In [10]:
import logging
import threading
import itertools
import pandas as pd  
import numpy as np  
import scipy as sp
import ruptures as rpt
import json
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.dbtables import FileModelStore,DBModelStore
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
from iotfunctions.bif import PythonExpression
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.anomaly import (SaliencybasedGeneralizedAnomalyScore, SpectralAnomalyScore,
                 FFTbasedGeneralizedAnomalyScore, KMeansAnomalyScore, NoDataAnomalyScore,
                 SaliencybasedGeneralizedAnomalyScoreV2, FFTbasedGeneralizedAnomalyScoreV2, 
                 KMeansAnomalyScoreV2, BayesRidgeRegressor, RobustThreshold)
from mmfunctions.anomaly import (FeatureBuilder, GBMForecaster, KDEAnomalyScore1d,
                                 VIAnomalyScore, SpectralAnomalyScoreExt, MatrixProfileAnomalyScore,
                                 NoDataAnomalyScoreExt, ChangePointDetector, EnsembleAnomalyScore,
                                 GMMAnomalyScore)

from mmfunctions.bif import InvokeWMLModel
from mmfunctions.Customanomaly import AnomalyThreshold
import datetime as dt
#import pymc3 as pm
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.mixture import BayesianGaussianMixture
from sklearn.pipeline import Pipeline
from sklearn import metrics
from pandas.plotting import register_matplotlib_converters
import matplotlib
import seaborn as sns
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

In [11]:
db_schema=None
'''
class DatabaseDummy:
    tenant_id = '###_IBM_###'
    db_type = 'db2'
    model_store = FileModelStore()
    def _init(self):
        return

db = DatabaseDummy()
'''

with open('credentials_as_monitor_demo.json', encoding='utf-8') as F:
    credentials = json.loads(F.read())

db_schema=None
#fm = FileModelStore()
db = Database(credentials=credentials)

if db.model_store is None:
     db.model_store = DBModelStore('jswmam', 5, 'bluadmin', db.native_connection, db.db_type)

2022-07-12T13:35:29.849 WARNING iotfunctions.db.__init__ Unable to locate CORE API URL.. using base API URL
2022-07-12T13:35:44.882 INFO iotfunctions.db.__init__ Data Dictionary is not available.


In [12]:
db.model_store

In [15]:
df = pd.read_csv('../mmfunctions/data/AzureAnomalysample.csv')
df['entity'] = 'MSCIL_Devices_01'
df = df.set_index(['entity','timestamp'])

In [16]:
from mmfunctions.Customanomaly import AnomalyThreshold
jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}

vsmi = AnomalyThreshold('value', .9, 'Min_J1','Max_J1','std_cycle_J1','outlier')
et = vsmi._build_entity_type(columns = [Column('test',Float())], **jobsettings)
vsmi._entity_type = et
et.name = 'IOT_MSCIL_DEVICES'


vsmi.auto_train = True
df = vsmi.execute(df=df)

2022-07-12T13:39:16.187 INFO iotfunctions.metadata.__init__ The entity type is not connected to a metric input table.
2022-07-12T13:39:17.047 INFO iotfunctions.dbtables.retrieve_model Model model.IOT_MSCIL_DEVICES.SupervisedLearningTransformer.Min_J1.Max_J1.std_cycle_J1.outlier.MSCIL_Devices_01 does not exist in table "BLUADMIN"."KPI_MODEL_STORE"
Here 1
2022-07-12T13:39:17.614 INFO iotfunctions.dbtables.store_model Model model.IOT_MSCIL_DEVICES.SupervisedLearningTransformer.Min_J1.Max_J1.std_cycle_J1.outlier.MSCIL_Devices_01 of size 102 bytes has been stored in table "BLUADMIN"."KPI_MODEL_STORE".
Here


In [2]:
#======================

In [55]:
import os
import dill as pickle
model = None

f = open('/home/markus/src/notebooks-attic/MODEL_Invoker', "rb")
model = f.read()
f.close()
arr = pickle.loads(model)
df = pd.DataFrame(arr)

In [9]:
from srom.anomaly_detection import GaussianGraphicalModel
from srom.anomaly_detection.algorithms.anomaly_graph_lasso import AnomalyGraphLasso
#ggm = GaussianGraphicalModel(base_learner=AnomalyGraphLasso(alpha=1.1))
ggm = GaussianGraphicalModel(base_learner=AnomalyGraphLasso(alpha=0.2), sliding_window_size=30)

In [4]:
#df[22] = df[22] + np.random.rand(2879)/10


In [10]:
ggm.fit(df)

GaussianGraphicalModel(base_learner=AnomalyGraphLasso(alpha=0.2),
                       sliding_window_size=30)

In [86]:
ggm.predict(df)

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -2.130e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -3.160e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -5.278e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -5.087e-04
  "graphical_lasso: did not converge afte

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -8.530e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -4.393e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.938e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.307e-03
  "graphical_lasso: did not converge afte

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 8.425e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -3.292e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.383e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.122e-04
  "graphical_lasso: did not converge after

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.742e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -8.193e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -8.976e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -8.262e-04
  "graphical_lasso: did not converge afte

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -7.592e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  "deviation of the data is probably "
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -6.976e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation

...

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 2.255e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 2.649e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 2.630e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 1.044e-03
  "graphical_lasso: did not converge after %i

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.110e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -2.691e-04
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.159e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -5.537e-04
  "graphical_lasso: did not converge afte

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -2.052e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  "deviation of the data is probably "
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 1.413e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 1.480e-03
  "graphical_lasso

/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.596e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.734e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -1.530e-03
  "graphical_lasso: did not converge after %i iteration: dual gap: %.3e"
/home/markus/.local/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:316: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: 1.914e-04
  "graphical_lasso: did not converge after

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.57655241, 0.78456193, 0.73394926, ..., 0.53621948, 1.03798035,
        1.00168166],
       [0.47539742, 0.8499178 , 0.72657758, ..., 0.4353255 , 1.05979025,
        1.05408836],
       [0.41373909, 0.82410362, 0.72345003, ..., 0.32935645, 1.05582046,
        1.1419076 ]])

In [11]:
arr = '{"predictions": [{"fields": ["prediction"], "values": [[[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]], [[1.6543474699192282, 1.0264375101108159, 0.27353033117801734, 0.5562844956803586, 0.8204602935649494, 0.9408279536123464, 0.4497521763158148, 0.5363080761507307, 1.128725745884734, 0.8609090671041016, 0.23472573878252045, 0.5049773983539427, 0.8840075158156478, 0.6971096601297309, 0.5281030870188883, 0.5413923880530157, 0.6374925842767438, 0.6718188505150461, 0.6910731752867305, 0.26012158336914415, 0.2715186445264077, 0.46790206434630405, 0.08326923621258207, 0.6419306859684406, 0.6345864207246743, 0.6615152603367358, 0.899132621038523, 0.5678771349790092, 0.680167062488023, 0.40121907744737156, 0.49722183073254206, 1.048178475324258, 0.48159258663395577, 0.6524289023913175, 0.25167724956073845]], [[1.5122126762460015, 0.9293257283604499, 0.22493292140474647, 0.5639600066162087, 0.8552708815796746, 0.8693252360732665, 0.4226892375430784, 0.5645711909864122, 1.135429186072553, 0.8482029324956043, 0.2425594099229597, 0.6378288526279654, 0.933116303774433, 0.7068787823355123, 0.5981896047585646, 0.6181700374057526, 0.7109515004316239, 0.6872077935823377, 0.7672149857384494, 0.35754144002973115, 0.33130918614630467, 0.4694290983560748, 0.08258921141083544, 0.6160266833277366, 0.6008469137383654, 0.6336105839211185, 0.96187285872862, 0.6972611253234562, 0.6940394366548452, 0.37878933897671296, 0.5481900210931027, 1.0383743381512025, 0.4973307158831973, 0.6427453760024945, 0.2646850170181108]], [[1.540516115898789, 0.942980570792456, 0.2955068380346417, 0.574500126322999, 0.7965907930554126, 0.8396756655920858, 0.44520323377763776, 0.5957620359270697, 1.1006090695410153, 0.8457308166426333, 0.2526757041614963, 0.5870879084253652, 0.9277693981459854, 0.6912793113465591, 0.5273020425400647, 0.6775701055064656, 0.5075736504782783, 0.7016155623523422, 0.8338580072412239, 0.4090494954754573, 0.4005991708134144, 0.4319362009243176, 0.08815193089593859, 0.6171103538016133, 0.5498103819291661, 0.6585596007287184, 1.0010304722719754, 0.748823638520587, 0.6762966245147003, 0.4277147574457454, 0.5592773409103258, 1.0808376578784045, 0.5058662818172963, 0.6252627110882537, 0.2976283813133629]], [[1.4750340885608169, 1.0229131267297629, 0.3032721327965791, 0.56490631942781, 0.5570780442145328, 0.8567746798399392, 0.42019293214378606, 0.6491813578074236, 1.1394703085554503, 0.8480119361995007, 0.255327619314296, 0.5878193597560658, 1.0654724075547244, 0.7052353083294848, 0.5018360157681716, 0.6986638389753257, 0.561090293766709, 0.6954757400274447, 0.7933474209072393, 0.45111291469393844, 0.42481039967560985, 0.4060440833435207, 0.08069916946197822, 0.6242231472258424, 0.4553763073269641, 0.7284680245605021, 0.9554332118370468, 0.7639160685279074, 0.6762160464207794, 0.5595149272877419, 0.5818019533129843, 1.070930813514754, 0.51566138689336, 0.6203991310505602, 0.334413041694994]], [[1.4089573133460533, 1.150575190275386, 0.32265160498909984, 0.6546701053718035, 0.468640826949005, 0.7845689462827743, 0.45658499180840606, 0.6812945608521804, 1.316408172337604, 0.8887420856053625, 0.24628036610978432, 0.5748475538990765, 1.096997203073342, 0.715058305470848, 0.5415076930977831, 0.6538949834532811, 0.49779796469701143, 0.6944701716009178, 0.8470482854778832, 0.5051103545243381, 0.4608160619350128, 0.4092148977360567, 0.06751261957338561, 0.6239766209890907, 0.4955587070953609, 0.7237262539104996, 0.9314115790066682, 0.897713786249432, 0.6582990256535937, 0.6177249573238268, 0.6035238901840962, 0.8517562881080256, 0.5331897441636919, 0.591709256624789, 0.39418165845397896]], [[1.4364876854361837, 1.228208955558335, 0.24352190813887997, 0.7540944602547792, 0.559531962284789, 0.7887141302703419, 0.45869295167482604, 0.7587431955540613, 1.2890481331349666, 0.8601568734453903, 0.2695929795720902, 0.9538011386289871, 0.9907851034350955, 0.6821832648550168, 0.6102472624889167, 0.6124458381809816, 0.5403654346637983, 0.7328392990331095, 0.8361993980562848, 0.4999004639248811, 0.42893057449431526, 0.3419959784704374, 0.04629130447364302, 0.6062287363583878, 0.5276101802439631, 0.6127330832538969, 0.6351670150598662, 0.8547225275165486, 0.6643848101058711, 0.7121977525054535, 0.5880305972943722, 0.830349967863802, 0.548708995188384, 0.6431817683181125, 0.43567634020295587]], [[1.3819888527068316, 1.2093093089983113, 0.27354425659989823, 0.9553964844288786, 0.6657792659385607, 0.7529212012814772, 0.4308978315569946, 0.792231863173713, 1.1899938016688822, 0.8503555296941914, 0.2816674861741961, 0.9333187021035335, 1.0428277725399298, 0.6574505788388927, 0.58445697517578, 0.6124224130270575, 0.5621045847687659, 0.6726316486827613, 0.8447271550865041, 0.5081263378250245, 0.4846225546987979, 0.44204385087066544, 0.05841839063295964, 0.6246693979490856, 0.4831151548817131, 0.6020350687587446, 0.5408488197226862, 0.8031579018462512, 0.6786931380125771, 0.8545697963991903, 0.5854919352025787, 0.7606455228171873, 0.5520753497923622, 0.6934369428381044, 0.4808229124791672]], [[1.301654669265586, 1.1875609943464194, 0.24600752534638348, 1.1990224478928728, 0.7100187280026318, 0.825402489383754, 0.49069385717494296, 0.772903719450668, 1.0671613263639852, 0.7379780778122854, 0.33145412790365814, 0.8869727708399554, 1.041180007765339, 0.6515893666849129, 0.6491279126436796, 0.6885279873274941, 0.507509184986644, 0.6907843754658447, 0.8819796377610701, 0.5228379780386354, 0.5448575228948754, 0.39858118754059346, 0.05521997837984232, 0.6246322557784253, 0.4120756091455784, 0.5783701857861601, 0.444411362248404, 0.8231325592417821, 0.6817027421018585, 0.8661584647938536, 0.6080020726307878, 0.804082126401461, 0.5589006893049064, 0.7266527510036287, 0.5087821256868227]], [[1.3204037374774968, 1.3172271859473303, 0.28578548670224213, 1.3169321772915445, 0.8511323447842616, 0.8045680924929224, 0.4594452822253167, 0.6710472613342131, 1.0370964539325924, 0.6501853167595262, 0.31639176316349726, 0.8421537165129236, 1.0232513050150802, 0.6928807371003225, 0.5783600746648148, 0.6265503534745065, 0.5397992516038124, 0.6884965637833964, 0.8029510426290953, 0.5301003632924413, 0.5632824965192254, 0.4477922366209966, 0.06031329431745568, 0.5341552335010635, 0.36524402641420084, 0.5690497006164832, 0.41181731315511905, 0.7999536789339241, 0.6845459238600957, 0.8519912473739106, 0.6960785443940128, 0.9762326013693962, 0.5757316608306731, 0.7932106404220642, 0.5127127493836763]], [[1.380960454202983, 1.3755329236687381, 0.26895973093338477, 1.4375122422699942, 0.9471218646969521, 0.7942510775735363, 0.4638970905870977, 0.6343621857425412, 1.0201104890948873, 0.5679408417885947, 0.334993715463929, 0.8642467363058652, 1.0331882044119545, 0.7397140070864794, 0.5537882852203686, 0.6315674474698201, 0.5298962208411119, 0.6747324649794473, 0.781393374951125, 0.513612049339853, 0.45040921578239, 0.439338540084852, 0.047373005102116214, 0.4809002139537394, 0.3760406458649162, 0.4509844289963571, 0.40125269190107676, 0.8610041672194758, 0.6885373726806341, 0.9064657665699525, 0.776749955957519, 0.9963036681871246, 0.5909010185246955, 0.8552216535302708, 0.5426807393619641]], [[1.4332665670357683, 1.366819255852507, 0.2536617611736708, 1.4799501008869258, 1.0354598192199267, 0.6743988099744401, 0.43121995235873056, 0.5901478984237434, 1.0604549620800832, 0.5471895408399073, 0.35270175952936594, 0.8661513940451965, 1.0265198213764017, 0.7662525371691202, 0.5670319059800313, 0.6391081948890073, 0.5957467631368585, 0.6900876732817576, 0.8340262764904067, 0.5071993244331441, 0.4330060146359381, 0.41749327339765013, 0.0526447509867401, 0.48693770698013944, 0.4238514586915585, 0.36561821345466006, 0.3856012618038542, 0.8911823487819801, 0.6990866283421673, 0.9590112431907343, 0.7737424621958489, 0.9549895886351725, 0.6040624532620118, 0.9437704055275774, 0.6216240915335051]], [[1.5488404354893126, 1.3382746095368063, 0.26525634850581303, 1.563845094905866, 1.0769085731601709, 0.68321956445202, 0.43422254328303767, 0.4729179730619891, 1.0434038828858947, 0.5244891410955923, 0.4268010483393495, 0.8018216856606045, 0.8042271774479337, 0.775189194610815, 0.5606363199228013, 0.5940995989434741, 0.5714244015933347, 0.6675682565811114, 0.8180409732547664, 0.5454102974879477, 0.41599518045010075, 0.4004060870284622, 0.04454278466308441, 0.5544628761037329, 0.45339892877193455, 0.423703967430584, 0.41188060804144166, 0.906575055134849, 0.7133790929576713, 0.9389990908184498, 0.7388625784473262, 0.9746632140913455, 0.5981043570114435, 1.0132109783095062, 0.6950012841364293]], [[1.65800198929901, 1.2599214420095728, 0.24222183105477268, 1.5783896312101735, 1.0418736025052326, 0.6944074180089632, 0.4472147014837523, 0.4295410590475066, 0.98971625757544, 0.506311650215885, 0.3899884652600929, 0.9297255221878858, 0.7792559044537444, 0.8536184423824071, 0.5685707837463345, 0.6769217717471978, 0.5912539435184414, 0.6626352896607721, 0.8091933050700858, 0.5252744829471685, 0.495312023975376, 0.4205323926774268, 0.05147962678712167, 0.5144979555540974, 0.5187927604486237, 0.5737468351730082, 0.489622910347414, 0.9251009262045053, 0.809057672011892, 1.0107561163845311, 0.7532720247628789, 1.0606408717090636, 0.5969240234367238, 1.0468664724152534, 0.7510632336469949]], [[1.6551338360374683, 1.201026191480647, 0.2502957035474377, 1.5086423475372852, 1.0456329454655107, 0.7309766541952631, 0.4706015762663452, 0.3978593254210816, 0.949487554320407, 0.5106689817035515, 0.2678638697009976, 0.9134751354487194, 0.6826429241166445, 0.7377975626730694, 0.46807804247218304, 0.669602903209173, 0.571394770094563, 0.6957897443879011, 0.8452947461716964, 0.5572000537081693, 0.5247164525840706, 0.3819476841520061, 0.04812391776841667, 0.5025025681624343, 0.5372191084096494, 0.6907720290294468, 0.5543550186125922, 0.9296114868421551, 0.9972808608234558, 1.1294338490557365, 0.8868960895337128, 1.194206401549834, 0.5791540953529548, 1.1143563921416983, 0.7863566476704182]], [[1.7163100036686574, 1.206923435417671, 0.30671618361125375, 1.3852088519262231, 1.0678727275761997, 0.7305261899001617, 0.4451164860574797, 0.3913457576181675, 0.8823686003824288, 0.5281010710594407, 0.13597092640532307, 0.8315557967739322, 0.7147671048813788, 0.6942014110580824, 0.44797107783384765, 0.5770313012419909, 0.5499128563913581, 0.6844717687673721, 0.8207730258963264, 0.5854293366462975, 0.5956472535301398, 0.3877693769547954, 0.04935625383527743, 0.47088467898414194, 0.45456078145769097, 0.7595926075163639, 0.5791596591546481, 0.8624764462874992, 1.102364749511785, 1.2685367208180178, 0.9497125959328931, 1.177461413071637, 0.5686978472628036, 1.0942876454714556, 0.9123804532220434]], [[1.7253993018590927, 1.2544371677745687, 0.30429003089394596, 1.379648435258112, 1.0801083932255648, 0.7319476296093788, 0.4025776046651256, 0.44847350411796666, 0.8535668490963009, 0.5494192243156547, 0.07125215600652884, 0.8567792887863073, 0.7151132681371304, 0.7290446965117452, 0.44739291665678244, 0.5739881352667828, 0.5674385865840925, 0.6737917338916389, 0.8146679266310771, 0.5732317271327252, 0.6281745304996247, 0.4067749696826636, 0.03531978844716389, 0.40239522684982865, 0.4681375184959399, 0.8317472093809356, 0.5492342776077149, 0.8028744455024821, 1.2124443936426905, 1.2016694601402973, 0.7330270303673296, 1.253878951026027, 0.507471647986234, 1.059366123984337, 0.9175931245259445]], [[1.7442907274684831, 1.2389453597788622, 0.28851864141533046, 1.3872132878443224, 1.07093809673586, 0.7372631207735447, 0.3596710138967932, 0.5064100787853424, 0.8353516839515147, 0.5604921479044084, 0.057018982645391236, 0.8710367369295116, 0.7124984141005832, 0.7139578652411824, 0.4650951305903939, 0.5860217771069287, 0.5464910382206506, 0.662320090761374, 0.7571693301427458, 0.5684292379146645, 0.5818391283579422, 0.4185493015525458, 0.028145652536299755, 0.33024555863827887, 0.476727051962316, 0.862609956620321, 0.5480328487912739, 0.7855593253520571, 1.2783742898526322, 1.1959131836417678, 0.687948721239913, 1.3423779440351458, 0.48296744345498477, 1.1324786751478395, 0.9570045976378556]], [[1.6760152562205513, 1.2082867360135525, 0.29171314050017133, 1.2963308867836585, 1.067936556065757, 0.7873076215006223, 0.3997321571234912, 0.6803446795110478, 0.78646607739293, 0.6137472228802885, 0.08032086672140862, 0.8419317207684944, 0.7199888595981758, 0.7410136899449965, 0.46391453123230925, 0.5269937631034975, 0.5542224959452062, 0.6567299878353979, 0.7872499723048655, 0.671556239034601, 0.5912336443884632, 0.5150924020139294, 0.02836216768743189, 0.3896610148551555, 0.533540538561103, 0.7581419045225252, 0.6095365304012801, 0.761634017457558, 1.0770808469887787, 1.1696188154399603, 0.649786856425836, 1.2199950046227364, 0.6331435148366193, 1.121428280401933, 0.9258911471754107]], [[1.54162061884831, 1.221911771569187, 0.28198071598165714, 1.2759084564512784, 1.053931321705667, 0.818398398872314, 0.4078008209040078, 0.8519138241516011, 0.7466976961477965, 0.6375915434861215, 0.0727934053805784, 0.820273497376895, 0.7092276820940967, 0.7573903851099046, 0.4348280144394833, 0.49727383258783947, 0.5400361157727447, 0.6582135482918492, 0.7798591968032854, 0.6363815934969335, 0.46867123545265443, 0.5266593675494077, 0.07906634490941672, 0.4762951618247186, 0.6317922282430071, 0.6921856818353782, 0.5826919569196727, 0.7574732860978973, 0.9635739949891778, 1.0627697792497037, 0.6641227950987227, 1.19979888378372, 0.7888587836942915, 1.1016861041050396, 0.8605446379187951]], [[1.4779992248105327, 1.254826675450597, 0.3004308090956484, 1.3033996543540307, 1.0898974508307577, 0.7286428650830865, 0.32423743233531677, 0.8523905006392078, 0.6849448495159505, 0.6427924340444933, 0.0691315312558064, 0.8794682350722626, 0.700841044216149, 0.7628611662534053, 0.44311015721201136, 0.49313814725363797, 0.5890193108361238, 0.6372623780991435, 0.7633180106310026, 0.648518679013508, 0.4512327203606617, 0.47024989186691235, 0.07135042615530685, 0.5305268904922158, 0.7025341911239272, 0.6811810561263607, 0.5656801972901304, 0.7014582025143173, 0.8559386921029302, 1.1206167132690732, 0.6444398263844968, 1.1556020409694743, 0.8277496356349032, 1.0733116318971503, 0.8629587801765557]], [[1.5140368386217546, 1.1988546050787234, 0.31887828548438, 1.31651156464367, 1.1125441108407772, 0.7070674447473029, 0.2952718315898965, 0.8501846047554042, 0.6268926131896286, 0.6199151306927061, 0.0846078612890378, 0.8928625119800201, 0.6873020703350083, 0.7554372329415966, 0.3803013691559605, 0.49482114529600413, 0.6430052480745909, 0.619552997612892, 0.7921524308228283, 0.645262329624758, 0.46024716124137155, 0.4494730687218108, 0.061769995942656246, 0.5215501387975039, 0.5958666781241615, 0.6126080263783477, 0.49467790012078544, 0.609484582104917, 0.7075617699780143, 1.2603218253574773, 0.6137881048163798, 1.1171291495099123, 0.8998887550331973, 1.0913538258280557, 0.8284130649581933]], [[1.4639178284248149, 1.1686265196608319, 0.4630513912860707, 1.3065480594011643, 1.098564194900597, 0.6966940662754223, 0.2597485909300895, 0.898806864508489, 0.545455105110696, 0.5449693484854996, 0.08062696169273117, 1.1069129280825865, 0.6152252803479321, 0.7091288060703901, 0.38719683556252177, 0.5471338100191538, 0.47867422375015006, 0.5876717966747598, 0.7228849033085445, 0.5098170374144032, 0.42931236810290496, 0.41786940391716143, 0.05719730931860684, 0.5324966132805518, 0.49254438547802276, 0.6298160213903994, 0.4498231088049116, 0.4895957181440471, 0.6453108965631972, 1.4377010406781396, 0.5624668117972014, 1.0314926966544276, 0.9292934078988413, 1.1384869703569074, 0.8543370473380182]], [[0.9080430531243561, 1.2210838659853818, 0.5173724350534976, 1.2918629002542674, 1.097687827640418, 0.6969593192633172, 0.2962302749370276, 0.8728555445170127, 0.5300925071707561, 0.5200075774689558, 0.08463259366348191, 1.1287345105776532, 0.5771360113369207, 0.7058378606897, 0.35461387048356835, 0.5619263637198046, 0.460849370999845, 0.5981706887323346, 0.8224362832010814, 0.3707899700895, 0.3758187605680269, 0.36130611650946987, 0.059508216659862674, 0.5466965837702417, 0.529203333201244, 0.6282355421591596, 0.4121201016337718, 0.47676915484048965, 0.6422620762554725, 1.4272210523440043, 0.5480157360059268, 0.9225506810509374, 0.8611016250680673, 1.1947877334216037, 0.8497662960121178]], [[0.9332419673698307, 1.2582089258037228, 0.40768657312343815, 1.2276951123076996, 1.0592536723363781, 0.6404285055247498, 0.28076343572164664, 0.8922967789535643, 0.48749757731169174, 0.511623633171927, 0.0750073744690623, 1.1393592791828442, 0.5629392420857822, 0.7221165566661976, 0.36693221930154224, 0.5959918267391056, 0.44341267785879723, 0.635148922621486, 0.8690661955312035, 0.3875016715814151, 0.4156299084770029, 0.3999957960354864, 0.06491860762399104, 0.58530268732961, 0.7471986833906323, 0.6392542476854624, 0.38893981687278734, 0.6126193785680245, 0.6437680716079757, 1.5051616827411345, 0.3014456864649281, 0.9591168162779784, 0.930397006314287, 1.0617454336885759, 0.829654138673065]], [[0.9915653900220764, 1.2306265994327712, 0.3874664416798764, 1.2445198404571314, 0.9871742993000784, 0.6464213466692001, 0.28061712440782627, 0.8994310859671677, 0.5542744171783366, 0.5520390227899306, 0.08108752648837572, 1.2101717695583922, 0.518240128387562, 0.6882970372838348, 0.36267501948499214, 0.6032002481741845, 0.40418004134612995, 0.6414219035801372, 0.8869043744769576, 0.371127589686524, 0.44038224061746284, 0.34136705252937904, 0.073057611796145, 0.627853698523156, 0.6449474961123871, 0.6238015233385394, 0.3434257425645708, 0.6116290366373298, 0.593700474755458, 1.5574505690974574, 0.2759204816184756, 0.959246298503359, 1.0072907285071373, 0.8810850031137578, 0.8007557750613958]], [[0.9802339564376619, 1.1984828384410355, 0.47859220745391506, 1.2279475276344942, 0.9382897908971353, 0.6481013796559913, 0.3013042124216663, 0.922328199375132, 0.6696650937533053, 0.5821298836842157, 0.07585791265510604, 1.184810020316983, 0.5081428459915367, 0.6916390979084589, 0.36891469861359233, 0.5447204107336551, 0.4255287713102277, 0.6106263053018672, 0.9146652080289003, 0.33009865814126566, 0.40667787601446215, 0.40132223819469975, 0.07124698987408623, 0.6481442158866915, 0.6607185920675496, 0.6200625439938252, 0.28464356634128296, 0.5724692985134603, 0.5146430618725388, 1.574782151119439, 0.2605668918323183, 0.9428518145054783, 1.111556465142249, 0.7980235890401692, 0.7470868135242383]], [[0.9112472525456173, 1.1715571180153097, 0.3667000897813737, 1.149313464196318, 0.9500047917254899, 0.677510949676696, 0.3372226264015239, 0.9435735214139216, 0.6821265714623845, 0.6097091706676613, 0.06342727090059902, 1.107368831981649, 0.45798525517737804, 0.6523140502658753, 0.35679993835009627, 0.5342641304330582, 0.40787473775976707, 0.6008606715858529, 0.95568852404185, 0.33253482333331613, 0.4264217436513031, 0.44689586882893817, 0.08130934364230653, 0.6664964373999639, 0.6223243087590814, 0.6098815771095802, 0.28223760665215775, 0.4515901046069112, 0.48914803981169486, 1.5071594244612379, 0.2953440538226705, 0.9101822969410417, 1.1400890536621118, 0.8032636119188272, 0.7105273623112223]], [[0.8632785935373528, 1.1137039278369856, 0.35905623217287796, 1.164349552379551, 0.9169489087908065, 0.6534388399415796, 0.33605567776780104, 0.9771067349571734, 0.6607032286880595, 0.6089829631094564, 0.04787675379287771, 1.1421067582537643, 0.4605114617961385, 0.616907993736351, 0.3616332757436419, 0.5555899308044047, 0.3721100735137266, 0.611911867488869, 1.016128552878743, 0.3426563493765965, 0.4216552235017624, 0.4488232685070904, 0.06623221621032593, 0.7084583265951453, 0.6474386143797822, 0.598800325111644, 0.28628002750700354, 0.4234058719178958, 0.43795555256983343, 1.4644415861312754, 0.26178151475386957, 0.8368099854750577, 1.1566568104733026, 0.7873309596845088, 0.6911022627041987]], [[0.7421513681058227, 1.0625907069052656, 0.38251216497527407, 1.1011391264069064, 0.9095318479017542, 0.684161030361444, 0.3682903869454279, 1.0103925356827397, 0.5841910796362728, 0.5729312031163556, 0.04317790760932457, 1.1446370001896877, 0.5364475894469918, 0.6556388961415198, 0.3584162270110019, 0.5435802060137368, 0.35526024170731746, 0.58513030998508, 0.9774914248532502, 0.3784443725645946, 0.413021615745096, 0.45222283009009073, 0.1025289135225091, 0.6880784927690156, 0.6261324076530519, 0.5460217041543594, 0.3678343226918326, 0.3950302320377836, 0.37298385157099656, 1.4312098037597607, 0.2708068198631924, 0.7331657912001981, 1.1741472997419962, 0.6714369001945024, 0.6381824238046603]], [[0.6909003282074692, 0.996238485320423, 0.37828315831549075, 1.0688104276442294, 0.8758883482717815, 0.699849322158337, 0.3713999834307175, 1.0028555978133589, 0.5937056768866911, 0.5475529865732655, 0.04525530755344058, 1.1238638248557227, 0.4767334648019377, 0.6262863538111307, 0.3730781405562778, 0.5834933732826162, 0.33842521959841626, 0.5744803491921182, 1.0015760989972595, 0.3901416080957489, 0.40487694011928527, 0.4474820088569797, 0.13262865335103566, 0.6379761478101715, 0.5192151183724188, 0.4444977863221721, 0.42596216571463164, 0.38458582929792406, 0.2988052624351062, 1.3773429619303372, 0.26815495185040317, 0.6017862215505221, 1.1760228709864542, 0.6417260480097915, 0.5663887835294785]], [[0.6729497912337367, 0.9813014767294512, 0.34706325541175514, 1.0497143278896908, 0.8925712716857547, 0.7289440571089596, 0.36693690117371314, 1.0427978407845164, 0.49928163874691456, 0.5526634825421252, 0.0495332706169597, 1.168378861302159, 0.4286393024413374, 0.6051084913173811, 0.3812103890253999, 0.5408733596636874, 0.39986402110810276, 0.6191505398892574, 1.0617028879236403, 0.3754366738101995, 0.41691013838110186, 0.46904580066546653, 0.255931025784223, 0.5038770695794157, 0.40568013443064277, 0.411132749282708, 0.359018353483773, 0.3835393826333901, 0.3125022788706272, 1.4395641843934492, 0.258109090336487, 0.4964290146160252, 1.207124591758348, 0.6432973633938315, 0.4374859730240115]], [[0.6447188749870104, 0.8964623019212484, 0.3400913767461774, 1.0473460692681278, 0.9310616412205353, 0.7160780218164706, 0.37000567767803, 1.1403109378772533, 0.4657068038391924, 0.5774297645725748, 0.07285849892780508, 1.2550136821453848, 0.3319315619469937, 0.5813638823465884, 0.3998814898781539, 0.5538524146367236, 0.4683674254861139, 0.6586734404907535, 1.0545410309336756, 0.3484575320483221, 0.3787609431173685, 0.443721517538256, 0.1539069517646475, 0.43967232056947503, 0.39036691644624477, 0.3939729204662259, 0.33670598706102756, 0.3996346999325424, 0.30099300999256673, 1.4004833062274693, 0.24941508653736502, 0.5743466474802232, 1.2292484397737118, 0.7096784683432249, 0.5525698620738271]], [[0.5940399273255207, 0.8313342832475092, 0.3476543905577947, 1.0433151044717488, 1.037640155207866, 0.7173855993779044, 0.37438257530633184, 1.1899011698902502, 0.5227447648394734, 0.6397102487490853, 0.09774921992563529, 1.2207376721690621, 0.26858128891969246, 0.5859397861171269, 0.5075341096014171, 0.4967587619444943, 0.4331344144274709, 0.6630537922808125, 0.9148741763613394, 0.2912298716367365, 0.31831453437052126, 0.42520860551761797, 0.05357177819436379, 0.4473049873212588, 0.41906429952094393, 0.2922033136329634, 0.32704547320229077, 0.4359562477928582, 0.27099767525002716, 1.408130822145337, 0.22850205878597923, 0.6107541149437404, 1.2364916495176081, 0.9942093977848522, 0.8052201670554731]], [[0.5731865542731053, 0.770432741621053, 0.35664243918725813, 1.136362461355693, 0.9937288267094184, 0.7055830070564733, 0.3734513494291034, 1.169180072898216, 0.4830715106264817, 0.518618157378141, 0.07737672414333419, 1.159837982621668, 0.26289106771368276, 0.613173162702912, 0.48936717979151195, 0.4790384541814716, 0.43407481344330046, 0.6878936026847395, 0.92687949369868, 0.2498233058091923, 0.3080700828391233, 0.42092009810160036, 0.06490182180090356, 0.47717077563660637, 0.48394343531862954, 0.28107791337784294, 0.3198904828354341, 0.4796273373929929, 0.21157969162765056, 1.4332260952505507, 0.2124686746491535, 0.6526276114164253, 1.2377652654172686, 1.2790721218659806, 0.9892001563938162]], [[0.5448051077934842, 0.7339559044976898, 0.45264177545375084, 1.174447072758197, 1.0025424449034703, 0.6948194836931205, 0.38401822366181443, 1.1248821063768448, 0.5445540675643042, 0.4963372985123454, 0.08464645782851186, 1.1474578868290628, 0.26327051911918237, 0.6782787028751704, 0.5013315010311931, 0.5234398813209697, 0.45872210094568666, 0.6672788074782694, 0.8579276983382333, 0.25670607889654995, 0.30601935317186263, 0.4286344400560406, 0.043350908184085185, 0.4965982479772051, 0.47335857483216914, 0.27353552345420323, 0.3259240582034612, 0.45425724567471665, 0.20370187044016438, 1.4122773454980728, 0.22466179246168597, 0.667598465136595, 1.277317618298582, 1.4448483266346384, 1.1054436141516184]], [[0.5209884410156136, 0.7102908136183317, 0.5063923746110082, 1.1824410988312029, 0.9637178001130469, 0.6825514955129723, 0.36184712928477364, 1.035469236142942, 0.5365378916847953, 0.5356892347493079, 0.08859348759966701, 0.9833197791692858, 0.24558274856800283, 0.6923939992010966, 0.5732808335331304, 0.5680011296581946, 0.4234167634515371, 0.6860128298699881, 0.8900148780825252, 0.40579549814731947, 0.3086882386689658, 0.48331337596672586, 0.06697007666712611, 0.5157774867794249, 0.5386011329784511, 0.25149781398513116, 0.32669176155422996, 0.45087756557945535, 0.19767516967257875, 1.3633025633081164, 0.2868453624199463, 0.6614350328944836, 1.352848647967802, 1.532073328728237, 1.178536470899748]], [[0.5045938603028605, 0.6846623061447654, 0.46214283918610044, 1.1404005147607599, 0.9392396802961412, 0.6886487357804374, 0.3556056639580659, 0.8897854265914684, 0.540537526668912, 0.5020171072687949, 0.08743849652824515, 0.8509697066170685, 0.21948548061468193, 0.6083186516113592, 0.5550980553920166, 0.41119653749240037, 0.39318922570213344, 0.6645460165447995, 0.8604857918411422, 0.5252770037733486, 0.380923245184955, 0.4203984316760855, 0.0303159266732957, 0.5027575425886381, 0.5535347963406018, 0.25154779666525484, 0.3361533527502545, 0.4550675411379245, 0.1597848083769775, 1.2915622623812437, 0.3611797280607404, 0.653677349152319, 1.4453274217074692, 1.627709357556271, 1.2392804361504417]], [[0.48154011028019056, 0.6146250179892981, 0.5135241513421561, 1.0737325392299413, 0.7720314158970656, 0.6947498839337227, 0.3479956642442047, 0.7441193155003676, 0.520137521687878, 0.3396494720720614, 0.07685373355433175, 0.7558553644202664, 0.22041586066942928, 0.60395919374239, 0.5209395355064335, 0.3962807077917738, 0.41944172563934623, 0.6341806197420362, 1.0783671481097905, 0.5178395151895685, 0.47811128464183206, 0.3466512053940214, 0.05762828719961395, 0.47579242066749067, 0.5767931516976149, 0.2597046092809322, 0.3549513837480008, 0.45416023180244736, 0.16061737184805283, 1.2533499282051948, 0.42916091733857353, 0.6460385601870082, 1.5029031786613607, 1.5660210793191776, 1.2729681356016505]], [[0.4550046612491876, 0.540093962309734, 0.5091335805066283, 1.017520646002188, 0.714160072200811, 0.6908656106744615, 0.35573999778567555, 0.680771216981748, 0.5609012232406012, 0.39449921292563506, 0.07770403499802325, 0.7780442150980417, 0.22151564018403655, 0.564059209584911, 0.521395265313693, 0.4060938476091698, 0.3741059380562769, 0.6010659943324869, 1.0660200803798576, 0.37007301248436747, 0.4833521169245528, 0.3036223357293619, 0.02170244187475875, 0.4280820949898124, 0.5681653459953171, 0.27178631466004444, 0.3737913531415214, 0.4522944492806964, 0.16310092000946672, 1.210098714167312, 0.4860204290221357, 0.6192957058308113, 1.4495395411366037, 1.5981990382663473, 1.37852206853955]], [[0.4105719321333737, 0.49815035385539863, 0.5142871826685392, 1.0179395685021568, 0.8156461087634463, 0.6947888304027657, 0.36254023520726697, 0.5994297886076113, 0.5745354856276887, 0.4637076179634767, 0.07576867860656966, 0.7543351057346843, 0.23597581637169762, 0.5498861925628626, 0.48937393599112206, 0.4117835589835523, 0.38740652246935176, 0.5780091548605845, 1.0975312721842392, 0.28341095578407427, 0.4448916163893031, 0.20735035526196832, 0.018594735965981928, 0.3392296273524701, 0.5391238866883646, 0.2533700071894155, 0.35649691098983893, 0.43474403264075356, 0.20074510840976628, 1.1880950449976493, 0.5066883574810763, 0.634421920942615, 1.314167590396152, 1.6364406835642054, 1.4712943555945304]], [[0.34332833567389026, 0.4750399652924402, 0.5219887632590593, 1.0706683326598254, 1.0264439777224637, 0.6964307738221662, 0.40205854239844585, 0.6863827409747676, 0.6079306481053914, 0.518225199101261, 0.07600359972953746, 0.6844571640492869, 0.2442565900892693, 0.565894203920646, 0.49335252786192846, 0.35487566400319215, 0.42037885558189464, 0.5319269807854747, 1.0749648823076547, 0.23042309923016258, 0.41255058018795154, 0.17403134151715216, 0.022792576698548832, 0.2777726473140928, 0.5125874459050987, 0.2398724545442838, 0.36472100752482856, 0.44075710998021156, 0.25539764612406934, 1.1689893046357478, 0.496808582350012, 0.6303377914707031, 1.2327972121106083, 1.502929743996432, 1.4173381082999825]], [[0.3202331525281426, 0.4701201435526893, 0.5302019377210064, 1.042344847577205, 1.058193002714939, 0.6662686108054161, 0.3963486351391605, 0.7246086996771017, 0.6127307581584949, 0.53452885630068, 0.08279249952563467, 0.7096052312428365, 0.2431617141310363, 0.560896864186551, 0.48865268154700653, 0.35019702251253737, 0.41821785482041113, 0.5208982805596185, 1.0574933972503415, 0.23542057845620143, 0.35970177518253466, 0.17902929605974538, 0.020212632460214767, 0.22722483698441726, 0.5534240363922582, 0.2605136359204959, 0.36753590780419504, 0.39040832517373425, 0.2752681149280519, 1.1021923442869788, 0.5169640256969738, 0.624152199323929, 1.1871552175988902, 1.5194693456325274, 1.4905219510278154]], [[0.28970607635187895, 0.43308104654212287, 0.535478103695146, 1.2042410798237522, 1.0784863677594543, 0.6880608304366227, 0.4058982924587272, 0.7188051522110277, 0.6480648216846652, 0.5879868253794776, 0.06389395210827502, 0.7202863723867321, 0.24736789082935334, 0.5466439384588715, 0.5073190289976888, 0.33525558374894937, 0.4089650714126952, 0.48971455317366386, 1.0644680819478798, 0.21446662856843263, 0.30199397439385384, 0.21292476971078644, 0.020568184467664417, 0.19534470238434232, 0.5980545451998172, 0.2928836436067482, 0.35942594448684473, 0.3266808643146158, 0.27370531247037805, 1.0036881024803326, 0.5464941648111761, 0.6029325446173484, 1.1439473945541487, 1.5492031367104449, 1.5708962447019874]], [[0.267130148887148, 0.41068495827966756, 0.5136795649067547, 1.4221003621076649, 1.1112143698827166, 0.701545923602698, 0.41492323666401465, 0.7003370921844686, 0.6372550103299846, 0.7330408274794256, 0.06467814978472872, 0.7759523447067829, 0.25023939304243337, 0.5636682440384823, 0.49251989111829486, 0.3391899116725797, 0.4242951856872238, 0.4763278617735265, 0.9490882351341753, 0.2159715342196102, 0.26403117661228553, 0.2290659431667731, 0.02069243124964966, 0.1597042808826652, 0.6362648107086484, 0.30936313595869075, 0.39189231037212235, 0.2618808408576393, 0.30064570024548304, 0.8729785949938587, 0.6059797704130071, 0.5582211868797884, 1.085084652931524, 1.4981071486423054, 1.5512850366734208]], [[0.24732859942292945, 0.4321377546687769, 0.5068382638021611, 1.4917908458094096, 1.1093425404945942, 0.71357006821108, 0.4184974965305536, 0.6477790684860678, 0.5521260603736792, 0.7278868726929261, 0.13815631113510413, 0.7821474145878127, 0.28059555778441914, 0.5615684257177491, 0.587077440624839, 0.3473843417340901, 0.44894099920479213, 0.49934396769289835, 0.9842333015883895, 0.1948955355683077, 0.2206150418123697, 0.205151121143361, 0.024191945841235782, 0.13452434783353442, 0.6569742765824925, 0.3234828873303275, 0.3975367812445079, 0.25637519910156203, 0.33934575423422275, 0.7727719589304747, 0.6580205125302538, 0.5143510541470426, 0.9855656223670133, 1.371482510377378, 1.5325101203140132]], [[0.23703567365464484, 0.43135133409557497, 0.5144602446425139, 1.5230156230332517, 1.0956211495304637, 0.6985123083095891, 0.44572146420934317, 0.6646438990657126, 0.5758791553243092, 0.7898589143228562, 0.11959436082759436, 0.7511620765011746, 0.2867709245426123, 0.6010210476352731, 0.6621146318620623, 0.3234286044871785, 0.3930132025667942, 0.5013735874092191, 0.9537282347127738, 0.19463449427296103, 0.21929059992957675, 0.1894425923697201, 0.027663802902965884, 0.10756572300656846, 0.6746663544801954, 0.34137393345749045, 0.42515498539493546, 0.2731599172642808, 0.35983856634165673, 0.7296474877137629, 0.7431138088069613, 0.5457728347941965, 0.902899010815124, 1.1954048198454048, 1.4598799361149004]], [[0.23686884401196118, 0.42887840250685544, 0.4954309290165353, 1.6997378635597633, 1.1153986323739784, 0.6905679391230921, 0.5019429243651445, 0.7179050797748061, 0.5767553884789394, 0.9400836431516557, 0.10079674453455638, 0.8034207086022719, 0.30046626979827157, 0.5933874367947081, 0.6337166245656954, 0.313180243572272, 0.3961670656986631, 0.47799380347355136, 0.9435991939170363, 0.20295536076401283, 0.22589390567658313, 0.14778961322482576, 0.03487533851155926, 0.08449073564239848, 0.7588672588845218, 0.3655530182279142, 0.44811919019843444, 0.4664234536585258, 0.4427396584269729, 0.7262274373392259, 0.891073312135161, 0.5693625300317904, 0.7927707270411748, 1.0688483432736489, 1.4264829317416143]], [[0.24299179953672484, 0.4339185026071478, 0.5039757428622613, 1.658646556133368, 1.1506755351985634, 0.6989655501014216, 0.5048102444502492, 0.6289728239142446, 0.5615907237745146, 0.8672346653591141, 0.056645471159872396, 0.7867733758386695, 0.3137248103667416, 0.5894427584009451, 0.6272607146981231, 0.34117356070642474, 0.4126803512017869, 0.5027494110907157, 0.9593349736635837, 0.17089757910048545, 0.19592129762778765, 0.1376007251865437, 0.04509640995057326, 0.09185257452543644, 0.743640199028115, 0.44851885527770285, 0.47404296521071326, 0.4673638661619865, 0.511739789513884, 0.6950296962802699, 0.9087425284587487, 0.5990547624716498, 0.738158751794521, 0.9259821937745216, 1.3360302154203745]], [[0.24679036693813863, 0.4315586541201337, 0.4804978203852703, 1.567431381161573, 1.2952729674385586, 0.6938665335547832, 0.5073887108148186, 0.5511677600808031, 0.5560433126087114, 0.689559684498457, 0.048057721527297474, 0.8306082780130726, 0.35698800943254244, 0.57752897740057, 0.5811494022541972, 0.3785254568309892, 0.42328171753406735, 0.4912318128792925, 0.9584576948628832, 0.17327578232126642, 0.21543958843299021, 0.22452306659234972, 0.05069452012217156, 0.20752945124325214, 0.6870700263255567, 0.5531729275299546, 0.4722701892611435, 0.4547521251717645, 0.5880918832105835, 0.7474518098527364, 0.8960263502514201, 0.5699049115102475, 0.7198042524279864, 0.8682335892438438, 1.3537864851292625]], [[0.24807943985002473, 0.6643242018785734, 0.4234768629344722, 1.3152202109196982, 1.3157439939940112, 0.6678832466041171, 0.4561753521844006, 0.5543487893478438, 0.5121903021965729, 0.3783968563892269, 0.06825338152599636, 0.8771873367321122, 0.5579369286425282, 0.6042336556847708, 0.5180914421164211, 0.3865846036925705, 0.40996851503315623, 0.5255683172362192, 0.8858633346014357, 0.16823686191952497, 0.22974222098653355, 0.3846155972274971, 0.05005016856473052, 0.3353558295669143, 0.5847350451002655, 0.5680822743053879, 0.4664596048971224, 0.4651026029033246, 0.6256366254583476, 0.8642368045594796, 0.8842143343507042, 0.5916083949092843, 0.7119433678156251, 0.855433435051842, 1.3152688229731164]], [[0.24524865469996943, 0.5298664425313673, 0.44292878225881027, 1.1928809136591856, 1.2919089207900525, 0.7172430351324304, 0.465755380340106, 0.44840458650883397, 0.49024579474127555, 0.3283949091091909, 0.06106492735973343, 0.9558503197282942, 0.5316972189332254, 0.5433251894020074, 0.6845551706608337, 0.2809697576728302, 0.4828360868152809, 0.497160008785116, 0.897351351200976, 0.17508926201114405, 0.22249373072140505, 0.4980402673167671, 0.046735067722196995, 0.406304812866362, 0.4868555535306639, 0.5553974593000317, 0.46598771431117514, 0.5319309388609444, 0.654426969086617, 0.9595718305875166, 0.8881707003092096, 0.6921602489649242, 0.7601640532122914, 0.8548102772597334, 1.2968364946871886]], [[0.2776724568564168, 0.5253248918103034, 0.48450037747753916, 1.037637600539969, 1.22144303304441, 0.8192839315596764, 0.5575378566688214, 0.3700438969346243, 0.4616293384434084, 0.3057746855119705, 0.006508741592810162, 0.6671158332107034, 0.6796723097227824, 0.5423638231923232, 0.6273895927412168, 0.36709335944634847, 0.5604258799572022, 0.6786557940896637, 0.9456920899350143, 0.16963170629507063, 0.23823439477089098, 0.41370026408307564, 0.03571915589967607, 0.47104073877079045, 0.44987673551814966, 0.5452945802974456, 0.5228625457300557, 0.559907175049891, 0.6667743100487735, 0.9952728286678844, 0.8461214641435357, 0.7269055920395442, 0.7500320600291905, 0.8542008509569302, 1.2686006236545007]], [[0.19239226495172132, 0.519256632159786, 0.4513207681597592, 1.0330790246598216, 1.2542528682576035, 0.8554759339354445, 0.5412290649362071, 0.41105085473432357, 0.4733204369246829, 0.3137987220781441, 0.007060606227727923, 0.6764424930781998, 0.635760020969016, 0.5466129185013062, 0.6148920891062973, 0.3199311577011992, 0.6417574532178101, 0.6442749281336388, 0.9457667768166148, 0.16156288772708907, 0.22890814011363192, 0.2771391856019426, 0.029703487094724645, 0.5552829234092109, 0.5418402710014496, 0.49171066608835856, 0.549909682653005, 0.5643340281656175, 0.6627511260171569, 0.9870720218247061, 0.8347658021580132, 0.8726867644555558, 0.7386926322334425, 0.8717627639341929, 1.2650817047517984]], [[0.19338695398412564, 0.522325931748095, 0.400321923237528, 1.0875023228411975, 1.2580029244944706, 0.8909707899488823, 0.5633672218966485, 0.5280930736712109, 0.46025916822445384, 0.2927712087690098, 0.039528383697566415, 0.6149473760661647, 0.6969048490916196, 0.5125700332589802, 0.5849380905815218, 0.3334328278423454, 0.6286330464652561, 0.6906348006521947, 0.9140982288058057, 0.1634083822435605, 0.1992057674197579, 0.18996517270500657, 0.025164480230895132, 0.7533681636105125, 0.5746087894904337, 0.4412735289194232, 0.5382835417796384, 0.5799757530882046, 0.6704145407152148, 0.9757381727230403, 0.8040775226741387, 1.2566168189505778, 0.6941744667944524, 0.8463784020084009, 1.2527949541631367]], [[0.19635531407268153, 0.5317543748580347, 0.34791480607308845, 1.0232733503385303, 0.9121708746527195, 0.9787374740225551, 0.6330058952865227, 0.7073844650154303, 0.6941427557442811, 0.30130533644131074, 0.040851450377307405, 0.5877084324171398, 0.6771751412203573, 0.5076829271429669, 0.4945278655142896, 0.3275096654121179, 0.5934278086267106, 0.6903958419717315, 0.9996254261792323, 0.16649648777894283, 0.1884527729257035, 0.1476473980208004, 0.027998883942818756, 0.8925497140581524, 0.6193782994816063, 0.3836191989622606, 0.5794311984154148, 0.5451495905900281, 0.671413867683218, 0.9165861349533968, 0.7741296419145477, 1.466305021707976, 0.6399419980042811, 0.8235729956526212, 1.2434159115445502]], [[0.19192689738400237, 0.5883978878927263, 0.3505413920483978, 1.0246522365098298, 0.9949366251249635, 0.8790480733389805, 0.627385571385987, 0.8646645208447286, 0.8088266763825336, 0.33042480860479495, 0.03522769448861485, 0.6087097647095192, 0.745424939872869, 0.4959525461035084, 0.46574686662954656, 0.3229479507489768, 0.615417939168554, 0.6754019641967963, 1.0317690530626107, 0.18488987659259615, 0.223447473336112, 0.144379860640048, 0.029267171640614507, 0.9166633892091731, 0.7414788953684642, 0.34612546075556266, 0.7172973778515332, 0.5377875809168928, 0.6234676844954126, 0.6781987924110824, 0.7623959722588016, 1.4896210372132146, 0.5561686458805029, 0.8197088920614417, 1.0189841022080603]], [[0.17353231865577545, 0.5603252882054507, 0.36833123358712005, 1.2375957263051984, 1.0638279645244322, 0.845933968294245, 0.5502743723481736, 0.9454800487197885, 0.9634866097702548, 0.352145753461011, 0.047356048899908, 0.6334943977259139, 0.8014207189499376, 0.534610695214722, 0.41527783270967644, 0.3112019433515263, 0.6476789030288935, 0.6689278157271008, 1.0919744279763859, 0.15888705811518644, 0.24237519242860223, 0.16478473952938566, 0.050263671727358746, 0.9267110622990595, 0.9042834393556295, 0.31313158147374803, 0.7589015965705936, 0.5396796122265655, 0.5310222779563685, 0.4823514226349833, 0.6574749903412894, 1.4950203183672657, 0.3971284665981021, 0.7284505953428762, 0.7310782894909633]], [[0.18266788121659244, 0.5867517761234519, 0.3708420520316207, 1.228864561384903, 1.0597700980877662, 0.8441815902697871, 0.5641306586527708, 1.010166074790345, 1.1077232668576174, 0.3704010343818877, 0.04604661286737164, 0.6086342135174678, 0.8365951742943848, 0.5155065569257267, 0.4199629244375952, 0.323378627100233, 0.6453408103015551, 0.6794446222384135, 1.0705611958772705, 0.16262818127555168, 0.2514989403452893, 0.1732923232590491, 0.07296431095095046, 0.8954701520743993, 1.032913280277383, 0.28800555989292287, 0.7422171514650096, 0.623021806404871, 0.5059266953804044, 0.35594661503714464, 0.4831655304473588, 1.4586144584532152, 0.26959480446434525, 0.6018773036426806, 0.5928311824974051]], [[0.17899953748913605, 0.7072638668977139, 0.35366147101756124, 1.2935522731293276, 1.1530755376146675, 0.8794440571357713, 0.5217988101445213, 1.0326843126577814, 1.2148616193415367, 0.3573452804763904, 0.05278304295729911, 0.5757791288321779, 0.8358893981944533, 0.4735091031634687, 0.3706226751221593, 0.3187102045390996, 0.7203884945632153, 0.6925141974445383, 0.9818781046089132, 0.21495524071249383, 0.24658917840458086, 0.2171355476863604, 0.079233010906144, 0.8767798309566835, 1.085816517430536, 0.26487316736587674, 0.7247340112270377, 0.7483604483037397, 0.45995839555009926, 0.4358462102193108, 0.44768140220859276, 1.4715142560146242, 0.3466296766082039, 0.5522245111371746, 0.6005246862104878]], [[0.18017815642291518, 0.6970258667933273, 0.38710625889953965, 1.3404282895185573, 1.1980588631795843, 0.8904258821945528, 0.5403629115755764, 1.000292878819298, 1.270779246407594, 0.31946171244366856, 0.05729653220101553, 0.5702675466334746, 0.8466679815283764, 0.4680548630986986, 0.3555400173964193, 0.393611950995557, 0.6992347868607212, 0.7657002580550243, 0.8977837470962904, 0.26015477093115, 0.23187069203743815, 0.23731651699946013, 0.07889388300703708, 0.8055070536484517, 1.0886141485637626, 0.2002869454785495, 0.674015195714863, 0.7612419015362978, 0.2970320758293642, 0.5081958434561641, 0.35216409723592657, 1.5347430445221222, 0.4371210065316709, 0.6339066919412486, 0.5273723625647728]], [[0.23598346078488097, 0.6103040582032417, 0.40289501919867643, 1.3182258562329339, 1.2007717413597883, 0.8911957807507083, 0.5432145029332014, 1.0034728924069272, 1.2402855628954006, 0.31162349776549547, 0.031782725176951174, 0.5790599507619772, 0.8716845749020096, 0.5422470825525634, 0.32935819134010536, 0.4021825034957034, 0.6417799879174211, 0.7796048281316805, 0.8635272035223598, 0.2715291668081533, 0.22586759690009636, 0.2528076284892222, 0.08509339234065463, 0.7221840057162594, 1.0605879875861353, 0.23509245279958382, 0.6909907479678011, 0.801267908216753, 0.27849336097394295, 0.5752535122050404, 0.36708363601056104, 1.5364299767080234, 0.5400023762495105, 0.6879139514101296, 0.5457056598376253]], [[0.21449323857629896, 0.6367160584724866, 0.4015702766697288, 1.3050097074568217, 1.0488388426381225, 0.888913326956853, 0.5048078697308891, 1.0106022935365864, 1.238971414059307, 0.3190472138966406, 0.041274932324225064, 0.52295820012727, 0.8687200770631036, 0.5322593642127331, 0.3334967164620151, 0.389846474459457, 0.6983365035772879, 0.7726481490265307, 0.9237678234149344, 0.26411073514955824, 0.22876652431497496, 0.25122802201777517, 0.09357951456448138, 0.698022261949713, 1.0392092263606127, 0.4374230716909577, 0.7319425495187541, 0.812557735978392, 0.358436069398418, 0.6798950352643389, 0.38969026483697444, 1.5300422395418973, 0.6879303098093037, 0.7548843550063241, 0.6844526195284344]], [[0.2072249040184065, 0.6783193217433852, 0.4019997777471474, 1.3106623303264213, 1.0968752674376665, 0.8675122090929619, 0.5125948866763741, 1.0222401595284787, 1.3428690760112034, 0.3255016394424044, 0.015318510688253878, 0.532816552307062, 0.8626294753571875, 0.5457380046329037, 0.32249960263314525, 0.37973401573094223, 0.6278425813078221, 0.8185392522283753, 0.9415970553446842, 0.27791191092210255, 0.2285541757339943, 0.25352628448402115, 0.09398295127961802, 0.6410665765183667, 1.028895919655911, 0.5354195297353115, 0.7211714447540973, 0.8701776883277393, 0.510608593705101, 0.758769384741336, 0.42705394046783063, 1.4584541658296157, 0.787777710553628, 0.7984072111679476, 0.8428706048955388]], [[0.19450658174139335, 0.6819691006269191, 0.41937207785938246, 1.2756187136388943, 1.138511198266543, 0.8577513714613338, 0.506456078502137, 0.9945515082974588, 1.4615269036085847, 0.32777038897133615, 0.02815449662260437, 0.4758576645581014, 0.8376499871797229, 0.4717087302854775, 0.31452281224954287, 0.3766649665036459, 0.6293232279474537, 0.8165923725458899, 0.9558724864678213, 0.28990920400254416, 0.24940272575232675, 0.22508349500297942, 0.09870820248551371, 0.6089996652997435, 1.0585567151860038, 0.67929317192159, 0.6982987735848512, 0.8671115673992111, 0.5780683622496505, 0.8458863188127937, 0.421621305520379, 1.297020251871898, 0.8151040768409101, 0.8433742004156981, 0.7802235257620365]], [[0.30634081935147256, 0.7748374093735039, 0.3575913632584199, 1.3057208476341466, 1.2102133789189269, 0.9219186864138023, 0.530458726609957, 1.0457165082668176, 1.666922137319795, 0.33975231482655743, 0.03412587968533551, 0.4778655509265256, 0.8274354808316022, 0.4366966327726628, 0.2557036234440659, 0.3415892140065495, 0.6696946651407389, 0.8093002928126569, 0.8942935612209656, 0.3048731574381913, 0.2613297998924534, 0.21694043647620656, 0.10515400266198421, 0.6727452087767205, 1.1604972749757425, 0.7923184637370569, 0.8435929855642208, 0.8993270571478416, 0.7001653257250515, 0.8815663769752575, 0.4288055645120944, 1.056596862643716, 0.8443299943342071, 0.7993405614085793, 0.5889038393393164]], [[0.590438296216311, 0.8291966741736996, 0.3444126730242683, 1.2954764016055758, 1.226161667168657, 0.9289545676589436, 0.548941084741858, 1.009399615148422, 1.6951347582591247, 0.35073118184244395, 0.017724816128645408, 0.43374530097281333, 0.8189216493145832, 0.4390411266878834, 0.2522467857899455, 0.32455854123943984, 0.6598167058573655, 0.8324947019071192, 0.9047639646087635, 0.32978875403573205, 0.25724593446426247, 0.19665285500684104, 0.10686663205567999, 0.8176929788137697, 1.2568709672056673, 0.7173952602678713, 1.2195580871025329, 0.8831792016210014, 0.89102735808351, 0.9286260533843826, 0.44380598728995524, 1.0488769451425868, 0.9299881429430388, 0.7363258889602919, 0.56751175105804]], [[0.40046987106062903, 0.845376813382172, 0.36688311542342955, 1.2987140747619188, 1.191713582446868, 0.9372598970874619, 0.5620914318821832, 0.9632776455317327, 1.6393400336022825, 0.365961858370023, 0.021259893949164985, 0.5502993778262737, 0.8252022584618841, 0.5530749829585156, 0.23822383601593966, 0.3829454302353782, 0.6188644956181244, 0.7688617969840392, 0.9535814504578446, 0.34024722016320186, 0.24783330371273327, 0.1989759808875795, 0.10147297160815284, 0.9591504457923754, 1.2331366457762922, 0.3730473235150098, 1.1387996663159963, 0.8431386061111075, 0.8141842364815766, 0.9551055408098348, 0.4311115129929096, 0.7785154628013419, 1.0096039880156655, 0.6667474470881898, 0.4199999917725048]], [[0.5316147703380945, 0.855525152357705, 0.32110137775408926, 1.417474931750573, 1.2752569883768459, 0.8634477366145459, 0.47423027794604705, 0.9023554924644481, 1.5785608439106897, 0.3640285513474811, 0.021039901089625238, 0.49759617318387594, 0.6935370960592069, 0.535629483978758, 0.23783974944854563, 0.40512330523337825, 0.6099256272761313, 0.7356265378817854, 0.9389015635107034, 0.33895257263910483, 0.25210506792821996, 0.20723931206485502, 0.11120198742804331, 1.044098060195686, 1.083025643901633, 0.23259917331472396, 1.0143164453480902, 0.7793162355843786, 0.5950805322641219, 1.0548611527799563, 0.448790263535354, 0.6811501616360021, 1.0580488823101533, 0.6531285497292327, 0.5746510239443523]], [[0.4086620580458176, 0.8774623917174286, 0.3272930674848787, 1.3975283991045286, 1.2511729515241785, 0.7460611289291545, 0.3966843103854206, 0.908352257417615, 1.5269921091931327, 0.35907533134966274, 0.025399450605980034, 0.490220738328337, 0.6183315639175844, 0.5378647586959668, 0.23784332361721738, 0.419540671297135, 0.59661856414804, 0.7734186086159962, 0.9252887966699987, 0.3652416436523057, 0.234588921756813, 0.2236212711885866, 0.11840053440645873, 1.1684466638569961, 0.9883967527690637, 0.23843098726784745, 0.9176325455321837, 0.6961090342960765, 0.45237369747101935, 1.076579140402952, 0.4179616196156668, 0.6892940576611156, 1.1872391588140596, 0.7905345202947601, 0.7119124482720438]], [[0.4295060470828238, 0.8285430650206614, 0.33631836320103525, 1.394878239148093, 1.2336072500138355, 0.7550576313294495, 0.3955930141180878, 0.8690419077074771, 1.4204431619890414, 0.35293163983263304, 0.041243312255846765, 0.46669185014075754, 0.6424428884778743, 0.5506258899193999, 0.24711406375812656, 0.405355782101124, 0.5392217119887315, 0.8064846055090511, 0.8910073734411352, 0.3596033059596677, 0.23342723511688862, 0.20878400340823405, 0.1628350520419573, 1.170526548748631, 0.9531920887876394, 0.2681056079078505, 0.9242704656284041, 0.5633441021753767, 0.4336308375686866, 1.0944801077917177, 0.3717596204348965, 0.655395640200158, 1.208059549592479, 0.8466677256720265, 0.8214952701856333]], [[0.39862529106903555, 0.7937107956456472, 0.3444317385987352, 1.295828526845213, 1.1172158092768614, 0.6981430274182127, 0.3876602874505878, 0.8812289012453386, 1.1574779116311884, 0.3307030873291701, 0.045790564249482704, 0.5290669053375197, 0.7299483730743843, 0.5794519070528101, 0.25510865531046745, 0.42161663822619444, 0.5245506980007484, 0.8115254376701835, 0.9015428230813555, 0.36878278546787185, 0.24514694200063158, 0.19106102478718745, 0.1318174760296262, 1.092600488418732, 0.9078049510083057, 0.29743787980668734, 0.9384967557548352, 0.41996393230247264, 0.4204430736011352, 1.1407575703392614, 0.4324300859221173, 0.6259211448602054, 1.223806751181693, 0.8430025036699372, 0.8671721594107425]], [[0.37920235126234536, 0.7349554163078684, 0.35657268558836575, 1.260374293391123, 1.0670329563574947, 0.6965691523958022, 0.4002050409125623, 0.8372518423692696, 1.0085127025643064, 0.3381019592640758, 0.0636444078186369, 0.5481716660662425, 0.8355421293439727, 0.5928954676148506, 0.22935600888470253, 0.3921338403178175, 0.4954547304624545, 0.8011412238359075, 0.9041557528378688, 0.333807728748197, 0.2580016419371136, 0.21234755196274338, 0.1650066713730205, 1.0872961410522621, 0.9203042147875757, 0.324829461412732, 0.9805592179300344, 0.4703550253049988, 0.3965150134814711, 1.1609694444273264, 0.6023611453312082, 0.6439946856215673, 1.2302141461535034, 0.8020996879694191, 0.904304544825298]], [[0.3583108054083163, 0.7346517169721518, 0.3204196361132662, 1.0879890773392096, 0.9788330931710364, 0.7074494141079908, 0.4140478425199574, 0.7304199750620646, 0.9488889317277464, 0.2798794780277517, 0.06958691262006596, 0.48132578993291397, 0.9106589106460516, 0.5502022380087805, 0.23576098578998075, 0.38629397158213685, 0.5111061331349717, 0.8257379535512153, 0.9224674842138669, 0.2640660778564655, 0.2657483671506368, 0.2191590383610038, 0.13536137181719265, 1.1493390788626339, 0.9707329316381411, 0.32711465110458987, 1.0351975032583725, 0.5601950306356976, 0.3663745525701732, 1.1319608429528714, 0.7371800598623423, 0.5793038938613081, 1.0535637701315228, 0.7486842264025356, 0.9620884182745739]], [[0.35077599623958067, 0.6740388865464226, 0.33271797235109113, 0.9474192949286306, 0.9805001021862221, 0.6917732900372919, 0.4361360855422116, 0.6305407694366554, 0.9570401686690602, 0.352363427067662, 0.07475712355577653, 0.5213988463841418, 1.0153480635453744, 0.5262315314217768, 0.25324265067967966, 0.3966502822997535, 0.5176141407204543, 0.7817593784261314, 0.9601695884536399, 0.23789911008687556, 0.30791281119484115, 0.20595134520428998, 0.1362612418597755, 1.1050861186238425, 0.9738135116681984, 0.32532186385818995, 0.9287547284934324, 0.5275308885156742, 0.3696686363538164, 1.0117092584411669, 0.7851653828699616, 0.5419756815322194, 0.9553395998311462, 0.7771102010640082, 1.0062827748037049]], [[0.34151209211792977, 0.5727557521372432, 0.31106373255086844, 0.9405666023105441, 0.9692842049929381, 0.6820330955976411, 0.39676588250331096, 0.7170933475449919, 0.9941112846038711, 0.34744876377450157, 0.0853520622402106, 0.4946085502700156, 1.0299445648262315, 0.5251475179176861, 0.2572829285790604, 0.3720706544283543, 0.5135652618731716, 0.7804480060538088, 0.9008995909973667, 0.23498327080890963, 0.29313870403897524, 0.21195511501004421, 0.12444128350532394, 0.9746295476662143, 0.9746020175904208, 0.33357400331274145, 0.6292479377950491, 0.4968810096372088, 0.3332292606541566, 1.0082515267795025, 0.7795970559551004, 0.4803750832534982, 0.9086984889442264, 0.8054622254108141, 1.1008880065236402]], [[0.32885207991212584, 0.5391644575543483, 0.31237940492294025, 0.973290921503718, 0.8946755319923705, 0.672838239264319, 0.37270582803639496, 0.8121677851547623, 1.003629392307425, 0.41896054620390494, 0.1034526116160922, 0.5320903605119371, 1.040080665397353, 0.48252131835619416, 0.24687500501967696, 0.3833819147515387, 0.5243957440172293, 0.7575683392724476, 0.9198162671240425, 0.22737955754107952, 0.27189585843002817, 0.21099336081977632, 0.134158271890426, 0.9409403695184688, 0.9748942626105906, 0.34401973542484, 0.4750601907928209, 0.47213523799379675, 0.31354573118707535, 1.0714079391441957, 0.8071740461262371, 0.43191469218173983, 0.8855271939516283, 0.8370233797181823, 1.1575743854390117]], [[0.31000792046644954, 0.5191294275219915, 0.3000305258707818, 1.044820333361237, 0.7934203542676488, 0.6661966971551094, 0.3845542320156855, 0.8255134634934488, 0.9928334783788778, 0.5014032703360842, 0.10680402267629568, 0.549618687054289, 1.1373744577211369, 0.5029330800985988, 0.27137764157071886, 0.3520891421431137, 0.5410863691541566, 0.7524458142362784, 0.9391358768216183, 0.208834506058997, 0.2601571097499527, 0.1956787236851683, 0.11629820629943742, 0.896661657663879, 1.0342135983648646, 0.29795117551521705, 0.37937615957852977, 0.480617111495391, 0.29121132234722263, 1.0536703346559086, 0.8854476302582271, 0.3729825211873755, 0.7392382408531395, 0.831956231969015, 1.1801138408300307]], [[0.3529168246934271, 0.46503451294210796, 0.2644195686617547, 1.0618209775239993, 0.7146209809278108, 0.6750516698524678, 0.38225381978746253, 0.8378813675718024, 0.9683114605998995, 0.5844417098425541, 0.12866254190790005, 0.6273697149029127, 1.0988918406995858, 0.5118451657938683, 0.291074029558148, 0.3624057282472575, 0.5585916229993233, 0.7580306356360929, 0.9282263870258427, 0.2110892966625677, 0.238057620428895, 0.2056529662354994, 0.1073117470282909, 0.8370890882763415, 1.1045135346123263, 0.23908978890769145, 0.357659293421908, 0.5054446247396015, 0.3197870398820275, 0.9947087673992183, 0.9212514977700501, 0.3416613187798673, 0.6734349967720276, 0.7350547727765798, 1.1372974419112691]], [[0.36413922189643155, 0.4793188191536182, 0.2749102726404468, 1.0744765375189393, 0.6255143781041077, 0.6687634549453448, 0.37698543375005034, 0.8795362096511482, 0.8996360750908855, 0.6306408635275048, 0.15921678747928708, 0.5629702592251449, 1.1076054614069843, 0.589637643866748, 0.2875782098337668, 0.38755611308828075, 0.5312001124572743, 0.7678407683806969, 0.9126568059690462, 0.22477028414032493, 0.21982668357626586, 0.2043176244779814, 0.09946578970641243, 0.8154622074598887, 1.1796806668981084, 0.2106011401829302, 0.29514226932147325, 0.5408259430233978, 0.33626825407592786, 1.044141859908398, 0.9310713226724242, 0.3337866865490363, 0.623364491347276, 0.6804413588862717, 1.0899290963824628]], [[0.3719434737308497, 0.4945241883454313, 0.31407260285511895, 1.1795925879199787, 0.5983526911531142, 0.6617902836743533, 0.3995138527489748, 0.9925991795035419, 0.8050425532618416, 0.6119258618085507, 0.20470637690657228, 0.6472904840378653, 1.081214221383346, 0.6016899780315135, 0.26672861408353166, 0.35572496567695977, 0.5233556550268759, 0.7233377745773095, 0.877308606981194, 0.24950712417870682, 0.21230534530775536, 0.2490485610121165, 0.09909563477617629, 0.8139434732344345, 1.205830717948774, 0.17141803994438448, 0.2956427083891392, 0.5903972641794498, 0.3317122193304636, 1.0623789805937085, 0.9547260412615182, 0.3835129200951379, 0.6172597140694751, 0.6204211128294985, 1.0368136765300418]], [[0.3267185895806652, 0.49434896121914046, 0.2919325889634984, 1.1875322586426662, 0.6297763577891033, 0.6432276075950898, 0.39549621865168344, 1.084081031966785, 0.7328430689426002, 0.562763528044336, 0.2270763253903343, 0.679363189899973, 1.0054041859271239, 0.6499420284213011, 0.2379027283181627, 0.3297730123410195, 0.5587393037980638, 0.7900085477335426, 0.8157175087016681, 0.3787693035291107, 0.2576369846523621, 0.32462767130020354, 0.19484888606833423, 0.8214081512341216, 1.1712749293436227, 0.16349320198940254, 0.35474005445588697, 0.5556844625088464, 0.25763114328708053, 1.1974520102284876, 0.953760538205105, 0.45985140249444145, 0.657166141261468, 0.5503252043408632, 0.9296350510699274]], [[0.3377822264180188, 0.5139281352820837, 0.3181601442381834, 1.282368767815517, 0.534710638466368, 0.630316010376353, 0.3857708365315231, 1.1115029375797847, 0.5984686154663496, 0.4798051827247957, 0.21020205760170552, 0.6538111537135463, 0.899592766106296, 0.662921726694143, 0.23531687724737924, 0.3294473862150836, 0.5928821363388166, 0.7218387042714491, 1.0048288053703893, 0.3250180158810049, 0.2568332331615363, 0.3303310373473001, 0.16238837288723967, 0.8020386546357143, 1.1173185982709952, 0.16508980567920378, 0.38125724949809103, 0.5735504071659459, 0.19715155132710793, 1.1366999200960275, 0.9595098544655123, 0.5411705023611119, 0.6386766629712537, 0.5197733282575274, 0.8165326206431319]], [[0.3525781183365858, 0.4224161506281015, 0.2714398208053075, 1.464094786204731, 0.4685518128801003, 0.6142364593195317, 0.393114957399178, 1.0381050082554202, 0.7344755629303212, 0.487339181937719, 0.1902342605249943, 0.6208113528874927, 0.9025107429426423, 0.6956836552808813, 0.25222596140955145, 0.43830360487765807, 0.5892948016125239, 0.7476360397097801, 1.0811635030735636, 0.17781547505798628, 0.22600510555768127, 0.3434293391235177, 0.15610150248996357, 0.7949361996598024, 0.8934544110630804, 0.15756105249633218, 0.4043290110991174, 0.5260378846530791, 0.18562846588409537, 0.9323051766902587, 0.9772194687044446, 0.7052273390494799, 0.6054887843248067, 0.5622593541160884, 0.7941102931963988]], [[0.43185385096850126, 0.3604886190823363, 0.1801970959235463, 1.3897279824699682, 0.5398052399526774, 0.6156615943646804, 0.39144480643450724, 0.9726148301115632, 0.6198786846523148, 0.4259678805204942, 0.1835983458178001, 0.6335424127434506, 0.8530371809625517, 0.6793591216144688, 0.33379778399760596, 0.4253291677774633, 0.5595092554842229, 0.7115953736299269, 1.1314490138174356, 0.13039223685191048, 0.22941585185471225, 0.274225796005075, 0.13111916649120461, 0.7864755788478751, 0.7077681186042177, 0.14299876448435667, 0.4530233907097687, 0.5583859259246999, 0.22716749896558408, 0.9078951243821292, 0.97249215594606, 0.779671226745362, 0.4999209842322234, 0.6121088122889059, 0.9040300898410155]], [[0.4640624856668598, 0.33182808161553057, 0.22846694001047482, 1.2392903194613039, 0.6359960710746063, 0.6264467414394799, 0.421344888609183, 0.9371850004015101, 0.6353687515419727, 0.4003032403138915, 0.17719266156171443, 0.700911083088398, 0.825660588535407, 0.6646439163418455, 0.38604253025622115, 0.36198836745646823, 0.5601657858281021, 0.6874492634051351, 1.0157071041961183, 0.1441852734731582, 0.23840270204142472, 0.2727939775873236, 0.12983294597531161, 0.777064584894547, 0.569147880290706, 0.13195615810435365, 0.45393710329336134, 0.5980383466274168, 0.2597806964391042, 0.9788027430422821, 0.941359171939893, 0.7351107815443515, 0.41076672073157505, 0.6268451593375088, 0.8918737827492846]], [[0.5043153625155175, 0.3504512359327331, 0.2302406036294627, 1.0537893784851198, 0.7009411620223016, 0.6305389015217758, 0.42601565039172373, 0.8742921912300494, 0.6680503636028805, 0.3852737145401578, 0.16590701742224434, 0.6499505966923294, 0.8073061099426021, 0.6991901152298642, 0.4342100998199101, 0.41478883255993926, 0.6169186357982307, 0.6893710799801537, 1.0016891593291752, 0.15961668339051294, 0.2656785269263628, 0.27527292379293306, 0.11704228348531719, 0.7357176549378603, 0.4886532514064944, 0.13603332115195904, 0.43020819421983514, 0.6562674743373685, 0.29428511414439984, 1.0380661590402918, 0.9371221453414906, 0.6956194694572904, 0.3138233288181166, 0.7631459612829214, 0.8524816857034607]], [[0.5488108363182406, 0.3559915483171401, 0.262220681166604, 0.8414024806001239, 0.751439518654361, 0.6445006569105877, 0.43640771552708524, 0.9115241770051636, 0.6679977283335001, 0.3014004967698939, 0.15322676678494018, 0.6425289033525018, 0.8047892953871872, 0.6443168510863333, 0.42724761565649394, 0.42607416437273693, 0.5354647330989516, 0.6707426406217041, 0.9085091813450311, 0.17363965726832875, 0.27427436597633625, 0.2838274398846437, 0.09835893339114579, 0.7229002832870335, 0.46706063915659074, 0.13385422642201528, 0.4643886743770643, 0.6268232345789991, 0.3115695920993654, 1.0922023220658135, 0.9292905877711217, 0.6799254753002042, 0.26076986921170187, 0.9040897124604003, 0.7903056794688432]], [[0.584239190516733, 0.3823377111184687, 0.2640956515809567, 0.7364699255256912, 0.7229150920202264, 0.6864821026861273, 0.43274871727803754, 0.9137512043520382, 0.6928188598316751, 0.27017231908065736, 0.15784615543377642, 0.6274510680213581, 0.7524641825466754, 0.6613534089349418, 0.43374379089898546, 0.3913934812933467, 0.5868557411806916, 0.66747736334742, 0.9355395121371479, 0.23971228599561611, 0.2802203249714338, 0.2715657866808501, 0.09872100747597178, 0.7074962337436435, 0.5534318589422245, 0.12469679182225332, 0.5125272768444121, 0.5806533337327996, 0.32345888571276193, 1.1688614126624337, 0.8656060794680358, 0.6846919377240877, 0.2462430089987178, 0.9647885157580172, 0.7425573715567972]], [[0.6246457654675999, 0.379318175638309, 0.27411598615272087, 0.7135196871238703, 0.6475383507500673, 0.6772149142230999, 0.4301540494510111, 0.960417791290427, 0.6646269312207782, 0.27312481763542457, 0.18295930350712752, 0.5697802122534568, 0.7041544666716002, 0.6162417351418781, 0.4081549245483175, 0.38843879646346613, 0.5695642792161667, 0.6702746177730383, 0.9965053530489646, 0.2668737636237656, 0.2903596677889182, 0.2955631732581676, 0.08969130674434594, 0.6608459492471201, 0.5998354558151004, 0.13378130774624208, 0.48312120804313263, 0.500258736945127, 0.3040769354793085, 1.1929079481580398, 0.7999841406185588, 0.7002931850906782, 0.22474950792746762, 1.0034114958841864, 0.7198504625647488]], [[0.6570070550958432, 0.39472689365935676, 0.273817263189072, 0.7091884124038474, 0.5979112622330542, 0.6796750939086308, 0.41502704461083967, 0.934934482065416, 0.6434210958061946, 0.31990219009492404, 0.19311569943282825, 0.5520858079338755, 0.6070271631633086, 0.6052962880918891, 0.4336351806193348, 0.3244778008687668, 0.5935639413724172, 0.6500742270891731, 1.0275197189637841, 0.29970564521479304, 0.31122541250731034, 0.3188181756407694, 0.09104652387172077, 0.6332076818596826, 0.5544679186834712, 0.13245223309131682, 0.4862302279353856, 0.4766750218426726, 0.29159219689725485, 1.2267849978132102, 0.6313043636737588, 0.7450832022537432, 0.22258717659110733, 1.0458792960255046, 0.7063793892855533]], [[0.7316310941425719, 0.5423808030295959, 0.2851322358120918, 0.7090650534835172, 0.5422246161105497, 0.7010289345866997, 0.3754396914269106, 0.8990134161866337, 0.5977510657214355, 0.41957244339882027, 0.12712018582253032, 0.5460341680649274, 0.5732461858781637, 0.48517587087528213, 0.43628180707649344, 0.3088033776505186, 0.6609497241618462, 0.6987150797057362, 1.1210777127015068, 0.3341852034453102, 0.28682040525221436, 0.3540857513459531, 0.08554840948058544, 0.5919196517191618, 0.5513272165457468, 0.12839326104770699, 0.4420574041349231, 0.43969763783732546, 0.27411560547109237, 1.1911207091558245, 0.4831694256800785, 0.7150642871955216, 0.27765242733826523, 1.065518225549971, 0.6957436106886029]], [[0.7837731729121826, 0.6777748525529361, 0.2877806510992062, 0.7517797670433161, 0.5604856969912858, 0.6933517672195162, 0.39172304804670405, 0.9282555794150159, 0.5953892106534544, 0.5268618154934817, 0.10530003336861496, 0.5279789813379518, 0.5829993714526239, 0.49317945774399047, 0.45374370183433155, 0.3141032004659915, 0.7858636437036863, 0.6947966996041881, 1.1500510400203061, 0.3846250999440112, 0.26079994302926013, 0.38045833612578606, 0.07898754550076091, 0.5436917300260206, 0.5646448204738669, 0.1274483634506563, 0.43037304837457185, 0.3626433094281881, 0.2655673928314467, 1.226181484049373, 0.4184750277101435, 0.7181348742421476, 0.31785293561658784, 1.076336239428054, 0.696668813173154]], [[0.8391048220447738, 0.7068208831654643, 0.2971355570541202, 0.758528813381512, 0.5768571070318758, 0.6707200286529302, 0.3762985206866568, 0.9689812645228568, 0.6512640230450175, 0.6097943699081523, 0.09562928608972592, 0.5135615286021107, 0.5879415664003366, 0.45346066475042984, 0.4588547168087717, 0.30201667431244805, 0.7347768962641867, 0.6357991586563312, 1.1497254604625282, 0.39683418092238276, 0.24008344213303728, 0.4238599470614657, 0.08220958781906948, 0.4719151559957308, 0.6007963957966133, 0.10125535743045083, 0.37130683227962075, 0.30869251013682575, 0.25364211918470986, 1.1890951481491767, 0.37641366243003704, 0.7373931013124179, 0.3969459943148713, 1.0759882369548692, 0.7094975161803405]], [[0.8685188449395016, 0.7684901790938974, 0.3005831763232202, 0.7411724255714861, 0.6000034371429874, 0.6736010582480381, 0.37595834858027577, 0.98560579063405, 0.7697307932108268, 0.7165477641394584, 0.17331577188305608, 0.5037554848521233, 0.5383113880513128, 0.4870841114313238, 0.4579413515456025, 0.3953649562580347, 0.9196975383577666, 0.6327802158584981, 0.8880379814402298, 0.4720828540012676, 0.32967488053888644, 0.45741155932079813, 0.10405545265277133, 0.38283999830589993, 0.6420846020000912, 0.10292949605068868, 0.35056668427101534, 0.3156532822899907, 0.25898686262975346, 1.2104440293090628, 0.34849766095946666, 0.7630452962539082, 0.4392602706741249, 1.1086985950472603, 0.8247164543865264]], [[0.8461528413520499, 0.8642867551096318, 0.29260417591089516, 0.7359519832023127, 0.7031997070348144, 0.6749635476258433, 0.39709763285674576, 0.9895991073930888, 1.077659129070498, 0.7129470219249088, 0.2149002573311798, 0.537636739690452, 0.5568611870309625, 0.504047900741285, 0.45186358018559203, 0.39924134638261316, 0.9758294980678952, 0.6416982093223872, 0.8883212267742611, 0.4913655111862705, 0.3531010413912652, 0.43763292358671196, 0.11186266254118331, 0.3978957769182966, 0.7162538867869086, 0.12393689324706109, 0.3316963282198676, 0.38600535294625304, 0.2808623947134672, 1.2550039465215568, 0.3607971660694246, 0.8490041643254249, 0.45010978243540967, 1.1707430905917802, 0.9661895975543617]], [[0.877441970113, 0.9096261286279016, 0.33305640171631123, 0.7851950273258714, 0.6883718873247506, 0.6744009432206826, 0.38289628629776346, 0.9217758292841973, 1.1300413804594829, 0.7106555245520885, 0.1085465266632085, 0.5282298303795736, 0.5810539714246039, 0.4855604211988507, 0.44014160306025324, 0.4671184346247774, 0.8220509783766425, 0.6877529169643897, 0.8726530787221217, 0.4491359099696464, 0.33662592781733014, 0.42130536787560846, 0.1167409407830085, 0.41929367831090275, 0.7711463841029341, 0.14596038841061, 0.36099550056570456, 0.37390597924378116, 0.2885769364773169, 1.2001133300271096, 0.43015152488602026, 0.9080572950043477, 0.44377147016392093, 1.2089874944324595, 0.999801193969662]], [[0.9179086509816874, 0.9691273605372398, 0.3512255080743554, 0.7337822021844231, 0.7849164476413859, 0.6261741660768367, 0.3890006818916972, 0.8381894415806757, 1.0259331563259562, 0.6607556583766232, 0.0386410915657683, 0.523066171049221, 0.6683572420903231, 0.5074645976179473, 0.42112666114813746, 0.44895275479474694, 0.863763357262337, 0.6265524342675222, 0.8161517236933724, 0.511401250919423, 0.31081788208787187, 0.4241854917211072, 0.06799569232704719, 0.4356815967927167, 0.7891397496956518, 0.15372198199321607, 0.3734852508919977, 0.36043319210020175, 0.2874902983103808, 1.092852008975082, 0.5690043491010874, 0.8741290848184238, 0.45093382028330986, 1.2002264373964548, 1.0216902110180224]], [[1.0628224625319909, 1.0119628867992674, 0.34997014704169993, 0.7502922888314661, 0.8210493621788347, 0.7876654895005204, 0.3937680161533676, 0.8105641501754153, 0.9214553744154381, 0.6687661417417521, 0.027659787934269632, 0.6009636170098445, 0.6873853597429909, 0.5521227840801066, 0.4174631935922561, 0.41896909603869625, 0.8741804512947152, 0.589468681276798, 0.8267159879640078, 0.5160460920543893, 0.2638377487182164, 0.3882580804090239, 0.0668038498048347, 0.4954336312485313, 0.7914939137269676, 0.1700230249932822, 0.37841065710848304, 0.33016819264174946, 0.27391182741629255, 1.0260283758009687, 0.5945760148412004, 0.8897800469741517, 0.5418028277861644, 1.1682886932989773, 1.131646944347049]], [[1.1576900906800198, 1.053146638866158, 0.364180406625032, 0.9089346346332048, 0.8500323739952511, 0.8112252344827806, 0.392424850080677, 0.7919947219570371, 0.9394920324661126, 0.7144332007098992, 0.05695092491659737, 0.6405410507463581, 0.7577242870464884, 0.5507755205083176, 0.39642784003747666, 0.3875233624699104, 0.907737500600874, 0.5198211250475201, 0.7558275164594154, 0.4889121835942677, 0.20459951733289541, 0.3455749572806458, 0.08165524397396175, 0.5222266632688779, 0.7861269784587858, 0.15460831943110281, 0.3445908092109373, 0.35491056897812834, 0.3252082085650111, 0.8595003968384495, 0.5624998681442182, 0.8598171367332064, 0.5296712439730069, 1.1956776425387459, 1.1750053249225323]], [[1.1864525959896701, 1.1555437136897966, 0.38301690108609227, 1.0286182771084675, 0.8183203954006733, 0.8448742543953828, 0.39864252531211664, 0.8080843404147693, 0.9528690242826758, 0.6993820918208348, 0.07405466713610087, 0.6835156154363877, 0.7553931246281655, 0.5614906498675544, 0.3955244065160668, 0.3719254794506708, 0.7776769628564214, 0.49101129417666567, 0.8736181060152888, 0.2929930822770117, 0.214287554032918, 0.3302734108764693, 0.07099652808720625, 0.5225798972833939, 0.7952694328500404, 0.1639297237200587, 0.3377072763869412, 0.5304432789994783, 0.43477670449817474, 0.716898841889678, 0.6019698885179783, 0.8108864108370907, 0.5828735743084994, 1.2091563049913052, 1.1596370762098123]], [[1.2762202685841504, 1.1994624282440476, 0.38576169433221374, 1.004532035380628, 0.8262684289107443, 0.8472962323853268, 0.40673871465179595, 0.8812559920899738, 0.7509264461958627, 0.6567828622756356, 0.08484699030073811, 0.7195751414630169, 0.764856171272875, 0.5428331384961071, 0.4452947002147402, 0.3631893501303576, 0.7471022569158918, 0.4698476308411357, 0.91874420031291, 0.2106553058788564, 0.20055494254820994, 0.41391788548893327, 0.10220279605844339, 0.6850294181608523, 0.8276724586784272, 0.16276286413745772, 0.3871882616418619, 0.633660909643726, 0.47229290949547975, 0.7166182057082765, 0.666523794950246, 0.626707274247043, 0.637287340268837, 1.1728673803325427, 1.1093097267024035]], [[1.045450975474644, 1.2361786991452057, 0.4295524631206231, 1.060338904354861, 0.8673693222927826, 0.8325103996922782, 0.4076973117005257, 1.0220585348077644, 0.8756343381471092, 0.5667745925198858, 0.09741583151720593, 0.7879483575842653, 0.6545001893201394, 0.5663982912502787, 0.4034827726996422, 0.3576480902300294, 0.6968077911050444, 0.4583032556291977, 0.9884242996448287, 0.19277455513005243, 0.1881025320145646, 0.38043082868821604, 0.0269019507556979, 0.804164105313389, 0.8219078584130595, 0.17645487304668583, 0.44999585913795465, 0.6993289160533999, 0.509987428766895, 0.6335281187141855, 0.7026601990795793, 0.5892607800540072, 0.7721774526621936, 1.161545487659298, 0.9591334541690748]], [[1.1379174177191298, 1.25739609342617, 0.4346733271235518, 1.0715143367560183, 0.9221929641884932, 0.8097259302723374, 0.41216245827751724, 1.0112977222692148, 0.8227004606651765, 0.5534467863426603, 0.12213344052455447, 0.770798225133767, 0.6386061647867354, 0.5778894704505246, 0.4884023136701827, 0.3326933278593513, 0.7263905029924411, 0.44941046678918883, 0.9439901728115516, 0.2432099838836212, 0.18463717009063296, 0.3998341546806849, 0.025292166784517057, 0.8939496032601213, 0.7915185589696496, 0.19551924861843428, 0.6463130931588943, 0.7470858294711282, 0.4993804293707051, 0.6172207645880199, 0.7423028829972595, 0.6216827114862291, 0.7895821945841164, 1.1419379814462736, 0.9705156706325817]], [[1.2024170302805652, 1.2566536868081486, 0.41250195435784953, 1.1271752615688244, 0.9814837973666102, 0.8097016058072342, 0.41587648042140146, 0.9991282268369063, 0.8159469557088052, 0.5687260798457088, 0.11683281326080139, 0.8066875741981637, 0.6490768576762166, 0.5863973211728544, 0.4543171789051585, 0.3331752033854619, 0.6941174212130977, 0.44974048124966437, 0.8756221482936641, 0.2290137163167324, 0.19327691783638312, 0.40566519065517614, 0.025941772322001747, 0.8073698386046986, 0.8021660590843985, 0.20010633912609405, 0.7134669658691439, 0.6948292375916382, 0.5248371540553782, 0.5740033089174044, 0.6951216206719977, 0.6711218258655133, 0.8256856223051685, 1.0374604753690182, 0.9696843145452297]], [[1.3018983488311155, 1.2107119450517816, 0.42106628677955393, 1.1516585226595195, 1.0241744216381896, 0.8164887301269552, 0.42254270542220107, 0.8246335318201088, 0.8280609718936384, 0.5972722694464643, 0.14406357236209405, 0.8012847621103221, 0.6502613629803028, 0.5890652612214954, 0.47785685874153655, 0.3688854603124117, 0.6883648946705624, 0.46801211356127514, 0.9101438439621252, 0.2795348502101016, 0.19152346184282715, 0.4213440881065419, 0.014165086821353454, 0.7070682607766335, 0.849035204602901, 0.20965894945966618, 0.679332148343949, 0.6649484788324889, 0.5415346531897484, 0.5176468011081112, 0.63260620069075, 0.6940130796892751, 0.8637282443283756, 0.974230547694291, 0.9238759972806593]], [[1.447557517871715, 1.1557382899132447, 0.41551950355703543, 1.1588558589358842, 1.1636122062208312, 0.837272817304733, 0.40789182795634027, 0.6293419992995627, 0.9032775508977013, 0.6348330457100322, 0.16264555724408403, 0.7760041914166635, 0.6235578381729188, 0.6116817299127347, 0.45785126900945494, 0.36432147503316425, 0.7101811819442037, 0.483551715947039, 0.9232174837040841, 0.29499330181058586, 0.19544576148280743, 0.42641372587994897, 0.012109684681103077, 0.5941495472995086, 0.9177822694645436, 0.2173276683993165, 0.670845927741239, 0.6951622204517895, 0.5588780139835614, 0.45548964196829295, 0.6666705691377557, 0.7704615864446802, 0.8932986305105541, 0.8877828022622548, 0.9101176932888273]], [[1.5012468775854435, 1.0828927720774306, 0.4216895889427207, 1.2003399019174643, 1.1408096732653912, 0.8274663137237246, 0.35920660177118585, 0.7038925097205662, 0.8888072739828967, 0.6977424874935624, 0.16536690630973028, 0.7589017326284146, 0.4605878503774078, 0.6168784193529893, 0.42933475631504225, 0.3619704043323775, 0.6921659188768315, 0.4800682577074553, 0.8300973551459979, 0.2799342875914714, 0.21492452677047602, 0.400003866883354, 0.022612611546214377, 0.5426848297048661, 0.8393350404173314, 0.22073594143025888, 0.6593537885212124, 0.6401913670098849, 0.548215221899504, 0.4137883045804627, 0.6745134830167212, 0.7523906407363503, 0.8383044899148993, 0.7287839641728482, 1.0038012768886193]], [[1.4497834148006663, 1.0522377719464329, 0.4446075196584468, 1.2253949329539293, 1.0652555677472089, 0.8894747947552686, 0.4086855233494161, 0.7156388966440893, 1.0769178927631216, 0.7233168258959376, 0.15580036299433198, 0.7247351555786019, 0.3223646773517922, 0.645863076922863, 0.5353374050052835, 0.35379777832779924, 0.6737546176215231, 0.45425364165484805, 0.8257052775502614, 0.3141979009162257, 0.20882228801664268, 0.4880836029499669, 0.025934821082504445, 0.5063411327596328, 0.7488188535449178, 0.20337669108337764, 0.6182178744532851, 0.647845574645268, 0.4733710920308841, 0.42791669887095357, 0.7202643612296746, 0.8216223437700889, 0.7646333346237704, 0.6330508738110452, 1.121837465729186]], [[1.4184570880249563, 1.0418398501963162, 0.4744367544891897, 1.2611604888688035, 0.981810522501831, 0.8998929882998552, 0.464709792925389, 0.7697081181758765, 1.159058180426054, 0.6202181279674107, 0.14050264341579155, 0.7578100538809431, 0.3047309367579337, 0.5960612380480386, 0.604595487750567, 0.35727404449345096, 0.654486829973838, 0.4910651065427257, 0.7054048435627357, 0.33706499490033914, 0.19845029965671876, 0.605697044755618, 0.035730672978050194, 0.4179703120923755, 0.7031862768923905, 0.20222268153364648, 0.5699417437528709, 0.6473624951676913, 0.4191604271514766, 0.46257243341422133, 0.7789810287279907, 0.8531117615343272, 0.7110916609605128, 0.6085108344180523, 1.103235763837794]], [[1.3138808446712893, 1.0133117013992792, 0.4514236740069167, 1.228448106625498, 0.6775850587015312, 0.8098784560848592, 0.4244068314828598, 0.7960174491898772, 0.9972954786290384, 0.44839855207117124, 0.16187627429649967, 0.687834020704243, 0.2282274513090872, 0.6475681091057957, 0.6032059966543393, 0.33685751145876075, 0.663723179878591, 0.48520742643120074, 0.6549158338929335, 0.34035474494403806, 0.2121675343469027, 0.6069855614299505, 0.054812109873370214, 0.3148787591043428, 0.7225429719531329, 0.19072538490713714, 0.5230664305143435, 0.5706593418862729, 0.32256103215902465, 0.4583538682736752, 0.8536887077404134, 0.8632486909395208, 0.6113145126090038, 0.5296163404251416, 0.9342818957559313]], [[1.2790081240161564, 1.169019047914897, 0.600166151215825, 1.1833329621403967, 0.38902839306235054, 0.7724016561077129, 0.42190813400098015, 0.8157958327717445, 0.9261743353632101, 0.3907259753385247, 0.14912781209552403, 0.676121938691622, 0.19626840142382235, 0.592426110269102, 0.6679571578372626, 0.3543966363737908, 0.6364883681175556, 0.4990804088735128, 0.6095213920207406, 0.33501539729614155, 0.23587278528997463, 0.6386843135748314, 0.04670516839643893, 0.21129140771285948, 0.7242210767758295, 0.18091305927589846, 0.427081611849538, 0.5405859181131011, 0.2889857359053202, 0.5754517960542628, 0.9224718892585098, 0.9459332506140341, 0.5761896590172468, 0.5078066763684604, 0.884803091069214]], [[1.2130843840153744, 1.1836753428884534, 0.6101283354303196, 1.21303400487521, 0.3023394966395906, 0.7439963520285835, 0.4158025782228014, 0.8075273479977878, 0.8828942945773229, 0.3003029932178015, 0.15561177332800266, 0.6961898636675208, 0.1956380266749813, 0.5831940842418997, 0.6203658467840814, 0.3796336493175916, 0.6102880721777315, 0.47916361432490345, 0.6090360417334897, 0.30847499346047, 0.2776045102591478, 0.597092002831756, 0.04190502848568228, 0.1350270909739607, 0.794867298813076, 0.12212258088151773, 0.36177270542120055, 0.5368570916688021, 0.28785759037234904, 0.6483585990603595, 0.7966408585168141, 0.9346948372798505, 0.5169602558339912, 0.5831188610139053, 0.8666783234588773]], [[1.1622511549923724, 0.9989928904145113, 0.5919205028038464, 1.2443399374682929, 0.28630598644866256, 0.7183654931132053, 0.41603548274207774, 0.7986830054954801, 0.8766946177133816, 0.3059082566383539, 0.16729049030581494, 0.6985060623315757, 0.19956150677467366, 0.5807789988252343, 0.6366911687928332, 0.38422065933823857, 0.5828392973546611, 0.47509459159222195, 0.5442858662727708, 0.3119504799299295, 0.32759670294508497, 0.5678961403393357, 0.041058085236643706, 0.11748859571498405, 0.7902217289066508, 0.10953761202926937, 0.3028323009512932, 0.5771415285226007, 0.3273618710803313, 0.6333247234285386, 0.7313442601409363, 0.9144380129611298, 0.3601094809500358, 0.5365725501962972, 0.8593601922184528]], [[0.8345023768248798, 1.0039088180213958, 0.5077299726735824, 1.2408148394094232, 0.28933011745413006, 0.701716563734569, 0.42052683328557894, 0.7829306181022873, 0.7708911635182645, 0.3122534454480077, 0.17836622563371057, 0.7122803345709894, 0.20277859527324782, 0.6035431766097012, 0.5341223959222963, 0.3547305488306383, 0.5573760294579997, 0.48676216304410974, 0.5639194450581789, 0.30293853054793984, 0.2847918242495557, 0.5634912041596714, 0.06019107722359124, 0.09732915933087621, 0.7443252604871853, 0.12760400569110603, 0.2678867421978387, 0.5947610811438768, 0.3164628710128651, 0.6464036794027319, 0.7140227973012002, 0.8463343183632124, 0.3101510127830357, 0.49886933302780245, 0.7436502026342757]], [[0.8160449221599999, 0.8395985753584414, 0.4022451614717114, 1.2598664666469572, 0.2894598836066721, 0.6851628416276088, 0.4107666073981593, 0.754898846792478, 0.846923514947846, 0.41243824108999755, 0.17807336154085232, 0.6956079863919374, 0.1997158785768844, 0.5989755729029416, 0.531127660350303, 0.4064563160031219, 0.5319296177241162, 0.5229650145325337, 0.6472232833893642, 0.28752457095703643, 0.2721837681673603, 0.5575062557642017, 0.05776545657997019, 0.12960250353881475, 0.6830103038873157, 0.15459624828489105, 0.36602054533889483, 0.6741249539798108, 0.32344822124268285, 0.667746676083057, 0.6938112361653392, 0.849808195627577, 0.28604794575922776, 0.4986851445813682, 0.6324794675765975]], [[0.8651414353048862, 0.8331026247990052, 0.3828317500496907, 1.255774772826425, 0.29103701855815967, 0.6526010709157744, 0.4065474212207786, 0.7947168860488555, 0.8794927187368987, 0.4943986366326631, 0.1791447095157269, 0.6635515181276868, 0.19730751976110505, 0.5833488425721239, 0.36601609204178986, 0.4635219805072387, 0.5305837613244185, 0.5376094040092678, 0.6712171239792808, 0.2482223704039286, 0.27364291403556146, 0.5802386511715855, 0.044681790327559784, 0.23635730538378652, 0.6970438839709757, 0.1831067058074614, 0.5215819279734553, 0.7417177167948266, 0.3275701667862564, 0.5932127453390839, 0.6755079835319211, 0.8431887517104341, 0.3598348353934866, 0.5273664064849893, 0.5514295755824821]], [[0.8594214961642455, 0.8316044359867368, 0.3360172237731516, 1.2300296162806905, 0.3720449089707419, 0.6803518016130745, 0.3880329602096319, 0.7829446505417623, 0.8549574212170826, 0.5152208520778151, 0.26676050455205685, 0.6482774742861755, 0.2042682600507451, 0.5805854723570367, 0.34074497052470254, 0.4538877462144078, 0.5927017310868428, 0.5723753620325597, 0.76518958248374, 0.21898897902845976, 0.23457116782436033, 0.5262758758407586, 0.040144778033587, 0.4285673994431423, 0.6473354002451803, 0.19865584652843066, 0.7703706450548999, 0.7555696972766084, 0.3356280646631725, 0.4750039180797395, 0.6384006894556495, 0.8587535072697333, 0.5099989475312418, 0.6332011874092329, 0.5208795590066606]], [[0.8457864766875628, 0.7852069261917515, 0.3448475665066656, 1.2675373401681034, 0.36625196163853624, 0.6375586716631572, 0.3865957786997017, 0.6946164332722333, 0.7342790011146917, 0.49354409763388724, 0.2881597332421115, 0.6200580959675591, 0.20411781210882585, 0.6038338410220114, 0.36692756929863773, 0.4423606335719682, 0.6449585246169864, 0.6091968009942254, 0.7754286782967317, 0.20440749463567626, 0.2532441979424918, 0.5417521801056653, 0.038464447081122094, 0.5691204577805391, 0.6175221311041473, 0.1674606720633443, 0.9119808901453772, 0.9184864539587019, 0.3150001726348747, 0.39573222279917564, 0.6809115602835192, 0.8786094333219903, 0.7204927935860975, 0.6360803704280613, 0.5823900789085132]], [[0.8223399358065542, 0.7137279751702419, 0.32803585769364074, 1.2789874542126782, 0.52561833165485, 0.626252906841934, 0.38455175286791243, 0.6739285361647392, 0.8334976641455463, 0.465625547523786, 0.23884356946062463, 0.8126184563177621, 0.11234487877720577, 0.5673541955737548, 0.2870305615422508, 0.4287514919841128, 0.5523922144554627, 0.5154768414244352, 0.7014145059492278, 0.17137793616097405, 0.22428504713019837, 0.45456933138147676, 0.03887338803411932, 0.7264278348023258, 0.675138961308678, 0.12125492819941486, 1.031597088756687, 0.9356449775942954, 0.2949534820726226, 0.3509813442859231, 0.7380010041967505, 0.9148083837201066, 0.8448158957830785, 0.6363502673881596, 0.7481020030082965]], [[0.7670584938655365, 0.6974000238519746, 0.31746754440673053, 1.217736206145741, 0.7091427648419762, 0.6531491499104541, 0.3797255217212152, 0.6784123549119636, 0.8194913889355505, 0.42774898088662694, 0.1661480792952598, 0.8235480644417501, 0.10227909764217256, 0.5694014699021069, 0.26383799433345945, 0.425249150997247, 0.5499254349206776, 0.4986072922133622, 0.6071994882683849, 0.16171720404870338, 0.23123446961975605, 0.4181480109278279, 0.0403287553709403, 0.8457707561266852, 0.7237481789704743, 0.11876083387445652, 1.0661605560528655, 0.9601080613140434, 0.25897954895041586, 0.33096572544716923, 0.7420448083941295, 0.9360087507590213, 0.9437651696799985, 0.5893600593735737, 0.934154714040959]], [[0.7214286520891693, 0.7465096211543594, 0.30695650860293694, 1.1448862159599984, 0.8511352459254983, 0.676830738612753, 0.37462522008909305, 0.7013029696118862, 0.7840985025914915, 0.3905308428159209, 0.15705655469928018, 0.7665454014460733, 0.09809112126382193, 0.5824331604777827, 0.24970701191842892, 0.42811727568140245, 0.44696720710228854, 0.4921023838956654, 0.6195144721692372, 0.18099303307555034, 0.23218459367216143, 0.43340323322767116, 0.043330962888539666, 0.9544994313843338, 0.6766560894084661, 0.1674843979285675, 0.950804881208385, 0.8794983879545943, 0.242942309556778, 0.34166026130087107, 0.7260896411509881, 0.944371274988607, 1.04426706842684, 0.5408607105677252, 0.9441496575622088]], [[0.7044903117321007, 0.7375957701875957, 0.26957917116547525, 1.1013509657521556, 0.8965079293938143, 0.6979188253369839, 0.3873580160133838, 0.7577481600303346, 0.7009979069356754, 0.3743123553404727, 0.15250327123009155, 0.7350677531364143, 0.0816668788680146, 0.5906013671770847, 0.2165881831215166, 0.42422537205933386, 0.37829433406517154, 0.5026945869729406, 0.6247281979603996, 0.1865629110870234, 0.23592103797965636, 0.420916232747858, 0.0458166665806841, 0.935206960589673, 0.6148054928655573, 0.34302386916524824, 0.8824676750626346, 0.7887770002362192, 0.25544078242873924, 0.42322262628439733, 0.680726760864536, 1.007210945794904, 1.0768836947704106, 0.47066318686207487, 0.9388371651499524]], [[0.6707843590470761, 0.591298221507835, 0.2609889713740313, 1.1081933565704232, 0.9749046630281359, 0.698818223829667, 0.3845042607237523, 0.7493413173264647, 0.6560333635926429, 0.36059140159033265, 0.13862480993977977, 0.7288261394364828, 0.08388366686579182, 0.6478353199859845, 0.21349074897632134, 0.45182863151160546, 0.35611605829116977, 0.43366329239364065, 0.5500402925245061, 0.1871745122591836, 0.23442699052720295, 0.39218599093617773, 0.04745090766430839, 0.9017298594883627, 0.4916819768926217, 0.4549993677397637, 0.8448277685901884, 0.7731015572970585, 0.24198411978263662, 0.49424575634111534, 0.6032188927165941, 0.9708717981940298, 1.0948154061259352, 0.425940887996629, 0.9072454004550199]], [[0.6454829675752682, 0.6066763378886518, 0.3316199548597726, 1.1617788902499842, 1.034133983223499, 0.6987302807236921, 0.38849561357406875, 0.6859940808993269, 0.6295158726834553, 0.41330598813742014, 0.10757622345495119, 0.6995091949436523, 0.07964194344300671, 0.6683039079207849, 0.16634698435360754, 0.3941666335336581, 0.3846751521574743, 0.509850100806985, 0.5395798078133568, 0.18880705058390362, 0.2201239167732328, 0.33135415635880605, 0.04872511378987475, 0.9073569906810919, 0.40549609616894855, 0.43990321789126596, 0.8096342197654776, 0.810816151048743, 0.2557314957812524, 0.55447759153802, 0.5517272858862974, 0.9075927558071448, 1.124467073085663, 0.45377291688455546, 0.8881887514939456]], [[0.6052838377928367, 0.5867612842749428, 0.3318315785675965, 1.2812681720465855, 1.087094746256056, 0.7009734311145299, 0.3946299016075152, 0.697078144796417, 0.5777951471420488, 0.4522717546949494, 0.10022124729658227, 0.7294002421383812, 0.09926591562975984, 0.642609392263866, 0.15459084446909563, 0.3612614147282203, 0.33984018695715834, 0.49546733055746817, 0.5476393787730596, 0.1830308961467565, 0.21221641444927508, 0.2971070113840695, 0.04981396755430698, 0.9213819782046841, 0.4658656835599719, 0.3590591310605694, 0.7721675961605254, 0.7782823251739062, 0.2360197296742771, 0.6387430532738526, 0.5355830591231202, 0.8512835579297631, 1.1007534259584797, 0.3915774743592265, 0.8410884861743341]], [[0.5799422175396778, 0.5459892249155249, 0.3089750357281664, 1.2741042822806263, 0.7188663638879589, 0.706239790328782, 0.39467073060095825, 0.7550725307185648, 0.5519866090182806, 0.3556871507090783, 0.08063645497333628, 0.7605199390500477, 0.11055156354618291, 0.5979392323214198, 0.1611359535887473, 0.4170874948484177, 0.40678929584069345, 0.5256681912799148, 0.5293721212867486, 0.18210061188986698, 0.227082084263199, 0.30175064502889454, 0.04917885438744351, 0.8382568416322713, 0.5008117029271432, 0.29475525734968333, 0.713683737113195, 0.8256769685885332, 0.2549471781800773, 0.7144399704095807, 0.5075020367878764, 0.8238994836815834, 1.2132441407226455, 0.28209242037266846, 0.8122055826931539]], [[0.5214187828411263, 0.5208670517503824, 0.3415602874128397, 1.3127905900172174, 0.7699126022960231, 0.769673741356678, 0.39263916737774995, 0.7714969102149416, 0.4969110518405658, 0.32821732781983215, 0.08565193236407057, 0.7906870557541354, 0.13238096042643557, 0.6146625235522135, 0.13635798304792743, 0.4258585183647011, 0.38755536222936915, 0.5141156597458478, 0.5236436318340562, 0.23848466790992895, 0.2898840111848124, 0.33590078567857623, 0.04353935979978037, 0.8895156273559697, 0.5378401089488949, 0.2822063768417307, 0.6688235426912609, 0.7990814978118848, 0.30237235150829367, 0.8809659770062193, 0.5644768989145386, 0.9051675607007601, 1.2610485999680496, 0.24440373303426105, 0.7339689029626897]], [[0.45357596534178113, 0.5460986063128015, 0.352959624518492, 1.3897485360490736, 1.0412227971984447, 0.7040412218906678, 0.3821892820307412, 1.049987634800152, 0.4671546348595407, 0.34151043448783514, 0.05833456497475255, 0.7730363441335433, 0.11004338030965866, 0.5968212489821609, 0.16397639493287758, 0.4324642398655152, 0.4619666589429213, 0.5453743267476615, 0.5410131467667538, 0.24547177635518447, 0.35669881952776733, 0.3404574545037767, 0.04587471008983299, 0.9147589255932229, 0.6870653322004097, 0.24258405139254896, 0.6823502766179624, 0.7626591066755579, 0.4080623935661709, 0.902729642446423, 0.6258388971332233, 0.8929549997895028, 1.3537060876791869, 0.2569756283788449, 0.7356165302068172]], [[0.47130487128009935, 0.5757709836862605, 0.3891812989425293, 1.502642149127422, 1.3535586319057686, 0.7033152865973809, 0.3805331067870426, 1.2308625506016964, 0.4299815707164813, 0.4061279861565139, 0.069044906361587, 0.7028121802829999, 0.06871939371860464, 0.6109576497404077, 0.15835221960229084, 0.43850990479947916, 0.38657122710173153, 0.5897847683142832, 0.5385463988274704, 0.28514395955978566, 0.36524084253488454, 0.331117365364235, 0.04552437851200497, 0.9114695119337147, 0.8235585860701644, 0.22289626275255817, 0.6932095001780793, 0.7178430196617802, 0.4786633097851041, 0.878126319035482, 0.6702026878538139, 0.8835000495748024, 1.3684609795413163, 0.3065589500918664, 0.780293221658769]], [[0.4072996345938881, 0.5206976309949527, 0.3908648054884375, 1.5166225174566001, 1.5318300455561165, 0.6957729660842563, 0.3797364938813037, 1.2327552748433117, 0.4537786091132035, 0.3651904915584472, 0.06060897717540041, 0.7295078104982337, 0.11938034850217213, 0.6260766300190697, 0.1276364629970055, 0.4757761024351189, 0.4299398652841397, 0.6179917946662323, 0.5963132617753112, 0.28636122352684534, 0.3382030250107143, 0.31041323931259757, 0.047583363884663596, 0.8539154317200686, 0.8463820840822069, 0.2214886974963929, 0.7111852296996842, 0.6775975293106503, 0.5465010647059763, 0.83248868131612, 0.6339494416575222, 0.8796353135223842, 1.3629393390023916, 0.29296584930531655, 0.8094624568111957]], [[0.29624407254259805, 0.5285174088478096, 0.3903932053787944, 1.4421038544420568, 1.6390247921211167, 0.6845087304006858, 0.37482552627959864, 1.3096710556714715, 0.41469423182123416, 0.3330868645555138, 0.059609446174690474, 0.7368628570647581, 0.14548125480590368, 0.6294745158442554, 0.10483448997194877, 0.42183409292606144, 0.4258366957313612, 0.6397428886481322, 0.6028661858136611, 0.28149045396261363, 0.36529562512598407, 0.2883897646913494, 0.05717958727011978, 0.7909546593622145, 0.8780842814631858, 0.23179709271284715, 0.6805309974097326, 0.6391399155992638, 0.5383972131618755, 0.8349416172671196, 0.6094693540036236, 0.893907445824356, 1.393873262644083, 0.22664163662577627, 0.8185360362537006]], [[0.2565834433481913, 0.5212596759901871, 0.394656789343808, 1.300512371045322, 1.6653447127508505, 0.6727013075551875, 0.3690602743402203, 1.4074481184795762, 0.4387437206324256, 0.3458699113936324, 0.05472688391203653, 0.7758340488071074, 0.21636168811636597, 0.6715275139773402, 0.10554382184978614, 0.4410922077070627, 0.48228897029692175, 0.6243652789962783, 0.6281026541756557, 0.2883079891983428, 0.35122559966424033, 0.26834616770552666, 0.06729137854313556, 0.7306799049266588, 1.0067956855708569, 0.1964854300757934, 0.6746722820353266, 0.5870742291392558, 0.5785040354927349, 0.9217689635228599, 0.5088043261557849, 0.8514915225507451, 1.3391564123989705, 0.18410701531123444, 0.8888315229026279]], [[0.22558567394291404, 0.4871561157950078, 0.3959762226197612, 1.2866132012935525, 1.7819194870224335, 0.6571820536350046, 0.3628356061587353, 1.415543026473504, 0.4281478668537644, 0.390734231258909, 0.03838155059006033, 0.8018485058370027, 0.2945231975994498, 0.6922742519765217, 0.1155369837741729, 0.4927161982576308, 0.46336205492951854, 0.6339290746605701, 0.6775693498173132, 0.224891007674762, 0.4156907559177557, 0.27514600281869384, 0.0625013823284449, 0.6568639063869312, 1.149548222858952, 0.19118828748516292, 0.7139643891806838, 0.5587295490475126, 0.5703070107098595, 1.0868144669077255, 0.46793830785457624, 0.8699385975414793, 1.2345198990579442, 0.1913315670129252, 0.9756966104403609]], [[0.20639846329379657, 0.4942227716119167, 0.40020855580289755, 1.1175044078495238, 1.856192685087381, 0.654399035319179, 0.3691751329052042, 1.4531847128830568, 0.3897592203233226, 0.3667391657911361, 0.033861681154332066, 0.8335300283151481, 0.27989302237899266, 0.674847350303589, 0.10154837443082837, 0.4795805818677161, 0.3937196445145206, 0.6006324215251996, 0.7381800347558487, 0.22555545029201296, 0.40138202944744106, 0.27613089417918474, 0.07144583595057041, 0.6049345221838706, 1.2388444389151916, 0.18092043348546602, 0.682423108938391, 0.530180161642256, 0.5669838542061287, 1.238302758241924, 0.4203213257691798, 0.8637121524893694, 1.1674060403154334, 0.2103923047115151, 1.1531738384646941]], [[0.24719169138041847, 0.5435149991281256, 0.3693199034232828, 1.0572750055015485, 1.860409697869614, 0.6841724877826796, 0.3877463376446355, 1.4602801549802131, 0.37184854629425157, 0.31152077510729204, 0.023308777066055286, 0.8937685281131156, 0.306963201296365, 0.6894679935189937, 0.10117923419027319, 0.49686415772408077, 0.3948284693650304, 0.5535931377125693, 0.7382444482925666, 0.25689861185924745, 0.3958453539616807, 0.24762280214834037, 0.08122261156555492, 0.6031336257580093, 1.2716046432427617, 0.18370094373352544, 0.6176044685172473, 0.5070367937077158, 0.6014055392410491, 1.3291554508738335, 0.3741662224185255, 0.8792542804556669, 1.0726278678463037, 0.230985723474013, 1.3436793207440494]], [[0.30715509713765654, 0.6084213637849782, 0.356158815133715, 1.209981404988549, 1.8227944943923315, 0.6935444430673467, 0.4100884886175979, 1.4416479521041676, 0.39410842415407127, 0.27743038652208596, 0.04322614697941288, 0.8319182381922718, 0.34258874863904565, 0.7074313738866741, 0.11722434016763839, 0.5088609811757897, 0.38578351947930406, 0.5509436577144682, 0.65204463333824, 0.4073885153182336, 0.4565380360650066, 0.2222007810132154, 0.10123542252445258, 0.6868662549309945, 1.2559553403427903, 0.22946315217970065, 0.4808710398124686, 0.6591215770367833, 0.6605817470216104, 1.5045358635247914, 0.33928488818031544, 0.8461187284064646, 0.8955436876080378, 0.24593052115348868, 1.524078989454561]], [[0.338701273806565, 0.6745621435712923, 0.34572069995170707, 1.053563441873899, 1.8053156005161752, 0.7041958013039735, 0.41640748242665987, 1.4479014371013355, 0.4457306928360657, 0.27317481555716683, 0.060963461638032575, 0.8174983254772729, 0.3442208892033405, 0.667294144151287, 0.10304068720775664, 0.5232160196148018, 0.320432436532467, 0.4969988412943849, 0.6236351674404816, 0.4029692451242467, 0.46687471603312425, 0.20675450923342686, 0.09620677515332184, 0.63315188525092, 1.1878361781750704, 0.298691955615743, 0.40083789912811374, 0.6484200263894876, 0.6314625151385862, 1.4972477499352004, 0.3234423257108357, 0.8470206594723806, 0.7898849123899259, 0.18013675021034092, 1.4604069923378655]], [[0.37756411566062176, 0.7754382360400522, 0.35600339517870455, 0.8672764474001384, 1.7423404681274004, 0.7091974305022111, 0.40242813724552395, 1.2524909972676865, 0.5006459759093498, 0.3231474370161583, 0.06772609068424995, 0.8587397565163823, 0.3733639384234171, 0.6840157960085768, 0.11283249344642654, 0.6224829554073633, 0.32326317331116994, 0.4801391805743695, 0.5376785055473225, 0.3834110301369208, 0.4301443645803267, 0.2749129408872143, 0.10846832198711233, 0.6194179626971976, 1.3097176016786258, 0.4294520466438603, 0.42650278551043846, 0.6401910098491028, 0.7577818817990745, 1.49741599372257, 0.31267546575549, 0.8084877963923955, 0.7280470366993251, 0.16811972450829155, 1.3374647783106912]], [[0.4790796303650625, 0.8165354240953184, 0.3486408469791078, 0.8281259479661328, 1.6331009996571422, 0.7684400597160136, 0.4721693672569624, 1.026132864604517, 0.554897054777365, 0.38988963759833983, 0.02992046909120794, 0.7984164221153327, 0.40140762775644767, 0.795624176660464, 0.11342043388874823, 0.5734228231181457, 0.332307933871666, 0.44824753265363604, 0.4813292940586706, 0.41080162658262365, 0.3858193176123162, 0.3377702701769636, 0.12076738195365039, 0.5740302853605099, 1.5182127693260576, 0.5310091440762019, 0.5508223082665707, 0.7564535808228485, 0.8313907117042828, 1.4447577192533285, 0.3890237555525522, 0.740554004294648, 0.5686006209069073, 0.21242042934974464, 1.171305587581917]], [[0.4985591861409572, 0.9010304972347245, 0.3256793400390732, 0.7964815843984346, 1.6234081058724907, 0.6512617330560371, 0.3731886150706852, 1.1261544707502957, 0.661943898127447, 0.3899868856242332, 0.02915851215733124, 0.730134388794739, 0.45552174652063315, 0.78931875316522, 0.11387430084530831, 0.5538164388897425, 0.28720714723331614, 0.424933859675997, 0.46377767433957684, 0.42611340942564435, 0.30507455878168277, 0.37097620303887874, 0.10085727905911719, 0.4824242841061553, 1.7019861603892545, 0.711686413275278, 0.6829507843815017, 0.8981717425047993, 0.926567999812381, 1.4895764879910343, 0.47912060007905544, 0.6929814433820827, 0.43847573153941644, 0.26419576741157036, 1.0529391642827042]], [[0.568796902055881, 0.9492124098021105, 0.3260738693859436, 0.8278132627415268, 1.726597124879052, 0.6635405075888711, 0.3720254457008001, 1.1631977289363808, 0.7665846115406789, 0.36932253018159605, 0.019963424322757044, 0.6978725785319749, 0.4966884522613735, 0.8669177073628803, 0.12569448804109684, 0.4946017502061534, 0.2951128377812442, 0.4773938466965863, 0.474391060140822, 0.32550147822808306, 0.26358357309932495, 0.4438256124113168, 0.13947847212723585, 0.32659335643011067, 1.7470807271315776, 0.7980811779328877, 0.709135154521975, 0.9387162680022727, 1.026947121812925, 1.3485831406333655, 0.5502017773532484, 0.6659902927216387, 0.4641941198527915, 0.33618635629672555, 0.8334489885659087]], [[0.6327740801028094, 0.8950360736224563, 0.32150722540325083, 0.7760715296719013, 1.6430411334247523, 0.6334456137942823, 0.35979468499915435, 1.1917348819316662, 0.8890655587985833, 0.35614392423614566, 0.027323483460006244, 0.6869434700863624, 0.5735884834953953, 0.8420480053244637, 0.18507967144039122, 0.49428455894303214, 0.31034571867155125, 0.48018888660169534, 0.6211235220898512, 0.25691380975132516, 0.24963849367168836, 0.267170373098019, 0.22305449661008753, 0.23847316365124255, 1.7643518357575365, 0.7895436478868296, 0.7055002148462897, 0.9733554789348257, 1.0970814975468595, 1.2408788187533346, 0.6123849124102916, 0.5893366331754339, 0.7086937990204677, 0.3655455903387935, 0.7672276460986998]], [[0.6650244834446893, 0.955962411714856, 0.3197758358816549, 0.8757071835278307, 1.4749132917859185, 0.611723105042849, 0.2983967988301032, 1.2266442813027372, 0.9200300090273114, 0.357142279624139, 0.02515319197665592, 0.6989509784945096, 0.6868896813748568, 0.8588589558805275, 0.18122226455724166, 0.4940934000694758, 0.31656139274545103, 0.4778776129853768, 0.6092051365463029, 0.20698320906375367, 0.20128061936225716, 0.23826154709491476, 0.14546583827986664, 0.1628935243895578, 1.7312529216309234, 0.6488499688595593, 0.7266224516323582, 1.0233784484100898, 1.0846675084772788, 1.1629854408404456, 0.6241461641724205, 0.47878009738652894, 0.6796722979993122, 0.34426411569558407, 0.5994790078998211]], [[0.9419296822100583, 0.9703915241413672, 0.29730465570660575, 0.9251565853297513, 1.298214769559129, 0.6596907687792626, 0.3247461105579237, 0.8543009558734012, 1.0077875517452353, 0.3261943094428126, 0.03567364745026231, 0.7836224763856161, 0.7696857417764766, 0.8518874586688046, 0.2769176543449926, 0.45534424773016047, 0.5133770694345611, 0.5268284375326198, 0.6444493281543314, 0.1687976691407319, 0.21939935287823165, 0.24925975900216446, 0.041152623592302, 0.136696240011906, 1.715647079831016, 0.560742989104062, 0.7200208183475041, 1.1816580146166749, 1.0222291781178945, 1.1070768322067164, 0.5892107219168026, 0.42774648822478667, 0.7014941716195624, 0.3692139894750322, 0.5937783166219489]], [[1.0315493050519595, 1.0096739684028668, 0.2977013238688709, 0.9824039725691498, 1.2145117286759106, 0.6527231392506526, 0.31998771388654945, 0.8781837033778409, 1.0857002237974134, 0.30951414933910093, 0.06664285328127476, 0.7773116854754045, 0.8041319577906969, 0.8540003918309462, 0.29573400280693807, 0.4351491695487175, 0.5081151542070906, 0.5910828687812688, 0.7751248861346318, 0.17681910146862198, 0.24470917545288717, 0.3446723968172357, 0.04334781301060002, 0.22352821146169174, 1.5644384875374593, 0.3535253450593654, 0.7412766373049738, 1.1573800542827493, 1.0342894419432342, 1.151163761623643, 0.566774117092472, 0.4342559599244791, 0.7360921920749878, 0.4511537928806438, 0.6299144818195357]], [[1.089525718604924, 1.0097516335221264, 0.2899973874288438, 1.0516599625230798, 1.149845299622561, 0.6656671729357041, 0.34901579495427504, 0.8673420337830149, 1.1233085701802143, 0.2478220637391612, 0.07442877719716207, 0.8545231119331682, 0.8160927690090191, 1.0166875632343233, 0.27416571000911166, 0.4337435144107077, 0.4273382749397666, 0.6143203528211095, 0.7816979671339567, 0.1755080266835517, 0.2515696078676939, 0.3294471609262383, 0.04259603390647548, 0.35293202078889696, 1.457772155287283, 0.25359102388157273, 0.7645040835007898, 1.121764855254183, 1.066856848804206, 1.1828193333996393, 0.5375243024879574, 0.5454747168054157, 0.6524186165072395, 0.42545327614446576, 0.6729975504383592]], [[1.146155165744012, 0.9726368320226221, 0.29040390245908515, 1.2090978972936555, 1.1313803390553034, 0.6596411613813855, 0.3284175221895858, 0.8739205085552045, 1.1729237986086514, 0.24546293082385798, 0.018169077157418195, 0.8584188151677032, 0.8185248946815592, 0.8848769779109461, 0.36961879641262596, 0.43274386151840927, 0.42374365750154497, 0.6478910352812994, 0.9061824226404032, 0.17576486452365897, 0.22766109122431993, 0.3948347686405743, 0.04178699186557097, 0.4886073064047139, 1.425019866294572, 0.2593043726169916, 0.8052536851932857, 1.0914298768765727, 1.096685120235198, 1.2630305668889568, 0.4533913297268901, 0.632945021288136, 0.5608454925933282, 0.4070885538589928, 0.7044701456846348]], [[1.2918701062988511, 0.9073766485583046, 0.30190467603562293, 1.407237037948126, 0.9972475033907982, 0.6741860614362803, 0.3637922533212788, 0.7900514049473082, 1.173982559532762, 0.2010925150860059, 0.036228543173422646, 0.9441927302214922, 0.7957686787649417, 0.9015507504233443, 0.5091569049705652, 0.4581116136698845, 0.4216648992036325, 0.7040974236357102, 0.7667670472703924, 0.19208065226548987, 0.21723709019278659, 0.29918056477936117, 0.06945424878436311, 0.6136566410463675, 1.2451200174344548, 0.27267538572260686, 0.7869054578638199, 0.934668555140514, 1.1452145929176296, 1.2794406460452863, 0.4085382324477107, 0.7019491124944931, 0.46716003026654696, 0.39219718015152366, 0.748250048059385]], [[1.3046138042481679, 0.820556849844227, 0.28248041733504653, 1.5475456505753924, 1.0490565768461166, 0.7227153088998395, 0.3692803602187894, 0.6162517381375431, 1.2700679221981077, 0.17540712820906562, 0.04772249723040522, 0.6369668537145696, 0.9140935330464279, 1.1111920479856776, 0.33646987012898405, 0.475096602080501, 0.6221779820153792, 0.6734222962795987, 0.8480105575013059, 0.2616383101054501, 0.2611627217567223, 0.23761254187669928, 0.05316480355881799, 0.6176558630148551, 1.2208884605083345, 0.27349164621891187, 0.7513032411741271, 0.925273408362186, 1.1692798195166982, 1.330288306162121, 0.40819130948354587, 0.7344472918538707, 0.46351712891714403, 0.39231833371623304, 0.7664132365203006]], [[1.3194508706414525, 0.7635840699832549, 0.2821663897410959, 1.3848666410709813, 1.0702144650647307, 0.7210571824967863, 0.3682686210002323, 0.43832965297935644, 1.2922253959901777, 0.3178271902254406, 0.04586832612532338, 0.6138949594435359, 0.9644007810699924, 0.9532238077892012, 0.32562532429885443, 0.45468767068483784, 0.4808881518634889, 0.7147082580170928, 0.7839778936179636, 0.34342685466221473, 0.2845376720963736, 0.18694283208420648, 0.05197130223763242, 0.4101173372442898, 1.226374771618529, 0.24417882272753694, 0.8402882971549911, 0.8754948345027054, 1.1903192802457039, 1.3194123305317813, 0.41374555387153616, 0.7380953653549347, 0.48898884759768924, 0.3652969421719409, 0.7935012016993699]], [[1.3219020845535927, 0.7101834728229198, 0.28655048018781576, 1.1791871619835308, 1.1358349969591015, 0.735038027198224, 0.376739915091854, 0.40843951790953686, 1.240584923142432, 0.3735401178267919, 0.024558004138363884, 0.5994454451511106, 0.9703455514191885, 0.888482010820688, 0.3298188552324602, 0.4448686651298337, 0.49146847400040683, 0.725840215335215, 0.7895377391685943, 0.4274047740502501, 0.2966103642846925, 0.23501623314540776, 0.16667262548781722, 0.20353752828325689, 1.1939651375329343, 0.3099633629738314, 0.9530514974684927, 0.8050880131934922, 1.1608733132228308, 1.3123616945284702, 0.4601480872109548, 0.7146410405915633, 0.5612342962990609, 0.4277953324156066, 0.7675895000171356]], [[1.3128009680420072, 0.6791521065536867, 0.2839914369187993, 1.1071897375280928, 1.159007884492127, 0.6828171288623264, 0.4186355507167184, 0.4558225622766704, 1.1776176205378395, 0.47594637109645876, 0.019095556641375227, 0.559735260014773, 0.9836925892453441, 0.8359008396640859, 0.4010185844520247, 0.462150245785399, 0.5436091070862558, 0.7084948163341305, 0.8062425234864736, 0.59724026126631, 0.45383056110097375, 0.34760737663597246, 0.31741657001655377, 0.29978275952302214, 1.184488960352187, 0.41133518536141583, 0.9766294514992615, 0.7973969942411329, 1.123921957842416, 1.3119037583949407, 0.606993660620206, 0.7068357417793182, 0.6217730265044725, 0.5546548704927511, 0.7316816441317897]], [[1.4509538262815105, 0.6934931868472571, 0.27957836709218536, 1.1128583559075245, 1.0482705433729362, 0.687900105799141, 0.39991382506816603, 0.5409787142728587, 1.1170719789366046, 0.6181374722913071, 0.018727585599336936, 0.5892186699023332, 0.9791840662722118, 0.8137393486143667, 0.3523953311001113, 0.44157262416171345, 0.5271630341994371, 0.7303122959339873, 0.8457913989752098, 0.5794954458834097, 0.5705014977549684, 0.3515988715361148, 0.19047217695270158, 0.3990462998805928, 0.8787837313750548, 0.42640948428293074, 1.0023969444925316, 0.7418046862473506, 0.9762226484449379, 1.2989543782576591, 0.7629823963306452, 0.6805503674684825, 0.6477959694255006, 0.5649176593773158, 0.6980124822913126]], [[1.4512993387497648, 0.6187002203640308, 0.2827518406504547, 1.13910481614928, 1.105928848381135, 0.6598140456734651, 0.41669264980214027, 0.5177274026548262, 1.0857249095276043, 0.7201205975387642, 0.011537229468523808, 0.7204568977411463, 0.9775003043996725, 0.8128654342497744, 0.36875174262806426, 0.4195346000612083, 0.573283513132053, 0.766871071894042, 0.8851478087373245, 0.5548478479767229, 0.6177589498787768, 0.4495425675245278, 0.15381651273679606, 0.489065405790997, 0.7007386642084976, 0.40149240874852477, 0.9873767474151467, 0.6186149173385344, 0.7665808254853669, 1.3122554622925282, 0.863869258891306, 0.5936009652646622, 0.6318760803366754, 0.5356907571102724, 0.5867464946841356]], [[1.4954083989164593, 0.6527220779747533, 0.2847958286605206, 1.2324258622934823, 0.9206404522613049, 0.6677372823908265, 0.40628412328450314, 0.5220756940798817, 1.0680275292879051, 0.8064341349461492, 0.024105070578236833, 0.7382709938340282, 0.9856596255819605, 0.8185559600627625, 0.37557844254157835, 0.4092248179456652, 0.6469521499266983, 0.7497196834697608, 0.9538982851664879, 0.5336026920473589, 0.6192505194232211, 0.5810085342819349, 0.15654036773386315, 0.5400652387497736, 0.6053917775898173, 0.42141625840643127, 0.9821390816708342, 0.5500194169760108, 0.6879625197579031, 1.3248356320985364, 0.9553403524974875, 0.4671066900886327, 0.6390837354415472, 0.41857726658779204, 0.5158078406042079]], [[1.5303395691495947, 0.627891572834146, 0.27652511033579913, 1.3085681213249254, 0.8286212149790901, 0.680519602859341, 0.40018667450411144, 0.5121878747123327, 1.051354749852906, 0.8722118072121429, 0.020284091619521476, 0.7561479166821057, 0.9670731517992759, 0.6275809843025868, 0.24655506001729474, 0.3839545371196388, 0.4299887588561907, 0.6659275867087995, 0.9540160291518334, 0.330637886318103, 0.4951567148611005, 0.6039173768138378, 0.10992493679212803, 0.6027220467233334, 0.625393325260222, 0.4199321963719112, 1.0252001730282068, 0.5146431943423412, 0.6438728941278985, 1.3284902950251063, 0.9830017167177374, 0.37491646771200804, 0.6662059856434595, 0.32584824778448707, 0.5069400014497394]], [[1.5739956592153752, 0.6372359455614017, 0.2620541617097601, 1.3180867067148812, 0.7818353782689593, 0.6901154662576857, 0.4013788087556597, 0.3729438644716806, 1.0341202895517987, 0.776198128732201, 0.008725975924071165, 0.7127630662236475, 0.8977666223301342, 0.6316045649741817, 0.2277302343248933, 0.35898085918784534, 0.4422511599757023, 0.636203760531879, 1.0293996929917753, 0.3031913300873416, 0.418263491313404, 0.5249172764532295, 0.11247454170308446, 0.6252729143325855, 0.6660876375209187, 0.3789778501182864, 0.7907449456527655, 0.5211704772004824, 0.6500018878742071, 1.2909836762838132, 0.9549555618231891, 0.4742493279175848, 0.6958181726606576, 0.2934609315475561, 0.540193526617514]], [[1.643945852472469, 0.6430753968530851, 0.27785124496127894, 1.291478178295298, 0.7147456104063725, 0.6982497006668014, 0.40755841154986183, 0.3893901851522238, 1.0078209591812715, 0.772522516302637, 0.010010183477002615, 0.7081237929223176, 0.8299301516272526, 0.5598538372387174, 0.22769524444797548, 0.37916751190023057, 0.40713406490880655, 0.732432247564163, 1.0049081895027787, 0.29706928320569626, 0.3979099120591638, 0.49574345171090317, 0.11196390791598894, 0.5848688799518718, 0.7439905659412214, 0.3761321501868928, 0.7606080441803156, 0.5205486818185507, 0.6654818533527718, 1.3139287372121302, 0.9526043427647722, 0.6266423559921799, 0.7953246431471109, 0.3155610044428445, 0.559946421535874]], [[1.7199850003846149, 0.636786719691492, 0.26022572306884867, 1.2726610350531282, 0.705944860719022, 0.6839119023713438, 0.40670917263370265, 0.4064439471357258, 0.7784419677643827, 0.7402957027663535, 0.01029770580487238, 0.646204792444983, 0.7933522829309252, 0.5405537552684261, 0.2252134850399981, 0.35675824550360236, 0.4553643733410695, 0.7163361314013711, 1.0675386676243397, 0.348419977397749, 0.3901668201379996, 0.5224579580370874, 0.11139470931358741, 0.5431923394111204, 0.7635338914294427, 0.3551462425788471, 0.4496487029189732, 0.5314995208177242, 0.6691793520575614, 1.2908759062229747, 0.8943704454758225, 0.7612080866611775, 0.7214927866474716, 0.30859577106567543, 0.5777644448235404]], [[1.7122234890969947, 0.614915819863572, 0.26985541896230775, 1.2410189531865834, 0.741170032203749, 0.6848281157846211, 0.4157390922972144, 0.40512599740091415, 0.7761450717063451, 0.783225148293247, 0.013547864084931114, 0.6571808284224231, 0.6816000065926087, 0.5414145170940962, 0.19723206058217718, 0.3489748351148425, 0.3607003921698072, 0.7403354932309261, 0.9915805594854075, 0.3912267916214893, 0.49526274773930123, 0.5629166748826975, 0.11075287140968207, 0.500445951380192, 0.6462250086295573, 0.3720650133703214, 0.3411550001111562, 0.5249679472224769, 0.6742658293935184, 1.213382986215521, 0.907755889887083, 0.9592134148010147, 0.8164423184306526, 0.30810870771035964, 0.674193749255622]], [[1.7592914709337757, 0.6142505420353608, 0.24948820739591143, 1.2250292658005955, 0.7853755174186076, 0.6848990449344178, 0.4154631083963408, 0.41166325056621844, 0.5789930863585404, 0.7199218665911161, 0.006025697259878809, 0.6605161516857789, 0.6400335917693827, 0.5199146505508826, 0.20166730919350068, 0.3065040324700937, 0.34695400355295264, 0.7573148483912948, 1.044825318946383, 0.4674355925112561, 0.5260678065321331, 0.5693032516548764, 0.11056375704276142, 0.47980788280687325, 0.7135603111261564, 0.3632313406974658, 0.3333191113156199, 0.5380263988676448, 0.6749464297715831, 1.1287405912262862, 0.9688015150352598, 1.018020729575913, 0.8652726291594235, 0.36473553929387026, 0.7561404901398265]], [[1.7938329376614894, 0.6534156355171556, 0.2377738662439439, 1.1863364859436354, 0.7173903504567075, 0.6738824732723938, 0.39331673737371364, 0.45199034942383065, 0.4938027773371004, 0.7143673378072467, 0.02996255607320101, 0.7334640812830461, 0.6323524732236954, 0.4893052107386133, 0.19077135716990182, 0.30601103981492395, 0.35255295561105704, 0.7774674305931966, 1.018320586534829, 0.5050486466688078, 0.514494545205944, 0.6369906392789346, 0.08902776803856488, 0.47482398662388714, 0.7659336926584344, 0.3421729987747227, 0.3866910343142843, 0.5383745401059692, 0.6539613632296271, 0.9962549586617377, 1.1592849829000658, 1.009715281758539, 0.8646935394006714, 0.40275493330491013, 0.7402044687621163]], [[1.5721124072797452, 0.6497840180808094, 0.2332127099664329, 1.1077107681771523, 0.8121119055373675, 0.7035869015241103, 0.39673983235931765, 0.488677007291089, 0.4753931114193404, 0.7429573131398255, 0.030470146514452408, 0.71016680263615, 0.6256930973892856, 0.48394668089242826, 0.19175901413831983, 0.2902191231665042, 0.3668186977529757, 0.8060565508059578, 0.9984025053434987, 0.5017021848532317, 0.5061655000955193, 0.7148177415791059, 0.07077442611481452, 0.4409692912133359, 0.9186244746722827, 0.3780410634842779, 0.46620970502686476, 0.5452135542732492, 0.665302263257228, 0.8695696115636871, 1.31857953344411, 0.9618600931187233, 0.815319113065959, 0.4458299800661574, 0.7988108298238057]], [[1.4213179947446943, 0.6604394752516474, 0.24436294505194872, 1.0480414017185953, 0.8125984451388956, 0.6850146212014986, 0.3988280100284416, 0.5259615048126196, 0.43051623544113593, 0.7013697685605152, 0.029999345968768448, 0.6627915034772911, 0.568799357449654, 0.48886362993479004, 0.2119570653628618, 0.2961059688565081, 0.4126041726016053, 0.9068657168259364, 0.9926068678818163, 0.5392800082258256, 0.5013437605240021, 0.767669625557566, 0.060556775801825384, 0.411123385864862, 0.9761420426236824, 0.4229836896394565, 0.4422593426421338, 0.5920595602330355, 0.6695180259980426, 0.878379586768558, 1.27747609554672, 0.8361407940140586, 0.8258367297947935, 0.47715476483154673, 0.8429752494774514]], [[1.213301258319005, 0.6781553697593161, 0.26296507686281495, 1.0659965708941646, 0.8415141056479429, 0.6685832714508224, 0.40007823370656337, 0.5460679411130558, 0.42536079409658145, 0.6658302972273076, 0.031392518288737375, 0.6548572468836915, 0.5405372846117645, 0.48014872725256874, 0.20188444464558164, 0.3066407295515067, 0.4387017697583543, 0.9066891545294065, 1.0656124932344404, 0.510358473254511, 0.4605724756459165, 0.7633102059625264, 0.05702766321034274, 0.39521506929616795, 1.0809838853871816, 0.39466436550864253, 0.43303573302657883, 0.6455906796030715, 0.6705825730286867, 0.9473153169440984, 1.333114942670634, 0.7890431970720286, 0.8824186116253169, 0.5450931940236019, 0.8219365887687318]], [[1.0376360266063736, 0.6726453073380109, 0.25829145611793763, 1.2061943934027088, 0.8674336033587641, 0.6529733766038612, 0.39774681993334343, 0.576392068079807, 0.4349733379262427, 0.6853826057293098, 0.04253495237734872, 0.7285720915035732, 0.5270692020671733, 0.4734119080881681, 0.21472999761130793, 0.3142026090742437, 0.44473298442296605, 0.9352400675486772, 1.0761687108775786, 0.45549361360356866, 0.47635444709839253, 0.7596267423614889, 0.056513633790463584, 0.39461005112560965, 1.2157121140373524, 0.35767234102998585, 0.4576018921170474, 0.6844232042147471, 0.6725966998136041, 0.9559157164692973, 1.4139724823049802, 0.7551179485515049, 0.8059033572611332, 0.5896824066337585, 0.7697660364628927]], [[0.8535279040372205, 0.6777351175851075, 0.2723316901862905, 1.3784508657865697, 0.807581748882439, 0.6250360150385436, 0.4078158418629029, 0.5279845273654715, 0.3468340991980471, 0.815323897470452, 0.047194468602167895, 0.7780108355765194, 0.5569225258335683, 0.4990896464801877, 0.2523756941897987, 0.3266318340552826, 0.4455820710090439, 0.9019500969770158, 1.083057434295916, 0.39077460567824496, 0.48134205335050617, 0.7975038918811694, 0.05548958937378735, 0.39818139084924586, 1.2909255169813276, 0.3023661052679874, 0.5364626814364133, 0.6836526137084871, 0.6645581567644485, 0.8871531388255335, 1.2708117009254323, 0.781430054030094, 0.7062537679848919, 0.6181609502099834, 0.783854073349916]], [[0.7871155300568128, 0.7061494052039736, 0.2811238585385832, 1.5834247925494278, 0.7842990699150765, 0.6185124711319077, 0.41877331475825486, 0.537952127272112, 0.31330849278135386, 0.9593322803259574, 0.06413773137360705, 0.7827097855946566, 0.5800369014818331, 0.5178238565711631, 0.2597249441546802, 0.35325114585094275, 0.4285126254895793, 0.7673562136652973, 1.0765648538092973, 0.3823154070696707, 0.2751846636112786, 0.5223368883764038, 0.0565092591035016, 0.39070754939716273, 1.2629895273839375, 0.2372364918185776, 0.6307731959196221, 0.7110628187617029, 0.6517618206815805, 0.9062785524600385, 1.2245037453001584, 0.8213981238038106, 0.6506483079326786, 0.6433946969853549, 0.8285831753054997]], [[1.2310455421037003, 0.733003398039946, 0.31110566945696033, 1.7467935195491227, 0.811794538570044, 0.6247051436722697, 0.41356444968531525, 0.5553150272192242, 0.33128264313653255, 1.0369777166378786, 0.07175415638158342, 0.845648873159673, 0.6678359893544974, 0.5368150763693179, 0.2541132986866346, 0.32779052222590804, 0.41095805245028494, 0.7074336022974734, 1.0450089594446872, 0.3654651112766174, 0.17033463241257424, 0.3954675238651193, 0.05343653992062329, 0.41267888325201507, 1.1861232133860073, 0.2252483426191187, 0.587829672343911, 0.7042239183762466, 0.620589610766491, 0.9684737783690839, 1.2338803568719316, 0.883225187292759, 0.6680342047532303, 0.6533920838216926, 0.9169403920064821]], [[1.4350673237030387, 0.7439164647515536, 0.28774950370370184, 1.9406741833468, 0.9412967820691751, 0.6091693624459078, 0.4244297992605416, 0.5031356592190609, 0.3030070796888368, 1.0846434835516816, 0.04624341064555926, 0.7629417062493862, 0.7210414867480874, 0.517001283040778, 0.2604745368481533, 0.32968597485740914, 0.4379866423113068, 0.7460654955301993, 0.9017640296114736, 0.2899719042446627, 0.15935804584458935, 0.20682594119148187, 0.04788634536133496, 0.4164007614024021, 1.0944744329316634, 0.24347289520172516, 0.6061823121925954, 0.6482639397734649, 0.5645522523013952, 1.041113662320236, 1.2246361409067879, 0.9177058952604704, 0.6750104170734428, 0.6354151326926749, 0.8996404187657843]], [[1.3756519962204146, 0.7161162617914771, 0.263162460390814, 2.129387058777254, 0.954996971644064, 0.6076374431559929, 0.4138362207731957, 0.5227122587996141, 0.2794716507935549, 1.3321944482269772, 0.04154421010879569, 0.8925607612039657, 0.7473404782227706, 0.49528441926409034, 0.27430670492713544, 0.32501928113853723, 0.4396313912070448, 0.7609649974321047, 0.9665253529210606, 0.3217938945116973, 0.1525699726305348, 0.2580303917598462, 0.0376499397855756, 0.4410548418977718, 1.0669317079016183, 0.3898752042928783, 0.6160144348875347, 0.6240240180347028, 0.510777947545006, 1.0576420388012537, 1.1974073800451872, 0.907129471552307, 0.7351520942527014, 0.6372258464777121, 0.9236575571856199]], [[1.4040116914968226, 0.6955116588781741, 0.2590425694611928, 2.2380485228006415, 0.9617702671012351, 0.578140091615784, 0.4070537222571219, 0.5120579546420534, 0.26270896010080597, 1.4762575494841612, 0.0345751210058031, 0.9679559921292773, 0.7712666689674613, 0.49663476279528357, 0.346348116824783, 0.319191126875652, 0.4394760462824383, 0.78868696014776, 0.9623608719759893, 0.32040905952328197, 0.1671230990606251, 0.23407172777976576, 0.031337832446705675, 0.47594811399346026, 0.9992054289345833, 0.4509091308095616, 0.650545850472154, 0.5711343397047434, 0.5093343099793621, 1.0511853627162902, 1.1572449096417818, 0.879351050792168, 0.6671329901183616, 0.5659924262849058, 0.8681240241660069]], [[1.3647257990936739, 0.7365000384062583, 0.260354660074616, 2.172486383373874, 1.0404046323351892, 0.6692768717189188, 0.45455394811161576, 0.4972080441091944, 0.2393851692846346, 1.5751156145364975, 0.03233312365904901, 0.9361255485612937, 0.7807407031064072, 0.49439530324106684, 0.3284538342020293, 0.3069108008635699, 0.47535469490019644, 0.80044834440603, 0.9963411238699426, 0.32341616584265415, 0.18344896575537664, 0.2133742903787887, 0.02966106992101713, 0.47819733679462373, 0.9939367818947248, 0.5097949328020162, 0.6848606302678466, 0.5280826190295467, 0.5182618675582383, 1.0422586955931212, 1.0919439888635898, 0.8491881989011474, 0.6227824304791475, 0.538550252173589, 0.7589091828184654]], [[1.280284595885186, 0.7981183949783403, 0.27258209287096086, 2.11987863176857, 1.1023203725048374, 0.6669439755493711, 0.45204566277148006, 0.4978478395340453, 0.21871734192712466, 1.6491299724836628, 0.06374680295793449, 0.9016504285951813, 0.836693125379985, 0.48985533260793346, 0.3631753445711879, 0.3186743824545497, 0.4203991690652581, 0.7870319684669296, 0.9095932513330721, 0.2634940166275019, 0.15748059030616915, 0.20062648910608671, 0.028553001432520504, 0.44590864363173843, 0.9443253530558988, 0.5348995736181561, 0.7197158536264829, 0.46271308631267605, 0.5244624678598868, 1.0397208932547968, 1.0258321848225442, 0.8300311473054416, 0.5853462414471879, 0.5104917999334568, 0.6413608391598749]], [[1.238314826016363, 0.7898901895655107, 0.2777432888386657, 2.0732132568087893, 1.085016177508601, 0.6682954295821948, 0.4611095016072698, 0.4880732931631426, 0.21904804412219742, 1.671724714808055, 0.07387529591126954, 1.0449230577920114, 0.8430321705262153, 0.5127045930887644, 0.36727974924568557, 0.3347528466776145, 0.41551521241536116, 0.9195794979653331, 0.8152042311095631, 0.23310356961235285, 0.17273731011957905, 0.15079549624610425, 0.027813802182396728, 0.4093685455972357, 0.9527211093913831, 0.5321238261452113, 0.7112136725451417, 0.4877592075114157, 0.5349783579642097, 1.00809155517727, 1.0026367721701588, 0.8109385843732924, 0.5190208594969301, 0.5366200540989569, 0.5327992939766119]], [[0.8687273366449317, 0.7911474519423128, 0.2860525024525547, 2.0128732563056912, 1.00575615457545, 0.7013523224256535, 0.47978978999935673, 0.4687938011019168, 0.29676074619195475, 1.6785510105413821, 0.09315119230629601, 1.0058602527410447, 0.8426091440169785, 0.5017235798341526, 0.3965160908614171, 0.33856341484997055, 0.4275956384973904, 0.9530125106171381, 0.8481656815481332, 0.27025132900224014, 0.19285825833137557, 0.11707159408759255, 0.03419617458347328, 0.34804110270107635, 0.9372897322873274, 0.5447838108579293, 0.7009472280859439, 0.5128246064457553, 0.5645038707364016, 1.0003521099779644, 0.9924197369891568, 0.7416132185788897, 0.5032632188158305, 0.6507216247501013, 0.47015527126916934]], [[0.7923279961369818, 0.8576530478663881, 0.28237667685795403, 1.9941588968070603, 1.3215987547301398, 0.711144635633466, 0.5058986413778368, 0.45583225054381493, 0.3392630963905765, 1.4279837702395017, 0.09412917713831857, 0.9880206762616223, 0.8731095748918725, 0.4870319586555329, 0.35668449945511355, 0.3279170495476572, 0.4735348613300949, 0.9522741921074911, 1.0185140422276548, 0.294030917166233, 0.1648443545654175, 0.10150361710099307, 0.035948403437789586, 0.3271838067568641, 0.8426410014287573, 0.5707255784515939, 0.6910128070918415, 0.5503193455517756, 0.5520506644114339, 0.9989376478292513, 0.9539229669024345, 0.7231118035975368, 0.5152362875838545, 0.7965152611375526, 0.44308362254999045]], [[0.629717964082806, 0.7110661243988448, 0.2841749024009003, 1.9987297389386625, 1.332791640483586, 0.6694461924503036, 0.443357062696981, 0.4345609944046643, 0.35402925729221135, 1.3442528041050759, 0.09762395190017993, 1.031361156952779, 0.9291994706589004, 0.48232920733258533, 0.29472462198935984, 0.3212697602145276, 0.5320457992162112, 0.999765378716605, 1.0183642262570298, 0.3361018646451521, 0.15354650240100307, 0.09870487753239172, 0.04143413850756803, 0.38718611934692687, 0.7258669971904289, 0.5552178606374203, 0.6811857629409708, 0.5642223120533286, 0.49991619347769933, 0.936690755611338, 0.8962058291663406, 0.7656824038359427, 0.4920543497409684, 0.8729075513282922, 0.44925476530398933]], [[0.4853282169138008, 0.7020806228026504, 0.2993736150082599, 1.9684649021946758, 1.2942632662623326, 0.6970374742346148, 0.4243199425251761, 0.43102433030935466, 0.3740246868278448, 1.2168822093846092, 0.10443305507978157, 1.0374973756604584, 0.9743569914233607, 0.4648175375623995, 0.28751234954866417, 0.3715123769942956, 0.5235116602864899, 1.027967867084358, 1.0432846837474152, 0.3190526572838811, 0.1744661368273281, 0.0769365726272093, 0.0206645662632768, 0.47850450849875764, 0.6678840061073378, 0.5546179634623067, 0.7319359000884023, 0.5883042416844733, 0.36777030809968014, 0.9055439329132693, 0.8198394979592535, 0.8060922929303963, 0.4598419382433425, 0.9087556568894322, 0.48471180794578345]], [[0.43118317958850927, 0.7093937718318739, 0.2896527331039648, 1.9322527379522076, 1.2629627398876928, 0.7041199751252345, 0.4189168693416533, 0.4644962033242562, 0.4074080554282665, 1.1323158517026972, 0.123156040585366, 0.8496294537483885, 0.9848923483935932, 0.4363065367970484, 0.3003318559765601, 0.31184255602356475, 0.5032277115219144, 0.9003114477229625, 1.0342875425964437, 0.27267127732506047, 0.18295875784269694, 0.1145283393190537, 0.021121902863481236, 0.6156049269912139, 0.7037554724556471, 0.6865551285175013, 0.9392474552555076, 0.6391593250585075, 0.35391870030697864, 0.8814388824571637, 0.7262089016725386, 0.79070322157982, 0.45193492574651506, 0.9630059193159244, 0.5228043407353938]], [[0.2869350371477992, 0.6128722852086524, 0.33953270041824335, 1.8284124458685704, 1.1840970753813913, 0.7274649111229554, 0.3933513566566889, 0.4900497868409981, 0.42526004325507716, 1.1553757706478833, 0.12232074354943945, 0.7989469898545865, 0.9978252906998566, 0.4266931329997795, 0.3069840758401977, 0.2957601114970295, 0.4628115425462971, 0.8315814134693509, 0.9137110389834068, 0.26897877461952896, 0.2103124139057619, 0.10086533481060174, 0.02216393479743982, 0.6208762633554774, 0.7477513844498309, 0.5352172513227111, 0.8233796041427623, 0.6607142898893271, 0.32620222161404866, 0.8912142729137088, 0.6723327388968348, 0.7439660379610138, 0.4203943446720191, 1.0803307443674224, 0.5561734852233579]], [[0.3281692762822045, 0.5454516364675213, 0.4690036175713498, 1.6196714298100283, 1.028837133057611, 0.7911326463919575, 0.38611444568032377, 0.5038638493375739, 0.44105281930022033, 1.1742688875989673, 0.11922659901410892, 0.870093618141172, 0.9769235924987432, 0.4307843373957752, 0.3223114764677911, 0.2642045820906717, 0.49708084696722915, 0.7438856213753449, 0.9242035017531127, 0.2595570889391819, 0.19065562804646302, 0.1046117664469636, 0.031192717790287528, 0.6430815686081274, 0.9277147123123679, 0.27522405370241243, 0.7115911442510319, 0.6738865778774059, 0.5076073728602422, 0.8926248437342599, 0.6944279031418183, 0.7342430844052088, 0.40680361792450115, 1.1969168627621585, 0.5955420974665229]], [[0.22781595560775858, 0.5735784618796929, 0.48028528139999616, 1.4425092262585837, 1.1137157596612617, 0.7619000399865383, 0.4510132974268305, 0.5158686642857618, 0.4658783205762979, 1.0123664429980428, 0.11329932950049287, 1.0240344770362388, 0.9783656698732983, 0.484539437344232, 0.315768713971462, 0.26400942607888267, 0.5017705202316464, 0.7856661523919933, 0.9053126281904202, 0.26611537755345394, 0.20140246460660033, 0.09477520687733086, 0.03526078944513539, 0.6523906062474993, 0.9672342768848414, 0.12455712430162902, 0.5959077972789653, 0.6873868136084529, 0.511509305289513, 0.8775183590076636, 0.7413946648770806, 0.7718817680893268, 0.3855160063069075, 1.2456654030173264, 0.7232745050666487]], [[0.29944210855638403, 0.5205716070336388, 0.4355899904486613, 1.3117380290659024, 1.2058499502049562, 0.7486856460418827, 0.46562501960537617, 0.557339324611196, 0.4482712688711985, 0.9533598611100311, 0.11944098277093049, 1.0794410633018476, 1.003930293146195, 0.49858090704833946, 0.281703099670438, 0.28226136221921866, 0.4974235601669148, 0.7765138201436597, 0.8054287152434493, 0.2943761984496879, 0.25829851604556286, 0.09344211258215118, 0.03490060679219412, 0.5943699291046185, 1.0289232836977886, 0.11673306341359191, 0.49779427759827544, 0.7752608927036396, 0.4427011686025639, 0.8280605739962529, 0.7709239711909324, 0.8131846072944073, 0.36259725697459544, 1.286903995181186, 0.7953189988963261]], [[0.38133047162693917, 0.5440594516309237, 0.4690906837218266, 1.2721363090610356, 1.2646731388754657, 0.7155250070919135, 0.4020438472254306, 0.6275408571019223, 0.42651407261723945, 0.9894668944405387, 0.08820613220863656, 1.112271900535598, 1.0589347787739014, 0.4880395066256674, 0.3309852251431181, 0.27461490240411324, 0.5308801763831972, 0.769182128366894, 0.7323308895683291, 0.4296899603654014, 0.29654006130601873, 0.09224406643298307, 0.021318934083892547, 0.588111888431653, 1.1960607098838776, 0.12044838232778865, 0.4133351026840439, 0.819942204563204, 0.5169630150400224, 0.8372804286502626, 0.7654753693849714, 0.8928551532257403, 0.3757351167993739, 1.1748704157501677, 0.7848178411865514]], [[0.5172338222879976, 0.48072081489873997, 0.4108565398300194, 1.1806839593000844, 1.285483015492646, 0.7313180939996796, 0.4178084896054682, 0.6630557200814737, 0.40993514412411425, 1.0907094290518091, 0.08735954346699026, 0.9531892697949014, 1.1161700862144512, 0.5200625817177102, 0.370209410733028, 0.32489884552703674, 0.5427131545700428, 0.7851685522747112, 0.7442426964639609, 0.4305870238187932, 0.2813337127179606, 0.13656883720727556, 0.018627893530949364, 0.7446135390926876, 1.331607870360851, 0.1334742913979446, 0.423184220702695, 0.7974352553842882, 0.46022105845353944, 0.9384787463485829, 0.7376242668876701, 0.8780783985846953, 0.39014730741829357, 0.991115891521691, 0.7604644652345749]], [[0.6175957109496267, 0.5819080467763029, 0.5292783959532215, 1.1392385344040528, 1.2629630771841507, 0.7216559825341197, 0.4112198509840793, 0.8502403063905604, 0.41494831141479793, 1.058000918787086, 0.10974144907545716, 0.8621935747188447, 1.195727343030011, 0.531073360726575, 0.33241026137170226, 0.31376672670475664, 0.4930215364184901, 0.7960100405011301, 0.7202084657264973, 0.4604369096007274, 0.2938660987855287, 0.21022834065025764, 0.01496158751783791, 0.8668589261385626, 1.4370935756458036, 0.1734190681866068, 0.5012495123525463, 0.8436996969648805, 0.4560032708061045, 1.0269027343570014, 0.7104356064402381, 0.8659782189465132, 0.5360766741412928, 0.9657705368648679, 0.7922816573073397]], [[0.6196558301056626, 0.5624755296318947, 0.5042680253118739, 0.9798975453988248, 1.2220322593465647, 0.740864743431841, 0.41061181298803767, 0.8808769158136547, 0.5067383856649784, 1.0320975263717835, 0.0836628780053541, 0.756597952420446, 1.1328129116407513, 0.48433320429146337, 0.3469404109980826, 0.3243036623060717, 0.49127244599956416, 0.8035229751781154, 0.715772725503521, 0.39270211562017726, 0.24000233602792637, 0.18589014488185507, 0.017391502769198847, 0.7771701251903174, 1.5228386618195198, 0.14717785457804883, 0.43950270389673635, 0.8218531706552435, 0.5168523395740348, 1.018094798149348, 0.6347825913771518, 0.8759853011956439, 0.7270737240242648, 0.9322825472337322, 0.8457657358165818]], [[0.6466094017860088, 0.570030785191128, 0.5137933276769023, 0.9038742575227915, 1.1925881102656308, 0.7267938104468195, 0.4127225455240823, 0.8015372710748321, 0.5214307751274527, 1.2192820499775872, 0.09707745768011258, 0.7438491996195409, 1.0718081265213124, 0.5183968448126024, 0.33370085133300226, 0.3512823212783127, 0.5101858813132154, 0.8485885814530356, 0.7070929739892065, 0.3501931777960638, 0.2736239842573376, 0.18103366292098244, 0.02783051191332815, 0.7681396865798615, 1.5269025192696821, 0.11494960859578285, 0.4284318008429459, 0.8354478737675815, 0.6953984865040108, 1.0035953109295108, 0.5159391371101711, 0.8804450423451288, 0.7105362040632983, 0.8852024010232994, 0.9181086534778958]], [[0.6286725636426076, 0.4806449959991289, 0.4359327321840439, 0.9233253053995973, 1.19011771896101, 0.7092313364403884, 0.4071384185665028, 0.8802610822247916, 0.9352983939015385, 1.3406193826837596, 0.09488023135743692, 0.7058938117974562, 1.0636949041883053, 0.4468753988375477, 0.3299185527223878, 0.37043343227199715, 0.5055684442945259, 0.8874136092252999, 0.7103835959148854, 0.3364500089680357, 0.2602807764599789, 0.18583307562581772, 0.022954279319088666, 0.7545516574057409, 1.4417849868425299, 0.13766833762214797, 0.5363803419635359, 0.8596784065854957, 0.850813555709411, 0.8730320994618146, 0.47908955227003913, 0.851845885362673, 0.7056647886098244, 0.847085986583805, 0.9564088474785132]], [[0.5646502888108009, 0.4274416888661071, 0.3935559186978032, 0.9498096420481899, 1.2981958153830204, 0.68758368569339, 0.3943997606707154, 0.9734162935201692, 0.9375669441574885, 1.5116553200896274, 0.11051240869382317, 0.7514137856778245, 1.035592783948449, 0.4476499556810197, 0.477648046078869, 0.38601898562393633, 0.5047468309343557, 0.874940311708534, 0.7037831712921025, 0.2977820586182192, 0.2508249415452795, 0.1941955446195407, 0.02617624896142917, 0.7844997359164041, 1.4187425619175082, 0.21512990548896274, 0.8156434538346312, 0.8185765702490579, 0.9195087654524479, 0.6890127495245756, 0.5143506227944322, 0.8317594462857149, 0.6844599766121262, 0.7904606077739008, 0.9255384201088932]], [[0.5114821535150793, 0.45819228452051497, 0.38604410763364727, 1.06143618134838, 1.288631136968406, 0.6849394144379004, 0.39189215350370343, 1.0407782570369486, 0.8508273884236017, 1.5845463920614913, 0.19085072427212288, 0.6723429178434309, 0.9671495729483992, 0.44318468374803527, 0.41613439231392263, 0.36197499020093465, 0.4796989360828385, 0.8668946187375265, 0.7429721510628304, 0.2862833279147327, 0.24997857812172836, 0.16305873364375445, 0.07508067331975052, 0.5830546982774301, 1.4813487257841902, 0.22361485977953935, 0.7781405075926158, 0.7588324322726263, 0.8315049667472105, 0.5918362307552814, 0.5306342755404976, 0.7653485741577986, 0.6662673368547382, 0.7127079635134379, 0.83570887811001]], [[0.4972080943741017, 0.40471807456854825, 0.40719315080371665, 1.203352230551585, 1.3002020806653067, 0.6918159108891064, 0.3869751785000726, 1.0916505211881002, 0.8317070782086848, 1.6135589828563623, 0.12366975943468775, 0.7075400713295381, 0.9602112544478286, 0.43564249582373166, 0.3687859013014898, 0.3427288101100874, 0.5100893183067079, 0.8154935583651435, 0.8186355822948952, 0.28486214357040834, 0.23602083441678023, 0.1758122325168035, 0.04359302890388915, 0.45264022097655804, 1.551892636298002, 0.372607926271309, 0.7400995127094325, 0.702576969837706, 0.7552621677769775, 0.43604181365748657, 0.4807264230689933, 0.650482771124998, 0.6416501200593974, 0.651431140084139, 0.794675377359189]], [[0.5001521511000663, 0.4041167215945612, 0.3977162255711591, 1.347249808814203, 1.2734214969899802, 0.7024567661577318, 0.3819559199069094, 1.1124816786839051, 0.8682775995098395, 1.6329110057235054, 0.12539152952921867, 0.7334856093170022, 0.9166275689702854, 0.43938097379850627, 0.3470907907222207, 0.2894749178176297, 0.46535989479999207, 0.80562560560456, 0.8107657584623116, 0.260846170198219, 0.2409615169599661, 0.13802609464246532, 0.012513784441619963, 0.4468588949077437, 1.585092305822353, 0.502000434145897, 0.7345507868007428, 0.6343824958455895, 0.6538955868920899, 0.39353879017815296, 0.49362420635180204, 0.5489512428300809, 0.6487605173051709, 0.6315879327821032, 0.8308492376636955]], [[0.44149908550184, 0.40228741816302627, 0.40729755853556476, 1.505327561839602, 1.2896096923124456, 0.705350422060975, 0.3865652799355685, 1.126535116475408, 0.8781410385085338, 1.6293413745076493, 0.2171100567864789, 0.7968795455700068, 0.9505882147203928, 0.46716782346545827, 0.34180456566887907, 0.294989143302655, 0.4663351116899251, 0.7688138572703801, 0.8234070411643298, 0.22420228553125693, 0.22321049507772586, 0.16043758191119606, 0.02350698476774469, 0.4459453351147336, 1.530744088401988, 0.5761086546214146, 0.7405259605492694, 0.6088999766717084, 0.5324521508131381, 0.38399699837689505, 0.5605647165266183, 0.47119082320300376, 0.6433151823197814, 0.5914255716095527, 0.8980090626182258]], [[0.3926431184122697, 0.407170614466273, 0.4010842286355836, 1.4676834481601888, 1.2867994760854122, 0.6779365280489636, 0.40002845915651336, 1.1241313834395772, 0.7847813893704696, 1.5794309638002302, 0.3691794630270525, 0.8362671984560976, 0.8210425427231918, 0.4632761876915703, 0.3656188755044041, 0.2834495326688997, 0.4683945561422504, 0.7618409271128257, 0.790182222459495, 0.22256101177227972, 0.2118562831029518, 0.1672602686708144, 0.03580135826477909, 0.5033524696807004, 1.468174274866569, 0.618253307227998, 0.6833649119177224, 0.6264869487762049, 0.414577305879209, 0.4968698382428802, 0.5915867269628134, 0.46603087364264106, 0.653876589502955, 0.5018354691574423, 0.9374626346773818]], [[0.3930015051669765, 0.4360791000794574, 0.39595667341720386, 1.5577195656515834, 1.281452310647273, 0.6782783878185898, 0.37791321759080365, 1.1439640146200158, 0.7135697661193409, 1.5531568325946705, 0.4372611142366764, 0.9085485710316175, 0.7690741339135337, 0.45899971558461666, 0.3698583385871149, 0.2997376765392728, 0.46408174210446695, 0.7449418201370449, 0.7929687911865717, 0.22540731409994097, 0.22476038899792192, 0.2049731680877075, 0.015635862300533494, 0.5277407322710254, 1.362377640786347, 0.624303297224855, 0.7344484865900884, 0.6460664278380328, 0.3880979129956055, 0.5815974911278577, 0.6191136264733798, 0.5298305971641589, 0.7087247714590821, 0.3569966856353872, 0.9259914793138166]], [[0.40242259831173466, 0.4482882670027095, 0.39839972176815586, 1.6015454242333793, 1.2572370173457785, 0.6947498477641267, 0.3764406898810288, 1.1076732441761175, 0.667136715787494, 1.4770299575558445, 0.4166347513480901, 0.9133940487165534, 0.7578001437957382, 0.4557472297251238, 0.47830437041003143, 0.2975096098820843, 0.4693829635985737, 0.7141718374235827, 0.8105053799958721, 0.2249213514279389, 0.22473631502797833, 0.23238344948562306, 0.0119360098959819, 0.5302028289716113, 1.206419345919816, 0.5976834247192542, 0.719852133034285, 0.6726559188613961, 0.44487003232447897, 0.6447830599184026, 0.6147619256977392, 0.6322509425791554, 0.6668710166830532, 0.29260941992195905, 0.8938721051669246]], [[0.393061351788263, 0.4344981896861324, 0.3965434064511385, 1.5125069960202007, 1.1662748007077421, 0.6948975753990803, 0.38034776938020404, 1.0234485162675209, 0.6016054471369201, 1.4539691542028201, 0.48098955996780135, 0.8888558583821865, 0.7276554895133023, 0.4612049039758841, 0.460377889997552, 0.2766239247331399, 0.5100033249225864, 0.7416360546004568, 0.8294163504583136, 0.2564382165551653, 0.23746412792662835, 0.23103225227598212, 0.015686005364824118, 0.4874536007247253, 1.0749081328133958, 0.5768191450458854, 0.5865586516696221, 0.6362877165133156, 0.45932382955373274, 0.6844641685529042, 0.6470804189440407, 0.7372666295087832, 0.5908118185093327, 0.25382416674684594, 0.8608127067390271]], [[0.39509806378123774, 0.46620646011612454, 0.3900999410304541, 1.414743327797907, 1.1081576837573628, 0.7088373523231664, 0.37908348031318045, 0.9144926821902604, 0.5401127233207403, 1.6162284279774097, 0.4767946709647189, 0.8841125860585414, 0.6705289104306813, 0.4706305754764581, 0.4768387480543002, 0.2847146576662667, 0.5212946565167109, 0.6989365730117103, 0.8554265073711153, 0.26432234099147467, 0.26564213475829346, 0.27737429336651453, 0.017665612498051396, 0.526678543983019, 0.951970229903255, 0.5555603163589757, 0.716417585125297, 0.49521475367826107, 0.5068523779371079, 0.6887887996676066, 0.7341900695335206, 0.8462727813363556, 0.4457196340277852, 0.2791337298080437, 0.7995785892179927]], [[0.2987300048008772, 0.43585146229877314, 0.39086074395046777, 1.2900729721334143, 1.0951583920472796, 0.7131217897319986, 0.38683790136432544, 0.7991433153689599, 0.5260335948660979, 1.6796162639271213, 0.3166930479029925, 0.8832325600757813, 0.5442886942102522, 0.4609115863813264, 0.43272430315598853, 0.29106053675343924, 0.5230464252927679, 0.6905793644371918, 0.8313070108988525, 0.2241149038761307, 0.25077420701391423, 0.25998513735335993, 0.020013740374298164, 0.49102648848597624, 0.883927760200252, 0.5219108797874749, 0.8417011309624612, 0.4040365430717324, 0.5603340920307558, 0.7069795136524962, 0.7118578726612474, 0.7954450352800977, 0.3950563515045259, 0.38338539460288723, 0.7406266779793762]], [[0.2925732437473536, 0.42289335557550556, 0.436396167586935, 1.1554482297083266, 1.0967095596436092, 0.7482789057785251, 0.3826030844612397, 0.6919697463905048, 0.5056685068813235, 1.6863358015831809, 0.3296854202323738, 0.8833273087285951, 0.4962201406645337, 0.45486685413940076, 0.4102971575301944, 0.3112833370076996, 0.5165350658283102, 0.7031889754268412, 0.8321240322223447, 0.2092415190707756, 0.2604127629633547, 0.24924520996487554, 0.015395973074701877, 0.3763188903339912, 0.8429056546868237, 0.5087443349030492, 0.8358709803831648, 0.394173160784254, 0.5863884347741252, 0.7395111127700029, 0.7180374080040559, 0.7490087410725498, 0.3681240817017867, 0.5784273765470229, 0.6904806213248452]], [[0.34221090460166914, 0.43272064570435964, 0.4279166227928237, 1.1073119838653156, 1.0997306248784526, 0.7431593840417491, 0.38768171091726333, 0.6357598943450493, 0.47491342002150383, 1.6517839267786483, 0.3240109027094342, 0.8624985227428077, 0.4586111639963393, 0.4690488547445613, 0.3997765383291721, 0.3085200456249396, 0.4991345177516285, 0.6995671417591189, 0.7859029374825175, 0.21855573937928552, 0.23970875962882426, 0.2421250965864842, 0.013270794087549675, 0.28854933155062834, 0.9117422688184507, 0.50582597370497, 0.3971123598033268, 0.36691431080244713, 0.5772828390282206, 0.7754116617961497, 0.7475208710332157, 0.6841647986805512, 0.34412579004552385, 0.7308160863551219, 0.7154780266652008]], [[0.378640648773546, 0.46070459749032205, 0.44354069120439255, 0.9130385510598813, 1.0771700494139766, 0.7606516145650096, 0.38293635588371977, 0.6351183811752409, 0.5388588454842518, 1.6217402499801665, 0.3184449167062733, 0.8638959717543329, 0.41337241510259914, 0.4657228933775384, 0.41267963958487036, 0.3214999604229834, 0.49867234689107137, 0.7793676312214777, 0.7882003848440219, 0.2125101964627833, 0.23487515776478718, 0.20163553599934694, 0.009908855057652766, 0.2262241382380031, 1.0399287409153262, 0.523365770066361, 0.3384544947592154, 0.3982571638798858, 0.5518395297870183, 0.615352190998397, 0.7357741197968362, 0.701947061282125, 0.3136987912031258, 0.8704922970451754, 0.7875323050851655]], [[0.38360917549398504, 0.4351426270205171, 0.4217178855422764, 0.9328647380399078, 1.040740673661247, 0.7459633795826779, 0.3875670470069708, 0.7039678858434785, 0.6094397840475634, 1.7104490098573601, 0.36545349033504837, 0.844779857306821, 0.3802374211624992, 0.48121482574101243, 0.43169839548415107, 0.30971006478025176, 0.49337364252278104, 0.7680470815374768, 0.7864531185640555, 0.21183970352900955, 0.23687594727225098, 0.19184618901227302, 0.016229909145970355, 0.25477290112567885, 1.182882875598953, 0.5269835024962679, 0.3064954565368996, 0.4288215030467652, 0.5033681492800476, 0.5277885133694258, 0.7436330204292185, 0.7994117540023167, 0.27779726780746195, 0.9468104848966206, 0.8132619254748402]], [[0.4447594106270306, 0.41626927282075316, 0.46016704036203115, 0.9902280922854549, 0.8760405297209574, 0.7764951377548874, 0.3895688543326492, 0.7580392345394551, 0.6954799370350927, 1.4184192396620228, 0.3605981067782982, 0.7651759565673908, 0.36119471917131096, 0.4191404228175495, 0.4055405133097721, 0.28412351897741517, 0.547835652387357, 0.7736498782464856, 0.7805400477415392, 0.22416670078133605, 0.2325068501913385, 0.19905062164016013, 0.01565260856687076, 0.28861175063628086, 0.9811196911643607, 0.5275556249937522, 0.2938287579663724, 0.4098673786999807, 0.4683650335516593, 0.5293545829611443, 0.7217126646833607, 0.8940050652620362, 0.2906701970653891, 0.9838124119083136, 0.8238542603187908]], [[0.46944441552096833, 0.42102421490307, 0.44141155758935313, 1.0378398817653949, 0.7530233942421912, 0.7266531945900836, 0.38610861679154884, 0.7868793669489872, 0.6336926043533427, 0.8739566362675445, 0.3561291477084683, 0.8292374746860418, 0.3426905029498076, 0.43365679732404994, 0.3953616704076604, 0.2930663626845974, 0.5396887265229755, 0.7169864651802159, 0.7586111855726334, 0.23617947176051096, 0.2513936517620675, 0.1891408711617062, 0.020587510661577706, 0.34187721945602617, 0.5812972832043195, 0.5377705305413495, 0.27043344738392255, 0.40999737598825364, 0.4643739319722428, 0.5853391348829557, 0.7170001289002953, 0.9381810608160575, 0.3253059855502868, 0.9714781546280152, 0.7805798446435976]], [[0.5150371104248186, 0.43829827397498655, 0.4471635266911891, 1.099931789387021, 0.5601898724456889, 0.7142022891784863, 0.36749465731126396, 0.865018291242672, 0.6647059053475829, 0.48854032910165474, 0.3461562004558958, 0.8443425514590395, 0.33102333293811564, 0.4353062724672564, 0.3648930990103967, 0.3182474161687465, 0.5514590834393194, 0.7459689320080751, 0.7515728828511674, 0.29359336090349025, 0.24875523888420223, 0.19197137997850405, 0.012155042449713938, 0.3517604774863351, 0.42448147381444173, 0.5577926992117914, 0.24809307044462864, 0.4675612928922319, 0.4506350365223259, 0.6008614568149875, 0.7170968798651205, 0.9904416546036016, 0.3476863111566112, 0.8898196278509795, 0.7490150169093891]], [[0.5520122492359496, 0.43198881900698344, 0.40925642925630534, 1.135854742624057, 0.4001830467316124, 0.6963285372444138, 0.3787516854344032, 0.8677484453117663, 0.7005303587019576, 0.23166704587485332, 0.3561494493926326, 0.9742305476771647, 0.31219416979750736, 0.43773522544143095, 0.42726600943980675, 0.29332842192215935, 0.5419096540461507, 0.718570408408783, 0.7552045078545454, 0.3138685384723682, 0.24368385003086157, 0.2167871861193118, 0.020186469508882192, 0.37912322777872465, 0.318566874054076, 0.5121145268360074, 0.23922671763560927, 0.546974283943884, 0.4204117948978234, 0.5997777272619549, 0.6701978119191181, 0.9929784136432047, 0.3848745125610585, 0.7979885399098595, 0.7504389453006466]], [[0.5934483472888166, 0.4109989309835276, 0.4085603212732293, 1.2079275617902034, 0.3850840337720431, 0.6704224572832969, 0.35987997962914886, 0.9032142970360924, 0.66975638249423, 0.18559166268416283, 0.3304061503239731, 1.0494526621283398, 0.2643154521117603, 0.4574850419503726, 0.4297827934950584, 0.28257505182592735, 0.54960300749134, 0.7309092847326917, 0.744726330200331, 0.35411182086704496, 0.2573073724809021, 0.22540066943770876, 0.020825609951929735, 0.39024208403052396, 0.2769995263896281, 0.40761494128102815, 0.2293210862176524, 0.5460006606209997, 0.39883246328159455, 0.5756921172441041, 0.6025842608842231, 1.0321507066769617, 0.40706742776060906, 0.7462161922661517, 0.743851857987626]], [[0.6677271596655866, 0.38286241278707467, 0.41211079036874926, 1.1636814099667734, 0.3009937990511433, 0.6955082770811387, 0.36894616276239917, 0.9037559242444386, 0.6160774005605093, 0.24361292171324403, 0.3175254380970134, 0.8854733057791576, 0.23474063499168712, 0.4721976889275168, 0.38768495312362145, 0.28611867609677044, 0.5537111961424244, 0.6934253256863879, 0.7181394955107079, 0.4034308274308933, 0.22059466884428894, 0.23413158167020176, 0.021932125989468802, 0.428667110028702, 0.30727644750019834, 0.31397244567491966, 0.2547415534590956, 0.5469596860029017, 0.3434629743502101, 0.615732622921422, 0.5142066364296918, 0.9636019015525031, 0.48627136448588976, 0.7195225848352109, 0.7766269792779831]], [[0.7166016492333884, 0.36737500270772905, 0.39557189443768004, 1.2565112495470174, 0.306995826464285, 0.6749003881107252, 0.3564033178284336, 0.9476117161027657, 0.6348286822623861, 0.3296067391566203, 0.29424515774282034, 0.7981076977912487, 0.2994286963810528, 0.4749381472642864, 0.40046149794292685, 0.29200359255774694, 0.5896085280468737, 0.6730870943369549, 0.776955630816999, 0.46232332198187365, 0.2361700904285461, 0.266707221617998, 0.023233926464853446, 0.4785577699907117, 0.35312012706688, 0.28858077761463063, 0.28729080567050863, 0.5812495571990481, 0.3419868611587656, 0.6407643285985327, 0.484206273625835, 0.9803161604848756, 0.5204810045037102, 0.737185486711801, 0.811263465726036]], [[0.7309393037889275, 0.3779118400824059, 0.4132513119831627, 1.3688001169146657, 0.3117459875625542, 0.6160785797914464, 0.34093433532965056, 0.994550044403111, 0.6532634839396677, 0.36902135764379945, 0.2976305341583341, 0.6931100471674421, 0.3746316744050766, 0.48490395956429555, 0.3925374487110397, 0.31995057730006216, 0.5565322447939289, 0.6873885623466944, 0.7947049295350435, 0.4425069214792629, 0.23987408767068855, 0.30006163617747683, 0.02887183280470466, 0.4939495713172955, 0.41005324018161204, 0.24009878262190337, 0.3107753669269827, 0.5727756318858426, 0.33899546738496783, 0.6412854871094245, 0.4690896255796596, 0.9775417232894805, 0.5446985733014501, 0.6941607763039435, 0.8380192531221293]], [[0.7335742967490873, 0.3789140039394248, 0.41092401431533687, 1.5065340401953482, 0.31844603337627314, 0.605583270149154, 0.3525750846424102, 1.0551947866109015, 0.6782039745571111, 0.41425589767881554, 0.2955420009930855, 0.6758739116180819, 0.4169367275364339, 0.48619831811109915, 0.39415035818055877, 0.29856195303234756, 0.5457050017891436, 0.6880652034928512, 0.8265138679241827, 0.4088164741346572, 0.22019447443766801, 0.32728572464902145, 0.03180904372482649, 0.5202161058763872, 0.41360465436143967, 0.2239851902482557, 0.33551588975979313, 0.5406152122728642, 0.32302518590650187, 0.639410611744821, 0.496252007307031, 0.9609515600991598, 0.5914714923558939, 0.6473998449855329, 0.853919952101521]], [[0.6833730033959111, 0.4017267150808015, 0.440260920238542, 1.6194256472005695, 0.32266035684038263, 0.5862943922724844, 0.3780678419615293, 1.1032099432869764, 0.687271228569031, 0.4643226283053633, 0.2978618253187168, 0.5844015260006199, 0.4507173858833074, 0.5376585796411644, 0.4095602514499681, 0.3032227798287279, 0.525608753652739, 0.726507806969845, 0.8270324904366803, 0.3574407233801472, 0.22985039773213228, 0.4059664198396954, 0.03666847206432833, 0.4807318631421974, 0.3568606898257853, 0.18149732788364176, 0.3663939381744723, 0.5359213992370533, 0.3664298677223041, 0.6304072903014557, 0.5551702208546906, 0.995670033590536, 0.6294253521545327, 0.615117350227091, 0.8528315062014594]], [[0.6942984801952573, 0.41855727300450074, 0.4433833296812587, 1.660295035143315, 0.32158329404054264, 0.4324116271560382, 0.22181119607613448, 1.1447604173335035, 0.6946582311621117, 0.5159955426333043, 0.277497710921798, 0.5662206587115677, 0.47744425641125265, 0.5364341415673088, 0.49240999821090464, 0.3038139012944603, 0.6340705206545902, 0.683525270564976, 0.9135602976086488, 0.36489483284702334, 0.21604824949005338, 0.6089837209914182, 0.04353057667195015, 0.4781009019760127, 0.396948691373735, 0.14659811699955144, 0.36181361105549525, 0.5734980837703946, 0.4099629608758899, 0.6305395505959024, 0.5989683972887554, 0.9904188051563292, 0.628502415987213, 0.5348529952814428, 0.8238601318114726]], [[0.7206589944285913, 0.4407962562537698, 0.44010894418127655, 1.756104182703365, 0.3181560311200398, 0.41902716736005613, 0.23385523328293492, 1.1316110641903718, 0.6780774124819142, 0.5353560931046937, 0.34470835658513865, 0.597966852737609, 0.42600557205080836, 0.5410964975558814, 0.5258447090840395, 0.4486714270802431, 0.5135143644493647, 0.6498822747103358, 0.732960620299228, 0.36755800797085686, 0.19897889228727003, 0.4497656514753523, 0.060146643625064465, 0.4873692772828869, 0.4425325973286752, 0.18043090594524905, 0.4028686032977272, 0.5456554804099052, 0.5131258369380429, 0.6257760377706003, 0.735311083580616, 0.9427447787060981, 0.6564661323736308, 0.5155095955186798, 0.8144214247466972]], [[0.4597553713685975, 0.4460008803839438, 0.40931010695021697, 1.7680123460991561, 0.3137902270443706, 0.5394632056185962, 0.3435398236038073, 1.12187541851229, 0.6886635208193098, 0.6637938873619217, 0.34472038029857255, 0.6156727753825091, 0.47905967173266156, 0.5484648039525437, 0.5366255365380654, 0.4740001627225322, 0.5030831210513249, 0.6770585198288392, 0.8214941979289934, 0.31316401318930065, 0.17989965923557658, 0.40110137169407434, 0.06968905513868733, 0.5197275428769141, 0.39146735796962073, 0.24444286472827975, 0.3939393732136616, 0.5045862236397962, 0.6101022032242236, 0.537727795026936, 0.8796243390931124, 0.9282794645401804, 0.6315786278063867, 0.49554344985483323, 0.7467119525354151]], [[0.4570411971032861, 0.446763698434234, 0.4643550103547269, 1.792752813517513, 0.31256294265218854, 0.5359611331443374, 0.35242238812958904, 1.098116803357607, 0.6851190031532691, 0.6685481720594264, 0.3652705670871421, 0.5907138008362931, 0.6566649363195014, 0.5385833984718313, 0.45592574768194144, 0.4477886071982343, 0.5526142513917661, 0.6889065107393317, 0.7825987350935952, 0.251881415836772, 0.16109215664623575, 0.38789543106999563, 0.05973231960027179, 0.4827782965616336, 0.35800382824507404, 0.3098247104327965, 0.4039742254098188, 0.3360160381548259, 0.685858008781275, 0.5200914449121077, 0.8878660759796861, 0.8735354547806207, 0.6409711029132934, 0.5033793379073659, 0.7145115766599842]], [[0.4328631529741231, 0.44199564533248364, 0.46501467334289764, 1.8940639225707154, 0.31837336661008414, 0.5041301156788696, 0.32658694147207895, 1.0563768538741416, 0.6353988832684997, 0.68035421550647, 0.3622420968008518, 0.6196781533513352, 0.8328583032374439, 0.5178087083718166, 0.4879130198195781, 0.41178927327883985, 0.5483124955577636, 0.683523120520676, 0.7777612672152786, 0.26273727392469376, 0.16932502801564542, 0.31561329933383725, 0.08603615739631064, 0.4699657772424355, 0.3270246482716576, 0.39804735788845935, 0.4117545934421647, 0.26626611859592786, 0.7534389701377221, 0.5024709851928137, 0.8339419717530109, 0.7733642631326816, 0.6652919484855198, 0.5136060273995953, 0.7139051268573584]], [[0.4209886315705732, 0.46372742275045264, 0.45307678631265874, 1.9083983553640038, 0.41154765415268185, 0.5736694418527868, 0.42484237728276597, 0.9463058840338001, 0.6201971311305415, 0.7298790429134336, 0.2984987432155169, 0.6491760998776093, 1.0063918146241222, 0.5487180501867117, 0.35669324795433405, 0.40775507930694455, 0.5131924013633935, 0.6676679192190278, 0.6553503758849858, 0.2714803952590836, 0.17021214367793905, 0.29303823364711806, 0.045238518986782036, 0.4406210686377979, 0.3297858234557376, 0.4738467235916316, 0.39748143666391544, 0.24001935278928344, 0.8131658884491691, 0.48528989089137436, 0.7508402771239022, 0.7029524382060865, 0.6568352972216904, 0.5429533166342209, 0.7422664759392095]], [[0.40579270487723096, 0.4855618618659937, 0.44920344170581095, 1.908177292394469, 0.7055656536669621, 0.5913682005048981, 0.40871131169528174, 0.8746804155966756, 0.6136096828682098, 0.7360727984805336, 0.30427879032170857, 0.6416162809709574, 1.0654285556263066, 0.5460067961688726, 0.36682254723559343, 0.3878581066764791, 0.5326298717465352, 0.7204854970853234, 0.6553644464806495, 0.2892526004326371, 0.16377471451016284, 0.29090119639266687, 0.07108224343736913, 0.4536459916811437, 0.33657732148309516, 0.49686426245773757, 0.3752038081824494, 0.26515468076772164, 0.7967023256989685, 0.5016998881303466, 0.6334051196228689, 0.5975816381120708, 0.6625977727491822, 0.5614874561352674, 0.7983928783687688]], [[0.41446614992256675, 0.45087973949487525, 0.38959975118574164, 1.851848426993588, 0.762589409343745, 0.6262409377997843, 0.3856057493003184, 0.8520194332525212, 0.6138390976098113, 0.8257920685872426, 0.31328092752338355, 0.654232288858383, 1.1390798547958412, 0.5647183673342195, 0.37590844614118984, 0.37442646052295303, 0.5014555246979178, 0.6911793924058205, 0.6474825811213679, 0.3084364073376904, 0.16901207465922996, 0.27345206248182474, 0.056483272327590706, 0.5119854595191511, 0.34020341695848383, 0.4929631607559732, 0.3727873808228834, 0.38130388851728675, 0.7897878009216627, 0.5403546374119312, 0.5575707011108707, 0.5217507364642853, 0.8577527201062831, 0.5482799600901572, 0.9124026004079511]], [[0.43026660950656803, 0.43708798801757787, 0.3943515360251382, 1.9861892690279384, 0.8709243210231323, 0.6383297438976617, 0.3785294209885891, 0.8788676182249758, 0.722586326673392, 1.0344030231669679, 0.39197983455879915, 0.644522129416573, 1.1152971372813232, 0.5579898308732234, 0.40400714502620705, 0.38906110157814033, 0.4966676431301327, 0.6606896608720378, 0.6292992512005338, 0.2860593039857182, 0.17620216619171772, 0.3169261494530759, 0.08352139329411848, 0.5117186986288604, 0.296881881168704, 0.4742235173986364, 0.39940393540190583, 0.4380933867676009, 0.7789305811535261, 0.5268823824320875, 0.5960450615387802, 0.4673401661671452, 0.9204665412079494, 0.4988571881518641, 0.9328107283480269]], [[0.42369306593859923, 0.4670639225123391, 0.4579261746915764, 1.830148930813605, 0.9231886008954363, 0.6283073259470396, 0.3745630909141585, 0.8800066830280381, 0.5843707717368771, 1.1024777950974007, 0.3138689662752781, 0.6681648093245144, 1.069597826714918, 0.5819768241601505, 0.37277048436235266, 0.36402577270974956, 0.47421277218714786, 0.6553134051269651, 0.6569560013112763, 0.2819429153621177, 0.16878089744603791, 0.26536281232299597, 0.08435488430608165, 0.49107914922815704, 0.28320086435653025, 0.45911888758037206, 0.42768583572884394, 0.4919886670192649, 0.7748812974436514, 0.5188569862289384, 0.689902662374891, 0.3820059954757464, 0.9582616748244512, 0.48153300831176693, 0.887081062236057]], [[0.5185125801641455, 0.4805357441154526, 0.47319405702985673, 1.8037012455988588, 0.9384982551186992, 0.6338568282961801, 0.3806548366768191, 0.8737088682744477, 0.5772707331103731, 1.0014423613739059, 0.3211545969231864, 0.694310576004496, 1.0335700721003476, 0.603708303842517, 0.39267993375623866, 0.3537163931123505, 0.42501562175201235, 0.6401236912051766, 0.6849313997975712, 0.3146668513197275, 0.1643837606095151, 0.26738433611879325, 0.09583807534957098, 0.4823949500066209, 0.25337256310478373, 0.441671885040127, 0.45366060916562506, 0.4261806972890512, 0.8024092689629351, 0.45773130232555653, 0.7443701524673008, 0.37243937609790584, 0.7404760826587282, 0.5095785801930623, 0.9329679447526357]], [[0.622135564646652, 0.5076807385439699, 0.4700047209815513, 1.6159003171518007, 0.8680994275114069, 0.6186314358427751, 0.3861990467197246, 0.8574328345427014, 0.5592137050100223, 0.8338574727155122, 0.1788802296859596, 0.6911479740380764, 1.0689233724764329, 0.6306563567058281, 0.397517020591143, 0.3674379148290737, 0.43394749522173703, 0.607265546208784, 0.7472072695304965, 0.2075611313158039, 0.1646255523105802, 0.20912366345932878, 0.05146845405726073, 0.36099198012371686, 0.2288565735833574, 0.4271839474104181, 0.40873308319965107, 0.38858991463100145, 0.8435091111388769, 0.3905187218996716, 0.7785236079209645, 0.4660537865612959, 0.7714568811896101, 0.6123714283213344, 0.8963525635859435]], [[0.5196491967821797, 0.5614379485045496, 0.5330302037502015, 1.3702965056214014, 0.7887345640347068, 0.6312004036843076, 0.39091127082151383, 0.755446675548997, 0.49330020265155966, 0.7913550327280297, 0.12463389890808138, 0.691608117793819, 1.023887107102668, 0.6422286084934583, 0.3773449592337412, 0.36661794122621605, 0.39791046783742234, 0.6322617175070719, 0.72110706767468, 0.22668329181339103, 0.16106602691434463, 0.20148362258438451, 0.05686252033377878, 0.28919359492908514, 0.2588346931776272, 0.409065521044846, 0.4381543182577178, 0.3589115640121215, 0.8746914329440505, 0.3651357275590894, 0.7493444249089272, 0.5074494624542097, 0.8839590029015976, 0.6395734008000842, 0.8380628882658395]], [[0.5043231604610459, 0.5872019201610267, 0.5432403095238836, 1.3331883597245673, 0.7499550802338886, 0.647930334932725, 0.3798929826765224, 0.8157177550377357, 0.4478283923724937, 0.7941356547600877, 0.08462006050434415, 0.7055582697947931, 0.9463607822836928, 0.6345082566311253, 0.36339520664486674, 0.3707613562822907, 0.3897501972465022, 0.6475808811059094, 0.7161197451226292, 0.2057020770584821, 0.16565233001680618, 0.1285355673260446, 0.061399442926708, 0.31533222484975, 0.2623188681016242, 0.405102243346378, 0.4907263681992678, 0.31788889382225044, 0.9580558531042962, 0.32750959524435663, 0.742737273211826, 0.6066155062981656, 0.9944239820646215, 0.6259222439384056, 0.7988509655371933]], [[0.5085433304567448, 0.5770253791090896, 0.5140859552852086, 1.2169730098203124, 0.6728867633575698, 0.6119111739739564, 0.3358934305219602, 0.701969297557134, 0.42091082250328776, 0.79665365073828, 0.08634181156088092, 0.7811352812770231, 0.8832245656462676, 0.5976042163519141, 0.35115090193947734, 0.39109914594584116, 0.38964441417781115, 0.6207880058976085, 0.6872770844800209, 0.14593663408022983, 0.2120009113538318, 0.15382893556542876, 0.07210136083703583, 0.37253135306430285, 0.26467025015939727, 0.431195637656449, 0.6576287630963673, 0.33948688715559683, 1.0982127548340022, 0.3019845049205308, 0.7575695169260124, 0.710726467919375, 1.117260483407213, 0.6152870373189421, 0.8226791959765597]], [[0.6291292798088264, 0.5284798922725846, 0.4591571285721714, 1.072483344151929, 0.6664963210841021, 0.6314551754290612, 0.31269221695918037, 0.7025208162098991, 0.37985397533054766, 0.7924834587983116, 0.08438995589950626, 0.7678912186635376, 0.8206935311018678, 0.6073431166159926, 0.33907834245589524, 0.3850006794895231, 0.3916367038492606, 0.6278158216464584, 0.6760485729640715, 0.1475574128497446, 0.17197811677743197, 0.1526287721852152, 0.09621060092478824, 0.4955057416249796, 0.24844945413392058, 0.4466131185804542, 0.9355014301599688, 0.3357380429208808, 1.14147037116656, 0.3237212790696694, 0.7849982079774811, 0.8014706798512988, 1.1723382733510108, 0.6919262525516647, 0.7723105970867064]], [[0.7074000974154522, 0.49668532070097016, 0.4199406811606865, 0.8805049939794101, 0.700027211987226, 0.6780106248838463, 0.34215183794341203, 0.7461548891426346, 0.4892018076663419, 0.738662310560336, 0.09523327009026102, 0.7227576234732733, 0.7553557298356751, 0.5789363502817927, 0.3857275231607236, 0.39267660774845226, 0.45510489521993264, 0.6413217773813131, 0.6898171105215372, 0.1145547727918917, 0.1778620924229713, 0.16292506097653292, 0.10198873655726338, 0.7351938888017024, 0.24051204785169317, 0.4527025457924911, 1.2706994705096952, 0.29879158776428494, 1.156665839971615, 0.5165002161171457, 0.7611338743711111, 0.824750812421785, 0.82015399393356, 0.887136402445112, 0.6969540131035652]], [[0.6888878157461726, 0.5425556232585926, 0.49490436778325764, 0.6992456313609428, 0.6917094692065104, 0.6367638883418012, 0.2759774015813048, 0.727192303051339, 0.7333426963102038, 0.6834935646259043, 0.10231955120320214, 0.7016455568103332, 0.7287490528159983, 0.5448922196797299, 0.38190818235075896, 0.3866042903400627, 0.4775551703726093, 0.6408646638510248, 0.683046938814217, 0.11213186448628733, 0.164511204754364, 0.14478802944952296, 0.11128722068259421, 0.8623851532052191, 0.2242024200767118, 0.4474975136155322, 1.3397086267894254, 0.29421999913410335, 1.1454436076844932, 0.6653493403428258, 0.735786931513871, 0.8388543034496997, 0.7354099151211325, 0.9762137560136215, 0.5646712050099882]], [[0.7065998416958069, 0.4711117596609957, 0.4122985960353357, 0.6248443576048015, 0.7307442616549752, 0.6390601443145687, 0.27271840634833144, 0.7583553544885782, 0.7481136322309049, 0.7080306108208166, 0.11915137944098088, 0.6961967033557779, 0.7953446222858309, 0.5497784965464133, 0.36397126082739845, 0.3810958477338441, 0.47634938234423185, 0.7094397016307129, 0.7019445091122469, 0.1035050108262961, 0.1898671170205635, 0.13638879012011898, 0.10376853161909987, 0.879989589376487, 0.22872877541906708, 0.4339210484721895, 1.2874144158183694, 0.27383264747661396, 1.126959596234933, 0.8208203635322333, 0.6874103490960106, 0.8253877689950799, 0.7151669280899258, 0.9962578709270675, 0.4826613237986408]], [[0.8469187001047931, 0.46556955172538145, 0.4028901825516621, 0.6235961257790363, 0.7454375904595829, 0.6654997822772764, 0.3080600610444183, 0.7922747761269859, 0.7859984405121199, 0.8319902759605269, 0.14979591596864703, 0.6989420685929224, 0.7963429445557567, 0.605325258276501, 0.3671647712430064, 0.37656221407522217, 0.4751922519095461, 0.7247176669836672, 0.6577481865683517, 0.11870120789307809, 0.19901988769552487, 0.136814489140621, 0.10193023010214682, 0.8858813168546861, 0.23145577290465136, 0.43150816561736594, 1.2189772203753386, 0.26567407691117645, 1.0454136556697458, 0.8662950784686749, 0.631581530182104, 0.7964231480703928, 0.711288493983825, 0.9055998881023221, 0.4408409624729749]], [[0.8899369445317221, 0.4954679965855018, 0.4585475305552428, 0.59506477995814, 0.7256641103566585, 0.605982260051242, 0.3036931722271258, 0.8411799017428814, 0.8042054661672866, 0.8259582777098162, 0.1602166448075904, 0.7630469868421208, 0.7079315145036125, 0.5933969827477832, 0.3670192981759379, 0.3714732417656211, 0.4637749792323377, 0.74408718947838, 0.6731953133091476, 0.11255348980160401, 0.1911746211794625, 0.1304502840812368, 0.10485312491998967, 0.9120880504270397, 0.25666950612808925, 0.4189463199960171, 1.0847965456433208, 0.29643022256225726, 0.914170908268022, 0.8712167274615692, 0.6209853956927625, 0.8059634838512839, 0.7369684990421528, 0.7627456635881127, 0.43214859060570066]], [[0.9629983746288422, 0.49756765725310337, 0.46089297791956885, 0.5961567536508274, 0.6751095142640919, 0.6112925860185436, 0.3036028303799553, 0.767186557402467, 0.8017949945729507, 0.6769335365396749, 0.16203027729701036, 0.8086799750130738, 0.4580589896126735, 0.5875538294266966, 0.4193686591929009, 0.3674341482938293, 0.4725840406180758, 0.7229444113979933, 0.700125107441618, 0.15041503004979923, 0.20571979927329842, 0.192414517950351, 0.07692156891807472, 0.8880412688932114, 0.28105546518358304, 0.4161742197490441, 1.1549565329436275, 0.3249864687854153, 0.8389032148977975, 0.8461664974639846, 0.5685910771234511, 0.8705439031743976, 0.7403871622374386, 0.6865847330382269, 0.42711328102339063]], [[0.8924388843999046, 0.5506050146460344, 0.4692271537795242, 0.5512782762061851, 0.6945374268481094, 0.6832058939121213, 0.3494983717903555, 0.5554013883252128, 0.7907290201941654, 0.7646362042321452, 0.15436012179601694, 0.8437402124997277, 0.3939995956451008, 0.5810060363147318, 0.476600581807898, 0.34733132719078197, 0.45079265715204264, 0.6784940810868297, 0.7938075875992876, 0.15723873639377117, 0.2329285781281067, 0.25605870734034475, 0.07303077368347584, 0.9061696451794767, 0.3173666919113415, 0.4010862053580677, 1.105617734767564, 0.34669185861947815, 0.8741445672611121, 0.7843028001159051, 0.5815023184649684, 0.8603543757058418, 0.7196976238573678, 0.6245091546036772, 0.41976168203436437]], [[0.9235095117827485, 0.5635135896566517, 0.4994586628612738, 0.6910654073737694, 0.7049665167759731, 0.6674409788213318, 0.37253340014866926, 0.5652813237969454, 0.7490852167305381, 0.5878354686539271, 0.14681594989082788, 0.7865395119228593, 0.3746661489890303, 0.5843465698341606, 0.4007638358066397, 0.35491110322664493, 0.5698960940169369, 0.7850127842996536, 0.8304597280900325, 0.17998468973259646, 0.23019826541356542, 0.27803349901971885, 0.07085881981550034, 0.89297401081778, 0.34125614615029665, 0.3911119236207792, 1.0156042934779443, 0.37006730661819215, 0.9577468861398901, 0.6471617855375328, 0.5231614760652303, 1.0025128242510268, 0.8201713344662506, 0.39571135556581033, 0.4558403695353193]], [[0.9572136094751433, 0.6398983379893886, 0.538418440914046, 0.8551001176029462, 0.72444088171419, 0.6631712125377148, 0.3979814885300663, 0.5597189966095257, 0.793388211845088, 0.5305466258964, 0.16075152089116185, 0.754622190347165, 0.4202738722383865, 0.6215660195202835, 0.40681738149254554, 0.37911652297611564, 0.5863257316846632, 0.7935547558560657, 0.885350470129165, 0.18064500952485374, 0.23008195916359422, 0.3292393425102278, 0.07038889271299628, 0.938260521501225, 0.3120736645357513, 0.361946577920625, 1.1365623038659303, 0.34143527684736674, 1.1300570149474203, 0.5860347725374244, 0.5493382922162842, 0.9990053813499166, 0.8792840353789131, 0.47935418265171287, 0.5610039514952243]], [[0.9855974247044795, 0.5910386745353016, 0.5315614485394249, 0.9613715193842196, 0.7335687503930209, 0.6924338335424511, 0.3972737780776082, 0.5947496502790697, 0.8008092920675108, 0.6375025656612325, 0.16856819625020997, 0.7519998550325578, 0.41444877386231915, 0.559002050450917, 0.43568960069564433, 0.3518983299749146, 0.6034927253992808, 0.7684280815486937, 0.8273839156661262, 0.19114487534867397, 0.2287715293099713, 0.29619338859020755, 0.0754037832205884, 0.8454815441773379, 0.24399504495245106, 0.32834538541381714, 1.0715131522464347, 0.28248726072628016, 0.8911468252359286, 0.5739149765044287, 0.5232274772733954, 1.0342804769536857, 0.8279263733926125, 0.5967914521795897, 0.5373375674871101]], [[1.0494431563439712, 0.5511273931014173, 0.5450822131165791, 0.9489210634726974, 0.7378412568491789, 0.6832261082774344, 0.3955418286936707, 0.711715355311132, 0.8284007931746913, 0.7815359914811526, 0.18199025827050294, 0.8314180124493785, 0.5872983497412361, 0.5714698479501625, 0.4584247745945206, 0.38035336868040204, 0.5803039325153991, 0.7609598520826842, 0.8117047558886743, 0.2144031849123238, 0.22359731350582762, 0.30719241208003745, 0.08232856764623592, 0.6828049898086661, 0.1646126298305865, 0.2895401281580542, 0.9027492622655059, 0.26750322387306796, 0.4955039510133975, 0.6008772365058548, 0.5091497864058807, 1.1363580615501634, 0.7186906593583436, 0.6672493385821425, 0.504042986432345]], [[1.1091119335600572, 0.5321546236479935, 0.555810248335908, 0.9461461836874232, 0.7617344284865823, 0.6673298749347077, 0.3763354951860979, 0.7838204819550689, 0.815769504128762, 0.8565117181327645, 0.1861998465673589, 0.8034260898278159, 0.5360174897518362, 0.5694452616887986, 0.44600118921942455, 0.3699506256752616, 0.5823333139695543, 0.8394121118499789, 0.8058657493574986, 0.20026642911350018, 0.2288381082082937, 0.26027197027835175, 0.08355740074246577, 0.5336761080039133, 0.1759689473217014, 0.251413455434296, 0.8178562005690972, 0.2868483298489799, 0.28497905929568945, 0.5485469342163585, 0.49672561173514096, 1.1885188402753015, 0.5030248962599873, 0.7922593769695061, 0.44302746824313477]], [[1.0753934844931192, 0.5477422402595351, 0.48833459258645406, 0.9512831408618586, 0.7789089152692281, 0.6434249485967432, 0.41539060077074585, 0.817996130389162, 0.7998005234256962, 0.9804808024539162, 0.2094649855169607, 0.7967622611175227, 0.6045797450298955, 0.5838192564433976, 0.4337770465316235, 0.3813460264108013, 0.5662215370539243, 0.8865429141653565, 0.8578575370157309, 0.2321857178003101, 0.245655987617672, 0.26864912931663665, 0.08172042706013295, 0.3475427546780798, 0.37265471772589004, 0.2099333633032449, 0.7004578379132508, 0.35690320924939123, 0.13995545175945234, 0.45161945247785273, 0.4789320129673475, 1.1685831418507795, 0.30895889673496046, 0.9881093097793434, 0.3622012828356521]], [[1.0687697321197247, 0.5244404789163961, 0.4422364745571551, 0.9250488118867193, 0.7890594714049526, 0.6747447938790712, 0.3975769027562736, 0.9295363962117887, 0.9644304552744175, 1.0950783536592454, 0.2272445737569923, 0.7885342916358133, 0.6503646099106088, 0.5809672375767656, 0.4457872635714949, 0.3803407291269315, 0.5593867955744121, 0.9006266962878613, 0.8391981809810535, 0.2432523582824076, 0.23219501522605546, 0.2734170946884209, 0.07761365072765747, 0.3106148605303068, 0.6213937535444142, 0.22517931089041182, 0.7415867858146737, 0.42948477772648985, 0.11554282011685113, 0.40062961709722156, 0.47579566549509167, 1.1242497103068192, 0.3034013515771711, 1.0517045579585578, 0.30146957285157183]], [[1.064020022011618, 0.5118331323321084, 0.4233182244989001, 0.8874645536212624, 0.8567426842918398, 0.6929365933643206, 0.3862247794967868, 1.0221996755144815, 1.153444791849897, 1.2949268754409649, 0.20283214736674565, 0.76835769520805, 0.6589336107495778, 0.5835347448165964, 0.46528073321897956, 0.38421711065012365, 0.5486014153448466, 0.887286218624056, 0.8246122028289589, 0.28949401199952784, 0.24150549055132386, 0.2816872894461312, 0.07638173008730088, 0.2951727896954184, 0.8587724778871251, 0.25179568055449136, 0.6719009228899511, 0.5469398938030723, 0.10063605330853431, 0.29972613466090825, 0.4657566623719659, 1.1375908792116052, 0.47630450626944604, 1.0879742299956496, 0.3966422796981397]], [[1.0377783118150872, 0.620804882080104, 0.5379945500512929, 0.8407068342143829, 0.8123775434055014, 0.7241396736840371, 0.39065200364294694, 1.0141546361597387, 1.1424517173843243, 1.2698094797157733, 0.21171479630589052, 0.7826664170293198, 0.6285789990307321, 0.5552150570229125, 0.4722857777478861, 0.3507692377029676, 0.517870838452045, 0.8830799067712782, 0.8846853358125797, 0.32459105698729, 0.271791523449019, 0.2995937441608274, 0.0736847477092539, 0.30535009828339765, 1.0225727226604748, 0.29266052979573876, 0.6225411888299958, 0.5977093084271882, 0.12177761372151052, 0.2692317434648747, 0.5279830290135361, 1.0169582603199312, 0.5180809840714945, 1.1197888283692545, 0.39148312516742545]], [[1.2517700767539996, 0.6324810768171245, 0.5221146437461434, 0.8162485961695258, 0.7606825400553355, 0.709292302415079, 0.40733894175495833, 0.9572391395237827, 1.0372320982360133, 1.2040095858284687, 0.23946901332674442, 0.7735588443642687, 0.5822648302599922, 0.5620849476920872, 0.48724664434134374, 0.3449427465192446, 0.5223321336940543, 0.8973760800737032, 0.9421010698861166, 0.34107544949785285, 0.23945873870204726, 0.28458008024844755, 0.07162164088756605, 0.35420788788034807, 1.1705157155249282, 0.2946603084285095, 0.5526298256235138, 0.6103931751522874, 0.1556237030882396, 0.2710233724280495, 0.7068987320851058, 0.8951151762994591, 0.5700516320199971, 1.2014465789471893, 0.4653429585928212]], [[1.1623592598552182, 0.6460125147672802, 0.5432927269931711, 0.7580353244277911, 0.7314872174022007, 0.6959993405618701, 0.40792596936989445, 0.8922228200188331, 0.8664764184264124, 1.2741842196497384, 0.2460391723127846, 0.7851104984523188, 0.5875540903323356, 0.559703384294354, 0.45346903957902407, 0.3438274228951515, 0.5177863179559242, 0.9047438998928797, 0.9521407200626915, 0.3532956001183603, 0.22572125335688958, 0.2762494970172188, 0.0748170080276245, 0.38936765009442365, 1.253948905895488, 0.2729425022071984, 0.584724660290164, 0.6155355365916494, 0.16014693061480806, 0.2880273864204746, 0.827895670873968, 0.742608080977174, 0.5674713096061219, 1.2127468240477723, 0.5483542453987518]], [[1.1031639818125107, 0.6652372637519018, 0.5430178370685259, 0.7201751570541857, 0.9878575810305916, 0.7035677792352077, 0.4086707229402793, 0.8826446343497445, 0.787629206547915, 1.500342335694639, 0.24912900768379137, 0.7876973856110638, 0.6340799867616413, 0.5482426221180692, 0.42399900970166154, 0.3467820853932325, 0.5322972985279434, 0.8833972009128579, 0.9649044746472721, 0.34779236790940116, 0.20607245361308452, 0.27804082104215644, 0.07320652869556978, 0.4745521203960849, 1.3551475522248542, 0.2544775650854701, 0.6159849398078021, 0.6159271153607209, 0.1526949530430058, 0.32433496167552933, 0.8510654827549644, 0.8041468389910631, 0.5765193440794951, 1.2116830318714702, 0.6406481547678389]], [[1.0975138512811944, 0.5556275987895649, 0.4551648073742559, 0.7344871626734825, 0.9296281747884055, 0.7053903998862566, 0.41073650051442323, 0.8560902183548651, 0.7533587196895981, 1.398586767395238, 0.23674380324386468, 0.8063787048047055, 0.6020884891627212, 0.5485779161526696, 0.42483932221381304, 0.3535290110614524, 0.5487295587980305, 0.8842765687501113, 0.93269953416777, 0.3693925183574796, 0.21231448834605796, 0.25260427098872856, 0.048043297024751457, 0.5479248476738228, 1.3950865171846578, 0.24770449882640355, 0.6804836577509834, 0.6361864084935903, 0.16849959154602953, 0.3668026585103989, 0.8515428923009599, 0.787980401696251, 0.5758806730288797, 1.2668439586838005, 0.683727415843589]], [[1.0955466774966374, 0.52300644476369, 0.442440676105052, 0.7193895304482687, 0.85345680121117, 0.7441444492192236, 0.39881529831793705, 0.8623817061230001, 0.7712478278380673, 1.347213214105612, 0.2307576228788359, 0.7725110304624807, 0.6535205206704877, 0.5356817773473828, 0.3848101736048543, 0.35167224211792314, 0.550055736866498, 0.8943487427648701, 0.8266306270615007, 0.3233726003050373, 0.21219391918052458, 0.1848226530161724, 0.0207406828404881, 0.5296412525542542, 1.3308351506724834, 0.24822203609246052, 0.7270133484278106, 0.6910975706320521, 0.21960749741676427, 0.4395073768143013, 0.8169688301584417, 0.7028811685516176, 0.544653588076734, 1.2795588367785975, 0.6572887579628098]], [[1.0160430712177728, 0.513678118302306, 0.4311688919332871, 0.7322775489047014, 0.7087955002130065, 0.7563806649504919, 0.3947844713708121, 0.8578697525086507, 0.7715635629228885, 1.2205453736812462, 0.21237511495560357, 0.808421845623381, 0.7200705633813296, 0.5209291221459486, 0.33689954951708934, 0.33908707883420786, 0.5655026140412586, 0.9317134024736657, 0.8478839867259043, 0.39485981308689927, 0.24746676160051426, 0.193780005645233, 0.028074954313299767, 0.5592734561930781, 1.350123423560114, 0.23789895603073596, 0.7475529576594313, 0.8350543671012799, 0.2882459798757658, 0.4756745483141408, 0.7317378239032282, 0.632455018072829, 0.5012890844540813, 1.2414047486964421, 0.5748097469300498]], [[0.9373548924753095, 0.5339604578497094, 0.4367244569700351, 0.7196403984480979, 0.5535270885492029, 0.6884647103399794, 0.39839975182209014, 0.876936578079625, 0.7709582270464157, 1.1321819578077008, 0.20861023195393458, 0.7810155938423946, 0.6947631307532898, 0.541771133234159, 0.25467214893025203, 0.33556239945393634, 0.5631484052603121, 0.8927940360345878, 0.8825966617538776, 0.3681028911487772, 0.24152642193439702, 0.14794245176706888, 0.011944899712381708, 0.6267500524609375, 1.3922093190846057, 0.22102106507963026, 0.7909213367721353, 0.9013810185842875, 0.3102103752927289, 0.511962974960264, 0.6867505870661739, 0.5776645386180571, 0.42154073561657956, 1.2109993280461793, 0.5429563627758766]], [[0.9073230304939239, 0.45847511794529006, 0.4275238153701091, 0.8039685896765438, 0.4895247212310775, 0.7781650839295429, 0.3980045121691592, 0.9193611332081701, 0.7989937054201824, 0.9989700977490223, 0.16527566965101026, 0.862745021151883, 0.6188052689768276, 0.5401854114402416, 0.25284864482990155, 0.34488485826163573, 0.5482631176066095, 0.8780037650877431, 0.9205666596582329, 0.38925508431535666, 0.23487498305960278, 0.16975165713737492, 0.011035950642318697, 0.4918921612673116, 1.3585123068594596, 0.2119649196674025, 0.779341379643805, 0.9084399985177627, 0.28767868359308074, 0.5538962489000694, 0.6372811688544191, 0.4623448078991637, 0.4410739033691592, 1.144569528596623, 0.5529328456618008]], [[0.8793040975487896, 0.45485382967124, 0.4870217414475343, 0.8565976300644657, 0.396428790738323, 0.7659177790086447, 0.3879846825871859, 1.0005415459640028, 0.8263191338338487, 0.9026501421449952, 0.13993931958490766, 0.8437144921699462, 0.6089061945964387, 0.5456633038126885, 0.2524110623795397, 0.3152830386110071, 0.5247089018484106, 0.8520570928494494, 0.8032185123918378, 0.4967094201779699, 0.24848501539649478, 0.15103761262249057, 0.011140741946475598, 0.48476926459449526, 1.2295530653384936, 0.22917761905295464, 0.7167742639932244, 0.9953583079758522, 0.2717986447740154, 0.5418275228977182, 0.6135181307930093, 0.4005325560404839, 0.486390059474297, 1.0476924408741608, 0.5526068016239065]], [[0.7991726745290756, 0.4560614019623002, 0.43171642202737265, 0.9505365086238247, 0.38745197969871614, 0.75135494467785, 0.3821823500591204, 1.0094620467348006, 0.8685478096221754, 0.8309068150409372, 0.14953842613182494, 0.8149306913809194, 0.6098495421632886, 0.6155142243080759, 0.2297124509082758, 0.3313712762031717, 0.5334748509309141, 0.8416231504272147, 0.824241595221997, 0.45136985365619064, 0.2338057458146551, 0.14792569584730164, 0.011734065448351599, 0.4968556472975467, 1.1621406515604356, 0.2533830152860881, 0.6558659260908162, 1.0145175394238075, 0.2602449644450816, 0.5215370086771057, 0.5954567250169247, 0.3776526869338219, 0.5594913772840803, 1.005518654767057, 0.5317166765242903]], [[0.7220644350979883, 0.5379509788240943, 0.47006485245408663, 0.9701276080229769, 0.46108198272084205, 0.7651059532782942, 0.3719248115464415, 0.970322103453032, 0.8418495022711903, 0.7074687810865443, 0.08366078217672683, 0.8112253318233469, 0.6240733020115348, 0.6741409068477258, 0.23599520748890299, 0.32417261624997395, 0.5547246791170726, 0.8834202815887664, 0.8066360101523156, 0.4016270004263065, 0.20292906662438187, 0.1417092461830964, 0.008928237650162317, 0.5076706859033209, 1.0291999156819673, 0.26147321132923873, 0.6777707199286747, 0.892621325231798, 0.2693919593618463, 0.5518431511543018, 0.5496016059547617, 0.39992357065222617, 0.5566710590499307, 0.921818534513964, 0.5390589564652436]], [[0.7191590007734888, 0.551088035859231, 0.4781275993462816, 0.9672473321786494, 0.43735565462306597, 0.7632800669571491, 0.37334660542485704, 0.9054921058483991, 0.9181133590230016, 0.5979693658192018, 0.0786902847558061, 0.8241482844676913, 0.6171363772384635, 0.6178591429360021, 0.2546494789237769, 0.34049799651812485, 0.5761402879985649, 0.8864844071105222, 0.7965802763926277, 0.43763219259665476, 0.21937039685145057, 0.13291085572091405, 0.009368540068607802, 0.5337978131702859, 0.920558089245463, 0.2670575698743073, 0.6814480828164091, 0.959638035562745, 0.27726628352536586, 0.5920574016123148, 0.5999706508259904, 0.46272278376866405, 0.5767785483980892, 0.8894870847438736, 0.6838775612757193]], [[0.6254244476277777, 0.5701515426958026, 0.49938067821433374, 0.9849235844484951, 0.5124242173110461, 0.8798943379218807, 0.35802998036367567, 0.9484386930975918, 0.981355694399429, 0.6208523121678893, 0.08873856498864108, 0.7675274535214891, 0.6321639479190839, 0.5836862155001834, 0.25862130792234084, 0.33376129054118553, 0.5698381806535231, 0.9488133207845828, 0.7710317510126103, 0.3939016278947281, 0.2176096641682979, 0.1658529747308484, 0.008264791233553219, 0.5705148965705531, 0.85056270825559, 0.26822222729711864, 0.631277081963169, 1.019373898874361, 0.2930832591090424, 0.6573813523471822, 0.6586417626250218, 0.5140914434252474, 0.5653278878138149, 0.8509640462469217, 0.8558889115683443]], [[0.632483609759985, 0.6179763841289551, 0.5229522524190454, 1.1091934590302053, 0.30709631261589054, 0.8020938298760263, 0.35872325604782296, 0.9648652153742117, 1.0105857702740866, 0.6527812730080348, 0.09402344759822646, 0.747825380618875, 0.6467056981405723, 0.595437096106904, 0.32951676524542484, 0.32874484203328597, 0.6186400589002027, 0.9116627940000019, 0.8021313059070097, 0.43249382721727686, 0.21296958857324375, 0.17607660914182022, 0.010194148665012321, 0.5689038944191038, 0.7819790571767558, 0.26928973635585807, 0.6525660019227364, 1.016055394127001, 0.2901901671343822, 0.706156873500267, 0.7236416510678446, 0.5461228054653289, 0.5906001246857282, 0.8642135611367476, 0.9988064370503525]], [[0.620744182886021, 0.6211472052926557, 0.5770630931064611, 1.2169467443320685, 0.3059839128776251, 0.7414158123450751, 0.45959501986274093, 0.9661245250587072, 0.9621566913683703, 0.6571996834606337, 0.07546235828886193, 0.7658325420138765, 0.6231934127182965, 0.505875856574123, 0.2971386823035724, 0.36218034668441057, 0.6264715670482213, 0.8912902404170584, 0.7235833758327859, 0.44399118172682916, 0.21975377316795414, 0.28392617739848214, 0.013027190293850901, 0.5643468414913471, 0.7344327352983901, 0.29284740358852707, 0.6605738056510695, 0.9658013181120264, 0.30305136873660066, 0.727826980688767, 0.7279298432510433, 0.5794358946876931, 0.6978440545225255, 0.8448987216893181, 1.0327469639054891]], [[0.6124898233137601, 0.5980739408262976, 0.5790107852734868, 1.3426228438096126, 0.36698058414599305, 0.7659845846447668, 0.44953649101899995, 0.9441788151545667, 0.9110167043252773, 0.6737464074429165, 0.10776452858965863, 0.8374811321150184, 0.6642371409776118, 0.4911399290626612, 0.30765440267164174, 0.3932196600597416, 0.5431193769106704, 0.851357747998273, 0.7358987262434515, 0.4425391923609866, 0.2376328440838998, 0.27370642369318454, 0.0178459040190853, 0.5650247681482085, 0.6967094894042126, 0.2868944530111478, 0.6325934420932657, 0.9807097180483875, 0.3262612964501149, 0.7771635954401108, 0.6784007114146401, 0.6247082509841999, 0.8033610101104951, 0.8390967672477505, 1.061030118308865]], [[0.7668873741255808, 0.6857257113067048, 0.648294959530838, 1.3595239789629547, 0.4159477171609363, 0.776725316338952, 0.399042111459404, 0.9181644477951351, 0.8859630799298978, 0.7270673124111634, 0.14403651577313675, 0.8564825460991161, 0.6182968618893656, 0.48871541333759855, 0.2809128487558043, 0.35926779527259356, 0.5301246410402378, 0.7610029922265389, 0.7337552475137361, 0.47851680296129373, 0.22590377271993778, 0.27524420145408957, 0.017699102299979206, 0.6487660106752509, 0.6901662989230513, 0.2605448003594579, 0.5928635569606533, 1.012740621854216, 0.3742110407957491, 0.816859439047164, 0.6376325374080962, 0.6573910677994819, 0.9542323190640329, 0.8448975091780442, 1.073169194738159]], [[0.7864793794428242, 0.6954499316379934, 0.6767154945151885, 1.4106808641837356, 0.4803914661671115, 0.7993971722676918, 0.4023691511550517, 0.8424137878624238, 0.860064661605502, 0.7816133699877783, 0.1761877873032846, 0.8407612406541061, 0.5962362328488973, 0.49902899912135945, 0.30089130082603927, 0.35309348247422917, 0.5918872866381837, 0.7370584050500116, 0.7734829710124091, 0.4478953192778522, 0.2048606423075916, 0.2792397003593779, 0.0344742239525426, 0.6147540176358764, 0.6709540186717103, 0.2560962671558174, 0.5085141119332739, 0.9163997926156472, 0.3905990534808207, 0.8623622961706146, 0.5825266484874134, 0.6387474375316862, 1.0524237360058748, 0.8346582164461225, 1.0836036655812316]], [[0.8704452907246949, 0.6946933734217197, 0.669499329179014, 1.5432566083528083, 0.5439514931265175, 0.7641380034428024, 0.48576728602691205, 0.7439164178436244, 0.9184336933448201, 0.8482718686645818, 0.1795013115446037, 0.8753983855059011, 0.6034037552350019, 0.5009309586311924, 0.24589893212405017, 0.3557619262062133, 0.5999560471385469, 0.7408937803033244, 0.7767235146885442, 0.452422170688716, 0.19014501654058885, 0.27011144152852495, 0.08242367336324116, 0.525296893301616, 0.6810721570706231, 0.2416532740761716, 0.39548325470309637, 0.7751455167460974, 0.4060533550302411, 0.9279603056439881, 0.6394218345239402, 0.6162912427071932, 1.0988628253985235, 0.7887576887604797, 1.1357025116621036]], [[0.927018154031872, 0.6401055011028284, 0.6036214401788385, 1.6896069042129322, 0.6025685064660713, 0.744985478696863, 0.5126477652286727, 0.6309625136474802, 0.923963209046934, 0.8565516407723386, 0.18133408175828794, 0.6979988275996121, 0.6043104748326973, 0.44261835871345767, 0.2584238040101042, 0.36561835492023087, 0.5961405765797616, 0.783701206985488, 0.7667734700920158, 0.4379356464872761, 0.25821830968673953, 0.2656043739576245, 0.013149049095324375, 0.5207075411435564, 0.6321702270374134, 0.23760912999702377, 0.30564197371712043, 0.6791290965581143, 0.4143756434693704, 0.9756017439400884, 0.7062539123795858, 0.5711267977601373, 1.1362329589031894, 0.7648209543564157, 1.1749595005346747]], [[0.9435929977746866, 0.6361710621074805, 0.5842773898389944, 1.6430337916026674, 0.6842157916293954, 0.7851809437323853, 0.5518843884442874, 0.526995279468044, 0.8240913384189925, 0.8935694669450611, 0.16684772017014482, 0.6542362599074987, 0.7174160842769126, 0.4482408047445148, 0.33551910603202423, 0.3810991759666559, 0.5618907084427398, 0.8370392923325085, 0.7831533322131745, 0.47075146011602453, 0.37170494379417907, 0.23077496361828745, 0.012643853666332215, 0.4694453787054557, 0.7208971044268442, 0.2697582279920337, 0.23521049058259436, 0.6672945094654854, 0.4222562596618098, 0.924785087691056, 0.7225969061671402, 0.5579752914253828, 1.1836945080767372, 0.8573673206169278, 1.230761282239856]], [[0.9233955787575072, 0.7254778607278664, 0.691019425909829, 1.4050901907491713, 0.8061011616115303, 0.7999477856051909, 0.5385038265633878, 0.42789694976428533, 0.7182822991949906, 0.9090235743205309, 0.1616652688321828, 0.6032641389826439, 0.7657165396120346, 0.45064132839594984, 0.27402495062597015, 0.36133791642698215, 0.5465941019223196, 0.87532713891071, 0.7950830928718653, 0.5307378584879376, 0.46260289576280117, 0.22206184376594282, 0.01147090701101244, 0.4033006803546082, 0.8562866741243428, 0.3389079717662119, 0.26155613538096933, 0.670318334085567, 0.48237088947098805, 0.8931282425906193, 0.7412659681811957, 0.6041188260473492, 1.2677894770817049, 0.9230434276599376, 1.2334659859859909]], [[0.9045442299107948, 0.72029042846741, 0.680766679684619, 1.18040397388923, 0.9263409372153455, 0.781958705641052, 0.5361147304123219, 0.4805699358688008, 0.6677462372479281, 0.959758171836465, 0.16353965069091217, 0.5847931433985571, 1.0101794992147841, 0.48054021779624867, 0.36241616816252814, 0.33710665752143265, 0.6792416206995231, 0.86271899111187, 0.833379378060099, 0.39218032023716887, 0.38389002465052385, 0.18730422775483135, 0.010216401931970317, 0.3919980736979843, 0.9222840477133836, 0.37733749270529776, 0.3015281695945684, 0.5478156110721484, 0.5858900173329922, 0.7932556226483257, 0.6593482838289729, 0.6076924219094768, 1.255389514518308, 0.964936687343745, 1.2890996275063438]], [[0.8961482623601822, 0.6625563709644022, 0.6370641455251989, 0.9935099411245745, 1.0276500057237004, 0.7223291204703748, 0.569494182281388, 0.5025331527589844, 0.679423760378578, 0.9669279776493415, 0.1785735181583196, 0.7208302771466377, 1.0606441960859132, 0.5472106121905882, 0.47666134449084613, 0.3534953539970925, 0.764697472694758, 0.8276881235135234, 0.9023144551316336, 0.27560166872309527, 0.31018229631375627, 0.19265037263531104, 0.008316340348781606, 0.37902327241101735, 1.0160116101044212, 0.4257888647882848, 0.396520140418529, 0.5817577281570119, 0.7838761404291104, 0.6065474459767858, 0.5827511236871517, 0.6357526400429176, 1.2360917475488384, 0.9593285475496466, 1.263261209763739]], [[0.8654731599198885, 0.6771264387244361, 0.616381571828015, 0.9478746052872739, 1.1294996174624634, 0.7072666288215078, 0.5721122000750718, 0.5089171119097973, 0.7646257723576017, 0.9652916242102583, 0.19054172999748625, 0.6397921077464579, 1.0641392054793484, 0.6316164348668507, 0.5783572223082947, 0.38573936268485476, 0.8035657111391271, 0.7844106233078085, 0.9887515417485127, 0.41785131735813297, 0.3290506294098953, 0.19803913798616177, 0.008435335023423418, 0.3587062883884138, 1.086093767819066, 0.5643344721069166, 0.45492395035089683, 0.6342422402873784, 0.9159675370432651, 0.6420267590972922, 0.5779658597691992, 0.6956690516553519, 1.2194744661012187, 0.9730442686189826, 1.265856345072585]], [[0.8530430309730617, 0.6363116592260781, 0.54186115496623, 0.9393075955482043, 1.2407331107667265, 0.8989417449470365, 0.6801993989913275, 0.5192670850493203, 0.8665845740172602, 0.9132632343403266, 0.19959848239778594, 0.644067354630842, 1.065942100602129, 0.625719574532923, 0.6533307592462628, 0.536725192156985, 0.7508841440332519, 0.8698855330896742, 1.0253649561205491, 0.4269653454634479, 0.3079343375940365, 0.20857841119960463, 0.008775022525610052, 0.31598629067207773, 1.0676195448997732, 0.6416566382213295, 0.45141418486668444, 0.6080070290765701, 1.0061219738125555, 0.7424413172701927, 0.6471902342715479, 0.8190847755921671, 1.172209225964874, 1.0055339341529108, 1.25226886026998]], [[0.9407009894468675, 0.6761871695300379, 0.49919418366301027, 0.8645858162733706, 1.3235622963775633, 1.1014847606364178, 0.8185941399067049, 0.5288702936361818, 0.8103768031154633, 0.8787882284350342, 0.20113148926723778, 0.850944769266156, 1.075736066410572, 0.6694182171826484, 0.7480530185600774, 0.5601146401425605, 0.893085696636455, 1.0215012695925856, 0.9274547706906386, 0.44235275221076464, 0.3767383930521342, 0.19673543086803202, 0.0090056304479786, 0.290960536247345, 1.0299289820898325, 0.6091572210712124, 0.41057402723378833, 0.5699554249751633, 0.9445725074411248, 0.7823488322718399, 0.7297099746635414, 0.8125087814557272, 1.0733605347893005, 1.1674729135836417, 1.2432002133318631]], [[0.9863568556911164, 0.6949208936258882, 0.5078342581213833, 0.8338423308571641, 1.4148910213234238, 1.1027913940200056, 0.8138877059920084, 0.516466745142699, 0.9017044956322728, 0.7731664800699368, 0.20301476331372276, 0.9180087252488492, 1.0668503953736532, 0.7291706177067732, 0.5732584741449442, 0.5285403533271406, 0.9377329621844733, 0.9726050672651086, 0.9676297007709674, 0.394869430010375, 0.3862945643702432, 0.19014201913870793, 0.009762596925872988, 0.3135282102424014, 0.9933500666541726, 0.5512324232598289, 0.3876616739340851, 0.5225458764531584, 0.7989262292131325, 0.6684450686893381, 0.7402560627694479, 0.8298205829125018, 1.0009151215026637, 1.3151485347452736, 1.2394079611403441]], [[1.0060394446356846, 0.6815681737131964, 0.4968529664691256, 0.8366948006681428, 1.4593175222297368, 1.113080548755789, 0.8231536010001471, 0.5103213336804125, 0.9119790395382495, 0.6944545893599525, 0.2342998997196712, 0.7657617110806718, 1.0440879388532933, 0.6177934639835833, 0.4798197599302011, 0.5392890543084854, 0.8724259550722464, 0.9967969602901479, 0.9109476883090615, 0.40097487957889455, 0.40602185412312747, 0.18992876754220595, 0.00902080881280527, 0.3231083895725146, 0.8976849117600009, 0.487214672395477, 0.3821424838560321, 0.5111683354953724, 0.668143746391456, 0.5193987145601919, 0.7594309197383188, 0.8337616039027558, 0.9095831896709177, 1.3239144781902448, 1.2347289705830764]], [[1.0330722789141769, 0.694958709636362, 0.49226586563532226, 0.9139555945195753, 1.4874163294352432, 1.046861667497762, 0.7613585683835397, 0.5331591227492045, 0.9382301978437126, 0.6938208814195788, 0.23896654751563773, 0.6778299417192117, 0.9813670311572441, 0.5706687072570756, 0.5309846636520512, 0.5642732990719078, 0.7254508557071019, 0.9947568867876809, 0.9023251886138178, 0.4232095196500951, 0.4364807315634276, 0.18939624371410801, 0.012368880415937361, 0.3370708876578311, 0.77667306535243, 0.4043117012537533, 0.40398893496962585, 0.48048399034934475, 0.5687705373001101, 0.3367959715562359, 0.7535805398051355, 0.7961560117522534, 0.7960753494897115, 1.3605070520707745, 1.2531271887287114]], [[1.0621826732326165, 0.6886141344255248, 0.48863103100919303, 0.774237064012079, 1.4712871049623126, 1.108618272968865, 0.7204878933113479, 0.590105515950618, 0.9661352772773117, 0.579325746246784, 0.24288016732180046, 0.6343468515850041, 0.9488281619668371, 0.5163549654418338, 0.533674364409993, 0.5659199212019388, 0.6991223629259041, 0.9898716618636, 0.8519353907578215, 0.39781974799867914, 0.4463040050146033, 0.18917673132182722, 0.028208945821489625, 0.3172082292305297, 0.6812021178784431, 0.349568152912534, 0.41952410979699994, 0.49435649214470834, 0.5006253650838381, 0.2109603981843281, 0.7589309072884665, 0.728256413170941, 0.7061825563767476, 1.3831034738622825, 1.2770000375450405]], [[1.1514543283511056, 0.6757550045810875, 0.4576459847155237, 0.9575199579294803, 1.4779255427210831, 1.1943397612794815, 0.7302189446431515, 0.5850594611261675, 1.0748817448846681, 0.4664009141764427, 0.22286519340369365, 0.5984404741439842, 0.9304281686743532, 0.5165712542378417, 0.51916158347879, 0.5775062179138037, 0.6708195940448313, 1.0685064235665123, 0.8679279381972672, 0.40295641987976993, 0.5626639565113418, 0.20727589048684159, 0.06932541577526127, 0.3082338547563672, 0.5690954801172765, 0.3070369956293464, 0.44833199951745306, 0.5214131512475774, 0.4860084193141985, 0.20118286363649884, 0.7714144882027338, 0.6655770881661146, 0.6120164599593301, 1.4053128195050832, 1.3056405097064]], [[0.7572202482679471, 0.6083814647859993, 0.4161382143406317, 0.8185496105479275, 1.2641858498580514, 1.0231601139436328, 0.6515373881160669, 0.4949012080297307, 0.7802004075074277, 0.34235431905663916, 0.09615578902181389, 0.5261054858830179, 0.6422487938394384, 0.5357432847554647, 0.6047885810188762, 0.5857880967288208, 0.6522233605591152, 0.9743402700838121, 0.8251421422847827, 0.3304138745715106, 0.41031842550168673, 0.1948170828772147, 3.4870886584447467, 0.19974931014858194, 0.4312466174585301, 0.18433759439634023, 0.39153723029120097, 0.3811779595795335, 0.27780611702831803, 0.22071288673872158, 0.5599417345145996, 0.45835242670198506, 0.33313113427026714, 0.8766741004255576, 0.8236757638160034]], [[0.679734621938364, 0.7227612993035819, 0.535506183524393, 0.9083049250877511, 1.2434719348410417, 1.0334374616061341, 0.680253572002265, 0.4821397040343132, 0.7502237455193743, 0.31710784096549943, 0.12311053136409195, 0.5922699576944717, 0.6369490239041364, 0.6320238235811961, 0.510612819912577, 0.6173488857020929, 0.565052276518129, 0.9180749930993267, 0.862054459033492, 0.4156507820737554, 0.4009505047405049, 0.1999726188629194, 3.4870886584447467, 0.16875405931621862, 0.4112805100537632, 0.18703936998172443, 0.45576378786904104, 0.3173518646931964, 0.2147198412231725, 0.28259709075715245, 0.5922440436344462, 0.5542002598364277, 0.3072473610509779, 0.859131546870523, 0.7880884609150167]], [[0.6459717907668272, 0.7228640194162346, 0.5264720487011378, 0.8023279023898675, 1.1873350899387909, 0.9432397325185622, 0.5841866451817387, 0.4695835093181382, 0.6756013880759465, 0.3856849488846411, 0.1665608218432802, 0.5582524460475571, 0.6681730458813004, 0.6629002117999583, 0.4682363031038462, 0.6207264387889382, 0.5495304238635766, 0.8302618012715995, 0.8342824074639916, 0.23188710544507918, 0.28938615551866537, 0.2394606816417268, 3.4870886584447467, 0.1717532118358237, 0.38491990268646337, 0.2647893012352921, 0.48861246962811317, 0.27468999070437133, 0.1955059935924715, 0.3310603457684722, 0.6296537335243251, 0.702518789382879, 0.3334884175973216, 0.8429216097160241, 0.7809029779655212]], [[0.6195107795882143, 0.7275172959141654, 0.5406342907942581, 0.8630341248042162, 1.121956268316348, 0.9353623172338308, 0.5771514569508743, 0.5290539018933877, 0.5915525954255401, 0.43992495989327307, 0.16097482124853563, 0.570613560769486, 0.7846903891131756, 0.6623813443528932, 0.6130567331203376, 0.587787618125638, 0.6238753486589732, 0.9745164829905508, 0.7938135169177045, 0.20624083994145576, 0.33755796084768397, 0.26538688961367113, 3.4870886584447467, 0.22764438476380164, 0.37101613552960727, 0.3259570314963902, 0.5005265205907732, 0.23374396085984125, 0.18201728867655648, 0.3302548270963931, 0.6547667408560253, 0.805563742068455, 0.3931171346579067, 0.764706120776914, 0.7549266300086075]], [[0.6139701227931478, 0.7284480528225414, 0.5470932262029763, 0.8005903878499487, 1.0326138295105787, 0.9541448262741795, 0.5642374321891532, 0.6095505466894755, 0.47824470223117826, 0.47821239473000854, 0.21316465862776157, 0.5777742243159462, 0.7873391632864937, 0.6236011153836535, 0.648275731468922, 0.5848612235045957, 0.6055226246759945, 0.9776750842063658, 0.7604292174727475, 0.2243647815517495, 0.3594228683339509, 0.25974144556101053, 3.4870886584447467, 0.2784982568346539, 0.3594218567647115, 0.3440574660154608, 0.4728233785817998, 0.2077045264050092, 0.17609165483774722, 0.34917594850520867, 0.6524283737574381, 0.8587541916215089, 0.45121571129514115, 0.7072857807358264, 0.7090225069497074]], [[0.5672046809945132, 0.6583826776988311, 0.48353839123312176, 0.680081423366879, 0.9574941086666725, 0.9464341129045833, 0.6405673077658751, 0.5382044510923074, 0.41429637344624987, 0.43485881512895735, 0.19480886105141584, 0.5936519975912194, 0.7684416957776179, 0.6049282559666451, 0.5765711125577704, 0.5580469870787234, 0.604183793863648, 0.9094378931830328, 0.8167885854182644, 0.23560312528164917, 0.37039098647958424, 0.24974712230850848, 3.4870886584447467, 0.3049888571632732, 0.4094660737494292, 0.34774548768744185, 0.40844510451036553, 0.23839961044568642, 0.16403421145044617, 0.34904498423735275, 0.6014037474186913, 0.8247098202320421, 0.5207166114521956, 0.5944560162687555, 0.6355600996676694]], [[0.46658149071078836, 0.6536754172065075, 0.4700564325129975, 0.7151300636711221, 0.8695988111275545, 0.947207479932364, 0.6116150491042973, 0.507555010179063, 0.36579199279296876, 0.4096846085983039, 0.22632136067120584, 0.5884438378756899, 0.5548516552670377, 0.5949957255709811, 0.5431703075696981, 0.5836669562494654, 0.5385193520107473, 0.9002765232171845, 0.7177448914794876, 0.2042906570464546, 0.3743400464132163, 0.23825854122216317, 3.4870886584447467, 0.31968369869183055, 0.5448306896944508, 0.33840462674599187, 0.39863419041155224, 0.3218000670774433, 0.16259961811776522, 0.3489471255256231, 0.5173056488955825, 0.7344137833394202, 0.5340122368618383, 0.4480576105179692, 0.5201119879996161]], [[0.4280109782792319, 0.6174803476137318, 0.46615238279105375, 0.7530093900156137, 0.7900793090598082, 0.8925436554769877, 0.5496955605002432, 0.54092169994915, 0.3542018143415326, 0.41958769044937605, 0.2205489529337826, 0.5826860776188442, 0.42913989294688093, 0.5678003848112083, 0.5819380899054168, 0.5949960286767587, 0.5331648402410796, 0.9141678311061434, 0.7417392893802901, 0.17592820533102815, 0.36708166087004, 0.25176579320863673, 3.4870886584447467, 0.2950906170129901, 0.6606399255232064, 0.33239574977411496, 0.3602935724829182, 0.422847311743589, 0.15551618163668351, 0.30663285191733874, 0.42149738581270013, 0.6245686741989475, 0.5308604347399498, 0.2889846256752202, 0.45471635234375574]], [[0.3817131422968494, 0.6487465588065238, 0.5194354785445584, 0.7666184515283607, 0.7705114793158362, 0.823601314841816, 0.5187687038870975, 0.5244910126615437, 0.3624403878974994, 0.44857952237137666, 0.15375720223875025, 0.6007569757293367, 0.23392764461553106, 0.5627060828491601, 0.4899798417781728, 0.5391016671339417, 0.5471036907203776, 0.9353616260209233, 0.7158384072146595, 0.1613204434228105, 0.37026588623109846, 0.2739378221633457, 3.4870886584447467, 0.31942382135627057, 0.7662726057288612, 0.31379722627416246, 0.3535139237731612, 0.48584276945241833, 0.17598312405112881, 0.3197556299936857, 0.31750200632304476, 0.47843782661840634, 0.4762737506724961, 0.16905474829357434, 0.5232211534899749]], [[0.330906909931011, 0.600837547754451, 0.473946827684089, 0.7872120828154454, 0.686623528721612, 0.7948786809302961, 0.549561901567688, 0.4832060748418643, 0.3394504378083376, 0.4317866899018861, 0.12428946306396355, 0.5832901725331796, 0.5201192458654504, 0.5739097197139972, 0.45377295307040527, 0.6079523770440804, 0.5497330821907248, 0.8972728184909016, 0.7350496494032046, 0.14756948650464652, 0.35479272165887354, 0.25119443658890817, 3.4870886584447467, 0.3053208555140141, 0.8574423069248097, 0.3903429805604866, 0.33490257568908327, 0.6131649335003295, 0.24846446795140598, 0.3357413015458054, 0.26782304697339454, 0.4237753913651173, 0.262715904307795, 0.10831524139325086, 0.5626390510687522]], [[0.3145938330426047, 0.5315487917782019, 0.4345763905486677, 0.6930656360632363, 0.617961290912757, 0.7214896753474574, 0.3879629733554068, 0.5589136034577675, 0.32198872556847996, 0.44827153694889244, 0.1229637571535993, 0.5764337561497448, 0.8042205485520071, 0.610957152470261, 0.4699267281684408, 0.5876781094897928, 0.5622043253044295, 0.8875269712000908, 0.7962244355472428, 0.14239944016490436, 0.3805687497733373, 0.24269212530775006, 3.4870886584447467, 0.30205835846038875, 0.9403730028666943, 0.43362004002265475, 0.32799937623745623, 0.6851375177543806, 0.2459368228796541, 0.34693587321283226, 0.35639195672934637, 0.43326493661316656, 0.20055908009362083, 0.15337772651718853, 0.5412362470510416]], [[0.29772534281971835, 0.4580146824139635, 0.4164628872877042, 0.6641694433962482, 0.5901294958859779, 0.7321162816982678, 0.3920178509526479, 0.6291667179315501, 0.28121864673381614, 0.4733097908227252, 0.1115429321483328, 0.5809715736667358, 0.8050689243502525, 0.6329665722053942, 0.383053789272698, 0.42439391379488434, 0.5849256018627954, 0.7912031571000793, 0.8278547203716662, 0.13390730326620792, 0.3869201267085425, 0.23587780143441817, 3.4870886584447467, 0.33196507690375493, 0.9775866652793713, 0.38188123491366877, 0.2603566662916924, 0.6865339836956028, 0.2396082211184721, 0.364096811739403, 0.37474673903803213, 0.40844801773021266, 0.16916513750632972, 0.19859665116769243, 0.5148193259303357]], [[0.30009010859820245, 0.4569315289440652, 0.4430357802060184, 0.6102994426903068, 0.5720639963929841, 0.745977070485778, 0.3900248793644197, 0.6007869328750084, 0.28726733581494757, 0.48616153475179447, 0.10822740864125388, 0.5890667455734921, 0.8151311601564434, 0.6457203980235378, 0.36426957108373575, 0.3785978050102009, 0.5468601676196981, 0.7628629413526895, 0.8774696961761617, 0.10749588964942425, 0.3577031768735819, 0.21759064444687176, 3.4870886584447467, 0.4667885835606833, 1.0024693972764642, 0.319969756239736, 0.2751241837464462, 0.753862359839359, 0.3625047502240685, 0.3677869089295452, 0.38590747108483836, 0.4130725147174865, 0.29368553609286596, 0.20357419100000362, 0.5242244615238247]], [[0.29389510978658556, 0.4994260171579934, 0.45187568125207866, 0.5525983892870638, 0.5329834429674722, 0.7938245594940895, 0.39033452408914393, 0.5744305825601324, 0.2923561303595075, 0.4668388304654497, 0.10279673868326826, 0.6101539517644858, 0.8170099785673834, 0.66652873715133, 0.35983642767303, 0.33659521746155574, 0.5598541935196025, 0.7295799836507599, 0.9024452141665953, 0.10738465339676889, 0.33000449138713517, 0.21681584391350947, 3.4870886584447467, 0.6010932032054067, 1.05291949747279, 0.29633332464861856, 0.3142762209665207, 0.7446181873920665, 0.4014823208026882, 0.3451405338191033, 0.3774329653517118, 0.4234885510898595, 0.4122022213273868, 0.19037950399944, 0.5306205189939894]], [[0.43815054182929614, 0.4199543419707976, 0.4935992100799694, 0.6002275105949625, 0.7075459269193596, 0.8059306122420227, 0.380446845568151, 0.908271903747801, 0.5371814943827848, 0.5645696690779678, 0.13928923721125883, 0.593425612155678, 1.2647998610459905, 0.700506510481073, 0.391267058034168, 0.3619957309440769, 0.5964678317884288, 0.7383600639522019, 0.9991615373496637, 0.09567877598237771, 0.3580230233742886, 0.29323522390522705, 0.16129574111620168, 0.8778910207486041, 1.2642018930726944, 0.46290116732178865, 0.35070553062514503, 0.848277803783016, 0.6892714912223868, 0.5429915022606496, 0.5547229093856395, 0.5390267029389706, 0.8620786228875702, 0.2082875217090189, 0.9675253675836326]], [[0.3893839835900936, 0.3384920117783377, 0.5378642399797051, 0.48318085226576146, 0.6076413759228683, 0.888780381947273, 0.3658270823555883, 0.8406586050435655, 0.5433760478628478, 0.5048474085548735, 0.13528708789164567, 0.592822645551452, 1.2960983702511575, 0.6019647525339794, 0.3733010574375672, 0.3489246392312433, 0.6472549696044028, 0.8214928208794201, 0.9461964518281071, 0.09908251508794141, 0.38761820467449903, 0.3144528454300425, 0.06791006476684394, 0.9273157138205764, 1.2404110172919383, 0.4327239333868875, 0.3606547373085811, 0.8465729187957509, 0.7206326597106115, 0.5639475065301356, 0.5633781428680885, 0.49726107078159065, 1.0071795290843057, 0.2008850380553572, 1.0133370071281393]], [[0.4020208668463105, 0.3803898388213023, 0.6375491240565372, 0.40314160046209224, 0.5635106565719323, 1.1009504391877658, 0.3614671599125431, 0.7724582673444735, 0.5952489594250222, 0.4551253207374292, 0.12828967561304128, 0.5642909018483145, 1.3181392420495959, 0.6211343582394198, 0.37638817635714505, 0.3601508770007301, 0.6425141150408593, 0.8563396069359258, 0.9187093739169567, 0.10861971197048848, 0.39886788618391195, 0.29254562840407006, 0.17406401150822326, 0.9899159656948018, 1.1748780379129566, 0.449837519774689, 0.3542433957092921, 0.8112248494044599, 0.6932581694284781, 0.5851039526187729, 0.5889884332180463, 0.5537977396402847, 1.1008596931663233, 0.20824313112024945, 1.0742180429099255]], [[0.4093094568915051, 0.351376538286844, 0.5450467313051293, 0.30494230165512726, 0.5084532000584957, 0.9275444061602621, 0.3928491216017519, 0.7346553564687983, 0.5489771159689892, 0.40641379756557683, 0.12354974797516052, 0.5726404047818299, 1.3744614462504074, 0.6195782526161571, 0.3830425876519928, 0.35815999275507615, 0.6767988809007807, 0.8441496610893479, 0.9341897540715859, 0.10284424367423191, 0.395140647918923, 0.34085477843282413, 0.14699849649677266, 1.126234216174696, 1.1249810194193601, 0.5277604706372441, 0.2845065670740129, 0.746141165117658, 0.6502666594140898, 0.6460468546410775, 0.6012313998228965, 0.5079829077391649, 1.1783951710661569, 0.18842954180101598, 1.071555056389457]], [[0.5017330914389355, 0.33084915592525843, 0.5644917437903261, 0.25576372971322975, 0.46992480561044764, 0.8920028282984012, 0.585586488440742, 0.6926915824995465, 0.5488111921018642, 0.40822161638564364, 0.12341999490428485, 0.5872183824713462, 1.438245348413337, 0.7224284095179521, 0.37137640379816905, 0.34427797898096957, 0.6983550223378902, 0.7408027095354799, 0.951801762939186, 0.097253779386216, 0.28005922838843783, 0.35058450396450114, 0.12491393731787147, 1.2019375025797563, 1.0428824098780236, 0.5438547173400233, 0.3706311542962317, 0.6830811306938722, 0.6624717059402407, 0.5886662606255872, 0.5647423152647082, 0.470959244420985, 1.1618325194447885, 0.2753387908678625, 1.1024406249036494]], [[0.9584557675220546, 0.3405724298691625, 0.5139401351768713, 0.2767419340355489, 0.4874414065237209, 0.9835092097074625, 0.5204081247900039, 0.6304132670770758, 0.5540659802494097, 0.44639974605982097, 0.15092555193832613, 0.5746483489765426, 1.4140613418207972, 0.6924096325624667, 0.36453902754785145, 0.3734729075692338, 0.7289268358999307, 0.6944693722131882, 1.0705830909225302, 0.09713615464922512, 0.2206170215631481, 0.41602672462248236, 0.11893393109710156, 1.289236284767814, 0.9837804352538688, 0.6124676776029174, 0.41747575985422103, 0.8048965380406546, 0.7308714788845526, 0.4563930756601415, 0.457565861770418, 0.5074791658334014, 1.121975519490445, 0.2569493732644881, 1.118068709094744]], [[0.9682361801646999, 0.3298031929936104, 0.5177238095627921, 0.3545528245940183, 0.5181234140879418, 0.9985414114359941, 0.4612835817270262, 0.5708656630317708, 0.6651987305419937, 0.4821175492729487, 0.17682528235728856, 0.5662592150262291, 1.3930921648052563, 0.7121402945255441, 0.35862887339279054, 0.3740545620937664, 0.763769311359002, 0.7026774757869549, 1.0390853840472576, 0.09910180503676747, 0.21637236900160745, 0.44343614646364715, 0.11656668098823997, 1.3279900179715678, 0.9561346925938966, 0.6078242806565675, 0.3807084189767407, 0.8956462073628632, 0.7295049628227127, 0.34872886250832014, 0.3600764698028275, 0.649107369318624, 1.1269731158321559, 0.3303517493527841, 1.1298232462151883]], [[0.9496129569423366, 0.3403779374156539, 0.4374054183523304, 0.382488146954072, 0.5254892287448444, 0.9819976775441734, 0.4693323556223644, 0.5378512381090683, 0.612081809029748, 0.5200819434036471, 0.17429800894968545, 0.5681979918654945, 1.3611758819945887, 0.6846311253471057, 0.3514176744591029, 0.37182976140356305, 0.7345080097102905, 0.7093593865002888, 0.9249439678314132, 0.0932057730117645, 0.21823722881208032, 0.45270247997795277, 0.120218307827322, 1.3427484567723202, 0.8418453028480775, 0.5932909705876569, 0.3580075669657874, 0.8673555363123107, 0.7664254613578083, 0.3052817434119379, 0.2975374826258781, 0.7276573648422414, 1.1346829456962517, 0.4929473879018787, 1.0936848386977616]], [[0.6534985238364284, 0.34957043293577705, 0.4215165045532526, 0.4102490274140636, 0.6478277720953264, 0.9608173996797695, 0.44910238562757454, 0.4973632784479689, 0.5134748174780861, 0.5434805646777319, 0.16889295349044828, 0.5898040941042838, 1.3642435888653819, 0.6808458966219526, 0.34825639454111634, 0.40904521314036213, 0.6848112679593888, 0.7059135432299801, 0.6882978272063964, 0.07386130011307204, 0.2419969267901552, 0.45809139059760007, 0.054071954449686384, 1.4136339344652646, 0.7071209530797549, 0.5579838984250948, 0.29463551249537323, 0.8498866436963013, 0.747226772521101, 0.2908589026485361, 0.30465175923997234, 0.7040513269565916, 1.1589435504293524, 0.5961903351674107, 1.0551346849955094]], [[0.6317241370891407, 0.35366952106412775, 0.39080323823215957, 0.37476145555348983, 0.5502693157431284, 1.0723891782513448, 0.5265185873642386, 0.4497267624367656, 0.5293198040700234, 0.5788395350783471, 0.1583490872440713, 0.5859777067562681, 1.3125728047630316, 0.6583219269351006, 0.34338458275677064, 0.4286388380902192, 0.7178904343759646, 0.6689532591911156, 0.7217197895805596, 0.08766458203668485, 0.235634228409715, 0.48751991549429163, 0.030044042136924586, 1.469258194848468, 0.6101568264183215, 0.5292330891169608, 0.2925839113473346, 0.7037865194011814, 0.7591045110725513, 0.27036108395549424, 0.30495977806616137, 0.6366610997900712, 1.1715446928634605, 0.47562181813626414, 1.017907741868881]], [[0.5390777376638414, 0.3614561504297489, 0.3802364420374821, 0.3794977894429497, 0.5476759665345987, 1.0259236803214193, 0.5197046649030506, 0.3886647061308643, 0.5913786721234018, 0.5868907172229689, 0.14892584523636193, 0.5653244694811167, 1.334999492008869, 0.6798830162706602, 0.35873502177625993, 0.4257252768423586, 0.7176010283322269, 0.6715211423371477, 0.8335210072793298, 0.09410620129559281, 0.218274846312226, 0.47105357749675947, 0.02833135955422079, 1.318304390895513, 0.5281065158570559, 0.4871059339037044, 0.3094365791369477, 0.7234616490581127, 0.7189398892408791, 0.26853247182453077, 0.30099241280181405, 0.4967199580571705, 1.2066120613511915, 0.43954763838393274, 1.029297546958474]], [[0.5529212169082308, 0.4054440346071138, 0.38986581323738595, 0.41463154464352703, 0.6061598395178364, 1.0025458846034403, 0.5066723487710569, 0.3613659033001588, 0.6070123082812791, 0.5532521312486745, 0.1322561533703071, 0.5683935645418839, 1.3305521174612975, 0.6531457435872866, 0.3446719791326731, 0.33358199835634017, 0.694960430038888, 0.6257998658094317, 0.8326124592816989, 0.09384321091756923, 0.24078125740292305, 0.5142723288779676, 0.09111277350762556, 1.2184360402897207, 0.4490733835958379, 0.4305463057409756, 0.29402262427095077, 0.6907611926064295, 0.7127264829869219, 0.307183990938809, 0.31215570669016973, 0.46479176537408295, 1.2135855434658878, 0.5082568606480348, 1.0676268113299747]], [[0.5995336323413653, 0.42139340636562933, 0.3865869154003357, 0.41599756129493576, 0.5436654729233619, 0.9618648420195279, 0.47800974533386675, 0.3686437100750909, 0.506186327804284, 0.48239420425967633, 0.11940351462210408, 0.5882657141593243, 1.3125161514627206, 0.7287923395307674, 0.33845292330623555, 0.32922738550616004, 0.6750363735901868, 0.6222556416704548, 0.8595669582140859, 0.06438068908696032, 0.2545729331609634, 0.5780143707651756, 0.07399692281172224, 1.0850586728917988, 0.4431388827226654, 0.42112402214097683, 0.27499982413272195, 0.5963359050344469, 0.7031006446716561, 0.359240450576613, 0.3682626198560072, 0.4762173683800655, 1.2043347608354185, 0.5441071639919806, 0.974549828615058]], [[0.646392378748045, 0.5153997601146899, 0.36600703683699104, 0.4242895435056625, 0.513972779249433, 0.9394169744000724, 0.48676081248016023, 0.4248481001521073, 0.49758535252108826, 0.5255392465065553, 0.12954300864036264, 0.5958361329877297, 1.2639461315373548, 0.5902075039750434, 0.3281207810025048, 0.31850357251301564, 0.6461548671267192, 0.6064063423140964, 0.8194989458028493, 0.06950264605702829, 0.23756651432860706, 0.5750793418425872, 0.05379257026372332, 1.039708362791214, 0.4191171377595395, 0.4061483653628611, 0.2735092393665653, 0.5656296603273593, 0.7024362103993039, 0.4336819183936614, 0.4939910299715881, 0.5170622442122838, 1.1870741126915694, 0.6275610043235129, 0.8302098200617778]], [[0.661378939447641, 0.6067866479112713, 0.3422488175020884, 0.4231001298994121, 0.5280625129688199, 0.8235420656024808, 0.44613385777803577, 0.5435431511544482, 0.6198440975046887, 0.5968285259950632, 0.1523574909057744, 0.5943304634360065, 1.1561379609845959, 0.6078966826643681, 0.3199139754935294, 0.3416016103723633, 0.6287667182550741, 0.6260287840296591, 0.8074524109100472, 0.10224384803467829, 0.2262362460411113, 0.5675854100069884, 0.0674151852277562, 1.0195603602385623, 0.4207843617698856, 0.3974482498504964, 0.26552103066837857, 0.5402488306004966, 0.7233116739320178, 0.5428249032509685, 0.6589526134978698, 0.6826163196680003, 1.152940585010289, 0.7089430620677857, 0.7772190981698899]], [[0.7958176666987121, 0.8061942031309333, 0.435864179368887, 0.4273960467215556, 0.5634700274381621, 0.760028916539232, 0.43211736324745154, 0.6538346073836155, 0.6168893595290429, 0.6357925230522563, 0.20720171356352785, 0.6077390100296572, 1.056518008704713, 0.6007269188038856, 0.32127623140364114, 0.3538599896682137, 0.6687360196487488, 0.6224457481634503, 0.8269452890164823, 0.1400769175116504, 0.19209926207378336, 0.5789040908201465, 0.04024139037330668, 0.9866031572193916, 0.44338646622324923, 0.35967302788527655, 0.28635691127482066, 0.5852403204285983, 0.7637869681202176, 0.6492685226728403, 0.7391250346304244, 0.822790273078505, 1.1237667672013552, 0.8282925376434327, 0.8470809100900885]], [[0.8816855671318111, 0.7881220654953445, 0.405411802667487, 0.4458605607845541, 0.6167794711608845, 0.7754633109640781, 0.4501393353080424, 0.5743553276331257, 0.6975298995230726, 0.6255751392393248, 0.18981064407471365, 0.6020373108882058, 1.068527550470247, 0.5594657254697013, 0.3030845345544899, 0.36645466101004365, 0.664947288715529, 0.6522856242907418, 0.7904147255787128, 0.22029915198114342, 0.20939950303351812, 0.6247975268007375, 0.04309568596764292, 0.98441903813579, 0.45887476821877343, 0.3228783668311904, 0.27912783343674613, 0.5429719226608543, 0.7833953259998916, 0.5699310968991986, 0.6515623331809915, 0.6659820296789268, 1.0962953306528047, 0.9187250500781861, 0.92379290218134]], [[0.6007474495869682, 0.5788163000350734, 0.30796903383604646, 0.5226643724263831, 0.6406925076969494, 0.7380310267366802, 0.4450429063170925, 0.540653550039025, 0.686790794857201, 0.6066725980511611, 0.18634677779918474, 0.6676070582049854, 1.1818798931524188, 0.5701482243126845, 0.30163925539494507, 0.3561928680174419, 0.7456209089737328, 0.6672053428624086, 0.8645474925950826, 0.2869604618467698, 0.29589163002271385, 0.5209907253739033, 0.043827327258290125, 1.003721226779247, 0.4871061242681697, 0.3161711317380883, 0.25630029791327863, 0.5812310218973786, 0.8238058437422526, 0.4338457037663848, 0.4744213810385177, 0.5931483826621436, 1.0749866648042925, 1.0151959462841007, 0.9717991495289479]], [[0.6453435703632656, 0.6453958820852658, 0.2710566593906944, 0.5223971777771862, 0.6419908627507313, 0.8746253888629765, 0.5519228553138524, 0.4851828165553397, 0.7180156368998191, 0.6225440577761998, 0.13340733482156686, 0.6298642512561503, 1.0353901206382496, 0.5836010251748012, 0.29193205375368003, 0.42686147296033916, 0.771790787513215, 0.7081035694288524, 0.8462086978488161, 0.23776479876030804, 0.2812968281167306, 0.4371232579905411, 0.04109540329639097, 1.0248256548231478, 0.49167131814510356, 0.3155477576815477, 0.24164934535407134, 0.5311228481060168, 0.8123997366058581, 0.411659774083642, 0.31443236810405933, 0.5899673118258778, 1.084534177662766, 1.190527246376388, 1.0072322569813053]], [[0.6004577986041986, 0.6157287860151106, 0.28321615722704724, 0.5397002138074366, 0.6423914311816804, 0.8740574120896778, 0.5386771988640425, 0.4595604553120497, 0.6906174997359287, 0.6897046599465071, 0.12894245907055615, 0.6203691974768684, 0.9442743387293386, 0.5971256512521987, 0.28991745705030325, 0.4409330904962849, 0.7713253146787908, 0.6979104691047436, 0.8493065599130394, 0.2370106022731997, 0.286917664808451, 0.4073488011130557, 0.04004114951732202, 1.0727731564704324, 0.5217467087963401, 0.32136741320792744, 0.25073256944920297, 0.5138858547397279, 0.8090957328955797, 0.39690195341595425, 0.21631068746528392, 0.657561834987453, 1.0750268085347132, 1.3633232269979243, 0.9521211870825814]], [[0.6533996062995202, 0.74331794102035, 0.28672231347632154, 0.5423698571702532, 0.6444874746690501, 0.9404064143463581, 0.5600366386349139, 0.5072129392882342, 0.7196625249592044, 0.7021150349710442, 0.10706800728865767, 0.6232377336342848, 0.8337047282582467, 0.620363982356753, 0.289099252037563, 0.4482347909476968, 0.7387524621681248, 0.7258282326061016, 0.8974394613999293, 0.20840146779614732, 0.2561840111365997, 0.41893472752070304, 0.04075706037868271, 1.1111380173445018, 0.48867894087850083, 0.33576519221675083, 0.2281658215950322, 0.4211403498757439, 0.8147754552359451, 0.4619162385575486, 0.17231491550201855, 0.7300415094018333, 1.0593123906096644, 1.3811804074226388, 0.841832286078436]], [[0.6907656836093189, 0.6068701784971112, 0.276723004492499, 0.5789419119464421, 0.6186721273641531, 0.9071240081907973, 0.5767597817060236, 0.765139153766331, 0.7166935674735331, 0.6858416707959958, 0.12836426384639715, 0.5871083474491918, 0.7092975920476041, 0.6972226019176847, 0.3022687107456357, 0.42790812236529935, 0.7053257571918967, 0.739068780770676, 0.8720530503752972, 0.19575127980114237, 0.34349889761897817, 0.4582551935878847, 0.04000320676737827, 1.1099308807568478, 0.5443838521525256, 0.37265711363776677, 0.23381156438935044, 0.3210378611270421, 0.8121186097985095, 0.5253796008120954, 0.24006902041946332, 0.795983084732796, 1.0142359094347906, 1.5474142614685782, 0.7668669535127057]], [[0.8056949111039964, 0.5682801905041983, 0.28996089310838874, 0.6068813604148162, 0.6046605580754636, 0.9345786937971774, 0.5857296337245306, 0.8498667588455561, 0.7392729416885753, 0.6732290778572245, 0.10057609343290325, 0.6212337283635433, 0.651146057629599, 0.8003271496700279, 0.29966829748493023, 0.42537258863144534, 0.6898989532553755, 0.7566517514619979, 0.8537386087693846, 0.1978579129246485, 0.2768164069266708, 0.4513892876643518, 0.033157941659840484, 1.0762052918489347, 0.5729613345785229, 0.42877587366899017, 0.25727049305018496, 0.26529502376417896, 0.8059103982103729, 0.6107933637308297, 0.31794082023269127, 0.7882695288037571, 0.9769499787615201, 1.6220937998037304, 0.7498560040722521]], [[0.8719979325235274, 0.5488162993901873, 0.31677160801220844, 0.8090948271866518, 0.5987570658335941, 0.9629226382539726, 0.6437778946418891, 0.9914126831694904, 0.7467935127191926, 0.6991289404504756, 0.08446090334122022, 0.644363936528769, 0.6743371674241989, 0.7437465338957989, 0.32377615936877624, 0.42568282684739145, 0.6603005327222013, 0.7682221079777551, 0.881847358314107, 0.20828588307979967, 0.24342560064683966, 0.3842714441956758, 0.03345909109836431, 1.0567166448143654, 0.5250380016933454, 0.4597191331774046, 0.2431817365936349, 0.23271455709777722, 0.7903062603288168, 0.5863238187976418, 0.39505957652660184, 0.7510278214725228, 0.9394806696826841, 1.7494239008388615, 0.7536865020890005]], [[0.8996842569553727, 0.5275225766224834, 0.28312766281339763, 1.0695632556029477, 0.620096250387425, 1.0006876558591389, 0.6737717353106756, 0.8872691831223074, 0.8422792307284952, 0.6844719593555143, 0.03194361217791621, 0.6443350954617351, 0.7097122814493754, 0.7570999671759171, 0.3149038763144276, 0.4261033469536709, 0.7006120755289834, 0.7481538653259747, 0.8574366189272385, 0.26728141358637114, 0.23306859350825432, 0.3292806139567526, 0.0337632758159354, 1.0093285957776041, 0.5135142676047943, 0.42633779955864487, 0.22006126555873, 0.27506774966296954, 0.7459174690424636, 0.5665220732906907, 0.3587156390243674, 0.7543133740173578, 0.8432234316171554, 1.7912525622482125, 0.7485084202945393]], [[0.9057265705088211, 0.5626564684794143, 0.2802205254009098, 1.238203558053308, 0.6737620996836833, 1.0171003192951482, 0.6687557303669978, 0.8599520353920812, 0.8853260672405366, 0.6342548501238902, 0.0336025351367193, 0.6527182353037131, 0.6501694487123129, 0.6733931587192288, 0.2939699405886288, 0.4230376751724032, 0.7089615381360495, 0.7581674924808413, 0.8268033047326102, 0.24954616289782178, 0.19480327652394042, 0.31356832664276946, 0.037435444020179856, 0.9918747718184384, 0.5004880589959383, 0.3936690746767722, 0.2268276903079514, 0.269337515425091, 0.6813002368850727, 0.5180886612017533, 0.24745734966164473, 0.743117876536391, 0.7630151401060976, 1.7672490571127915, 0.7546963084993483]], [[0.8623759101067416, 0.5216859541683447, 0.3190806498282527, 1.4889322041163122, 0.7003821128484116, 0.9732631348651613, 0.6436310425382896, 1.0617856505678742, 0.8446593773963484, 0.8694837398240746, 0.0729533900973692, 0.6567069519963354, 0.6637281761692557, 0.664548675620617, 0.299016129734597, 0.4472286658786703, 0.7056081563486636, 0.7969141307334736, 0.8322921913964929, 0.27550375568907925, 0.2077247355849598, 0.3127883081593209, 0.036843598292293084, 0.9555573875686163, 0.48027510567288867, 0.3745840136440909, 0.24305099917461973, 0.27853609455879674, 0.6049012832279561, 0.514862909874684, 0.18921130037999115, 0.7467456884862405, 0.7557007967538627, 1.709941300964768, 0.733277692456207]], [[0.8917130377045477, 0.5601274051919989, 0.3062330922796306, 1.5767483351166047, 0.7021481843416207, 0.9903887230483273, 0.6337862928302985, 1.0318704339690976, 0.8443243251335179, 1.0699879891365522, 0.05131987440653037, 0.6495220390263836, 0.707007935059919, 0.5876847473346657, 0.28998423315753696, 0.471530199556359, 0.7854194878193022, 0.7769039464137747, 0.862543727717197, 0.28474152393115304, 0.20350711499214824, 0.30360156502796976, 0.03693134472694029, 0.8783113624185521, 0.4667788278681049, 0.36456953837547973, 0.2634468546907895, 0.30730320378089937, 0.5164920836067974, 0.49607629816197224, 0.18507004845019448, 0.681708151795988, 0.7439291167024089, 1.6985413203492699, 0.7158640117560058]], [[0.8556805573497768, 0.6186339207845155, 0.3143848565991784, 1.680667711219458, 0.6712238198197407, 0.9593701822503291, 0.6180712757589659, 1.050407314162885, 1.034963691783815, 1.21897553719154, 0.039851556943730354, 0.6352509923602326, 0.7908887014171929, 0.5603370787774993, 0.28402392963862244, 0.47562333013778224, 0.8062409662798908, 0.7775419310109886, 0.8548671944427587, 0.28616937066343007, 0.19420261523722254, 0.29987971227575344, 0.039514537863697116, 0.8075994754636603, 0.4915433635845122, 0.37868772872391976, 0.2254633587964331, 0.3564125519644958, 0.4604832754288587, 0.47295858152702325, 0.21626199274312707, 0.6715194819735951, 0.7972105119476245, 1.8723202661080214, 0.7083443072164978]], [[0.9512272228432476, 0.6594379943930951, 0.33618268241589055, 1.828130706959458, 0.677249724759305, 0.960444242396426, 0.6412394234693736, 0.9511539915070844, 1.0153037032918575, 1.3782566268042578, 0.013249064186832474, 0.6694699211723946, 0.7905330632852035, 0.5814848726514256, 0.30486739526949336, 0.4920401453461495, 0.8830313622936616, 0.8346928891113679, 0.7782920272577843, 0.30707949180262656, 0.18665584910554273, 0.3208793588168912, 0.043449322169778204, 0.7524807855248621, 0.4871358693505158, 0.4155486970871725, 0.23081685680502856, 0.4085677657735357, 0.4386400721465543, 0.46548591839403053, 0.2845598376304544, 0.7228837530365841, 0.8957963567698581, 1.7080961483324093, 0.7570532683930749]], [[0.9038772899606831, 0.6820033176605171, 0.3118320519082712, 1.7820750479772585, 0.8143814487674184, 0.8679402425106775, 0.6038254387081698, 0.6955427706021791, 1.031620716338086, 1.4526787008427333, 0.061842217968528235, 0.6417677912726111, 0.7184122533120394, 0.7128322663728075, 0.29956169339532895, 0.46914361269941407, 0.8796353488151978, 0.8046671760569539, 0.7010824319565702, 0.25243545334677486, 0.26908994613632464, 0.3253720133186096, 0.04650093370042825, 0.7143600303350361, 0.4817874983880136, 0.45964301497568955, 0.25645756154109745, 0.4105406606314876, 0.42861664047008574, 0.44311528058622185, 0.42295800340606793, 0.7653044677199897, 0.9061790600471102, 1.5915546078764393, 0.6843871241003804]], [[0.889651231050269, 0.665122991323962, 0.29284045813244036, 1.7054939684190207, 0.7816174929156408, 0.8429519285274152, 0.5996765823201217, 0.5914921572914553, 1.0796266073260588, 1.3273167381930058, 0.1447483775951589, 0.7216905066546399, 0.4946756378916002, 0.6971743570937148, 0.30927861716684013, 0.46009864189329486, 0.8907749465822015, 0.7911671021232585, 0.8259368194634402, 0.2887153848191478, 0.31493961347083543, 0.3948914936253995, 0.0530860120161645, 0.6850930284997859, 0.4433365388465137, 0.39807987456913785, 0.35416867810478064, 0.427730770587549, 0.449955598950443, 0.4269108775552151, 0.5012054802229107, 0.7885960103116707, 0.9315446951785153, 1.560935724796859, 0.6662375018035285]], [[0.8947129669625696, 0.7051674460755448, 0.28493876776135146, 1.617045507824482, 0.7520711165611105, 0.7756439916851198, 0.5585682680319672, 0.6373082199565735, 1.1583695431880412, 1.1416835982955473, 0.2616342595334112, 0.7256157730712977, 0.4043317854453055, 0.6795140508206784, 0.32084468783296566, 0.497036566663036, 0.8907085055496315, 0.8111690742963948, 0.8565128663389943, 0.2902041916348088, 0.3302466573993089, 0.3819122717048751, 0.11157460172108473, 0.7050109851430395, 0.4316455186881307, 0.32921447175100865, 0.4368124637939077, 0.425812580847654, 0.4531615680400646, 0.5038813837547014, 0.5272187878185784, 0.8212157085300898, 0.936478764856, 1.62108100002406, 0.6682224682214027]], [[0.8909418207235424, 0.7208487177409454, 0.2861023388954503, 1.6110381711196984, 0.7245423184714602, 0.7832737584489419, 0.5560170906279348, 0.7520463661494527, 1.2392723458034902, 0.9807274022274947, 0.2578352824620457, 0.6380786938194446, 0.47748254377342514, 0.7227236008501601, 0.3319489467543786, 0.5007407111139329, 0.8842199755669768, 0.7874330437826103, 0.844530512568031, 0.2940545561411254, 0.3598608980482581, 0.3841895097279907, 0.036776235558464695, 0.7705364515954447, 0.42545325228587777, 0.32756129563345954, 0.412937702124058, 0.4568295159288366, 0.45025073283979916, 0.5249617589761824, 0.5413815844328239, 0.8420289018657606, 0.9274924008546823, 1.676612847009953, 0.6547905140553449]], [[0.9544814111844084, 0.6548907533112038, 0.30887467190157647, 1.6335758579158421, 0.7090939390623873, 0.6687415926153171, 0.46244539515250455, 0.731575079930047, 1.2511196952124994, 0.9719839331468904, 0.31256435887550527, 0.6045473967553637, 0.5703131895821097, 0.6411622631099506, 0.3134238397425265, 0.4429395312019079, 0.788748471804719, 0.7950254648409972, 0.974085954158428, 0.25509553701001797, 0.3638850149748033, 0.3436763422025442, 0.020814222403145705, 0.7669869206627773, 0.3791023147014505, 0.4481323844949176, 0.44740341401995654, 0.47152371962743433, 0.578419819427507, 0.5773174585302395, 0.5434854503363283, 0.8346205270664704, 0.9104145243943378, 1.7419832092404268, 0.5971505002776315]], [[0.9201240617013261, 0.7257594171411541, 0.28543134147071564, 1.5546357506353732, 0.6938322950239898, 0.649173436767768, 0.4501158200607373, 0.7338906621609795, 0.9936281577766712, 0.9536612913912108, 0.3070994731104763, 0.6365649499035488, 0.6377993424115773, 0.6369946036069063, 0.30236137486759573, 0.49023173288676014, 0.8203502550791368, 0.7106138379487159, 1.0382202093649315, 0.2009150372098017, 0.31316719732859, 0.33341492255496674, 0.019076559517908077, 0.7177782993976827, 0.43193879106914956, 0.48666166480701356, 0.4332726713067565, 0.5506600789157882, 0.7079683161538075, 0.5971218220896056, 0.5075270672053162, 0.7835292073163883, 0.9198657899897869, 1.5774874019061607, 0.5078946801605732]], [[0.9226830490103752, 0.7786479499072639, 0.282282409569461, 1.4890497509654081, 0.693315550253417, 0.6255227508428622, 0.4512717991686612, 0.7431761679144493, 0.8029496235666717, 0.9898779061899601, 0.26077505757521735, 0.51471048882517, 0.6235697704352358, 0.617666037718939, 0.2902704838406398, 0.46958113740233387, 0.7906125442621641, 0.7607080765307481, 1.0529336521482136, 0.23937492222023155, 0.3445563043273814, 0.3937056291343037, 0.023522025678667534, 0.6390655614672907, 0.38753101227123604, 0.43472555929669665, 0.4268096375918775, 0.7120434347356777, 0.6820340956902405, 0.5280754248020976, 0.4665814861648343, 0.7150549636800714, 0.9107018100632012, 1.3474787240919963, 0.4654886706483423]], [[0.871321377484301, 0.776371919442556, 0.31908898370559086, 1.4525395226484623, 0.727039891199382, 0.6083723356487709, 0.4235022084376999, 0.7270891741378144, 0.7564010883834726, 1.0064060549819045, 0.22868357375610693, 0.5087099621818361, 0.6320726114378872, 0.5886579427928091, 0.30684189725728506, 0.4645482997497298, 0.7827067815798273, 0.7464361993390214, 1.1251004571851428, 0.23007279244979817, 0.3468011353469779, 0.44521603124033415, 0.024003326357840686, 0.5142881629159954, 0.4435873964651474, 0.2793866286916048, 0.4347656592102629, 0.7106531931199116, 0.6452255220611853, 0.4883996024144755, 0.4637981656299367, 0.5902832628445901, 0.6449149229984261, 1.3620354231482743, 0.48500654990822134]], [[0.8619673574886921, 0.7289183371097125, 0.36059756827264866, 1.4604413906857785, 0.7489759689839575, 0.6136907706984411, 0.44075016377693665, 0.7392608453628854, 0.8062931835598401, 0.9297961753477761, 0.21296616218509856, 0.4806200312346577, 0.674826018252632, 0.5785711887927331, 0.3065904357563497, 0.4567418520138081, 0.7308126993217462, 0.766608323093112, 1.1971901266308091, 0.27177284304705795, 0.37412734738420245, 0.497627738339392, 0.02416742792484902, 0.3517374807075986, 0.49148054971386157, 0.20754480094794725, 0.4255878170388145, 0.7659673173256473, 0.6922132797950811, 0.4746560817746045, 0.48239379491389467, 0.5783841722498522, 0.4708900474509657, 1.314579198915951, 0.5090025118270299]], [[0.8648122261737377, 0.7170384331646296, 0.3163343519096743, 1.4595093620161428, 0.7270936354897279, 0.613101349448949, 0.4191893333989237, 0.7387449542521103, 0.8600255769700897, 0.9042200644186424, 0.21408032532821297, 0.4878947636953992, 0.6272771018197634, 0.5764451718624708, 0.29410165984861847, 0.4422248790172618, 0.704583253274589, 0.7398081833264203, 1.1400408590487534, 0.20118629533936977, 0.4233426872814421, 0.5616360319895866, 0.0233766169686784, 0.3990577043246184, 0.5382725902621497, 0.19636533544945567, 0.42539916456715915, 0.8967990326952306, 0.6675410650244754, 0.4239141733648624, 0.5448507681548327, 0.601439398501448, 0.3524214974047296, 1.3548932613432623, 0.5606901344734065]], [[0.9345819271545064, 0.724384970938448, 0.306716303868437, 1.3627399519720802, 0.7531647768577864, 0.6102835199929302, 0.4138344200531928, 0.7924809366683736, 0.8721301795047737, 0.9907867924456814, 0.21938375921448405, 0.6414420642208496, 0.6459075912612173, 0.6565788922371429, 0.31778247014854416, 0.564889584182142, 0.714554200758301, 0.723266931394579, 1.02021085632832, 0.14549560238805836, 0.31925849251888827, 0.44161991553343505, 0.02405342553862423, 0.4482124756358171, 0.606567103989307, 0.20605275304722404, 0.4654744911294413, 0.8280902936073491, 0.6510720552122937, 0.3555178553994789, 0.5668207096955191, 0.6108277180552868, 0.2842575382323338, 1.286553823488514, 0.6298804980458423]], [[0.9659757279776093, 0.702747236941871, 0.3411878093884132, 1.3140710066857764, 0.8858866984249463, 0.6250353423529467, 0.4114774692488459, 0.7331695757074334, 0.8820579222252017, 0.9921031443708551, 0.21784689341959151, 0.7523701050674452, 0.6862186958936023, 0.6033630608942935, 0.32598086705131135, 0.5790501411331394, 0.5967055470860674, 0.6418773818208126, 1.0772777525763404, 0.11398998367965124, 0.27228606855761006, 0.5823841712077638, 0.028784330520543248, 0.4311445000601673, 0.6133139340698331, 0.1759931792325145, 0.4959650221916516, 0.8037595204576591, 0.6759527192149849, 0.329493601299179, 0.5883683968236759, 0.6214710134523938, 0.3048378361630699, 1.3485373284453628, 0.6593183308440639]], [[0.9260723789580778, 0.7335066349560933, 0.34850270845749587, 1.2124750317388615, 0.9534751905613956, 0.6005248975131554, 0.40023891117099836, 0.7470708144550523, 0.840096364955064, 0.9961068826710976, 0.2129790770560158, 0.6495978279538339, 0.8088600882733492, 0.6434416373841731, 0.3295212283915061, 0.481725013618425, 0.5769267484578939, 0.6236579422725359, 0.9712322663503061, 0.14886890617045392, 0.2344361387694564, 0.5721643389323221, 0.028408073119182434, 0.43874276581067906, 0.6409534861472894, 0.15117907586776602, 0.5173995345075675, 0.798870867790715, 0.7569383824782379, 0.3586237392100988, 0.7368534917215219, 0.649820614516667, 0.3862945549244902, 1.3590761549507002, 0.6947727079094517]], [[0.9399856966841774, 0.7019497870818546, 0.31895622277996877, 1.3156176508256123, 0.8654491908002002, 0.6815034492959521, 0.4456973316467158, 0.6725732164845254, 0.7911512038292925, 0.9449477786932767, 0.20956638441222675, 0.5990437440035595, 0.9057878386005638, 0.6418713569685834, 0.4035152854805486, 0.5227184615351868, 0.5976202319524557, 0.6360326555307285, 0.9299874422393256, 0.19327928011131942, 0.23134340233806502, 0.5841869859277519, 0.028161741367797245, 0.4750070980718268, 0.7213607902206749, 0.14020065240578572, 0.4687547544383435, 0.8654877061845891, 0.869597317273582, 0.42310569692198274, 0.9750692405746824, 0.6632261930526383, 0.4612478295098843, 1.369254401845702, 0.6731598563729291]], [[0.97226464365375, 0.6792090362489045, 0.3300346190598108, 1.3574413196901463, 0.7348637186824204, 0.7519519848766041, 0.4626511702059347, 0.6601248084776614, 0.709872923466977, 0.8759594904444349, 0.20997580082540088, 0.6380254041923293, 1.0145000777242936, 0.644707796401089, 0.3621307205912213, 0.544393005630181, 0.6040811276019368, 0.6490194648072611, 0.8789824372982858, 0.16570883220586582, 0.24044431915986703, 0.5280885252526997, 0.027136916621652206, 0.5557306362307901, 0.644048945957417, 0.1403263247449795, 0.4600364513214095, 0.9437526896730624, 1.0082087268213096, 0.4599021810564813, 1.1912208912715267, 0.6638693944313072, 0.5590778585661939, 1.2630743539132039, 0.6542771460815338]], [[1.0667319384329492, 0.7310765850644161, 0.3413634880152502, 1.3869367672832105, 0.7230117216945227, 0.8102594225266028, 0.4811921485349691, 0.7130033745748037, 0.6512753831109301, 0.8709608106640199, 0.21241539019305644, 0.6178928407190478, 1.0715963859814903, 0.6645449465606793, 0.36107673866548085, 0.5733421108781523, 0.5582674669577132, 0.6587950397525721, 0.9002108865024068, 0.16354212405081728, 0.23305796806902507, 0.4016620384089616, 0.03101380424157348, 0.6109898624871413, 0.6243647656049081, 0.12018716098081866, 0.44354921473849535, 0.9864784319037307, 0.9716084588492553, 0.4853280968732882, 1.3538748124444695, 0.6729531703115593, 0.5953064229356115, 0.9961556079984315, 0.6667139861409369]], [[1.1787487655504747, 0.7420016383370788, 0.3327951677846641, 1.3414450080398708, 0.7481872489631243, 0.7900437922534993, 0.4509333342354086, 0.7449486344242304, 0.5982352921035383, 0.8490356065980754, 0.21557533016741065, 0.6278587543943364, 1.0910416133635144, 0.7448828895079949, 0.35894434909001516, 0.5937643142931615, 0.5454841599399627, 0.6704593693352634, 0.9142680973511582, 0.1648864936561242, 0.2251293038593716, 0.3308428695119042, 0.03093391755403603, 0.5845341469341682, 0.653321122376008, 0.11283722726579234, 0.4254089294253933, 1.002313346458951, 0.8943974089033486, 0.5202770383068629, 1.3330971023438134, 0.7280247669580402, 0.6094088118726411, 0.9435377644743979, 0.6385884452806961]], [[1.506425703791719, 0.7345583482381426, 0.33106733238316677, 1.25435820977024, 0.7780657571615981, 0.7765977125691058, 0.43963411081613446, 0.7992218727030389, 0.5858556097760457, 0.864236663302349, 0.2203099809033045, 0.7052724275288329, 0.9644410683361161, 0.9016221259152053, 0.3766316363909704, 0.6016300373508177, 0.6510207359286628, 0.5963529017056359, 0.9251731053183601, 0.15880408317728656, 0.22980936098260754, 0.29679105704426323, 0.03054897181685554, 0.5460732208074768, 0.879906279530881, 0.11334843914295997, 0.5367939356764129, 1.018173608198429, 0.920514284898946, 0.5733107101111947, 1.3029212079785355, 0.7390352947765773, 0.5873235942036923, 0.7585302768462832, 0.6691268147279024]], [[1.405371154362999, 0.6875050367098009, 0.4415931473740903, 1.1162731500261232, 0.7718128712865954, 0.6506785109274296, 0.40871996609345873, 0.7350333695792163, 0.5472527200216927, 0.8613216580475319, 0.21777322351606843, 0.653111335764025, 0.924623806733552, 0.9284115577322962, 0.3603961931705021, 0.6159464676466111, 0.6544167388647967, 0.6084725311750154, 0.8803636523506095, 0.15747400451656218, 0.2240503997285429, 0.3030565760343636, 0.02958943199429094, 0.4757075619956376, 0.9335997870770452, 0.09700515744567878, 0.5883367535798032, 1.022145319550695, 0.9518112204872333, 0.6073910984512989, 1.274273995737895, 0.7678162817562375, 0.586178835995524, 0.589708892466993, 0.7117201219339091]], [[1.1664933909937738, 0.6819268531616023, 0.3129578577709752, 0.9623889780907904, 0.7153435974862812, 0.6661053140931928, 0.37326049948042883, 0.6689109349951052, 0.5797492227337534, 0.8534964236610914, 0.21400165038535418, 0.6711199280827223, 1.0102450960147653, 0.8163877381215473, 0.350473378559333, 0.5680829802571487, 0.6070369703165912, 0.6116419347364812, 0.8948596467008192, 0.15685482797033745, 0.23509859260783786, 0.28989644964961997, 0.05013173941817216, 0.45502548375304364, 0.8329000245443077, 0.08676251939272235, 0.685681284930707, 1.0475571505341068, 0.9918331363919767, 0.6517923834248265, 1.2646137177641648, 0.7849402960712567, 0.5932694163575813, 0.501195329723588, 0.7542502318762445]], [[1.1122338636615712, 0.6541844852617891, 0.31514033819867837, 0.7549999058441105, 0.6618707622076645, 0.6215867332447026, 0.36322068901924065, 0.6740303565519468, 0.5197587041991995, 0.8360013217939204, 0.22090416985587924, 0.6860937053448968, 1.1044344250115132, 0.8266013298098296, 0.3267744004618346, 0.5614810646002399, 0.581047316881633, 0.5611160782089322, 0.9152143852488046, 0.16098132361810863, 0.207271549562513, 0.303577774855055, 0.06445724265654011, 0.4688754763113456, 0.766108111858143, 0.09336198851568776, 0.7555018914596023, 1.0910837787964742, 1.025109481368494, 0.6255850094268992, 1.3203657085604057, 0.8048678787006467, 0.590938480734327, 0.4271976686004761, 0.7195343053037262]], [[1.1330566194324914, 0.6175106482573322, 0.30336566887151994, 0.648090289227814, 0.6774137011466338, 0.6057025222432034, 0.3275527001761458, 0.6982497924700187, 0.4367720348221539, 0.8306398183982855, 0.24094697841961601, 0.6703747980362403, 1.1676893906766779, 0.7629372615409526, 0.3372252700571326, 0.5793836066750065, 0.6014587269537873, 0.6227790829043904, 0.8978235093045293, 0.20002460764180868, 0.19980420024375511, 0.3409595695483968, 0.048062652896399216, 0.4824907360195845, 0.7663128133562469, 0.1349963002782492, 0.7639453626146528, 1.1501633205079629, 1.0802994512798165, 0.6489573972445696, 1.343102619498943, 0.8120209125959714, 0.5401588746928049, 0.48801765731468205, 0.6457555002140083]], [[1.1174418504147499, 0.5908571433253613, 0.34740795642150546, 0.5323116525335325, 0.7345795541732154, 0.5791531142748683, 0.2948562836014893, 0.7107865837248766, 0.43379029816195747, 0.8219805675222774, 0.26898939942056155, 0.6713768161986382, 1.2625565309592741, 0.7651937002793976, 0.3574316177977095, 0.5962166508997798, 0.6149358086842736, 0.6712400679318309, 0.9517078002606807, 0.25357049464988396, 0.17925330013060717, 0.3527086262870816, 0.10422164407138484, 0.5296742969936279, 0.892732898940277, 0.1818349067601775, 0.6466657563484268, 1.2039902756481244, 1.130216720141428, 0.8509543387180708, 1.2689014988467373, 0.7773843221128568, 0.5005811643194887, 0.5893265886406632, 0.5768105459358684]], [[1.1081720372128805, 0.6265128339378058, 0.345110139580069, 0.4928461738018661, 0.8007625221953224, 0.635806393153607, 0.34564610069747553, 0.6871728029248378, 0.4075865930482323, 0.8311489338036191, 0.24195245553082748, 0.6465737482189515, 1.3241932617064736, 0.7311614200321348, 0.3924413998169679, 0.5766534098253028, 0.643445009281426, 0.6493537637590296, 0.8916503695293583, 0.3145371086141593, 0.16380094049855165, 0.3595719046491892, 0.08264938398714283, 0.5215333919954852, 1.0278169740943954, 0.2511200224880368, 0.3752149932582841, 1.2109333419776591, 1.150592918682916, 1.0517186857164809, 1.1373968148190834, 0.6823174653163653, 0.4814386425375695, 0.6320135797393331, 0.5289365544599494]], [[1.1751482426626199, 0.6680300171114041, 0.37643056597623026, 0.4489086815357064, 0.8116375761943446, 0.7175841296832286, 0.4436113940352716, 0.7746638495971825, 0.46269883639519127, 0.820126880562667, 0.22224840321491574, 0.6258596951511177, 1.272148933341798, 0.690981748678497, 0.32748424726710934, 0.5704642938248592, 0.649219650016808, 0.6317630059962629, 0.7597211550709668, 0.2227269028210384, 0.17457953408738214, 0.361722999036634, 0.07921700936021497, 0.5109193036737754, 1.0534047771542738, 0.32620425263970987, 0.34361942072811935, 1.1205035557104088, 1.2404211888160188, 1.2615223340338495, 1.0251282099104717, 0.6010756489675887, 0.5054883433144962, 0.6220490823448632, 0.551238744371891]], [[1.1902932837148974, 0.7008749406763437, 0.44443782902562345, 0.44626866186213354, 0.8095306425878732, 0.6654530300402635, 0.3663333330671741, 0.9266477992045654, 0.5097022263093456, 0.7817417429696796, 0.20605672550217308, 0.6251834706973489, 1.284507690364435, 0.6864934966538869, 0.36092813784259586, 0.5965293516673453, 0.6391299194253824, 0.6049803586806372, 0.7031474971206086, 0.18949672054653005, 0.17703961129000229, 0.33206810657767527, 0.06237118329480905, 0.6371273437666084, 1.0813098185800312, 0.2345148668986147, 0.26731831174291043, 1.0030458559011566, 1.2821618890996285, 1.2441258938353226, 0.8808532333861241, 0.5751141800711345, 0.5186068214357993, 0.5815928889794082, 0.699735179576305]], [[1.0638048677017995, 0.7369551725355379, 0.41960058582313386, 0.6001672863802336, 0.8456236377382281, 0.7454425658425466, 0.42564897807593405, 1.035478258730294, 0.5267566509412327, 0.7288455979224829, 0.18266762349912052, 0.6677320737628546, 1.3141113044703112, 0.7070145955871017, 0.31669389969038564, 0.5936501753875152, 0.5884951527343155, 0.5373722202898972, 0.6269758120778195, 0.2369136175060996, 0.2056542052469526, 0.31438920254372743, 0.057491492309650685, 0.7141022583131638, 1.1139465556692556, 0.21295201448180423, 0.22050472106240102, 0.8360914951519179, 1.0084889583053833, 1.1861385211952227, 0.8066558896595329, 0.6262164173933055, 0.5617738546364047, 0.6143047736479164, 0.809661278469228]], [[1.058246699417647, 0.7575059707503919, 0.40581511564098716, 0.6801971405900299, 0.7957595390435785, 0.7655643837427908, 0.4238190477964452, 1.1201346046063967, 0.5917943845511843, 0.6908838459043469, 0.1742934568962177, 0.6930156969092036, 1.390672494361768, 0.8261220777648295, 0.31906231660869755, 0.6061086490527577, 0.6392992152695233, 0.5612230574941912, 0.5712857907441635, 0.22209467601857927, 0.23276649433672325, 0.3014350312727192, 0.049859169085379906, 0.6678823321271778, 1.0474191177630034, 0.1741704710146054, 0.29740242655645033, 0.7219171302628903, 0.8231209231630207, 1.1064543444886723, 0.7666627476362625, 0.6589147791467675, 0.630505011550267, 0.6432489849731855, 0.8996911129966201]], [[1.0518607803549416, 0.7772542131930795, 0.34256688031657245, 0.7201632147361188, 0.7800438071964019, 0.7404069857429793, 0.3216858363008549, 1.2121069631153722, 0.6370788019774383, 0.672401028233231, 0.24710899063095734, 0.6278405206252783, 1.3932525479398112, 0.7660732045902334, 0.35274538807960243, 0.547090680521972, 0.5443683202933272, 0.5635781961633402, 0.5394442456746282, 0.2128502882911237, 0.24843532843849275, 0.3085099217979578, 0.0685539956522987, 0.6586830567201603, 0.9813473553776295, 0.1844973089387169, 0.2705803295271954, 0.5836387013207438, 0.7672181495726786, 0.9228233311295797, 0.6939582553709781, 0.6523306422051828, 0.6966205076827829, 0.6095744567328618, 0.8688671708295665]], [[1.0510630663212235, 0.772693229801469, 0.34081585852929386, 0.8169192039518687, 0.7799465217839593, 0.7698082402298642, 0.33109474890864166, 1.1958901564948257, 0.5879603304571295, 0.6829760317706409, 0.42100408477168594, 0.5951914707564933, 1.3358993530660639, 0.8045255467704695, 0.31861860248375845, 0.4941729248658826, 0.5615661442252348, 0.5339067377918474, 0.556606601797721, 0.15747607742137626, 0.3167656776887658, 0.3369872925265882, 0.07847486411565821, 0.5820709159683191, 0.9345965152765293, 0.1893119381473386, 0.2439760920521749, 0.476110885436615, 0.7316021237120786, 0.8662398094488661, 0.6255634539097827, 0.643551867808716, 0.7606732651130284, 0.5565123611984781, 0.8026188849448297]], [[1.027724518134392, 0.7605327661154558, 0.3366218040513853, 0.8020062353501431, 0.7575452821487005, 0.7547492222969816, 0.3586706444143804, 1.188246232668641, 0.5583905369224909, 0.6468308086253329, 0.44066806012735876, 0.5845638718992805, 1.235563116233049, 0.8498855265148169, 0.3676311734972729, 0.4880023089327148, 0.5447445314608235, 0.5250056485656185, 0.576880823320994, 0.12683332306683553, 0.2748996824257319, 0.3542178013177033, 0.07484105267035868, 0.4645080208364864, 0.9269059388817598, 0.22495344844944126, 0.24517416621681315, 0.4149134039498939, 0.7200460966019723, 0.7557051937190447, 0.608870640806467, 0.624983649887972, 0.7864372237616248, 0.49451962302209584, 0.7796742420813487]], [[1.014969669132896, 0.7660028605353775, 0.35162759947563194, 0.7862763595655549, 0.742803845812309, 0.7627667413674413, 0.3684151087408927, 1.1642442752926365, 0.6088139851155301, 0.644307682935747, 0.417745046427033, 0.5889034935326002, 1.1067979588904422, 0.9252367940023857, 0.3653778945953883, 0.4893631465054231, 0.6064878950280062, 0.5317347241956805, 0.6016105067312486, 0.1515669367688162, 0.2710882869305284, 0.38889490003101673, 0.07761482208168086, 0.3571236469206911, 0.9748178182727454, 0.269706407697716, 0.24850664166146963, 0.4748057853466606, 0.7097993564129551, 0.6400764228416479, 0.5808188263392087, 0.600894955706722, 0.7623422119345411, 0.45482090587262586, 0.7665575579930485]], [[0.9861342028360761, 0.7785323068931012, 0.2973315640124372, 0.7018944372507192, 0.7331037314294897, 0.7479479480722747, 0.3244840632905912, 1.091853216709474, 0.7311823549038627, 0.6086966758681892, 0.27493467784527714, 0.5841235959819343, 1.0582361505463262, 0.9651680680152455, 0.3868297537310614, 0.5055464686717591, 0.6862946021769827, 0.5531052048827323, 0.5933510588710857, 0.1534088467880563, 0.21497660510755426, 0.4691297237749975, 0.12650917094897396, 0.3487006578817461, 0.9605817260066924, 0.31315228318142346, 0.2699793641116081, 0.6129146943911572, 0.7471064298664727, 0.5951481615791532, 0.6298262152600598, 0.5770230225998809, 0.7471308839209918, 0.4342217422184362, 0.7716941984618988]], [[0.9365553173176511, 0.8082816927678285, 0.3090510786396658, 0.7444819132978622, 0.747285608362178, 0.7376467267002867, 0.29631814015361513, 0.99235562584919, 0.8437991657964738, 0.5585596854842303, 0.15895338520597907, 0.683153112970117, 1.1317963861470484, 0.9556521578133799, 0.3688066182918275, 0.519038845193634, 0.7620235031269311, 0.5184776009320982, 0.6228717844724492, 0.22131878549636008, 0.2514310621136651, 0.6243738064730258, 0.12376399108027586, 0.36512019349897984, 0.8835599438594547, 0.37123242857505484, 0.31073265972148734, 0.726534452679102, 0.7692265030823601, 0.5912081100625028, 0.7059992039692361, 0.616824041152997, 0.7691999214936776, 0.403845842184258, 0.7875487178979446]], [[0.9499475815707165, 0.7914286178323653, 0.29298004909458536, 1.0193079812317527, 0.8079000862926577, 0.7835701637758967, 0.34219957344485197, 0.9255487046803128, 0.9405401431946437, 0.5198232714344987, 0.12157349457097522, 0.6986512033082021, 1.0573609224701104, 0.9073046833133412, 0.37744451262610695, 0.5406818423585567, 0.8514553355606118, 0.5010012287226834, 0.8156862577687864, 0.20336003806556716, 0.1854386786632058, 0.5809132559611112, 0.11856911738911477, 0.3762083662850314, 0.8274865011966059, 0.392106349176375, 0.5042731580316574, 0.7782494965359127, 0.7906245901012768, 0.5992209248994701, 0.7875651440931126, 0.6738225770236403, 0.9052513411239101, 0.4068769016683646, 0.7572259275694265]], [[0.9535827235243434, 0.7837796991246091, 0.3106814402191237, 1.1413827775045706, 0.9113451187629589, 0.7628414966314785, 0.30866526988195314, 0.8788840845748351, 0.9404431457353609, 0.5833507989933251, 0.10303694205900717, 0.5735340024661424, 0.9238195659526314, 0.9195298659789093, 0.3792568707324717, 0.4650820171448786, 0.8063121242818103, 0.5009022466174256, 0.8849211564749224, 0.13751059131500598, 0.18944517385472637, 0.5162609761526982, 0.11489284740766792, 0.37905143468190633, 0.7357208696963737, 0.31416679696159433, 0.5553377449534929, 0.7773679185346668, 0.8261550274037953, 0.6387381237851787, 0.8396572981221446, 0.6970293685230642, 1.0309387564286228, 0.4477762462050134, 0.698697390922286]], [[0.9636343138455876, 0.7743460924955377, 0.2882460206273243, 1.2424611122221, 1.0247465402374507, 0.7770894052551964, 0.33337120207657595, 0.8683378467218378, 0.8294414986891114, 0.6152524327407978, 0.10628833526993431, 0.6878545324558801, 0.9203472651781851, 0.911409412171715, 0.38765312210127234, 0.46880615126282454, 0.8183852452123314, 0.5185548760644632, 0.8644405707335516, 0.11186113895625865, 0.15656279544412144, 0.46177114376115874, 0.10806969901780378, 0.40278302139387184, 0.7028690278308021, 0.2759470859042912, 0.654314829802779, 0.7181954095518334, 0.8502235257709284, 0.6834557982624246, 0.787424139515404, 0.727704157337663, 1.1084280213787037, 0.4935881328306736, 0.6358494461208659]], [[0.9572163136418801, 0.7520201384065117, 0.2624255233209812, 1.0923540249394221, 1.0272013996558098, 0.8091283708752898, 0.3594344238010175, 0.8952984332898378, 0.8266593938540757, 0.6484827964763196, 0.11229968386280537, 0.7565109597923575, 0.8838381929593244, 0.9252159468144946, 0.3933291591643386, 0.4991848931116999, 0.7541610457613891, 0.5276630787162826, 0.9031800531703051, 0.13481119640713213, 0.13408684594591885, 0.4000419768332279, 0.0991204678783118, 0.4029982002037224, 0.6115094052982826, 0.27756416305861964, 0.6480853501035138, 0.6324882408284285, 0.8616442457317474, 0.7212518639628154, 0.7322031420766745, 0.6954512966709034, 1.0961414568571142, 0.4939841608501301, 0.6238342567987867]], [[0.8666159000190489, 0.7573834156872066, 0.2570559094835071, 1.1188785359415563, 0.9909442151192216, 0.8416048959773107, 0.3999627174947545, 0.9362961473405544, 0.8553133566297949, 0.6613832410280701, 0.118073759699579, 0.7280658437381543, 0.8186612523601204, 0.8901761696438067, 0.39553817695406807, 0.4724325064407827, 0.6988143731245959, 0.516520407395963, 0.8275140197155926, 0.16161642758255124, 0.14920070587995, 0.3724247052979732, 0.09550844075194477, 0.38940882203956595, 0.5519794020923328, 0.2622048725050413, 0.7499300891442102, 0.6023896123635765, 0.8773070882574285, 0.719949833339069, 0.6287698494539947, 0.6287694049357104, 1.1033644167308054, 0.49444408756518066, 0.6232601185307465]], [[0.8624929156039495, 0.7686265559813554, 0.24991977352574968, 1.0650609500885777, 0.9921148498990813, 0.8773426286403181, 0.3894072466095685, 0.9426827141100222, 0.8320955487910526, 0.6487120883299484, 0.12436068216522148, 0.6973322484451843, 0.7246212808484616, 0.8631994379219905, 0.41753857510235864, 0.465935071325143, 0.6821876867455509, 0.5239815556039182, 0.8362961341752408, 0.1522222053887986, 0.16410039501151139, 0.3317594370275969, 0.09435154211838259, 0.3854129792396325, 0.5118974374901956, 0.2589320264461824, 0.7656172737609195, 0.5283832536142556, 0.903734303814195, 0.6625423225298055, 0.5889836367942104, 0.7173969770059936, 1.0570050694266246, 0.49797998549682204, 0.641415008093821]], [[0.8646093096159103, 0.7537830056344088, 0.24563472134359338, 1.067047081040078, 0.999363094066101, 0.9369996773570807, 0.37664640423174917, 0.9488654013698731, 0.7728536293809309, 0.6879623294030839, 0.12413095366850474, 0.5612539870095892, 0.6430618246910188, 0.8757033510383483, 0.3833989247614814, 0.5576831326248223, 0.785981113656802, 0.5584790295889844, 0.8524421644574678, 0.17093364787499815, 0.19976019216890653, 0.35072696607337744, 0.0942216243031579, 0.36879439290864835, 0.5426593222942051, 0.25985686850307455, 0.7099812585520097, 0.5179744671069204, 0.9341976154896999, 0.6614891277352253, 0.5842898606017014, 0.7853856829695649, 0.9918706311492028, 0.4790286802655741, 0.6802255590878409]], [[0.8618410362969166, 0.7364520138023133, 0.2530750599525735, 1.0635950826017346, 1.0234067909922484, 0.9266502972898295, 0.38151697051535166, 0.8337651475769194, 0.7358041846927419, 0.7238741733685197, 0.12168309568919819, 0.48525082616897264, 0.5971460840163636, 0.8556248202086496, 0.4167306444388887, 0.6018522952467655, 0.7956102202822125, 0.499050571699883, 0.8868085850569913, 0.2061846770618489, 0.23794667654374707, 0.17095246945971107, 0.10017019415797396, 0.3450537834370079, 0.5337309608729596, 0.24169272139621686, 0.6778199484214235, 0.5588442597944905, 0.9534246395584226, 0.6623688689756067, 0.64176389501197, 0.8155585584616993, 0.9347620606367102, 0.45697972902209816, 0.7138638690902108]], [[0.8679681802918451, 0.7397565541133696, 0.2486350880384276, 1.0722803240055194, 1.023577978521575, 1.1388841081398633, 0.3724469293770786, 0.7658832557693132, 0.75202845759849, 0.6707898058280021, 0.12004261024796042, 0.4872433178702267, 0.47858184552867583, 0.8599365795066969, 0.4104598455910343, 0.47233000153380345, 0.8006103067420022, 0.572232056443725, 0.9259829244530837, 0.20498441664002254, 0.2813431284847342, 0.03219303766808352, 0.10238591511968849, 0.41988648136792445, 0.5352316682771541, 0.2202904636910768, 0.6823393297346108, 0.5349819045366058, 0.9535338221666804, 0.6747024168731743, 0.6428345673486833, 0.7916190683305495, 0.8557190622986021, 0.42899794591913354, 0.662026235794051]], [[0.9056696686497115, 0.7472939186203353, 0.23721044554730691, 0.9957573296506517, 1.0316295845344208, 1.2037378651202175, 0.373710947137196, 0.6897637846316889, 0.9149888354534047, 0.7314119633065769, 0.12081250238085225, 0.5500347789828502, 0.40744639982197506, 0.8751123829537446, 0.4233744358701128, 0.5039038390481302, 0.8158295934805729, 0.6239087924216785, 0.9194323994122772, 0.22195459270632198, 0.26838076726966204, 0.32159731275050624, 0.10407688532526613, 0.4624543159835588, 0.5568767157313104, 0.21042306199864405, 0.6775696703912757, 0.5408933673546202, 0.9559942038873297, 0.7054277178293191, 0.6784398456389714, 0.7731688118169138, 0.7903965747878174, 0.40342870700961525, 0.5857894857697066]], [[0.9489415778185245, 0.7584210437891925, 0.23856792556883222, 0.8067374124652298, 0.9579468316498192, 1.2143543434992852, 0.37401778730741236, 0.38533412737024736, 0.9742487324300229, 0.6996148558716475, 0.11858110045110162, 0.5295290749231135, 0.5407019686447506, 0.9171451328772475, 0.4374093649366002, 0.6127419363677286, 0.8011079563698209, 0.5661344095123686, 0.945298858174579, 0.24916867904834047, 0.27852311088630743, 0.5857060282691793, 0.13753461587415214, 0.434445863563041, 0.5312866476593117, 0.18321326828385318, 0.515241870605547, 0.5356476463089933, 0.9525305330196632, 0.7218582269706697, 0.701431412312943, 0.7552019960607459, 0.7472055778123057, 0.40089852986718244, 0.4966724083626096]], [[0.8633164202097823, 0.7275474020072076, 0.2424643754664767, 0.7649726359767554, 0.8745333335993055, 1.367788782049388, 0.3796206374056231, 0.3560193591369157, 1.0918844130824295, 0.7212556232589927, 0.11787654961329297, 0.5541242362332983, 0.5862010157115546, 0.8877957803414331, 0.41225847536466037, 0.5112776404181312, 0.8234064793226161, 0.6034961030235615, 1.225052582428258, 0.25070560170120476, 0.2935116930205265, 0.7873311956522014, 0.13301568875969097, 0.4539118162257774, 0.46776018002558306, 0.18778685095106135, 0.36966042657795606, 0.5110126816281715, 0.9504565380560474, 0.7200671342397921, 0.6732587293834342, 0.6937387565484501, 0.6561316662498944, 0.32423737531155894, 0.4051900770650395]], [[0.8674823764519324, 0.7241325420499256, 0.23986585402569016, 0.7358035630830309, 0.8082919786171456, 1.3479850965793427, 0.37170242368067674, 0.38094666669780175, 1.125048104480828, 0.6789210156412988, 0.10711730730581098, 0.5929390455313243, 0.8549482458993569, 0.848321952237036, 0.38969727150618116, 0.39552173853908434, 0.7888050447074552, 0.6352854395843688, 1.3369309213332348, 0.23669318208908108, 0.31043783879802356, 0.8551003457562141, 0.1251007202291782, 0.4610210417703462, 0.45682277790671677, 0.2649087407197962, 0.3161348940071753, 0.4626388691941621, 0.9195011642025671, 0.7148224203015074, 0.6718152531969228, 0.631562695705094, 0.5906134043409095, 0.21037775933990893, 0.34361815626643943]], [[0.8651065201283148, 0.7043375733454352, 0.2471089473146979, 0.7506470922394518, 0.7884187938124237, 1.2937861498897911, 0.37168092102620603, 0.36004680782224485, 1.3353505654507694, 0.6736425141504268, 0.10244918691583643, 0.5979238203221349, 1.13107158412241, 0.8204557060195701, 0.3837974998732577, 0.37225289672869916, 0.7627315028925751, 0.6326440469135857, 1.3701918640371757, 0.24192856605252316, 0.32744512007117993, 0.9054105346488192, 0.1339408060522168, 0.4544733480979881, 0.45238814182734743, 0.35736632943757185, 0.3175488937305262, 0.4572210757230336, 0.8714011874770737, 0.6593026843226524, 0.6462961407372829, 0.6255473960572836, 0.5665203809218073, 0.2094739317081568, 0.3853147243113578]], [[0.9013934240090564, 0.7418083191438869, 0.2373946580963341, 0.8074691755312076, 0.7077912946985221, 1.352702205364737, 0.390607674678169, 0.3676751564818539, 1.325081402097413, 0.634019971274513, 0.09015400307188667, 0.5789807654921981, 1.1913009340880731, 0.8805015132479516, 0.3695779631501659, 0.3625848198463565, 0.8440392091425517, 0.7220895011286153, 1.4653698667283073, 0.22699195115341875, 0.3272370873305023, 0.9548757102315099, 0.21452703780596488, 0.4567067515985445, 0.40316564047689907, 0.3665302773092483, 0.3318441490232444, 0.40283900235011016, 0.8409493142456158, 0.6486710091999657, 0.6723575978908307, 0.6299122187673285, 0.5464238533298864, 0.2883815835966383, 0.47191940408392763]], [[1.1032673791529652, 0.80666299815607, 0.2539655656185029, 0.9160621970676772, 0.6643523454348442, 1.1324761720467098, 0.3919505694050352, 0.4537851617091581, 1.1525571306201052, 0.6014693351903022, 0.08779493873003608, 0.6437338095261552, 1.2484241890739138, 0.8152885238781318, 0.3693894713097413, 0.3588961890855284, 0.8575110646153223, 0.7066043146328653, 1.4894463022252662, 0.2199479365953686, 0.32760849585747537, 0.9188701530220121, 0.17130238241364104, 0.4542845077962492, 0.4071785365359471, 0.3921807079242122, 0.40922469438170944, 0.3833839613804196, 0.8186184798980515, 0.6623905845265035, 0.5719548020565511, 0.5596953464133602, 0.45113473773805296, 0.34416610626345434, 0.5806087247717926]], [[1.1640661672268018, 0.7210748879795897, 0.27316671269406256, 1.296709053805958, 0.6845172242616122, 1.1655545163425296, 0.3859360828929735, 0.6522426157697792, 0.9444293600919097, 0.6825308098480467, 0.09580489175027851, 0.6011907912434782, 1.3066953671682324, 0.829590839422778, 0.3501412644124912, 0.354500839994742, 0.822722852355102, 0.6892470815734939, 1.4712545861243012, 0.19827438809034767, 0.31468000474994695, 0.8985384137285182, 0.04776010224828715, 0.49237553105524207, 0.46292216660936425, 0.39614899936663084, 0.4525572419525738, 0.322379330089834, 0.8183199336490478, 0.6650050029367002, 0.5490799735626164, 0.5352838128322805, 0.42706073493145935, 0.40650983774449045, 0.6128479405477785]], [[1.1471317940472, 0.7188057127935241, 0.2593391841400182, 1.5841461810840713, 0.7313788103843732, 1.1357277473931089, 0.3855694028790204, 0.7093865658386571, 0.803224865963159, 0.6320959184619727, 0.12072545172174745, 0.6289977518364193, 1.3348457274441734, 0.816939414661983, 0.3469088286750952, 0.35565609889339106, 0.7647114409723285, 0.6402670204317593, 1.50012738862337, 0.22586233206556672, 0.3947432462769729, 1.0011453715841807, 0.09222250207961552, 0.5556546813890817, 0.5084749628312977, 0.396253987713303, 0.4649171796190984, 0.32068776824712136, 0.8020808420144661, 0.6732848072296438, 0.7189734082194668, 0.5709900070931208, 0.4930669003386818, 0.4273265945476208, 0.7021288786674682]], [[1.151355748445002, 0.7723434101882903, 0.25706866236180853, 1.674367916652407, 0.8197338658255233, 0.9626934903723546, 0.39656219220400535, 0.7240440032383578, 0.7400595027641633, 0.7544100916301865, 0.10388483480093173, 0.670181489031147, 1.3452533025662115, 0.8093795579992337, 0.34931754207725263, 0.34473625569822486, 0.762115283384931, 0.6185665115864416, 1.5778396105487218, 0.23395836254280036, 0.355637583150787, 0.8055440300875949, 0.026344785628411777, 0.5837551434040739, 0.4312677954439851, 0.39771952907596, 0.4208668461084988, 0.2792622719745517, 0.7764100006013239, 0.684497827692535, 0.7484874484045042, 0.5049613180577606, 0.48315074256730456, 0.38451428573381413, 0.41214426238056123]], [[1.1486120356797511, 0.7930341909398158, 0.2967220383941572, 1.8990873747149317, 0.8704008755664814, 0.9958698110115833, 0.4338958838467107, 0.7609368931330558, 0.6836666092994907, 0.9257696497830772, 0.21272709101656612, 0.6746287330991512, 1.3072271169112402, 0.8720184058935305, 0.347185219178739, 0.34955082207862204, 0.8251964714063019, 0.6129672866537166, 1.453153414964607, 0.2682341163940726, 0.29332587187499837, 0.7438863018751276, 0.030023708762373702, 0.5491390766822336, 0.40988560300651444, 0.4882996488919781, 0.42246654151098567, 0.26852633600609777, 0.7320284419991563, 0.6631615937409234, 0.7384258777149975, 0.4335528454590395, 0.49854703190045524, 0.3093688310960825, 0.3438658712775079]], [[1.004783750619049, 0.8060170524605065, 0.2773675659774089, 2.0781123461610145, 0.8893241272298897, 0.9604515034274301, 0.35622776901896747, 0.6796984945566902, 0.6223499175234384, 1.0126559695666466, 0.08755446731882605, 0.6485353390709467, 1.3075675101257607, 0.9028053413466117, 0.34993219599676084, 0.34758845678816286, 1.0716297512812465, 0.6433350364564165, 1.4739431081024685, 0.2959858559857024, 0.28289820759336126, 0.820349583635564, 0.03771493378702045, 0.516403183202565, 0.3840474766342912, 0.6688062905478587, 0.41881569040352984, 0.2604735084594689, 0.6940456701924074, 0.6362394575306778, 0.8183363458708419, 0.37410974716488976, 0.5319377081412532, 0.251740835709309, 0.34250400733923275]], [[0.9107769307323992, 0.7958539023206394, 0.2760504319987331, 2.141184297783974, 0.9131287816604193, 0.9397718880217103, 0.28635686026337204, 0.6038483354980865, 0.5887464022639852, 1.0819606510862436, 0.08466557141489026, 0.6473404758437321, 1.3299587187953692, 0.8946413335976959, 0.3798637144018105, 0.3721610596323276, 1.215642861970776, 0.6955800080324323, 1.438497803811374, 0.32382456294310613, 0.2834442890018561, 0.9289319801851668, 0.032208101883697146, 0.48871634182055945, 0.3394536863573111, 0.7595259210004824, 0.4006064896661051, 0.2687596497656072, 0.7274134411314418, 0.6686358130952326, 0.8299506124598612, 0.3858435894628136, 0.5730093665979248, 0.2648594872811524, 0.4416077386839958]], [[0.8555621639764037, 0.833766055664436, 0.3046518552286949, 2.0161748941827535, 0.9348707298591574, 0.9179635522615393, 0.3624859299520893, 0.4000682571618852, 0.5781220750138678, 0.9047434949972069, 0.06899781487969725, 0.6402306457862336, 1.3306540011038228, 0.8423662139382376, 0.36890814998309324, 0.3654785390138554, 1.0988970305501513, 0.6769558511677009, 1.457222225574335, 0.2974083096288388, 0.27585434252495655, 0.8338412197268668, 0.03921612306684671, 0.5868433283699606, 0.35412679358119303, 0.8347175324589583, 0.3939578457511274, 0.27882503175415646, 0.9058187345117497, 0.7585261459654318, 0.8562723713090827, 0.491950640903294, 0.6189435651276165, 0.3109600228716712, 0.5123373089021983]], [[0.8276063720713331, 0.9188810303856035, 0.2853859663912462, 1.8713401977374362, 0.9346663743495054, 0.8598616776927367, 0.29008646869817706, 0.37059609098208496, 0.6701417392627134, 0.5950546383898602, 0.0442442780610551, 0.6190649220529644, 1.3292285135499196, 0.7909492511816374, 0.38186454888314914, 0.3762617029587042, 0.8510816798582503, 0.7289141620518365, 1.458787673945335, 0.2912543333522364, 0.24065620336900048, 0.7590325023596043, 0.042530312648385075, 0.7316821343553248, 0.3872962783035309, 0.8397180061020988, 0.4668901010217439, 0.29205457265031454, 0.9200133046318708, 0.8435414170736244, 1.0480528843255823, 0.5234996028408592, 0.7212139401792088, 0.349769420344615, 0.5961505395741704]], [[0.8378677829668728, 0.973561746211725, 0.30417999724900785, 1.848544240938614, 0.9902634163500222, 0.921852101564242, 0.37025310982822257, 0.42377091375375986, 0.8038255453358127, 0.6280127363396217, 0.048986789447240324, 0.6325390667234903, 1.2842406328394778, 0.8242928869571253, 0.4158308812304589, 0.418436466849574, 0.787202009073366, 0.7168955010778612, 1.280807797204735, 0.3045551423379649, 0.21847433364933977, 0.7119658347052615, 0.07826394825955482, 0.793781322530084, 0.405535071852998, 0.881966159948794, 0.5138204406024023, 0.30863789762991345, 0.9697325613220158, 0.9181743828129918, 1.0775745067569291, 0.5390215266038588, 0.7735339062608242, 0.3756526996798702, 0.6569113290996528]], [[0.5932601402731805, 0.8863685155780074, 0.3111087275384814, 1.5956048489526145, 0.8497261796986999, 0.9058205930123847, 0.3821806115456672, 0.470711466368963, 0.6451733961651702, 0.5005614995849286, 0.040131884938521274, 0.6285698215218112, 0.7755629496554477, 0.7498894546692787, 0.3819170073358736, 0.4143395173150023, 0.6403187635556238, 0.6296963363923664, 0.9876712046730162, 0.21272994081306976, 0.17759303206425153, 0.41812363755161985, 3.4870886584447467, 0.5173269977600832, 0.39935447856516537, 0.5855070271140396, 0.3668024015344822, 0.3278855351301593, 0.7565980718282264, 0.8044092539610008, 0.6085791272258825, 0.30336947664637876, 0.47142886525212147, 0.2031165178209157, 0.5686946916579474]], [[0.598396066136582, 0.8873030457597384, 0.3067093953816289, 1.5849838288490854, 0.9136409708424653, 0.9204056925975115, 0.40325044865241305, 0.5986861859394699, 0.6476826042831068, 0.5677475141927751, 0.023703233594367382, 0.6008033146193864, 0.7376062318579829, 0.7410591521027187, 0.3802739459016227, 0.43440992352189634, 0.6501250109723976, 0.6520249471854658, 0.9861411843033974, 0.232956524878825, 0.17856985356593047, 0.3948044721130127, 3.4870886584447467, 0.5016868711233167, 0.5634407647652818, 0.6292871329405842, 0.4032582706690026, 0.375263557965339, 0.8217879699119104, 0.8633878158483562, 0.6210982950861185, 0.23392697213214947, 0.4682249096754223, 0.21887441518893777, 0.620022487084093]], [[0.5325244826717473, 0.9213957134383177, 0.31192060212731193, 1.565713077515478, 0.9855178769435575, 0.9657827934875314, 0.3903667526261562, 0.8735031660723571, 0.634864984986693, 0.637422502755437, 0.026046451586913662, 0.6203722181075819, 0.6849302600315506, 0.6690633954480194, 0.38098578537818245, 0.39669544095809894, 0.5086540970594908, 0.6346916900508318, 1.0485797579384961, 0.2333803131336864, 0.17850609574767423, 0.36334663936274875, 3.4870886584447467, 0.4252215224850422, 0.8139791743488413, 0.7371613721906296, 0.46565842567224175, 0.7263896746369576, 0.9874892016268146, 0.8910509995393925, 0.6421029826741234, 0.22200850011882528, 0.4649790604624545, 0.29572742094426047, 0.6658577552605708]], [[0.5569242631348968, 0.8816285008411736, 0.30117388732301453, 1.5920952046062489, 1.0014652890600668, 1.0043994937592529, 0.3918099562511273, 0.9593346554966871, 0.661470106069244, 0.7007786259100097, 0.015531932009899774, 0.6097292395511844, 0.6965137208584997, 0.677983006837719, 0.39898266803354293, 0.4129665130079032, 0.5527248526616272, 0.6777952386364052, 1.0241652132308627, 0.22208545994865442, 0.18639623275676287, 0.3511195344281033, 3.4870886584447467, 0.37722418894730786, 1.1045252578209506, 0.7333246300030729, 0.4367543399323577, 0.999137519477655, 1.0488473275163859, 0.9010847429839174, 0.6463377173519886, 0.2577703826552551, 0.4675578308566414, 0.36881568233006834, 0.6785062296324573]], [[0.5606187574308457, 0.888300430169424, 0.30597473971512723, 1.5848983883532242, 1.0053031537984358, 1.1337842770623627, 0.4060251559865023, 0.99777266538906, 0.7202935950109631, 0.762081923466055, 0.039275280021331724, 0.6683469756168514, 0.7374071014624748, 0.6821622337120443, 0.40972724945421707, 0.4533171354098037, 0.5547814923428354, 0.8140009673675379, 1.026534438465608, 0.23782739845950812, 0.20568641174576113, 0.394477932621315, 3.4870886584447467, 0.3362040349076928, 1.2267863345450118, 0.6221947587902193, 0.4020493881639704, 0.9520875185863367, 0.9675736840771043, 0.9053439912518524, 0.6300334545496437, 0.32419169687684773, 0.4589797742777679, 0.45925427218833226, 0.6707481583323317]], [[0.5828682493762849, 0.8864979131961078, 0.29097555125445745, 1.533088317122044, 1.0374774182463038, 1.1519042674600612, 0.4068573039447614, 0.9159526990438952, 0.8252961541626129, 0.8324004050246774, 0.017671153629582055, 0.6758147965983915, 0.8164816792641314, 0.7079546698363419, 0.41477620713358004, 0.46294953648619874, 0.6057314188743963, 0.765343063716247, 1.0139738612731308, 0.3529064851288391, 0.24501157824041928, 0.43800142517674423, 3.4870886584447467, 0.28362295682629346, 1.249363720812012, 0.5122099209957306, 0.3907553208165929, 0.8703466602274808, 0.9061955467174095, 0.901641238543619, 0.6544261246401444, 0.37153742368631804, 0.45261548743414304, 0.4905067042505895, 0.6287119965881698]], [[0.598303286321417, 0.8672725754531789, 0.28619171328947896, 1.4897469296580175, 1.0428895008338173, 1.1695981355543155, 0.4094959055685774, 0.8640365737728569, 0.9436186437327314, 0.8845431353549893, 0.014768540825331569, 0.7072639335793708, 0.9839622254980533, 0.706041174089518, 0.46500547488854066, 0.4900807892519814, 0.5955672676830641, 0.7888693258814541, 1.0025426410054705, 0.5369583279152952, 0.3559242125771734, 0.5349505207104293, 3.4870886584447467, 0.22406659501772724, 1.1903568182766324, 0.5140294581649032, 0.41978151833317157, 0.7997714359853119, 0.83701912130941, 0.8996120512889118, 0.6215497834976398, 0.3637301501648081, 0.44357632132667835, 0.5242541609376206, 0.6104731249797724]], [[0.5955520702205662, 0.8778398001091549, 0.2727998629203308, 1.4382385033499794, 1.0671046787394232, 1.1360385518680611, 0.4091871377210646, 0.7824319745031493, 1.0334876566294127, 0.91512554555179, 0.013936431148572908, 0.6807259339072729, 1.0355807462917381, 0.7329787496356723, 0.39668708840766215, 0.4719234606363505, 0.5619929882007133, 0.7867102812544018, 0.9802945316812219, 0.6483444525865298, 0.47943791986554124, 0.6257396372659049, 3.4870886584447467, 0.2558554597110255, 1.1767754944450828, 0.5385326864257125, 0.45882579194530637, 0.7554719029264568, 0.7713819967490072, 0.8929760863064318, 0.4859744808241492, 0.32149440424861886, 0.41006934267892703, 0.5758332169323446, 0.5735778982900248]], [[0.6103070764081419, 0.8985239186806474, 0.2884713697687643, 1.379201520503338, 1.0704925932751914, 1.0857136863478898, 0.39827664967959114, 0.6687669787412638, 1.155003875955183, 0.8733988153328961, 0.010394843494808969, 0.6113641210803464, 1.0465061354740144, 0.70641199621807, 0.40634453533184484, 0.46056856646596966, 0.5109212702395115, 0.6789180926069092, 0.8785007778878974, 0.6175259652195895, 0.46122466687101615, 0.6295924439568016, 3.4870886584447467, 0.2783187420938555, 1.1059315089312178, 0.5307595882240295, 0.4406906746787428, 0.6931701958767235, 0.7253542505844017, 0.8569413270680318, 0.23843799656184833, 0.24377561232610573, 0.34480975473870534, 0.5893114000996698, 0.5414994034786608]], [[0.6298058898139665, 0.9011128750387053, 0.28722990582097435, 1.3088471089587732, 1.0508083098915768, 1.0623303193499496, 0.41202856114059405, 0.6313020665332449, 1.279242478345778, 0.8740737628449924, 0.005781048474719398, 0.563574420059689, 1.0674347993545834, 0.7103142019082207, 0.4201134822929509, 0.46357579291178563, 0.4754596998619196, 0.6182744585478471, 0.8328557780541698, 0.5828021003288373, 0.4513223869720218, 0.6190359989964926, 3.4870886584447467, 0.31688329400077764, 1.0982911495485532, 0.5246108187881762, 0.434373487344119, 0.6670328662373977, 0.7263905800398369, 0.827619686323958, 0.1697858385299498, 0.20539472980996598, 0.36343705415564964, 0.6194021173824573, 0.529094522910911]], [[0.5607364319932711, 0.8774942771133641, 0.28907130614334015, 1.2333699244393885, 1.0328991303644297, 1.0137273668779563, 0.4191902579231801, 0.5845795957593807, 1.3640346996699906, 0.8570573741091041, 0.00400914172406001, 0.5815514833404525, 1.0624063012946738, 0.7193402345429024, 0.39564301346137626, 0.46267713321455417, 0.472280117385248, 0.5833313690363037, 0.8049172977769612, 0.5247364384711173, 0.4430801132401891, 0.6550783094162249, 3.4870886584447467, 0.34322610389296454, 1.0748468685717159, 0.45793090449808393, 0.41623648276692904, 0.6499080756689594, 0.7253044276947147, 0.7796873949472509, 0.17817270520448453, 0.19609097408063328, 0.36360995626870446, 0.6134243569388198, 0.5198223191504612]], [[0.5491328630147992, 0.8772805479470663, 0.29881442287497817, 1.1704482954127935, 0.99839750163916, 0.9653503044934331, 0.4224036272154863, 0.6157413402469141, 1.4197827166094146, 0.8436318385411339, 0.005244286496593908, 0.6133251124740611, 1.0750464445003003, 0.7011622197618897, 0.38324126595225944, 0.3963912084592316, 0.5032517302502988, 0.5838821146060618, 0.7935540187486394, 0.48337417412439854, 0.47225297922952686, 0.6374524538051157, 3.4870886584447467, 0.3925641227256872, 1.0287405414962816, 0.37997463385976704, 0.4002629419411055, 0.6454415611784595, 0.7109452969516715, 0.674658110610558, 0.2670263494602499, 0.2273042804028885, 0.41736370541741835, 0.5763906242474486, 0.5291748633965612]], [[0.5308166213710843, 0.8662355415214922, 0.29434148573851543, 1.0975266512577997, 0.9502278485772399, 0.9391988123323893, 0.4905963605069073, 0.6133691426308165, 1.473724015450693, 0.8197428690463797, 0.0054892631523850875, 0.6217515197105021, 0.9635994383876885, 0.7105644355860723, 0.37400396656174667, 0.41589164484581714, 0.5294832284068033, 0.5770234967656045, 0.7046853969811566, 0.40717783089582305, 0.4954335896736406, 0.6170431819481217, 3.487088658444745, 0.4684103756215544, 1.0271963221431837, 0.3045916533333122, 0.4196704024526361, 0.6818468747377695, 0.6682268638270742, 0.5331146208283688, 0.30911275944227234, 0.300924890715215, 0.5112922947099776, 0.5744343033793227, 0.5459786524212186]], [[0.5430928827572612, 0.8160467954187345, 0.28115281333004943, 1.0286559241346531, 0.9196652162265481, 0.8424394611152505, 0.4268316992027905, 0.589350012431385, 1.5342477657346965, 0.7888927276454499, 0.004544095845848322, 0.6607118865544266, 0.94152943598684, 0.6813605002103167, 0.39347766247754107, 0.4268078472036041, 0.5230434539977437, 0.5528699318939008, 0.6819281529354353, 0.37203030408064386, 0.48140171489992645, 0.6206945575563652, 3.4870886584447467, 0.5000229480274179, 1.0295573182621873, 0.2574964626445131, 0.4290618703804018, 0.6514014269449544, 0.6346087692876146, 0.38694893989251267, 0.3615263919242556, 0.4252939020565517, 0.5234063619768448, 0.5651028257392278, 0.5547121270320456]], [[0.555986172225575, 0.7981795565753115, 0.28606742487446374, 1.0001469122328133, 0.8435072008997417, 0.7826020313930745, 0.4419573354616837, 0.5731651848117996, 1.5949185660651057, 0.7412448616666162, 0.017639119334213006, 0.6140090139873644, 0.8866977816025432, 0.6115261636211191, 0.30656952588095676, 0.4243148447380638, 0.4238765798406675, 0.48312390225036417, 0.6411478628757186, 0.3225024270542482, 0.4515901949265308, 0.568295493229031, 3.4870886584447467, 0.5250273172111496, 1.0724394247432227, 0.22334415780397504, 0.511354681523376, 0.79972314592168, 0.6012017467789565, 0.4049601149881944, 0.4626578257442311, 0.6439517449839591, 0.493709777745049, 0.5448430801412543, 0.5674818343607593]], [[0.5579505814942389, 0.7716968299216825, 0.2997899403373338, 0.9538750072412614, 0.7526354222634283, 0.7148363678616465, 0.44947862617031786, 0.5865775794432204, 1.6346355466170572, 0.705029421187123, 0.009426388384040458, 0.5709646101398391, 0.6786252762812739, 0.5781753661540182, 0.3218456067292296, 0.4216325262297942, 0.435853285784874, 0.5114453407250565, 0.6762296072320753, 0.2964095238354532, 0.3880613862392278, 0.5563696796574666, 3.4870886584447467, 0.6807605837809869, 1.0667890075874504, 0.2372648988573945, 0.5682497019269129, 0.92017124104818, 0.5319855228821605, 0.43369940547527863, 0.5631947819197494, 0.7223485811685761, 0.4325052778157994, 0.6043476638753746, 0.5839981624677102]], [[0.5959464367749795, 0.7672789449101635, 0.2909706073692148, 0.8637202434433005, 0.6938461346339613, 0.7003363465306023, 0.4851340479314019, 0.6205172783479687, 1.639548346072585, 0.6656353167075657, 0.005771117961354562, 0.5324956630635822, 0.5096207219887245, 0.529384414304, 0.2906679153517392, 0.4274621627811187, 0.4146681217636863, 0.4305216219571033, 0.5782159602810744, 0.2803201411888166, 0.3380045338733507, 0.5489639707702516, 3.4870886584447467, 0.7743525075080908, 1.0600772512729086, 0.26535263218983085, 0.5939528033905205, 0.9103555631832361, 0.5046120813581473, 0.45848743321903973, 0.5984807313282643, 0.7601415526992406, 0.40436996849803675, 0.6974720494180329, 0.6226965699672737]], [[0.6531477594291039, 0.7459986616764565, 0.298700959618074, 0.8278602721148964, 0.6512067872913205, 0.6329469265822398, 0.4673646741914421, 0.6872273442256195, 1.5981836330272794, 0.6197110616877887, 0.0015170401694573252, 0.5452098625988497, 0.39585392072897374, 0.534506670658813, 0.3046560850942507, 0.42606224670372117, 0.4332059103129131, 0.4591545112027242, 0.6227665511163754, 0.27333144557586425, 0.31191865459457935, 0.5162199424821879, 3.4870886584447467, 0.6580365031669547, 1.103956984373369, 0.2707469831619965, 0.6535249463572529, 0.8221623763054626, 0.519385603625278, 0.45404247437645134, 0.5572846435139178, 0.7171060307445525, 0.42520642178328594, 0.6620636075908062, 0.6680878218468487]], [[0.6724019571216893, 0.735744242757144, 0.29071862081820715, 0.7982438393240917, 0.6154515003573335, 0.6006858774874537, 0.44241416518134624, 0.7642396960237472, 1.5334475081675918, 0.5749762081089953, 0.0014674066155896134, 0.5669980771203846, 0.33467782873003604, 0.5422758031119224, 0.3598816637913331, 0.4002601617154343, 0.45652793924221197, 0.4795422002445181, 0.6484531231958, 0.3052141504091957, 0.291814271678598, 0.47863728045615794, 3.4870886584447467, 0.6746659228033576, 1.1244804358311666, 0.2773181849239061, 0.6804840176300273, 0.7845101528480327, 0.5311760075709062, 0.4596215651690063, 0.5211786920028669, 0.703260132259367, 0.4346449009632808, 0.5649518416318808, 0.7198647921921948]], [[0.681797878359643, 0.6706525239412467, 0.244616184179048, 0.599060478627244, 0.5280958560795237, 0.5090406948710893, 0.42619122228769885, 0.8423181713542249, 1.4496763195606706, 0.5095182105675451, 0.0006657345705040744, 0.5694108189024231, 0.36919491197387444, 0.5348384255983414, 0.2905095719650187, 0.42181772727565914, 0.5439334986188524, 0.5008794112048933, 0.6255639771730181, 0.34280308489523714, 0.2884418353789714, 0.49101136824353464, 3.4870886584447467, 0.6758727849478334, 1.186670311885417, 0.282250933546343, 0.6658542979343489, 0.7846135981924875, 0.5240936214506418, 0.4913537151453105, 0.5394652272782772, 0.7395153730592945, 0.45802197693000046, 0.5362581961831554, 0.7271869213435248]], [[0.6570046265272507, 0.6938145036417636, 0.265713052622201, 0.6300643465280868, 0.5027691315708929, 0.5636923689776967, 0.40931737335232987, 0.8394046057113335, 1.3416389739301984, 0.48223706905075153, 0.0006925861058894203, 0.5156447323376361, 0.38357359546257164, 0.5125979515631167, 0.26586714764305236, 0.45074155057652465, 0.5530216412204951, 0.49108409325709546, 0.602694648418064, 0.3985854718838992, 0.320704429284062, 0.5044132435863365, 3.4870886584447467, 0.6850276856804142, 1.2578937625851248, 0.24526957375441757, 0.6638992813254414, 0.7649656701250873, 0.5096447302965608, 0.5271286781884896, 0.5550533417281421, 0.7558632214475276, 0.45223080553660255, 0.5609791855176938, 0.6928769394322549]], [[0.6363209830384338, 0.6916324335320145, 0.27269641608836287, 0.6916542984474441, 0.47225721086145966, 0.588457947966292, 0.4084478650430845, 0.8250687069893994, 1.2045446689007633, 0.44793583853745444, 0.0006863612673056192, 0.5181793102375758, 0.40047522100052163, 0.5173904274377441, 0.2814846238839578, 0.42002268106173823, 0.5794614636689643, 0.5032324719190071, 0.5832988415348577, 0.4257699625948037, 0.328117055937847, 0.4925950157014181, 3.4870886584447467, 0.7070630943409402, 1.3419336801858086, 0.21365156398817067, 0.6763358894948917, 0.7184580345464879, 0.5142657346212397, 0.5234718041554063, 0.549427273628242, 0.7441400981547763, 0.4316129924275238, 0.5912476136373256, 0.5472288365912059]], [[0.6573130129809152, 0.6024910043030867, 0.21064872801914494, 0.6347407883292557, 0.44145241202136576, 0.5919510306628619, 0.40026885247322813, 0.8238987499210301, 1.076826162411991, 0.41603365502590195, 0.001395631245915456, 0.5410395794379057, 0.39890114703164314, 0.556485720845636, 0.2885202549831527, 0.4315174926400618, 0.5336289049859937, 0.5058848871899291, 0.5697944752651588, 0.42650159268223714, 0.339996597120271, 0.4801021440321978, 3.4870886584447467, 0.6914071002219393, 1.3852015458922091, 0.18603182213811628, 0.6714374058686006, 0.638234586373797, 0.5276954660010151, 0.5183797377848929, 0.5466602221861819, 0.7836618916071132, 0.39099839389058955, 0.5991943174846833, 0.45948836084901457]], [[0.6648343548550306, 0.6138224847403054, 0.22715469608327535, 0.5522987651665352, 0.4119444026416838, 0.566694501551495, 0.3847095628632915, 0.7821987448171503, 0.994013278162311, 0.41375901698182127, 0.0027121639678429816, 0.5510030366896979, 0.4291218882307127, 0.5815661491265208, 0.18982664554033693, 0.45350874382545425, 0.4996570579684431, 0.4817568796412729, 0.47304021188886936, 0.41068267527505364, 0.36542827972285885, 0.47849315271883835, 3.4870886584447467, 0.6872778224034399, 1.3695515360478019, 0.1624318680740445, 0.6549614636586254, 0.6089430945412053, 0.5479082589566103, 0.5024584372466301, 0.5505865048573031, 0.8207611738368212, 0.34960543251271037, 0.5851400028217641, 0.4663158222607249]], [[0.6976081498644846, 0.6173061758983712, 0.25976227304902866, 0.5164929656253248, 0.4085521802957913, 0.5626451699763838, 0.35783441834832524, 0.7614902688976775, 0.9707620886239804, 0.41479122309163696, 0.016991957647392905, 0.5579386415341124, 0.39416413035776277, 0.5738455018214235, 0.2241827298781902, 0.4551612385690436, 0.44741043698682187, 0.46613686505679097, 0.5261296074703289, 0.42186497607022344, 0.4117324932671339, 0.49870247162572456, 3.4870886584447467, 0.6649341188631166, 1.335252723566888, 0.1369865139941264, 0.668017409856268, 0.5839323031773278, 0.5660608519191836, 0.46888697528668294, 0.5463502756404736, 0.8344517410645838, 0.31693841085972807, 0.6011082464466556, 0.4270667591738902]], [[0.728236601630067, 0.5428535975201983, 0.19829388903651785, 0.5326222638177458, 0.4598287232046794, 0.630935049669713, 0.3542985958522279, 0.7393289319094771, 0.918277319092863, 0.4701854127900461, 0.01112696807342127, 0.5498258394234162, 0.28770658240736147, 0.5678179502411899, 0.23935974591031117, 0.46881709545861017, 0.3397120337316046, 0.44926693390740224, 0.5214340896402612, 0.4137407150773138, 0.44352648848524356, 0.5268387385763291, 3.4870886584447467, 0.657126204051574, 1.3114209164759998, 0.11072161479100283, 0.6627973198737523, 0.5791402507300649, 0.5661298328805889, 0.45030930660032664, 0.5447349559261364, 0.8533033061529283, 0.2950291310129405, 0.6865785074637645, 0.4375200018710803]], [[0.711773835091495, 0.5480701510002144, 0.20866813671640705, 0.5109232229058026, 0.5207789008052433, 0.6206414443680248, 0.34817353811754376, 0.7257612492141775, 0.8293599404678175, 0.5195724985258364, 0.0009688581790248806, 0.5379166569247306, 0.28737179881940034, 0.551460600849345, 0.24648972083286907, 0.45352509098750954, 0.3618885170624009, 0.4520189436257055, 0.5432363863524985, 0.3902780363851571, 0.5058128907671312, 0.5856838584505805, 3.4870886584447467, 0.6602602009852863, 1.260630225376857, 0.07357690353438975, 0.6385203868062139, 0.6046339557826406, 0.5486180919266467, 0.4316977584974116, 0.5454813341855953, 0.8698143093157487, 0.28468265431923095, 0.6710811675235717, 0.4355806091533855]], [[0.6958986876669415, 0.6297334507012502, 0.2574003519578259, 0.5248645529470808, 0.51026197861842, 0.6725661361991184, 0.33131597657232686, 0.6913157223086934, 0.7503255479509403, 0.5629946870091785, 0.010066263249356255, 0.5249065535733368, 0.32984459110106445, 0.5430536125394569, 0.24513851124472252, 0.4393969450872157, 0.4060399806141913, 0.4657891379552303, 0.5984198871643411, 0.36392544656757325, 0.4949346737474266, 0.5956501370963398, 3.4870886584447467, 0.6522697485265296, 1.0987704927726054, 0.0399265883795889, 0.6084999896703588, 0.6071767463735898, 0.5416670322254261, 0.438984595033211, 0.5438688801059325, 0.8565841941522414, 0.2975195998019086, 0.620809536175658, 0.418992529708077]], [[0.7103573508229102, 0.6392036422806104, 0.2642604516548673, 0.5799011327293251, 0.5396254924120005, 0.7093928216005249, 0.34093864237104826, 0.6792665904198707, 0.750382617583846, 0.6034792175511513, 0.03487535697070988, 0.5884676151471961, 0.42020780386570417, 0.5797343850995534, 0.24424745003346343, 0.4148233057139278, 0.4337718232061102, 0.4363905978900492, 0.6172268066634533, 0.34170673496896553, 0.45102170468789193, 0.6098608387072112, 3.4870886584447467, 0.6442143312861512, 0.8056329355916765, 0.021485966292125284, 0.5694835014654132, 0.5608108563809244, 0.544562445719539, 0.4491736947238494, 0.5475233948560023, 0.8234056450555765, 0.31641449452933285, 0.5975946164055923, 0.42730743644589525]], [[0.7198516501269865, 0.6452361947738852, 0.26578779352154397, 0.6191128430454518, 0.5867782074104779, 0.7209955767616427, 0.3475428249491879, 0.6948348259420112, 0.7120007224717971, 0.6345527535580495, 0.021507499240844108, 0.6010969412640019, 0.38223603747957335, 0.5874248452712246, 0.24897506411468512, 0.3985774440512616, 0.3969786184096946, 0.45589441100423855, 0.6539970959470189, 0.31387762464799634, 0.43246589739197716, 0.6516576001491177, 3.4870886584447467, 0.630293755378196, 0.528499159058401, 0.012266813359424152, 0.5290435496545477, 0.498171294109432, 0.5382921906177298, 0.4502450720415766, 0.5525006927710803, 0.8237719744246229, 0.3270470736404603, 0.6412299423716323, 0.4775272456714793]], [[0.772473214494113, 0.6587540533063379, 0.2726802407997943, 0.6467581842773662, 0.5895911122799948, 0.6715645120013174, 0.34955409849362634, 0.7594748681126227, 0.6881668702817779, 0.6406510604033128, 0.012716581603886648, 0.6412941530123915, 0.4057645914379236, 0.5975378364532897, 0.25085325125704144, 0.4112307566935326, 0.39163157796683545, 0.4622263590308771, 0.6593476719890329, 0.28689832918914265, 0.40653160419017986, 0.6481408774871666, 3.4870886584447467, 0.604259343417556, 0.5370860002313615, 0.02527906958267198, 0.5139792249629316, 0.5558240107453656, 0.4715781107510896, 0.4645340197988799, 0.5577545774124579, 0.8449147504878334, 0.3740886187493142, 0.6186749601742604, 0.5115203320531525]], [[0.7699573541570255, 0.6383809018789826, 0.2725036936911301, 0.6615397577460671, 0.5576552089991434, 0.6445378332419835, 0.3587574636500702, 0.828435875850263, 0.6833206902144096, 0.5687668602696835, 0.008793108784448155, 0.6003748037180777, 0.40453337696284436, 0.594673209666086, 0.24819972234282747, 0.38140786239839225, 0.4153115502611828, 0.4597652603327734, 0.7202877916960793, 0.3022482790259473, 0.39884476257859486, 0.6472340653881722, 3.4870886584447467, 0.5619439597046798, 0.42089004809471003, 0.08223282650651648, 0.4925366109892272, 0.5330331922183478, 0.39551909792471907, 0.46347433240596475, 0.5534667574394856, 0.8506205609967545, 0.4246189642693824, 0.6588608981664562, 0.5381370507075787]], [[0.7547349142251492, 0.720069230900287, 0.2714566639431751, 0.7252134632391576, 0.5064903995248697, 0.6928340302950036, 0.37388951803427684, 0.8822418435853007, 0.5983751408286914, 0.5826083172251546, 0.0051904180623224, 0.5982425620484757, 0.4537595925215526, 0.5943652557157318, 0.24228614146615166, 0.38412436596550653, 0.4365364056104565, 0.4457764842179139, 0.7186177844534216, 0.306350917167197, 0.4060937700561761, 0.6093666698188523, 3.4870886584447467, 0.5249023692242654, 0.4173070528898037, 0.115746389465457, 0.455118022806521, 0.5392349130980931, 0.3447514393348255, 0.4929725906365006, 0.5513319674223192, 0.7905065178705095, 0.46022801515624046, 0.7728777983849755, 0.5375715539265938]], [[0.7157244279956401, 0.7507210351418051, 0.2712878175744024, 0.8105646669512181, 0.46987270228013917, 0.6971630916808342, 0.3685180063790034, 0.9845898124758152, 0.5339986957537274, 0.5459367716313029, 0.004484958947125927, 0.58357341080177, 0.5362976492272536, 0.6192411211782591, 0.2432938607483465, 0.3911012420824329, 0.4267401664179171, 0.43996270143327554, 0.7383951845940417, 0.3411484672884437, 0.3443598029335562, 0.5467208246610274, 3.4870886584447467, 0.5320794860726452, 0.3859125458201974, 0.14366065706344153, 0.519158524298757, 0.5461862515128022, 0.25309925084468216, 0.5099167698835352, 0.5605775997765505, 0.7562562092685197, 0.4350250737986313, 0.7667440642681778, 0.4985232630182798]], [[0.6607999907562809, 0.8140544080187301, 0.27421583116222764, 0.8708475143763423, 0.49837738289088296, 0.6449816458244366, 0.3746930879362349, 1.2032330452193667, 0.5980911766582373, 0.5017790030361267, 0.029672172544902065, 0.5757135404496245, 0.3939745024962736, 0.6178309669628628, 0.25813883444961006, 0.4410184693839545, 0.4538644697450631, 0.4097771364998738, 0.746166750555073, 0.28557633525686943, 0.3258979379965019, 0.4569964003742909, 3.4870886584447467, 0.6197545583202931, 0.3660880652351847, 0.0762321301298113, 0.5095094360820486, 0.6040522226196761, 0.25994108625321877, 0.5519043569048612, 0.5662250914929143, 0.708926652027843, 0.500267087169946, 0.7699875777332335, 0.4668451534175607]], [[0.6227692263000822, 0.8116199155224619, 0.26375447335112573, 1.0153260566758018, 0.6009211281388085, 0.9633239733309384, 0.429605781288402, 1.325063659715178, 0.6512043021218384, 0.42632139540814695, 0.025768977389632162, 0.6154448824358766, 0.4778287798171035, 0.6426423945685376, 0.2624681982519709, 0.42944183601453945, 0.42065896434342165, 0.39243199224234904, 0.7257737334671839, 0.2669065036908347, 0.2850656639328407, 0.4043688373507585, 3.4870886584447467, 0.677378885593304, 0.3627014900160429, 0.06598671823443564, 0.49441412945396684, 0.6319035649696866, 0.29136007052945406, 0.5919512694248306, 0.5549106216126451, 0.6997508251379656, 0.5875164055391964, 0.7060801219012385, 0.45949985703894625]], [[0.5249785255742541, 0.8268385563490175, 0.2528920621415727, 1.119268608988414, 0.7042763223268964, 0.9025661793596317, 0.3316473351590404, 1.540719394164974, 0.7199390773201277, 0.27052299858543183, 0.029480682695836195, 0.6082071473648841, 0.515673901978136, 0.6376558469274192, 0.2703356871939011, 0.42621099701741527, 0.5016358062383289, 0.39977985341128597, 0.723085098585872, 0.2780362192395089, 0.32133589769762205, 0.30658403578176446, 3.4870886584447467, 0.6889654482473602, 0.3345153470655332, 0.0740663705053807, 0.48211726012639783, 0.6284671510942387, 0.3349387625856671, 0.6457156473055623, 0.5423955407043666, 0.6223529680340707, 0.60274110305957, 0.5781417272483971, 0.4447934559320671]], [[0.5404930953950923, 0.8603458391933047, 0.27038929511730586, 1.1573295122773875, 0.7224006473539801, 0.9961459486366887, 0.3602572768550103, 1.6072147086938713, 0.8439743228673434, 0.4795972761469931, 0.026950508158394497, 0.6030100995455452, 0.6084445856144776, 0.635935773167654, 0.30014534250172675, 0.4439648661410892, 0.4993728571839784, 0.4065873105812392, 0.7149875213554582, 0.2903654798145994, 0.33509456673602667, 0.17912918885394077, 3.4870886584447467, 0.6609107227064243, 0.33994801141580155, 0.08478300793911561, 0.45514414518411545, 0.5648743398517031, 0.36045229671835705, 0.702087023218917, 0.5768436221628036, 0.6068588502616697, 0.6191419835495171, 0.590615101219492, 0.4589040214926799]], [[0.573683770541338, 0.8246794625377845, 0.23445487553507965, 1.1975868791605355, 0.7475198834492418, 1.0266175600995444, 0.32176001324380027, 1.560858913148846, 0.9191135738617694, 0.5422844118265917, 0.021552837187230858, 0.6105274169375308, 0.6027005262418301, 0.6346939393366695, 0.29881889648147797, 0.44026195487178793, 0.5005972866013189, 0.41010843687702647, 0.7040031125531006, 0.30499953521378714, 0.3647527545453826, 0.19618900821418223, 3.4870886584447467, 0.6345672838716678, 0.338418249158951, 0.0828811987852909, 0.48157599827217423, 0.5643348401060448, 0.3736309154443852, 0.7317830555561667, 0.6109356985411466, 0.6007543758867265, 0.6247715116260593, 0.5402048216924791, 0.4540679674506423]], [[0.6463246444943653, 0.7948107487673178, 0.2157601119913229, 1.214692888449157, 0.7544293765949102, 1.069969365829143, 0.3228096578344258, 1.5271910222477663, 0.9847771884228141, 0.6418597577371881, 0.06483175003350841, 0.6153564301861855, 0.6409187571001977, 0.6330823661875975, 0.2876037760922378, 0.4466924696342547, 0.5014882406657604, 0.41643329947463237, 0.7006672377006085, 0.3188738163369769, 0.3577171666207043, 0.3284733487833842, 3.4870886584447467, 0.6355470533405887, 0.34943962704434256, 0.07840377727457376, 0.4885501898194711, 0.5638306385289698, 0.37791711340185463, 0.758656964807619, 0.6193890224591497, 0.6486757934785257, 0.6443853017612063, 0.5115245325464137, 0.44952476616101156]], [[0.7440794748423165, 0.7516382621851412, 0.1928985885079994, 1.248215438126644, 0.7710904020283524, 1.116119676376053, 0.33242179875138766, 1.4498065927611077, 1.0371588865229464, 0.5168053920438483, 0.06540043537318312, 0.5792208122597085, 0.7331418735895985, 0.6061313623077866, 0.30266109078381287, 0.44335546391733216, 0.4974011548940116, 0.42113472298371935, 0.7147100506484316, 0.34367014956962616, 0.4199349205003765, 0.4497083548666446, 3.4870886584447467, 0.6428252494266256, 0.3466861829392207, 0.06600019409528071, 0.451173975207705, 0.5437581580611226, 0.37619054784481654, 0.780803928018352, 0.6217608754011357, 0.650244849511448, 0.6482434714122766, 0.5217432454481711, 0.430039438940946]], [[0.7487165742139643, 0.7549520110276502, 0.1644643355551067, 1.2607405759826542, 0.7674534477662152, 1.181415941293385, 0.34720368268567847, 1.2609174379938721, 1.0901288860282996, 0.5123789919346984, 0.08927793859749444, 0.5974205903782082, 0.8059699085620526, 0.607152859377438, 0.2544104241887095, 0.42838965871821577, 0.4836192252493746, 0.4353254590736965, 0.6827165756306741, 0.38114067215345976, 0.4852305351179602, 0.5430260827071289, 3.4870886584447467, 0.6502578432614243, 0.34974752752434224, 0.060773449732831136, 0.38033770369774467, 0.5471482934552581, 0.3665198266812073, 0.8025834275810253, 0.6243433261793635, 0.6224103519171306, 0.6604100160950666, 0.48225983080196577, 0.4085034802783839]], [[0.5663351104827501, 0.8859168541571093, 0.16192599922656853, 1.210442083696475, 0.7767130888916506, 1.182779989437427, 0.34530005529586455, 1.1373207617524168, 1.149269342381813, 0.5076243960488643, 0.08243804162268847, 0.6308804098683152, 0.9588612504098801, 0.6089768901855817, 0.2638459193091408, 0.41322918519250984, 0.5407000462966143, 0.46989811089570777, 0.6417011727357136, 0.4323827796566932, 0.6554151169472026, 0.6984462113601329, 3.4870886584447467, 0.6509582178750224, 0.34694432409589393, 0.05823179440677012, 0.32559543553288284, 0.5414316104016502, 0.33700826892473806, 0.7839310026653854, 0.6415148217773652, 0.6042392118945087, 0.6700919488579329, 0.38401575986686687, 0.389402103573561]], [[0.552007869896685, 0.8567638387535458, 0.1815726174011028, 1.2240825981359011, 0.8563730781417112, 1.1945565980958806, 0.34473117939266756, 1.0185525568605969, 1.2161577000976245, 0.538546315171209, 0.08146326016110732, 0.6586602614008314, 1.0143553795413751, 0.6058135986136107, 0.2620432176944164, 0.41759044416812574, 0.5175961411115706, 0.47110191607773194, 0.6328014998270473, 0.4500503716339402, 0.7475198084414105, 0.7484252044639776, 3.4870886584447467, 0.6317511844454347, 0.3491459591140554, 0.06071869925538531, 0.31028562087087164, 0.5987260472471251, 0.30224670608834386, 0.7843337625587476, 0.6676004804748846, 0.6268512017497756, 0.6728168214203193, 0.33423770149161225, 0.37987166805395045]], [[0.5813977056837514, 0.8678359236817028, 0.21505111961291062, 1.1839214609711597, 0.9629422299477529, 1.1653801047791887, 0.3311320161343633, 0.9457940498155915, 1.242048755013106, 0.5796608367600843, 0.07479683359724143, 0.8568344421213951, 0.7932211153050515, 0.5858548128874125, 0.3411147011419321, 0.43005681286260433, 0.6272932278449319, 0.508344970317974, 0.7197453657336674, 0.4451196126071252, 0.6632688310248602, 0.6747924668970176, 3.4870886584447467, 0.6058832575588344, 0.3964478753692458, 0.06566149966884, 0.2531036315006715, 0.6578078048160279, 0.2890119557096842, 0.7503789528744134, 0.6855649902223535, 0.6337519771975346, 0.6585443982977262, 0.2311192419689937, 0.39094493691263343]], [[0.6093471196164576, 0.8360184764035388, 0.21277765518644082, 1.0874728041187192, 0.9956280301157723, 1.1466156461362949, 0.3433783735951971, 0.8920166777357724, 1.2515132903720292, 0.5952014229982814, 0.06767283311071096, 0.8290613307406208, 0.8380933053069215, 0.5783093850909651, 0.36723256604322707, 0.4067563270244804, 0.5869649253222062, 0.46352200421814654, 0.6472838239755018, 0.4504240520465377, 0.6382490466256416, 0.5661358491067682, 3.4870886584447467, 0.595003693248729, 0.47068109629868427, 0.07577525195511922, 0.2615164240666036, 0.7348524949633637, 0.29348777183299646, 0.7090657361588297, 0.6925237261641424, 0.6158381685088057, 0.6347345859253553, 0.16908848152855496, 0.3902166973372903]], [[0.5442950075936995, 0.5392018794910503, 0.2860323424715052, 1.0022579507358007, 1.0769884578710458, 1.0716458431072209, 0.41297631068239865, 0.7724954463948541, 1.1964861829239823, 0.6153632543560423, 0.07579571029608784, 0.7964002098667805, 0.7916706367984159, 0.5971551220919389, 0.43619660391628773, 0.43085423067053985, 0.5378687470381982, 0.5050867733249644, 0.6134999584307556, 0.45064360915232116, 0.6746394177321864, 0.4979233104074659, 3.4870886584447467, 0.6051895144041571, 0.537460076747162, 0.0881138868200173, 0.27269684022708496, 0.8994023864927502, 0.3237609445435823, 0.6460589808077604, 0.6918274349240969, 0.6122213186922763, 0.597753287470532, 0.26339650896960387, 0.5628616048474737]], [[0.5581855938531555, 0.6064406865184784, 0.3087406212073006, 0.9116962166969644, 1.0849115760483188, 1.0650275375646219, 0.4036043674270315, 0.8256096377981047, 1.1454450966628735, 0.6234884142468952, 0.07566838982541106, 0.7935483360710254, 0.8892462597994122, 0.6669348531659166, 0.5043677012920986, 0.4352697192070345, 0.4817732951767756, 0.5121440812525111, 0.5522566934886717, 0.4211068816409591, 0.64287941901508, 0.5209731694179647, 3.4870886584447467, 0.5257109350844611, 0.5633262510812045, 0.08934651298880436, 0.2543036548465995, 0.9377316590377509, 0.30453942437398973, 0.5872109314057943, 0.6809271381370472, 0.6057425270122392, 0.5354469403331404, 0.36143485087742944, 0.7178857464472186]], [[0.5983292550047223, 0.6730595738653965, 0.30772315510526327, 0.8260683375630444, 1.1374727159790392, 1.0491060635227907, 0.3935152335761362, 0.9242119561193727, 1.0942510809620822, 0.6362578933622868, 0.07356986640127106, 0.7875154536830592, 0.963879222620216, 0.697722476452961, 0.5236767499447941, 0.4239017290741668, 0.48914787236436175, 0.5362855669542128, 0.5964411703474342, 0.38015866465356196, 0.5207667523798563, 0.49876943760054626, 3.4870886584447467, 0.47035689682684634, 0.4488137889801631, 0.10002146915538138, 0.2615327266195705, 0.8396412845057292, 0.27273055316296735, 0.5201139417132083, 0.6463562567713411, 0.622582416092474, 0.4745074312517912, 0.3709776367810295, 0.7345340456654578]], [[0.6335169642925578, 0.7160597053041835, 0.2914705904612201, 0.756230209110634, 1.1785175365699303, 0.9727480314681074, 0.3695158200203336, 0.870326541246717, 1.0232953131235978, 0.6173846668504633, 0.07165978489876224, 0.7975307171305384, 1.0268077839186112, 0.6657449665152769, 0.5202597933890537, 0.4037025225461399, 0.4609791640491061, 0.5242089529630923, 0.67033023915046, 0.3819089591139614, 0.5281725934885839, 0.46795969288728767, 3.4870886584447467, 0.567173617150311, 0.3827004921867987, 0.09902662546407667, 0.26221530055363845, 0.7343018266557205, 0.25804216523431855, 0.47204982951926255, 0.6098470628830681, 0.6089726588253868, 0.44874898379792955, 0.38453184359517156, 0.6446084232903195]], [[0.6554280588382562, 0.7536732173171571, 0.2865386629601009, 0.7318162164294829, 1.2247447977194643, 0.9261094398161794, 0.3574764281293923, 0.8384681949724656, 0.9314414369089037, 0.5710999336452922, 0.07248994430465941, 0.7855476084344988, 1.040199731303562, 0.6547219991904657, 0.5211610818471532, 0.416646385111871, 0.4789302506930291, 0.541806734137381, 0.6976614330144606, 0.35012643015267564, 0.49203040762638617, 0.455325128035984, 3.4870886584447467, 0.641212064585839, 0.32868997232358615, 0.091264256148815, 0.25253837803752033, 0.6782957775300507, 0.2624808250294419, 0.5471267094839263, 0.5566254587823926, 0.5408524261945042, 0.46125950739628274, 0.42962478094767387, 0.6556596026548421]], [[0.6674597188176696, 0.752313386168515, 0.28281638296970124, 0.7254850816036542, 1.2238542238621002, 0.9057968995482357, 0.36891187899998956, 0.7722211309380902, 0.7969080983955215, 0.49512768975690646, 0.0667892721049835, 0.6655858485566071, 0.9214742794358344, 0.6387081582347542, 0.5188920338095723, 0.41049512639075536, 0.5993779669870533, 0.5875016040617502, 0.7602668672197352, 0.317272903380728, 0.409072348449714, 0.4573326946250863, 3.4870886584447467, 0.7149270344435104, 0.29246063097398417, 0.0884588432212771, 0.2641009880624241, 0.6820688609100509, 0.28412879651136613, 0.46524594310966566, 0.45255199781206584, 0.4347280222342716, 0.47783226155304165, 0.4324277922008053, 0.6136337683957459]], [[0.6928741613082127, 0.7554370019470101, 0.28183589775326306, 0.7122260820808042, 1.201873001180112, 0.9116358291347282, 0.3855796301043544, 0.7361204416129812, 0.6321239630731259, 0.39118846256201933, 0.0675498320214548, 0.6599674955386966, 0.9122639367874874, 0.6511286668025168, 0.4687466094135761, 0.4196041654951388, 0.606241286958443, 0.638898907535167, 0.8515198298050963, 0.32144329335138544, 0.38771863230701153, 0.4394739078229806, 3.4870886584447467, 0.7884535005817811, 0.27186860656835304, 0.09451292121659108, 0.2481920839671496, 0.7469813313479166, 0.3405205497642014, 0.4391775975763654, 0.4257479254997658, 0.3650127813469824, 0.5192590638283171, 0.46973127340533605, 0.5892445303099564]], [[0.6832173376786638, 0.8152610509206024, 0.32038673194229533, 0.6232318440564917, 1.16780632563514, 0.8365619248287597, 0.3929957121508487, 0.7027611594656882, 0.44706099023142154, 0.42724726134932456, 0.07763534545665783, 0.6668852426590336, 0.9032076923918736, 0.6234742552321331, 0.5346896085793083, 0.43079701943419846, 0.6006481871816779, 0.6765269639521367, 1.0223141108981206, 0.35738111615356716, 0.3803425058202474, 0.43982663452456566, 3.4870886584447467, 0.8172538065666388, 0.3124169798208089, 0.07976976227526672, 0.2623068181821808, 0.7841075252695523, 0.40896015983837924, 0.43647243210834685, 0.31889975119515024, 0.34883831107163404, 0.5603909936140793, 0.32740935554657163, 0.5618019952280441]], [[0.7005700682765135, 0.7967603765910845, 0.2998504601328317, 0.5908987243718944, 1.1233033340949374, 0.8204399014296045, 0.3854663787630502, 0.6145072856638079, 0.40113089544702485, 0.4388218888836266, 0.16260116643888234, 0.6231094762014779, 0.7979834026472874, 0.6327554831014375, 0.5103637994918164, 0.4713041227689907, 0.5581400137987843, 0.6854098077348866, 1.0369223180194884, 0.3684483260661091, 0.3454671071575016, 0.46871305081152537, 3.4870886584447467, 0.8365522232205166, 0.31890680312457004, 0.06799990696235338, 0.29706196169470744, 0.7799989521623713, 0.45362012638787735, 0.41472882727949, 0.3048772596078184, 0.3621882064479049, 0.6100206322663357, 0.1486779483060273, 0.5515452995714121]], [[0.6958955779071272, 0.7769678845146171, 0.29233172200646873, 0.6190551544551719, 1.0143426877388821, 0.8508284012487054, 0.37222697826816326, 0.5619614103610022, 0.4209566204149162, 0.4438771149360147, 0.14987390443498677, 0.632866912409849, 0.7888316766177322, 0.6238276148813728, 0.463478125780965, 0.43905661238996596, 0.5500036096141212, 0.7122920046426515, 1.084387247167439, 0.33848247367522444, 0.32341843956856586, 0.4861348711513053, 3.4870886584447467, 0.8281313834563343, 0.31200384840617773, 0.07799273425573368, 0.3254763122673135, 0.7425563495946177, 0.4507952405653846, 0.35280522209787024, 0.3547378519230348, 0.3836996590662412, 0.6420478788619246, 0.09014435609429278, 0.5503814266425648]], [[0.6942677476392751, 0.7936407522906833, 0.29170784716533604, 0.671511296858923, 1.0019449840457024, 0.8139133252501125, 0.37832438188696926, 0.5061798230927229, 0.4648029863677422, 0.43215440230915525, 0.12029111916311067, 0.7123664624307802, 0.8371916010031395, 0.6297175819610858, 0.479090210346241, 0.43949639200999036, 0.5562279016242639, 0.7184313651223359, 1.1051858316986745, 0.3681009919941254, 0.35503687634461123, 0.4806685825510936, 3.4870886584447467, 0.8009018650078791, 0.3005031291759177, 0.11494213672676759, 0.3541604982469869, 0.7177763219844624, 0.4232171907817516, 0.3083094073692173, 0.41735705232209686, 0.43975401348790755, 0.6223534410516234, 0.07601278677605314, 0.5690606302413468]], [[0.5687648807304815, 0.8284882054317694, 0.3506628467443951, 0.7068787530209268, 1.0021696571206995, 0.8431140136978656, 0.38398713828709735, 0.4748109021405795, 0.47204327798147666, 0.4441102849808392, 0.09607819181242198, 0.7894224165018765, 0.8222330709449919, 0.6089321236606908, 0.4379133027751392, 0.449880941529887, 0.5474879676618822, 0.6950555006001728, 1.0884817821505357, 0.37653361658616136, 0.36388932588673195, 0.45035140416701697, 3.4870886584447467, 0.7418928736044524, 0.31115892121065536, 0.1657318630944428, 0.3846670068880044, 0.6956560198988251, 0.42934098794448494, 0.3102040523245768, 0.4599355064281897, 0.4446928030082976, 0.6010431977079865, 0.14637755483463252, 0.5823462258928047]], [[0.5779603999100033, 0.7707824644492854, 0.3333782190864193, 0.7602189135113514, 0.9702902582909646, 0.7838312649824458, 0.38979902525793914, 0.43634720024431195, 0.47654722841334923, 0.4548573586359733, 0.10348344787408717, 0.7825436734992177, 0.718067555570955, 0.5836873005635332, 0.44209202338999865, 0.4510951164122654, 0.5489584690728845, 0.648095235031238, 1.0170624845343985, 0.4028904865311248, 0.3883391465660246, 0.4667437022621177, 3.4870886584447467, 0.6839508712993032, 0.29796259880981724, 0.21784197829032192, 0.42393984886772673, 0.6677683016697245, 0.4475930656202034, 0.3956263599802833, 0.49371432182420677, 0.4572542656598377, 0.5566085060627108, 0.2341503167385601, 0.614145478215671]], [[0.5890768945488023, 0.7547419252693172, 0.3566459436409345, 0.8395274741376604, 1.181464888806668, 0.7481457358313945, 0.39974989405226896, 0.30737953964437914, 0.4762596982114143, 0.4197481255768786, 0.1179142050201713, 0.7351628729192733, 0.6586510598581613, 0.6103788086207127, 0.4215591046300017, 0.4311478074954428, 0.578026451987209, 0.6457165101426863, 0.9452163625688574, 0.37615099788657014, 0.3845147103958978, 0.4813031232183134, 3.4870886584447467, 0.6563943113351087, 0.3423158117613503, 0.25705968933556594, 0.4665027099106581, 0.6573385474062917, 0.4352338452541724, 0.4545326818275572, 0.5346376223369793, 0.4510027521295792, 0.537634855294977, 0.32502168235534784, 0.6056937231078415]], [[0.6353100391257303, 0.7668929968849197, 0.3813958202586839, 0.7993377836369853, 1.1948411047588388, 0.6885558792645562, 0.4051053530348817, 0.3088691276665645, 0.41834830737259954, 0.3742458225496003, 0.11491302907913956, 0.7020110082420785, 0.5897669247553555, 0.6193951147830473, 0.4448823283810636, 0.43376701491391834, 0.6435205724521413, 0.6466093406712132, 0.9357968550858511, 0.3914395931486118, 0.4428092596010293, 0.5512623196938076, 3.4870886584447467, 0.6517056262218003, 0.41312554215096503, 0.26886625142495335, 0.5109324984977227, 0.6648362738250069, 0.44118872310442214, 0.400809937653187, 0.5364611497094418, 0.4008158779086274, 0.5126081356369572, 0.40381813477894574, 0.6249983753720925]], [[0.7114171499562234, 0.765014639321121, 0.377458830508257, 0.854979343469193, 1.2224280898195423, 0.6125545191971306, 0.4045930891434106, 0.314283387711101, 0.3737885028645548, 0.32619359937248527, 0.11182133677192764, 0.6676273536539402, 0.4303985252739422, 0.6078874750615878, 0.4847804862250044, 0.4244979219016991, 0.6613544725822487, 0.6270899918260397, 0.829871732004664, 0.44626721528738833, 0.4870200212047376, 0.6211583252435268, 3.4870886584447467, 0.618500610048494, 0.4116563156784677, 0.24625142636697042, 0.46549491050363434, 0.5963187435832911, 0.484401291829636, 0.2549013781722831, 0.549960812594731, 0.34206800198066506, 0.5054369510926389, 0.4535204164271171, 0.6505251251341695]], [[0.7592111620013404, 0.7450909489409763, 0.37316983537649606, 0.8236169530241091, 1.1845868983945733, 0.5499141934266978, 0.4200854650160044, 0.3138030902905784, 0.3832840084627802, 0.3460103858290347, 0.13887411439189018, 0.6655030867165808, 0.4144049970216341, 0.6180136566242554, 0.49177387202490397, 0.4253805942094143, 0.6701690455519098, 0.6366177310789329, 0.8097821855489935, 0.42267939774642715, 0.43055015066443453, 0.5772883210638718, 3.4870886584447467, 0.565446384762515, 0.38241597002066763, 0.24021066489011728, 0.44024579990476564, 0.47241430374300164, 0.5219582134416211, 0.21893248816201855, 0.5669470516761703, 0.30654249586478105, 0.5267098230869804, 0.49625298981043026, 0.6420602353667615]], [[0.6856319351284409, 0.7014247283646988, 0.3786642093055222, 0.8427250736966063, 1.1270212723813833, 0.6417142819800506, 0.4322816570687109, 0.3079934896279105, 0.3841944847201475, 0.35624076500043145, 0.1457820956647173, 0.6436118272860097, 0.6200797387289292, 0.6041258636296539, 0.4881111745327919, 0.4721049480845617, 0.6484113785360601, 0.6312790610141938, 0.9248226805181968, 0.46904322941285115, 0.3917722836078183, 0.5246822973601901, 3.4870886584447467, 0.5183767714682037, 0.3557807042127063, 0.25617326229713955, 0.45540512557502055, 0.4232311889420984, 0.5198622323749189, 0.21890693475468997, 0.539096503358151, 0.2841901832557822, 0.5470988991131075, 0.5391973815064401, 0.6090758290926198]], [[0.6142922803283335, 0.6847691614378908, 0.3837285426875802, 0.8355729350847616, 1.0746504216843327, 0.6045366921676387, 0.3727440922230263, 0.29466670312870247, 0.38542598268236106, 0.37507296455061967, 0.2127011452605182, 0.6410165282975964, 0.4590152931229091, 0.5876352999483545, 0.4731145580270122, 0.43399740707461376, 0.637661081178833, 0.6180219066057602, 0.9635101106906292, 0.42902044856763133, 0.3818670125338536, 0.5662753715805957, 3.4870886584447467, 0.4493382007164586, 0.32801811742872766, 0.34874916319285887, 0.4327660531822013, 0.30599198802483696, 0.4827816517379274, 0.22124839029369883, 0.47536978991116907, 0.24076074487011745, 0.5340089635192005, 0.5461521137548603, 0.5761630109145357]], [[0.5717824759207798, 0.6568146712133893, 0.3993833675965922, 0.8091483208127528, 1.0213114619501236, 0.6596759971550119, 0.4523283142605066, 0.3497839970320912, 0.38221928426603496, 0.3847971323446522, 0.19538266948709376, 0.6322428065007675, 0.3281112843672089, 0.5745466761557543, 0.4546128339630698, 0.4559559482300063, 0.6490419645670216, 0.6478572297382048, 0.9778482122604879, 0.3933326521764665, 0.3681768982932228, 0.6072378667381546, 3.4870886584447467, 0.4327149882479631, 0.2902869932833635, 0.41201272145740664, 0.44547679140021557, 0.28278730528408796, 0.43589281929007645, 0.27266714175543216, 0.4290869515686265, 0.1877219074783423, 0.5229142612934707, 0.5704857525889562, 0.5982557435350523]], [[0.53051475226756, 0.6514936162228223, 0.3526142564313335, 0.7704474058267275, 1.0435866931233562, 0.6704946475832192, 0.4083533268322841, 0.3473312041764382, 0.392411865862541, 0.4944922462834954, 0.056901200861897094, 0.609777416172376, 0.336709301499759, 0.5949191389261934, 0.4300367051855949, 0.42122940381553986, 0.614513414752026, 0.6412011021959334, 0.9648238442716734, 0.4075719895875546, 0.3417297490306051, 0.4733323370076349, 3.4870886584447467, 0.47309790489575204, 0.27732075577829246, 0.47370291796057445, 0.3962059860973156, 0.2871718950130868, 0.4179267818088983, 0.32405471881436176, 0.43194617323338896, 0.21705752955115035, 0.5263790600617583, 0.6042796489190734, 0.6341623908380063]], [[0.5210428402068965, 0.60663390304096, 0.35617661783827004, 0.7829822911011179, 1.006123341130052, 0.6755098452665678, 0.43427206991620576, 0.39157524091844625, 0.3384246891863673, 0.5668484599988655, 0.0545282430583993, 0.6113408335984016, 0.28152671120003303, 0.5907876448812028, 0.3551388538377629, 0.49411508597743825, 0.5911926994251016, 0.6317912466935469, 0.9638217370500073, 0.4757760014209974, 0.36563194325051374, 0.44417455000610756, 3.4870886584447467, 0.5126578808442279, 0.2765333331979267, 0.48847330570584124, 0.3499865009582418, 0.2849617459084061, 0.418862733604648, 0.39983032446984684, 0.42740204725466224, 0.24299190319644454, 0.5489023776711424, 0.6244523142308381, 0.648970617016444]], [[0.4930808855941085, 0.5833386295701593, 0.37635539860337275, 0.78591789682666, 0.7563162498698717, 0.6990983741038193, 0.4559435014728943, 0.423526186271666, 0.33404334340584646, 0.6593876210639963, 0.05035571950532464, 0.6006486175886472, 0.2963116955339702, 0.5685027485030134, 0.3676970123797573, 0.49129652105790167, 0.5856467265945683, 0.6100991361902818, 0.970667378308466, 0.4006488419615356, 0.2776067655506357, 0.3894963030884208, 3.4870886584447467, 0.5310318750833268, 0.3096708045986849, 0.5306237593848357, 0.32280081035265007, 0.2839501061335225, 0.4245499627579984, 0.4239351917284519, 0.41256669790911193, 0.24437395717553864, 0.559526721269472, 0.6419178731050603, 0.6362748760189397]], [[0.4616568982397995, 0.5791730011748526, 0.3969813076642291, 0.7831169454023658, 0.7824945057148098, 0.6967626988458886, 0.4745337645539568, 0.5033362183512784, 0.34251223822632954, 0.7092408011729857, 0.06111872404933491, 0.6234306941494325, 0.29102545847027655, 0.5575469638922173, 0.38937567488786334, 0.5592602955132052, 0.5799086330493181, 0.625384845509012, 0.9928061824793037, 0.39817616373944553, 0.24059709266370866, 0.45165762170943, 3.4870886584447467, 0.5454417408829152, 0.3275981391532813, 0.5600785355904164, 0.3399271207361997, 0.28653654493020786, 0.43405528046749875, 0.4579929364518478, 0.4028564044846059, 0.2404679879142287, 0.5554218785222671, 0.650685380243189, 0.6361120659608395]], [[0.42544613953796484, 0.5596361400972933, 0.39597522326578416, 0.7058973158947189, 0.8299537486805464, 0.7588051089515659, 0.4725811530938617, 0.5740740783317749, 0.33613554567190823, 0.7012466654163589, 0.04907572033336872, 0.6426284040848169, 0.2717043019131556, 0.5749895077402344, 0.42473165707437166, 0.4645818098501696, 0.5729600915731207, 0.6330739569783999, 1.0512368608298102, 0.4917552657985895, 0.2440893550559331, 0.4891231889069607, 3.4870886584447467, 0.540570595930534, 0.3219676651576804, 0.5556288710195567, 0.31323588017390264, 0.28996309168202056, 0.44301919541314816, 0.49710419460327504, 0.3970330172391865, 0.23026109595703256, 0.5404744604697097, 0.6482859669719939, 0.6223018861857484]], [[0.37780249711974956, 0.5487542233420044, 0.37801680333110566, 0.7078421925877446, 0.730809304176568, 0.7196242785024762, 0.4724012555630209, 0.5228355267984954, 0.3427016806607576, 0.6990006328327949, 0.05409212564044538, 0.6349885686171639, 0.2553635845358836, 0.5751248491001155, 0.3456304324020581, 0.5063472136419541, 0.6083858757691365, 0.6411621878772448, 1.0503362758307642, 0.4957823152667622, 0.2486871168364716, 0.4864321923656353, 3.4870886584447467, 0.3341175858740018, 0.3356324458574883, 0.5336381953838016, 0.24240071255113194, 0.292364514505365, 0.4438391459405361, 0.5175514656220965, 0.4074491201685133, 0.2241045740187879, 0.5206651648188083, 0.6352919429635925, 0.6192798317558706]], [[0.46589226018678936, 0.5621022037879084, 0.36849039427907865, 0.7824750838170885, 0.7885644999966264, 0.72924297166134, 0.4615320663804905, 0.5152605021926432, 0.3655467069855381, 0.6825326088435613, 0.05198309865387047, 0.597807333551041, 0.24336917420043916, 0.5850403923486586, 0.3722546503033164, 0.5170925105811024, 0.6334310656194675, 0.7409427860311775, 1.0915112219978427, 0.5490690020476545, 0.3035266074925195, 0.4860060003322354, 3.4870886584447467, 0.26029011396907764, 0.3400893429993279, 0.5383691742408587, 0.2117571912472403, 0.30767149118170417, 0.4496822234466985, 0.5010028272776723, 0.4125896397250678, 0.23740540567673346, 0.5090662012891652, 0.6232925662019845, 0.6360654581213716]], [[0.43753732576324106, 0.5266059406941177, 0.3718161897310852, 0.8186488798798773, 0.8275210018118775, 0.7272925475199217, 0.4522884589599645, 0.5129317161033312, 0.3869945947056717, 0.621093472854985, 0.06335660405995808, 0.574255861542584, 0.1925505892589144, 0.6100064308930024, 0.36677913867150425, 0.5097908172998363, 0.6266521197471109, 0.8332451765199619, 1.078099938553795, 0.5407003090620088, 0.337695031944792, 0.6193827545169885, 3.4870886584447467, 0.0983234225660093, 0.3094722550802562, 0.5154819564962905, 0.10298923907003524, 0.3174993749475692, 0.43871040574493336, 0.5072069533989063, 0.3887624154467698, 0.24109475440717612, 0.481245471360717, 0.5899045811245749, 0.6262792971225389]], [[0.42785775730388775, 0.5118209855536735, 0.3906425497759275, 0.9639567006007648, 0.8628579416991818, 0.7194022191581126, 0.4474053270665521, 0.5538527705687503, 0.37469386485108347, 0.5778184977921159, 0.0534869426673235, 0.6136335062978955, 0.39681067862984476, 0.6351640928276234, 0.369033371022346, 0.46150534598978954, 0.6726901296367671, 0.808189379796353, 1.0873430097295593, 0.45998399722823624, 0.35378201179744034, 0.6172112649394207, 3.4870886584447467, 0.0906544468586733, 0.2842736088907531, 0.49537137720914803, 0.08593738849333848, 0.37051568518204114, 0.4557505274706371, 0.5281088534891871, 0.3933386646171847, 0.23557590057937994, 0.4882984440351954, 0.5515976340876176, 0.6113584748382337]], [[0.4260194683242483, 0.5184048646522907, 0.3672028796079928, 1.0299229525478417, 0.9051879838481622, 0.6754502500084312, 0.4591646002229828, 0.5353393519644332, 0.3718756732865175, 0.6047860151873132, 0.03657296201195421, 0.6331949079245607, 0.4036466621810889, 0.596179453713461, 0.3588506464086185, 0.48466760923050844, 0.6605329182109105, 0.8309590334902522, 1.1059379188864709, 0.4647344879691626, 0.3533882731663497, 0.6612943665453518, 3.4870886584447467, 0.09225635063312551, 0.278946891841658, 0.47486015690276534, 0.10008860503458555, 0.40655808833419294, 0.46022996764643326, 0.5531868402624657, 0.39085624773777927, 0.24101917211837837, 0.48470149267345813, 0.5241399769387705, 0.5926749014578495]], [[0.4547706262131543, 0.44545537451975326, 0.34208428388395473, 1.1805905729111077, 1.0257516462802967, 0.6864443050806213, 0.35973214878044196, 0.5167470159773396, 0.3706951039916472, 0.6802758450351785, 0.030813722635702555, 0.645666409608405, 0.4073014565251592, 0.5710493576368189, 0.3582908162684767, 0.29886307114276045, 0.47052735551922675, 0.6611041495063027, 1.0889541812263834, 0.4220893228403322, 0.3071831929753953, 0.538585937157092, 3.4870886584447467, 0.13919559666900588, 0.28058234724485465, 0.42298442242100276, 0.13258386763293178, 0.41441979331821704, 0.45848083483220375, 0.6029441255906217, 0.40477898903393905, 0.27085646739227454, 0.4784861803102862, 0.5658763703023828, 0.5822375884191109]], [[0.52945261615879, 0.4597413772494452, 0.3409983877783704, 1.1992740095512913, 1.0194296639331473, 0.7017810982504051, 0.4032757265248814, 0.4178469197534525, 0.4164945307690707, 0.6109946078117723, 0.03198088207503706, 0.6472605229631613, 0.365215926645377, 0.5349487369153116, 0.3349401052283141, 0.34261632572102696, 0.41696519623803163, 0.7240078762490123, 1.1361634723634249, 0.5667633022301127, 0.36918594055361986, 0.5906818543206224, 3.4870886584447467, 0.16882074049667423, 0.22974507944312053, 0.3433070897534005, 0.16252781481106668, 0.4193463066502722, 0.42934492835098925, 0.6355714144303904, 0.4536276165221682, 0.35146680881740644, 0.4584137659748665, 0.5893463539305264, 0.5957772221700749]], [[0.5979378451231565, 0.44522829779035616, 0.3321828908624095, 1.0944688266503007, 0.8367460191207489, 0.779910964071497, 0.38379791218655723, 0.3628788097258797, 0.46934954021164077, 0.5255417616144389, 0.029581150946130747, 0.6568201750251507, 0.2862914342254134, 0.49701041417511715, 0.2867401341527798, 0.2636705862633859, 0.5708458872430111, 0.7140408512005305, 1.201713827856559, 0.44363897893514925, 0.4379047503445748, 0.624165570724763, 3.4870886584447467, 0.16185903426514758, 0.22072116800091268, 0.31365208570421466, 0.15011131058014612, 0.5210168439428772, 0.4011852011073987, 0.6767894440410303, 0.4839478494463442, 0.4505993018863232, 0.3913422235989702, 0.622304204239253, 0.6249147424887522]], [[0.4774478173829575, 0.4622192298722935, 0.3196577716340845, 0.9297800359360885, 0.7242084741661793, 0.7390600578424409, 0.38654217729275714, 0.2749898645817451, 0.5829232693151419, 0.5412662887287574, 0.02733568387973126, 0.648464585975834, 0.44955840055857466, 0.48648227526822757, 0.29257471334992036, 0.24070571281831787, 0.5752093634683855, 0.6613986657524457, 1.1372129987085977, 0.4224271462322272, 0.4515815100204387, 0.6669678571252042, 3.4870886584447467, 0.11661512413770231, 0.233312394921938, 0.3051134345407883, 0.15270036111974336, 0.5702350326578554, 0.38852044207073355, 0.7463007062613258, 0.4630884936268954, 0.46679329768860167, 0.3564055295393638, 0.615764845818114, 0.6624060652748389]], [[0.5781191909710377, 0.45633215439673175, 0.30798911394060907, 0.8062286852270182, 0.6511381528462538, 0.7719282764631737, 0.3673389562401671, 0.2727609273820887, 0.6756125662314032, 0.5325409049567078, 0.01595024450652887, 0.648531255918287, 0.35119232817645585, 0.4937691756531988, 0.3157735937673395, 0.23831100283002205, 0.6854373438921613, 0.6712333733669515, 1.0774666914404387, 0.44359941833770233, 0.43059801420346694, 0.6856486449812975, 3.4870886584447467, 0.10104226323728516, 0.5299452614768347, 0.3062147334389843, 0.148633379572229, 0.7925857555445938, 0.3863434073298494, 0.7852415773598107, 0.4906567409245252, 0.4761201315265782, 0.3260662402339283, 0.6271089122202272, 0.7017823630602211]], [[0.8059700837879371, 0.4423248457865865, 0.3052104358476924, 0.9022389137091966, 0.7924311876082233, 0.7128219051418153, 0.36270361541315804, 0.4024408755997425, 0.6623598688048913, 0.61836846774482, 0.02881774060132839, 0.7132919654867865, 0.3353314053609096, 0.5048249581385119, 0.36281971527077383, 0.21202141248637557, 0.779627088011693, 0.6348118838770384, 1.054831575264165, 0.5621448062839054, 0.5349646020833084, 0.9264404252068099, 0.2085996459192768, 0.19076604658268215, 0.8101980366231029, 0.39198063945449896, 0.2701068135182161, 0.9489653600550555, 0.47522789156880685, 1.301540538828791, 0.7513856947039838, 0.5532896630808636, 0.4804511932991817, 0.9994144865146322, 1.2569826735807057]], [[0.7893301262533474, 0.4396789165305124, 0.2973366811995711, 1.1160751817264218, 0.7675473407972918, 0.674543764056245, 0.3618936982161506, 0.671197441741977, 0.6093974432795535, 0.6162534817885573, 0.056440436614570114, 0.719346985374853, 0.26984020551605903, 0.5828015126470116, 0.299326869688114, 0.20508301409390056, 0.7807694572345087, 0.7050083861605191, 0.8753276521750957, 0.5982958097661065, 0.5092795414494522, 0.9903459950516448, 0.12217854890733483, 0.18584250809538055, 0.9640255811209254, 0.28741470541257275, 0.27430614450306684, 0.8437620704612787, 0.4489692656516052, 1.386291802898802, 0.8682227721102698, 0.4465891471973856, 0.4964322312319861, 0.8671974660446782, 1.4257674144515997]], [[0.7988828966028201, 0.4236895531177861, 0.3346119268599649, 1.2038000098249115, 0.785059611106109, 0.6561987035264001, 0.39924824926865876, 0.8824755271876682, 0.666145093421462, 0.5879931556497435, 0.05376689635734915, 0.6962871508706737, 0.25377303111486926, 0.6945938048779923, 0.3785366504027686, 0.1964233711099428, 0.7935358042469476, 0.7008465230677066, 0.863791324113582, 0.5273481635835768, 0.6618917930015601, 1.0206946301622168, 0.12436784669937975, 0.19272355549264863, 0.850467208155967, 0.2891209936639403, 0.27875349136998817, 0.9518769641007068, 0.40211966995028553, 1.3842683023742415, 1.0162666543352197, 0.4442681870731703, 0.6139606852962651, 0.8105782162089634, 1.4323949175944328]], [[0.8261017248665568, 0.44172806243583185, 0.31887975463146795, 1.2276000981580821, 0.7591500662918391, 0.6629104898305738, 0.38344278970380985, 0.9453896674855418, 0.6998617850100332, 0.5695377369238934, 0.07073026553529178, 0.6613104450787537, 0.2341816683118595, 0.6578393023154712, 0.3587626943790412, 0.16199027090770302, 0.5394591490173066, 0.6481665692189968, 0.8191366012137442, 0.5442492268333259, 0.7568257271309151, 1.0774390323475433, 0.0777732024780318, 0.16739531284458653, 0.8019702513839296, 0.31027661517749383, 0.3116302131039791, 0.861885164736935, 0.35090559017785133, 1.357770649781829, 0.9724620481841808, 0.4632155011907494, 0.6579746816359626, 0.7509323258287445, 1.4065164455931496]], [[0.8892599084224876, 0.4509132231336942, 0.320335431886775, 1.12451071859153, 0.7486701361311584, 0.6841718871186353, 0.41765478763506136, 1.0939560778689266, 0.7818817786403214, 0.5638459412536849, 0.05551690835194831, 0.6698750745481357, 0.18363351699831804, 0.6347533894348687, 0.4520404737619432, 0.1641561203385402, 0.5947519292213019, 0.7027927503585576, 1.0158737933914592, 0.5500973926903681, 0.7306187537887083, 0.9485465711763348, 0.06476218663448849, 0.1637881749492528, 0.7311807770824041, 0.3101017104458225, 0.36427387528844646, 0.9015043445002981, 0.30667447238845535, 1.28496501655635, 0.8939365115124163, 0.4250403635015723, 0.7046237636980426, 0.7254743113475113, 1.334888733492869]], [[0.8814071774663108, 0.4451284737167833, 0.31944073819738633, 1.1295142862792962, 0.738804037283182, 0.6819720344462568, 0.4253784346277141, 1.155690369441022, 0.787817583904213, 0.5379456884521512, 0.06253382993780301, 0.6667024673818229, 0.17711115930407978, 0.6649052598830039, 0.3429851539313015, 0.17976882485929505, 0.7800612158961394, 0.5813340098874817, 1.136434534208141, 0.6046944221375632, 0.5363678457829177, 0.8927886908325944, 0.05631055528449872, 0.16150965967218095, 0.6665762655568892, 0.3493930178264191, 0.38716072062132934, 0.8569409925515175, 0.2936264939971372, 1.1212921342254476, 0.8356174097828322, 0.5140320199890667, 0.695853544407419, 0.6731679515958888, 1.1504340842577583]], [[1.0529237118585208, 0.44085369520596895, 0.3289789365222022, 1.0988355953301157, 0.8019847182224865, 0.6850726517122689, 0.3956204227987512, 1.239106021461914, 0.8209486587185968, 0.5167859089814202, 0.06949638343690347, 0.7120270803983277, 0.19741282334894233, 0.6157022634973092, 0.5037248934653966, 0.17470013199062828, 0.8876070651138371, 0.5402411252854296, 1.286136612619591, 0.5426189155806586, 0.48163187773111027, 0.961442229301515, 0.0524949973255326, 0.16048215294399149, 0.6171368404755477, 0.3808689761495381, 0.4074626298871691, 0.8794056874682737, 0.2835396178768884, 0.9334524238978177, 0.8012152691419658, 0.6303292377849755, 0.7200446451385888, 0.7465387397068282, 0.9603432984628983]], [[1.1083533450163021, 0.4023031969157307, 0.32972705649945633, 1.1158849989398623, 0.817517013807368, 0.6441079442198216, 0.3545224227049596, 1.330261525714549, 0.7667012769189763, 0.4467929004638086, 0.08928566761044043, 0.7323003503636927, 0.22598459046095087, 0.6660072833685896, 0.4159604246415277, 0.22092351935165505, 0.8336380698354835, 0.5231244648919136, 1.2075283837352697, 0.5310618611114466, 0.3239713865373091, 0.949526014985616, 0.05130246327633087, 0.1547924927106924, 0.5037160218087685, 0.38364209485131107, 0.40195148680688375, 0.7202884631398079, 0.353141206927076, 0.7287980757078985, 0.7511847496966821, 0.8133073295740054, 0.7938389471101698, 0.7264212813204398, 0.9124137458897172]], [[1.1199637162095564, 0.3964620725144998, 0.31948548158615603, 1.0611563802360855, 0.8519742399007317, 0.6696225843913579, 0.38214108299427196, 1.3237527586005498, 0.7515590348816428, 0.3536765314324898, 0.08490882754724338, 0.7268934277060634, 0.24398450550292766, 0.6668300228194532, 0.397662295906721, 0.21973509435985766, 0.86788361001938, 0.5490672327202165, 1.2354506000077428, 0.5701331194188163, 0.3523844130041646, 1.00435598216546, 0.05241336929046976, 0.14815151659709502, 0.46261793502188847, 0.39435153812408774, 0.37767229734268914, 0.6490128444704937, 0.4102347929543174, 0.5925497520291404, 0.9306883318483288, 0.8951260093375781, 0.8357572952374298, 0.6999490422481041, 0.9745654416255998]], [[1.15235793215501, 0.37857500956093526, 0.30912224195716814, 1.1063945485887947, 0.8787840291937737, 0.6411904633515737, 0.397845129873973, 1.3303928668295306, 0.7344545182845228, 0.24323923429674732, 0.09434946334678175, 0.7076791650939652, 0.3003116674609358, 0.6589026444009922, 0.44131236285765785, 0.2263128848910957, 0.8605329289910909, 0.5586323718575733, 1.2892756832083452, 0.6189454743977857, 0.3271728026837714, 0.8868008800696677, 0.05536282046707575, 0.14494023717532833, 0.46245076787424017, 0.41653253047007005, 0.37757007469103343, 0.6121844354135282, 0.49495960314423054, 0.6915009408211372, 0.9303703905845496, 0.9306297144977735, 0.8547202757230115, 0.6562613824991372, 1.1110184995160353]], [[1.2198954798967643, 0.3696002660652534, 0.3163365723834301, 1.2862055972890194, 0.9454801594884404, 0.6316349961807861, 0.4327417995440658, 1.3601593729557957, 0.6296568971785124, 0.25516316164515196, 0.03571570977496524, 0.6698301214988003, 0.36692433266561647, 0.7080905014561543, 0.574613794332981, 0.22178483708025432, 0.850282512791188, 0.591676062481877, 1.3563578986793439, 0.662894571711901, 0.3430173571827556, 0.6849172383671852, 0.055970873504161155, 0.14100239929641153, 0.462481116104989, 0.44750571399840294, 0.36702660372226487, 0.5318899126037631, 0.5728929958698677, 0.8822150395508737, 0.9758010763927212, 0.9591776166795853, 0.914552917010814, 0.6068788366064087, 1.1729455367682982]], [[1.2999263861562094, 0.3512949026676968, 0.32593906550700275, 1.350055849365122, 1.0516900685740942, 0.6241066912105172, 0.3865930858152442, 1.382683103524212, 0.6843244160525089, 0.30049172786639056, 0.029893975646933107, 0.6570216654938882, 0.4674315735710005, 0.7196091894807009, 0.5661650816554133, 0.22073990604972182, 0.8561506604645304, 0.5520057540685205, 1.2591036991335436, 0.6654176210245384, 0.4069615616997829, 0.7707752321799434, 0.05804069373250665, 0.12826501027085588, 0.46153320081691884, 0.4449149168631842, 0.3395899460047218, 0.5251056550080755, 0.5755123837452328, 0.9633158673903324, 1.0295191693271368, 0.9702313429917215, 0.9151429632194228, 0.5728031840113694, 1.12658640566901]], [[1.2401202907727806, 0.31921935049363215, 0.3388378394541891, 1.2908085724336809, 0.9336808913279827, 0.6487648255335643, 0.3856081898950795, 1.379777452923218, 0.772155008300379, 0.36082832737243753, 0.039776771897838265, 0.8027557997626253, 0.5667791590796677, 0.6774867999484766, 0.5277940842697183, 0.23020401791989537, 0.8503530904413725, 0.5574660446236462, 1.3329402466774156, 0.6435344496042129, 0.41289645994278484, 0.6568157592495665, 0.058571049521009395, 0.12539098814752211, 0.4596029603306431, 0.38004900691776455, 0.3234018233276339, 0.5368149774588997, 0.6548084860819146, 0.9729827249914724, 1.0401817744900859, 0.9940122022144048, 0.9437732249071799, 0.534008459272584, 1.1826787500084226]], [[1.158614890483309, 0.34786382526439913, 0.368308123343809, 1.314230154423846, 0.7771007836479529, 0.6502494248826869, 0.40064000831049407, 1.3358870848796007, 0.847026963610208, 0.3905116012184231, 0.05977447916535186, 0.809182809483832, 0.6134326090439419, 0.6724423291935268, 0.490301631491432, 0.21605753351336837, 0.859916167892495, 0.6288269464790773, 1.4502080397397115, 0.7467947612614471, 0.48682939403865866, 0.6656994719524458, 0.05974950152328993, 0.1027610987158809, 0.46025767236897824, 0.35576155304986734, 0.31481314300599117, 0.5582487425224194, 0.7363866921005229, 1.0162238149257306, 1.0705222134559969, 0.9943632952495005, 0.9457786371919212, 0.49858286657635603, 1.291334423968419]], [[1.079792689625191, 0.37383469859279694, 0.3508146520559561, 1.230476969345222, 0.6533470114713839, 0.6533595974712406, 0.40635135233567093, 1.2269100494296445, 0.898803696546373, 0.42082098838317367, 0.09371044893284863, 0.69846216392494, 0.6581090346671299, 0.7399407544405996, 0.4667301209119018, 0.21461422317954676, 0.8656458010616438, 0.6340370527968319, 1.3465753906461013, 0.8041393285523181, 0.5062021694592971, 0.7018609942960689, 0.055294470319797874, 0.10825861894036193, 0.44887690189778506, 0.3173771387981109, 0.32089143208450643, 0.6155075757407298, 0.7826077920917285, 1.0205159635792107, 1.1050190404795996, 0.9703453990607024, 0.9668134553172842, 0.5156804680279241, 1.3888801214339153]], [[1.0939244071008158, 0.4360637400161571, 0.3125112064551761, 1.2747835242473688, 0.6159378109307807, 0.657813660603473, 0.4190224588724941, 1.1406507774108183, 0.9458979605236042, 0.4651397491308217, 0.1269472884743061, 0.7227734923088909, 0.6315210310012105, 0.7820072690949602, 0.5130931766120987, 0.20679299337876317, 0.8700406348809859, 0.6752180372917654, 1.2121718288506946, 0.8480792239339434, 0.6186326127943533, 0.7550046397993446, 0.055155354821749915, 0.0997576512249746, 0.4674691024010169, 0.22863707624038937, 0.3316212654034555, 0.7369204239367962, 0.8807281897622999, 1.0549279678843493, 1.1767821915066463, 0.9826401812363745, 0.9692088941631959, 0.5700657004126923, 1.460715590246693]], [[1.029251519642573, 0.47245678716651984, 0.32619185512724636, 1.4138071670508228, 0.7402383555165233, 0.6531525658627447, 0.41083255522176587, 1.0567623656001708, 0.8440222770841382, 0.4618764965484695, 0.12990313396673303, 0.6939936629081427, 0.5824000712249553, 0.7632958216070306, 0.41534281795517614, 0.22694566063099236, 0.8608371586202435, 0.7092278012432617, 0.9585631139691051, 0.8901312934114424, 0.6514136723536104, 0.8077299988888533, 0.05096664524145798, 0.08934208482627065, 0.5328421602878723, 0.18869630217091993, 0.34716400803865194, 0.896163704713357, 0.9052487762819134, 1.0572135437644286, 1.1275846980630777, 0.9961929751241763, 0.9752701322421393, 0.6244061573714133, 1.2724886366965187]], [[1.0015591191031727, 0.5086075250428648, 0.3396246763858204, 1.5834355689788886, 0.763777712994063, 0.6482080003206979, 0.4075850210214138, 0.9797332174402393, 0.8469634330374467, 0.4167536987923155, 0.11350091087419029, 0.6548080294541239, 0.5504323392548219, 0.7676446447935037, 0.3557842316819918, 0.3270712206539892, 0.7921210412443982, 0.727782351237074, 0.8955158196994493, 0.861670818365597, 0.7027062069049256, 0.8337209599532156, 0.047566068464840225, 0.08796825065408559, 0.5505624695594337, 0.17909272646925414, 0.37209823966353667, 0.9271334995701728, 0.8068882483172364, 1.0511179408721032, 0.9738907722435463, 0.9968378362937803, 0.9963156612731227, 0.6287146786004476, 1.1956935162319577]], [[0.9042632947879921, 0.5437957318225237, 0.3863914993450466, 1.6373637230500893, 0.8097654012012516, 0.6522683167869205, 0.3900070186778493, 0.9421682776081092, 0.8226030893292282, 0.3979382056868148, 0.08820846303747144, 0.6590640697521158, 0.5240789206578734, 0.7587927572426063, 0.3478698906376313, 0.28668311106023125, 0.7604634022767467, 0.7715032289062047, 0.8624424978370211, 0.8796603990860299, 0.7449798515191529, 0.8423630807718323, 0.04968991925814555, 0.14116791009736385, 0.6124552042483351, 0.20338242729935008, 0.38820168684105033, 0.9764491260485659, 0.6927231738327904, 1.055761148061372, 0.8413034309579579, 1.056582683436285, 1.0102402380453412, 0.6811236409177508, 1.1440141593108726]], [[0.8612684736854137, 0.53864072984299, 0.3968780606062564, 1.6376727408323277, 0.8397875557092849, 0.6573893570992455, 0.3799266020573756, 0.933914291151084, 0.5689068794070931, 0.3877018218534743, 0.08519908495109449, 0.678724046351874, 0.4802456034430349, 0.7661794589431534, 0.3722915080276984, 0.32881024021846555, 0.7716141431069925, 0.7724469027747091, 0.912582430021122, 0.8632770262641524, 0.7280213025543857, 0.8799123553681942, 0.051106576685280826, 0.18855804670076748, 0.5797179641762513, 0.22968712041671374, 0.4001697565170658, 1.0307250875863057, 0.5597247739751511, 1.0687561964283434, 0.7547493340463765, 1.0086408840160301, 1.0115748617919527, 0.6679562734399966, 1.1772779027543547]], [[0.8534511581014457, 0.5502647639240738, 0.40067548443286044, 1.4908329903778497, 0.843668121065533, 0.6698408971474656, 0.3824768416976804, 1.0593636425342208, 0.6686201076210088, 0.4833700152835141, 0.09363611302711479, 0.6668190530053466, 0.4945823402850319, 0.7945861214244205, 0.4095471628882228, 0.349894491631103, 0.8444750096511928, 0.7573698794451159, 0.9596140515337233, 0.9513345410023121, 0.741573863711842, 0.9535495254986843, 0.052836520638474306, 0.2594002049916277, 0.5917749462349172, 0.24994578228787506, 0.4421215478467979, 0.8900009411111295, 0.4337338592301931, 1.0379464794885085, 0.6421414164969432, 0.9432277483068229, 1.0294362031260045, 0.616276642707857, 1.2206753464128433]], [[0.7815060150196043, 0.5994762840167991, 0.3946615352339703, 1.4425248471769554, 0.8365335685744587, 0.6580162221219236, 0.37192389087591365, 1.465515535770317, 0.8160724017875459, 0.6500778833068377, 0.11792125906576693, 0.5837319534051406, 0.4996750755755438, 0.570815722400813, 0.3803790756982699, 0.3221250149965697, 0.8996581066472991, 0.8153876890428985, 0.9729614572390013, 0.8392755806894474, 0.6697168544250982, 1.0677195804559552, 0.05490832184862444, 0.4029090772630667, 0.6095571404918947, 0.2716555884573263, 0.7919322946573584, 0.8810544812280727, 0.4035185262391811, 1.062473955587079, 0.5518911711311357, 0.8242259835031858, 0.9921703610785453, 0.4384417749092834, 1.2054261150191667]], [[0.7177805018579013, 0.6029618977466241, 0.3695800428138329, 1.3294647582114485, 0.9508850523434205, 0.670324613387681, 0.3720013164785688, 1.4014279786742319, 1.0026455521856277, 0.717432686337395, 0.11430730489695475, 0.5294293193603417, 0.46780448633294125, 0.5643538598871434, 0.416863728437833, 0.3429860445426806, 0.7792451915622051, 0.7236181627525208, 0.8275124512454808, 0.3357215446483349, 0.38102021491062044, 0.7889156218215807, 0.05574000492075784, 0.30230833034591664, 0.5686981717196347, 0.2521900292368069, 0.7878599060533861, 0.8557617597732164, 0.3801159116487715, 1.054223743403106, 0.5429981580676351, 0.6586958287085297, 0.9361013853016964, 0.3781020076412194, 1.0631675071991817]], [[0.8229876208486839, 0.6417760474740363, 0.37955300944241044, 1.2559883174467594, 0.9464969860104001, 0.6720033930885041, 0.3812517479316528, 1.2769553407482896, 1.0936138453989461, 0.6629866550662491, 0.12825232618108595, 0.5288279705701255, 0.4756814837795979, 0.6098283927684021, 0.3508256467319256, 0.33636502018237674, 0.7353578620607751, 0.5970835083482788, 0.88209023469793, 0.19069337409882614, 0.2680443079925008, 0.7455647744148648, 0.059281182186903814, 0.32785289845041377, 0.5482600044973096, 0.2616228376630467, 0.7637507823143386, 0.7079637380656405, 0.37260595704532407, 0.9604977236966334, 0.5167164263398221, 0.5146609311210842, 0.8732759456506567, 0.384353695763873, 0.9928291957207188]], [[0.8656209348651872, 0.6917303238666678, 0.3880424193033993, 1.139743887905756, 0.935879014625993, 0.6939272839733064, 0.3886050229623065, 1.1851890438739308, 1.0791263306774863, 0.6205173255289175, 0.12734034244313414, 0.5045408019391224, 0.5487712574810721, 0.5698295402834626, 0.4011784019769369, 0.3568648442547738, 0.7309801868144027, 0.5763228252559747, 0.8941792603461246, 0.17126365450723174, 0.2603491923073873, 0.7360023549100675, 0.03602387103664667, 0.336745802927777, 0.5364923470562926, 0.2711327462697064, 0.8381659438615253, 0.7473547895135689, 0.3774929268560143, 1.004548310876546, 0.5270461397644859, 0.41651457304982603, 0.9063571990119164, 0.39608007906425147, 0.9813775578561024]], [[0.8599449082499486, 0.5890633203035165, 0.35985086284626744, 1.037592883465577, 0.9461647029916348, 0.7006609394505904, 0.3929681923048829, 1.2092192364152805, 1.0662760215116158, 0.5961115509843018, 0.1348497884647663, 0.49453417755115087, 0.7159017680071482, 0.555567024861461, 0.3490272592185983, 0.3849451712803981, 0.7593505865832014, 0.6606191929274515, 0.8613170307889761, 0.15060793419654564, 0.29303228255991154, 0.7403345668513112, 0.03470878812374918, 0.3893190933926988, 0.49097080694506007, 0.2727551619217241, 0.977259823358152, 0.8244129814201243, 0.3484848690451432, 0.9977707099971209, 0.5756390799965401, 0.3264734781914328, 1.0517447784647704, 0.4058183930825477, 0.8848573136609781]], [[0.858574623959599, 0.5867562581128972, 0.36787147671499726, 0.9740931090553989, 0.8715687501632089, 0.7202804082645, 0.3749925047553251, 1.2909359866559855, 1.1183026840750356, 0.5141200339192318, 0.1354851712157485, 0.4723354866873942, 0.7734657818670487, 0.5276861300322757, 0.3483345977601514, 0.39295880257407456, 0.8022466332392019, 0.6820676124378332, 0.8720141335441445, 0.09402656821817106, 0.2549173899038093, 0.5162005496043739, 0.03606311221242173, 0.43017289016059784, 0.592178972512695, 0.2723546225003952, 1.1240065660619796, 0.9603338280699468, 0.3385946151202148, 0.9799745497440231, 0.6832226719659271, 0.2858186433977227, 1.218554389933684, 0.3984399637974745, 0.8315175064538914]], [[1.0362711393398314, 0.6071374892123242, 0.39478644658330614, 0.912960525408789, 0.7219743568028428, 0.6927030514895473, 0.37935236800722766, 1.2106616373253611, 1.065533115316291, 0.49115178955525574, 0.12019074007123952, 0.46568117320605906, 0.871169076957852, 0.5504912465292968, 0.41427319249873185, 0.46152627320437495, 0.7058128262750909, 0.5672084248188567, 0.7315071416656322, 0.07360729006259203, 0.23211709239889086, 0.42223934430448823, 0.031978699917628264, 0.39479168158657774, 0.6845382249548292, 0.2815360655332919, 1.0117106638522324, 0.8166652345924335, 0.3596072620669443, 0.9602342920724224, 0.7291705288108314, 0.31420014222657766, 0.9893417162425169, 0.38638820087489495, 0.7500126188013778]], [[1.0936556654902685, 0.6238359633424946, 0.38308131784517985, 0.8662533754795401, 0.803322789919271, 0.6968203163162558, 0.3825104135637438, 1.1226696580118793, 1.0037989181821498, 0.5087686578207428, 0.11736790663304641, 0.48978454812158023, 1.1275639153750094, 0.6064391968982821, 0.41080340547088723, 0.4761128396450649, 0.7245813188073932, 0.565443908625695, 0.6999339730305406, 0.0803365511133467, 0.2199305601078308, 0.41668287214391136, 0.033779027090273486, 0.4037451429829648, 0.8818181620614942, 0.3475350821963531, 0.9088674838037947, 0.7703506665902877, 0.4027310975668299, 0.9606889707507289, 0.6753804225062241, 0.37702931715704135, 0.842061625481056, 0.3899348272637877, 0.6394436901560414]], [[1.2573768412601418, 0.6470166317012342, 0.42470871129857435, 0.8292986043046363, 0.8256964406387128, 0.7062035204762929, 0.3785206626196809, 1.0410038597192692, 1.0186303771087224, 0.5399381590247823, 0.1052484669982043, 0.5059069992002061, 1.0942017806823419, 0.5679186079713903, 0.41179851147899665, 0.47339714167433244, 0.6828109736145996, 0.5817012502275029, 0.6763151825352971, 0.06900360514105147, 0.21949102733316495, 0.42996726905782623, 0.03799046346704785, 0.40999295501386956, 0.7382337026156603, 0.3960252034607053, 0.8605637227043983, 0.6460984753731356, 0.4164228331494536, 0.8818876821089591, 0.6451540598617521, 0.4674574321422861, 0.8003717405940053, 0.38255198361392434, 0.6282374282426099]], [[1.3212456351119917, 0.6474568732936865, 0.41834793960288286, 1.0291588019176734, 0.9003207058933188, 0.6968448908159286, 0.3696751661987368, 1.1454387615004311, 1.094342816629778, 0.5990145046962304, 0.11659347201565218, 0.5013151598839574, 1.092862102997392, 0.5612531076691589, 0.41813855651978704, 0.4829476010338084, 0.649302938280606, 0.6091284553543515, 0.7079263179310893, 0.05411062658335175, 0.2152885263713777, 0.47448731918328113, 0.039139552515671694, 0.44069437722410587, 0.6159642591504473, 0.41466895214594235, 0.9524901082520179, 0.6455080672162892, 0.42870663956726807, 0.8343807938822234, 0.6237307681564989, 0.49247194042756437, 0.7799882353107122, 0.3860218206550181, 0.5995496083776627]], [[1.2991181907361615, 0.637058117357716, 0.41628941426506927, 1.0992996928695458, 1.0234320212907146, 0.7071438490749703, 0.3736268343240875, 1.2134825717082247, 1.1963287157347153, 0.6726148590022394, 0.14662963437632343, 0.5179789760243909, 1.0020242188964013, 0.5229789688741596, 0.4052029776304265, 0.4808367676202595, 0.4890664146084317, 0.6552677241657479, 0.7510344103716998, 0.07824726119140074, 0.22082748857362622, 0.47511536548493805, 0.0584874485343349, 0.44852001765806915, 0.5020529747569895, 0.408865408088977, 0.8241310552312306, 0.6258880818536781, 0.42639060851081867, 0.8053295085423885, 0.5740833570129382, 0.45397563569919963, 0.7929170306807993, 0.40487593126806837, 0.5560285600954598]], [[1.3242647794497755, 0.5768307786969131, 0.405859892588569, 1.0911121900901253, 1.0492518838018807, 0.7022167340233698, 0.35643054621181347, 1.1693833593902878, 1.3293993210247244, 0.5979865895544739, 0.18383760293902873, 0.5162538740784062, 0.9125395248757429, 0.40026062565378173, 0.4075861045386538, 0.5499012114428773, 0.5446806720891744, 0.7356071898547838, 0.8628457558166875, 0.12391581921039879, 0.19234924869308467, 0.4675791972158989, 0.0526305767006108, 0.36632665508419576, 0.383738825120401, 0.4062596318389153, 0.5463095467707462, 0.5446879233960269, 0.3974678228354916, 0.7873944836736784, 0.5841567810781841, 0.42311759028911133, 0.7918102458234013, 0.40220574550346383, 0.6366703984344835]], [[1.3580794616914384, 0.549778850120393, 0.4166106420491118, 1.105988175513717, 1.0860073588601853, 0.6948040349487404, 0.37442706947921667, 1.1098940669148698, 1.4316063603840528, 0.49572992145631617, 0.28486039772644567, 0.5128413181411619, 0.8297303648956503, 0.40916694982828006, 0.42682538146690013, 0.5387753839433536, 0.5627865519497812, 0.7191918578828526, 0.9139277391014071, 0.11185899129052508, 0.17209088192840766, 0.45602327266786274, 0.05515745056148971, 0.35149829558189893, 0.35815341266654943, 0.3971592001481633, 0.4715005556462996, 0.45050451926810536, 0.36887243818848725, 0.7964922629862048, 0.7359769818109461, 0.3891733235090994, 0.7894333011028141, 0.4621530365169709, 0.7789429048782579]], [[1.3891066877488076, 0.546232462889608, 0.433905957242458, 1.140209905994579, 1.159708066677581, 0.6734401503373877, 0.36105135757148543, 1.138492215622274, 1.5219984922708534, 0.443063309394648, 0.16719606702306053, 0.5125432799927787, 0.7627309284223518, 0.42005882075066076, 0.4265584892739079, 0.5115883280055898, 0.5593138194406551, 0.7777360853285672, 0.8913242798840592, 0.09095031744390375, 0.17664750781170724, 0.4397259981846755, 0.05342605135546072, 0.3630801735374289, 0.3339440881430725, 0.40445520133882085, 0.483887556437509, 0.4332877960299843, 0.33715786036456075, 0.7827895478658609, 0.9934710638827833, 0.3712931939698302, 0.7459745737405294, 0.3753301809064338, 1.0086817602775104]], [[1.19525172209867, 0.5204380224155272, 0.4424665752238134, 1.1858593321744364, 1.2622893525472412, 0.6871779090553947, 0.35045538536180343, 1.2129303260064919, 1.5853253686513926, 0.40140403079571485, 0.14944001953861047, 0.5112828182829223, 0.7651065330303015, 0.41236276133393573, 0.4068752648407588, 0.47709446670983036, 0.5370861564395351, 0.7782857951438207, 0.917497579948686, 0.08693780929593126, 0.17676104872878667, 0.43219478835313074, 0.05053686671431015, 0.3618508725874599, 0.251995064081433, 0.4072709721397921, 0.5730701954895893, 0.48859578972638723, 0.34715990481301856, 0.726710509096971, 1.0466390453977767, 0.3766144100304154, 0.6399793270632672, 0.4415852586081409, 1.0931927381836892]], [[1.1888654424098404, 0.5435050149369332, 0.42614182489028135, 1.1953444483190792, 1.4211313057027344, 0.6779885181366592, 0.3451039466824866, 1.3058230061636946, 1.6312272616819161, 0.37698927895182116, 0.11846002506407496, 0.5067996869727894, 0.8673774545673322, 0.40528125512674973, 0.4329053051745945, 0.5664399243688351, 0.5053083872847294, 0.7538852118623818, 0.9589024949620355, 0.09217675369514317, 0.18229004184520597, 0.3601689688639216, 0.05072356164674503, 0.327929488886057, 0.3969146591075206, 0.36348536500134965, 0.6827230321790166, 0.5489746257458006, 0.3522540150242918, 0.7183566240293946, 1.052086185811282, 0.4260191040677991, 0.41211214638329313, 0.41789461270222794, 1.191919726684661]], [[1.0825124888440676, 0.4740527453784519, 0.46129451714962755, 1.2341801855445527, 1.379960348679747, 0.730147574162755, 0.3495304807648469, 1.3085207840300297, 1.5525654996472358, 0.3879793922411159, 0.10639724186348795, 0.5225096889819238, 0.9405468250532306, 0.39152530014612746, 0.4325662972437673, 0.4779010337351145, 0.49743594835914595, 0.7728174704786925, 0.9106217018906757, 0.08963550223635905, 0.18300939621045792, 0.3378328703547878, 0.04951857240594818, 0.36602741395056515, 0.34849574376254044, 0.34697547736061074, 0.6831457995088189, 0.6438699854343065, 0.33346308590408624, 0.7035570971369591, 1.0049776877731007, 0.4707643088295726, 0.32118289074450806, 0.5590949099429341, 1.1261434703248971]], [[1.0286489325852557, 0.4247806310084733, 0.5104648778309632, 1.2295632321178813, 1.3408506453087194, 0.71543691221737, 0.3524675405820973, 1.2610398625755148, 1.4978138907296559, 0.41071726863343183, 0.12193468394009635, 0.5273749401900224, 0.9453260212831192, 0.4086187325772085, 0.43883914566822374, 0.46443947397818464, 0.5029463479145587, 0.7857723129534223, 0.9199327407644446, 0.07567468362050211, 0.2065415552669377, 0.3306289779892922, 0.05766053456591845, 0.38287957924373234, 0.27316558653502393, 0.34389894493289863, 0.6473313482625951, 0.6454594517345117, 0.3147456306803561, 0.6673565155881971, 0.9823283365292509, 0.4700833434895502, 0.37640399635552196, 0.6025120069008505, 1.1630751509643944]], [[1.0484297346629219, 0.39016290443589996, 0.5258953594937128, 1.20741332643195, 1.3476262577980922, 0.7305440215315844, 0.3599298759454193, 1.1342869332859764, 1.440227736968948, 0.4716268261436845, 0.18060384251674605, 0.521891253397635, 0.9834614960176701, 0.44969995772115784, 0.43304653686353894, 0.5045719528477324, 0.46625431401173395, 0.8071941843063407, 0.9528162741952411, 0.08642736182196958, 0.22082988654231417, 0.35811910759448096, 0.0480171201227908, 0.3212959760411874, 0.23161068738728963, 0.3473187162814112, 0.6055406674589453, 0.6750673731994655, 0.29507668873386195, 0.569051768693523, 0.9714063094536856, 0.37207741554629187, 0.4546791321447703, 0.6619861722682236, 1.2656750559062204]], [[0.9391078148653966, 0.35529689450623403, 0.5197198597783302, 1.1954466546942304, 1.398431060573897, 0.7425344294784905, 0.3693242437372166, 1.00353095256164, 1.4553653074802528, 0.6451153524697171, 0.18172022243569907, 0.5438445313741578, 0.9628597034894864, 0.4556256954865566, 0.43795573106956964, 0.4862044307049658, 0.4802994592990073, 0.782465739992217, 0.8591600599422642, 0.08340367078980107, 0.21828459171426323, 0.395839475690219, 0.04660225757497342, 0.24559665699890385, 0.1743729060114681, 0.3179771796956061, 0.5743779187536662, 0.6788496191086999, 0.32346019619920074, 0.5410628453547003, 1.0289690557378457, 0.3038267917121641, 0.6246739859730333, 0.6645350769186245, 1.2677366903333427]], [[0.8200625373013053, 0.3420861279461566, 0.5317629972045761, 1.153519587196167, 1.5641307049637563, 0.726048145147522, 0.3617691343664793, 0.8915951508668982, 1.407346941967761, 0.7526626665241433, 0.2027678798697432, 0.5460629057832229, 0.9415976430919488, 0.5059024121081739, 0.4249410700854177, 0.45935996908174737, 0.5376979409056502, 0.8192911980317654, 0.8582070258398522, 0.0859608977123266, 0.2156885854780521, 0.4129076505221127, 0.04670651044251053, 0.14606186627039525, 0.1767162969687704, 0.26385739051547985, 0.5040577133034398, 0.6686288370086387, 0.36875811824544075, 0.5715544288444682, 0.9921331349254137, 0.2674696743941774, 0.6931925151774461, 0.635175237130996, 0.8820133730821929]], [[0.7698032762803815, 0.34040369601086595, 0.5520578796144817, 1.1874410211362296, 1.645553137930066, 0.7933535820787824, 0.401715627459417, 0.845114471996096, 1.334919681109207, 0.7718069618134307, 0.17194830774538783, 0.5426722841848642, 0.965493505279384, 0.5264253998522589, 0.6108406110266904, 0.6972868698641905, 0.5545515368323848, 0.8292086954726005, 0.6783719032553406, 0.09466269112360029, 0.22015568926328372, 0.4162729396465254, 0.044486089678158924, 0.0875429202693987, 0.20417129488973268, 0.2299705564423185, 0.43754517693094275, 0.6813942197821479, 0.41207111691880055, 0.30694843893076285, 0.7010655908730457, 0.32212629136016446, 0.6904361797683861, 0.7075484882518777, 0.48238389551076233]], [[0.7664427348863145, 0.30654334827843155, 0.5636240194028597, 1.1346558490770347, 1.6680092554926391, 0.7186091766513489, 0.3604791539699205, 0.8267569461003321, 1.2846176682642223, 0.8623013520693577, 0.12077425197348451, 0.5478824691999904, 0.9626696589205287, 0.5144960116889798, 0.6226473022601451, 0.6991728771286958, 0.557441737498634, 0.7780345749781847, 0.6287434208093192, 0.10784562064554884, 0.22480388870665924, 0.4207319989909023, 0.044916004419472015, 0.07685715155596592, 0.23027469186387955, 0.21568427104146948, 0.41125044136478817, 0.7054738595438699, 0.4656169910222244, 0.18873258143876806, 0.4847806914623063, 0.3884669764522564, 0.7225569645289656, 0.7794898675488976, 0.37304711658751494]], [[0.8372040299421111, 0.4089047733235521, 0.48959891600031813, 1.1496600650946733, 1.6392861037237991, 0.7177179385648509, 0.35168611485931955, 0.8566931820552242, 1.1656910127950726, 0.8449075075852339, 0.1334464438358418, 0.5538536349091538, 0.9643588279267192, 0.46413276072775983, 0.5899740099344823, 0.6727905501219149, 0.5725592284430487, 0.7441993078373585, 0.647921978401813, 0.09109081246285156, 0.23322530077290277, 0.39695258814679024, 0.03988968702258007, 0.08154868306839622, 0.2813632744647623, 0.205750348437923, 0.42464654538949576, 0.7784664501530436, 0.503424668193761, 0.22509989177388423, 0.26770535174497667, 0.4487140905373941, 0.7094872049315504, 0.8306963693059985, 0.33720378363090653]], [[0.8726496857441881, 0.33085103162573626, 0.5336310770363455, 1.2060006107007315, 1.6327593103124909, 0.657288361838479, 0.3025025176695164, 0.8453109846511244, 1.1786822214848978, 0.7954926513918188, 0.13749458643610352, 0.543674794903573, 0.9317122357968783, 0.4576216170490156, 0.6356563417851966, 0.7012406620193979, 0.5901718941854361, 0.6991052856062614, 0.6335583566207585, 0.10385926743369733, 0.25157741904877, 0.4359019830482826, 0.039822946560628035, 0.08386795915544831, 0.3236993405057515, 0.23243471049858783, 0.4468018850067977, 0.8294799868609894, 0.528775316562063, 0.3303764356985348, 0.1251306177833359, 0.5265893524434228, 0.6782767772240157, 0.8461817790708723, 0.45252137939038095]], [[0.8663162304667262, 0.3374569850374609, 0.5496674672639368, 1.2132428216589044, 1.6030633652539774, 0.7627181973828341, 0.3359494301864907, 0.8331735296231526, 0.9364993993817325, 0.757860659518816, 0.18683102855352984, 0.539655674032748, 0.900879175474754, 0.5576429259399293, 0.5991503646609389, 0.6770692833035125, 0.5911859825903318, 0.7544710679009399, 0.6222558751862097, 0.13289547665348833, 0.24399446837810335, 0.4146401731608821, 0.04045691413063239, 0.09770301446824481, 0.36938401917050834, 0.24208644301291438, 0.4159604147226328, 0.8697832684906701, 0.5565569623679254, 0.38447176511845566, 0.10702360631344665, 0.6671579626371125, 0.667406493020472, 0.8590815885218053, 0.5566888277166456]], [[0.8854128911350736, 0.3380759169908208, 0.5592053934399055, 0.8271737312432526, 1.4045643656564746, 0.6747995774121794, 0.3372482610790405, 0.8642516747882059, 0.5485204659082901, 0.5480512311956024, 0.20307411528415065, 0.5514892748994346, 0.9260510818897878, 0.5126442932128332, 0.5278170670619597, 0.475468823964859, 0.5492922071006501, 0.7209493525625883, 0.6224136845003512, 0.17357419013962777, 0.2530036107617769, 0.41053984203389293, 0.043776893127886476, 0.12369767823401867, 0.436207514862018, 0.27488709251537996, 0.37933946700512433, 0.92173811985957, 0.5612830176919037, 0.38256653341939706, 0.12597810040838958, 0.7651893675644901, 0.47776802048666267, 0.8790441306481249, 0.5831548031208311]], [[0.8164355365504703, 0.33669011170402174, 0.5616897383795609, 0.7457760153874557, 1.422582306422592, 0.6995657722463828, 0.33843688910014713, 0.7986372072215169, 0.7967138959529382, 0.4661175158001314, 0.2190559219435439, 0.5863172536745, 0.9693821720832838, 0.4664109806029934, 0.5215450054347051, 0.667504952009874, 0.5428911334358051, 0.6538460370296206, 0.5785900389925003, 0.1840698011079579, 0.2622348257683038, 0.41946790746452123, 0.04396753927977047, 0.146748567396094, 0.574728427986789, 0.3046080236138477, 0.3227844765290562, 0.9933212700704003, 0.5623160178179661, 0.3661256637707655, 0.1507990642863829, 0.8220633348967704, 0.35536133858921454, 0.9197263163668525, 0.5873661288327328]], [[0.7764648901177429, 0.32114432544038496, 0.5648118756080831, 0.737794695406544, 1.457055479047186, 0.6943396478232372, 0.3403446382322849, 0.6839783929124119, 0.702204892367884, 0.3962205215596012, 0.2337691875366122, 0.5770464713227266, 0.9651051740493619, 0.48484069650065276, 0.5081836575762506, 0.46888067066485956, 0.4915153619078415, 0.719537609966923, 0.564233410116466, 0.20415944406366787, 0.2576563468032803, 0.42053691123082526, 0.04260076491164666, 0.17231195241229041, 0.6550292012351073, 0.3057248523076402, 0.2855117485377854, 1.0184578948569938, 0.5812078995774319, 0.33899512020317724, 0.1632812128173562, 0.850710374195017, 0.2614064691257604, 0.952114659012695, 0.6412103981165155]], [[0.7574675228877941, 0.34772014664190765, 0.5158892824050862, 0.7513400382143894, 1.4680642417907976, 0.6772567564814629, 0.35718195896727395, 0.6794324046266763, 0.6853834143951854, 0.37215619424008767, 0.23476685414498055, 0.5990162925906206, 0.9946040556247531, 0.4814492460285986, 0.481095598767536, 0.4448510457470156, 0.49281936102523605, 0.7444426839967624, 0.571566925615399, 0.20204061246813737, 0.25114124110937613, 0.41707227751176823, 0.046356394880958086, 0.19029070782148605, 0.6906065237379468, 0.2878660702560572, 0.24174505013606987, 1.0179724897940483, 0.5824778575857606, 0.32086457415499137, 0.16595887021476474, 0.8034406275681286, 0.24129996632640716, 0.9337430940425118, 0.669625283927534]], [[0.7648289811041572, 0.3585620169569915, 0.5223959105549194, 0.784397314979222, 1.473267853221099, 0.677531476458435, 0.3643229567852838, 0.6502856547340528, 0.65929688408165, 0.35603826005926764, 0.24585654699473325, 0.5799772614896154, 0.9901189046899512, 0.485331416464468, 0.4892663152187851, 0.4526156605890094, 0.5155739834765232, 0.6909272682933931, 0.5584080424863281, 0.24828518528027888, 0.27800692905790925, 0.4089276208659952, 0.048152013592156144, 0.21338479841355568, 0.6603046074352971, 0.2588803882085475, 0.20340557565684225, 0.9252753080386151, 0.5781464610408144, 0.341203357162513, 0.15696452153946672, 0.778003261815203, 0.2647635078440973, 0.9218152517847726, 0.6528157553965503]], [[0.6804654625458806, 0.371398494319252, 0.47765105075640424, 0.8043454591842686, 1.4911576483168105, 0.6666957576700534, 0.38540453014369835, 0.6775295127407657, 0.6334981737789538, 0.36376630667726395, 0.20622886004322794, 0.4851147555234633, 1.0068762748742777, 0.41650827582691113, 0.48954806300624004, 0.4651651552783618, 0.505258751202285, 0.6378880338835716, 0.5709075260259001, 0.20891577488542035, 0.2871658687551529, 0.382740935984097, 0.03724654730787173, 0.22745439395360173, 0.5408345973424323, 0.20277462002851787, 0.1955960428855048, 0.7810535208399259, 0.5852935820713384, 0.3932174562572441, 0.16815564414630257, 0.7819398529532011, 0.3331673071874041, 1.0457840479846698, 0.6408618684318763]], [[0.6170350727795377, 0.3743809797119282, 0.4690908033891975, 0.7964413856057242, 1.44703841200469, 0.6696384711539123, 0.38501215690754, 0.6662310021987845, 0.6174288595038762, 0.3593117813186999, 0.20320022665493986, 0.4749513226003617, 1.0072175526949028, 0.42312327432475105, 0.4244683513315733, 0.4664565865111699, 0.5077267245620011, 0.6381222930393372, 0.6361663744551672, 0.16389021065661924, 0.29310977968802504, 0.34579820682332374, 0.018083877517079665, 0.2768996547794441, 0.4513101957510889, 0.1483957033802495, 0.18010722869671775, 0.7098854827725362, 0.5872921680725107, 0.37127406662369183, 0.23396017771194555, 0.7375099933472962, 0.4060952908519886, 1.128377003768926, 0.6215685223399124]], [[0.5821429791935303, 0.4195408330516741, 0.4028153103985367, 0.8257807725762107, 1.4238239412871074, 0.6824242174281199, 0.38218282731095243, 0.832906310494868, 0.6041052463600521, 0.3509740168122798, 0.21312068155330305, 0.4782475354679979, 1.0587041184018247, 0.3569807360643993, 0.4574878588822695, 0.502764029616744, 0.4229514068894701, 0.6039909200140003, 0.6372190850661617, 0.2545314613056632, 0.2881444549136843, 0.4772948192984635, 0.022890550944373853, 0.3084931708597617, 0.3412804957042276, 0.09557607224149597, 0.18578150910004332, 0.5569373386999508, 0.5765496794752746, 0.38230730672830787, 0.28235875369487884, 0.6324482462930191, 0.47147543247173596, 1.1510924126183841, 0.5632575246658471]], [[0.5624527269119146, 0.3847974353590521, 0.4442869184193772, 0.7925514884630867, 1.380703616941982, 0.7005276383219756, 0.3995063344163199, 0.8509382843395155, 0.6158962242375161, 0.35088435246217237, 0.2877217033837769, 0.46318244409151943, 1.074056901955776, 0.3504610381230981, 0.4818085046578621, 0.4607248435161624, 0.41935348219136803, 0.6010426101895389, 0.7132057187534503, 0.3262643641401434, 0.31616367395311124, 0.580728097949921, 0.025503568635939604, 0.30638355168720877, 0.29455985673666935, 0.09526296243558408, 0.1701289637022415, 0.45833916806626407, 0.5463516314944272, 0.4536026616532762, 0.31584440913798784, 0.5484231789596664, 0.5605346848449355, 1.1832328030590722, 0.4934057315509397]], [[0.5779984903932704, 0.34003230810450874, 0.28517395861320666, 0.8281008406741395, 1.355171540907322, 0.6933491721333866, 0.41238371122711076, 0.9678326012388755, 0.6078376087165837, 0.3505564735242713, 0.3243502197365299, 0.4814930054324502, 1.1352913734234007, 0.37163826455514964, 0.44930338656364804, 0.4321728653876762, 0.41175888157425705, 0.6387753472013575, 0.7721392711117758, 0.3098542617252103, 0.3134525783033518, 0.5001458306513284, 0.030139003971799715, 0.289965610380996, 0.296370800673452, 0.1224451914878007, 0.1644608721072748, 0.4049713975264555, 0.4631457163203763, 0.4880726303537783, 0.3376541740655693, 0.5108608315477849, 0.6267807689366227, 1.1491189671574888, 0.48532848888162106]], [[0.6735061739581021, 0.43580506015147213, 0.28097089466066083, 0.8225684430421456, 1.380051911072655, 0.6903008994638073, 0.3926588969314757, 1.0083334216277235, 0.5377705805710105, 0.3602898592852809, 0.30978971935582467, 0.46609438867681935, 1.1939677861045381, 0.3827120855746511, 0.4421949692431839, 0.44957367274007143, 0.4026875825690252, 0.6709184358415738, 0.8409839190673332, 0.27987213024652574, 0.29144011996250874, 0.38115353307626654, 0.013786430843634583, 0.3291186604405758, 0.35722889878093156, 0.1820237732035771, 0.15364147241709927, 0.4124765562696763, 0.35689028186813376, 0.5187095875993457, 0.3436940524875213, 0.4905910928488199, 0.7134888656247199, 1.0769866868784814, 0.48354752399127565]], [[0.6839608323898314, 0.449328592310462, 0.24208477715092563, 0.7896494751868541, 1.3440033204345698, 0.7203429725432815, 0.37787228412165025, 0.9882162716896582, 0.5551566374299837, 0.33507722039066046, 0.3080877763488663, 0.561783044569309, 1.0834195926305907, 0.39018275735206476, 0.46904250740276227, 0.4607615255858485, 0.3930670141964197, 0.6426304073510339, 0.7609471290527108, 0.3265553402813, 0.3173083542953037, 0.29736805419328854, 0.011174768981915346, 0.3757474971942185, 0.5106370642066527, 0.28531303000556996, 0.09958824942445321, 0.5365409998023156, 0.30655572660769304, 0.5091126334970228, 0.32372430646765527, 0.45997985830624405, 0.7640221798030348, 0.8761684496213445, 0.5109835113357439]], [[0.7641180673552905, 0.38557930897239123, 0.23430385968869616, 0.8084764775003634, 1.2722799133028198, 0.768972998619623, 0.3880734926202078, 0.9688736602705423, 0.6361302840798855, 0.2796793349464643, 0.2976862819370357, 0.5603543406397198, 1.1449170533818847, 0.3973943381737009, 0.4847177485244323, 0.4850720428310279, 0.3821635293473292, 0.6452933786033264, 0.7610682308137776, 0.29755980060795034, 0.2988658373833787, 0.28946082911692006, 0.013201003794881452, 0.5342542818462606, 0.6248843458503024, 0.38865583437806267, 0.06291627684344259, 0.6691810695867064, 0.34417555942719197, 0.507493605670353, 0.2553103446428173, 0.42617784780144546, 0.7045465986375501, 0.5935593719028356, 0.5276506434586218]], [[0.7612216623499688, 0.40080842137057565, 0.21695436205743093, 0.8610293108502729, 1.2325665165633444, 0.7719132767963343, 0.3820171422734788, 1.010009709280286, 0.5310854066868038, 0.20856327537984948, 0.3034586980954539, 0.557778013130722, 1.061447979210046, 0.40068801483703076, 0.4680797238958393, 0.39751491243664583, 0.4096102118725916, 0.6807137084980491, 0.8174385504569472, 0.30139146896113533, 0.28720947539959985, 0.2620396538853297, 0.016759054265924264, 0.6375797998079, 0.8812864451951588, 0.5047200475712579, 0.1390324224456177, 0.8233815286242159, 0.35693581785845113, 0.5919753269622452, 0.2951836211157923, 0.39412870490205854, 0.6727547138480489, 0.6603401323630802, 0.47835177363952597]], [[0.7610401578089093, 0.37132540444123235, 0.22465774577779063, 0.849401165150348, 1.1951704856003704, 0.7647063619012268, 0.3806026703592014, 1.110410608035986, 0.5566645113456635, 0.1598793158091641, 0.3078989267562371, 0.5737164306015506, 1.1489518420238314, 0.4206246658188674, 0.4962693940118169, 0.37019628022254253, 0.41064512655254637, 0.6954371243015687, 0.818741290469893, 0.29891398942261177, 0.27827896804899865, 0.29557221319969074, 0.017514126082618427, 0.5859760658766295, 1.0974056676491786, 0.5607149558579445, 0.29267205256337964, 0.8776572419532378, 0.3569853192749298, 0.67252439887762, 0.35495970860227144, 0.3729171881523417, 0.5985814079298856, 0.7811479000854544, 0.45730731757197723]], [[0.7688258895744167, 0.3941003516184025, 0.2417117940219928, 0.8192404480230182, 1.14483922668492, 0.7262940123739938, 0.38046026868096033, 1.1685993137297184, 0.5170364245774812, 0.18068256003724176, 0.26859452206425505, 0.5854455574502319, 1.1063706327855667, 0.4232522557501018, 0.5051152406002086, 0.3627479674746763, 0.4053571556856105, 0.7032190483681073, 0.8631258748171156, 0.2839220448758608, 0.2685416472248005, 0.2544646182249888, 0.021967877521537928, 0.5409808390097488, 1.2665780470963555, 0.5037316226110439, 0.2592700537496849, 0.9253481077015638, 0.3353711075171879, 0.7076609742575, 0.4044125703944895, 0.3697006012297985, 0.5142389861506349, 0.9074957611335612, 0.4634452726773985]], [[0.8238196557093538, 0.4080127748856144, 0.2475392793324915, 0.9149625400401898, 1.0569685055666795, 0.7500268918772756, 0.38560575188539514, 1.2386585295347363, 0.41978925724499194, 0.32017920708315606, 0.15935587942617435, 0.5769343583386541, 1.0558140219326368, 0.41043078201674166, 0.5365097340650005, 0.34969402921565174, 0.3847352174429359, 0.6850528326744576, 0.835656564431509, 0.29261823484553967, 0.24639720185775132, 0.23964866716999123, 0.038179220805417816, 0.45935889664620666, 1.442246293697667, 0.4817816468206338, 0.2209439053587732, 0.9342328667327677, 0.3373106003567384, 0.7083346149357721, 0.5226886188869726, 0.43124551276800704, 0.6049327753624086, 1.0074223018731754, 0.4551937356301042]], [[1.0376856515984976, 0.3867305263707611, 0.21931131552441113, 1.195669012233076, 1.0990343862693925, 0.7391532954763758, 0.4043157445017484, 1.2665160335395909, 0.43676690899412773, 0.44184356724058327, 0.13979647355317049, 0.5806900783973381, 0.997758738138415, 0.42762734770320787, 0.552285013546294, 0.3503363402418357, 0.39926937506444415, 0.6897922898491974, 0.8278721114576646, 0.27558792551816624, 0.22248294639001948, 0.24257642639174357, 0.04173413047172937, 0.40142589863653666, 1.5609617980019457, 0.44583427456513913, 0.20640716550294216, 0.9191129983758556, 0.31733089554689287, 0.6926364991930586, 0.6289622579212997, 0.42483713365203307, 0.7209197358415693, 1.025254330553765, 0.46415334810096187]], [[0.9259456455322805, 0.3822304476769781, 0.2198778150698874, 1.415663889500349, 0.8123296900595465, 0.6887035909043817, 0.3879559101575831, 1.2768513213751984, 0.4758183865593434, 0.297471622308106, 0.13507367217696215, 0.5648990762639386, 0.9296399374181363, 0.4169680347510083, 0.5325135372703784, 0.3851635189378192, 0.37093132268930307, 0.7404992023678181, 0.9468465077921001, 0.24157300657841735, 0.20483284858489953, 0.19186801537980186, 0.09093550723010152, 0.351284968552726, 1.6808507822547276, 0.455680259683812, 0.1880117145748519, 0.8968884921201828, 0.2541068715222496, 0.5677896184462472, 0.681933312384359, 0.3797351654375241, 0.800891589010758, 1.0968163953658232, 0.4445392840638348]], [[0.9676847130904116, 0.36516993458969516, 0.21760085526235415, 1.5415811299695317, 0.8057506970192041, 0.6755210158512734, 0.39394607177746016, 1.196775393673601, 0.4573385188533321, 0.24015938918459964, 0.15639405834221806, 0.6058487655829019, 0.9261206986129297, 0.41046252009740225, 0.5211914200120686, 0.375711092966858, 0.39417844289614223, 0.7844735441235475, 0.911849641099356, 0.28541035222607125, 0.2027502751923876, 0.1947663814583039, 0.10432745745447272, 0.3068467990775524, 1.7617170620717977, 0.43173857926837256, 0.1395186016834088, 0.8879094097799604, 0.2230979348947805, 0.5090959184965029, 0.6937983077475997, 0.3212055038419326, 0.833595975426584, 1.084248622918318, 0.4412012810147551]], [[0.9683866271966282, 0.3147058840775343, 0.1947333529972386, 1.6740758005788858, 0.7759906975521419, 0.6333840136082081, 0.4279921003507478, 1.186075752427937, 0.4356976614154852, 0.20595255628678039, 0.19413232893543964, 0.6650091506457687, 0.8783717818915374, 0.3913034129679947, 0.34736640283912057, 0.38063583884497054, 0.4168761279009502, 0.8527268856747539, 0.9390106268798737, 0.3608052876520831, 0.2650119902724099, 0.16446759921421772, 0.17929911000719104, 0.24047453150760079, 1.6795777916172487, 0.38633920872495364, 0.13644265879566259, 0.7351664635399384, 0.25162156634033506, 0.46729168828826767, 0.7730300028946857, 0.3313964024934427, 0.914995797435959, 1.1360224198169748, 0.38595874810419206]], [[0.9913630151498456, 0.24979742741570998, 0.1596570455980776, 1.783135254459069, 0.6865114743160516, 0.635667247431247, 0.4244763674480915, 1.1960879669828697, 0.4341123578545794, 0.3002055955724988, 0.19110661901372938, 0.6552812575389637, 0.8534039542846839, 0.391183258318008, 0.35170376438527495, 0.3840949653378626, 0.40347101137964, 0.8300315221651031, 0.9308818679960971, 0.39057871818290285, 0.3601072519493784, 0.22035699748467413, 0.176002412324053, 0.23585335686701497, 1.5334575964982968, 0.39132688566784846, 0.1292901950000372, 0.6173843540407495, 0.30336198005076365, 0.4782312774670404, 0.8419170465613532, 0.3898093628740416, 1.0535720386136807, 1.1784890322900763, 0.36769151064346683]], [[0.9784124901348532, 0.24180703638738416, 0.16042998086483795, 1.9533079871558674, 0.6252453275294938, 0.6465004133079979, 0.40708388102779486, 1.1870560581082357, 0.43969528522243323, 0.5288005538274888, 0.18824563215266266, 0.6819019270747226, 0.8152578023091182, 0.4409223872249405, 0.36371801775844464, 0.40420085962869656, 0.4218732716751962, 0.8608827257836097, 0.9180776107891351, 0.35494689934629226, 0.2956824344359068, 0.32778131861253446, 0.17401054312305422, 0.24706753014103947, 1.285920475953342, 0.42927777528659494, 0.13956691647743721, 0.5141799255894256, 0.37368017575638357, 0.4921527339508517, 0.8983914428394024, 0.4675403618770517, 1.1285580177762835, 1.1797189402097936, 0.4126409751760691]], [[0.9183315905698414, 0.26121397180085654, 0.16205625842555027, 2.0781855233670323, 0.5333486533175951, 0.6339888606791524, 0.4082551145902205, 1.19533142023945, 0.46329232722473024, 0.6346803247149375, 0.18364495815510934, 0.721185297572299, 0.755807640573716, 0.4266562750937377, 0.4096099206576511, 0.4261692273095349, 0.43488188764648256, 0.8976099316894276, 0.853343755816901, 0.2967928847988358, 0.33504529362652474, 0.30213342465071136, 0.17473582854731512, 0.27168729520210994, 1.1895013097716265, 0.44116955043074013, 0.14703042652211928, 0.5010704448009455, 0.446480749842517, 0.515736500439386, 0.9398891838202676, 0.5621268184476124, 1.0614697273196885, 1.2115880560728434, 0.4927178932107164]], [[0.9030633619418315, 0.2519540714290422, 0.16450547785414255, 2.205948548215972, 0.46065172840317875, 0.6340939420965986, 0.4065292822346008, 1.1664860148061171, 0.48567866551263356, 0.6811400794095919, 0.17867229667109524, 0.7620438266598433, 0.7240213574404696, 0.40200972291728965, 0.4259441664067542, 0.3498728560283342, 0.425079355017442, 0.8169297113415188, 0.8635666651019045, 0.26356724416477617, 0.3350336568024921, 0.31575285246740503, 0.16650524802261507, 0.2917086858254168, 1.1293850013974578, 0.46779713675523915, 0.1612792278694853, 0.4931702896853431, 0.4976849486621735, 0.585721481231663, 0.9751201545915598, 0.6545761919508121, 1.0456386611315345, 1.2201080528579185, 0.5393043513920986]], [[0.8485133767719901, 0.2639189759833236, 0.17578869467287617, 2.2475971192668966, 0.3951822323163031, 0.6253012259874917, 0.4098364501346632, 1.1008925476906133, 0.5166327326454243, 0.7163571354663328, 0.17701130602186152, 0.7640531810415159, 0.7140388949128892, 0.4050654591049938, 0.669216068893369, 0.30826051668845267, 0.41363012710957126, 0.865013364208679, 0.8821521494050594, 0.3181840977586142, 0.3471189855293892, 0.34229374290011005, 0.1627233826907392, 0.3194719435654617, 1.0966736673194615, 0.4608348749772837, 0.22388763226538055, 0.4463399339932448, 0.5383319184715075, 0.4546284822654201, 0.9708347798419961, 0.7484455647790742, 0.9765802181298411, 1.1557891710442127, 0.6026649732912903]], [[0.8448640936072245, 0.24401979463414053, 0.16282081050834132, 2.2687600847630915, 0.44317069333531395, 0.6343906257189829, 0.4124674630662414, 0.98448920440492, 0.5391385307435906, 0.7875355381451954, 0.1464657009504795, 0.8347615145518863, 0.6750136338903242, 0.4069330646977847, 0.5815767157668028, 0.3150747211275855, 0.4218133594058861, 0.8790283113106641, 0.7649083460335775, 0.3461752587903488, 0.33828627998016686, 0.3389518198934823, 0.12701260927646868, 0.3557150208557524, 1.0865928709966806, 0.2650998898857101, 0.2758646636047184, 0.447041971087805, 0.5739209685269397, 0.3092050230179608, 0.9501535983231926, 0.7909646560093091, 0.9355544965602062, 0.9555149160292866, 0.648669463448901]], [[0.8982719284254229, 0.24660079211204555, 0.16034180616616078, 2.3336311592946823, 0.32501374300337815, 0.6390072126031767, 0.3876290089284137, 0.921920622463121, 0.5400497254601461, 0.9890989907658362, 0.46027237225459966, 0.8099644563683921, 0.6594145205353991, 0.4077389954666496, 0.7226842577360126, 0.3110361319048733, 0.44411964334769616, 0.8303728980779481, 0.6753305378766019, 0.355840863781648, 0.3816341482563934, 0.35054327470053515, 0.40917056291584614, 0.3793755867844546, 1.0592729869523234, 0.15125496856662696, 0.3459633833312763, 0.4667382658181635, 0.5993752277226579, 0.41480193624677775, 0.9105867726894457, 0.8226113173668177, 0.9340499760294607, 0.83537909971762, 0.6561355064679608]], [[0.8359839732117582, 0.3026966820176771, 0.21243947238062488, 1.848059421505749, 3.7882351812831665, 0.6087562432547413, 0.3852275702097628, 0.8783725031374923, 0.6010202900088235, 0.8946862680124702, 0.24664493150665395, 0.7884922867894057, 0.49690271015223064, 0.42482559877047243, 0.6605253809544023, 0.30586067550200735, 0.4223533622736323, 0.7762157795524194, 0.6701668493737633, 0.30464946900688417, 0.3555464045999226, 0.2385369619851644, 0.10375994188993622, 0.35233339646158074, 0.849162717039704, 0.1128694181874301, 0.40132443354765257, 0.5058312982446476, 0.5806902295891533, 0.5018074221584403, 0.6920670478966604, 0.5800127445149449, 0.6188831808955475, 0.5824215994344101, 0.5468369540325492]], [[0.5361247683937495, 0.2294308664956537, 0.17338849994609876, 1.8904670964599444, 3.7882351812831665, 0.5964606420029458, 0.4042490451531204, 0.8846461316460548, 0.36779899616336775, 0.8747810683378325, 0.3418320178695603, 0.7623750914213732, 0.49141806021995893, 0.421928665363939, 0.6455799331840968, 0.3120848238624332, 0.421811802783652, 0.7870103126885034, 0.6630520364102509, 0.2882178188963198, 0.36179707693901697, 0.25341252462470465, 0.1689456415312745, 0.41287381837034, 0.8409229928644679, 0.15280982484724565, 0.4396416794705542, 0.5263167457774635, 0.6317506050023393, 0.5942679870597423, 0.655438536338018, 0.556181372530454, 0.633843983629545, 0.5651396473916863, 0.5317128374920355]], [[0.45945919288480963, 0.22284638023523695, 0.15886711124782044, 1.916339947722072, 3.7882351812831665, 0.5826886435541919, 0.42008838533533144, 0.8835985843418641, 0.39175462828725116, 0.7928788217628695, 0.48636741725724275, 0.7565006598408743, 0.5236744125246352, 0.4486315430034985, 0.6660007405724895, 0.3204073679952088, 0.44604464226065754, 0.7730304599502952, 0.6595723216802257, 0.24338769207895605, 0.30896993670000106, 0.28344566311773645, 0.2664544285244198, 0.46827179949611974, 0.8022514827044294, 0.1881772181416918, 0.4674082609418299, 0.5333206193302744, 0.6503853230899467, 0.5532868322661281, 0.6160261643204606, 0.5281998653547917, 0.6200925034631803, 0.5237600722949185, 0.4985668275861355]], [[0.39924716003763805, 0.2817094341919505, 0.23038806858255176, 1.9645175978472744, 3.7882351812831665, 0.5754728512590666, 0.4401722923808178, 0.8755535659525845, 0.35538374107300735, 0.6566629536395836, 0.4676229209249575, 0.8827568872798933, 0.5778997234762642, 0.494540728522983, 0.5502951610825078, 0.39837056263398263, 0.44181195868386136, 0.7986716998436375, 0.7285941111796211, 0.2237962731483809, 0.2819316629690829, 0.24258391258499049, 0.2691620258747912, 0.49944179312955145, 0.7230404822679982, 0.26298396057031326, 0.491203574342839, 0.5239303950850147, 0.7224137085430016, 0.569657539427247, 0.5827319284705595, 0.4983993640532032, 0.6352948245483395, 0.49170094329685365, 0.4630566648602873]], [[0.43510311117788925, 0.27426216139749116, 0.19898558738593747, 2.218569287792232, 0.4313849097606528, 0.5823191231222762, 0.4786068396801431, 1.1208614633555805, 0.5717526117203959, 0.9274339513934704, 0.723206803479892, 0.9791222083810946, 0.7331760670189088, 0.4713562218662945, 0.6320918421643011, 0.3477977769646311, 0.4808897086784666, 0.8215846620065673, 0.6647532878680027, 0.20890915234044383, 0.2731826999119782, 0.3510910769553357, 0.4335703760275554, 0.5989993907418707, 1.1225568498821379, 0.3415956270368408, 0.5073711742371405, 0.4647344597185471, 0.8195863635223487, 0.9225072789242046, 0.7320430422691834, 0.7358355172792416, 0.9385463505422054, 0.8152000554979518, 0.5389961168386994]], [[0.34839565058360966, 0.27282384452724784, 0.2025791175043033, 2.2051223334163, 0.5105552304769146, 0.5758887583175346, 0.4231415293031948, 1.1647078278464422, 0.6424788009171876, 0.8003828560501932, 0.716690728084305, 1.0425166541492406, 0.74207263954827, 0.4560668788708841, 0.6796359699505194, 0.33532011839813003, 0.4660779753733568, 0.8437346498085924, 0.7115795152684536, 0.19125050119828546, 0.31062428189893737, 0.3397699114213026, 0.39000442939609653, 0.6359715188778814, 1.128712424918326, 0.38375930520222906, 0.5103803174607857, 0.42963860294802547, 0.7709420609506893, 0.8324887150552078, 0.7320029122640912, 0.7300564572593677, 0.9591421322831544, 0.8635867077109223, 0.5327121885371705]], [[0.2627067299614, 0.2807771572126482, 0.20847612155669631, 2.1932684284068413, 0.6804418079427721, 0.590478880576306, 0.4335340369669429, 1.1940974168336678, 0.6772949709833795, 0.6891817350743402, 0.7412031498972158, 1.07436085024447, 0.8205734282913923, 0.43258881527497367, 0.5944183542025998, 0.33991812083622797, 0.4485468410504653, 0.880365246813716, 0.7497575743763404, 0.20255100037491336, 0.36246823591947797, 0.3303437976488209, 0.3913812722893659, 0.7017834618645861, 1.1353013981375484, 0.4057083144149155, 0.5333336847835987, 0.4540634679551486, 0.6768597848966733, 0.7901933598939926, 0.7572057331289346, 0.7183797921774577, 1.0191236207244128, 0.9264600124823721, 0.5498072552833554]], [[0.2663724650843742, 0.27822209692878586, 0.2055481364064698, 2.1737408702289707, 0.7959767151043395, 0.6151716321484808, 0.4428229642278011, 1.2012235338264068, 0.6971285691525806, 0.6622555369668429, 0.739404502526217, 0.9515092788329762, 0.804114260062402, 0.4191863543028881, 0.5374817155537641, 0.34648972932801414, 0.4153681835165739, 0.8942296479288829, 0.7572147850500737, 0.2692855532771202, 0.3835209363947873, 0.375968638993715, 0.3871815331658218, 0.7702402381234601, 1.1327175436766233, 0.41648381898270764, 0.5799457616913416, 0.5011428321410825, 0.573276248159189, 0.7696600548331288, 0.7384875807841709, 0.7022371814632073, 1.0392694791848889, 0.9843743281653723, 0.5577210796412564]], [[0.2490447250970386, 0.26453269499317766, 0.18537164634051892, 2.148293691367919, 0.8773265610810175, 0.5950823320479632, 0.44479466594065775, 1.22267375544657, 0.7107706574761976, 0.6858655680526189, 0.6317218422577464, 0.9910796232922456, 0.895236260891704, 0.3812334055106998, 0.7451986541973796, 0.5251581935843158, 0.42856576232038224, 0.9329222065563118, 0.7449278579891103, 0.248998762309261, 0.3304313259594943, 0.3341174376024134, 0.3587156711799156, 0.8123423381423994, 0.985311352785448, 0.45888304036318295, 0.6151738848515578, 0.5484861506587204, 0.5842729916990499, 0.7557861299268028, 0.6829963932553029, 0.6681078838285038, 0.9749963451016641, 0.993772043313432, 0.5186787413080169]], [[0.2430348570445909, 0.34034539738522873, 0.28691136469560763, 2.0720330002209004, 0.913701538079676, 0.585024646811672, 0.46849167385851725, 1.2507941696757119, 0.7405920949528604, 0.7046112811913057, 0.5094700570187146, 0.9626533886296865, 0.9390137044402075, 0.5144301401052489, 0.8415268307918602, 0.5181056076865079, 0.599457196283962, 0.8834919800738295, 0.773768168261421, 0.2289884789398834, 0.31361328303541447, 0.26364325063459204, 0.25654588635972386, 0.8490768867251175, 0.8519213027886401, 0.4152126206153241, 0.6483979550167396, 0.49368511094652795, 0.5966610163286206, 0.7806788684558621, 0.6459209314611212, 0.6598649944201466, 0.8506873043787979, 0.9394633626531481, 0.44375084565842393]], [[0.23199655564257377, 0.35741747597744444, 0.2550200896583277, 2.100294317662565, 0.9567162405370511, 0.6029852135958439, 0.4155979550668788, 1.262985113961995, 0.7709710442005382, 0.7252181164688454, 0.31840292752982435, 1.0012784562064287, 1.007581723405286, 0.665927533794959, 0.756689019236535, 0.512705394363991, 0.6018254311212174, 0.8271094490696225, 0.6438529331529035, 0.22002141822667257, 0.3144893083880092, 0.24186448223021967, 0.17111292750166604, 0.884877292756075, 0.7059275775831148, 0.4388347120518945, 0.6914782442917524, 0.44015919225946387, 0.6300381310742202, 0.7708296737246445, 0.676609109161943, 0.672666763263414, 0.7182400864150138, 0.9238790426098882, 0.3223118029733226]], [[0.22588921353329516, 0.37854202105551193, 0.3439922814383013, 2.0299485089592477, 0.9692737988724394, 0.6146460787493, 0.3866207687174784, 1.2676507493869202, 0.7996604691701655, 0.7593943314969558, 0.3106794131063009, 1.118045826434774, 1.0009422194047795, 0.5548874888920645, 0.6539068341718208, 0.43343610711469893, 0.531889312171546, 0.710439324551046, 0.5581393880095745, 0.23325778776849593, 0.2939943490018007, 0.2819735625573293, 0.20551470904246139, 0.8760668875309441, 0.5758518875820522, 0.4477943475392251, 0.6616715898376604, 0.345326860052877, 0.6911324892241911, 0.7541392813478163, 0.7055628589842603, 0.6894441813443297, 0.5528375728511935, 0.8782324221339899, 0.22776106004973642]], [[0.21436128389975048, 0.361889766362948, 0.32244739478579965, 1.9774534928399023, 0.7988575663071611, 0.6262712713657584, 0.42635431798862006, 1.2473878131446288, 0.7815673266836142, 0.8374453297391916, 0.315278205018867, 0.9634421058276016, 1.010659145467706, 0.5495368274268606, 0.6061236616579488, 0.445617764939189, 0.5109119577878074, 0.5953905379523674, 0.532894518805852, 0.24304918173424153, 0.29392532479997024, 0.28831990583864897, 0.27242065894944134, 0.8260601180670902, 0.5005103901845415, 0.4808502767860511, 0.6455886604684179, 0.2246513924962044, 0.6755631479005229, 0.7368180461699501, 0.7384665796531815, 0.6816756618348645, 0.4170592337264819, 0.9602868239975961, 0.17387847938122106]], [[0.21633860998857737, 0.37400281395693635, 0.3866484520034373, 1.8715093686423365, 0.793463727273969, 0.5987779103146114, 0.4631120010904268, 1.2498004988261617, 0.796831811741917, 0.8438735868500007, 0.30281832335083236, 0.9477682706265553, 1.0824021670198947, 0.5588828272100075, 0.5629670628453931, 0.44529837545557727, 0.5638725473841507, 0.6200154044679114, 0.5600415761311492, 0.22091723127733523, 0.27862001459550906, 0.28305142071890965, 0.3060994183741, 0.8280983514495079, 0.45274526346885846, 0.5129637838545509, 0.5787376152343758, 0.20086202100303113, 0.6527013322976258, 0.7216189061792159, 0.852391186098983, 0.6620999363785731, 0.36356607157635945, 0.982794454226942, 0.13386344934839073]], [[0.24628211594593605, 0.38572026202625664, 0.3931720086866794, 1.754178077931934, 0.7930009515816416, 0.5378603285704078, 0.4174526454055618, 1.2389488231908978, 0.8023781840649034, 0.9223377593311919, 0.2866898660814115, 0.9042067264829785, 1.1348796919390818, 0.5335455316628481, 0.6140190071967098, 0.48887074127893393, 0.5548330230215678, 0.6476839807688779, 0.6278741909897416, 0.21567392328800697, 0.26682342692016414, 0.27039894450609836, 0.2216397420020228, 0.8036116880779131, 0.45957010522407027, 0.45466849994540925, 0.6483247972610111, 0.1873663175248706, 0.7245064916036127, 0.7116926507019996, 0.9157884488331302, 0.6862265013990397, 0.5617930041219978, 1.0275677973264343, 0.13712846310763166]], [[0.2600614765161919, 0.38327925649002054, 0.40989687076211706, 1.7656116385057297, 0.9147514104800905, 0.565712510279521, 0.42262950757773443, 1.1812809882390192, 0.7879386416839453, 1.027668425393995, 0.2724301281477414, 0.8968929333693296, 1.1734914440314088, 0.5363695530701681, 0.6508027483860446, 0.5064198828356257, 0.5583601020973264, 0.6352881887761908, 0.5995982092003336, 0.20419558961104856, 0.269932827720114, 0.2750380147173959, 0.20970070526411982, 0.9442543831215195, 0.4919716942717788, 0.4109235564499866, 0.7789661008098437, 0.16085863831849756, 0.7358071009439102, 0.6493830756259891, 0.9862673164888529, 0.7540199342429494, 0.9122230453540241, 1.113276515308856, 0.12778287261068544]], [[0.2568654932930756, 0.3805921963507579, 0.3993399107192481, 1.7330509429854357, 1.0030311300497432, 0.5843709983669463, 0.44375926109946906, 1.1144088988396013, 0.7827386996818125, 1.154730074976098, 0.2625940770882315, 0.8888740049752815, 1.1498642695297279, 0.5344881438015067, 0.6272585889073459, 0.502931705290184, 0.5112720803449228, 0.6409371913059632, 0.6050733907086826, 0.20692105426870755, 0.29035165546211605, 0.2864462324692217, 0.16649139920962955, 1.0538271454464345, 0.5294871454577009, 0.3653809243082706, 0.9054494316288189, 0.16667096015660948, 0.7219245746017451, 0.5945850949050913, 1.05552677862198, 0.8317803847784586, 1.278835745426735, 1.1971916267177476, 0.13098313659890293]], [[0.28880770472841044, 0.4078272814316253, 0.3932589146961255, 1.6439777197794259, 1.043882810142787, 0.6009220972303502, 0.46133494354228605, 1.0671966111709057, 0.779366247552462, 0.9235723164401063, 0.2430501152421879, 0.9130174740723915, 1.1853425394295498, 0.5487395232402361, 0.5551944889580789, 0.48490508047458847, 0.4895947740107977, 0.6470404444641746, 0.632370395043736, 0.23140085039601777, 0.4043712975268331, 0.2804993264178922, 0.18197981625547976, 1.0716089806402014, 0.50070602413867, 0.3678820175477804, 0.947521978616052, 0.15920960515634755, 0.6508552000293412, 0.6157972784524506, 1.0690028973119419, 0.913218703673379, 1.3312718928731408, 1.2702839070573777, 0.18316625185273722]], [[0.26463757513653885, 0.4412354313799438, 0.3756425604206421, 1.5829306168484147, 1.0591563686538361, 0.618138250460351, 0.4354166955215151, 1.0652980716554215, 0.7754415108001291, 0.7354958439574264, 0.2549257439478289, 0.9436746770358977, 1.2640443497370775, 0.5548016551474735, 0.5518676506846159, 0.4896754842206063, 0.5665322011394835, 0.6527592115023538, 0.6203093792501202, 0.2182830765913938, 0.38111750393038907, 0.30019576339699455, 0.20017037009603117, 1.0509800561603075, 0.3945827905678778, 0.39613418762856156, 0.8902291189411562, 0.17212420668419365, 0.5837558755746812, 0.5964435180784124, 1.0358426829953755, 0.9214427224352397, 1.184430536635996, 1.3387940719831364, 0.23695863953631757]], [[0.2861460556855573, 0.4331197542724235, 0.38577232700597763, 1.6140387692715916, 1.030380600734558, 0.7348688191807808, 0.503425823943697, 1.0543900210797017, 0.7504244711885842, 0.5102184187075731, 0.23346406608150114, 0.9328685569812183, 1.3372948406207994, 0.5348785801780259, 0.5826190086497873, 0.48742212963067555, 0.583249803265624, 0.6547885798346282, 0.6088389459619337, 0.19890447297047453, 0.3176145867739733, 0.3402071579511916, 0.21800112282061473, 0.9495741582700769, 0.33800945572942187, 0.5196252079024082, 0.7703686368189222, 0.19500430477564706, 0.5526403073447896, 0.5167539922405726, 1.0238473809236017, 0.8685022558225769, 1.0362741729496077, 1.3457132406446948, 0.26106040028025207]], [[0.335427498748331, 0.4290358503592748, 0.4199338597664758, 1.428048257950794, 0.9760974545834818, 0.6237738751214696, 0.4627307138890364, 1.0136506301356325, 0.7462548345646739, 0.4857878883345746, 0.2242804965199925, 0.8985090232048332, 1.331507012398991, 0.5292054075028045, 0.5418048673735101, 0.4713361759942297, 0.5865568224354857, 0.6880874548260654, 0.6645335629494576, 0.1843871758786052, 0.27907448869373763, 0.32938179353802544, 0.2266552730001058, 1.0390127910369333, 0.34189946852300845, 0.617006807044888, 0.6350944184504218, 0.3366690263818421, 0.7117837040835596, 0.4497041062253943, 0.9890092815134044, 0.7201361289869208, 0.8399990960585353, 1.342135988463358, 0.27013918955572275]], [[0.3619466554502838, 0.45533260909819967, 0.43914473175290414, 1.260100475907034, 0.9583777935611894, 0.6426642428826999, 0.44115733720463407, 0.9248265205585355, 0.6695059736449103, 0.711447090506516, 0.21712054127521024, 0.859684198827591, 1.3773024870681652, 0.5593573228684725, 0.533215135709266, 0.47028619736783106, 0.48315681002715316, 0.6099702470425556, 0.6639171909632677, 0.22759316555010622, 0.2667805054543815, 0.3164150415876023, 0.14475661725385858, 1.2383235008456557, 0.36761819014466973, 0.5564659035653141, 0.6555117205895479, 0.5860432620818662, 0.701608675718173, 0.4971684349308411, 0.9352107552519713, 0.6692072786395021, 0.8383977549221802, 1.3567518263651859, 0.2887530463388014]], [[0.48552701828195216, 0.4541893675949136, 0.4326568423318039, 1.1974430721557514, 0.9770441346192692, 0.7354770867340926, 0.5294934668176583, 0.8058152048006662, 0.6926330064153696, 0.9035311494803115, 0.2063367984476932, 0.8448198350620078, 1.3952584357753917, 0.5543860007407015, 0.5142795595600946, 0.473867495277952, 0.4651226934021635, 0.6382979023762358, 0.6828342416955129, 0.21284708820840634, 0.2123039178978613, 0.30064243628970855, 0.1286084978506375, 1.3063302266211823, 0.4173405741712487, 0.4615728809260475, 0.6012222720342324, 0.7777729972751212, 0.6795562082779198, 0.5116029958087641, 0.8458493672246465, 0.6963650343700464, 0.8092208349177493, 1.3070101203162774, 0.3004670433980977]], [[0.475904847631353, 0.45152966731649796, 0.4217959507074055, 1.1469043858865513, 1.028636110388248, 0.762022548607338, 0.5393895015435866, 0.9422889871860444, 0.6413263036217272, 1.015341229466773, 0.2013928975166141, 0.8770240152394018, 1.3881850381070298, 0.5640557244656272, 0.5296145462343012, 0.5086252347364599, 0.4658783931458538, 0.6803992490157864, 0.6764470361277528, 0.18753758278748744, 0.1385010488468353, 0.29547515839669053, 0.10324764868344162, 1.4518621892279162, 0.5304081492471427, 0.44772407479507786, 0.5877681594669855, 0.8819322652866455, 0.7071204907087809, 0.5419089002227175, 0.7131057751095642, 0.7394593469170161, 0.8196688534445994, 1.2850047919831906, 0.30818420710419003]], [[0.4981206806717565, 0.4438966746235115, 0.4038778905703895, 1.207251314335654, 1.0422051010462368, 0.7422916971097564, 0.5190019964897215, 0.8864237207729458, 0.5933605075149544, 0.9283706544181267, 0.20635561407111547, 0.8217317642718728, 1.284479036197887, 0.5742553991875943, 0.5626945863364097, 0.5457556109214421, 0.470148618121923, 0.7110136525307253, 0.6877971304036821, 0.20545156527274586, 0.14390770435760844, 0.24589684793581584, 0.09421516260832041, 1.742995770735157, 0.5811151614320614, 0.4121100635819259, 0.6531789830263781, 0.8539964533753173, 0.7245637692964202, 0.5207660214515905, 0.612429077106537, 0.7706841787362837, 0.796622479239304, 1.3157260835760982, 0.3145268855320017]], [[0.38915162773766704, 0.44169065770047433, 0.39354734327092056, 1.185914118615735, 1.0600156468744648, 0.7560821252296945, 0.4732214326762443, 0.7399902626009999, 0.5430986769740517, 0.8424552347861617, 0.19491935551145625, 0.8190097035863017, 1.2457010421590515, 0.5531985820412568, 0.4639555652998095, 0.607015084362435, 0.6299895770141659, 0.8995931258221533, 0.7932192328655461, 0.24406975473320622, 0.16672950992134133, 0.2033791800748731, 0.07419149397210817, 1.8749986379638435, 0.583447994292954, 0.4002750919352351, 0.7637880427868283, 0.8360976915025184, 0.7191322678594934, 0.5000147342363583, 0.5415696544315227, 0.7929459041516818, 0.6382267640139593, 1.2939023716621578, 0.3430138435684903]], [[0.38000279895495886, 0.44806782985444077, 0.37766622726743093, 1.2089743589473774, 1.0452970300165996, 0.7427463228066054, 0.4355947188296255, 0.6364017613733823, 0.4896445526429845, 0.7657022237011042, 0.1725358979483065, 0.6839975217514584, 1.2354915769752122, 0.49843448101211674, 0.34139327659873125, 0.5397299185454988, 0.4735654476905389, 0.7989727804850675, 0.7987443175506153, 0.22027508534795423, 0.15952395416531162, 0.20138184455803448, 0.0647689633569828, 1.9266995335972799, 0.5967036639572715, 0.3693656327412493, 0.8052669270077597, 0.8912614773904283, 0.7237709104330892, 0.4454591689855102, 0.42239675728021775, 0.8019598923769242, 0.639618885125841, 1.3115223522409907, 0.3660585349482075]], [[0.3739181491041754, 0.4369110534819577, 0.36760543196138235, 1.205971456297639, 0.981792707466419, 0.744642783447284, 0.41721018964035417, 0.5821069751823273, 0.4457410926651143, 0.4964742149104381, 0.20274758979745852, 0.6615851082894617, 1.2999028911285864, 0.48371025205995016, 0.3463519144113768, 0.5611727849605705, 0.46054237707391804, 0.7662138382359611, 0.850282561276422, 0.21820904209487307, 0.19714910382763354, 0.19296238079869518, 0.04718881312023619, 1.9477115210097316, 0.5984094748207913, 0.32968314621169426, 0.8286927674648237, 0.9161185321216626, 0.72869225817852, 0.6218433907429584, 0.4087964378718115, 0.813398264513294, 0.6053623501230502, 1.206271946043464, 0.36709834668011043]], [[0.41484355009111873, 0.43504944806058615, 0.37045856103372315, 1.219009840742838, 0.9395865170633553, 0.7842136589920614, 0.41831029119732666, 0.5704777695099794, 0.4068263342679826, 0.3220902698020595, 0.23660091771621716, 0.6662902751977898, 1.3162792056384864, 0.48014509507341496, 0.3430939418865957, 0.5491860584884944, 0.4042284425132159, 0.6889012745995189, 0.8710670847138582, 0.2075368624688084, 0.20493825454431192, 0.24894008709656681, 0.04457737574666377, 1.892383142637513, 0.6489258488925674, 0.3360910678884065, 0.8199316816712431, 0.970955250621935, 0.6982505945431092, 0.7087480800191823, 0.4430124371548431, 0.823158063554057, 0.674137976562394, 1.141361389618997, 0.40386724941065816]], [[0.49252710706733815, 0.4695193421318741, 0.3455240186610097, 1.2476950700898453, 0.9132763048431048, 0.8192909051862789, 0.4456530529655885, 0.6053517208651801, 0.4973339825958423, 0.31377646651092994, 0.21312331314370314, 0.6732832615955617, 1.298940908276887, 0.6122884567202196, 0.31871176340463453, 0.5702041658627238, 0.4529524778385389, 0.7029766815743852, 0.8196878930192131, 0.20542025389459284, 0.22814921074813765, 0.24696379570916915, 0.045562263503879416, 1.7787513093874168, 0.7401900964854393, 0.4076686291695797, 0.8319668648630398, 1.047466836089198, 0.6448669024932012, 0.6527503973885267, 0.4816461979143627, 0.7601738285884596, 0.7198451136564015, 1.1771060728642637, 0.4044384981274294]], [[0.9515196441736214, 0.5339377462889554, 0.3334260863428842, 1.355590126435127, 1.007000939927972, 0.7765229052330024, 0.39259007132034923, 0.7477600687985663, 0.640705438797431, 0.3206082334282459, 0.2108427845164057, 0.6701575002503286, 1.248633731990943, 0.5984759303595082, 0.3249839496798634, 0.5323994791916297, 0.4501318639367865, 0.7005183914848709, 0.835956218588622, 0.21038601814525007, 0.224491730340371, 0.2172259066638656, 0.06919926564732197, 1.4371885600593484, 0.8526412642035281, 0.37001299928682024, 0.8421159213478316, 1.1754488370069132, 0.5595673679975828, 0.6557784556674033, 0.5736652117082692, 0.7381032028940726, 0.7386573737191335, 1.1704583262742523, 0.37067504683267616]], [[0.9507796165845714, 0.5998356395574712, 0.38518356674216425, 1.4012421936463149, 0.9946231702739146, 0.7978159876019584, 0.379175886169232, 0.6836280455039664, 0.849918075231284, 0.34004216127013287, 0.18022203586427316, 0.6300077591967114, 1.3271861171591612, 0.5903618903540836, 0.34598811331054447, 0.5531572195444495, 0.4872682223601434, 0.7480854392362416, 0.8451695255317231, 0.19772572475584552, 0.19799941621093278, 0.19041242647763176, 0.07460620277546405, 1.268017141950039, 0.8252552257621351, 0.4787518443809468, 0.7532625606219834, 1.2018544510370834, 0.6136575522218164, 0.5487803408134044, 0.6380242375781067, 0.6862111933671372, 0.7195290121659823, 1.1630674059096602, 0.32862380783181294]], [[0.9431442978413189, 0.7467711137351758, 0.3940863222843244, 1.3527029056363473, 0.8481325618248323, 0.8387355654196267, 0.3858963409833879, 0.7099309817554975, 1.0211512531746596, 0.4117029892988166, 0.13425265188260355, 0.6526938397605916, 1.2677460542019316, 0.5805138671780992, 0.3746427124741408, 0.5049740563742146, 0.4953652147101131, 0.7859162167797867, 0.8272578387418299, 0.23576529210536723, 0.18981376459035343, 0.20744202035598575, 0.11669821906833056, 1.1193694992083982, 0.8318287688021448, 0.4684817654366431, 0.4414891004100753, 0.9861095109544762, 0.5644654567389297, 0.5416538778991167, 0.7425317827311, 0.7770510419713741, 0.7846352066125287, 1.150766892051503, 0.3087739268502101]], [[0.9107153940361553, 0.623263559121755, 0.3876489966134272, 1.3259228041324207, 0.8158099598174958, 0.9222254181638485, 0.40085574998870444, 0.6668514820720068, 1.1462456736994113, 0.46000307768696314, 0.1412358293688708, 0.6532920150137015, 1.183537501226241, 0.5611802815369877, 0.34499190749624314, 0.48573706644072545, 0.4790842328776949, 0.7926230830864104, 0.778843563107968, 0.2446580506386415, 0.18565124767572722, 0.26745777960599265, 0.11690823242686262, 1.1739277799061165, 0.8506032277755662, 0.47608287365719415, 0.3541234002566138, 0.8538480764980552, 0.4346893636712662, 0.5614630789313229, 0.8983691678000978, 0.9829445585330269, 0.8710756970208311, 1.1730874502341169, 0.348451550508057]], [[0.8588391509559308, 0.675179202573224, 0.39056976229808527, 1.361453246738516, 0.8639272638392823, 0.8595096282417822, 0.38291910163655285, 0.642500256496345, 1.235147738235066, 0.46592327168763004, 0.15379823117842711, 0.5900695088558308, 1.187492677528024, 0.590941532425671, 0.3513051482709945, 0.4349475739853824, 0.4712651861133039, 0.8020519309255193, 0.7855514061768436, 0.20379485683158202, 0.19812883183810853, 0.22648967943510553, 0.11007966107597006, 1.1449599777937767, 0.8866216313657771, 0.4615090945225864, 0.49962837722984144, 0.8702856763816632, 0.3395535478852112, 0.6942519334078185, 0.9187174516779888, 1.1943533623273728, 0.8634645538311884, 1.1995909221540741, 0.3793668906407739]], [[0.7942990548492975, 0.703133518637442, 0.4130885983548114, 1.3078497279835817, 0.8815898109633282, 0.8749857047994715, 0.4167856949022944, 0.5957110244822438, 1.350071634746326, 0.43795648283475863, 0.1492568899189373, 0.6246095804591941, 1.1784072500089886, 0.6491660955339239, 0.3022708628273381, 0.45160539293304214, 0.5262679707523266, 0.771462979204398, 0.8293000703537448, 0.2091997684192994, 0.22369370073210273, 0.22449021951432502, 0.09326638713957092, 1.1521734730544213, 0.9469710059107448, 0.4491937276164969, 0.6220177571974653, 0.9891181045636201, 0.3569453244602642, 0.777309232371027, 0.8919241387517394, 1.1334096305351353, 0.8420647862805632, 1.1125245753022748, 0.42605016389090067]], [[0.7006398027809325, 0.7353563247701198, 0.4385954244159862, 1.2818977439848274, 0.7787240539160691, 0.8673526686220336, 0.43549114831979774, 0.5951583144093385, 1.4306277632302435, 0.44657258106581627, 0.1971017505590127, 0.713652324810019, 0.5963954403010796, 0.7422401786553382, 0.20141958852523084, 0.48372557013564543, 0.5881633857515506, 0.740358046790494, 0.8161643032964223, 0.27926838057166936, 0.23088859327442607, 0.23497588581387224, 0.09181747057140682, 1.290260327641842, 0.9727875376756459, 0.43879563650751396, 0.6886303879766003, 1.0869283796300628, 0.3505543449331465, 0.7952078468511974, 0.7523023335267804, 1.0569597123554217, 0.8365783469270391, 1.0314238174788828, 0.4727876432729017]], [[0.6245408731205734, 0.7861277043355882, 0.4420527218279581, 1.2692382234849924, 0.7712842555591741, 0.9391197755027758, 0.36198052740750636, 0.6088491853715606, 1.434663042687502, 0.43663000215232545, 0.12725873216268305, 0.8210151352030014, 0.48222378115990205, 0.7570754758335173, 0.2997429513008934, 0.4808418944951235, 0.5480200117433263, 0.7422638645141926, 0.8411044627221658, 0.29552931017284917, 0.21005431281528464, 0.2699577152213387, 0.06962461322640129, 1.3972279491987634, 0.8335627553212657, 0.4688937271469264, 0.8698880887291345, 1.040871820014297, 0.38448581302012275, 0.7957288775732693, 0.6195975893276124, 0.8481134345637666, 0.8433579162454089, 1.0584868030604289, 0.5047314762332809]], [[0.5377956019507576, 0.7698808714365374, 0.4392492537695735, 1.1843851083552233, 0.7838605355784516, 0.9037186483946937, 0.3766029694050968, 0.6510421288551285, 1.4532311246988712, 0.4442962919836542, 0.10963232929397902, 0.7516022136507409, 0.5527181276862967, 0.7655943370009342, 0.3596827871178134, 0.48644419714005604, 0.42550090535952556, 0.7321672485186125, 0.798212218626412, 0.29588283785936653, 0.2082734974065257, 0.26409040725706934, 0.09032590307127465, 1.540982446585732, 0.6441289096417149, 0.4986908341238903, 0.8525467842436302, 1.0404067262871717, 0.42101610533121486, 0.7699851824554836, 0.5349031442602822, 0.594102087503166, 0.8900460848269641, 1.0469636678406138, 0.5157565407658643]], [[0.4802774830778862, 0.7964921745115301, 0.4315484336667143, 1.1082246149977144, 0.7491201977983827, 0.8759662841066364, 0.3740175033963965, 0.6806590762201157, 1.4625176506395428, 0.4288032373469224, 0.10892968149810464, 0.8605179417069819, 0.6533280276351396, 0.8133966045717402, 0.36524612351293756, 0.4755934557516053, 0.44651737533036934, 0.7226747604237231, 0.9640844356227443, 0.31801411960146786, 0.21335753510472327, 0.2609716086427247, 0.07054053707512042, 1.6016438906409929, 0.5429242880070706, 0.5558482179743702, 0.7962947061868975, 0.8476137845556377, 0.43159591086139837, 0.7356899153209309, 0.5945169714253735, 0.46706269036691195, 0.9194411217646137, 0.9682187216920333, 0.5431965778866984]], [[0.42422882159579833, 0.8065843221205649, 0.45281635130991893, 1.1234474454250931, 0.7393219922519616, 1.0740263847118885, 0.4048983278163601, 0.6687772092594833, 1.4347238812111334, 0.3955636711528209, 0.09504346415856892, 0.8417596465212596, 0.7822802960543573, 0.7785579676276828, 0.38341439901430147, 0.44915225771884393, 0.49395236267143616, 0.6728349013954966, 0.9686182961263897, 0.287350620337808, 0.23333996196854864, 0.2549044406879984, 0.08439834932963672, 1.7305061920930545, 0.5152385028119146, 0.6721599274780273, 0.7811614499665178, 0.8009606080845277, 0.4792526111326799, 0.7873275068879089, 0.6650627852675748, 0.4115531708375421, 0.9354558777016588, 0.7716333760318475, 0.557324023285499]], [[0.36921958897560964, 0.6976288596189777, 0.36274441975720995, 1.0680186212643727, 1.0671962854852417, 1.256300038210015, 0.3706426554676529, 1.0093887333254088, 1.1617041182398715, 0.3979502173401491, 0.08357232435152341, 0.7293489053423559, 0.871858764649592, 0.9702735810233708, 0.3810197083194671, 0.4758880871178328, 0.5295120760330039, 0.7521711903848238, 0.9468043855246373, 0.313649808785403, 0.23847972526427944, 0.22291510126781758, 0.0776381129958004, 1.799180980345989, 0.5299104672206583, 0.7985072465032961, 0.8313443084074728, 0.8053134586154334, 0.6840128920447583, 0.8511631725814774, 0.7184247044813846, 0.35668313734615814, 0.9436253104410548, 0.5416566054247995, 0.5642419920897703]], [[0.32829012355458465, 0.6359585365911751, 0.3657747559187582, 1.1649315264796007, 0.9820687913817203, 1.3987626136186546, 0.34244894054216696, 0.6701691797912565, 1.0368344353176133, 0.2400596558006376, 0.060426712495209006, 0.7214550840974154, 0.9240735089892889, 1.0565850676226831, 0.40478133463409166, 0.5290316730750527, 0.568679765177325, 0.8114600252804742, 1.0019797300358109, 0.34305875667801033, 0.21933851620809258, 0.2552888556356123, 0.03189189030156606, 1.7852081307790393, 0.265695605601745, 0.6619594546836503, 0.8767194747920954, 0.8092935984551941, 0.6942288095403328, 0.8861486293402832, 0.7462089641091978, 0.3776855531362945, 0.9352032784913564, 0.4737240765577189, 0.540529419561754]], [[0.268336113442317, 0.57502476654805, 0.36911429730211276, 0.9672006367491128, 0.9992064425093302, 1.4190609574162139, 0.3241761174213894, 0.5867373694173597, 0.953442236320455, 0.26590784000627343, 0.06902606720197214, 0.7178940656132329, 0.9872471949244901, 0.9751648941572517, 0.473139668750508, 0.48341082042980255, 0.6027840432349361, 0.7647734077260184, 0.7976179614214827, 0.40380348761323615, 0.2162816814570709, 0.25464748769681755, 0.029446162127462518, 1.858235638901272, 0.11723164070760796, 0.6288110002841008, 1.0533663822912087, 0.7211390157043591, 0.5601573835427553, 0.9306185098758193, 0.7707978461203293, 0.48908477197240147, 0.9418310916872604, 0.3316903899788777, 0.5037656182389714]], [[0.29357163903948724, 0.5534886932287266, 0.353137796140431, 0.9048025823599972, 0.9826050502220413, 1.669344326964527, 0.351500890625998, 0.8357477461910716, 0.8398006073345035, 0.33107253232290673, 0.06625052933347578, 0.7052072809060391, 1.0429612011475258, 1.0407223148137597, 0.42737247234151166, 0.5241728627700928, 0.6194427947231493, 0.7748649306710471, 0.7787879800798725, 0.47844744770672765, 0.2079869554255182, 0.21910703373694346, 0.04111205177369899, 1.788340129595554, 0.22732647012184387, 0.5472235771749383, 1.2059631729601565, 0.7062215834040131, 0.49638562661726676, 0.9537208447876088, 0.7876261791664544, 0.6331555272508753, 0.970019276668601, 0.20106312914241292, 0.4090569033082442]], [[0.44747453146390864, 0.5427835992467847, 0.3683163468628551, 0.8045415132404137, 0.9321686852271378, 2.0032698535560254, 0.5237812102346677, 0.825799146219648, 0.715399207377748, 0.3953968248697006, 0.07241209819810002, 0.733231866785184, 1.13895538062719, 1.0220939197842256, 0.42674091688689475, 0.5507479226757347, 0.5672908863937269, 0.7858634119548126, 0.7708157279239303, 0.42260477958404996, 0.20525179090900073, 0.2099159526992161, 0.038512444725663175, 1.6255234302375596, 0.2848345072580275, 0.5104093033758105, 1.119564920321694, 0.6909827862758283, 0.4824393711241386, 0.9597821650371456, 0.8729891769619924, 0.7494329844348101, 1.0050962731703228, 0.19104613549760674, 0.43045272461784134]], [[0.3301380532397893, 0.5057805696466289, 0.37855933327673263, 0.5587587637977185, 0.8436831638854849, 2.157187919366363, 0.4721530128186778, 0.7351377898566126, 0.6240487779649119, 0.3679903591185083, 0.09903079609135519, 0.6856814137131693, 1.2067924834568329, 1.0424831376267527, 0.39538924514802043, 0.5020932169635486, 0.6598835915964191, 0.7062643188449418, 0.8084343402499934, 0.2619187313198203, 0.2397665130428661, 0.24534518087361, 0.040372183399254, 1.53788804028797, 0.33520964901675177, 0.50839074502456, 1.1082201046001476, 0.6207568725045937, 0.5391979470484256, 0.9776605057732867, 0.9521649914410358, 0.7645457593892798, 0.9495196798915786, 0.23945224757271627, 0.41251211174017444]], [[0.31474295047058465, 0.5121050254617915, 0.3781544719115547, 0.6552277912656297, 0.7430013635580534, 2.25145456458905, 0.4526539141736596, 0.7326061228820206, 0.6383118182682389, 0.3625922852800213, 0.14118489840036635, 0.7139035328551285, 1.266627555920174, 1.1102648984672667, 0.4062131572394052, 0.5043831534157562, 0.6246974905640392, 0.745592841228631, 0.8084526208682491, 0.22684399513459402, 0.2240716024009427, 0.25565338065341203, 0.05057200492313206, 1.5152385900290655, 0.36691232028868415, 0.44491075244007644, 1.0497025419356811, 0.569364002964547, 0.6317889663424807, 0.9733099785521505, 1.0151588809564243, 0.9190020290521442, 0.8298816080501143, 0.4285193722363445, 0.4179450519999446]], [[0.36764525568211337, 0.5299665938368396, 0.39728291506587665, 0.8326014020532462, 0.7554224516256345, 2.2894797827119526, 0.41157205819369136, 0.6743594050848404, 0.6743280438810848, 0.31358992952275544, 0.14829186996835247, 0.7005032297155618, 1.2027077144591471, 1.0279034045107618, 0.41132471514810676, 0.5290341460677835, 0.5716772689377306, 0.6765072682231686, 0.8001087369028729, 0.19987971825086837, 0.20777097756135604, 0.28152055532975934, 0.06529738121402172, 1.4755825571824588, 0.40608949302672326, 0.41899557815583743, 0.9859269030274831, 0.5376579795061327, 0.6456635781829447, 0.999293595055275, 1.076582586755829, 1.0414986410015319, 0.7213504374642086, 0.5034355144903538, 0.463542460514313]], [[0.34700295480392807, 0.5193703305219541, 0.42124855667933037, 0.883543802270755, 0.7520594145078584, 2.4140599765427675, 0.44105588834517206, 0.7199475282153842, 0.6693825663788381, 0.31990734836153334, 0.16115875532607468, 0.6679494087308795, 1.1799917392422876, 0.9193980549470925, 0.4036398702827816, 0.5341154758996222, 0.6016354636794639, 0.7857913022308871, 0.8283592285584287, 0.18822714171729948, 0.19880114913917257, 0.2368277953326268, 0.04403450049426878, 1.3308441832326703, 0.37844041914966986, 0.3806589036739791, 0.9066174682846533, 0.46671146194968854, 0.6685768638933643, 1.061323631088544, 1.1566062115162006, 1.2081838631422885, 0.6044107220285009, 0.5999752762230439, 0.5260679348537719]], [[0.44424055028462084, 0.46632799026851823, 0.3324112880664344, 0.9190852809527236, 0.6493289852363524, 2.496682460397894, 0.48873668441284535, 0.7148893307396817, 0.6512729691177652, 0.3776945298708128, 0.1950272399512343, 0.6319168636380095, 1.1801265061757815, 0.878591776270197, 0.39286373084932524, 0.532933283244258, 0.6096406424024144, 0.7818653510551218, 0.7782863256145853, 0.18150764974020922, 0.18212714302441357, 0.23682364368243597, 0.05229192425202295, 1.1551366653310717, 0.4463690690812392, 0.28896843934454614, 0.8868017840476317, 0.4764553156306345, 0.690048807358065, 1.1167304116872778, 1.2633510450396064, 1.2817879274486341, 0.5062969766127926, 0.6187211832698386, 0.5472871585269257]], [[0.4260065792193115, 0.4694387786711592, 0.3364343326059259, 0.9773368253080272, 0.6189014771152077, 2.4197823060676305, 0.43334144772987887, 0.761108984282085, 0.6624239836578678, 0.3921449603302777, 0.24644278710299503, 0.5869086454734551, 1.1867280193273542, 0.8716568532945865, 0.33014633214456157, 0.5536006868489618, 0.5907952219488921, 0.7861589721114575, 0.7873007079446072, 0.19824769137427856, 0.17802961528133415, 0.20892680109425918, 0.08096010714482214, 1.1277654471611462, 0.5227486727446695, 0.09985047121167304, 0.6771208109936331, 0.519164416115658, 0.7533613744120036, 1.1377681262262112, 1.3463901730836136, 1.2747547680833333, 0.4191536642018617, 0.6249734455864087, 0.6995826530711258]], [[0.47210455791139, 0.4636197679008355, 0.3312508114318543, 1.0825270573094818, 0.5673350089633262, 2.4352463853765838, 0.44501626008358186, 0.9021775271317174, 0.6545681894987984, 0.4140012319983443, 0.2944545924618034, 0.5902819835204574, 1.145226634725066, 0.8607936217748806, 0.34656856277953907, 0.5189190094817172, 0.5932621008492367, 0.7961015274788181, 0.7835691846471591, 0.2184912179120967, 0.17848178815425583, 0.18641426367937863, 0.08608008448220156, 1.0517181925574568, 0.6231778138552637, 0.08374505223613074, 0.5720784590603261, 0.5443585706168612, 0.7061325635248147, 1.1484061049298673, 1.4018619191031614, 1.322136422493306, 0.324328416240347, 0.6145601695143834, 0.814911409163253]], [[0.3797882498577657, 0.4811088768232642, 0.33620320608295223, 1.184344494123584, 0.42381647542292067, 2.3250432715817917, 0.39815062233906173, 0.8618311193476416, 0.5629211657475149, 0.42660997839820103, 0.4017898035056722, 0.5907122510441212, 1.1248517018397117, 0.8408501874754366, 0.35072645365603095, 0.510175058531421, 0.5684117096230958, 0.7950131529663579, 0.7412126118194988, 0.2042812094881979, 0.18893152394748738, 0.1821783306815731, 0.09449399092110478, 0.8525516498654855, 0.6890706615144933, 0.13724834118249982, 0.5209996916282906, 0.5214535675816552, 0.6993186952467835, 1.0575885781287795, 1.4338216950139828, 1.3824620570605517, 0.24254535992453957, 0.6470261776220207, 0.8393377892889522]], [[0.38797442186734177, 0.4788652213629173, 0.3349537681228607, 1.1687375983473536, 0.34298244688008384, 2.2940644601951083, 0.4094702846292574, 0.9297718997977554, 0.46501339306189704, 0.4264833722590589, 0.3935508523106586, 0.610113537697057, 1.115867052438627, 0.8250089744722428, 0.3474309514070826, 0.48973511867224073, 0.5076927567536713, 0.7699787875853112, 0.7643664884839357, 0.19429264978018962, 0.21185296646942586, 0.19517218938563916, 0.09601514275108386, 0.7756797860670667, 0.7757872967107933, 0.18837509001100275, 0.45420125379399645, 0.47833313773088715, 0.7190673499142161, 0.9890579316976167, 1.4484706798604932, 1.4155559440422638, 0.2253869352117861, 0.6535569899304652, 0.8020246306039664]], [[0.3668478028928712, 0.4575304353625512, 0.33020403187536174, 1.2527717274181331, 0.3236347245357831, 2.2704669690050614, 0.38262993222998143, 0.9658007733864515, 0.42737616065023115, 0.3833619942940394, 0.30886316735441066, 0.6189262344342431, 1.1322560184035364, 0.8626094552704539, 0.3521958061286056, 0.4956081446840238, 0.44749312575018785, 0.7686214970556928, 0.8298876633387993, 0.21508647987309587, 0.1940829869757425, 0.21269592771065549, 0.10870857354550725, 0.47259484107365246, 0.8878493582096112, 0.27585390258889464, 0.41766105175025303, 0.43398811463475684, 0.6194832529992449, 0.9712930963513143, 1.4642925979897057, 1.4421343257365158, 0.2557753292962205, 0.5905037639244876, 0.7791365284243945]], [[0.36055273127172904, 0.4265497307136341, 0.33461026617678147, 1.2120612264192574, 0.5134815951124472, 2.221839390283304, 0.38121432544645767, 0.9671184518209142, 0.4410424047247954, 0.35630576815523163, 0.28678566646559966, 0.5888420486452217, 1.1546056863143057, 0.9174079032308988, 0.36951395723738845, 0.5009017619701605, 0.43155358899346663, 0.7656465757758725, 0.7860240654032872, 0.20533252609624802, 0.22314849891839403, 0.3062193571523719, 0.09688830810018217, 0.31276076938431313, 0.9482056733628346, 0.3604059767522194, 0.5295564705785967, 0.39675879370977307, 0.4887803806762375, 1.0098621260057798, 1.4786553690473758, 1.4577934595587039, 0.3612989126353879, 0.5726680943038445, 0.7971329254099541]], [[0.3422919442790239, 0.4117589838741563, 0.3394514952677906, 1.2404323190586128, 0.71144763774371, 2.2225781856375098, 0.4050878507650981, 0.9594727386265536, 0.5167231942553239, 0.3228543340673852, 0.29165285915360634, 0.5859520531877078, 1.1836868900537751, 0.8962400235154242, 0.5508280451018328, 0.6274786602837278, 0.4327986610186234, 0.7001353761354026, 0.7444328800596514, 0.18632113804279127, 0.25420089156309816, 0.3025597264437352, 0.10002731888857855, 0.24042590800665026, 0.978892943295715, 0.43004939036917855, 0.6065131626690715, 0.35463055138675614, 0.48338420355272665, 1.0354400993854016, 1.4834989420165314, 1.4618638010809093, 0.34665986357166767, 0.6128095581291357, 0.796124817235752]], [[0.34403234091629753, 0.4004295203122752, 0.3433213975574979, 1.133167359417689, 0.8142856874315878, 2.141072016277312, 0.368539565159095, 0.9486269725162391, 0.5786730761726241, 0.3004713926640489, 0.31258036371642345, 0.5759932689065879, 1.1991602868110809, 0.8737807914943507, 0.5924456926056931, 0.5806446484097134, 0.4255805692024166, 0.6741643910038685, 0.7078422326284534, 0.17484705471334572, 0.2661709059988576, 0.25029594129516153, 0.09619717404689312, 0.22918511182533277, 0.9934962225749905, 0.4845252345487174, 0.6105904260076648, 0.33391907709698004, 0.44190762733727074, 1.0508805366119178, 1.4895654029107546, 1.49712977188803, 0.36044458573580607, 0.640821181321031, 0.7961000325711316]], [[0.3511346715416571, 0.40006864941028963, 0.3391553898645387, 1.0433468425200534, 0.8799083683144305, 2.142181376833042, 0.39635177440510677, 0.9400846444773372, 0.5810707062746947, 0.3068504202665808, 0.3295582059086314, 0.6333863543550573, 1.152648778566898, 0.7428396527621864, 0.6272543397822556, 0.5504943431944099, 0.38963173645084614, 0.697893965455774, 0.6785988483662091, 0.1693077300362742, 0.20912613095780525, 0.22910173973728948, 0.09437811793862813, 0.26401262985055307, 1.0150564018819352, 0.5192170022676244, 0.5644318969140242, 0.31897684874505566, 0.42253831292283334, 0.9889481500850293, 1.486205897793268, 1.5076318763257148, 0.2968200530722892, 0.7003477521315916, 0.7862633234661851]], [[0.34237613463127226, 0.408294404324762, 0.34087685929504724, 1.0347322016937992, 0.9038896534914211, 2.1055837536603623, 0.40101389452479896, 0.8990774999913376, 0.5971693291343566, 0.3049918731529582, 0.3296106424547903, 0.6435156215542, 1.0845156921724186, 0.7991964044583435, 0.7184611536098904, 0.5123227382811885, 0.40285472376835074, 0.7088283369369781, 0.6450591627242278, 0.21039056679155427, 0.25318035910773107, 0.20847152995708262, 0.058225205732784044, 0.28614759776378484, 1.0317074293426451, 0.5366783139345738, 0.5359269989206222, 0.3071927942215129, 0.4311325644249671, 0.9610534809441567, 1.4833684103030764, 1.4981735916135726, 0.2615763938707533, 0.729447822739763, 0.7847230035324212]], [[0.36397540333475054, 0.41518803169960716, 0.34342721047702973, 0.9828606207110133, 0.8980800162010427, 1.9522271741586885, 0.3565803119053904, 0.9423074685505425, 0.6132753226363569, 0.3089956859882279, 0.3532591232817825, 0.6731745237617107, 1.0115068296870107, 0.7552087968363234, 0.7128495315968577, 0.4894395082584385, 0.3991138419357957, 0.7178095934188289, 0.6111487477398989, 0.21082819924151847, 0.25644751539512, 0.19954321531086458, 0.04677437459006274, 0.27518366867640054, 1.0667982824476487, 0.53168225871587, 0.5741182615381004, 0.3019803769572179, 0.4168496408273178, 0.9163912227689981, 1.4757211750248143, 1.4979681167056649, 0.27953628583837525, 0.7736591836725697, 0.817895187746225]], [[0.3357928698024687, 0.4202136655714114, 0.34424589144373524, 0.9414276165201376, 0.9104909285759799, 2.0012624782002892, 0.38196055113936705, 0.9583260539196399, 0.6232507585107421, 0.31596207492365747, 0.3390623253663707, 0.6365400269555571, 0.9178238067637308, 0.6962633098956922, 0.7287247078351802, 0.4718093426581279, 0.3936286604631586, 0.6868193839040047, 0.5884801182381643, 0.20445907646981276, 0.21127093966849664, 0.23406281841561494, 0.06139998672888819, 0.27408962497753153, 1.0001045478680823, 0.4971935414008698, 0.5839539175497889, 0.2720242609795755, 0.44457024474999673, 0.9002353629852573, 1.492226173800589, 1.5104447775843002, 0.27759969524898925, 0.8032965198585964, 0.8359427963745197]], [[0.34438401084474213, 0.4370458584458137, 0.3491121405340049, 0.9784908196276825, 0.9662848639140073, 1.956461607516311, 0.414263110625194, 0.9642873156220667, 0.627631558462888, 0.3335999821486316, 0.31472730630987544, 0.6345378360277438, 0.8628768741626283, 0.6920846400093472, 0.6901504006313031, 0.45381923292631243, 0.4201925275710122, 0.6765701077114705, 0.5915941508683998, 0.18411217992219836, 0.23391598395972435, 0.2821942442257104, 0.0832717174097763, 0.2932526898740424, 0.9908324026134838, 0.47107902570990867, 0.5928279162115278, 0.294531192266139, 0.5248492519759667, 0.8875147857591783, 1.549158806240119, 1.5602403627583834, 0.28119039648246846, 0.7835313980459091, 0.8769432363926145]], [[0.38160026183072737, 0.4214045103491143, 0.34738452090108357, 0.9857752505525796, 1.012524829206667, 1.8982346698087627, 0.4437326914492227, 0.9469063912009601, 0.597982884332447, 0.31084386298345257, 0.28347302198072166, 0.6710836624908498, 0.7983473760617315, 0.6465658847271739, 0.635768719033768, 0.548403892573328, 0.4310869863961373, 0.6305298111408653, 0.5609613358299343, 0.16452981773768682, 0.2837614028612774, 0.2598307678886349, 0.0736531518532173, 0.3332022557552003, 1.0227480019570732, 0.4464388362703092, 0.6049226655280988, 0.3864950273175127, 0.5767918939643136, 0.8576151020345395, 1.5103625597866817, 1.581213568641927, 0.30645227992276314, 0.7498863609893015, 0.9000373396070296]], [[0.38726114859907235, 0.4168527793530633, 0.3544254883063232, 0.933849163140882, 1.0239848095251467, 1.8733465010752752, 0.4814720703188953, 0.9817561099497583, 0.5430964881049815, 0.2682626707389943, 0.25337427469119034, 0.6759599687586609, 0.8339949054766413, 0.7322466079410488, 0.6157368979144835, 0.5673667034671146, 0.4210381071007966, 0.6354062267997415, 0.6243588683798154, 0.1833568786622545, 0.27702924481626284, 0.2520468418725056, 0.08156288236478235, 0.38106225724729836, 1.0781553830861959, 0.4327678974719815, 0.605993368679812, 0.44216246861007336, 0.5992167609251411, 0.8828178495964167, 1.3704839219428668, 1.556529047907878, 0.35266034459783957, 0.8454024092430539, 0.9607884520923049]], [[0.39528407465124415, 0.47497096396492317, 0.4122173195156853, 0.9200073305467015, 1.036752369152367, 1.687138433292181, 0.40797523250723533, 0.9683885427351493, 0.504169910308682, 0.2504613564140803, 0.23784746357173037, 0.6694623862148383, 0.6221219738474796, 0.7856000645983803, 0.693650728010472, 0.5938854403347669, 0.41711152587302625, 0.6612119240553372, 0.720785185252728, 0.2357979663265284, 0.28312443905593754, 0.2483074564614788, 0.03323740287811833, 0.4510034803367117, 1.1500941632325485, 0.36238485813039967, 0.6247565563471382, 0.5397285662356686, 0.5459269278916685, 0.8821410608322322, 1.270359758275809, 1.4860423618751635, 0.3949292156574335, 0.846515915631227, 1.0365906970767165]], [[0.3886022513941788, 0.41250555680942214, 0.36259396278625206, 0.951576128215506, 1.0410536204110126, 1.5508709424001017, 0.36351383476976157, 0.9133630911099521, 0.48957164160744165, 0.2528981384408039, 0.24113935783204454, 0.6676960151191016, 0.5543196304186067, 0.7716085144166095, 0.7899174857202274, 0.5602593748574237, 0.42426807667009814, 0.6446281237398885, 0.6780101415083678, 0.21588968775071865, 0.23088338176480616, 0.20738440291565988, 0.023775249525861544, 0.5047824810661692, 1.2476010855588515, 0.2879510142895526, 0.6174497532229672, 0.6118920617763481, 0.45645723621027745, 0.8872374715854454, 1.1804573388993331, 1.4368796495803426, 0.4395026533659576, 0.8882819943623396, 1.0561453876076927]], [[0.37956127887044155, 0.4037433418267264, 0.35676405795389826, 0.9348905903160266, 1.0088907043952746, 1.3865656912399653, 0.34172153748368783, 0.8666168056462442, 0.5143118240097779, 0.26170147321866055, 0.27019299103911193, 0.6916365324163345, 0.5614323979906539, 0.7607038710388524, 0.7946932364395416, 0.5904534736583676, 0.4172716182628039, 0.6412299045833205, 0.6524285968066357, 0.22378704155976728, 0.22255387294236895, 0.18074957733381147, 0.01819986868242179, 0.5596137333602954, 1.2438086337982721, 0.26139378103738686, 0.6297764238420984, 0.6664672413406448, 0.4033521562096838, 0.9083658218653214, 1.064474284553782, 1.3944262377464112, 0.43861481901161337, 0.8966988841150214, 1.0285229900600203]], [[0.353798727561743, 0.4042157635206538, 0.347678901735983, 0.6938821963033639, 0.8041174694388087, 1.0630484993899734, 0.3356426532395214, 0.8628087945138185, 0.5159293020236204, 0.2623263066908935, 0.2515546307865451, 0.6383665810832129, 0.49288096520792657, 0.7355717603634597, 0.4713622076054434, 0.43671805747343495, 0.4017481971121848, 0.5956271248981448, 0.71796192175861, 0.2606784539482576, 0.22611952811942693, 0.15998663804457608, 0.01573145513301193, 0.6458715430821229, 1.1783373240388704, 0.2506534028448568, 0.5889451245974155, 0.6933029457446771, 0.35758914786303836, 0.8726514349597476, 1.0175827956710486, 1.2687745728575606, 0.47804902305182273, 0.8126536413938781, 0.9614406719424775]], [[0.34815181817070084, 0.43554975834424403, 0.3755636156093183, 0.731684935748974, 0.8486972659106521, 0.9889117755715681, 0.34284296059702, 0.8585050564688191, 0.8374799738334491, 0.9882670021081055, 0.24387496056131797, 0.9895346825093654, 0.37415201686030436, 0.7825718860587476, 0.49040653375105103, 0.36007310855043984, 0.3118913933756264, 0.5812053901318859, 0.6484955040924167, 0.2501118494357013, 0.1642437696409506, 0.13837889255460645, 0.00647894015065872, 0.7330651036730705, 1.0779182603974933, 0.24016673931656177, 0.5824904015803799, 0.7081377084839731, 0.2418924532540056, 0.7999241743420047, 1.1243902757110376, 1.1195665133702482, 0.6775225871314287, 0.8298755280272512, 0.8791846191855228]], [[0.34110432372533483, 0.40675875809073336, 0.3523199915008792, 0.8093120643257004, 0.8117171669267725, 0.8431172681943058, 0.34931707947788604, 0.8180329563245898, 0.903578570451685, 1.150149714302932, 0.24563069213598282, 1.0135137994960157, 0.4209341334447658, 0.7967299735558733, 0.4723226268765098, 0.39271083515888805, 0.3240294358494198, 0.613903129593568, 0.695751074158747, 0.21387227330510628, 0.15331425334069002, 0.11692387578778118, 0.01550701882315753, 0.8233981556085548, 0.9052613343631342, 0.2511976133874053, 0.618363351136086, 0.7017511087256068, 0.23005922357221698, 0.7832843741982083, 1.0566454548190611, 0.936379889835889, 0.7793128695495346, 0.8370000982950483, 0.7827547234593892]], [[0.3273275996141437, 0.4078713368054415, 0.35889906395994026, 0.8892474117827791, 0.79598526216518, 0.798941956298578, 0.3625289597215188, 0.7909030418597204, 0.6820551161797415, 1.1520672135228198, 0.2238763359117958, 1.040309279983902, 0.48669650117271956, 0.7168321890942666, 0.5056076478374665, 0.3213448066122857, 0.34051530703544625, 0.6857553666429388, 0.7953612862157695, 0.25451435649048865, 0.14887057873264375, 0.15670532810437018, 0.013969375116087979, 0.9174751243966222, 0.7360428369316754, 0.37176668952335795, 0.6786536001812584, 0.6940289810833836, 0.23317302526329153, 0.7749826516427032, 1.0181697203471238, 0.7320054017395825, 0.8314595867020071, 0.8666591461954585, 0.737545107987138]], [[0.31282916025663576, 0.38909698269588067, 0.35742762280591706, 1.030325095159927, 0.8034802390671982, 0.8789028673313376, 0.41077369876124586, 0.847163299539655, 0.6576487482011669, 1.122614999511201, 0.23926363138038526, 1.0618184952915954, 0.5338577697174421, 0.6606697284833783, 0.43886393439661936, 0.3863566109818001, 0.5106664080631576, 0.7363382638743193, 0.8523575376562961, 0.3016724867603612, 0.14164802919215708, 0.1540497691912207, 0.013486913041017602, 0.9731921424318812, 0.682557976503864, 0.44125345560198714, 0.7051111175204691, 0.6898472455383866, 0.2324401348236424, 0.747715478297579, 0.8998091124881783, 0.6373870882424192, 0.8405030830989224, 0.8264967471533994, 0.7281161209957825]], [[0.224773853229426, 0.409693674141576, 0.3343978654150357, 1.1280579223089784, 0.8074554731148487, 0.8837520117522302, 0.4022442686639086, 0.8832725734215914, 0.7127906847182611, 1.0460354498170203, 0.2627590929855796, 0.9489022936814102, 0.5679103006067707, 0.6354094963065389, 0.4634361648480701, 0.3426362198917158, 0.5446247171637547, 0.740014609180971, 0.9014846557430176, 0.3264189672906396, 0.14516658338669253, 0.18957056938173308, 0.018919299934196256, 1.0339849711679325, 0.648250624407865, 0.4320411933624436, 0.7184765599913717, 0.7578421853604655, 0.22544198205109178, 0.740769531863513, 0.9158112241292446, 0.5632925680388734, 0.7577981297463612, 0.7222789653122119, 0.7434950383215455]], [[0.23365354111715902, 0.40460865598653317, 0.3470421990114033, 1.1693633476830958, 0.8490766507251185, 0.9307233903188442, 0.38661947237873995, 0.899718032221923, 0.7193005230522601, 0.955597062154083, 0.2855140807489559, 0.9874911337075438, 0.6088638185032713, 0.6260591446283144, 0.4538752325732328, 0.36595902153999915, 0.6725572424143126, 0.7758280744444817, 0.967915443809799, 0.3419894347817533, 0.15165861126517324, 0.21539249701082072, 0.016312122624247628, 1.0842773762158968, 0.5694261514951892, 0.3751319723294671, 0.675321174146679, 0.8259814801604355, 0.21233016009165104, 0.7994255357535739, 0.9462101747768578, 0.583020979943368, 0.7138500257373814, 0.6836037401213717, 0.747525219734576]], [[0.21812215524665768, 0.4022638893906258, 0.3469937160259634, 1.1537902381142306, 0.8671980410884629, 0.8653169506069729, 0.3880191666742829, 0.9468670234502635, 0.7389633066836193, 1.0547251721327333, 0.3120436250310761, 1.027150499425546, 0.6213148436805181, 0.6031835449082471, 0.4668670212263409, 0.34352196161405346, 0.716912581018773, 0.8006667993659815, 1.1072373012738477, 0.3621660600943195, 0.16309514478430812, 0.2607006860919444, 0.01565749783768655, 1.1011869447609544, 0.5218142294814732, 0.3661079544790648, 0.6269753426093421, 0.7707421209737862, 0.33304290183914076, 0.8067974221095409, 0.9880897677658945, 0.6147836440569154, 0.6809797153264167, 0.5879914315478824, 0.77333473075961]], [[0.20840623079324913, 0.4009175756286778, 0.3420047331308906, 1.1455907862179193, 0.8679084830343875, 0.8704862093738834, 0.38588513502229993, 0.921320782467615, 0.7751223142425719, 1.095713180663884, 0.32661551744004125, 1.1058363953665298, 0.641398645782675, 0.6259713870690586, 0.45257877963713833, 0.32788174211261495, 0.7350571914672085, 0.8086008691168229, 1.1372762353256354, 0.40376465009338774, 0.1652149276246808, 0.27764208068579666, 0.0335992034088127, 1.114686343677064, 0.4979665869375808, 0.36000115686931333, 0.5790278667651348, 0.7668227605504988, 0.41041434924751086, 0.8549689871698748, 1.0212763701480179, 0.6575496639202367, 0.7501617218247767, 0.5565817859118325, 0.833123565882435]], [[0.20114662404138337, 0.41058203514545705, 0.3450828988376323, 1.0904929034028583, 0.8686418869484851, 0.8696483849586474, 0.3846824738108616, 0.9291202945366803, 0.8085454191486746, 1.0821996334083297, 0.329979795629849, 1.0094047784653764, 0.6366481681381126, 0.6458210086669419, 0.46740842536471955, 0.32313750337500136, 0.6334124900502085, 0.7588125102620724, 1.0444379108736475, 0.41531103089893684, 0.18609878667609847, 0.2716768027674461, 0.018524097391831938, 1.0801464494116637, 0.5543469085735923, 0.30534361703343904, 0.5010336731265366, 0.8150909739739989, 0.435150157070075, 0.9558660133384961, 1.0603271649024286, 0.7134257888730282, 0.7096848081702684, 0.4968029056173488, 0.8928979032522328]], [[0.18087176745670971, 0.40605200622787463, 0.3386627823746673, 1.068639017583098, 0.8646224430410139, 0.8222425657764554, 0.3896893574143681, 0.8358933810830714, 1.0026323032685918, 1.1927839880252529, 0.34783443333317876, 1.005064630005508, 0.6310419502083746, 0.6043544125071953, 0.5061982246560496, 0.32428965456005643, 0.6188045293286257, 0.7222792983761149, 0.9899850893966102, 0.38573676515943833, 0.20726093238400592, 0.2823843661700118, 0.028747326237430492, 1.0168608113312256, 0.6740719524248346, 0.3323528685805147, 0.4004317214077616, 0.849016718754978, 0.46820512439882234, 0.9438689591547642, 1.0581276077175863, 0.7917632689624061, 0.6146511610227431, 0.4696138252243299, 0.9828463480295359]], [[0.17548872930384926, 0.4088255870167522, 0.336593896296653, 1.0808567104188531, 0.8977388209405865, 0.790912302403266, 0.3960447358583165, 0.8399543466296816, 1.0931196517735522, 1.1711563194541428, 0.4666145640404165, 0.9139280759023264, 0.6151483339097813, 0.620384757588685, 0.41534533405347956, 0.33475741410956705, 0.4680511629123515, 0.6696297922310163, 0.9613816470231767, 0.3936498088446612, 0.21705030051108243, 0.29066841173201685, 0.01329449597820059, 1.0633291372120364, 0.7093951249116567, 0.3847046309243254, 0.3895691061553962, 0.8214478059590009, 0.49545538426596364, 0.7827456797569907, 1.0311051330500782, 0.8407857401410694, 0.5610080669183621, 0.48488456526235013, 1.1169391709683545]], [[0.17331012330198106, 0.42943560278372017, 0.3448701823442207, 1.161713252165514, 0.8705483088359847, 0.7954493166308145, 0.39157673602176046, 0.9765105603905417, 1.1352646893398595, 1.1480655640438004, 0.4769520443745998, 0.8866339379249133, 0.5913142792557198, 0.6366312407643835, 0.4091875540367364, 0.32649671152055904, 0.4410955682629625, 0.6378560228867292, 0.8999284434089301, 0.38665310269906555, 0.23365171601193788, 0.3164646519551401, 0.015279212682114554, 1.1434768514586438, 0.6327323944723657, 0.42657641058779616, 0.5378118715298283, 0.7373032716084197, 0.4789142698197207, 0.6292620406502465, 1.0211591855995903, 0.9147443280070644, 0.49618961541565976, 0.6061694340966628, 1.2180041613658605]], [[0.1607420698133015, 0.41400563245047606, 0.3246788829692412, 1.2840623530045836, 0.8764622312032055, 0.7765315286043206, 0.38956577876032183, 0.9798067258130919, 1.1232841780293135, 1.0608609897644325, 0.42249767147238443, 0.8711506080482591, 0.5482911530102846, 0.6526776644879702, 0.41384427474193414, 0.30124340553619783, 0.44257174372808206, 0.5483231337741594, 0.8014290285485705, 0.4129639092703333, 0.26034714000773324, 0.3331540242366499, 0.01236751991802586, 1.1441787988701122, 0.6013346145671645, 0.478252104265669, 0.6645790202574208, 0.7210875211423611, 0.4955750350818923, 0.5189608864400153, 1.0198506464942085, 0.960676064270561, 0.4594231605598813, 0.6439891062194851, 1.2161602636738245]], [[0.154523348461072, 0.42080466427862906, 0.3196214863986523, 1.6228687979286767, 0.8190262125997998, 0.896410864798592, 0.38283509261059984, 0.9422479489200566, 1.0943429162409313, 1.0461774173852392, 0.46899673772225536, 0.8558365136097564, 0.5478689554932255, 0.6910325164361908, 0.49293557683120687, 0.29649343538294604, 0.4050898492172269, 0.5373015099304512, 0.7805345748926155, 0.4381797680671203, 0.23682343056255423, 0.3780335381265798, 0.016276325252311413, 1.1589007034882495, 0.5807322162691468, 0.4950476155764515, 1.1575393914303223, 0.7605221006142664, 0.8803386063982709, 0.4573254839353035, 1.0185773918290677, 0.9338987647848058, 0.4629019914661311, 0.6697765869874419, 1.1828756627052701]], [[0.1560793069870505, 0.4192544284248708, 0.32066863483419716, 1.9276686373145546, 0.769654342881243, 0.9076765769613505, 0.361915570711546, 1.0040909355233392, 1.1069180343564329, 0.9435200529687131, 0.4306220393614455, 0.820182401276471, 0.5316143359924009, 0.7264207835312809, 0.5061882869151988, 0.28022045638228316, 0.3960113661898875, 0.5313596507294638, 0.780967675466284, 0.36813220488073983, 0.2290777817641771, 0.3991259982692558, 0.018102902865654434, 1.2763579846736968, 0.5311484320468425, 0.49575677845642185, 1.5979473309118053, 0.6588049683724896, 1.2680752026976927, 0.37849749323772786, 1.0142635787048988, 0.7979064065962165, 0.46466978149647176, 0.6607642255227772, 1.1605577398615687]], [[0.1516355760979405, 0.43783436885945004, 0.3182435687388824, 2.18119197106751, 0.8702547547351253, 0.9849786275461427, 0.354361383912034, 1.2420146978655404, 1.1451495698820628, 0.9009146060615418, 0.6127996582973789, 0.8108025837767373, 0.5062892492601049, 0.768716961011992, 0.5527335226474033, 0.2748337701807715, 0.39337238036776345, 0.5090962120000633, 0.8280726151086271, 0.3012204555835538, 0.2198309118418761, 0.4138928687911651, 0.017482313181676618, 1.3179525268812267, 0.47122894768098655, 0.48228947316787524, 1.6427610718254306, 0.6318343305235049, 1.2523619662019612, 0.29694000307897417, 0.9956562487059702, 0.7084891736949813, 0.4408984493028345, 0.6143746789889069, 1.153707701328778]], [[0.1638263013904436, 0.45860607500055595, 0.33560192058779337, 2.2943865297564883, 0.9008994474729781, 0.9926484629739338, 0.36297578292890276, 1.3252005374224165, 1.0363958530190507, 0.8482633683383576, 0.66121287580874, 0.8344014454665409, 0.5061817692124011, 0.6942447019182783, 0.5887142217601145, 0.3060231801665777, 0.43330496785989014, 0.5595058457030131, 0.8727628299160339, 0.28489043647923606, 0.20065908002345081, 0.4213648465718629, 0.04220420868968558, 1.4093276611482906, 0.382969225413574, 0.5130460159299627, 1.7285134446025698, 0.5899104250768222, 1.294603758180719, 0.21886235015098324, 0.9231131236849163, 0.5675403074217925, 0.416450659104342, 0.45336669054287937, 1.1298177366744453]], [[0.19059666751744583, 0.4190185806097975, 0.36130297103629955, 2.3553318619647046, 0.9448271156574564, 1.0666768020268338, 0.36566458960642273, 1.0514110517532242, 1.2196174955422827, 0.6451538256680002, 0.6448419478566136, 0.9475467895670026, 0.5578117431116703, 0.6904872708308608, 0.534424557949551, 0.3335640301734202, 0.48296809708258226, 0.5356416718333139, 0.9078913186972963, 0.2659117073560824, 0.19006863575949465, 0.41995848467947544, 0.04882280217489167, 1.4994455269039426, 0.3720665367143421, 0.565048149975844, 1.967107404485283, 0.6506374713510843, 1.344956582972518, 0.20765957948866542, 0.8173276598416694, 0.4497040369055302, 0.40521924665289444, 0.4651125999660515, 1.0364059029120467]], [[0.17976505155654657, 0.42455497062396186, 0.3702299173108155, 2.378474965736604, 0.9937324336071911, 1.042367656474331, 0.3667549548600338, 1.0813254625708413, 1.2780383301593845, 0.5173306942015432, 0.710009361401982, 1.0768159435806277, 0.5943593688902147, 0.6752023224782944, 0.48818855831907443, 0.3988369919657194, 0.6478124932024364, 0.5383518410161341, 0.9614557488125048, 0.25285093853386625, 0.170050236571284, 0.44291589199619485, 0.029483573309883465, 0.9821461336414992, 0.5094001530848347, 0.5862175962896791, 1.6578984528332916, 0.6147334227436264, 0.5794868659572208, 0.3428694376106835, 0.6680440663791452, 0.3570543080370262, 0.37648630739040856, 0.47557049425381276, 1.0014179784228736]], [[0.1805510633330615, 0.42714366206720183, 0.37404550435165407, 2.3666951005639296, 0.9832499384831629, 0.9877472845785896, 0.31716121206484893, 1.1196597978223761, 1.247875304687202, 0.4399497026337943, 0.7330051577323711, 0.9626403235563455, 0.6316674878712583, 0.6275193387723892, 0.4814408505621309, 0.39366998335560077, 0.5264388884044833, 0.49158145595553154, 0.9557420794983474, 0.21498992896498703, 0.17263533630978364, 0.3920977948768327, 0.022400915346727234, 0.45106583613603735, 0.5074682277744348, 0.49182131156776104, 1.2996991480600615, 0.6633536111436128, 0.2918698248765018, 0.6404865424953314, 0.5586602397007637, 0.3367114573119527, 0.3723413489086347, 0.4933175927536713, 0.9057164280746586]], [[0.18121183291088913, 0.41962834688765926, 0.3725545784262596, 2.355667619111749, 0.950541732868803, 0.988899677031735, 0.31686731678609814, 1.124363008085754, 1.1418673277840303, 0.45419420917529507, 0.7299169061951367, 0.9352184308614876, 0.6565396688811564, 0.6768870155410691, 0.48853443927813384, 0.3924014790452741, 0.5279282594749335, 0.523191212719482, 0.9845763947016015, 0.2193363403919421, 0.14248449628026488, 0.22238639008050629, 0.02406013971877375, 0.3298247781739094, 0.7026944355424963, 0.38234490464022086, 0.670541934303666, 0.7175568812312254, 0.24265506872843218, 0.9370945546278795, 0.5414150218817746, 0.36705372728121954, 0.3623923659274821, 0.5517070766078267, 0.8810124026936772]], [[0.18161013417070762, 0.39165655404311994, 0.3607656053751345, 2.379062956040146, 0.9910399796006057, 1.01691777555203, 0.3145488003818415, 1.1380749887974284, 1.0687712760483732, 0.5822808979271161, 0.7426371188708292, 0.8943680727455907, 0.6134889217051895, 0.5781295704903909, 0.4934623659680497, 0.34905778553220645, 0.529128024639574, 0.5167632091381658, 0.9273087311146497, 0.3112211098179035, 0.14749313368513448, 0.17815797113496923, 0.020297832556689, 0.3833788669492449, 0.7549549329549949, 0.433037459439347, 0.4599505210364149, 0.7398275516450963, 0.31978235117576714, 0.9855185623048547, 0.53056732368179, 0.3832972597648131, 0.34162045408624564, 0.6318902492220106, 0.7894255783496475]], [[0.1812501489332443, 0.38822030629163434, 0.34992287532729527, 2.111758412167748, 0.9013532243465896, 0.882734269121235, 0.25369925639285545, 1.2246032223996743, 0.9279562687786073, 0.6830776248004936, 0.6439104770302087, 1.0045837688364112, 0.7512492230803038, 0.6099080252143213, 0.5548634671209838, 0.35011264849876156, 0.4593151142341827, 0.5037501079357672, 0.9642949939562779, 0.20164969888948886, 0.19424469530156432, 0.132923913504235, 0.018189730964111788, 0.41661740072192, 0.9030833032372025, 0.4712150359980951, 0.6601752597791027, 0.7596120780300234, 0.5414345394648507, 0.8713574660144487, 0.5805323223239549, 0.378567092795432, 0.311954022852375, 0.7604897653366468, 0.6815070688487581]], [[0.18151989918269637, 0.3988675687170612, 0.3502209913893714, 1.9689109636030888, 0.7437462344555992, 0.9378319542790889, 0.3083759653653205, 1.254052174792826, 0.8539924692826266, 0.7056920536472864, 0.6181776231285329, 1.0112791821284117, 0.8002655440283821, 0.6461015744023362, 0.6330898946657377, 0.34300347731226943, 0.408272538719996, 0.514886383456194, 0.9950205445101168, 0.15257534616236568, 0.20222326937052199, 0.13742055281451965, 0.01389894718269101, 0.4538775019793153, 1.0452454477941358, 0.4980597177411894, 0.7807231663057618, 0.7934561791274568, 0.6400786860826181, 0.8080139542667264, 0.6265387692108798, 0.4020232449950435, 0.31572857044842195, 0.8685128566901112, 0.6915692639567759]], [[0.18144057018189994, 0.39424088880481134, 0.3558010174803955, 1.8740656178946034, 0.6999685901182294, 0.9274205096436408, 0.2830857632810674, 1.1708266083029129, 0.7617364502928148, 0.681918169368118, 0.5367016999986984, 0.9650848511486175, 0.8048485243601531, 0.6851297229752287, 0.6256749893964779, 0.3390629392514184, 0.4289408577716092, 0.5179166508231883, 1.0026720865350305, 0.1106105436918079, 0.18385510958706636, 0.13392445831304048, 0.040395542726211854, 0.48739008864673483, 1.1621878023991623, 0.5050758870350098, 0.8454373413932323, 0.7757408476565952, 0.6173456346463859, 0.5577316116335906, 0.5691880501358219, 0.418431522752557, 0.33958913311588906, 0.82732429506053, 0.4860240805409305]], [[0.18148455744403189, 0.4040400780221445, 0.36447381646336086, 1.8442534706886655, 0.6615110289319646, 0.8966576064794349, 0.26890743844135934, 1.0642391988159092, 0.681075806045923, 0.6117655498576845, 0.45152795575023735, 0.9246848134737572, 0.9244215870543218, 0.6715290092943459, 0.5706005140297918, 0.35380688388599624, 0.42370549134014523, 0.5196494668176728, 0.9914644232935328, 0.12116686828370882, 0.1741325576007618, 0.13785481744784586, 0.03634549489194128, 0.5298844972945146, 1.2816722072134692, 0.49963457252851673, 0.8557949253803528, 0.7704984328865419, 0.6391183918214409, 0.4474721485455597, 0.38768297338769975, 0.38548491267209706, 0.36714868661289846, 0.8326895115941593, 0.48190225775437345]], [[0.19151935305806891, 0.36981934914372105, 0.40509273936426116, 1.7527103654339904, 0.5978773095590709, 0.8307075490565246, 0.21046833382458263, 1.0595647661253835, 0.6371262512803736, 0.6091082530917106, 0.3031894383764636, 0.9571139703088267, 0.9194326355706415, 0.6736672030030615, 0.6605659331122689, 0.36668224637069835, 0.46891334145930763, 0.524484573692227, 0.9957907370728437, 0.1101356261725185, 0.18972586444153688, 0.15386418315196543, 0.03261775918235591, 0.5873072305653887, 1.390438658970043, 0.48422905933582383, 0.8055722977492369, 0.7417950533443711, 0.6171745922925859, 0.48857535823143683, 0.272490671133045, 0.3230043763130107, 0.4163181981567686, 0.8352871385232468, 0.5179201979397577]], [[0.1929576698880164, 0.3726382815119368, 0.3948638782011352, 1.675797897845091, 0.6200385929466593, 0.7605120659764157, 0.1782885384456812, 1.0783069206655846, 0.575236918506795, 0.6462602243957671, 0.3250175369339983, 0.9524805128899669, 0.9227276676359009, 0.6059390079834799, 0.5774417324950045, 0.3795868739072848, 0.4937006610363984, 0.5421593256506397, 1.0683895797966623, 0.10481809822216621, 0.15558822173457942, 0.16113458060777372, 0.020366158970432712, 0.6383245039641438, 1.4288692139148844, 0.4988923074153857, 0.7777248681347555, 0.766561930218133, 0.6009706594000374, 0.49136860389279513, 0.1946472169218253, 0.3393625297297439, 0.5704722933133578, 0.8031334240475105, 0.5613314888725752]], [[0.18877491457411186, 0.3960818474401375, 0.36057344182654716, 1.5349162699313321, 0.41473917936858434, 0.6862805154695656, 0.16291874419173846, 1.085497419783303, 0.5969818793374393, 0.6488536568431607, 0.22032963230439218, 0.9482842967974499, 0.9346436293122922, 0.5875729451758083, 0.6219376843210518, 0.3870308933950553, 0.5435522461959624, 0.5331133088052684, 1.068373228737528, 0.1149572554672676, 0.13014128357121332, 0.20366560225539804, 0.029640271246766138, 0.659760881432088, 1.3940784555096855, 0.5151555020334617, 0.8265940364342799, 0.7634052625104089, 0.5220612176466689, 0.5090832840160692, 0.2189451971881327, 0.38414399619267126, 0.6617279381068308, 0.691353535896951, 0.6293761912102055]], [[0.18874210178623863, 0.37979638423732165, 0.3708452981551692, 1.5081161322997862, 0.4059387098108367, 0.6315553124549317, 0.18660088461093918, 1.0713481855991978, 0.6579259208432333, 0.6399052122235076, 0.24680302860322112, 0.9094053862090501, 0.9568866349852556, 0.5978729220158965, 0.643056262132502, 0.5116990993611452, 0.5001070119220385, 0.5661546351130644, 1.0611775619664434, 0.14431102602341228, 0.12713778649136057, 0.27338711400616933, 0.04670140162430741, 0.5969122989494833, 1.352193200184249, 0.46958988401323953, 0.8866644663779075, 0.6492907500266158, 0.5590874865388697, 0.5112047023499579, 0.2518701257563958, 0.46357708044181434, 0.6741208393095883, 0.5356167608939179, 0.7329727834589296]], [[0.20835165297938937, 0.37954082737246686, 0.40631305586252825, 1.3518250896407626, 0.4220403563894556, 0.6296451673920568, 0.22781814594457359, 1.0882994561831534, 0.7006397836127384, 0.714059706767765, 0.15299811661393592, 0.5377404627370386, 1.2717608145414967, 0.6263037292712239, 0.4973396183928841, 0.5603410097787944, 0.48864096937762924, 0.6902251171254307, 1.162767292362528, 0.1635542994897375, 0.1487986713393738, 0.2828758871465375, 0.10378572390855896, 0.5420548706672167, 1.2697602237719703, 0.4563220983752829, 0.9235058106757051, 0.587817183810235, 0.5874179670455129, 0.506932164044856, 0.2953535818728448, 0.49860829046348953, 0.6972671330426268, 0.36870065988574185, 0.7515627252302216]], [[0.22726496497144377, 0.38615358572150416, 0.39444457327864496, 1.1639005627052814, 0.44420997797660794, 0.6079681048623098, 0.19415291531779394, 1.106123619789382, 0.6719090597393392, 0.7140297605228508, 0.1463035878313236, 0.5317650189928871, 1.1393726536288606, 0.6290247078546097, 0.5486566537439151, 0.594611338372762, 0.47913282275370694, 0.675239990416034, 1.1370465012535709, 0.15448764789095726, 0.16964755027465453, 0.21482373441037161, 0.11860500222011623, 0.5369501273600775, 1.1307097872739111, 0.4091731019092083, 0.8814739397463003, 0.5780204197906593, 0.5886641842565279, 0.524925150508005, 0.33223962278243946, 0.5455754215585724, 0.7001103576819481, 0.31299649518616546, 0.752456050909223]], [[0.23431650067322307, 0.3466849143291031, 0.4214866165217268, 0.9540476082482277, 0.46329345971914015, 0.672452386908516, 0.2579437632247299, 1.0914852612772696, 0.7303709139176192, 0.6935077840617909, 0.11438036919649491, 0.5302040028031948, 1.0409000001232354, 0.664302129469186, 0.5033719717559536, 0.5960242049390387, 0.48417675322786596, 0.6783561677533831, 1.0282619316633017, 0.13729906216712934, 0.16308978461076354, 0.18367088002367726, 0.1263377291282143, 0.6514909967086546, 1.1383712297777362, 0.3811377898554603, 0.8943352465357314, 0.5651246804611991, 0.5961890265583567, 0.5380197595307096, 0.3421199294418702, 0.5462260218557107, 0.6813377946185294, 0.31047700469561734, 0.7411531336653203]], [[0.25481803013898985, 0.3404373871986311, 0.4253777552904954, 0.7548746927429824, 0.49320272020730827, 0.61487204465796, 0.30182458761191466, 1.0071281993466417, 0.7083467635094901, 0.6255884539266913, 0.10464715529007373, 0.5421939307296105, 0.9497850778919248, 0.6586555669360581, 0.5186435946162538, 0.5457182617716734, 0.494827785132387, 0.7509187235383441, 1.0275296923103159, 0.12838567266938564, 0.18311353573689876, 0.1423567891630667, 0.11109931910928914, 0.6981833601766221, 1.292128081487025, 0.3576297034280216, 0.9595614890584687, 0.5578249084662046, 0.595635157475537, 0.5430159823584271, 0.37440601333999013, 0.4686265341020622, 0.6829015075527656, 0.41444932077918784, 0.640859499350885]], [[0.19260144806055454, 0.3365780902413902, 0.4417114452166011, 0.7404374190978418, 0.5026782848078757, 0.6364802884071398, 0.37780898496756493, 0.9434461089222199, 0.7114014825970101, 0.5130668620549511, 0.0831506576149203, 0.5978566742400978, 0.7385022854343033, 0.604760558534239, 0.3471358729037715, 0.365792371426665, 0.48676542169468495, 0.8027374579856681, 1.019837998579663, 0.1284802160877517, 0.19894714301180544, 0.13222748772606652, 0.09810531349156026, 0.7342113671124768, 1.3740336235742154, 0.32544349959520374, 0.9538728140790004, 0.5687827633197053, 0.5419458872268554, 0.5429768922020433, 0.37097855023313675, 0.46567525972321583, 0.5983914989712806, 0.462683355245393, 0.565580592624196]], [[0.1996234965426736, 0.34352144364566417, 0.41658600021107495, 0.7483234069367617, 0.4994994742587181, 0.6384200941472509, 0.4068272627784051, 0.7904302078417016, 0.6957263814609898, 0.4871930889919426, 0.07429962619393832, 0.5518356332670779, 0.645389740917753, 0.5606363595306026, 0.46946254819760386, 0.3343673025945606, 0.493450601641201, 0.7760136404681355, 0.9199051902416504, 0.15539497695227003, 0.1887260324550367, 0.10798864063238126, 0.1080789766341844, 0.7107601413173377, 1.425368883157994, 0.3076399589613374, 0.9061696173025181, 0.571323771296672, 0.5201989438749965, 0.5414917397746283, 0.34430559123128957, 0.45915788734975027, 0.5617702736763516, 0.48539645070346954, 0.4739512409182838]], [[0.20470960357330945, 0.3437092624806998, 0.4279863291253913, 0.7783068445528607, 0.5085279344253375, 0.6068272185703985, 0.47462799252178156, 0.7952673054275615, 0.6348766423135669, 0.45577771832105585, 0.090947531504214, 0.5612428509559202, 0.639487234925873, 0.5798013286822742, 0.4624716608211434, 0.3468960462505468, 0.48399025022573416, 0.8013132602194705, 0.8329451497094197, 0.16112891367121035, 0.2059549882883828, 0.11059661013226613, 0.09939351218958858, 0.6605878340360479, 1.4715457475743585, 0.29784512304231175, 0.8226924068158098, 0.5513017359950714, 0.4999979816547134, 0.5241311668984796, 0.3549651732190847, 0.48078990017582746, 0.5536912960582016, 0.5183373298803092, 0.3925998384578414]], [[0.21935918687300568, 0.3634662428003377, 0.416304179860903, 0.7688813233333494, 0.6533811591763117, 0.6155295051105404, 0.48622776796084033, 0.8469951898296798, 0.4451041136197774, 0.38220506889262407, 0.12047507992389413, 0.548006727495067, 0.6198396225124017, 0.5973675127252254, 0.45057976585752463, 0.3611963131507414, 0.5065896395262374, 0.826787401676196, 0.7593440357040322, 0.17145253412079875, 0.21169188884589363, 0.10804424587391787, 0.10005965479026233, 0.6043938890360274, 1.505867980016121, 0.23413146847471922, 0.7433367897129101, 0.4913768571540389, 0.4856102604595609, 0.4578106790350961, 0.3826714129646295, 0.5630658579633467, 0.5872396458407529, 0.6737908768319407, 0.3084089019402174]], [[0.2373639162704074, 0.3768528847763677, 0.4150811559369079, 0.9254935573883778, 0.5084328735485066, 0.6763441550942407, 0.5131457620369811, 0.8563993008021455, 0.39289311804807037, 0.4912934200031728, 0.14434481423064785, 0.61078652089986, 0.5271926255049135, 0.6138400161631434, 0.4216921226824096, 0.35987390963715105, 0.5648104137578213, 0.8225491290585936, 0.713602307632957, 0.21270740130449722, 0.2017054293036627, 0.11079174477181647, 0.10372844008829807, 0.5994926219183704, 1.4850654436160597, 0.22934387265634693, 0.7708601474690249, 0.48715930175838473, 0.4487986327757286, 0.4535666683690698, 0.40268023344885784, 0.5884468262338709, 0.5777533698372433, 0.738528634689789, 0.3422309472510596]], [[0.24059564151789356, 0.37381735929897325, 0.3934664876179988, 0.9775667213472997, 0.44777193138137567, 0.6734104287157736, 0.5382212898796517, 0.8247574916813065, 0.5254506930531705, 0.6394411594382661, 0.17393087345127123, 0.6053930109235348, 0.5143564417929022, 0.6430158460183539, 0.4539652732438994, 0.3469410528033584, 0.5859097965196992, 0.805596097889361, 0.7127014584731532, 0.22595751624056482, 0.21698950437887898, 0.11251624370076539, 0.1498594364661379, 0.6405051840129782, 1.3603121635162907, 0.31931225185358564, 0.7909966948643695, 0.5550232008717972, 0.44367656818742135, 0.447969400700238, 0.4673166029529423, 0.6537217421778598, 0.5845120450669298, 0.7382752273967053, 0.3788379703319511]], [[0.24780757784169857, 0.4131510797316005, 0.4076366352509026, 1.0176742561903978, 0.4168413540011333, 0.6786957169504171, 0.5052469393900036, 0.8003974616101048, 0.6771427617507405, 0.7208250176921078, 0.1620133215200925, 0.6435169371601284, 0.5546801854543495, 0.5853533891244891, 0.4404999878412303, 0.35355945015555956, 0.5527992845923067, 0.7353362059756541, 0.7100941865001846, 0.206707965325193, 0.22380220515820964, 0.13603073225675466, 0.15393322304242796, 0.6506466012441077, 1.2469525825023258, 0.5440170455196085, 0.8042713836510833, 0.6685203835259101, 0.6914514943671383, 0.4171905625329979, 0.5332040950709351, 0.6334804494119979, 0.585433708891153, 0.7437640904829064, 0.44939924148742705]], [[0.25261857699217305, 0.48422605737051483, 0.4213814484423537, 0.9503440617434585, 0.41653976894676614, 0.7268435365265085, 0.5923978264229307, 0.7639523058287073, 0.7477925418512233, 0.8497539075168845, 0.14810175891119043, 0.6206263061550213, 0.586068450555397, 0.6041067954071746, 0.3241975122150977, 0.4058658427701355, 0.5055052714314255, 0.7350489454098099, 0.6048614355570996, 0.1849733241013988, 0.22252267994378494, 0.15165261271022334, 0.1020986543002201, 0.6458870141465352, 1.1656282538961773, 0.6430546518714666, 0.8702747019055275, 0.7178545201089832, 0.8797466553190414, 0.3939006557409428, 0.5329192052120671, 0.6306946404533891, 0.5650375723972592, 0.7055153655658017, 0.4670558465379657]], [[0.26702645659491714, 0.5207093796547315, 0.43048113367995, 0.9590974535404356, 0.42040183012534066, 0.7701892906213212, 0.5717629715578059, 0.7314220158135009, 0.785279869855068, 0.9666434298863233, 0.16432450429371265, 0.6751970738571507, 0.6152616594642761, 0.708851738661596, 0.34874031772405073, 0.42449534158882224, 0.5522271203978284, 0.7092614807598562, 0.5947123216259622, 0.16097548198041225, 0.23598732951222914, 0.22079495965743723, 0.10934005207436931, 0.6502772740174165, 1.1415725952736022, 0.7092096417222213, 0.9584872218355822, 0.7552276671181524, 1.1079963435693585, 0.3913953200646187, 0.5255091125718947, 0.6041000494945986, 0.5579285197583355, 0.7061627770489554, 0.5481577027120741]], [[0.28282776742490534, 0.5487475753973152, 0.4530119700101227, 1.0130010027995104, 0.4584409905952399, 0.7914299626004555, 0.5381880982621797, 0.752199485352665, 0.8197496758007893, 0.9951240760871285, 0.16018709429419048, 0.6541453777501578, 0.6280282580616485, 0.6692991389179486, 0.3544198106675201, 0.43334620876553764, 0.5651673825215392, 0.6827843353461424, 0.6086163698118656, 0.14575696389271692, 0.20973297497985113, 0.24846236697159574, 0.1003756683161342, 0.6497537875060166, 1.0860969929199822, 0.7754720965669071, 0.9841963708378938, 0.784583387147122, 1.3334149784966192, 0.3927148785068106, 0.5454169982263712, 0.5331990212373905, 0.6096261771148853, 0.6842544629297815, 0.6110170024077566]], [[0.22171816089409369, 0.6548676499811786, 0.4012459601132674, 0.892597112657582, 0.3159921671205792, 0.8317610045586861, 0.5122829985053038, 0.6925581938151597, 0.868004086416574, 1.0347561031346393, 0.09394491129649703, 0.6653081415287552, 0.6286633501530527, 0.6316570198126347, 0.36964139598021756, 0.4122684897162103, 0.5511574685086835, 0.7754188197203288, 0.6557865492617747, 0.13810071292810114, 0.19619935165638727, 0.23071099955245053, 0.10537090115935745, 0.6605305811432148, 0.9400283665963939, 0.877712241191531, 0.8979323475233668, 0.79639068037345, 1.5320169669798511, 0.3606246145047285, 0.5500758927056579, 0.45811453682115977, 0.6158362210046013, 0.6723832124979794, 0.6829037790842062]], [[0.21447624870737592, 0.7069006086733007, 0.43772002204390803, 0.8347818327731784, 0.32017802583523136, 0.850063675703378, 0.5021429792479415, 0.7825100891596184, 0.9908061269381708, 1.2158222098774747, 0.09328807748182018, 0.6663045273479457, 0.6552827942704502, 0.5874533384723153, 0.38802620838893004, 0.40738561227312453, 0.5192721941038112, 0.7421186966015862, 0.6556792918772925, 0.13428920340626938, 0.20123826253286053, 0.18138571681297005, 0.10399802309741547, 0.6665680868961138, 0.818360190312858, 0.9631617340961809, 0.803483251579003, 0.8125165515901083, 1.6347139674325355, 0.3844424272010492, 0.5525729004376998, 0.4861634575034044, 0.5572760709253237, 0.65482365694538, 0.6997917405204588]], [[0.23407540839793317, 0.6929283235665631, 0.43501350841744846, 0.8118873616651923, 0.34703556082275355, 0.9169782811186317, 0.466400671399264, 1.3033795095426377, 1.4239907702761925, 1.5214646672110033, 0.08201627646442762, 0.652930781917133, 0.6815251372177452, 0.5793929192676602, 0.41402845222540435, 0.35965186160533946, 0.5152387531723006, 0.7879106510273411, 0.6456631222910976, 0.15863860133075636, 0.2136249857221796, 0.14280027233879825, 0.11655513070513968, 0.665910628744605, 0.8005259563030656, 1.0188910567308682, 0.786374079086529, 0.8210039823320866, 1.709649457775383, 0.3700796705525504, 0.5626523462800558, 0.5026362689281537, 0.5591539156171625, 0.6755401404546906, 0.9172502578447335]], [[0.3205151000965544, 0.8429286724646082, 0.3369730424840837, 0.8318550494075874, 0.39693126459216077, 0.9656207921802704, 0.5085641475401735, 1.087697559160237, 1.489530514857509, 1.5983410708386185, 0.08981754081667001, 0.6246453233760465, 0.6756928639007813, 0.5584230677943209, 0.41079427589153, 0.34955401450189105, 0.5257006966822295, 0.7833156581089954, 0.649102575225107, 0.1628735440711102, 0.21927024527876593, 0.13711002972789332, 0.12018309655784892, 0.687459246741396, 0.8180887273379913, 1.059204195158871, 0.7644667483847594, 0.8619804942131294, 1.6864170712724482, 0.3515865338404405, 0.561641839160228, 0.5285851752588352, 0.6126942406307185, 0.6327440206272041, 0.9872937500429733]], [[0.30903106284199977, 0.933436354449879, 0.4039182807421554, 0.8607054089035657, 0.3984603939286563, 0.9477244353877569, 0.5102513845493034, 1.0083987138382777, 1.5133146221055445, 1.4989868426683497, 0.0985515727027391, 0.602174493054558, 0.708400064150124, 0.574543889076454, 0.35003608196053365, 0.3296092589004207, 0.5359072841426273, 0.7150406927779414, 0.6605978443875147, 0.16640384788211113, 0.22026958744809705, 0.15920147087918668, 0.10776089478240926, 0.7476874916948983, 0.8859031042743681, 1.0726029279870075, 0.7896041834208551, 0.7449967420447278, 1.6710053911275329, 0.3469093538204353, 0.534914787366073, 0.5583032186782148, 0.6061785227458202, 0.5505010135889705, 1.0025300335813179]], [[0.334914157837196, 0.8048153149405745, 0.35570798598657877, 0.9626812922877366, 0.39898157337031187, 0.8968896079757901, 0.4496662405381582, 1.009938524713751, 1.4199218862560459, 1.1667570394546725, 0.08043602295991287, 0.6550693314458975, 0.8505543832254059, 0.6428815558391511, 0.3531775554010888, 0.3221009010940483, 0.5640501570353549, 0.7485825705553777, 0.6550642527576629, 0.167653407250011, 0.22540029442798126, 0.20471708436565308, 0.07887258179986906, 0.7408047795188495, 0.9457412246458463, 1.1175024547344659, 1.062941272270092, 0.6760625685234204, 1.6790597213533085, 0.33191305458182624, 0.453483609837633, 0.5705122265054767, 0.5636389949379808, 0.4606417472024012, 1.0657950134699503]], [[0.3018754396268637, 0.7329382948954581, 0.33817876370133465, 0.9445225117635432, 0.4061457967088252, 0.7858124680275824, 0.42204013880706087, 0.7655052238730684, 1.141456375652492, 0.7980011080323907, 0.044139994909469134, 0.5617554713982867, 0.9259577902448818, 0.6132835130375045, 0.38956761710543203, 0.3343248105443937, 0.5923846388001887, 0.7446795494580366, 0.641273791715705, 0.19336759420543537, 0.21413652640240136, 0.2034092372308664, 0.06582548493960824, 0.7332518739123721, 0.9973390155928819, 1.1242494738540434, 1.1117165057025962, 0.6037021025570634, 1.7197262871219778, 0.3738744635641145, 0.4222714370690608, 0.5754713450090936, 0.5558106630065387, 0.40971717267839874, 1.0947393924098159]], [[0.2670832135620964, 0.7841442561570704, 0.3855431234251375, 0.9356453230445754, 0.3972218119642815, 0.7840219237655608, 0.42823676586421616, 0.5208350695504808, 0.985591535869342, 0.701694589005675, 0.034840993788579196, 0.5234978121141017, 0.8598348932830409, 0.5834909101868625, 0.4099139543411479, 0.3252986488827909, 0.6396824009793705, 0.8349561897638256, 0.6208320240279717, 0.28471279803586175, 0.2433347755256531, 0.193822358905536, 0.0918978982243321, 0.7661791582204789, 0.9912617697294354, 1.1229271290845315, 1.1830141594305745, 0.6207805851111698, 1.8045698571637978, 0.47594055363627774, 0.4006761790433251, 0.5860328435877219, 0.5474796025878687, 0.3976513583611567, 1.1847446178429828]], [[0.237997697319503, 0.8127329336052187, 0.38320541459750807, 0.9017437004275333, 0.3817442620778541, 0.7878200036986427, 0.4315264643059824, 0.5197260640962695, 0.9416515616596162, 0.6566811894688848, 0.027582516092839935, 0.48319198838502453, 0.6827950836234581, 0.5318098665858999, 0.3335541734806872, 0.31432525483266976, 0.604666520822084, 0.7904339948470078, 0.6432273625761215, 0.2714149200797952, 0.2365025287937138, 0.18877306583653142, 0.06951410978382286, 0.7565699665850972, 0.9348788334508736, 1.110694937561544, 1.0563200580123335, 0.6171905839815378, 1.8321911927042445, 0.6624873790810044, 0.4369302823606246, 0.6158628514852266, 0.5546330835178639, 0.3627047975868926, 1.2719617378929686]], [[0.24857896500398258, 0.7523294538092864, 0.38376866308992563, 0.9284930406541514, 0.3762308342032783, 0.8237593134066277, 0.4476044405584342, 0.44195291947157433, 0.9090590907746452, 0.7048842000930292, 0.03517228717446575, 0.4446570019772835, 0.6178728613075677, 0.5265627240091686, 0.2732769130119601, 0.30500498134041754, 0.5883013607365456, 0.7742852500937285, 0.6661788852921767, 0.2580616033298506, 0.2036871231552413, 0.19185770618805992, 0.0732432421563317, 0.7610818994675113, 0.882508459177524, 1.117110363594732, 0.9715629486346311, 0.7070022239086841, 1.832638094424719, 1.0576641883287983, 0.5329832085271223, 0.6272219378826072, 0.5435106893046506, 0.3053370389332962, 1.4296068230821812]], [[0.24188644063760723, 0.7779979765879622, 0.4079496688691866, 0.9600321273253769, 0.35572747123022463, 0.8035694141193404, 0.44340562612312384, 0.5279819254833704, 0.9448112095967378, 0.6901830571097284, 0.029575532478224316, 0.4122491464140819, 0.6150030637573197, 0.5091004685004992, 0.27587901632546913, 0.3180255205785021, 0.5689681095238606, 0.7996702455533571, 0.6653770386557681, 0.19009793177815354, 0.20162469967304977, 0.16596897580982173, 0.07888287986171491, 0.7541475182314892, 0.8490844953947342, 1.1231873211739756, 0.9387204311822277, 0.7238294014135007, 1.831336681147729, 1.2181101471732032, 0.7103728002613304, 0.6518646442438687, 0.5517344861727141, 0.3476583112321701, 1.5509059774796963]], [[0.24444737647435966, 0.7233943848821169, 0.41657020937977357, 0.8713757184801673, 0.3527124296009746, 0.742050948354613, 0.40194621984963663, 0.5456394169642677, 0.8984605116363356, 0.6701617476650169, 0.013655291330929218, 0.4063200237183441, 0.6145278497138972, 0.5180880427156211, 0.29073696643659613, 0.29516390032796025, 0.5780531061114942, 0.7555716172718395, 0.6659436176882909, 0.20093029356447184, 0.1440713569379644, 0.14343363407222182, 0.08198277669102771, 0.7266555143929457, 0.7892911851145088, 1.1249274994203553, 0.8798107975260687, 0.6629548890439612, 1.8633243827420722, 1.1520690225220487, 0.8130877436347184, 0.6832761800204366, 0.4163157199746602, 0.3492026047877955, 1.5444059336394944]], [[0.26214917068360954, 0.7605282258657609, 0.424662182204105, 0.8969004990615901, 0.33885399873043276, 0.7099583893256034, 0.39706436627908936, 0.6245298712226799, 0.9082431454873026, 0.656083676603164, 0.04562172446302835, 0.5134891888028293, 0.63994559416428, 0.6133211869252053, 0.30206932094514966, 0.2924521479008017, 0.5656959350914501, 0.758523305922703, 0.6642432858556505, 0.19669346891370787, 0.13741485025846273, 0.1492643809406827, 0.09405772985524447, 0.7660974809959555, 0.7672216025427632, 1.1223832132525116, 0.8160738211047427, 0.5990152994619293, 1.8938124050480023, 1.0285720612826088, 0.7359547310264121, 0.700757764240094, 0.3961666738590466, 0.21645370146252557, 1.4978408598864514]], [[0.2652692421799517, 0.7689417606897899, 0.4281988618212113, 0.9444744504981293, 0.324632884759947, 0.7157672744135033, 0.37714694849256364, 0.7067673232967191, 0.9606042526643449, 0.7319224560303951, 0.06579624369608565, 0.433655751171781, 0.6564201353293182, 0.5903577218029666, 0.2711043176767523, 0.27756865137747155, 0.593469746922848, 0.757089603896468, 0.6757540076290773, 0.19752715995950593, 0.12348432086408243, 0.1702276843332463, 0.07536345250967436, 0.9061289774306044, 0.7044456432035262, 1.116125999705741, 0.7790910527621597, 0.5590091181908154, 1.9253210031575372, 0.9345062959906271, 0.6567158023402888, 0.7116800958587753, 0.4427324365152083, 0.11254220626076335, 1.3409889626979445]], [[0.2375778401141917, 0.7513843004240265, 0.4044457248930533, 0.9773176060969653, 0.29795564089089577, 0.6882752366582623, 0.38153941902073657, 0.8210617227863812, 1.1528204857879139, 0.9069721471073046, 0.09366629187582237, 0.46804883663977603, 0.6370703694335935, 0.5526268216591891, 0.25453076580652373, 0.2639237462716204, 0.624814253499331, 0.7582374164617092, 0.679074559771442, 0.20170899703250922, 0.10546756331360826, 0.1420359067176927, 0.07662382025161199, 1.0220653397139983, 0.7311536579583803, 1.1126603988303851, 0.6809124188470311, 0.6083146893383788, 1.9659641192529294, 0.9196032981450224, 0.6827672634823108, 0.7265159435812334, 0.5371635758705399, 0.13153243141329848, 1.262200040099717]], [[0.23824835902190178, 0.6805353652363536, 0.40233234928721057, 1.0609101925380278, 0.2927724818445564, 0.6646631517207013, 0.36517665302274066, 0.9089770142408392, 1.282986992170551, 1.0779911430681401, 0.07550872409842842, 0.47485405357522004, 0.6681481079039315, 0.5601721177192407, 0.24827367022649166, 0.2729020686928082, 0.6384352631861677, 0.735527598074011, 0.6874078530369537, 0.18251300444054727, 0.1056426222143731, 0.1198240231099074, 0.06848590006613817, 1.1338819564583642, 0.7757680370014324, 1.117537597412025, 0.6733136379898839, 0.665945169221222, 2.000736572386192, 0.8804797247337335, 0.8060722889505458, 0.7456875138621705, 0.619513398470609, 0.2755973436816564, 1.2264321395276703]], [[0.25385417231253965, 0.6234290364700319, 0.3579509323691769, 1.0692591069166593, 0.30230784658588117, 0.6841508875772813, 0.3636694682078259, 0.9746041415737721, 1.265647365323173, 1.008134996092352, 0.049904052587975756, 0.4861622525616051, 0.8244171165092726, 0.5704610137192205, 0.29116384189215255, 0.27833747523489816, 0.6440416842485983, 0.739563076245272, 0.7246517452981049, 0.2087232700259971, 0.11871205125155361, 0.1363503267809063, 0.06524954824058515, 1.1458607883599492, 0.8221630690913497, 1.106477244928757, 0.7034952919664741, 0.7016050839183032, 1.9925854052404806, 0.8136782246650812, 0.7926712885789189, 0.7988303252266479, 0.6846070052403695, 0.3215443178067998, 1.1860957239587169]], [[0.27860463884364806, 0.5183375856197627, 0.2989726404066106, 1.117032925683, 0.3041993533739633, 0.6976393077390243, 0.42350254905374274, 1.087096361246067, 1.231188096952326, 0.8471534816546411, 0.03954269011626355, 0.47875414787268256, 0.9013766938253394, 0.5334475176230931, 0.3217484899599758, 0.2870050424607596, 0.6131801457175632, 0.742836597392659, 0.7322554479894408, 0.22517580706382406, 0.13643576507849886, 0.16487567190666055, 0.07034226799891026, 1.2105169901618877, 0.8416689642157802, 1.0871002043822213, 0.7473447199099253, 0.6666151318429696, 1.9347520413715016, 0.820407132018662, 0.6886382732219349, 0.8420879475078054, 0.7597195420465171, 0.28004545205079223, 0.9344522213325968]], [[0.2731043146391301, 0.5081803531120054, 0.3209544619790014, 1.14936094888587, 0.3044024869972016, 0.7021245785284276, 0.4613629244473795, 1.1529213077809823, 1.133538952039687, 0.6857027821475228, 0.05323491075365484, 0.4792232503060213, 0.9677755040630838, 0.516413290321917, 0.33208450164593584, 0.3298526012364599, 0.6393613989117233, 0.7359998661623484, 0.8140395839944352, 0.1955938442491632, 0.14708001667066853, 0.16118335147483304, 0.06483008762549511, 1.2225481533782667, 0.9077016637900888, 1.0655100979720475, 0.7497108423581968, 0.6067877613433171, 1.8084860885838292, 0.8261532355495237, 0.6802234238540465, 0.856645198987264, 0.8335191281634711, 0.2717667051778806, 0.7748094312672635]], [[0.39673420691765726, 0.494914920488906, 0.42089227018035097, 1.2063590297785245, 0.30523932267759313, 0.7042013565891242, 0.4940338760540445, 1.191870239187922, 1.0742673714534183, 0.6159837356328269, 0.06152996706187548, 0.530294066791127, 0.9722993736893021, 0.5324444086564182, 0.32520349674736376, 0.3620211385979624, 0.5962077071880458, 0.7432697834950289, 0.8069809558733603, 0.27314698522214165, 0.13741350650402065, 0.30847959284552773, 0.0587363612011873, 1.2065480003644748, 0.9785744319249584, 1.051542598332841, 0.6732449415628005, 0.5983975916840436, 1.6823926678055727, 0.8218018812255521, 0.6891314627395795, 0.8356996122993953, 0.9094949506880987, 0.36574296205384965, 0.7707533331174241]], [[0.36459163381727144, 0.5146434192880849, 0.4234517348994574, 1.236944710053158, 0.30611166779496046, 0.6559849046353213, 0.5051699843425002, 1.2205643382242921, 1.0217719722784646, 0.6230279095018073, 0.07259783463017505, 0.5004457446831488, 0.9425313976289789, 0.566026629596283, 0.33724649657484324, 0.3193869633666771, 0.6754028715578604, 0.7407796003911922, 0.766888299112912, 0.3678505894267501, 0.13714283118193513, 0.2719749113705681, 0.07290096383164649, 1.189003246628655, 1.0104932573748509, 1.0256749614717977, 0.6046622222311773, 0.6637063762841047, 1.6162570823641862, 0.7982820315123613, 0.7205858773540528, 0.784818100753154, 1.0024962640294368, 0.38338314496480697, 0.7416166925117166]], [[0.34829308223797106, 0.5209362168203534, 0.3645546961123082, 1.2702193940760549, 0.30709692541994393, 0.6636090771829353, 0.5160078598643483, 1.2155063102925532, 0.991212010081322, 0.6942883152356072, 0.0633051726487389, 0.5343211581140059, 0.8805901325275269, 0.5875419842805925, 0.2868510594558485, 0.34078933866422806, 0.667306097872981, 0.8004518109804358, 0.8349114094902741, 0.41331365054764946, 0.2396945972076189, 0.27207135960709994, 0.04020704461090545, 1.1618534205946225, 1.0124309256075046, 1.0008080108918802, 0.5129602229349128, 0.730278355089386, 1.5783413616677906, 0.751461989758883, 0.7161212953489468, 0.7228898270072979, 1.0649508294101149, 0.3775007905658231, 0.7122184007316003]], [[0.31790439685449495, 0.47594790035148893, 0.30462920507895913, 1.3877982192446243, 0.3046934430269598, 0.6463301898071581, 0.5480266652543065, 1.2029393594205253, 1.459157781574594, 0.7190297987205577, 0.05412489007401275, 0.533338506122994, 0.8784848596183046, 0.6630542282794822, 0.27949497464448325, 0.3522930106654868, 0.7267370291780455, 0.7397156993656178, 0.7989546227056673, 0.43337881348925966, 0.258894361555886, 0.19882955917719378, 0.05319669694433998, 1.1833438468970743, 1.011774989583114, 0.9707701454031373, 0.49984723576975976, 0.7262791636574609, 1.592712058752661, 0.72289236399097, 0.6955802592351253, 0.629164138047251, 1.0583090357928653, 0.36117617638088295, 0.7078454580678605]], [[0.3652873735333107, 0.42602037666044684, 0.29198704588974983, 1.3439873809300398, 0.3043531275600877, 0.6384235200834654, 0.5791049296461617, 1.1799630297113797, 1.4291909509419174, 0.7598540829981986, 0.051852680146714625, 0.5318611086316558, 0.8134242392729163, 0.6711092947689676, 0.3132089719173809, 0.3526016935333023, 0.7122890042944988, 0.7897379092653869, 0.7616793849865877, 0.45022166538595043, 0.2760924809521152, 0.14092436581866472, 0.04943410160310227, 1.0783878985362298, 1.030394940566716, 0.9396986833765943, 0.5367898059130041, 0.7257664770575802, 1.731201131710328, 0.7337176672238742, 0.6647046162432529, 0.5178819855625061, 1.0871901125225911, 0.3136011188874286, 0.7097658498813445]], [[0.4451936550477217, 0.4137625030048409, 0.2906272060057177, 1.2207380166436255, 0.30173612951446027, 0.564173825338531, 0.5781392150186195, 1.1631892209639412, 1.4357482830839121, 0.7939026492143031, 0.0415309777364478, 0.48806171456073677, 0.7992522782823086, 0.6840912028814508, 0.4069426552849356, 0.35364115755071457, 0.6589939227114556, 0.866131594258857, 0.7503174782920483, 0.4428091028747148, 0.2855634873935431, 0.14954449952142393, 0.05952221053776241, 1.034198817039155, 1.032928236678158, 1.0039876338804616, 0.6014658759884584, 0.6798154328692235, 1.8923200720549636, 0.7599259443776083, 0.6734727102180671, 0.45463776349973695, 1.0792299400348868, 0.2565894315251943, 0.7348619130250285]], [[0.4873227174381414, 0.3653384375912877, 0.24965945450398203, 1.2603959677735213, 0.28972756767071883, 0.6376054228690804, 0.6761913126677255, 1.1325110017657525, 1.4128676808599416, 0.7591168393546611, 0.04348683015991664, 0.5049592246634445, 0.805668940308524, 0.6804809746473957, 0.3584482333821252, 0.35332598653344915, 0.6897519410771816, 0.8840944538389724, 0.808059993826508, 0.5240930125821237, 0.34876053894160847, 0.1806523102413691, 0.06794918822474194, 0.9999450532846607, 1.0586285424197892, 1.117627052326885, 0.5631944442801868, 0.657881307627265, 1.989612074287923, 0.7964833870685273, 0.6880996582908292, 0.4122647486188731, 1.0471966598141746, 0.20256788224798716, 0.794382896511928]], [[0.5894213259605945, 0.3455845220956517, 0.24449174739933194, 1.1419053898855647, 0.26804867535036436, 0.5759664924754376, 0.6893001022379872, 1.1048520274904157, 1.4266226879108013, 0.7390784421839159, 0.04414250869714578, 0.5561293183008783, 0.7480769835955724, 0.7083155407854638, 0.4567805964526404, 0.38847388817635453, 0.6641276862121562, 0.8186479839754104, 1.2926709507656478, 0.60217631013451, 0.37642544814128776, 0.450660741977642, 0.049681507289643975, 1.0023914217923735, 1.0536031277031972, 1.2540493338865266, 0.550592594280642, 0.703912487775678, 2.030625606803352, 0.8150971615441698, 0.6738632845275724, 0.4228594596857116, 0.9488030614921328, 0.19267765740814746, 0.7763967266839029]], [[0.6398326047809637, 0.32878920020269625, 0.25216374302956657, 1.1498385121392356, 0.2413700139694439, 0.6395821185245368, 0.7200523819359306, 1.0682195846396603, 1.472179745425247, 0.7908340646429831, 0.04947938127463259, 0.5607381626151011, 0.7107980509314689, 0.7094357949747243, 0.5724009869124718, 0.37266512517438344, 0.6906868393799557, 0.7791622924472383, 1.310520715074985, 0.5988601379772852, 0.44765017664102547, 0.6397665884058119, 0.06425888598241832, 0.9612711373040849, 1.0418164894745958, 1.3435430988203714, 0.5395358207964582, 0.7404813146611962, 1.999706504966096, 0.8300448297574998, 0.6418843128356944, 0.4411317339003794, 0.9308579463181121, 0.17383472741925124, 0.7657812050911725]], [[0.6577429119563851, 0.377878212368734, 0.23055122292585234, 1.1109525759491563, 0.22886778993533163, 0.6717748841111169, 0.7707631375882877, 1.0058832825160913, 1.4641929428993397, 0.8207033886929678, 0.07114516987112764, 0.5241628692228153, 0.7651154004297915, 0.7449086540392688, 0.5398547031591341, 0.37257155078111515, 0.6276161110787595, 0.7745336722716054, 1.2431165281045933, 0.4614929739472371, 0.4737400943683395, 0.815398244318046, 0.037046094063822185, 1.0319064304162973, 1.0254069674872568, 1.3839703062898132, 0.5242610455261634, 0.7806145147041653, 1.9191279975375093, 0.8606574714123533, 0.6270605261935538, 0.4602120487070006, 0.8999438653272722, 0.1987232127824734, 0.8232598329823192]], [[0.73522043515466, 0.35680234282496476, 0.2176376558568507, 1.1012342875553585, 0.2218519872230872, 0.697062144317548, 0.7605804124090256, 0.9566078239081743, 1.4802893154902517, 0.8580722337205724, 0.09889908136827716, 0.5619702885941817, 0.7636847784894474, 0.7656575689615077, 0.5504347213951459, 0.3918602539185899, 0.677519512305794, 0.9386021490126488, 1.2986692339143426, 0.20019742626154863, 0.40141742998642954, 0.9039166517175339, 0.03551556899088585, 1.147689040346509, 1.0410272915402436, 1.39810469713706, 0.47044006477197153, 0.7005306458876439, 1.8128341850719292, 0.9297766208344983, 0.5875250441101101, 0.44578796471279947, 0.8575672380522634, 0.3050338538925974, 0.8641306438674932]], [[0.862670559916195, 0.49195061851804, 0.29336283961941007, 1.056440447522404, 0.21556676583397938, 0.660196920029537, 0.8094263170626166, 0.9436421325536046, 1.5138933144990614, 0.895324832766951, 0.11957127163253733, 0.5526819319849399, 0.8543462186584934, 0.7546666054287162, 0.5857298540168255, 0.4071827119549406, 0.7013420768020858, 0.8345344530509913, 1.3783237631688823, 0.15746611399522903, 0.3435975682242248, 1.0031688921855724, 0.03476912721050289, 1.012751896556962, 1.0601505706504735, 1.319985459310077, 0.472648046594772, 0.700068363247395, 1.6790338072664182, 0.8571375320726193, 0.607192258023571, 0.4253768420730909, 0.8101375061884513, 0.3739497455952741, 0.8584785321865023]], [[0.8935869637156343, 0.5243411422066877, 0.29731213601560047, 1.072509554796879, 0.3296355518877351, 0.6995052509998123, 0.7199240464913353, 0.884512997714901, 1.4350749315999258, 0.9017378958690992, 0.1333795031483729, 0.6362659513978703, 0.7798436441142356, 0.7805098822241658, 0.5795989095420457, 0.42970921444081556, 0.7331570710327941, 0.8608148936272304, 1.3731641195543394, 0.14945025118772176, 0.31209347455317493, 1.0969704946784753, 0.04212089268670169, 0.8541797595371861, 1.0523068906319974, 1.2913309274050575, 0.48698758335670733, 0.6711551565561064, 1.5435105263992135, 0.8483840516184361, 0.691534576914897, 0.40993019381781237, 0.8107735580456098, 0.5630520411470721, 0.8552624619556635]], [[1.0068945280932313, 0.5587820775107605, 0.2839042677520003, 1.0200210260758813, 0.3526649842679069, 0.6865190500108329, 0.6894185785352691, 0.8608966883198087, 1.4366549824529469, 0.8669904919532883, 0.154396034905428, 0.6067490832397694, 0.7409849849803445, 0.7685261913052506, 0.5768867444802308, 0.454362218050832, 0.7731579508829127, 0.920611930886876, 1.399074690174835, 0.19497069014899276, 0.322379362836445, 1.0422275451140517, 0.04472802812864468, 0.8562957584469129, 1.018851286789186, 1.2578403520650274, 0.4536335577518261, 0.6788673243923942, 1.452970390282044, 0.8701001919557609, 0.6950104261194364, 0.40195192406762015, 0.802821954786126, 0.5968776029603791, 0.8538398003680351]], [[1.2797614940303825, 0.5724197220965717, 0.27813601619241685, 0.8766732167847681, 0.39817229396249665, 0.7234859630271593, 0.6746283557340956, 0.8023249470123253, 1.4492929395950718, 0.8469275732554377, 0.16112159596318365, 0.6037661259726513, 0.673377691508956, 0.7792331013168048, 0.5568620770203754, 0.4264358366154817, 0.8489438795435882, 0.9148148329192161, 1.3031899426847282, 0.203965730434219, 0.2763944710572571, 0.6797450479035829, 0.04146539069796343, 0.8300627239086459, 0.9812976352020083, 1.223823386346952, 0.5468309736880042, 0.7463870686874257, 1.401736790530694, 0.8601409023097472, 0.6707418436063293, 0.4032467793645679, 0.7767654704367145, 0.4981961702103996, 0.878529921906162]], [[1.2906616561261415, 0.577609324423396, 0.2680188737631394, 0.9168762354824191, 0.4681945323582188, 0.7868592288646978, 0.69084322610753, 0.7491658559998906, 1.480880365921011, 0.8597154883567906, 0.18492327219738672, 0.6025231646748168, 0.644092755433904, 0.7787436873903648, 0.5891755493258303, 0.4300455773902041, 0.8394252825065399, 0.7695604205110027, 1.1695140085117435, 0.20533119532457744, 0.28244037796365556, 0.4109679533769047, 0.05264862045637772, 0.8071458694660003, 0.9098769022584692, 1.1664091924153226, 0.6154677530952675, 0.7263673943576026, 1.3426621875702918, 0.8809173079874325, 0.6942490686297348, 0.3817242167675968, 0.7646821771057264, 0.42738342434649174, 0.9235139518212049]], [[1.4966710754737602, 0.5618615754473794, 0.25898860412169156, 0.8605524642586129, 0.5777928133149379, 0.7606679722080462, 0.6761412956638373, 0.5985210946058479, 1.4504819752668163, 0.8342949047329209, 0.18962007208794912, 0.585904676888557, 0.6168814938296846, 0.7940645165271487, 0.5800627862585783, 0.428375202396, 0.8630149523320713, 0.7701589282603597, 1.186105343661857, 0.2508626059711076, 0.35903175920157143, 0.3780178282885637, 0.028674530340453003, 0.8247291413968381, 0.8343663641730725, 1.06871838401203, 0.5859746277820197, 0.6774956855427383, 1.2305765646763813, 0.9192136344282036, 0.6949836587310617, 0.33707957190129545, 0.6720722601077993, 0.3910374266272815, 0.8924384788450357]], [[1.512814438809326, 0.564995815991157, 0.25736132207948065, 0.740197984187388, 0.6715769591339834, 0.8054455128661024, 0.6712315461648446, 0.39582349460178967, 1.3592426383312999, 0.8410163073140194, 0.19281268492328058, 0.5710613260886455, 0.6421919185446411, 0.7717353908902245, 0.6018609066362988, 0.4395685483217724, 0.8931819339831522, 0.7555230477679342, 1.2644548487253138, 0.33828206025424523, 0.34822674455172653, 0.3632232261261017, 0.03985444794694266, 0.8038071307193775, 0.7061734123508773, 0.9446136797390965, 0.587860964963516, 0.6659040478771572, 1.012765705303088, 0.9250756228016082, 0.7391306114051505, 0.24929908907823278, 0.5200384069043174, 0.35915362714536786, 0.8257545049376516]], [[1.40185861086372, 0.6055094312289839, 0.2468003699846828, 0.7231270350201627, 0.771760732517043, 0.8008921344424964, 0.659152717503685, 0.4195945972278635, 1.2152474530537398, 0.6122058620220292, 0.2008140703031533, 0.4959582886527938, 0.5952966309647734, 0.8150470556749077, 0.5788683458035478, 0.33363277837918126, 0.8338114618563032, 0.5856858786626172, 1.355481863835432, 0.4602068843594558, 0.38679733006603423, 0.3713977302270983, 0.04605373728722105, 0.8242646251738336, 0.5513279128266778, 0.7947353174047378, 0.5814448815073664, 0.5753318428009189, 0.7901297610448625, 0.8684756174201748, 0.7282352029614609, 0.42803172083237295, 0.41289026390000727, 0.3783457042943832, 0.7593086592401703]], [[1.5254264627089236, 0.6787884545107725, 0.23779512507147177, 0.8044169884816038, 0.7532324380788218, 0.7805402660833909, 0.6393929914228311, 0.4431577064631616, 1.0703269569556355, 0.45542942485905313, 0.15879323466114673, 0.5277113904711623, 0.5635630429908134, 0.891552243341218, 0.533475669229871, 0.35716390103450135, 0.8605051199193093, 0.7239416411781872, 1.1552670320767446, 0.48550946332277056, 0.3917934906731103, 0.43694505474356227, 0.04529400622267489, 0.788168203596038, 0.4130568404944907, 0.5379118287819062, 0.5481586273316026, 0.47656044662627567, 0.6210579881976066, 0.9700981092568322, 0.7607122137579958, 0.6721705766580015, 0.35024673579408694, 0.4605842340822317, 0.7036487682968633]], [[1.518184368422419, 0.7084038520064732, 0.2268025139778888, 0.8733246850010863, 0.7906574699186162, 0.7567792187622618, 0.6202392021883472, 0.5536626295204977, 1.020773287529153, 0.3510058700383769, 0.16275998696261984, 0.5251121579589701, 0.5558562378212206, 0.8815741887452404, 0.5131171879283771, 0.38164614121791185, 0.8905376787458656, 0.8757771000223503, 1.1241069211659727, 0.5017171407898654, 0.4880808268705812, 0.4788108158955847, 0.04024015611539306, 0.8130517461379551, 0.44218691935843, 0.41972379715109165, 0.6875978309218214, 0.30523467003737104, 0.5183710074534695, 1.1314820180334069, 0.7527390093985915, 0.6775787414466576, 0.33147439928417244, 0.4926517985937824, 0.7644666823009777]], [[1.4965777304794141, 0.8829865871565739, 0.2362010659441311, 0.9918615111177108, 0.813753540657062, 0.7330683054699624, 0.5695264872253409, 0.7226397353975466, 0.9471381194286947, 0.2871994759301899, 0.22465004913965386, 0.5359446465669273, 0.5600847648457778, 0.8856694871308156, 0.49649980788312087, 0.4003238835225954, 0.8828120263192223, 0.8729341715248129, 1.1813463364203776, 0.5130930097492145, 0.4976946516430848, 0.5553660135717337, 0.04273150222454357, 0.8724417357062927, 0.39860870043938085, 0.4378704108026654, 0.5985749790151772, 0.2262820529261392, 0.506089102202364, 1.3172360018346554, 0.7527232485173985, 0.7033810534938854, 0.3155040566771987, 0.6212561850095708, 0.8127612726753359]], [[1.5286428166603112, 0.9405661945737634, 0.2416251990041059, 0.9754477535062014, 1.0938270581042597, 0.7078974085591174, 0.574679081794397, 0.8547532066846909, 0.9283977800225062, 0.27476506215231755, 0.23025729004879297, 0.5382268809429809, 0.658749914604996, 0.8581461999028839, 0.5724661369986273, 0.4326488185659014, 0.8682282078176708, 0.9104033761714871, 1.236596946392769, 0.5358409915811356, 0.49259734177073744, 0.5686958527319999, 0.04032649712743784, 0.8870976796801279, 0.37399394102404004, 0.4752921849663343, 0.5041131869417954, 0.22713844660972668, 0.5127419141924517, 1.344973338021317, 0.8509552101513655, 0.6780973996374065, 0.36877577162084574, 0.7694407600150155, 0.9258905597302043]], [[1.4062125426134116, 0.9925514985608971, 0.24113197222037142, 0.920150110822417, 1.1438792935707294, 0.69564855175161, 0.4802474944310868, 0.9252405470552237, 0.8838181609990525, 0.266833127204369, 0.3190744884105601, 0.47737966870829684, 0.844561869402418, 0.6482750779260746, 0.5194486750106635, 0.43444538232464236, 0.8614015512377662, 0.86471542587493, 1.1867667898004859, 0.5279465381073547, 0.47047775741405934, 0.6082811601775419, 0.02620017453414614, 0.8488629469892479, 0.3197675761953437, 0.48818029218510517, 0.45588815164001245, 0.22238491247689754, 0.6052944663074308, 1.330811547910736, 0.8196541689890253, 0.5900205629034909, 0.42506518495383716, 0.9833372565732919, 1.0473456480876155]], [[1.3119432722637556, 1.0329289506488384, 0.24009239273332084, 0.9158034400562296, 1.2084851581484992, 0.6875340584535649, 0.42339542758039683, 0.9699734717652408, 0.8611022256752572, 0.2658138438630788, 0.3249391619681755, 0.9745682638468169, 1.097460814258429, 0.6587241807754287, 0.6011094709846179, 0.4315825372742289, 0.4511163660256209, 0.6633943209483042, 1.243386284077773, 0.2767034041281571, 0.37293419675223743, 0.6046956442134982, 0.026479124311655146, 0.6925110984316529, 0.2884380418229878, 0.5058199546065323, 0.41987727719696155, 0.35848321865670374, 0.8046334776302957, 1.3337620034466986, 0.8040762623673978, 0.5044877612589488, 0.4252550337205845, 0.9850200520238014, 1.0670151575181699]], [[1.2417477503229077, 1.1661916174704312, 0.2881713009300596, 0.8482854026483131, 1.275216287286248, 0.6868046456270676, 0.4364791547106328, 0.9751744256275261, 0.8270711260992946, 0.2635196892177036, 0.395781261589068, 0.7906562546432908, 1.1300028499192438, 0.5773955050523754, 0.6307257438311663, 0.43051901709808027, 0.45593430284576486, 0.7532328450753555, 1.3097157206692374, 0.24060131509069943, 0.3328147623960602, 0.6513561157168022, 0.020237963210660948, 0.6023114392236691, 0.28848620837819483, 0.4842979042090465, 0.410638480775285, 0.4354123764623266, 0.9772794732116521, 1.2746708893889185, 0.7846455256240471, 0.4298267439592688, 0.42962874273492835, 0.8887857605822767, 0.9742013304528321]], [[1.1599348351059695, 1.153348533456841, 0.255208930876331, 0.8473337036394284, 1.331374230360566, 0.671694039095982, 0.42310065974575006, 1.0038853989673053, 0.8079224995694879, 0.28466812621951204, 0.3761034171940228, 0.8097215201566653, 0.8980529737983762, 0.5979798490076103, 0.5665304856965843, 0.4272344576661975, 0.45027207326377505, 0.7378059694440495, 1.2928114187645185, 0.2521961436914364, 0.3432962464718531, 0.6541211328347062, 0.020400812055251877, 0.6464900324921415, 0.22588802144530115, 0.4608646867168718, 0.4308238442350615, 0.4263244791535015, 1.0330915564868037, 1.1704214776599167, 0.7295755612606093, 0.42666069893298836, 0.4494309229295479, 0.83860357302201, 0.8516005513523053]], [[1.231998169888898, 1.2054724786875728, 0.25476285532248794, 0.859554006051577, 1.3789397846234934, 0.6629428595702922, 0.4326853558891219, 1.050110717463479, 0.8315664375282725, 0.342714634030357, 0.3171561722273356, 0.6969148700801371, 0.7748583388682271, 0.5985040768529426, 0.43292943756695684, 0.4033615959573783, 0.46574846173935897, 0.6142400047192451, 1.1541499588069497, 0.26494903362327804, 0.34877261512123436, 0.5956037657801243, 0.024916345506904386, 0.6733294857001086, 0.16934787173325438, 0.3886280594794077, 0.476189804322538, 0.4001170667943378, 1.0870987333222886, 1.0552319978159441, 0.6630284447677935, 0.45463372475329256, 0.4297307794969426, 0.8483353579392121, 0.6710529484547859]], [[1.2228703824527238, 1.2238548121786477, 0.2478702333366617, 0.83876599697235, 1.458431797565719, 0.6913659468271072, 0.4085877703284993, 1.0681976532625415, 0.8557335703660385, 0.3821023340419547, 0.2863142365534157, 0.6736023841147197, 0.6140102647077171, 0.5944048133952018, 0.4412348902065335, 0.40524779753936935, 0.4832599553134336, 0.641189740467533, 1.1041271044745724, 0.2249524722583735, 0.32115113072437795, 0.636503303992044, 0.03051188599195888, 0.6635696733597267, 0.1556679783270906, 0.26371441832496223, 0.5403741117071409, 0.24033435782965148, 1.1907842763030951, 0.8831724144584681, 0.6929323097013997, 0.529842205685683, 0.39960979831125176, 0.8690077821636351, 0.5904097534259568]], [[1.153392061920072, 1.2802182290027169, 0.2587985643338221, 0.77418924618245, 1.5020964691986878, 0.7120837916108833, 0.40953113348198195, 1.029970197602675, 0.8691063825983062, 0.38951850877900274, 0.2739469133131518, 0.7669647069214348, 0.5582563162602961, 0.5919286973461492, 0.5000271589214564, 0.4170114875124816, 0.5627908275471415, 0.6094804798856849, 1.188010888912704, 0.19557791468513383, 0.29298537013062365, 0.6658807057298519, 0.04382600088819416, 0.6379274466567785, 0.2154152339138239, 0.35489567348771217, 0.5308814850972232, 0.2007087338420603, 1.1262390008245173, 0.6627212109526299, 0.7860432935969046, 0.5406626843949566, 0.39184286117127926, 0.8700097118087433, 0.5409500414921862]], [[1.1485027267221826, 1.2154402512662805, 0.2567128343897367, 0.687054410881013, 1.5894149087082403, 0.7018156466052198, 0.4103938230851963, 0.99227419657627, 0.8778035603290724, 0.39327814892188717, 0.26750652946793174, 0.7679377494637639, 0.5700788438526205, 0.6560934191242126, 0.5044698008223004, 0.44049901271998626, 0.5528098956060683, 0.5627880140545198, 1.1430698881834345, 0.1895168576892782, 0.28968586606861835, 0.5983298012096487, 0.049570693995590207, 0.7445915153978201, 0.20505773616923137, 0.3409715398962539, 0.4461038868194816, 0.20277681566840194, 1.0031419736204663, 0.4901268359131269, 0.8569104287769677, 0.5535291958777514, 0.39134251691231814, 0.8669636010494215, 0.4860589628116787]], [[1.1214296125200869, 1.2775412846733778, 0.2563588830273953, 0.6521360513305522, 1.6162040660512726, 0.7146596684868101, 0.40007333606560846, 0.9216795682020607, 0.95963382940296, 0.4297779545050268, 0.28207713139347157, 0.7109631646684509, 0.5381053218048204, 0.7100198258630117, 0.4866525958552357, 0.3941161746238539, 0.4996183650290317, 0.5464928767489786, 1.1193598157146805, 0.18833872513096472, 0.27587890623957395, 0.6017999791097465, 0.04864423173034184, 0.7927355914724006, 0.2852418810586847, 0.38184566377692636, 0.4230655300389539, 0.19519753907486548, 0.8954057668410004, 0.42693347732567444, 0.9153844034769709, 0.6062306554860704, 0.3849292075238574, 0.8243804312294412, 0.4242214920460303]], [[1.084606779854843, 1.3302523535558655, 0.2573987989747793, 0.6029764002182745, 1.5956800333630656, 0.7218680809992186, 0.4257628750834689, 0.9063792698753386, 0.9309186463572543, 0.4044223173673896, 0.2930314328615422, 0.7185651791248074, 0.5228530910889116, 0.6920414927392313, 0.4789925185860677, 0.43533099063740577, 0.46067089047536014, 0.5637670740068869, 1.0587320532085926, 0.18475509478080762, 0.2795332388834355, 0.4849801223380743, 0.07272018837915106, 0.8441186806631218, 0.3015399214003352, 0.40756106702651257, 0.46856677585026574, 0.2312456179948675, 1.0124315795273404, 0.37729996635932933, 0.9631918782603412, 0.6647756932094291, 0.39489667531447287, 0.7853362866853495, 0.40253801273114287]], [[1.038130877367947, 1.361611582057461, 0.24777332872008467, 0.6236345406632757, 1.6397807534729534, 0.7368406327525957, 0.394679614564722, 0.9620492117409423, 0.9447027081838509, 0.3648232922078021, 0.23643513728609802, 0.6950354496921527, 0.5215512880503169, 0.6340129072879715, 0.5333560214666475, 0.43460792709600526, 0.4686065114893446, 0.548839530738922, 1.032868362161314, 0.19937128483200525, 0.2903978064833591, 0.4260060505105622, 0.03379784977621363, 1.0058357115635272, 0.2638310464654862, 0.36320091909244956, 0.5571650273404791, 0.40143510327046483, 1.135958875159914, 0.3724868626213411, 0.9954811206339373, 0.6249082673214728, 0.39207516387468144, 0.7534658817961013, 0.3953944565541]], [[0.9945204129138283, 1.3459124340290376, 0.24928083512623878, 0.7281330856562673, 1.6753700027750411, 0.7423863985721468, 0.3988636118128974, 1.0418397093946274, 0.9390976891605828, 0.35226499768227804, 0.1864044673834296, 0.7267693962548227, 0.4562817021216339, 0.6318988690839853, 0.5291761832024102, 0.43289204814429555, 0.45218359644032047, 0.5145146371079468, 0.8692734082986221, 0.16259957580220594, 0.2988260494779519, 0.3283561332143226, 0.0617787727522973, 0.9915986191210007, 0.21204081905808644, 0.34395599928785225, 0.5612878556503865, 0.3456181184998351, 1.0906156339654782, 0.3980738910590345, 0.9836764073496927, 0.6282622700872642, 0.3915124237049553, 0.7373804883994091, 0.44423200799863133]], [[0.959943401290679, 1.3638731616948905, 0.2512204622236477, 0.6994351608288334, 1.3484536947787336, 0.715866399598728, 0.4202879815146466, 1.091463902698196, 0.9745873502803053, 0.35911471834190783, 0.17965228853391707, 0.7088038786100019, 0.4203614065592505, 0.6588109866181324, 0.4938476340071228, 0.4453892173237185, 0.44735327900471367, 0.48652148822765057, 0.8910039339607874, 0.13538961961985763, 0.2905469722126238, 0.32321558203268624, 0.04994178682452887, 0.9908135699474199, 0.17864842932953595, 0.3451074245901271, 0.5465715417434426, 0.3659738061141523, 0.9928818621661406, 0.5053510022557752, 0.9880133538789785, 0.6694112293001542, 0.3875948862856804, 0.7066754398377061, 0.45696552214893604]], [[0.9993435556746454, 1.3259404722397201, 0.25127672673592005, 0.7298440941407379, 1.3271075094264106, 0.7017052739040581, 0.39161261120736746, 1.0605684845966716, 1.001199331876306, 0.38357148744630865, 0.12901817256248305, 0.6848456474537613, 0.43482247530301166, 0.7077530009551694, 0.48397402873454987, 0.42984117692550416, 0.45771343332272274, 0.5107263107249227, 0.9076844182949042, 0.12290928521916833, 0.2729203685390707, 0.3225922979043242, 0.013379483379531504, 1.0522541056921166, 0.17521533688694124, 0.4075615044425292, 0.5873639431079041, 0.3412031748280703, 0.9191075699703173, 0.6528950178775299, 0.9805649791039257, 0.7164943246048792, 0.42597792042766525, 0.6860509899517587, 0.5096561615845858]], [[0.9964655992460458, 1.3199352312445172, 0.23852692938919784, 0.720754193880171, 1.3331172340759982, 0.7086554234382554, 0.3935416127898482, 1.0487780996756941, 1.1103788740845628, 0.40750773974650767, 0.07974316535335366, 0.6765589223244437, 0.38215734701732573, 0.6847129896066295, 0.3954085201143649, 0.4516002960294441, 0.45016755499310673, 0.5314345637465807, 0.7816029458383591, 0.13154304679374643, 0.2376593917575616, 0.29221440081614813, 0.01641992206461745, 1.0345537795219848, 0.19995611416034068, 0.5466394676980324, 0.5156327099360786, 0.337325486041667, 0.8503558686915709, 0.6640056220025512, 0.9418826856816102, 0.6958575015438184, 0.4136523242243838, 0.7467190659258796, 0.6182843539626715]], [[0.9864615137922581, 1.3351044864377988, 0.2421848888098695, 0.7287531779238595, 1.3569502449009163, 0.7223960287557779, 0.397384896217331, 1.036341061466541, 1.0517495066662506, 0.38873685465412744, 0.07714935408981632, 0.7195273099460606, 0.4550422177962635, 0.7108834253624063, 0.336557964578524, 0.45749316597990763, 0.40319268133756125, 0.5748064216573022, 0.793277966192872, 0.13573041138816863, 0.19560791993520488, 0.27045784211736806, 0.014811604550007938, 1.0217189703873086, 0.21890987533461054, 0.4280895715674141, 0.5061490369400711, 0.35408821316185285, 0.7687168032982235, 0.6283121377209345, 0.8753541877102633, 0.7586188560796185, 0.3678780056331009, 0.7642604642787597, 0.5454455355825958]], [[0.9909948078730205, 1.300807080492869, 0.252714079410188, 0.763830433398889, 1.3648554301757365, 0.6980126442790968, 0.39343096854720794, 1.0309756081399843, 1.0272252734187688, 0.43935750783465993, 0.12120036482925114, 0.748345811470164, 0.5372343391581917, 0.6935441469749126, 0.33696854776861973, 0.4178002421026288, 0.46879973754523196, 0.6325653471296062, 0.8320605444292268, 0.15198316301879206, 0.19269336868190767, 0.2874900241960326, 0.015522484207049594, 1.0031624993411263, 0.2573801485402167, 0.3454039915385978, 0.5046276237893247, 0.37528737065867684, 0.7881528288357704, 0.6150900009646043, 0.7934285446612523, 0.7903754792985009, 0.3515789561774062, 0.7813344830055005, 0.4301775527816941]], [[0.9852082523807253, 1.2866720817577924, 0.25304321645642225, 0.7847478670334945, 1.3547471325905076, 0.712056705750251, 0.40234011960117316, 0.9843347846331956, 0.9748957320513734, 0.4466969505265054, 0.11736220769541508, 0.7740581250879308, 0.6362298787746624, 0.6787316195384787, 0.354458223281584, 0.41072554852543885, 0.4780986025099172, 0.6600486056522975, 0.8112651946263395, 0.14000233100234724, 0.18769298616148872, 0.33297864859010945, 0.0204449393300176, 1.0052776451159182, 0.2905184456557286, 0.32796797445148085, 0.5755716256834164, 0.39283662946043285, 0.7613602156532684, 0.5806506034702956, 0.7381241634706587, 0.767549484594807, 0.430476528372129, 0.7822783844933248, 0.4553889516386851]], [[0.975162520756802, 1.3550182056029398, 0.24892448061863243, 0.8053774892294407, 1.3269845072736217, 0.8058895313342704, 0.3797584468349944, 0.8315653531709077, 0.8388658512028758, 0.40722567013479877, 0.13845613241813465, 0.8588218555170298, 0.6946775155812683, 0.6846962792603983, 0.3404805292148986, 0.4336499909573829, 0.5254274938505117, 0.6992701361538888, 0.8653444655018805, 0.13099651894518882, 0.18108801927990315, 0.37639850392783175, 0.027333094938553903, 1.0526106712092713, 0.30291012843014414, 0.2992098291666154, 0.863020101715231, 0.4352314505182243, 0.5809472690329618, 0.49054241217206485, 0.6492920604804548, 0.7669650272716706, 0.4722917961649117, 0.7895331448767713, 0.45295514821685845]], [[0.9861241349553309, 1.3900570623671828, 0.24604997651228327, 0.7627839394078794, 1.3345074287417809, 0.8010224885396463, 0.37685693122681374, 0.7855800087025986, 0.828382341165999, 0.3249752816371454, 0.15316541121273133, 0.8535227246620964, 0.7358071974594573, 0.6478076690949899, 0.3332433776541694, 0.43038973888855603, 0.5232021999510033, 0.7252779472878809, 0.8845425143061674, 0.13436470616449683, 0.1724705485117023, 0.44117170375817005, 0.037534623587969926, 1.1262304009586988, 0.2933070945269093, 0.27630688907928375, 0.9402371324938457, 0.4533547198927543, 0.4792273715428494, 0.4601413660371054, 0.6582647048295285, 0.7532238492154731, 0.4969940054313015, 0.8125984671296134, 0.471351704125053]], [[1.0567436295871657, 1.3793697034485453, 0.2461878776831245, 0.7734657038568286, 1.3168922012261985, 0.783499682089654, 0.3837613566164231, 0.7424598428459313, 0.9153552153074173, 0.28699082317830044, 0.14046164803667716, 0.8491771096039934, 0.7110769226101047, 0.6531638306593258, 0.309149062896159, 0.41528582693673394, 0.5046318455274856, 0.7278132064653041, 0.8908174546486991, 0.13160984444786347, 0.22631023149795534, 0.5142498313151715, 0.04985214392198997, 1.2081178850989933, 0.2772303985542053, 0.2623204789745388, 0.9373812526766341, 0.4888636105094063, 0.4660319188926828, 0.35709439653825314, 0.5708839337472125, 0.7549739458808524, 0.5118639614790677, 0.803445637085642, 0.5809739920333628]], [[1.2399408961863632, 1.3646948134904684, 0.24452837616331577, 0.8728335989506555, 1.2469768142031983, 0.7777258319376879, 0.3940002010150179, 0.7383809777372815, 0.9707261788112715, 0.27446302313259047, 0.1271075122555916, 0.8454107135206166, 0.6719715984641255, 0.646246078468085, 0.3223196560991683, 0.41676427708060443, 0.5368055508462686, 0.7591244352355915, 0.9260797596966666, 0.1465558200406128, 0.2129106770307618, 0.5314350450673049, 0.039779947084945436, 1.1993641206302081, 0.2953810109652934, 0.24916807506475408, 0.9135328861292149, 0.45089518826434016, 0.42193298398984724, 0.24137744282930287, 0.45282300598603237, 0.7486949637870461, 0.5237294886929393, 0.8149456932633405, 0.6182287927342978]], [[1.2107435594765588, 1.3530256245679517, 0.24315425442335725, 0.8963842078930698, 1.2702106178781292, 0.7796384087033195, 0.4030394501374338, 0.7051594157627238, 1.0008502990734809, 0.2433603244295427, 0.11072352358538123, 0.8383187711612368, 0.7151381419741094, 0.5767672532469805, 0.3400240166384876, 0.40909269627720146, 0.6480053594139813, 0.8183868610148448, 1.0049504283109787, 0.15321277046285334, 0.21289303749921343, 0.5362166669362457, 0.03850493391851953, 1.2793853768996168, 0.3422757075255951, 0.23200463323501053, 0.9505991141979504, 0.41354365584892006, 0.3591868429637192, 0.21720127605046918, 0.4498380988987648, 0.7483147261274172, 0.6220658850438971, 0.6679114202041725, 0.6824028740500445]], [[1.1551794135499684, 1.2745903194695307, 0.24860269650246286, 0.8871942092240026, 1.3221242095888055, 0.8374027231249263, 0.40450133369705704, 0.6220656482496423, 1.0327661922550324, 0.2263510195199729, 0.11510193622371476, 0.8420139468598482, 0.790098255798821, 0.5924782088556179, 0.34505484459088254, 0.45084846506645315, 0.6229120297825856, 0.8165749144824537, 0.959087760861415, 0.16192040034665559, 0.20729130064771092, 0.5238277320716584, 0.07067970832306744, 1.2505678378445824, 0.38956420190547153, 0.2417160652837067, 0.9598765121603028, 0.4195975819131014, 0.36715735702206753, 0.1344972515977887, 0.6478694456805314, 0.786424799830856, 0.7613043499843839, 0.46530172298133504, 0.7553180958357693]], [[1.1735774037488778, 1.2620169529212932, 0.2500202154329266, 0.8242931956730268, 1.3210258917009257, 0.8446166409447986, 0.4298541081257511, 0.47388058552718165, 1.0344135099542922, 0.23851858556945288, 0.1221281622875468, 0.8557691620609871, 0.8553773108947793, 0.5872237303918252, 0.30520659864845356, 0.4371484850724406, 0.6111117279740776, 0.7918988715934975, 0.8656542384222373, 0.1673093795661233, 0.20437449525781623, 0.47975529287230934, 0.06861262374357441, 1.2866580168887618, 0.38658480387610095, 0.26398399441087217, 0.921915284940767, 0.3663957439808223, 0.3141815724793117, 0.15334483270113702, 0.7422675048088179, 0.8780055818457595, 0.8890261539881696, 0.2950754981655123, 0.8639952697049558]], [[1.1868968470399384, 1.2126140674617643, 0.2386234205097809, 0.7922690347806208, 1.3442300751050462, 0.9490900699520308, 0.45865059291049226, 0.38034597084479843, 1.0976164848500658, 0.33496991820091504, 0.12867263378044494, 0.9415299982865639, 0.8947604342067776, 0.631676010280507, 0.30126712000894185, 0.4802878596327884, 0.6115831283318836, 0.7985865309862676, 0.7774795885894302, 0.17438354997776417, 0.1792156356261353, 0.5234126149834479, 0.09227803271277027, 1.24680655854071, 0.36412402072086214, 0.24581222244494835, 0.9313613536879282, 0.3295811847989194, 0.24691579667479438, 0.2534485372833389, 0.7698530682244653, 0.8766000989840381, 0.8898625228272492, 0.2759806428065986, 0.9581664564679322]], [[1.1896213571784202, 1.2117954707318628, 0.24128378088951397, 0.833075551031961, 1.2790089908923694, 0.8777439391977475, 0.4073521234861307, 0.40423052783366564, 1.0899785744920156, 0.37101800605797153, 0.12285399172357553, 0.94238279700502, 0.862015666442657, 0.6339481255914414, 0.2897938120099376, 0.4564525514754656, 0.6148016880379576, 0.7618033514046545, 0.7402391234495975, 0.1840492647617617, 0.1696233632437761, 0.5122664345104417, 0.08131911151757601, 1.1455833258744303, 0.36164753284243445, 0.23719729250586816, 0.9360716417805008, 0.3208486875480838, 0.2558971881642631, 0.33547472667631306, 0.7993968296021334, 0.7709849960762275, 0.9056353588492407, 0.2814478096841561, 0.9480351051617538]], [[1.235422978488789, 1.1740067118882882, 0.25710198060877376, 0.8553147447257203, 1.1421097200871757, 0.831928916244632, 0.3918397498120616, 0.4411173500172613, 1.0155354610122533, 0.3504671641791267, 0.1197823200273915, 0.9378805142756748, 0.8234950380750877, 0.6242697758792581, 0.2904950528718808, 0.44695129508678555, 0.6257394934354394, 0.751933564421821, 0.6990552140448826, 0.1938835956584585, 0.16670638333722998, 0.5058494919743819, 0.06789604305538206, 0.9965441952678246, 0.35764018015903, 0.28120527036618637, 0.8829594965163929, 0.31507549056449047, 0.29169522868401054, 0.39944068461487037, 0.9240566967782742, 0.7056499150661276, 0.9186775431564169, 0.31038705324694926, 0.9246505387655608]], [[1.2962423005319954, 1.2286462523898543, 0.23912848574076273, 0.8446463448634582, 1.144062241388502, 0.9146146166961003, 0.4123509663660351, 0.6317040017696778, 0.9664396321134456, 0.3601972553540168, 0.1178059150092985, 0.8853031346046879, 0.7931639321794131, 0.682037623815765, 0.28641682092014054, 0.5084323253840182, 0.6319225334029759, 0.8091356064047435, 0.6453650916356579, 0.22574457378556434, 0.17460546916256525, 0.5131119315931163, 0.06834931310516451, 0.9698105484428157, 0.3776738470710025, 0.35140831085943625, 0.8677648652310912, 0.30555009323121085, 0.2988381103143974, 0.4635535870713841, 1.091919987767033, 0.6302176482849284, 0.9435528831865956, 0.300291966712162, 0.9119105103124622]], [[1.3019938728351221, 1.3199498338523314, 0.268010886882628, 0.8240455175782995, 1.1473206521967518, 0.8509281506761167, 0.42166858274591673, 0.5109686203003034, 0.9378298425144288, 0.3060763907639857, 0.12040233293715974, 0.9291607773431536, 0.7889201385058247, 0.6563730314499446, 0.27958502738731855, 0.4882222184180269, 0.5962914482868704, 0.7839924644758527, 0.618601195538065, 0.22626450335595388, 0.1754211238511325, 0.47082285432182913, 0.0962772652545981, 0.9785231735359216, 0.3992059862203793, 0.3778346886216496, 0.8336305559880266, 0.30903074837017686, 0.2796182700645344, 0.5003201132146518, 1.161269478174776, 0.5401641133060254, 0.921610722533359, 0.3539834408867945, 0.9318834956633597]], [[1.2974220874067957, 1.32090589878569, 0.27680564317058376, 0.8124342353329089, 1.1782212954607294, 0.8463696635834639, 0.4470225002578036, 0.5171692963201001, 0.8925492292781261, 0.28891001186864745, 0.1336010719328155, 0.846344146399437, 0.8314463667870069, 0.6299735265271387, 0.2939756981247712, 0.45640593238363475, 0.6317880219591503, 0.778590688821212, 0.6740009995467362, 0.2512740815583662, 0.17405142681119004, 0.42041572348177425, 0.058425461129637965, 0.7982920854787643, 0.40936741697529844, 0.4070617318283612, 0.8322885837716812, 0.31155709924984953, 0.2736440388227153, 0.5720685217123417, 1.2028695474432662, 0.5475175076001911, 0.9103759619672369, 0.32216475364233826, 0.921891549315576]], [[1.3923301350943464, 1.30089711077497, 0.2737522580995204, 0.7871968702497811, 1.2063062018320356, 0.8658661868960218, 0.4468803451528326, 0.5041998968057803, 0.8629838480356105, 0.23610577305322455, 0.10470835903597016, 0.6302106163918539, 0.9876261226658818, 0.7208927187731512, 0.3360570273665017, 0.48240117582276004, 0.834177703578203, 0.8146356048623925, 0.6532931809354487, 0.2925193091306172, 0.18479295612229546, 0.38776474434867825, 0.06480904784118076, 0.6934857449069511, 0.41307615258057007, 0.42199046812558344, 0.7451631365323139, 0.30762615669567916, 0.28760284358370997, 0.6330037580912905, 1.2673475845080278, 0.5848952711771981, 0.9200701717598765, 0.26905716793918333, 0.8959527339654973]], [[1.4043535035644366, 1.3508206253761668, 0.29393120607460166, 0.7848989058451907, 1.188861492103455, 0.861856779368469, 0.4291993828477594, 0.5008515469064553, 0.6390510300613028, 0.2046456312986894, 0.07169636869011622, 0.6149213925186267, 0.9401478305519628, 0.5637860925358448, 0.3472659117807877, 0.4587956138232501, 0.8151713558785325, 0.8302328410127413, 0.6684195476485315, 0.34054367703502747, 0.23272925385254642, 0.40943411826940257, 0.06777188872782428, 0.648495035401184, 0.4146048053521406, 0.43308349761056275, 0.6893173261568937, 0.2914590460253869, 0.28536031495052355, 0.6570140190617769, 1.2420163183030484, 0.6321330874939043, 0.898249792774267, 0.2606804090207134, 0.8816881667073266]], [[1.387164981858626, 1.2939348140021392, 0.2821333107660149, 0.8056947144906876, 1.1672289278849257, 0.854991788838754, 0.42639363363075017, 0.4939583227474115, 0.46163277677670117, 0.17581213473285034, 0.082555015264179, 0.6025782903554525, 0.9562119377655676, 0.5871441906522072, 0.29922557470003214, 0.4749131406435656, 0.7777323684552884, 0.858868294831186, 0.6867064442340528, 0.34457863771658825, 0.2574714824054526, 0.4159445025025137, 0.07044807450365276, 0.5501343526183324, 0.38697969149661166, 0.4846059769343554, 0.6325728067328886, 0.2804396128550531, 0.32825362112313333, 0.6978948736879415, 1.2678160085189463, 0.6500142393255397, 0.944402978821745, 0.30835707643346955, 0.8660789954974417]], [[1.4778644643926993, 1.2736057210257872, 0.2808716970476699, 0.7829957981877369, 0.9935751633415709, 0.8531557378062218, 0.41352206472086267, 0.5161048346086133, 0.36351238359784066, 0.17129967610805819, 0.11528480221644394, 0.5506498270516432, 0.9089051019532182, 0.6048915514204201, 0.2613668130131134, 0.5171133504172497, 0.6426930906847746, 0.7291219803249047, 0.7047136567103467, 0.3738019358467131, 0.1928266911684793, 0.4336258517651806, 0.07204735759735131, 0.39364133345428065, 0.31430135973952644, 0.4446936868541771, 0.5589768211468821, 0.29365096752304126, 0.3196817898552688, 0.7287083779097365, 1.1741242069782905, 0.7092786035535762, 1.0117507596710031, 0.33482715675127106, 0.9874254392029485]], [[1.5618731762834217, 1.2786451658482547, 0.2938868541744545, 0.759469325001424, 0.954675409406057, 0.8615626897380939, 0.4349599801952504, 0.530570943674539, 0.3254077305634649, 0.1708613456894174, 0.1177110426676681, 0.5423069795159561, 0.8488669313692229, 0.6382433286542297, 0.26787474479822726, 0.5080917334743202, 0.6453762528968222, 0.7240383078616817, 0.7111197550968257, 0.4036431599804903, 0.22052846714719368, 0.44020665169235984, 0.06255584279106682, 0.2895604262610571, 0.22349874437857048, 0.4169880696946113, 0.5359927192108193, 0.2931883437326045, 0.3259856159881028, 0.7371558599006601, 1.124258449404992, 0.7021725937131469, 1.0641544425231608, 0.33533759566323257, 1.1772891383561064]], [[1.5395511561087283, 1.2102142550897521, 0.2919811216446567, 0.6974501336064187, 0.9070144956743771, 0.8760383071486999, 0.4297007858175013, 0.5567865325794126, 0.23579416856233826, 0.18142987827353635, 0.13350168345311814, 0.5425973234871858, 0.7877490199278485, 0.6340904011280368, 0.2604711238442513, 0.5155581195299154, 0.6268595222496421, 0.7089966480341221, 0.6428450916057216, 0.47369257012684196, 0.23859266611973956, 0.4858885022843865, 0.05382042381647949, 0.26900570833326654, 0.17851340749664718, 0.4288289776324351, 0.5058542527251498, 0.2923979698177122, 0.3407785795838805, 0.7438012624843462, 1.072629333382157, 0.6574419414593502, 1.029305480822273, 0.40072505233706, 1.096311967696645]], [[1.485434487505898, 1.2057798482674205, 0.29128868611684045, 0.7225064082772743, 0.8811738428693495, 0.8423668453950656, 0.44813807265033107, 0.540536462697056, 0.2413467960881502, 0.18911512803898356, 0.14411294041329628, 0.5755913516691407, 0.7380057628804687, 0.5940975209791077, 0.2420942803389073, 0.5093999851330862, 0.6563108954911392, 0.7406211080870644, 0.6766066022302923, 0.4534988814874666, 0.2238561865421254, 0.3596190420950782, 0.042879032117944305, 0.3145664112864428, 0.1258099421384693, 0.38925517545616106, 0.5024343121518057, 0.2969162440059837, 0.3587977968096501, 0.7398758816640343, 0.9553145747120757, 0.668716465391487, 1.020106069266403, 0.4150443009098792, 1.025174221233275]], [[1.4871200095684292, 1.1893478092094496, 0.2631682637484598, 0.74129614485914, 0.8389988135999331, 0.8423920679958485, 0.5140382255533763, 0.43390624052981847, 0.2772937957969963, 0.22635565242839026, 0.15023437073399235, 0.6116938234414497, 0.6633409599835294, 0.6015732745136085, 0.25912981345446706, 0.46620266983332403, 0.6440951565355435, 0.6736139761361808, 0.68223315196137, 0.421083947494981, 0.21260386139936144, 0.3650518807445556, 0.09649501921727027, 0.3375060158515242, 0.06458501564253322, 0.2939438484820853, 0.4901492576478702, 0.32768730292415826, 0.4019195683991387, 0.763224234371385, 0.8427113462443706, 0.6704410167204019, 0.9962757135471311, 0.41790794537436726, 1.002575343498259]], [[1.5077893118017172, 1.1655085768173898, 0.24655410900862318, 0.7046982144518311, 0.7864889304061491, 0.8273762953812378, 0.47006207572530523, 0.36393802364016375, 0.31080900927390653, 0.3108145287581594, 0.14028364521632022, 0.5663628272702813, 0.6749632682334687, 0.6113883903409274, 0.2533427381963818, 0.44005778590812744, 0.6960455068709555, 0.6291596808559371, 0.6509452583832371, 0.529692600458957, 0.31640577954820415, 0.33118849854868976, 0.04463775163290053, 0.3172605353642157, 0.15799268272336708, 0.3325016867377838, 0.4130086702180521, 0.3432169323017148, 0.6247678602749428, 0.7922331426275078, 0.7713385129048589, 0.6638742620093584, 0.9624414173461388, 0.42594211808725374, 1.0430355835759262]], [[1.5329340039571393, 1.0777846470096106, 0.23749232204748533, 0.6916021967116424, 0.7805430883803937, 0.8228125975697573, 0.4454474563598344, 0.40603933247044244, 0.3344629300703796, 0.4407064538102665, 0.0933906142611964, 0.5680187933610173, 0.6788736743241593, 0.6390227516398483, 0.33025309850373746, 0.4401637896120999, 0.69478539846025, 0.6605550250707435, 0.6998250784744371, 0.5849601634350059, 0.33472700521385956, 0.3517136805752224, 0.05130449766870457, 0.2735647552925522, 0.2971134176843627, 0.3766396268629922, 0.4024280390978055, 0.3401802108699889, 0.7511159676825898, 0.8089527872544354, 0.7375877862482632, 0.6492369969987402, 0.9255507583275697, 0.4469584205922889, 1.0532302201563404]], [[1.6335138996857215, 1.0273698786956342, 0.22754510774387873, 0.6662871297013807, 0.9469414361095917, 0.8397973386522832, 0.42199028636571734, 0.4420380597533945, 0.36232105483046584, 0.5414933573111492, 0.068468968199351, 0.5892211560998903, 0.6022333474272736, 0.638880547312239, 0.3286119975526353, 0.4197023163629431, 0.7154334720290924, 0.6225805091309389, 0.6691794293436568, 0.4409135557479645, 0.3533158097398441, 0.42132918670549246, 0.06245638517656278, 0.23511740910548984, 0.3458243734197054, 0.4005916293781789, 0.3354647922505744, 0.3379558572971274, 0.8382169584057975, 0.8680648946803213, 0.6850711641286356, 0.6234694082938333, 0.8888279209841952, 0.47564318554382995, 1.0077082419932823]], [[1.735191901402938, 1.0681937895411642, 0.24104779195232093, 0.6345218654782382, 0.9464100518425931, 0.8339486660163304, 0.46888584022703994, 0.43987918955099736, 0.39260998185697893, 0.6112893323745214, 0.07909508265518825, 0.6318434866996177, 0.39225176437967174, 0.57885734110191, 0.36268914814042436, 0.3990404738931954, 0.7486917110251081, 0.6803242169701367, 0.6674616136162683, 0.21788371152944547, 0.2977287174127521, 0.5038934545022029, 0.07009586608022564, 0.19662613941569762, 0.42460824312126366, 0.3827971072817843, 0.32132416783865597, 0.3538670046001002, 0.9057283836901463, 0.8845742370509941, 0.6880735216470001, 0.7266253414993653, 0.8359875774724774, 0.46177866822211866, 0.9497138423420357]], [[1.7301050477263797, 0.9921477326836525, 0.23495402134539528, 0.6743660222523876, 0.9462568936970466, 0.8568962337279147, 0.405081939742187, 0.46203377927888345, 0.3955416564390963, 0.6373399840896214, 0.1369086933060067, 0.56187412828826, 0.3185897251303788, 0.5213301610966588, 0.42178716598595783, 0.36570551715613686, 0.707478682494281, 0.7635293630001274, 0.7122064731063353, 0.13487282795815922, 0.27658715914366866, 0.49732319218216886, 0.03804050810062807, 0.18192101525390364, 0.4936671070448524, 0.3888197935898092, 0.32260260177549216, 0.3548360099098713, 1.0000573196335973, 0.89062249399115, 0.6880561037237324, 0.6504444670279108, 0.7402408704440129, 0.4521233055242305, 0.8552214358494672]], [[1.722037629567914, 0.9912227255124505, 0.23017149162906975, 0.681841824476915, 0.855967551963743, 0.9141030901449065, 0.4265226150006656, 0.40524524512571447, 0.49076076269746566, 0.5583171893639187, 0.18885723906238944, 0.5371674454422366, 0.26828272042263507, 0.5249136566014303, 0.4327627652454607, 0.381450741592128, 0.7033060347047704, 0.7672802638288868, 0.7233761766425025, 0.16343928271260122, 0.3489138946577372, 0.4434625271513134, 0.04992921162831467, 0.18614992576751632, 0.5319484539689645, 0.4072600845871353, 0.29761064773269125, 0.3915318716059313, 1.0087703025848054, 0.8741683113734251, 0.6199780604761531, 0.5506628407855012, 0.6211735488666392, 0.4367092748254175, 0.7154924389485292]], [[1.7495575896205633, 0.9897846546580669, 0.24001263399556538, 0.690266246161787, 0.8921800154692385, 0.90481563354258, 0.41857104492674047, 0.40443105098392496, 0.6107945360718032, 0.4971785469571449, 0.20450390851665828, 0.5511287970671627, 0.2753597350454562, 0.48422970410790234, 0.48989078749989085, 0.3858332346694143, 0.7005200746291758, 0.8287745442270866, 0.7504477773128617, 0.18705793009794414, 0.35914849659574216, 0.44912077358491875, 0.05129464672270101, 0.20571569323470568, 0.5878836956223804, 0.3927170245235979, 0.2834500096023689, 0.3978936775625402, 0.9133399656847978, 0.8734996497502434, 0.5792610527545976, 0.5435351948087203, 0.5663074113647371, 0.39035493318274167, 0.6457123304935853]], [[1.7909980800311942, 1.0148775339473637, 0.24608094924689694, 0.7228391135062331, 0.8713986518342043, 0.8842514083769701, 0.39687419955940073, 0.42675497815879204, 0.6793961811282028, 0.49149412359771283, 0.22087446998866594, 0.585604366985321, 0.2603430449679456, 0.49287252908309215, 0.546178643356815, 0.3786877536039476, 0.7039428395840965, 0.8344276459073829, 0.7808883705024791, 0.19221197706808035, 0.32141000669580855, 0.4288303804040234, 0.06087645248974672, 0.2187140065006604, 0.5726739147605591, 0.3385941257533088, 0.2619658202364578, 0.4250925412585835, 0.8634649705789782, 0.8990768482729463, 0.60553642418843, 0.5017940829457783, 0.5387503409414801, 0.3593874786404262, 0.6130978017859716]], [[1.7669183292859767, 0.9141269625620458, 0.20480025915436884, 0.6533025568441421, 0.8223985888380503, 0.9257567369501865, 0.383686582937165, 0.40905899891694153, 0.655041671693568, 0.4516860746086435, 0.22413234289785866, 0.6893559346475843, 0.3550265425131073, 0.5042680268179645, 0.5826753277621255, 0.37396922935436977, 0.6531226985480969, 0.8251841957087909, 0.8521352531756599, 0.1963153784027897, 0.3229296611664209, 0.43797622825604066, 0.05972800326610243, 0.21563775224720255, 0.5386696217105793, 0.2026563450358939, 0.2584046224133978, 0.41583471455155097, 0.7011666276775548, 0.8878031123910926, 0.6039178575064409, 0.5455146496267753, 0.4770207437627566, 0.3232022489940848, 0.5581387772174666]], [[1.8710090490150924, 0.8513663059582134, 0.1617318172070199, 0.6830813134591067, 0.7927429047038248, 0.8947112157672903, 0.40680005078456705, 0.4852063654997616, 0.6575224012708318, 0.5032228372349768, 0.18264385964235863, 0.6845718414692075, 0.4158409307490042, 0.4871832741733989, 0.5896130474325811, 0.32740701625015756, 0.6768603479067927, 0.8586667040275043, 0.8895276610276633, 0.18375974349864593, 0.27904066563672936, 0.3800528293805494, 0.09110016678364301, 0.22752094908911505, 0.5469580095868737, 0.13376155551213176, 0.25861353055319947, 0.45417685895258825, 0.6940892500441436, 0.8456883114709084, 0.5316753074926284, 0.42384737919252924, 0.5016297312946862, 0.2871130007801786, 0.4955551705630563]], [[1.8063978561850953, 0.8970665908743967, 0.21534069415939572, 0.623567647788465, 0.843569890207539, 0.8313953450202473, 0.40827695904404154, 0.4347540918065731, 0.6214173106650949, 0.5734817546932018, 0.20150874338726887, 0.6755681481338123, 0.38599784714317964, 0.4647178975844509, 0.6197164970167721, 0.32601559141248543, 0.6405901485712213, 0.9675456267447569, 0.9127002468479319, 0.17384457343673757, 0.2630800749103525, 0.3809601151157229, 0.09201307186146195, 0.23229998097167412, 0.5720262309500537, 0.15396633331036652, 0.2852357633970211, 0.4776820496554689, 0.7043686350374303, 0.8081320236911952, 0.38200518649467996, 0.27927187709933937, 0.522323699590247, 0.2794195862972301, 0.5604425610289845]], [[1.8055703633903126, 0.9279392811804585, 0.2286911824329464, 0.6154043934212676, 0.8745693641198958, 0.7616036191127291, 0.3744549365844823, 0.4038273021842479, 0.573536448156039, 0.6179882814846899, 0.19127574200977293, 0.7742701257184951, 0.2969458111875764, 0.5089636121122946, 0.7063915994234093, 0.3319880248253662, 0.6364590634796075, 0.8531019321537656, 0.8068348339481578, 0.16497407341199916, 0.3080895154478758, 0.34148998678478154, 0.06721774552941864, 0.21303412176817496, 0.6140714234276754, 0.15087233531378508, 0.4283790367657278, 0.6578937400702476, 0.6470790248122338, 0.7295787499587851, 0.25822147720288635, 0.15028947328289022, 0.5441096246183901, 0.27039535310090596, 0.5609471077421451]], [[1.8394353001107098, 0.9335258458008158, 0.19152238320473736, 0.6217481806498762, 0.9146324097081997, 0.7905451849906056, 0.3965985852397422, 0.3167113456623122, 0.5244987908895704, 0.6312247842200287, 0.17749267337844365, 0.7496725176478635, 0.1964958549909292, 0.4922570518272063, 0.7037980951326135, 0.33779936984444314, 0.6226595708909627, 0.7960752830049929, 0.7753918305470809, 0.15910383155425487, 0.31836245417197706, 0.3371927511153634, 0.1314416999211747, 0.19137664390911474, 0.618594460973395, 0.22565156392407001, 0.4701345892259107, 0.78164075287074, 0.7125766337235048, 0.7246178361637516, 0.1749165211463603, 0.10316341488377792, 0.5255557425661126, 0.31620319686372006, 0.4940501905362128]], [[1.8597131968704117, 0.915442591447279, 0.15272454360450538, 0.6263149154050967, 0.9925892542323911, 0.7540279465837895, 0.38162853333948893, 0.28538394818161616, 0.5520605349283245, 0.6087381955067306, 0.15065359384283586, 0.7345034867915115, 0.19095434061392239, 0.4896428629452153, 0.7093330814354586, 0.3426101966275266, 0.6423119498768803, 0.7850785066855777, 0.7849878755106358, 0.1599392940054735, 0.3405385608929442, 0.32301386977686375, 0.1216426756409747, 0.19673371490039854, 0.5872461852301712, 0.21089859669836822, 0.4782889699346182, 0.8421575349701216, 0.7416589036210255, 0.6764850515716208, 0.16580928293044214, 0.08304940346760159, 0.5767495948218294, 0.3720705812915012, 0.4856691985152935]], [[1.827565781348776, 0.939141320448466, 0.15147895607041068, 0.5813959357648644, 1.058117125655304, 0.727401686764879, 0.37451132943588683, 0.2640341494320515, 0.6253391457113249, 0.6167787605737511, 0.14520721886633167, 0.7532570897750017, 0.22742826237691727, 0.4993026768203293, 0.7374479871751076, 0.3430219031132594, 0.7087885379403666, 0.7940874236003848, 0.7342117327183421, 0.15142829723875384, 0.3314193208663544, 0.3235672873701222, 0.12934537741883406, 0.16496203677220933, 0.606847281162102, 0.17714868875067438, 0.5153048605315658, 0.8955577101588799, 0.714836714416884, 0.521494279944128, 0.16057314547680243, 0.11415450283108713, 0.6424354204507472, 0.3879825350903628, 0.6036017333122075]], [[1.754935395757414, 0.955746182285249, 0.1648971140219314, 0.579541296315183, 1.1391981346163067, 0.6889689396721885, 0.375356851172843, 0.30243012046878415, 0.676265520898709, 0.5989971436453765, 0.12608772456093514, 0.7431920742868809, 0.2435745946814401, 0.5003787418428577, 0.7065856105668431, 0.3391096552036946, 0.6989101730603519, 0.7898003061047767, 0.7398843203924731, 0.1615745535570484, 0.33240005201679546, 0.3288585499075888, 0.11108795281019253, 0.1646868001384142, 0.6529721409958913, 0.16444400431080836, 0.5175178152505207, 0.9324277230296683, 0.6851586885530587, 0.4197813674881471, 0.18274667629890412, 0.18128594137335524, 0.671555598840034, 0.39219417995022343, 0.6436046979043432]], [[1.7008668447223685, 1.0606197610674242, 0.2114669671166886, 0.5787354814993311, 1.2186931269283918, 0.6858222931203102, 0.3793877661544649, 0.32518559592749074, 0.7108192708123754, 0.6050059432141888, 0.0973901178178325, 0.6781740592544788, 0.25078630810625524, 0.4593213913058602, 0.7073614562893276, 0.34008419949506224, 0.6873476877545579, 0.7560245483700471, 0.8272071689593242, 0.15501316337476334, 0.38300281042149686, 0.3551048086383083, 0.08447453629319904, 0.16576209026514793, 0.9124819932649463, 0.15056388363164555, 0.5137672842367529, 0.998224268255808, 0.6694313301777705, 0.3453771073614584, 0.19181548224912406, 0.2560992919704399, 0.6888809212981108, 0.41598739475231616, 0.6570390987497796]], [[1.7213687019218928, 1.0835401885406932, 0.2364490832108852, 0.6969897900466094, 1.272670467684697, 0.7028195842564864, 0.3787565647781541, 0.31056689418156347, 0.8133608822191937, 0.5960445219278877, 0.08707847497703251, 0.7097507238867523, 0.31268775436941254, 0.45128228867031683, 0.8618718404305261, 0.38702310532162115, 0.7136305704510258, 0.7507510814716115, 0.813152571526006, 0.14770651085707898, 0.3693213112610136, 0.32830806850274696, 0.07937432230068522, 0.1679143177108829, 0.6932556156954821, 0.15600345238070312, 0.5234320901775438, 1.0337296698230969, 0.6765254438018764, 0.3903011261508216, 0.21755249341371644, 0.20354390401494005, 0.6318225606185789, 0.41062325718510867, 0.5442832111157428]], [[1.7140482279468598, 0.98866940967896, 0.24463106395446388, 0.9406943592509511, 1.3055498170474722, 0.7073106389346393, 0.37960067421282506, 0.3354127068555162, 0.8428033657390346, 0.5873864572772309, 0.09390383672438082, 0.7073832461031878, 0.4463963782182867, 0.43218744090593486, 0.8718101785060643, 0.43664076238723554, 0.7193646161113962, 0.7691727146146446, 0.780910130177576, 0.1520961622948477, 0.3591960421730737, 0.30631698351620745, 0.08316871673573754, 0.19748431612158102, 0.6592822272377844, 0.176632703733994, 0.55362613795354, 1.093947743701906, 0.7294660066764442, 0.4067049965889735, 0.3505726904321629, 0.26753108652799373, 0.627681510761158, 0.45228598071982473, 0.5836262337259123]], [[1.55284444171865, 0.9971499884937596, 0.21655605855632443, 1.1240296489885009, 1.3612211351943846, 0.9222040818120003, 0.44109689087950665, 0.30748938018663, 0.8286822888702292, 0.5948010473464251, 0.08244420093587985, 0.5968574081687338, 0.496827526955282, 0.4133992734066292, 0.8397162355210142, 0.325980509890559, 0.8022042989453702, 0.7598851496655946, 0.7409402590478947, 0.16778622134467572, 0.3569874781090434, 0.307784409904588, 0.09432780753843822, 0.23039848119173256, 0.6467475368248481, 0.1547893777611476, 0.5603635272444145, 1.2222785531587061, 0.7140881220694442, 0.4525579902329378, 0.4712801672917171, 0.37801788374366824, 0.5873486388635483, 0.4528025988145742, 0.6400212333091098]], [[1.4457220441660708, 1.11852193684806, 0.2012087531059188, 1.0458839218682523, 1.5049692234528336, 0.8246892937153294, 0.4444106749557833, 0.37824972767160137, 0.7868516357526733, 0.6290050347879634, 0.07305638598789585, 0.520080335665364, 0.48841584161877927, 0.3869280144937889, 1.015190576622388, 0.31685471325024694, 0.861289206367368, 0.7337773296502498, 0.7400318498713172, 0.17188702208776135, 0.26586154104957266, 0.3080646845543973, 0.09853804466533617, 0.23265940351526404, 0.5904612860595576, 0.15536667365233972, 0.5672874175881559, 1.297596575704491, 0.7303888925021901, 0.4860450210091387, 0.5269757220769649, 0.4693563999203826, 0.6105993755503751, 0.3846428360155657, 0.6600678592549425]], [[1.4739786579429865, 1.1887740426706253, 0.2429966868437145, 1.0915967010770031, 1.5860234826154131, 0.8052486425721352, 0.45523521554274804, 0.4386328288398277, 0.8227028164184524, 0.6302682275763175, 0.06593030557417655, 0.5984857617149029, 0.4886129907171056, 0.4524942043479896, 1.0972108390619686, 0.3265381901783876, 0.9552984764953828, 0.724408588388329, 0.7221821527181018, 0.1709021915190285, 0.28447625166333007, 0.30699473211363615, 0.09970069891940783, 0.2381761972074673, 0.5883234606130602, 0.12037954407611702, 0.5696859233493545, 1.3395068194605513, 0.6699381837314171, 0.52144330667864, 0.5489796383992338, 0.4474642599877731, 0.5888901767735555, 0.34542677692735807, 0.693532285471619]], [[1.3816306339669886, 1.2165773576908867, 0.26990482906245766, 1.1847982666609898, 1.6330924617065647, 0.7918889685270916, 0.4641713955250757, 0.45529854440390183, 0.810042575703056, 0.553576229520495, 0.06029854170445819, 0.6511205048488689, 0.5305624010583875, 0.456918420071469, 0.8214593195861535, 0.3566729115855325, 0.7678446612093347, 0.7544646846370615, 0.7221856543015509, 0.15373679120261632, 0.2505970943208034, 0.30344853243985476, 0.0998726869089182, 0.2453462960444377, 0.5225119592514101, 0.1141778801574536, 0.5177166991393671, 1.3083434514715933, 0.6996997091635733, 0.5975526729624225, 0.5689560087585761, 0.4039866566346338, 0.5544328226415655, 0.36684330652997954, 0.6215874727329402]], [[1.1803055334478847, 1.1774264694131074, 0.2704325747352601, 1.316733581584173, 1.6412507819793967, 0.8385244110540406, 0.502526069780359, 0.504288140494584, 0.7125689553541262, 0.4407375686856249, 0.048770335677425065, 0.6251246540397798, 0.5756814415309794, 0.45544086070868744, 0.8635143866549543, 0.34869464038778986, 0.8209919762048371, 0.7536917185745343, 0.711575960226535, 0.13758109585514613, 0.27583046436400477, 0.2932131585058946, 0.10111061870542482, 0.23557321755460692, 0.582568906350801, 0.14200786926731243, 0.4956614292382875, 1.269644433621104, 0.7073166041095805, 0.6213776166524994, 0.5993954594887008, 0.39115621396966155, 0.5017111973445402, 0.45723697497787774, 0.46527408310262075]], [[1.1256924790337133, 1.1872193398385917, 0.2695458342318961, 1.423860986667956, 1.5699769624857711, 0.9726572425153259, 0.5230072760161881, 0.5685624826725149, 0.6295838348176677, 0.3927822667687218, 0.05692388408634695, 0.6416633858591316, 0.741498575947964, 0.44129729062045464, 0.6125942501664823, 0.3442467535252879, 0.7677550565102034, 0.7529179711683519, 0.7062874674347784, 0.10478396019775701, 0.25580896388052754, 0.1728232590906518, 0.11189739501940921, 0.21342721811980814, 0.6449140698951841, 0.17789709703892154, 0.4708672904264496, 1.2368593468450402, 0.7377870495191908, 0.5979590644456838, 0.6170879850338168, 0.45233072158795035, 0.42089193491706783, 0.5287657174612999, 0.4695050071189415]], [[1.0967639937967224, 1.21806230121735, 0.27362965377705023, 1.5431678616443048, 1.549198008847967, 0.9572335662407465, 0.5077318948054518, 0.6773983200431508, 0.6607161195074853, 0.4742605368725227, 0.04495634763557691, 0.5958900462137783, 0.7023578330007226, 0.44222931905689, 0.6220313292213591, 0.3226291674319035, 0.6868577076008782, 0.7468311246359023, 0.6836359449296703, 0.09629639903294707, 0.255156957251379, 0.1338389055553554, 0.10649347830841238, 0.19218485941729246, 0.6794270236213087, 0.1968909588605154, 0.4104232887112514, 1.122811843190462, 0.7935127956125465, 0.6143985551744412, 0.628231373446929, 0.54271791007405, 0.3621785758258955, 0.6042146586477007, 0.47055051913169377]], [[1.0666950419590457, 1.2924273503032149, 0.2712703262791854, 1.787469779397193, 1.550138954577128, 0.9299981660444755, 0.49450290776542233, 0.7465252427502118, 0.7400917746520256, 0.5182775709960833, 0.042110681803378576, 0.617635360418116, 0.6306381351031004, 0.4404157576223845, 0.6160808636554194, 0.32342258921245937, 0.700766118115028, 0.733035980678553, 0.6733663810616143, 0.09718748144977254, 0.25640773129225547, 0.11412359332573072, 0.09862292590848923, 0.16474674763803165, 0.7111899167979607, 0.196694402299467, 0.33858257648557777, 0.9895274461307668, 0.8646901297827841, 0.6386036267568347, 0.6515750980058984, 0.5895464167651767, 0.32030106833507627, 0.7215430442603084, 0.47094114609267235]], [[1.2107983140803427, 1.3503905888536656, 0.32666930566020524, 1.7887864443394108, 1.5618979380361264, 0.9047873257139452, 0.484178684302739, 0.882675338035261, 0.7457737646881205, 0.4276614223800579, 0.04229857280778493, 0.6746419115763518, 0.7197573739759072, 0.5342356070861343, 0.715090753673451, 0.3268610568257718, 0.8105383679206336, 0.6883543983110788, 0.6577405649987645, 0.12459503644765035, 0.21650507425404836, 0.17156075261043274, 0.09213558783921554, 0.1393794229774482, 0.7764858326153228, 0.1747410222321824, 0.28344108294562553, 0.9206587012426835, 0.9232241109914561, 0.6473431926895614, 0.6701399002692847, 0.5539117759097394, 0.3101813723514238, 0.891676727856886, 0.4728566944729937]], [[1.2778215724857414, 1.3938545046353714, 0.3207304535201704, 1.7884506609565163, 1.5376870860018745, 0.9413378556597055, 0.49739545789479545, 0.87462105606338, 0.9050065247126402, 0.4063957758863828, 0.04390578907257894, 0.7235571223667103, 0.8062152323256468, 0.5715864988632794, 0.6687157399963994, 0.3440873665615336, 0.6561962990755035, 0.6223924509357266, 0.682436307576088, 0.10407213344076238, 0.17478727741165012, 0.1691727698010174, 0.10449673420509537, 0.13124407920480965, 0.8664808206327025, 0.15034531668955248, 0.2946853021765865, 0.9516102383566084, 0.9797521849616331, 0.6681614313885028, 0.7097020427513755, 0.5466187332396881, 0.3883449379797677, 0.8678395695299024, 0.4871220179664576]], [[1.3396900064017851, 1.3547393380314576, 0.2898957609845508, 1.7347202128125574, 1.4765410178061957, 0.9698322091793883, 0.5055221156363165, 0.97802616996245, 0.9722415214505999, 0.4624205961005747, 0.04243046998465669, 0.6712698950201312, 1.0906291722780388, 0.6157232575338634, 0.5734429176122395, 0.37231294043041047, 0.6220867065979394, 0.6883709065310988, 0.8034701021967271, 0.14076023364603235, 0.17225445957795915, 0.21909397900326194, 0.09952547348093668, 0.26323070169491464, 0.894739395518724, 0.1234021989931088, 0.49319932768787045, 0.939176958369641, 1.0052767326874033, 0.6754476231958079, 0.7261326079690407, 0.5411079840100803, 0.4250837345906987, 0.875042995296738, 0.4956803324021324]], [[1.406262987998955, 1.2211495127208911, 0.33083209179194817, 1.6133669650506663, 1.3918985533122283, 0.9534370022034666, 0.5143185045641034, 0.858677233632009, 1.054915314114341, 0.7210756779234322, 0.032751223804404886, 0.6826490789275229, 1.1776963417463202, 0.6317887402831269, 0.6321219424891613, 0.38138840298026205, 0.625666869524665, 0.7108689799213735, 0.8964058240172458, 0.13384964488615944, 0.16491059906250471, 0.2117563651254778, 0.10731548750324366, 0.528942954516688, 0.9052418923947424, 0.1009286065864855, 0.7441358361368353, 0.8203029265957742, 1.0058624109103231, 0.7132371186502833, 0.7453290839480219, 0.5967621805121299, 0.36838904882879375, 0.8895156665121866, 0.45170446199564185]], [[1.4125213757213984, 1.1875491293877896, 0.3402719775424723, 1.6100747484870912, 1.275243078840744, 0.8735510564863609, 0.5126970020136528, 0.8113351595470091, 1.0240945844369103, 0.7683522414777078, 0.04027200192260946, 0.6847519120361634, 1.1107117707667156, 0.6099835356529041, 0.45429274795111174, 0.3800492313727193, 0.6107770785951154, 0.7467933897382808, 1.0477546626914918, 0.14669980416866885, 0.18969718163846516, 0.2244918635791089, 0.11760524185373197, 0.576864969618172, 1.03976289331405, 0.17550205775985697, 0.6908396418184517, 0.6387293619465504, 1.0045281758714657, 0.6869776938972271, 0.7505245077557694, 0.6467284412628944, 0.31847067424026765, 0.8922134526801497, 0.47285166344709817]], [[1.4897206631769406, 1.1623299265970664, 0.33368279038329396, 1.5145158935377085, 1.201020843729506, 0.8435138168994649, 0.5217203476331767, 0.7875908676229215, 0.9490299870814303, 0.6664931599780454, 0.05409396218250243, 0.6764396535717518, 1.1134734495998226, 0.5948033470578422, 0.4499336965952808, 0.3855335179829326, 0.5584372377206943, 0.7833174781078358, 1.1568999649956146, 0.1825184958823064, 0.19612220541414757, 0.23059950662008438, 0.11502724999016817, 0.6671829318173705, 1.1481405006667673, 0.3086301512973999, 0.6708402683905532, 0.5673412729314892, 1.007034655557569, 0.6661912558721779, 0.7635980821602096, 0.5678688926355104, 0.26882361391838716, 0.8735797583320809, 0.46166189133033475]], [[1.5226409297579924, 1.1214278710609678, 0.33667919081124337, 1.3814227815798488, 1.1714175886345868, 0.8792747837275923, 0.570494103868474, 0.675398815124019, 0.8626830503052698, 0.4957003473203102, 0.16445944150479244, 0.6670351042220042, 1.177888446748037, 0.5828849646254592, 0.44503468869063284, 0.3761825409708537, 0.5352036554645978, 0.7726517988485294, 1.1943007297193904, 0.2274376305221412, 0.20043408087366485, 0.26879561973114885, 0.05165455638448481, 0.7170318507292088, 1.184506689139544, 0.3321423880752622, 0.7204275082383086, 0.5420680812056402, 1.0058848464777976, 0.637490589351897, 0.8044336762574367, 0.5605541049753364, 0.20079164674008743, 0.8633947240561766, 0.4417046591557672]], [[1.7077017984885743, 1.3734221446546775, 0.3019349014784678, 1.429381064725273, 1.6334079128278853, 0.8675326903435197, 0.6001437953843596, 0.6879734926958478, 1.0109763829455647, 0.4271058648841006, 0.11156060118438185, 0.6700691172494793, 1.1734119763776565, 0.5638140864210457, 0.42917348611180584, 0.3643411615558094, 0.5468045852283467, 0.782537068534786, 1.0951701768312916, 0.2590955598774099, 0.19253003147290848, 0.3093288170851758, 0.06371738901327034, 0.7937406059476613, 1.2242324125002275, 0.36526635163899135, 0.7378463696895214, 0.6436211180336744, 0.9066705855684081, 0.684596877819027, 0.8447973035444793, 0.5276193572233652, 0.22399331599299374, 0.8523740650078688, 0.4496834933501467]], [[1.6383727101928307, 1.0697963917759499, 0.3018861648020986, 1.1859068517160134, 1.6630398504330142, 0.8764141008534284, 0.5253207721434108, 0.9102670322943505, 0.8083290345862604, 0.44238307781833797, 0.1422827140489723, 0.6932550928262946, 1.1671926140893973, 0.5683243861663159, 0.44292020446672614, 0.3703257397916206, 0.52892675918635, 0.7641111385321071, 1.0562429198583883, 0.23296521665132852, 0.17485287002311514, 0.3500863106949163, 0.06187580945698766, 0.8541807751280015, 1.3409718063093221, 0.36277796036408977, 0.7593234116249374, 0.6880456148708567, 0.8843929933415691, 0.7286521326281725, 0.8512747495023358, 0.46250037696315577, 0.2215727524623747, 0.8815373407044287, 0.5728922785959254]], [[1.6315586868503842, 0.9000126272389862, 0.3420098271663864, 1.1981848144243872, 1.5120410210820707, 0.8509971486052299, 0.6140059559233235, 0.9535054715224199, 0.601388554488218, 0.47963010107076387, 0.17620275169009159, 0.6550611655063818, 1.121999535642772, 0.5704302921775171, 0.43969581555141746, 0.3756958572230007, 0.5221964720313272, 0.7683802514971385, 1.0466145247582386, 0.22249742891596444, 0.16917247497440724, 0.3148728363269719, 0.08094721703535335, 0.8303136923371026, 1.4526502812781894, 0.3122157164117027, 0.7453440442582057, 0.8654167040437062, 0.9051309268078289, 0.764088750189158, 0.880223731325586, 0.41106062545757727, 0.21146611850652555, 0.8938835437278376, 0.5757288741283342]], [[1.5928047129868155, 0.8493084755146593, 0.33587278501576023, 1.2437503780550105, 1.3968415916111017, 0.7941420974413831, 0.5328567136350432, 1.003063307271948, 0.49817940784544723, 0.5282639071686781, 0.23647231899090204, 0.6770934205890575, 1.0402061328654144, 0.5420397368881388, 0.40701295490766354, 0.37358474753054993, 0.5245199043440734, 0.7585800516951664, 1.0771125845301572, 0.21503316934991196, 0.18309452076859845, 0.27893555837168627, 0.11758099271335509, 0.8016824671976186, 1.454956381232035, 0.2756836982745883, 0.7911189707423972, 1.0206850906817642, 0.9989721932901332, 0.7961235458105969, 0.9105627227275741, 0.4815598834824262, 0.24484456115020595, 0.9018053532808201, 0.6412955094671285]], [[1.6093715509695388, 0.8561210127344382, 0.307927726840287, 1.2830996754645365, 1.1092051928306526, 0.8119722764127986, 0.5493100809197333, 1.0828107297315412, 0.47317905507416336, 0.5642317583888781, 0.26899406725599173, 0.6421722823170934, 1.047219425722804, 0.5549161430300606, 0.39759184454990115, 0.41055627814346374, 0.5259756123405392, 0.7586542081570633, 1.0212264618709128, 0.22378438883961138, 0.17707268338618914, 0.18547484863746552, 0.05759770480811416, 0.7567441944854612, 1.530226099673305, 0.25233893698684473, 0.7834062504846618, 1.160440449435006, 1.1001195823867196, 0.7805767497718705, 0.9040944806339368, 0.5527560112331011, 0.3303545997291874, 0.9131740292322308, 0.6510855344847701]], [[1.6758170209484775, 0.8581463369063747, 0.273375542673431, 1.2651211119062642, 0.9645525584738077, 0.7922464258597315, 0.49992884268474813, 1.0763587518044186, 0.5779783978838403, 0.5982094514825619, 0.34886611075507806, 0.6530387560392417, 1.0798875269318466, 0.5548957526348628, 0.383234623040104, 0.4224017887774125, 0.4883609517699733, 0.7317744592215716, 1.0185396011821368, 0.22437849327868176, 0.1902694493285224, 0.1547288507063253, 0.08187804607936348, 0.8129905314367822, 1.6267009480420196, 0.39889132512356984, 0.8171702968059084, 1.2274571439881472, 1.290545719660588, 0.7345214768858598, 0.8781518383249238, 0.5540558721649055, 0.37336782420351755, 0.9020130481988367, 0.7090672686227]], [[1.6438218806470601, 0.8681997590078743, 0.24922106596472937, 1.3473225831537445, 0.9291028396581724, 0.7295386741854349, 0.4443814139004232, 1.0461287653099258, 0.751956287189153, 0.6474288362972086, 0.17661521657910423, 0.6217232221672573, 1.1059945944176062, 0.5722761609027633, 0.40323838073997087, 0.4670101782033321, 0.4703086362682934, 0.7507135511875379, 0.9726016821681263, 0.2613246099352725, 0.1769858709398717, 0.12371692076294191, 0.08445122328398776, 0.8576603351826019, 1.5853907422665774, 0.5336338471138034, 0.8302286529915159, 1.245451835292851, 1.483542490339881, 0.6088971775691936, 0.8575736860744655, 0.5673335801174847, 0.5126424986222724, 0.9010056783254625, 0.7929776138365147]], [[1.6087703643031979, 0.8134551588778626, 0.23719290140322374, 1.4144341926774073, 0.8813313647416223, 0.7142711874930727, 0.42114284098934807, 1.0059540502197484, 0.8194364518564501, 0.6610919300268568, 0.14874782812546541, 0.6118630086115223, 1.07617549182994, 0.5657238002173852, 0.397411941691246, 0.5061468421465147, 0.5015627962451638, 0.7292551800936071, 0.9263903869345012, 0.2821252882576861, 0.26564127987155745, 0.1348621250994989, 0.060513188253974816, 0.8556966621140745, 1.4984477844800708, 0.6314451660725366, 0.8730652694214361, 1.2306594832424809, 1.5599080972888115, 0.5651837280757137, 0.8547667270342113, 0.6235977259247236, 0.6737855235044528, 0.7723271947356902, 0.8514380254949194]], [[1.5565281341062058, 0.8284165247338182, 0.25003374005034434, 1.3869273109303513, 0.8026255312966519, 0.7212838181337415, 0.4024982178966034, 0.9722134587934995, 0.8825591561879161, 0.6384562549699632, 0.1704031850873064, 0.5935460042906515, 1.0894742700794902, 0.5733175315278727, 0.36859160237440425, 0.5114829044118661, 0.5339893536791547, 0.7707622330356969, 0.9180216814898358, 0.27235250234972136, 0.33934566817240464, 0.22737005172535457, 0.04072370637538397, 0.8006926632080203, 1.3659279264535293, 0.7494539908584676, 0.8022340441277656, 1.2108430932592782, 1.569775340278799, 0.47613715071557355, 0.9544417297496646, 0.7281102221387687, 0.7120816449688725, 0.7193930725401227, 0.998595462850753]], [[1.5980427035553493, 0.8164342240890846, 0.2709144304807523, 1.2481926058086903, 0.7330378909358035, 0.7434509687519474, 0.39992670281709797, 0.9798297225463897, 0.9256667007412642, 0.6184092366654302, 0.16425869494547762, 0.5721994121905343, 1.060497866039622, 0.5743833883839644, 0.35404854798137386, 0.5260913112529089, 0.5423199097330208, 0.7746227163426918, 0.8917960948666197, 0.26758852305066577, 0.3086200074226123, 0.3216074250990625, 0.0295848678032411, 0.7636381965401953, 1.3107465381540846, 0.7745216274768478, 0.8063381196618065, 1.2025982380435385, 1.6653030914139608, 0.5597547236841205, 0.921111344357319, 0.8194675650980552, 0.7281011739335831, 0.8905549224303762, 1.1005618132724035]], [[1.5495454121735213, 0.8392201854978303, 0.27563775684207614, 1.0686229303852977, 0.7765814652977828, 0.7724792453211875, 0.41254595253327625, 0.9692591344417691, 0.9906053642464229, 0.6650023576495382, 0.1369441414524032, 0.565952735482715, 1.0321569177042844, 0.5615482554628262, 0.3302505012631853, 0.5413325702248726, 0.5035222408591786, 0.8208483992411322, 0.8966541359239533, 0.20928579025119795, 0.2536755531134143, 0.3963148568258827, 0.02904243772339907, 0.7662541169469791, 1.404728948702938, 0.8079713222641849, 0.6866460608288385, 1.044016297561024, 1.8923374566964735, 0.7373559888291972, 0.8709472827054612, 0.8525156160004556, 0.7423928093233918, 1.0110268456878184, 1.074039269356413]], [[1.4930482585632467, 0.7822770451596667, 0.28628890077274866, 0.9152962863305152, 0.820494716420452, 0.8023928380495036, 0.4011177982960633, 1.0259069469616457, 0.9913830583199996, 0.7927725778075994, 0.11985554422633579, 0.5382819894904731, 1.0056143497151033, 0.5866299364754208, 0.3141941299680305, 0.544418008104198, 0.5167153033121125, 0.8052209334394397, 0.9123630328613239, 0.18547280143945774, 0.2588707135528089, 0.5124818383666393, 0.03199775051950722, 0.7718188564510036, 1.5545618360758788, 0.8763026000811218, 0.6206651497313975, 0.8817923282173191, 2.1634624301285514, 0.8422760651652759, 0.8319823716012038, 0.8289160095462539, 0.7824497831003767, 1.0237198431886414, 1.0823734428968252]], [[1.523753509757205, 0.7943214549995543, 0.2990678117268343, 0.7856953885390772, 0.8378640938673979, 0.819442650967249, 0.40503509923004055, 1.0469403578022378, 1.0721759358438891, 0.8605047746290614, 0.1181382302672721, 0.5491929533307707, 1.007868959023192, 0.6005333780426418, 0.298227331016523, 0.5715383571555218, 0.4864361029286932, 0.8388386958529233, 0.9412992176988509, 0.1827316605845434, 0.23042115480282024, 0.6346409088196049, 0.052261534707151426, 0.8307643379857372, 1.6850578221448522, 0.9414069894971757, 0.5377581863177995, 0.8263467313919546, 2.328900825055477, 0.9158124294251586, 0.8285931500547145, 0.817703783409019, 0.8659531293660359, 1.0246290083762897, 1.1650325395207586]], [[1.546050560761715, 0.8714335289682998, 0.31871850876659885, 0.8372202372186569, 0.836808776448728, 0.7732408806226398, 0.37560025359465443, 1.117750809650433, 1.1087936462555734, 0.9087256678792915, 0.11363027401750922, 0.5700779198161782, 1.032956936853585, 0.6158236747982629, 0.3003616066473054, 0.5902679712762924, 0.4800000846042147, 0.8155829556153176, 0.9919950448371337, 0.1610254819670987, 0.14350011017978642, 0.5107352791397284, 0.059107679995198836, 0.8752122905614778, 1.716999695753215, 0.8089551378552645, 0.41731037830430917, 0.8170321375674093, 2.302332218827816, 0.9910466923861097, 0.8059913575917692, 0.790135277472963, 0.9303387666985217, 0.901899088774786, 1.1158824968788061]], [[1.5621740952903602, 0.8717763080336085, 0.3316677675196568, 0.8890076650430145, 0.9007798135772505, 0.7538838405824189, 0.3716152900895109, 1.1483453743707637, 1.116485219643899, 0.9648933375596853, 0.08634593956336953, 0.5892625191944872, 1.0945506692636138, 0.6282601212789904, 0.3324097340287917, 0.6159847544942849, 0.4980475430573057, 0.8413256379426621, 1.0848833949194745, 0.1710836851371842, 0.14183538238304685, 0.49681599164914314, 0.03803376517283736, 0.915761597676351, 1.7799713202367045, 0.7119790121156496, 0.29533545636273456, 0.8334076040439853, 2.2287875895227645, 1.0104490143781313, 0.8193805157203982, 0.7724450752965497, 0.9583700472510258, 0.7599297270859473, 1.0444916561512239]], [[1.5316951707563076, 0.8079719118656713, 0.2958929045613337, 0.9303217667724973, 0.8913052279213158, 0.7527938196859996, 0.3897834635132426, 1.1209205187959848, 1.192423838476798, 1.092900850969643, 0.08815473590966241, 0.6232237076892059, 0.958513405678816, 0.6181694932810838, 0.3362243623453448, 0.5456258702250274, 0.5227618840559183, 0.8051898775051602, 1.183810770708138, 0.1883740091568016, 0.14246492076567763, 0.410825323067589, 0.05500783056316563, 0.9484133613889605, 1.715924608031181, 0.6458304309250378, 0.31831372604278435, 0.8626434854345197, 2.107904825538231, 1.0452374656214714, 0.8687948054825098, 0.6761083272668292, 0.9497671465155555, 0.6636291798849416, 0.9594465383067367]], [[1.4227799606163005, 0.9256439005328305, 0.32757793812995434, 0.9549067691243951, 0.9183346706416218, 0.8375187923520688, 0.38980170586349067, 1.0917826859800774, 1.15957843469175, 1.2594933693927515, 0.08679706807060372, 0.6420158550208703, 0.9292263774311249, 0.6079754611677062, 0.37536961167696437, 0.4936438157144606, 0.5346984144824384, 0.8467822925748518, 1.3228455342867356, 0.2023744524962363, 0.1443156681975976, 0.3645285867136817, 0.060605895342638876, 0.9763526505169964, 1.6916000641224807, 0.5580824898738783, 0.5309519622576152, 0.8705778538199878, 2.070019866097952, 1.103868644296211, 0.8845505128286204, 0.5681972668679178, 0.9881709321458609, 0.5629292303048198, 0.9038314443738451]], [[1.3349404879951083, 0.9710675974874687, 0.3506341347431308, 0.9708232075343696, 0.9290787387011181, 0.8446237930899367, 0.388339063859148, 1.0020362637119558, 1.0988113477248542, 1.2767241789972932, 0.0664919838624883, 0.6423028970066718, 0.9398066805370522, 0.6242600986757934, 0.3644741468313071, 0.47977138807094055, 0.543731721688993, 0.8814305836093416, 1.3339444151150672, 0.2269270220557119, 0.14602172711216785, 0.3214571139481821, 0.06271413299240716, 0.901054372190522, 1.6870207839489575, 0.5352296817522126, 0.5730963100525382, 0.9058799076190844, 2.077202986392222, 1.1374671362441129, 0.9006542523254931, 0.47221604129312933, 1.005224324789187, 0.49292436402262485, 0.8106175148618678]], [[1.167926496397501, 0.9926527599049445, 0.3382977629585356, 1.0090028617082203, 0.832016239037456, 0.8332886613096393, 0.3771398233412268, 0.9600869955643314, 1.001979462182848, 1.2792168715108403, 0.06383153525108079, 0.6018437951760284, 0.9331653687323912, 0.6577897399769446, 0.3400018586512138, 0.5283631962916822, 0.5674338678141677, 0.8582828574494983, 1.2644055563159342, 0.26546251524809367, 0.16081333666264777, 0.2984073791832677, 0.061699213129708726, 0.8490405707246076, 1.6446028617575261, 0.5668980829828345, 0.5820203563445698, 0.8949152442974202, 2.049363602355391, 1.1400469949512642, 0.9072653120610723, 0.43345773513082403, 0.9642878086100239, 0.4797111434275844, 0.8006252592302928]], [[1.002106493405626, 0.941989907317874, 0.33033319847001913, 1.0252856297929438, 0.7556393214545196, 0.8324408112368853, 0.3915934972212055, 0.9333345447086312, 0.9145948473210006, 1.3133317932212163, 0.06378518964988271, 0.5971015323019662, 0.8690694331458281, 0.6576571722940019, 0.33768968979717906, 0.5235702394376474, 0.5295883492159403, 0.7655290517762324, 1.2468122956109797, 0.24261492366374, 0.16422946924823026, 0.28572804727840206, 0.07241560897264505, 0.7909146500179158, 1.6439222833949643, 0.599981887538846, 0.5752396822393255, 0.9339295220589501, 1.997769828629519, 1.1183139240293036, 0.9343708400147739, 0.41078961058740066, 0.958396912556704, 0.4637926506477001, 0.8819888192045324]], [[0.8593948488128866, 0.8457701907308869, 0.3110891176051381, 1.0552259433270166, 0.6229387793428018, 0.8153960883855141, 0.4031176009100456, 0.8569004173386441, 0.7360029542661164, 1.3831916271960973, 0.12011807123834309, 0.6661049920768112, 0.9276219234164671, 0.7055178704267431, 0.41981733897029894, 0.5450991665748479, 0.5352143477257, 0.6936319435225495, 1.1303656803532416, 0.31718405344412, 0.18114719387467634, 0.2611488646917452, 0.11047795894839312, 0.7241911397771749, 1.6428237817786906, 0.5885725983933009, 0.5276628788487608, 0.9185724903544225, 1.8776180724196634, 1.108257034735671, 0.9522809736558364, 0.33210741363528595, 0.9143667658372052, 0.4158790623363599, 0.9505496927255774]], [[0.8886270660222458, 0.978830356918974, 0.33131934189709294, 1.0444507324206205, 0.4149037065741301, 0.7589011974840222, 0.38672477714593095, 0.8862327711796153, 0.4470604318429249, 1.3515277337114817, 0.14720042891104124, 0.7150304719652879, 0.8006019155244497, 0.6807432755346718, 0.44599478165199813, 0.5256698108290291, 0.5582978254787494, 0.7369598614051072, 1.0204878574422178, 0.3445246083451898, 0.20099302780160977, 0.2854542658037933, 0.06665482731651312, 0.6868244954218887, 1.6354713567858692, 0.6520023933367308, 0.4128090213392254, 0.8389429267397541, 1.874002158827711, 1.1074256668362934, 0.9601610391033645, 0.2878584885737457, 0.9550355203169243, 0.4573080924690918, 0.9383870382563643]], [[1.188885870052478, 1.0448489235511758, 0.2938866700648027, 0.9916252442129143, 0.40264883703730087, 0.7219228241563982, 0.37889994349669465, 0.9711346500568562, 0.4424952923780257, 1.4335447229957525, 0.1748738220739237, 0.7702555556493734, 0.7498879383925466, 0.6797317618386869, 0.40523587395710303, 0.5079612891419727, 0.5522806010953876, 0.7463137668003061, 1.1179446111665814, 0.3314764172144958, 0.22488995401417577, 0.2757477358703385, 0.05460544525145108, 0.6598782189086833, 1.6722408526702177, 0.6575959373581809, 0.42332857537238033, 0.7849466908764662, 1.9248318295737867, 1.166021894312121, 0.9507793812745651, 0.27050467505435366, 0.9237449283436137, 0.5371536632742472, 0.8663878283731021]], [[1.2711669626148225, 1.1648190622134962, 0.31080260387471786, 1.0420136777644817, 0.43335841850692725, 0.770701951556302, 0.37764425107148214, 1.0223234102629686, 0.5436368639975384, 1.4570241944677136, 0.21064503774715754, 0.7584500684602982, 0.6823920771335972, 0.6219366714894785, 0.3870301652124437, 0.4885271281668687, 0.5475179033358175, 0.7030474946748384, 1.2446620477497619, 0.2680422238188288, 0.2782760748971089, 0.28847310670700266, 0.08020328920624613, 0.620729575768962, 1.7053279184902472, 0.6777530461492429, 0.4562522311956904, 0.8531331394215794, 1.9055587608549829, 1.1863794140821504, 0.9472064761971195, 0.34697857025178874, 0.901121438952561, 0.6119642138110334, 0.8157348770503299]], [[1.2280620116764076, 1.191328637191495, 0.29828666007440535, 1.0184490732539293, 0.4975734213656967, 0.8643271705211195, 0.38506630220217447, 1.002199072409141, 0.6935079280647658, 1.4296911904910377, 0.20509737713683424, 0.7542187565082346, 0.5788617747672231, 0.5748951385620426, 0.38602717242716084, 0.431145471725361, 0.5266481326630534, 0.7376326409480816, 1.1499729062185675, 0.2386259710189332, 0.2762642639627956, 0.2762071934615466, 0.09563854149998899, 0.5213431673397585, 1.6862260176020896, 0.6963722383628483, 0.46326395784356095, 0.8597505705893234, 1.8564273520090495, 1.1858430295126567, 0.9704096443584093, 0.4040843971058341, 0.7948589928907577, 0.6125631673021266, 0.799759478965548]], [[1.1759845161305078, 1.2021598184941944, 0.3022882761008396, 0.9701077436361549, 0.5462869515281292, 1.013494209135386, 0.4060958041518366, 0.9243037988116007, 0.7588970517553022, 1.4221750212016773, 0.20692397012156, 0.7075022962273464, 0.5363417621043896, 0.544145936886572, 0.4008946014923585, 0.40942322935813386, 0.5353370955357336, 0.7678700851478549, 1.1615461351487848, 0.24109743322520016, 0.24705893962636563, 0.3245765076470159, 0.13639857792481996, 0.46983332773452424, 1.606829944650552, 0.8498509691520507, 0.48061345259422283, 0.8205093705646184, 1.8545502713825868, 1.2191643673818746, 0.9871678711767539, 0.433649651573072, 0.6785398057175203, 0.6571769033977409, 0.8042406004949496]], [[1.1920426678396279, 1.1897347502661184, 0.2801391464663891, 0.8769124762199629, 0.5695299101382109, 1.0160538151278793, 0.41080404712272284, 0.912897556336177, 0.7895160336957393, 1.4205160119349824, 0.21502786184762046, 0.8306743546876282, 0.5224177873922445, 0.5009620658252528, 0.4512421154372522, 0.3597520474818116, 0.5812783460449805, 0.7602932720334379, 1.1858260560605849, 0.34938588488296746, 0.20969507008385543, 0.42528083236391395, 0.1585750109976012, 0.4548244162607679, 1.4876987672177018, 0.9219847906184222, 0.5159311447735112, 0.7982936091679645, 1.8454881722705931, 1.201191920908131, 0.9844613417956454, 0.46645947440519664, 0.6049665119527976, 0.7031356177491661, 0.7629996018604148]], [[1.123637194704143, 1.2218129177926673, 0.28290262682690476, 0.8068551976217107, 0.6045552151292626, 0.9925224034839665, 0.40781584058019443, 0.973772565625968, 0.8710701603830142, 1.4593311697959876, 0.22105405007658058, 0.7832047665077178, 0.5519140006219471, 0.6530821787466241, 0.4568214539956154, 0.36907126559363995, 0.6241109138744866, 0.7509676313366748, 1.1009580884009693, 0.4425202970930654, 0.20012733859304624, 0.44031350375316697, 0.2118262126741513, 0.5250095556955102, 1.3651824861519075, 0.9353912876463686, 0.5735496949734986, 0.7311114281761919, 1.8290962421549577, 1.1648092000778372, 0.9286859513885882, 0.5372492484489036, 0.5779178699356715, 0.740310905425676, 0.7384936955087062]], [[1.0892425091155946, 1.2295155603391679, 0.2843992712378281, 0.726820235803889, 0.6082174446112849, 0.9950674129514097, 0.4295554763397094, 1.05454889294275, 0.9373959228501638, 1.3989949296280664, 0.21202025309601435, 0.8698889284431908, 0.6241466118271488, 0.6320496306518212, 0.44462326927634765, 0.36385933829982287, 0.6151426114205583, 0.7126639999729385, 1.0848617733100385, 0.4077663205204128, 0.19214150711872235, 0.3898520669587721, 0.19772246766596363, 0.5626344555546983, 1.3212832582617624, 0.9612671031419413, 0.6704015107825163, 0.7750406078254299, 1.7668358041558716, 1.0709484712847663, 0.8616629307085923, 0.5831879105070004, 0.48958308761583225, 0.7635932131602041, 0.7284285539976475]], [[1.1023888533227955, 1.187424601922294, 0.2888482111655543, 0.7165639513760971, 0.6216061881591204, 0.9687422558474434, 0.39610754859430786, 1.1870770264128017, 1.084190782017336, 1.423692547548272, 0.2249809514887885, 0.8675054861244249, 0.7256811458384544, 0.5569415200196309, 0.4600202814949352, 0.3768092018571799, 0.6764877387269715, 0.712266178356047, 1.0889329283432336, 0.3487340791628668, 0.18327520720290263, 0.2827791927855033, 0.15982616232903554, 0.5834943762398338, 1.269724761608433, 1.0593388616932204, 0.7701875765423403, 0.7691304102830072, 1.6572209449897808, 1.006072805548413, 0.8317059686504189, 0.6927576104170217, 0.4040302156383062, 0.7681981724403883, 0.7347611332874899]], [[1.1988669356266741, 1.3503170684882952, 0.31315550898934646, 0.7602112041755577, 0.6049043770340948, 0.9279688406857103, 0.3931818440845021, 1.3202667125468575, 1.1782834884665268, 1.4075120621024673, 0.20225384396778112, 0.7786298398815292, 0.79249486025378, 0.5029421001492755, 0.42526425032137793, 0.37577843289840396, 0.6437968755196863, 0.7077064456890558, 1.1097845661762666, 0.39683966151875333, 0.1593617967120994, 0.21565527515686117, 0.17201924286683745, 0.6095377089509095, 1.230560240725076, 1.0911729905236374, 0.8036057986068359, 0.7950405376486283, 1.4837538498305942, 0.9931214718893664, 0.7931695066251786, 0.8089070707588697, 0.39652549779933416, 0.7602986362033795, 0.7621570801830952]], [[1.1794801576996767, 1.3104778838249553, 0.3309978413559092, 0.7451419838156932, 0.6312682553404967, 0.9269064623750206, 0.4027373224269902, 1.4583237017554747, 1.1660031583372896, 1.3568882442560728, 0.15999146943461784, 0.7159085146540148, 0.7851221618187565, 0.519175445130891, 0.428291669473451, 0.383042459920271, 0.6849473145389163, 0.7151792898483944, 1.1026313834112003, 0.4209288433944144, 0.17091651322934764, 0.259683743127504, 0.21112428884873405, 0.6569547883883011, 1.1734114187050944, 1.0978273531402785, 0.7715357377420043, 0.82176810869741, 1.3370352962339498, 0.9430666899849729, 0.8009215547105336, 0.8767455526230352, 0.6224804895137461, 0.7625941038220162, 0.6741738255617455]], [[1.093707278193692, 1.3322254161513944, 0.28080979945434564, 0.6462096518338902, 0.6813972544085325, 1.0085766916728043, 0.4423291785841937, 1.5491252318556445, 1.0621936257472144, 1.280091033052082, 0.14352552306105892, 0.7307337766069261, 0.7967916203817218, 0.5346848765842217, 0.4158117195862159, 0.3883382498820003, 0.706508885686057, 0.7556023884892934, 1.1528165658880307, 0.4585875688059703, 0.18126977990882986, 0.29305274945222737, 0.2123943688238132, 0.747795371323293, 1.0503582624833907, 0.8610632570305561, 0.7865947072578927, 0.7238395683973384, 1.2329624860277233, 0.8556925010524167, 0.8535882186368243, 1.0351837574047233, 0.6355527404918953, 0.8799984761651378, 0.5347323724112678]], [[1.0681228382401424, 1.3957957400524768, 0.31520893727534754, 0.6462402019419669, 0.9497895516156811, 0.9523053915846577, 0.42984566938138336, 1.6423805996570067, 0.9295654254821514, 1.149415222312605, 0.15418767592260885, 0.7177382857924355, 0.7441810724597153, 0.5230044658505018, 0.4496488616760588, 0.4174227293529833, 0.67574812457257, 0.7778651592753958, 1.0607205862843574, 0.3733566594765624, 0.16800657822575146, 0.31370907702016526, 0.18527525532791045, 0.7644805817930129, 0.6484917143028166, 0.7874973573022217, 0.7677132221933051, 0.6013607408046231, 0.8508132877756115, 0.7299973587208162, 0.922451285611833, 1.110496048992422, 0.5095585466901928, 0.994505082964043, 0.509438566654636]], [[1.1437429403513901, 1.3778305536315403, 0.31766474670044015, 0.6510285960806331, 0.8863643346931587, 0.9233345881932742, 0.38456437833097484, 1.5786442262724338, 0.9745945230674868, 0.9816753258931403, 0.15947821722875452, 0.7255188080671424, 0.7576352182845785, 0.5253171865617888, 0.4691055928139104, 0.36764243173251787, 0.6565392131655644, 0.7367398169794743, 1.0643049344047983, 0.3185138703195648, 0.17592430206460757, 0.29303504761611887, 0.1379536135723976, 0.7612206916425183, 0.4951063074449401, 0.6252440710005457, 0.7917289671114152, 0.4871360480874267, 0.5200501321131601, 0.6315226383502681, 0.9234448117557859, 1.113574574173435, 0.3747949337452165, 1.0512298199081145, 0.49463625998788996]], [[1.1381284978029331, 1.2771059687241235, 0.3754952014505855, 0.6520289782440317, 0.8903883709056295, 0.8642101961005291, 0.37693270756312397, 1.5679757704280677, 1.0846098679523215, 1.0196134872938891, 0.23244134954899492, 0.6381522548650735, 0.7022288076906079, 0.5230984281519917, 0.47657995726145513, 0.39839718821170633, 0.7131022481291481, 0.8049866001389111, 1.0985307782329514, 0.31004501936751805, 0.16492250851300777, 0.2721596661381908, 0.11091199378843537, 0.7067020307748881, 0.5818230433484026, 0.6402072926962146, 0.7778453509951234, 0.41492570254632694, 0.6065425711551476, 0.4948049205643905, 0.9160438069549572, 1.0540102113309313, 0.49655757458705324, 1.1468919263953996, 0.491542736686312]], [[1.120492506483586, 1.194292286805641, 0.36757882976967166, 0.7106808253257962, 0.9518105189606478, 0.9661168685734796, 0.37942671117809434, 1.57505580237318, 1.1410056363438221, 0.972784168242105, 0.29723464874988204, 0.7216916376880119, 0.6678095480917914, 0.5443377147762123, 0.531644694913268, 0.3683636250253094, 0.7788652545095265, 0.7694097167749714, 1.170132065924455, 0.3027546895196521, 0.1544828451962008, 0.2723819070121991, 0.11153261060863363, 0.6891341141465717, 0.6868339269622498, 0.586922910018223, 0.7661979247063448, 0.42640437770861594, 0.6836436613542961, 0.37667799571989236, 0.8797461117472116, 1.109501505220534, 0.5897201552689292, 1.1865581836982462, 0.49935701415981026]], [[1.1612840612432107, 1.3586874781054785, 0.33092058643897054, 0.9071985919160929, 0.9454777219193303, 0.9325931602583166, 0.3870485229043216, 1.6016651243987927, 1.1265669084842544, 1.0863130493398685, 0.32617740290643815, 0.5473744963434914, 0.5381141712211113, 0.6045879972685456, 0.5299344521972083, 0.33991516247593573, 0.6744912822571645, 0.7242716233480992, 1.06715066270385, 0.3589376444122674, 0.12839604321306253, 0.29383437225512665, 0.10205833871849673, 0.6401882141422297, 0.793500744299869, 0.5707359095335174, 0.697455771348774, 0.36755840250725463, 0.6954357168513547, 0.28529291041073135, 0.9520122336086294, 1.1243765863339814, 0.5289257761521582, 1.1843377593021196, 0.46117437195527256]], [[1.183001956771486, 1.2999052195542178, 0.32805984264072746, 0.8873535890695603, 0.8954821575151156, 0.8452995158762129, 0.3870917550591599, 1.635413936152944, 1.017307221409109, 1.2969159004946254, 0.32676171947773003, 0.5825474558488142, 0.5786105828856586, 0.603097957267958, 0.49056666931931253, 0.3301030030807224, 0.6661687926343617, 0.7431529881568798, 1.2006170922214907, 0.31798531921044854, 0.12794314816545477, 0.31025144939769145, 0.08458105057418, 0.6199465456413945, 0.647881920294361, 0.496898336657131, 0.672373763520209, 0.363166515759971, 0.7319019263186637, 0.2801248947492893, 0.9923444806329997, 1.1742725225416868, 0.5870637600386601, 1.013331904680839, 0.37249882197335754]], [[1.180634935350819, 1.2203782127117258, 0.33954798942634073, 0.871321929544072, 0.8640283802722161, 0.8172769097662398, 0.3766865764016804, 1.616760924441581, 0.9928028495651531, 1.344335970443114, 0.29597628250196406, 0.6614311000222703, 0.7134774295028017, 0.5968048844207093, 0.5133453482638397, 0.3331924939982556, 0.8399580604106668, 0.749297091469912, 1.245878747993805, 0.3692111764346587, 0.14137571939233484, 0.3238057297690726, 0.10504701183953372, 0.6186579090150711, 0.6433771455942022, 0.3196451611699188, 0.6861695072386573, 0.3780692781694922, 0.6514481457803771, 0.34246782741226073, 0.9772818420403866, 1.222406787545952, 0.7785978917397612, 0.9742555305056033, 0.32991291165343734]], [[1.146493043129506, 1.0769385498333348, 0.3294810833758994, 0.8659833350738937, 0.8438484459875191, 0.8507164812698221, 0.38022133031132815, 1.5601502173946622, 1.0013389983372174, 1.2646858452642946, 0.2570203955739959, 0.6106262019758439, 0.6992549736215358, 0.5980363334286722, 0.5066148778466368, 0.3048440294804486, 0.85834339418661, 0.7246392128531782, 1.3390207714639688, 0.42789025483544774, 0.16322135814713867, 0.3695840436149941, 0.05738552280605128, 0.643197207161327, 0.6367903394027326, 0.3134089100026912, 0.6623137172677611, 0.36022851409518764, 0.5428252782656355, 0.3540941936961238, 1.0173904207796463, 1.1714688103985325, 0.9503415207076824, 1.0177105590709903, 0.3199878515879774]], [[1.0685271751661543, 1.0389091117358917, 0.3000880721012628, 0.8373683648267372, 0.800068681906316, 0.7150569649713671, 0.36814443282812503, 1.4760626319258958, 1.003008771234858, 1.3878266504736736, 0.258705882857732, 0.6312560363037141, 0.7837677889596449, 0.5958106749252947, 0.34656463502293355, 0.3078264117904682, 0.7492593033267472, 0.739709900209934, 1.3918271041406327, 0.45270938623488044, 0.19255316616516588, 0.4761751811187017, 0.043476008001311836, 0.6693092770317339, 0.6458671110961209, 0.30217406304028893, 0.631060828362457, 0.3851892077407449, 0.5729444607926228, 0.4527856796206591, 1.0356904392657094, 1.1669858262442323, 1.065401367208129, 1.0128968686633648, 0.2945969805599108]], [[1.0103851681637335, 1.0199552719160547, 0.28962506372504454, 0.7947915160015102, 0.7410232985311729, 0.6994225362539241, 0.36215511443811144, 1.5260656691503591, 1.028887167991346, 1.289178161841921, 0.2925252190006272, 0.6387854932065296, 0.8000005522532315, 0.591166990708019, 0.29996845001175493, 0.29596986682739324, 0.7608365058776331, 0.7367872954126615, 1.3256312962795647, 0.3998248033500361, 0.27999207891461225, 0.5237898803394562, 0.026409381072670303, 0.680828123136742, 0.6045276831330136, 0.2641527652383139, 0.6281655711446479, 0.3850061355733583, 0.5779786845993534, 0.5556677279787325, 1.0482685363010118, 1.1910616103796432, 1.120637852105304, 0.9995881347637328, 0.2614118448881316]], [[1.0811246699298598, 1.135606481283921, 0.29206006818497915, 0.8156059763326893, 0.6867079721292452, 0.6737825434426448, 0.36258833084736447, 1.6391180097111686, 1.009844320863784, 1.2989975580873951, 0.2723605609176384, 0.6550802340957513, 0.8947554056895053, 0.5870822789967827, 0.30991023056696687, 0.3028248015763285, 0.7125384025775005, 0.7317113336629417, 1.241624667460596, 0.26968267789704137, 0.3285478876591252, 0.4611349787028978, 0.014042638290969255, 0.7187587301707825, 0.5380707551914705, 0.24879331098871033, 0.6326199550627385, 0.446104261380896, 0.6085326928268705, 0.7363523243580501, 1.0337611411617236, 1.1340385188233213, 1.253988616877019, 0.9168258400768466, 0.21284566031343205]], [[1.1031571904309425, 1.0616382373057602, 0.3131631172194445, 0.9265267330155933, 0.7227078447139454, 0.6817006030944296, 0.34176490223096795, 1.6280182011586666, 0.9842222296405121, 1.287257676522887, 0.2674794386594966, 0.6778909285635581, 0.9202427393299548, 0.5783744948199827, 0.29228243162274037, 0.29151882319354666, 0.7016113868412311, 0.7177385179140857, 1.2368409708258143, 0.26318098714140337, 0.3829437724466358, 0.47453204844595365, 0.01772449937608376, 0.7452333629534705, 0.5752073700534601, 0.318090414587243, 0.6634707458720821, 0.4743633725346486, 0.6112527421621394, 0.7984927407291966, 1.0284728992032703, 1.1228144439926626, 1.3240603336587635, 0.759401997588116, 0.25758269704467895]], [[1.2681336095598372, 0.9623273614477984, 0.401231028551998, 0.943468115295856, 0.7293337134357392, 0.6869993646341228, 0.36114490998357374, 1.6896445467715406, 1.0199979857561905, 1.2589695680458153, 0.26317114081267856, 0.5646318042093875, 0.9088388336729163, 0.5909517759978624, 0.2988492813566055, 0.31168195799578824, 0.7211479292243643, 0.7823420937962482, 1.0491505463997692, 0.21450802909575692, 0.23150231488789502, 0.5304148698154236, 0.02292797044697369, 0.7654052034358916, 0.49990868896942764, 0.35353956312156165, 0.6600058055320789, 0.4731947134536796, 0.6411375217044829, 0.9161908446585634, 1.0319214519418174, 1.1065423575369855, 1.3565391998423357, 0.6702612886068637, 0.3572358076388119]], [[1.3392657291547496, 0.9969518134875457, 0.4358559456008634, 0.8902313738705931, 0.7594288414808278, 0.7083635530434966, 0.3598327030224896, 1.6532205288035993, 1.0429534417970328, 1.3306128491543565, 0.25952753475354273, 0.5604728698871388, 0.922850176878678, 0.6768290373295675, 0.3038908733069533, 0.34769028329643054, 0.6907779993724839, 0.763762126239788, 0.9554452087220493, 0.22637794200674824, 0.24308018436957451, 0.5645766977958725, 0.029289500199752964, 0.7943836946276046, 0.4349556977413872, 0.3899712034506293, 0.646849912737903, 0.4680386081014245, 0.6566470390076664, 0.87613732022231, 1.0310412244436291, 0.982687146100691, 1.3951854586489074, 0.5996650623838284, 0.35245749663402737]], [[1.3243965793052659, 1.085706697880919, 0.38253599662803944, 0.8902075469716964, 0.8024683683315635, 0.7170792423597374, 0.36542380099521465, 1.5729450613741363, 1.074813269259498, 1.4745075684786242, 0.2803514964948459, 0.5642618497741313, 0.971308278675141, 0.6445869998572418, 0.3057134852892234, 0.3446668199782075, 0.6381203543831439, 0.7481061966693047, 0.9015197959956197, 0.24227468313029704, 0.24346629654785484, 0.5440710809047823, 0.011391929219434285, 0.8232037181380542, 0.3774379185255321, 0.41463839778406203, 0.6525066483558498, 0.4437426986806595, 0.6828661682538995, 0.8718322517055852, 1.0293921707441918, 0.8703020252576563, 1.4071511918051123, 0.526865238063195, 0.33144185505769186]], [[1.4080954899428297, 1.2321731693223932, 0.32037461678161155, 0.8796800633142696, 0.913494170578405, 0.7272272968938387, 0.3656611834333544, 1.543522746324766, 1.0713024289020887, 1.5628091523111838, 0.33415263203461565, 0.6052921841776611, 1.0136608674597387, 0.6303632166311228, 0.2591690910905582, 0.33850943147319007, 0.6132674196779329, 0.7664033753617859, 0.897077064324228, 0.2585674849745342, 0.20978203859519667, 0.4813688587262356, 0.009170662696097783, 0.8356679767564212, 0.4363631032210045, 0.43170663026601064, 0.6336442544187632, 0.48542012224187425, 0.6759932697324303, 0.8500499893404351, 1.0231677054126609, 0.7772746778190248, 1.4041809894263055, 0.5031303381201134, 0.3070172249206391]], [[1.2926415705054604, 0.9476907666817271, 0.29785234822866186, 0.8345776050464031, 0.60885387577379, 0.7415091346133741, 0.3680430391011266, 1.5401717008215383, 1.0536874554555489, 1.6248499413505493, 0.3027352363198211, 0.6214287576134317, 1.0254081125098828, 0.6086892537021322, 0.2647763325426533, 0.3240473577643822, 0.6167940911564012, 0.7895230300124552, 0.9658986078632017, 0.24842010074986254, 0.2158834268125544, 0.47431565778334034, 0.036870205392535736, 0.8430708075317239, 0.5416354541338841, 0.4190027821192695, 0.62143342910448, 0.5892344345910786, 0.6583469949653561, 0.8383740716866034, 0.9841125216648309, 0.7406684800211842, 1.3752387483141217, 0.4437506531903791, 0.34596819105297705]], [[1.1962246010581101, 0.779639520588218, 0.275499988211879, 0.8300814301917177, 0.3342725729283703, 0.7773725762789209, 0.3723639336946875, 1.538898606106165, 0.9786351757544673, 1.588787512432892, 0.2687356153915424, 0.4708034006154874, 0.9864042567364901, 0.47195413790782487, 0.31098949374642454, 0.39618533996721694, 0.5526822766535029, 0.7622622207950999, 0.9465382709311878, 0.21965688452747062, 0.2769569640573939, 0.523543486756342, 0.0363871459150307, 0.8231581090936522, 0.5610683856446841, 0.4071813105001385, 0.5982122468310864, 0.5487548512447429, 0.6597545825500539, 0.8287272073248096, 0.9676550021746542, 0.8007185803298325, 1.3536200529911717, 0.4208066635468847, 0.45941815498569333]], [[1.178975664490061, 0.7744622159637783, 0.34453186814222464, 0.860683457439155, 0.3367690748033503, 0.7652399533063151, 0.3691549027498936, 1.5958552151351957, 1.0560856838622055, 1.5922520407605467, 0.24947897228116528, 0.48507178397431205, 1.0147243671859463, 0.4544294346254293, 0.3502400620201297, 0.3964426058249608, 0.5151463071979354, 0.7647268951799663, 0.9084031495135361, 0.21452361921408764, 0.26223894255517866, 0.5346624544988459, 0.026300440544861733, 0.8520901686082221, 0.59648968046407, 0.38284862501269323, 0.6386331463384363, 0.5193451170672123, 0.6640405492548652, 0.7582886399566287, 0.9525716639572492, 0.8055182764285956, 1.3161849006250366, 0.4093707871805826, 0.5993353492370499]], [[1.1287286443378517, 0.8663670948172398, 0.3622524706985453, 0.8301168842708562, 0.5075852964352141, 0.7675588857302644, 0.3685917960323653, 1.6519987625558077, 1.1747925491148308, 1.558675358185266, 0.23887995222716896, 0.48720016358514634, 0.8985751337392728, 0.44357595648135617, 0.30915192688453097, 0.4045010099977086, 0.5349412019205951, 0.730619315368034, 0.8693931942750956, 0.23977914295183367, 0.23113713313369216, 0.4770078118828709, 0.05301669704886448, 0.9774905082328083, 0.5711266433390548, 0.3785521693952108, 0.683162886071814, 0.4146289929620083, 0.6728764156846185, 0.794348392607036, 0.9101187220424538, 0.8172827398258434, 1.2433998008057594, 0.4358041778920466, 0.719061557285543]], [[1.131193907816288, 0.8551114856832184, 0.39230985825693454, 0.8052740681185102, 0.36072951165627276, 0.7496823747379682, 0.37034321495580963, 1.5693982429430413, 1.1277350917256665, 1.4395288235129904, 0.21364758438242137, 0.493058016750784, 0.7026728542494185, 0.45074069227366437, 0.32059447994237755, 0.44256520133711574, 0.5277625467941861, 0.7118109605526629, 0.8087731490496366, 0.2625310468827462, 0.22644128682539877, 0.46256739875258535, 0.04364005934919163, 1.081283350593945, 0.542510229209123, 0.3924044267035287, 0.7401955288185313, 0.34380856505780555, 0.715790263063769, 0.764946112646932, 0.858354062755303, 0.6758162289342825, 1.1807568425455832, 0.3261582930026957, 0.7774102943281909]], [[1.1089194133106883, 0.9032053667337804, 0.35040544736095014, 0.7834666976759452, 0.49216105467184623, 0.6772258428977818, 0.37151700771653556, 1.5963919892934046, 1.1249981806030935, 1.4367171161714647, 0.18664494986504726, 0.47315048981209373, 0.6112454572422681, 0.455977379794481, 0.37299405677773, 0.4656430486668931, 0.5399145402577672, 0.7377294976994893, 0.7920151160963036, 0.252953207328545, 0.23271814869324758, 0.39088883335758295, 0.04440266206990956, 1.0464701297758912, 0.4951891640695154, 0.3911578313201326, 0.7006438877698788, 0.36187771538247726, 0.738697058354722, 0.7931767580742068, 0.7195797635355687, 0.6047258892633969, 1.156046631380677, 0.32219432125313235, 0.7385995544872574]], [[0.9876920234157922, 0.9764630101159072, 0.333408279523316, 0.8130414271331947, 0.39562208672305826, 0.6645709630705185, 0.3707179048302762, 1.6243900516423229, 1.1642618503578341, 1.3618274899556981, 0.1711475554360296, 0.4835003399813477, 0.5911740669246791, 0.4450240933760928, 0.34777716913945494, 0.44029920082117135, 0.6152953925793285, 0.7454963379471351, 0.7508645840038036, 0.2536264185201051, 0.22631657336938626, 0.3896026800827867, 0.015990348491129312, 0.9904455899061839, 0.48363464628459896, 0.4034787564382908, 0.4438543846279246, 0.3760068327658628, 0.7565988471173698, 1.0088826297598616, 0.48018948302255604, 0.5439847656924621, 1.100376274606576, 0.39360946746113507, 0.7596812972659335]], [[0.9065472484496688, 1.0071286708966032, 0.33671074639638565, 0.8222094912486231, 0.5036146731953449, 0.6787237472941035, 0.3702849121257981, 1.5782934539025613, 1.1497710461267399, 1.4029427484258372, 0.14018256092900253, 0.4890819083327923, 0.5967275204065312, 0.4402283397125767, 0.3197337518927999, 0.46425910460083164, 0.5438364958118952, 0.7777756473712167, 0.7778490148354269, 0.2512642491334864, 0.22978306558986272, 0.3698930307376538, 0.02025852541134082, 0.7729246453787911, 0.4710340495628237, 0.4089045752609367, 0.22312903422543962, 0.40992392123355503, 0.7499262860007742, 1.0805208766995982, 0.4694914219686298, 0.5440539200143755, 1.0278721471653254, 0.41604061908883583, 0.7054311538039514]], [[0.9014364318730009, 1.0348418169113134, 0.35617968243623943, 0.8399886961134836, 0.5675109421682543, 0.6797253145625479, 0.3682051388451773, 1.5210402305944548, 1.1647848467298585, 1.428388833019275, 0.13786309988825068, 0.48947028546769705, 0.6131227683227288, 0.36497844451726597, 0.2532685062901575, 0.464472315303403, 0.5885845373298011, 0.7411532653222428, 0.7950270409874238, 0.23242160338853834, 0.23818433181872758, 0.3383728159640025, 0.02798070593814957, 0.6107005175493853, 0.49666189621199563, 0.4031540787505458, 0.20603912052017898, 0.4902213053497748, 0.7211739849969684, 1.058000814164271, 0.4692374066819334, 0.5511441264368617, 0.9135488905371341, 0.4811146300954425, 0.7273736859601537]], [[0.9152348120496544, 1.0852289385591238, 0.3456178860380643, 0.9027248089151283, 0.6108283102147302, 0.6705226903262462, 0.3685759711390425, 1.4252176685183886, 1.127198923698899, 1.301770982619785, 0.07186278740005766, 0.4630824019166858, 0.623001337582725, 0.3683722585088457, 0.26021100646007617, 0.44093693951478774, 0.5548308886329072, 0.7326706755538429, 0.7877133806833084, 0.21496254307445853, 0.31743533202456653, 0.28238149650297595, 0.0497042528737966, 0.4692034428291627, 0.4761851165909188, 0.40685313795285377, 0.07657324299559246, 0.5377700007550051, 0.7039606137259757, 0.9681054390104405, 0.4300068378480031, 0.5767976028423377, 0.7473545886940381, 0.4944203417278621, 0.7482596048282439]], [[0.9457339377027782, 1.190813794616443, 0.32796130361632647, 0.845827513404968, 0.7024426605922697, 0.6750427347527039, 0.36706152207797477, 1.2422058825285687, 1.1541287398162785, 1.0532356446166435, 0.10825177867624242, 0.4599149640154967, 0.6592122839620347, 0.35560940300730853, 0.23076670016724926, 0.4541769791829263, 0.5963738021839192, 0.7464638434108674, 0.7907434318406066, 0.2050367794450739, 0.31839260025682425, 0.2826625999733939, 0.0729365525262303, 0.39256119895508834, 0.5169172517555561, 0.36273785758180155, 0.0649049873939433, 0.613208224557971, 0.6765332843034149, 0.9127301936691514, 0.4170045710346497, 0.5720229338694061, 0.63418592882417, 0.5421484003654051, 0.7728850110230487]], [[0.9055877108387604, 1.1968651707848923, 0.37272020340031253, 0.773604573738025, 0.8198458307633562, 0.6883539433425394, 0.36750305869669386, 1.147677294214383, 1.091338729125566, 0.9411514983139254, 0.15421955564872838, 0.48513550711537257, 0.6827641822676089, 0.4487727160814735, 0.2751264270044096, 0.4218251805365386, 0.6813332564629913, 0.7621686762004278, 0.7898875764879069, 0.20343774593397065, 0.2607862763746864, 0.32621066714441255, 0.06356139298074595, 0.34694214011052066, 0.5588570488803689, 0.31608631347148053, 0.07233194182119378, 0.6587875266746606, 0.6642061112849104, 0.8669782734523486, 0.4210840523313151, 0.5519857453285173, 0.6165648911859788, 0.679472635748251, 0.8195297971522864]], [[0.9278994079026677, 1.2468681650921367, 0.3463359265655171, 0.7282088316606286, 0.9071581948155758, 0.707932167926175, 0.37427656250781577, 1.0912800764912305, 1.0857946150419056, 0.9620618093424185, 0.1560729246398834, 0.4679562066873689, 0.6706910424594645, 0.44075522644180953, 0.27014679955055815, 0.4057522861684386, 0.651718993918926, 0.7773324957363599, 0.7640156607161941, 0.19189151471175542, 0.24061109470649125, 0.3378423835176195, 0.05422708164798766, 0.3223747034361139, 0.5462370729358104, 0.3014048250176293, 0.06420226104934537, 0.6786990383397653, 0.6787286100530485, 0.8359100734731923, 0.4260538675842237, 0.5464245840785168, 0.5667603392520613, 0.7171731784603668, 0.8786769286366822]], [[1.1070042567114173, 1.3654953291698628, 0.3989353707742468, 0.7645380283024262, 0.987238079940792, 0.7329692528559573, 0.3708799874951242, 1.0309685961076314, 1.1142656593732903, 1.0688017163096546, 0.1019681061582078, 0.4604505377159825, 0.6896641752727672, 0.42546287895172774, 0.2710599180299843, 0.37180166133972464, 0.6504935900624067, 0.770266030319135, 0.6250336059393626, 0.1844875007709837, 0.24703585551405036, 0.3458437218204552, 0.04202751169027649, 0.31336702489492696, 0.5285852514036906, 0.3083899655890058, 0.051937000893211197, 0.6998643131439186, 0.6600977308870104, 0.8281314845443328, 0.4677814862772176, 0.5491401288483434, 0.5127642374602897, 0.7751319439603959, 0.9004666220848687]], [[1.0872767493694484, 1.3602069241020232, 0.3773049382117267, 0.8054085627403453, 1.047793265625359, 0.737255100457187, 0.36958990329867025, 1.0517206558789665, 1.1259281257541713, 1.0553544609485275, 0.07246918322029593, 0.4676006047834431, 0.6627471453260998, 0.43034996450336477, 0.28317278432861964, 0.3774383054306509, 0.6728615854499702, 0.7673470910260067, 0.6500834309751267, 0.17614864596304028, 0.23569392532140934, 0.3957648586525707, 0.0444858710911221, 0.30782869824242187, 0.5249029415362485, 0.3274925020064727, 0.043714659504023756, 0.7204146676301051, 0.6307279622877945, 0.8268888277619629, 0.4753472022671438, 0.5651349041765477, 0.4702924253009807, 0.6828012389995788, 0.9013961292422742]], [[1.0498631002566787, 1.3612788946802727, 0.34227190411017194, 0.8330591213384546, 1.1173391012575007, 0.7701569282852873, 0.3854064312174116, 1.0085935908958459, 1.1726154036555707, 0.985674153477482, 0.06184004832302682, 0.4755545845675754, 0.7027901675987834, 0.45140471014667943, 0.2885029959656809, 0.35225191639724646, 0.6632009781449042, 0.7679245540941784, 0.6964051608537801, 0.17533109419765983, 0.18796382213719381, 0.4074507841730712, 0.05850882522127106, 0.3049799965569918, 0.5117767493376094, 0.3542080683674773, 0.05055084471393226, 0.732947706865643, 0.6562072193650847, 0.8353386947568924, 0.5028136981894831, 0.5950523550272773, 0.4458098465839384, 0.5885285451160364, 0.8846081415341217]], [[0.9760993536125956, 1.3219891323470887, 0.3467458453192348, 0.8351516222512729, 1.2008148970150434, 0.7414605780321897, 0.3929496996758189, 0.9663994556057802, 1.225627908314003, 0.9260476576873466, 0.04988664412297625, 0.4830052002532048, 0.6955071035263471, 0.4334738062538252, 0.30693553888885494, 0.3303938470648612, 0.6521345330001536, 0.8619554526152875, 0.757100157574184, 0.1727519526293147, 0.1809682269166183, 0.4079763307802994, 0.09353715883844857, 0.33070186776775445, 0.504936946070649, 0.3743106726316721, 0.07353735859186007, 0.7500554647611974, 0.6523515569954705, 0.8463241305058543, 0.538509711535212, 0.6196473646723026, 0.49681993182798634, 0.4598760201724499, 0.8132150361320833]], [[0.9402126695361139, 1.343997232381647, 0.3149845073908102, 0.8516339457086366, 1.2501956464982862, 0.7237130007845851, 0.3784491870267118, 0.911289670806561, 1.111251234538488, 0.8277787848112708, 0.04438425246898134, 0.49868156955512466, 0.7315772870293487, 0.45340328878585345, 0.3081735493282576, 0.33404688807326555, 0.6601925150510215, 0.8786395455746654, 0.7552409967808353, 0.17860652358333753, 0.1919452818103085, 0.3891835155791863, 0.10380175143183143, 0.3612718504669091, 0.5659352471396755, 0.42581050835908396, 0.0897206160883662, 0.7230026484662864, 0.663742568512397, 0.862934059890527, 0.559793047144154, 0.6202448640873724, 0.5611842435317607, 0.3819281610208701, 0.7574371416032178]], [[1.0036380937572706, 1.3881685671922925, 0.3180816192867453, 0.8635483810281326, 1.2753762662516177, 0.7251268916827014, 0.3640130026253985, 0.8133811301054854, 1.0338007263071614, 0.7588757250949221, 0.05117305166360149, 0.5340011119801666, 0.737256705840035, 0.44992440309563553, 0.3027931939146383, 0.3286025870680671, 0.6711310227636453, 0.9261396183765982, 0.7493542573134537, 0.18062539884367101, 0.1967405711697673, 0.32608008867750915, 0.09812638225077337, 0.29029994238344853, 0.5610061608282143, 0.5615443544526532, 0.10756045325478594, 0.7443025639667709, 0.6920608838205156, 0.8672363453244318, 0.6285107423151438, 0.6031639501092367, 0.6587091731311705, 0.4237541393068192, 0.7209695008628059]], [[0.9641421938452321, 1.3554144635713772, 0.31379597382671576, 0.8879882318772865, 1.2855184458298576, 0.7126994421348218, 0.34645149350185866, 0.7510883840660795, 1.004420487550901, 0.6834095398412835, 0.06550155458469759, 0.553596547125038, 0.7195297514788008, 0.44361036076919036, 0.2956924600170097, 0.32589926536687003, 0.6691209462880436, 0.9154149727320955, 0.743288823378243, 0.17026532663458963, 0.20856491344324063, 0.29132796115609794, 0.08489493182573028, 0.26073352694813473, 0.5478434244448906, 0.6931885806998153, 0.08819250185572333, 0.7628430133010708, 0.7057154794261516, 0.8633689673151766, 0.6818220348681573, 0.5869189274159877, 0.776184947866356, 0.47607890543403797, 0.6896628581970634]], [[0.9365157142112195, 1.3684348049637707, 0.31450182173729824, 0.8642036546352974, 1.302316055222261, 0.7224669168060612, 0.36103151680362006, 0.695712761813379, 1.0128147670361787, 0.679322943134885, 0.084468008919144, 0.5910588827084184, 0.7147457077374464, 0.44714306774738566, 0.2951479010436665, 0.32961228535171844, 0.6330097317161437, 0.9280225657648293, 0.7550653058178208, 0.16548088025820684, 0.20418142392335273, 0.28844147522438457, 0.08714719136508622, 0.2734826242312376, 0.5823888027599038, 0.6793535792023526, 0.09495324373623398, 0.7549772353527029, 0.6803596408801773, 0.8489453137920426, 0.7247625610402411, 0.5725767693444153, 0.7629481661039825, 0.4475759111694799, 0.6429339772700007]], [[0.9734948940072243, 1.377355194291963, 0.3123229604812161, 0.8777104385307353, 1.2849534961601394, 0.6943532568718698, 0.36783999303715237, 0.631821951365967, 1.0194038011432123, 0.6524705848955215, 0.05186782772598242, 0.5597283397238454, 0.6971564672744663, 0.41493403180350463, 0.3096676644205444, 0.33611789766001543, 0.5269293523816724, 0.9606569129660594, 0.7732166512759292, 0.1696661916060014, 0.18673144467780328, 0.25259108591554236, 0.12531382525592244, 0.23111471211053453, 0.5835414092384352, 0.7057881449001352, 0.12631641460799464, 0.7396054338714535, 0.6972068080520655, 0.826489352842768, 0.7512467944027287, 0.5622819948164416, 0.7393919793301018, 0.4327525740174284, 0.6216596722245789]], [[0.9731493378006151, 1.3324240205701234, 0.29830862913827827, 0.8598058352091368, 1.2832614099980648, 0.6809603586023287, 0.34357870280469965, 0.5789555249576855, 1.0491263130432764, 0.5181167354735646, 0.03851771306154323, 0.5270505523403484, 0.6603425418878768, 0.4403696345155321, 0.3404149403282979, 0.28909949124912554, 0.5888421159412804, 0.9877999565236124, 0.7943437285232804, 0.1941770758046604, 0.17720296262967156, 0.28349846187158456, 0.11528261773887406, 0.20401540864867007, 0.558677895745442, 0.7527606167193739, 0.22614263267616008, 0.733586132073821, 0.738369159595619, 0.8147533686187609, 0.756715167610204, 0.5518947575040372, 0.7240036573077063, 0.44653584621048403, 0.5702387365463543]], [[1.000728391157895, 1.3339844835988828, 0.3123721893467394, 0.8711902423486225, 1.287306144835736, 0.6721597683491277, 0.3195895534190133, 0.5320550380851254, 1.08484796930275, 0.5026224575578007, 0.03581291266015343, 0.5170848261412287, 0.702342373853396, 0.45628953177087594, 0.39999073405829494, 0.30061746079278795, 0.5774202186065811, 1.0151598721558506, 0.7994125716856411, 0.29389010900229645, 0.23570391161262053, 0.34097563003371983, 0.12480369531346483, 0.21082789970834181, 0.5119848229407649, 0.8380637235753431, 0.27710830456220625, 0.7274371277626436, 0.8336242647981427, 0.7774025798644714, 0.7496727513576398, 0.5271026243630783, 0.7352553090340419, 0.527340853724119, 0.7432996578632001]], [[0.9720041314055643, 1.16519895064421, 0.34939614793151674, 0.9153655650392142, 1.249015317200626, 0.6955043561874192, 0.3569284283117872, 0.46174650140539947, 1.0794491708055811, 0.4488681404846453, 0.046315924428253906, 0.4994826246704194, 0.7406843390579765, 0.42581229757678707, 0.3354733640623463, 0.30741569089663356, 0.5755116324960308, 0.9881457097995049, 0.7485509745260804, 0.23362478957195243, 0.22896376375017474, 0.3436416181425075, 0.10344482620535836, 0.19392906458682602, 0.37876698657862423, 0.8185942941069579, 0.3261072763652286, 0.7548936478078736, 0.8342126472461262, 0.7799190645803044, 0.7469283652199505, 0.5032681107105311, 0.684061672379096, 0.5896567494703566, 0.7019091199875227]], [[0.9221163581500673, 1.0720438639749768, 0.37941279802890937, 0.9749033900648945, 1.178829918733928, 0.7109054749302355, 0.35467037287482445, 0.3763502182294074, 1.0473101451915467, 0.41706633823105843, 0.04228249464288821, 0.45800931473284856, 0.7842854203097102, 0.40913566588927697, 0.3097487552473862, 0.30433117487395855, 0.5657677314893371, 0.9290834431089074, 0.741751177144931, 0.14436028806929463, 0.24881518848136844, 0.3347919646314439, 0.09719362407798608, 0.2182292983413594, 0.27529815865834095, 0.8102504891015844, 0.3116763954700052, 0.7984681963046605, 0.8373806319342674, 0.7804618145050137, 0.7634865595036044, 0.4913496075233781, 0.6467786172962695, 0.6606533948089827, 0.6777910194333654]], [[0.9062337586830109, 1.0507945846704914, 0.35150868423847925, 1.0089781918582734, 1.1049366170593533, 0.6779480142265868, 0.36883928564469176, 0.3565059447258151, 1.0123116244718955, 0.3746604465475273, 0.05452572842172563, 0.48903137455787016, 0.8115624202176035, 0.4427079365656651, 0.31505465309265834, 0.3176389670799013, 0.6045822337881028, 0.9068065543125223, 0.7407767726570149, 0.1503740697263894, 0.24124964062248383, 0.2805512990822888, 0.11898048320860836, 0.2599029205778882, 0.24476832299271453, 0.8005796636495193, 0.3031711654899799, 0.8398534949293546, 0.7980471365209142, 0.7836375316464357, 0.773241868173831, 0.48385396445332907, 0.5883382015532985, 0.7571168163601072, 0.5318860993424473]], [[0.8935320194254587, 1.006187182578799, 0.3268491428930823, 1.0859234748425497, 1.0463303472210141, 0.6646902236429062, 0.36435796510259144, 0.31718244396175144, 0.9848199813143318, 0.3387618765781526, 0.0719882253868312, 0.5050239276405804, 0.8572868759565797, 0.4425144624438424, 0.31091310594504673, 0.34043545292300215, 0.608662648195923, 0.9528053197488695, 0.7319555660453585, 0.12772948627999398, 0.22590080622216396, 0.31554378151184975, 0.11219889716811358, 0.36253816636826053, 0.28098462303723226, 0.8018960353392472, 0.31541121211324985, 0.8678459843091464, 0.7005039096714919, 0.7846084609164662, 0.7752854010619054, 0.5023960818944255, 0.5973828023860321, 0.8016685513016726, 0.6331450088357071]], [[0.874515564733622, 1.0028207892327492, 0.30942666430128296, 1.1954072409047427, 0.9737051654922415, 0.6737318097165169, 0.39169514950930245, 0.3341869806430048, 0.9486948786220298, 0.36488843226559065, 0.06995695618691616, 0.5045966718125141, 0.8981960130726749, 0.4278223484475087, 0.3017246284978867, 0.3823604164094415, 0.6062833643062981, 0.9683498433456795, 0.7715847832326099, 0.15271643395149082, 0.21794863262176223, 0.3231148124220368, 0.11683795291059021, 0.43891542828416186, 0.3715084111117829, 0.8278825649733879, 0.39326465491536444, 0.8675400086267693, 0.6018154342052108, 0.7791565298642498, 0.799918239558087, 0.5344576728389427, 0.6116086845773636, 0.806277153985582, 0.7810629219540071]], [[0.81009697547834, 1.0478215161395117, 0.3314461685551465, 1.2354678944888486, 0.867593024665579, 0.7346092353370604, 0.41066296060093016, 0.4104197488206195, 0.8999544669360856, 0.5361646527956798, 0.0475234290153722, 0.4704084274024229, 0.9631677759242918, 0.39183483580104683, 0.2860138580888507, 0.3757812034460474, 0.5975327302387263, 0.9175614369188857, 0.7438405283489216, 0.16374145778978902, 0.21584419086316203, 0.3212290981286589, 0.15591534931337156, 0.474027849379385, 0.3864229581785804, 0.7980108551671462, 0.4952040120877481, 0.8150999365260787, 0.5366597578788144, 0.683702741871493, 0.7143081748042456, 0.5754133529024089, 0.567822706362298, 0.8297196349832983, 0.9405836302155105]], [[0.7650127662464551, 0.9550508029401873, 0.3382014821186867, 1.279895486199912, 0.7219211288949885, 0.763437224104077, 0.41444723615653706, 0.5162630614331547, 0.869576354780335, 0.7978202277202334, 0.08899273725066437, 0.4675094639656588, 0.98453621774662, 0.41114043437860903, 0.3256575541175547, 0.36935495651152916, 0.6269116801237775, 0.8774470729516937, 0.6737537078884985, 0.17431733768307991, 0.20419438630185666, 0.35001760365072904, 0.1182919145477763, 0.3848814465889618, 0.3656524859286031, 0.7811256455929925, 0.6112575656497669, 0.6592239385340447, 0.612551996833979, 0.6731023914385186, 0.6135928059687408, 0.6141205320436667, 0.6100391181603297, 0.8355564602828971, 0.9491222776592044]], [[0.7298903317754939, 0.9356030585204373, 0.34361594989260313, 1.3617685551944378, 0.596460631351202, 0.6604934820972939, 0.4286311641073105, 0.5659780542848667, 0.8948251407507685, 0.875739861689961, 0.07462893432229062, 0.49940925645631185, 0.9642607200001028, 0.5194288760735368, 0.3167187930001761, 0.3609476163469637, 0.7164868160321685, 0.9078063046500527, 0.6491507598260098, 0.16522499998618764, 0.20760754776977638, 0.4526968726485475, 0.08775121162125682, 0.25913987348905876, 0.276448129157156, 0.7549140682098814, 0.5800841651839024, 0.6218240339934857, 0.48611566889275504, 0.6263240877205797, 0.46326471977753003, 0.5809447445799989, 0.6459165804035059, 0.8778223278696935, 0.8321428494897631]], [[0.6706111298863606, 0.8929616021754705, 0.35993182251566425, 1.4079630659295266, 0.5163394879683703, 0.6693224177071637, 0.42481907116241935, 0.5268091259904351, 0.9409815923348128, 1.0947604473141364, 0.06722923107275958, 0.4084428269658272, 0.9984075958427696, 0.5123693707804247, 0.3081686229303813, 0.374615408958372, 0.7245641597579333, 0.8255289240361574, 0.6702252987867253, 0.18832750624381933, 0.19860881498996683, 0.417949887909177, 0.061463679224247676, 0.1908782916228533, 0.17232471963391427, 0.7460550085389608, 0.5160317309907527, 0.5542954100021675, 0.3924600152158737, 0.6389769084208333, 0.5065219206311377, 0.6640535523741526, 0.6260033807620824, 0.9048257149707248, 0.7325494325078092]], [[0.6161767488366279, 0.9143435297327154, 0.342501285938251, 1.4708466589835358, 0.5661396735132064, 0.671115363875723, 0.44223226994276393, 0.5513527524282502, 1.1128130892720471, 1.2854832676534627, 0.06533753092977322, 0.4163531414428397, 0.9787963862908342, 0.45733325780077305, 0.3110808383934583, 0.38472378866939355, 0.684933383416066, 0.7831668610720548, 0.6760720114949667, 0.19590618453734399, 0.21263004819249998, 0.41974630780780336, 0.07328099419475011, 0.16227388326513897, 0.1554991093789062, 0.7084494715951182, 0.5026685579377975, 0.5914580296575531, 0.36114116203289576, 0.6842846159295763, 0.6702253143389334, 0.8248380066386057, 0.5902693587659372, 0.9324845714231996, 0.7353673787616295]], [[0.5461530606015913, 0.9656727299569462, 0.3525596391448922, 1.4538887952865756, 0.4893409245248735, 0.6756619074021969, 0.44171729081754574, 0.5734369292503266, 1.2409874130096135, 1.0994746437653222, 0.07782542020756374, 0.4244709341412749, 0.9303375540167615, 0.47857791084345747, 0.31360284793877286, 0.3730422863422214, 0.7348812260704074, 0.7683453135301195, 0.707229728646156, 0.2076739648863396, 0.2157205856078066, 0.41965250559749456, 0.12995520710013617, 0.15771831718105006, 0.21145904899827578, 0.6608777438820808, 0.47012047325306794, 0.6182648104677013, 0.2923424378036479, 0.6695021307020288, 0.7722752390956373, 0.8324210241546307, 0.5980185594937165, 0.901020937075104, 0.724374989040977]], [[0.6654557070456312, 0.9597245285497952, 0.3240210814470306, 1.4704394539935504, 0.6995785784763662, 0.6714548578300814, 0.440494681846058, 0.5533594571377656, 1.317692883666966, 1.057187844287225, 0.08661179784949449, 0.39212996751853363, 0.9172097565457562, 0.46092474211115986, 0.32037870418620823, 0.3520672731875987, 0.713449445697008, 0.7955255253136659, 0.7221943360538701, 0.2228087212749412, 0.22208798779190364, 0.3699926007188178, 0.10795615056372632, 0.14905444218181357, 0.25402557901767414, 0.6478040255814608, 0.41215805442665177, 0.6278601010865449, 0.3220120480151516, 0.6228726027358009, 0.7505086425762284, 0.8110218645835304, 0.45832617579373003, 0.8458025726818115, 0.7726072751420059]], [[0.6469330802297091, 0.8243962566811233, 0.3387111361748546, 1.5055033800040931, 0.3231410864988349, 0.6926904702037766, 0.4374537450664456, 0.5679043089046594, 1.2052374929993215, 0.9153245951518056, 0.09196627947804223, 0.40355867142991175, 0.8318679880984472, 0.48794992713682706, 0.3347263838599736, 0.3654746144171389, 0.5881055865514091, 0.8190912799203645, 0.7230672252416198, 0.2289227187012149, 0.23798653242996032, 0.4072375870044319, 0.040384473382085576, 0.1377074814746579, 0.2980455498751702, 0.6350525883351621, 0.38056473631619586, 0.6498127669227504, 0.34448812671395235, 0.45791082209875356, 0.6903739659882195, 0.7929209027759744, 0.4427439841650874, 0.7821736925984908, 0.8279612788376818]], [[0.645685189920107, 0.78780722458308, 0.36576098166885046, 1.467108951229575, 0.33297325462032634, 0.68447035901705, 0.4301559839829266, 0.5990114515945044, 1.247191915171657, 0.8464362463784869, 0.09003908564178113, 0.4167230650455087, 0.7113509238804288, 0.48589621118594867, 0.3487194627342841, 0.3639207056289063, 0.5787308695083505, 0.7845662544590152, 0.6549317951801883, 0.25267927609158747, 0.23515681434735175, 0.3991597994449225, 0.04325153806819332, 0.13422903799538055, 0.3160477803714637, 0.6318314519952788, 0.36712763942868926, 0.7194730821596993, 0.37249708943387067, 0.4949845722762741, 0.7647722489335025, 0.7647181210120213, 0.48364148167285664, 0.8316115331800036, 1.023494102131502]], [[0.6368787611869379, 0.7917789716477882, 0.4111859538781609, 1.499049068677561, 0.4143772598999532, 0.6572208959818179, 0.4238537573181975, 0.6077836298122958, 1.3758887569851452, 0.9326292971905881, 0.11699009468468863, 0.39640541564655085, 0.6650062077743832, 0.4598338619910335, 0.36487685454080054, 0.3402779048743324, 0.5700583217329427, 0.8950870538783364, 0.674981070599682, 0.231667320830992, 0.2638036746913257, 0.4130619858541797, 0.04619407514141842, 0.11832297037611861, 0.2602008052741597, 0.6605014635036237, 0.343181091651673, 0.7007305683222244, 0.3907294345317015, 0.5734590743165552, 0.9141166634413431, 0.701178221533277, 0.48337780790559715, 0.9336020762308219, 1.1812802917126106]], [[0.632034228408024, 0.7407414901875936, 0.42080345191603785, 1.3788662621978638, 0.44817270436688766, 0.6638786482361025, 0.4236589227029976, 0.6238414196656878, 1.0766728159734718, 0.9287161969076096, 0.09548462024079911, 0.4239187868239631, 0.53029988082848, 0.4249971345827939, 0.38258588212127914, 0.32999645413310974, 0.48507120759081584, 0.8980344244474627, 0.6844272018143083, 0.2961350179399832, 0.28743003189414523, 0.44231753304613924, 0.026421815409291005, 0.11461216440981026, 0.19373732365293492, 0.6960305517216542, 0.34358626539243553, 0.6814269485313181, 0.4095755893299601, 0.6451519002833112, 0.945633430063207, 0.6626041777154252, 0.46767173966479336, 0.8651199964466902, 1.036595186171798]], [[0.5478978995791484, 0.6602497851391576, 0.41855134668436816, 1.2221229136917855, 0.6279725822876571, 0.6643082721311657, 0.43951262977687205, 0.6415563246906602, 1.0328786612467242, 0.8961775557618409, 0.12198888242499303, 0.45041211395259884, 0.329363637820662, 0.45410719649291487, 0.3486787941280692, 0.31874845012095754, 0.4164822568568529, 0.8486719579471869, 0.7494144795193554, 0.29151889946889437, 0.3204851362250483, 0.4670583111053097, 0.02933436188795467, 0.0947200714384327, 0.18352749093018858, 0.7071374950988862, 0.33861557004359977, 0.6316596868111932, 0.45121045448173197, 0.689376465010764, 0.9549309863426685, 0.6416159010762398, 0.4601364459039108, 0.8226133573189587, 0.9279188613445785]], [[0.5081872588453578, 0.6239791571558805, 0.49756625936370213, 1.03866790249435, 0.9183758995764711, 0.6380147532484959, 0.4815458507952086, 0.6685926606010238, 0.8921280780546572, 0.8627245572928006, 0.07128128831812358, 0.4676749285592636, 0.23247525780078865, 0.4533598023998355, 0.38908977593462507, 0.3081314466977495, 0.46173261497989804, 0.8104880853707546, 0.6154354568509879, 0.2721864687067146, 0.3261382801716949, 0.4783619522950988, 0.017063909863255993, 0.09673549877630247, 0.18390687218864324, 0.7258326669354118, 0.37414974233203585, 0.5857121024015547, 0.5074421252602884, 0.7051241799665617, 0.957909501013161, 0.606356044404186, 0.4954564547557798, 0.580947636777041, 0.7691927054159992]], [[0.46198866946228956, 0.5660138044258627, 0.5186035498475108, 0.9050295942958424, 1.0255153954869403, 0.673988885217616, 0.4677233132090008, 0.6675058301372073, 0.8669177941847415, 0.8269024169162371, 0.09195482387286119, 0.5287020291076925, 0.17686419574846973, 0.41747557668637647, 0.4202788199186515, 0.3414846075480442, 0.4540529696908022, 0.8488612729448652, 0.588226272001105, 0.2134302095426085, 0.3362875076034388, 0.49148088862559974, 0.009867284584679692, 0.10354388681048912, 0.21809718446556808, 0.6952897092016903, 0.3692787311320417, 0.5540965669025079, 0.5329557864173431, 0.6905754582099761, 0.9244225497746448, 0.5637832301158032, 0.5177207093958709, 0.42183435870622765, 0.5748875587542158]], [[0.39660666675296263, 0.5832122478956532, 0.5007637218830909, 0.873021784961359, 1.1410108461730608, 0.6782805644532695, 0.48830952929501426, 0.660434391678029, 0.8162313968472537, 0.8371204156098511, 0.07673620193091642, 0.47856327053759173, 0.18244022796196221, 0.4584346643923309, 0.3749638948087381, 0.35771846184509176, 0.5227297873201777, 0.922775972479319, 0.6649101893034302, 0.21196342445900485, 0.33423626583159566, 0.5016350435929119, 0.04446469754907548, 0.1248106292841582, 0.2593159874222423, 0.654710912800792, 0.33728392631085674, 0.49578358324736194, 0.5537840043658392, 0.6837265805781381, 0.905320234759069, 0.5696992469665407, 0.5392842738968219, 0.3943272016623346, 0.3687965003843521]], [[0.3750044192389521, 0.48496327561009445, 0.5725197570384303, 0.9718108816752123, 1.244445294526565, 0.669237278905106, 0.434494322382211, 0.6633288964082826, 0.9020402743296643, 0.8611038004554628, 0.07936169090463562, 0.4768565697472842, 0.1852082448919625, 0.45847208290302766, 0.4161929323656319, 0.3723303959931971, 0.5557937338603828, 0.9326714025900295, 0.6500368376855445, 0.21931685631607917, 0.3692368832216163, 0.5454304493968537, 0.017913352922863612, 0.1384881029370083, 0.3508853745515974, 0.6772068614585389, 0.2733133789595879, 0.43442316716998, 0.675030998151249, 0.7389599708784051, 0.8541581368405832, 0.6486442759707012, 0.5941825750255149, 0.4156596508933008, 0.24339081429654816]], [[0.46393126688800373, 0.5227901763397357, 0.6669104575026383, 1.1570341540024718, 1.3003473221755073, 0.7217490152559007, 0.499060816382073, 0.6633224017734217, 0.9368321978227203, 0.9103716794458634, 0.08889298421708614, 0.4793196155562815, 0.23662146313553828, 0.4731427880984833, 0.433412674233201, 0.3589077762793894, 0.5263004348799053, 0.8735530845624304, 0.7910152529979411, 0.17663584312694502, 0.3148827368231597, 0.704031583025416, 0.058637806392502434, 0.16110786438385394, 0.48546909608068156, 0.7541558741174599, 0.23712836936468024, 0.5583322043085855, 1.0618139586835487, 0.7485508433069199, 0.8144135222300002, 0.6930466688230071, 0.7270470976696416, 0.5010002712899687, 0.22986659203138793]], [[0.37188085377374386, 0.49630334036326706, 0.6375392276430756, 1.1089177837880404, 1.285966547339064, 0.7340076455094735, 0.4974339148442758, 0.5958497948028973, 0.9747591074544433, 0.9677363247070885, 0.08154839373916145, 0.4944139119909444, 0.28925420049382083, 0.5031168787783846, 0.492569722347822, 0.3331878909282604, 0.5144910352449946, 0.7134072392203623, 0.8163502593686676, 0.22535653386155768, 0.32358362828461956, 0.9053066522992879, 0.038876634610827916, 0.1958069098371363, 0.6637719597817344, 0.7589095371268662, 0.28165014561114216, 0.8195778293818505, 1.4870137152294065, 0.7300991194743947, 0.73120832168474, 0.581908810326786, 0.6513189777978445, 0.45944246334622246, 0.2957326284374967]], [[0.34051451924792125, 0.4944594184931606, 0.6591382218852014, 1.0406685568467158, 1.2758497119575838, 0.696734454957145, 0.5128935733181517, 0.5855324989792726, 0.9296664762402751, 0.8785671543214078, 0.08458229836671889, 0.5043129830567986, 0.30924301322140113, 0.468557003182817, 0.48780263857421974, 0.35876464027348803, 0.502944287986933, 0.7121437796443831, 0.7585387783035897, 0.2543815388386673, 0.3514684382316854, 0.9119550943548524, 0.07888497885174502, 0.27524871349916463, 0.8572667254211297, 0.7487514536848956, 0.2820317119877753, 0.9498901496041516, 1.5393852982678708, 0.686557272226763, 0.5009825308621205, 0.49830825225230824, 0.5800917718890598, 0.46267282866076265, 0.368243945670804]], [[0.40419592329583437, 0.48883826007593867, 0.6751536071577455, 0.8727523967246499, 1.2557631831997418, 0.6794768917095282, 0.5272247867859734, 0.5883473989959436, 0.9136472036647144, 0.7216817868909495, 0.0946704307035095, 0.49376470496834723, 0.32216493446297023, 0.4592560490278829, 0.49290033573961806, 0.3468264488536691, 0.5409118791483991, 0.745644866296663, 0.710296405624943, 0.25022948056270977, 0.3683689624929121, 0.8313922882848469, 0.05067976680990735, 0.33168342537031953, 0.9426197398025586, 0.7340757087540984, 0.2713432128714092, 0.9488862785110744, 1.515611570626584, 0.6738557747467326, 0.31504923772504934, 0.4376243057484791, 0.5215354919853501, 0.4675387444687864, 0.42761740584047925]], [[0.4177650965227432, 0.50863142836081, 0.6411257080059485, 0.6536134548146841, 1.2391814818467124, 0.6893957189802015, 0.5890304210565933, 0.5446322056908255, 0.8997442270013526, 0.6732245471846364, 0.0867958438673317, 0.5168838282138999, 0.30397072024230704, 0.47068908947264787, 0.4854946353049037, 0.33419700847551875, 0.5345396110089019, 0.7227583784694925, 0.7279156454674134, 0.24566451631847597, 0.354384742527826, 0.6488641527236171, 0.053772338775256036, 0.3612724529280531, 1.0872673236710595, 0.6829572147507135, 0.26030414221788095, 0.8905689858829655, 1.3815231882427996, 0.6534781366411624, 0.2487063424858823, 0.4119699085428218, 0.46745714245254133, 0.518970733780015, 0.429219614100917]], [[0.4517305641464373, 0.6661625430351429, 0.6205579906310619, 0.5585455441366984, 0.9894686384708106, 0.6877187534878821, 0.600012501167958, 0.3582453861809071, 0.44146056292497976, 0.3992534082468622, 0.08301729824072782, 0.5256951281987354, 0.3009289858714837, 0.4535986522089433, 0.47538629281247474, 0.36145989793620803, 0.5316250836679973, 0.7072832896424144, 0.7457298378558297, 0.2497671966831233, 0.33256286385141637, 0.4989076511270973, 0.05120334467286866, 0.4427584450584477, 1.2978916140786563, 0.6231984012122457, 0.2737439152332539, 1.070808914700267, 1.3214726779860484, 0.6122175292343079, 0.17569900037877295, 0.4383717465151483, 0.46628929725028334, 0.7033252159708707, 0.44357715945990006]], [[0.4102085689618572, 0.7086453166193303, 0.6253079166930703, 0.5035142929799036, 0.9261244155805252, 0.7322368542517915, 0.4727183187508318, 0.2978566111291596, 0.18233922685921947, 0.24733405545749887, 0.10559552585780149, 0.5503902802294387, 0.3321579663832492, 0.4632094517693933, 0.5029336585827273, 0.32347251397019827, 0.5355324508669297, 0.7023469483885567, 0.8099571040358965, 0.2670726231108199, 0.39917598629076373, 0.3393859062980865, 0.07307800960653413, 0.5805705197713167, 1.34885894964292, 0.610551594382265, 0.40338413899838865, 1.19031711158443, 1.3465062985450653, 0.6207079475933079, 0.18002798869059863, 0.5394315055172064, 0.4947003697478738, 0.8559993901971517, 0.48421906228102396]], [[0.3848851081801705, 0.6904052121640477, 0.5755929982282086, 0.49483024277114457, 0.9978418068079307, 0.6893338561852772, 0.4876889983089754, 0.266611576572455, 0.17778468606566739, 0.18695218038066888, 0.11533399085074125, 0.5370471764412612, 0.3229594907526172, 0.45995332747221773, 0.5066078752760794, 0.3357985267853492, 0.5512178413760243, 0.7820209394713442, 0.7904422573449958, 0.2899210250241946, 0.39403407616500236, 0.29375850553808514, 0.07315004059573604, 0.8430931008705056, 1.4523423126618606, 0.6104240343747591, 0.5890277144749447, 1.1069006671515078, 1.5132626558245097, 0.6535627682364932, 0.24300135617227597, 0.804530090677311, 0.49749840453596056, 0.8771257479798058, 0.5169077885426295]], [[0.37250138504078484, 0.723797670613082, 0.552877741616702, 0.48010097659872386, 1.0450633189568406, 0.6649260145885417, 0.5018158401727444, 0.24469852596397437, 0.25177664358948837, 0.12640834055732003, 0.09441815137205248, 0.511222971198565, 0.32152329157022386, 0.4529162234402601, 0.4834269674562454, 0.4527165260279732, 0.5083210526572404, 0.7661506761038506, 0.7975102921076683, 0.3011756265946627, 0.36103286926796785, 0.300004209908017, 0.07748199268709886, 1.1066583268443038, 1.6529983586339456, 0.6954969386713639, 0.7366171990671944, 1.0463593283917896, 1.6666374627511376, 0.6752916462260483, 0.3256136013775557, 1.0682888673901392, 0.5092088607837071, 0.768235071762055, 0.6017092074680973]], [[0.3321416890877963, 0.7301063108296, 0.4141240755471055, 0.4986939834195895, 1.0530993365061485, 0.6938975418252875, 0.4899452214145488, 0.22388534652973657, 0.41856615556990706, 0.18645831988662032, 0.1282673493071728, 0.5095362859249603, 0.32393829850899614, 0.44102285726606705, 0.48582378734733817, 0.4168315677252916, 0.44224369830820603, 0.7356119084490751, 0.7298672134014387, 0.3110163318848612, 0.36062212098837754, 0.29451651034081416, 0.05527800845090917, 1.2477738887999796, 0.2573255212126311, 0.3172222974930904, 0.6657458151341268, 0.6881818043641114, 1.0510400539584994, 0.6601070853593693, 0.4069698913616373, 0.9712997631296825, 0.5528692951205492, 0.5876140225043782, 0.7698632125505211]], [[0.33548870731864755, 0.7618676062649123, 0.3906691460550325, 0.5533866702464378, 1.092003339742302, 0.6763550183672773, 0.5133745943503274, 0.2473408603845818, 0.5187887357064612, 0.20031720523728558, 0.14134605562866734, 0.5102034753286746, 0.3245811366382537, 0.45909520269657395, 0.40001758128015646, 0.4574850745726008, 0.46853165686297543, 0.7431239497022213, 0.7498245951932822, 0.3280451852432713, 0.36379087827186496, 0.32375713012184787, 0.07811236381098514, 1.1735330918126874, 0.28709894172887124, 0.3057433000495833, 0.6018066527676714, 0.6943146553071922, 1.0648792153484832, 0.6711255629999997, 0.6411988032664552, 0.983490482965673, 0.5790320994608353, 0.7843115303992287, 0.8891516127232639]], [[0.33741266732541664, 0.7210399697283771, 0.3821749940562541, 0.5673911974825165, 1.047735535787339, 0.690523002103226, 0.5111018959955242, 0.4294780253372455, 0.2919403007469208, 0.14327191375507004, 0.1278868012743562, 0.521667895909993, 0.21572573135386877, 0.47585740175959507, 0.4610423828991666, 0.3813008034884119, 0.4131645975082323, 0.7597781336953829, 0.6734289913943307, 0.36379463455949856, 0.322177460954019, 0.3432656729704249, 0.09690219691904953, 1.1727001048616135, 0.3015260868664923, 0.2706766431704588, 0.5809698219624639, 0.5505209826824433, 1.0834934385307124, 0.6494147465239192, 0.6249450162429057, 0.9810934065420005, 0.596043098030534, 1.0226087041257719, 0.9555027835983414]], [[0.33985420141141864, 0.6942977783496719, 0.40712560784307794, 0.5562497863730251, 0.9900235506380193, 0.6733533961947289, 0.5075417165087032, 0.5551536538631084, 0.3656592532698933, 0.11491927574109981, 0.14746160401182992, 0.5149331812239427, 0.25037434452897317, 0.46697138980727504, 0.4074121900155651, 0.3706212534489062, 0.38611310362631796, 0.7642742854970479, 0.6816859412490222, 0.3703382374077405, 0.36123394856699015, 0.35641333567547145, 0.11170701382626183, 1.1454503680910004, 0.3613596010949207, 0.23079607195522012, 0.524554959817949, 0.5637359097400277, 1.0451369173570573, 0.6536439890315925, 0.6663031552414393, 0.9179323566778264, 0.567941179451823, 1.1457614825982816, 1.005782327387066]], [[0.3159737646943537, 0.699073791504953, 0.37498835901161187, 0.5826010480296214, 0.9923193807988266, 0.6720850474003883, 0.49094283796231675, 0.7529550568473451, 0.4857187807423804, 0.13378160248397705, 0.18005863424914417, 0.4976986277807803, 0.2826483616291611, 0.47787908778646715, 0.40647636799986847, 0.37932233286783235, 0.4106656252870785, 0.7820079150908799, 0.6686724870460856, 0.38643808599169205, 0.4425284959403922, 0.3672677805124035, 0.1467584932037539, 1.0249059178986963, 0.3971586171941481, 0.2576944437179931, 0.48636545217184896, 0.6075296489814512, 0.9991892114720942, 0.6858328473921369, 0.8144101242011308, 0.8248319991709806, 0.5279562328879221, 1.1992299479493367, 1.0428998473005717]], [[0.30042102417582695, 0.673552651248332, 0.39104060653309225, 0.6230931726439802, 0.9158641620628002, 0.6999964490587275, 0.43795669659534675, 0.9167726037532167, 0.517898177072819, 0.14662929217620824, 0.10579342867342209, 0.47126011810942936, 0.33178309903590353, 0.44872683975237937, 0.34834555314838106, 0.3961335120426004, 0.4202485108270931, 0.8063325660409042, 0.6337542119369778, 0.4196395171996219, 0.44798727929006715, 0.36701491553012394, 0.1806820198770776, 0.9232305158902037, 0.4696387840042674, 0.305105083564837, 0.4834717334449131, 0.6549719890321426, 0.9593542925728238, 0.6176903079451341, 0.8189502223195078, 0.6420737747255649, 0.4670540053607567, 1.2058567952273325, 0.9980109423133876]], [[0.32987716796513006, 0.7520335823518944, 0.3851144896669636, 0.6611192997274493, 0.7299670777716307, 0.7044698227693429, 0.4368585996663191, 0.9523772741635488, 0.379825974039024, 0.1295901653490934, 0.04487659097280145, 0.48967194427350674, 0.3943685557427233, 0.460146236491916, 0.3517171153260422, 0.4004829200860984, 0.40632475077813207, 0.8098173790823889, 0.639884883262968, 0.34686696531051886, 0.4224932254008947, 0.36539723967668025, 0.09971180380843447, 0.8381735314204617, 0.5355925495896413, 0.3917942857302459, 0.5197399928947373, 0.6678817253355296, 0.9491540580151308, 0.5571029265192482, 0.8932140690530661, 0.5032907696311233, 0.3837593434401086, 1.1732268221293054, 0.8913676573956377]], [[0.4086787094415487, 0.7208796618380511, 0.3407158602379189, 0.7547253141187912, 0.6422069200342794, 0.6933588647824328, 0.46070304510433935, 0.9059517848735817, 0.3480150971021792, 0.12185265512305254, 0.05402288643157451, 0.48346624632386725, 0.4724309129629964, 0.462590750808254, 0.37327017097178444, 0.3811754924885704, 0.4213830108226241, 0.8520206759192266, 0.6351033618007214, 0.3482785638298096, 0.4179760225746219, 0.3857739086431844, 0.12717966954202828, 0.8295321976679287, 0.6084350279055981, 0.4811132839514024, 0.4520520735198936, 0.6718488351197446, 0.9791590148008533, 0.6141215784478581, 0.9595302069806079, 0.4999137625345531, 0.36839455062938165, 1.1599435946134715, 0.8189921410906655]], [[0.5066459156466995, 0.7283514462640006, 0.3262149240611242, 0.9903221491874615, 0.6917277634371188, 0.7058442485683099, 0.45283306806400075, 0.858812224132901, 0.3581582823747327, 0.19520634784479124, 0.058026090550270915, 0.4798021009684009, 0.5531838990956759, 0.455133888595377, 0.38536627121035355, 0.329483425555966, 0.36995944466118713, 0.7070520799014763, 0.6257951876739434, 0.36254341865720063, 0.43457582959286584, 0.3925407479366114, 0.16667698995631347, 0.9542705077046802, 0.6875318582319796, 0.5537226508789127, 0.391345143686454, 0.6313204801405647, 1.0656253905710578, 0.6561494117541966, 0.9775812506908781, 0.5207416078911983, 0.3599638416869258, 1.121003696417442, 0.7115787070341781]], [[0.5022788306831598, 0.7832655529333776, 0.33373997783143006, 1.173653823564519, 0.7241154791960382, 0.6351159645472926, 0.43424113393785113, 0.9208562667820296, 0.334362050239967, 0.2953013719325213, 0.04986690251056661, 0.4902275962822507, 0.6202920432948984, 0.4630994555244043, 0.35905673328287985, 0.3302628845287503, 0.41314002449647425, 0.7207909145962526, 0.6388737844034218, 0.3723417090299127, 0.4783972517757913, 0.3801351050788443, 0.15971487375292096, 1.020267273139452, 0.7390167076177898, 0.5949101295192059, 0.47774792645103703, 0.6146318302192367, 1.129840254798985, 0.7300948165896989, 0.997346117578263, 0.5488360214692622, 0.3909065926932701, 1.0814214311932684, 0.627833836265959]], [[0.4937910550389926, 0.7801148973551255, 0.36452284073536956, 1.4283798341838656, 0.6851580747169733, 0.6552936333893058, 0.46033950175507005, 0.8336451116636137, 0.32975985205646896, 0.3461149084659566, 0.04325855920289544, 0.4792635002561988, 0.6809179565160794, 0.4642674401112505, 0.46689307674244657, 0.3842313740893136, 0.40902606696033234, 0.6623754949039111, 0.6587556822584373, 0.3398900880168766, 0.4191394238264327, 0.36251280797201585, 0.14983013711907422, 1.0139382460232622, 0.7999716511359198, 0.6052698067961196, 0.5147122943917761, 0.5932968941311153, 1.1763834420484565, 0.7096608913330796, 0.9550237651129828, 0.6157979867125829, 0.2696453230179041, 1.0528799643360092, 0.6860781991913272]], [[0.4711225916330936, 0.83184634899481, 0.4214991819617072, 1.687914236238503, 0.6666277488229679, 0.6921357280738695, 0.49572130799397296, 0.7083858461713441, 0.32442204733338054, 0.3678296268957227, 0.08455918178601851, 0.47494018967392465, 0.7962700120322148, 0.461296545631359, 0.49539775433388195, 0.44959810716332127, 0.41912442691781177, 0.6584856175126064, 0.6803775496232713, 0.3814562131046979, 0.4376900829587777, 0.3824453581328501, 0.1778504413541545, 1.0314484831160313, 0.8503073240413237, 0.6025647811830322, 0.5566690558589598, 0.5801726383297261, 1.1800161225981052, 0.7767596145325422, 0.9054078137342088, 0.6891580164821438, 0.3747503427964755, 1.00818569954856, 0.6319552689435253]], [[0.5017369860942384, 0.9473122231851018, 0.44972025884641714, 1.6591862747277306, 0.6559402001337862, 0.6283935623074168, 0.46372282389873926, 0.6586259974501176, 0.3651651802895688, 0.44027623101594693, 0.0854957919918692, 0.4562276100419681, 0.798258963134013, 0.4696560252069286, 0.5043050410081227, 0.5228935358803464, 0.4064387623426527, 0.6707094488648083, 0.7194178817712025, 0.40216913466819626, 0.45785067188181056, 0.3757269110240002, 0.16625571180226872, 1.0245972053755747, 0.8580351507167077, 0.6443892624109404, 0.568569073759639, 0.5672885172100331, 1.2034333556843553, 0.7837390217510078, 0.8337081186829118, 0.7671841867890373, 0.6445445772081123, 0.9863119276806647, 0.5246880040653874]], [[0.7195661573212766, 1.0152758305251295, 0.4645458123982604, 1.8336782627972903, 0.7056901200460336, 0.6732922385367475, 0.5168811126769559, 0.6294381992652666, 0.3357354324184765, 0.4449526358458571, 0.08691511483904983, 0.4413149894543633, 0.8590709885702104, 0.45747459119776807, 0.5620257118522269, 0.6411403044648046, 0.49063316266834367, 0.7218241627258476, 0.7712116902097493, 0.4064567612651595, 0.5037178488696799, 0.3716849060854889, 0.17506596419323717, 1.0100896977373164, 0.8937924326073552, 0.70638477758566, 0.559346897410816, 0.5607163947789353, 1.2162338351456992, 0.7517140008978528, 0.7707911583464904, 0.8221231652019674, 0.687024582952236, 0.9505113036791776, 0.4377752949276787]], [[0.8435764909676817, 1.0418125431480814, 0.4823979048422668, 1.9659209655104217, 0.8552548141951811, 0.6571649203283538, 0.4924688228825207, 0.6236840209021093, 0.32252753414933233, 0.4584954487338003, 0.08999059541690352, 0.4447299983559474, 0.8927337177738688, 0.4448425703444714, 0.6778328247102468, 0.6760788506292251, 0.5237157192076457, 0.698271868914644, 0.7597919167999443, 0.4014496555960906, 0.5387600866770657, 0.3767064289042066, 0.1383596156109305, 0.8937017178787354, 0.9405072837905882, 0.7778791409745052, 0.5648374448682522, 0.5582679066531236, 1.2027540613016439, 0.7488690177012964, 0.7274614762819369, 0.865489713253125, 0.6756839823293085, 0.9115778403169961, 0.43779572564514624]], [[0.8371018451205188, 1.0743193354904297, 0.49114733713174574, 1.9940866003232243, 0.9878253685273327, 0.662986932102615, 0.5082418390011376, 0.6526443184144772, 0.3724661636378153, 0.5284385776612497, 0.12133690018019075, 0.4883976683207546, 0.9554111660589641, 0.49467975222122834, 0.713627467843019, 0.7030926508916805, 0.5163035332180623, 0.6916860733430374, 0.8187668215159842, 0.35177483405868615, 0.5436193460959056, 0.3253574325293198, 0.13115860794631842, 0.7439872503423026, 0.9159454524159927, 0.8065691857550004, 0.5643697611271341, 0.5440408428991301, 1.1970332831557196, 0.7082480553047694, 0.7141387190011423, 0.9088252973722173, 0.6907776261083666, 0.8843057068837507, 0.447068756670939]], [[0.781928812595202, 1.159922657728193, 0.4994432129083731, 1.9524293241334894, 1.092659735466174, 0.6218486284670569, 0.47327713000617927, 0.6850401438864535, 0.4292215970360006, 0.620279870426969, 0.19208222865346322, 0.5112946322810734, 1.081512231223714, 0.5157810223342391, 0.9069720062613345, 0.6957063189181557, 0.5551872031633615, 0.6758051698269555, 0.8485225736760046, 0.421620411202422, 0.543188527166254, 0.2545782390295566, 0.12912711471245295, 0.6113319870328913, 0.8270912745913036, 0.8076351247325673, 0.573466834159353, 0.519044263226656, 1.1972592047428625, 0.6025603188320999, 0.7345081676143089, 0.9217762839144563, 0.6722063547479022, 0.8727680521049885, 0.48365104967600614]], [[0.8667739512954172, 1.1583485843687518, 0.4697612314759873, 1.8350315290941592, 0.8822025579073309, 0.6495337830977389, 0.4491724618579903, 0.6975642444190423, 0.5239668442510464, 0.6834500990850485, 0.09337875528415987, 0.49683140186638797, 1.1599847819844002, 0.525985641128611, 0.9294874308232691, 0.613661622229346, 0.5523579527036112, 0.6655394416881227, 0.8530596392904104, 0.40413360071702553, 0.44055886242954556, 0.2881612943082572, 0.1406703299956421, 0.41944863396423676, 0.7316553702936428, 0.7127084547064313, 0.5053569866008996, 0.510490865316036, 1.1064016981155598, 0.560044737482601, 0.7569196764323622, 0.9883781530245042, 0.6277813747462888, 0.8635618301267736, 0.637135314077077]], [[0.941330052714759, 1.1475342983042913, 0.46695722816712965, 1.7960167093354504, 0.8676855548815217, 0.662373035996823, 0.4645056881305918, 0.7118403451974404, 0.6532473582732672, 0.7350660706291026, 0.09367973224068976, 0.5748233632426432, 1.1679639336305727, 0.604304615856261, 1.0929209597615022, 0.4375594421516439, 0.6014102791869879, 0.6770107015953237, 0.8477080886258432, 0.4273092652703476, 0.31628348279664403, 0.23572317515726082, 0.13541795875059004, 0.3817506620007769, 0.543983154212005, 0.6501696274595256, 0.464391426595377, 0.5082739028437446, 1.1210835809041755, 0.5735276232956763, 0.7806071023079285, 1.0114865779062359, 0.5197773924908375, 0.9831709720487825, 1.0251175541482782]], [[1.009062676321997, 1.1185728121879053, 0.4619166658252248, 1.7772938844113781, 0.863521599004504, 0.644834434775923, 0.4551930726877942, 0.7520650197117392, 0.7945535545899542, 0.7554836918071753, 0.09411621007040229, 0.5548908025121162, 1.1692775959025845, 0.6337092332352499, 0.8652276193602909, 0.4445253713472277, 0.5390638860403308, 0.6594729584336927, 0.8280899816623686, 0.394609310719384, 0.2168160069063096, 0.2049155461697139, 0.11778892375045524, 0.2990678719842671, 0.45636738194373183, 0.572308014164645, 0.5310268826554252, 0.5151356617604796, 1.0966126691425127, 0.5627207565023479, 0.9282996033044758, 1.0297930410432181, 0.4679129762504538, 1.030232162781993, 1.3529449180056412]], [[1.1707917413717193, 1.0955844933107781, 0.39987423327373883, 1.7189587130180517, 0.877206276570692, 0.6436323001860049, 0.44664425638358585, 0.8684771764744649, 0.9206135469121943, 0.7697136731975178, 0.09547202384416084, 0.4974871916485966, 1.2465824602474942, 0.6374683067730846, 0.937917107612307, 0.4437216451280245, 0.5536172547706617, 0.6674234275650242, 0.8497501449399545, 0.4310554480232923, 0.28722967449608106, 0.30097942164077696, 0.10289725031422622, 0.3342270957902864, 0.3787659676727443, 0.5799007657539325, 0.5112980030725929, 0.5071154825920207, 1.0219447745576713, 0.5112614232430196, 0.8199325297114841, 1.002184481909361, 0.4334368581825554, 1.032624082215616, 1.349484163021109]], [[1.4050269219238438, 1.035890004180572, 0.34046945723478605, 1.6943535955842297, 0.8918106157368834, 0.6506560734352687, 0.399890001947226, 0.8772305536385817, 1.0528537740172736, 0.7794780724505155, 0.10765628433186897, 0.5533908809377825, 1.2174989435330146, 0.6339054006731111, 0.9764446128921191, 0.4696675116660951, 0.5652719772433673, 0.6849356499889986, 0.8235861416143189, 0.4845170275331706, 0.2461472104005658, 0.4437246287430678, 0.11850654484764846, 0.34649357204460535, 0.3570678908341849, 0.5981303981397788, 0.4455330553855229, 0.47656508344723814, 0.9816218378488393, 0.5002275199401517, 0.655792258687055, 0.9847555887071419, 0.42991166716988816, 0.9086370776685627, 1.0484159423019714]], [[1.5144892946370154, 1.0237182113248309, 0.3066434469551592, 1.7177266125553274, 0.8931063594941772, 0.656124772243089, 0.397231842606446, 0.950526250806734, 1.228132900053785, 0.8021431642928242, 0.13901500389643753, 0.5281179323025527, 1.2205946694419276, 0.6157070637202249, 0.8798167215130098, 0.4311711001727203, 0.5006634982290169, 0.693967589821411, 0.8320527648936817, 0.4545422590209103, 0.26779923738236894, 0.47156000742459137, 0.08832500747539698, 0.359580786387458, 0.3134411349229954, 0.6138057946013653, 0.4712932215007853, 0.43220922045913057, 0.9547868332845518, 0.5677193500583613, 0.4726087314126647, 1.0124463484021506, 0.46934981884068544, 0.8080399362563384, 0.904252710901958]], [[1.5278832527160766, 1.1653763843859526, 0.33255325462098684, 1.6691437756496923, 0.8796675477956957, 0.6669480974207574, 0.396990967028245, 0.9409993284185342, 1.327958294936621, 0.7731312683388807, 0.19983276387047455, 0.5238009529562417, 1.195177030564107, 0.6235492078317094, 0.8762037353675503, 0.41117252780154856, 0.5142331029815272, 0.6928743333513794, 0.8377931213451613, 0.3973297008371184, 0.28310576244773034, 0.484005310736254, 0.10026278681268315, 0.3499622908882204, 0.2966019262521274, 0.6213316539054656, 0.4414529780471639, 0.4068478889918459, 0.9018944010111558, 0.6625101647679046, 0.313771342097478, 0.9989515858937075, 0.45877976154320615, 0.6565129138524746, 0.8662570969216761]], [[1.5390887317134494, 1.0559323664171036, 0.30040716433020587, 1.6037643936374615, 0.891183517196777, 0.6425198411157119, 0.3909824448348989, 1.0223149618372867, 1.4538406984035233, 0.7473994246179343, 0.30913641783593493, 0.4944335216900746, 1.2533331261596836, 0.5736140029911267, 0.7197779329652512, 0.41032799943038123, 0.4633836052029972, 0.6730822572504511, 0.8348670839058716, 0.38065875011705874, 0.1388986013624367, 0.4379652018627847, 0.09478978847399136, 0.31690248048379077, 0.23942835116049224, 0.6446913458130854, 0.4365736387019686, 0.3689413098439479, 0.8598684086536819, 0.7076183881169436, 0.35166102011144973, 0.9769806423079743, 0.47598785798455573, 0.606006963225072, 0.8560302911686368]], [[1.54746467100256, 1.0124544131697946, 0.30007392496307916, 1.5256210024775216, 0.9199391049562154, 0.6328035741591171, 0.3803653149427404, 1.1226124696823245, 1.4878245477859637, 0.7047484028873475, 0.38742995855198187, 0.4689763532073765, 1.2046903428924685, 0.6297213844567991, 0.6498902629554871, 0.4259137011680053, 0.48074243018475826, 0.6882193730380932, 0.8294299179506064, 0.37836231624438754, 0.1324803212446048, 0.46531616149416766, 0.07016307682514483, 0.3102797894099152, 0.19769924969216282, 0.6550752803221683, 0.4345125852154893, 0.3209527804746222, 0.8391285003372493, 0.7287012187206992, 0.5648058887445799, 0.9612339430347692, 0.5066970845719635, 0.5316289729925743, 0.8805377290755243]], [[1.56778925417433, 0.9919496827636705, 0.2756876251812237, 1.4640169582412632, 0.9397283158011567, 0.6836438310710051, 0.40367208953827566, 1.136995229752962, 1.516408844357204, 0.7328348738862562, 0.3487700819445968, 0.4947849914960968, 1.2024139926736825, 0.6969345041006358, 0.6535713581562841, 0.4634825999247121, 0.45339694931711705, 0.6730986338875476, 0.8366041734270766, 0.4128662594313115, 0.13606313138893195, 0.2933842576671512, 0.06582569924028589, 0.3079022418327223, 0.19791768863866688, 0.6710892276409397, 0.4306323104285461, 0.2689997257340811, 0.786636837261836, 0.7116234752323924, 0.4522167332146425, 0.9124738798276875, 0.5431056314461433, 0.4449660828210196, 0.818262208594008]], [[1.5941440136624343, 0.9538384029643835, 0.27880303166441933, 1.4495545988219638, 0.9467223249924812, 0.6917460547860076, 0.39490438342431583, 1.1309520055239604, 1.5583817816958636, 0.649624626985026, 0.31749126271280514, 0.5521415456898883, 1.1465650469563036, 0.7507260591738283, 0.6427371087293682, 0.4212268219437799, 0.4436171027105241, 0.6418417876503413, 0.8567064508616217, 0.41916939604463377, 0.12515448398662743, 0.25305901831889, 0.11945134096814153, 0.2778478923121338, 0.1981731417191912, 0.6884042871972651, 0.4727599616114872, 0.23572478528860072, 0.7168242319944594, 0.7056539077816841, 0.4037060269989421, 0.8381818167933079, 0.5960036809654492, 0.2962365736668041, 0.7575990097972413]], [[1.6276885066573308, 0.9429329050145873, 0.2910318654284135, 1.3801396780785713, 0.9394094737450616, 0.7164666738833039, 0.3967125882451365, 1.1374207356957449, 1.539813017648492, 0.6479358861964837, 0.29013623582646314, 0.5217121057179691, 1.0440920024579923, 0.7761360998486808, 0.610128637036666, 0.29837869997685684, 0.4687475323971392, 0.5910107411880411, 0.7326012628829093, 0.4250432133339751, 0.13944425081105571, 0.2615571568865068, 0.0893703604350059, 0.25857378117791013, 0.2723149461432822, 0.6270030468021006, 0.48357401336411343, 0.26561783629298974, 0.5820926266309436, 0.7123110469674427, 0.42606164528927076, 0.8114840603709361, 0.6646045072693185, 0.21786061580541166, 0.7115835755827674]], [[1.650110751173972, 0.8781991279228671, 0.29057488307767687, 1.3645822744226397, 0.9259333208796053, 0.6982798903034924, 0.41352785539345427, 1.0652406274654125, 1.5161800422629765, 0.6287099279145625, 0.24976979978813663, 0.5437181340436211, 1.0476336787494844, 0.7536984897400968, 0.6017088573054926, 0.30183964966541615, 0.5095101410204503, 0.5519528218553723, 0.6803900836528036, 0.4635982521246724, 0.14383440229672603, 0.347636679197977, 0.08305737476354408, 0.2636687753392323, 0.5051917733565018, 0.5353864615431966, 0.5191062997926493, 0.3255903464392316, 0.49077131391028217, 0.7143989201238197, 0.5532873800938758, 0.8306467289216102, 0.7176022445954657, 0.3550900630293562, 0.715667258820446]], [[1.64638455006517, 0.8366982741713589, 0.25545374092345485, 1.3057874430404615, 0.9340873274090876, 0.7239490536053459, 0.4312689984966658, 1.0579966738942792, 1.5321412945588413, 0.6078658667294067, 0.2205154827529754, 0.49939332411684534, 0.8295767688996508, 0.7135709336521544, 0.5764268105989878, 0.33164500848240086, 0.4192657413460328, 0.5309426636186099, 0.7668457957809452, 0.5388206412185647, 0.14665207017860607, 0.3257394954360826, 0.10980472940258998, 0.2873501001595156, 0.4801122651160463, 0.4868085803586932, 0.5484283056473732, 0.4674148669382856, 0.39738044182569776, 0.6791092488160575, 0.5422114958908006, 0.7732422718899117, 0.7029930544499636, 0.4402087273441019, 0.70533811708155]], [[1.6528391373390143, 0.9050926621809587, 0.26163117065866637, 1.2658015166791279, 0.9300832724552665, 0.7087167268650387, 0.4193047415682414, 1.187908575820596, 1.4984606580212347, 0.5765494851786143, 0.22478954733883919, 0.4735699502662276, 0.7218718295498943, 0.6376790858057244, 0.5581187075060171, 0.3337957540918135, 0.411941645371221, 0.5296159659234587, 0.7687101463911873, 0.5522445096857295, 0.15468270846304483, 0.27903782358955354, 0.140963900640748, 0.349274914898848, 0.4271551253969402, 0.43126385396481737, 0.6152456246776133, 0.7497151384006018, 0.2453301146125595, 0.67394271682923, 0.5783468458283543, 0.7028647115277968, 0.6908148855041429, 0.5363039076955751, 0.7691800470659331]], [[1.561428934898889, 1.0096528683146153, 0.3135461084058532, 1.1847754704381483, 0.9614206995905563, 0.6950852021615295, 0.3936057785958686, 1.2315334191146945, 1.6266899620665591, 0.5558821300828463, 0.22818703803010526, 0.4797860086706769, 0.7696072165247733, 0.618941855044252, 0.6755697775440828, 0.3451267082563113, 0.46460314066911446, 0.549783019660605, 0.9046208791099808, 0.6536751097183658, 0.18047056083538462, 0.2649569997150193, 0.1645103602200904, 0.4368317888840527, 0.38937994616496, 0.37253556971406765, 0.6732540239004804, 0.732176272430441, 0.15404352503501967, 0.6553146591148107, 0.5920918291719162, 0.6337497598124504, 0.6838132297156079, 0.5559480315060736, 0.8316020236201644]], [[1.5747710281809866, 1.050412500579022, 0.26394276011990925, 1.0855074499987505, 1.0040912886891777, 0.6988295729675139, 0.38356665510893756, 1.2186904481606955, 1.6548970608150337, 0.5396329164821647, 0.21752440099067183, 0.5104648046035474, 0.7680593462755159, 0.6048329134095348, 0.650149329223221, 0.3130561305764321, 0.45318059725851445, 0.5392090627860859, 0.9109158247369809, 0.761051488593594, 0.17741714940863534, 0.2444086639717421, 0.13885806648889923, 0.5410851821383269, 0.33366021777333077, 0.32568225738847, 0.66997639821834, 0.6094925974725602, 0.1315957118622596, 0.6366387165062115, 0.629362772516937, 0.5848422255771861, 0.7052834363005, 0.581350044555188, 0.8624795566739702]], [[1.653090730095035, 1.0715544604602931, 0.2606954522971212, 1.0918648859671238, 0.9900015802911932, 0.6832796638968548, 0.3876222009550817, 1.24104576057589, 1.6564730773531524, 0.5421655947090617, 0.18575406688946516, 0.498338482205123, 0.7511751523378307, 0.6034033959056274, 0.7821958316389317, 0.35117800571082436, 0.4370110381695298, 0.5882697669248388, 0.885487185595523, 0.8035590329428772, 0.16662026202959235, 0.23421765752927254, 0.1427517602191997, 0.7145812950213987, 0.3218710099472384, 0.2777496729797903, 0.7353996157699578, 0.5668408911498797, 0.11741621685906628, 0.6319174243938681, 0.6602655076745371, 0.5048591367839793, 0.7314922986763033, 0.64153094552751, 0.8417155808948671]], [[1.692141294358647, 1.0079347724450332, 0.2716933257664526, 1.0030523807692124, 1.0003692819536658, 0.6942792402247095, 0.3999979354066359, 1.1798171633771535, 1.6888201853611569, 0.5068007123470805, 0.183490279854948, 0.5091900853710366, 0.695366921325004, 0.7290180653133547, 0.718646964761819, 0.3518805345136732, 0.4302374035317368, 0.5973636007252016, 0.8518965867574788, 0.921121589110339, 0.14978388001235649, 0.2517379267465863, 0.15152533022555348, 0.8032753636102967, 0.397364511868571, 0.3507660574264392, 0.6226066649569297, 0.536985209450801, 0.14669532168630656, 0.6395634835849009, 0.6775780348174657, 0.4077479713380252, 0.829914102372463, 0.7915754673059741, 0.8514584620929164]], [[0.8691823700645906, 0.9392712677588827, 0.2682593049549382, 0.9985736565010268, 1.0065441972624898, 0.7189554048572333, 0.3919100747559248, 0.984774586430577, 1.847860542607729, 0.5753221294311589, 0.18896195945071967, 0.5070287911934069, 0.840057248065248, 0.7464275322899008, 0.7217420686740006, 0.35187905678150877, 0.45844893875022086, 0.5402574645781534, 0.8816708600429927, 0.9334212652789716, 0.14693654027563957, 0.2679705222903711, 0.23021978042088304, 0.9422271956431574, 0.4527595428496072, 0.5469820278131284, 0.4985027361432981, 0.45185446139112756, 0.20346468053418149, 0.6519276244176284, 0.6334839850464591, 0.3341631980512049, 0.9315096443243887, 0.8376859128919605, 0.8820765607515679]], [[0.7954094036256887, 0.9684413564474607, 0.31337381841107903, 1.066468493658391, 1.017042829956047, 0.7183123140986453, 0.4333722755516362, 0.8555944763863114, 1.7620328697583603, 0.5033829261000358, 0.19993089826183774, 0.448923694214947, 0.9477123664068255, 0.7787661680676707, 0.7178639050280935, 0.4525161976812411, 0.5386221574329836, 0.5403904146929986, 0.8543073251137644, 0.9943515130251496, 0.12952968551580446, 0.22719145015822698, 0.22021700721568693, 0.9632651727390812, 0.5243403753530388, 0.9661784828365751, 0.4798880685591931, 0.4381770349392222, 0.39560145613610953, 0.6493165333120346, 0.5174393891186938, 0.36763711214064176, 1.0363366009323114, 0.7979065097505476, 0.9604352607078538]], [[0.7219327657807264, 0.9586790149765341, 0.3133099067330113, 1.09435488210269, 1.016503278624335, 0.7546733313708631, 0.46565515814634295, 0.8238136206201458, 1.7151842457278663, 0.47688195554406687, 0.20965455834346294, 0.47028720919184736, 0.9430999877386068, 0.7920230062576813, 0.6433667141461121, 0.44964185250272526, 0.562403141306918, 0.5355728466994223, 0.8470099858023289, 1.0415763047785376, 0.2355939980163126, 0.24520701914396656, 0.210937744853793, 0.9016315378193551, 0.6123273169014078, 1.0422825457149334, 0.5520203914139146, 0.4493344019540235, 0.5993346569419933, 0.664483994775737, 0.5812056534307237, 0.4776142565880671, 0.8706337503498385, 0.7530716062428786, 0.9872779660594768]], [[0.668399620428062, 0.9940487748925289, 0.3108114103887274, 1.0891761534345368, 1.065292089312179, 0.7413157921539208, 0.4616028676586829, 0.8123861703406671, 1.5595317790739576, 0.4680235797676157, 0.2315840333951624, 0.48283359087182737, 0.8770116842528792, 0.7062897760504625, 0.626661359993973, 0.4524458503488673, 0.5444212313801751, 0.563428758164681, 0.8476549167670472, 1.0375155222897137, 0.3236119707857532, 0.18693573054270898, 0.2173059414117817, 0.9466664389008768, 0.6656874927433457, 1.0756974992215194, 0.5828074887504952, 0.4682518077231293, 0.7140728076390408, 0.682117170417027, 0.7427108680207596, 0.5326976819531531, 0.7371047053187634, 0.7854888755168751, 1.0194905736695943]], [[0.6609397536472221, 1.0104769719516766, 0.3092404685397818, 1.1524070092931828, 1.073384829552813, 0.700594028371784, 0.46079791322427727, 0.7846758792433608, 1.4811613655562414, 0.4556006212020949, 0.21308568369521988, 0.5031650428337013, 0.9114556212487464, 0.7111031963922407, 0.5013932005728245, 0.4316720247039204, 0.5932362087162597, 0.6211333499353604, 0.8497112980840228, 1.0361733166609346, 0.32289683391795426, 0.19479253139288646, 0.2196375171441396, 1.0475948900993166, 0.6756056889508468, 1.1530429462481375, 0.43972556882540725, 0.46642167038671545, 0.6713893431955955, 0.6721748221661892, 0.8808360373712645, 0.5236465761079253, 0.7885212483878357, 0.849588317997963, 1.0343929991218883]], [[0.6737969177897358, 1.055361261267972, 0.2913369254298936, 1.2527534599900512, 1.0745721543296574, 0.6522882604342854, 0.4392145369933205, 0.741239862046807, 1.462669583537027, 0.4309814846737905, 0.17408109160255175, 0.46407769061988924, 0.9310105437032145, 0.7057917899733137, 0.4856077997032124, 0.4122768098855789, 0.5914809689109517, 0.5695283944839873, 0.9331884062632709, 1.0243059221217505, 0.3108769194781723, 0.20493866519280607, 0.20443765581453222, 1.0850899787537125, 0.6835110997482211, 1.0480308152072533, 0.3456323907972776, 0.42372792906738566, 0.578390850661177, 0.6623247152085817, 0.8764607859973925, 0.5419079261145319, 0.8563936479553005, 0.904268585105305, 1.013073744451104]], [[0.6666847691123277, 1.0878523630485546, 0.2819400899771099, 1.319130495315971, 1.0588321225971231, 0.6467977089979746, 0.4329483574541023, 0.8050581957256641, 1.3218078499148196, 0.40835652774547765, 0.16125940715684073, 0.48537813634948834, 0.9391714707369238, 0.6839907165016526, 0.48611261725487226, 0.39533175263577824, 0.6997319686529772, 0.5586754838589905, 0.8954943199704412, 1.0566212302124354, 0.3624126221641121, 0.21101351140828398, 0.20080037400965967, 1.120128686768428, 0.6738975794453776, 0.8954300767528154, 0.26709631691845, 0.5116948636666585, 0.48467708406424487, 0.6718635600883346, 0.8514475448090959, 0.6259027786581571, 0.8757512114702302, 0.9874265549001049, 1.009485712734406]], [[0.6349317211466717, 1.1675979738622306, 0.30056438897757326, 1.3492360146891864, 1.0561039504620728, 0.5777750050391657, 0.400482159019276, 0.7943660448542151, 1.1948681715913703, 0.45308181789318286, 0.13776224037881224, 0.49599601155471795, 0.9953120315395498, 0.7176358234123539, 0.4591687036659154, 0.36775770166132304, 0.6454377805118323, 0.49033424471484277, 0.8101082014927111, 1.0939367971600085, 0.4196164892665934, 0.21195428526496327, 0.18482586384008728, 1.1389039677555488, 0.6700689581896044, 0.800497742234928, 0.30575400660834867, 0.5302596130010184, 0.5176295306773506, 0.7035847643404746, 0.9480002742005078, 0.6958724270855889, 0.8779776090068483, 1.0466489643604981, 0.9710451308581987]], [[0.6498557327722577, 1.15723059718856, 0.283138348107255, 1.2842661434020988, 1.084644701815177, 0.6070198016420614, 0.4172442728166914, 0.7954241217293823, 1.4146141172577587, 0.5013247253329359, 0.15494895337207965, 0.5001967941607713, 1.1408061446503703, 0.7262416605902207, 0.4448652038539611, 0.34058920187646213, 0.6950611344874774, 0.5152410311802152, 0.8645170505742916, 1.116287289530014, 0.5023692310385981, 0.18221342843301533, 0.18780823452115242, 1.135427554395398, 0.6681138492664083, 0.6925771231111832, 0.38036838034544374, 0.5270018608328032, 0.5877600769480468, 0.7328794276916442, 1.0899394154965254, 0.8648725705191983, 0.8702565982736804, 1.0833173177947133, 0.959188096806343]], [[0.6721111507400407, 1.1414390101875205, 0.28204406877443905, 1.2843766916319899, 1.099336314716572, 0.6148388182373418, 0.4332564035435563, 0.8134055825872292, 1.3533362535275995, 0.255643021841082, 0.13430449996049432, 0.47732030877016085, 1.1353479851717738, 0.6871651009889048, 0.4342680865196257, 0.3431402977661947, 0.7165662708174124, 0.474647842997001, 0.8737983560959715, 1.0771523773407021, 0.5385975738216862, 0.26389994497914576, 0.1844949914426698, 1.1395738121446357, 0.6397047492769785, 0.47789624175206064, 0.6046853675254122, 0.6259071416267099, 0.57364832335223, 0.7982010815369922, 1.0300472570298185, 1.058751292792163, 0.8597183722820788, 1.1726372594478258, 0.9197641883589989]], [[0.6628731194202501, 1.1563035269616468, 0.29136293044303646, 1.2399982014356088, 1.2077316656402002, 0.5828014046318954, 0.3963272062516735, 0.871883008202563, 1.1112777264821603, 0.42339385257690454, 0.1364888048911574, 0.4798360170116103, 1.2432266588102658, 0.6834693737005846, 0.48325004489500956, 0.329609018927278, 0.6869720402523993, 0.48255554681506185, 0.9166768343051174, 0.9348292391065686, 0.5249936979700575, 0.29217664671653687, 0.12060998022548268, 1.0671166857979013, 0.5921090145878469, 0.3402502549492957, 0.629808671774921, 0.5967446447727194, 0.5843558017225053, 0.8588661461655605, 1.0340418831400102, 0.9834350324819459, 0.8967677074862938, 1.120066582760939, 0.9505153070619143]], [[0.6507516856853162, 1.1604383456996745, 0.27726877004924905, 1.137718972507752, 1.2378441690157158, 0.7244151147717809, 0.3514176167958863, 0.9363501303768043, 0.8180381382647789, 0.5288069040394721, 0.08540758961032696, 0.521508742398477, 1.1508870593867866, 0.7351539654042614, 0.4343579846317428, 0.43761185620426735, 0.7264631350841745, 0.5550499919435804, 0.9919562478117403, 0.7069039000521962, 0.5145567252510379, 0.31476902382836064, 0.13164131830385156, 0.9171151938486671, 0.5870780699611842, 0.3149222391822116, 0.6556839470335156, 0.596987274472266, 0.5826947285907773, 0.8544715514169132, 0.9824501401940832, 1.0050414932848408, 0.8375530033447329, 1.0954522718059967, 0.9512006895322767]], [[0.6496676828554905, 1.1367681248524284, 0.2798452341350736, 1.1412663320069816, 1.2144992999232622, 0.6136303337044123, 0.3502082203325205, 0.852778928255756, 0.6852741749653334, 0.706920394408671, 0.11934058091465152, 0.5421772651038831, 1.0760516182212905, 0.7298857316061861, 0.35727293160711715, 0.5705808630662584, 0.719974442369751, 0.5042989420310456, 1.1576336801873963, 0.6920541300864274, 0.4977132702484801, 0.3508954798593368, 0.09436990126701152, 0.8190706043627134, 0.609359916719289, 0.3077966944859631, 0.7095644373085923, 0.5250831432718783, 0.5826998628029155, 0.9709919899498327, 0.975700740109742, 0.9303514436860345, 0.8072137538921026, 1.058859404612061, 1.045190066190857]], [[0.6367368376093118, 1.1537155327713864, 0.3103148151231708, 1.1380673338083862, 1.189644594789638, 0.6151095215747764, 0.38209214319568346, 0.7284475891422239, 0.6478020310971386, 0.6899178432294083, 0.1634600191501404, 0.5340117653262859, 0.8026022500498244, 0.6771189606182715, 0.3913346317428607, 0.7632767085829291, 0.7479167265385203, 0.5107186626191791, 1.149881020414778, 0.738402329981296, 0.4151688437058194, 0.35649829309729686, 0.12099464825200003, 0.7280682127264375, 0.6292838309803581, 0.3878866950146346, 0.679623190176113, 0.5037602386111499, 0.6172089384351835, 0.8081866265381429, 0.8046883502960183, 0.9311775168794572, 0.7770987359410034, 1.0208033910767635, 1.024281416521627]], [[0.6369590583906883, 1.1135622061799308, 0.2920675956566608, 1.1261762762137408, 1.1884273726662606, 0.6252736191499215, 0.39180431973747476, 0.6714383400147793, 0.6382039533306638, 0.5079423097400155, 0.20017731963409008, 0.5346413074459371, 0.7057235335155885, 0.6515959521358861, 0.3887633892316578, 0.7220062437285243, 0.86080091599244, 0.5161677130579485, 1.1187088403448155, 0.703887200870773, 0.46060354314514534, 0.3589994121819149, 0.10739322403881196, 0.5690417721904545, 0.6296045731482232, 0.4641384050577947, 0.44110934077418607, 0.5832023039866168, 0.6285343829730391, 0.6794489729349995, 0.7830436259300548, 1.0050111952909884, 0.8295024469733076, 0.9582079855021901, 0.7857810397615521]], [[0.639549607627617, 1.0962395748628404, 0.2887891994549309, 1.2751999574036357, 1.1517399996743707, 0.6680329963932707, 0.41983694462358134, 0.5751533666111099, 0.8964030379274244, 0.3907187649885409, 0.2187639550352184, 0.5170625559700939, 0.5741140205576862, 0.6500771645627224, 0.3853519346028418, 0.6951371765408896, 0.823968316066753, 0.4888991431094384, 1.1367576387477984, 0.6310437881642971, 0.4315244335999975, 0.33089625695312347, 0.09309682062731345, 0.5527116240606755, 0.605568874262528, 0.5112116569825953, 0.33283062509146455, 0.601080776537317, 0.6170184366104552, 0.6569629844927478, 0.8425871431109065, 1.065699664155285, 0.8520227232912833, 0.9229788280293774, 0.6067942369813615]], [[0.6418300301415799, 1.0551647389450762, 0.27990850596500366, 1.2474679057102407, 1.1985082668622642, 0.6790197653443657, 0.40306099968960013, 0.421150801493726, 1.2294588153304213, 0.44695245457914073, 0.23872354167986765, 0.507498372696453, 0.5403364558818217, 0.648476789506708, 0.4113270091134107, 0.7140038064022999, 0.7821073319204079, 0.4910635449144849, 1.1143437611859128, 0.5506425809637246, 0.39108227958235314, 0.3117870166278226, 0.06962387981395933, 0.5367895732421173, 0.5206269765523672, 0.4943752400366749, 0.3408739076665516, 0.5930324286020664, 0.6268538574012371, 0.652714816688767, 0.9060607282018492, 1.1175156546601683, 0.8651338595525628, 0.9597678519813981, 0.4866236971727934]], [[0.6235146143990846, 1.045690654631816, 0.2756667046408648, 1.12249494822161, 1.3092503173229009, 0.6769690100139154, 0.4066260773345678, 0.5172408499225268, 1.2623005130579958, 0.3973378292687583, 0.2648635993492715, 0.5264223168769684, 0.3794193753494404, 0.655738508536416, 0.43159741185137546, 0.6529718321417498, 0.7827961484198444, 0.6622885356716257, 1.1173427423061697, 0.48508088009833744, 0.377520879342632, 0.3644292176676221, 0.08182290786123614, 0.45056383961402624, 0.4006161468193933, 0.4282762500802373, 0.33018892356284235, 0.5159467960271414, 0.6474028279158208, 0.634788168511208, 0.967802978059457, 1.1805485684810728, 0.8783151541285468, 0.984903885561614, 0.4019549105404099]], [[0.6075494174217464, 1.0300942474456107, 0.28311532271442524, 1.1240796439738432, 1.3139655582760557, 0.6742250729871674, 0.4609851602420375, 0.636794226011269, 1.230934061158179, 0.4015868351708447, 0.20442569647195707, 0.5249889594682708, 0.5725786240622069, 0.7263573376758554, 0.4460298477585034, 0.6558532186292889, 0.7622007917967415, 0.657607895261964, 1.15792872501227, 0.4301161170026328, 0.33102673737023175, 0.3969992459033095, 0.1317356504108989, 0.36857172792846205, 0.3319291653132482, 0.2376639892503708, 0.3311524558398004, 0.4487008593254035, 0.5048313614437501, 0.5680738312928901, 0.9924088239078451, 1.254251121197043, 0.8933234567519359, 0.9742991198378124, 0.41317378404972394]], [[0.602214351166311, 0.9844780341395649, 0.2845963532356017, 1.1367350751325116, 1.2701608211192883, 0.6626718578901948, 0.41987554362304885, 0.6088558810565644, 1.0393886451967114, 0.4104747219947653, 0.20312019972812678, 0.5055881358355361, 0.7540426984158521, 0.6751240035779813, 0.4917551629799511, 0.6247987901083455, 0.6512057293781378, 0.4926835067612946, 1.1471708837751065, 0.4428873587900949, 0.34563817234576716, 0.4062274425536687, 0.1397213891640602, 0.3457385157055549, 0.28353688294542123, 0.19312545631867523, 0.3535176050469212, 0.4044910036022422, 0.4556465236206724, 0.577252694001209, 1.0196480813008328, 1.322445277938502, 0.9082793558290386, 0.9425518917540621, 0.4527535994045495]], [[0.6067560266791698, 0.9274273570060567, 0.29870843098874483, 1.044892486964355, 1.2841095181958684, 0.6392589984335403, 0.4141923345686911, 0.5152505088650218, 0.8715070920636944, 0.4329559688646316, 0.21778169088205873, 0.488897286256177, 0.8759744757481847, 0.6715985069441379, 0.5438125100033815, 0.6476663689423072, 0.5638425999351584, 0.4899884132446598, 1.1451534341184144, 0.4242958016725878, 0.3511539809358034, 0.4145874767381956, 0.17906176392404638, 0.4301799844202755, 0.2523420207720085, 0.204802249195881, 0.45644323484942784, 0.3626606057457609, 0.5690969891832309, 0.6061436213302417, 1.0246864023788076, 1.3693257430668757, 0.9296942473451, 0.9284885991937886, 0.5165197284535443]], [[0.5909894522474678, 0.9321020451342313, 0.2841459493382454, 0.992170151548537, 1.2865998446100626, 0.6320416351948085, 0.39925681593095674, 0.42974045015652756, 0.7449405418131853, 0.45611045160312996, 0.2504649133727732, 0.9275754511394342, 0.5243142761006298, 0.4311495930473125, 0.4467295159655113, 0.4012611355844319, 0.23692467966029024, 0.2940043003638686, 0.7248392279925182, 0.25916606896336736, 0.3205827293647301, 0.3017695435065642, 0.17065870836662034, 0.5076563604591386, 0.2623535218788273, 0.1714374080951092, 0.6234590382275969, 0.43235299196067944, 0.46921249879134797, 0.5894343221650494, 1.049273690877746, 1.3993684079818118, 0.9131013762922962, 0.9251993881663111, 0.7222983732029564]], [[0.5703601205104016, 0.9169774468446222, 0.2803018484959254, 0.9210187117850022, 1.2899405609871013, 0.6094620917407796, 0.40389391832802496, 0.4181646050413878, 0.7437464881866406, 0.3967843591088419, 0.2672310500817795, 1.0658383094426833, 0.5359823951004555, 0.3914156944949875, 0.45239488616027246, 0.4893644500111869, 0.24727005962821713, 0.31045688026273904, 0.7364522209229911, 0.1677769856139594, 0.27085049750461543, 0.28513401338830846, 0.18802474773976355, 0.6661481339081108, 0.2974745392647882, 0.19765197741169174, 0.6874078745890472, 0.5020864233578225, 0.4751936650029176, 0.5829590287209769, 1.0411599434140257, 1.3965196825918418, 0.900412754846701, 0.9046662197008849, 0.8387356891568079]], [[0.6155232596545157, 0.8309997220111177, 0.29933315027379714, 0.9478731829420207, 1.261779113796396, 0.6128174124510738, 0.4177324881096073, 0.41960220647628005, 0.780001605727501, 0.31514351689706716, 0.27666412395891615, 1.111498427769528, 0.5660553148186023, 0.4000144758742125, 0.45140133287907214, 0.4732575324817826, 0.2643819308204518, 0.30393615215001046, 0.7580510363942702, 0.17910217486505708, 0.25842935097609127, 0.3259619007901915, 0.16884081720656263, 0.746629634940742, 0.3911794054989584, 0.21966223344607413, 0.7103076908731437, 0.4779242119920589, 0.4947628898937851, 0.5768531604034928, 0.9991489709712719, 1.3633147355894293, 0.9172216111122597, 0.8703421103564708, 0.8091615424884457]], [[0.6004712716127862, 0.8711707633359291, 0.30949555730088824, 1.0066592783097335, 1.213991031481822, 0.642497655842545, 0.4292752447548085, 0.424759520731214, 0.7453276182877443, 0.35926612206805447, 0.2960967129534039, 1.1026507230197427, 0.5827667343151803, 0.40471597029164375, 0.41231797593229347, 0.4613957999600954, 0.270291923513537, 0.3044953513360411, 0.7227394107974627, 0.19420234832033714, 0.24409651990579628, 0.3392821509508645, 0.16637173864760826, 0.8390962130808948, 0.3281910238043565, 0.20499184240123552, 0.6850085530236825, 0.44556807449492153, 0.5076686290967434, 0.5297368824056523, 0.8824223401717748, 1.3489122649518088, 0.8931541466202702, 0.8361784814075188, 0.9294428944142141]], [[0.6063373520245642, 0.8822213818512431, 0.34234950700952155, 1.0694314249363437, 1.1669467573804453, 0.6514799984038617, 0.4303875078872955, 0.48718342190087804, 0.7454875034136443, 0.4044912254014401, 0.3057016393049017, 1.1133425236861607, 0.5957374680831777, 0.4157437014823248, 0.39845866497814747, 0.45450754237012536, 0.2534427576932277, 0.3135582515469129, 0.7368232923511144, 0.21379656329118418, 0.23428669297130422, 0.33683640296957945, 0.15993967584028992, 0.8462831229961344, 0.3793418081680933, 0.20128337625801723, 0.6441826416879235, 0.4472816955958406, 0.5387874550535394, 0.4801709742660183, 0.7651613484004439, 1.310766951647571, 0.8724373330325192, 0.8081089839925554, 0.9383922736486998]], [[0.607434517314811, 0.914722920869929, 0.3669594641854855, 1.134958613291778, 1.1174846763637276, 0.6417466051511258, 0.44134479903970003, 0.5091931757988388, 0.7710774530967867, 0.4015352689518559, 0.28384274682872646, 1.1213571530868944, 0.5864962477280335, 0.40935242992119586, 0.32523532509187825, 0.45235722718098437, 0.24543646707965106, 0.3136855994271482, 0.646468262381561, 0.20671666593025473, 0.20555624851714285, 0.33142617756211823, 0.1588913561405092, 0.8647924644730115, 0.3676671237577386, 0.19545436563932184, 0.5487536148663623, 0.3689346481301451, 0.5764903075583606, 0.4136205946348889, 0.5654225954224064, 1.2566914315277415, 0.877885008621073, 0.793250545997001, 0.9131921007291255]], [[0.6358573219375616, 0.9038542564184577, 0.3364848266563612, 1.1314424398928145, 1.081681154274989, 0.6638012914024094, 0.4523917987837064, 0.5542785952286603, 0.7807170084175643, 0.39248836848471746, 0.2971035930511644, 1.1268678380106105, 0.6065136115505436, 0.41813400746904844, 0.27877244682558255, 0.4752038593571589, 0.2446965823199196, 0.3179279666982681, 0.6456842450662981, 0.1495965095244337, 0.17587633513111245, 0.27358456780692336, 0.19308608365405233, 0.8900052862441115, 0.47959518297697734, 0.19792197397840255, 0.5130201778048435, 0.3664904489660227, 0.6128880480085914, 0.3757630900326363, 0.4154599114009949, 1.3054914573851675, 0.8587136265807066, 0.7589957774573419, 0.902015047145064]], [[0.6471066212404061, 0.9150290362418312, 0.29246494867238754, 1.1461533251931442, 1.021646231428048, 0.6566399870202919, 0.4554495587686824, 0.5252995360889398, 0.7813561332460971, 0.39457384514308286, 0.27905918967765825, 1.0964271749750971, 0.558383032683449, 0.41356547097202806, 0.28478898427315325, 0.4526027109587779, 0.2622184435000526, 0.322459424191587, 0.6633038165824909, 0.08255945306744356, 0.14508483163415528, 0.15552659459653204, 0.1793032165756442, 0.9048565109798115, 0.5310544929673086, 0.1785250681682633, 0.4821475656706584, 0.47708271098775856, 0.6544150002722221, 0.3675740154744308, 0.31280162550516427, 1.3156025882762439, 0.8629849642091136, 0.7657071145613965, 0.9033409480046872]], [[0.710495790535803, 0.8813680798868793, 0.31606627884415445, 1.124166276343248, 0.9931290069623155, 0.661713128211393, 0.4380871711422476, 0.5683760556981962, 0.7251595260130196, 0.45752032710126217, 0.27168828917216575, 1.0666304464619296, 0.540965256391349, 0.41458662078350245, 0.27234702527667703, 0.4582697342196691, 0.2028320087980895, 0.3247713122580855, 0.6379154664066605, 0.05988382008103183, 0.14420233919639708, 0.11669241180805487, 0.17609490135540676, 0.8676566733719031, 0.6631301980879012, 0.16861272742528832, 0.5206396851127824, 0.5774230108303784, 0.7164053005621398, 0.34135095493957146, 0.35378938824386497, 1.2535586549635735, 0.8263572952282006, 0.7619138737654254, 0.9321416057936968]], [[2.036947299572411, 1.0204382422395923, 0.3092065653855182, 1.100291237701039, 1.0422948918640769, 0.6280094102400029, 0.42291360397794797, 0.5554458201899367, 0.6756047377681941, 0.482402188548941, 0.24309984504444898, 1.0758588309037076, 0.5394473047569995, 0.4092658232944506, 0.2595501815457808, 0.47474840431800736, 0.19679133719347286, 0.3006178124078164, 0.5856622139831028, 0.04532641698279563, 0.16650002849421114, 0.10089511155270164, 0.17152378515653133, 0.8048091027201212, 0.8709944360771938, 0.13418566038158158, 0.49829703437424044, 0.7032384493600021, 0.8544944197909965, 0.3462573386608522, 0.612169747526957, 1.0809309519545511, 0.76564023138323, 0.8607402514139547, 0.9831328758907567]], [[2.05902610531033, 1.0221188687993459, 0.2999274077916186, 1.1022063442183396, 1.0346237575512491, 0.6172761359985892, 0.4024915127154658, 0.49889363926178476, 0.7156355320372736, 0.45503336024493507, 0.2403351022657817, 1.0825705564297259, 0.536975758980387, 0.41251454783291086, 0.2632611380952612, 0.5107574144777393, 0.1836161734224055, 0.2958738001991258, 0.5620377264496114, 0.053848667608427825, 0.18565928918745855, 0.11064538478434877, 0.16166602221331014, 0.7620138457578902, 0.7962652131589549, 0.12320150120335563, 0.5962378006135307, 0.9332342575857345, 1.0123991572688742, 0.3932080345782627, 0.7249243720867695, 0.9207688019713987, 0.6270240838008455, 0.843505404236869, 0.9695230290874856]], [[2.068678678187087, 1.049053407788138, 0.2797244684710242, 1.039301106883487, 1.0978936234476635, 0.5858703928336056, 0.3910444745402287, 0.46559466154198104, 0.7773141974686172, 0.40455179210677183, 0.24394481877922056, 1.088802296534663, 0.5341042899501528, 0.42168375520829793, 0.27863079899828713, 0.44698887557180866, 0.18077788333649475, 0.2909418146631695, 0.5490227945806603, 0.04968515821933888, 0.20227099721736988, 0.12413968863987258, 0.1678398959542962, 0.7455473521994975, 0.8189677439828922, 0.12695493124283852, 0.5330678516948395, 0.991025653105264, 1.1035477286972684, 0.5012941108644553, 0.743767700032935, 0.7551507155746493, 0.46596349536281756, 0.7601151775741926, 0.9718299488632222]], [[2.0664978929279463, 0.9661747318687979, 0.30614981495550064, 1.1007182948361183, 1.0177070533692891, 0.5913618413244326, 0.38825141778472383, 0.4379942646109798, 0.6737130861982251, 0.36690664589841127, 0.2609528425199285, 1.346085750438042, 0.5153180240006479, 0.6861283096720375, 0.2567297687588239, 0.4884117765921465, 0.23720896655359447, 0.3035576681871379, 0.5399666326826629, 0.05148469476964118, 0.16410381824763295, 0.10911826246261569, 0.16739153036179824, 0.7247591906259175, 0.8636832790165849, 0.1265843239834227, 0.4803286492552864, 0.968286139157677, 1.191577319789535, 0.5481169693528706, 0.7152571027812231, 0.7642265213679975, 0.3591448455546654, 0.6306158484969505, 0.9508206155517371]], [[2.054463845689107, 0.9655653161364274, 0.3106507389189941, 1.2060569083156867, 0.9210579693789485, 0.6311887324675258, 0.39142438865355, 0.44305305686938395, 0.5932871581241499, 0.36167929077901495, 0.26090678670406264, 1.3822889519508947, 0.5123863157757591, 0.6818583937456786, 0.2447873311773393, 0.47448647720942116, 0.2520474154115063, 0.2994841302810808, 0.5365890636354365, 0.0530060980300477, 0.14955801257591717, 0.0929358568559542, 0.16279362493326796, 0.7095526132321912, 0.9224039286891721, 0.12362771384386008, 0.5099397181910553, 0.9475696131972994, 1.2551197422566516, 0.5667467940857935, 0.684953473302008, 0.7712893142965839, 0.31389535519637046, 0.6384515059442903, 0.9139815853166159]], [[2.0207919894376833, 0.9823292709449919, 0.30748024686466, 1.2551920656034548, 0.8172768973624605, 0.6494639547843654, 0.3763899538359894, 0.4409557264989737, 0.6116197724932788, 0.29559143509558017, 0.2561685247581339, 1.401936406208725, 0.5407931330778354, 0.7153315921457563, 0.2389275913917449, 0.5001838517842134, 0.29751499009508964, 0.30403566304555196, 0.5207042877364545, 0.060740932588372536, 0.15275714678616337, 0.08586546060741618, 0.14910615937463367, 0.7362568429538804, 0.9133453267704522, 0.1357489123418802, 0.5435750986152674, 0.907553260207887, 1.2525917257202215, 0.5352094555239063, 0.6566991991342035, 0.7880970236948882, 0.296925519650879, 0.6303461472899358, 0.900750331954485]], [[2.029543578054184, 0.9622521807794053, 0.31485031385274287, 1.2387613474193704, 0.8044264581138468, 0.6505312936301878, 0.3642404744394222, 0.4399864386017274, 0.6795327246527165, 0.3237582370850959, 0.2473189200668835, 1.4335284294198338, 0.6282936119784701, 0.6991106417410882, 0.2607129544480198, 0.47739487590754903, 0.24952188336052972, 0.2995065539480293, 0.505531103389099, 0.08622911619276763, 0.16576814328781617, 0.07647737928120507, 0.1253965590216582, 0.736784817844325, 0.8184024572762644, 0.19689346843148514, 0.5930111329193515, 0.9261050231322642, 1.220775028301783, 0.4465195204306829, 0.676706439732444, 0.7730907142375918, 0.3023091728010575, 0.7195606333115405, 0.8828271265969249]], [[2.0410373097804246, 0.9888436738050955, 0.30956057909324697, 1.1933153818916926, 0.8766773958266665, 0.6629340530715828, 0.37395868820295314, 0.41619669750844607, 0.7549699010841084, 0.4059819126296201, 0.1516373584060286, 1.3210856326920593, 0.5663367522048811, 0.6522096879476575, 0.2403877792092104, 0.37645201975861875, 0.1872050631096691, 0.30671941628177724, 0.5046176003040227, 0.08297254914667473, 0.17805278862364157, 0.07536690973020468, 0.11089801559913752, 0.7355814669491678, 0.7223455569500061, 0.274637688647733, 0.6708432020182948, 0.8851083089550044, 1.126196395586887, 0.40421527889309744, 0.69384375117253, 0.7408002006958182, 0.3466086281509838, 0.7932238612320139, 0.8160114645408569]], [[2.06494169561659, 0.967714233792939, 0.31465509435749994, 1.183545497379992, 0.9363040303246639, 0.6606129202499154, 0.376955162944806, 0.4205680125370405, 0.779666432578128, 0.46249599978098677, 0.09386101327717593, 1.2959611182838513, 0.5278579720726547, 0.6906259352887031, 0.22551407357795072, 0.39237761517330505, 0.18563367630435212, 0.3110055238718099, 0.4826421637628582, 0.07135779955373828, 0.20143015189944136, 0.1015026793836164, 0.11791735966194022, 0.722297078948408, 0.677968809164156, 0.2866340574141778, 0.6727254978550185, 0.7150393482829335, 1.07355392102146, 0.4638282126071939, 0.7612827593570504, 0.7714937087517371, 0.3948060242684007, 0.8102646383945753, 0.7469258969322222]], [[2.092086275601689, 0.999327367058849, 0.29185429859458845, 1.2433180300075324, 1.0055385815818862, 0.6673845119050786, 0.3719563515524579, 0.41315332089843804, 0.8183148604043811, 0.4148854321396082, 0.05141613876206047, 1.242479866112895, 0.5065328857191582, 0.6892358583382938, 0.23298923974284858, 0.38233597318872903, 0.18702413033136955, 0.33251111264808597, 0.49761222579361175, 0.08549480182982765, 0.19724514975203922, 0.10762110985000606, 0.09834303007147935, 0.7071945209733914, 0.5730184435284854, 0.25505999930278334, 0.6627552468788475, 0.4453975756890689, 0.9692526346939349, 0.5355709785431646, 0.8662447161914656, 0.7787844190887345, 0.4346691184637556, 0.7833662476497429, 0.7311817112520324]], [[2.067750915091335, 1.0149375110588714, 0.2990598941464281, 1.1631428511153314, 1.050447644507978, 0.6595197914010971, 0.3893374249849687, 0.431317262549657, 0.8253562838368016, 0.4892586248564583, 0.22044457614816265, 1.17741885282024, 0.38291557877468685, 0.6289873873307195, 0.23466330581898465, 0.37042804760722786, 0.19933093861880052, 0.336413605294695, 0.4948275992739216, 0.08149855614769769, 0.19327216890421622, 0.12445640104297054, 0.20541684996699355, 0.6800044117830581, 0.554052727018574, 0.21773173053569134, 0.5467752344851434, 0.3832043422025684, 0.9347152562816137, 0.6015778842885855, 0.9019610485546414, 0.8132740207052634, 0.5016871277975725, 0.736896425361476, 0.6883361413050642]], [[2.035268672953564, 0.9942077449299254, 0.29962380176969616, 1.0522220938067988, 1.0586852110118157, 0.6264739775288408, 0.3749802740445506, 0.43176116844164375, 0.764579537180949, 0.553985123125708, 0.21960878170388964, 1.2657996203915505, 0.29874256554793743, 0.6034828041127891, 0.23304066012229496, 0.3644767348623749, 0.2018359802865828, 0.3159065508698137, 0.49441083957228005, 0.06413052310446118, 0.20115259136428548, 0.1322614773877506, 0.1893349621325382, 0.6461819797508697, 0.48143498130076284, 0.18904653238886043, 0.48143503523221387, 0.3481033387522489, 0.8825103915520218, 0.53693274630303, 0.9415545986424927, 0.8231504947172101, 0.6176747648392042, 0.7333843618194478, 0.6597453753448221]], [[2.022902496854014, 0.9166873746069261, 0.3138805643846665, 1.0600609031170005, 1.0750679040585158, 0.6664389308699643, 0.3868440605879573, 0.4475121187244547, 0.6251261842958912, 0.5195202990597512, 0.23570125988923746, 1.2585607422890988, 0.21432998679804638, 0.6651800817747442, 0.23887103014086475, 0.3639342694697527, 0.19582518194363197, 0.3271869707999254, 0.477216056744203, 0.05539149430192236, 0.19991830722962634, 0.15842738798670192, 0.1780053650224723, 0.6244799867348741, 0.48918906343836327, 0.18154283225894255, 0.4540117730117207, 0.39690496488602245, 0.8092554184112888, 0.46502424550491883, 0.9634632930052016, 0.7450204928991201, 0.5288823688449056, 0.7368062497758856, 0.5634106731815497]], [[1.993449292728136, 1.0003571754330747, 0.32052984326370326, 1.037589756632837, 1.0409221389809038, 0.6617821770454027, 0.39094804948460865, 0.48417852940455375, 0.610382507327354, 0.4759472579393353, 0.25932036848721945, 1.315731726405346, 0.2081892375763198, 0.7220697093748295, 0.21400947270012582, 0.24118821753175806, 0.19685891233169622, 0.3080397321486734, 0.4475714991726728, 0.054381589396434116, 0.20538979600759413, 0.18511710895317596, 0.17550334173782178, 0.6005612159066893, 0.46801308555336973, 0.1654129698818235, 0.4602009104606495, 0.3925807879659333, 0.7289515417592387, 0.4316903455742691, 1.0130075620035035, 0.7180579747036566, 0.5239686882101717, 0.7543326611132355, 0.5647630659557222]], [[1.9510794249555432, 0.906170701691313, 0.29768287176536173, 0.9948443740455373, 0.9917836398906228, 0.663480987037812, 0.41279592229584383, 0.575595044072458, 0.49801824733102323, 0.44359325023556273, 0.2553020325256341, 1.304267828008566, 0.19029860243554408, 0.6658189548372443, 0.22415570350741357, 0.24280381897207826, 0.19134906415459432, 0.3295983844199001, 0.4244500484345897, 0.054343480313426895, 0.20877384532277143, 0.19485009833058387, 0.1624686526422623, 0.6538889072328776, 0.43449490258259954, 0.176617775641365, 0.46738883552078536, 0.3295767204737558, 0.6541167572592312, 0.4345802531171522, 0.9963141070884403, 0.7144616008205315, 0.5120439211509267, 0.7512056611085856, 0.5701143996751303]], [[1.8747474803486877, 0.9016304244414068, 0.3565712688769957, 0.9937431947377593, 0.9535267950444564, 0.6578267124440922, 0.41692589529128793, 0.6698145080830968, 0.5280301277518531, 0.42521368129436404, 0.22293744591450815, 1.2688605369526544, 0.11932659753732033, 0.6775971164421005, 0.21908579944710868, 0.2516971511738346, 0.21007575040936458, 0.33786142785104323, 0.4429237678007971, 0.09032605474354705, 0.21216517421011458, 0.21940847442204792, 0.18555593559542033, 0.6981031090968135, 0.390279549199382, 0.3172941092607591, 0.4424464916437224, 0.28209240968619737, 0.6130754466927206, 0.4893917507148096, 0.9390218211055037, 0.8109325433016428, 0.5244415258204724, 0.7538308806958058, 0.612477181210665]], [[1.7683726860931803, 1.017360488521476, 0.34561638983466364, 0.8615392829111981, 0.9724052904732755, 0.6373066844583006, 0.459369045181422, 0.71548719081922, 0.5947302379111364, 0.5694439637733404, 0.19539064288755015, 1.2835853322429311, 0.15223484979800034, 0.6466781989638866, 0.219166074421815, 0.2503814311697714, 0.2377656331440288, 0.3424975763960319, 0.43876510132108787, 0.10937189774566097, 0.22084338026921757, 0.25768558279641474, 0.20974538386971464, 0.7754275826269238, 0.32077355008125685, 0.4505273753982693, 0.4186911733812301, 0.24725939974410382, 0.7440208223724368, 0.48949658672426033, 0.8354959463241408, 0.8954437662472416, 0.5453691845170865, 0.7712611070529125, 0.4688390886674353]], [[1.7992497391044304, 1.0237355600644094, 0.31423157679488883, 0.7119043150922926, 0.99073671574108, 0.6064306948710856, 0.4312260715999814, 0.8205586594726155, 0.6752262700411279, 0.6775736483648227, 0.23032307732065016, 1.2358459989637596, 0.15466578971555708, 0.694612677095914, 0.2049196579419414, 0.24444362728022784, 0.21263157588338577, 0.32206165333061265, 0.4601529815527007, 0.11567737254780727, 0.22326452562792756, 0.2902182665261903, 0.2324918546370418, 0.8218610940798083, 0.2605486788866094, 0.5038528298320833, 0.37459756907017555, 0.2414113240707605, 0.8977962439987436, 0.48222377437998293, 0.7600034653380994, 0.9702028641548074, 0.5707132576553029, 0.7940246947167037, 0.3108907284044358]], [[1.7359584894008149, 1.035782983708703, 0.3031376875457909, 0.6852013444967517, 1.0062572109339802, 0.6188901132390867, 0.413509354173663, 0.9264017663082589, 0.6504842371213924, 0.5969305913977979, 0.35386079918882674, 1.2200108722973542, 0.14110968126685244, 0.6181352643840201, 0.2086262159958985, 0.25049008203145995, 0.21145702205299408, 0.32052253266071323, 0.4584041799865436, 0.11891268013681347, 0.23419933496215983, 0.2945058310064472, 0.16161534272815, 0.8167396539411985, 0.27786245232477613, 0.46857061859425975, 0.3676895546051357, 0.24714390394246438, 0.8674462011330804, 0.43812438568235335, 0.7879870867519532, 0.9086409931971584, 0.4422537787019167, 0.8197103488159299, 0.23916801045004635]], [[1.6660739775552296, 1.0557197374525322, 0.29204799854846925, 0.6405098425975474, 1.0199035305684605, 0.6265878923242092, 0.42392701620907836, 1.027320142014671, 0.6487025244700286, 0.5017615457228013, 0.32005657638201196, 1.2575965148905606, 0.10460462090236351, 0.5954712454061578, 0.21112608775944633, 0.25028369571036496, 0.21272178784517187, 0.3283990927183088, 0.48447762518028115, 0.1598511950869605, 0.22102342262739705, 0.4284676612316907, 0.16117378483272154, 0.7991222749844382, 0.2749746692401465, 0.36463684752169934, 0.3585497614244431, 0.2745288280572684, 0.8163865625201603, 0.4544889283646413, 0.8168649251598243, 0.6564267284866333, 0.3513620077547813, 0.8680208333747856, 0.36652200844675553]], [[1.1565538540945453, 0.9658146681643169, 0.26912460847815806, 0.541109508347637, 1.0434560567722104, 0.5748141727184897, 0.4373199135655984, 1.138261283858494, 0.6842205332646449, 0.45501934032892977, 0.30090462050328004, 1.3076612331247508, 0.21255892478925362, 0.5973314590425868, 0.21596474135325436, 0.252944781094841, 0.2157547474289695, 0.34292673390285416, 0.5002622247187576, 0.3631617411866881, 0.3197496778425764, 0.6123690655332676, 0.11257243181328434, 0.7580086668132271, 0.3284366067390111, 0.3269447457917929, 0.36213721584294034, 0.2686599580987592, 0.9614292153671757, 0.45253189616089573, 0.8518230077672906, 0.49159657083127895, 0.2825504343834689, 0.8300901906811177, 0.49918277626923013]], [[1.082377368486247, 1.1528479330213017, 0.2758409768373366, 0.6523119182250081, 1.0775283872946835, 0.6167090699619624, 0.5044363944808322, 1.2244237304462269, 0.6489541945196448, 0.41799762585273914, 0.2703381865369576, 0.7251777551771607, 0.15277946071366566, 0.8716084868263395, 0.2795870122333851, 0.49231001482516135, 0.5091866050933161, 0.6277277794672568, 0.6933206648051704, 0.45352819915438025, 0.45074439531104743, 0.5710696278254936, 0.12324573488293292, 0.6809865206084036, 0.5122172916002535, 0.2463110812476671, 0.37853136449157554, 0.3306435956697115, 0.9503091597812324, 0.4191074470425389, 0.8359341881052245, 0.42486788167833206, 0.26566748774160776, 0.8405847260926356, 0.3067148097275687]], [[1.012833124603479, 1.1435509790029739, 0.2506598655120191, 0.7761812441405227, 0.9738595772631565, 0.6560075448461811, 0.527040766009539, 1.2956643575319855, 0.603655716321663, 0.4619138273510087, 0.2775298403799332, 0.7030349806508029, 0.19558360331445485, 0.7935089320350293, 0.31184149593082733, 0.5126383279844311, 0.5740941810239037, 0.6058962903478042, 0.6477912786068851, 0.5100096666261191, 0.3960127835795564, 0.6417970751138086, 0.044051294313512804, 0.6748757692268499, 0.5326017093674856, 0.2560528943695322, 0.35862507910167557, 0.29242898296602504, 0.9298154004263102, 0.4239093336394176, 0.7812454462135112, 0.39615834817778756, 0.2833738823489049, 0.6104570145189838, 0.2572145674760489]], [[1.0592141512730346, 1.1427884780935325, 0.2247194800184644, 0.826957275546421, 0.9764222038349848, 0.625615910058029, 0.49450047556114923, 1.393638990038673, 0.6113045883708177, 0.4046132267255472, 0.2700096915156672, 0.7225480539587795, 0.21813405273591774, 0.7947037463577215, 0.3333972646325287, 0.513771803768809, 0.5579619451085934, 0.5747882764898251, 0.6204057771736314, 0.5463094745518984, 0.39989295150743454, 0.6989170800770181, 0.0614867440144502, 0.8007266503340906, 0.5189523819936824, 0.24985959332174718, 0.3206470057992884, 0.2482938386219734, 0.8638501497366093, 0.38658542140292157, 0.6635323981465249, 0.39061816046490905, 0.2829471054800371, 0.4791482920378107, 0.23063489966106065]], [[1.0917977970484873, 1.1472115852721958, 0.22120303027644211, 0.8606476309344939, 0.9504081356687649, 0.5766465983508735, 0.3908586013918183, 1.4192696131393971, 0.5838262292431202, 0.385276776278146, 0.20791509798457758, 0.6542681320098318, 0.2643859766654688, 0.7960515288151528, 0.2726039161069669, 0.4023788361400549, 0.542501540388027, 0.5680254721615969, 0.5855060199227551, 0.47380006419914567, 0.37170067467477136, 0.738958153202651, 0.03877653305563357, 0.7885586002732596, 0.458018147395878, 0.25096830092280664, 0.3323906085430762, 0.2751404868663837, 0.8345978414831693, 0.41613212339152417, 0.6026381719354675, 0.3909357725465741, 0.2870082226330671, 0.5496618138899027, 0.23777174729537628]], [[1.0563505724597377, 1.1042362380372726, 0.2584126621455372, 0.8670082662037181, 0.9812409485510287, 0.5727104576668808, 0.3477990568806377, 1.4397331439676055, 0.5499955144980668, 0.37342302485760304, 0.22789299974334504, 0.9474900799041488, 0.29669852461925406, 0.6278559793131525, 0.24737494369030438, 0.40680294378325554, 0.4653216117568917, 0.4242546946558238, 0.49031457674413165, 0.3595109433050471, 0.38455837389641856, 0.6378061628480829, 0.03830612521974978, 0.789746237439767, 0.4458614340652648, 0.23931746781065832, 0.38718281783141495, 0.22723408890451877, 0.7345559440634608, 0.5141812476560759, 0.6038693645143841, 0.44128858465599263, 0.363374152405734, 0.5941248588720076, 0.26019829821326107]], [[1.0651962387107974, 1.0614848056805122, 0.2528799013073492, 0.8873362122381971, 0.972101060718795, 0.5591042245008375, 0.30110350290380866, 1.4655627929903576, 0.5087588130044254, 0.3719054515813078, 0.26223772385218636, 1.0354883832349784, 0.2793187164686238, 0.613255463554097, 0.25380984283286306, 0.40829318397652914, 0.4485352368081945, 0.407976368851091, 0.4969783935562323, 0.3953531489966218, 0.37764280058777394, 0.688720811595202, 0.04463464024382975, 0.7667914416242743, 0.5400746075187345, 0.21613062354482576, 0.4044224495720554, 0.23840876621866475, 0.7080527815815987, 0.5395588801384399, 0.6085328186807453, 0.5010531227179182, 0.4643841980663478, 0.6873084659920857, 0.29763323946125064]], [[1.1072642825111219, 1.0378466635137498, 0.27112151619626085, 1.0666705124024867, 0.9463597550905929, 0.5370847696972134, 0.27011652915228135, 1.4865243158975343, 0.5334991562885503, 0.42301738729753807, 0.31497504655241604, 1.0073896729123453, 0.36153965792783826, 0.6011424542106321, 0.22385562565530542, 0.47849508743516966, 0.4137310956306275, 0.40324157574924707, 0.5039101825106614, 0.3551998558967401, 0.40620175646122875, 0.6680461500618283, 0.06714282775405442, 0.6723069607781615, 0.5514972140169425, 0.18292677069869834, 0.4137728900742661, 0.21732434792980232, 0.6496786492050385, 0.6467758664536789, 0.607272609696416, 0.5447755090672461, 0.6447868411949234, 1.0084620494801557, 0.4381405669134679]], [[1.2056694924410563, 1.1161809656243413, 0.24622560272035404, 1.0569922298417815, 0.7238022340057922, 0.5364194255384759, 0.26215751157044387, 1.5066343414828143, 0.5929665619631471, 0.4728862583741341, 0.34310182570766246, 1.0700546225442633, 0.387688390679935, 0.5994559384578246, 0.21768142467312063, 0.4381665231261714, 0.40533784511690374, 0.3555002517073499, 0.5931313198563759, 0.22375285687295648, 0.38786002094726785, 0.5906728334135813, 0.06387236013017165, 0.6183318870850747, 0.6258497048461349, 0.15401618266902106, 0.4215893115419036, 0.20451590467294958, 0.5254247326603441, 0.6739076397953148, 0.6821777172449353, 0.6231921474982602, 0.8015520436399202, 1.1627739239094443, 0.685723605299814]], [[1.2005695259404348, 1.0310380718461951, 0.22758420357615844, 0.8608037337334995, 0.9277830162969588, 0.5268383226813013, 0.2604248493410367, 1.549854874312503, 0.6069055159518466, 0.5085966089298755, 0.3553357459541814, 1.022536174625176, 0.4415273709673784, 0.597071221970236, 0.2677368001012736, 0.4351088310104323, 0.3958733795888397, 0.3617718599414574, 0.5458489253656256, 0.2412366401237016, 0.3294590549843093, 0.5582146263063407, 0.049631523064297314, 0.5623584530901049, 0.702451133696015, 0.1294053054268781, 0.46932805239305575, 0.2586958088552665, 0.45389555684503785, 0.6429826625044476, 0.7301233473156366, 0.7173145681455804, 0.958702680540477, 1.2237033412431493, 0.9076507771285889]], [[1.193396052637787, 1.0585881469580805, 0.23076137806305025, 0.7679978336336848, 0.9134785897661422, 0.5576463249783491, 0.28533052018061045, 1.4842056259281915, 0.6692421187612091, 0.5149341529615842, 0.38430831572422497, 1.0041961569459479, 0.4810369461566507, 0.6048887045886562, 0.2847653972649738, 0.449348363166277, 0.37663387738303, 0.38442800533132526, 0.5801093268153754, 0.2937148953186138, 0.33450358923393264, 0.4996185088833576, 0.05209753546175741, 0.7559542360698218, 0.7066851863050292, 0.11870490148069174, 0.4959161506511711, 0.26729811429472916, 0.4254416321470672, 0.6017148616620372, 0.7581290664073692, 0.8083095099836807, 1.0739844548386746, 1.2206119932441033, 0.957479528140796]], [[1.1478093372167972, 1.027552073285833, 0.22951337998992521, 0.7964286629965802, 0.8778772677416966, 0.5036691360598937, 0.2441542595483939, 1.4678945545362858, 0.6947743685489496, 0.513169142530354, 0.36372031275451233, 1.038581803281513, 0.5350181571599096, 0.602392997447103, 0.28012637787760414, 0.43262891309704654, 0.3760526784390934, 0.3910142118170224, 0.6002638195831127, 0.3060851341820398, 0.3444921792091649, 0.43368693031625344, 0.04049725034491594, 1.0643879440364548, 0.67144863971648, 0.10641623667676295, 0.48379549626751533, 0.218535818564425, 0.3952896867371274, 0.3267645589823364, 0.8717860106203517, 0.9068969636098514, 1.091532183851687, 1.2047456174633706, 0.8447175339301375]], [[1.1312751012008393, 1.0243933273085934, 0.23800789019697072, 0.7728527644338499, 0.8383152259081488, 0.4637604756749262, 0.19157802427191128, 1.3394958473813001, 0.6878147384778568, 0.4925630289348085, 0.24103168974898248, 1.0657042631709626, 0.5890967278798652, 0.584357889498144, 0.24401926780907385, 0.45005476545632633, 0.44596356129182047, 0.43640843320783773, 0.6690894593040662, 0.27455443640600286, 0.32263721055778294, 0.35855657202449664, 0.029498918507374454, 1.2566825163697186, 0.6110170763290299, 0.1009719846578428, 0.46727296882359426, 0.17895704757548192, 0.40389414520856765, 0.314715133240588, 1.0212251363197062, 1.0055397309289107, 1.042048394819946, 1.1936511373459624, 0.8414821809400055]], [[1.1069296208185184, 1.0934609072595494, 0.23463799034349958, 0.7648851047166851, 0.714257649216741, 0.46195470809623895, 0.19054298468302966, 1.2274668345586637, 0.6829499926800432, 0.4715942264108368, 0.23018884184350924, 1.1225241626699893, 0.5991831970976316, 0.5383356092288706, 0.2073668076807466, 0.4364651786402751, 0.4503855585682863, 0.44691602048859724, 0.7117445131740476, 0.22693719513194865, 0.2664730341507922, 0.3201617362706428, 0.025048195778613672, 1.230005176114998, 0.5516998826618338, 0.08843325066865909, 0.5738408227815002, 0.14270953202762926, 0.47306160817192056, 0.22491194739797918, 1.1381590739973138, 1.1037586569918, 0.8540779512929837, 1.2327477206945268, 0.8558528346805043]], [[1.117576841986477, 1.0649740854008736, 0.2960560563146311, 0.7925371065160371, 0.7289703779412187, 0.4608333265048573, 0.19200422188837896, 1.1812016171598445, 0.7198436605109566, 0.46551864172955015, 0.1868795658211381, 1.0897711347991015, 0.6388955407055212, 0.5308190612954388, 0.22731440985020818, 0.4347251639188884, 0.48518998061974705, 0.4770709923954662, 0.6576845617421764, 0.27627755183954905, 0.2852974661582259, 0.24514916527421016, 0.024795392642873712, 1.1708075222746337, 0.5205689456805799, 0.07567118194712408, 0.647738579410565, 0.15122085543861935, 0.5036470430160442, 0.24076719103721012, 1.1802758003024822, 1.1847112896582361, 0.7203347695322911, 1.2637954189056588, 0.847105673558807]], [[1.0975279773471822, 1.0401404052340135, 0.32539204944829053, 0.8659598592579787, 0.834717646329131, 0.5734942148235553, 0.32093091984250055, 1.1335448502380303, 0.6712254312241481, 0.4414970926023457, 0.17376921871313372, 1.0528559786012164, 0.6728406391829365, 0.5649056295300021, 0.23405825914780937, 0.4407736863195826, 0.5694234125575615, 0.5470045673538217, 0.6701009084600144, 0.33889266763624737, 0.29687425975033926, 0.19073688333857006, 0.02502360983187956, 1.1027957174445278, 0.5609357349095624, 0.0706195573708919, 0.6589244960774058, 0.1719945726974559, 0.5457856265339354, 0.2820137905739736, 1.2285534160738236, 1.2595179902299372, 0.701646272804895, 1.3621286804364245, 0.9642657291554415]], [[1.1007698896716196, 1.0215510139284474, 0.3469053889069327, 0.8979310867150262, 0.825495928528083, 0.5836272244725718, 0.3236470922127216, 1.1256064330248041, 0.5977990456549109, 0.42685471041097234, 0.14411208545604431, 1.1073469140484982, 0.6962661288666752, 0.5690106550060399, 0.23961650175608618, 0.4358896407245454, 0.5689751237184353, 0.5385674925767345, 0.6653496865643856, 0.2942678538591746, 0.2721751072442234, 0.22848702271608362, 0.02461215344836171, 1.0659230730348057, 0.6766439034504207, 0.06496058232594032, 0.72525367173655, 0.21946799650785956, 0.5989911041380525, 0.3486303046552526, 1.281103550262469, 1.2592984366198183, 0.6494453549811159, 1.3983436749224398, 1.16202004589114]], [[1.061785044228816, 1.0491355669666875, 0.3250569818700898, 0.9403529036911277, 0.885052397483488, 0.5813942349138386, 0.3228297840221875, 1.0788550076958312, 0.5947168622163528, 0.45977302345965454, 0.1442563943151413, 1.2229759059416807, 0.733707246728242, 0.5877985786100476, 0.23280102618243542, 0.4300846543684751, 0.6606146448753337, 0.5298186022024821, 0.6823563531816215, 0.29880950236057563, 0.2833177274562404, 0.22714670942079465, 0.010996432058035814, 1.0729150474723619, 0.7110852515680026, 0.0796463938604163, 0.760504260407875, 0.2574620845591928, 0.5849190125067244, 0.43073634738331246, 1.374762918182971, 1.1351862094643774, 0.6871215167280061, 1.3705112604974294, 1.282290418637644]], [[1.0205980455441053, 1.1767689933666914, 0.263717843272774, 0.9548191376760877, 0.9150201259778175, 0.6148068089148775, 0.3735635675063898, 1.0474123303821266, 0.6610175199698122, 0.4795370312331454, 0.17716259329425027, 1.4038657771025944, 0.7225851972320682, 0.5479863305797289, 0.25889242433150417, 0.42483431806013905, 0.9115294425923679, 0.5737404590510198, 0.7263779710774976, 0.2520387621920393, 0.23232794787870567, 0.21370658044907176, 0.008170971366658522, 1.07059587899251, 0.8679053614939523, 0.10764614812959905, 0.8193392577209554, 0.4834314572934283, 0.569617218237607, 0.5064489248522196, 1.4725692932698806, 1.0747922832416423, 0.7326273926171085, 1.3737542806706895, 1.2575454767824237]], [[0.9721306359542887, 1.202390878675589, 0.2689085671761467, 1.0069483766094565, 0.950761149411617, 0.5575961614548577, 0.29331882874449855, 1.0221778550964378, 0.7198039935134818, 0.43392526378985286, 0.14050115559829693, 1.4026667778944888, 0.6453935176934733, 0.5515309819978216, 0.2547046372586896, 0.42644198593441784, 0.9150493667458998, 0.5374267693700113, 0.7235667489341522, 0.29657093881074176, 0.33825004154561567, 0.2543888746317305, 0.012604540583811661, 1.039423321574011, 1.1514724812603747, 0.15203838563406857, 0.7340843592851587, 0.6220781519717468, 0.5891368882793653, 0.5601700544444108, 1.586019221723816, 1.151212492359969, 0.6830146671853188, 1.337896058417807, 1.043179109136794]], [[0.9301746611205817, 1.2286581156901573, 0.2694150479322372, 1.017991531069831, 0.9973812619948246, 0.6110915602716642, 0.353314135473286, 0.9880149620082876, 0.7700002067823647, 0.4630825221174596, 0.12910636163041628, 1.267626885074588, 0.6255002966737044, 0.5609646188033376, 0.23768603915671652, 0.4283748526999014, 0.8042044222276767, 0.5708578145067662, 0.693560297826957, 0.2698790329481736, 0.29881089706276376, 0.28835373163744654, 0.006034691366983254, 1.0463866578264729, 1.4223758394937762, 0.3393142573083956, 0.6688747639512844, 0.7501029911726707, 0.6877634388704388, 0.5964719212887485, 1.5524202454301674, 1.1081187311328162, 0.6944064114792691, 1.1990921128224197, 0.7344452929664189]], [[0.8963684593626235, 1.2453441339446634, 0.2747453116632124, 0.9963587423346588, 0.9961249521268618, 0.5879663413600739, 0.3267687060266189, 0.9940846014101983, 0.7845457223399462, 0.4587704783797921, 0.10382687807634862, 1.2576460027683085, 0.6150916023522663, 0.5780577932620571, 0.2470284434442128, 0.42549825730637825, 0.7943388220142891, 0.5655053681569588, 0.670301300876409, 0.200878433206079, 0.23576243983347436, 0.2933650816023833, 0.011729942845374253, 1.0596707821396802, 1.761730980740305, 0.646244195296009, 0.6429444423116742, 0.8187101932960648, 0.8890948368669734, 0.6505569398654857, 1.3879477272824206, 0.9799574285934936, 0.7191787913596789, 1.0459891834196688, 0.4452465018153642]], [[0.9880409294291795, 1.3157938003248582, 0.27001600065798476, 1.0048680926208253, 1.0002431054858247, 0.6196376794765778, 0.33950915782855007, 1.2578887896148485, 0.9884056558526567, 0.4221065045360528, 0.16304089634835972, 1.3697149450146275, 0.5752876022828508, 0.5518877340667208, 0.24467507099044183, 0.4329043813927669, 0.8716501240876913, 0.5753378940138635, 0.6180488552705321, 0.12781071349695072, 0.21332035074948325, 0.3102946970611698, 0.02001367787710421, 1.0712273258149045, 2.12030183605262, 0.8914234606779539, 0.6463873058545271, 0.8954966807616763, 1.0636154147745107, 0.7637670018590501, 1.2385431527036048, 0.8513907613273456, 0.8074299938614096, 0.9721388187624179, 0.4072572472231783]], [[1.1015291759968675, 1.299552070787596, 0.2723870483794815, 1.005939009112328, 1.0004759115532216, 0.6171487807105267, 0.3369092818043788, 1.4403829370500296, 1.0730176927964106, 0.4992677783135347, 0.17066001634592567, 1.209235337781548, 0.5617295825690127, 0.5707104016444537, 0.31573591733582973, 0.42845335367796733, 0.8399998604840069, 0.5770868460205039, 0.6343321221167548, 0.10824186977302364, 0.1723401065932936, 0.30887478457861367, 0.02353488205892828, 1.0667863615611655, 2.2167329143707697, 1.0749344212224308, 0.6341374822380357, 0.8416025949272619, 1.2592143037696837, 0.8371794526499217, 1.1656643787393857, 0.7374475950285757, 0.9155309321068738, 0.9412559311617461, 0.5245529522024273]], [[1.0735522870913872, 1.270889627503117, 0.2626335470486379, 1.0223882463898233, 1.013523374294472, 0.5665838849405325, 0.2571395779814147, 1.4988378816418533, 1.0204801750452421, 0.5874846732922342, 0.13884936890278704, 1.2349919804741645, 0.5667330640516163, 0.5887260837029971, 0.3656897549847884, 0.4373058739530695, 0.8902074885230045, 0.5855029488217265, 0.6271739435100518, 0.1226368966187048, 0.16263587037159027, 0.26506840424332034, 0.017660737447543727, 1.0449583969254421, 2.186462390714955, 1.080428277389771, 0.5133280029813044, 0.834309941814864, 1.3293304505335084, 0.9084059017176094, 1.1112944031090652, 0.7191103296236921, 0.9550255467184607, 0.9323045292350136, 0.6026482453607793]], [[1.0203967923776402, 1.239782162759548, 0.26142622693007705, 1.033855260578571, 0.8662489259927371, 0.5800213700723923, 0.25207704178582885, 1.4244894284067375, 1.017097586083751, 0.593449448805014, 0.1431335797164187, 1.1834256933195288, 0.5813975510321414, 0.6355969043041425, 0.364718893170659, 0.45243863869258905, 0.8775908797677165, 0.5858009358982438, 0.6765591692673149, 0.13799117208768819, 0.15567217716019843, 0.2874367885695266, 0.016748767282730812, 1.0099260669208647, 2.048898422183999, 1.0721527448712074, 0.4726012942513961, 0.8104155920226347, 1.3651072220477687, 0.921091749355374, 1.0704559850135311, 0.8083189762559291, 0.9861793782262869, 0.9429995088113678, 0.6458921003436755]], [[0.8830998895667089, 1.2432651577268274, 0.27531447102842593, 1.0309745247864923, 0.8583689790895221, 0.5555621850754295, 0.23205247848351415, 1.278277600712178, 0.9373152178737253, 0.5015317509782591, 0.15625424690820297, 1.057780215997837, 0.6322777742236345, 0.6800293419362597, 0.3687384403733188, 0.45446888420807763, 0.8864055510268534, 0.5614984211900182, 0.6804869177203446, 0.14957654467279485, 0.15537788717990375, 0.3268822543129697, 0.019146103185717044, 1.0163448982007832, 1.9163770279868637, 1.1638426619318079, 0.443778695870088, 0.8425634487781082, 1.4041395016146907, 0.893923494603184, 1.0410001794361792, 0.850799557597939, 1.054840136514731, 0.917829900547855, 0.6846300352038845]], [[0.8340865523719416, 1.2311855807866894, 0.2825323106608543, 1.0751698256102924, 0.8426522885318726, 0.5708845847911964, 0.24612440716138326, 1.1379408529942643, 0.9199427111169217, 0.542214426085477, 0.13995863566921207, 1.1311190408420149, 0.6120353874822446, 0.6595595908589771, 0.35643249511423336, 0.40256896257517716, 0.8744923897422797, 0.5683989720502783, 0.7153203294319801, 0.15574794582483115, 0.15855144172574054, 0.3441976934166972, 0.021742646021722156, 0.9986263017532204, 1.696622856514161, 1.1958755312133889, 0.40388858047954035, 0.8030117540685244, 1.481400108142869, 0.8592496414626818, 1.0058182470380572, 0.860217122536647, 1.0704043647904664, 0.9499057083701413, 0.7273924099140319]], [[0.8501087245469909, 1.2689109357017354, 0.29584692875558943, 1.1042404690137049, 0.8375873071620767, 0.5860904326647645, 0.26634397671411414, 1.0870447677656008, 0.9652308623524598, 0.5434625778905593, 0.13742221784688008, 1.120401501208014, 0.5780285885439413, 0.6500145336246279, 0.32681628697969567, 0.4311607425908398, 0.9295809073627694, 0.5554149760712903, 0.7082323109079753, 0.1618654388002394, 0.16866093477626087, 0.31763551767635123, 0.014798111296177351, 0.9441219768790818, 1.6181480544799758, 1.260213658993945, 0.3295017892608873, 0.782476612177377, 1.50992342603752, 0.8259940381236632, 0.9427984654912568, 0.8327916429449489, 1.107932876530874, 0.948138658620056, 0.8163130751907278]], [[0.8904674728460118, 1.3760475840933577, 0.31058648195332084, 1.024393214062152, 0.8233582358678533, 0.5893712098353129, 0.2623725739851548, 1.0746303474981305, 0.8945063777590317, 0.5390389682083727, 0.14901413161151653, 1.1349648877243452, 0.5681436814736462, 0.6295704590785535, 0.3419767904857808, 0.44725025390941653, 0.9076849319939955, 0.5370774254755171, 0.7038425846824359, 0.17738099407947494, 0.17666928527315096, 0.3049780321458217, 0.01829932606361817, 0.8900968339174922, 1.7115290885994063, 1.1392016542441208, 0.2979301082856072, 0.7731194722740417, 1.5210218258505181, 0.7785335320970743, 0.8732999365769016, 0.814723316209296, 1.0113677660523541, 0.9197865202713853, 0.8404995696662212]], [[1.0779346243441488, 1.23732053131564, 0.28838718083913656, 0.9971317089461191, 0.8021635157541037, 0.6378603439143974, 0.29504599242882745, 1.1192772583630808, 0.7775095347473108, 0.5522236410525307, 0.13448943400654645, 1.1273691351615607, 0.6516341476114148, 0.6357125470387206, 0.3489860369630498, 0.44142240073308914, 0.9728221660770694, 0.5572445808546354, 0.71493581730275, 0.19715903975537086, 0.16831185451895736, 0.32050287634651653, 0.030655809803489326, 0.8241677689029014, 1.7943919876249161, 0.9955061837533599, 0.27207799579581743, 0.8003582767662906, 1.4897530712500373, 0.7349869798103034, 0.7930902392253458, 0.8336704923364158, 0.8829187905493325, 0.8770248825747228, 0.9116279739554513]], [[1.2005725497330642, 1.1918643273766172, 0.2837846687630358, 0.8925562449644867, 0.8479507901986826, 0.709977817974868, 0.3239484620718981, 1.121062151174797, 0.7036149547562164, 0.5818880782351773, 0.10013178576219564, 0.9501913844735361, 0.6315061522095455, 0.6423264665069583, 0.33570541127494413, 0.4204209941378829, 0.854941532006465, 0.492708974102784, 0.6761956432768751, 0.20852265286767066, 0.15747048652276885, 0.3254057829600461, 0.07581874940402766, 0.7632213369083938, 1.9515260591291308, 0.8909962731970803, 0.30776765606272627, 0.8723826186280097, 1.4747701194254161, 0.7146505891018524, 0.7282391913223345, 0.8355328364793134, 0.7865469231318998, 0.8367225377349741, 0.9570034647516704]], [[1.2305872638859903, 1.1390079588927309, 0.27999980787247275, 0.9475516969189806, 0.8544072235659976, 0.762945392030451, 0.3565437461364084, 1.2321468013379546, 0.7304892999691732, 0.6416463090622602, 0.09694664412937545, 0.943866463172337, 0.5949113041466636, 0.633545815357931, 0.36842375702733887, 0.42241419245440925, 0.7460854043538773, 0.497432469040846, 0.7346431274955746, 0.20397645776332543, 0.16226802332265494, 0.32159982767231454, 0.06286688346595151, 0.7866327418751267, 1.984993251615994, 0.7400500533365197, 0.38090261947595394, 0.8423425075211184, 1.2784297206575692, 0.6905760016056894, 0.7636430236968199, 0.8636293590482713, 0.7653718559385001, 0.8312742939673683, 0.911298737095402]], [[1.2082708228274255, 1.1131219598240232, 0.26857036964562914, 0.9697516900357498, 0.8595904408268309, 0.8178698139340548, 0.33826723303376394, 1.283869637068509, 0.7390347320683888, 0.6688106956544289, 0.0941430009601775, 0.9382390607585539, 0.4845259921916617, 0.6212332665919353, 0.373407703287056, 0.4252150457080619, 0.7448547582260239, 0.4932943091583676, 0.7409262599417323, 0.2382845064948878, 0.1784536374699413, 0.3511730074953109, 0.03652534757197817, 0.7759519948805753, 1.888075989603387, 0.6226825852820664, 0.40432452635779204, 0.8346135822718941, 1.139221211623875, 0.7354167295026054, 0.7423514016169672, 0.8626810273682343, 0.8408359960342391, 0.8941295569815679, 0.9086380357831793]], [[1.1836552917848682, 1.0897440198055528, 0.268571555413537, 0.932651381057808, 0.8316462859901143, 0.7717496209780106, 0.35179684030418146, 1.2879578198110604, 0.7381197172854046, 0.6512542439039468, 0.10259669938986866, 0.947976838147905, 0.43915917952378003, 0.6301124837072547, 0.356568243514085, 0.4143500805577284, 0.7393504441533696, 0.49558694475263887, 0.8083242472623922, 0.2147899975584346, 0.17203966928909242, 0.34305604247876387, 0.04477053015341882, 0.8554431556505524, 1.8811566411004546, 0.5062443309701588, 0.4443708393279106, 0.8219170107086348, 1.0946278894992036, 0.7563996859122626, 0.7010102132587652, 0.8787884999275473, 0.9103359950996636, 1.0076010477366568, 0.9100872091689303]], [[1.2151632874828797, 1.1242033830536402, 0.2864627924286966, 0.9243517630867999, 0.8309532633952723, 0.8151133675604296, 0.35950592152230404, 1.2107935988415695, 0.8038814063530567, 0.6384942723126873, 0.11805786150481115, 0.6276343934765758, 0.48450623486717026, 0.690318999528083, 0.3711760221694186, 0.41549638133374345, 0.6918397010160053, 0.5555631320465388, 0.8687467517100071, 0.17810487985889528, 0.16796154188132337, 0.39407634591062835, 0.06410697646888232, 0.8500095619819092, 1.8561116931218837, 0.44572280566612016, 0.49720496937481395, 0.7973365906163673, 1.0753725514074435, 0.7567505348705459, 0.6338653091211257, 0.9473499059413417, 0.986708320149172, 1.0987281220812828, 0.9252005087988107]], [[1.292825145945355, 1.170387522320412, 0.2880389411837467, 0.8059907070030049, 0.9173224327144447, 0.8690937838067184, 0.38734222517168954, 1.1498782074690497, 0.8663644721915936, 0.5062493108471077, 0.1336097483563531, 0.5458630361833314, 0.34605848524284694, 0.6626015079571008, 0.38972334540282816, 0.42287906262259944, 0.6765778493939409, 0.5529422341353583, 0.8048676317681895, 0.15288353986705833, 0.18066344368803028, 0.47607983783346763, 0.08649442407810609, 0.7306678891729125, 1.8186085573861224, 0.3985651067530193, 0.5623137999925192, 0.8471145457097567, 1.0728468362945769, 0.7250543065189814, 0.6719335097320704, 0.9782348156092009, 1.062650571029919, 1.1230730639854685, 0.9554864157757725]], [[1.4028598791556681, 1.0351861652477763, 0.26571004744085286, 0.6989317097269272, 0.958266309687422, 0.941175225204341, 0.39793237042801, 1.1311450385432331, 0.89788618494143, 0.42578946489777614, 0.09636099525847282, 0.5772644024203881, 0.3195146885818604, 0.6324522367800753, 0.3873584096226733, 0.451952504104002, 0.6710907192919215, 0.560848827764403, 0.7933202157372059, 0.15111702132082205, 0.18569621388484292, 0.5334376536452434, 0.04248314723531403, 0.7828124659614792, 1.6114885988164809, 0.2630660941657542, 0.6659929367547981, 0.8215517404215965, 1.0239019726631569, 0.6706318498041852, 0.6150108955363026, 0.9671993378907651, 1.0916031379709097, 1.079440814719617, 0.9597128562857214]], [[1.3816209958673298, 1.0121229114529404, 0.2677104172218025, 0.7413583764259829, 0.8626410341767922, 0.9844149818873627, 0.4069175628915324, 1.0490269356604922, 0.9304666859945241, 0.39405350225625313, 0.08356310417454636, 0.5695428982761762, 0.34058314386417265, 0.6342717880018525, 0.3923558014216264, 0.429477589406424, 0.6159821981994414, 0.572344522826693, 0.7895717012016138, 0.16763255009624692, 0.1748490427260837, 0.6077990561076234, 0.044925720195118535, 0.8170186652737664, 1.3670646851435972, 0.2801085050625558, 0.6751684993615176, 0.7101387865202118, 0.9596308951115481, 0.554605570933141, 0.6191109776743187, 0.9293131321228165, 1.0242551985422796, 1.16064509469629, 0.9878192321497142]], [[1.3483732248361564, 0.9641518287385902, 0.27478466348462666, 0.6964067459001824, 0.8229428885375674, 1.0230575846527437, 0.4007422267261197, 0.9891424820927573, 0.8612228140560931, 0.39741667980894274, 0.05561196587090291, 0.5169517566644722, 0.3890040633579456, 0.551331515923046, 0.36270138190950707, 0.4114371103779919, 0.7084369585313398, 0.5851162709003617, 0.8028113882538569, 0.13987308297304973, 0.17282481291629637, 0.6064972046721736, 0.05102648665914407, 0.914686855952056, 1.279173715499719, 0.3023256866142989, 0.6168845633175475, 0.6723158866872674, 0.9217509293656665, 0.5380811092215452, 0.6320951619331531, 0.8687939532262232, 0.9106126234055307, 1.259664967830363, 0.9515131940923809]], [[1.3189825066640828, 1.0035544203120401, 0.2661983201267855, 0.7705324862100716, 0.8183705119850136, 1.0911546171005266, 0.42194027034937315, 1.00868291810138, 0.843052197454301, 0.4407181199129244, 0.03570840979347356, 0.5140442122506467, 0.5198708175603051, 0.528501813466225, 0.32825974846216843, 0.42880884639168293, 0.7768874612454002, 0.6061698494883834, 0.795212214967896, 0.12287134084421625, 0.16337112057308223, 0.5331502164637439, 0.04713560335187185, 1.0806583789600082, 1.223651629334663, 0.30956835318588066, 0.6218651972309872, 0.6702551703917372, 0.9250194669310685, 0.623616954631141, 0.6572768434274243, 0.8883107778691769, 0.8781948175989089, 1.3803547796792748, 0.9308929346549977]], [[0.9873636377125882, 0.8580630901129513, 0.2657558236123854, 0.6572972323346287, 0.6105127434643867, 1.108446278118873, 0.4057756832482345, 0.9559214785912182, 0.8939993854176942, 0.522875055936026, 0.045263000156369554, 0.523435498611611, 0.64058984013343, 0.5160114977665406, 0.33458259423383385, 0.4234211172291158, 0.8539386720251474, 0.666248330770653, 0.7932570245080046, 0.12374883889273236, 0.15216682869283582, 0.45662270280281336, 0.04301063613585435, 1.1142022450830882, 1.278567902310209, 0.2867517602030199, 0.6429111209847109, 0.6224735008252783, 0.9060804973247364, 0.730802175186449, 0.6737460840178452, 0.8763909784021526, 0.852101385500654, 1.426802637870881, 1.0191368683088957]], [[0.9453880931854588, 0.8802829619135724, 0.266117682470342, 0.6187550922744752, 0.42577585417128466, 1.2036527034915618, 0.40100184931247435, 0.8249903451636463, 0.9097392460251568, 0.5872032901273008, 0.053405920678338104, 0.5331873908528191, 0.6438745660127712, 0.5102276116878302, 0.4293829596723767, 0.42747490789383147, 0.8663816270485307, 0.6414683983371032, 0.805454472344683, 0.11942545253157313, 0.1490822056539356, 0.45610223546577366, 0.058887756900936096, 1.1136120650519477, 1.4317705680942006, 0.278048317998461, 0.6619297790880517, 0.5840314135885566, 0.892598242151375, 0.7679495477473576, 0.6850358911634438, 0.873116797524737, 0.8116654969455951, 1.4693719600646038, 1.0803821620997713]], [[0.9693497266346538, 0.9034228054350999, 0.26687792947553474, 0.6319963586919887, 0.38720931855810514, 1.22311433056389, 0.4059961637651913, 0.7951224185321768, 1.0065764949549685, 0.6200940203708672, 0.056432035413933324, 0.5494945227772088, 0.6450959004042343, 0.5081104925016359, 0.4348438936082096, 0.40369358372022, 0.7969998884405215, 0.6499020009543445, 0.7579917230484465, 0.13762340305577658, 0.1609219270639727, 0.5070245588617117, 0.05193568443384211, 1.0747295777222212, 1.7667472808322735, 0.2727637398456304, 0.676302366359764, 0.6489668394428508, 0.9435874959029595, 0.8140872472426128, 0.6804476681776364, 0.7878138786919135, 0.7608121598525636, 1.4961862728824913, 1.0697406914571894]], [[1.0135662668214127, 0.9256223487289369, 0.2680112672727349, 0.654742054075033, 0.3808214766875806, 1.2386446944095995, 0.4068666174043153, 0.7604109194080246, 1.1290923867835905, 0.6435385523656851, 0.08411133203647948, 0.5280159329265764, 0.6644706327150895, 0.5472336436230717, 0.4173079256098967, 0.4018112249343624, 0.7297288188126886, 0.7835022145636835, 0.7727932172299875, 0.129084301589106, 0.17850050641409346, 0.5031379387697974, 0.05642308751937884, 1.0487823987627454, 1.823142965841241, 0.25931276119025937, 0.6981070199447152, 0.6303400573088056, 0.9926061291699009, 0.7270781879602675, 0.6629218653493194, 0.7772980210452574, 0.680843779162648, 1.5457169631686063, 1.020731744828634]], [[0.977429280478131, 0.8899947832112967, 0.2743808225433782, 0.6875725930621972, 0.3494020135793168, 1.1205187907100969, 0.38196194268819705, 0.7364324383681797, 1.1894437609325932, 0.6556450826282304, 0.10099741295866434, 0.5297281862919335, 0.6679889272009485, 0.48780076021815433, 0.45485482898878704, 0.4403349286175019, 0.7134228725698871, 0.7529557083294989, 0.7450948684670917, 0.1506594580792886, 0.1817315786646783, 0.46283304613133763, 0.07042961868435735, 1.0329502609238708, 1.838866622353407, 0.2198901053022415, 0.666226225540623, 0.6306827833669446, 0.9622907467859085, 0.679453596811042, 0.6487352984351975, 0.776055264844374, 0.6618342647938737, 1.5888821736744334, 0.8045544450228072]], [[0.9718338306748187, 0.7585077679225816, 0.27201934349329965, 0.7048764701346188, 0.3436218462845674, 0.9948900233944019, 0.3748607299175635, 0.7266348318187862, 1.290378583024795, 0.6628985138006976, 0.06816175993963317, 0.5610190000029285, 0.6394837597564955, 0.479617697834976, 0.39732360100796965, 0.4619349848689749, 0.6959306205928144, 0.7235821221622191, 0.6705703316223441, 0.20328793037915463, 0.18262162450896596, 0.4544748294304025, 0.06988797565331137, 1.0168861348180385, 1.8286854439138178, 0.2644832447359896, 0.6791562017711195, 0.640034940607174, 0.9315414358253847, 0.6553064024160998, 0.6290499101682026, 0.7719041900679464, 0.6559479791183073, 1.5118281937074944, 0.5824558692127841]], [[0.49414567703982426, 0.7602033522167735, 0.27026645555928636, 0.6267794004182097, 0.3783590573328709, 1.0930907186710686, 0.3753545047556873, 0.7492767930319055, 1.2279906681765835, 0.6799736973632894, 0.0663024345517941, 0.5746617285035909, 0.6869685921326001, 0.5274441004314729, 0.41216651845986024, 0.4669519378319923, 0.6726520297768335, 0.7379430522441542, 0.5948426490926588, 0.2061222246869452, 0.17409775010552103, 0.4458625029421547, 0.04041091389732115, 0.9580936157313166, 2.02025431287271, 0.2940787259471634, 0.7220525068886103, 0.6878662094461823, 0.8967711846014361, 0.6668975538109068, 0.5812400102727346, 0.7574047094831275, 0.637552150281256, 1.5215824772591118, 0.5704829836782281]], [[0.47364003574155944, 0.816704250011392, 0.2773353237904056, 0.6394011104953174, 0.38066434789520925, 0.9968039265422461, 0.37083829620449116, 0.7119086789932989, 1.2860983937841635, 0.7030302839373221, 0.057499664523409016, 0.6014330303719657, 0.7033363041506542, 0.5064112073154583, 0.3343063339296789, 0.4738513560174996, 0.7971669392081975, 0.7526443513854447, 0.7627366094307163, 0.18196177652358356, 0.20240979077378551, 0.47954007823341066, 0.04332404908250351, 0.801692255884383, 2.1710581093094143, 0.3237089220124182, 0.9259942861667674, 0.6834513033519698, 0.9164263857525123, 0.6292823907794679, 0.5642888040822684, 0.7490531509090441, 0.5969670675078452, 1.5643493386777763, 0.5761218905560561]], [[0.4729639158666683, 0.7459320067343391, 0.29362656426463474, 0.6176101220523331, 0.36505689677267505, 0.920142852570484, 0.37343800117960907, 0.7476765241036074, 1.3209453380641552, 0.7180233974932773, 0.05464757040420322, 0.600074599582501, 0.7189403095922916, 0.5418785496673268, 0.37748772863927915, 0.4836475472643518, 0.748241822600552, 0.7213019855611048, 0.8092380450609991, 0.16028427852200208, 0.24582533861012476, 0.4343442446886614, 0.05765057371152748, 0.550399392255417, 2.0893275830907942, 0.35285887296830815, 0.8204693587488687, 0.7551475872819293, 1.008082616423776, 0.5881197482739967, 0.5247371296918149, 0.8051438025295492, 0.5386389686506368, 1.5390945149683968, 0.3936177909343589]], [[0.46225664748971385, 0.7905832709705415, 0.3139229163038362, 0.6339002702812707, 0.381879943049468, 0.866744823282091, 0.36034250393158457, 0.7277799986997122, 1.3670486043969245, 0.722282790161908, 0.06724811933138464, 0.6351443583459715, 0.7459430421618233, 0.520371576509877, 0.4354382442517082, 0.4710921916239875, 0.7367440512377763, 0.7298264866563507, 0.8566279353393297, 0.15828445152825593, 0.1974386750500662, 0.4008752914847318, 0.050627710545152145, 0.5742748838038233, 1.968626931234206, 0.36434931073647625, 0.7495657203924831, 0.7606649963734062, 0.995605088635257, 0.4969395874784292, 0.4544631180044539, 0.8248255609683879, 0.5108599994599772, 1.508597295687063, 0.37679769673865793]], [[0.4586824494878133, 0.7540173740313033, 0.28256613567435324, 0.6887885826526612, 0.3803656059155973, 0.8829156860190298, 0.3566607429266159, 0.7682187195291181, 1.3568208923172966, 0.7226261500822568, 0.06779930912303389, 0.6010109846986345, 0.7859501708274302, 0.5082150523067689, 0.4594189419879516, 0.5151806066624436, 0.6717083100828294, 0.7250587892078989, 0.8380481977802808, 0.16995035189337604, 0.1967122477057403, 0.3841197937031181, 0.055434105068653206, 0.6381454218788143, 1.9458179701391298, 0.3239710201542681, 0.7771215203684921, 0.7097174487125115, 0.9743011124182606, 0.4277732806846569, 0.43938171891877875, 0.7943009163999162, 0.5100520544152686, 1.4673797573981502, 0.5666137594477215]], [[0.4348492277121786, 0.8092013588846482, 0.2793700067638265, 0.8364266332764752, 0.3188077727519987, 0.8851303615863204, 0.35407232254553733, 0.7848078382120911, 1.190779834007506, 0.7168817051919456, 0.09423588756317619, 0.6069639443140075, 0.7947176777974481, 0.59961284036741, 0.46946918439201296, 0.5261948302860502, 0.5690472519275994, 0.7093329248228389, 0.7280122848997231, 0.2038509714146317, 0.19422038864568109, 0.4169204452883934, 0.06134927750288716, 0.6492416955602336, 2.03249845513628, 0.26747180512079033, 0.8147333382094718, 0.726456603133599, 0.9621306217161807, 0.3847928166531955, 0.4673140375842386, 0.624990781215714, 0.5304336116136883, 1.3649695229963323, 0.7240939865244744]], [[0.42152073170465226, 0.845972994684881, 0.3081026949324133, 0.9459802033269279, 0.3428558321686659, 0.9225084697757217, 0.3652274198961003, 0.9651376039133429, 1.0215227588068838, 0.7108593683382277, 0.09281166707692158, 0.6520288686362792, 0.7853189823682512, 0.598820068188359, 0.3908363226187457, 0.5381088662910102, 0.6157854995070027, 0.724801576236222, 0.7906289643947725, 0.20978595494829166, 0.199848058753764, 0.5767924801424995, 0.10935882733321114, 0.6701255735249372, 2.0953615633641283, 0.2841770840428287, 0.7911185001050565, 0.8314965985605705, 0.9419661545214566, 0.3600323354239784, 0.4148944661876117, 0.42776694895915607, 0.5532470583917375, 1.3343961004470886, 0.9271665172704613]], [[0.390231712990915, 0.8758986465170571, 0.26861618706750573, 1.062548475863504, 0.2804701611202649, 0.997702077171079, 0.3545488012725018, 1.0582748297168307, 0.8109092418405437, 0.7173643483325727, 0.09637585037174314, 0.7188207134909725, 0.7963357788532841, 0.6131992445085432, 0.3980759986360517, 0.5279771858947901, 0.5665185785646976, 0.731693998496504, 0.6685263324618499, 0.2115399594198586, 0.18985361936458225, 0.44519870887129287, 0.04975363461465356, 0.6585053801250074, 2.104352164413677, 0.27914502262173807, 0.8264381725984637, 0.9053179284922256, 0.9452630562821608, 0.42514020171644873, 0.37955262717149374, 0.337636687274848, 0.5708361073348672, 1.203091040029673, 1.021974296815197]], [[0.3947616001012903, 0.9027213984778479, 0.26128192768489544, 1.0596140164690722, 0.3171497916201068, 0.8960366055935514, 0.3559008776381945, 1.0836114840990456, 0.7153470838403251, 0.7353169602284659, 0.0962648295427568, 0.7510802591692731, 0.8563265467526734, 0.6550182140565097, 0.3974998878993373, 0.524714947550939, 0.635815268373639, 0.6584301468353314, 0.6871803907596037, 0.23746800636587867, 0.21397290266355082, 0.4768583188129061, 0.0699189076343697, 0.6288595120713654, 2.0896458920974457, 0.2294801602900547, 0.8408888859057186, 0.9018677472081897, 0.9510845703190478, 0.47894192171421446, 0.3639698443980662, 0.45874947073663513, 0.5941170906065671, 1.0706293053875378, 1.0856194969751065]], [[0.35127755269878136, 0.9459423542646037, 0.2641418398960851, 1.124169825918695, 0.32238282107125305, 0.8881769189630013, 0.34841112627223775, 1.069760141271363, 0.7355156987464887, 0.7538731104354975, 0.10255159322842526, 0.9142513045982755, 0.8395650559867445, 0.6137718132799383, 0.40989689957277375, 0.5362385334718947, 0.7680555120166951, 0.6487525829014785, 0.6154626948145221, 0.1365834832479247, 0.220091966592865, 0.24780322353627776, 0.03951211853876431, 0.6306666122601068, 2.0886739744157756, 0.21883733364603186, 0.7817860239570189, 0.9168872438974931, 0.9678892040994828, 0.5833431894466968, 0.374566565582628, 0.34145612192960795, 0.5930599439244518, 0.9599802473859387, 1.077158370004205]], [[0.3265966944076725, 0.9816538408021644, 0.2710425131052805, 1.1908213125280247, 0.3215455955533082, 0.8302483972548429, 0.3411789029535832, 1.0977655051360107, 0.7553468078874765, 0.8194144410269358, 0.09495692475340535, 0.7610467283851794, 0.8248580344290977, 0.6242836353044412, 0.33035935383875437, 0.44967031459928586, 0.7858302858592132, 0.6773502008703947, 0.5564533998839561, 0.09777670526663723, 0.1629240132271381, 0.16723279880909786, 0.04530915250450602, 0.715419824943893, 1.9898775875583854, 0.21810053514033512, 0.8238213351471309, 1.010285875815527, 0.9557277647531405, 0.709614523356268, 0.4690085053477949, 0.38431896860095655, 0.5952685435938083, 0.908744945632703, 1.0428481441767952]], [[0.32039816749917294, 0.9345085103046661, 0.27975104521642635, 1.2257814792807866, 0.3174228611618224, 0.7855220500235753, 0.35328147220048645, 1.168439904004371, 0.7894305280701772, 0.9458093310737727, 0.06706621935263143, 0.7546628369426363, 0.7810553513578522, 0.6009063679625822, 0.3497080012255584, 0.45001152797594957, 0.5768671478844764, 0.6941415977073236, 0.5460855086769272, 0.1165678583195005, 0.14870639430612287, 0.15817005590435246, 0.042137832342759404, 0.7834151789612704, 1.8955672100721344, 0.27457593938602454, 0.796482991458152, 0.9600384040989999, 0.9281210407673881, 0.7224957725209408, 0.5314064748902639, 0.37564634203644603, 0.6103932961000811, 0.7815756168536739, 0.9669764822699123]], [[0.3064080552670793, 0.9536027643742143, 0.31504686623481937, 1.3001382182488246, 0.292933678036415, 0.7917253080718348, 0.3510965471302911, 1.2183405744828344, 0.7992319399677242, 0.9200180883185451, 0.07718177475217831, 0.7094091116348804, 0.7227912705419599, 0.5872656549230035, 0.3368013727723105, 0.4454283756027979, 0.5184292232898107, 0.6828407546192556, 0.5841923774334432, 0.13568748176703171, 0.15729717951040986, 0.16703760573361726, 0.04910958620773865, 0.9171515988442567, 1.8604921467202407, 0.31484927470369906, 0.848865126047252, 0.8803893951090098, 0.8744237912084021, 0.7714556544965788, 0.5506021525754544, 0.34089594220299035, 0.6016495477467609, 0.7157456110645756, 1.040054613951493]], [[0.29325812317920424, 0.9528605801154416, 0.2998476084204226, 1.3201827674004774, 0.3535158994533245, 0.809159684461624, 0.35506035878276054, 1.2431379819249344, 0.8008810069576682, 0.8629765581095392, 0.09285310211118733, 0.7384836359311822, 0.6376768294018458, 0.6417161125371953, 0.29611377423577423, 0.4595306528408586, 0.5440809429904755, 0.6760353982395093, 0.6174766002997896, 0.13446072712871787, 0.15140660001725964, 0.15355686034834948, 0.059733326579608725, 0.9066450356707554, 1.8167965439134612, 0.3379021139936097, 0.8622987468642977, 0.8268031058764056, 0.7639205363903759, 0.7790421266294114, 0.5799183665169264, 0.30047323105659013, 0.6016280796881418, 0.6724342908209051, 1.1255451331563535]], [[0.29615612550554604, 0.9520372497301569, 0.3035528700607373, 1.2775917517229622, 0.5142544276917932, 0.8248343901366963, 0.38770346566920955, 1.2466954878175551, 0.8396731931885895, 0.8576333213830782, 0.10781171164436508, 0.7213234104949879, 0.6042727493282155, 0.6718842503252556, 0.30768324087002286, 0.4146849894587965, 0.49738457955303395, 0.6868837207046301, 0.6612004449804341, 0.0968148650116872, 0.15401494969861146, 0.1441211073801331, 0.08482923029425775, 0.9071140915937489, 1.7420183982095865, 0.35205673842864155, 0.8506889042227412, 0.8051587243031529, 0.6284754680681318, 0.8488555275160656, 0.6474750248572527, 0.29636225543191863, 0.5793584492232015, 0.6047035986542258, 1.1568435633999519]], [[0.2775109300742622, 0.9401742890967322, 0.30878372328906917, 1.2082576083060008, 0.5463846868157627, 0.7987164406489962, 0.4077990904090123, 1.286599877557764, 0.9264271225733459, 0.9066499389287441, 0.10071573957903013, 0.7270779304573638, 0.5709271085043514, 0.6740367286308365, 0.29400886526105696, 0.4261607190775393, 0.5357606093373138, 0.7047941105363137, 0.6681992416254094, 0.10832148610181863, 0.15395224699789473, 0.16464601950778046, 0.07352116033425683, 0.9507789685805252, 1.6585927441053971, 0.36019014689492, 0.8108252899688105, 0.8068132850755333, 0.5526128904012151, 0.838099198696818, 0.671895631716031, 0.3827332174944188, 0.5184501096649213, 0.6211410818436562, 1.1410775091106355]], [[0.27372702526564674, 0.8952970773146784, 0.313476401267168, 1.1920634860339354, 0.5839675991952038, 0.7863211151225737, 0.38724832750158966, 1.258146770126942, 0.985164996910844, 0.9605655996169871, 0.15189805014307167, 0.735593111991931, 0.47944766667903094, 0.720304564230219, 0.2955059212040178, 0.41003247715046875, 0.5976237112304374, 0.7136390422113698, 0.6307930885003042, 0.09386241464748973, 0.16188512073387226, 0.15526498538636774, 0.057659551887727895, 1.0837275257066488, 1.562902612153777, 0.3738870006519913, 0.7581782514341193, 0.8624051701267201, 0.48498799931468867, 0.913161698146641, 0.6293761635601236, 0.408258528413174, 0.5137486679500903, 0.7171524856214941, 1.1454673640616277]], [[0.2563325921741925, 0.8950819770315398, 0.31814366392444937, 1.1647591430455473, 0.7047857891339869, 0.7488089508727334, 0.38376901365967103, 1.2192437421607432, 1.0572997412311178, 1.0581997916390287, 0.14713210187099018, 0.7116528359624832, 0.4189998449410137, 0.693751740322865, 0.3017686429426373, 0.4269667482965376, 0.5191557821655841, 0.7337466452726208, 0.6366669482561502, 0.08966758280489329, 0.16256259466427528, 0.14561239587823635, 0.056284472270527665, 1.1308047154489023, 1.4877559943913357, 0.3309703007081554, 0.767446061713107, 0.8488629831060639, 0.4769407795432325, 0.9319571817043302, 0.599422285064565, 0.4777484758423099, 0.506683756436616, 0.7303602066538348, 1.154608658917204]], [[0.24991854977429387, 0.8961449137078605, 0.32727284141484486, 1.1732382995181787, 0.8086720808492442, 0.7475301299055559, 0.4114308515792649, 1.2554139577771721, 1.149062100150258, 1.269763377372915, 0.14601609172895802, 0.6167034701818384, 0.3838729654674981, 0.6613241376803001, 0.2900868899721992, 0.45007403303549154, 0.48792425537167805, 0.6947865683624006, 0.6342212103741405, 0.09829610402717781, 0.1739704244097549, 0.1446207623768577, 0.05564331735972786, 1.10386121362402, 1.4232298502052778, 0.2783421210167148, 0.7387520544849907, 0.8547554501564374, 0.49311213972204154, 0.9459018278103681, 0.5274534892015385, 0.5094719445147927, 0.4414955813508785, 0.7180184452489358, 1.1764646924848674]], [[0.2799407661188015, 0.8608294555295694, 0.3148233600558309, 1.0692103453974338, 0.8527306290271836, 0.769564750229528, 0.3833904171418043, 1.2640513507822615, 1.207317402594641, 1.246607329315237, 0.15609593652113196, 0.6510563551877901, 0.2858011289393187, 0.683395187499495, 0.2884836325759793, 0.4606362137550699, 0.5121308640755554, 0.6715790572083334, 0.6304495438140078, 0.12922501632642508, 0.19905840257025545, 0.15211223183777917, 0.07152330466942491, 1.1051888610572198, 1.3659107810109439, 0.30042694163550265, 0.661890878049659, 0.8371234406797545, 0.498053123963089, 0.8676376952991501, 0.4314707278251399, 0.5244632749681687, 0.35838953511971006, 0.7357911502510299, 1.171549547579797]], [[0.26527264295860215, 0.8291836183346327, 0.30413115721712036, 1.1471306742383802, 0.8646529623553454, 0.768966862327972, 0.39076081437178994, 1.15534394928298, 1.2540622555661018, 1.1027036718160232, 0.16889602442710736, 0.6322485771008773, 0.24929274466069457, 0.71885400806712, 0.2985477571945713, 0.4825400276790258, 0.5466732258186884, 0.7240591844913034, 0.6052589307570414, 0.1395122429298994, 0.19017095757515687, 0.1677490152818662, 0.01835381012717518, 1.0524829931001152, 1.2144338067199691, 0.3310899418160383, 0.5736945321787762, 0.85960992848321, 0.5003648473325106, 0.8333380715995118, 0.37227917239433383, 0.40695843347284893, 0.3674132766985262, 0.7495177426267217, 1.1971116677937528]], [[0.2538889125644811, 0.7961595091518193, 0.32035826372782994, 1.0519716466121574, 0.8579306209240427, 0.7990561716896489, 0.3958130672699084, 1.057496475067435, 1.3092159142236621, 0.9900043683281181, 0.18008336694328356, 0.8351811377514925, 0.5525120769914822, 0.7492775118044797, 0.28689112331134087, 0.43728966598515867, 0.5119163866948184, 0.7401103030707774, 0.6014536711286185, 0.1463395663398731, 0.16755343421949412, 0.17570392387256673, 0.017433552767483804, 1.0296517911079286, 1.075549822820784, 0.3832821292157358, 0.5526659200519777, 0.9059275088629806, 0.5536786849367998, 0.8173901434238602, 0.2912048840450341, 0.29226785895974616, 0.5539782532675837, 0.7723144009121763, 1.1755973859939337]], [[0.28541560402903177, 0.7802143738605705, 0.3495971881853925, 0.9281480668265613, 0.8859464276220255, 0.8540398222005992, 0.3963705517106932, 0.9225234372441591, 1.3519151958537734, 0.9680315764135143, 0.17805338686905564, 0.9126825845403335, 0.28289119820781233, 0.7081734464844892, 0.29240743103029054, 0.4504632807881442, 0.510517868808926, 0.6784712398874087, 0.5976396961746546, 0.12982018827023883, 0.1811597267360531, 0.11073153687158557, 0.015861999995535357, 0.9157647501141009, 0.9099137904155293, 0.4642707724374753, 0.4948295104218958, 0.9674782515615772, 0.5865753828801761, 0.7889584905455294, 0.2390824877440783, 0.2859215818905344, 0.6332751830641874, 0.8056070842586838, 1.1568275191050787]], [[0.31859305111526254, 0.7865314033272872, 0.3435846349041567, 0.8625461314942482, 0.9115042191959998, 0.9647495548619084, 0.38916357000810536, 0.7414374544628317, 1.4091084763766903, 1.0708377470509283, 0.25052167120733515, 0.9253528992475535, 0.26412937403102454, 0.6728455859664565, 0.285719664107165, 0.45869131961627246, 0.5592158177647742, 0.6674999503826188, 0.6188306498195035, 0.1591876748157844, 0.185934984784064, 0.07735435164955046, 0.014355246472220706, 0.7253147101764863, 0.8121687172693487, 0.49361763731598685, 0.3854799457382023, 1.097558504681314, 0.5992225025329587, 0.71768322381693, 0.2969853296822491, 0.3804936622850284, 0.7303374398449896, 0.7707952694026855, 1.0334652457794384]], [[0.33094520626931134, 0.7506752133633916, 0.32632257711795, 0.7817485216666001, 0.9391503443309623, 1.1583933081848605, 0.3882053751513944, 0.5604613842511095, 1.463301061839387, 1.1949626422731552, 0.25563196361127793, 0.9564262189555464, 0.2690912990119678, 0.6142536937660124, 0.26822571946282364, 0.42896001422016655, 0.5922330783075409, 0.6317289188462561, 0.6538550424799309, 0.1949969662493709, 0.1851240227389401, 0.09730187768879346, 0.012915691900009161, 0.5603651128548334, 0.6799695696711041, 0.5658227818054815, 0.263689750034777, 1.1633806485906426, 0.632715132897568, 0.5771683334426434, 0.4845015425960991, 0.543294212152854, 0.800534474544011, 0.7622998861155587, 0.8726306161350087]], [[0.27517938160019795, 0.7176511136792615, 0.3430938168378491, 0.6990510087153756, 0.8646449904127209, 1.1661693389907082, 0.4124168550757138, 0.4886281925232302, 1.4535630865862932, 1.202210913557455, 0.22441862346841215, 0.9460281654151703, 0.2666310240783303, 0.6103012843573898, 0.2728769656786866, 0.4178694276646288, 0.5892770108191354, 0.6486202683775164, 0.6905822219590966, 0.23605908259000807, 0.1932638214792569, 0.07856405551573263, 0.010468928758567501, 0.4630180911499002, 0.598975912135121, 0.5919456480825134, 0.2486731015883557, 1.160674712730208, 0.6891757048869638, 0.44962384280165624, 0.6274005731084575, 0.5642390504423175, 0.9218759281336655, 0.7650537237962045, 0.7781957314833587]], [[0.3311726418263814, 0.7330218211127058, 0.34648804634856506, 0.6311794270881131, 0.7816566754561758, 1.1572871167453809, 0.45855368911202515, 0.5244419001412406, 1.40914333213698, 1.21544892481028, 0.22931981943769608, 1.0319411034204995, 0.2678941233229841, 0.56416496944331, 0.31985332608518136, 0.39367026092525415, 0.6061007265510296, 0.6482950120368881, 0.6772601735901236, 0.2511407537063, 0.2028627693887281, 0.07240029952631608, 0.053699355852797115, 0.392675741537892, 0.5455431676462504, 0.6913060949483476, 0.33213289007143354, 1.2565066657239579, 0.7496358259996254, 0.5096190226873442, 0.6253436669018779, 0.5319673712137013, 1.0315849493178542, 0.8327184889671015, 0.8831178078489299]], [[0.37708597803893934, 0.7955066263851144, 0.3543652515104139, 0.5978084379463015, 0.786637852224362, 1.1065426931899052, 0.47684407284071223, 0.6047700258271124, 1.333353770393598, 1.1381909413359215, 0.35848595134031846, 0.9169579292720709, 0.2693595829987694, 0.5428218845897066, 0.33903490680828746, 0.40532304903059746, 0.6760575387027685, 0.6632402458601996, 0.6558207843434442, 0.2524367232146272, 0.2142901450226593, 0.07549787739894019, 0.05362432845568743, 0.353314771988705, 0.5723830216654493, 0.7705580452785661, 0.42267322886894343, 1.0636372407126444, 0.8376861796578314, 0.3876346232180303, 0.6676056896849074, 0.5503731342631293, 1.0655602672912985, 0.8629708486511334, 0.9682104186588305]], [[0.3686370859985779, 0.8065692248485574, 0.41279556100030046, 0.6045636245305521, 0.7618544001346224, 1.0987866678684983, 0.4933127102027256, 0.6885493705692616, 1.2689349305538586, 1.1094560558403077, 0.30921138692039607, 0.8921797621401699, 0.2791001065029377, 0.5637680469314377, 0.33914274100013725, 0.4205013819263004, 0.7144212021288949, 0.6514783525392283, 0.6663512313310773, 0.2591195760943168, 0.2512691869770615, 0.0909387741796657, 0.09264549653870179, 0.3118194060957476, 0.6073081155278228, 0.8168615189912708, 0.5390195938918072, 0.908923629861426, 0.9231842280711028, 0.29637315381378937, 0.6997564886560027, 0.550137210780049, 1.0561573870735323, 0.8453258751330073, 1.0579666878497629]], [[0.24980607548885153, 0.8177569801256634, 0.4122351168037296, 0.6253406005842213, 0.7555282722590906, 1.2162476998646214, 0.5172708382770541, 0.6767392377542671, 1.1798066374486855, 1.1644901193402997, 0.3713229661456088, 0.8989225371531407, 0.28454368944230657, 0.5927296898435, 0.3325492505607544, 0.4005221789306018, 0.7221053054364502, 0.6595961185991568, 0.7032955183187293, 0.27264993898506434, 0.2654637924731862, 0.09330382503340981, 0.11439804378361564, 0.32337610968916697, 0.6352953426334706, 0.8168559906253061, 0.7114572214739591, 0.7969310225544788, 0.856045286400308, 0.24335003793260887, 0.7618929624127289, 0.6054083178624575, 1.0469012588224276, 0.7613604435815602, 1.0362083954818566]], [[0.24561134072616275, 0.7758041702811583, 0.4325094661561518, 0.6742266937744563, 0.7305090882585056, 1.251049446867773, 0.4998188205910293, 0.7185120187655762, 1.0994864522024108, 1.1011356092256759, 0.27209207965709126, 0.8878047494763942, 0.29571887247663453, 0.5695496091976942, 0.3141314561597022, 0.39991979819835033, 0.6161008057444406, 0.789438525996545, 0.8299202215430733, 0.17075967200243314, 0.2527989066616414, 0.07467515800684638, 0.10224536654899134, 0.3658712238268427, 0.6081911409674992, 0.7210113880118313, 0.8650056531148133, 0.6228875850001389, 0.8131108632260362, 0.22898422153111694, 0.7117152605432787, 0.6233206923558043, 1.0197966558027323, 0.7219988134675535, 0.9669685016404307]], [[0.29382457098234893, 0.8119463355916061, 0.4800281510666964, 0.7055299197848003, 0.6969267575809674, 1.229345208752855, 0.4653794817261244, 0.8724401811503882, 1.0369833811752072, 1.019195163594026, 0.19481827723599165, 0.8501280905966466, 0.29054777812528576, 0.5828357280459764, 0.2957410729722474, 0.3931716992914355, 0.6118153246790434, 0.7582102457910798, 0.8444919533202786, 0.20391413970902655, 0.2937326688912813, 0.07732444729644258, 0.08599622482847508, 0.4232127324445692, 0.5718935665155366, 0.7224055481787947, 1.0439514455937675, 0.6423027122629079, 0.7575555670425393, 0.1939483404482928, 0.6890408477100308, 0.6066477787291329, 1.0136517700990635, 0.6716975684470681, 0.8583251269866179]], [[0.2765011029818118, 0.9085542877140121, 0.46638534643009477, 0.7002085766700823, 0.7646377052696004, 1.1952838386504965, 0.4789539107902762, 0.8902357270401594, 1.0087408367863606, 1.0275373956152443, 0.16726661722886876, 0.8251875656407237, 0.28342943878954163, 0.5882261695604754, 0.31268449741331705, 0.3827845946530336, 0.6256643121757159, 0.6769991687529598, 0.8862331731670515, 0.246041913259644, 0.35391793000099575, 0.09000349276896547, 0.08200818673308388, 0.5250174563777184, 0.508665476873492, 0.8084425230136576, 1.1833271075670382, 0.5969792890809229, 0.7470752366642084, 0.20852957426751342, 0.6836268472964255, 0.5629003402646702, 1.0299254218668386, 0.6105347005418685, 0.7484655327842775]], [[0.27448430793841416, 0.9743382863989689, 0.5010376888207982, 0.7224578323858797, 0.7172363530343175, 1.2020675431235002, 0.4546421194096539, 0.9265996089471156, 0.8938354067885277, 0.9278479776338441, 0.13116767425803283, 0.8124769898549495, 0.2050268477463958, 0.5475633069374823, 0.3567631874898786, 0.3650389840628364, 0.6200067805706417, 0.7108466277615962, 0.9780541789791661, 0.25633021103676357, 0.2716266203409532, 0.11166132340465157, 0.09047987485919341, 0.5862741546822242, 0.4392470083953632, 0.8631041747231643, 1.2800846217119932, 0.5259698738764822, 0.6734862755384192, 0.3170233395659086, 0.7210156006286425, 0.5546449231252772, 1.0681138680599935, 0.6376195953936729, 0.6349818631761588]], [[0.22319838971266268, 0.9373328285495793, 0.4798577477570206, 0.7394257648128186, 0.5390067838221497, 1.128093988936439, 0.4418067275575706, 0.9324159832200959, 0.6125776982039679, 0.7317250055042139, 0.10849627396136065, 0.8014961483190985, 0.2133302943556319, 0.5451019935028747, 0.34998025181156023, 0.3450502045270784, 0.5778691828433309, 0.7276572784281667, 0.934507561475818, 0.2502505489732995, 0.24985533314709754, 0.11482766675565441, 0.09695896713965012, 0.668401425631501, 0.43823688096009794, 0.9532286503940226, 1.3929679815411975, 0.550673837265858, 0.5802992307928422, 0.5574034848827929, 0.7900719354765753, 0.5612495866007882, 1.08335869247803, 0.6067979321856358, 0.5082330531191882]], [[0.184395801005638, 0.9185251452374998, 0.4535462510630037, 0.7658156276371116, 0.34672234734925245, 1.0654296549152114, 0.43054466419918913, 0.9065483223860875, 0.41847558389434436, 0.4531884295783454, 0.10371923557600232, 0.7872720745219579, 0.21068838649272376, 0.5135390295538838, 0.39289308642239396, 0.3453692477518332, 0.599665968841132, 0.7214843185374373, 0.8856201455255135, 0.23281182363426625, 0.248610587102987, 0.10325644316666122, 0.10214857780719089, 0.7652093501836436, 0.4218640362241894, 0.9774413360941305, 1.4424425218743264, 0.6488281301889588, 0.7852351717872024, 0.9326709730887461, 0.8902256821517378, 0.5653310452991882, 1.0895446355722558, 0.6619370957705957, 0.4867834820453556]], [[0.19053390572908413, 0.8683069692783382, 0.48758352455921994, 0.7716812251053596, 0.27735261536442724, 0.8427142073289544, 0.37248903878364303, 0.8801090095267873, 0.3557958088139364, 0.48065025451373333, 0.09436431247817176, 0.7707147274606525, 0.21172974829260185, 0.5043941218409852, 0.3429936541758889, 0.38404487536267395, 0.5860291334730721, 0.7047276548295365, 0.7327305957919301, 0.2289365889712345, 0.2569236041465288, 0.07920778151087597, 0.10285986366658173, 0.8062993604558738, 0.4010931476132638, 1.0090470299637377, 1.4108461890248079, 0.8278081631074069, 0.8598148687902866, 1.243431283820504, 0.9493761263539894, 0.6083600278989703, 1.0927557481984818, 0.9654826367942246, 0.5426386068703676]], [[0.2711123979763916, 0.8146582728377504, 0.46489417220540386, 0.7689126424685062, 0.41623431169930203, 0.7187843825755962, 0.3484303411682613, 0.9522225886901309, 0.5258599549510043, 0.5952032683771078, 0.10205521474436416, 0.7504345122972149, 0.2555377509440981, 0.4989897293087895, 0.335475040529049, 0.3641661956485555, 0.6032885095530738, 0.7254545118798139, 0.752996811647145, 0.2278693641401388, 0.25349256351543853, 0.06415206107951495, 0.10924554333416811, 0.7965581336910866, 0.39735597354433055, 1.0460028587071848, 1.2937586314721017, 0.9606560024791524, 0.9191526737981855, 1.4309093684427077, 0.9990118801981295, 0.6504266315069507, 1.12078418718625, 0.9674895297636594, 0.5293673590403104]], [[0.3459213973733916, 0.8257969297132183, 0.45835088497723975, 0.7681190936349745, 0.6686052658157748, 0.7159665027287944, 0.29881738567738003, 1.0018253056668307, 0.6006990012120788, 0.6626719598985041, 0.10352727900542204, 0.7704966857746198, 0.29698907284917786, 0.4955161659641789, 0.3383209544746336, 0.32220282182701454, 0.6092499978742134, 0.7435787046060257, 0.7382637704625099, 0.2146609564259356, 0.2577492211738849, 0.07995661516069996, 0.11602778114902412, 0.7584944791951607, 0.38043677249339347, 1.0318424115104696, 1.1156598262024229, 1.0500858283394299, 0.8938405810206035, 1.4728944324326716, 0.996450425249629, 0.699873973864988, 1.136856556354605, 1.1084751077413304, 0.5509958117179847]], [[0.3847315409068071, 0.9176867077024506, 0.45015248159473287, 0.795504730456907, 0.5467412596897547, 0.6967589706394156, 0.316716767346534, 1.1188549180477996, 0.613851276390905, 0.6489039076060368, 0.10422938991367753, 0.7363159551934715, 1.0437690381314448, 0.5774151983336279, 0.2959355419532056, 0.35006760064144504, 0.6191731391285659, 0.740570899069206, 0.7160943782705691, 0.1603297512688381, 0.24601010750015873, 0.11058518602729381, 0.1021760560719998, 0.7103398914682729, 0.34519598780559735, 1.025375392889631, 0.9520011450130857, 1.165073571113449, 0.907458205314708, 1.4541497772814842, 0.9527305125114109, 0.7491735073571983, 1.1073384494807772, 1.1481589226671114, 0.5016790558109018]], [[0.415101154893561, 0.9784603878527451, 0.465286297179743, 0.8110212311098557, 0.5508380045457346, 0.7465292650856179, 0.3803471432349735, 0.8290930011712647, 0.6110378421067828, 0.6482372555040166, 0.0966149517834263, 0.7734799458047817, 1.1209187452097522, 0.5721072512613008, 0.38125873679184463, 0.3391768126740917, 0.4992172394580553, 0.7471231097678066, 0.6975817410335028, 0.1459740803608037, 0.2574120921295817, 0.10126840407628032, 0.14465863232756787, 0.6956451222269595, 0.2745998741824728, 1.0712459561252718, 0.7855575794136953, 1.1185512589377153, 0.8470567906093072, 1.5083219209066345, 0.8991493276125768, 0.7631295918670944, 1.0897485322833589, 1.1758958666890127, 0.42013068717880153]], [[0.44834768272295356, 0.97207039833241, 0.3906885341925048, 0.7935269163200149, 0.7066373986191848, 0.798508690188767, 0.37382495257242154, 0.839706191442591, 0.5616393203011667, 0.6734470090854593, 0.09436929699135763, 0.7022548063678935, 1.2160759056489456, 0.5682221481696861, 0.35559788313279816, 0.3436823476363211, 0.5740117793072153, 0.7655929314763166, 0.7451673893779314, 0.14929307362830432, 0.27919436591598995, 0.0831064392074182, 0.12444907680498322, 0.7636337877919233, 0.3249034157894297, 1.0653466525989586, 0.6555819290764106, 1.1147942437509786, 0.8539146896132535, 1.5009464745798051, 0.7755551095977794, 0.6369409003765765, 0.9943646757703506, 1.2004524517891333, 0.3647226590336549]], [[0.5018046088337034, 0.9789638865773246, 0.399155036606731, 0.8933297693327504, 1.1832066404961217, 0.6956015055525977, 0.3101002673398679, 0.8242290448676646, 0.5314188286931725, 0.6328982385258632, 0.09212554284563065, 0.670068133210024, 1.3021977209120221, 0.5666763971676244, 0.36351696587043597, 0.35594055052677587, 0.6382372610856328, 0.7574353000318026, 0.7901646008698283, 0.15558541952209265, 0.343382041419795, 0.11241030659882578, 0.1013126454180802, 0.7541863132245061, 0.481209319080445, 1.0110924759288265, 0.5581640796276451, 1.177107352271073, 0.8234977856497092, 1.5058808293484454, 0.679849078175641, 0.6029853236759004, 0.9698966935562177, 1.2958646652327643, 0.4261868140934826]], [[0.5418936053061468, 1.0042216673282363, 0.37862570124668715, 0.9813536151667381, 1.2097690651691564, 0.7719613125321227, 0.32995565997065657, 0.8296697855992186, 0.4739040416617428, 0.5186731750444828, 0.07577575809608185, 0.7080913130671218, 1.2795968105504303, 0.5815297471141381, 0.35974362835594853, 0.37899121834653027, 0.6743939337550005, 0.73656813828799, 0.7896669920454261, 0.15579373864103352, 0.2936935128160808, 0.13145293837634114, 0.08619684076210121, 0.7795329429499247, 0.562003782443177, 0.8759750445452097, 0.4795040320298355, 1.2366663218807465, 0.7662431985183378, 1.561995329168226, 0.6047370658053292, 0.535296489880805, 0.948209105627551, 1.3149754309081358, 0.4847294341480456]], [[0.5795163755334067, 1.0772939752902835, 0.485395437716197, 1.0084146950926784, 1.228521292946064, 0.7192519279117872, 0.2862675603640038, 0.8621971798325847, 0.5281924762901408, 0.46402577916679977, 0.05474156679193256, 0.7392541036703176, 1.2670166271111758, 0.6012498738967542, 0.42648055925354295, 0.43986070038603714, 0.72778281271938, 0.7256328338304739, 0.828728057473481, 0.1727157798139961, 0.2867789288931417, 0.13540764719022252, 0.09169422361086627, 0.7850060013363152, 0.5194753610769498, 0.7828564756658619, 0.43089176078237407, 1.2529546884708083, 0.701520484348213, 1.5271519069529274, 0.5146362697376464, 0.46067661714003344, 0.9029558684263503, 1.313184760442281, 0.5400692349647119]], [[0.6383267985618439, 1.094742866611604, 0.6227771895780536, 0.9638716844044781, 1.2453930287426813, 0.6635187053186633, 0.4121737773964028, 0.8386987739650467, 0.7032609535736742, 0.3925503184413809, 0.05678648354849739, 0.7580220601196703, 1.3014063415240065, 0.585024525563598, 0.397686746161923, 0.40922864390895575, 0.7109567998387589, 0.7242583844564969, 0.7826931185655885, 0.23644993253032087, 0.2692564159866357, 0.1949468922591132, 0.10557601673305157, 0.7746244703343877, 0.5834363373438354, 0.6537188039905715, 0.4189234291898243, 1.2132404854110526, 0.6199553574363752, 1.5513507359164103, 0.40489179510711193, 0.4361177621822107, 0.8137487518512507, 1.3467778323113375, 0.6041957708578567]], [[0.7403118271688917, 1.0934268472683697, 0.7135066329767523, 0.8834302023583867, 1.23486864008638, 0.6360377561218773, 0.39923698366287325, 0.8625982344467655, 0.8194476079099531, 0.4074975014102474, 0.0547351042826562, 0.7199460059794341, 1.272007789974949, 0.6095549041618282, 0.36696409373255473, 0.43670428426062213, 0.676110117814583, 0.698149583483955, 0.7315274735700289, 0.1918803128840556, 0.23238096185369603, 0.2279297301325918, 0.10840292583165946, 0.7530265804675729, 0.610574372832577, 0.5835208549994048, 0.43167649144019504, 1.1950256009487963, 0.6529573187134399, 1.5379631338848114, 0.2725086069448608, 0.3808612858031701, 0.6300934183215702, 1.4151916086132372, 0.6136132521093378]], [[0.7665253198145903, 1.2310246743889335, 0.8173597172519649, 0.8538101347685878, 1.0041022834124567, 0.5849102014553134, 0.49769580336795116, 0.6954465263295038, 0.8119383959429856, 0.6394778986325512, 0.05082534710355274, 0.7015344029089212, 1.2935674160321788, 0.6369973431147307, 0.3774955755596851, 0.4445184063966161, 0.7022670154137157, 0.6628131697344276, 0.7272782932718331, 0.22337239693493405, 0.21980469517617907, 0.3306279329111754, 0.12005886467273222, 0.7207708652312397, 0.6196846081041508, 0.3548345609252054, 0.5067999373793761, 1.050254540141923, 0.48389524981405946, 1.6139589011466353, 0.21976815811682104, 0.3143903495462297, 0.5243135901686975, 1.4611891431155848, 0.5553142098407546]], [[0.8033358347769082, 1.217481965259108, 0.938543160525887, 0.8420536163932961, 0.924134329395258, 0.6144977159150866, 0.5889324959222785, 0.847180221937208, 0.8203674379604192, 0.7297085135747254, 0.07623552185183868, 0.7337061575544337, 1.3130465441160042, 0.6393905860110343, 0.3369023142933384, 0.4539616851533216, 0.7146389352059086, 0.729293321072683, 0.73014050804048, 0.24229769464776352, 0.2178297486071743, 0.42447426137122657, 0.10687833622071669, 0.7230040842813962, 0.6472452652901682, 0.30920831783009106, 0.49994355177671224, 1.0528876586527665, 0.4640174529361612, 1.650961500577679, 0.23505019270151176, 0.26773092061433346, 0.562075453095514, 1.5490982358823775, 0.5212055746240587]], [[0.8083789820682394, 1.1742808161915925, 0.8909335073221867, 0.9927081427729522, 0.8968925302494735, 0.5992642913096227, 0.5554879844023801, 1.03399329737917, 0.7926555156699557, 0.8259997287375539, 0.10651445055174195, 0.7157186262139692, 1.3848592615987632, 0.667111806196331, 0.3409334319141403, 0.4663014481814574, 0.7053658840477017, 0.7276030372472769, 0.7198080323490482, 0.2442681527462789, 0.20380020075885774, 0.4997288192310776, 0.0925584919911841, 0.5045426968355468, 0.6837764342132553, 0.26906002520681693, 0.42505713694745545, 1.0194879135399662, 0.42993971337231485, 1.4256795915553993, 0.1671017858675166, 0.22477502942076996, 0.6344599152070871, 1.5249847691543872, 0.48478311035568444]], [[0.8086736584014868, 1.1947167527063387, 0.8599035752981483, 1.0554529198641651, 0.9712917322116698, 0.5807007638003424, 0.5372204259102638, 1.2750857069810717, 0.7256559176645984, 1.0022538385449327, 0.13515733252223938, 0.7400465698763905, 1.4733295236805735, 0.6546327081083969, 0.34724236526030083, 0.43266811143493633, 0.6972230573805733, 0.7381135499222387, 0.762248534340713, 0.24124562157934168, 0.16833748587806463, 0.5141005199695186, 0.0969637836663925, 0.37023084172947374, 0.6278332782330276, 0.32764745384966076, 0.3158276900894934, 0.9427516711694216, 0.4191693597690357, 1.1582316228390737, 0.15220411285809765, 0.2015777427937755, 0.6462988047094594, 1.5246622671713455, 0.5011359518028251]], [[0.725567426184925, 1.1241123456559003, 0.8139715398491597, 1.0537190096415825, 1.0006973207156382, 0.6281992120262998, 0.6000598003436439, 1.4464642957568445, 0.7404123098778027, 1.0774839758975303, 0.1285873227728719, 0.6912003525549046, 1.501990516898462, 0.7146070364895437, 0.3456034642612674, 0.460293667404742, 0.6845669480254759, 0.6894771489554745, 0.8553734135364038, 0.20616729658924227, 0.18406066279950828, 0.43845227671384446, 0.12833541289660522, 0.39892921472709064, 0.6500828498040392, 0.40315899170870206, 0.267218992777732, 0.8255876532298984, 0.4043665504130364, 1.1157368306278408, 0.24389751254836622, 0.23945222922299633, 0.602302521222445, 1.5961982966024304, 0.5354531260120134]], [[0.7203288691705625, 1.0470883227539576, 0.7321958522773412, 0.9976848818203189, 1.0055238540245204, 0.6266698140218181, 0.5720639104319951, 1.5947303358931024, 0.6784210708461181, 1.0233271846515417, 0.13012001390987754, 0.6389467452958136, 1.5012507160611621, 0.7011625088815729, 0.3384583258753696, 0.46404078192182624, 0.6897757351433529, 0.6801864860292797, 0.8872686163815163, 0.15947782673482186, 0.21392836656683284, 0.5339099763742621, 0.10175026820292016, 0.3959002478460032, 0.717654509083125, 0.36477877140424947, 0.23060432704056788, 0.7400700732366854, 0.37944561889630407, 1.0584464426602318, 0.4002958068703224, 0.3069365621357054, 0.549065573493364, 1.5360078302335571, 0.5688115070854377]], [[0.6743829651859139, 0.9092762217256098, 0.6826385037029361, 1.0853835849410534, 1.0229443170897454, 0.6476565895728966, 0.5279636798954492, 1.646487969603413, 0.624080192222326, 0.9506407058985259, 0.1181575847212272, 0.6217048271396691, 1.4881160772089337, 0.6916230881193779, 0.42554549819266796, 0.4823790146557704, 0.6810213287046193, 0.7153585040673479, 0.8489923667531167, 0.15575286752016698, 0.21551476134148734, 0.6053154220216984, 0.04911132936317502, 0.42733430690488117, 0.7576853698667223, 0.2802152554718687, 0.2373684933511795, 0.659532843525415, 0.3527084530365024, 0.936357921904589, 0.5216849202438087, 0.4721663944109545, 0.46960455660072187, 1.3805942433256821, 0.6008872880459292]], [[0.6818472978514883, 0.8715784715155264, 0.6425186724331244, 1.1395125559951373, 1.0337539702440863, 0.6695908945575731, 0.5385543262183456, 1.6810180514260562, 0.6468478737051907, 0.9008620692799254, 0.11590274117220659, 0.59696384043779, 1.502951283295134, 0.6903850847593742, 0.39194889286010137, 0.4520066434331937, 0.6988834329843077, 0.7323084307493637, 0.9101069736853812, 0.1726229359949303, 0.25439533429757216, 0.6659324999892993, 0.04500302889702838, 0.511045240417479, 0.7466069390320172, 0.27825834890042733, 0.29153810345772346, 0.6305736326589599, 0.35256621073282723, 0.8672846439739135, 0.6145334413616022, 0.5799067910379374, 0.43993953279974185, 1.3034682412575425, 0.5986784144159453]], [[0.7034344617854709, 0.8614648141670177, 0.6092539977409388, 1.1158117123018774, 0.9667811931753996, 0.6815118569139897, 0.532716167234682, 1.6957192480384646, 0.6228181733416264, 0.7418453343033109, 0.1129113764684817, 0.5839448773379531, 1.5356594950007123, 0.6911884722856403, 0.4188551878609257, 0.4808579871080749, 0.7523628825305759, 0.7649761203974401, 0.9280626362151023, 0.22931448087283268, 0.28507468331679314, 0.8683360924720404, 0.043730585572086436, 0.558052202649437, 0.7111097358475968, 0.34589119180002736, 0.3568888257115582, 0.6004781090439559, 0.40161111347984696, 0.8779918719649897, 0.6657089213736025, 0.6592499740420369, 0.44796475467089597, 1.2130584547174479, 0.616616919324762]], [[0.724673181535832, 0.8449702188989336, 0.5871359676804018, 1.151269390264824, 0.8429924792851149, 0.6965053515935731, 0.5310604775405672, 1.6541970168694804, 0.6443907005787368, 0.7594124718996613, 0.11231033467193631, 0.5613512076980299, 1.561328724122781, 0.7091569306721831, 0.4163280028898686, 0.5199732405500745, 0.7826628280200374, 0.8030408145561293, 1.1968962732393424, 0.23902532683854727, 0.29648535050524893, 1.032947633540946, 0.12808247302563303, 0.6201008127634704, 0.7637584097427366, 0.6022835960725513, 0.4374939700822156, 0.5705506778829141, 0.4604078368114734, 0.885813016211729, 0.815244391714229, 0.7810904722699088, 0.46003458451573437, 1.087549432734165, 0.6762392925171128]], [[0.7607558131738329, 0.8369468139741227, 0.5694596396156854, 1.2083408751272662, 0.8283466788979255, 0.6944008722701936, 0.5060487782763218, 1.5987906885158252, 0.6411121160759865, 0.7610207740875855, 0.10934325183539696, 0.5544973518930505, 1.5907227257875831, 0.6889740676298669, 0.39303001574078744, 0.5411129277764424, 0.7606370632013287, 0.7996159030597715, 1.287541524544046, 0.27206923558316387, 0.37151399036479343, 0.9790151440981013, 0.18520961717325102, 0.6071891106034115, 0.911092915031685, 0.8474587517594, 0.44719384301003623, 0.5358147807029745, 0.5126808772527662, 0.917222711868285, 0.8944874474084072, 0.8366846185630148, 0.46436611330532396, 1.0289844759156768, 0.7915165379188032]], [[0.7725128050666913, 0.8416473951908212, 0.5579040095736261, 1.2624258926911285, 0.8454372991643195, 0.6750485613782988, 0.5129812554502341, 1.5684096385127526, 0.6541511972079261, 0.5768173061027557, 0.12613858059095184, 0.5516698522078731, 1.6359825343606031, 0.7142744721509193, 0.4064012744066396, 0.42026781013027736, 0.6992054107619246, 0.8541150426337989, 1.4675937479183585, 0.3139731456151997, 0.43293116858181396, 0.8563743036670766, 0.14773491978538755, 0.5938492149449344, 1.0473526797672066, 1.0890248930039257, 0.4154001929057467, 0.3294360453013474, 0.5009899933072085, 0.8939784051264467, 0.9349866736055807, 0.8604371171163738, 0.456895391554505, 0.8719369763494529, 0.8461892169397358]], [[0.8527879957681704, 0.8101547212998121, 0.5526146523689205, 1.3495275934230677, 0.8888868183129621, 0.7110090965828834, 0.5168954736619417, 1.5907877138410822, 0.6786793427569018, 0.4958360847889271, 0.14356764217559193, 0.5507711783326406, 1.674717863058014, 0.7854275965719631, 0.42060801313639834, 0.43778245662261417, 0.7330565175006487, 0.8021533954267197, 1.486295983135462, 0.3312992708955213, 0.45702650979507353, 0.7412901532279826, 0.15889290880318202, 0.6162676038605313, 1.0285027166564045, 1.1376358694790145, 0.3011882374219345, 0.22511721768189868, 0.44773933828576784, 0.8277816612739188, 0.9290247441777615, 0.8628896369910324, 0.442685772644682, 0.8108706552797942, 0.7764061626518342]], [[0.7676765204358346, 0.7168199090302665, 0.40366701191037285, 1.4639020283484114, 0.9163376448766702, 0.6923879099859016, 0.44971711364852374, 1.5887776387631634, 0.6440231641705133, 0.5417221415262389, 0.13739237435265295, 0.5425102931416997, 1.6940002046485167, 0.7523589177031591, 0.47114008025051307, 0.39559811911488596, 0.71573081659889, 0.7588505346023653, 1.433943507911548, 0.32695230799532154, 0.42794360537107606, 0.6304154128144612, 0.1323283446859727, 0.6351887617047476, 0.9451007235112516, 0.9639055085950431, 0.16666112368829616, 0.1625786871240298, 0.36848879725489053, 0.640871759350606, 0.9030643684396729, 0.854325666844097, 0.422698129959177, 0.7350950517921742, 0.651531979671726]], [[0.7304322858623269, 0.7309089235939692, 0.40707670129063456, 1.398529296738585, 0.9696382584892175, 0.6822499902881398, 0.44294469736335973, 1.5419578405628165, 0.5131751324966363, 0.6262996174146114, 0.10341769067806401, 0.5477613344765593, 1.64860368111668, 0.719259663783172, 0.45676303568602933, 0.4169615676605644, 0.7168693266588231, 0.7713618066916521, 1.3929493156857458, 0.32260265925642384, 0.3852248812204236, 0.6576113480739846, 0.11453220345789773, 0.6486291014381104, 0.8293482031955979, 0.8501032760751687, 0.085447465965101, 0.1608681764028897, 0.3863551857009945, 0.5855399745120922, 0.8909118303617518, 0.8602827119434074, 0.4064004107783567, 0.6613545436318027, 0.567350140913177]], [[0.7131601431353553, 0.7218768656172292, 0.3921058856098638, 1.5004621854191027, 1.0438964424347994, 0.7100091565759035, 0.42475527188950996, 1.5238794536217994, 0.617694895422205, 0.7181608897879508, 0.13660715024468817, 0.5847300080229869, 1.6017848163750004, 0.7413038671945755, 0.39392995436693057, 0.40660686845804356, 0.7676831907266956, 0.7614676009704582, 1.325240437815981, 0.36213084279454355, 0.414772612702125, 0.7655161053226067, 0.10907662601556832, 0.6795122181615891, 0.6665838943367115, 0.905371841171111, 0.10645939688753026, 0.18573440244254763, 0.4306299035504815, 0.5833755358267723, 0.8977249840414212, 0.8865898418103083, 0.3895499722950627, 0.5740182971373597, 0.44233540402431876]], [[0.7477230804980916, 0.6946102692970203, 0.3592081039064492, 1.5031785136224216, 1.0873362426142565, 0.7565385349145421, 0.43285618660094444, 1.5240860934626548, 0.6966404700004121, 0.7556618055744085, 0.09369246623506657, 0.5819409258386347, 1.528086017029834, 0.8314922385375085, 0.3667477979351181, 0.40204484225520626, 0.8898343150401611, 0.7476995706721863, 1.3412999509252255, 0.41107074474269933, 0.44725762895247895, 0.8222289915041495, 0.11200348744350151, 0.6955903205669642, 0.5338053823176373, 0.9836200450788523, 0.18584345824209958, 0.2434859357363201, 0.4408064118455618, 0.5997844068867954, 0.9218482766990304, 0.9021055258241876, 0.36931698965543075, 0.443067486592113, 0.3089598712949128]], [[0.7417757370301034, 0.7223220250344657, 0.3267306627058097, 1.425234416902544, 1.1421111703955447, 0.7490717010952983, 0.46480374045997414, 1.4608187345470063, 0.6846296227050377, 0.7657281586896623, 0.10334729352008523, 0.5894570231245269, 1.3818841837976865, 0.8872921225249171, 0.36691364717413644, 0.40030099065166924, 0.7872103150537668, 0.747052133477255, 1.3464950190275697, 0.4502921427908858, 0.45682568102146687, 0.8579268993169257, 0.10697973021982807, 0.7141778209301493, 0.49510052550411043, 1.0482367470618876, 0.3330798756351201, 0.31400481665993907, 0.4567925204200271, 0.6017542588975889, 0.9440102027147832, 0.9212687056801614, 0.35573601317184966, 0.4045968114585441, 0.13336318357329296]], [[0.7344514171335452, 0.7296507850292486, 0.3046438565453864, 1.3255529915954272, 1.0968197676864495, 0.7477254891016076, 0.49382229701623326, 1.3761330205395477, 0.7066955589284876, 0.7286301793347315, 0.10182843750742454, 0.583897862787719, 1.267714254797743, 0.9145606153116598, 0.34041881703805665, 0.4023511440504305, 0.7827935588871089, 0.7503716622852757, 1.4032532481571025, 0.43210151422744514, 0.4192890045806984, 0.7685425664944725, 0.10846377215395624, 0.7274063197362056, 0.4535160733529151, 1.0534084784172673, 0.4021798609642029, 0.30568969326506207, 0.45486452503933317, 0.6261614800068677, 0.9896799344363642, 0.9517392710726889, 0.353332334773793, 0.3488500494786321, 0.15290081966579902]], [[0.7085804810989953, 0.790559051786925, 0.31175447101596054, 1.4146614014743544, 1.090603727648166, 0.7130341151012558, 0.4911768676949792, 1.3029973174174065, 0.7010395586205731, 0.7127226860878879, 0.06665152852967826, 0.5761593181129852, 1.1919856370729138, 0.965724027997944, 0.34150968425182826, 0.37175734869321775, 0.6793022028967008, 0.6906280361452592, 1.253989705939824, 0.4105734491035403, 0.3976903859786617, 0.7153582943349065, 0.09618242074112578, 0.7098907082578292, 0.4172723436569286, 1.0241856212257558, 0.40976549838474163, 0.32798149405239974, 0.4191843604565687, 0.6444553453896977, 0.9959766635446056, 0.9649648132856267, 0.3469558073314852, 0.34070968324742734, 0.28707181767717893]], [[0.7079898420028891, 0.7243400768409436, 0.3130110700142148, 1.3804133165824433, 1.074439663309928, 0.7083132306700566, 0.43977952112161, 1.2673804612246522, 0.7160601733773955, 0.6902906732331219, 0.05764772594813439, 0.5649213464780805, 1.1844146851386732, 1.021386693370326, 0.38766533940663733, 0.3583575870243155, 0.5674838945796765, 0.7176839335564655, 1.1512864186230956, 0.41344274523834756, 0.4048398907403557, 0.685763277896133, 0.09586863062318018, 0.6930263861770123, 0.42948228770826813, 1.0024031520763734, 0.41303207476543236, 0.3723804921023266, 0.3965143797304226, 0.6496949468251947, 0.9986871771752698, 0.9624023839926227, 0.34843483177649254, 0.4293940043464687, 0.4410969168785165]], [[0.69540231866467, 0.6578366941836029, 0.31387327501721896, 1.3559756126484854, 1.114973970466513, 0.7471491721571771, 0.4354940727080685, 1.300590203969621, 0.7173748869040306, 0.6780848412044036, 0.058569190569471186, 0.5673186346551367, 1.2396049255677721, 1.0243757461354033, 0.46763285957831907, 0.3822894507526525, 0.5321327779053548, 0.6662438007613695, 0.918786472025872, 0.22104286095396797, 0.24809783584910733, 0.4152244156912519, 0.10935810855110281, 0.6793355936259098, 0.41407332681899545, 0.9716869307639077, 0.3977030420006832, 0.36789683486441627, 0.37579108491567925, 0.5500897990895823, 1.0069610064822643, 0.9222067802408696, 0.44335752643368637, 0.5009225491749092, 0.5949569482544665]], [[0.6992136146541, 0.6369423946970807, 0.2969819474049852, 1.2431097278266305, 1.137847042184491, 0.7310533442069862, 0.4369680368196842, 1.157223112918214, 0.6821039375660942, 0.6380344806938107, 0.05699064333331476, 0.5653305814886483, 1.2224696110552458, 1.0451379171063622, 0.42657335734698304, 0.36512734248289963, 0.44610929957086254, 0.6719421647690463, 0.9466955969081354, 0.18825514281170388, 0.244037939496491, 0.23437859070287403, 0.1427486738085308, 0.6714089934132711, 0.3787911456864541, 0.8780345821722737, 0.41140775241686833, 0.3834466480815639, 0.39224939614261, 0.4877944156312723, 0.999622566401752, 0.8516650063971742, 0.4856838870815397, 0.5672007820488779, 0.6944829933479612]], [[0.7214988789775418, 0.5975528787424607, 0.34140525680876666, 1.0612318433545862, 1.1349387291495063, 0.6991555551759363, 0.43731481072745887, 0.9987797711584626, 0.5263651071510365, 0.5728501363750007, 0.05412158853318012, 0.5840404774978576, 1.268391933266458, 1.104295513591367, 0.4660631329463683, 0.3794993328749624, 0.5760317150484772, 0.636653604178515, 0.9090651265615198, 0.18689201846111284, 0.23721266626508808, 0.20573303522489633, 0.13823275404910745, 0.6468527237218437, 0.39086362452425416, 0.7885204941321605, 0.3890653474529643, 0.39856233791396317, 0.41015760786032235, 0.44099086528473647, 0.9442416182665965, 0.8199572672962275, 0.4560654234067868, 0.6467102935904112, 0.7440309239404149]], [[0.7248984429631623, 0.5341656367856557, 0.34231501291162036, 0.9410909843857919, 1.262484406057621, 0.673353234484408, 0.3140820669791654, 0.8343186357420987, 0.4394991792193577, 0.45593253082749796, 0.07203434225303777, 0.5867370152154952, 1.3649497263230383, 1.081318813483081, 0.3830234016312225, 0.32360157895372066, 0.4581192425663557, 0.6258776093080772, 0.7892933249793999, 0.21658026529268776, 0.25759442481895745, 0.2108069472639611, 0.14324849520249777, 0.605132977374298, 0.4024409133438762, 0.7800003079037285, 0.33342155227942943, 0.39227797585490354, 0.42701992644247994, 0.40829875252460873, 0.8560549185512475, 0.8143833900078008, 0.44555184608863097, 0.5770376302362268, 0.7648449099549463]], [[0.7398107862459712, 0.4623218212293495, 0.41191299283752303, 0.7878086179465366, 1.364493997772775, 0.7360557870969149, 0.42232661795255444, 0.7131117666585831, 0.3882493552449978, 0.3927439886671192, 0.0638606258161066, 0.5692287046973298, 1.4988051422020552, 1.111493709621425, 0.38365012105988733, 0.30879659633246015, 0.48202086700404084, 0.6316914087069769, 0.7402441394918429, 0.2341462340995954, 0.26468662595164655, 0.21573674495541817, 0.14757867461083574, 0.5364285411947612, 0.41282988116645586, 0.7923102326531127, 0.3062461771637298, 0.42127390927609853, 0.44708868909637245, 0.3722369064218281, 0.7845984500452947, 0.7594607650521482, 0.48371451567979523, 0.6321004003638909, 0.7607494400876517]], [[0.7873807602344268, 0.4471893286097225, 0.5128403323261401, 0.6899678757813407, 1.4016355720734208, 0.7422301339775725, 0.45469259924722194, 0.701399352671159, 0.4600703986950133, 0.37407480109034397, 0.07053007789392834, 0.5560299521251517, 1.593601706304697, 0.9271300928462869, 0.4181769105906756, 0.32084529519611765, 0.4560791573988262, 0.6389655714711938, 0.7448949331907696, 0.1934873905049696, 0.27321953775797314, 0.2166809608438465, 0.15174248706062984, 0.4875935611228014, 0.42050132367585724, 0.7652274528867526, 0.28237238177851576, 0.44705301481251225, 0.43348736982702296, 0.3703723079175426, 0.6727825407701157, 0.7516157904916159, 0.5179860991657157, 0.676040199527595, 0.7418066030931852]], [[0.8161235160099539, 0.5822413003110362, 0.44853359414351734, 0.7293245644701134, 1.479198604865983, 0.6524432616564576, 0.45671939828131747, 0.7041656371874259, 0.39681702692070775, 0.30900816430715994, 0.07643873624033896, 0.662728137719075, 1.5849970583123572, 0.838412963144082, 0.407266194192861, 0.3292608416906813, 0.5318682464073434, 0.6501978800005015, 0.7483663934066078, 0.1824055194860272, 0.2831603687942931, 0.22361445678223585, 0.16774775268811762, 0.38701081711065183, 0.4473252159402656, 0.7276303603843131, 0.2366562264150563, 0.4711755905005596, 0.42360407274308814, 0.3590298963095607, 0.5128791179320388, 0.7752811785025597, 0.5964653810967245, 0.6942343237253199, 0.71261379347531]], [[0.8185782172980101, 0.6262610386737673, 0.48901399684259284, 0.671606293875695, 1.5156481724645121, 0.5299109825991581, 0.41524060404707586, 0.6435915463810782, 0.3840857085934198, 0.2752411063028437, 0.06843718611648437, 0.6715422273450952, 1.5678568406374598, 0.7881410372774915, 0.3961685552328073, 0.3265147089474851, 0.5349662860104328, 0.6424713904715231, 0.7429876885489781, 0.18418569466614057, 0.2846240983210647, 0.20397576916532312, 0.189056153431741, 0.30193105808243775, 0.4501275327749421, 0.6827091788028568, 0.21972774646269716, 0.4680009269958531, 0.4074788927354659, 0.3689918179775965, 0.41334766103919385, 0.8477510213873469, 0.6639734102015802, 0.7285686852626774, 0.7179965401218399]], [[0.8654668648231385, 0.7208354996044173, 0.6389483012246091, 0.7669460293801815, 1.5215239763267017, 0.5575314372407945, 0.4209164693442279, 0.6296825898680358, 0.4058901632097384, 0.2287577002428875, 0.06871259571832335, 0.7010832100901527, 1.6218302587744668, 0.7762716521467413, 0.34794447640116866, 0.4042186445823081, 0.5804374841950386, 0.6434085350172464, 0.7387104985525205, 0.19901219128043893, 0.28482202674365814, 0.2036480444488208, 0.20566395907571602, 0.26366008884009023, 0.42149272173992214, 0.6995123686802908, 0.20733986511943542, 0.4883960420792559, 0.43072749615716244, 0.3881203990402119, 0.3954272792802156, 0.8729726058102651, 0.7176714684413079, 0.7803364114568774, 0.7417497343627175]], [[0.896895718196555, 0.757914197471631, 0.6384399226791471, 0.7924935969003346, 1.5302452411162069, 0.544120605666034, 0.3975610497319779, 0.6185589551958719, 0.43562209783384837, 0.23331209375751694, 0.07784939133886623, 0.6554588243415322, 1.6152540504095083, 0.744359043495609, 0.347075232244474, 0.38077563636151135, 0.5473603697511435, 0.5877562049262282, 0.7507247767442045, 0.19568000763111992, 0.28431694453114087, 0.19036510928995606, 0.20098002073125087, 0.23520705289970406, 0.34341461059159556, 0.7190669369859446, 0.18961809007248065, 0.5679760816896525, 0.54544199996917, 0.3622555983873058, 0.37954236594507657, 0.8195649011249727, 0.6629140764516926, 0.7921129731542595, 0.7027209167132549]], [[0.9203260869888996, 0.8144619246028881, 0.6828755569792012, 0.959210066532751, 1.5628018814443763, 0.531925244962269, 0.40253964391428076, 0.6645814292256518, 0.489688989027541, 0.19670773109654852, 0.1006829629006123, 0.6256947320651383, 1.6127067815677105, 0.5812622403179927, 0.33993017219249033, 0.36973300251240965, 0.4962876586296814, 0.5631417418636825, 0.7519894370876097, 0.17432152826407404, 0.28485451683302065, 0.1918697392493999, 0.20246811427958483, 0.28337133638598055, 0.2919635539000446, 0.7144143799326013, 0.19152118311596672, 0.7340931309016713, 0.8613728853030438, 0.3889987523167603, 0.4615814040373152, 0.7853566261936664, 0.5942446460709199, 0.7925321638751508, 0.693808820403967]], [[1.041940374200777, 0.8372131846276653, 0.7813798486366592, 1.3508688690297974, 1.6067667886632062, 0.5281256154965785, 0.43012743741197845, 0.6707051103667925, 0.7449807027200612, 0.16640451831056996, 0.1129899986869342, 0.6341173197394682, 1.564052567616587, 0.636592413580318, 0.3390160379454189, 0.4026743232123423, 0.4751596178071792, 0.5611657602333142, 0.7782389975180344, 0.1490664542896975, 0.28167788676912187, 0.19407992908971328, 0.17607329141720976, 0.3820753999537426, 0.28712081285875407, 0.6600158073182203, 0.1695577395744671, 0.8657854394599827, 1.0955260309253534, 0.4212706338187766, 0.4858989067198487, 0.7556820776674817, 0.5787296253413015, 0.7870060674100896, 0.7191364683382502]], [[1.1525566667718712, 0.8102458493225483, 0.6990971192053411, 1.5148839319424774, 1.6401052139197294, 0.5383190722275519, 0.41891177408242847, 0.6597951795779382, 0.7895903478370316, 0.14979694279447514, 0.197295228084903, 0.6359528641358073, 1.4874233443219136, 0.6600860730359601, 0.35332424224586784, 0.4271708566532126, 0.4281981217302898, 0.5412369304694784, 0.8143718694401196, 0.14301375422589113, 0.2640868496278598, 0.21835493485773835, 0.1763848547521343, 0.5111229101528812, 0.34330280176487404, 0.6841685494698897, 0.2657996585320921, 0.8661727962265411, 1.0205788079133202, 0.43439350228802265, 0.66513200217944, 0.8143837193915355, 0.7259016083457954, 0.8226763984543988, 0.6900409205130424]], [[1.093640172390843, 0.7788141054586314, 0.5946359007896103, 1.5608867477677615, 1.6614374455695748, 0.549598502369835, 0.4196095416729708, 0.6264303398610717, 0.6555750012272695, 0.13840386717162212, 0.1460612992150187, 0.6499657021349281, 1.4330793890911353, 0.6337884925123396, 0.35640120604138203, 0.3891697171993735, 0.39982294892718717, 0.5146452321732938, 0.822046254487904, 0.15171466146083687, 0.24353288077187285, 0.23275913218125907, 0.1994449548689671, 0.6048744703953595, 0.386205320795528, 0.5521444911802861, 0.4212493600147379, 0.839221110952715, 0.7914682960254391, 0.5002068795486411, 0.8863885039022986, 1.0337589076381888, 0.9202529160597877, 0.8606114960622506, 0.6623190091401563]], [[1.116821827491675, 0.7949957839630581, 0.593388308284768, 1.692964942125353, 1.6615726400750201, 0.5363783091244221, 0.4415829517309683, 0.5827666552999745, 0.6116215471121018, 0.12708636696933998, 0.1521503434458551, 0.6679721805561232, 1.4588682207461887, 0.6350851278569016, 0.2836503386740137, 0.36917378796119105, 0.38166189616068574, 0.4966931353753584, 0.8474559334514155, 0.15204517200984813, 0.24234840034279703, 0.23205088920342687, 0.230371917824988, 0.6329066675469758, 0.4078626650045821, 0.5396101329996915, 0.39842694342011464, 0.7608559704656637, 0.5932099197251629, 0.5925102245871197, 0.9886840707366716, 1.1197938000488472, 0.9906635204491878, 0.8746764739462822, 0.650592520537465]], [[1.0914183640451252, 0.7775115273607934, 0.5941761786153337, 1.632829268778253, 1.6760340518351637, 0.5070328591952045, 0.4488140528530582, 0.5839683950359079, 0.5721930102897612, 0.1222019041360054, 0.11073958043547566, 0.6866055866081471, 1.472625772039751, 0.6561932226837205, 0.3283175937866035, 0.38365030403809874, 0.3817844055980851, 0.47656426269971497, 0.863122986915719, 0.1271453287054254, 0.2362518506532374, 0.22800772673491387, 0.1924799815904231, 0.6263094850245239, 0.44623924405106447, 0.6123189080346116, 0.3861125023898755, 0.7316610929301703, 0.5282419324798896, 0.6538769826163147, 0.8526886201809545, 1.1082209632825315, 0.9713664784709912, 0.8688198684073952, 0.6461555403809908]], [[1.08428334634484, 0.7708180747566413, 0.6023200210265669, 1.5716748917511176, 1.6714286831737835, 0.5186628055488001, 0.4477271008566457, 0.6584006741175608, 0.5077661093196357, 0.11062036585884791, 0.09392642230914339, 0.5895581755449142, 1.440556370174349, 0.6300668577040952, 0.24878430701332888, 0.32150076916339054, 0.38828127412762115, 0.454641697379089, 0.9100602786772309, 0.11663468499744306, 0.22531638366293996, 0.19086215207457763, 0.2033492730684436, 0.5711060546005284, 0.5999674912254386, 0.8108068279351808, 0.37431069145445445, 0.6840470675557867, 0.5456994498486946, 0.8538035141073493, 0.7155501963998816, 1.0721922958451229, 0.9201909358686504, 0.9034118668284955, 0.6744929794308597]], [[1.037079642841493, 0.7690272918295091, 0.6174856357222749, 1.6329253211798322, 1.5968910211615952, 0.5173663458027676, 0.4511131525533904, 0.5823465625034504, 0.4882976381810391, 0.0847487127214287, 0.07266484493260746, 0.5753996455506394, 1.4025117280220476, 0.6506094225041408, 0.2472214072413667, 0.32758576727968114, 0.3979830717934534, 0.49452848967841584, 0.885675118724716, 0.07662561222271812, 0.2316807974981323, 0.1650514427197608, 0.205523223506773, 0.5166023546795289, 0.6101131072940856, 0.788316418825425, 0.4581667570986148, 0.6326957749466519, 0.632204998981647, 0.8767103633038643, 0.7289497924717216, 0.9910015210046529, 0.8752641157535715, 0.858227543675474, 0.6232003187743533]], [[1.034064951723893, 0.7482412824184994, 0.6155860864531242, 1.6401889958578815, 1.5283144717311976, 0.5431011048594011, 0.462427130994663, 0.601324398160696, 0.43555091254635747, 0.07952493244656125, 0.07354658318096356, 0.6227932518282951, 1.3510723186210287, 0.6311526474459697, 0.2514486670164533, 0.36970177823370315, 0.36080984578038017, 0.5212456300446546, 0.8633075549668354, 0.0574656494844266, 0.2514522052133653, 0.1697287937127172, 0.15980153099324204, 0.4797634042229014, 0.5712098719611571, 0.6476536759616454, 0.5626608506990044, 0.723809467886307, 0.5409785932570973, 0.9117261205825271, 0.8156713340384222, 0.9507262261396382, 0.8470914563650537, 0.7575713322284898, 0.5343240470250563]], [[0.9901648009363329, 0.6346961301653459, 0.5953240709972896, 1.6721389132507616, 1.4276243619965838, 0.5642761694714331, 0.4505190105153548, 0.6041138612823246, 0.4820558765430639, 0.18451888820181195, 0.06967610391307574, 0.5624732320277668, 1.3275802177933695, 0.557616211597804, 0.3335130078352616, 0.4846025179575911, 0.38344479315088265, 0.5669344852760576, 0.8670692219683049, 0.034323051490610046, 0.27045485755967086, 0.20371959361961267, 0.11641387007171612, 0.5262040120531878, 0.6297072201299557, 0.5130803711229195, 0.5230678874510605, 0.6963869002095882, 0.473416447853994, 0.923352628383027, 0.8628959609869862, 0.9269598624819181, 0.8428815054995711, 0.6795978881926977, 0.484059532343971]], [[0.9752371799916442, 0.6007020577883179, 0.5689425216410793, 1.606263872352914, 1.3225107516955688, 0.5689195633847431, 0.43397470457869414, 0.5994122060609475, 0.5672342000693062, 0.26926996478237586, 0.0659122640299436, 0.5617162415275183, 1.2865992673476294, 0.5834461260575372, 0.3231256988477885, 0.3781781078091764, 0.43104595290271774, 0.5117586823078291, 0.6952636825983218, 0.03345840411907916, 0.2528133687849, 0.2323177142746783, 0.140613660098035, 0.5508367522319473, 0.552711761712235, 0.47510177055156927, 0.4096670677048899, 0.6239746098494243, 0.3621281296658414, 0.8738111060380521, 0.8690609187405627, 0.8869598725578477, 0.7803225960758371, 0.6590056877256274, 0.4423799260118839]], [[0.971739824526605, 0.549247082513822, 0.545108607142375, 1.5541621444600917, 1.2499939569281722, 0.5823935895806038, 0.4757433839833919, 0.5999930412590688, 0.6125371860675193, 0.3163702868712932, 0.06513270126057757, 0.5317433777828448, 1.2128915009032673, 0.6357729986912336, 0.31877731958035516, 0.44114862883771855, 0.4514756972884447, 0.4733785461516305, 0.6514876452498611, 0.03483229778768546, 0.2471046630062046, 0.2103776759939096, 0.09848736673684332, 0.6458534077218286, 0.5493239305484167, 0.48128289128446017, 0.37221922729625695, 0.5682118044362678, 0.3144325716661297, 0.8483157505593659, 0.872391818680742, 0.8650396208099613, 0.735247276062308, 0.6588555616267802, 0.4132149226795777]], [[0.9406436230003759, 0.5279719166601103, 0.47159252605796004, 1.5106395944000406, 1.2173792171304605, 0.5877984452461285, 0.40164158669206285, 0.49314484674123477, 0.6369172382431483, 0.30906424153270595, 0.07965956431007583, 0.5427702853507923, 1.126428332661948, 0.6368430428420156, 0.30390055537152993, 0.4414597468991329, 0.4343715161258451, 0.46787251780130323, 0.6205950557040218, 0.026108890586311898, 0.23196095917345344, 0.19761605325144138, 0.10517959230717874, 0.8042182964010761, 0.6054442273750572, 0.5651868367869619, 0.3859751638496021, 0.4798946463473792, 0.33420536190310607, 0.8178912484040624, 0.8943620997149724, 0.8603560870500154, 0.6654578603169082, 0.6258752665160912, 0.3930119714908782]], [[0.8603341350791492, 0.5397263184820813, 0.514715241070552, 1.491350943897564, 1.1767932040879028, 0.6278972663591891, 0.36171833664597036, 0.5103975222573209, 0.6494908117013921, 0.2949764765520517, 0.11395146718502067, 0.5423133664076021, 1.2368514375908706, 0.6579846423016128, 0.25069254499907206, 0.412818393384085, 0.44728745437926587, 0.47505935237489294, 0.5832164223725056, 0.029270995340349215, 0.22655773358531572, 0.18092685240780793, 0.11833374143580017, 0.9728102822371066, 0.45738683463827734, 0.608162757019888, 0.3682190984648338, 0.23963918690207142, 0.38226820424286595, 0.8833859287338097, 0.9307080238410497, 0.8411550819693402, 0.5304814537436777, 0.5966202623672101, 0.34164418391710893]], [[0.8009734924394442, 0.5092088170577613, 0.46182977757145527, 1.4283387576669269, 1.1539815493005687, 0.6486830592014305, 0.38417100257148545, 0.5227950659758193, 0.7425499243957365, 0.29253499609604916, 0.10897635210984655, 0.5329751321552159, 1.3514349596020558, 0.6238151565050575, 0.2621194814067468, 0.41913948346502117, 0.428248844958786, 0.4661772472147048, 0.6146886804294547, 0.04237900908543629, 0.2575662310729559, 0.17621757795561194, 0.11846802013455046, 1.043290677252305, 0.4052274783629794, 0.5392661071812019, 0.365537017324318, 0.27760761845849974, 0.47166668797171746, 0.8221625277944702, 0.9717441288021083, 0.7963501300385628, 0.429672937358373, 0.5496672220221792, 0.3209063807656404]], [[0.7747456135152103, 0.46863057533164176, 0.4152545186321054, 1.3828886651722108, 1.1004589336777792, 0.7210587763759082, 0.3628209937836224, 0.5398683139754106, 0.8753434402686724, 0.32408249122923377, 0.13855979688771788, 0.5190264341356818, 1.3628950252882226, 0.647011998033205, 0.2176676863998943, 0.4283865386949295, 0.4000835566806794, 0.5029937143827479, 0.6191452576434004, 0.035698406560531054, 0.25599450675330315, 0.17347106743374588, 0.06555372824699995, 1.0017759127378127, 0.38204149291871864, 0.3958178446123174, 0.35158707197542416, 0.4569514923882084, 0.5295606200842229, 0.7709122894724737, 1.0132659981985122, 0.7524250816894672, 0.38490904194945325, 0.47634615218028253, 0.41899765397665634]], [[0.7207119915086396, 0.4300763626921764, 0.41624237117860363, 1.5623415040022566, 1.1072603343018241, 0.35137942379196807, 0.08971909621893365, 0.5134050157146196, 0.9843869821063057, 0.37041645370759974, 0.1274523606457266, 0.5126142631459019, 1.3197502390957507, 0.730960623075581, 0.22575656340549313, 0.5466207238502926, 0.43954798799340855, 0.4958889908660341, 0.6036428279825523, 0.030921883356372415, 0.28821122981011865, 0.1763200595824634, 0.0730562486876364, 0.9847074731530008, 0.342752341179116, 0.21298753651951327, 0.24181287282378, 0.5702837395026452, 0.5020955056434309, 0.7463200389459297, 1.0511235204360037, 0.7089673819089638, 0.36258718961456143, 0.5543989591365635, 0.5886045723382974]], [[0.4366402862843809, 0.4128558732258908, 0.5074555344669287, 0.36998874089026557, 0.3999191424861622, 0.4323264014928433, 0.4166556336902915, 0.37753058662776073, 0.38066751011617417, 0.24083541778245743, 0.8238585625999513, 0.5380465825697971, 0.34668196327136724, 0.548392642847571, 0.22088650097579232, 0.45676116877394746, 0.30120978070067045, 0.4472035422983923, 0.5617662345281436, 0.05077948356492251, 0.20334266062494918, 0.0871116654644078, 0.024490465224494683, 0.21410065294948485, 0.43503863611127624, 0.3648941821340038, 0.5354403261018396, 0.6127366952392386, 0.7779219908179418, 0.4788681351815879, 0.48251243686501566, 0.8280446408107671, 0.6781529613264872, 0.5054529728894042, 0.4732976175236684]], [[0.45150016662983916, 0.4385920075960469, 0.5285036510651273, 0.4000738761975456, 0.4162440886317001, 0.47028887871487746, 0.44272089062914116, 0.412727713455863, 0.39775673125195965, 0.306818497858062, 0.814344425732914, 0.5478459173630514, 0.36293958917966973, 0.4898261303343275, 0.2427155562078659, 0.47122984757264536, 0.350868101595287, 0.5247623013743159, 0.6426625729730063, 0.0597349258601656, 0.20177595226685432, 0.09642420401228131, 0.021065979442174643, 0.3557445145181156, 0.5480468944438219, 0.4721701664657821, 0.5867187666103557, 0.6613395627224843, 0.783223352662267, 0.46338711356906953, 0.4919679404197468, 0.7929494936732757, 0.6766981401484997, 0.5711356643556977, 0.5079023982697932]], [[0.45753177264181655, 0.45045520871725064, 0.53780532681917, 0.43505114681563717, 0.425474629952249, 0.48824375782623186, 0.45519934609870105, 0.42706195985145834, 0.4154385052695504, 0.3431167618055263, 0.808533693750394, 0.5361101558709189, 0.4398191544712632, 0.439099020161007, 0.2741329131568493, 0.4916329366444442, 0.45700758962576693, 0.680156846163592, 0.7313557911322708, 0.07694549057467348, 0.19609542685564085, 0.10773241731909874, 0.015168880247418155, 0.4267125636149309, 0.5962491710892331, 0.525943261408125, 0.6137125343374463, 0.6902103837257839, 0.7787665332437808, 0.44357072085970695, 0.5090243246845386, 0.7706417724934447, 0.6772022994681842, 0.5788936933449234, 0.5053690374588764]], [[0.45979515165556517, 0.4547326654665754, 0.540880429454219, 0.4559801091985579, 0.4298573159405313, 0.49536539601150825, 0.4567634415595554, 0.4132883139649642, 0.4240082097585847, 0.3534369589759344, 0.8084819134610091, 0.5675106190100304, 0.37979093722568963, 0.45576386943366554, 0.22419162063780862, 0.46683953068938533, 0.45399978866912305, 0.7208656649678165, 0.7101390392226617, 0.07965623656395102, 0.19642132135047016, 0.1103102882175509, 0.013743195423061724, 0.42395520300132883, 0.51508468068069, 0.4949110017993053, 0.6196142646440768, 0.6797157572310563, 0.7708432592095245, 0.4217482990171287, 0.5271916474527721, 0.7759450181815064, 0.6796158288164778, 0.5711155044341867, 0.4530169975956949]], [[0.46146421133285676, 0.4577178994389727, 0.545135744837992, 0.46026320789616715, 0.43342074292992105, 0.49981383336649865, 0.4582466529572696, 0.39985994073982245, 0.4316705444660307, 0.36181292366208867, 0.8085493706637437, 0.5567597105750179, 0.33915200009408575, 0.44467488029800456, 0.3029186005154968, 0.4391886144651993, 0.5276783060631907, 0.7197078917507064, 0.7516979605530444, 0.10272610996614479, 0.20520828324614793, 0.12089795514672634, 0.01524191477717628, 0.42427363212240915, 0.4574722548152747, 0.4774250254152952, 0.6283072539573101, 0.6718902449114103, 0.7697650726374048, 0.40995211441345286, 0.5343365531644305, 0.7654594826068333, 0.6808654424839817, 0.5771164508266466, 0.43138966922472277]], [[0.4625927646931165, 0.46024368634365964, 0.5471074005342782, 0.4601258934203611, 0.43727164311507627, 0.5039753739725655, 0.4597902408047132, 0.39578950040608163, 0.43851633165926, 0.369895498772676, 0.808568484353575, 0.57367746268494, 0.3559613896808037, 0.473735289742945, 0.2906750860820747, 0.514135706492863, 0.4189186313048609, 0.6562480038903743, 0.7881424487071258, 0.16331472468601127, 0.22950699043237408, 0.15853392031168506, 0.02409025528603339, 0.4278822083518611, 0.4270927757078621, 0.4703294369881656, 0.6375736597508288, 0.6686509008527519, 0.7726100925740216, 0.39183196171539736, 0.5406685877341445, 0.7550628302707998, 0.6815153329478405, 0.58307374645214, 0.42586417201607335]], [[0.46217157712038104, 0.46266435024196584, 0.548470511770734, 0.4601614284081632, 0.44171220169809855, 0.5086440814594342, 0.4607443356805069, 0.39477105883200003, 0.4459966182534758, 0.37687224140612574, 0.808241120664036, 0.5417641688015014, 0.3286844262729297, 0.4691146555755463, 0.23093014421933478, 0.671971752195775, 0.39826870342095344, 0.6502664400984073, 0.7083371766802327, 0.1973619463351294, 0.22950642856253944, 0.1516898730508725, 0.020584200400195037, 0.429502600323681, 0.4144120969010169, 0.4663326680642751, 0.6432664697301531, 0.6694065185204485, 0.7784330632215755, 0.36876598086849904, 0.545574484264989, 0.7397628514864485, 0.6823610136489275, 0.5872382168867959, 0.42789180644474256]], [[0.46105943304542385, 0.46526263700966064, 0.5501194535503182, 0.46420799208933616, 0.4468978403130312, 0.5137712848395257, 0.4622779246969403, 0.40389376515072256, 0.4532800642882796, 0.37943644248827973, 0.806486302382912, 0.5141420794217778, 0.29870492921147296, 0.44546923013864176, 0.22184519056275387, 0.5580823795729241, 0.4452330792134341, 0.6812759941915455, 0.658773897362602, 0.18610050268262776, 0.20959173837945358, 0.1611155861976205, 0.040849941114479656, 0.4338650985220389, 0.4120013460507313, 0.461320525007718, 0.6463590365927199, 0.6694117535212524, 0.7828754843337056, 0.3547526835067911, 0.5473239803214356, 0.7191528387650168, 0.681821129354379, 0.5849738885356072, 0.4307932477298253]], [[0.46001240214259487, 0.4679352023534176, 0.5506609679591448, 0.4750234599945193, 0.45174847126115647, 0.519922378636944, 0.464070369056718, 0.424554202153802, 0.45429457182669897, 0.38154464299175134, 0.8047747785732577, 0.5242873514695815, 0.30692899295452036, 0.45628143045359115, 0.24858709229390596, 0.4982195987814272, 0.4753178879796601, 0.6849143561075579, 0.6626722554109921, 0.16762883476383753, 0.18874248226591803, 0.15490880467498702, 0.0342176440790063, 0.43308128517261885, 0.4192323312626699, 0.4602833973826509, 0.6481678915156708, 0.6699675532265035, 0.7859451288048512, 0.36351431202355516, 0.5588232268070417, 0.6974601241622524, 0.6738172447083031, 0.5758201165223806, 0.43748240327443155]], [[0.45873183830486897, 0.46994188301426665, 0.5508265123968743, 0.48119922897189005, 0.45496298567262583, 0.5236901015715316, 0.4655870402237631, 0.4424517059373669, 0.4503119010150903, 0.3795034112675542, 0.8019193220902545, 0.46075054503478546, 0.33492462549376256, 0.44723951356985625, 0.23809021852759682, 0.6232500664112447, 0.43803440823497675, 0.6368541420042587, 0.6650755645581676, 0.15634582966483893, 0.19552663110173382, 0.14581480060419547, 0.024665056774770586, 0.43848578625292667, 0.43059521166940107, 0.45916443904552834, 0.6507661276516516, 0.671940842006558, 0.7897849718378427, 0.4124391015978136, 0.5636754163478185, 0.6848960559395632, 0.6641112061707367, 0.561261399261205, 0.4295812156439883]], [[0.4576843581095002, 0.4719490591712281, 0.5513687429818945, 0.48505538795901015, 0.45758429070551526, 0.5275022621092618, 0.4675967576748489, 0.4587137453914758, 0.44408738454287344, 0.37680363792000054, 0.7998090001786151, 0.4298290632474968, 0.3555408707235352, 0.45890656690767706, 0.29418394764969447, 0.6001647003639639, 0.4528321790022112, 0.584335691615187, 0.6994051185995238, 0.1579560421554908, 0.21829360337547485, 0.14238478362640655, 0.016865866961061304, 0.4446759829904841, 0.4446549991095468, 0.4589493383341427, 0.6548366489410757, 0.6752130808098826, 0.7955726360695614, 0.47300859306181625, 0.5715998341012027, 0.6804047272214097, 0.6553408874851341, 0.5399803140250327, 0.4264753805944119]], [[0.4560302696863099, 0.47264577704611854, 0.5508698640931837, 0.4868236673987213, 0.4585235679735736, 0.5290644158874528, 0.46863455422348, 0.4680998434431177, 0.4356225172352176, 0.3762563267655998, 0.798501250401746, 0.5406353045066246, 0.3468269603253611, 0.49541720144892876, 0.3028974152497226, 0.44780660689467455, 0.47342221838926823, 0.5949002717387817, 0.6930225861596224, 0.16461494471838997, 0.20932262940411114, 0.13147202245118694, 0.01120791656162445, 0.4546308822357652, 0.4564513647947094, 0.4582021314270457, 0.6592131734313431, 0.681164901056956, 0.8019756378585293, 0.5205350736505772, 0.5834778861126855, 0.6794819287458767, 0.6506536650626656, 0.516831024213095, 0.4364682010738088]], [[0.4553657320086141, 0.4732277700153813, 0.5514222150263558, 0.4884337622262522, 0.4579515785005649, 0.5290455094349538, 0.4689415241800459, 0.47239247087432756, 0.4262861611864771, 0.3761184793056496, 0.7981573830025019, 0.5210619464909878, 0.30810796392527995, 0.44887018183695726, 0.29296326047815946, 0.3897895458861789, 0.4795066972488574, 0.5494107883097987, 0.7272952060930065, 0.2117886538149053, 0.2482463222808311, 0.15412253987332714, 0.013752572707244374, 0.45736060202804224, 0.4698921795204648, 0.4592781477589803, 0.6647710564044245, 0.6859472181296471, 0.806553657923654, 0.5473027100361252, 0.6052721930180609, 0.6871180677114683, 0.6490654736553493, 0.5008458674856209, 0.43737655595048763]], [[0.45472799074663134, 0.4732269970517958, 0.5512992795545166, 0.4894973503591103, 0.45736516483106116, 0.5289127277615178, 0.46880002647252245, 0.4728628353596409, 0.4123400358516956, 0.374482685747154, 0.7977307760855448, 0.5595731111508842, 0.26597507555866473, 0.4093249918855807, 0.28180275030622526, 0.3684389132805883, 0.46386912545465453, 0.5168704978000138, 0.7527439214048459, 0.24527539612762528, 0.29303666695506125, 0.19364869103661642, 0.016543795646134625, 0.46835156974440184, 0.48056738833289236, 0.46080509123189983, 0.6719279318670504, 0.6920292638255748, 0.8093564578258853, 0.5559315042239441, 0.6323507789786154, 0.7025178763727689, 0.648453888616294, 0.49278823626885865, 0.4272321995200069]], [[0.45341465819601356, 0.4709543198014595, 0.5511405299405288, 0.49011773338918185, 0.4550793869848835, 0.526496340737882, 0.46681212867697974, 0.46319379234756286, 0.39938529738780143, 0.37162661834174626, 0.7980626829916813, 0.6575140236149721, 0.31385109834374425, 0.43097210193799673, 0.278156357936363, 0.37012662986516, 0.4902098921292489, 0.5121120243467797, 0.7612126856052049, 0.258505382707102, 0.3479946609437476, 0.25930755317076115, 0.017415838409365312, 0.47865883146437066, 0.49324945974168216, 0.4580041624111498, 0.6816811227395514, 0.7000821936632675, 0.811987296410074, 0.5581596913633198, 0.6460996331216282, 0.7259197704652278, 0.6498465836556488, 0.4891073492835589, 0.40968323185841904]], [[0.4526530949155654, 0.46852322314512757, 0.5496699099383315, 0.49615755593291844, 0.45171068941140263, 0.5223104044415701, 0.4642829292858047, 0.45474510349315106, 0.3896203601923202, 0.36768612475575996, 0.7995831449693906, 0.6549589074810203, 0.3270201963433015, 0.44222584354402106, 0.2702843174643555, 0.36942426709157217, 0.4659336638114404, 0.5098579864928945, 0.7371195640207122, 0.23715549728238666, 0.3531841252218697, 0.2834150317179995, 0.02413086598870451, 0.49075616704472597, 0.5012528174639825, 0.4553152533054502, 0.6896729852891823, 0.707237760674793, 0.8136622562665089, 0.5558748753452494, 0.6464769025711198, 0.7500175159407361, 0.6539890627624466, 0.47992789558963767, 0.40294861806484006]], [[0.4522312117470484, 0.4665242286015925, 0.5488197857026762, 0.4994291254290225, 0.44788319898619133, 0.5182828353611793, 0.4621047672090555, 0.4474551372801552, 0.3788066190448077, 0.3635362329149495, 0.8013445367355994, 0.636569275000926, 0.33365119416623645, 0.4348817076914248, 0.2741838463948399, 0.3724219783285266, 0.49302980723655165, 0.5148837123809096, 0.6924588441628742, 0.21104583316818182, 0.3443321906013849, 0.22390517930752749, 0.02017699809015406, 0.5116957480453012, 0.497835489909483, 0.45625716405329353, 0.6939611521812368, 0.7111083154953907, 0.8103438384310258, 0.5561344206710523, 0.630199531512073, 0.7733634602224703, 0.6579602262953732, 0.46431994374578717, 0.38657463196661035]], [[0.450793326300129, 0.46237570078815615, 0.5462972228992283, 0.501598545725193, 0.441947750588661, 0.511457750149942, 0.45844237033216106, 0.4429507555124197, 0.371592955478647, 0.35791818548435095, 0.8038895053448071, 0.548598214756067, 0.2616060396077061, 0.41911418293490665, 0.22994683176238045, 0.3807301469128443, 0.5069936272383703, 0.5278530857355808, 0.6653684329342748, 0.19833975441769514, 0.31966422491065466, 0.15803358829001302, 0.03601685523241663, 0.5369699909114263, 0.4974508567029107, 0.45573007573567453, 0.6963892934406377, 0.7168468170802045, 0.8047838808821512, 0.5514072126877232, 0.5937007606623886, 0.7733274876197231, 0.6642634079678822, 0.45053383353392684, 0.37868801295806653]], [[0.44989519939140654, 0.4587682254443104, 0.544623387223943, 0.4969249027271912, 0.4365648332932743, 0.5067098040991302, 0.4553256295567352, 0.43873848771326995, 0.372615695461885, 0.3516622992717411, 0.8060974792367874, 0.5624105700039019, 0.1743581026461617, 0.43248834384788837, 0.243273042599843, 0.37884844572606735, 0.5447739538981269, 0.5168670428527609, 0.6490061704204918, 0.17661943935710933, 0.29643323457053994, 0.13091943307193166, 0.024364726377321073, 0.5638637255271447, 0.5067499998342717, 0.4561339728408002, 0.70316451595405, 0.7230678202743088, 0.79693006814434, 0.5471690433842832, 0.5713541646433417, 0.7756540145215389, 0.6718296603947289, 0.44329586154849643, 0.3696353320696736]], [[0.44859439995358635, 0.45535283484230715, 0.5423564022690347, 0.49337561488583453, 0.4320633187411881, 0.5020668954516271, 0.45288627451142127, 0.4351698267674894, 0.37753702315566773, 0.3482348020290845, 0.8079517965604075, 0.5608073482785835, 0.11338576736038747, 0.44591054506976335, 0.23524926035790467, 0.38300326628239134, 0.6084215784413048, 0.53688303861013, 0.6137070839920029, 0.18057792247248067, 0.2974276993936949, 0.17736205642807473, 0.025349751410587866, 0.5824686519106015, 0.5150444171172758, 0.4617312014576721, 0.7077385321425413, 0.7300344972530886, 0.7896745293559178, 0.5365839408271412, 0.5579788579785586, 0.7766637714932099, 0.6774705376705834, 0.4375387885296873, 0.3574185006308256]], [[0.4479261035608664, 0.45319898203308173, 0.540284668690717, 0.49525046601015743, 0.4291782650571658, 0.4987751020924207, 0.45147933735999873, 0.4335678443293668, 0.3842679862903028, 0.35032313349180993, 0.8093162269976442, 0.5600291326567948, 0.14752583857514553, 0.43672794886573507, 0.19167808252541835, 0.3881469351629131, 0.6381129306777641, 0.540616781342971, 0.6121760971975831, 0.20459433073469957, 0.3210105349475061, 0.23182483945110238, 0.031250207175180164, 0.5988314784848106, 0.5293225436538899, 0.4728515061771219, 0.7130038803407457, 0.7377109186298487, 0.7815941677010214, 0.5261292169850104, 0.5646625439395423, 0.7773545086337017, 0.6837934952624656, 0.43309188150523664, 0.3589263532782241]], [[0.44863270502234703, 0.4519252784250605, 0.5390846075681337, 0.4965292718010157, 0.4264880588580247, 0.49607526221147324, 0.45154616486304106, 0.4327619496556469, 0.39380976044937793, 0.35696512178175155, 0.8107283244094861, 0.5619173565195783, 0.09906236982462531, 0.4704015076503013, 0.18918664651613332, 0.40053807245916734, 0.5661960527826095, 0.5315284343619318, 0.5772606235094973, 0.18182418377856135, 0.3111075139245833, 0.25252750519395795, 0.044654765695838164, 0.6047922938873224, 0.554434429857556, 0.4919001030759074, 0.7184763865919884, 0.7461834369927332, 0.7739637389041385, 0.5177830793244561, 0.565304843427094, 0.7774779486218549, 0.6928983612650725, 0.43187926651290065, 0.34923929347484706]], [[0.44982232899077995, 0.4503947938117357, 0.5375994913602788, 0.49612321461524195, 0.42289416146352615, 0.49269152363830593, 0.451247506806382, 0.4325467557250674, 0.401102000855057, 0.3669014794346306, 0.812160863812154, 0.5994272401894222, 0.09356097844423765, 0.4762902764205575, 0.1924501093103989, 0.40829373879507014, 0.5718759040011185, 0.5163687560743982, 0.5284084221378808, 0.16080276209350008, 0.2964823878939607, 0.2710836473334863, 0.0530265861620077, 0.6013501975834402, 0.585849414697062, 0.5273538291999916, 0.7200247821246664, 0.7540337744944827, 0.773647183309604, 0.5182127770826862, 0.5630584480439232, 0.7739429820827579, 0.7037030633735353, 0.43467631476629653, 0.3436871725528563]], [[0.45197941306525785, 0.4512396347642934, 0.5378238224667078, 0.5010849010840192, 0.4222567504816354, 0.49254547387765446, 0.4531759615460444, 0.43265519930327, 0.4095553418561324, 0.3781115150755261, 0.8124261569833349, 0.5854323035001121, 0.13043686009721936, 0.442398161667482, 0.23881289874914208, 0.388058169397157, 0.5719415181131864, 0.5662889853138338, 0.6975071488938351, 0.15667477933026042, 0.28814424435080965, 0.2680925347499447, 0.028085092464421618, 0.6011769920194864, 0.6280017262996956, 0.5792910521044945, 0.7165475329343947, 0.7629908303940172, 0.7713226403476904, 0.5124544937548197, 0.5632555740938902, 0.7686715638845543, 0.6997784806222368, 0.4398243854535103, 0.337588034635095]], [[0.45324907680012666, 0.4514995289189756, 0.537022855663043, 0.5031888689815757, 0.42086489334309657, 0.49217422257793536, 0.4563775971718125, 0.4312682880241845, 0.41955911439062005, 0.3985371431747496, 0.8123423745024265, 0.5406908910022976, 0.15037706939538145, 0.40978119787809625, 0.23464660552827096, 0.3827548625334244, 0.5804217037526268, 0.5836237324446794, 0.7068254433497314, 0.13711010049333772, 0.2841488026655313, 0.28316638233325214, 0.035707651760611185, 0.599770906170783, 0.6742729513371757, 0.6395651866920449, 0.7132053407467736, 0.7790061395330131, 0.7800292717880954, 0.4981062189801942, 0.5709363821218221, 0.7606634435628346, 0.6952549566601852, 0.4458943203297982, 0.34947454561063485]], [[0.4523636949552199, 0.44894846987898296, 0.5358455480969792, 0.5111566643531967, 0.4173075088226399, 0.49156681311912226, 0.4561924358672439, 0.44231722827679454, 0.4255621082322991, 0.4117925467879351, 0.8121439222186114, 0.536212094343014, 0.17354802470199576, 0.41597939505849657, 0.22832284801167413, 0.4007057442050304, 0.6329867677946865, 0.5963913313242076, 0.7300168396548364, 0.15299748445862793, 0.27269011592120096, 0.3511279743929868, 0.0456279036466925, 0.5927284249243836, 0.703124225680664, 0.6895419785711947, 0.7098698243976027, 0.8027844324438245, 0.7988198076171193, 0.49291406382045655, 0.5758969024813362, 0.7523627407238211, 0.6924231593900989, 0.44865840870300483, 0.38233562738470495]], [[0.45088672759285076, 0.44780214050481665, 0.5346974025073905, 0.5156161151806911, 0.4157869976318184, 0.4908901185989078, 0.4550946094959459, 0.4537175097491307, 0.4270666745307309, 0.41655057361169107, 0.8113706967502934, 0.5370935321674772, 0.19570817704675517, 0.4106829519592588, 0.229807102839998, 0.4143650826958039, 0.6243399989684102, 0.5959508463624351, 0.7729445809276609, 0.1472980741670244, 0.25906292261448083, 0.35113169693219254, 0.03769236448401764, 0.5961139007483958, 0.7081709136039918, 0.7187509431655977, 0.6994958715221395, 0.8165860800971316, 0.7982341157745774, 0.4989812869352814, 0.5798902990334093, 0.7431858718297264, 0.6879339595018612, 0.45059548328557814, 0.4074497047738119]], [[0.4492313937125826, 0.44850656081587487, 0.5345180255570481, 0.5223812327452249, 0.4172601803259121, 0.49360033206021636, 0.45681061305252685, 0.4570557680305308, 0.4288125840312753, 0.41763666682572, 0.8087349304817374, 0.4735431162811157, 0.1846967942108071, 0.3963275031519648, 0.26533378213196995, 0.417074323774866, 0.6201080955541466, 0.5930667190270908, 0.7761291454546266, 0.15284464016197052, 0.22452162568615164, 0.343044584556147, 0.030124802324434964, 0.612594074789168, 0.7039576248841595, 0.7063001852514359, 0.6973870605911817, 0.8068677726228664, 0.7907479654364691, 0.5036097469793037, 0.5735415898855175, 0.7303773682979975, 0.6821045888761359, 0.45229281035982144, 0.4096586966097755]], [[0.908165094290777, 0.7973996643752481, 0.9852122201603449, 0.8453977854034491, 0.7783626301221723, 0.7871657663511642, 0.8480100092066218, 0.8336590513330171, 0.7821680299993087, 0.8148136963686776, 1.5605664214179382, 0.4465495012013904, 0.24640572586099874, 0.3729591764006628, 0.2569459694375836, 0.4770127863929834, 0.6431288409092611, 0.5009908434124554, 0.8778036323336487, 0.17009516818349724, 0.28032872310775664, 0.35262123528751127, 0.040763070286784324, 1.132258879147459, 1.0893494703302107, 1.198130229184527, 1.2697110588554503, 1.3635100806780402, 1.2687024366285053, 0.9121383922358823, 0.9804167884260362, 1.358912358433591, 1.3205841594123087, 0.9083731917484628, 0.7951140894803163]], [[0.9091403427638002, 0.8057533476016052, 0.9883501371193855, 0.8722612206953211, 0.7897022994829411, 0.8118231986165667, 0.8601255878901455, 0.8119537571144442, 0.7918672345641797, 0.8284880968030366, 1.5524520584302275, 0.4546863219145102, 0.17704874242146848, 0.3966713264604638, 0.24857653624022655, 0.39818549093952504, 0.6083552206502116, 0.5331498189909589, 0.8307436417258085, 0.11403672864663791, 0.29284279599788654, 0.3821319913111394, 0.0446616661157741, 1.2145729441949418, 1.1097665620355757, 1.1327952963862633, 1.283469879890065, 1.3611528560666146, 1.255762403833968, 0.9183585596176047, 0.9638329462675385, 1.3599372735436082, 1.2971826926753063, 0.9364025119479255, 0.9059136392195134]], [[0.443838332304374, 0.44095035950229383, 0.5285510422267177, 0.5395151743979221, 0.41195174840343274, 0.4892860404467838, 0.4505198874834328, 0.43980732215208956, 0.4254101887340768, 0.4157588461135141, 0.8079226817141395, 0.5804208184856686, 0.16670989874841863, 0.4578969639146101, 0.24214063804958025, 0.38475542650859595, 0.6212887954657821, 0.600105666679428, 0.7980099468928774, 0.12448931190316725, 0.2585030539112737, 0.3292172234034819, 0.03293743255604936, 0.6592764843915091, 0.6661533730629772, 0.6162860876015303, 0.7227114556967935, 0.825622085199225, 0.7584089969944366, 0.4830555412253851, 0.5114265648800886, 0.7359021192692197, 0.6620390419286794, 0.44107558308778044, 0.4595400697134029]], [[0.43260540483293897, 0.41408485119733757, 0.5117810987377005, 0.5258867602562252, 0.38510205867871905, 0.4538394274395571, 0.4244355033011768, 0.43625194914170107, 0.40199553124125176, 0.4037170140564897, 0.8222158735276206, 0.5927259444436577, 0.21034057071544038, 0.49145426950106286, 0.2496057844842401, 0.3888040206910858, 0.6004454017482886, 0.6026507360088204, 0.7839410311691174, 0.12520382106459532, 0.24979379917890832, 0.32065645099468165, 0.03217716124214646, 0.6308150242126209, 0.6635611285052061, 0.5534491631934952, 0.7392724839676248, 0.8501144136027952, 0.7292535252740124, 0.4739078016989956, 0.4677694478475668, 0.7482936752538394, 0.6586268666987989, 0.39094705718674094, 0.4420244531209595]], [[1.0988750999759938, 0.7749915168162997, 0.37409463644980545, 1.2699321019901322, 2.2452572440478007, 0.630365302758923, 0.40886711915842755, 0.5158117255475718, 0.7465922178181318, 1.0752461542115934, 0.09915877083474663, 0.5675429888423629, 1.1957576221651083, 0.36194285405184157, 0.2554877774867684, 0.3362296999079042, 0.5196341880434597, 0.5345657348800923, 0.9307668722863804, 0.12526775585708502, 0.3276981449244564, 0.45649693025964144, 0.0709506536416691, 0.12951737107146508, 1.1867284135994813, 0.7006295153856117, 0.08736729060403928, 0.8388241730336105, 0.9745040305782418, 1.3212158874975524, 0.8680208517446657, 0.7580253955656697, 0.799734446045449, 0.6500638120490725, 0.6485292930259107]], [[1.1388008919474168, 0.8036463501030847, 0.35364113677997217, 1.384307128095068, 2.1484210775733343, 0.625312408775135, 0.44796252856002583, 0.46630549240289854, 0.7693075375288788, 1.0141386419916958, 0.11510313347379873, 0.6069815702688934, 1.1985202873300027, 0.4374288478750297, 0.29604355378998737, 0.3441906497001582, 0.48131623083815983, 0.5826845989373218, 0.9011081641521004, 0.12979928708270952, 0.34887785233597596, 0.4206147125136603, 0.06902645591529365, 0.13057155950174743, 1.1727472125453056, 0.644352051907473, 0.2194824946006835, 1.0377829360945776, 1.0024239672797868, 1.1763794223899158, 0.8366382617956147, 0.7213929575260816, 0.6742829561379028, 0.6502844532139875, 0.6565346964455868]], [[1.121023269660666, 0.8801972195391001, 0.34012541887686154, 1.3790646614242033, 2.0890056520132116, 0.6706061113710116, 0.4052422144518276, 0.48707550950198175, 0.7234397443493269, 1.0538380069500604, 0.13256063113113173, 0.623665764536131, 1.2189098464908534, 0.44675475775626133, 0.3403748918169035, 0.35902955021912036, 0.5383540172865391, 0.5764727867425727, 0.9554726643598321, 0.15352741181362767, 0.35958945127701725, 0.4235640361303502, 0.06993867625048948, 0.22624155782188146, 1.167071310059466, 0.6337937437508201, 0.3645986294523642, 1.0416504404104066, 1.0313883779170823, 1.1242083579682038, 0.8096863341768703, 0.6703764259136151, 0.6376381090870908, 0.6322518449140174, 0.6366872275765014]], [[1.1213795935696487, 0.8942117583745066, 0.32445171240465775, 1.3024704464090024, 2.091564896108979, 0.7174140808398101, 0.42545658893231897, 0.5645033677636795, 0.6577602562598317, 1.1782282662122576, 0.178532708990541, 0.8042677095466162, 1.2091898943660175, 0.5044602506017148, 0.3467175698753758, 0.33660635804322425, 0.4638445076818856, 0.5343584473109749, 0.8543469323500903, 0.18293075132978287, 0.3864043056542125, 0.3792376302886293, 0.07177659985156254, 0.25081487717171497, 1.2031130645410093, 0.6262396800076323, 0.49538990982276343, 1.1995858361409688, 1.038076188250769, 1.1800051298631482, 0.7759762365919225, 0.585742887326784, 0.6224812467185077, 0.6353140758832905, 0.671963247206819]], [[1.0781501685890937, 0.874033322987851, 0.31368080367795814, 1.1056125806271615, 2.0412703299012103, 0.7320158215689978, 0.3956896512473001, 0.5430905245549651, 0.6508612259458291, 1.2556315020967517, 0.1722335911473859, 0.8894841630491931, 1.1328587724337074, 0.5443624764495539, 0.3747305426363497, 0.2692787473599594, 0.48418927281275753, 0.6157919308143156, 0.979804060025762, 0.0988702722085523, 0.3855020964028005, 0.3210272320000641, 0.07671603296921617, 0.3084077392819943, 1.15070363926195, 0.5793408152334496, 0.6240071493679114, 1.1879225582368445, 1.0032708054893642, 1.11884863377671, 0.8597614587896163, 0.5212423713408862, 0.618821879053367, 0.5978080713436218, 0.5982630106977596]], [[1.0592594256172374, 0.8623773535406152, 0.3074149848776717, 0.9016813384716538, 2.054772691767774, 0.79475377701071, 0.40686836014658945, 0.5471807353164648, 0.6957547488538445, 1.2972010369297093, 0.09424131517926085, 0.8748334206057016, 1.0445250478684998, 0.566479793493855, 0.3318215528551392, 0.2565781441049151, 0.4340461096213247, 0.6112183492419577, 0.9785469968005037, 0.11389543241972183, 0.39731380148967615, 0.31709347849974745, 0.08577758003337727, 0.3794798201676538, 1.1616601109680338, 0.49610724789407135, 0.6875264130822616, 1.1993977003506786, 0.9547973924342174, 0.9167565125901608, 0.9098295539284473, 0.47892451055004703, 0.5622950227688133, 0.583728696906372, 0.5316288700552854]], [[1.108822906466185, 0.8695262916035423, 0.2977575206140728, 0.7832212341704087, 2.066171450528894, 0.7818468153594194, 0.42804060159345725, 0.6345596529959561, 0.7342496399394313, 1.2816707927708304, 0.0899476395003741, 0.9093294249273154, 0.97229850988656, 0.5950670952111132, 0.3827400934978334, 0.2930085012977776, 0.4198540629024767, 0.6126169189934902, 1.0947010683327598, 0.12344146871725487, 0.39790205132992673, 0.31821378673824774, 0.09384710437603698, 0.4025077828018296, 1.0758287911444664, 0.4656853392271533, 0.7097625391999359, 1.1017180443798642, 0.8943497525812214, 0.7609041618594934, 0.7824087669482218, 0.45068154052914766, 0.557833317339469, 0.5897497977499822, 0.5562863513954679]], [[1.0960571136621826, 0.8243629913692823, 0.31202609265827574, 0.6901011232456791, 2.0195678178884764, 0.8011263940692714, 0.3997786543244196, 0.669678634724048, 0.7533363899149075, 1.2000832759083786, 0.07512914646354535, 0.9186524117379993, 0.9820356461417825, 0.5410823191976799, 0.3747840142296339, 0.26585250933756543, 0.4179977545735716, 0.611911156646371, 1.1006878105846016, 0.11104258881235424, 0.3923534519163885, 0.3540144454875754, 0.09874308852748934, 0.43367435043915126, 1.0149603233715583, 0.42497752160432034, 0.7662215459847936, 1.1523992152575078, 0.8382973000524805, 0.6617024604953972, 0.7048180290051438, 0.41150624573164507, 0.5519625354399313, 0.6997614344959582, 0.6096321737818644]], [[1.1560832950405306, 0.7915806045583216, 0.3335172510984791, 0.6709104551189164, 1.916762496256646, 0.7476628043265381, 0.42039144042860344, 0.6103412858368669, 0.8533935429451834, 1.063205177781283, 0.07841643139616328, 0.8686587630841027, 0.9622690357183875, 0.5414398000802481, 0.3843132296659299, 0.26807633317479157, 0.4154971630385159, 0.5829112524157778, 1.0777514150128849, 0.09678214350638792, 0.39212458788692983, 0.3670057531882238, 0.0878382664050318, 0.4152152714386335, 0.9758830508238349, 0.33428644138452845, 0.7678878738853093, 1.1067691971336426, 0.7715617878042957, 0.6432263744680475, 0.7102464456002306, 0.3889746828953111, 0.5614834682731749, 0.6491156155249457, 0.575769957783038]], [[1.145324928055531, 0.7544432575575757, 0.3059647218879009, 0.720685050859879, 1.8427441148918389, 0.8079521895498036, 0.4148300753170917, 0.6223215710430753, 0.8671017381282953, 1.0411708284171237, 0.07247927210906421, 0.8032232063840379, 1.068544556780432, 0.5511118979280871, 0.38149853140253626, 0.27004557154569236, 0.4089067164310015, 0.5966120734974076, 1.0847631637647335, 0.06930021706540962, 0.405428626028833, 0.40454982091139635, 0.08808860109331132, 0.39712544811057626, 0.9087636051778556, 0.4391115804801305, 0.7874732287352373, 1.1521088965793682, 0.7411740121690737, 0.54291385540071, 0.7976322174792054, 0.5851811954520659, 0.6356539339482432, 0.6417070830358023, 0.5365298750742903]], [[1.243415677825538, 0.7449324908127896, 0.30473530378517494, 0.7405668568930572, 1.9126014478314275, 0.7474675922557457, 0.42327374321936245, 0.6116439553929552, 0.7402565028044201, 1.0273210722365853, 0.06408213295604363, 0.786459104193795, 1.0739297633158587, 0.6460069587549194, 0.38639113430006433, 0.3003261518340064, 0.4151814421081704, 0.5815789525784353, 0.9462279035565868, 0.07098277160137034, 0.37347179894269744, 0.3974876382472702, 0.0898640270350948, 0.38705749406681, 0.8935432954029086, 0.3966736666853411, 0.817477418592115, 1.1038505285048719, 0.5895352046127091, 0.47277987689094914, 0.8318070964930352, 0.6264223089961809, 0.7178848144799568, 0.5444207390046316, 0.5142897995489055]], [[1.3422626975811942, 0.6880065546119133, 0.3203974554543526, 0.7736585270837979, 1.9363338680951125, 0.7566871441887999, 0.42753302190474035, 0.644960118831347, 0.502595679062255, 1.0319238908335437, 0.07316811747766676, 0.8487677230521073, 1.0850197770759815, 0.6753101993616172, 0.4467422915981319, 0.4542580807880901, 0.4772211188255492, 0.5985138716668871, 0.8059052496391259, 0.0718033373041125, 0.33925475249402315, 0.36946715767460747, 0.10138264346372727, 0.3654061198769857, 0.93505257586397, 0.384594961066554, 0.8498544178087412, 1.1336854692812626, 0.5245836054258315, 0.3972923583214215, 0.8779715594855408, 0.6551411152105693, 0.7503951491315561, 0.3892344342837133, 0.4273603153091613]], [[1.3561499495476594, 0.5894032947299064, 0.34066230677258713, 0.7884949651507474, 1.8429606602899526, 0.7485214678886469, 0.40364649283544146, 0.6698432221780607, 0.46753468371999946, 0.9616854468557257, 0.09459216227712108, 0.8582367354388525, 1.095241010201737, 0.6101130000768114, 0.44915593205615656, 0.42774506124938233, 0.4876375965836304, 0.5871285263988277, 0.7606580872696292, 0.09251322269950243, 0.33066847544200717, 0.36664330362030223, 0.10723226225298013, 0.3282173763971411, 0.9926701677001688, 0.37942989169583574, 0.8292574191824398, 0.9994552629840758, 0.45718095182721685, 0.5373037489482934, 0.868955793628366, 0.6761851958242404, 0.8092864916325134, 0.3712929041786954, 0.4635393189574948]], [[1.4013741795133163, 0.5988483626023344, 0.3532751641872125, 0.837081331341335, 1.8271582814686438, 0.6797915041344459, 0.40039956763659024, 0.6467174429981073, 0.4705871117368926, 0.9194328236586642, 0.07478806027983133, 0.8465031447836722, 1.1271741010491119, 0.6707374808422069, 0.45366716041166355, 0.4649209950485487, 0.5656942869113835, 0.6147759420702021, 0.740484685954881, 0.09022141759424636, 0.3280141220764715, 0.34806043805031195, 0.10032013387882538, 0.3360946664591522, 1.0218635721322413, 0.36133944152629227, 0.8078516811138605, 0.9251704325055288, 0.5534991183873358, 0.6130861312811668, 0.8264324221130659, 0.6932175737115989, 0.8086169911256189, 0.408695776581137, 0.48214922477709565]], [[1.4184850473352024, 0.5877007729721416, 0.3875860149563012, 0.8335510721800694, 1.773121788485438, 0.6446770329523751, 0.39199353418334176, 0.6130531905174639, 0.47229174738199764, 0.8463534576944891, 0.0479374682707317, 0.8314838760876495, 1.1592306216814312, 0.6241402765493645, 0.45430014076977554, 0.4366384939885968, 0.5566836882642763, 0.5906323997510541, 0.7314168353915341, 0.09330148656976432, 0.3521901055422546, 0.3262340193698164, 0.0964742484261411, 0.4143397897904765, 1.006475168029841, 0.17856447642855586, 0.8584409385991295, 0.8035565759645742, 0.502255639562757, 0.7480867261842704, 0.7478779769959087, 0.7038850063461155, 0.8107144638130457, 0.36672953974048533, 0.43344577345702673]], [[1.4359834674261356, 0.5484479708171227, 0.4109124613997196, 0.9766243786717725, 1.6829945517567895, 0.6673572938924862, 0.4029155607385636, 0.5939374538872841, 0.5152880592654823, 0.7813970921561704, 0.03994346300964825, 0.9004341090460808, 1.1699367790086415, 0.6061503727711064, 0.43915043628065337, 0.4112860770295767, 0.525171491257045, 0.5888473920241063, 0.8132093961464976, 0.1399750060354254, 0.3929887013194654, 0.35768086655726455, 0.11077209915105328, 0.5629520868921063, 0.9636901138661982, 0.16232320129952024, 0.9069722099201487, 0.732006549908025, 0.33663025697735216, 0.7744300826716141, 0.685489709790821, 0.7184356746329599, 0.8039479776805072, 0.22575682514315004, 0.3993306592332496]], [[1.4434995113630005, 0.4774559405797745, 0.3813660873639443, 0.9559089122731671, 1.612135659585986, 0.6757624010515839, 0.4066921744676492, 0.576041742233024, 0.4784847901991617, 0.6457897422315273, 0.051337707963564475, 0.9152259321909388, 1.1831338196219434, 0.5970721073426625, 0.4001560760778232, 0.41259952446047043, 0.5171413393912269, 0.604140865263209, 0.7268828861084129, 0.13720943218809706, 0.4045133717456037, 0.4464883079540359, 0.12236082782238107, 0.7601750569078591, 0.9289242254589685, 0.27965569977290883, 0.9586337272984551, 0.6731748771584304, 0.33577514319055035, 0.7213580190919382, 0.7061389244481575, 0.7149257878489027, 0.7130777699985129, 0.20317947432416414, 0.37190425420807316]], [[1.465632468206871, 0.4074515623626963, 0.38097984479038316, 0.9149868279797593, 1.6017526290419783, 0.6572786197300443, 0.40276270742078063, 0.4809566850226357, 0.4620410842390593, 0.6060337406005365, 0.03735130106052817, 0.8603133049746314, 1.2066547167546915, 0.5595434834411452, 0.4239552537157378, 0.4721919009978697, 0.4892361420700163, 0.619358003106953, 0.6985136997552797, 0.1545665232485652, 0.3752283949844595, 0.5734189010186153, 0.13102001859916435, 0.9111038840649752, 0.8376426903840872, 0.35594415793391065, 0.9632595285572279, 0.6264327694019349, 0.3713427190930486, 0.8053837464631435, 0.7206663061873615, 0.7565144581955404, 0.6427731935387067, 0.21742180488666743, 0.3452510570894393]], [[1.4984596578269507, 0.43723038173360845, 0.37699664206648903, 0.833063629838424, 1.586306716230625, 0.617263104466587, 0.41223110275927755, 0.49346896760537556, 0.4412207142813202, 0.6070438498401884, 0.023788869256448755, 0.8444058311985663, 1.2339404406433945, 0.5569300316055521, 0.47426983303340947, 0.4236879458598691, 0.47016548585576967, 0.59306882818586, 0.7323538120377234, 0.1987893539247429, 0.3432053906355275, 0.6301213881166693, 0.12465516194369157, 0.9446780863420994, 0.8288160929212219, 0.38523346181030627, 0.905663272672939, 0.5529007505045125, 0.3699711780693503, 0.8581342987848328, 0.7707006298054143, 0.8729619609963838, 0.60482925179724, 0.2520551513703746, 0.35621204843356424]], [[1.5018523303564333, 0.5151176550873625, 0.36408928443110344, 0.8418097538122938, 1.5532792725230173, 0.6360933077538409, 0.3953275160490117, 0.5153246475679408, 0.4380251721115266, 0.5203360916629419, 0.01816035551700335, 0.8539837947053237, 1.2516607683044814, 0.5708725311925219, 0.49643526640943136, 0.40904171361938313, 0.5182391651452274, 0.5838423777480315, 0.7230135173300182, 0.22231608603170816, 0.30781688155353254, 0.7271123755366965, 0.12325966746592006, 0.9579794259307061, 0.746791720462017, 0.4129554488747282, 0.9111242402063284, 0.5437419811360559, 0.41988988439318264, 0.8570553772733625, 0.7539066525049671, 0.9478581212613628, 0.6350823304819195, 0.2851769740161305, 0.48781367643930057]], [[1.5363778813773459, 0.37820314291050394, 0.34411382351134534, 0.8524539249231353, 1.7338123271173727, 0.5879462628403277, 0.4123193063296377, 0.5218276601958893, 0.6331903116561794, 0.3884973386376528, 0.013986286511108786, 0.8769810246493814, 1.1230835811608497, 0.5697236974649281, 0.4626514383776357, 0.3986898145353919, 0.5736179677092423, 0.5499775270509922, 0.6945101182044924, 0.27358329650574154, 0.3157744923917148, 0.7144546274686698, 0.1408230430169115, 0.9011915417105003, 0.6639472955990641, 0.41358284573095117, 0.9116963739666227, 0.5209702363966701, 0.45923582097078, 0.8718168236428647, 0.7819078988328854, 0.9082924357678028, 0.6178549757302281, 0.2845185482897159, 0.5534397541627027]], [[1.5958108342029638, 0.36399845346527254, 0.32710457219116507, 0.9393577404862961, 1.806679854761674, 0.6180543338497323, 0.42350596513640054, 0.5482785431058266, 0.6216258908516925, 0.3276277214667276, 0.01532254415687567, 0.7759954808379048, 1.0611650627928104, 0.6552219191291296, 0.433285976724737, 0.4361433547397361, 0.5939369526530911, 0.5672324459024232, 0.6081425026185984, 0.28304390053618056, 0.2863234621175611, 0.7282110005176621, 0.07075884428342669, 0.8333950081180976, 0.5315157839742897, 0.36702722448861874, 0.8682456629623407, 0.5279795227849287, 0.4798945594942933, 0.9932000893458409, 0.7531644809506877, 0.8760494955558716, 0.6479356818552084, 0.2652609984626515, 0.6872264236085599]], [[1.5528473546073331, 0.3642382820025196, 0.3467217442617656, 1.025239134722077, 1.770614532176109, 0.5926571822465336, 0.4317335453185832, 0.5096716899600227, 0.6829985917803688, 0.29305853332234594, 0.011198633429541327, 0.5796072456878616, 1.0032418333328916, 0.5604326209561419, 0.4046816468804677, 0.38479828645477865, 0.596283419715593, 0.6335932422185265, 0.5938535122162063, 0.24791781722161585, 0.2249976342914037, 0.5830709013717629, 0.0204774208188264, 0.7717403044597075, 0.5017282838448696, 0.346153577167889, 0.8325738294938378, 0.5386847096481692, 0.49319856326598677, 0.9974444993492129, 0.6895206633560084, 0.8862262163635115, 0.6480640092606711, 0.23296995609223903, 0.9220422608761746]], [[1.539070174416923, 0.3775234898457124, 0.3131174698902194, 1.1163741628375683, 1.73082367426807, 0.6222854343177553, 0.43270347668347925, 0.4377852825948288, 0.7673085142123832, 0.24911318880838526, 0.01990789213086941, 0.48785810811160873, 0.9213497093063283, 0.47788277818396635, 0.37612775529222753, 0.38450910445701053, 0.5912346798007495, 0.6195825121846628, 0.5657134700193209, 0.22135927589230758, 0.2294319559475278, 0.35977345684652995, 0.04651406077386644, 0.8212482098008111, 0.523544816312158, 0.3833705738601274, 0.8309117544889024, 0.572224337187972, 0.48911734417134534, 1.0097613072542115, 0.6169217066204653, 0.9045958043759941, 0.5606612118086169, 0.3115080755087731, 0.9759116492487049]], [[1.3799503282861116, 0.37103447375468024, 0.3319246652285305, 1.1514690504262766, 1.64354705701106, 0.6125265703240608, 0.40532513582633695, 0.4102286889573476, 0.7544090486334304, 0.23776057843657677, 0.02247539376948459, 0.4867350283347205, 0.8145741845946581, 0.48575347422390847, 0.3730550886476033, 0.3815792337911174, 0.6417887323476893, 0.5853113499831732, 0.558692538540248, 0.2224486599264111, 0.22681851622554533, 0.29495514093995445, 0.04494586201640263, 0.8570706279579525, 0.5565762744682758, 0.4138601020244544, 0.8926948345427104, 0.6338131834566892, 0.5228161198398463, 0.9662904478818187, 0.6502455693954367, 0.9863055585715619, 0.5091292156735483, 0.37829642709249317, 1.0780477590243818]], [[1.4452737292238969, 0.35528707671620513, 0.3685809301658778, 1.2243884793771413, 1.3030762393086914, 0.6099017564385276, 0.40897369284209467, 0.4790366903610039, 0.4352601115882079, 0.22511758093506695, 0.021529400362209263, 0.4871228193343664, 0.6863835938444, 0.5026203507641247, 0.42713728635220166, 0.3827665570253271, 0.6512180001688196, 0.5828706471835812, 0.5695278065898945, 0.20998857660885456, 0.237030419372252, 0.2752021139543659, 0.04282734569468595, 0.8489435181251886, 0.4429768967729282, 0.4574092710273516, 0.8348206458326597, 0.5691988546677577, 0.5349421833516907, 1.0230604906280132, 0.69399412481737, 1.0332736994894363, 0.4768509310681049, 0.40874830655093614, 1.0940196112876936]], [[1.5231262869756887, 0.3642363958468414, 0.4282983783815203, 1.323593744138979, 1.2683831518461064, 0.6159696015252141, 0.40009618209092257, 0.5100995114913078, 0.39685294888242156, 0.22526032075408908, 0.04077852327418055, 0.48669667032062314, 0.5545811428708938, 0.5070431709846323, 0.5501276821362446, 0.41797416517984043, 0.6950388710256707, 0.5852536323062347, 0.615881715592062, 0.16746619332875395, 0.24514542991107185, 0.3022282519105665, 0.04226719853393686, 0.8610755941508947, 0.3990624928843621, 0.5232333673166041, 0.6805172615863654, 0.5600395311094596, 0.5532248888443687, 1.0681857702583724, 0.7050959358679993, 1.0513320281392284, 0.4268808700679988, 0.4536663262137384, 0.984434596900045]], [[1.528591037662896, 0.3777415421040607, 0.34880492573348076, 1.5753775983321125, 1.3692678233799476, 0.6049595898050224, 0.39461566253245645, 0.4701532396060709, 0.38335421740709474, 0.24215760997101765, 0.009383553105350802, 0.4737842465782436, 0.4060604820177423, 0.47084561197340474, 0.5074981303208601, 0.449767389074452, 0.6802688147709959, 0.5699718563774473, 0.5696879906007375, 0.13701874903070765, 0.2394537404826051, 0.2848887351799566, 0.04532024218561945, 0.8882378497076928, 0.37508037003299005, 0.5397774463096252, 0.7010140153387268, 0.542342765865035, 0.5606778897589058, 1.1196903124280597, 0.7345194274225846, 1.042070463093443, 0.42937940623942394, 0.4812797526083421, 0.8067079430911718]], [[1.4761592015777736, 0.37627269671008645, 0.3521961394628802, 1.7314131677979927, 1.4690798041331747, 0.6074700007894748, 0.3968377417929126, 0.5775693073442634, 0.4004476529221478, 0.23577190339074827, 0.0024927044704159563, 0.46540615149924136, 0.4175550159997612, 0.41737958699248295, 0.45472965673364957, 0.4655043027937771, 0.6808349841687817, 0.5880497687616256, 0.5899859610098185, 0.12342060573265427, 0.23274202153007625, 0.28022784955847446, 0.04620201915463601, 0.830769744940067, 0.34979458046822676, 0.5322136346737868, 0.7388515507893214, 0.526298145865851, 0.5786882647954988, 1.057305308757103, 0.7917525935163692, 1.0236754150409755, 0.3862790812368177, 0.5021229148653121, 0.8353171856845393]], [[1.4576252003967274, 0.37735588293807903, 0.35483359283472304, 1.8047643856563294, 1.4409142219573017, 0.6025013334875149, 0.3891447611988857, 0.6722334166012933, 0.5134524589842041, 0.3375552180375381, 0.0022756554687821677, 0.45231321364512533, 0.5058611878482004, 0.43041844123333955, 0.41916667501252863, 0.44033879702650397, 0.6579934905181095, 0.6425224487934493, 0.6122457215170074, 0.11992431831391663, 0.23466408610208517, 0.28586729332962235, 0.0456010233143124, 0.7797452798335985, 0.327407613516579, 0.5359453844827948, 0.7173191557996625, 0.4357998795054676, 0.6562334055956122, 0.9188997147882105, 0.8618739375157616, 0.9884513849097183, 0.46765294952950215, 0.5413479630725481, 0.9169097282958827]], [[1.4410777102147105, 0.37939701488256716, 0.3681420314481866, 1.9275501035303135, 1.4504359001136315, 0.6107593762921486, 0.3963798707654123, 0.7603445250138929, 0.6226083117525543, 0.3809946161228015, 0.0030555524630525837, 0.4563025680672759, 0.7629299320636637, 0.46603195308717116, 0.4129347853533581, 0.41417662801534433, 0.6873799088171728, 0.6517619710943865, 0.6103334549531323, 0.1188238687136176, 0.23822598735405717, 0.31471181981023655, 0.04614749980135678, 0.763136434057903, 0.3250546972756171, 0.5587001759156395, 0.5967153278737014, 0.3878737951159007, 0.7350899693005284, 0.8427599200663571, 0.9037558347894584, 0.9372858476142287, 0.4924218526876928, 0.5679038219148947, 0.9954952787597905]], [[1.423912089487907, 0.376446889598702, 0.3066607335017877, 2.0019431764390943, 1.415549169633202, 0.6034014887837948, 0.3985288782531865, 0.8197259061320856, 0.7126547878833651, 0.40722476643640226, 0.004035523596935474, 0.4519270936599497, 1.0113260416814573, 0.4092855583028798, 0.3805220940411963, 0.4036070984956841, 0.6726830950069422, 0.6611505077563378, 0.6413095476791233, 0.1282109608131301, 0.2471721489688139, 0.3159673115036995, 0.050213126425268186, 0.7411623471702172, 0.36729010949570906, 0.5196105168476293, 0.5379527919089515, 0.38952481679077544, 0.7654583617432831, 0.7736235203927256, 0.8633635286479394, 0.8866506284248309, 0.5484462048449905, 0.5819381078387545, 1.058189200795033]], [[1.3558807437813662, 0.3746798438526081, 0.34179542801790064, 2.004405649493881, 1.4141230980043613, 0.569600543573829, 0.41294244682306364, 0.8979490577567776, 0.8225185766754131, 0.41895095810560157, 0.008070593565322822, 0.48223316821641254, 1.2266102727452075, 0.41767251136985717, 0.36371480467096035, 0.3806309783084029, 0.8297856200755868, 0.6844430179467069, 0.6323589199867022, 0.1379824336622169, 0.249195935369672, 0.3101571945119083, 0.0516802950073308, 0.7027770535674238, 0.43160099790224893, 0.43236845387462763, 0.5566584896915481, 0.4750436874076188, 0.7309175834956321, 0.6717582823034116, 0.832211341948629, 0.8444720035681399, 0.6112785417096603, 0.5908754727065002, 1.0864702193111158]], [[0.7843218944667757, 0.3797794095537247, 0.31973520718326454, 1.7826703541079862, 0.9438096597096088, 0.5706137503367843, 0.3843896809778408, 0.8148766196480434, 0.7860676950865682, 0.41347823403357575, 0.012537217035766112, 0.4662664418919862, 0.7817855560722005, 0.38685567820821964, 0.3451532732502798, 0.35967317211448613, 0.7117996773870461, 0.7114823710049543, 0.659401177027005, 0.12599994937759884, 0.20461852014363963, 0.2101621616651403, 3.4870886584447467, 0.48152941771395236, 0.358176021318538, 0.2844109137004171, 0.3327417222210882, 0.4379474600870037, 0.5282238509570075, 0.40417265398649754, 0.5541785049636109, 0.5276073052540886, 0.573139153206426, 0.49958372613492025, 0.8000499486173139]], [[0.7606687651983821, 0.3825512341061721, 0.3096479319778629, 1.8357773080762092, 0.9119400394955841, 0.5678119965217135, 0.3949712851456942, 0.8653023326595373, 0.8498980729766908, 0.43877630321651884, 0.026212302986173403, 0.46050854518136347, 0.7619919743245662, 0.38751951712457994, 0.34441967661442163, 0.36281064434896537, 0.711074962756003, 0.6633299644925452, 0.6494455926955618, 0.1363673169772165, 0.201597268102728, 0.21542715984743055, 3.4870886584447467, 0.43106193780942914, 0.39103083358076657, 0.22896925376963786, 0.2408087190016038, 0.4815324546831453, 0.45764462634353104, 0.38720882630729136, 0.5897236080759269, 0.5466017442705475, 0.6603579814208576, 0.5220527218670786, 0.8241489963910423]], [[0.7476209030590916, 0.38075727895761086, 0.31398531393636264, 1.8578161930858454, 0.8722368054307787, 0.5698320393491206, 0.38974281659081467, 0.898626493233521, 0.9002866708921188, 0.498202415915956, 0.02448938755263555, 0.47094661476722877, 0.766584400395971, 0.39381269189799584, 0.3378500719668467, 0.36102204949372446, 0.620156991960722, 0.6693749716340507, 0.6540477544771162, 0.1297500025535196, 0.1984053261100061, 0.22666929839212874, 3.4870886584447467, 0.41960636111313127, 0.5136695485020335, 0.19447387655682702, 0.12344190183761183, 0.43968207512146573, 0.4213160629123765, 0.39334790437665856, 0.6377598372177345, 0.5944521113860943, 0.702086642052855, 0.5229420010458892, 0.8574691952285926]], [[0.7890779345203383, 0.39830634050394703, 0.3527535989965508, 1.9143351815783545, 0.9132800240926939, 0.5922916903284932, 0.37826266307282824, 0.8911269924348661, 0.8408432225410523, 0.4379405755351679, 0.03691431457072434, 0.4558692304216557, 0.8026107958850828, 0.393497454142672, 0.3159866020236285, 0.3829974314526823, 0.5755168642683614, 0.6752397910628942, 0.6283658036486067, 0.13265953792976182, 0.20067237986230746, 0.2275059965265256, 3.4870886584447467, 0.4384374573752047, 0.6459917278434952, 0.1632585308026756, 0.15266473371022976, 0.4414690868122264, 0.3962397832654414, 0.35934877917121905, 0.6873639370892032, 0.6410613847503517, 0.6923200657724586, 0.49415155915443565, 0.8688606729948072]], [[0.7996104571556404, 0.4045392204200593, 0.35822239028859004, 1.9677212108636442, 0.8911403115019387, 0.5942823146076374, 0.37739375032463457, 0.8773773796504316, 0.795263456836458, 0.37434664133244755, 0.0453692072150121, 0.4562701269802192, 0.8197886902324676, 0.3911005774739892, 0.3340043566814106, 0.3756806305353726, 0.6061223376067091, 0.6951946602447217, 0.6439601086022897, 0.13612664101566474, 0.20634379839212064, 0.23359210756211463, 3.4870886584447467, 0.4245817168725705, 0.718140692390137, 0.13456348876929675, 0.20565863079001984, 0.5255813879312408, 0.3776322915710111, 0.3401525194219286, 0.7290612005897112, 0.676182660806204, 0.6569124118210299, 0.46948339654724525, 0.862457719047162]], [[0.7824949704759963, 0.4050414391937045, 0.3512776138046502, 2.0233976322885985, 0.8553494153407117, 0.5870143032085687, 0.3807083167513766, 0.8677600396512837, 0.7623362604663658, 0.2896576274826552, 0.05420811664918869, 0.46591535214268637, 0.8123129196342089, 0.4008802674984987, 0.3595872422610269, 0.37087620030115487, 0.6167215836740035, 0.7145932156768218, 0.6491655019421009, 0.13270321965328602, 0.20785616010694308, 0.23734922340016745, 3.4870886584447467, 0.3559112908226236, 0.6997422816292018, 0.12687852213742284, 0.25787410197000843, 0.6461419100035752, 0.3707694361462054, 0.3253114808402207, 0.7714647282954553, 0.6929179358255609, 0.5857389984283585, 0.44724520531086714, 0.8460873913218451]], [[0.7695468133342706, 0.40934793473360154, 0.34719921077802685, 2.0764663778950023, 0.8862213440702131, 0.5931019913425577, 0.38436199935262294, 0.8601398439045552, 0.7467974209226216, 0.3322786927759618, 0.0636121432328627, 0.4590873321136732, 0.8168656370961862, 0.39936681431697657, 0.3611328923032368, 0.3823072103271429, 0.6408111083363885, 0.7177007747422789, 0.6638516606242013, 0.12397652517640595, 0.21362227366906822, 0.22157542878306247, 3.4870886584447467, 0.3020129542258194, 0.641016967384664, 0.1262904615362852, 0.2571682282526647, 0.7876773675847195, 0.36539614980457347, 0.31315293590811866, 0.7847719759186318, 0.7348027645466251, 0.49942891727132094, 0.46662204421409315, 0.828683839162264]], [[0.7401316060054681, 0.4041755718761507, 0.3514170793245752, 2.0937931332746, 0.8389963793494086, 0.5680320004482926, 0.3790596838974389, 0.8540391664070783, 0.6988291706607628, 0.45492033978791363, 0.07551058694494972, 0.4426553436552716, 0.8220005830465748, 0.376935708569627, 0.35842259997683673, 0.3688642024907882, 0.6350459883951753, 0.7193702591882267, 0.6789940818603548, 0.10474769085287222, 0.214681974997398, 0.19746399365215717, 3.4870886584447467, 0.2819856286751217, 0.5425811613820906, 0.12335717702868398, 0.24024482924957039, 0.7846018666438198, 0.3635903710789605, 0.47864413767585584, 0.718197958980638, 0.5114563033589057, 0.451428132300707, 0.32974360081649556, 0.8996165698913544]], [[0.7599192549376873, 0.3989268662942441, 0.3573257058835626, 2.147959704522318, 0.8051025165175579, 0.5693844358749129, 0.3813977808785819, 0.8468769265651271, 0.6407000073136875, 0.4851960615985019, 0.08241234391200965, 0.4504646594941326, 0.8156417175355453, 0.3814872581044989, 0.3386415846694643, 0.38166899062862225, 0.6218751186259922, 0.6485046036250847, 0.6807983568419123, 0.09288080791403064, 0.21583623026372215, 0.18365144797822675, 3.4870886584447467, 0.28280403785314856, 0.4713849481083052, 0.13787374789828566, 0.24398738852317883, 0.7810794561020056, 0.34328835219697024, 0.6001591959812813, 0.6550476193282628, 0.43614528991356777, 0.4169868937701787, 0.30451697409233625, 0.9644460046595886]], [[0.8080128367546511, 0.4114579732402441, 0.38374700382783705, 2.0149070495944095, 0.7487141254103564, 0.566138980315413, 0.3995678233340809, 0.8778803640632318, 0.6179465970121782, 0.5340669461289406, 0.09272358072903528, 0.4503588224058641, 0.8185396775937765, 0.3847506917339686, 0.27899417381559627, 0.3824134939870721, 0.6435949988078187, 0.6375622596676276, 0.7238524053769274, 0.0777333675891414, 0.21018316456183628, 0.17649953018198072, 3.4870886584447467, 0.2884252799639433, 0.34452040493098873, 0.15400414009560182, 0.2450957574293864, 0.7646689124441618, 0.32516732310935836, 0.6662335784038012, 0.5911896845926816, 0.37879287558424757, 0.399570449696002, 0.29046575811766706, 1.0242124486087083]], [[0.8594285210859757, 0.4080901305299873, 0.3824327095323924, 2.0577911857618663, 0.6927482339251961, 0.5872726305466147, 0.3971608470921613, 0.8817458085842478, 0.564369488374649, 0.5040898524676685, 0.0931672908170269, 0.4475573498621829, 0.8090886481820695, 0.3640995386871419, 0.32167292365503375, 0.3284125418110952, 0.6522794655733065, 0.7395783483373745, 0.7626372515119892, 0.07679871404747124, 0.19412921843143865, 0.18806813362006694, 3.4870886584447467, 0.28066341834448344, 0.2559705568800981, 0.19239443322768618, 0.20977818959021868, 0.6816586577695699, 0.3104407279771744, 0.7157307917431296, 0.5280829006265929, 0.35379217827647547, 0.3619170671129941, 0.329353103956741, 1.0648822868579682]], [[0.9118360887845566, 0.3968365156051893, 0.377914132155415, 2.1041300652485337, 0.6036117796465155, 0.597998555421164, 0.39490278404198575, 0.8930698191331512, 0.5523231531331132, 0.507017024684236, 0.11481849445728201, 0.472551505282983, 0.7929326354905435, 0.37536375535473787, 0.3279810323913991, 0.34615065368587916, 0.6537033607306533, 0.7683090971694058, 0.7407670108767668, 0.08942043921823628, 0.18824035667038097, 0.18874331905576514, 3.4870886584447467, 0.28343790088698073, 0.24355585742797453, 0.2258984463060919, 0.15923674887038597, 0.589414901514898, 0.32815624472916377, 0.7214966845374842, 0.4746213120490975, 0.38208018946691424, 0.3151649132339578, 0.37629432583960293, 1.0949266545049892]], [[0.9718920501517293, 0.4058656678397821, 0.3971529393531746, 2.137617779204086, 0.6093234985942326, 0.6119403212987198, 0.39060538845366366, 0.910102578445107, 0.553050162264201, 0.5670580738338127, 0.13014576091586139, 0.4855480424935974, 0.7876155997274853, 0.38374287834770265, 0.35157065851003544, 0.3350685700722969, 0.5934348444587009, 0.7815756650551078, 0.7127698913790671, 0.08741468174346154, 0.1806339394970049, 0.1717935734403974, 3.4870886584447467, 0.1877557509077658, 0.22295218772410003, 0.21736319750244407, 0.14764232739323158, 0.5563394662148295, 0.3583003736283511, 0.7043762465604725, 0.44784934059372095, 0.4062539123507941, 0.264984550304888, 0.40123949038315954, 1.110959764903908]], [[0.9574507241894952, 0.43034441585265437, 0.42909916207573884, 2.1080037749414537, 0.6326821085664154, 0.6059616576266866, 0.37493762133355923, 0.9410094820483855, 0.5503008086384523, 0.6143731759755124, 0.1429193991588404, 0.48409953190413557, 0.7682060011442166, 0.3849523838074058, 0.3692293103058924, 0.3367383109022538, 0.5897993624674838, 0.7826408433228955, 0.7092477314904164, 0.10137051613199646, 0.16812422773539276, 0.16190964475182335, 3.4870886584447467, 0.1535217774825643, 0.221325594718421, 0.239868280584062, 0.1457230278176846, 0.5543489756790478, 0.35207578129989026, 0.679072578550675, 0.4354434166481017, 0.4313774261905392, 0.2500852940069822, 0.4064455725587345, 1.113528640536079]], [[0.9772528437238797, 0.42938890669365526, 0.4343596628173859, 2.0956389518363285, 0.6598212968775162, 0.6028836852384298, 0.38091858740891016, 1.0142777557575098, 0.5937760841451161, 0.6923521899373262, 0.14492137207286804, 0.4859019113677215, 0.7527449907082382, 0.4085162266021544, 0.36182745636741803, 0.362846291421121, 0.5698271249856119, 0.7755961812461649, 0.7208839240118214, 0.09917080198579911, 0.17474246828967469, 0.15317285834063354, 3.4870886584447467, 0.1476228435201451, 0.2817109925800474, 0.24499875093022824, 0.14561299323845633, 0.5774850370982305, 0.3345436728490364, 0.6617638756303548, 0.463683190842309, 0.44191012156033005, 0.2388396217443103, 0.4123663392713235, 1.1070894099080326]], [[0.9775645179759296, 0.46344252427021115, 0.441050046303052, 1.9826581811137305, 0.6532430353315622, 0.6054192220629764, 0.40118202998727387, 1.0441763049996249, 0.6733508506705713, 0.7909506445032746, 0.1430990917871325, 0.48158057630457, 0.743537025134184, 0.4148942985273128, 0.344635174727218, 0.35919642666739005, 0.5709696549240523, 0.7666201736806743, 0.7351462820466573, 0.09751361290186897, 0.18003853224586952, 0.13342099292319998, 3.4870886584447467, 0.1489460108823803, 0.3230741562036482, 0.24398729018178067, 0.12616096216062933, 0.6264152937115517, 0.3195947026112868, 0.6566093771373276, 0.5128929854304565, 0.4600413927019858, 0.23049247075681314, 0.3896287036765295, 1.077743538963833]], [[0.9857897224145247, 0.448982367965719, 0.4343983068241839, 1.9020584857774767, 0.6219584596199743, 0.5886204062561374, 0.37010930003273285, 1.0013409042974408, 0.7196305750279021, 0.8383837678830246, 0.13356726024166146, 0.4706179623765808, 0.7346663787123263, 0.39500004306132397, 0.34139809742208466, 0.3589403423233536, 0.5757850952951098, 0.7737968513781555, 0.7098379314706644, 0.10508686039455292, 0.18365206774709386, 0.12935170172524263, 3.4870886584447467, 0.18475404461848496, 0.3215657066301645, 0.22521979053027272, 0.14407307739434985, 0.701591134215919, 0.3053749936425215, 0.6701045327755966, 0.5332985927322693, 0.4540174259921842, 0.2247175298686615, 0.3233786914523988, 1.003745220224941]], [[0.9112385499078164, 0.49210186984168836, 0.4684297664244838, 1.6872687367633317, 0.5103834341102708, 0.6227957302871718, 0.3683119041084752, 1.1187911066093756, 0.7534588846676638, 0.9491825069832243, 0.1177335137981024, 0.4605395336072591, 0.7098709562278966, 0.41113495074235185, 0.3437339404760672, 0.40466598115736907, 0.5821833750094771, 0.765821957436241, 0.7143022895806103, 0.0679668895175925, 0.17549018577802292, 0.13418943996432453, 3.4870886584447467, 0.2272836632520487, 0.32050703999466773, 0.23652190760878797, 0.175981295178212, 0.7834119025653197, 0.30927618212723296, 0.6768274633001408, 0.4520157219755837, 0.41736864794894013, 0.18483806361496743, 0.19961528058434416, 0.9094284173186018]], [[0.9727467372502749, 0.5211588321849296, 0.4877924814586074, 1.6116398291276606, 0.4400740238003022, 0.6283021179040449, 0.3829866303322767, 1.1119907315686728, 0.7372434460611257, 1.022694489488423, 0.12391879205990994, 0.46623424829849175, 0.6979711393998138, 0.4003320572185824, 0.3540496280806418, 0.37537428196997535, 0.5711859853218693, 0.7681599089956301, 0.7166896069253365, 0.04757938541400744, 0.17309036807810746, 0.17815566577500525, 3.4870886584447467, 0.2657664604725312, 0.36329756672945107, 0.2800331999237854, 0.21916697660079168, 0.8755253067155101, 0.34990335773474457, 0.6968102814159347, 0.418515509989268, 0.45296680300973713, 0.1916466931433788, 0.14291175094030373, 0.8374024665113868]], [[0.7977896091446466, 0.5527867904014658, 0.5037059239875896, 1.4983677224849303, 0.43299643605701715, 0.6884656196717615, 0.43948855640978934, 1.1969534184940303, 0.6588011645251204, 1.0052856041650773, 0.13159921416992396, 0.5662680490064249, 0.6816796449953628, 0.44351726360914967, 0.339508979849953, 0.3667165338003049, 0.5636921291535137, 0.7636364083661564, 0.7267787088650027, 0.04395517058054943, 0.1855855167317525, 0.19116239881952152, 3.4870886584447467, 0.29218796334359576, 0.33305686088423586, 0.42935481878907056, 0.2668661209605133, 0.9999282152826623, 0.574436790527754, 0.7083743563488104, 0.3999747018873837, 0.5207768065113232, 0.21490249594016692, 0.1647063258483435, 0.7804908846040587]], [[0.8213980389306343, 0.5558652983815812, 0.5173024443133796, 1.4839257187018184, 0.3420341186008474, 0.6427769261939948, 0.3931387239442428, 1.078116377901326, 0.6134045644659756, 1.0459077369931298, 0.14029216828397892, 0.5786518399245074, 0.7263211859767842, 0.49278361832440387, 0.3446003264677483, 0.337849831662585, 0.5425533491256618, 0.7786188960995251, 0.7654414330761079, 0.03640537104171496, 0.17441809863957028, 0.1868654687523434, 3.4870886584447467, 0.3036832803449433, 0.3635019024075306, 0.5032111299910274, 0.28645581225125555, 1.062567419863439, 0.9191303784923822, 0.7267063126134123, 0.40527181599415274, 0.578579481792945, 0.21707457914408945, 0.20181746665845057, 0.7268567107520602]], [[0.7873430139417645, 0.5816734469240379, 0.5326614164763073, 1.4009818611112577, 0.4534916889530894, 0.7901562967734082, 0.3855084207189532, 1.048854186548603, 0.5776564279713814, 0.8854478240487879, 0.14552584990562398, 0.5990274360876824, 0.7730975537400085, 0.4885721794960058, 0.36308883281868154, 0.33312196608447175, 0.5031351714976873, 0.7375496882756751, 0.7602306763280482, 0.05158576634326062, 0.18032328849249366, 0.1897710479029553, 3.487088658444745, 0.3165841368545449, 0.3664520694993654, 0.44794969752663094, 0.2896075250907069, 0.9505555617273191, 0.9820682516131368, 0.7591214686453263, 0.4055436691094011, 0.594893461655543, 0.24177716454638007, 0.2740802442249621, 0.6804346397874734]], [[0.6817617039333079, 0.5766400714754687, 0.49629312816776017, 1.3576613590077353, 0.2755954654227389, 0.6690653736354251, 0.38564034258921237, 0.8696127087427235, 0.48495284365804847, 0.9114811568948818, 0.16459465558471367, 0.6391323702532172, 0.7826381564189211, 0.4770016114352406, 0.3772874383501683, 0.3160575829888095, 0.5225201535927877, 0.7537467296713498, 0.7221186723668165, 0.07575575540618987, 0.19817423726494388, 0.2118717311013846, 3.4870886584447467, 0.29658624419141344, 0.34419876316170883, 0.37753202273219116, 0.28425178027686837, 0.8512750895942531, 0.6728527816278717, 0.7220810064188229, 0.4115170584519161, 0.5669874751445081, 0.2600262972340791, 0.3738717510171454, 0.6620508622846586]], [[0.6291788816294434, 0.5537065491766475, 0.4621891599269154, 1.381549021117932, 0.2587464874703609, 0.6750260074725243, 0.3500448953228226, 0.8074525598565363, 0.4499919442597596, 1.0159354452900298, 0.12623484359118842, 0.7354805479669011, 0.7703268549615614, 0.5063903178322896, 0.3520185556871461, 0.31268561495867914, 0.504481017993831, 0.7961507692139431, 0.6841602455850768, 0.14040242239537393, 0.18506884413681757, 0.1971461307659311, 3.4870886584447467, 0.2970843489368609, 0.3650125755834144, 0.3391577130886237, 0.2603270142792248, 0.7706606381115095, 0.4850887972729978, 0.669130639107852, 0.43470409348357864, 0.5808647963096314, 0.26448801130641064, 0.44761461657155366, 0.6549970106959468]], [[0.5890033096110452, 0.5589339640123625, 0.4728407664816213, 1.3988760113200391, 0.2467302676354229, 0.6809980762502692, 0.3345111640587499, 0.7131239306804655, 0.4284286440422083, 1.0658813536173506, 0.11791219242372136, 0.7813629346708553, 0.7517270630104137, 0.4969376347927251, 0.3475364077595031, 0.3168880855886331, 0.5493569115011325, 0.77603727179332, 0.7098766685358726, 0.15994285260526336, 0.17397378550466144, 0.20409704022359135, 3.4870886584447467, 0.29144845104869044, 0.3541260357934439, 0.3402166663880042, 0.20671950452502436, 0.6968221206841533, 0.43724611121208473, 0.6425455107976025, 0.46022479009150496, 0.5885529102031827, 0.2759343208869228, 0.5667307856767939, 0.6437534626168673]], [[0.35825363374563984, 0.5996705312325524, 0.4890640060885002, 1.322566272891566, 0.24469552559105837, 0.6868880509271071, 0.3487398844163889, 0.6387474357410271, 0.4229868124991024, 1.0437928925997297, 0.11219564192842377, 0.7545415750685145, 0.6909387425482942, 0.5038617166717108, 0.350894781726084, 0.37749285752593964, 0.5348586846747192, 0.7723660041717495, 0.7399197938479614, 0.15485610497830787, 0.17246740793824256, 0.19528137436236326, 3.4870886584447467, 0.2884771492512984, 0.3264927975779276, 0.390323567871219, 0.1562822844017126, 0.5711094270829533, 0.3827708340234629, 0.6005078177977781, 0.40703316546488777, 0.49493235988584383, 0.292720565344596, 0.6299363495802889, 0.6203506421772493]], [[0.34281259477723225, 0.5879152497123902, 0.47147834389816734, 1.1923358190236313, 0.23077588960771958, 0.6623073796534091, 0.37547918274815284, 0.5404485270678352, 0.4725674182684796, 1.0909830304750459, 0.09820831842392673, 0.8085005615806071, 0.6723790262355889, 0.49576793160669697, 0.3464013675419908, 0.3951691934643437, 0.5199013240634129, 0.7513421173826347, 0.7112702757924341, 0.15250238945417605, 0.17933179596291365, 0.17159268464113941, 3.4870886584447467, 0.30344253501404733, 0.2982749985615879, 0.36743294428859474, 0.1620115740626149, 0.47104912899916185, 0.35764932205431094, 0.5918367145593869, 0.41806231126090054, 0.4335886725418252, 0.3226547746615416, 0.6925888849152642, 0.6049454172367026]], [[0.3213525508336049, 0.5899458544324183, 0.45430837939952884, 1.0797394549379336, 0.21198339895985938, 0.7004682671548867, 0.40168203484068, 0.4876657285907091, 0.5402914416890399, 1.1126827965493524, 0.09323609769085053, 0.8003657908334152, 0.6601949762645225, 0.5028216847792669, 0.3453081580559242, 0.4077600004939226, 0.5488807794763925, 0.7848768428804307, 0.7360362099401307, 0.17537920117205769, 0.18247721990043092, 0.25869434857204576, 3.4870886584447467, 0.3767169058588082, 0.3172346274766563, 0.35496240007142166, 0.18553994230600823, 0.4810238674821691, 0.3695808779793378, 0.6150319628381469, 0.47055576606936167, 0.4841043730472912, 0.327627962167344, 0.7850600276836799, 0.5887659121819481]], [[0.29007213454131714, 0.5840636808130023, 0.4609785274883872, 0.992809953723125, 0.19058879190626135, 0.728482471903402, 0.395056561838712, 0.44603673293071655, 0.574233974778325, 1.101207046500685, 0.10525915440630942, 0.811231883578343, 0.6771303081920741, 0.48549501039022824, 0.3470085789024435, 0.40091403822629124, 0.5374385899224321, 0.7556681650105896, 0.7087703926812852, 0.16924740698027896, 0.19096360382918792, 0.2851241079133971, 3.4870886584447467, 0.44586044609312403, 0.32376760737501453, 0.3638305486057889, 0.2227170641494059, 0.49890592684427354, 0.4465338898436897, 0.6455199968576839, 0.4772635239158872, 0.5587290055999852, 0.34118891073021573, 0.8363401499239123, 0.5776232332410516]], [[0.2886569727865159, 0.5667706895545646, 0.4361638185840849, 0.833223663381085, 0.19102780910635098, 0.721500943602086, 0.39304672774478394, 0.39677642594792284, 0.6052810507377695, 1.1502538713690929, 0.12123341962326858, 0.6756066471667408, 0.7673471965429846, 0.45066016407148685, 0.36042730744183005, 0.42112021719769854, 0.6153247745958449, 0.7847202136067564, 0.7002158885206976, 0.17597194202416644, 0.19631780692794026, 0.2821167291933166, 3.4870886584447467, 0.4262537548419343, 0.33725732559111776, 0.3650617028526699, 0.27617259919539355, 0.5466392485443523, 0.5276916531415252, 0.6609244276867066, 0.4899764307425214, 0.618443541142099, 0.34360523649124647, 0.8850976620363606, 0.564880930251963]], [[0.2635773772554964, 0.5681949830840921, 0.43937296002910237, 0.7713790548282724, 0.21869554699619037, 0.7330402208513576, 0.389726142362665, 0.4059563380837184, 0.6857014274941047, 1.1374122719043258, 0.1491456975161103, 0.7357495488148936, 0.8100159618727516, 0.6043587897212184, 0.40557034687834215, 0.41295562558853294, 0.7355077254113089, 0.752340577767411, 0.6608814550288376, 0.09423652291906487, 0.18180113217971403, 0.28459030788400563, 3.4870886584447467, 0.37088186507403414, 0.36354649715343046, 0.33743010406115637, 0.3300953139697748, 0.5216782503283082, 0.5365897453515025, 0.6513995328414014, 0.5438603681317125, 0.6204923154710461, 0.3253799369410457, 0.9439600019248593, 0.5234452599435462]], [[0.2338084159264503, 0.5551757125972104, 0.4518257779766134, 0.721030885001759, 0.25623571673199513, 0.7696773036249643, 0.3855850134128134, 0.4226857570613922, 0.7901003071786171, 0.9712099268109123, 0.16184827215830658, 0.7575531152554325, 0.7502887152134838, 0.6242380704574799, 0.39834640579446334, 0.4258718897060516, 0.7441770925593064, 0.7277656217775529, 0.6820687475939795, 0.08750309216607632, 0.18265328781006016, 0.31026954225157655, 3.4870886584447467, 0.3075632231222425, 0.3815874169810598, 0.29757568499640713, 0.321397255314567, 0.5599755783400396, 0.5531089034769042, 0.6204052916263414, 0.5840379088223662, 0.6360699356586521, 0.3060196297650666, 1.0515515412391143, 0.445859015508054]], [[0.2320306349137915, 0.5491701510846003, 0.45383757762308446, 0.6671552749791382, 0.26131796423707837, 0.7848653479523996, 0.37816402781489916, 0.46789155487228795, 0.8118199979084232, 0.9210876716777163, 0.17929085268799952, 0.7649591610910391, 0.6801003768337559, 0.6762241980165965, 0.3433775246285773, 0.45496215287954245, 0.7776126364271767, 0.7301408724968029, 0.7014768735562801, 0.0995490726123438, 0.18294658803897532, 0.29932578281976074, 3.4870886584447467, 0.24533497332180618, 0.393982294433555, 0.26936178979213254, 0.27447611455835697, 0.5689813684538038, 0.5454164128419746, 0.5606922006051935, 0.549348929146945, 0.6629358940260915, 0.284734276779692, 1.0350616698479032, 0.34363129930196534]], [[0.2769394355999132, 0.57442393033084, 0.45593769302941656, 0.6135467358955007, 0.36837298273572344, 0.826830581564354, 0.367907351309395, 0.5058706876818035, 1.275601685923161, 1.1602087551830897, 0.263492715119645, 0.9164879801055268, 1.08028190771615, 0.7091221485774275, 0.37052706319179407, 0.4947950122895247, 0.8199274581883278, 0.7538143557045052, 0.7340372446737563, 0.0680013960583693, 0.19544776744731923, 0.4198348004689036, 0.051271318816271165, 0.27587541213357836, 0.46089217446394287, 0.41181224131557836, 0.24723345758684256, 0.7545043140110848, 0.6257244137057657, 0.7152593244979453, 0.8262190642607594, 0.7620661590884421, 0.45782869316132224, 1.486829009204277, 0.3653383957192497]], [[0.2665407731503124, 0.5314545803907923, 0.3944701857355231, 0.5115638493306746, 0.35116933162317837, 0.8057979909344136, 0.3883609702762969, 0.41967913786787786, 1.273141617389384, 0.9764764951975227, 0.28002710069806874, 0.9137904716946152, 1.1646271270369941, 0.6954543692928425, 0.36575230890361654, 0.4961291085863838, 0.8251814373930931, 0.7319232123673087, 0.7302980585450323, 0.06635726049906862, 0.1945903386744157, 0.31336599880519933, 0.0322632798866746, 0.14646819318615484, 0.6093124512516002, 0.3554445107425863, 0.20930746607548295, 0.7242277566544846, 0.6744121395943171, 0.6249266125800488, 0.7089368787512531, 0.7136409714152302, 0.39316145780937223, 1.5563067711006395, 0.5731814679634056]], [[0.23845865101867936, 0.5961391736367266, 0.4503807547090061, 0.49975478754423214, 0.34014056623441946, 0.8775619862383061, 0.3757768281931021, 0.42517135251151117, 1.23956757028481, 0.9551428994634192, 0.2600901303261717, 0.9383386113100893, 1.2389947636599978, 0.8132519337471065, 0.3744063188765948, 0.4941036759648113, 0.8643883346589742, 0.708258406678617, 0.763717295632973, 0.07507643135457381, 0.18676503975565262, 0.257971762043509, 0.02802092974562491, 0.11249788951041984, 0.8224447991414306, 0.36865702475981343, 0.28184165068333505, 0.7324203223499566, 0.7600220458785326, 0.6464006560850956, 0.7643981168147904, 0.7271895634255063, 0.29526934716580666, 1.673200003133629, 0.8685555835159937]], [[0.21274513187400806, 0.5867781841743639, 0.46392249778807926, 0.46081084494408586, 0.29470295098767657, 0.8944193690732027, 0.3963462875052706, 0.4119843695231813, 1.2016810462229237, 0.8390363975005632, 0.2695080517171468, 0.9586344934794349, 1.394918936831553, 0.8146509341105564, 0.34925823819216373, 0.47948249031839607, 0.8643014975912837, 0.7512237856579183, 0.7884749817300741, 0.05700145593925765, 0.17779661160686908, 0.2258838213311668, 0.031065719399000505, 0.1462030150641495, 0.8772114978843943, 0.41415471118279557, 0.3358062300587898, 0.784824297769586, 0.8140881132301226, 0.7943860348420781, 0.9029790864724092, 0.7359541845398854, 0.3251461109646302, 1.6288844495840906, 0.9443220200708671]], [[0.1974211906417424, 0.6082337440757921, 0.5111480761574075, 0.433277972813346, 0.2711248779275196, 0.9760972845092865, 0.36947214170088066, 0.3146553002748393, 1.120015804637929, 0.8936021842410918, 0.2635956334419857, 0.9945182502745247, 1.3798724180785131, 0.7659689913763359, 0.35147826078403854, 0.47287258531366916, 0.859193202609242, 0.7405509161063462, 0.7669809762304243, 0.06058111095830776, 0.1647332702045075, 0.21077639956917055, 0.03161585657033497, 0.18548804192854546, 0.8104425307393901, 0.4195588662715699, 0.41624886501478986, 0.7913532652820243, 0.8744124435756611, 0.8343905619084053, 0.9396810477387199, 0.6244869994958884, 0.21953038993757817, 1.6387952101778778, 0.9326949025260034]], [[0.22847457293343626, 0.579484321837898, 0.4959555772313944, 0.3997144172172982, 0.2587435977061834, 0.9658250300217148, 0.381709531784927, 0.24644011369899782, 1.091016645478843, 0.8879766035634898, 0.25657096119832323, 0.9696243001236404, 1.3778526631563963, 0.7909867866601634, 0.3391939569314796, 0.47866792708603634, 0.8489291585096824, 0.7221463458767458, 0.8172791784441741, 0.08588087169881103, 0.15995648001805068, 0.1944730383871931, 0.034045793769589196, 0.2205830074132226, 0.8846564191351713, 0.4076371083798034, 0.4963541717940198, 0.898627263918784, 0.8993135427068186, 0.9014482662408703, 0.9489572166550849, 0.5705387529186268, 0.23822969722795678, 1.6955884265259842, 0.961972165385824]], [[0.2031434205788098, 0.594224636254855, 0.5017428756520613, 0.281021272263279, 0.2489236926102587, 1.0330632995672704, 0.38093167999131017, 0.1439314686412161, 1.1710780104806644, 0.8464719055871734, 0.2817517720371372, 0.9510041705683653, 1.3631494491915461, 0.7997764551532804, 0.3409847507355199, 0.47067793349652554, 0.8209076396405605, 0.7006066536101252, 0.9032632262859099, 0.13045304987251727, 0.20192170402485607, 0.23062187281953156, 0.03548673057019224, 0.27387873722909006, 1.1002571651284856, 0.3978110115064593, 0.5776147723449205, 1.0850600863991826, 0.8492819539911453, 0.8852597622148495, 0.9534943567149778, 0.569304852767953, 0.3059967415916974, 1.6593571093076354, 0.9338690273887197]], [[0.2021254194217944, 0.5722121114173303, 0.49577644477674015, 0.28476552735784316, 0.2520192370744229, 0.9806937532697635, 0.3883967308063126, 0.11184565597206514, 1.165479992156596, 0.7241369304350066, 0.28361758557392486, 0.9398070048863014, 1.323371748279286, 0.7973760491823898, 0.34030258983365413, 0.5183009126176555, 0.7478310055318095, 0.7130336259692103, 0.9266815981554957, 0.12580823607715524, 0.18294267721470125, 0.24204908203364361, 0.03533073487026057, 0.24708804280578509, 1.069884552728044, 0.4343943117908134, 0.581145790038106, 1.0891701612084186, 0.8405162937638148, 0.8911665561691728, 0.9688756749408751, 0.5737712669630755, 0.37263336986697604, 1.637986816799019, 0.9949451835217912]], [[0.1980049678944823, 0.5566499864064375, 0.47062476014072263, 0.30295782639132823, 0.24836241414866003, 0.9997013656214413, 0.4013049907836275, 0.09761690611623147, 1.1470034937600968, 0.7793554909996027, 0.29022234206122427, 0.9376661316345384, 1.2666616268776352, 0.9509038480798764, 0.3415126870261168, 0.5214034235541863, 0.7958262608260945, 0.8273520432016679, 0.9312066265093558, 0.12921711051163234, 0.15053607734141622, 0.2508397780321794, 0.034350620804966966, 0.21566977016286237, 0.9784931550178648, 0.48897450707163065, 0.5612865152136884, 1.1125607314384491, 0.8298286385990161, 0.9233986255791822, 0.9958332967203496, 0.5667064955678622, 0.44774931697749143, 1.5708901756862121, 1.0851648729361565]], [[0.23967429736050577, 0.5709114111573066, 0.48884773508202095, 0.34060600376640876, 0.2182950967634007, 1.0162381766465385, 0.3946270738419255, 0.0974258861202201, 1.117555323901795, 0.782435799515784, 0.2742868107914048, 0.9000749508157299, 1.3404093772248464, 0.9391678731949706, 0.3361238697444271, 0.515659436485654, 0.8387840968446787, 0.866078607928787, 0.9678507722914438, 0.14939318819477232, 0.14101285688239204, 0.28416636709658616, 0.03382936291753517, 0.2069592773555028, 0.8540539483570677, 0.48681658565437014, 0.54463048030716, 1.0905619597500196, 0.7742841980591703, 0.9381599283581565, 0.9292137757971983, 0.5451411912662232, 0.48173396628373333, 1.5405007257364967, 1.0701339115440185]], [[0.3898325254758548, 0.6015773296227441, 0.5383063085456541, 0.4767747346718528, 0.16655630142525707, 1.0647922988837166, 0.3878885233770197, 0.11654284248936952, 0.944637942331231, 0.772653412243772, 0.30216240391399696, 1.0154886888641632, 1.2772619001495111, 0.8118024828504995, 0.3362208832814171, 0.5171138682674185, 0.7326831960905302, 0.8014977564257619, 0.9130038769119608, 0.17905344453320998, 0.16130267940108148, 0.27515585564716877, 0.030500952523708567, 0.2031070647362401, 0.8004948411837811, 0.5197331721916809, 0.5195032315624432, 1.098137353941953, 0.7228539221540564, 0.9566483576376551, 0.876268352771211, 0.5421746688129367, 0.5720424023256128, 1.3314287840321146, 1.0467769368944193]], [[0.46922137581541495, 0.5757633895199199, 0.574399421181638, 0.5654823599069175, 0.12675280950953452, 1.0571888722223366, 0.384578441183918, 0.18345548939620182, 0.8331338654842804, 0.7225602092712267, 0.3395905572918089, 0.9889075645688861, 1.347931756391245, 0.7695540779102998, 0.34231722095449646, 0.4699594325821732, 0.7628343149368284, 0.7150374587857663, 0.9489334284467911, 0.19141215948562285, 0.17900094782894538, 0.29351370009482286, 0.02927607382611125, 0.19071731698541622, 0.7951207595443397, 0.5603796186704535, 0.517313185060549, 1.0671339791912848, 0.7383546842924095, 0.9581331312738011, 0.9021822682374694, 0.5387159058393473, 0.6794798093149885, 1.0517728817718326, 1.0319797793370051]], [[0.449545792153677, 0.579303039550766, 0.49551077242852154, 0.6697078228742761, 0.17437278999434447, 0.9953395112141994, 0.3816096723841, 0.29707264669204153, 0.9114237894813937, 0.6640593599444238, 0.3403762370214139, 0.9417092389618549, 1.2986696913093212, 0.7069684526816431, 0.37823646389061993, 0.4467927157101347, 0.7270787996284551, 0.7188506917921504, 0.9351638959021858, 0.19415778625705357, 0.20921871694408173, 0.4078823667976036, 0.017995295020459958, 0.14997334282678113, 0.8155745737471751, 0.5213809979832097, 0.5263717169927671, 1.070565434858981, 0.7505950712849079, 1.0405534043343723, 0.9260562064958568, 0.6231576372699487, 0.7558553567252708, 0.8908315084309115, 1.033132898693156]], [[0.4448357152480368, 0.571340237044299, 0.49016663817915307, 0.8286541427061477, 0.35954807240750475, 1.0028836732825834, 0.3587938167179598, 0.48435069274476883, 1.1250642579940175, 0.6257501108898191, 0.31130834802868246, 0.9887361991428343, 1.1806480467127618, 0.7230078934848909, 0.34177719772610515, 0.4725336860572723, 0.7530384892640324, 0.7142002875684024, 0.9422421937271401, 0.3880957323080507, 0.28617866850178336, 0.5792541184797606, 0.01551239357632261, 0.12446966495496484, 0.8323544117687884, 0.3085948592329947, 0.5335028647387793, 0.9119155648751099, 0.7668482480590901, 1.1486307441896013, 0.9690656634741608, 0.7410389972528659, 0.7913653224153379, 0.8721011335611337, 1.0213827314988788]], [[0.4235897435722715, 0.5859272440141099, 0.4856889539526885, 1.0149721164526897, 0.3168601555373034, 0.9125286763799622, 0.3726772794904167, 0.6392994034661619, 0.9972209289562708, 0.6046405237501353, 0.24732901934566243, 0.8400766975650871, 1.233054689288011, 0.8537307464812334, 0.3367551594284156, 0.46659293332864293, 1.1624203502147763, 0.73085916429604, 0.9106677893204957, 0.4131085969100688, 0.24346585098837023, 0.6035305851281378, 0.015221233918045161, 0.11682026274355836, 0.7784746352746955, 0.13463044568103424, 0.5489765033598402, 0.7468921965132698, 0.728407771112179, 1.1506419349168342, 0.9549765675863299, 0.7393778730867737, 0.8125835026006967, 0.8647546205261507, 1.0323517369360422]], [[0.3861667057993827, 0.5678615359446005, 0.45813387168650527, 1.112592748850384, 0.3884152529638067, 0.8322781234460938, 0.3853132501768977, 0.805794411184291, 1.0508244063838772, 0.5821095508768541, 0.22621007497763845, 0.7891813511506144, 1.0991630730672781, 0.8622355851021253, 0.3371344860113582, 0.4553824709673907, 1.1471712780457566, 0.7434355790298661, 0.9806135791152797, 0.3654927880084693, 0.20075320987990147, 0.49168228086840654, 0.015636789505500882, 0.10357729419455969, 0.6237999344791552, 0.11492938041138168, 0.5693991455277775, 0.49101529973504643, 0.6244191639415136, 1.1399073140862896, 0.8255444393512171, 0.6608455701268443, 0.8000021181852587, 0.8836216191481326, 1.1016832674433585]], [[0.3834370187579212, 0.5649797025543186, 0.44933013032083424, 1.2079551923441614, 0.3981408030019149, 0.7882386558625938, 0.3857464045795667, 0.9191003071547377, 0.9521747221965582, 0.5713329892127716, 0.18460724130708855, 0.6783112906960241, 1.235253851119432, 0.8654816042184242, 0.3461333568896554, 0.4278525843278427, 1.0634487289981887, 0.7810995078411767, 0.985249902019153, 0.27494679361540486, 0.18593764155352177, 0.38511130888560335, 0.016079447514004773, 0.08429613135049452, 0.5722584822688739, 0.19058495914388762, 0.5704418139364624, 0.4094905228513324, 0.43741014275152995, 0.9975449820892006, 0.858820124300839, 0.5421909815126645, 0.7807385216140206, 0.8712566852890604, 1.1284982493735527]], [[0.4019216709180878, 0.5570347452468811, 0.4229482804817886, 1.246530878237638, 0.5430943275563592, 0.7601548913709621, 0.3852752090489886, 1.0779280864835052, 0.8987701305669961, 0.5976635702309671, 0.1624098888261724, 0.7372041474764899, 0.7789222874464117, 0.77996793407091, 0.3600367049815145, 0.4103863674746807, 0.8926047402010915, 0.6451719112306873, 0.9153218464279874, 0.341563258105454, 0.16528370771675763, 0.331753661700428, 0.016113051635742913, 0.0983716917624225, 0.6894941218613415, 0.2604545709694821, 0.5706111598034757, 0.3582436862356055, 0.41215435612907597, 0.8178042949520401, 0.935124598301915, 0.47567001165178213, 0.7196288182243056, 0.8286519392992469, 1.0505308967767735]], [[0.3858075903858046, 0.6360597130287623, 0.424948792921684, 1.2938168905642686, 0.7408340137082653, 0.8168607357996693, 0.36275607655998504, 1.1294267912457352, 0.958800767399894, 0.718266136360714, 0.15060950240581963, 0.7235002910952419, 0.912680360759539, 0.806025461840613, 0.34622410701875206, 0.4675165535121616, 0.893116462286889, 0.5901336479979644, 0.8634697122274655, 0.4773447469458032, 0.16151478443850026, 0.3142846565411851, 0.015749604371819895, 0.09079221639956642, 0.8596421202040515, 0.29859199808702175, 0.5740608491216281, 0.49624681919192437, 0.41345655756569377, 0.6984428168619653, 1.029407743107901, 0.49282279707488297, 0.6670661396244966, 0.8058375081169763, 0.9442507276359524]], [[0.3912998005747453, 0.6056370506096128, 0.3818602718171929, 1.3623368437254684, 0.8801492066624615, 0.7871888172651306, 0.36956063553187113, 1.1735456598858367, 0.9843152023838441, 0.7286792437394881, 0.12327965639425936, 0.6379819899835542, 0.9762084186270126, 0.8193582901133643, 0.34924680001705277, 0.4776713774849428, 0.7281315769822734, 0.6158054771512255, 0.9655617380820805, 0.49020091215938877, 0.19144839740621183, 0.313239904330956, 0.014765007164060196, 0.057387464392013005, 0.8979714735693252, 0.2911893111874322, 0.5512437578262235, 0.5754871194182061, 0.4177648614243487, 0.6156874416147128, 1.1056007391678762, 0.5745249058776174, 0.5242191798907527, 0.7779285599043213, 0.8874849735972195]], [[0.4230666490214006, 0.6064176848643429, 0.39164000106964997, 1.4556894412564079, 1.002956871335006, 0.7637665332896976, 0.3958256462777894, 1.1854299963484796, 1.0168863775502595, 0.6495773227030724, 0.13033827238937695, 0.5860772112574764, 1.056403113029179, 0.7852405549783095, 0.2836019709369544, 0.5019258664051455, 0.7092315024123431, 0.6047865813247251, 0.9252648311807949, 0.4775993992004538, 0.18336409973866025, 0.3190649207380937, 0.01393734876959224, 0.048388434616083054, 0.8384474044395953, 0.30091978523677726, 0.5372323860242514, 0.5481141347783447, 0.37295665347539764, 0.5368962773749308, 1.0769351263859952, 0.5724861117862706, 0.4336186971514198, 0.7497541100713943, 0.8470733612505276]], [[0.4458024362353803, 0.5994157100162145, 0.3969512361146541, 1.5335062415997638, 1.0996987725131082, 0.7766687044193472, 0.4505660411094141, 1.2185940946176463, 1.151700349600016, 0.6404580119572133, 0.19329533957340989, 0.5751727824240066, 1.1349124634603807, 0.8041864439897703, 0.2802467562824019, 0.5246597015136942, 0.7625629639392365, 0.6257774756872159, 0.8835159956019458, 0.4598550712419787, 0.166295931940414, 0.3317282582983672, 0.017009735207250173, 0.05817143607421682, 0.7521916570861469, 0.3302557234974131, 0.5292518667196398, 0.43604040081961576, 0.4051139834656475, 0.5364769514175799, 0.9709026773574182, 0.6921004282946659, 0.4220955496968068, 0.7178216938634157, 0.8453269377898147]], [[0.6749848286621067, 0.5694407524053972, 0.4081255969113278, 1.5358100159322434, 1.0522525357532082, 0.8036505801905379, 0.4208122763925627, 1.2874009802837025, 1.1651735049058032, 0.5391421862257894, 0.24161138945645344, 0.5886625756599045, 1.1849795857320093, 0.7091731092535504, 0.23057980320251603, 0.4209268321750361, 0.731402591583035, 0.6369572875092256, 0.8136020915987242, 0.4707967427051498, 0.1665071802930796, 0.33962189696760586, 0.018159557813655195, 0.08082804793692971, 0.751256438471138, 0.3134429223241501, 0.5014891822116534, 0.4370143687161234, 0.379690088456606, 0.48405811222812767, 0.9666603479740589, 0.8039376477821457, 0.4272464003362514, 0.8400869081256619, 0.8178256577711718]], [[0.6781469661388, 0.4827348399945336, 0.4356150966104534, 1.4635536439718815, 1.1095026462986821, 0.7104484765897681, 0.4062879139947005, 1.2526178746188152, 1.165830475749754, 0.5373551274731582, 0.282098463493734, 0.6017269531539243, 1.18692390528699, 0.7121287287681901, 0.23088716449084684, 0.3464166023662244, 0.699234758699387, 0.6585689369642456, 0.9017852789431702, 0.4902742066299264, 0.1634372970205064, 0.3819571378886519, 0.017645916512566195, 0.0969921157043038, 0.7689929836305114, 0.3130520856810608, 0.4610394530774793, 0.5033115659244729, 0.4223593474278632, 0.5134466672619047, 0.9572076422848856, 0.7722523190372572, 0.43058197394587816, 0.8449354022004545, 0.8076978602000958]], [[0.6985897845310902, 0.5364775154591193, 0.4270398628968683, 1.4317745505167705, 1.1625593807814643, 0.6722260322915671, 0.39973089715043597, 1.1989728055543036, 1.2606655389874613, 0.6057291474400097, 0.279794333507102, 0.6008836798858781, 1.196406327436222, 0.7266202824762745, 0.2303352742031567, 0.34561873791796405, 0.6696856259310943, 0.6657522092814103, 0.911907751911512, 0.5048924118633382, 0.16814925635161418, 0.39617079846319664, 0.017213129129063844, 0.11325572297111527, 0.7725683853348149, 0.3234807394287759, 0.4054427400951901, 0.583342383891224, 0.5059927722061112, 0.5130128871890086, 0.9263154209899728, 0.7241657972018827, 0.41674688255927633, 0.8093985709209325, 0.7805933747473839]], [[0.6970766096476362, 0.5675609124421559, 0.42846928446838206, 1.422715507046242, 1.1234658843522285, 0.6733530612985025, 0.39269756004822165, 1.1101653301310985, 1.1647202203278524, 0.6798739834315662, 0.3001001085810274, 0.6071561456981529, 1.11826175999169, 0.7346815622239643, 0.2459393051428184, 0.3641573014005608, 0.7024604238881594, 0.6866684218992403, 0.8830135636645331, 0.4940499539762785, 0.17170306201669272, 0.36554754987237614, 0.017831852629676598, 0.1633359478960354, 0.7628728301210569, 0.3518621148579568, 0.3660431714093469, 0.5874300285198257, 0.617676732999827, 0.4421444611891434, 0.9068131440396034, 0.6241521275220702, 0.3921586046104854, 0.7865717097092472, 0.6973723923117167]], [[0.7087138397809314, 0.52333838483455, 0.4059421468015247, 1.3866289865042998, 1.188481797293887, 0.680204986475331, 0.3834579600386712, 1.0375361354599384, 1.0963349695557372, 0.7699384507841376, 0.32732808774616884, 0.5982745351397379, 1.249970013097243, 0.7521043276669921, 0.25130841938342385, 0.4029356678008009, 0.6184483657789785, 0.6507537791441955, 1.0564798625380338, 0.4584843438261221, 0.1860051706144196, 0.3822336577449155, 0.0199893319429468, 0.2508032368050147, 0.7478020337018302, 0.4117441575200087, 0.32438500322785613, 0.5696697145249932, 0.7473796088443728, 0.4298072896413576, 0.8949486688665522, 0.6751871215484155, 0.37651449491109085, 0.6970263343992555, 0.6537652549618449]], [[0.7195515510849955, 0.5500339385626191, 0.40977707883425785, 1.343449694409867, 1.2913481609327544, 0.6813926047064289, 0.38328089035422075, 1.0396487250749757, 1.1207991597668803, 0.9144007959786304, 0.35052377509028987, 0.6045858697272136, 1.185857822480999, 0.7361374855630191, 0.23614761033343878, 0.4415809541863688, 0.6560634542787747, 0.6540428787513386, 0.9758867109973932, 0.47508967146025155, 0.19302917380653456, 0.4255693993555152, 0.02320145186704882, 0.3544118606449334, 0.764814962369611, 0.5346625038663784, 0.2820437230590879, 0.5510238968875598, 0.8488160801131859, 0.4166673954661789, 0.7974157614681023, 0.8401534763676246, 0.3829547547144594, 0.6466853959045897, 0.611942390277322]], [[0.6571808721217895, 0.5201201125712709, 0.42342479193909566, 1.3088672792600664, 1.2253950010975274, 0.6813588766083627, 0.38273429102750084, 0.9881912373069914, 1.3190224863860065, 0.9580702751809173, 0.3901682065679517, 0.5996152293992869, 1.1711363141624833, 0.7161739300423204, 0.24619394967142594, 0.4381541963946469, 0.685713599064257, 0.6916213233792036, 1.04180087186805, 0.4445253710730021, 0.19383913207068432, 0.4280136654611276, 0.02842402247576191, 0.4574721879986091, 0.7028036840201345, 0.6460221283364247, 0.2748677389024934, 0.5282816314326467, 0.9799395079244486, 0.45279220959295885, 0.4791533991879739, 0.9903157868194263, 0.42123818769915855, 0.7319751388572793, 0.5527155428903656]], [[0.6036434428695407, 0.5490144407223332, 0.4168578192147393, 1.230333095194104, 1.184971966881094, 0.6693189141986267, 0.40554251431603794, 0.9647789633991661, 1.4022276592690048, 1.0788412156963014, 0.3722105279517878, 0.6019353379944614, 1.1630667867189068, 0.6173571218515963, 0.25858385823701774, 0.41138399724632857, 0.6747148483125862, 0.7205985059001428, 1.0127879452493969, 0.3706610706101195, 0.19402225978560736, 0.4523038097287914, 0.06870701594555592, 0.549143824604841, 0.6247708635294138, 0.7186146567281279, 0.27695149287390125, 0.5496484154560315, 1.0523587391814457, 0.5193351154216239, 0.2077332575734452, 1.1009056334996594, 0.44135286313369493, 0.726302962534692, 0.5212601210789751]], [[0.41640871459254614, 0.5330698067775176, 0.40046662501334984, 1.0863618128625507, 1.025259999547399, 0.6397038195226289, 0.3993877315159599, 0.8105589230371865, 0.874620442754168, 0.7447759374363192, 0.29155191013859033, 0.6086116104525954, 0.7417806776520248, 0.5902388278238866, 0.263728198829412, 0.41444949369648554, 0.7079508279402894, 0.7190294001784765, 0.9024289937574099, 0.3829005699090672, 0.2042334743498734, 0.33594978225369676, 3.4870886584447467, 0.33603376448190847, 0.492087509474773, 0.47535534624841547, 0.2567528994560272, 0.5460983918020939, 0.7750295802932445, 0.4960581391489437, 0.15334401944864479, 0.6454903647799768, 0.2936860178083459, 0.4647445573099196, 0.3930513583653848]], [[0.39285956250411136, 0.5378494820827145, 0.4039575582043541, 1.094074303033617, 1.0401147516687377, 0.6028327196154661, 0.416706086504402, 0.7975037158811638, 0.7738830001751125, 0.7204791020252255, 0.2811524835922151, 0.6202543599472977, 0.6007772210672558, 0.5839678090558993, 0.283695535268393, 0.3754300140486504, 0.550041756080777, 0.6968287247171274, 0.9049387934057782, 0.38458034806817476, 0.21963656348399335, 0.2570152416366207, 3.4870886584447467, 0.3158190559730226, 0.41499338709566846, 0.40020686495717306, 0.2877264480010985, 0.54229625673113, 0.7926082962258845, 0.5489157867846312, 0.17136161964991772, 0.6253147877147116, 0.34147990708412995, 0.3786070797815172, 0.35097922505616497]], [[0.4099426440255691, 0.5851045666196538, 0.4380392329133215, 0.9891589691885336, 1.0328564794664943, 0.5937325909690176, 0.42770414246719235, 0.7867791080491248, 0.6275448286090819, 0.6075060731052588, 0.31467500938210946, 0.6155700741919093, 0.5611419007649121, 0.5512036422581605, 0.2579272622499032, 0.3998078111605081, 0.6081245009336282, 0.7038979043330706, 0.8555649545108241, 0.19622357271379565, 0.2431547283885669, 0.2391596699512127, 3.4870886584447467, 0.3181371292210121, 0.3213782189475063, 0.3972700074031271, 0.3109217097905346, 0.506773212318482, 0.8129811650210563, 0.5965571965523023, 0.242837804682883, 0.6138610824756092, 0.38364138879532406, 0.3630700070125562, 0.35474026724869695]], [[0.4167152584615962, 0.5701418829475744, 0.4654417215597396, 0.8265438921056614, 1.0332015994105859, 0.6202609082404699, 0.39732712943405185, 0.7150310373300306, 0.5438136541618546, 0.6331358083044383, 0.31891361528285317, 0.6094278766042799, 0.5771795212744244, 0.5498985228389234, 0.22989533846951174, 0.4047973677142148, 0.6096082389496689, 0.6937134018282157, 0.7907070065898772, 0.17199789046110142, 0.24013946305441297, 0.22748160514943225, 3.4870886584447467, 0.33033943479819533, 0.27623180603278086, 0.4367062670773921, 0.33280620070029043, 0.4992895705785754, 0.8475621945051733, 0.6360214975932732, 0.2586339689054722, 0.6063151078341715, 0.39844683019404237, 0.2845181944992254, 0.3545859621134556]], [[0.4101409701255312, 0.5901694060475681, 0.4954956297286076, 0.7158138557607359, 0.9888341848375113, 0.5859094142265722, 0.4614889049499984, 0.5887201649270856, 0.6278741091808858, 0.7938030789680899, 0.2932836669006205, 0.6104216884923861, 0.5242134312796333, 0.5519009471538757, 0.2393398844059468, 0.39636897109383695, 0.5984556428241325, 0.7320195537682745, 0.7691730476475631, 0.2091492436655783, 0.22323965990087308, 0.22069050819853528, 3.4870886584447467, 0.39353356789513994, 0.20419076272076192, 0.5133337823264754, 0.3353508072868534, 0.4592952584130374, 0.8824464207820495, 0.5422591786382434, 0.28407620357173813, 0.6148069322704198, 0.1933974145913417, 0.24640403194415095, 0.3300196182083378]], [[0.4035501328178399, 0.5704738536616882, 0.49093241420455963, 0.6197073966760545, 0.9519814207161219, 0.5815853275194751, 0.45437610317576116, 0.5826259577319481, 0.5908241909785354, 0.8127476333478424, 0.2680622199861158, 0.6023127766592712, 0.5250933246143378, 0.5564651631540309, 0.26244528902440256, 0.3774960673705203, 0.6391031002838985, 0.7495910396695086, 0.8107294572167778, 0.31373562751354306, 0.25183463056308925, 0.2190382308813058, 3.4870886584447467, 0.4072922923880451, 0.18828845841212616, 0.5755682888797977, 0.3394192161678984, 0.46237241432029985, 0.8804415192904134, 0.5219034202973477, 0.31843431981196446, 0.6161436272423051, 0.2874796643753399, 0.2520219518486013, 0.32599977161753396]], [[0.4636898426718811, 0.5567848978969897, 0.4935148518626892, 0.5947875112030787, 0.9270230440346945, 0.600994426525331, 0.44443389329999927, 0.5866019002150817, 0.5648723283507678, 0.8054308368371481, 0.2576483536712023, 0.596211647427316, 0.598345803709611, 0.5517366434540895, 0.2303592372383219, 0.40512350910957207, 0.6089648749674049, 0.7119588290758768, 0.7854659605927756, 0.32706910537551726, 0.2944577250157159, 0.22953838487733352, 3.4870886584447467, 0.43019559968131293, 0.18550072635754877, 0.607903361786048, 0.3452953758364479, 0.47759758409385583, 0.8748011095255426, 0.47823447714950945, 0.3529266380551803, 0.624770420590776, 0.2308326990848577, 0.28816143527038784, 0.29508445212562084]], [[0.4619346498067123, 0.5377160506595358, 0.5020166137018943, 0.6163631050082466, 0.8404822427177044, 0.5968879653140338, 0.4636501663341577, 0.592309296655629, 0.5497169037718715, 0.8562568288430725, 0.22295059069191303, 0.6009175757251471, 0.6356799957152762, 0.5547072489552625, 0.21501157260681972, 0.3940774490762743, 0.6181778076078763, 0.7163176322996786, 0.7827912709451754, 0.2727070416410145, 0.3017283462583099, 0.24970342715152327, 3.4870886584447467, 0.46381734985562656, 0.15646131028328103, 0.6149435004586132, 0.3244775788330429, 0.47952158197551187, 0.9188821286959876, 0.44730268061071327, 0.4045624892691506, 0.63711418314396, 0.27773957603250354, 0.30660724224570335, 0.22489543975581316]], [[0.5554086351595993, 0.5282446477406407, 0.5146135468222608, 0.6474458132468199, 0.8076524495052999, 0.6342192099559791, 0.43116232881532834, 0.6317054681705961, 0.46367206240299463, 0.8034838621067963, 0.17768899854457645, 0.6015749310397659, 0.709287917394761, 0.5946268370607497, 0.23990753161034872, 0.3977839191377157, 0.5983768405271164, 0.706281092162465, 0.8610724013021565, 0.2074894807432736, 0.29628318899683437, 0.28404719888594754, 3.4870886584447467, 0.5198908344646267, 0.1721441606380611, 0.56768386478322, 0.3003779744488865, 0.4702849052556346, 0.8281896089359249, 0.4329020591083437, 0.4441511739033488, 0.5990023810823016, 0.32472995868991766, 0.3494665619551923, 0.15066808512724778]], [[0.5571029149248157, 0.5133082886018752, 0.5204843217565414, 0.7711588457378702, 0.7757207169585263, 0.6233294889105738, 0.40438405488580637, 0.6003981608045891, 0.49329980275904733, 0.7701395227777235, 0.16151642242964914, 0.5922269565467247, 0.8188241415403308, 0.6367605306508848, 0.23390865890366594, 0.41395045707800193, 0.5611928384651675, 0.6652146549562459, 0.9140927871328454, 0.15622769399462255, 0.2611937624577248, 0.29977336826698425, 3.4870886584447467, 0.510080737963077, 0.1914358690832783, 0.4964731642636381, 0.24308882984852798, 0.44971372441807655, 0.604074524943525, 0.40164286281344597, 0.45333399368892324, 0.5408008639550252, 0.36714520707461207, 0.3246621167917635, 0.12000243529343035]], [[0.5135603200325538, 0.49549359084786543, 0.5044807403497553, 0.8228566861961171, 0.7555972055505752, 0.6164659881867169, 0.39581871569052446, 0.609536484379547, 0.5823828254282872, 0.6625194855063434, 0.12853763595858125, 0.5756587926737897, 0.8987743896871168, 0.6893204415174872, 0.22429726883541615, 0.6145757580552657, 0.6357753433444115, 0.6349214739116562, 0.8627524970490443, 0.1753045543163093, 0.22664278107666186, 0.2468759099257019, 3.4870886584447467, 0.4184397065130229, 0.25230171440294563, 0.4609424813319904, 0.19493249191565165, 0.42318741254304865, 0.532903053536557, 0.2755441944471235, 0.42936578094068467, 0.5687143034590552, 0.3907316737740998, 0.3276736442579699, 0.13473945751979044]], [[0.5112903219745114, 0.46419173528088503, 0.49118116351743146, 0.868972508033379, 0.7235003794392063, 0.6113925676017766, 0.3957455838089439, 0.5824070730269725, 0.6843710780743423, 0.6161933717152458, 0.09650813826069658, 0.5711439824776784, 0.9052015922839964, 0.7417885160471492, 0.26335028615422684, 0.6169700095916978, 0.6012557230543565, 0.6226604983318287, 0.9405587813491006, 0.16216255741874663, 0.254885493936477, 0.2207817191906589, 3.4870886584447467, 0.36384369783951515, 0.31796134072751436, 0.42801319377856545, 0.2007692390205923, 0.46081676119460346, 0.5041182189325352, 0.16154801690297388, 0.4121869070403712, 0.5660654019823679, 0.4467582082249462, 0.3410471679232061, 0.15382996454100883]], [[0.48997706831392046, 0.4261902917190329, 0.44068843158394044, 0.9252934150506333, 0.6506656813610934, 0.5925714779284852, 0.4556721194757763, 0.6130044573155883, 0.7667739296493628, 0.547937947435698, 0.10409121611601163, 0.57876762230835, 0.9552046532457521, 0.7590277705938416, 0.32403398420458307, 0.5676477522287584, 0.5989711188601163, 0.6145928443616773, 0.9589146981356934, 0.1707008580953971, 0.23916432218606754, 0.22611141044657046, 3.4870886584447467, 0.34607111217085906, 0.3506817737065188, 0.40690405787051737, 0.2043005863823823, 0.5653542566224953, 0.45980546446629444, 0.08938474091837785, 0.42041036919116515, 0.5460853136852821, 0.45677080338179227, 0.36902366180444096, 0.1995517781337743]], [[0.4869996988011994, 0.417749228548751, 0.44300787611863457, 0.984981990822235, 0.636220966925385, 0.5961982505670543, 0.46953199352336467, 0.582356554095431, 0.8516002766437188, 0.4857064650989526, 0.08301406301625422, 0.5670080448177197, 0.9315214787040115, 0.7636056511605467, 0.3023916684729956, 0.49728228085489457, 0.5821867064654231, 0.6089847721945416, 0.9262644344809627, 0.10675002477329315, 0.22186524465620483, 0.2593994004747524, 3.4870886584447467, 0.30665375271597206, 0.40858260383032075, 0.40753773401898186, 0.2615199028596112, 0.6615664926933507, 0.4358179993755972, 0.12812552243378172, 0.49392982577339145, 0.564549585529355, 0.4722409093533592, 0.43601345595680235, 0.25161758213007923]], [[0.5296727619684374, 0.46402322953236286, 0.4400983987417826, 1.0522660919459312, 0.7025933849114396, 0.6103655179770366, 0.4187197764195505, 0.5713572265707761, 0.8481945475748058, 0.5242792966398316, 0.0576186956702328, 0.5604342520784183, 0.916766229196204, 0.6729848335947243, 0.3018269644361554, 0.4502673252642191, 0.48232246507426985, 0.5668342996464069, 0.93737636458177, 0.11349574328281492, 0.24390627923829183, 0.23804849788258256, 3.4870886584447467, 0.3039780660474789, 0.45166316415899427, 0.4463901216837079, 0.2552256013608274, 0.7014030522007971, 0.3883127709724429, 0.20367724555820818, 0.510521938080658, 0.5608317541634866, 0.4759843905898611, 0.5082261418071123, 0.3229859111627324]], [[0.576483534461347, 0.39529303167718866, 0.4535515413793319, 1.1207683110385713, 0.860261424437622, 0.6177814517230952, 0.4044712476947277, 0.5589222977444844, 0.8256218908628786, 0.5583528643689867, 0.03228390805777061, 0.5764067592777774, 0.8894437243023263, 0.6800825754299193, 0.32082162266002623, 0.4665145033196548, 0.4495061734802205, 0.5631617436968782, 0.8176118018249943, 0.1195201892703418, 0.2734161377825313, 0.2257683704426706, 3.4870886584447467, 0.27236564205271335, 0.49216865365213813, 0.46986669522116903, 0.25321760097322055, 0.7299578535350577, 0.33562151595451234, 0.20528529326602524, 0.5520168354376034, 0.5320480842266451, 0.5041659061870503, 0.5325623915561511, 0.4205155797041853]], [[0.5945475206395227, 0.40580899053914204, 0.4390505546583884, 1.2666774338605522, 0.9095695442564176, 0.633267889108176, 0.4334527479174407, 0.5501402974363164, 0.762489439400924, 0.5783014211669922, 0.052748867779534726, 0.5996481190920369, 0.8907827962295477, 0.6133365606638634, 0.30170846827277487, 0.47103901078908883, 0.4690927358818302, 0.5818660372190418, 0.8347588707854091, 0.1364464558018615, 0.2790591990410815, 0.24389343685761028, 3.4870886584447467, 0.28769570588267646, 0.5485389852172937, 0.4699201861777463, 0.2561751903753796, 0.7108414341025036, 0.32619284700949125, 0.37662295949926156, 0.55748194950955, 0.4658789851939871, 0.521746212231555, 0.5454133268528967, 0.4217124870040352]], [[0.6402164826086117, 0.39676694415270186, 0.4489054118890067, 1.3168342410853255, 0.8987024779294654, 0.6258045993896986, 0.43385592032425235, 0.586863355601603, 0.6942027500906894, 0.6914472725701185, 0.04797733701333427, 0.5123540012719581, 0.8852634713767864, 0.6089428574616851, 0.3230415461866163, 0.5282145182666211, 0.5238129525007285, 0.5846804270025147, 0.8694937801243661, 0.1529478824080505, 0.27763362645454304, 0.2246189920624008, 3.4870886584447467, 0.2782157444069978, 0.5577297358323978, 0.4454092261319805, 0.26657466678126124, 0.6814584093758993, 0.31105422108845315, 0.45251499460176553, 0.5707171912622624, 0.45175656517855306, 0.5280127479786471, 0.5939277445983928, 0.42658100029980894]], [[0.6656158831537589, 0.39296412240030754, 0.4437908871254055, 1.3617785467613912, 0.8575529324653932, 0.6237387872402134, 0.5092048528906847, 0.5545771991148778, 0.64313959906591, 0.7136527178323833, 0.012099262906631169, 0.524213227085095, 0.8810207235994127, 0.602675337477075, 0.4023026737113212, 0.5147729434148135, 0.558651514307268, 0.619033304330693, 0.9172540518383563, 0.14376053429927077, 0.3026102146035903, 0.22190615336526734, 3.4870886584447467, 0.2800748240755554, 0.5853944982023302, 0.4368213617470375, 0.2733585603386197, 0.6929296098076771, 0.33891368064201666, 0.5698327736748902, 0.5943628673554068, 0.4345937959538346, 0.5222656677513562, 0.6088512425215806, 0.40085585140318714]], [[0.6128068054549758, 0.4021764972283946, 0.4260242647213534, 1.3823585586091671, 0.5984545522716828, 0.6305327839088685, 0.4635195231404103, 0.47775532732848036, 0.6875063762268293, 0.7052423499027057, 0.005086153939402187, 0.5926320948683332, 0.8759502989239987, 0.6502741619816331, 0.41332055161964387, 0.4724479774642236, 0.5021757549143595, 0.6249540192679199, 0.8958696202901073, 0.14138303123780013, 0.295771394305273, 0.20368187670645885, 3.4870886584447467, 0.21985453408547517, 0.6185626727634319, 0.3919962231796619, 0.2852421315578605, 0.6886280507375729, 0.3466701851446339, 0.6184447577758578, 0.42428568521107984, 0.3819016161406848, 0.5198502661898252, 0.6351648368390326, 0.3556934882544782]], [[0.593419582660412, 0.4129125891831227, 0.42526597741539174, 1.3827014086170855, 0.5133567504337885, 0.6128604133580594, 0.4626234034288895, 0.46897384253169, 0.7503080963666029, 0.7496811946671613, 0.007617330056489699, 0.660315218947419, 0.8731179350163195, 0.6786851394128629, 0.47029836986041745, 0.5234067233241136, 0.4819814436340265, 0.6274753275727699, 0.8497699794676712, 0.14926523437549852, 0.3245509411353654, 0.2012399028304808, 3.4870886584447467, 0.18662096120854416, 0.6290206677593073, 0.3321201140804947, 0.2757444273681121, 0.6628187029062018, 0.3829887457810909, 0.6350349748443509, 0.3726206285961685, 0.4117785294494899, 0.5118770855161037, 0.62265749491224, 0.3463653337996445]], [[0.5692821363826729, 0.40375859517374213, 0.3955031980678103, 1.4888361178443068, 0.41544394469791956, 0.5974515009362893, 0.4806509250940931, 0.4506527192553068, 0.7460973686054488, 0.7382847693178437, 0.009271632947378072, 0.6542828730839999, 0.8534708484981852, 0.740815491883658, 0.4373531084929858, 0.49599762199086395, 0.49655524436830123, 0.5707383136790106, 0.7776715013793052, 0.14991056029003996, 0.3110082400413677, 0.21272344314648922, 3.4870886584447467, 0.14363223631128363, 0.5874690084418241, 0.2132439893107112, 0.2375238830806005, 0.5643349575126796, 0.43972292644590183, 0.7026239630925956, 0.3181439644248218, 0.46423361911417393, 0.5156851301814366, 0.611824450459506, 0.35986905016500614]], [[0.5992742509754989, 0.32997590905371443, 0.30860639712137256, 1.4286758956967216, 0.390210634369613, 0.5628010002219332, 0.4944884801095246, 0.518279980985855, 0.7107214087664105, 0.6978456060942024, 0.013705591207061149, 0.5954505341006002, 0.8247720598145779, 0.6216041147710988, 0.4360673892571604, 0.5135800108454827, 0.5142433738028468, 0.5639017442845766, 0.7735345315553018, 0.2043400628939065, 0.3146853048887196, 0.233747751907048, 3.4870886584447467, 0.12854426619795278, 0.5785630893982103, 0.14629328270274825, 0.2262040751222845, 0.4972095691140186, 0.48888089538027696, 0.7604295295612673, 0.2932901960828356, 0.5215388647061376, 0.5174314637659928, 0.6112400734781356, 0.3722057489309282]], [[0.5922748232611293, 0.30286889413891455, 0.27503601367291797, 1.388656896037074, 0.3978814975754891, 0.5901421422878229, 0.46324489126486557, 0.5544310815328987, 0.6730798672824188, 0.6507780420063973, 0.024861412807690813, 0.5719200039930493, 0.8525480012871143, 0.6734096993805538, 0.3532991808840621, 0.4924945445650236, 0.4500126243356931, 0.5454414589498795, 0.7750005042970931, 0.23949456324053717, 0.36005324133040056, 0.2227977770719895, 3.4870886584447467, 0.1373500767472975, 0.7127882572846997, 0.13029771955188613, 0.23865183741597096, 0.5561047468679506, 0.4843204974578432, 0.7568017147087449, 0.2670907213012531, 0.5964915447183372, 0.502775477328303, 0.6097207937417247, 0.38807011847313405]], [[0.6007467251527678, 0.3282184504250179, 0.3207311101402446, 1.4801475948131295, 0.4021824857836054, 0.6063272250185069, 0.41690338247235575, 0.553376507030289, 0.6343629229811529, 0.6381962991570727, 0.1615778283000935, 0.46822964865175576, 0.8875983772717904, 0.6029569339292872, 0.31098483224883744, 0.522794505337136, 0.4302932699892734, 0.532232653651834, 0.8228383744842656, 0.2358311527771832, 0.3561489577603493, 0.23908011478844543, 3.4870886584447467, 0.18296039940840936, 0.745244129920674, 0.12609601751466604, 0.23951275699387, 0.5414891066446544, 0.4713298517128873, 0.7601822899549272, 0.2462955043786228, 0.7078115030586636, 0.4708044564447425, 0.5826132439178964, 0.41910070149623013]], [[0.5712949527849005, 0.29121714905824736, 0.2651587854558881, 1.47741827122946, 0.4098307802158768, 0.6115544769580852, 0.3997208372890698, 0.5811905161838965, 0.5833566840588615, 0.6636057272174172, 0.16363973009725136, 0.4882560721097241, 0.8748421282712939, 0.6104268875872326, 0.3358847434305977, 0.5642326585225462, 0.42447229541776343, 0.5350637385001283, 0.8509985567490597, 0.25405383703832407, 0.2704212840112821, 0.2638060012524569, 3.4870886584447467, 0.20522448521533898, 0.7821227148600355, 0.13068073942938913, 0.2661383650040262, 0.5173568425140991, 0.4941957829179585, 0.7548211971757721, 0.2060291385703106, 0.7399346450519142, 0.47366341023718994, 0.5809903898827763, 0.4270694316999658]], [[0.5406144823929167, 0.25509548715662633, 0.23113780725457322, 1.5136616426195513, 0.4397265359663708, 0.6179542819850188, 0.39510669256297515, 0.5821340437057698, 0.5489375761006625, 0.6939075089438814, 0.2261154425402762, 0.4754477437052035, 0.8465043286540945, 0.5648840970010203, 0.3482438269007607, 0.5534306704590859, 0.42706798850295136, 0.5418649847738868, 0.9068833665361389, 0.270491004882912, 0.24076419747814654, 0.31004253340148213, 3.4870886584447467, 0.2065534252217721, 0.7753015936788459, 0.15478438523124777, 0.3077047251437483, 0.43729613711950105, 0.5372713995644218, 0.7328854290311749, 0.21417518907607275, 0.7723099280507773, 0.4683751485399233, 0.599216438737377, 0.41747963076199046]], [[0.5397281272138564, 0.24815234366067906, 0.25278322721739155, 1.456154716871442, 0.4359706489076667, 0.6486159613756176, 0.38633294912710636, 0.6002084542988831, 0.5246187904938568, 0.711919396915824, 0.26148265616162597, 0.4718879185094136, 0.825088579428408, 0.5521945553077834, 0.3317972764363251, 0.5408649710752833, 0.48765198075316263, 0.5463277989638335, 0.9380567881203994, 0.28935732863122376, 0.20726359236883296, 0.355898908738634, 3.4870886584447467, 0.19619581428739907, 0.7799196354414166, 0.16610833925739255, 0.32068543771067504, 0.4483529242648642, 0.5500149388559886, 0.7179867752348006, 0.27870458081752747, 0.7362519576484537, 0.4717920385371617, 0.6122119158386434, 0.4505174883120964]], [[0.5284824104488491, 0.2662173498442105, 0.26474433367100947, 1.3831835253389033, 0.4799610451210432, 0.6526651520438095, 0.38184489204827365, 0.6374592238383422, 0.5791908945844985, 0.7634471472624289, 0.29193480779827513, 0.4643260554281494, 0.8109392904201818, 0.5673108942469809, 0.33737287685441886, 0.560179374448996, 0.455191140598607, 0.5323513106070507, 0.9248921536812416, 0.2982057486739322, 0.19367599879617842, 0.3847105972183146, 3.4870886584447467, 0.19033406665957064, 0.7254538379005776, 0.1931481700224727, 0.3244353749101293, 0.4417114946266818, 0.5316625169319152, 0.7389379722329336, 0.4299622607529311, 0.7410689226218938, 0.47635896619047224, 0.6145462648183885, 0.4999931292756971]], [[0.5603554888830544, 0.281121228563214, 0.21698107870714012, 1.3564188828178325, 0.5468008931778646, 0.6440131394416896, 0.3800184573244265, 0.6828081912636752, 0.6475835675354769, 0.7658050116941763, 0.301076173189907, 0.41730319792683157, 0.7406670847385597, 0.567943003331834, 0.34666749120831175, 0.6038243195648285, 0.4471750715335605, 0.5372565759968644, 0.9443097513024767, 0.33883056763582947, 0.20631783787659208, 0.36773930893132056, 3.4870886584447467, 0.19812029426740352, 0.6487369925217872, 0.1966359944027, 0.3443882966239563, 0.40976975967978163, 0.5005135374080987, 0.7597230571882866, 0.5698316148661764, 0.755034768071897, 0.48282137787793167, 0.6004381672825696, 0.5326972984748691]], [[0.5619982194477284, 0.36972075191093057, 0.346482380693538, 1.3273799928303731, 0.5424583625918042, 0.6397380409148286, 0.38081211611043053, 0.6941991214228733, 0.698786297156851, 0.750931375604686, 0.3172016451549627, 0.4664110543844754, 0.6597335305781542, 0.5898865569332961, 0.3497107549051617, 0.579979272133714, 0.5357221588929179, 0.5650193286130019, 0.8950164292474997, 0.36447561695281666, 0.18402461050254948, 0.3959810479653013, 3.4870886584447467, 0.190211121867152, 0.6678373827263973, 0.21230973154649127, 0.3449330127884056, 0.38092171381027773, 0.4722772786645746, 0.77009947727889, 0.5751675986047822, 0.7487583823686867, 0.49012504948528385, 0.5927037471315991, 0.5564371660666958]], [[0.5757441267522765, 0.3920480068242491, 0.36271015152884756, 1.2765077699680116, 0.5250197789506471, 0.6368884639803396, 0.3756851799591151, 0.716399694141344, 0.7891002607325236, 0.7929867544121936, 0.24881054121438267, 0.46056752792214034, 0.6713092995021042, 0.5815335539342803, 0.3492985458125064, 0.5777786226618916, 0.532914661947268, 0.5524781942104838, 0.9535561240079407, 0.382336742167682, 0.17977961545968993, 0.3994708257608779, 3.4870886584447467, 0.1693820590903669, 0.6885828354891097, 0.2202233366061795, 0.3429217083263696, 0.38873532622352125, 0.4520995337930818, 0.7682670059646878, 0.5355835925198751, 0.7577169761724736, 0.4892815448472228, 0.575071413523803, 0.5736847139786643]], [[0.957841868248425, 0.4975868260020999, 0.3894578237766211, 1.3823905094765934, 0.7739912439582088, 0.636630116185501, 0.40833580318787366, 1.2189593254861946, 1.3016348596255773, 1.3306415649043206, 0.2688129114957465, 0.42239322231297016, 1.1909195940420148, 0.5690278072481452, 0.3482619085768808, 0.6602482461632336, 0.503311549791114, 0.5679772151558862, 1.1213980057211475, 0.44466520284980293, 0.1910416379750295, 0.5363338896367129, 0.1657705162609946, 0.25809708487299615, 0.8118867777925121, 0.31465249539203666, 0.5530099844034542, 0.4000500384201312, 0.6461647458660223, 1.0342257866026823, 0.9048476704003974, 1.320040943138249, 0.8505032195225635, 0.7185089298735228, 0.7922565261670844]], [[1.0103638341202459, 0.5432233258469824, 0.37190013641010167, 1.285881150623744, 0.7276159806998745, 0.6419295034489759, 0.3916499112125854, 1.2446262730547324, 1.7015177690272258, 1.381906301185876, 0.26459964001048275, 0.4059336022409922, 1.3289975098585065, 0.5906166146508015, 0.37137322721210414, 0.683086110495781, 0.5425604987194945, 0.5699494420631613, 1.1806484287292762, 0.4324025605929763, 0.22079062143046266, 0.5061177480040298, 0.07619989423111845, 0.23296114318333488, 0.8998697193710573, 0.377481286661548, 0.5412366451609734, 0.4758249915760797, 0.8255387257296168, 1.0407497035300093, 0.94061126084159, 1.321772966239156, 0.8371010874018947, 0.7342776219028389, 0.7692632165931732]], [[1.0560275490688276, 0.5270975047217956, 0.3827146369222983, 1.2830969772181868, 0.6728572408241256, 0.6667644509011061, 0.39113123010008727, 1.2386927349089454, 1.7495498574135584, 1.199478779605012, 0.237005595526521, 0.4386848098791467, 1.324472964105137, 0.5901224283965276, 0.39178435786072047, 0.6711234134209125, 0.5883138747430865, 0.5707883808523109, 1.0761218015717309, 0.4316840617168267, 0.22878064921761876, 0.5242913539827397, 0.059636910069719835, 0.20310063465075123, 0.9151844610583811, 0.36518962345962, 0.538847929165677, 0.49543300767553755, 0.8001796152373974, 1.0053534488664797, 0.9578489096469909, 1.2993053079751875, 0.8379802270276259, 0.6856805913751909, 0.7116552021532787]], [[1.115407942774206, 0.5131165950510133, 0.35288278149706886, 1.2646892100557294, 0.6824384944271144, 0.6355972276194406, 0.3877187946826929, 1.1750674060741244, 1.7932159459810284, 1.0004016358286267, 0.20115732498026817, 0.47905955554781043, 1.2793240237623917, 0.6184227392150838, 0.3702133738387147, 0.6369455235625145, 0.5939705100057722, 0.564389481126541, 0.9946681929429051, 0.4534392393963263, 0.2235439878839316, 0.5424905739915925, 0.06231990891442081, 0.2020684253040061, 0.905019687411663, 0.322466309100052, 0.5312263160086432, 0.5444083055121973, 0.7655245959816562, 0.9681211482820813, 0.9781358211135802, 1.2060989965131519, 0.8649599556092162, 0.6485499674357924, 0.6771840796772769]], [[1.148072728741912, 0.4893525357423175, 0.32529901057124716, 1.2332819766630299, 0.7012625946397183, 0.623512522246213, 0.38392618385431976, 1.0736984069271107, 1.7943086658805774, 0.8487066117402051, 0.21322573440098427, 0.48737411963234645, 1.210809031278195, 0.7336928371231415, 0.40450199950413873, 0.5771565791096686, 0.5886773788195585, 0.5337349651057702, 0.9942100468169308, 0.46735790704803126, 0.22239534502554195, 0.6147073544337385, 0.052764430177194566, 0.22688057496309227, 0.8559546728228895, 0.3698444943691217, 0.5482416394600307, 0.6020386227058591, 0.7358443111623405, 0.9241283664993786, 1.0572245359864443, 1.0619279844691554, 0.9442160565230677, 0.6709560487906674, 0.6973365776127881]], [[1.1870300839615373, 0.4761498107861911, 0.33709947515660793, 1.2450496018147512, 0.7132262430597216, 0.6906345574706159, 0.4158038786840772, 1.048680087950398, 1.7832744996630465, 0.8030822977433317, 0.2181704821735048, 0.4758749167259994, 1.1933156278964085, 0.6902011191063677, 0.42530553256333853, 0.5246101216923564, 0.651243555366296, 0.5629670479193296, 1.0615361270433583, 0.459647525857511, 0.20648125325612043, 0.6895089954306524, 0.04507317012338196, 0.2262051654652128, 0.8224773553630477, 0.4109625990317259, 0.5550024716832376, 0.6512253095127043, 0.8195308356209368, 0.8867574562465954, 1.1383544762703697, 0.9208861701584765, 0.9833937771765997, 0.7410752351765475, 0.69052180742666]], [[1.2337406854499977, 0.5284891690557623, 0.3947028315327695, 1.3382508621480762, 0.7262800647463953, 0.7054274004220771, 0.453277099641699, 1.0697899471163874, 1.7784326074578036, 0.80071112689711, 0.2282415718957155, 0.4628898008395446, 1.2105421523700817, 0.6620508670411602, 0.43759301386454, 0.5309390975016758, 0.6827024158278605, 0.5519470144840167, 1.0668603916111343, 0.40847886207945483, 0.21753114419441594, 0.6867114644167231, 0.04440139053419467, 0.24185491568089557, 0.7691370972193281, 0.40146847400975494, 0.5906676743976405, 0.6923156221574329, 0.8488119118721922, 0.848668832650162, 1.1920561594491188, 0.7988936092475363, 0.9670802744731495, 0.7950575537626416, 0.751975815055151]], [[1.2653225955422776, 0.5073331995909758, 0.3217846028816084, 1.4212106593476366, 0.7438157448626905, 0.6590700999030727, 0.40018931043577904, 1.125019689501513, 1.7082541548110273, 0.756693315831767, 0.1761035803562065, 0.46606800781878177, 1.21822134616431, 0.6354557481633398, 0.42140191612628347, 0.5173800104055566, 0.673296576278644, 0.5628040689162319, 1.068279837100007, 0.3735571960633681, 0.2693625295630315, 0.6886212549803921, 0.0434150646159191, 0.2864994534374474, 0.7057948246115514, 0.4111561763623654, 0.6072547742692781, 0.762156433019295, 0.8385645096374906, 0.835706581558687, 1.1946440500271747, 0.7052619494615235, 0.9194244222242447, 0.7704437596273324, 0.8665169942835]], [[1.2069099187671029, 0.5918353936235614, 0.3985189343661133, 1.5305492209677243, 0.8071212335273994, 0.7322186822594685, 0.45375208479538776, 1.1158791242820332, 1.5551211958554592, 0.7466606612345809, 0.18683683924182862, 0.4567944239550358, 1.2434288162486062, 0.5556581566888157, 0.4338067918858861, 0.4588717375194693, 0.6912332020201937, 0.599622623628267, 1.0511743799805977, 0.34835954214050957, 0.2757295353188203, 0.7869214605042418, 0.04617194817821511, 0.4373223343251751, 0.6856314037481912, 0.4582910219457929, 0.6168867111260479, 0.7592569317593219, 0.8539616082479002, 0.7825882894643341, 1.1025288757728833, 0.6428347521210385, 0.8945664147898846, 0.7900072980010153, 0.9491333664697441]], [[1.1774524687707226, 0.537833125748021, 0.3330858424530011, 1.519314570337484, 0.8371046109241659, 0.6373886426662916, 0.36532364823452973, 1.170065094683184, 1.4429440679032757, 0.8389360676711812, 0.19705363969042478, 0.4705503627968157, 1.309071095365716, 0.5206813713117415, 0.4311732442818875, 0.43576986164418074, 0.7170293209628044, 0.595869605667262, 1.024075617333202, 0.31029483125943313, 0.2878816390284729, 0.8600609262955345, 0.043508754609885354, 0.6083614915397482, 0.6955052060988156, 0.5077820818538348, 0.6607013133109391, 0.7813945693473402, 0.8827997532456374, 0.7940439977533673, 1.0135101849131125, 0.5897380020204939, 0.8769009927691405, 0.6851852424684975, 0.9364521439250917]], [[1.2353154259413146, 0.45503616327424723, 0.3702671415011673, 1.4831854222187746, 0.7913794450517633, 0.6429096061433484, 0.3737880373544098, 1.1827581634883317, 1.2688362436733676, 0.9473079255780003, 0.2023112040068927, 0.7451491845480375, 1.2938350983914653, 0.6299367286678088, 0.3391178364258051, 0.3677502245231637, 0.7615574761209614, 0.5561021849636929, 1.0009948798876334, 0.3174759564745064, 0.3099691721274942, 0.9748033008041432, 0.03692251941209858, 0.6261401584712173, 0.688952054438152, 0.5729434624157346, 0.6899940290857112, 0.7590995522716579, 0.8513368203742359, 0.8056048152134252, 0.9503130839700983, 0.5573642828430448, 0.9043050927959528, 0.6009759568335067, 0.8762799672333019]], [[1.1847650027925019, 0.5587995022079648, 0.4381233887661932, 1.645725640675641, 0.7394702417241161, 0.6651250548249217, 0.3866642256543413, 1.1696286718843214, 1.1619741419064864, 0.9318821853121086, 0.19490929091920442, 0.7420488644262534, 1.3189889997894653, 0.630800281586678, 0.29425321178342306, 0.3738238207626068, 0.7456167305057768, 0.5551387012312428, 0.9744582826231745, 0.3410551300675813, 0.346790799771042, 1.0048923921408905, 0.0409579194837274, 0.5106495013195416, 0.6843621710394028, 0.6667317466755891, 0.6844699554167295, 0.741926262838198, 0.8783593177568912, 0.8508442297870201, 0.884158709629203, 0.574914469903791, 0.9421825220612889, 0.6995977300767704, 0.8056276715484711]], [[1.0942070734417564, 0.6612690341267852, 0.4577035665349195, 1.7230968478966284, 0.7002930513458516, 0.6844754175564685, 0.4110788615421613, 1.1488003952402166, 1.1229553510578576, 0.950389322248168, 0.19003561264082164, 0.8561753553714044, 1.389066429399175, 0.6653688444186151, 0.30514496018083076, 0.3622325576484967, 0.8030096061326496, 0.5672092203581098, 0.9378515621209664, 0.3791944394900406, 0.3844737693252419, 0.9903988064590485, 0.04222255416304207, 0.3741542096359571, 0.7335815756099168, 0.6956459834718334, 0.6585965210617932, 0.6884280042396533, 0.9256808588304475, 0.9535296707855424, 0.8504253031222941, 0.6339778187021475, 0.9967208906323919, 0.8752489086112818, 0.7108326087882508]], [[1.1539805348182623, 0.5880143296098339, 0.36179226204395787, 1.7496027039123432, 0.6380871500106998, 0.6836406044312358, 0.43666190592499265, 1.1123717529473416, 1.1054581857073007, 1.039332263943464, 0.18798801762887896, 0.800347735439115, 1.4381598180562236, 0.670814610921739, 0.29679952777816804, 0.3727096496613625, 0.811856155090143, 0.5825456169878647, 0.978021002173046, 0.4168427716276646, 0.40581395709497864, 0.975981842231483, 0.04374706896063435, 0.2904204774425042, 0.7574322929227865, 0.6894377994697153, 0.6559488241181171, 0.6512029436739853, 1.0768679745458662, 0.9772986980907686, 0.8086587076654781, 0.6877005305782047, 1.0473139175338388, 1.0515267670644084, 0.6346860829124269]], [[1.2005563294796653, 0.6056493605231188, 0.3371703833557989, 1.9014336642295513, 0.5286395567606319, 0.6843924200652134, 0.42417452386594745, 0.965915145484692, 1.1317620495339855, 1.141170020096377, 0.16166804167167706, 0.816235476203893, 1.4347038889441608, 0.688937365728457, 0.31335561569472664, 0.3983909942050473, 0.7615013530721063, 0.5840487757929844, 1.0351114227847589, 0.35997805813904427, 0.4880358483779499, 0.9141640021636437, 0.0459728397351842, 0.23243101857013518, 0.7496840754707081, 0.6633535199463343, 0.6211128679948611, 0.6868420596100718, 1.2713182146832533, 1.0574645689177975, 0.746679797003681, 0.7185959346153628, 1.1326096402911485, 1.0962308404086718, 0.5310139310726327]], [[1.2327241714736537, 0.7635202907699375, 0.3692505244391556, 1.9097285684484997, 0.428326856711961, 0.6945924291686194, 0.42803968321450114, 0.8003832256202125, 1.247637725242317, 1.0471878586125685, 0.1256190269819339, 0.7761949835385241, 1.4420296160501036, 0.6924186436542851, 0.2707023925589497, 0.3915707858264018, 0.6334104340066908, 0.5441545559268918, 1.1023393910772357, 0.3344860534359984, 0.38728673203124786, 0.6205219667709239, 0.039902164091264306, 0.30527187432617764, 0.787560370450174, 0.3966173439960111, 0.5561966349812992, 0.6977421054507315, 1.1455793703950932, 0.9771518168007991, 0.6917335326614799, 0.6879969187670143, 1.1626556740347342, 1.1266341939121594, 0.4353034416566225]], [[1.2159851532516064, 0.7491186244549751, 0.3866954600303751, 1.8731893520400442, 0.3398072127693511, 0.7007525211583706, 0.3957848482946996, 0.7438071122097683, 1.2563235029395767, 1.0214243429137322, 0.1524487038906945, 0.7423363595690079, 1.4345939096275715, 0.6879777038466683, 0.2954707007665061, 0.38788718142502987, 0.5586162366048428, 0.5121855961503035, 1.127805587617318, 0.37629371596580413, 0.3086874738846258, 0.49050067307679657, 0.035987615326559745, 0.5074314167482292, 0.838815117104063, 0.3178547089656012, 0.4614606723686082, 0.6852563836589971, 1.0128444350933847, 0.9572923765383183, 0.6649303577931935, 0.6295085210711452, 1.1659130576352015, 1.140278138754303, 0.4266072416349632]], [[1.2323891973364112, 0.7458258148762844, 0.3811749458900928, 1.8639784502477263, 0.3535090910562769, 0.6553918673091655, 0.3802309998262208, 0.7081535530883485, 1.212921502673609, 1.0222275057212302, 0.15366155948879792, 0.882988042426131, 1.4123605976301081, 0.60921169631286, 0.30947526511987145, 0.37044824567606427, 0.5403979805221804, 0.4493909265647462, 1.066677736875218, 0.35017313957215157, 0.3196916931478375, 0.46247577467614487, 0.029922475616634407, 0.6709280100836544, 0.8146723319188582, 0.28387031362164916, 0.4264242783087944, 0.7325301797352382, 0.9643257661484375, 0.9180597779297169, 0.6387290458746663, 0.5351932912765054, 1.1084387766822061, 1.1431691013613245, 0.402014390494556]], [[1.2828874182453636, 0.7142892069964352, 0.3880076785438084, 1.8123223167190408, 0.33855970374098276, 0.6552109701424562, 0.37023489770549844, 0.6681197641057277, 1.230657761756581, 1.0435977101162792, 0.17088699806992708, 0.8505529638682237, 1.3582771315995883, 0.6201808752605209, 0.27754500102717544, 0.35762047429487515, 0.6577488748613327, 0.439962062201983, 1.0525290315451141, 0.3444690621400284, 0.30222875942812455, 0.4411001420154247, 0.030371352107998927, 0.799997270042121, 0.7724250226297493, 0.32066556860834616, 0.39520601651900256, 0.9201331765972399, 0.843653404352559, 0.8680969150238405, 0.7879237614420269, 0.40771795852102466, 0.9516051764736881, 1.1559796689569677, 0.5307866312788141]], [[1.2897525615404983, 0.6882856792693636, 0.36122109426734994, 1.8754642142777778, 0.34002626851502105, 0.6437663831911635, 0.37668440691233673, 0.6371301307686117, 1.1316983567957895, 1.1067299651621725, 0.20197035229851817, 0.8592816645615865, 1.3142479908357618, 0.6193441749643502, 0.281181830184907, 0.3401432881127848, 0.6634437728102687, 0.4613789114623812, 1.061635605343163, 0.37711039312095174, 0.29530082800914903, 0.42839028515924854, 0.046113228576661336, 0.9203138984292525, 0.7326208211403827, 0.278488677252856, 0.31906337883609004, 0.9254351098159922, 0.7501776080376459, 0.7971489618169807, 0.8341191447075, 0.2903675821262137, 0.7788876020917226, 1.146359225261003, 0.68273632233078]], [[1.171140222479728, 0.6162431586549648, 0.4587750458902936, 1.9828630370273448, 0.3515473084535008, 0.6752243909636062, 0.37903053600816117, 0.664476450832, 0.8569001400021221, 1.1424450681631184, 0.23763054171520753, 0.863032361347765, 1.2858433146507369, 0.6682013913827114, 0.28647668840286294, 0.37593483731617516, 0.677867092460849, 0.4733618908711559, 1.066560655421012, 0.3716953648780699, 0.2761673140025772, 0.41496960621360346, 0.04665875854723109, 0.9834548090764854, 0.716273451480381, 0.2366028624636335, 0.22816270828457158, 0.9125309938615674, 0.7029931615499585, 0.7878774048056045, 0.8124737150188057, 0.2179684466380101, 0.6525194995330513, 1.13491005224884, 0.7557384253887234]], [[1.1053005719649023, 0.59483732300163, 0.43138279059163986, 1.941837597448961, 0.3515175432905764, 0.6827454021799955, 0.37717016881653503, 0.7629008153749806, 0.8362268736663767, 1.112578787481655, 0.24519933605548822, 0.8618609210407753, 1.3123053550125314, 0.6695019403150628, 0.2760276383494224, 0.34637023537290973, 0.7412333171549375, 0.49006305208586953, 1.0947469824254004, 0.33533325614203185, 0.14510457311668182, 0.39832369220602554, 0.050992853025031246, 1.0204121417072758, 0.7665198943191929, 0.196687504359482, 0.20408741144077552, 0.8573440119442952, 0.687270282223209, 0.830294829269186, 0.7649939187021042, 0.2104077316275681, 0.5796240515011846, 1.124402028206999, 0.9092883180394549]], [[1.029409559395641, 0.6464515903881625, 0.5514694462701877, 1.9489728498936971, 0.35550147792048925, 0.6757096081709106, 0.3780148222448968, 0.7285057597734224, 0.8574796545456085, 1.0316989905943816, 0.22371857469498035, 0.8776060166936938, 1.383760018170602, 0.6732242329741073, 0.2821383381524141, 0.2944214264118976, 0.6265873457183622, 0.5089940263462727, 1.0117961536372981, 0.3091161399308265, 0.17777507058463055, 0.3170324593638529, 0.05144212462155837, 0.9562915721311047, 0.8931115693667591, 0.2286249960662902, 0.31231220340521154, 0.9591620777836162, 0.7157534346158525, 0.8240695143409813, 0.6712996600607116, 0.27678919231234544, 0.5615543159677017, 1.1398350854154164, 0.8439407588533304]], [[0.9736308016281379, 0.5556182225549469, 0.4350820446241138, 1.7794678322348199, 0.358766567149985, 0.6788923347606965, 0.3865960202640078, 0.6274045038246856, 0.8147604798438368, 0.9482821203767969, 0.2645077130584912, 0.9437180010822185, 1.4364579572755405, 0.6587922448134338, 0.28099925380960583, 0.26375532178278593, 0.624992999211702, 0.4921481947303143, 1.065009314759716, 0.266111807898167, 0.22081249865928232, 0.2862000257859074, 0.05144997305541201, 0.8698400202714005, 1.0176804312274037, 0.26838968562234294, 0.36725293064731135, 0.9976011839866648, 0.7763786848118419, 0.8504376996396932, 0.6079400014294507, 0.34378173895611625, 0.6931867595328589, 1.0871521944428473, 0.8316083925886039]], [[0.9440087706415723, 0.4915790475791164, 0.3516839824386172, 1.6648020619563935, 0.34216516867390645, 0.6845296455175182, 0.38208800145050104, 0.5508869489526862, 0.8365507963391194, 0.8212437214090244, 0.22832841238827778, 0.9694503251828064, 1.484496577465301, 0.6666853804935393, 0.3133881183368316, 0.24960716342732675, 0.5974762245477601, 0.475489868647736, 0.9320703815439303, 0.23090473480686766, 0.29994186153579494, 0.2717052075825224, 0.04903758330784549, 0.8659185358425534, 1.14410562570313, 0.2998009868431034, 0.40139093452988495, 0.9063691702820891, 0.7736369570714857, 0.7842104052234946, 0.6393040020317305, 0.45126989792247835, 0.7489089805292547, 1.0039747484863044, 0.8378898585755139]], [[0.7398501933692537, 0.5294658301665909, 0.4508589791763094, 1.4026934050201116, 0.3023166569517885, 0.6645736157699194, 0.38238170495890644, 0.5318997178903473, 0.8484255318669184, 0.8876708035974565, 0.16543352799089656, 1.080177726789193, 1.490214005088181, 0.6080909038229418, 0.30889949716329096, 0.22778789905931834, 0.4538707068024439, 0.47649031732521985, 0.8546022559150468, 0.22055609307931656, 0.3438372182120941, 0.3236887496316574, 0.05073273600654121, 0.8909433176638832, 1.2047664354130994, 0.4475489412090146, 0.44479676255450945, 0.8137574865560635, 0.7881562160991145, 0.7335169364258378, 0.6481891525379093, 0.5495663338631622, 0.7894753192644556, 0.9495415162047666, 0.9346644248815796]], [[0.7633823791688158, 0.5370864234123981, 0.5783913764923739, 1.3400829841995785, 0.369539881111149, 0.6612013745211077, 0.3816744863434244, 0.599107541450419, 0.8931401726646941, 1.1135806489854945, 0.12012837630153056, 1.1427807781834662, 1.4941539572024873, 0.6045901892734789, 0.31066174835423727, 0.2344617758888855, 0.53744645832314, 0.44653158295043804, 0.9467617785702438, 0.19912133205475024, 0.37813531271404993, 0.4999513899262054, 0.13370552405739838, 0.9027443217677342, 1.1918421797165628, 0.4038779608323222, 0.482377107964136, 0.7199712359780327, 0.7182094919075017, 0.6511512436795401, 0.6446718615342655, 0.608114611146336, 0.8021284250031283, 0.9323823029709222, 1.0556489256846726]], [[0.7234695213790412, 0.4925402529912373, 0.5236551615271606, 1.3048909827113988, 0.3429887517957013, 0.6348076841177384, 0.3793829820563945, 0.6285279994914182, 0.8295915136505018, 1.1226810300328829, 0.016944925849462153, 1.0242081452539264, 1.5378647922838546, 0.5922478564200123, 0.3267377464091922, 0.25598925509470427, 0.4754419824553943, 0.43486419840409196, 0.8105780570030375, 0.23324056540006124, 0.42693426208064356, 0.7002268125505997, 0.08826795479229155, 0.9185170855625946, 1.1403132972480847, 0.29652894989544326, 0.4567118570156816, 0.4619806751937545, 0.6642863121091173, 0.5265297210290767, 0.6750228732549886, 0.7132049636735217, 0.8282111970752719, 0.9430915474546173, 1.046828424963448]], [[0.7489080593334106, 0.4778680275279191, 0.5381450302089918, 1.4499168388388377, 0.3560763651544383, 0.624192931158077, 0.398255550587472, 0.546643529882537, 0.8445774317126941, 1.0984240855657887, 0.03930891284640282, 1.0518910950698497, 1.5245635329944134, 0.6005554915529692, 0.34781002647099307, 0.22918308045054087, 0.4230413010893447, 0.4331709833809285, 0.8598622658943819, 0.2379597719030358, 0.4487197204968791, 0.817276817727553, 0.07772700376202574, 0.9225684401659195, 1.0227897277010332, 0.34944207096642277, 0.42947451368442474, 0.36042739718229283, 0.5719552828722607, 0.48450645412144755, 0.6284144921979544, 0.7057391409318076, 0.8239810597786422, 0.908697556473164, 1.0610381510884843]], [[0.7399636016129936, 0.4659920701774227, 0.5122933187439312, 1.4896662541669188, 0.37781668518193934, 0.6224822445127045, 0.39739686191895573, 0.5706419894839461, 0.9431766947002138, 1.0308298803442937, 0.043166008829667334, 1.061390081218175, 1.5455645011884376, 0.655586008491277, 0.3598753180082522, 0.24759112140154543, 0.41886420342260233, 0.4559022303497125, 0.8031024227862793, 0.3026920905014813, 0.4233743254561324, 0.9939941992179988, 0.06806958128218976, 0.9938426966642527, 0.8335846813833395, 0.4444888677633844, 0.4431895421492977, 0.42930722497433854, 0.6356730234312803, 0.6046799526311375, 0.6056017428578473, 0.7191109139296205, 0.838711142625002, 0.8668752202994103, 1.1215151726993065]], [[0.8375745917293114, 0.47292124655037643, 0.5332308726667874, 1.5357718534903333, 0.38079724431661865, 0.6489664374668684, 0.40764834436433284, 0.5699117750721501, 1.0070029089461006, 0.9181686808601304, 0.04322715506847089, 1.0186190840497926, 1.54522010910707, 0.6675443979722774, 0.3629926870350935, 0.2546375758793879, 0.35983133388508026, 0.47040190013505145, 0.7000787362869014, 0.3495332533724799, 0.4201483442016989, 1.103151624959011, 0.07243509033511855, 1.0868979106155245, 0.6911528968084301, 0.5272231567316217, 0.5005089562129417, 0.45384637229578917, 0.7191047580200101, 0.6691459056094363, 0.6042230408823643, 0.7454610870145555, 0.8868640090114697, 0.7760554557812187, 1.1655936716716364]], [[0.8898085817212007, 0.4788179043003946, 0.5489227659621144, 1.5164657089561613, 0.3713196596595506, 0.6569135700281907, 0.4124835490856331, 0.5900849694489503, 1.1152313379836365, 0.7281395330929935, 0.047676783752327395, 1.0571975801565316, 1.512449636041285, 0.7145180015581354, 0.3309618944182401, 0.2481799417503628, 0.37860166326745215, 0.45950289787978976, 0.7640936734941997, 0.32612977399255044, 0.4276467182629861, 1.1822471585177885, 0.06334836004937797, 1.0058574087629584, 0.5902448518781833, 0.586142171556999, 0.47845323075791135, 0.5036137903046427, 0.8192142143259837, 0.6313990609732305, 0.580205411476086, 0.8272308911824213, 0.8901374475678022, 0.6471471825276625, 0.878963385297399]], [[0.9112934389258414, 0.4562074444426497, 0.5559160387363378, 1.6374747013904025, 0.3531722146937964, 0.6484788921679261, 0.3979883726313828, 0.6615420655964049, 1.1763250696711745, 0.7133213272300201, 0.024721092668873283, 0.9295071578826891, 1.5343669905570108, 0.6966738760023498, 0.3083593373776082, 0.25012468784154784, 0.40452573058961305, 0.4924304563323181, 0.8181983246530427, 0.2846349921663075, 0.4456262174512603, 1.0952635842600884, 0.06334646542525464, 0.8981099352991895, 0.4861569909997224, 0.4729728004529673, 0.4507578809956003, 0.6009744214282995, 0.8333944967045597, 0.6174323484944835, 0.591082096247502, 0.829852869042712, 0.9021862705217362, 0.5956732520171736, 0.7675308382143614]], [[0.9268244701289389, 0.4306563425724693, 0.5389174074200581, 1.696191686070471, 0.33396904224029395, 0.6141787975189548, 0.3932328177222779, 0.7466883609464567, 1.1731471020727773, 0.591669956184533, 0.01040287946546987, 0.9214263039836041, 1.5616792513513476, 0.7238941617787552, 0.3146024790192393, 0.318629619525495, 0.5150328332351234, 0.5456684087264454, 0.8123254400511395, 0.27952017548900576, 0.4589096067079105, 1.0797872221457279, 0.06287874325273464, 0.8771751508012231, 0.4445316128284962, 0.4755069703256852, 0.5057584319270924, 0.5848097117185023, 0.8539646794789969, 0.6012791743659862, 0.6858253178302632, 0.8556603297615706, 0.9032107659702902, 0.5553858396938137, 0.7063977797022389]], [[0.9533420542065552, 0.42408016107612934, 0.5002045189199414, 1.6130146348635939, 0.39462710123661127, 0.6273215990293233, 0.39827806477096855, 0.8524839075429319, 1.1773924777247236, 0.4970622774868414, 0.007265944778266028, 1.0160298794044516, 1.5878311625843415, 0.5901622989686082, 0.34875754524011837, 0.3171783483956965, 0.4854908360975999, 0.6135797184454705, 0.809250725596077, 0.28367601875623893, 0.46883296029461846, 1.0124014536900297, 0.0624731006031494, 0.7507554885489389, 0.4817088689903536, 0.4950922599649998, 0.47494322237926045, 0.5715561651091902, 0.8274069383308801, 0.6205675613471878, 0.7211923725672731, 0.8738242414667433, 0.858647517073732, 0.4787813788973598, 0.622743829131431]], [[1.0092655529719226, 0.4133900331038486, 0.4825321093725995, 1.6207251322687553, 0.3484729215305674, 0.6227016006289212, 0.412789797302834, 0.8844786631276218, 1.164617426678991, 0.42651062325873595, 0.012061849765836763, 1.0508062859235507, 1.5720490219210372, 0.5519749752066451, 0.3508515456864928, 0.2929274492362546, 0.3949333325007842, 0.5775897536350478, 0.79290638827117, 0.28392248060580594, 0.4032824029600288, 0.9490170980821457, 0.06299122162958926, 0.6622141191810147, 0.4270051007487792, 0.5262255176868473, 0.4159129806730089, 0.5624107032955744, 0.7709805782307293, 0.6288526446099652, 0.7323130137093313, 0.8823297376568875, 0.7991683622462018, 0.49271014314703837, 0.5420930695027943]], [[1.0985562419330461, 0.42739418792549383, 0.4887372921336386, 1.5220506340470565, 0.3100221803495026, 0.6419644238257118, 0.4086639484133151, 0.8179936027716489, 1.0703351517322206, 0.442452294983983, 0.05741437931824199, 1.0421004506301252, 1.5570996988167956, 0.5563611193951492, 0.310428480500474, 0.2947775854171279, 0.41773750994659375, 0.6192662465946986, 0.8441273250892606, 0.3096907571130997, 0.35700267910694944, 0.8528860955624227, 0.0592374669530599, 0.6162668335685555, 0.3419367942842528, 0.4088655124842444, 0.4401016786920561, 0.5762858361359574, 0.7896157795220557, 0.6048183591643802, 0.6994961660856869, 0.8779773973643452, 0.7782207099574767, 0.47979197514095767, 0.47691984520336306]], [[0.9040479811450401, 0.5164667521854789, 0.48883450948164253, 1.4128108484462376, 0.34230262770606573, 0.6513128301102926, 0.4530384616479826, 0.7338383767657539, 1.0700415913393888, 0.40519807208505965, 0.07390397495656034, 1.062818246435866, 1.5577438897947882, 0.5308141729912101, 0.3076087212052909, 0.2864411128304921, 0.44713344469776706, 0.5962470831534123, 0.7583598298949008, 0.32710759153480407, 0.3664646095760518, 0.8298326248392169, 0.05697577243118458, 0.5604138814186196, 0.32466705281993, 0.15568613447277602, 0.5007017663617824, 0.6025694800079852, 0.6517940867007763, 0.5340081322567551, 0.5766845909667606, 0.8734157579082236, 0.7760731219879953, 0.41286857940171917, 0.3870976445352463]], [[0.8579242517579766, 0.5047079236028782, 0.48349467868937723, 1.4189727874081481, 0.3755557383400485, 0.6443042510904371, 0.4628446409739655, 0.6023895673126991, 1.0264303426442087, 0.46197808632861337, 0.07745145851776751, 1.0690534147507336, 1.588431735275785, 0.5972115904172669, 0.3129491299941931, 0.30237654947036774, 0.4194883331134597, 0.6179318407077556, 0.7712868005090312, 0.36832996147918773, 0.3676651237302751, 0.7903388200951054, 0.058158998470004206, 0.45354348869683214, 0.37537403764469207, 0.0954527718148594, 0.5053252038784206, 0.5835960170536526, 0.4343482582302314, 0.4928448370409854, 0.42169168642594845, 0.885743029442962, 0.7896371196904872, 0.3841905092810353, 0.31192784523200745]], [[0.9087985608563742, 0.48428790311536485, 0.4480233899490932, 1.1512676462660003, 0.38078129964141894, 0.723999441676199, 0.5182937763500326, 0.5484024010843436, 0.964947907436304, 0.45283472520341406, 0.09920977334319342, 1.0929292928832197, 1.6280337561579885, 0.6174641356011741, 0.32981010899229113, 0.3326188182839037, 0.45949755798469094, 0.6242647311476164, 0.8141411623498687, 0.5192867005938241, 0.4081712714240401, 0.7589655631797683, 0.05947392913063303, 0.4219827172732423, 0.4837613589235352, 0.07115235912231566, 0.5025727371946226, 0.5113552969004389, 0.23802484372030686, 0.4543474980913788, 0.2519396790217958, 0.9406092551367666, 0.7844085966682188, 0.3813593611972581, 0.3228918315083482]], [[0.9522797369119593, 0.5329658838323703, 0.415860494743143, 0.9785549166622268, 0.3781497176678872, 0.7463930693406412, 0.5169608972022045, 0.4728969214858613, 0.9790669570579658, 0.4459222803860416, 0.09190383667109893, 0.9283922149232722, 1.5133660569817493, 0.616697169306601, 0.35941214143219835, 0.2927757497393483, 0.41664052700740584, 0.6064713215557512, 0.7924770087150549, 0.5316968891587874, 0.39387392667651555, 0.7186119966829804, 0.05534245350137265, 0.31056724468625363, 0.5899022043082605, 0.060279597015470365, 0.525671304911753, 0.5026377865627427, 0.1398770751497571, 0.37764632977050283, 0.24496149682988505, 1.0100718259922914, 0.745968614426979, 0.37400011696835184, 0.3780689092757875]], [[0.9722182109639395, 0.6241984586551884, 0.44414357702585894, 0.7109351064249063, 0.3490113495803589, 0.751457329584059, 0.4775609300258994, 0.5181352084221675, 1.0477202002546342, 0.47364109386487613, 0.07310237716022298, 0.9324446496114298, 1.478177235426002, 0.6370976067040862, 0.35658762841382985, 0.2688956106466487, 0.4454393168263815, 0.6037043092632435, 0.8884387609306992, 0.40845279559875813, 0.4206958677276845, 0.6737929104736478, 0.06073029168693083, 0.3289752529052099, 0.6184040515670997, 0.05936456904902593, 0.5528529013643568, 0.4629783970654461, 0.10084249449423816, 0.33941430745341483, 0.3317489052385792, 1.0394887808187834, 0.668300032087571, 0.4127852074395466, 0.4096482126607389]], [[0.970999617635088, 0.6547613269175725, 0.47267660539572764, 0.6537266736315694, 0.3818905722833881, 0.767591269382669, 0.4769128260944454, 0.5007687532792914, 1.1944342649963988, 0.5527152913952125, 0.1264703367793853, 0.9391647915993877, 1.4327375388047368, 0.620062721053608, 0.3519082435509636, 0.2790108156358403, 0.4978964384118808, 0.6114310823080051, 0.9271892683143457, 0.48027777995928106, 0.38049142260056934, 0.6120500671957585, 0.05808779466097217, 0.36534045817185756, 0.6382739448548014, 0.05629470494246358, 0.612638696986306, 0.41146862194691414, 0.20277298034359573, 0.46042294998125544, 0.4857959273492576, 1.0549863339062016, 0.6006823795990232, 0.3608560124583297, 0.46318359599103875]], [[0.966435741923412, 0.6865175965748254, 0.4089113776685007, 0.6208096674265717, 0.39227624676449246, 0.7301421776361053, 0.4486195496703461, 0.6359966481523996, 1.1390982975522284, 0.6315270020752366, 0.11157785406119944, 0.9364768748323474, 1.4830226814490943, 0.661870789455935, 0.3112744949938794, 0.2879944884220629, 0.39891399365578056, 0.5841425091852532, 0.8785918324593576, 0.7722385866827972, 0.5471954309281948, 0.6588851544412153, 0.0798941165405527, 0.40611889872856144, 0.6350216935066758, 0.06984613493573982, 0.7345335549929634, 0.2687658241030043, 0.2851432325174743, 0.6000075290458395, 0.592801696935081, 1.0427074936659093, 0.48790673969049847, 0.33867540820316555, 0.5798529685088307]], [[0.949729098332101, 0.7068132766139623, 0.36902789409951076, 0.5756652443122297, 0.3772618752265476, 0.705340362661659, 0.4763995061648984, 0.7864652898870421, 1.0402455075213606, 0.6947816082858818, 0.07527203354576592, 0.9589101040269425, 1.4800034271865317, 0.6234759851504191, 0.31564422942956216, 0.28332849371704405, 0.37083997095675114, 0.6129743202650586, 0.8019893196539698, 0.8208561100052072, 0.5572327659174947, 0.6744003150659272, 0.08093194649240457, 0.47903558039157657, 0.487023739348259, 0.12818595372983327, 0.8226070983226096, 0.29086103031671506, 0.43845412868671174, 0.6966333342988207, 0.6197065385300785, 1.0431262015953155, 0.47256001099361855, 0.4358141913791449, 0.7013930855394863]], [[0.9451714549224709, 0.6946181551282069, 0.3441319382135243, 0.5698623668847516, 0.3905529606861465, 0.7098205464134657, 0.4832092248108718, 0.834049195985381, 0.9184242169606449, 0.6003121445002862, 0.07062940694507357, 0.9474272134559356, 1.4013439225893825, 0.6118092677178644, 0.3578265993834636, 0.2571083179544754, 0.38927616558270317, 0.5930528719306234, 0.7748410837963193, 0.7179123323862372, 0.5378932885728852, 0.5714839360907369, 0.08007609419556397, 0.5544300068123945, 0.41329743232436833, 0.17114894447515994, 0.8652552912692231, 0.294510386233791, 0.5183485625122948, 0.3222128305899569, 0.5402991861970605, 1.0562375620096112, 0.624198459747431, 0.5978864439276921, 0.7485759578456849]], [[0.9456127411710474, 0.8004825415318098, 0.3318372387783023, 0.5571845174695109, 0.3769926238430123, 0.7296128297952206, 0.45824196059017425, 0.866795146081547, 0.789327413051524, 0.6648266658617543, 0.07986312902459154, 0.990057436708754, 1.4952899484368878, 0.6003294016802314, 0.3597095591249226, 0.278287206475912, 0.37610152217076764, 0.5918700155989111, 0.8267095215509029, 0.6855324458985486, 0.5033202587607267, 0.5124735793464036, 0.08073019212759872, 0.6371139562410435, 0.3450971121219212, 0.2400861651757199, 0.8766207165545175, 0.28844260962677987, 0.5714871142385736, 0.31399366228563136, 0.4539952605158001, 1.0155156175811866, 0.8222726357727481, 0.6935049949972836, 0.6885672625727084]], [[0.9532900733866748, 0.8474224641958387, 0.32685847606553387, 0.5784817172477911, 0.4946567580254772, 0.7164252117846934, 0.38432361698896494, 0.9098114569592481, 0.7117378045525751, 0.7238103675952829, 0.12621180067557963, 0.6122216573406085, 1.683387759965043, 0.7666460601555416, 0.32236120437765764, 0.28425549495235547, 0.3550996691173116, 0.5563226232103232, 0.8979858317306598, 0.7215448791700827, 0.4804571536308717, 0.4489829406596933, 0.08245956631804288, 0.6969265998334997, 0.3379381336395451, 0.3121032079647865, 0.8576281895449884, 0.25284724619283905, 0.6826538821009129, 0.3167710179904277, 0.4244137131908547, 0.9233154921944963, 0.9610752237707054, 0.7733759559267921, 0.6101906469833893]], [[0.9913671109621996, 0.9794566028236651, 0.32423485827618237, 0.5624345597903141, 0.5993897824822867, 0.7354975593045973, 0.36246997682789844, 0.8952374218090191, 0.652533433438407, 0.8145692586811613, 0.12777947246615382, 0.6043674168810801, 1.7561434982071527, 0.718243992587678, 0.33123860496938073, 0.2969879394521096, 0.34038989322015856, 0.5368959885716001, 0.8700156261022345, 0.793238065912607, 0.5020209566777544, 0.4575377011303277, 0.08313573296457868, 0.7414453319183442, 0.3417760645677758, 0.3575266082016796, 0.8676877833460248, 0.23074790442356502, 0.7697523096086591, 0.37704356578382603, 0.4457279417035743, 0.8321116864153676, 0.9960837848058992, 0.8352478372085048, 0.4835154537070605]], [[1.0447626735598023, 1.1479458419936395, 0.33788874730805646, 0.58945859521862, 0.690446968392568, 0.7440496168699413, 0.3663637762485071, 0.8915728077929277, 0.5708074689446193, 0.7415857796392304, 0.0955782703351277, 0.5814687224787638, 1.6583772580922995, 0.7204040549338896, 0.36353358733100183, 0.2984399677849149, 0.3161198403814348, 0.5483921032016675, 0.8216921396739016, 0.8356211408398659, 0.49500042359017177, 0.46426575948892035, 0.0688128412855123, 0.7586105897314959, 0.34510588487238597, 0.39112109586189114, 0.8582449687565912, 0.234618126931753, 0.8380551403130443, 0.4123567899042172, 0.5166093022331377, 0.8377074400546464, 1.0353444373277156, 0.8949467731475309, 0.3932396462992532]], [[1.0454414877304448, 1.1240268491952616, 0.31833189412463786, 0.6088370448711321, 0.7550507603113472, 0.7497081665304646, 0.38854842379635557, 0.9571709280123798, 0.5114933794144156, 0.6594353193070175, 0.1372848221224117, 0.6054471279768696, 1.625809538094546, 0.6955072351062472, 0.31146220257507085, 0.30111646924376984, 0.30145124051471117, 0.5372997750133051, 0.8156560805284918, 0.9090708808059935, 0.5175566892136225, 0.46527733495943124, 0.06056615245523852, 0.7575862995217936, 0.3159318158464125, 0.43282181649686224, 0.7790258521372067, 0.22058431181366545, 0.8632433927253514, 0.42644607270523305, 0.6049412970808796, 0.9375068758165728, 1.06757017744614, 0.8668948134289112, 0.3453508022301004]], [[1.0463649585501558, 1.1762244292515516, 0.3043392629951666, 0.7020438242877058, 0.7750176995249024, 0.7493676439991435, 0.3996587048852796, 1.0292508103657945, 0.5338862090368904, 0.5762308242910461, 0.1508565143724906, 0.6107566479747462, 1.551947629878101, 0.6817397372670987, 0.2985951295153378, 0.30731403659169043, 0.29538291621321666, 0.5933659031657854, 0.7976229107293775, 1.0069242841799686, 0.5701589349383711, 0.5106175030262131, 0.05980976289913654, 0.6841039074410142, 0.27207635020430804, 0.4489100106264301, 0.6960858647818624, 0.2239430260781027, 0.8301426615890668, 0.4372793682831645, 0.6766964811868856, 1.08300986221199, 0.978126553787009, 0.8254362697513014, 0.2989174195631096]], [[1.0526720025202945, 1.398655067058642, 0.31894403666374166, 1.066774576501691, 0.8318240385260169, 0.6600788573336518, 0.30556078748525684, 1.1333656094417262, 0.6721711529291219, 0.4665232634943683, 0.1732851113305851, 0.5995090293069716, 1.434602907305059, 0.729342165328885, 0.2965096373840447, 0.31270207953063145, 0.2935474935470083, 0.5938973127133205, 0.7438763868807022, 0.9856660468783955, 0.5687506471291064, 0.5240951741070458, 0.06774992522558053, 0.574916977794349, 0.23339255489458866, 0.4660080901746658, 0.5758735651816184, 0.20695111196908247, 0.812200102026464, 0.46963519698970835, 0.722801839900642, 1.2087000016058116, 0.9074455362193978, 0.8990830506375197, 0.25628508211797046]], [[0.972268246659382, 1.3322228622518306, 0.3199945498492596, 1.0888252474516857, 0.8934931076518411, 0.6503223451193669, 0.3142771952208837, 1.1503721254841115, 0.7122497288243317, 0.38676927068162714, 0.17603254356889558, 0.5923979843782223, 1.4275115835992147, 0.6844668320599304, 0.29779823066564737, 0.3058353148479564, 0.30208812328174484, 0.5707035646942579, 0.7438065952987936, 1.036321150942694, 0.5709988368930505, 0.5936727550284844, 0.07626551885459265, 0.5558313418865142, 0.19619141868496903, 0.47824166368038323, 0.4845424980907219, 0.21626073064120757, 0.7978064468014413, 0.5058773212019065, 0.7442150854550857, 1.277208116307166, 0.8114475904071492, 1.0090949617728093, 0.2340981287561829]], [[0.9701503561265792, 1.18267607895588, 0.2939018217425658, 1.1456883730976057, 0.9815195952094408, 0.6686618233992532, 0.32745723856048986, 1.1860738445242465, 0.7509089698741803, 0.30907549796609857, 0.2061084749836167, 0.5430018257776976, 1.4357923696002224, 0.7123365799930577, 0.29556130337291653, 0.29809331095951985, 0.31358199252061575, 0.5606209971747584, 0.814937893682937, 0.9182686623451719, 0.47593944994670073, 0.5246165983120237, 0.08125284286314866, 0.6020116330515102, 0.27285496927471453, 0.48639095025330015, 0.42611164231695176, 0.2896734321083812, 0.709235796296069, 0.545826858498214, 0.7568876483904052, 1.248229166763243, 0.7610784581486392, 1.0772963616903246, 0.2625163654259681]], [[0.9403919012620496, 1.0652954420639384, 0.29005120180898686, 1.1381904493470076, 1.0365172311520146, 0.7572886377315999, 0.35524251381377964, 1.2224545417747916, 0.7532840992942677, 0.2456806950284932, 0.22600988315375514, 0.5693482714581984, 1.4281119300278182, 0.6886374468477978, 0.28405587113321346, 0.2852435997940376, 0.3023702281609426, 0.5413459316936063, 0.8093576141714058, 0.835116802146822, 0.448393367363055, 0.46508023596718995, 0.11215194255783892, 0.5747812318116938, 0.3011334056216246, 0.4891511404391051, 0.3956126205998021, 0.31008380897173393, 0.6047243638391002, 0.594889973682575, 0.7950829422932759, 1.0552761137593634, 0.7347611584993041, 0.8466064080176398, 0.32685743094778336]], [[0.7591584262695784, 0.9200274533149845, 0.267827652596961, 0.9587346444220838, 0.6628077334154152, 0.7772390719380033, 0.38879640128740633, 1.06886156985807, 0.5154648932390815, 0.16511462296986168, 0.2141440045110985, 0.5510771232749918, 0.7828152547993406, 0.6757973168063345, 0.2787556043710374, 0.3336767026767469, 0.3313856095324694, 0.5586376396541033, 0.73722436961718, 0.7036301082374465, 0.41510655471051755, 0.43142416404673534, 3.4870886584447467, 0.40212618447561566, 0.21997164294931434, 0.3276498480381688, 0.35722696650929087, 0.2726359591250641, 0.3692117271351019, 0.5322771105229449, 0.699079093138212, 0.6358210986799258, 0.4752219269610792, 0.44234453716005817, 0.2974596225637033]], [[0.7510498945648851, 0.9326570353249807, 0.27708664878009276, 0.8760671402126259, 0.7241152637425885, 0.7985587754279345, 0.40542784358472567, 1.0508667378837673, 0.5078311966424434, 0.21753121997445818, 0.24931469822889074, 0.5713702584622993, 0.7581545879010324, 0.6352468138912473, 0.28015997426105477, 0.3677057993169596, 0.3465676384998394, 0.6741955096213135, 0.7415822994270059, 0.5286072116875442, 0.26369726774287355, 0.5223976474988922, 3.487088658444745, 0.37109096575389666, 0.17955949666965787, 0.34832464452744366, 0.40380901030462196, 0.25486869794836126, 0.3477796464401858, 0.6167515265771852, 0.7192454002614086, 0.6171377665958915, 0.45880913690923786, 0.5155032378516735, 0.39327449440139617]], [[0.7533590044774072, 1.0064531372723287, 0.2764861969658617, 0.8643946028727051, 0.7259164334201484, 0.7402114933881301, 0.3591231173249697, 1.0116976231464485, 0.5478663286787824, 0.32335241229488054, 0.25896536520976454, 0.6026485537826873, 0.7594654518883657, 0.6215577558521708, 0.2857232710610792, 0.3240924151453965, 0.37523027007526455, 0.6797430139368448, 0.7424021050306198, 0.3857392255988801, 0.1818933126673658, 0.5188220670247641, 3.4870886584447467, 0.36006838756404497, 0.16679572098574907, 0.36140453170712783, 0.4727195194652274, 0.259729663266338, 0.3218840810959752, 0.722010201727219, 0.7379958593026721, 0.5678828289739539, 0.4672857028072792, 0.5333740827495853, 0.5236384865714153]], [[0.7509705971171123, 0.8498373895465592, 0.26952655930741937, 0.8425120339383809, 0.6979413461290762, 0.7783971907299475, 0.3877550410681473, 1.0114098781835412, 0.5529286787726101, 0.28922744886488666, 0.2810449424125493, 0.6151004681755281, 0.7900805288995566, 0.6195838809095725, 0.2866046564675556, 0.3027027534183726, 0.3843184517105391, 0.7341862503317667, 0.7448367769036865, 0.36380582672945205, 0.16478517966346906, 0.4487436798857276, 3.4870886584447467, 0.3490004155473113, 0.19661207484360865, 0.3540945545510097, 0.527977131869687, 0.2678007653578184, 0.30256873779006416, 0.8073794841578671, 0.7470204980830741, 0.5182451313144432, 0.43661348853336535, 0.5498897070632224, 0.6481338702490793]], [[0.7423203755035315, 0.8237785144587764, 0.2715171115924954, 0.8252112369839661, 0.6860935159306047, 0.7920459228550503, 0.39808390561837176, 1.0181469294995567, 0.622470520566491, 0.29740951257991327, 0.30464574399729083, 0.6393078520922986, 0.8146743193923541, 0.6223853053434142, 0.2804620107937421, 0.3047880579120223, 0.3801305694704078, 0.7014173305294329, 0.7754802502776549, 0.348761921931517, 0.16281658854008307, 0.4333891747601176, 3.4870886584447467, 0.38343879250528146, 0.2312742431677068, 0.3477552710329651, 0.6248906390159851, 0.2795630114220473, 0.2877931311590053, 0.8700711163461132, 0.7499940873214855, 0.4660735229474715, 0.4275756367028859, 0.5622645995087309, 0.7330855760735323]], [[0.7284969317264937, 0.7960247172615277, 0.2664347262592314, 0.787347412901331, 0.703170400768343, 0.758027728745485, 0.37707079870605165, 1.0319063467332925, 0.6039595557899583, 0.2841555695450493, 0.32023976230218837, 0.6424910867577847, 0.8031100543861648, 0.5852648329607947, 0.26952626612890895, 0.3026893461359277, 0.4611969763379571, 0.7262878966096259, 0.7610580167888188, 0.3307814627208272, 0.21313310520773726, 0.4102000436191172, 3.4870886584447467, 0.45831600925280247, 0.25045068240162666, 0.3337044188072265, 0.7407103439075473, 0.28922327640458845, 0.27961570829114296, 0.9259361482345302, 0.7354468917658057, 0.4259526563678505, 0.41649501267480404, 0.580980317844564, 0.7919899316913844]], [[0.7132604855782207, 0.7214036336301937, 0.2664353748146274, 0.8084535725395511, 0.705186784254532, 0.7061687322559363, 0.36980462910097905, 1.051700937137078, 0.6476314163255028, 0.29911987445201416, 0.3034944140637988, 0.6193627411760689, 0.7948820009199642, 0.5955703903310534, 0.23829463241678092, 0.3023839162482814, 0.4496853113171882, 0.7843778689193608, 0.745780296328074, 0.2881733516389813, 0.20877155708026107, 0.31806886365668263, 3.4870886584447467, 0.4687233216578392, 0.2897608445554536, 0.30380665299967813, 0.8219039601994614, 0.28720928917580135, 0.27448781127230887, 0.9708157117087712, 0.7200004367872227, 0.39857563624643505, 0.4349350352060496, 0.5978833833052825, 0.8188144707730651]], [[0.6694348115382647, 0.5985822374141281, 0.2974970281563699, 0.82861984824976, 0.7258480540773006, 0.6815017263178271, 0.369624772169989, 1.0964396094065427, 0.6789163871658452, 0.32868400412984133, 0.2671543914460743, 0.6038682093328386, 0.7491326216752381, 0.6313430189466587, 0.2104184558049012, 0.3333219847038963, 0.4488052971557646, 0.7749830201191841, 0.7191299367784145, 0.2747528643539808, 0.22725213625514434, 0.21935798196479392, 3.4870886584447467, 0.4911245367761684, 0.35707726615106405, 0.2568084706155469, 0.8937676225861926, 0.2852297007379195, 0.26274209790915726, 1.0137642743735715, 0.70341449049102, 0.3753044273553646, 0.4621033948854132, 0.6370581722785335, 0.838700144135187]], [[0.6457714257105339, 0.5377848104909504, 0.3116868509432069, 0.8469216908358164, 0.7206121059428188, 0.6831524535177592, 0.37105030374713455, 1.0784935866730971, 0.6729215340361294, 0.32595984399341826, 0.25613727201887404, 0.6475387152693417, 0.7135186390460059, 0.6247160960505889, 0.23009226426959425, 0.30768000768584136, 0.4654744328310626, 0.7913093902788947, 0.7291932351982999, 0.2767560277035277, 0.2662542810538354, 0.19078485700405154, 3.4870886584447467, 0.45903526948027074, 0.3919925045451597, 0.2123069321953312, 0.9491941884043148, 0.2810284252442635, 0.2511952067841558, 1.0443650474050532, 0.6908288852780957, 0.36064688614721363, 0.4970205730582321, 0.6560708054464289, 0.8570615401289771]], [[0.6167998024620627, 0.4528695935867365, 0.3255533496086003, 0.8520713463906482, 0.7209555001286603, 0.661028894133728, 0.3725778315661737, 1.0639950949522468, 0.6666209240385825, 0.41631485316779954, 0.18252945423988323, 0.7475431349763528, 0.6659820346420973, 0.5574723734727358, 0.18985092116612695, 0.2824599931731734, 0.415789704618199, 0.7510407505298464, 0.7085243089975992, 0.24777251804973383, 0.2369859924700205, 0.1671919531329572, 3.4870886584447467, 0.3989430712399932, 0.35793465159028925, 0.1807955606259687, 0.9817488772687996, 0.26779000847313594, 0.23938874305151905, 1.0710405127727105, 0.6876653523530147, 0.3442975227302483, 0.5179264886621564, 0.6671976653417626, 0.8670759150591325]], [[0.6005063272341387, 0.41215584979046227, 0.3389585737582052, 0.8170384326851785, 0.7697862403729954, 0.6701039594764714, 0.37172506898846946, 0.9653688585027458, 0.7071653079964689, 0.4002101247108629, 0.16738123304065203, 0.7406754325749484, 0.6345496895719827, 0.5555321454524174, 0.1518402709062524, 0.284334405392052, 0.40893753818692724, 0.698047675019147, 0.6266709802833347, 0.17583635498162586, 0.21029205307320278, 0.1257870397860017, 3.4870886584447467, 0.38250785080736943, 0.24383752500844288, 0.14453931441778534, 1.02416071916067, 0.2498143911924791, 0.23612655141386243, 1.1023072997728804, 0.6917064574666105, 0.328306944476819, 0.5375569092760178, 0.6748532570592087, 0.8898398764336728]], [[0.5875263500583804, 0.4064534534798252, 0.34011646400154305, 0.8270207735610364, 0.8079186541897936, 0.6841678805141967, 0.37389232965031843, 0.8951347933736916, 0.7521791125921411, 0.42565018057747633, 0.1607460065180437, 0.7461821835691389, 0.6511078644778943, 0.5968738111036007, 0.20458105221621484, 0.2483062901148444, 0.41958312860201374, 0.7001729374210259, 0.618105271837665, 0.15886620274250668, 0.21082451127384277, 0.12190206790895214, 3.4870886584447467, 0.3488993225883424, 0.15476481217878882, 0.12600333731863522, 1.0285717555918574, 0.23230581226749114, 0.21864045232086377, 1.1172504697474879, 0.6896019533624315, 0.32004762248809043, 0.5458077117135353, 0.672060382511189, 0.9079202079673262]], [[0.571774535233785, 0.39622938005693553, 0.34177639158133205, 0.75373573335428, 0.8551635297163487, 0.6883614017756685, 0.37292459037797066, 0.7714935717460457, 0.7651696126427409, 0.4699063736189375, 0.15415025751997333, 0.7565729529031675, 0.6757382675147873, 0.5573623212176553, 0.27408381837868756, 0.26307782113074546, 0.45127457200259746, 0.7423863590043291, 0.6809677643485441, 0.1729775212263369, 0.21327653702485905, 0.1361421615876292, 3.4870886584447467, 0.39363069452669164, 0.09900864175650383, 0.10859391896959408, 1.0445698195313706, 0.201641345753318, 0.18826670653391425, 1.120423506762588, 0.6874132304000404, 0.3092644148951717, 0.5628047454953238, 0.6725705194640947, 0.9149247950021819]], [[0.5500544239911701, 0.39198490645850115, 0.34657306529120396, 0.697372102934449, 0.8772343637281733, 0.6917059328185473, 0.37543583100015415, 0.7514963375019219, 0.7590189800137419, 0.4974588963316322, 0.13438195753902754, 0.7894626417093857, 0.691295081031072, 0.5477262288905653, 0.2091375475085239, 0.27842794650474095, 0.462254197970694, 0.7508762319830768, 0.6768734573004247, 0.19317475929237643, 0.22800873772599714, 0.1705499932109258, 3.4870886584447467, 0.43083471757935915, 0.07926853310023325, 0.0865305501927934, 1.074038227958972, 0.18546512035346208, 0.16676401455836726, 1.1029121963853261, 0.7056087948734845, 0.2944959606100529, 0.5788852743772006, 0.6877900851461162, 0.9087921433958619]], [[0.544216657478687, 0.38844809765866584, 0.3438819748590246, 0.7255588273402522, 0.8836492102566976, 0.694839043109255, 0.37402641203564646, 0.7888446928201492, 0.7798188187343503, 0.48617803093903256, 0.10755512168448851, 0.7397841508339169, 0.6737162651195033, 0.5627243420621909, 0.2499770751336563, 0.2925744852516494, 0.4809909093205603, 0.7396822430727351, 0.7006734671280233, 0.1697660364966937, 0.21431779784516028, 0.20465465537731142, 3.4870886584447467, 0.4578648206756924, 0.07233159839969926, 0.07014619108245967, 1.0861792214895623, 0.1786991008797415, 0.1787687289500294, 1.0866346993003044, 0.7406552883219666, 0.30466195323531897, 0.614325309740106, 0.6836314906654992, 0.898271858286148]], [[0.5311378646173108, 0.3891920591697986, 0.34376816271568755, 0.7126033776776024, 0.9153525649055292, 0.6952137164997128, 0.37410213339529996, 0.8660015332715115, 0.8181564925284859, 0.49033154530728373, 0.0926889161344427, 0.7405352819735751, 0.6814709987956389, 0.5619306768674623, 0.26020812178045255, 0.2847695977587956, 0.4894317951493462, 0.7401594234833477, 0.6883237733875289, 0.14380101969899395, 0.21414474053834615, 0.2296977343669383, 3.4870886584447467, 0.47770002431070885, 0.07272676205764686, 0.06126030063239361, 1.0969540378172475, 0.1865945707745283, 0.2198069133943073, 1.080670165382568, 0.7070457430832379, 0.4083824418287378, 0.6414373163039361, 0.6911404715372946, 0.8826016959472416]], [[0.5295166808685463, 0.38800043664785083, 0.34366647994384575, 0.6901080963490958, 0.9556213181073783, 0.7200843353062141, 0.37403536505632956, 0.8498954221665869, 0.8437173016166294, 0.4885225272809943, 0.0947282186440295, 0.7220234758112697, 0.6926130706383229, 0.5669190302091418, 0.30066470413361257, 0.27618885495711315, 0.49216877519801494, 0.7151270626673114, 0.7416642343429121, 0.11807159274055945, 0.2153737961724601, 0.22716060026412166, 3.4870886584447467, 0.4975988987206785, 0.0698271159658465, 0.061888657419787205, 1.1120484541604816, 0.22168544392487113, 0.2601726991806874, 1.0801209730280017, 0.6260230244407552, 0.44284413735476297, 0.6670279303107617, 0.7175097348522725, 0.8718203354045245]], [[0.5094668891888741, 0.3837085041316115, 0.34437082166314886, 0.6814692214653804, 0.9647662928177381, 0.6849737359024706, 0.38177350179037983, 0.8663790194308447, 0.8585005018824885, 0.4966979974391704, 0.09046744302825249, 0.7826055244715846, 0.7070901803420984, 0.5552928884987165, 0.3074806480747885, 0.2892058868894475, 0.51774105057157, 0.7944523314816587, 0.7758609257833401, 0.11603610466632797, 0.2234520257637887, 0.22661964307247062, 3.4870886584447467, 0.5236494117954941, 0.07638722915474006, 0.07702515805984647, 1.1190034707589611, 0.21503813326544274, 0.21988557797913072, 1.0793431872873964, 0.49359788894774387, 0.40908068085062554, 0.7208629888972915, 0.7431361456657526, 0.8488723318321398]], [[0.49690972905777464, 0.37502380387454837, 0.3387580855041957, 0.6825035073536991, 0.9954074963888704, 0.6923024196933352, 0.37630230609086956, 0.9052455697399693, 0.7917424553481146, 0.5166008472716316, 0.09031743786301395, 0.767200296802349, 0.7346014968744866, 0.6150792484482346, 0.3230966081445461, 0.3221621974053309, 0.5308879270141657, 0.7432909172740652, 0.75517273707741, 0.1430707568852605, 0.24142840754467232, 0.23909177693889028, 3.4870886584447467, 0.5863181660872323, 0.08039447143323955, 0.17071424416185352, 1.1388259223679056, 0.23297605767345997, 0.19645257260650145, 1.0434810256603675, 0.4055391774149324, 0.48607256406406796, 0.7550535690513391, 0.7668367573341113, 0.779799649845193]], [[0.5464822765189749, 0.37476289328314244, 0.34204005497776213, 0.6570686284750369, 1.2411540001774066, 0.6914589452809874, 0.37790934782079305, 0.9710980457856501, 0.8635980539615196, 0.5390013316745892, 0.0786010817908756, 0.7753200817531657, 0.7237023076667558, 0.6289392756499957, 0.30453983622019987, 0.3617578766239601, 0.593592948424585, 0.7037634899370231, 0.7577094356449587, 0.14649477600550104, 0.23708743203773802, 0.2352672837102218, 3.4870886584447467, 0.589156808555791, 0.08472172801105685, 0.15700102950054243, 1.0929513390442753, 0.2254056532584922, 0.21743055689502527, 1.0063808658848092, 0.340014942876094, 0.5432613503341748, 0.7475148887723786, 0.7679544220556267, 0.6952003535674622]], [[0.4885389974148825, 0.376864423669134, 0.3431791095824972, 0.657328581882183, 1.3149253381814807, 0.6922321887028517, 0.3792353112640682, 0.9669678223217154, 0.8921163903894171, 0.5443934981708912, 0.06988749821952986, 0.7590349708479375, 0.7237416066296192, 0.6355137170508529, 0.29815462976028595, 0.43602368792311685, 0.5940617230063241, 0.6870560258524014, 0.7815680543618333, 0.14482048532378122, 0.22842327512685326, 0.25398067397926183, 3.4870886584447467, 0.5709434314182942, 0.08644691016760046, 0.14488141257170356, 1.034319868541821, 0.2096072750754978, 0.19536765522423383, 0.9165726206012877, 0.2513703829047909, 0.4792069375480621, 0.7237235533062465, 0.7590394374625241, 0.625031321151604]], [[0.3826804843069965, 0.3860191511542806, 0.3385394835808487, 0.6506569249623462, 1.3052202270909232, 0.688585152036983, 0.37506770517753646, 0.9538762808727701, 0.8875935269529162, 0.5449602251453394, 0.06285316483344894, 0.7692262299580795, 0.7495320105417196, 0.5848485819333282, 0.2893824414506717, 0.41542154471246207, 0.5619309949600788, 0.7297666124395426, 0.8231396496419098, 0.15039705552857158, 0.2230516227545742, 0.28365671372270995, 3.4870886584447467, 0.5508889833972643, 0.1083757149661696, 0.14787535300646043, 0.9452242306444704, 0.17981865154782428, 0.17198661578723767, 0.8379637447485091, 0.23434245627903527, 0.4386406722882555, 0.6983905233364076, 0.7298309654020495, 0.5872703593305791]], [[0.39231174571052974, 0.3855769522373275, 0.34538693453876124, 0.6688354137505197, 1.300627427665097, 0.7050769697893369, 0.37532482354819235, 0.8864506327274916, 0.9300043479072135, 0.5690198315983329, 0.07921961204399007, 0.8324190821084487, 0.7430960831942677, 0.5640201954810351, 0.27914560548073974, 0.3788823099311466, 0.4948041151405073, 0.8147823056536583, 0.8761868787286449, 0.16054587246839597, 0.21012865353133642, 0.30519792570803034, 3.4870886584447467, 0.5042929968517171, 0.134614333372089, 0.13565619373909205, 0.867529082230596, 0.1806424879767756, 0.21048662219977765, 0.7766703063115943, 0.25906826521517556, 0.4171227324432779, 0.667953836539587, 0.7253155025073086, 0.5643614211714495]], [[0.4072208584406549, 0.45305140257021687, 0.329229719279866, 0.6662914317736968, 1.2660949239485686, 0.699816188238308, 0.3753543559826804, 0.8472385695067114, 0.9836707859005902, 0.614111621452283, 0.08129084644174944, 0.7961291052601311, 0.6690374615022681, 0.5590000258289726, 0.2857422354816328, 0.4040155719226307, 0.5002975440525229, 0.8199991558392505, 0.8564809353016267, 0.16612249069954588, 0.2007122777082597, 0.3117898244564533, 3.4870886584447467, 0.478587804380964, 0.1715523234711725, 0.14110003015704115, 0.8043792157098026, 0.18885042335427066, 0.24138422998615128, 0.7158978507370876, 0.3027256750554337, 0.42083210885275957, 0.6423328804857771, 0.7127728461679254, 0.5570262266307382]], [[0.433015115712436, 0.5087295619480523, 0.3129562566224802, 0.6752538393588161, 1.218545988041205, 0.7297911760843052, 0.3846590168475273, 0.7484995489622255, 1.0269475700365918, 0.6498252538171572, 0.11626419594249142, 0.8009249632436243, 0.6438062769233872, 0.5494348579232748, 0.2772925001986324, 0.40511637976161996, 0.4985693799839091, 0.8187561864889954, 0.9106452087252985, 0.17145603525088876, 0.19296971632721993, 0.34767335642425523, 3.4870886584447467, 0.39657734220393986, 0.22439752480436684, 0.1853318998004594, 0.7232894471358071, 0.2014973456866857, 0.2583941340823538, 0.6464197906096141, 0.303134769927903, 0.4586007319545181, 0.6119702726394212, 0.7072898192440351, 0.56457933227921]], [[0.44198461030919833, 0.5002523322037904, 0.3124922311265279, 0.6472956145952872, 1.1309432180605385, 0.721147162467586, 0.4160553864887568, 0.7092782508458662, 1.0001703568521951, 0.5984411068954842, 0.1156038286894605, 0.7137436921357593, 0.643237969606021, 0.5467551353756284, 0.2764666842615262, 0.3712938728262737, 0.4689702610979294, 0.744415832504658, 0.8893215446627003, 0.14813030970908547, 0.19966174962490552, 0.34491000112684234, 3.4870886584447467, 0.32561102459542224, 0.28424366827675196, 0.23708111573185114, 0.6707282196504946, 0.2395355067098594, 0.31806380667998824, 0.5708339765331947, 0.30677366806934014, 0.46768874388670933, 0.5523572363634927, 0.7247486664391671, 0.5566459685210171]], [[0.5311818320846622, 0.5489500282029411, 0.29448869511901676, 0.6565332846898508, 1.0630148069135104, 0.7568814271364264, 0.4113751343397764, 0.5905635651981839, 0.9588831369598801, 0.4732826055754644, 0.10991529176458587, 0.6897613238912366, 0.6474480498585694, 0.5397012937905716, 0.26413627941916723, 0.343640966129554, 0.4681712543764672, 0.7533986020328812, 0.957711276822229, 0.15187905549763703, 0.20248403914489208, 0.2962982789091771, 3.4870886584447467, 0.2507678210381007, 0.3107147785477472, 0.25578113301667094, 0.6412913484503636, 0.22823233490395814, 0.4354815374288644, 0.5084413942654358, 0.29985587368788325, 0.47689664934654136, 0.4861304320804905, 0.7318698372887142, 0.558761436480799]], [[0.5204218193021862, 0.5275608394448679, 0.30390089777279017, 0.6761818959695945, 1.0076469294581145, 0.7558208919591863, 0.4039582938625588, 0.5360086640395766, 0.9544059137592282, 0.39250086935367445, 0.09839153673210789, 0.6829444740227772, 0.653103856226946, 0.5427666587620918, 0.2771869546266998, 0.3589724416347625, 0.4629780709961845, 0.79620544644543, 0.8762265407222696, 0.17070166544070287, 0.21937372728609883, 0.30371560201565284, 3.4870886584447467, 0.20159347602845198, 0.32944430618018317, 0.23490153519586698, 0.6542499424764219, 0.21570640076286568, 0.5247279788671111, 0.5098219055881381, 0.3127616124051318, 0.49814983831220133, 0.4631060653192074, 0.7637136865670093, 0.5605674422264614]], [[0.5149566218108974, 0.48426993808938007, 0.3123182774897213, 0.6823402542548074, 0.9155614765025804, 0.7449042900390994, 0.3906852922639537, 0.501962226811085, 0.8963919105573221, 0.4450106183693063, 0.11782195809815625, 0.6589256225951647, 0.6569670880321818, 0.5593748263190101, 0.27839354332500404, 0.3496419959458422, 0.4429297867655161, 0.7976369848984097, 0.8089217223829743, 0.21043636171789049, 0.24852588155673877, 0.32575001078196403, 3.4870886584447467, 0.22996060892135523, 0.3402236014548199, 0.19520496247140487, 0.6922743236527367, 0.23420780377010902, 0.49277257851150297, 0.5091033636122777, 0.35218779711524495, 0.5092416586613249, 0.41967122722254224, 0.7663358756170691, 0.5689093104624731]], [[0.5368338955095904, 0.5167014961353942, 0.2991134804163804, 0.7237359857342673, 0.8408667185499742, 0.6975152676777419, 0.3799674798877225, 0.5022024791856761, 0.8550297651584007, 0.4814958372977655, 0.11432036131070379, 0.623479057834472, 0.6432688370341949, 0.5726559556050074, 0.28084897696841266, 0.3628263727316846, 0.43192879346818114, 0.8008565139543631, 0.7602049224990495, 0.2952421388622361, 0.3013672102223288, 0.3331833271327204, 3.4870886584447467, 0.2828678272257179, 0.33637856057086, 0.162992525183331, 0.7581725931157037, 0.2650007389279518, 0.48185809160857546, 0.4502580324459404, 0.3794643526701992, 0.5022570865983911, 0.41761667448405015, 0.7564646216672407, 0.5652301806940687]], [[0.5336163463744631, 0.5035639301409807, 0.3064600397221545, 0.744046027104805, 0.9521353927097579, 0.6908122702653526, 0.38078241970811666, 0.49202031964994936, 0.8092655928359963, 0.4191972460747242, 0.1043717646244228, 0.6288412862680685, 0.6171655732055176, 0.6115310266773598, 0.29583737441071956, 0.3467282153227616, 0.44562398923674607, 0.8207635243232074, 0.7118186439574921, 0.30631239235076335, 0.3630214121919672, 0.27548936677733565, 3.4870886584447467, 0.31599391283902784, 0.31767609129050006, 0.16973497173381383, 0.8264115635018, 0.25743261423506797, 0.5012682175210912, 0.4469786745670331, 0.40611116849787143, 0.5094309326945676, 0.43454451146447387, 0.7643356957539654, 0.5519214315448441]], [[0.5501041037384558, 0.5059862269665207, 0.30434408806479757, 0.6936786384196427, 0.9199416035759426, 0.6694831800828882, 0.38839457995936216, 0.5639555025814728, 0.8347134118916046, 0.35220969882202735, 0.07419604322507017, 0.6497408056768555, 0.6189098647067689, 0.6083759512713255, 0.29940119351260586, 0.340094348719739, 0.42191488182596115, 0.7564967382298412, 0.7074369332801149, 0.30211499761921534, 0.3600690672833353, 0.28678254970345285, 3.4870886584447467, 0.34885138015529255, 0.3401420203338858, 0.18118617654347877, 0.8887807941565651, 0.23485604779706742, 0.48762102572394594, 0.4147036642280566, 0.4362803384639552, 0.5534472240682473, 0.4526463063553482, 0.7709568883312172, 0.531583995180686]], [[0.5563122885096852, 0.5186689624859069, 0.30093988498488483, 0.6708816339450807, 0.879859781105068, 0.7276448219577742, 0.462444006745138, 0.5919234212021146, 0.846625875259085, 0.3780292558261819, 0.07590610649243953, 0.6839206834931522, 0.594071950902783, 0.6125477824916166, 0.3033533367832609, 0.3543166181358244, 0.4039514829897828, 0.7417997462510716, 0.721352093106507, 0.30181624748086566, 0.3395057843763028, 0.23298996498520994, 3.4870886584447467, 0.37918284546269154, 0.34915831465331115, 0.17762861521987344, 0.9734536364648153, 0.23443458772782105, 0.405218163594802, 0.39269036823785997, 0.4536501468337928, 0.6065953973729603, 0.43581472624120826, 0.7357376185213873, 0.5151592185675187]], [[0.5359445001967442, 0.4992963205290174, 0.3091049209410115, 0.6877052168750792, 0.846348543987259, 0.7042220238270627, 0.4463436497661005, 0.5873323284595737, 0.8549761498270075, 0.4004709071168891, 0.07989308874214411, 0.6427501142700711, 0.6334380125773642, 0.6186765162695997, 0.29911401057035025, 0.34355471940623705, 0.38380379223150485, 0.7144263726663043, 0.7432729915154755, 0.30584755717318135, 0.3513921156293662, 0.19975117716840057, 3.4870886584447467, 0.389678025739712, 0.3533015823297718, 0.22263277053197666, 1.047803098991896, 0.23836719053650804, 0.3878749421720069, 0.34316917786935275, 0.4700872951639874, 0.670380980093246, 0.43055750597762094, 0.6894266976388161, 0.5010153010743164]], [[0.5608510852912421, 0.5202952930202158, 0.30724264796926487, 0.7340427963009384, 0.7269483886579797, 0.6942609355968921, 0.45970937385608973, 0.5893301176859818, 0.8520910776174584, 0.43097174885115686, 0.07726232065267684, 0.6887025825415762, 0.7071402417956938, 0.6427737937788163, 0.3010462125716188, 0.3499811653778972, 0.3859828437133134, 0.7161653196960205, 0.7430024290296244, 0.28748823917001753, 0.36177785994175204, 0.21735378460712884, 3.4870886584447467, 0.3622236980727249, 0.36715790523239517, 0.24307496314896898, 1.09413914649113, 0.24867681921796803, 0.44222295740303613, 0.3484328389686584, 0.48861699718865226, 0.7031179983929217, 0.42795951006956323, 0.6565340556151615, 0.49809818527212424]], [[0.5639010283536348, 0.580131550737397, 0.30106847987636487, 0.8111508044487343, 0.7098571586172925, 0.6890447121252852, 0.38883665694018227, 0.4638043428827237, 0.8091057840489346, 0.46329333292910213, 0.08420584099079495, 0.6958735919394414, 0.7218493224822055, 0.6062693748342928, 0.32804541066311266, 0.35012112549180263, 0.38362906599469865, 0.7169406259410065, 0.7569124971941399, 0.2807727354511411, 0.34989293342279154, 0.34867831124799126, 3.4870886584447467, 0.28004974378032665, 0.39541295727919423, 0.2906980569227758, 1.038600004372268, 0.30293981855684793, 0.4557800485566522, 0.29655723141189066, 0.5235571586675867, 0.7250863684572955, 0.4342669035491403, 0.5863202178326495, 0.4849700191034776]], [[0.5723713431264621, 0.638899847857161, 0.3210199739260463, 1.157280738206973, 0.6220783109236878, 0.7048906655754592, 0.38294148079838325, 0.5921556334589952, 0.7982952086188702, 0.494136615797284, 0.10701864913245321, 0.6698006837436105, 0.9958574438523247, 0.5792082664753496, 0.3280677142756139, 0.3523170097287339, 0.380959099680263, 0.7368547736848138, 0.7103797000882996, 0.2937549068022848, 0.33590107557826143, 0.2524663183965686, 3.4870886584447467, 0.2338946955491143, 0.4839996101768011, 0.30848831124173065, 0.8771640865418315, 0.33950919749515723, 0.4416917318328307, 0.23838285641604468, 0.5267724994231393, 0.7094526660252609, 0.4232351104290308, 0.522750748741778, 0.48192016199371035]], [[0.5938247864868592, 0.7314190698996141, 0.2823799295818811, 1.5249621602708467, 0.5526968685094057, 0.7325151995582806, 0.3649882568591979, 0.6264640167079774, 0.8023597136764153, 0.4597931755583273, 0.1231116978401377, 0.6677583185443994, 1.1448453897988242, 0.5685771172125115, 0.33528219475605775, 0.3440671917881333, 0.3803749475984969, 0.7650499793380567, 0.7151013575596827, 0.3482472628851275, 0.33433415424533675, 0.23698342462634658, 3.4870886584447467, 0.2553824079020918, 0.529781877753051, 0.30772641020742764, 0.7724959177736435, 0.3571408511199079, 0.43582687234028594, 0.2615076376194107, 0.4922701937738373, 0.6892977245413665, 0.4126087910399175, 0.4490005669352315, 0.47986061526266766]], [[0.5852798329049123, 0.7892501543803399, 0.28091216646887557, 1.646103702777141, 0.519554003503744, 0.7052458371550672, 0.39093409942055274, 0.7363093083485526, 0.7640212429301763, 0.4301420820699349, 0.12763859031412364, 0.620045981402262, 1.1779779835515165, 0.5119924446287073, 0.3508920074761282, 0.3407515970115106, 0.3743421457412117, 0.76747301829551, 0.7414039752773279, 0.5027215378851583, 0.34573124786090004, 0.2394000320173898, 3.4870886584447467, 0.22359125695676307, 0.6121829242521892, 0.34077677062465533, 0.6553718770026249, 0.400806461703452, 0.4305707572591915, 0.24825374644465806, 0.4811338607895153, 0.6730899342929079, 0.41636776315590773, 0.38518009801130304, 0.47797749856609206]], [[0.5513738986123249, 0.9253120620208299, 0.3277441834006839, 1.8424016033430446, 0.4915373313174046, 0.7254737895255012, 0.4072782177840305, 0.8846505467292145, 0.7036787345088602, 0.41888161981666305, 0.13566012620694795, 0.5140603579676364, 1.1499255335467098, 0.4467985963337713, 0.32647306989206815, 0.36200611319860654, 0.44304133450373007, 0.794369265318132, 0.749554338621701, 0.5674383274189809, 0.35257065414348965, 0.23935976062656616, 3.4870886584447467, 0.19620953579880274, 0.8723653654037771, 0.5059553052338268, 0.5608813921626584, 0.5541040651341568, 0.4257599063033436, 0.2955402855050864, 0.4924947723017188, 0.7017701491173483, 0.4145867144070261, 0.33952829759547154, 0.47680765990837704]], [[0.5313497364924662, 0.9284140904307185, 0.32148180319877834, 1.9231895440451048, 0.5986042126358992, 0.7297256903264846, 0.4666749247014254, 1.0589368163978603, 0.6720907288166735, 0.4190981384849347, 0.14096232134867653, 0.5205878957371852, 1.129673036293525, 0.44764262950491424, 0.3338724903110601, 0.3559714664454913, 0.45928401124189827, 0.8012902066021438, 0.8074875327060342, 0.5430630704574293, 0.3356159166921133, 0.240268627180347, 3.4870886584447467, 0.17814495996892082, 1.1499813400367294, 0.6283302525633228, 0.49293306937503933, 0.8237725259862152, 0.518014361166389, 0.2972799592635317, 0.46727818982940195, 0.6963965772016403, 0.3955910954351053, 0.3394024404110532, 0.4799998736730009]], [[0.5342961372016648, 0.9200852691867742, 0.3329210308057001, 2.0046649184633947, 0.5930802894302427, 0.7060738632288897, 0.42412608417746067, 0.907599536414055, 0.6427789102889065, 0.41092379478599517, 0.14826575998836522, 0.5034247559155006, 1.041618491461195, 0.43811365036702926, 0.2831428635540556, 0.37516640532487633, 0.465426738518421, 0.831615043675484, 0.829443217787736, 0.5154875098021299, 0.34647280886764886, 0.23273238973295496, 3.4870886584447467, 0.1608455644589673, 1.1423789274850003, 0.5603953593114748, 0.4349695619270397, 0.8043230650356322, 0.5986429073768434, 0.2568942611895513, 0.462979967522937, 0.6870192575552653, 0.350232050603422, 0.25097725287975275, 0.4717903502083973]], [[0.487953330592249, 0.9248522683954561, 0.28854520965103014, 2.064868670871202, 0.6124329434467672, 0.6930728575384746, 0.42603421968620603, 0.8344080018810016, 0.5708270934188722, 0.41054585818720457, 0.16323065767176975, 0.49901934363698525, 0.9192488856296698, 0.40704469959824735, 0.273327606887491, 0.3704119699759525, 0.5006074681451999, 0.9123026721276424, 0.7986979323770769, 0.4760407955144768, 0.3184389442245833, 0.22291521003303816, 3.4870886584447467, 0.21236130907495712, 1.085152610173253, 0.49974090595046883, 0.39357407466053773, 0.7090115856918603, 0.6022939799409129, 0.2507125234729859, 0.5093719201243323, 0.6884225574448593, 0.32281228167221665, 0.2742678617608141, 0.46796888628358424]], [[0.4844612069013333, 0.9022857558516564, 0.29216903027275587, 2.0676942696632037, 0.6371118506669466, 0.6980738700553899, 0.43048618966752294, 0.8634741180366399, 0.5098111834773634, 0.4143049898286353, 0.1709138304016178, 0.5056454238880539, 0.9644384397050662, 0.45127831476747304, 0.34849036274186407, 0.3370268148798409, 0.5197411901968181, 0.8187944148899294, 0.837182494817228, 0.5260104240498991, 0.30431166176383984, 0.20929223945948727, 3.4870886584447467, 0.22993522651509143, 0.9951632219387762, 0.4552301118729246, 0.35425823261884243, 0.6824424593699603, 0.6189088396354263, 0.2677428253140556, 0.6128570603304981, 0.7308682475050775, 0.3078279225240113, 0.3321591028364672, 0.4594921572171743]], [[0.47052448594545826, 0.9069018206560459, 0.28854979201113545, 2.05657337411597, 0.5102193283383297, 0.6893750242182264, 0.41595979373596165, 0.9970043240732542, 0.46279025945600055, 0.4627701576104286, 0.175334671664289, 0.5885236995326383, 0.9347240699651742, 0.4882029188741117, 0.32568682535935184, 0.36244956315666343, 0.5647810205785578, 0.8150410955662044, 0.8677174004748858, 0.5340687013486386, 0.30338479310889255, 0.1868711278034497, 3.4870886584447467, 0.2547318252218751, 0.899980778756868, 0.3963100935668943, 0.34474164569835086, 0.6107084226382556, 0.6414801154590392, 0.26709874876061335, 0.7382909982523604, 0.781759666486443, 0.2851777165178803, 0.41025862727173834, 0.47546988662874934]], [[0.5052330559912859, 0.901949616006815, 0.31348913819146657, 2.0641493929919053, 0.5321286962198379, 0.6726672052026172, 0.40789652508665863, 1.035377990594718, 0.4889965444024381, 0.5353815395495622, 0.17931443512585352, 0.5663537122609833, 0.8743473856731534, 0.49629148704695847, 0.3541265675891727, 0.38246088190482197, 0.6017109150262832, 0.8332534980223503, 0.8949487164264873, 0.5147105965711591, 0.28722072976189594, 0.17569766771488843, 3.4870886584447467, 0.28432687963723186, 0.9621647759682742, 0.3573045160958574, 0.3739416888194363, 0.5752956957802173, 0.6946720373285369, 0.35841533693657535, 0.8446235002750265, 0.8028513969748827, 0.30514130368175885, 0.5178265003760074, 0.46852425184619473]], [[0.4745061251115516, 0.9788886461289545, 0.3288752388650343, 1.986804978735528, 0.5637661069934282, 0.66255660220397, 0.41409574688275974, 1.0754280846321072, 0.38056915371016686, 0.5709293671093851, 0.1859150369071022, 0.5514556880217999, 0.8145906014806703, 0.5111061814081966, 0.3373065333227871, 0.3838433260759152, 0.67895874747647, 0.8178833365927176, 0.9806836792429038, 0.48136903304874984, 0.30219128583944366, 0.17122125217859724, 3.4870886584447467, 0.35011655965894256, 0.9548453683824426, 0.33277955547247245, 0.3898200036547208, 0.5457849766931202, 0.7727398026884837, 0.4535517082728803, 0.8603277612905136, 0.7611047554701202, 0.2918232509174573, 0.5485182537067493, 0.42760031011979727]], [[0.4782570886126588, 0.8896452150068939, 0.29014142535933696, 1.9629097891891867, 0.6160383355728585, 0.6829211187970129, 0.4095060893809147, 1.1754145218430818, 0.3868655933517812, 0.5199336685816272, 0.20122109040609387, 0.5361614552894722, 0.7548790377101804, 0.4835996580012839, 0.3924568297254817, 0.36256562815481974, 0.7349099063167079, 0.8546774499129497, 0.9961254638152265, 0.47224909860721237, 0.3304257515363957, 0.18069212925767503, 3.4870886584447467, 0.4022406460401555, 0.9418333266143925, 0.3146403156640437, 0.38651558330041846, 0.561467997310442, 0.7980599109787756, 0.5298291249153346, 0.8265315916041304, 0.6615336526653592, 0.2760733067392538, 0.5795929129945456, 0.3939785171919652]], [[0.520225881482077, 0.8145081310890857, 0.3139793619305768, 1.8713465928077893, 0.6065377078212308, 0.6721796283925019, 0.38844041084410424, 1.2164150041096369, 0.37070682064926624, 0.5535826596926944, 0.21384451578773667, 0.5306164009088903, 0.7455454101306236, 0.4676704334037594, 0.3876750700006528, 0.3230611976217377, 0.7332641755481168, 0.8489054273028, 1.0389164324106717, 0.5576298414395955, 0.3505860120630616, 0.1859722904873199, 3.4870886584447467, 0.43498662922605436, 1.0599499237315726, 0.2965519918759674, 0.3930560398089632, 0.6297534344981357, 0.9271421575064741, 0.5624292517055811, 0.7786019861944566, 0.671669828617788, 0.22391143812639952, 0.6040801991323506, 0.37264868036566046]], [[0.583772342678007, 0.7634264668858373, 0.3021557314586824, 1.8086359898193658, 0.6435590581915381, 0.6740198706313815, 0.4106243599994954, 1.1996319331136172, 0.47515947869007097, 0.5672053326056742, 0.21157604760720256, 0.5263592495284478, 0.7144431925886497, 0.45977087276510864, 0.36879958760411546, 0.31786596838364967, 0.7001800626119967, 0.8523382356567202, 1.0270632364964332, 0.59028441318315, 0.3607564375639063, 0.19841684378146968, 3.4870886584447467, 0.44319541916644833, 1.1198909791378338, 0.2740469916755025, 0.4063086045698876, 0.6240425743091669, 0.9080822448318895, 0.5943261347314448, 0.7881716106896418, 0.684281373499397, 0.2045663575569077, 0.6576638867889126, 0.3901280951129459]], [[0.5773803885083473, 0.7414714822806416, 0.3115736159205511, 1.730743211230367, 0.6375410512262129, 0.6757357086833107, 0.41545562585047613, 1.1677626168643045, 0.6028417749865005, 0.6105521255207575, 0.23575850819018718, 0.5515311968539136, 0.6600283178829327, 0.4931090556114281, 0.35467006805456214, 0.28124943385622236, 0.6013131268350433, 0.8219266212797622, 0.9721140720033496, 0.6257263409528676, 0.3802757987737086, 0.19578438610571652, 3.4870886584447467, 0.4213500474640137, 1.1471040741688965, 0.251247516219551, 0.38038078357317373, 0.5935840571643992, 0.8726122560248692, 0.6036131078973489, 0.8590728061876154, 0.7004540571561502, 0.18369895958986776, 0.7313354266151313, 0.4367889355579214]], [[0.5920883693874183, 0.7248120695872005, 0.3088178788978833, 1.6896464729149032, 0.6379118430694428, 0.6598299699885034, 0.41599961118464357, 1.1136234070097197, 0.7039830436161323, 0.6380684491777695, 0.2879979627476029, 0.574522726342458, 0.6851185474231325, 0.49599153193815415, 0.3525555713079682, 0.2881385989769369, 0.5925241971199223, 0.8452345443682824, 0.9530978086761384, 0.6393836538720988, 0.37377860142369873, 0.20095557384790336, 3.4870886584447467, 0.35162008106657006, 1.113376915034763, 0.21878753531167053, 0.3482824962333909, 0.5777323233615212, 0.8247636017968693, 0.5988340647605943, 0.8605840995999423, 0.7316118567082408, 0.14446709678786765, 0.7643694444995863, 0.4282826968553244]], [[0.5731733731795561, 0.755096543633492, 0.33127100676124305, 1.5506951901199708, 0.6180220275594207, 0.650019359668262, 0.441736005137008, 0.8901058734644368, 0.8094630862097443, 0.6224244762815488, 0.23407682181323447, 0.5181871357799321, 0.691776957168712, 0.5400550263104458, 0.3375050975487347, 0.2789317821091956, 0.5895338530772842, 0.7676039643580608, 0.9921543268571977, 0.6340632123409924, 0.43753523145216633, 0.2361759216110949, 3.4870886584447467, 0.3042756755911226, 0.9490759464225802, 0.14568175466943104, 0.34545958747238614, 0.5163356986383836, 0.6372933235455183, 0.615658678632214, 0.8567979984755842, 0.8135001632674115, 0.12992507427095712, 0.7767719285422097, 0.44893599776173315]], [[0.5892613269297852, 0.7483560010914763, 0.3141789690767501, 1.5066402765343443, 0.6262585683527815, 0.6655426865120974, 0.4330267911966171, 0.8910820472085459, 0.8834861038392655, 0.5934351234263063, 0.22448222676333612, 0.5346110231326058, 0.7100118518105728, 0.5100413861020674, 0.34531437198521253, 0.25874660938945143, 0.5813307512410277, 0.7632447535381287, 1.0623341804744646, 0.632410471399892, 0.4754514435850725, 0.2830663547434478, 3.4870886584447467, 0.30067617880504394, 0.7428288807023147, 0.10959913052414884, 0.37790643492536274, 0.4503306417619523, 0.44654527704179997, 0.57760066934886, 0.843358273415096, 0.8072867509100695, 0.08239850419987033, 0.7912884619714124, 0.4177929023129025]], [[0.5949677165110263, 0.7401444799306389, 0.3149452832201732, 1.4577724451275924, 0.6331724305336514, 0.6881399847961903, 0.4352780059419464, 0.9018145022445406, 0.9487400682424306, 0.5575383948481311, 0.1939316657403265, 0.5766503626497386, 0.7241673417464876, 0.5366334926528719, 0.33328018011994315, 0.2487555285393343, 0.5639298209309, 0.7127875769393702, 1.0186625370629236, 0.638597841925537, 0.48249147223260636, 0.30245782159329193, 3.4870886584447467, 0.28963638288748456, 0.6831733959958604, 0.09815887038192525, 0.41159603427352365, 0.38770160193749803, 0.38098559429199363, 0.5561103352687312, 0.839689718831989, 0.8058902464931279, 0.07678288608393892, 0.7760434423283782, 0.37222118125456516]], [[0.5886473017284357, 0.74128479252029, 0.3105018123382226, 1.4016812231714664, 0.6459347555032502, 0.6882568945848544, 0.43512918690963887, 0.9018957240582163, 0.9902804614279461, 0.48016018211407585, 0.16481681587353983, 0.5617663310410146, 0.7256730039598493, 0.5702336128454095, 0.3321786511214563, 0.2503221748794034, 0.5719690305799546, 0.7121024375647576, 0.9650677939027353, 0.5903892948752052, 0.4102690074176359, 0.33105396310899127, 3.4870886584447467, 0.24455240062078049, 0.6434340788130675, 0.11015787301819996, 0.40569824610556127, 0.3968096415781377, 0.38623841175954177, 0.5428679270499406, 0.821928993461893, 0.793862840075513, 0.08680122527291607, 0.7489588320935527, 0.3505062096108419]], [[0.5990945658024038, 0.8078423829508381, 0.2949482019141151, 1.3830341060324578, 0.6829834907501893, 0.6702934824185536, 0.43390856871941447, 0.928878590428299, 1.0333099587160783, 0.4017719619841136, 0.1327618703703971, 0.5467812291087266, 0.7060281308226497, 0.4959392709842192, 0.3362498640849752, 0.262553502713626, 0.5615741371926124, 0.8036455947777008, 0.9965912528518273, 0.542614094220062, 0.39772785329214577, 0.33783026572308433, 3.4870886584447467, 0.25431134631935043, 0.6134157894657095, 0.15299180168324747, 0.39537703226821674, 0.40666377947600174, 0.4083798729110199, 0.5366255391544037, 0.8007494866715094, 0.7775776029858121, 0.16241980721039595, 0.7825574852320644, 0.38069835306359295]], [[0.5704534630762657, 0.713428568930457, 0.32429718234033666, 1.343387491953812, 0.6754200074901148, 0.681641874271745, 0.4338005368946406, 0.9196084119350958, 1.0695990697532407, 0.36913163983872355, 0.11982084221294886, 0.5933143682504007, 0.7165697462985647, 0.5024352333563398, 0.33523076648864236, 0.2785506272290012, 0.538061377520978, 0.8036232297680965, 1.0568954265635857, 0.48123469871121344, 0.37491766508268487, 0.29409187698185135, 3.4870886584447467, 0.27298263450579496, 0.6240978928393776, 0.19450837996730222, 0.3910964434285531, 0.4214828439045751, 0.44600528858940713, 0.5627561916800314, 0.7781207985241531, 0.7315667707822757, 0.24967031340645882, 0.8008058726757245, 0.4177215589797123]], [[0.4847958223822778, 0.571744698809026, 0.3106646618446548, 1.2919964091536813, 0.6767668468969121, 0.7079902815495454, 0.4332772074935404, 0.8375192273156662, 1.0647797199002202, 0.35619290160525374, 0.0652608481878544, 0.6001200286823523, 0.7125159379892588, 0.46976396798540176, 0.4456635882842538, 0.30201504543741553, 0.588851915581982, 0.788974218390393, 1.134283845217194, 0.45247035387000584, 0.32234814240038956, 0.265952822858469, 3.4870886584447467, 0.2698704976852139, 0.6136311217414907, 0.21395993789906148, 0.36518856843796466, 0.43459124765000345, 0.4715866398223494, 0.5582767548023992, 0.7823097484132324, 0.6763317681705237, 0.24173830074525443, 0.842905731003196, 0.4860766442632579]], [[0.44009892643730264, 0.569439717979154, 0.3153688615622547, 1.2039251279318168, 0.7257652061703194, 0.7125581449512037, 0.4279378758595382, 0.8137949244467886, 1.0437320134854449, 0.4055959253734748, 0.02993840349418355, 0.624518886643797, 0.7039491579272786, 0.46784689225749354, 0.4280973721234006, 0.3057865005684773, 0.570069406765761, 0.7694146928363761, 1.0930225397154338, 0.39831201116070813, 0.29126486546511104, 0.26493191918215553, 3.4870886584447467, 0.2592238394461813, 0.623621885581285, 0.25038790564951363, 0.3391762359983905, 0.4643895608907189, 0.5023860516539176, 0.5713085217231784, 0.7667160611344639, 0.6987715531327319, 0.22114098842722124, 0.8701919803374691, 0.5636241161898479]], [[0.4248872136039653, 0.4995251695008067, 0.32343381735644183, 1.2334568418688168, 0.7617916121557339, 0.7365597794053197, 0.4320053283709917, 0.7831414843555351, 1.041231070806223, 0.3328179465553609, 0.025891123338507804, 0.6160685326385288, 0.6372295510404589, 0.45676695729071864, 0.39217157132136493, 0.3025620905218575, 0.6076891437525848, 0.8412248252036381, 1.0502853427286276, 0.3681896193332502, 0.2631299737459115, 0.22152781913529562, 3.4870886584447467, 0.24924497180871533, 0.6056090881043815, 0.24634612628955804, 0.3004127950915436, 0.4860403067342423, 0.573844752133693, 0.595075289940764, 0.6750650650251913, 0.723203056105977, 0.19877970160761727, 0.8997219242394641, 0.6178051013365364]], [[0.44482280356647963, 0.4547509906884044, 0.33881795234022155, 1.2463540780036722, 0.7638795067500638, 0.7321112810400315, 0.419022105970148, 0.7756732732751173, 1.0644200795341943, 0.2727365402013875, 0.03340542951229709, 0.6024676710363455, 0.6767430290504266, 0.4622349895238328, 0.38640624098324255, 0.2953158717077278, 0.5267318499332343, 0.8530054113728089, 0.9942361634832246, 0.31165298075572556, 0.2548070325092137, 0.2593515675888367, 3.4870886584447467, 0.2636641888013812, 0.5898164356926157, 0.17713988104241327, 0.28825465187054466, 0.4986639993003117, 0.595632517446907, 0.6192330661563505, 0.6248321491495643, 0.704802659714332, 0.1976109416554958, 0.9103569023843662, 0.6481521963279209]], [[0.4284733175113129, 0.4337782852536653, 0.34173812036211726, 1.2379675836973005, 0.7898398381949604, 0.7347739029672944, 0.4187301584571468, 0.7893812807564988, 1.0519925988927314, 0.23725404916212073, 0.025827695403425506, 0.5945423873974999, 0.6350025489114547, 0.46026269881461374, 0.3325832863650424, 0.29686287379619547, 0.5339646433210765, 0.8572473441591317, 0.9426108782249276, 0.3219550018427396, 0.24983700872597187, 0.20885879305261673, 3.4870886584447467, 0.25977029632148363, 0.6026594141112874, 0.13336705837042254, 0.2533150095899984, 0.5140008426491367, 0.5954273701975445, 0.6398193024186427, 0.5596960291212965, 0.6432030273936549, 0.21372437548898274, 0.9307613575706507, 0.6324178493048456]], [[0.38662742619422363, 0.39732127798556727, 0.3577430748012831, 1.2052943834072263, 0.7984238270518692, 0.7488746530632261, 0.40147423495574686, 0.7955765423136709, 1.0870362872352288, 0.223243983386227, 0.03680230719648384, 0.6255571976650133, 0.6387715988470559, 0.47703968086111154, 0.32927507632379494, 0.2954283681248583, 0.5496949976950936, 0.8592689027549958, 0.8865574790363494, 0.34614218719561046, 0.259536512049154, 0.18941889328844566, 3.4870886584447467, 0.24976415425071155, 0.6442907095446285, 0.09624770457196327, 0.2288927536334432, 0.5684071101321286, 0.6154714575947164, 0.6450267610009346, 0.5024309621723955, 0.56770173518499, 0.223763861862725, 0.9374838446011319, 0.5906035522140562]], [[0.35165654228581783, 0.3837904745498106, 0.35523704865476313, 1.1541612005535244, 0.8292334551086094, 0.7682224624668987, 0.4084953373899437, 0.7880637097444407, 1.0593958667803398, 0.2143415375656523, 0.030369417570662645, 0.6300330336689781, 0.6741383838538301, 0.46285558727294546, 0.3484081480750513, 0.2859758372043548, 0.576891522579356, 0.8731921472971318, 0.8648503694511218, 0.3844756304136906, 0.26309700664101376, 0.19666834998494706, 3.4870886584447467, 0.26143055794247116, 0.6546626166250616, 0.07288473341679429, 0.23405936788302092, 0.588599317636131, 0.60288929871401, 0.6430605932854423, 0.450321397326823, 0.5706668318410809, 0.2390265692854326, 0.9416459713515154, 0.5478761461194576]], [[0.3130192764449105, 0.3655638375799761, 0.351573752651948, 1.196952225991995, 0.8407085549063351, 0.8180009524123231, 0.43447226541187267, 0.8519879111736015, 1.0712332992527365, 0.21137219040207775, 0.06272630110896292, 0.655497138507549, 0.6643966927603385, 0.47293023437099285, 0.29817561490178573, 0.2982099452920015, 0.6188903288055077, 0.8726647140500972, 0.8180325986726178, 0.41355281614324474, 0.2820186711646195, 0.2556835801191775, 3.4870886584447467, 0.2438148635604734, 0.691020819980902, 0.06634590889120838, 0.2592626511778612, 0.6798135288354097, 0.5996953244252512, 0.6253304675482221, 0.37732984103471157, 0.4729552994814974, 0.22261099598486422, 0.9558373840898988, 0.5770246586535304]], [[0.28052263333313426, 0.3557049662561201, 0.3674435917082898, 1.1601564832644526, 0.797631184616022, 0.7238739826220838, 0.37801054727865646, 0.8947737572179547, 1.0393296122111466, 0.23000623764458636, 0.09924095554418459, 0.7228842585313331, 0.7691760191156367, 0.49281719843432537, 0.39558476448883273, 0.2908937096339206, 0.5964489121203951, 0.7485458494802942, 0.7130639863304119, 0.4082256639323527, 0.27826726515625533, 0.23737037408506614, 3.4870886584447467, 0.2733239814647118, 0.629470062317886, 0.07266413757627214, 0.2718296690274661, 0.6865870873302757, 0.5847107152433304, 0.529775327566995, 0.3154902389600661, 0.3950339220679294, 0.19508172433554677, 0.9624934945574468, 0.5964297823719107]], [[0.28066511331324184, 0.37497475930040836, 0.37237119918028383, 1.0700893058968792, 0.8760144327051348, 0.678883068104714, 0.39604693564435167, 0.8890423926688211, 0.9773678887867873, 0.23849312837871592, 0.1506592846884047, 0.7399628192137649, 0.82645937487596, 0.5481624237120014, 0.45382713270387004, 0.29456634580118346, 0.6226130357058147, 0.6938817014248437, 0.8082076771412254, 0.41292233715111554, 0.28239433085769594, 0.21309964229201397, 3.4870886584447467, 0.3368872235506944, 0.5352590402736744, 0.08181678128584664, 0.311331901394714, 0.6741758894670522, 0.5624669934677515, 0.46325508364698553, 0.2915015157817914, 0.30202840268573394, 0.2071252550801282, 0.9569512156933092, 0.6309963860482364]], [[0.31847998699588986, 0.34805662046832303, 0.3542540666717756, 1.0358402456545057, 0.9169709238288031, 0.6932510637843812, 0.3752326961846545, 0.9068320301342602, 0.94731909622976, 0.24280464738795451, 0.16633009260261583, 0.7221376973946618, 0.8298861735728246, 0.5559709071366797, 0.4704187018243688, 0.30392130221375924, 0.5530223986352123, 0.7064235425477388, 0.9125180651786331, 0.45132139318887976, 0.27583971961194015, 0.20176600054600108, 3.4870886584447467, 0.32936616401055896, 0.47576493960634547, 0.06945518944557247, 0.35808972559722885, 0.6258059634334814, 0.5368564033353409, 0.4823711360069749, 0.31024363582122116, 0.20601828050560803, 0.21544854058426655, 0.9268090841072862, 0.6690525730693474]], [[0.38944932454038894, 0.36551183739404186, 0.3610893254155416, 0.9733370228216875, 0.9635057659473109, 0.6683551874788827, 0.3776932853014562, 0.8713075416706981, 0.8947877610203332, 0.27419899783781954, 0.11608981905320598, 0.7565432461096564, 0.8454546752049062, 0.5477250185457008, 0.47733041937988907, 0.3018326813448375, 0.5075896724750041, 0.6766100012064213, 0.8512868520718961, 0.5057593491635997, 0.29864091840602514, 0.18828719008535952, 3.4870886584447467, 0.31666300654284685, 0.4290467195778973, 0.07884668449457483, 0.37468088708994324, 0.5696155172804269, 0.5037777274456723, 0.44785962776197186, 0.33201469874708106, 0.1439564712356377, 0.23046594762873007, 0.8567813719315857, 0.7122849938179505]], [[0.44435946511383506, 0.3979070518229828, 0.34447007650936295, 0.8916834734316039, 0.9864781615024336, 0.6902650261559673, 0.37238955242618355, 0.8671333118215617, 0.8487110597102738, 0.2917100226825439, 0.11126447604786077, 0.7841288830329762, 0.8417501520134675, 0.5459937861426339, 0.41959715928177044, 0.3050175182423398, 0.5762159696555241, 0.6596425203505721, 0.8363931532377573, 0.48327037467477435, 0.3240622763439733, 0.18188147237742114, 3.4870886584447467, 0.2996898685945707, 0.36533160387349783, 0.0817438533911595, 0.378542151405262, 0.5538018508167025, 0.4702916922418652, 0.37591057020873736, 0.3124616677332632, 0.09986654584849147, 0.26025785951124414, 0.7657843530817546, 0.7456351175309365]], [[0.4390416074472454, 0.39925265007756366, 0.33479712095708114, 0.8238905093702991, 1.0260688089077394, 0.6800803971448294, 0.37350150480032795, 0.8317017323615709, 0.7628426769116808, 0.2850278083555738, 0.11202634062512708, 0.760061682445077, 0.8191775537808073, 0.531158055215168, 0.43191189114047374, 0.30423105891285224, 0.5512379815983115, 0.6494236187375257, 1.0291988035686352, 0.46241917950863354, 0.3263823950062142, 0.2160710426992321, 3.4870886584447467, 0.28758607691013527, 0.3148452157493259, 0.09412453544010925, 0.388168261062268, 0.5621814045538641, 0.4241470250731331, 0.3469811135695434, 0.26187474229617336, 0.09358123777628122, 0.2616779700113288, 0.6676861221813017, 0.7471613956253076]], [[0.7589948873489777, 0.3993295513389909, 0.31276110069669505, 0.8102950090305301, 1.2884976282631149, 0.7040912617627241, 0.3822835232903568, 0.892466528659039, 0.7256492550332381, 0.30873160507054687, 0.19271922658620944, 0.7952876401821147, 1.2876761916548178, 0.5792835610031752, 0.47259251150117704, 0.2901593716345966, 0.5606987607008246, 0.6294567028217211, 1.244522674188992, 0.5594616450192191, 0.34245049041746367, 0.2750303051185538, 0.09570002647806608, 0.3449978247226857, 0.3266412684844734, 0.16579776358670398, 0.5062606679656717, 0.7638748141081435, 0.46043792929112004, 0.4531243678545362, 0.3284182867620208, 0.17473218150532308, 0.37534616565042656, 0.7222234797779186, 1.266127929243201]], [[0.7957876246691462, 0.37720880895967734, 0.3404221364482819, 0.8085347868850563, 1.3326454890981245, 0.7018703093211214, 0.3802316169242462, 0.8568101558570349, 0.6371621047843639, 0.32512296834300236, 0.1355470799181738, 1.3655692782614544, 1.420322173753006, 0.5555477236133771, 0.425684561507584, 0.3003983543813639, 0.5449323749766429, 0.6372065964363587, 1.1022773450696715, 0.6281828858690105, 0.3109497652790825, 0.2630749015113594, 0.07597895487529682, 0.2509353275679446, 0.4295649205898547, 0.16693525759021258, 0.39999529039929077, 0.8091105727200205, 0.42660325762086054, 0.3722428362954053, 0.2898569329515476, 0.20960516961012546, 0.3443090411653377, 0.6125687136889869, 1.2697574994047462]], [[0.8450478129833954, 0.39364011236820645, 0.3486368433457568, 0.8181090052865816, 1.3698582617922828, 0.8118868248718888, 0.37486011728854, 0.8240168839365134, 0.5345233843779635, 0.26762311204194955, 0.11224619459167601, 1.116367013106254, 1.4033897297037128, 0.534040261343001, 0.3970756292791592, 0.28368914301933784, 0.5121719352481046, 0.6470657000964797, 1.280836989105207, 0.42255508366017497, 0.266961110878488, 0.2949635667168309, 0.037965004887494797, 0.20257335364029094, 0.44860524354699594, 0.19902676513330408, 0.3162004509181138, 0.6990170888380668, 0.4062543441443423, 0.45003891599798834, 0.2766016339715405, 0.2118666486099473, 0.33853170473352723, 0.4877524899771376, 1.2466454377401914]], [[0.8767597935744214, 0.4168906688027582, 0.3482815825364942, 0.7964378670132022, 1.3972531466511138, 0.8240979994874281, 0.3836636407341419, 0.804748627197062, 0.4156768004962358, 0.2487576870413869, 0.15078096700025148, 1.1201211540131428, 1.3883663870639626, 0.546854601797518, 0.38094038558634125, 0.2776078384676499, 0.5076330756509357, 0.6714033306869438, 1.330263051427154, 0.45384346685055915, 0.24353161919207963, 0.2720295244670737, 0.02760408500582353, 0.20446680536260212, 0.49830300979183506, 0.2632452518505803, 0.3026305345472975, 0.6889838771090089, 0.37601396638644163, 0.5767397413813538, 0.2755604971216006, 0.24654519428837118, 0.347652352351651, 0.38516917820795094, 1.202985664080079]], [[0.9038850265635878, 0.4812475849304225, 0.33982592666525624, 0.7600568432104011, 1.4169458488415563, 0.7223258472854056, 0.37432213741816095, 0.8499272811415403, 0.4389556144918108, 0.3815301637552479, 0.11264302610395058, 1.0872761599220242, 1.269100448604401, 0.5667054740298945, 0.4213799274369646, 0.3012781192639057, 0.4567568951684757, 0.6109636922688727, 1.360191872706662, 0.5190398947183398, 0.23590835966919071, 0.23190505524283356, 0.023883226831284736, 0.3630102092526886, 0.5450068820139768, 0.3145690754389139, 0.3191202522664357, 0.6499083963246877, 0.3533589563913002, 0.64776022021728, 0.2786948048624349, 0.3130549597938526, 0.39590993895616405, 0.3410493108750703, 1.1355093424295768]], [[0.9199224296694843, 0.5462359319841983, 0.3400868224159179, 0.6916848953877888, 1.412206576510572, 0.6685512581086506, 0.3616801917449222, 0.9221704661988485, 0.4760945493059993, 0.50054461632423, 0.085127807321798, 1.0324598000161764, 1.2052916069793964, 0.6084204671880413, 0.38840814895685905, 0.2713145929773778, 0.47095740083778787, 0.6287089329306583, 1.1935958642632905, 0.5357118836279736, 0.29632203330840656, 0.16026570518881444, 0.022846787604934218, 0.612126030550618, 0.6479516411282178, 0.36463943931646287, 0.3601523749101025, 0.7346268284082498, 0.3545712577767757, 0.7621445408543541, 0.32272898626068436, 0.3979512789731236, 0.56005832374439, 0.3199512956983249, 1.0481776470328574]], [[0.9160891282549177, 0.5800305139296438, 0.3379779633035172, 0.6854973537451043, 1.385841441014435, 0.6765727191208131, 0.36362484943852924, 0.8887422802098064, 0.5148759139471029, 0.5408318068498232, 0.09782080936960358, 0.9951359648365496, 1.174941977169698, 0.5895023729110698, 0.38765798858218803, 0.25921384930032804, 0.43976091546905194, 0.6082694547908523, 1.1298269918961845, 0.4883537472033808, 0.3376438016809187, 0.1414726597213207, 0.021244622203314877, 0.7703431352591521, 0.7339162110158487, 0.39512697473916963, 0.3811565588225215, 0.720726198160926, 0.35917141274210784, 0.716515229707456, 0.42659765095117563, 0.4218011228526125, 0.5854376455935, 0.35067200349762706, 0.9394735268403451]], [[0.8791341431042481, 0.5871854633149387, 0.3304972987970137, 0.7318405976212026, 1.3744715336789497, 0.6725805974920969, 0.36474791687502994, 0.9503797291818215, 0.5479363788352086, 0.5376436255225687, 0.12309752284974466, 1.0114734950445081, 1.1745312088728475, 0.5981951564144921, 0.3814906336069067, 0.275942299470921, 0.40935675706360936, 0.5804994976252593, 1.0160699463145513, 0.35423329776685697, 0.3031830753175267, 0.1536825412822424, 0.02053611330119036, 0.7717566723698461, 0.7866325066542936, 0.4250382428189111, 0.5069179842599119, 0.78604033991769, 0.340033236968584, 0.7276143924242788, 0.5119824948613845, 0.3936858422578254, 0.6722731078108792, 0.3650878286121218, 0.8294661246720125]], [[0.8904935774432452, 0.6114333696456669, 0.3225149196678108, 0.7923636235934277, 1.3896939236169012, 0.6301602155236281, 0.3636758522723044, 1.0557667789376914, 0.5833945261879241, 0.47603183413781014, 0.1394435224219049, 0.9616862876187078, 1.2207987640584834, 0.6057257255756799, 0.3774834243018104, 0.3091208006620527, 0.41212792536586085, 0.5756929184256615, 0.9622731754812495, 0.3313034685589554, 0.28449514889137484, 0.1931316609350753, 0.020331489250472512, 0.6991294386028784, 0.823348938390843, 0.4625659560161084, 0.6128895375326673, 0.6959232227751466, 0.2892587500615853, 0.7675321326021479, 0.4852085622648443, 0.38925149351130955, 0.6337792636895343, 0.37608119124200745, 0.6974697526242948]], [[0.9089886708801461, 0.6326251215994361, 0.3143601101628192, 0.8016024992948926, 1.481956358504136, 0.6828912204513958, 0.38517317777286264, 1.2402475786844391, 0.6256794770419634, 0.5716415093343888, 0.09688529108126837, 0.9358908515830364, 1.3823580033917053, 0.5940598320739623, 0.3876745755277992, 0.33504840709401185, 0.38868687585659456, 0.5580061963167555, 0.9622319436513245, 0.3228036363669635, 0.37159453664771186, 0.281685118365561, 0.019252790014810485, 0.5131144097359458, 0.8617381969121118, 0.5072371985534291, 0.6787849152755964, 0.6566144730559365, 0.35360576644488595, 0.9016004282685073, 0.29406301225852277, 0.36203099499107727, 0.5793731528150441, 0.3425104728058863, 0.75052512273983]], [[0.8946896349344067, 0.6754120443643268, 0.31031668430989057, 0.8189634642774506, 1.5087837892642595, 0.6058983625793756, 0.37411914978979294, 1.3440569973124008, 0.6650063488225477, 0.47979326256294474, 0.16612597225285122, 0.9784940726404372, 1.4647993670818413, 0.5043017704900923, 0.33726107682315054, 0.26597099282484415, 0.38034929434307574, 0.5367598754439346, 0.912092584774701, 0.3152569429536169, 0.368967575077849, 0.32445860659814285, 0.015383130811346868, 0.4203923676338917, 0.8745061569532845, 0.48466944933117834, 0.6884900090808317, 0.6372644838686461, 0.43809745764425, 0.9911271931633148, 0.25370102779160986, 0.34263551548611054, 0.5344157165280803, 0.32087424345077153, 0.6515680064726795]], [[1.0326396428367057, 0.644552250106981, 0.3336458264443397, 1.089936785185555, 1.2976717834145206, 0.5992842221685382, 0.3739401206632578, 0.7792072001361587, 0.6628013688503039, 0.46152589898715324, 0.104107743321871, 1.0269353898438554, 1.4511120623179372, 0.4986280645360306, 0.3165987149361617, 0.2644188113268693, 0.3082698144412681, 0.5612792355979357, 0.916075202007974, 0.3044388855334106, 0.3492048114471811, 0.3345365392656702, 0.009936673719764854, 0.4145704928698433, 0.8740631664060423, 0.4122028428056183, 0.7487720381325432, 0.5825689377663196, 0.44903505821274636, 0.9969934557528902, 0.3104074486479564, 0.4484389735133398, 0.43657054816987595, 0.37358405760944113, 0.6105294633672755]], [[1.0569966688870074, 0.6872265094077791, 0.3200341445386284, 1.3342521270763035, 1.3044482108952855, 0.6152503372896941, 0.4050834007326402, 0.7977156340605458, 0.6546084216457967, 0.4411184994316442, 0.08553756278804381, 0.9800473762149523, 1.3765481378335052, 0.5068809909117233, 0.35051394783812106, 0.2819602309343511, 0.29232569591453783, 0.5651738826952268, 0.9235206313849296, 0.28065072159281357, 0.3314225814788432, 0.33851991085118094, 0.007157192217877304, 0.44183902167334344, 0.8531379856274234, 0.4141975182660662, 0.7524009421416018, 0.6073801446430797, 0.48864405138430533, 0.9689962059141646, 0.42238566434531916, 0.4516687905318512, 0.3950730719009966, 0.4919980541971619, 0.5690435438452218]], [[1.1308775471233865, 0.6667453506957028, 0.39493001393726235, 1.484205987399547, 1.3025553439684499, 0.5583670606994178, 0.3735200111568688, 0.8738466798927431, 0.9194187103583878, 0.5620063837132325, 0.06318794780454122, 0.9648808944273051, 1.3085293811324976, 0.49709379739214615, 0.34801383523856133, 0.29948840334665067, 0.32225801102543133, 0.5372484410431074, 0.8734648385050613, 0.2585440005954365, 0.3516490816780074, 0.34056525360388773, 0.007406416294952612, 0.565140886096515, 0.835482137518172, 0.3843908166019746, 0.8105481360216767, 0.5945710670157173, 0.488529741542003, 0.9846147151253399, 0.5445839218218219, 0.44461765554083116, 0.4316845287378397, 0.6592149324630505, 0.5068417347950258]], [[0.7603028398029255, 0.6287475196002166, 0.3716747338577973, 1.647244871774151, 1.2436019695532967, 0.588029987859559, 0.37288160554135413, 0.8478231488357238, 1.1411162032411544, 0.5627153094008293, 0.06595039779386962, 0.977151858714303, 1.2725980864153377, 0.5073784790759541, 0.3062574985610359, 0.2764663022573142, 0.3545043083993815, 0.5579009942005855, 0.8801061637616747, 0.22117020710836063, 0.36103282435280803, 0.323832858295133, 0.0074710226216516745, 0.71543054814174, 0.8177540895588538, 0.32849203203802824, 1.0790056949425697, 0.6176507545775205, 0.45277595032651335, 1.0271371095368962, 0.643684831706831, 0.4190732077727363, 0.4648404534116135, 0.7093659247853089, 0.49286551084991925]], [[0.8524707461782883, 0.658602748084929, 0.3015039084327444, 1.7554948974698108, 1.2138598157637461, 0.5887257664007106, 0.3646421048622535, 0.7398468725556403, 1.5506368280399832, 0.48429235627687445, 0.07554497570651933, 1.011027228239901, 1.3060223600601235, 0.4864000197917001, 0.3305567182391074, 0.28331492143762876, 0.34481458037866364, 0.5641999783110956, 0.8876487293532673, 0.22721820604406348, 0.32243169135806404, 0.24809701135500917, 0.007506532931320506, 0.6646165986191799, 0.7954413171578998, 0.3509304840091438, 1.2126303738636395, 0.6614903185429672, 0.45956349606712643, 1.0374122076376349, 0.7397677634924773, 0.3907998588485463, 0.4879142348671075, 0.805056560589072, 0.4998508765970434]], [[0.8119113743302723, 0.5998311490866227, 0.3045581610060314, 1.7876757572771054, 1.17237118814704, 0.5948860329868018, 0.362788213734027, 0.7096860879490712, 1.694241555826421, 0.4816990181207012, 0.06308872707712283, 1.0595097216878113, 1.3271872046640367, 0.49080906871819563, 0.2507344312584691, 0.27041765799231093, 0.33272225991676635, 0.5654008122350966, 0.853969214872088, 0.21065470012625004, 0.3299031328139215, 0.2633450565065007, 0.006958651325666623, 0.5294969903990892, 0.7662475210651416, 0.35609770040055844, 1.2062052978096225, 0.7589945478322152, 0.3923354447789015, 1.0572943166066313, 0.8929451004688951, 0.3604053673936971, 0.5053400482277286, 0.8702924689465176, 0.5758161578364711]], [[0.7673428590215943, 0.6020351007279823, 0.31134465491130076, 1.8627618531626977, 1.1030228645870686, 0.5882875807067621, 0.3700983589887813, 0.6841015256666177, 1.7502041985222312, 0.4772900166798147, 0.06142043579056575, 1.0692620540373143, 1.329736155395476, 0.5044167185581474, 0.22157749642572727, 0.265322228113591, 0.3627884266468099, 0.5474371370823848, 0.8506892264932421, 0.19285241480321982, 0.31245841889515286, 0.2806583432635799, 0.0066747174080848196, 0.5136194408846935, 0.6946016091515378, 0.4105246562653555, 1.1926040895104288, 0.8135469103525772, 0.3909070335923225, 1.0640286728962771, 0.9548184277206215, 0.3755885287302652, 0.5581924778373026, 0.9958381616840588, 0.6986642669813472]], [[0.7569415400906636, 0.5318910250152287, 0.36303822450528517, 1.9539943719727504, 1.0491012684761525, 0.5930737903390605, 0.3718480959650858, 0.6802193457677481, 1.7726497103723637, 0.5014639880577605, 0.07531516394908842, 1.0793842546405759, 1.3230706302180921, 0.5026667958089373, 0.23428802363313256, 0.2719277454424113, 0.35939186927263134, 0.5096389269699881, 0.8090497810011823, 0.22854779001239867, 0.28388303163607365, 0.3146860189068166, 0.006332011787384116, 0.6946600897763745, 0.6404843140629944, 0.5040064361387413, 1.1948738183295085, 0.8987359534556866, 0.4044390046453246, 0.9639439800669398, 1.0155159803011142, 0.36857639850353713, 0.5623849411775608, 1.0366465266642004, 0.8081622521065783]], [[0.7373671977187151, 0.5838697701147743, 0.3615365626727403, 1.967799083196486, 0.9193991291369484, 0.5548236548346998, 0.3619816281540264, 0.7052660625788729, 1.7982530800390437, 0.5109220957548772, 0.09741043468628541, 1.1304824619086689, 1.3025690016428042, 0.5774605137159565, 0.30514996518503845, 0.30719351667087375, 0.31709000693082556, 0.5586868280398831, 0.866861118374697, 0.23089233561275574, 0.2673387091625663, 0.24803093379904934, 0.0066551360470561444, 0.8790019029774587, 0.7083651292904851, 0.4818329459344862, 1.107031451447324, 0.9952649547310255, 0.521335841273886, 0.9903250853020745, 1.0228471895414029, 0.33531834748363193, 0.5834131303795971, 1.1163530680155265, 0.9424239473863826]], [[0.6990646373499003, 0.5265491184312017, 0.37690016743636023, 1.9979004152296465, 0.7312154129729312, 0.5642470196926851, 0.3605057727807082, 0.7559428027583542, 1.6275946619902917, 0.48776997409320577, 0.11491989038868376, 1.0941190048717035, 1.273081150847754, 0.5916037185748289, 0.3002760952659235, 0.31740020393475843, 0.3186528167553972, 0.5575395537303698, 0.8805013727189039, 0.2277618769967167, 0.28915313662672404, 0.23668372949274358, 0.008061760156569278, 0.9464991879834713, 0.7583522605152954, 0.47596906078738926, 1.0667137166110094, 1.0435583798906385, 0.6126132785851962, 0.9829553663028202, 1.030629891336445, 0.2347450019435056, 0.547341689326952, 1.1937435057268004, 0.966216098668302]], [[0.6223100440297594, 0.5770220086289153, 0.36733210006316325, 1.891771172693183, 0.6261952025706018, 0.5626669769089476, 0.3578441479300618, 0.7837338058181431, 1.3837350715377434, 0.4845026041386423, 0.1185079770988609, 1.0955095460066395, 1.2151819807452973, 0.591998539642705, 0.35745520164157457, 0.3228712366842657, 0.30014784770299646, 0.5107597177215906, 0.758793102498585, 0.22438006628681695, 0.2940662338195089, 0.24423761399033594, 0.009819909490449412, 0.9370059518628144, 0.7526540394744969, 0.4378563550691193, 1.1089806383808742, 1.079103886286168, 0.677897057167081, 0.9647557534505058, 1.029784679174388, 0.26019726695173384, 0.48411105803295135, 1.2236842703182096, 0.9988023091340541]], [[0.5550845209342713, 0.6436055741760468, 0.355788528702324, 1.9053570217856317, 0.3833470932921131, 0.5704717571308988, 0.37766241987690374, 0.8143130236251933, 1.3028398799194598, 0.42463653569687476, 0.16035044352954092, 1.0245324954484833, 1.1479720778477378, 0.5843996103358415, 0.31072998817850583, 0.3504988498224176, 0.29322997897836534, 0.5183510841664922, 0.7282700420563899, 0.22596212408309338, 0.30980002734590456, 0.27426176630689447, 0.010552144355212725, 0.9166903942621933, 0.8042292945112659, 0.4214364482068767, 1.1701581029597894, 1.230713628005903, 0.7138690208821795, 0.9334959310676483, 1.0424950552323873, 0.27043644161046715, 0.4118663189652572, 1.259024755491899, 1.043305210513522]], [[0.46733043844575134, 0.6028332508343901, 0.3820429774000249, 1.921542405517613, 0.3199221620443622, 0.5970078009565675, 0.3845326090103801, 0.8309415462445305, 1.05931384190164, 0.3463905654927667, 0.14516527726519504, 1.0060002455072927, 1.1397561965189047, 0.586593487369092, 0.33240897436643757, 0.3320717319893829, 0.26860566136047337, 0.5122870107201876, 0.7012415612145387, 0.21926141217835848, 0.30488408960116925, 0.33045877156250825, 0.011263021217770607, 0.9185700890983337, 0.7107329993123872, 0.42751069873469505, 1.2084968285126998, 1.2232113194833052, 0.6720071207350548, 0.9388717435489299, 1.040073537002746, 0.32515738460617605, 0.31197536491854555, 1.2652625095029446, 1.1356401164521324]]]}]}'

In [12]:
arr = arr.replace('NaN','0')
arr

'{"predictions": [{"fields": ["prediction"], "values": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
arr2 = {"predictions": [{"fields": ["prediction"], "values": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[1.6543474699192282, 1.0264375101108159, 0.27353033117801734, 0.5562844956803586, 0.8204602935649494, 0.9408279536123464, 0.4497521763158148, 0.5363080761507307, 1.128725745884734, 0.8609090671041016, 0.23472573878252045, 0.5049773983539427, 0.8840075158156478, 0.6971096601297309, 0.5281030870188883, 0.5413923880530157, 0.6374925842767438, 0.6718188505150461, 0.6910731752867305, 0.26012158336914415, 0.2715186445264077, 0.46790206434630405, 0.08326923621258207, 0.6419306859684406, 0.6345864207246743, 0.6615152603367358, 0.899132621038523, 0.5678771349790092, 0.680167062488023, 0.40121907744737156, 0.49722183073254206, 1.048178475324258, 0.48159258663395577, 0.6524289023913175, 0.25167724956073845]], [[1.5122126762460015, 0.9293257283604499, 0.22493292140474647, 0.5639600066162087, 0.8552708815796746, 0.8693252360732665, 0.4226892375430784, 0.5645711909864122, 1.135429186072553, 0.8482029324956043, 0.2425594099229597, 0.6378288526279654, 0.933116303774433, 0.7068787823355123, 0.5981896047585646, 0.6181700374057526, 0.7109515004316239, 0.6872077935823377, 0.7672149857384494, 0.35754144002973115, 0.33130918614630467, 0.4694290983560748, 0.08258921141083544, 0.6160266833277366, 0.6008469137383654, 0.6336105839211185, 0.96187285872862, 0.6972611253234562, 0.6940394366548452, 0.37878933897671296, 0.5481900210931027, 1.0383743381512025, 0.4973307158831973, 0.6427453760024945, 0.2646850170181108]], [[1.540516115898789, 0.942980570792456, 0.2955068380346417, 0.574500126322999, 0.7965907930554126, 0.8396756655920858, 0.44520323377763776, 0.5957620359270697, 1.1006090695410153, 0.8457308166426333, 0.2526757041614963, 0.5870879084253652, 0.9277693981459854, 0.6912793113465591, 0.5273020425400647, 0.6775701055064656, 0.5075736504782783, 0.7016155623523422, 0.8338580072412239, 0.4090494954754573, 0.4005991708134144, 0.4319362009243176, 0.08815193089593859, 0.6171103538016133, 0.5498103819291661, 0.6585596007287184, 1.0010304722719754, 0.748823638520587, 0.6762966245147003, 0.4277147574457454, 0.5592773409103258, 1.0808376578784045, 0.5058662818172963, 0.6252627110882537, 0.2976283813133629]], [[1.4750340885608169, 1.0229131267297629, 0.3032721327965791, 0.56490631942781, 0.5570780442145328, 0.8567746798399392, 0.42019293214378606, 0.6491813578074236, 1.1394703085554503, 0.8480119361995007, 0.255327619314296, 0.5878193597560658, 1.0654724075547244, 0.7052353083294848, 0.5018360157681716, 0.6986638389753257, 0.561090293766709, 0.6954757400274447, 0.7933474209072393, 0.45111291469393844, 0.42481039967560985, 0.4060440833435207, 0.08069916946197822, 0.6242231472258424, 0.4553763073269641, 0.7284680245605021, 0.9554332118370468, 0.7639160685279074, 0.6762160464207794, 0.5595149272877419, 0.5818019533129843, 1.070930813514754, 0.51566138689336, 0.6203991310505602, 0.334413041694994]], [[1.4089573133460533, 1.150575190275386, 0.32265160498909984, 0.6546701053718035, 0.468640826949005, 0.7845689462827743, 0.45658499180840606, 0.6812945608521804, 1.316408172337604, 0.8887420856053625, 0.24628036610978432, 0.5748475538990765, 1.096997203073342, 0.715058305470848, 0.5415076930977831, 0.6538949834532811, 0.49779796469701143, 0.6944701716009178, 0.8470482854778832, 0.5051103545243381, 0.4608160619350128, 0.4092148977360567, 0.06751261957338561, 0.6239766209890907, 0.4955587070953609, 0.7237262539104996, 0.9314115790066682, 0.897713786249432, 0.6582990256535937, 0.6177249573238268, 0.6035238901840962, 0.8517562881080256, 0.5331897441636919, 0.591709256624789, 0.39418165845397896]], [[1.4364876854361837, 1.228208955558335, 0.24352190813887997, 0.7540944602547792, 0.559531962284789, 0.7887141302703419, 0.45869295167482604, 0.7587431955540613, 1.2890481331349666, 0.8601568734453903, 0.2695929795720902, 0.9538011386289871, 0.9907851034350955, 0.6821832648550168, 0.6102472624889167, 0.6124458381809816, 0.5403654346637983, 0.7328392990331095, 0.8361993980562848, 0.4999004639248811, 0.42893057449431526, 0.3419959784704374, 0.04629130447364302, 0.6062287363583878, 0.5276101802439631, 0.6127330832538969, 0.6351670150598662, 0.8547225275165486, 0.6643848101058711, 0.7121977525054535, 0.5880305972943722, 0.830349967863802, 0.548708995188384, 0.6431817683181125, 0.43567634020295587]], [[1.3819888527068316, 1.2093093089983113, 0.27354425659989823, 0.9553964844288786, 0.6657792659385607, 0.7529212012814772, 0.4308978315569946, 0.792231863173713, 1.1899938016688822, 0.8503555296941914, 0.2816674861741961, 0.9333187021035335, 1.0428277725399298, 0.6574505788388927, 0.58445697517578, 0.6124224130270575, 0.5621045847687659, 0.6726316486827613, 0.8447271550865041, 0.5081263378250245, 0.4846225546987979, 0.44204385087066544, 0.05841839063295964, 0.6246693979490856, 0.4831151548817131, 0.6020350687587446, 0.5408488197226862, 0.8031579018462512, 0.6786931380125771, 0.8545697963991903, 0.5854919352025787, 0.7606455228171873, 0.5520753497923622, 0.6934369428381044, 0.4808229124791672]], [[1.301654669265586, 1.1875609943464194, 0.24600752534638348, 1.1990224478928728, 0.7100187280026318, 0.825402489383754, 0.49069385717494296, 0.772903719450668, 1.0671613263639852, 0.7379780778122854, 0.33145412790365814, 0.8869727708399554, 1.041180007765339, 0.6515893666849129, 0.6491279126436796, 0.6885279873274941, 0.507509184986644, 0.6907843754658447, 0.8819796377610701, 0.5228379780386354, 0.5448575228948754, 0.39858118754059346, 0.05521997837984232, 0.6246322557784253, 0.4120756091455784, 0.5783701857861601, 0.444411362248404, 0.8231325592417821, 0.6817027421018585, 0.8661584647938536, 0.6080020726307878, 0.804082126401461, 0.5589006893049064, 0.7266527510036287, 0.5087821256868227]], [[1.3204037374774968, 1.3172271859473303, 0.28578548670224213, 1.3169321772915445, 0.8511323447842616, 0.8045680924929224, 0.4594452822253167, 0.6710472613342131, 1.0370964539325924, 0.6501853167595262, 0.31639176316349726, 0.8421537165129236, 1.0232513050150802, 0.6928807371003225, 0.5783600746648148, 0.6265503534745065, 0.5397992516038124, 0.6884965637833964, 0.8029510426290953, 0.5301003632924413, 0.5632824965192254, 0.4477922366209966, 0.06031329431745568, 0.5341552335010635, 0.36524402641420084, 0.5690497006164832, 0.41181731315511905, 0.7999536789339241, 0.6845459238600957, 0.8519912473739106, 0.6960785443940128, 0.9762326013693962, 0.5757316608306731, 0.7932106404220642, 0.5127127493836763]], [[1.380960454202983, 1.3755329236687381, 0.26895973093338477, 1.4375122422699942, 0.9471218646969521, 0.7942510775735363, 0.4638970905870977, 0.6343621857425412, 1.0201104890948873, 0.5679408417885947, 0.334993715463929, 0.8642467363058652, 1.0331882044119545, 0.7397140070864794, 0.5537882852203686, 0.6315674474698201, 0.5298962208411119, 0.6747324649794473, 0.781393374951125, 0.513612049339853, 0.45040921578239, 0.439338540084852, 0.047373005102116214, 0.4809002139537394, 0.3760406458649162, 0.4509844289963571, 0.40125269190107676, 0.8610041672194758, 0.6885373726806341, 0.9064657665699525, 0.776749955957519, 0.9963036681871246, 0.5909010185246955, 0.8552216535302708, 0.5426807393619641]], [[1.4332665670357683, 1.366819255852507, 0.2536617611736708, 1.4799501008869258, 1.0354598192199267, 0.6743988099744401, 0.43121995235873056, 0.5901478984237434, 1.0604549620800832, 0.5471895408399073, 0.35270175952936594, 0.8661513940451965, 1.0265198213764017, 0.7662525371691202, 0.5670319059800313, 0.6391081948890073, 0.5957467631368585, 0.6900876732817576, 0.8340262764904067, 0.5071993244331441, 0.4330060146359381, 0.41749327339765013, 0.0526447509867401, 0.48693770698013944, 0.4238514586915585, 0.36561821345466006, 0.3856012618038542, 0.8911823487819801, 0.6990866283421673, 0.9590112431907343, 0.7737424621958489, 0.9549895886351725, 0.6040624532620118, 0.9437704055275774, 0.6216240915335051]], [[1.5488404354893126, 1.3382746095368063, 0.26525634850581303, 1.563845094905866, 1.0769085731601709, 0.68321956445202, 0.43422254328303767, 0.4729179730619891, 1.0434038828858947, 0.5244891410955923, 0.4268010483393495, 0.8018216856606045, 0.8042271774479337, 0.775189194610815, 0.5606363199228013, 0.5940995989434741, 0.5714244015933347, 0.6675682565811114, 0.8180409732547664, 0.5454102974879477, 0.41599518045010075, 0.4004060870284622, 0.04454278466308441, 0.5544628761037329, 0.45339892877193455, 0.423703967430584, 0.41188060804144166, 0.906575055134849, 0.7133790929576713, 0.9389990908184498, 0.7388625784473262, 0.9746632140913455, 0.5981043570114435, 1.0132109783095062, 0.6950012841364293]], [[1.65800198929901, 1.2599214420095728, 0.24222183105477268, 1.5783896312101735, 1.0418736025052326, 0.6944074180089632, 0.4472147014837523, 0.4295410590475066, 0.98971625757544, 0.506311650215885, 0.3899884652600929, 0.9297255221878858, 0.7792559044537444, 0.8536184423824071, 0.5685707837463345, 0.6769217717471978, 0.5912539435184414, 0.6626352896607721, 0.8091933050700858, 0.5252744829471685, 0.495312023975376, 0.4205323926774268, 0.05147962678712167, 0.5144979555540974, 0.5187927604486237, 0.5737468351730082, 0.489622910347414, 0.9251009262045053, 0.809057672011892, 1.0107561163845311, 0.7532720247628789, 1.0606408717090636, 0.5969240234367238, 1.0468664724152534, 0.7510632336469949]], [[1.6551338360374683, 1.201026191480647, 0.2502957035474377, 1.5086423475372852, 1.0456329454655107, 0.7309766541952631, 0.4706015762663452, 0.3978593254210816, 0.949487554320407, 0.5106689817035515, 0.2678638697009976, 0.9134751354487194, 0.6826429241166445, 0.7377975626730694, 0.46807804247218304, 0.669602903209173, 0.571394770094563, 0.6957897443879011, 0.8452947461716964, 0.5572000537081693, 0.5247164525840706, 0.3819476841520061, 0.04812391776841667, 0.5025025681624343, 0.5372191084096494, 0.6907720290294468, 0.5543550186125922, 0.9296114868421551, 0.9972808608234558, 1.1294338490557365, 0.8868960895337128, 1.194206401549834, 0.5791540953529548, 1.1143563921416983, 0.7863566476704182]], [[1.7163100036686574, 1.206923435417671, 0.30671618361125375, 1.3852088519262231, 1.0678727275761997, 0.7305261899001617, 0.4451164860574797, 0.3913457576181675, 0.8823686003824288, 0.5281010710594407, 0.13597092640532307, 0.8315557967739322, 0.7147671048813788, 0.6942014110580824, 0.44797107783384765, 0.5770313012419909, 0.5499128563913581, 0.6844717687673721, 0.8207730258963264, 0.5854293366462975, 0.5956472535301398, 0.3877693769547954, 0.04935625383527743, 0.47088467898414194, 0.45456078145769097, 0.7595926075163639, 0.5791596591546481, 0.8624764462874992, 1.102364749511785, 1.2685367208180178, 0.9497125959328931, 1.177461413071637, 0.5686978472628036, 1.0942876454714556, 0.9123804532220434]], [[1.7253993018590927, 1.2544371677745687, 0.30429003089394596, 1.379648435258112, 1.0801083932255648, 0.7319476296093788, 0.4025776046651256, 0.44847350411796666, 0.8535668490963009, 0.5494192243156547, 0.07125215600652884, 0.8567792887863073, 0.7151132681371304, 0.7290446965117452, 0.44739291665678244, 0.5739881352667828, 0.5674385865840925, 0.6737917338916389, 0.8146679266310771, 0.5732317271327252, 0.6281745304996247, 0.4067749696826636, 0.03531978844716389, 0.40239522684982865, 0.4681375184959399, 0.8317472093809356, 0.5492342776077149, 0.8028744455024821, 1.2124443936426905, 1.2016694601402973, 0.7330270303673296, 1.253878951026027, 0.507471647986234, 1.059366123984337, 0.9175931245259445]], [[1.7442907274684831, 1.2389453597788622, 0.28851864141533046, 1.3872132878443224, 1.07093809673586, 0.7372631207735447, 0.3596710138967932, 0.5064100787853424, 0.8353516839515147, 0.5604921479044084, 0.057018982645391236, 0.8710367369295116, 0.7124984141005832, 0.7139578652411824, 0.4650951305903939, 0.5860217771069287, 0.5464910382206506, 0.662320090761374, 0.7571693301427458, 0.5684292379146645, 0.5818391283579422, 0.4185493015525458, 0.028145652536299755, 0.33024555863827887, 0.476727051962316, 0.862609956620321, 0.5480328487912739, 0.7855593253520571, 1.2783742898526322, 1.1959131836417678, 0.687948721239913, 1.3423779440351458, 0.48296744345498477, 1.1324786751478395, 0.9570045976378556]], [[1.6760152562205513, 1.2082867360135525, 0.29171314050017133, 1.2963308867836585, 1.067936556065757, 0.7873076215006223, 0.3997321571234912, 0.6803446795110478, 0.78646607739293, 0.6137472228802885, 0.08032086672140862, 0.8419317207684944, 0.7199888595981758, 0.7410136899449965, 0.46391453123230925, 0.5269937631034975, 0.5542224959452062, 0.6567299878353979, 0.7872499723048655, 0.671556239034601, 0.5912336443884632, 0.5150924020139294, 0.02836216768743189, 0.3896610148551555, 0.533540538561103, 0.7581419045225252, 0.6095365304012801, 0.761634017457558, 1.0770808469887787, 1.1696188154399603, 0.649786856425836, 1.2199950046227364, 0.6331435148366193, 1.121428280401933, 0.9258911471754107]], [[1.54162061884831, 1.221911771569187, 0.28198071598165714, 1.2759084564512784, 1.053931321705667, 0.818398398872314, 0.4078008209040078, 0.8519138241516011, 0.7466976961477965, 0.6375915434861215, 0.0727934053805784, 0.820273497376895, 0.7092276820940967, 0.7573903851099046, 0.4348280144394833, 0.49727383258783947, 0.5400361157727447, 0.6582135482918492, 0.7798591968032854, 0.6363815934969335, 0.46867123545265443, 0.5266593675494077, 0.07906634490941672, 0.4762951618247186, 0.6317922282430071, 0.6921856818353782, 0.5826919569196727, 0.7574732860978973, 0.9635739949891778, 1.0627697792497037, 0.6641227950987227, 1.19979888378372, 0.7888587836942915, 1.1016861041050396, 0.8605446379187951]], [[1.4779992248105327, 1.254826675450597, 0.3004308090956484, 1.3033996543540307, 1.0898974508307577, 0.7286428650830865, 0.32423743233531677, 0.8523905006392078, 0.6849448495159505, 0.6427924340444933, 0.0691315312558064, 0.8794682350722626, 0.700841044216149, 0.7628611662534053, 0.44311015721201136, 0.49313814725363797, 0.5890193108361238, 0.6372623780991435, 0.7633180106310026, 0.648518679013508, 0.4512327203606617, 0.47024989186691235, 0.07135042615530685, 0.5305268904922158, 0.7025341911239272, 0.6811810561263607, 0.5656801972901304, 0.7014582025143173, 0.8559386921029302, 1.1206167132690732, 0.6444398263844968, 1.1556020409694743, 0.8277496356349032, 1.0733116318971503, 0.8629587801765557]], [[1.5140368386217546, 1.1988546050787234, 0.31887828548438, 1.31651156464367, 1.1125441108407772, 0.7070674447473029, 0.2952718315898965, 0.8501846047554042, 0.6268926131896286, 0.6199151306927061, 0.0846078612890378, 0.8928625119800201, 0.6873020703350083, 0.7554372329415966, 0.3803013691559605, 0.49482114529600413, 0.6430052480745909, 0.619552997612892, 0.7921524308228283, 0.645262329624758, 0.46024716124137155, 0.4494730687218108, 0.061769995942656246, 0.5215501387975039, 0.5958666781241615, 0.6126080263783477, 0.49467790012078544, 0.609484582104917, 0.7075617699780143, 1.2603218253574773, 0.6137881048163798, 1.1171291495099123, 0.8998887550331973, 1.0913538258280557, 0.8284130649581933]], [[1.4639178284248149, 1.1686265196608319, 0.4630513912860707, 1.3065480594011643, 1.098564194900597, 0.6966940662754223, 0.2597485909300895, 0.898806864508489, 0.545455105110696, 0.5449693484854996, 0.08062696169273117, 1.1069129280825865, 0.6152252803479321, 0.7091288060703901, 0.38719683556252177, 0.5471338100191538, 0.47867422375015006, 0.5876717966747598, 0.7228849033085445, 0.5098170374144032, 0.42931236810290496, 0.41786940391716143, 0.05719730931860684, 0.5324966132805518, 0.49254438547802276, 0.6298160213903994, 0.4498231088049116, 0.4895957181440471, 0.6453108965631972, 1.4377010406781396, 0.5624668117972014, 1.0314926966544276, 0.9292934078988413, 1.1384869703569074, 0.8543370473380182]], [[0.9080430531243561, 1.2210838659853818, 0.5173724350534976, 1.2918629002542674, 1.097687827640418, 0.6969593192633172, 0.2962302749370276, 0.8728555445170127, 0.5300925071707561, 0.5200075774689558, 0.08463259366348191, 1.1287345105776532, 0.5771360113369207, 0.7058378606897, 0.35461387048356835, 0.5619263637198046, 0.460849370999845, 0.5981706887323346, 0.8224362832010814, 0.3707899700895, 0.3758187605680269, 0.36130611650946987, 0.059508216659862674, 0.5466965837702417, 0.529203333201244, 0.6282355421591596, 0.4121201016337718, 0.47676915484048965, 0.6422620762554725, 1.4272210523440043, 0.5480157360059268, 0.9225506810509374, 0.8611016250680673, 1.1947877334216037, 0.8497662960121178]], [[0.9332419673698307, 1.2582089258037228, 0.40768657312343815, 1.2276951123076996, 1.0592536723363781, 0.6404285055247498, 0.28076343572164664, 0.8922967789535643, 0.48749757731169174, 0.511623633171927, 0.0750073744690623, 1.1393592791828442, 0.5629392420857822, 0.7221165566661976, 0.36693221930154224, 0.5959918267391056, 0.44341267785879723, 0.635148922621486, 0.8690661955312035, 0.3875016715814151, 0.4156299084770029, 0.3999957960354864, 0.06491860762399104, 0.58530268732961, 0.7471986833906323, 0.6392542476854624, 0.38893981687278734, 0.6126193785680245, 0.6437680716079757, 1.5051616827411345, 0.3014456864649281, 0.9591168162779784, 0.930397006314287, 1.0617454336885759, 0.829654138673065]], [[0.9915653900220764, 1.2306265994327712, 0.3874664416798764, 1.2445198404571314, 0.9871742993000784, 0.6464213466692001, 0.28061712440782627, 0.8994310859671677, 0.5542744171783366, 0.5520390227899306, 0.08108752648837572, 1.2101717695583922, 0.518240128387562, 0.6882970372838348, 0.36267501948499214, 0.6032002481741845, 0.40418004134612995, 0.6414219035801372, 0.8869043744769576, 0.371127589686524, 0.44038224061746284, 0.34136705252937904, 0.073057611796145, 0.627853698523156, 0.6449474961123871, 0.6238015233385394, 0.3434257425645708, 0.6116290366373298, 0.593700474755458, 1.5574505690974574, 0.2759204816184756, 0.959246298503359, 1.0072907285071373, 0.8810850031137578, 0.8007557750613958]], [[0.9802339564376619, 1.1984828384410355, 0.47859220745391506, 1.2279475276344942, 0.9382897908971353, 0.6481013796559913, 0.3013042124216663, 0.922328199375132, 0.6696650937533053, 0.5821298836842157, 0.07585791265510604, 1.184810020316983, 0.5081428459915367, 0.6916390979084589, 0.36891469861359233, 0.5447204107336551, 0.4255287713102277, 0.6106263053018672, 0.9146652080289003, 0.33009865814126566, 0.40667787601446215, 0.40132223819469975, 0.07124698987408623, 0.6481442158866915, 0.6607185920675496, 0.6200625439938252, 0.28464356634128296, 0.5724692985134603, 0.5146430618725388, 1.574782151119439, 0.2605668918323183, 0.9428518145054783, 1.111556465142249, 0.7980235890401692, 0.7470868135242383]], [[0.9112472525456173, 1.1715571180153097, 0.3667000897813737, 1.149313464196318, 0.9500047917254899, 0.677510949676696, 0.3372226264015239, 0.9435735214139216, 0.6821265714623845, 0.6097091706676613, 0.06342727090059902, 1.107368831981649, 0.45798525517737804, 0.6523140502658753, 0.35679993835009627, 0.5342641304330582, 0.40787473775976707, 0.6008606715858529, 0.95568852404185, 0.33253482333331613, 0.4264217436513031, 0.44689586882893817, 0.08130934364230653, 0.6664964373999639, 0.6223243087590814, 0.6098815771095802, 0.28223760665215775, 0.4515901046069112, 0.48914803981169486, 1.5071594244612379, 0.2953440538226705, 0.9101822969410417, 1.1400890536621118, 0.8032636119188272, 0.7105273623112223]], [[0.8632785935373528, 1.1137039278369856, 0.35905623217287796, 1.164349552379551, 0.9169489087908065, 0.6534388399415796, 0.33605567776780104, 0.9771067349571734, 0.6607032286880595, 0.6089829631094564, 0.04787675379287771, 1.1421067582537643, 0.4605114617961385, 0.616907993736351, 0.3616332757436419, 0.5555899308044047, 0.3721100735137266, 0.611911867488869, 1.016128552878743, 0.3426563493765965, 0.4216552235017624, 0.4488232685070904, 0.06623221621032593, 0.7084583265951453, 0.6474386143797822, 0.598800325111644, 0.28628002750700354, 0.4234058719178958, 0.43795555256983343, 1.4644415861312754, 0.26178151475386957, 0.8368099854750577, 1.1566568104733026, 0.7873309596845088, 0.6911022627041987]], [[0.7421513681058227, 1.0625907069052656, 0.38251216497527407, 1.1011391264069064, 0.9095318479017542, 0.684161030361444, 0.3682903869454279, 1.0103925356827397, 0.5841910796362728, 0.5729312031163556, 0.04317790760932457, 1.1446370001896877, 0.5364475894469918, 0.6556388961415198, 0.3584162270110019, 0.5435802060137368, 0.35526024170731746, 0.58513030998508, 0.9774914248532502, 0.3784443725645946, 0.413021615745096, 0.45222283009009073, 0.1025289135225091, 0.6880784927690156, 0.6261324076530519, 0.5460217041543594, 0.3678343226918326, 0.3950302320377836, 0.37298385157099656, 1.4312098037597607, 0.2708068198631924, 0.7331657912001981, 1.1741472997419962, 0.6714369001945024, 0.6381824238046603]], [[0.6909003282074692, 0.996238485320423, 0.37828315831549075, 1.0688104276442294, 0.8758883482717815, 0.699849322158337, 0.3713999834307175, 1.0028555978133589, 0.5937056768866911, 0.5475529865732655, 0.04525530755344058, 1.1238638248557227, 0.4767334648019377, 0.6262863538111307, 0.3730781405562778, 0.5834933732826162, 0.33842521959841626, 0.5744803491921182, 1.0015760989972595, 0.3901416080957489, 0.40487694011928527, 0.4474820088569797, 0.13262865335103566, 0.6379761478101715, 0.5192151183724188, 0.4444977863221721, 0.42596216571463164, 0.38458582929792406, 0.2988052624351062, 1.3773429619303372, 0.26815495185040317, 0.6017862215505221, 1.1760228709864542, 0.6417260480097915, 0.5663887835294785]], [[0.6729497912337367, 0.9813014767294512, 0.34706325541175514, 1.0497143278896908, 0.8925712716857547, 0.7289440571089596, 0.36693690117371314, 1.0427978407845164, 0.49928163874691456, 0.5526634825421252, 0.0495332706169597, 1.168378861302159, 0.4286393024413374, 0.6051084913173811, 0.3812103890253999, 0.5408733596636874, 0.39986402110810276, 0.6191505398892574, 1.0617028879236403, 0.3754366738101995, 0.41691013838110186, 0.46904580066546653, 0.255931025784223, 0.5038770695794157, 0.40568013443064277, 0.411132749282708, 0.359018353483773, 0.3835393826333901, 0.3125022788706272, 1.4395641843934492, 0.258109090336487, 0.4964290146160252, 1.207124591758348, 0.6432973633938315, 0.4374859730240115]], [[0.6447188749870104, 0.8964623019212484, 0.3400913767461774, 1.0473460692681278, 0.9310616412205353, 0.7160780218164706, 0.37000567767803, 1.1403109378772533, 0.4657068038391924, 0.5774297645725748, 0.07285849892780508, 1.2550136821453848, 0.3319315619469937, 0.5813638823465884, 0.3998814898781539, 0.5538524146367236, 0.4683674254861139, 0.6586734404907535, 1.0545410309336756, 0.3484575320483221, 0.3787609431173685, 0.443721517538256, 0.1539069517646475, 0.43967232056947503, 0.39036691644624477, 0.3939729204662259, 0.33670598706102756, 0.3996346999325424, 0.30099300999256673, 1.4004833062274693, 0.24941508653736502, 0.5743466474802232, 1.2292484397737118, 0.7096784683432249, 0.5525698620738271]], [[0.5940399273255207, 0.8313342832475092, 0.3476543905577947, 1.0433151044717488, 1.037640155207866, 0.7173855993779044, 0.37438257530633184, 1.1899011698902502, 0.5227447648394734, 0.6397102487490853, 0.09774921992563529, 1.2207376721690621, 0.26858128891969246, 0.5859397861171269, 0.5075341096014171, 0.4967587619444943, 0.4331344144274709, 0.6630537922808125, 0.9148741763613394, 0.2912298716367365, 0.31831453437052126, 0.42520860551761797, 0.05357177819436379, 0.4473049873212588, 0.41906429952094393, 0.2922033136329634, 0.32704547320229077, 0.4359562477928582, 0.27099767525002716, 1.408130822145337, 0.22850205878597923, 0.6107541149437404, 1.2364916495176081, 0.9942093977848522, 0.8052201670554731]], [[0.5731865542731053, 0.770432741621053, 0.35664243918725813, 1.136362461355693, 0.9937288267094184, 0.7055830070564733, 0.3734513494291034, 1.169180072898216, 0.4830715106264817, 0.518618157378141, 0.07737672414333419, 1.159837982621668, 0.26289106771368276, 0.613173162702912, 0.48936717979151195, 0.4790384541814716, 0.43407481344330046, 0.6878936026847395, 0.92687949369868, 0.2498233058091923, 0.3080700828391233, 0.42092009810160036, 0.06490182180090356, 0.47717077563660637, 0.48394343531862954, 0.28107791337784294, 0.3198904828354341, 0.4796273373929929, 0.21157969162765056, 1.4332260952505507, 0.2124686746491535, 0.6526276114164253, 1.2377652654172686, 1.2790721218659806, 0.9892001563938162]], [[0.5448051077934842, 0.7339559044976898, 0.45264177545375084, 1.174447072758197, 1.0025424449034703, 0.6948194836931205, 0.38401822366181443, 1.1248821063768448, 0.5445540675643042, 0.4963372985123454, 0.08464645782851186, 1.1474578868290628, 0.26327051911918237, 0.6782787028751704, 0.5013315010311931, 0.5234398813209697, 0.45872210094568666, 0.6672788074782694, 0.8579276983382333, 0.25670607889654995, 0.30601935317186263, 0.4286344400560406, 0.043350908184085185, 0.4965982479772051, 0.47335857483216914, 0.27353552345420323, 0.3259240582034612, 0.45425724567471665, 0.20370187044016438, 1.4122773454980728, 0.22466179246168597, 0.667598465136595, 1.277317618298582, 1.4448483266346384, 1.1054436141516184]], [[0.5209884410156136, 0.7102908136183317, 0.5063923746110082, 1.1824410988312029, 0.9637178001130469, 0.6825514955129723, 0.36184712928477364, 1.035469236142942, 0.5365378916847953, 0.5356892347493079, 0.08859348759966701, 0.9833197791692858, 0.24558274856800283, 0.6923939992010966, 0.5732808335331304, 0.5680011296581946, 0.4234167634515371, 0.6860128298699881, 0.8900148780825252, 0.40579549814731947, 0.3086882386689658, 0.48331337596672586, 0.06697007666712611, 0.5157774867794249, 0.5386011329784511, 0.25149781398513116, 0.32669176155422996, 0.45087756557945535, 0.19767516967257875, 1.3633025633081164, 0.2868453624199463, 0.6614350328944836, 1.352848647967802, 1.532073328728237, 1.178536470899748]], [[0.5045938603028605, 0.6846623061447654, 0.46214283918610044, 1.1404005147607599, 0.9392396802961412, 0.6886487357804374, 0.3556056639580659, 0.8897854265914684, 0.540537526668912, 0.5020171072687949, 0.08743849652824515, 0.8509697066170685, 0.21948548061468193, 0.6083186516113592, 0.5550980553920166, 0.41119653749240037, 0.39318922570213344, 0.6645460165447995, 0.8604857918411422, 0.5252770037733486, 0.380923245184955, 0.4203984316760855, 0.0303159266732957, 0.5027575425886381, 0.5535347963406018, 0.25154779666525484, 0.3361533527502545, 0.4550675411379245, 0.1597848083769775, 1.2915622623812437, 0.3611797280607404, 0.653677349152319, 1.4453274217074692, 1.627709357556271, 1.2392804361504417]], [[0.48154011028019056, 0.6146250179892981, 0.5135241513421561, 1.0737325392299413, 0.7720314158970656, 0.6947498839337227, 0.3479956642442047, 0.7441193155003676, 0.520137521687878, 0.3396494720720614, 0.07685373355433175, 0.7558553644202664, 0.22041586066942928, 0.60395919374239, 0.5209395355064335, 0.3962807077917738, 0.41944172563934623, 0.6341806197420362, 1.0783671481097905, 0.5178395151895685, 0.47811128464183206, 0.3466512053940214, 0.05762828719961395, 0.47579242066749067, 0.5767931516976149, 0.2597046092809322, 0.3549513837480008, 0.45416023180244736, 0.16061737184805283, 1.2533499282051948, 0.42916091733857353, 0.6460385601870082, 1.5029031786613607, 1.5660210793191776, 1.2729681356016505]], [[0.4550046612491876, 0.540093962309734, 0.5091335805066283, 1.017520646002188, 0.714160072200811, 0.6908656106744615, 0.35573999778567555, 0.680771216981748, 0.5609012232406012, 0.39449921292563506, 0.07770403499802325, 0.7780442150980417, 0.22151564018403655, 0.564059209584911, 0.521395265313693, 0.4060938476091698, 0.3741059380562769, 0.6010659943324869, 1.0660200803798576, 0.37007301248436747, 0.4833521169245528, 0.3036223357293619, 0.02170244187475875, 0.4280820949898124, 0.5681653459953171, 0.27178631466004444, 0.3737913531415214, 0.4522944492806964, 0.16310092000946672, 1.210098714167312, 0.4860204290221357, 0.6192957058308113, 1.4495395411366037, 1.5981990382663473, 1.37852206853955]], [[0.4105719321333737, 0.49815035385539863, 0.5142871826685392, 1.0179395685021568, 0.8156461087634463, 0.6947888304027657, 0.36254023520726697, 0.5994297886076113, 0.5745354856276887, 0.4637076179634767, 0.07576867860656966, 0.7543351057346843, 0.23597581637169762, 0.5498861925628626, 0.48937393599112206, 0.4117835589835523, 0.38740652246935176, 0.5780091548605845, 1.0975312721842392, 0.28341095578407427, 0.4448916163893031, 0.20735035526196832, 0.018594735965981928, 0.3392296273524701, 0.5391238866883646, 0.2533700071894155, 0.35649691098983893, 0.43474403264075356, 0.20074510840976628, 1.1880950449976493, 0.5066883574810763, 0.634421920942615, 1.314167590396152, 1.6364406835642054, 1.4712943555945304]], [[0.34332833567389026, 0.4750399652924402, 0.5219887632590593, 1.0706683326598254, 1.0264439777224637, 0.6964307738221662, 0.40205854239844585, 0.6863827409747676, 0.6079306481053914, 0.518225199101261, 0.07600359972953746, 0.6844571640492869, 0.2442565900892693, 0.565894203920646, 0.49335252786192846, 0.35487566400319215, 0.42037885558189464, 0.5319269807854747, 1.0749648823076547, 0.23042309923016258, 0.41255058018795154, 0.17403134151715216, 0.022792576698548832, 0.2777726473140928, 0.5125874459050987, 0.2398724545442838, 0.36472100752482856, 0.44075710998021156, 0.25539764612406934, 1.1689893046357478, 0.496808582350012, 0.6303377914707031, 1.2327972121106083, 1.502929743996432, 1.4173381082999825]], [[0.3202331525281426, 0.4701201435526893, 0.5302019377210064, 1.042344847577205, 1.058193002714939, 0.6662686108054161, 0.3963486351391605, 0.7246086996771017, 0.6127307581584949, 0.53452885630068, 0.08279249952563467, 0.7096052312428365, 0.2431617141310363, 0.560896864186551, 0.48865268154700653, 0.35019702251253737, 0.41821785482041113, 0.5208982805596185, 1.0574933972503415, 0.23542057845620143, 0.35970177518253466, 0.17902929605974538, 0.020212632460214767, 0.22722483698441726, 0.5534240363922582, 0.2605136359204959, 0.36753590780419504, 0.39040832517373425, 0.2752681149280519, 1.1021923442869788, 0.5169640256969738, 0.624152199323929, 1.1871552175988902, 1.5194693456325274, 1.4905219510278154]], [[0.28970607635187895, 0.43308104654212287, 0.535478103695146, 1.2042410798237522, 1.0784863677594543, 0.6880608304366227, 0.4058982924587272, 0.7188051522110277, 0.6480648216846652, 0.5879868253794776, 0.06389395210827502, 0.7202863723867321, 0.24736789082935334, 0.5466439384588715, 0.5073190289976888, 0.33525558374894937, 0.4089650714126952, 0.48971455317366386, 1.0644680819478798, 0.21446662856843263, 0.30199397439385384, 0.21292476971078644, 0.020568184467664417, 0.19534470238434232, 0.5980545451998172, 0.2928836436067482, 0.35942594448684473, 0.3266808643146158, 0.27370531247037805, 1.0036881024803326, 0.5464941648111761, 0.6029325446173484, 1.1439473945541487, 1.5492031367104449, 1.5708962447019874]], [[0.267130148887148, 0.41068495827966756, 0.5136795649067547, 1.4221003621076649, 1.1112143698827166, 0.701545923602698, 0.41492323666401465, 0.7003370921844686, 0.6372550103299846, 0.7330408274794256, 0.06467814978472872, 0.7759523447067829, 0.25023939304243337, 0.5636682440384823, 0.49251989111829486, 0.3391899116725797, 0.4242951856872238, 0.4763278617735265, 0.9490882351341753, 0.2159715342196102, 0.26403117661228553, 0.2290659431667731, 0.02069243124964966, 0.1597042808826652, 0.6362648107086484, 0.30936313595869075, 0.39189231037212235, 0.2618808408576393, 0.30064570024548304, 0.8729785949938587, 0.6059797704130071, 0.5582211868797884, 1.085084652931524, 1.4981071486423054, 1.5512850366734208]], [[0.24732859942292945, 0.4321377546687769, 0.5068382638021611, 1.4917908458094096, 1.1093425404945942, 0.71357006821108, 0.4184974965305536, 0.6477790684860678, 0.5521260603736792, 0.7278868726929261, 0.13815631113510413, 0.7821474145878127, 0.28059555778441914, 0.5615684257177491, 0.587077440624839, 0.3473843417340901, 0.44894099920479213, 0.49934396769289835, 0.9842333015883895, 0.1948955355683077, 0.2206150418123697, 0.205151121143361, 0.024191945841235782, 0.13452434783353442, 0.6569742765824925, 0.3234828873303275, 0.3975367812445079, 0.25637519910156203, 0.33934575423422275, 0.7727719589304747, 0.6580205125302538, 0.5143510541470426, 0.9855656223670133, 1.371482510377378, 1.5325101203140132]], [[0.23703567365464484, 0.43135133409557497, 0.5144602446425139, 1.5230156230332517, 1.0956211495304637, 0.6985123083095891, 0.44572146420934317, 0.6646438990657126, 0.5758791553243092, 0.7898589143228562, 0.11959436082759436, 0.7511620765011746, 0.2867709245426123, 0.6010210476352731, 0.6621146318620623, 0.3234286044871785, 0.3930132025667942, 0.5013735874092191, 0.9537282347127738, 0.19463449427296103, 0.21929059992957675, 0.1894425923697201, 0.027663802902965884, 0.10756572300656846, 0.6746663544801954, 0.34137393345749045, 0.42515498539493546, 0.2731599172642808, 0.35983856634165673, 0.7296474877137629, 0.7431138088069613, 0.5457728347941965, 0.902899010815124, 1.1954048198454048, 1.4598799361149004]], [[0.23686884401196118, 0.42887840250685544, 0.4954309290165353, 1.6997378635597633, 1.1153986323739784, 0.6905679391230921, 0.5019429243651445, 0.7179050797748061, 0.5767553884789394, 0.9400836431516557, 0.10079674453455638, 0.8034207086022719, 0.30046626979827157, 0.5933874367947081, 0.6337166245656954, 0.313180243572272, 0.3961670656986631, 0.47799380347355136, 0.9435991939170363, 0.20295536076401283, 0.22589390567658313, 0.14778961322482576, 0.03487533851155926, 0.08449073564239848, 0.7588672588845218, 0.3655530182279142, 0.44811919019843444, 0.4664234536585258, 0.4427396584269729, 0.7262274373392259, 0.891073312135161, 0.5693625300317904, 0.7927707270411748, 1.0688483432736489, 1.4264829317416143]], [[0.24299179953672484, 0.4339185026071478, 0.5039757428622613, 1.658646556133368, 1.1506755351985634, 0.6989655501014216, 0.5048102444502492, 0.6289728239142446, 0.5615907237745146, 0.8672346653591141, 0.056645471159872396, 0.7867733758386695, 0.3137248103667416, 0.5894427584009451, 0.6272607146981231, 0.34117356070642474, 0.4126803512017869, 0.5027494110907157, 0.9593349736635837, 0.17089757910048545, 0.19592129762778765, 0.1376007251865437, 0.04509640995057326, 0.09185257452543644, 0.743640199028115, 0.44851885527770285, 0.47404296521071326, 0.4673638661619865, 0.511739789513884, 0.6950296962802699, 0.9087425284587487, 0.5990547624716498, 0.738158751794521, 0.9259821937745216, 1.3360302154203745]], [[0.24679036693813863, 0.4315586541201337, 0.4804978203852703, 1.567431381161573, 1.2952729674385586, 0.6938665335547832, 0.5073887108148186, 0.5511677600808031, 0.5560433126087114, 0.689559684498457, 0.048057721527297474, 0.8306082780130726, 0.35698800943254244, 0.57752897740057, 0.5811494022541972, 0.3785254568309892, 0.42328171753406735, 0.4912318128792925, 0.9584576948628832, 0.17327578232126642, 0.21543958843299021, 0.22452306659234972, 0.05069452012217156, 0.20752945124325214, 0.6870700263255567, 0.5531729275299546, 0.4722701892611435, 0.4547521251717645, 0.5880918832105835, 0.7474518098527364, 0.8960263502514201, 0.5699049115102475, 0.7198042524279864, 0.8682335892438438, 1.3537864851292625]], [[0.24807943985002473, 0.6643242018785734, 0.4234768629344722, 1.3152202109196982, 1.3157439939940112, 0.6678832466041171, 0.4561753521844006, 0.5543487893478438, 0.5121903021965729, 0.3783968563892269, 0.06825338152599636, 0.8771873367321122, 0.5579369286425282, 0.6042336556847708, 0.5180914421164211, 0.3865846036925705, 0.40996851503315623, 0.5255683172362192, 0.8858633346014357, 0.16823686191952497, 0.22974222098653355, 0.3846155972274971, 0.05005016856473052, 0.3353558295669143, 0.5847350451002655, 0.5680822743053879, 0.4664596048971224, 0.4651026029033246, 0.6256366254583476, 0.8642368045594796, 0.8842143343507042, 0.5916083949092843, 0.7119433678156251, 0.855433435051842, 1.3152688229731164]], [[0.24524865469996943, 0.5298664425313673, 0.44292878225881027, 1.1928809136591856, 1.2919089207900525, 0.7172430351324304, 0.465755380340106, 0.44840458650883397, 0.49024579474127555, 0.3283949091091909, 0.06106492735973343, 0.9558503197282942, 0.5316972189332254, 0.5433251894020074, 0.6845551706608337, 0.2809697576728302, 0.4828360868152809, 0.497160008785116, 0.897351351200976, 0.17508926201114405, 0.22249373072140505, 0.4980402673167671, 0.046735067722196995, 0.406304812866362, 0.4868555535306639, 0.5553974593000317, 0.46598771431117514, 0.5319309388609444, 0.654426969086617, 0.9595718305875166, 0.8881707003092096, 0.6921602489649242, 0.7601640532122914, 0.8548102772597334, 1.2968364946871886]], [[0.2776724568564168, 0.5253248918103034, 0.48450037747753916, 1.037637600539969, 1.22144303304441, 0.8192839315596764, 0.5575378566688214, 0.3700438969346243, 0.4616293384434084, 0.3057746855119705, 0.006508741592810162, 0.6671158332107034, 0.6796723097227824, 0.5423638231923232, 0.6273895927412168, 0.36709335944634847, 0.5604258799572022, 0.6786557940896637, 0.9456920899350143, 0.16963170629507063, 0.23823439477089098, 0.41370026408307564, 0.03571915589967607, 0.47104073877079045, 0.44987673551814966, 0.5452945802974456, 0.5228625457300557, 0.559907175049891, 0.6667743100487735, 0.9952728286678844, 0.8461214641435357, 0.7269055920395442, 0.7500320600291905, 0.8542008509569302, 1.2686006236545007]], [[0.19239226495172132, 0.519256632159786, 0.4513207681597592, 1.0330790246598216, 1.2542528682576035, 0.8554759339354445, 0.5412290649362071, 0.41105085473432357, 0.4733204369246829, 0.3137987220781441, 0.007060606227727923, 0.6764424930781998, 0.635760020969016, 0.5466129185013062, 0.6148920891062973, 0.3199311577011992, 0.6417574532178101, 0.6442749281336388, 0.9457667768166148, 0.16156288772708907, 0.22890814011363192, 0.2771391856019426, 0.029703487094724645, 0.5552829234092109, 0.5418402710014496, 0.49171066608835856, 0.549909682653005, 0.5643340281656175, 0.6627511260171569, 0.9870720218247061, 0.8347658021580132, 0.8726867644555558, 0.7386926322334425, 0.8717627639341929, 1.2650817047517984]], [[0.19338695398412564, 0.522325931748095, 0.400321923237528, 1.0875023228411975, 1.2580029244944706, 0.8909707899488823, 0.5633672218966485, 0.5280930736712109, 0.46025916822445384, 0.2927712087690098, 0.039528383697566415, 0.6149473760661647, 0.6969048490916196, 0.5125700332589802, 0.5849380905815218, 0.3334328278423454, 0.6286330464652561, 0.6906348006521947, 0.9140982288058057, 0.1634083822435605, 0.1992057674197579, 0.18996517270500657, 0.025164480230895132, 0.7533681636105125, 0.5746087894904337, 0.4412735289194232, 0.5382835417796384, 0.5799757530882046, 0.6704145407152148, 0.9757381727230403, 0.8040775226741387, 1.2566168189505778, 0.6941744667944524, 0.8463784020084009, 1.2527949541631367]], [[0.19635531407268153, 0.5317543748580347, 0.34791480607308845, 1.0232733503385303, 0.9121708746527195, 0.9787374740225551, 0.6330058952865227, 0.7073844650154303, 0.6941427557442811, 0.30130533644131074, 0.040851450377307405, 0.5877084324171398, 0.6771751412203573, 0.5076829271429669, 0.4945278655142896, 0.3275096654121179, 0.5934278086267106, 0.6903958419717315, 0.9996254261792323, 0.16649648777894283, 0.1884527729257035, 0.1476473980208004, 0.027998883942818756, 0.8925497140581524, 0.6193782994816063, 0.3836191989622606, 0.5794311984154148, 0.5451495905900281, 0.671413867683218, 0.9165861349533968, 0.7741296419145477, 1.466305021707976, 0.6399419980042811, 0.8235729956526212, 1.2434159115445502]], [[0.19192689738400237, 0.5883978878927263, 0.3505413920483978, 1.0246522365098298, 0.9949366251249635, 0.8790480733389805, 0.627385571385987, 0.8646645208447286, 0.8088266763825336, 0.33042480860479495, 0.03522769448861485, 0.6087097647095192, 0.745424939872869, 0.4959525461035084, 0.46574686662954656, 0.3229479507489768, 0.615417939168554, 0.6754019641967963, 1.0317690530626107, 0.18488987659259615, 0.223447473336112, 0.144379860640048, 0.029267171640614507, 0.9166633892091731, 0.7414788953684642, 0.34612546075556266, 0.7172973778515332, 0.5377875809168928, 0.6234676844954126, 0.6781987924110824, 0.7623959722588016, 1.4896210372132146, 0.5561686458805029, 0.8197088920614417, 1.0189841022080603]], [[0.17353231865577545, 0.5603252882054507, 0.36833123358712005, 1.2375957263051984, 1.0638279645244322, 0.845933968294245, 0.5502743723481736, 0.9454800487197885, 0.9634866097702548, 0.352145753461011, 0.047356048899908, 0.6334943977259139, 0.8014207189499376, 0.534610695214722, 0.41527783270967644, 0.3112019433515263, 0.6476789030288935, 0.6689278157271008, 1.0919744279763859, 0.15888705811518644, 0.24237519242860223, 0.16478473952938566, 0.050263671727358746, 0.9267110622990595, 0.9042834393556295, 0.31313158147374803, 0.7589015965705936, 0.5396796122265655, 0.5310222779563685, 0.4823514226349833, 0.6574749903412894, 1.4950203183672657, 0.3971284665981021, 0.7284505953428762, 0.7310782894909633]], [[0.18266788121659244, 0.5867517761234519, 0.3708420520316207, 1.228864561384903, 1.0597700980877662, 0.8441815902697871, 0.5641306586527708, 1.010166074790345, 1.1077232668576174, 0.3704010343818877, 0.04604661286737164, 0.6086342135174678, 0.8365951742943848, 0.5155065569257267, 0.4199629244375952, 0.323378627100233, 0.6453408103015551, 0.6794446222384135, 1.0705611958772705, 0.16262818127555168, 0.2514989403452893, 0.1732923232590491, 0.07296431095095046, 0.8954701520743993, 1.032913280277383, 0.28800555989292287, 0.7422171514650096, 0.623021806404871, 0.5059266953804044, 0.35594661503714464, 0.4831655304473588, 1.4586144584532152, 0.26959480446434525, 0.6018773036426806, 0.5928311824974051]], [[0.17899953748913605, 0.7072638668977139, 0.35366147101756124, 1.2935522731293276, 1.1530755376146675, 0.8794440571357713, 0.5217988101445213, 1.0326843126577814, 1.2148616193415367, 0.3573452804763904, 0.05278304295729911, 0.5757791288321779, 0.8358893981944533, 0.4735091031634687, 0.3706226751221593, 0.3187102045390996, 0.7203884945632153, 0.6925141974445383, 0.9818781046089132, 0.21495524071249383, 0.24658917840458086, 0.2171355476863604, 0.079233010906144, 0.8767798309566835, 1.085816517430536, 0.26487316736587674, 0.7247340112270377, 0.7483604483037397, 0.45995839555009926, 0.4358462102193108, 0.44768140220859276, 1.4715142560146242, 0.3466296766082039, 0.5522245111371746, 0.6005246862104878]], [[0.18017815642291518, 0.6970258667933273, 0.38710625889953965, 1.3404282895185573, 1.1980588631795843, 0.8904258821945528, 0.5403629115755764, 1.000292878819298, 1.270779246407594, 0.31946171244366856, 0.05729653220101553, 0.5702675466334746, 0.8466679815283764, 0.4680548630986986, 0.3555400173964193, 0.393611950995557, 0.6992347868607212, 0.7657002580550243, 0.8977837470962904, 0.26015477093115, 0.23187069203743815, 0.23731651699946013, 0.07889388300703708, 0.8055070536484517, 1.0886141485637626, 0.2002869454785495, 0.674015195714863, 0.7612419015362978, 0.2970320758293642, 0.5081958434561641, 0.35216409723592657, 1.5347430445221222, 0.4371210065316709, 0.6339066919412486, 0.5273723625647728]], [[0.23598346078488097, 0.6103040582032417, 0.40289501919867643, 1.3182258562329339, 1.2007717413597883, 0.8911957807507083, 0.5432145029332014, 1.0034728924069272, 1.2402855628954006, 0.31162349776549547, 0.031782725176951174, 0.5790599507619772, 0.8716845749020096, 0.5422470825525634, 0.32935819134010536, 0.4021825034957034, 0.6417799879174211, 0.7796048281316805, 0.8635272035223598, 0.2715291668081533, 0.22586759690009636, 0.2528076284892222, 0.08509339234065463, 0.7221840057162594, 1.0605879875861353, 0.23509245279958382, 0.6909907479678011, 0.801267908216753, 0.27849336097394295, 0.5752535122050404, 0.36708363601056104, 1.5364299767080234, 0.5400023762495105, 0.6879139514101296, 0.5457056598376253]], [[0.21449323857629896, 0.6367160584724866, 0.4015702766697288, 1.3050097074568217, 1.0488388426381225, 0.888913326956853, 0.5048078697308891, 1.0106022935365864, 1.238971414059307, 0.3190472138966406, 0.041274932324225064, 0.52295820012727, 0.8687200770631036, 0.5322593642127331, 0.3334967164620151, 0.389846474459457, 0.6983365035772879, 0.7726481490265307, 0.9237678234149344, 0.26411073514955824, 0.22876652431497496, 0.25122802201777517, 0.09357951456448138, 0.698022261949713, 1.0392092263606127, 0.4374230716909577, 0.7319425495187541, 0.812557735978392, 0.358436069398418, 0.6798950352643389, 0.38969026483697444, 1.5300422395418973, 0.6879303098093037, 0.7548843550063241, 0.6844526195284344]], [[0.2072249040184065, 0.6783193217433852, 0.4019997777471474, 1.3106623303264213, 1.0968752674376665, 0.8675122090929619, 0.5125948866763741, 1.0222401595284787, 1.3428690760112034, 0.3255016394424044, 0.015318510688253878, 0.532816552307062, 0.8626294753571875, 0.5457380046329037, 0.32249960263314525, 0.37973401573094223, 0.6278425813078221, 0.8185392522283753, 0.9415970553446842, 0.27791191092210255, 0.2285541757339943, 0.25352628448402115, 0.09398295127961802, 0.6410665765183667, 1.028895919655911, 0.5354195297353115, 0.7211714447540973, 0.8701776883277393, 0.510608593705101, 0.758769384741336, 0.42705394046783063, 1.4584541658296157, 0.787777710553628, 0.7984072111679476, 0.8428706048955388]], [[0.19450658174139335, 0.6819691006269191, 0.41937207785938246, 1.2756187136388943, 1.138511198266543, 0.8577513714613338, 0.506456078502137, 0.9945515082974588, 1.4615269036085847, 0.32777038897133615, 0.02815449662260437, 0.4758576645581014, 0.8376499871797229, 0.4717087302854775, 0.31452281224954287, 0.3766649665036459, 0.6293232279474537, 0.8165923725458899, 0.9558724864678213, 0.28990920400254416, 0.24940272575232675, 0.22508349500297942, 0.09870820248551371, 0.6089996652997435, 1.0585567151860038, 0.67929317192159, 0.6982987735848512, 0.8671115673992111, 0.5780683622496505, 0.8458863188127937, 0.421621305520379, 1.297020251871898, 0.8151040768409101, 0.8433742004156981, 0.7802235257620365]], [[0.30634081935147256, 0.7748374093735039, 0.3575913632584199, 1.3057208476341466, 1.2102133789189269, 0.9219186864138023, 0.530458726609957, 1.0457165082668176, 1.666922137319795, 0.33975231482655743, 0.03412587968533551, 0.4778655509265256, 0.8274354808316022, 0.4366966327726628, 0.2557036234440659, 0.3415892140065495, 0.6696946651407389, 0.8093002928126569, 0.8942935612209656, 0.3048731574381913, 0.2613297998924534, 0.21694043647620656, 0.10515400266198421, 0.6727452087767205, 1.1604972749757425, 0.7923184637370569, 0.8435929855642208, 0.8993270571478416, 0.7001653257250515, 0.8815663769752575, 0.4288055645120944, 1.056596862643716, 0.8443299943342071, 0.7993405614085793, 0.5889038393393164]], [[0.590438296216311, 0.8291966741736996, 0.3444126730242683, 1.2954764016055758, 1.226161667168657, 0.9289545676589436, 0.548941084741858, 1.009399615148422, 1.6951347582591247, 0.35073118184244395, 0.017724816128645408, 0.43374530097281333, 0.8189216493145832, 0.4390411266878834, 0.2522467857899455, 0.32455854123943984, 0.6598167058573655, 0.8324947019071192, 0.9047639646087635, 0.32978875403573205, 0.25724593446426247, 0.19665285500684104, 0.10686663205567999, 0.8176929788137697, 1.2568709672056673, 0.7173952602678713, 1.2195580871025329, 0.8831792016210014, 0.89102735808351, 0.9286260533843826, 0.44380598728995524, 1.0488769451425868, 0.9299881429430388, 0.7363258889602919, 0.56751175105804]], [[0.40046987106062903, 0.845376813382172, 0.36688311542342955, 1.2987140747619188, 1.191713582446868, 0.9372598970874619, 0.5620914318821832, 0.9632776455317327, 1.6393400336022825, 0.365961858370023, 0.021259893949164985, 0.5502993778262737, 0.8252022584618841, 0.5530749829585156, 0.23822383601593966, 0.3829454302353782, 0.6188644956181244, 0.7688617969840392, 0.9535814504578446, 0.34024722016320186, 0.24783330371273327, 0.1989759808875795, 0.10147297160815284, 0.9591504457923754, 1.2331366457762922, 0.3730473235150098, 1.1387996663159963, 0.8431386061111075, 0.8141842364815766, 0.9551055408098348, 0.4311115129929096, 0.7785154628013419, 1.0096039880156655, 0.6667474470881898, 0.4199999917725048]], [[0.5316147703380945, 0.855525152357705, 0.32110137775408926, 1.417474931750573, 1.2752569883768459, 0.8634477366145459, 0.47423027794604705, 0.9023554924644481, 1.5785608439106897, 0.3640285513474811, 0.021039901089625238, 0.49759617318387594, 0.6935370960592069, 0.535629483978758, 0.23783974944854563, 0.40512330523337825, 0.6099256272761313, 0.7356265378817854, 0.9389015635107034, 0.33895257263910483, 0.25210506792821996, 0.20723931206485502, 0.11120198742804331, 1.044098060195686, 1.083025643901633, 0.23259917331472396, 1.0143164453480902, 0.7793162355843786, 0.5950805322641219, 1.0548611527799563, 0.448790263535354, 0.6811501616360021, 1.0580488823101533, 0.6531285497292327, 0.5746510239443523]], [[0.4086620580458176, 0.8774623917174286, 0.3272930674848787, 1.3975283991045286, 1.2511729515241785, 0.7460611289291545, 0.3966843103854206, 0.908352257417615, 1.5269921091931327, 0.35907533134966274, 0.025399450605980034, 0.490220738328337, 0.6183315639175844, 0.5378647586959668, 0.23784332361721738, 0.419540671297135, 0.59661856414804, 0.7734186086159962, 0.9252887966699987, 0.3652416436523057, 0.234588921756813, 0.2236212711885866, 0.11840053440645873, 1.1684466638569961, 0.9883967527690637, 0.23843098726784745, 0.9176325455321837, 0.6961090342960765, 0.45237369747101935, 1.076579140402952, 0.4179616196156668, 0.6892940576611156, 1.1872391588140596, 0.7905345202947601, 0.7119124482720438]], [[0.4295060470828238, 0.8285430650206614, 0.33631836320103525, 1.394878239148093, 1.2336072500138355, 0.7550576313294495, 0.3955930141180878, 0.8690419077074771, 1.4204431619890414, 0.35293163983263304, 0.041243312255846765, 0.46669185014075754, 0.6424428884778743, 0.5506258899193999, 0.24711406375812656, 0.405355782101124, 0.5392217119887315, 0.8064846055090511, 0.8910073734411352, 0.3596033059596677, 0.23342723511688862, 0.20878400340823405, 0.1628350520419573, 1.170526548748631, 0.9531920887876394, 0.2681056079078505, 0.9242704656284041, 0.5633441021753767, 0.4336308375686866, 1.0944801077917177, 0.3717596204348965, 0.655395640200158, 1.208059549592479, 0.8466677256720265, 0.8214952701856333]], [[0.39862529106903555, 0.7937107956456472, 0.3444317385987352, 1.295828526845213, 1.1172158092768614, 0.6981430274182127, 0.3876602874505878, 0.8812289012453386, 1.1574779116311884, 0.3307030873291701, 0.045790564249482704, 0.5290669053375197, 0.7299483730743843, 0.5794519070528101, 0.25510865531046745, 0.42161663822619444, 0.5245506980007484, 0.8115254376701835, 0.9015428230813555, 0.36878278546787185, 0.24514694200063158, 0.19106102478718745, 0.1318174760296262, 1.092600488418732, 0.9078049510083057, 0.29743787980668734, 0.9384967557548352, 0.41996393230247264, 0.4204430736011352, 1.1407575703392614, 0.4324300859221173, 0.6259211448602054, 1.223806751181693, 0.8430025036699372, 0.8671721594107425]], [[0.37920235126234536, 0.7349554163078684, 0.35657268558836575, 1.260374293391123, 1.0670329563574947, 0.6965691523958022, 0.4002050409125623, 0.8372518423692696, 1.0085127025643064, 0.3381019592640758, 0.0636444078186369, 0.5481716660662425, 0.8355421293439727, 0.5928954676148506, 0.22935600888470253, 0.3921338403178175, 0.4954547304624545, 0.8011412238359075, 0.9041557528378688, 0.333807728748197, 0.2580016419371136, 0.21234755196274338, 0.1650066713730205, 1.0872961410522621, 0.9203042147875757, 0.324829461412732, 0.9805592179300344, 0.4703550253049988, 0.3965150134814711, 1.1609694444273264, 0.6023611453312082, 0.6439946856215673, 1.2302141461535034, 0.8020996879694191, 0.904304544825298]], [[0.3583108054083163, 0.7346517169721518, 0.3204196361132662, 1.0879890773392096, 0.9788330931710364, 0.7074494141079908, 0.4140478425199574, 0.7304199750620646, 0.9488889317277464, 0.2798794780277517, 0.06958691262006596, 0.48132578993291397, 0.9106589106460516, 0.5502022380087805, 0.23576098578998075, 0.38629397158213685, 0.5111061331349717, 0.8257379535512153, 0.9224674842138669, 0.2640660778564655, 0.2657483671506368, 0.2191590383610038, 0.13536137181719265, 1.1493390788626339, 0.9707329316381411, 0.32711465110458987, 1.0351975032583725, 0.5601950306356976, 0.3663745525701732, 1.1319608429528714, 0.7371800598623423, 0.5793038938613081, 1.0535637701315228, 0.7486842264025356, 0.9620884182745739]], [[0.35077599623958067, 0.6740388865464226, 0.33271797235109113, 0.9474192949286306, 0.9805001021862221, 0.6917732900372919, 0.4361360855422116, 0.6305407694366554, 0.9570401686690602, 0.352363427067662, 0.07475712355577653, 0.5213988463841418, 1.0153480635453744, 0.5262315314217768, 0.25324265067967966, 0.3966502822997535, 0.5176141407204543, 0.7817593784261314, 0.9601695884536399, 0.23789911008687556, 0.30791281119484115, 0.20595134520428998, 0.1362612418597755, 1.1050861186238425, 0.9738135116681984, 0.32532186385818995, 0.9287547284934324, 0.5275308885156742, 0.3696686363538164, 1.0117092584411669, 0.7851653828699616, 0.5419756815322194, 0.9553395998311462, 0.7771102010640082, 1.0062827748037049]], [[0.34151209211792977, 0.5727557521372432, 0.31106373255086844, 0.9405666023105441, 0.9692842049929381, 0.6820330955976411, 0.39676588250331096, 0.7170933475449919, 0.9941112846038711, 0.34744876377450157, 0.0853520622402106, 0.4946085502700156, 1.0299445648262315, 0.5251475179176861, 0.2572829285790604, 0.3720706544283543, 0.5135652618731716, 0.7804480060538088, 0.9008995909973667, 0.23498327080890963, 0.29313870403897524, 0.21195511501004421, 0.12444128350532394, 0.9746295476662143, 0.9746020175904208, 0.33357400331274145, 0.6292479377950491, 0.4968810096372088, 0.3332292606541566, 1.0082515267795025, 0.7795970559551004, 0.4803750832534982, 0.9086984889442264, 0.8054622254108141, 1.1008880065236402]], [[0.32885207991212584, 0.5391644575543483, 0.31237940492294025, 0.973290921503718, 0.8946755319923705, 0.672838239264319, 0.37270582803639496, 0.8121677851547623, 1.003629392307425, 0.41896054620390494, 0.1034526116160922, 0.5320903605119371, 1.040080665397353, 0.48252131835619416, 0.24687500501967696, 0.3833819147515387, 0.5243957440172293, 0.7575683392724476, 0.9198162671240425, 0.22737955754107952, 0.27189585843002817, 0.21099336081977632, 0.134158271890426, 0.9409403695184688, 0.9748942626105906, 0.34401973542484, 0.4750601907928209, 0.47213523799379675, 0.31354573118707535, 1.0714079391441957, 0.8071740461262371, 0.43191469218173983, 0.8855271939516283, 0.8370233797181823, 1.1575743854390117]], [[0.31000792046644954, 0.5191294275219915, 0.3000305258707818, 1.044820333361237, 0.7934203542676488, 0.6661966971551094, 0.3845542320156855, 0.8255134634934488, 0.9928334783788778, 0.5014032703360842, 0.10680402267629568, 0.549618687054289, 1.1373744577211369, 0.5029330800985988, 0.27137764157071886, 0.3520891421431137, 0.5410863691541566, 0.7524458142362784, 0.9391358768216183, 0.208834506058997, 0.2601571097499527, 0.1956787236851683, 0.11629820629943742, 0.896661657663879, 1.0342135983648646, 0.29795117551521705, 0.37937615957852977, 0.480617111495391, 0.29121132234722263, 1.0536703346559086, 0.8854476302582271, 0.3729825211873755, 0.7392382408531395, 0.831956231969015, 1.1801138408300307]], [[0.3529168246934271, 0.46503451294210796, 0.2644195686617547, 1.0618209775239993, 0.7146209809278108, 0.6750516698524678, 0.38225381978746253, 0.8378813675718024, 0.9683114605998995, 0.5844417098425541, 0.12866254190790005, 0.6273697149029127, 1.0988918406995858, 0.5118451657938683, 0.291074029558148, 0.3624057282472575, 0.5585916229993233, 0.7580306356360929, 0.9282263870258427, 0.2110892966625677, 0.238057620428895, 0.2056529662354994, 0.1073117470282909, 0.8370890882763415, 1.1045135346123263, 0.23908978890769145, 0.357659293421908, 0.5054446247396015, 0.3197870398820275, 0.9947087673992183, 0.9212514977700501, 0.3416613187798673, 0.6734349967720276, 0.7350547727765798, 1.1372974419112691]], [[0.36413922189643155, 0.4793188191536182, 0.2749102726404468, 1.0744765375189393, 0.6255143781041077, 0.6687634549453448, 0.37698543375005034, 0.8795362096511482, 0.8996360750908855, 0.6306408635275048, 0.15921678747928708, 0.5629702592251449, 1.1076054614069843, 0.589637643866748, 0.2875782098337668, 0.38755611308828075, 0.5312001124572743, 0.7678407683806969, 0.9126568059690462, 0.22477028414032493, 0.21982668357626586, 0.2043176244779814, 0.09946578970641243, 0.8154622074598887, 1.1796806668981084, 0.2106011401829302, 0.29514226932147325, 0.5408259430233978, 0.33626825407592786, 1.044141859908398, 0.9310713226724242, 0.3337866865490363, 0.623364491347276, 0.6804413588862717, 1.0899290963824628]], [[0.3719434737308497, 0.4945241883454313, 0.31407260285511895, 1.1795925879199787, 0.5983526911531142, 0.6617902836743533, 0.3995138527489748, 0.9925991795035419, 0.8050425532618416, 0.6119258618085507, 0.20470637690657228, 0.6472904840378653, 1.081214221383346, 0.6016899780315135, 0.26672861408353166, 0.35572496567695977, 0.5233556550268759, 0.7233377745773095, 0.877308606981194, 0.24950712417870682, 0.21230534530775536, 0.2490485610121165, 0.09909563477617629, 0.8139434732344345, 1.205830717948774, 0.17141803994438448, 0.2956427083891392, 0.5903972641794498, 0.3317122193304636, 1.0623789805937085, 0.9547260412615182, 0.3835129200951379, 0.6172597140694751, 0.6204211128294985, 1.0368136765300418]], [[0.3267185895806652, 0.49434896121914046, 0.2919325889634984, 1.1875322586426662, 0.6297763577891033, 0.6432276075950898, 0.39549621865168344, 1.084081031966785, 0.7328430689426002, 0.562763528044336, 0.2270763253903343, 0.679363189899973, 1.0054041859271239, 0.6499420284213011, 0.2379027283181627, 0.3297730123410195, 0.5587393037980638, 0.7900085477335426, 0.8157175087016681, 0.3787693035291107, 0.2576369846523621, 0.32462767130020354, 0.19484888606833423, 0.8214081512341216, 1.1712749293436227, 0.16349320198940254, 0.35474005445588697, 0.5556844625088464, 0.25763114328708053, 1.1974520102284876, 0.953760538205105, 0.45985140249444145, 0.657166141261468, 0.5503252043408632, 0.9296350510699274]], [[0.3377822264180188, 0.5139281352820837, 0.3181601442381834, 1.282368767815517, 0.534710638466368, 0.630316010376353, 0.3857708365315231, 1.1115029375797847, 0.5984686154663496, 0.4798051827247957, 0.21020205760170552, 0.6538111537135463, 0.899592766106296, 0.662921726694143, 0.23531687724737924, 0.3294473862150836, 0.5928821363388166, 0.7218387042714491, 1.0048288053703893, 0.3250180158810049, 0.2568332331615363, 0.3303310373473001, 0.16238837288723967, 0.8020386546357143, 1.1173185982709952, 0.16508980567920378, 0.38125724949809103, 0.5735504071659459, 0.19715155132710793, 1.1366999200960275, 0.9595098544655123, 0.5411705023611119, 0.6386766629712537, 0.5197733282575274, 0.8165326206431319]], [[0.3525781183365858, 0.4224161506281015, 0.2714398208053075, 1.464094786204731, 0.4685518128801003, 0.6142364593195317, 0.393114957399178, 1.0381050082554202, 0.7344755629303212, 0.487339181937719, 0.1902342605249943, 0.6208113528874927, 0.9025107429426423, 0.6956836552808813, 0.25222596140955145, 0.43830360487765807, 0.5892948016125239, 0.7476360397097801, 1.0811635030735636, 0.17781547505798628, 0.22600510555768127, 0.3434293391235177, 0.15610150248996357, 0.7949361996598024, 0.8934544110630804, 0.15756105249633218, 0.4043290110991174, 0.5260378846530791, 0.18562846588409537, 0.9323051766902587, 0.9772194687044446, 0.7052273390494799, 0.6054887843248067, 0.5622593541160884, 0.7941102931963988]], [[0.43185385096850126, 0.3604886190823363, 0.1801970959235463, 1.3897279824699682, 0.5398052399526774, 0.6156615943646804, 0.39144480643450724, 0.9726148301115632, 0.6198786846523148, 0.4259678805204942, 0.1835983458178001, 0.6335424127434506, 0.8530371809625517, 0.6793591216144688, 0.33379778399760596, 0.4253291677774633, 0.5595092554842229, 0.7115953736299269, 1.1314490138174356, 0.13039223685191048, 0.22941585185471225, 0.274225796005075, 0.13111916649120461, 0.7864755788478751, 0.7077681186042177, 0.14299876448435667, 0.4530233907097687, 0.5583859259246999, 0.22716749896558408, 0.9078951243821292, 0.97249215594606, 0.779671226745362, 0.4999209842322234, 0.6121088122889059, 0.9040300898410155]], [[0.4640624856668598, 0.33182808161553057, 0.22846694001047482, 1.2392903194613039, 0.6359960710746063, 0.6264467414394799, 0.421344888609183, 0.9371850004015101, 0.6353687515419727, 0.4003032403138915, 0.17719266156171443, 0.700911083088398, 0.825660588535407, 0.6646439163418455, 0.38604253025622115, 0.36198836745646823, 0.5601657858281021, 0.6874492634051351, 1.0157071041961183, 0.1441852734731582, 0.23840270204142472, 0.2727939775873236, 0.12983294597531161, 0.777064584894547, 0.569147880290706, 0.13195615810435365, 0.45393710329336134, 0.5980383466274168, 0.2597806964391042, 0.9788027430422821, 0.941359171939893, 0.7351107815443515, 0.41076672073157505, 0.6268451593375088, 0.8918737827492846]], [[0.5043153625155175, 0.3504512359327331, 0.2302406036294627, 1.0537893784851198, 0.7009411620223016, 0.6305389015217758, 0.42601565039172373, 0.8742921912300494, 0.6680503636028805, 0.3852737145401578, 0.16590701742224434, 0.6499505966923294, 0.8073061099426021, 0.6991901152298642, 0.4342100998199101, 0.41478883255993926, 0.6169186357982307, 0.6893710799801537, 1.0016891593291752, 0.15961668339051294, 0.2656785269263628, 0.27527292379293306, 0.11704228348531719, 0.7357176549378603, 0.4886532514064944, 0.13603332115195904, 0.43020819421983514, 0.6562674743373685, 0.29428511414439984, 1.0380661590402918, 0.9371221453414906, 0.6956194694572904, 0.3138233288181166, 0.7631459612829214, 0.8524816857034607]], [[0.5488108363182406, 0.3559915483171401, 0.262220681166604, 0.8414024806001239, 0.751439518654361, 0.6445006569105877, 0.43640771552708524, 0.9115241770051636, 0.6679977283335001, 0.3014004967698939, 0.15322676678494018, 0.6425289033525018, 0.8047892953871872, 0.6443168510863333, 0.42724761565649394, 0.42607416437273693, 0.5354647330989516, 0.6707426406217041, 0.9085091813450311, 0.17363965726832875, 0.27427436597633625, 0.2838274398846437, 0.09835893339114579, 0.7229002832870335, 0.46706063915659074, 0.13385422642201528, 0.4643886743770643, 0.6268232345789991, 0.3115695920993654, 1.0922023220658135, 0.9292905877711217, 0.6799254753002042, 0.26076986921170187, 0.9040897124604003, 0.7903056794688432]], [[0.584239190516733, 0.3823377111184687, 0.2640956515809567, 0.7364699255256912, 0.7229150920202264, 0.6864821026861273, 0.43274871727803754, 0.9137512043520382, 0.6928188598316751, 0.27017231908065736, 0.15784615543377642, 0.6274510680213581, 0.7524641825466754, 0.6613534089349418, 0.43374379089898546, 0.3913934812933467, 0.5868557411806916, 0.66747736334742, 0.9355395121371479, 0.23971228599561611, 0.2802203249714338, 0.2715657866808501, 0.09872100747597178, 0.7074962337436435, 0.5534318589422245, 0.12469679182225332, 0.5125272768444121, 0.5806533337327996, 0.32345888571276193, 1.1688614126624337, 0.8656060794680358, 0.6846919377240877, 0.2462430089987178, 0.9647885157580172, 0.7425573715567972]], [[0.6246457654675999, 0.379318175638309, 0.27411598615272087, 0.7135196871238703, 0.6475383507500673, 0.6772149142230999, 0.4301540494510111, 0.960417791290427, 0.6646269312207782, 0.27312481763542457, 0.18295930350712752, 0.5697802122534568, 0.7041544666716002, 0.6162417351418781, 0.4081549245483175, 0.38843879646346613, 0.5695642792161667, 0.6702746177730383, 0.9965053530489646, 0.2668737636237656, 0.2903596677889182, 0.2955631732581676, 0.08969130674434594, 0.6608459492471201, 0.5998354558151004, 0.13378130774624208, 0.48312120804313263, 0.500258736945127, 0.3040769354793085, 1.1929079481580398, 0.7999841406185588, 0.7002931850906782, 0.22474950792746762, 1.0034114958841864, 0.7198504625647488]], [[0.6570070550958432, 0.39472689365935676, 0.273817263189072, 0.7091884124038474, 0.5979112622330542, 0.6796750939086308, 0.41502704461083967, 0.934934482065416, 0.6434210958061946, 0.31990219009492404, 0.19311569943282825, 0.5520858079338755, 0.6070271631633086, 0.6052962880918891, 0.4336351806193348, 0.3244778008687668, 0.5935639413724172, 0.6500742270891731, 1.0275197189637841, 0.29970564521479304, 0.31122541250731034, 0.3188181756407694, 0.09104652387172077, 0.6332076818596826, 0.5544679186834712, 0.13245223309131682, 0.4862302279353856, 0.4766750218426726, 0.29159219689725485, 1.2267849978132102, 0.6313043636737588, 0.7450832022537432, 0.22258717659110733, 1.0458792960255046, 0.7063793892855533]], [[0.7316310941425719, 0.5423808030295959, 0.2851322358120918, 0.7090650534835172, 0.5422246161105497, 0.7010289345866997, 0.3754396914269106, 0.8990134161866337, 0.5977510657214355, 0.41957244339882027, 0.12712018582253032, 0.5460341680649274, 0.5732461858781637, 0.48517587087528213, 0.43628180707649344, 0.3088033776505186, 0.6609497241618462, 0.6987150797057362, 1.1210777127015068, 0.3341852034453102, 0.28682040525221436, 0.3540857513459531, 0.08554840948058544, 0.5919196517191618, 0.5513272165457468, 0.12839326104770699, 0.4420574041349231, 0.43969763783732546, 0.27411560547109237, 1.1911207091558245, 0.4831694256800785, 0.7150642871955216, 0.27765242733826523, 1.065518225549971, 0.6957436106886029]], [[0.7837731729121826, 0.6777748525529361, 0.2877806510992062, 0.7517797670433161, 0.5604856969912858, 0.6933517672195162, 0.39172304804670405, 0.9282555794150159, 0.5953892106534544, 0.5268618154934817, 0.10530003336861496, 0.5279789813379518, 0.5829993714526239, 0.49317945774399047, 0.45374370183433155, 0.3141032004659915, 0.7858636437036863, 0.6947966996041881, 1.1500510400203061, 0.3846250999440112, 0.26079994302926013, 0.38045833612578606, 0.07898754550076091, 0.5436917300260206, 0.5646448204738669, 0.1274483634506563, 0.43037304837457185, 0.3626433094281881, 0.2655673928314467, 1.226181484049373, 0.4184750277101435, 0.7181348742421476, 0.31785293561658784, 1.076336239428054, 0.696668813173154]], [[0.8391048220447738, 0.7068208831654643, 0.2971355570541202, 0.758528813381512, 0.5768571070318758, 0.6707200286529302, 0.3762985206866568, 0.9689812645228568, 0.6512640230450175, 0.6097943699081523, 0.09562928608972592, 0.5135615286021107, 0.5879415664003366, 0.45346066475042984, 0.4588547168087717, 0.30201667431244805, 0.7347768962641867, 0.6357991586563312, 1.1497254604625282, 0.39683418092238276, 0.24008344213303728, 0.4238599470614657, 0.08220958781906948, 0.4719151559957308, 0.6007963957966133, 0.10125535743045083, 0.37130683227962075, 0.30869251013682575, 0.25364211918470986, 1.1890951481491767, 0.37641366243003704, 0.7373931013124179, 0.3969459943148713, 1.0759882369548692, 0.7094975161803405]], [[0.8685188449395016, 0.7684901790938974, 0.3005831763232202, 0.7411724255714861, 0.6000034371429874, 0.6736010582480381, 0.37595834858027577, 0.98560579063405, 0.7697307932108268, 0.7165477641394584, 0.17331577188305608, 0.5037554848521233, 0.5383113880513128, 0.4870841114313238, 0.4579413515456025, 0.3953649562580347, 0.9196975383577666, 0.6327802158584981, 0.8880379814402298, 0.4720828540012676, 0.32967488053888644, 0.45741155932079813, 0.10405545265277133, 0.38283999830589993, 0.6420846020000912, 0.10292949605068868, 0.35056668427101534, 0.3156532822899907, 0.25898686262975346, 1.2104440293090628, 0.34849766095946666, 0.7630452962539082, 0.4392602706741249, 1.1086985950472603, 0.8247164543865264]], [[0.8461528413520499, 0.8642867551096318, 0.29260417591089516, 0.7359519832023127, 0.7031997070348144, 0.6749635476258433, 0.39709763285674576, 0.9895991073930888, 1.077659129070498, 0.7129470219249088, 0.2149002573311798, 0.537636739690452, 0.5568611870309625, 0.504047900741285, 0.45186358018559203, 0.39924134638261316, 0.9758294980678952, 0.6416982093223872, 0.8883212267742611, 0.4913655111862705, 0.3531010413912652, 0.43763292358671196, 0.11186266254118331, 0.3978957769182966, 0.7162538867869086, 0.12393689324706109, 0.3316963282198676, 0.38600535294625304, 0.2808623947134672, 1.2550039465215568, 0.3607971660694246, 0.8490041643254249, 0.45010978243540967, 1.1707430905917802, 0.9661895975543617]], [[0.877441970113, 0.9096261286279016, 0.33305640171631123, 0.7851950273258714, 0.6883718873247506, 0.6744009432206826, 0.38289628629776346, 0.9217758292841973, 1.1300413804594829, 0.7106555245520885, 0.1085465266632085, 0.5282298303795736, 0.5810539714246039, 0.4855604211988507, 0.44014160306025324, 0.4671184346247774, 0.8220509783766425, 0.6877529169643897, 0.8726530787221217, 0.4491359099696464, 0.33662592781733014, 0.42130536787560846, 0.1167409407830085, 0.41929367831090275, 0.7711463841029341, 0.14596038841061, 0.36099550056570456, 0.37390597924378116, 0.2885769364773169, 1.2001133300271096, 0.43015152488602026, 0.9080572950043477, 0.44377147016392093, 1.2089874944324595, 0.999801193969662]], [[0.9179086509816874, 0.9691273605372398, 0.3512255080743554, 0.7337822021844231, 0.7849164476413859, 0.6261741660768367, 0.3890006818916972, 0.8381894415806757, 1.0259331563259562, 0.6607556583766232, 0.0386410915657683, 0.523066171049221, 0.6683572420903231, 0.5074645976179473, 0.42112666114813746, 0.44895275479474694, 0.863763357262337, 0.6265524342675222, 0.8161517236933724, 0.511401250919423, 0.31081788208787187, 0.4241854917211072, 0.06799569232704719, 0.4356815967927167, 0.7891397496956518, 0.15372198199321607, 0.3734852508919977, 0.36043319210020175, 0.2874902983103808, 1.092852008975082, 0.5690043491010874, 0.8741290848184238, 0.45093382028330986, 1.2002264373964548, 1.0216902110180224]], [[1.0628224625319909, 1.0119628867992674, 0.34997014704169993, 0.7502922888314661, 0.8210493621788347, 0.7876654895005204, 0.3937680161533676, 0.8105641501754153, 0.9214553744154381, 0.6687661417417521, 0.027659787934269632, 0.6009636170098445, 0.6873853597429909, 0.5521227840801066, 0.4174631935922561, 0.41896909603869625, 0.8741804512947152, 0.589468681276798, 0.8267159879640078, 0.5160460920543893, 0.2638377487182164, 0.3882580804090239, 0.0668038498048347, 0.4954336312485313, 0.7914939137269676, 0.1700230249932822, 0.37841065710848304, 0.33016819264174946, 0.27391182741629255, 1.0260283758009687, 0.5945760148412004, 0.8897800469741517, 0.5418028277861644, 1.1682886932989773, 1.131646944347049]], [[1.1576900906800198, 1.053146638866158, 0.364180406625032, 0.9089346346332048, 0.8500323739952511, 0.8112252344827806, 0.392424850080677, 0.7919947219570371, 0.9394920324661126, 0.7144332007098992, 0.05695092491659737, 0.6405410507463581, 0.7577242870464884, 0.5507755205083176, 0.39642784003747666, 0.3875233624699104, 0.907737500600874, 0.5198211250475201, 0.7558275164594154, 0.4889121835942677, 0.20459951733289541, 0.3455749572806458, 0.08165524397396175, 0.5222266632688779, 0.7861269784587858, 0.15460831943110281, 0.3445908092109373, 0.35491056897812834, 0.3252082085650111, 0.8595003968384495, 0.5624998681442182, 0.8598171367332064, 0.5296712439730069, 1.1956776425387459, 1.1750053249225323]], [[1.1864525959896701, 1.1555437136897966, 0.38301690108609227, 1.0286182771084675, 0.8183203954006733, 0.8448742543953828, 0.39864252531211664, 0.8080843404147693, 0.9528690242826758, 0.6993820918208348, 0.07405466713610087, 0.6835156154363877, 0.7553931246281655, 0.5614906498675544, 0.3955244065160668, 0.3719254794506708, 0.7776769628564214, 0.49101129417666567, 0.8736181060152888, 0.2929930822770117, 0.214287554032918, 0.3302734108764693, 0.07099652808720625, 0.5225798972833939, 0.7952694328500404, 0.1639297237200587, 0.3377072763869412, 0.5304432789994783, 0.43477670449817474, 0.716898841889678, 0.6019698885179783, 0.8108864108370907, 0.5828735743084994, 1.2091563049913052, 1.1596370762098123]], [[1.2762202685841504, 1.1994624282440476, 0.38576169433221374, 1.004532035380628, 0.8262684289107443, 0.8472962323853268, 0.40673871465179595, 0.8812559920899738, 0.7509264461958627, 0.6567828622756356, 0.08484699030073811, 0.7195751414630169, 0.764856171272875, 0.5428331384961071, 0.4452947002147402, 0.3631893501303576, 0.7471022569158918, 0.4698476308411357, 0.91874420031291, 0.2106553058788564, 0.20055494254820994, 0.41391788548893327, 0.10220279605844339, 0.6850294181608523, 0.8276724586784272, 0.16276286413745772, 0.3871882616418619, 0.633660909643726, 0.47229290949547975, 0.7166182057082765, 0.666523794950246, 0.626707274247043, 0.637287340268837, 1.1728673803325427, 1.1093097267024035]], [[1.045450975474644, 1.2361786991452057, 0.4295524631206231, 1.060338904354861, 0.8673693222927826, 0.8325103996922782, 0.4076973117005257, 1.0220585348077644, 0.8756343381471092, 0.5667745925198858, 0.09741583151720593, 0.7879483575842653, 0.6545001893201394, 0.5663982912502787, 0.4034827726996422, 0.3576480902300294, 0.6968077911050444, 0.4583032556291977, 0.9884242996448287, 0.19277455513005243, 0.1881025320145646, 0.38043082868821604, 0.0269019507556979, 0.804164105313389, 0.8219078584130595, 0.17645487304668583, 0.44999585913795465, 0.6993289160533999, 0.509987428766895, 0.6335281187141855, 0.7026601990795793, 0.5892607800540072, 0.7721774526621936, 1.161545487659298, 0.9591334541690748]], [[1.1379174177191298, 1.25739609342617, 0.4346733271235518, 1.0715143367560183, 0.9221929641884932, 0.8097259302723374, 0.41216245827751724, 1.0112977222692148, 0.8227004606651765, 0.5534467863426603, 0.12213344052455447, 0.770798225133767, 0.6386061647867354, 0.5778894704505246, 0.4884023136701827, 0.3326933278593513, 0.7263905029924411, 0.44941046678918883, 0.9439901728115516, 0.2432099838836212, 0.18463717009063296, 0.3998341546806849, 0.025292166784517057, 0.8939496032601213, 0.7915185589696496, 0.19551924861843428, 0.6463130931588943, 0.7470858294711282, 0.4993804293707051, 0.6172207645880199, 0.7423028829972595, 0.6216827114862291, 0.7895821945841164, 1.1419379814462736, 0.9705156706325817]], [[1.2024170302805652, 1.2566536868081486, 0.41250195435784953, 1.1271752615688244, 0.9814837973666102, 0.8097016058072342, 0.41587648042140146, 0.9991282268369063, 0.8159469557088052, 0.5687260798457088, 0.11683281326080139, 0.8066875741981637, 0.6490768576762166, 0.5863973211728544, 0.4543171789051585, 0.3331752033854619, 0.6941174212130977, 0.44974048124966437, 0.8756221482936641, 0.2290137163167324, 0.19327691783638312, 0.40566519065517614, 0.025941772322001747, 0.8073698386046986, 0.8021660590843985, 0.20010633912609405, 0.7134669658691439, 0.6948292375916382, 0.5248371540553782, 0.5740033089174044, 0.6951216206719977, 0.6711218258655133, 0.8256856223051685, 1.0374604753690182, 0.9696843145452297]], [[1.3018983488311155, 1.2107119450517816, 0.42106628677955393, 1.1516585226595195, 1.0241744216381896, 0.8164887301269552, 0.42254270542220107, 0.8246335318201088, 0.8280609718936384, 0.5972722694464643, 0.14406357236209405, 0.8012847621103221, 0.6502613629803028, 0.5890652612214954, 0.47785685874153655, 0.3688854603124117, 0.6883648946705624, 0.46801211356127514, 0.9101438439621252, 0.2795348502101016, 0.19152346184282715, 0.4213440881065419, 0.014165086821353454, 0.7070682607766335, 0.849035204602901, 0.20965894945966618, 0.679332148343949, 0.6649484788324889, 0.5415346531897484, 0.5176468011081112, 0.63260620069075, 0.6940130796892751, 0.8637282443283756, 0.974230547694291, 0.9238759972806593]], [[1.447557517871715, 1.1557382899132447, 0.41551950355703543, 1.1588558589358842, 1.1636122062208312, 0.837272817304733, 0.40789182795634027, 0.6293419992995627, 0.9032775508977013, 0.6348330457100322, 0.16264555724408403, 0.7760041914166635, 0.6235578381729188, 0.6116817299127347, 0.45785126900945494, 0.36432147503316425, 0.7101811819442037, 0.483551715947039, 0.9232174837040841, 0.29499330181058586, 0.19544576148280743, 0.42641372587994897, 0.012109684681103077, 0.5941495472995086, 0.9177822694645436, 0.2173276683993165, 0.670845927741239, 0.6951622204517895, 0.5588780139835614, 0.45548964196829295, 0.6666705691377557, 0.7704615864446802, 0.8932986305105541, 0.8877828022622548, 0.9101176932888273]], [[1.5012468775854435, 1.0828927720774306, 0.4216895889427207, 1.2003399019174643, 1.1408096732653912, 0.8274663137237246, 0.35920660177118585, 0.7038925097205662, 0.8888072739828967, 0.6977424874935624, 0.16536690630973028, 0.7589017326284146, 0.4605878503774078, 0.6168784193529893, 0.42933475631504225, 0.3619704043323775, 0.6921659188768315, 0.4800682577074553, 0.8300973551459979, 0.2799342875914714, 0.21492452677047602, 0.400003866883354, 0.022612611546214377, 0.5426848297048661, 0.8393350404173314, 0.22073594143025888, 0.6593537885212124, 0.6401913670098849, 0.548215221899504, 0.4137883045804627, 0.6745134830167212, 0.7523906407363503, 0.8383044899148993, 0.7287839641728482, 1.0038012768886193]], [[1.4497834148006663, 1.0522377719464329, 0.4446075196584468, 1.2253949329539293, 1.0652555677472089, 0.8894747947552686, 0.4086855233494161, 0.7156388966440893, 1.0769178927631216, 0.7233168258959376, 0.15580036299433198, 0.7247351555786019, 0.3223646773517922, 0.645863076922863, 0.5353374050052835, 0.35379777832779924, 0.6737546176215231, 0.45425364165484805, 0.8257052775502614, 0.3141979009162257, 0.20882228801664268, 0.4880836029499669, 0.025934821082504445, 0.5063411327596328, 0.7488188535449178, 0.20337669108337764, 0.6182178744532851, 0.647845574645268, 0.4733710920308841, 0.42791669887095357, 0.7202643612296746, 0.8216223437700889, 0.7646333346237704, 0.6330508738110452, 1.121837465729186]], [[1.4184570880249563, 1.0418398501963162, 0.4744367544891897, 1.2611604888688035, 0.981810522501831, 0.8998929882998552, 0.464709792925389, 0.7697081181758765, 1.159058180426054, 0.6202181279674107, 0.14050264341579155, 0.7578100538809431, 0.3047309367579337, 0.5960612380480386, 0.604595487750567, 0.35727404449345096, 0.654486829973838, 0.4910651065427257, 0.7054048435627357, 0.33706499490033914, 0.19845029965671876, 0.605697044755618, 0.035730672978050194, 0.4179703120923755, 0.7031862768923905, 0.20222268153364648, 0.5699417437528709, 0.6473624951676913, 0.4191604271514766, 0.46257243341422133, 0.7789810287279907, 0.8531117615343272, 0.7110916609605128, 0.6085108344180523, 1.103235763837794]], [[1.3138808446712893, 1.0133117013992792, 0.4514236740069167, 1.228448106625498, 0.6775850587015312, 0.8098784560848592, 0.4244068314828598, 0.7960174491898772, 0.9972954786290384, 0.44839855207117124, 0.16187627429649967, 0.687834020704243, 0.2282274513090872, 0.6475681091057957, 0.6032059966543393, 0.33685751145876075, 0.663723179878591, 0.48520742643120074, 0.6549158338929335, 0.34035474494403806, 0.2121675343469027, 0.6069855614299505, 0.054812109873370214, 0.3148787591043428, 0.7225429719531329, 0.19072538490713714, 0.5230664305143435, 0.5706593418862729, 0.32256103215902465, 0.4583538682736752, 0.8536887077404134, 0.8632486909395208, 0.6113145126090038, 0.5296163404251416, 0.9342818957559313]], [[1.2790081240161564, 1.169019047914897, 0.600166151215825, 1.1833329621403967, 0.38902839306235054, 0.7724016561077129, 0.42190813400098015, 0.8157958327717445, 0.9261743353632101, 0.3907259753385247, 0.14912781209552403, 0.676121938691622, 0.19626840142382235, 0.592426110269102, 0.6679571578372626, 0.3543966363737908, 0.6364883681175556, 0.4990804088735128, 0.6095213920207406, 0.33501539729614155, 0.23587278528997463, 0.6386843135748314, 0.04670516839643893, 0.21129140771285948, 0.7242210767758295, 0.18091305927589846, 0.427081611849538, 0.5405859181131011, 0.2889857359053202, 0.5754517960542628, 0.9224718892585098, 0.9459332506140341, 0.5761896590172468, 0.5078066763684604, 0.884803091069214]], [[1.2130843840153744, 1.1836753428884534, 0.6101283354303196, 1.21303400487521, 0.3023394966395906, 0.7439963520285835, 0.4158025782228014, 0.8075273479977878, 0.8828942945773229, 0.3003029932178015, 0.15561177332800266, 0.6961898636675208, 0.1956380266749813, 0.5831940842418997, 0.6203658467840814, 0.3796336493175916, 0.6102880721777315, 0.47916361432490345, 0.6090360417334897, 0.30847499346047, 0.2776045102591478, 0.597092002831756, 0.04190502848568228, 0.1350270909739607, 0.794867298813076, 0.12212258088151773, 0.36177270542120055, 0.5368570916688021, 0.28785759037234904, 0.6483585990603595, 0.7966408585168141, 0.9346948372798505, 0.5169602558339912, 0.5831188610139053, 0.8666783234588773]], [[1.1622511549923724, 0.9989928904145113, 0.5919205028038464, 1.2443399374682929, 0.28630598644866256, 0.7183654931132053, 0.41603548274207774, 0.7986830054954801, 0.8766946177133816, 0.3059082566383539, 0.16729049030581494, 0.6985060623315757, 0.19956150677467366, 0.5807789988252343, 0.6366911687928332, 0.38422065933823857, 0.5828392973546611, 0.47509459159222195, 0.5442858662727708, 0.3119504799299295, 0.32759670294508497, 0.5678961403393357, 0.041058085236643706, 0.11748859571498405, 0.7902217289066508, 0.10953761202926937, 0.3028323009512932, 0.5771415285226007, 0.3273618710803313, 0.6333247234285386, 0.7313442601409363, 0.9144380129611298, 0.3601094809500358, 0.5365725501962972, 0.8593601922184528]], [[0.8345023768248798, 1.0039088180213958, 0.5077299726735824, 1.2408148394094232, 0.28933011745413006, 0.701716563734569, 0.42052683328557894, 0.7829306181022873, 0.7708911635182645, 0.3122534454480077, 0.17836622563371057, 0.7122803345709894, 0.20277859527324782, 0.6035431766097012, 0.5341223959222963, 0.3547305488306383, 0.5573760294579997, 0.48676216304410974, 0.5639194450581789, 0.30293853054793984, 0.2847918242495557, 0.5634912041596714, 0.06019107722359124, 0.09732915933087621, 0.7443252604871853, 0.12760400569110603, 0.2678867421978387, 0.5947610811438768, 0.3164628710128651, 0.6464036794027319, 0.7140227973012002, 0.8463343183632124, 0.3101510127830357, 0.49886933302780245, 0.7436502026342757]], [[0.8160449221599999, 0.8395985753584414, 0.4022451614717114, 1.2598664666469572, 0.2894598836066721, 0.6851628416276088, 0.4107666073981593, 0.754898846792478, 0.846923514947846, 0.41243824108999755, 0.17807336154085232, 0.6956079863919374, 0.1997158785768844, 0.5989755729029416, 0.531127660350303, 0.4064563160031219, 0.5319296177241162, 0.5229650145325337, 0.6472232833893642, 0.28752457095703643, 0.2721837681673603, 0.5575062557642017, 0.05776545657997019, 0.12960250353881475, 0.6830103038873157, 0.15459624828489105, 0.36602054533889483, 0.6741249539798108, 0.32344822124268285, 0.667746676083057, 0.6938112361653392, 0.849808195627577, 0.28604794575922776, 0.4986851445813682, 0.6324794675765975]], [[0.8651414353048862, 0.8331026247990052, 0.3828317500496907, 1.255774772826425, 0.29103701855815967, 0.6526010709157744, 0.4065474212207786, 0.7947168860488555, 0.8794927187368987, 0.4943986366326631, 0.1791447095157269, 0.6635515181276868, 0.19730751976110505, 0.5833488425721239, 0.36601609204178986, 0.4635219805072387, 0.5305837613244185, 0.5376094040092678, 0.6712171239792808, 0.2482223704039286, 0.27364291403556146, 0.5802386511715855, 0.044681790327559784, 0.23635730538378652, 0.6970438839709757, 0.1831067058074614, 0.5215819279734553, 0.7417177167948266, 0.3275701667862564, 0.5932127453390839, 0.6755079835319211, 0.8431887517104341, 0.3598348353934866, 0.5273664064849893, 0.5514295755824821]], [[0.8594214961642455, 0.8316044359867368, 0.3360172237731516, 1.2300296162806905, 0.3720449089707419, 0.6803518016130745, 0.3880329602096319, 0.7829446505417623, 0.8549574212170826, 0.5152208520778151, 0.26676050455205685, 0.6482774742861755, 0.2042682600507451, 0.5805854723570367, 0.34074497052470254, 0.4538877462144078, 0.5927017310868428, 0.5723753620325597, 0.76518958248374, 0.21898897902845976, 0.23457116782436033, 0.5262758758407586, 0.040144778033587, 0.4285673994431423, 0.6473354002451803, 0.19865584652843066, 0.7703706450548999, 0.7555696972766084, 0.3356280646631725, 0.4750039180797395, 0.6384006894556495, 0.8587535072697333, 0.5099989475312418, 0.6332011874092329, 0.5208795590066606]], [[0.8457864766875628, 0.7852069261917515, 0.3448475665066656, 1.2675373401681034, 0.36625196163853624, 0.6375586716631572, 0.3865957786997017, 0.6946164332722333, 0.7342790011146917, 0.49354409763388724, 0.2881597332421115, 0.6200580959675591, 0.20411781210882585, 0.6038338410220114, 0.36692756929863773, 0.4423606335719682, 0.6449585246169864, 0.6091968009942254, 0.7754286782967317, 0.20440749463567626, 0.2532441979424918, 0.5417521801056653, 0.038464447081122094, 0.5691204577805391, 0.6175221311041473, 0.1674606720633443, 0.9119808901453772, 0.9184864539587019, 0.3150001726348747, 0.39573222279917564, 0.6809115602835192, 0.8786094333219903, 0.7204927935860975, 0.6360803704280613, 0.5823900789085132]], [[0.8223399358065542, 0.7137279751702419, 0.32803585769364074, 1.2789874542126782, 0.52561833165485, 0.626252906841934, 0.38455175286791243, 0.6739285361647392, 0.8334976641455463, 0.465625547523786, 0.23884356946062463, 0.8126184563177621, 0.11234487877720577, 0.5673541955737548, 0.2870305615422508, 0.4287514919841128, 0.5523922144554627, 0.5154768414244352, 0.7014145059492278, 0.17137793616097405, 0.22428504713019837, 0.45456933138147676, 0.03887338803411932, 0.7264278348023258, 0.675138961308678, 0.12125492819941486, 1.031597088756687, 0.9356449775942954, 0.2949534820726226, 0.3509813442859231, 0.7380010041967505, 0.9148083837201066, 0.8448158957830785, 0.6363502673881596, 0.7481020030082965]], [[0.7670584938655365, 0.6974000238519746, 0.31746754440673053, 1.217736206145741, 0.7091427648419762, 0.6531491499104541, 0.3797255217212152, 0.6784123549119636, 0.8194913889355505, 0.42774898088662694, 0.1661480792952598, 0.8235480644417501, 0.10227909764217256, 0.5694014699021069, 0.26383799433345945, 0.425249150997247, 0.5499254349206776, 0.4986072922133622, 0.6071994882683849, 0.16171720404870338, 0.23123446961975605, 0.4181480109278279, 0.0403287553709403, 0.8457707561266852, 0.7237481789704743, 0.11876083387445652, 1.0661605560528655, 0.9601080613140434, 0.25897954895041586, 0.33096572544716923, 0.7420448083941295, 0.9360087507590213, 0.9437651696799985, 0.5893600593735737, 0.934154714040959]], [[0.7214286520891693, 0.7465096211543594, 0.30695650860293694, 1.1448862159599984, 0.8511352459254983, 0.676830738612753, 0.37462522008909305, 0.7013029696118862, 0.7840985025914915, 0.3905308428159209, 0.15705655469928018, 0.7665454014460733, 0.09809112126382193, 0.5824331604777827, 0.24970701191842892, 0.42811727568140245, 0.44696720710228854, 0.4921023838956654, 0.6195144721692372, 0.18099303307555034, 0.23218459367216143, 0.43340323322767116, 0.043330962888539666, 0.9544994313843338, 0.6766560894084661, 0.1674843979285675, 0.950804881208385, 0.8794983879545943, 0.242942309556778, 0.34166026130087107, 0.7260896411509881, 0.944371274988607, 1.04426706842684, 0.5408607105677252, 0.9441496575622088]], [[0.7044903117321007, 0.7375957701875957, 0.26957917116547525, 1.1013509657521556, 0.8965079293938143, 0.6979188253369839, 0.3873580160133838, 0.7577481600303346, 0.7009979069356754, 0.3743123553404727, 0.15250327123009155, 0.7350677531364143, 0.0816668788680146, 0.5906013671770847, 0.2165881831215166, 0.42422537205933386, 0.37829433406517154, 0.5026945869729406, 0.6247281979603996, 0.1865629110870234, 0.23592103797965636, 0.420916232747858, 0.0458166665806841, 0.935206960589673, 0.6148054928655573, 0.34302386916524824, 0.8824676750626346, 0.7887770002362192, 0.25544078242873924, 0.42322262628439733, 0.680726760864536, 1.007210945794904, 1.0768836947704106, 0.47066318686207487, 0.9388371651499524]], [[0.6707843590470761, 0.591298221507835, 0.2609889713740313, 1.1081933565704232, 0.9749046630281359, 0.698818223829667, 0.3845042607237523, 0.7493413173264647, 0.6560333635926429, 0.36059140159033265, 0.13862480993977977, 0.7288261394364828, 0.08388366686579182, 0.6478353199859845, 0.21349074897632134, 0.45182863151160546, 0.35611605829116977, 0.43366329239364065, 0.5500402925245061, 0.1871745122591836, 0.23442699052720295, 0.39218599093617773, 0.04745090766430839, 0.9017298594883627, 0.4916819768926217, 0.4549993677397637, 0.8448277685901884, 0.7731015572970585, 0.24198411978263662, 0.49424575634111534, 0.6032188927165941, 0.9708717981940298, 1.0948154061259352, 0.425940887996629, 0.9072454004550199]], [[0.6454829675752682, 0.6066763378886518, 0.3316199548597726, 1.1617788902499842, 1.034133983223499, 0.6987302807236921, 0.38849561357406875, 0.6859940808993269, 0.6295158726834553, 0.41330598813742014, 0.10757622345495119, 0.6995091949436523, 0.07964194344300671, 0.6683039079207849, 0.16634698435360754, 0.3941666335336581, 0.3846751521574743, 0.509850100806985, 0.5395798078133568, 0.18880705058390362, 0.2201239167732328, 0.33135415635880605, 0.04872511378987475, 0.9073569906810919, 0.40549609616894855, 0.43990321789126596, 0.8096342197654776, 0.810816151048743, 0.2557314957812524, 0.55447759153802, 0.5517272858862974, 0.9075927558071448, 1.124467073085663, 0.45377291688455546, 0.8881887514939456]], [[0.6052838377928367, 0.5867612842749428, 0.3318315785675965, 1.2812681720465855, 1.087094746256056, 0.7009734311145299, 0.3946299016075152, 0.697078144796417, 0.5777951471420488, 0.4522717546949494, 0.10022124729658227, 0.7294002421383812, 0.09926591562975984, 0.642609392263866, 0.15459084446909563, 0.3612614147282203, 0.33984018695715834, 0.49546733055746817, 0.5476393787730596, 0.1830308961467565, 0.21221641444927508, 0.2971070113840695, 0.04981396755430698, 0.9213819782046841, 0.4658656835599719, 0.3590591310605694, 0.7721675961605254, 0.7782823251739062, 0.2360197296742771, 0.6387430532738526, 0.5355830591231202, 0.8512835579297631, 1.1007534259584797, 0.3915774743592265, 0.8410884861743341]], [[0.5799422175396778, 0.5459892249155249, 0.3089750357281664, 1.2741042822806263, 0.7188663638879589, 0.706239790328782, 0.39467073060095825, 0.7550725307185648, 0.5519866090182806, 0.3556871507090783, 0.08063645497333628, 0.7605199390500477, 0.11055156354618291, 0.5979392323214198, 0.1611359535887473, 0.4170874948484177, 0.40678929584069345, 0.5256681912799148, 0.5293721212867486, 0.18210061188986698, 0.227082084263199, 0.30175064502889454, 0.04917885438744351, 0.8382568416322713, 0.5008117029271432, 0.29475525734968333, 0.713683737113195, 0.8256769685885332, 0.2549471781800773, 0.7144399704095807, 0.5075020367878764, 0.8238994836815834, 1.2132441407226455, 0.28209242037266846, 0.8122055826931539]], [[0.5214187828411263, 0.5208670517503824, 0.3415602874128397, 1.3127905900172174, 0.7699126022960231, 0.769673741356678, 0.39263916737774995, 0.7714969102149416, 0.4969110518405658, 0.32821732781983215, 0.08565193236407057, 0.7906870557541354, 0.13238096042643557, 0.6146625235522135, 0.13635798304792743, 0.4258585183647011, 0.38755536222936915, 0.5141156597458478, 0.5236436318340562, 0.23848466790992895, 0.2898840111848124, 0.33590078567857623, 0.04353935979978037, 0.8895156273559697, 0.5378401089488949, 0.2822063768417307, 0.6688235426912609, 0.7990814978118848, 0.30237235150829367, 0.8809659770062193, 0.5644768989145386, 0.9051675607007601, 1.2610485999680496, 0.24440373303426105, 0.7339689029626897]], [[0.45357596534178113, 0.5460986063128015, 0.352959624518492, 1.3897485360490736, 1.0412227971984447, 0.7040412218906678, 0.3821892820307412, 1.049987634800152, 0.4671546348595407, 0.34151043448783514, 0.05833456497475255, 0.7730363441335433, 0.11004338030965866, 0.5968212489821609, 0.16397639493287758, 0.4324642398655152, 0.4619666589429213, 0.5453743267476615, 0.5410131467667538, 0.24547177635518447, 0.35669881952776733, 0.3404574545037767, 0.04587471008983299, 0.9147589255932229, 0.6870653322004097, 0.24258405139254896, 0.6823502766179624, 0.7626591066755579, 0.4080623935661709, 0.902729642446423, 0.6258388971332233, 0.8929549997895028, 1.3537060876791869, 0.2569756283788449, 0.7356165302068172]], [[0.47130487128009935, 0.5757709836862605, 0.3891812989425293, 1.502642149127422, 1.3535586319057686, 0.7033152865973809, 0.3805331067870426, 1.2308625506016964, 0.4299815707164813, 0.4061279861565139, 0.069044906361587, 0.7028121802829999, 0.06871939371860464, 0.6109576497404077, 0.15835221960229084, 0.43850990479947916, 0.38657122710173153, 0.5897847683142832, 0.5385463988274704, 0.28514395955978566, 0.36524084253488454, 0.331117365364235, 0.04552437851200497, 0.9114695119337147, 0.8235585860701644, 0.22289626275255817, 0.6932095001780793, 0.7178430196617802, 0.4786633097851041, 0.878126319035482, 0.6702026878538139, 0.8835000495748024, 1.3684609795413163, 0.3065589500918664, 0.780293221658769]], [[0.4072996345938881, 0.5206976309949527, 0.3908648054884375, 1.5166225174566001, 1.5318300455561165, 0.6957729660842563, 0.3797364938813037, 1.2327552748433117, 0.4537786091132035, 0.3651904915584472, 0.06060897717540041, 0.7295078104982337, 0.11938034850217213, 0.6260766300190697, 0.1276364629970055, 0.4757761024351189, 0.4299398652841397, 0.6179917946662323, 0.5963132617753112, 0.28636122352684534, 0.3382030250107143, 0.31041323931259757, 0.047583363884663596, 0.8539154317200686, 0.8463820840822069, 0.2214886974963929, 0.7111852296996842, 0.6775975293106503, 0.5465010647059763, 0.83248868131612, 0.6339494416575222, 0.8796353135223842, 1.3629393390023916, 0.29296584930531655, 0.8094624568111957]], [[0.29624407254259805, 0.5285174088478096, 0.3903932053787944, 1.4421038544420568, 1.6390247921211167, 0.6845087304006858, 0.37482552627959864, 1.3096710556714715, 0.41469423182123416, 0.3330868645555138, 0.059609446174690474, 0.7368628570647581, 0.14548125480590368, 0.6294745158442554, 0.10483448997194877, 0.42183409292606144, 0.4258366957313612, 0.6397428886481322, 0.6028661858136611, 0.28149045396261363, 0.36529562512598407, 0.2883897646913494, 0.05717958727011978, 0.7909546593622145, 0.8780842814631858, 0.23179709271284715, 0.6805309974097326, 0.6391399155992638, 0.5383972131618755, 0.8349416172671196, 0.6094693540036236, 0.893907445824356, 1.393873262644083, 0.22664163662577627, 0.8185360362537006]], [[0.2565834433481913, 0.5212596759901871, 0.394656789343808, 1.300512371045322, 1.6653447127508505, 0.6727013075551875, 0.3690602743402203, 1.4074481184795762, 0.4387437206324256, 0.3458699113936324, 0.05472688391203653, 0.7758340488071074, 0.21636168811636597, 0.6715275139773402, 0.10554382184978614, 0.4410922077070627, 0.48228897029692175, 0.6243652789962783, 0.6281026541756557, 0.2883079891983428, 0.35122559966424033, 0.26834616770552666, 0.06729137854313556, 0.7306799049266588, 1.0067956855708569, 0.1964854300757934, 0.6746722820353266, 0.5870742291392558, 0.5785040354927349, 0.9217689635228599, 0.5088043261557849, 0.8514915225507451, 1.3391564123989705, 0.18410701531123444, 0.8888315229026279]], [[0.22558567394291404, 0.4871561157950078, 0.3959762226197612, 1.2866132012935525, 1.7819194870224335, 0.6571820536350046, 0.3628356061587353, 1.415543026473504, 0.4281478668537644, 0.390734231258909, 0.03838155059006033, 0.8018485058370027, 0.2945231975994498, 0.6922742519765217, 0.1155369837741729, 0.4927161982576308, 0.46336205492951854, 0.6339290746605701, 0.6775693498173132, 0.224891007674762, 0.4156907559177557, 0.27514600281869384, 0.0625013823284449, 0.6568639063869312, 1.149548222858952, 0.19118828748516292, 0.7139643891806838, 0.5587295490475126, 0.5703070107098595, 1.0868144669077255, 0.46793830785457624, 0.8699385975414793, 1.2345198990579442, 0.1913315670129252, 0.9756966104403609]], [[0.20639846329379657, 0.4942227716119167, 0.40020855580289755, 1.1175044078495238, 1.856192685087381, 0.654399035319179, 0.3691751329052042, 1.4531847128830568, 0.3897592203233226, 0.3667391657911361, 0.033861681154332066, 0.8335300283151481, 0.27989302237899266, 0.674847350303589, 0.10154837443082837, 0.4795805818677161, 0.3937196445145206, 0.6006324215251996, 0.7381800347558487, 0.22555545029201296, 0.40138202944744106, 0.27613089417918474, 0.07144583595057041, 0.6049345221838706, 1.2388444389151916, 0.18092043348546602, 0.682423108938391, 0.530180161642256, 0.5669838542061287, 1.238302758241924, 0.4203213257691798, 0.8637121524893694, 1.1674060403154334, 0.2103923047115151, 1.1531738384646941]], [[0.24719169138041847, 0.5435149991281256, 0.3693199034232828, 1.0572750055015485, 1.860409697869614, 0.6841724877826796, 0.3877463376446355, 1.4602801549802131, 0.37184854629425157, 0.31152077510729204, 0.023308777066055286, 0.8937685281131156, 0.306963201296365, 0.6894679935189937, 0.10117923419027319, 0.49686415772408077, 0.3948284693650304, 0.5535931377125693, 0.7382444482925666, 0.25689861185924745, 0.3958453539616807, 0.24762280214834037, 0.08122261156555492, 0.6031336257580093, 1.2716046432427617, 0.18370094373352544, 0.6176044685172473, 0.5070367937077158, 0.6014055392410491, 1.3291554508738335, 0.3741662224185255, 0.8792542804556669, 1.0726278678463037, 0.230985723474013, 1.3436793207440494]], [[0.30715509713765654, 0.6084213637849782, 0.356158815133715, 1.209981404988549, 1.8227944943923315, 0.6935444430673467, 0.4100884886175979, 1.4416479521041676, 0.39410842415407127, 0.27743038652208596, 0.04322614697941288, 0.8319182381922718, 0.34258874863904565, 0.7074313738866741, 0.11722434016763839, 0.5088609811757897, 0.38578351947930406, 0.5509436577144682, 0.65204463333824, 0.4073885153182336, 0.4565380360650066, 0.2222007810132154, 0.10123542252445258, 0.6868662549309945, 1.2559553403427903, 0.22946315217970065, 0.4808710398124686, 0.6591215770367833, 0.6605817470216104, 1.5045358635247914, 0.33928488818031544, 0.8461187284064646, 0.8955436876080378, 0.24593052115348868, 1.524078989454561]], [[0.338701273806565, 0.6745621435712923, 0.34572069995170707, 1.053563441873899, 1.8053156005161752, 0.7041958013039735, 0.41640748242665987, 1.4479014371013355, 0.4457306928360657, 0.27317481555716683, 0.060963461638032575, 0.8174983254772729, 0.3442208892033405, 0.667294144151287, 0.10304068720775664, 0.5232160196148018, 0.320432436532467, 0.4969988412943849, 0.6236351674404816, 0.4029692451242467, 0.46687471603312425, 0.20675450923342686, 0.09620677515332184, 0.63315188525092, 1.1878361781750704, 0.298691955615743, 0.40083789912811374, 0.6484200263894876, 0.6314625151385862, 1.4972477499352004, 0.3234423257108357, 0.8470206594723806, 0.7898849123899259, 0.18013675021034092, 1.4604069923378655]], [[0.37756411566062176, 0.7754382360400522, 0.35600339517870455, 0.8672764474001384, 1.7423404681274004, 0.7091974305022111, 0.40242813724552395, 1.2524909972676865, 0.5006459759093498, 0.3231474370161583, 0.06772609068424995, 0.8587397565163823, 0.3733639384234171, 0.6840157960085768, 0.11283249344642654, 0.6224829554073633, 0.32326317331116994, 0.4801391805743695, 0.5376785055473225, 0.3834110301369208, 0.4301443645803267, 0.2749129408872143, 0.10846832198711233, 0.6194179626971976, 1.3097176016786258, 0.4294520466438603, 0.42650278551043846, 0.6401910098491028, 0.7577818817990745, 1.49741599372257, 0.31267546575549, 0.8084877963923955, 0.7280470366993251, 0.16811972450829155, 1.3374647783106912]], [[0.4790796303650625, 0.8165354240953184, 0.3486408469791078, 0.8281259479661328, 1.6331009996571422, 0.7684400597160136, 0.4721693672569624, 1.026132864604517, 0.554897054777365, 0.38988963759833983, 0.02992046909120794, 0.7984164221153327, 0.40140762775644767, 0.795624176660464, 0.11342043388874823, 0.5734228231181457, 0.332307933871666, 0.44824753265363604, 0.4813292940586706, 0.41080162658262365, 0.3858193176123162, 0.3377702701769636, 0.12076738195365039, 0.5740302853605099, 1.5182127693260576, 0.5310091440762019, 0.5508223082665707, 0.7564535808228485, 0.8313907117042828, 1.4447577192533285, 0.3890237555525522, 0.740554004294648, 0.5686006209069073, 0.21242042934974464, 1.171305587581917]], [[0.4985591861409572, 0.9010304972347245, 0.3256793400390732, 0.7964815843984346, 1.6234081058724907, 0.6512617330560371, 0.3731886150706852, 1.1261544707502957, 0.661943898127447, 0.3899868856242332, 0.02915851215733124, 0.730134388794739, 0.45552174652063315, 0.78931875316522, 0.11387430084530831, 0.5538164388897425, 0.28720714723331614, 0.424933859675997, 0.46377767433957684, 0.42611340942564435, 0.30507455878168277, 0.37097620303887874, 0.10085727905911719, 0.4824242841061553, 1.7019861603892545, 0.711686413275278, 0.6829507843815017, 0.8981717425047993, 0.926567999812381, 1.4895764879910343, 0.47912060007905544, 0.6929814433820827, 0.43847573153941644, 0.26419576741157036, 1.0529391642827042]], [[0.568796902055881, 0.9492124098021105, 0.3260738693859436, 0.8278132627415268, 1.726597124879052, 0.6635405075888711, 0.3720254457008001, 1.1631977289363808, 0.7665846115406789, 0.36932253018159605, 0.019963424322757044, 0.6978725785319749, 0.4966884522613735, 0.8669177073628803, 0.12569448804109684, 0.4946017502061534, 0.2951128377812442, 0.4773938466965863, 0.474391060140822, 0.32550147822808306, 0.26358357309932495, 0.4438256124113168, 0.13947847212723585, 0.32659335643011067, 1.7470807271315776, 0.7980811779328877, 0.709135154521975, 0.9387162680022727, 1.026947121812925, 1.3485831406333655, 0.5502017773532484, 0.6659902927216387, 0.4641941198527915, 0.33618635629672555, 0.8334489885659087]], [[0.6327740801028094, 0.8950360736224563, 0.32150722540325083, 0.7760715296719013, 1.6430411334247523, 0.6334456137942823, 0.35979468499915435, 1.1917348819316662, 0.8890655587985833, 0.35614392423614566, 0.027323483460006244, 0.6869434700863624, 0.5735884834953953, 0.8420480053244637, 0.18507967144039122, 0.49428455894303214, 0.31034571867155125, 0.48018888660169534, 0.6211235220898512, 0.25691380975132516, 0.24963849367168836, 0.267170373098019, 0.22305449661008753, 0.23847316365124255, 1.7643518357575365, 0.7895436478868296, 0.7055002148462897, 0.9733554789348257, 1.0970814975468595, 1.2408788187533346, 0.6123849124102916, 0.5893366331754339, 0.7086937990204677, 0.3655455903387935, 0.7672276460986998]], [[0.6650244834446893, 0.955962411714856, 0.3197758358816549, 0.8757071835278307, 1.4749132917859185, 0.611723105042849, 0.2983967988301032, 1.2266442813027372, 0.9200300090273114, 0.357142279624139, 0.02515319197665592, 0.6989509784945096, 0.6868896813748568, 0.8588589558805275, 0.18122226455724166, 0.4940934000694758, 0.31656139274545103, 0.4778776129853768, 0.6092051365463029, 0.20698320906375367, 0.20128061936225716, 0.23826154709491476, 0.14546583827986664, 0.1628935243895578, 1.7312529216309234, 0.6488499688595593, 0.7266224516323582, 1.0233784484100898, 1.0846675084772788, 1.1629854408404456, 0.6241461641724205, 0.47878009738652894, 0.6796722979993122, 0.34426411569558407, 0.5994790078998211]], [[0.9419296822100583, 0.9703915241413672, 0.29730465570660575, 0.9251565853297513, 1.298214769559129, 0.6596907687792626, 0.3247461105579237, 0.8543009558734012, 1.0077875517452353, 0.3261943094428126, 0.03567364745026231, 0.7836224763856161, 0.7696857417764766, 0.8518874586688046, 0.2769176543449926, 0.45534424773016047, 0.5133770694345611, 0.5268284375326198, 0.6444493281543314, 0.1687976691407319, 0.21939935287823165, 0.24925975900216446, 0.041152623592302, 0.136696240011906, 1.715647079831016, 0.560742989104062, 0.7200208183475041, 1.1816580146166749, 1.0222291781178945, 1.1070768322067164, 0.5892107219168026, 0.42774648822478667, 0.7014941716195624, 0.3692139894750322, 0.5937783166219489]], [[1.0315493050519595, 1.0096739684028668, 0.2977013238688709, 0.9824039725691498, 1.2145117286759106, 0.6527231392506526, 0.31998771388654945, 0.8781837033778409, 1.0857002237974134, 0.30951414933910093, 0.06664285328127476, 0.7773116854754045, 0.8041319577906969, 0.8540003918309462, 0.29573400280693807, 0.4351491695487175, 0.5081151542070906, 0.5910828687812688, 0.7751248861346318, 0.17681910146862198, 0.24470917545288717, 0.3446723968172357, 0.04334781301060002, 0.22352821146169174, 1.5644384875374593, 0.3535253450593654, 0.7412766373049738, 1.1573800542827493, 1.0342894419432342, 1.151163761623643, 0.566774117092472, 0.4342559599244791, 0.7360921920749878, 0.4511537928806438, 0.6299144818195357]], [[1.089525718604924, 1.0097516335221264, 0.2899973874288438, 1.0516599625230798, 1.149845299622561, 0.6656671729357041, 0.34901579495427504, 0.8673420337830149, 1.1233085701802143, 0.2478220637391612, 0.07442877719716207, 0.8545231119331682, 0.8160927690090191, 1.0166875632343233, 0.27416571000911166, 0.4337435144107077, 0.4273382749397666, 0.6143203528211095, 0.7816979671339567, 0.1755080266835517, 0.2515696078676939, 0.3294471609262383, 0.04259603390647548, 0.35293202078889696, 1.457772155287283, 0.25359102388157273, 0.7645040835007898, 1.121764855254183, 1.066856848804206, 1.1828193333996393, 0.5375243024879574, 0.5454747168054157, 0.6524186165072395, 0.42545327614446576, 0.6729975504383592]], [[1.146155165744012, 0.9726368320226221, 0.29040390245908515, 1.2090978972936555, 1.1313803390553034, 0.6596411613813855, 0.3284175221895858, 0.8739205085552045, 1.1729237986086514, 0.24546293082385798, 0.018169077157418195, 0.8584188151677032, 0.8185248946815592, 0.8848769779109461, 0.36961879641262596, 0.43274386151840927, 0.42374365750154497, 0.6478910352812994, 0.9061824226404032, 0.17576486452365897, 0.22766109122431993, 0.3948347686405743, 0.04178699186557097, 0.4886073064047139, 1.425019866294572, 0.2593043726169916, 0.8052536851932857, 1.0914298768765727, 1.096685120235198, 1.2630305668889568, 0.4533913297268901, 0.632945021288136, 0.5608454925933282, 0.4070885538589928, 0.7044701456846348]], [[1.2918701062988511, 0.9073766485583046, 0.30190467603562293, 1.407237037948126, 0.9972475033907982, 0.6741860614362803, 0.3637922533212788, 0.7900514049473082, 1.173982559532762, 0.2010925150860059, 0.036228543173422646, 0.9441927302214922, 0.7957686787649417, 0.9015507504233443, 0.5091569049705652, 0.4581116136698845, 0.4216648992036325, 0.7040974236357102, 0.7667670472703924, 0.19208065226548987, 0.21723709019278659, 0.29918056477936117, 0.06945424878436311, 0.6136566410463675, 1.2451200174344548, 0.27267538572260686, 0.7869054578638199, 0.934668555140514, 1.1452145929176296, 1.2794406460452863, 0.4085382324477107, 0.7019491124944931, 0.46716003026654696, 0.39219718015152366, 0.748250048059385]], [[1.3046138042481679, 0.820556849844227, 0.28248041733504653, 1.5475456505753924, 1.0490565768461166, 0.7227153088998395, 0.3692803602187894, 0.6162517381375431, 1.2700679221981077, 0.17540712820906562, 0.04772249723040522, 0.6369668537145696, 0.9140935330464279, 1.1111920479856776, 0.33646987012898405, 0.475096602080501, 0.6221779820153792, 0.6734222962795987, 0.8480105575013059, 0.2616383101054501, 0.2611627217567223, 0.23761254187669928, 0.05316480355881799, 0.6176558630148551, 1.2208884605083345, 0.27349164621891187, 0.7513032411741271, 0.925273408362186, 1.1692798195166982, 1.330288306162121, 0.40819130948354587, 0.7344472918538707, 0.46351712891714403, 0.39231833371623304, 0.7664132365203006]], [[1.3194508706414525, 0.7635840699832549, 0.2821663897410959, 1.3848666410709813, 1.0702144650647307, 0.7210571824967863, 0.3682686210002323, 0.43832965297935644, 1.2922253959901777, 0.3178271902254406, 0.04586832612532338, 0.6138949594435359, 0.9644007810699924, 0.9532238077892012, 0.32562532429885443, 0.45468767068483784, 0.4808881518634889, 0.7147082580170928, 0.7839778936179636, 0.34342685466221473, 0.2845376720963736, 0.18694283208420648, 0.05197130223763242, 0.4101173372442898, 1.226374771618529, 0.24417882272753694, 0.8402882971549911, 0.8754948345027054, 1.1903192802457039, 1.3194123305317813, 0.41374555387153616, 0.7380953653549347, 0.48898884759768924, 0.3652969421719409, 0.7935012016993699]], [[1.3219020845535927, 0.7101834728229198, 0.28655048018781576, 1.1791871619835308, 1.1358349969591015, 0.735038027198224, 0.376739915091854, 0.40843951790953686, 1.240584923142432, 0.3735401178267919, 0.024558004138363884, 0.5994454451511106, 0.9703455514191885, 0.888482010820688, 0.3298188552324602, 0.4448686651298337, 0.49146847400040683, 0.725840215335215, 0.7895377391685943, 0.4274047740502501, 0.2966103642846925, 0.23501623314540776, 0.16667262548781722, 0.20353752828325689, 1.1939651375329343, 0.3099633629738314, 0.9530514974684927, 0.8050880131934922, 1.1608733132228308, 1.3123616945284702, 0.4601480872109548, 0.7146410405915633, 0.5612342962990609, 0.4277953324156066, 0.7675895000171356]], [[1.3128009680420072, 0.6791521065536867, 0.2839914369187993, 1.1071897375280928, 1.159007884492127, 0.6828171288623264, 0.4186355507167184, 0.4558225622766704, 1.1776176205378395, 0.47594637109645876, 0.019095556641375227, 0.559735260014773, 0.9836925892453441, 0.8359008396640859, 0.4010185844520247, 0.462150245785399, 0.5436091070862558, 0.7084948163341305, 0.8062425234864736, 0.59724026126631, 0.45383056110097375, 0.34760737663597246, 0.31741657001655377, 0.29978275952302214, 1.184488960352187, 0.41133518536141583, 0.9766294514992615, 0.7973969942411329, 1.123921957842416, 1.3119037583949407, 0.606993660620206, 0.7068357417793182, 0.6217730265044725, 0.5546548704927511, 0.7316816441317897]], [[1.4509538262815105, 0.6934931868472571, 0.27957836709218536, 1.1128583559075245, 1.0482705433729362, 0.687900105799141, 0.39991382506816603, 0.5409787142728587, 1.1170719789366046, 0.6181374722913071, 0.018727585599336936, 0.5892186699023332, 0.9791840662722118, 0.8137393486143667, 0.3523953311001113, 0.44157262416171345, 0.5271630341994371, 0.7303122959339873, 0.8457913989752098, 0.5794954458834097, 0.5705014977549684, 0.3515988715361148, 0.19047217695270158, 0.3990462998805928, 0.8787837313750548, 0.42640948428293074, 1.0023969444925316, 0.7418046862473506, 0.9762226484449379, 1.2989543782576591, 0.7629823963306452, 0.6805503674684825, 0.6477959694255006, 0.5649176593773158, 0.6980124822913126]], [[1.4512993387497648, 0.6187002203640308, 0.2827518406504547, 1.13910481614928, 1.105928848381135, 0.6598140456734651, 0.41669264980214027, 0.5177274026548262, 1.0857249095276043, 0.7201205975387642, 0.011537229468523808, 0.7204568977411463, 0.9775003043996725, 0.8128654342497744, 0.36875174262806426, 0.4195346000612083, 0.573283513132053, 0.766871071894042, 0.8851478087373245, 0.5548478479767229, 0.6177589498787768, 0.4495425675245278, 0.15381651273679606, 0.489065405790997, 0.7007386642084976, 0.40149240874852477, 0.9873767474151467, 0.6186149173385344, 0.7665808254853669, 1.3122554622925282, 0.863869258891306, 0.5936009652646622, 0.6318760803366754, 0.5356907571102724, 0.5867464946841356]], [[1.4954083989164593, 0.6527220779747533, 0.2847958286605206, 1.2324258622934823, 0.9206404522613049, 0.6677372823908265, 0.40628412328450314, 0.5220756940798817, 1.0680275292879051, 0.8064341349461492, 0.024105070578236833, 0.7382709938340282, 0.9856596255819605, 0.8185559600627625, 0.37557844254157835, 0.4092248179456652, 0.6469521499266983, 0.7497196834697608, 0.9538982851664879, 0.5336026920473589, 0.6192505194232211, 0.5810085342819349, 0.15654036773386315, 0.5400652387497736, 0.6053917775898173, 0.42141625840643127, 0.9821390816708342, 0.5500194169760108, 0.6879625197579031, 1.3248356320985364, 0.9553403524974875, 0.4671066900886327, 0.6390837354415472, 0.41857726658779204, 0.5158078406042079]], [[1.5303395691495947, 0.627891572834146, 0.27652511033579913, 1.3085681213249254, 0.8286212149790901, 0.680519602859341, 0.40018667450411144, 0.5121878747123327, 1.051354749852906, 0.8722118072121429, 0.020284091619521476, 0.7561479166821057, 0.9670731517992759, 0.6275809843025868, 0.24655506001729474, 0.3839545371196388, 0.4299887588561907, 0.6659275867087995, 0.9540160291518334, 0.330637886318103, 0.4951567148611005, 0.6039173768138378, 0.10992493679212803, 0.6027220467233334, 0.625393325260222, 0.4199321963719112, 1.0252001730282068, 0.5146431943423412, 0.6438728941278985, 1.3284902950251063, 0.9830017167177374, 0.37491646771200804, 0.6662059856434595, 0.32584824778448707, 0.5069400014497394]], [[1.5739956592153752, 0.6372359455614017, 0.2620541617097601, 1.3180867067148812, 0.7818353782689593, 0.6901154662576857, 0.4013788087556597, 0.3729438644716806, 1.0341202895517987, 0.776198128732201, 0.008725975924071165, 0.7127630662236475, 0.8977666223301342, 0.6316045649741817, 0.2277302343248933, 0.35898085918784534, 0.4422511599757023, 0.636203760531879, 1.0293996929917753, 0.3031913300873416, 0.418263491313404, 0.5249172764532295, 0.11247454170308446, 0.6252729143325855, 0.6660876375209187, 0.3789778501182864, 0.7907449456527655, 0.5211704772004824, 0.6500018878742071, 1.2909836762838132, 0.9549555618231891, 0.4742493279175848, 0.6958181726606576, 0.2934609315475561, 0.540193526617514]], [[1.643945852472469, 0.6430753968530851, 0.27785124496127894, 1.291478178295298, 0.7147456104063725, 0.6982497006668014, 0.40755841154986183, 0.3893901851522238, 1.0078209591812715, 0.772522516302637, 0.010010183477002615, 0.7081237929223176, 0.8299301516272526, 0.5598538372387174, 0.22769524444797548, 0.37916751190023057, 0.40713406490880655, 0.732432247564163, 1.0049081895027787, 0.29706928320569626, 0.3979099120591638, 0.49574345171090317, 0.11196390791598894, 0.5848688799518718, 0.7439905659412214, 0.3761321501868928, 0.7606080441803156, 0.5205486818185507, 0.6654818533527718, 1.3139287372121302, 0.9526043427647722, 0.6266423559921799, 0.7953246431471109, 0.3155610044428445, 0.559946421535874]], [[1.7199850003846149, 0.636786719691492, 0.26022572306884867, 1.2726610350531282, 0.705944860719022, 0.6839119023713438, 0.40670917263370265, 0.4064439471357258, 0.7784419677643827, 0.7402957027663535, 0.01029770580487238, 0.646204792444983, 0.7933522829309252, 0.5405537552684261, 0.2252134850399981, 0.35675824550360236, 0.4553643733410695, 0.7163361314013711, 1.0675386676243397, 0.348419977397749, 0.3901668201379996, 0.5224579580370874, 0.11139470931358741, 0.5431923394111204, 0.7635338914294427, 0.3551462425788471, 0.4496487029189732, 0.5314995208177242, 0.6691793520575614, 1.2908759062229747, 0.8943704454758225, 0.7612080866611775, 0.7214927866474716, 0.30859577106567543, 0.5777644448235404]], [[1.7122234890969947, 0.614915819863572, 0.26985541896230775, 1.2410189531865834, 0.741170032203749, 0.6848281157846211, 0.4157390922972144, 0.40512599740091415, 0.7761450717063451, 0.783225148293247, 0.013547864084931114, 0.6571808284224231, 0.6816000065926087, 0.5414145170940962, 0.19723206058217718, 0.3489748351148425, 0.3607003921698072, 0.7403354932309261, 0.9915805594854075, 0.3912267916214893, 0.49526274773930123, 0.5629166748826975, 0.11075287140968207, 0.500445951380192, 0.6462250086295573, 0.3720650133703214, 0.3411550001111562, 0.5249679472224769, 0.6742658293935184, 1.213382986215521, 0.907755889887083, 0.9592134148010147, 0.8164423184306526, 0.30810870771035964, 0.674193749255622]], [[1.7592914709337757, 0.6142505420353608, 0.24948820739591143, 1.2250292658005955, 0.7853755174186076, 0.6848990449344178, 0.4154631083963408, 0.41166325056621844, 0.5789930863585404, 0.7199218665911161, 0.006025697259878809, 0.6605161516857789, 0.6400335917693827, 0.5199146505508826, 0.20166730919350068, 0.3065040324700937, 0.34695400355295264, 0.7573148483912948, 1.044825318946383, 0.4674355925112561, 0.5260678065321331, 0.5693032516548764, 0.11056375704276142, 0.47980788280687325, 0.7135603111261564, 0.3632313406974658, 0.3333191113156199, 0.5380263988676448, 0.6749464297715831, 1.1287405912262862, 0.9688015150352598, 1.018020729575913, 0.8652726291594235, 0.36473553929387026, 0.7561404901398265]], [[1.7938329376614894, 0.6534156355171556, 0.2377738662439439, 1.1863364859436354, 0.7173903504567075, 0.6738824732723938, 0.39331673737371364, 0.45199034942383065, 0.4938027773371004, 0.7143673378072467, 0.02996255607320101, 0.7334640812830461, 0.6323524732236954, 0.4893052107386133, 0.19077135716990182, 0.30601103981492395, 0.35255295561105704, 0.7774674305931966, 1.018320586534829, 0.5050486466688078, 0.514494545205944, 0.6369906392789346, 0.08902776803856488, 0.47482398662388714, 0.7659336926584344, 0.3421729987747227, 0.3866910343142843, 0.5383745401059692, 0.6539613632296271, 0.9962549586617377, 1.1592849829000658, 1.009715281758539, 0.8646935394006714, 0.40275493330491013, 0.7402044687621163]], [[1.5721124072797452, 0.6497840180808094, 0.2332127099664329, 1.1077107681771523, 0.8121119055373675, 0.7035869015241103, 0.39673983235931765, 0.488677007291089, 0.4753931114193404, 0.7429573131398255, 0.030470146514452408, 0.71016680263615, 0.6256930973892856, 0.48394668089242826, 0.19175901413831983, 0.2902191231665042, 0.3668186977529757, 0.8060565508059578, 0.9984025053434987, 0.5017021848532317, 0.5061655000955193, 0.7148177415791059, 0.07077442611481452, 0.4409692912133359, 0.9186244746722827, 0.3780410634842779, 0.46620970502686476, 0.5452135542732492, 0.665302263257228, 0.8695696115636871, 1.31857953344411, 0.9618600931187233, 0.815319113065959, 0.4458299800661574, 0.7988108298238057]], [[1.4213179947446943, 0.6604394752516474, 0.24436294505194872, 1.0480414017185953, 0.8125984451388956, 0.6850146212014986, 0.3988280100284416, 0.5259615048126196, 0.43051623544113593, 0.7013697685605152, 0.029999345968768448, 0.6627915034772911, 0.568799357449654, 0.48886362993479004, 0.2119570653628618, 0.2961059688565081, 0.4126041726016053, 0.9068657168259364, 0.9926068678818163, 0.5392800082258256, 0.5013437605240021, 0.767669625557566, 0.060556775801825384, 0.411123385864862, 0.9761420426236824, 0.4229836896394565, 0.4422593426421338, 0.5920595602330355, 0.6695180259980426, 0.878379586768558, 1.27747609554672, 0.8361407940140586, 0.8258367297947935, 0.47715476483154673, 0.8429752494774514]], [[1.213301258319005, 0.6781553697593161, 0.26296507686281495, 1.0659965708941646, 0.8415141056479429, 0.6685832714508224, 0.40007823370656337, 0.5460679411130558, 0.42536079409658145, 0.6658302972273076, 0.031392518288737375, 0.6548572468836915, 0.5405372846117645, 0.48014872725256874, 0.20188444464558164, 0.3066407295515067, 0.4387017697583543, 0.9066891545294065, 1.0656124932344404, 0.510358473254511, 0.4605724756459165, 0.7633102059625264, 0.05702766321034274, 0.39521506929616795, 1.0809838853871816, 0.39466436550864253, 0.43303573302657883, 0.6455906796030715, 0.6705825730286867, 0.9473153169440984, 1.333114942670634, 0.7890431970720286, 0.8824186116253169, 0.5450931940236019, 0.8219365887687318]], [[1.0376360266063736, 0.6726453073380109, 0.25829145611793763, 1.2061943934027088, 0.8674336033587641, 0.6529733766038612, 0.39774681993334343, 0.576392068079807, 0.4349733379262427, 0.6853826057293098, 0.04253495237734872, 0.7285720915035732, 0.5270692020671733, 0.4734119080881681, 0.21472999761130793, 0.3142026090742437, 0.44473298442296605, 0.9352400675486772, 1.0761687108775786, 0.45549361360356866, 0.47635444709839253, 0.7596267423614889, 0.056513633790463584, 0.39461005112560965, 1.2157121140373524, 0.35767234102998585, 0.4576018921170474, 0.6844232042147471, 0.6725966998136041, 0.9559157164692973, 1.4139724823049802, 0.7551179485515049, 0.8059033572611332, 0.5896824066337585, 0.7697660364628927]], [[0.8535279040372205, 0.6777351175851075, 0.2723316901862905, 1.3784508657865697, 0.807581748882439, 0.6250360150385436, 0.4078158418629029, 0.5279845273654715, 0.3468340991980471, 0.815323897470452, 0.047194468602167895, 0.7780108355765194, 0.5569225258335683, 0.4990896464801877, 0.2523756941897987, 0.3266318340552826, 0.4455820710090439, 0.9019500969770158, 1.083057434295916, 0.39077460567824496, 0.48134205335050617, 0.7975038918811694, 0.05548958937378735, 0.39818139084924586, 1.2909255169813276, 0.3023661052679874, 0.5364626814364133, 0.6836526137084871, 0.6645581567644485, 0.8871531388255335, 1.2708117009254323, 0.781430054030094, 0.7062537679848919, 0.6181609502099834, 0.783854073349916]], [[0.7871155300568128, 0.7061494052039736, 0.2811238585385832, 1.5834247925494278, 0.7842990699150765, 0.6185124711319077, 0.41877331475825486, 0.537952127272112, 0.31330849278135386, 0.9593322803259574, 0.06413773137360705, 0.7827097855946566, 0.5800369014818331, 0.5178238565711631, 0.2597249441546802, 0.35325114585094275, 0.4285126254895793, 0.7673562136652973, 1.0765648538092973, 0.3823154070696707, 0.2751846636112786, 0.5223368883764038, 0.0565092591035016, 0.39070754939716273, 1.2629895273839375, 0.2372364918185776, 0.6307731959196221, 0.7110628187617029, 0.6517618206815805, 0.9062785524600385, 1.2245037453001584, 0.8213981238038106, 0.6506483079326786, 0.6433946969853549, 0.8285831753054997]], [[1.2310455421037003, 0.733003398039946, 0.31110566945696033, 1.7467935195491227, 0.811794538570044, 0.6247051436722697, 0.41356444968531525, 0.5553150272192242, 0.33128264313653255, 1.0369777166378786, 0.07175415638158342, 0.845648873159673, 0.6678359893544974, 0.5368150763693179, 0.2541132986866346, 0.32779052222590804, 0.41095805245028494, 0.7074336022974734, 1.0450089594446872, 0.3654651112766174, 0.17033463241257424, 0.3954675238651193, 0.05343653992062329, 0.41267888325201507, 1.1861232133860073, 0.2252483426191187, 0.587829672343911, 0.7042239183762466, 0.620589610766491, 0.9684737783690839, 1.2338803568719316, 0.883225187292759, 0.6680342047532303, 0.6533920838216926, 0.9169403920064821]], [[1.4350673237030387, 0.7439164647515536, 0.28774950370370184, 1.9406741833468, 0.9412967820691751, 0.6091693624459078, 0.4244297992605416, 0.5031356592190609, 0.3030070796888368, 1.0846434835516816, 0.04624341064555926, 0.7629417062493862, 0.7210414867480874, 0.517001283040778, 0.2604745368481533, 0.32968597485740914, 0.4379866423113068, 0.7460654955301993, 0.9017640296114736, 0.2899719042446627, 0.15935804584458935, 0.20682594119148187, 0.04788634536133496, 0.4164007614024021, 1.0944744329316634, 0.24347289520172516, 0.6061823121925954, 0.6482639397734649, 0.5645522523013952, 1.041113662320236, 1.2246361409067879, 0.9177058952604704, 0.6750104170734428, 0.6354151326926749, 0.8996404187657843]], [[1.3756519962204146, 0.7161162617914771, 0.263162460390814, 2.129387058777254, 0.954996971644064, 0.6076374431559929, 0.4138362207731957, 0.5227122587996141, 0.2794716507935549, 1.3321944482269772, 0.04154421010879569, 0.8925607612039657, 0.7473404782227706, 0.49528441926409034, 0.27430670492713544, 0.32501928113853723, 0.4396313912070448, 0.7609649974321047, 0.9665253529210606, 0.3217938945116973, 0.1525699726305348, 0.2580303917598462, 0.0376499397855756, 0.4410548418977718, 1.0669317079016183, 0.3898752042928783, 0.6160144348875347, 0.6240240180347028, 0.510777947545006, 1.0576420388012537, 1.1974073800451872, 0.907129471552307, 0.7351520942527014, 0.6372258464777121, 0.9236575571856199]], [[1.4040116914968226, 0.6955116588781741, 0.2590425694611928, 2.2380485228006415, 0.9617702671012351, 0.578140091615784, 0.4070537222571219, 0.5120579546420534, 0.26270896010080597, 1.4762575494841612, 0.0345751210058031, 0.9679559921292773, 0.7712666689674613, 0.49663476279528357, 0.346348116824783, 0.319191126875652, 0.4394760462824383, 0.78868696014776, 0.9623608719759893, 0.32040905952328197, 0.1671230990606251, 0.23407172777976576, 0.031337832446705675, 0.47594811399346026, 0.9992054289345833, 0.4509091308095616, 0.650545850472154, 0.5711343397047434, 0.5093343099793621, 1.0511853627162902, 1.1572449096417818, 0.879351050792168, 0.6671329901183616, 0.5659924262849058, 0.8681240241660069]], [[1.3647257990936739, 0.7365000384062583, 0.260354660074616, 2.172486383373874, 1.0404046323351892, 0.6692768717189188, 0.45455394811161576, 0.4972080441091944, 0.2393851692846346, 1.5751156145364975, 0.03233312365904901, 0.9361255485612937, 0.7807407031064072, 0.49439530324106684, 0.3284538342020293, 0.3069108008635699, 0.47535469490019644, 0.80044834440603, 0.9963411238699426, 0.32341616584265415, 0.18344896575537664, 0.2133742903787887, 0.02966106992101713, 0.47819733679462373, 0.9939367818947248, 0.5097949328020162, 0.6848606302678466, 0.5280826190295467, 0.5182618675582383, 1.0422586955931212, 1.0919439888635898, 0.8491881989011474, 0.6227824304791475, 0.538550252173589, 0.7589091828184654]], [[1.280284595885186, 0.7981183949783403, 0.27258209287096086, 2.11987863176857, 1.1023203725048374, 0.6669439755493711, 0.45204566277148006, 0.4978478395340453, 0.21871734192712466, 1.6491299724836628, 0.06374680295793449, 0.9016504285951813, 0.836693125379985, 0.48985533260793346, 0.3631753445711879, 0.3186743824545497, 0.4203991690652581, 0.7870319684669296, 0.9095932513330721, 0.2634940166275019, 0.15748059030616915, 0.20062648910608671, 0.028553001432520504, 0.44590864363173843, 0.9443253530558988, 0.5348995736181561, 0.7197158536264829, 0.46271308631267605, 0.5244624678598868, 1.0397208932547968, 1.0258321848225442, 0.8300311473054416, 0.5853462414471879, 0.5104917999334568, 0.6413608391598749]], [[1.238314826016363, 0.7898901895655107, 0.2777432888386657, 2.0732132568087893, 1.085016177508601, 0.6682954295821948, 0.4611095016072698, 0.4880732931631426, 0.21904804412219742, 1.671724714808055, 0.07387529591126954, 1.0449230577920114, 0.8430321705262153, 0.5127045930887644, 0.36727974924568557, 0.3347528466776145, 0.41551521241536116, 0.9195794979653331, 0.8152042311095631, 0.23310356961235285, 0.17273731011957905, 0.15079549624610425, 0.027813802182396728, 0.4093685455972357, 0.9527211093913831, 0.5321238261452113, 0.7112136725451417, 0.4877592075114157, 0.5349783579642097, 1.00809155517727, 1.0026367721701588, 0.8109385843732924, 0.5190208594969301, 0.5366200540989569, 0.5327992939766119]], [[0.8687273366449317, 0.7911474519423128, 0.2860525024525547, 2.0128732563056912, 1.00575615457545, 0.7013523224256535, 0.47978978999935673, 0.4687938011019168, 0.29676074619195475, 1.6785510105413821, 0.09315119230629601, 1.0058602527410447, 0.8426091440169785, 0.5017235798341526, 0.3965160908614171, 0.33856341484997055, 0.4275956384973904, 0.9530125106171381, 0.8481656815481332, 0.27025132900224014, 0.19285825833137557, 0.11707159408759255, 0.03419617458347328, 0.34804110270107635, 0.9372897322873274, 0.5447838108579293, 0.7009472280859439, 0.5128246064457553, 0.5645038707364016, 1.0003521099779644, 0.9924197369891568, 0.7416132185788897, 0.5032632188158305, 0.6507216247501013, 0.47015527126916934]], [[0.7923279961369818, 0.8576530478663881, 0.28237667685795403, 1.9941588968070603, 1.3215987547301398, 0.711144635633466, 0.5058986413778368, 0.45583225054381493, 0.3392630963905765, 1.4279837702395017, 0.09412917713831857, 0.9880206762616223, 0.8731095748918725, 0.4870319586555329, 0.35668449945511355, 0.3279170495476572, 0.4735348613300949, 0.9522741921074911, 1.0185140422276548, 0.294030917166233, 0.1648443545654175, 0.10150361710099307, 0.035948403437789586, 0.3271838067568641, 0.8426410014287573, 0.5707255784515939, 0.6910128070918415, 0.5503193455517756, 0.5520506644114339, 0.9989376478292513, 0.9539229669024345, 0.7231118035975368, 0.5152362875838545, 0.7965152611375526, 0.44308362254999045]], [[0.629717964082806, 0.7110661243988448, 0.2841749024009003, 1.9987297389386625, 1.332791640483586, 0.6694461924503036, 0.443357062696981, 0.4345609944046643, 0.35402925729221135, 1.3442528041050759, 0.09762395190017993, 1.031361156952779, 0.9291994706589004, 0.48232920733258533, 0.29472462198935984, 0.3212697602145276, 0.5320457992162112, 0.999765378716605, 1.0183642262570298, 0.3361018646451521, 0.15354650240100307, 0.09870487753239172, 0.04143413850756803, 0.38718611934692687, 0.7258669971904289, 0.5552178606374203, 0.6811857629409708, 0.5642223120533286, 0.49991619347769933, 0.936690755611338, 0.8962058291663406, 0.7656824038359427, 0.4920543497409684, 0.8729075513282922, 0.44925476530398933]], [[0.4853282169138008, 0.7020806228026504, 0.2993736150082599, 1.9684649021946758, 1.2942632662623326, 0.6970374742346148, 0.4243199425251761, 0.43102433030935466, 0.3740246868278448, 1.2168822093846092, 0.10443305507978157, 1.0374973756604584, 0.9743569914233607, 0.4648175375623995, 0.28751234954866417, 0.3715123769942956, 0.5235116602864899, 1.027967867084358, 1.0432846837474152, 0.3190526572838811, 0.1744661368273281, 0.0769365726272093, 0.0206645662632768, 0.47850450849875764, 0.6678840061073378, 0.5546179634623067, 0.7319359000884023, 0.5883042416844733, 0.36777030809968014, 0.9055439329132693, 0.8198394979592535, 0.8060922929303963, 0.4598419382433425, 0.9087556568894322, 0.48471180794578345]], [[0.43118317958850927, 0.7093937718318739, 0.2896527331039648, 1.9322527379522076, 1.2629627398876928, 0.7041199751252345, 0.4189168693416533, 0.4644962033242562, 0.4074080554282665, 1.1323158517026972, 0.123156040585366, 0.8496294537483885, 0.9848923483935932, 0.4363065367970484, 0.3003318559765601, 0.31184255602356475, 0.5032277115219144, 0.9003114477229625, 1.0342875425964437, 0.27267127732506047, 0.18295875784269694, 0.1145283393190537, 0.021121902863481236, 0.6156049269912139, 0.7037554724556471, 0.6865551285175013, 0.9392474552555076, 0.6391593250585075, 0.35391870030697864, 0.8814388824571637, 0.7262089016725386, 0.79070322157982, 0.45193492574651506, 0.9630059193159244, 0.5228043407353938]], [[0.2869350371477992, 0.6128722852086524, 0.33953270041824335, 1.8284124458685704, 1.1840970753813913, 0.7274649111229554, 0.3933513566566889, 0.4900497868409981, 0.42526004325507716, 1.1553757706478833, 0.12232074354943945, 0.7989469898545865, 0.9978252906998566, 0.4266931329997795, 0.3069840758401977, 0.2957601114970295, 0.4628115425462971, 0.8315814134693509, 0.9137110389834068, 0.26897877461952896, 0.2103124139057619, 0.10086533481060174, 0.02216393479743982, 0.6208762633554774, 0.7477513844498309, 0.5352172513227111, 0.8233796041427623, 0.6607142898893271, 0.32620222161404866, 0.8912142729137088, 0.6723327388968348, 0.7439660379610138, 0.4203943446720191, 1.0803307443674224, 0.5561734852233579]], [[0.3281692762822045, 0.5454516364675213, 0.4690036175713498, 1.6196714298100283, 1.028837133057611, 0.7911326463919575, 0.38611444568032377, 0.5038638493375739, 0.44105281930022033, 1.1742688875989673, 0.11922659901410892, 0.870093618141172, 0.9769235924987432, 0.4307843373957752, 0.3223114764677911, 0.2642045820906717, 0.49708084696722915, 0.7438856213753449, 0.9242035017531127, 0.2595570889391819, 0.19065562804646302, 0.1046117664469636, 0.031192717790287528, 0.6430815686081274, 0.9277147123123679, 0.27522405370241243, 0.7115911442510319, 0.6738865778774059, 0.5076073728602422, 0.8926248437342599, 0.6944279031418183, 0.7342430844052088, 0.40680361792450115, 1.1969168627621585, 0.5955420974665229]], [[0.22781595560775858, 0.5735784618796929, 0.48028528139999616, 1.4425092262585837, 1.1137157596612617, 0.7619000399865383, 0.4510132974268305, 0.5158686642857618, 0.4658783205762979, 1.0123664429980428, 0.11329932950049287, 1.0240344770362388, 0.9783656698732983, 0.484539437344232, 0.315768713971462, 0.26400942607888267, 0.5017705202316464, 0.7856661523919933, 0.9053126281904202, 0.26611537755345394, 0.20140246460660033, 0.09477520687733086, 0.03526078944513539, 0.6523906062474993, 0.9672342768848414, 0.12455712430162902, 0.5959077972789653, 0.6873868136084529, 0.511509305289513, 0.8775183590076636, 0.7413946648770806, 0.7718817680893268, 0.3855160063069075, 1.2456654030173264, 0.7232745050666487]], [[0.29944210855638403, 0.5205716070336388, 0.4355899904486613, 1.3117380290659024, 1.2058499502049562, 0.7486856460418827, 0.46562501960537617, 0.557339324611196, 0.4482712688711985, 0.9533598611100311, 0.11944098277093049, 1.0794410633018476, 1.003930293146195, 0.49858090704833946, 0.281703099670438, 0.28226136221921866, 0.4974235601669148, 0.7765138201436597, 0.8054287152434493, 0.2943761984496879, 0.25829851604556286, 0.09344211258215118, 0.03490060679219412, 0.5943699291046185, 1.0289232836977886, 0.11673306341359191, 0.49779427759827544, 0.7752608927036396, 0.4427011686025639, 0.8280605739962529, 0.7709239711909324, 0.8131846072944073, 0.36259725697459544, 1.286903995181186, 0.7953189988963261]], [[0.38133047162693917, 0.5440594516309237, 0.4690906837218266, 1.2721363090610356, 1.2646731388754657, 0.7155250070919135, 0.4020438472254306, 0.6275408571019223, 0.42651407261723945, 0.9894668944405387, 0.08820613220863656, 1.112271900535598, 1.0589347787739014, 0.4880395066256674, 0.3309852251431181, 0.27461490240411324, 0.5308801763831972, 0.769182128366894, 0.7323308895683291, 0.4296899603654014, 0.29654006130601873, 0.09224406643298307, 0.021318934083892547, 0.588111888431653, 1.1960607098838776, 0.12044838232778865, 0.4133351026840439, 0.819942204563204, 0.5169630150400224, 0.8372804286502626, 0.7654753693849714, 0.8928551532257403, 0.3757351167993739, 1.1748704157501677, 0.7848178411865514]], [[0.5172338222879976, 0.48072081489873997, 0.4108565398300194, 1.1806839593000844, 1.285483015492646, 0.7313180939996796, 0.4178084896054682, 0.6630557200814737, 0.40993514412411425, 1.0907094290518091, 0.08735954346699026, 0.9531892697949014, 1.1161700862144512, 0.5200625817177102, 0.370209410733028, 0.32489884552703674, 0.5427131545700428, 0.7851685522747112, 0.7442426964639609, 0.4305870238187932, 0.2813337127179606, 0.13656883720727556, 0.018627893530949364, 0.7446135390926876, 1.331607870360851, 0.1334742913979446, 0.423184220702695, 0.7974352553842882, 0.46022105845353944, 0.9384787463485829, 0.7376242668876701, 0.8780783985846953, 0.39014730741829357, 0.991115891521691, 0.7604644652345749]], [[0.6175957109496267, 0.5819080467763029, 0.5292783959532215, 1.1392385344040528, 1.2629630771841507, 0.7216559825341197, 0.4112198509840793, 0.8502403063905604, 0.41494831141479793, 1.058000918787086, 0.10974144907545716, 0.8621935747188447, 1.195727343030011, 0.531073360726575, 0.33241026137170226, 0.31376672670475664, 0.4930215364184901, 0.7960100405011301, 0.7202084657264973, 0.4604369096007274, 0.2938660987855287, 0.21022834065025764, 0.01496158751783791, 0.8668589261385626, 1.4370935756458036, 0.1734190681866068, 0.5012495123525463, 0.8436996969648805, 0.4560032708061045, 1.0269027343570014, 0.7104356064402381, 0.8659782189465132, 0.5360766741412928, 0.9657705368648679, 0.7922816573073397]], [[0.6196558301056626, 0.5624755296318947, 0.5042680253118739, 0.9798975453988248, 1.2220322593465647, 0.740864743431841, 0.41061181298803767, 0.8808769158136547, 0.5067383856649784, 1.0320975263717835, 0.0836628780053541, 0.756597952420446, 1.1328129116407513, 0.48433320429146337, 0.3469404109980826, 0.3243036623060717, 0.49127244599956416, 0.8035229751781154, 0.715772725503521, 0.39270211562017726, 0.24000233602792637, 0.18589014488185507, 0.017391502769198847, 0.7771701251903174, 1.5228386618195198, 0.14717785457804883, 0.43950270389673635, 0.8218531706552435, 0.5168523395740348, 1.018094798149348, 0.6347825913771518, 0.8759853011956439, 0.7270737240242648, 0.9322825472337322, 0.8457657358165818]], [[0.6466094017860088, 0.570030785191128, 0.5137933276769023, 0.9038742575227915, 1.1925881102656308, 0.7267938104468195, 0.4127225455240823, 0.8015372710748321, 0.5214307751274527, 1.2192820499775872, 0.09707745768011258, 0.7438491996195409, 1.0718081265213124, 0.5183968448126024, 0.33370085133300226, 0.3512823212783127, 0.5101858813132154, 0.8485885814530356, 0.7070929739892065, 0.3501931777960638, 0.2736239842573376, 0.18103366292098244, 0.02783051191332815, 0.7681396865798615, 1.5269025192696821, 0.11494960859578285, 0.4284318008429459, 0.8354478737675815, 0.6953984865040108, 1.0035953109295108, 0.5159391371101711, 0.8804450423451288, 0.7105362040632983, 0.8852024010232994, 0.9181086534778958]], [[0.6286725636426076, 0.4806449959991289, 0.4359327321840439, 0.9233253053995973, 1.19011771896101, 0.7092313364403884, 0.4071384185665028, 0.8802610822247916, 0.9352983939015385, 1.3406193826837596, 0.09488023135743692, 0.7058938117974562, 1.0636949041883053, 0.4468753988375477, 0.3299185527223878, 0.37043343227199715, 0.5055684442945259, 0.8874136092252999, 0.7103835959148854, 0.3364500089680357, 0.2602807764599789, 0.18583307562581772, 0.022954279319088666, 0.7545516574057409, 1.4417849868425299, 0.13766833762214797, 0.5363803419635359, 0.8596784065854957, 0.850813555709411, 0.8730320994618146, 0.47908955227003913, 0.851845885362673, 0.7056647886098244, 0.847085986583805, 0.9564088474785132]], [[0.5646502888108009, 0.4274416888661071, 0.3935559186978032, 0.9498096420481899, 1.2981958153830204, 0.68758368569339, 0.3943997606707154, 0.9734162935201692, 0.9375669441574885, 1.5116553200896274, 0.11051240869382317, 0.7514137856778245, 1.035592783948449, 0.4476499556810197, 0.477648046078869, 0.38601898562393633, 0.5047468309343557, 0.874940311708534, 0.7037831712921025, 0.2977820586182192, 0.2508249415452795, 0.1941955446195407, 0.02617624896142917, 0.7844997359164041, 1.4187425619175082, 0.21512990548896274, 0.8156434538346312, 0.8185765702490579, 0.9195087654524479, 0.6890127495245756, 0.5143506227944322, 0.8317594462857149, 0.6844599766121262, 0.7904606077739008, 0.9255384201088932]], [[0.5114821535150793, 0.45819228452051497, 0.38604410763364727, 1.06143618134838, 1.288631136968406, 0.6849394144379004, 0.39189215350370343, 1.0407782570369486, 0.8508273884236017, 1.5845463920614913, 0.19085072427212288, 0.6723429178434309, 0.9671495729483992, 0.44318468374803527, 0.41613439231392263, 0.36197499020093465, 0.4796989360828385, 0.8668946187375265, 0.7429721510628304, 0.2862833279147327, 0.24997857812172836, 0.16305873364375445, 0.07508067331975052, 0.5830546982774301, 1.4813487257841902, 0.22361485977953935, 0.7781405075926158, 0.7588324322726263, 0.8315049667472105, 0.5918362307552814, 0.5306342755404976, 0.7653485741577986, 0.6662673368547382, 0.7127079635134379, 0.83570887811001]], [[0.4972080943741017, 0.40471807456854825, 0.40719315080371665, 1.203352230551585, 1.3002020806653067, 0.6918159108891064, 0.3869751785000726, 1.0916505211881002, 0.8317070782086848, 1.6135589828563623, 0.12366975943468775, 0.7075400713295381, 0.9602112544478286, 0.43564249582373166, 0.3687859013014898, 0.3427288101100874, 0.5100893183067079, 0.8154935583651435, 0.8186355822948952, 0.28486214357040834, 0.23602083441678023, 0.1758122325168035, 0.04359302890388915, 0.45264022097655804, 1.551892636298002, 0.372607926271309, 0.7400995127094325, 0.702576969837706, 0.7552621677769775, 0.43604181365748657, 0.4807264230689933, 0.650482771124998, 0.6416501200593974, 0.651431140084139, 0.794675377359189]], [[0.5001521511000663, 0.4041167215945612, 0.3977162255711591, 1.347249808814203, 1.2734214969899802, 0.7024567661577318, 0.3819559199069094, 1.1124816786839051, 0.8682775995098395, 1.6329110057235054, 0.12539152952921867, 0.7334856093170022, 0.9166275689702854, 0.43938097379850627, 0.3470907907222207, 0.2894749178176297, 0.46535989479999207, 0.80562560560456, 0.8107657584623116, 0.260846170198219, 0.2409615169599661, 0.13802609464246532, 0.012513784441619963, 0.4468588949077437, 1.585092305822353, 0.502000434145897, 0.7345507868007428, 0.6343824958455895, 0.6538955868920899, 0.39353879017815296, 0.49362420635180204, 0.5489512428300809, 0.6487605173051709, 0.6315879327821032, 0.8308492376636955]], [[0.44149908550184, 0.40228741816302627, 0.40729755853556476, 1.505327561839602, 1.2896096923124456, 0.705350422060975, 0.3865652799355685, 1.126535116475408, 0.8781410385085338, 1.6293413745076493, 0.2171100567864789, 0.7968795455700068, 0.9505882147203928, 0.46716782346545827, 0.34180456566887907, 0.294989143302655, 0.4663351116899251, 0.7688138572703801, 0.8234070411643298, 0.22420228553125693, 0.22321049507772586, 0.16043758191119606, 0.02350698476774469, 0.4459453351147336, 1.530744088401988, 0.5761086546214146, 0.7405259605492694, 0.6088999766717084, 0.5324521508131381, 0.38399699837689505, 0.5605647165266183, 0.47119082320300376, 0.6433151823197814, 0.5914255716095527, 0.8980090626182258]], [[0.3926431184122697, 0.407170614466273, 0.4010842286355836, 1.4676834481601888, 1.2867994760854122, 0.6779365280489636, 0.40002845915651336, 1.1241313834395772, 0.7847813893704696, 1.5794309638002302, 0.3691794630270525, 0.8362671984560976, 0.8210425427231918, 0.4632761876915703, 0.3656188755044041, 0.2834495326688997, 0.4683945561422504, 0.7618409271128257, 0.790182222459495, 0.22256101177227972, 0.2118562831029518, 0.1672602686708144, 0.03580135826477909, 0.5033524696807004, 1.468174274866569, 0.618253307227998, 0.6833649119177224, 0.6264869487762049, 0.414577305879209, 0.4968698382428802, 0.5915867269628134, 0.46603087364264106, 0.653876589502955, 0.5018354691574423, 0.9374626346773818]], [[0.3930015051669765, 0.4360791000794574, 0.39595667341720386, 1.5577195656515834, 1.281452310647273, 0.6782783878185898, 0.37791321759080365, 1.1439640146200158, 0.7135697661193409, 1.5531568325946705, 0.4372611142366764, 0.9085485710316175, 0.7690741339135337, 0.45899971558461666, 0.3698583385871149, 0.2997376765392728, 0.46408174210446695, 0.7449418201370449, 0.7929687911865717, 0.22540731409994097, 0.22476038899792192, 0.2049731680877075, 0.015635862300533494, 0.5277407322710254, 1.362377640786347, 0.624303297224855, 0.7344484865900884, 0.6460664278380328, 0.3880979129956055, 0.5815974911278577, 0.6191136264733798, 0.5298305971641589, 0.7087247714590821, 0.3569966856353872, 0.9259914793138166]], [[0.40242259831173466, 0.4482882670027095, 0.39839972176815586, 1.6015454242333793, 1.2572370173457785, 0.6947498477641267, 0.3764406898810288, 1.1076732441761175, 0.667136715787494, 1.4770299575558445, 0.4166347513480901, 0.9133940487165534, 0.7578001437957382, 0.4557472297251238, 0.47830437041003143, 0.2975096098820843, 0.4693829635985737, 0.7141718374235827, 0.8105053799958721, 0.2249213514279389, 0.22473631502797833, 0.23238344948562306, 0.0119360098959819, 0.5302028289716113, 1.206419345919816, 0.5976834247192542, 0.719852133034285, 0.6726559188613961, 0.44487003232447897, 0.6447830599184026, 0.6147619256977392, 0.6322509425791554, 0.6668710166830532, 0.29260941992195905, 0.8938721051669246]], [[0.393061351788263, 0.4344981896861324, 0.3965434064511385, 1.5125069960202007, 1.1662748007077421, 0.6948975753990803, 0.38034776938020404, 1.0234485162675209, 0.6016054471369201, 1.4539691542028201, 0.48098955996780135, 0.8888558583821865, 0.7276554895133023, 0.4612049039758841, 0.460377889997552, 0.2766239247331399, 0.5100033249225864, 0.7416360546004568, 0.8294163504583136, 0.2564382165551653, 0.23746412792662835, 0.23103225227598212, 0.015686005364824118, 0.4874536007247253, 1.0749081328133958, 0.5768191450458854, 0.5865586516696221, 0.6362877165133156, 0.45932382955373274, 0.6844641685529042, 0.6470804189440407, 0.7372666295087832, 0.5908118185093327, 0.25382416674684594, 0.8608127067390271]], [[0.39509806378123774, 0.46620646011612454, 0.3900999410304541, 1.414743327797907, 1.1081576837573628, 0.7088373523231664, 0.37908348031318045, 0.9144926821902604, 0.5401127233207403, 1.6162284279774097, 0.4767946709647189, 0.8841125860585414, 0.6705289104306813, 0.4706305754764581, 0.4768387480543002, 0.2847146576662667, 0.5212946565167109, 0.6989365730117103, 0.8554265073711153, 0.26432234099147467, 0.26564213475829346, 0.27737429336651453, 0.017665612498051396, 0.526678543983019, 0.951970229903255, 0.5555603163589757, 0.716417585125297, 0.49521475367826107, 0.5068523779371079, 0.6887887996676066, 0.7341900695335206, 0.8462727813363556, 0.4457196340277852, 0.2791337298080437, 0.7995785892179927]], [[0.2987300048008772, 0.43585146229877314, 0.39086074395046777, 1.2900729721334143, 1.0951583920472796, 0.7131217897319986, 0.38683790136432544, 0.7991433153689599, 0.5260335948660979, 1.6796162639271213, 0.3166930479029925, 0.8832325600757813, 0.5442886942102522, 0.4609115863813264, 0.43272430315598853, 0.29106053675343924, 0.5230464252927679, 0.6905793644371918, 0.8313070108988525, 0.2241149038761307, 0.25077420701391423, 0.25998513735335993, 0.020013740374298164, 0.49102648848597624, 0.883927760200252, 0.5219108797874749, 0.8417011309624612, 0.4040365430717324, 0.5603340920307558, 0.7069795136524962, 0.7118578726612474, 0.7954450352800977, 0.3950563515045259, 0.38338539460288723, 0.7406266779793762]], [[0.2925732437473536, 0.42289335557550556, 0.436396167586935, 1.1554482297083266, 1.0967095596436092, 0.7482789057785251, 0.3826030844612397, 0.6919697463905048, 0.5056685068813235, 1.6863358015831809, 0.3296854202323738, 0.8833273087285951, 0.4962201406645337, 0.45486685413940076, 0.4102971575301944, 0.3112833370076996, 0.5165350658283102, 0.7031889754268412, 0.8321240322223447, 0.2092415190707756, 0.2604127629633547, 0.24924520996487554, 0.015395973074701877, 0.3763188903339912, 0.8429056546868237, 0.5087443349030492, 0.8358709803831648, 0.394173160784254, 0.5863884347741252, 0.7395111127700029, 0.7180374080040559, 0.7490087410725498, 0.3681240817017867, 0.5784273765470229, 0.6904806213248452]], [[0.34221090460166914, 0.43272064570435964, 0.4279166227928237, 1.1073119838653156, 1.0997306248784526, 0.7431593840417491, 0.38768171091726333, 0.6357598943450493, 0.47491342002150383, 1.6517839267786483, 0.3240109027094342, 0.8624985227428077, 0.4586111639963393, 0.4690488547445613, 0.3997765383291721, 0.3085200456249396, 0.4991345177516285, 0.6995671417591189, 0.7859029374825175, 0.21855573937928552, 0.23970875962882426, 0.2421250965864842, 0.013270794087549675, 0.28854933155062834, 0.9117422688184507, 0.50582597370497, 0.3971123598033268, 0.36691431080244713, 0.5772828390282206, 0.7754116617961497, 0.7475208710332157, 0.6841647986805512, 0.34412579004552385, 0.7308160863551219, 0.7154780266652008]], [[0.378640648773546, 0.46070459749032205, 0.44354069120439255, 0.9130385510598813, 1.0771700494139766, 0.7606516145650096, 0.38293635588371977, 0.6351183811752409, 0.5388588454842518, 1.6217402499801665, 0.3184449167062733, 0.8638959717543329, 0.41337241510259914, 0.4657228933775384, 0.41267963958487036, 0.3214999604229834, 0.49867234689107137, 0.7793676312214777, 0.7882003848440219, 0.2125101964627833, 0.23487515776478718, 0.20163553599934694, 0.009908855057652766, 0.2262241382380031, 1.0399287409153262, 0.523365770066361, 0.3384544947592154, 0.3982571638798858, 0.5518395297870183, 0.615352190998397, 0.7357741197968362, 0.701947061282125, 0.3136987912031258, 0.8704922970451754, 0.7875323050851655]], [[0.38360917549398504, 0.4351426270205171, 0.4217178855422764, 0.9328647380399078, 1.040740673661247, 0.7459633795826779, 0.3875670470069708, 0.7039678858434785, 0.6094397840475634, 1.7104490098573601, 0.36545349033504837, 0.844779857306821, 0.3802374211624992, 0.48121482574101243, 0.43169839548415107, 0.30971006478025176, 0.49337364252278104, 0.7680470815374768, 0.7864531185640555, 0.21183970352900955, 0.23687594727225098, 0.19184618901227302, 0.016229909145970355, 0.25477290112567885, 1.182882875598953, 0.5269835024962679, 0.3064954565368996, 0.4288215030467652, 0.5033681492800476, 0.5277885133694258, 0.7436330204292185, 0.7994117540023167, 0.27779726780746195, 0.9468104848966206, 0.8132619254748402]], [[0.4447594106270306, 0.41626927282075316, 0.46016704036203115, 0.9902280922854549, 0.8760405297209574, 0.7764951377548874, 0.3895688543326492, 0.7580392345394551, 0.6954799370350927, 1.4184192396620228, 0.3605981067782982, 0.7651759565673908, 0.36119471917131096, 0.4191404228175495, 0.4055405133097721, 0.28412351897741517, 0.547835652387357, 0.7736498782464856, 0.7805400477415392, 0.22416670078133605, 0.2325068501913385, 0.19905062164016013, 0.01565260856687076, 0.28861175063628086, 0.9811196911643607, 0.5275556249937522, 0.2938287579663724, 0.4098673786999807, 0.4683650335516593, 0.5293545829611443, 0.7217126646833607, 0.8940050652620362, 0.2906701970653891, 0.9838124119083136, 0.8238542603187908]], [[0.46944441552096833, 0.42102421490307, 0.44141155758935313, 1.0378398817653949, 0.7530233942421912, 0.7266531945900836, 0.38610861679154884, 0.7868793669489872, 0.6336926043533427, 0.8739566362675445, 0.3561291477084683, 0.8292374746860418, 0.3426905029498076, 0.43365679732404994, 0.3953616704076604, 0.2930663626845974, 0.5396887265229755, 0.7169864651802159, 0.7586111855726334, 0.23617947176051096, 0.2513936517620675, 0.1891408711617062, 0.020587510661577706, 0.34187721945602617, 0.5812972832043195, 0.5377705305413495, 0.27043344738392255, 0.40999737598825364, 0.4643739319722428, 0.5853391348829557, 0.7170001289002953, 0.9381810608160575, 0.3253059855502868, 0.9714781546280152, 0.7805798446435976]], [[0.5150371104248186, 0.43829827397498655, 0.4471635266911891, 1.099931789387021, 0.5601898724456889, 0.7142022891784863, 0.36749465731126396, 0.865018291242672, 0.6647059053475829, 0.48854032910165474, 0.3461562004558958, 0.8443425514590395, 0.33102333293811564, 0.4353062724672564, 0.3648930990103967, 0.3182474161687465, 0.5514590834393194, 0.7459689320080751, 0.7515728828511674, 0.29359336090349025, 0.24875523888420223, 0.19197137997850405, 0.012155042449713938, 0.3517604774863351, 0.42448147381444173, 0.5577926992117914, 0.24809307044462864, 0.4675612928922319, 0.4506350365223259, 0.6008614568149875, 0.7170968798651205, 0.9904416546036016, 0.3476863111566112, 0.8898196278509795, 0.7490150169093891]], [[0.5520122492359496, 0.43198881900698344, 0.40925642925630534, 1.135854742624057, 0.4001830467316124, 0.6963285372444138, 0.3787516854344032, 0.8677484453117663, 0.7005303587019576, 0.23166704587485332, 0.3561494493926326, 0.9742305476771647, 0.31219416979750736, 0.43773522544143095, 0.42726600943980675, 0.29332842192215935, 0.5419096540461507, 0.718570408408783, 0.7552045078545454, 0.3138685384723682, 0.24368385003086157, 0.2167871861193118, 0.020186469508882192, 0.37912322777872465, 0.318566874054076, 0.5121145268360074, 0.23922671763560927, 0.546974283943884, 0.4204117948978234, 0.5997777272619549, 0.6701978119191181, 0.9929784136432047, 0.3848745125610585, 0.7979885399098595, 0.7504389453006466]], [[0.5934483472888166, 0.4109989309835276, 0.4085603212732293, 1.2079275617902034, 0.3850840337720431, 0.6704224572832969, 0.35987997962914886, 0.9032142970360924, 0.66975638249423, 0.18559166268416283, 0.3304061503239731, 1.0494526621283398, 0.2643154521117603, 0.4574850419503726, 0.4297827934950584, 0.28257505182592735, 0.54960300749134, 0.7309092847326917, 0.744726330200331, 0.35411182086704496, 0.2573073724809021, 0.22540066943770876, 0.020825609951929735, 0.39024208403052396, 0.2769995263896281, 0.40761494128102815, 0.2293210862176524, 0.5460006606209997, 0.39883246328159455, 0.5756921172441041, 0.6025842608842231, 1.0321507066769617, 0.40706742776060906, 0.7462161922661517, 0.743851857987626]], [[0.6677271596655866, 0.38286241278707467, 0.41211079036874926, 1.1636814099667734, 0.3009937990511433, 0.6955082770811387, 0.36894616276239917, 0.9037559242444386, 0.6160774005605093, 0.24361292171324403, 0.3175254380970134, 0.8854733057791576, 0.23474063499168712, 0.4721976889275168, 0.38768495312362145, 0.28611867609677044, 0.5537111961424244, 0.6934253256863879, 0.7181394955107079, 0.4034308274308933, 0.22059466884428894, 0.23413158167020176, 0.021932125989468802, 0.428667110028702, 0.30727644750019834, 0.31397244567491966, 0.2547415534590956, 0.5469596860029017, 0.3434629743502101, 0.615732622921422, 0.5142066364296918, 0.9636019015525031, 0.48627136448588976, 0.7195225848352109, 0.7766269792779831]], [[0.7166016492333884, 0.36737500270772905, 0.39557189443768004, 1.2565112495470174, 0.306995826464285, 0.6749003881107252, 0.3564033178284336, 0.9476117161027657, 0.6348286822623861, 0.3296067391566203, 0.29424515774282034, 0.7981076977912487, 0.2994286963810528, 0.4749381472642864, 0.40046149794292685, 0.29200359255774694, 0.5896085280468737, 0.6730870943369549, 0.776955630816999, 0.46232332198187365, 0.2361700904285461, 0.266707221617998, 0.023233926464853446, 0.4785577699907117, 0.35312012706688, 0.28858077761463063, 0.28729080567050863, 0.5812495571990481, 0.3419868611587656, 0.6407643285985327, 0.484206273625835, 0.9803161604848756, 0.5204810045037102, 0.737185486711801, 0.811263465726036]], [[0.7309393037889275, 0.3779118400824059, 0.4132513119831627, 1.3688001169146657, 0.3117459875625542, 0.6160785797914464, 0.34093433532965056, 0.994550044403111, 0.6532634839396677, 0.36902135764379945, 0.2976305341583341, 0.6931100471674421, 0.3746316744050766, 0.48490395956429555, 0.3925374487110397, 0.31995057730006216, 0.5565322447939289, 0.6873885623466944, 0.7947049295350435, 0.4425069214792629, 0.23987408767068855, 0.30006163617747683, 0.02887183280470466, 0.4939495713172955, 0.41005324018161204, 0.24009878262190337, 0.3107753669269827, 0.5727756318858426, 0.33899546738496783, 0.6412854871094245, 0.4690896255796596, 0.9775417232894805, 0.5446985733014501, 0.6941607763039435, 0.8380192531221293]], [[0.7335742967490873, 0.3789140039394248, 0.41092401431533687, 1.5065340401953482, 0.31844603337627314, 0.605583270149154, 0.3525750846424102, 1.0551947866109015, 0.6782039745571111, 0.41425589767881554, 0.2955420009930855, 0.6758739116180819, 0.4169367275364339, 0.48619831811109915, 0.39415035818055877, 0.29856195303234756, 0.5457050017891436, 0.6880652034928512, 0.8265138679241827, 0.4088164741346572, 0.22019447443766801, 0.32728572464902145, 0.03180904372482649, 0.5202161058763872, 0.41360465436143967, 0.2239851902482557, 0.33551588975979313, 0.5406152122728642, 0.32302518590650187, 0.639410611744821, 0.496252007307031, 0.9609515600991598, 0.5914714923558939, 0.6473998449855329, 0.853919952101521]], [[0.6833730033959111, 0.4017267150808015, 0.440260920238542, 1.6194256472005695, 0.32266035684038263, 0.5862943922724844, 0.3780678419615293, 1.1032099432869764, 0.687271228569031, 0.4643226283053633, 0.2978618253187168, 0.5844015260006199, 0.4507173858833074, 0.5376585796411644, 0.4095602514499681, 0.3032227798287279, 0.525608753652739, 0.726507806969845, 0.8270324904366803, 0.3574407233801472, 0.22985039773213228, 0.4059664198396954, 0.03666847206432833, 0.4807318631421974, 0.3568606898257853, 0.18149732788364176, 0.3663939381744723, 0.5359213992370533, 0.3664298677223041, 0.6304072903014557, 0.5551702208546906, 0.995670033590536, 0.6294253521545327, 0.615117350227091, 0.8528315062014594]], [[0.6942984801952573, 0.41855727300450074, 0.4433833296812587, 1.660295035143315, 0.32158329404054264, 0.4324116271560382, 0.22181119607613448, 1.1447604173335035, 0.6946582311621117, 0.5159955426333043, 0.277497710921798, 0.5662206587115677, 0.47744425641125265, 0.5364341415673088, 0.49240999821090464, 0.3038139012944603, 0.6340705206545902, 0.683525270564976, 0.9135602976086488, 0.36489483284702334, 0.21604824949005338, 0.6089837209914182, 0.04353057667195015, 0.4781009019760127, 0.396948691373735, 0.14659811699955144, 0.36181361105549525, 0.5734980837703946, 0.4099629608758899, 0.6305395505959024, 0.5989683972887554, 0.9904188051563292, 0.628502415987213, 0.5348529952814428, 0.8238601318114726]], [[0.7206589944285913, 0.4407962562537698, 0.44010894418127655, 1.756104182703365, 0.3181560311200398, 0.41902716736005613, 0.23385523328293492, 1.1316110641903718, 0.6780774124819142, 0.5353560931046937, 0.34470835658513865, 0.597966852737609, 0.42600557205080836, 0.5410964975558814, 0.5258447090840395, 0.4486714270802431, 0.5135143644493647, 0.6498822747103358, 0.732960620299228, 0.36755800797085686, 0.19897889228727003, 0.4497656514753523, 0.060146643625064465, 0.4873692772828869, 0.4425325973286752, 0.18043090594524905, 0.4028686032977272, 0.5456554804099052, 0.5131258369380429, 0.6257760377706003, 0.735311083580616, 0.9427447787060981, 0.6564661323736308, 0.5155095955186798, 0.8144214247466972]], [[0.4597553713685975, 0.4460008803839438, 0.40931010695021697, 1.7680123460991561, 0.3137902270443706, 0.5394632056185962, 0.3435398236038073, 1.12187541851229, 0.6886635208193098, 0.6637938873619217, 0.34472038029857255, 0.6156727753825091, 0.47905967173266156, 0.5484648039525437, 0.5366255365380654, 0.4740001627225322, 0.5030831210513249, 0.6770585198288392, 0.8214941979289934, 0.31316401318930065, 0.17989965923557658, 0.40110137169407434, 0.06968905513868733, 0.5197275428769141, 0.39146735796962073, 0.24444286472827975, 0.3939393732136616, 0.5045862236397962, 0.6101022032242236, 0.537727795026936, 0.8796243390931124, 0.9282794645401804, 0.6315786278063867, 0.49554344985483323, 0.7467119525354151]], [[0.4570411971032861, 0.446763698434234, 0.4643550103547269, 1.792752813517513, 0.31256294265218854, 0.5359611331443374, 0.35242238812958904, 1.098116803357607, 0.6851190031532691, 0.6685481720594264, 0.3652705670871421, 0.5907138008362931, 0.6566649363195014, 0.5385833984718313, 0.45592574768194144, 0.4477886071982343, 0.5526142513917661, 0.6889065107393317, 0.7825987350935952, 0.251881415836772, 0.16109215664623575, 0.38789543106999563, 0.05973231960027179, 0.4827782965616336, 0.35800382824507404, 0.3098247104327965, 0.4039742254098188, 0.3360160381548259, 0.685858008781275, 0.5200914449121077, 0.8878660759796861, 0.8735354547806207, 0.6409711029132934, 0.5033793379073659, 0.7145115766599842]], [[0.4328631529741231, 0.44199564533248364, 0.46501467334289764, 1.8940639225707154, 0.31837336661008414, 0.5041301156788696, 0.32658694147207895, 1.0563768538741416, 0.6353988832684997, 0.68035421550647, 0.3622420968008518, 0.6196781533513352, 0.8328583032374439, 0.5178087083718166, 0.4879130198195781, 0.41178927327883985, 0.5483124955577636, 0.683523120520676, 0.7777612672152786, 0.26273727392469376, 0.16932502801564542, 0.31561329933383725, 0.08603615739631064, 0.4699657772424355, 0.3270246482716576, 0.39804735788845935, 0.4117545934421647, 0.26626611859592786, 0.7534389701377221, 0.5024709851928137, 0.8339419717530109, 0.7733642631326816, 0.6652919484855198, 0.5136060273995953, 0.7139051268573584]], [[0.4209886315705732, 0.46372742275045264, 0.45307678631265874, 1.9083983553640038, 0.41154765415268185, 0.5736694418527868, 0.42484237728276597, 0.9463058840338001, 0.6201971311305415, 0.7298790429134336, 0.2984987432155169, 0.6491760998776093, 1.0063918146241222, 0.5487180501867117, 0.35669324795433405, 0.40775507930694455, 0.5131924013633935, 0.6676679192190278, 0.6553503758849858, 0.2714803952590836, 0.17021214367793905, 0.29303823364711806, 0.045238518986782036, 0.4406210686377979, 0.3297858234557376, 0.4738467235916316, 0.39748143666391544, 0.24001935278928344, 0.8131658884491691, 0.48528989089137436, 0.7508402771239022, 0.7029524382060865, 0.6568352972216904, 0.5429533166342209, 0.7422664759392095]], [[0.40579270487723096, 0.4855618618659937, 0.44920344170581095, 1.908177292394469, 0.7055656536669621, 0.5913682005048981, 0.40871131169528174, 0.8746804155966756, 0.6136096828682098, 0.7360727984805336, 0.30427879032170857, 0.6416162809709574, 1.0654285556263066, 0.5460067961688726, 0.36682254723559343, 0.3878581066764791, 0.5326298717465352, 0.7204854970853234, 0.6553644464806495, 0.2892526004326371, 0.16377471451016284, 0.29090119639266687, 0.07108224343736913, 0.4536459916811437, 0.33657732148309516, 0.49686426245773757, 0.3752038081824494, 0.26515468076772164, 0.7967023256989685, 0.5016998881303466, 0.6334051196228689, 0.5975816381120708, 0.6625977727491822, 0.5614874561352674, 0.7983928783687688]], [[0.41446614992256675, 0.45087973949487525, 0.38959975118574164, 1.851848426993588, 0.762589409343745, 0.6262409377997843, 0.3856057493003184, 0.8520194332525212, 0.6138390976098113, 0.8257920685872426, 0.31328092752338355, 0.654232288858383, 1.1390798547958412, 0.5647183673342195, 0.37590844614118984, 0.37442646052295303, 0.5014555246979178, 0.6911793924058205, 0.6474825811213679, 0.3084364073376904, 0.16901207465922996, 0.27345206248182474, 0.056483272327590706, 0.5119854595191511, 0.34020341695848383, 0.4929631607559732, 0.3727873808228834, 0.38130388851728675, 0.7897878009216627, 0.5403546374119312, 0.5575707011108707, 0.5217507364642853, 0.8577527201062831, 0.5482799600901572, 0.9124026004079511]], [[0.43026660950656803, 0.43708798801757787, 0.3943515360251382, 1.9861892690279384, 0.8709243210231323, 0.6383297438976617, 0.3785294209885891, 0.8788676182249758, 0.722586326673392, 1.0344030231669679, 0.39197983455879915, 0.644522129416573, 1.1152971372813232, 0.5579898308732234, 0.40400714502620705, 0.38906110157814033, 0.4966676431301327, 0.6606896608720378, 0.6292992512005338, 0.2860593039857182, 0.17620216619171772, 0.3169261494530759, 0.08352139329411848, 0.5117186986288604, 0.296881881168704, 0.4742235173986364, 0.39940393540190583, 0.4380933867676009, 0.7789305811535261, 0.5268823824320875, 0.5960450615387802, 0.4673401661671452, 0.9204665412079494, 0.4988571881518641, 0.9328107283480269]], [[0.42369306593859923, 0.4670639225123391, 0.4579261746915764, 1.830148930813605, 0.9231886008954363, 0.6283073259470396, 0.3745630909141585, 0.8800066830280381, 0.5843707717368771, 1.1024777950974007, 0.3138689662752781, 0.6681648093245144, 1.069597826714918, 0.5819768241601505, 0.37277048436235266, 0.36402577270974956, 0.47421277218714786, 0.6553134051269651, 0.6569560013112763, 0.2819429153621177, 0.16878089744603791, 0.26536281232299597, 0.08435488430608165, 0.49107914922815704, 0.28320086435653025, 0.45911888758037206, 0.42768583572884394, 0.4919886670192649, 0.7748812974436514, 0.5188569862289384, 0.689902662374891, 0.3820059954757464, 0.9582616748244512, 0.48153300831176693, 0.887081062236057]], [[0.5185125801641455, 0.4805357441154526, 0.47319405702985673, 1.8037012455988588, 0.9384982551186992, 0.6338568282961801, 0.3806548366768191, 0.8737088682744477, 0.5772707331103731, 1.0014423613739059, 0.3211545969231864, 0.694310576004496, 1.0335700721003476, 0.603708303842517, 0.39267993375623866, 0.3537163931123505, 0.42501562175201235, 0.6401236912051766, 0.6849313997975712, 0.3146668513197275, 0.1643837606095151, 0.26738433611879325, 0.09583807534957098, 0.4823949500066209, 0.25337256310478373, 0.441671885040127, 0.45366060916562506, 0.4261806972890512, 0.8024092689629351, 0.45773130232555653, 0.7443701524673008, 0.37243937609790584, 0.7404760826587282, 0.5095785801930623, 0.9329679447526357]], [[0.622135564646652, 0.5076807385439699, 0.4700047209815513, 1.6159003171518007, 0.8680994275114069, 0.6186314358427751, 0.3861990467197246, 0.8574328345427014, 0.5592137050100223, 0.8338574727155122, 0.1788802296859596, 0.6911479740380764, 1.0689233724764329, 0.6306563567058281, 0.397517020591143, 0.3674379148290737, 0.43394749522173703, 0.607265546208784, 0.7472072695304965, 0.2075611313158039, 0.1646255523105802, 0.20912366345932878, 0.05146845405726073, 0.36099198012371686, 0.2288565735833574, 0.4271839474104181, 0.40873308319965107, 0.38858991463100145, 0.8435091111388769, 0.3905187218996716, 0.7785236079209645, 0.4660537865612959, 0.7714568811896101, 0.6123714283213344, 0.8963525635859435]], [[0.5196491967821797, 0.5614379485045496, 0.5330302037502015, 1.3702965056214014, 0.7887345640347068, 0.6312004036843076, 0.39091127082151383, 0.755446675548997, 0.49330020265155966, 0.7913550327280297, 0.12463389890808138, 0.691608117793819, 1.023887107102668, 0.6422286084934583, 0.3773449592337412, 0.36661794122621605, 0.39791046783742234, 0.6322617175070719, 0.72110706767468, 0.22668329181339103, 0.16106602691434463, 0.20148362258438451, 0.05686252033377878, 0.28919359492908514, 0.2588346931776272, 0.409065521044846, 0.4381543182577178, 0.3589115640121215, 0.8746914329440505, 0.3651357275590894, 0.7493444249089272, 0.5074494624542097, 0.8839590029015976, 0.6395734008000842, 0.8380628882658395]], [[0.5043231604610459, 0.5872019201610267, 0.5432403095238836, 1.3331883597245673, 0.7499550802338886, 0.647930334932725, 0.3798929826765224, 0.8157177550377357, 0.4478283923724937, 0.7941356547600877, 0.08462006050434415, 0.7055582697947931, 0.9463607822836928, 0.6345082566311253, 0.36339520664486674, 0.3707613562822907, 0.3897501972465022, 0.6475808811059094, 0.7161197451226292, 0.2057020770584821, 0.16565233001680618, 0.1285355673260446, 0.061399442926708, 0.31533222484975, 0.2623188681016242, 0.405102243346378, 0.4907263681992678, 0.31788889382225044, 0.9580558531042962, 0.32750959524435663, 0.742737273211826, 0.6066155062981656, 0.9944239820646215, 0.6259222439384056, 0.7988509655371933]], [[0.5085433304567448, 0.5770253791090896, 0.5140859552852086, 1.2169730098203124, 0.6728867633575698, 0.6119111739739564, 0.3358934305219602, 0.701969297557134, 0.42091082250328776, 0.79665365073828, 0.08634181156088092, 0.7811352812770231, 0.8832245656462676, 0.5976042163519141, 0.35115090193947734, 0.39109914594584116, 0.38964441417781115, 0.6207880058976085, 0.6872770844800209, 0.14593663408022983, 0.2120009113538318, 0.15382893556542876, 0.07210136083703583, 0.37253135306430285, 0.26467025015939727, 0.431195637656449, 0.6576287630963673, 0.33948688715559683, 1.0982127548340022, 0.3019845049205308, 0.7575695169260124, 0.710726467919375, 1.117260483407213, 0.6152870373189421, 0.8226791959765597]], [[0.6291292798088264, 0.5284798922725846, 0.4591571285721714, 1.072483344151929, 0.6664963210841021, 0.6314551754290612, 0.31269221695918037, 0.7025208162098991, 0.37985397533054766, 0.7924834587983116, 0.08438995589950626, 0.7678912186635376, 0.8206935311018678, 0.6073431166159926, 0.33907834245589524, 0.3850006794895231, 0.3916367038492606, 0.6278158216464584, 0.6760485729640715, 0.1475574128497446, 0.17197811677743197, 0.1526287721852152, 0.09621060092478824, 0.4955057416249796, 0.24844945413392058, 0.4466131185804542, 0.9355014301599688, 0.3357380429208808, 1.14147037116656, 0.3237212790696694, 0.7849982079774811, 0.8014706798512988, 1.1723382733510108, 0.6919262525516647, 0.7723105970867064]], [[0.7074000974154522, 0.49668532070097016, 0.4199406811606865, 0.8805049939794101, 0.700027211987226, 0.6780106248838463, 0.34215183794341203, 0.7461548891426346, 0.4892018076663419, 0.738662310560336, 0.09523327009026102, 0.7227576234732733, 0.7553557298356751, 0.5789363502817927, 0.3857275231607236, 0.39267660774845226, 0.45510489521993264, 0.6413217773813131, 0.6898171105215372, 0.1145547727918917, 0.1778620924229713, 0.16292506097653292, 0.10198873655726338, 0.7351938888017024, 0.24051204785169317, 0.4527025457924911, 1.2706994705096952, 0.29879158776428494, 1.156665839971615, 0.5165002161171457, 0.7611338743711111, 0.824750812421785, 0.82015399393356, 0.887136402445112, 0.6969540131035652]], [[0.6888878157461726, 0.5425556232585926, 0.49490436778325764, 0.6992456313609428, 0.6917094692065104, 0.6367638883418012, 0.2759774015813048, 0.727192303051339, 0.7333426963102038, 0.6834935646259043, 0.10231955120320214, 0.7016455568103332, 0.7287490528159983, 0.5448922196797299, 0.38190818235075896, 0.3866042903400627, 0.4775551703726093, 0.6408646638510248, 0.683046938814217, 0.11213186448628733, 0.164511204754364, 0.14478802944952296, 0.11128722068259421, 0.8623851532052191, 0.2242024200767118, 0.4474975136155322, 1.3397086267894254, 0.29421999913410335, 1.1454436076844932, 0.6653493403428258, 0.735786931513871, 0.8388543034496997, 0.7354099151211325, 0.9762137560136215, 0.5646712050099882]], [[0.7065998416958069, 0.4711117596609957, 0.4122985960353357, 0.6248443576048015, 0.7307442616549752, 0.6390601443145687, 0.27271840634833144, 0.7583553544885782, 0.7481136322309049, 0.7080306108208166, 0.11915137944098088, 0.6961967033557779, 0.7953446222858309, 0.5497784965464133, 0.36397126082739845, 0.3810958477338441, 0.47634938234423185, 0.7094397016307129, 0.7019445091122469, 0.1035050108262961, 0.1898671170205635, 0.13638879012011898, 0.10376853161909987, 0.879989589376487, 0.22872877541906708, 0.4339210484721895, 1.2874144158183694, 0.27383264747661396, 1.126959596234933, 0.8208203635322333, 0.6874103490960106, 0.8253877689950799, 0.7151669280899258, 0.9962578709270675, 0.4826613237986408]], [[0.8469187001047931, 0.46556955172538145, 0.4028901825516621, 0.6235961257790363, 0.7454375904595829, 0.6654997822772764, 0.3080600610444183, 0.7922747761269859, 0.7859984405121199, 0.8319902759605269, 0.14979591596864703, 0.6989420685929224, 0.7963429445557567, 0.605325258276501, 0.3671647712430064, 0.37656221407522217, 0.4751922519095461, 0.7247176669836672, 0.6577481865683517, 0.11870120789307809, 0.19901988769552487, 0.136814489140621, 0.10193023010214682, 0.8858813168546861, 0.23145577290465136, 0.43150816561736594, 1.2189772203753386, 0.26567407691117645, 1.0454136556697458, 0.8662950784686749, 0.631581530182104, 0.7964231480703928, 0.711288493983825, 0.9055998881023221, 0.4408409624729749]], [[0.8899369445317221, 0.4954679965855018, 0.4585475305552428, 0.59506477995814, 0.7256641103566585, 0.605982260051242, 0.3036931722271258, 0.8411799017428814, 0.8042054661672866, 0.8259582777098162, 0.1602166448075904, 0.7630469868421208, 0.7079315145036125, 0.5933969827477832, 0.3670192981759379, 0.3714732417656211, 0.4637749792323377, 0.74408718947838, 0.6731953133091476, 0.11255348980160401, 0.1911746211794625, 0.1304502840812368, 0.10485312491998967, 0.9120880504270397, 0.25666950612808925, 0.4189463199960171, 1.0847965456433208, 0.29643022256225726, 0.914170908268022, 0.8712167274615692, 0.6209853956927625, 0.8059634838512839, 0.7369684990421528, 0.7627456635881127, 0.43214859060570066]], [[0.9629983746288422, 0.49756765725310337, 0.46089297791956885, 0.5961567536508274, 0.6751095142640919, 0.6112925860185436, 0.3036028303799553, 0.767186557402467, 0.8017949945729507, 0.6769335365396749, 0.16203027729701036, 0.8086799750130738, 0.4580589896126735, 0.5875538294266966, 0.4193686591929009, 0.3674341482938293, 0.4725840406180758, 0.7229444113979933, 0.700125107441618, 0.15041503004979923, 0.20571979927329842, 0.192414517950351, 0.07692156891807472, 0.8880412688932114, 0.28105546518358304, 0.4161742197490441, 1.1549565329436275, 0.3249864687854153, 0.8389032148977975, 0.8461664974639846, 0.5685910771234511, 0.8705439031743976, 0.7403871622374386, 0.6865847330382269, 0.42711328102339063]], [[0.8924388843999046, 0.5506050146460344, 0.4692271537795242, 0.5512782762061851, 0.6945374268481094, 0.6832058939121213, 0.3494983717903555, 0.5554013883252128, 0.7907290201941654, 0.7646362042321452, 0.15436012179601694, 0.8437402124997277, 0.3939995956451008, 0.5810060363147318, 0.476600581807898, 0.34733132719078197, 0.45079265715204264, 0.6784940810868297, 0.7938075875992876, 0.15723873639377117, 0.2329285781281067, 0.25605870734034475, 0.07303077368347584, 0.9061696451794767, 0.3173666919113415, 0.4010862053580677, 1.105617734767564, 0.34669185861947815, 0.8741445672611121, 0.7843028001159051, 0.5815023184649684, 0.8603543757058418, 0.7196976238573678, 0.6245091546036772, 0.41976168203436437]], [[0.9235095117827485, 0.5635135896566517, 0.4994586628612738, 0.6910654073737694, 0.7049665167759731, 0.6674409788213318, 0.37253340014866926, 0.5652813237969454, 0.7490852167305381, 0.5878354686539271, 0.14681594989082788, 0.7865395119228593, 0.3746661489890303, 0.5843465698341606, 0.4007638358066397, 0.35491110322664493, 0.5698960940169369, 0.7850127842996536, 0.8304597280900325, 0.17998468973259646, 0.23019826541356542, 0.27803349901971885, 0.07085881981550034, 0.89297401081778, 0.34125614615029665, 0.3911119236207792, 1.0156042934779443, 0.37006730661819215, 0.9577468861398901, 0.6471617855375328, 0.5231614760652303, 1.0025128242510268, 0.8201713344662506, 0.39571135556581033, 0.4558403695353193]], [[0.9572136094751433, 0.6398983379893886, 0.538418440914046, 0.8551001176029462, 0.72444088171419, 0.6631712125377148, 0.3979814885300663, 0.5597189966095257, 0.793388211845088, 0.5305466258964, 0.16075152089116185, 0.754622190347165, 0.4202738722383865, 0.6215660195202835, 0.40681738149254554, 0.37911652297611564, 0.5863257316846632, 0.7935547558560657, 0.885350470129165, 0.18064500952485374, 0.23008195916359422, 0.3292393425102278, 0.07038889271299628, 0.938260521501225, 0.3120736645357513, 0.361946577920625, 1.1365623038659303, 0.34143527684736674, 1.1300570149474203, 0.5860347725374244, 0.5493382922162842, 0.9990053813499166, 0.8792840353789131, 0.47935418265171287, 0.5610039514952243]], [[0.9855974247044795, 0.5910386745353016, 0.5315614485394249, 0.9613715193842196, 0.7335687503930209, 0.6924338335424511, 0.3972737780776082, 0.5947496502790697, 0.8008092920675108, 0.6375025656612325, 0.16856819625020997, 0.7519998550325578, 0.41444877386231915, 0.559002050450917, 0.43568960069564433, 0.3518983299749146, 0.6034927253992808, 0.7684280815486937, 0.8273839156661262, 0.19114487534867397, 0.2287715293099713, 0.29619338859020755, 0.0754037832205884, 0.8454815441773379, 0.24399504495245106, 0.32834538541381714, 1.0715131522464347, 0.28248726072628016, 0.8911468252359286, 0.5739149765044287, 0.5232274772733954, 1.0342804769536857, 0.8279263733926125, 0.5967914521795897, 0.5373375674871101]], [[1.0494431563439712, 0.5511273931014173, 0.5450822131165791, 0.9489210634726974, 0.7378412568491789, 0.6832261082774344, 0.3955418286936707, 0.711715355311132, 0.8284007931746913, 0.7815359914811526, 0.18199025827050294, 0.8314180124493785, 0.5872983497412361, 0.5714698479501625, 0.4584247745945206, 0.38035336868040204, 0.5803039325153991, 0.7609598520826842, 0.8117047558886743, 0.2144031849123238, 0.22359731350582762, 0.30719241208003745, 0.08232856764623592, 0.6828049898086661, 0.1646126298305865, 0.2895401281580542, 0.9027492622655059, 0.26750322387306796, 0.4955039510133975, 0.6008772365058548, 0.5091497864058807, 1.1363580615501634, 0.7186906593583436, 0.6672493385821425, 0.504042986432345]], [[1.1091119335600572, 0.5321546236479935, 0.555810248335908, 0.9461461836874232, 0.7617344284865823, 0.6673298749347077, 0.3763354951860979, 0.7838204819550689, 0.815769504128762, 0.8565117181327645, 0.1861998465673589, 0.8034260898278159, 0.5360174897518362, 0.5694452616887986, 0.44600118921942455, 0.3699506256752616, 0.5823333139695543, 0.8394121118499789, 0.8058657493574986, 0.20026642911350018, 0.2288381082082937, 0.26027197027835175, 0.08355740074246577, 0.5336761080039133, 0.1759689473217014, 0.251413455434296, 0.8178562005690972, 0.2868483298489799, 0.28497905929568945, 0.5485469342163585, 0.49672561173514096, 1.1885188402753015, 0.5030248962599873, 0.7922593769695061, 0.44302746824313477]], [[1.0753934844931192, 0.5477422402595351, 0.48833459258645406, 0.9512831408618586, 0.7789089152692281, 0.6434249485967432, 0.41539060077074585, 0.817996130389162, 0.7998005234256962, 0.9804808024539162, 0.2094649855169607, 0.7967622611175227, 0.6045797450298955, 0.5838192564433976, 0.4337770465316235, 0.3813460264108013, 0.5662215370539243, 0.8865429141653565, 0.8578575370157309, 0.2321857178003101, 0.245655987617672, 0.26864912931663665, 0.08172042706013295, 0.3475427546780798, 0.37265471772589004, 0.2099333633032449, 0.7004578379132508, 0.35690320924939123, 0.13995545175945234, 0.45161945247785273, 0.4789320129673475, 1.1685831418507795, 0.30895889673496046, 0.9881093097793434, 0.3622012828356521]], [[1.0687697321197247, 0.5244404789163961, 0.4422364745571551, 0.9250488118867193, 0.7890594714049526, 0.6747447938790712, 0.3975769027562736, 0.9295363962117887, 0.9644304552744175, 1.0950783536592454, 0.2272445737569923, 0.7885342916358133, 0.6503646099106088, 0.5809672375767656, 0.4457872635714949, 0.3803407291269315, 0.5593867955744121, 0.9006266962878613, 0.8391981809810535, 0.2432523582824076, 0.23219501522605546, 0.2734170946884209, 0.07761365072765747, 0.3106148605303068, 0.6213937535444142, 0.22517931089041182, 0.7415867858146737, 0.42948477772648985, 0.11554282011685113, 0.40062961709722156, 0.47579566549509167, 1.1242497103068192, 0.3034013515771711, 1.0517045579585578, 0.30146957285157183]], [[1.064020022011618, 0.5118331323321084, 0.4233182244989001, 0.8874645536212624, 0.8567426842918398, 0.6929365933643206, 0.3862247794967868, 1.0221996755144815, 1.153444791849897, 1.2949268754409649, 0.20283214736674565, 0.76835769520805, 0.6589336107495778, 0.5835347448165964, 0.46528073321897956, 0.38421711065012365, 0.5486014153448466, 0.887286218624056, 0.8246122028289589, 0.28949401199952784, 0.24150549055132386, 0.2816872894461312, 0.07638173008730088, 0.2951727896954184, 0.8587724778871251, 0.25179568055449136, 0.6719009228899511, 0.5469398938030723, 0.10063605330853431, 0.29972613466090825, 0.4657566623719659, 1.1375908792116052, 0.47630450626944604, 1.0879742299956496, 0.3966422796981397]], [[1.0377783118150872, 0.620804882080104, 0.5379945500512929, 0.8407068342143829, 0.8123775434055014, 0.7241396736840371, 0.39065200364294694, 1.0141546361597387, 1.1424517173843243, 1.2698094797157733, 0.21171479630589052, 0.7826664170293198, 0.6285789990307321, 0.5552150570229125, 0.4722857777478861, 0.3507692377029676, 0.517870838452045, 0.8830799067712782, 0.8846853358125797, 0.32459105698729, 0.271791523449019, 0.2995937441608274, 0.0736847477092539, 0.30535009828339765, 1.0225727226604748, 0.29266052979573876, 0.6225411888299958, 0.5977093084271882, 0.12177761372151052, 0.2692317434648747, 0.5279830290135361, 1.0169582603199312, 0.5180809840714945, 1.1197888283692545, 0.39148312516742545]], [[1.2517700767539996, 0.6324810768171245, 0.5221146437461434, 0.8162485961695258, 0.7606825400553355, 0.709292302415079, 0.40733894175495833, 0.9572391395237827, 1.0372320982360133, 1.2040095858284687, 0.23946901332674442, 0.7735588443642687, 0.5822648302599922, 0.5620849476920872, 0.48724664434134374, 0.3449427465192446, 0.5223321336940543, 0.8973760800737032, 0.9421010698861166, 0.34107544949785285, 0.23945873870204726, 0.28458008024844755, 0.07162164088756605, 0.35420788788034807, 1.1705157155249282, 0.2946603084285095, 0.5526298256235138, 0.6103931751522874, 0.1556237030882396, 0.2710233724280495, 0.7068987320851058, 0.8951151762994591, 0.5700516320199971, 1.2014465789471893, 0.4653429585928212]], [[1.1623592598552182, 0.6460125147672802, 0.5432927269931711, 0.7580353244277911, 0.7314872174022007, 0.6959993405618701, 0.40792596936989445, 0.8922228200188331, 0.8664764184264124, 1.2741842196497384, 0.2460391723127846, 0.7851104984523188, 0.5875540903323356, 0.559703384294354, 0.45346903957902407, 0.3438274228951515, 0.5177863179559242, 0.9047438998928797, 0.9521407200626915, 0.3532956001183603, 0.22572125335688958, 0.2762494970172188, 0.0748170080276245, 0.38936765009442365, 1.253948905895488, 0.2729425022071984, 0.584724660290164, 0.6155355365916494, 0.16014693061480806, 0.2880273864204746, 0.827895670873968, 0.742608080977174, 0.5674713096061219, 1.2127468240477723, 0.5483542453987518]], [[1.1031639818125107, 0.6652372637519018, 0.5430178370685259, 0.7201751570541857, 0.9878575810305916, 0.7035677792352077, 0.4086707229402793, 0.8826446343497445, 0.787629206547915, 1.500342335694639, 0.24912900768379137, 0.7876973856110638, 0.6340799867616413, 0.5482426221180692, 0.42399900970166154, 0.3467820853932325, 0.5322972985279434, 0.8833972009128579, 0.9649044746472721, 0.34779236790940116, 0.20607245361308452, 0.27804082104215644, 0.07320652869556978, 0.4745521203960849, 1.3551475522248542, 0.2544775650854701, 0.6159849398078021, 0.6159271153607209, 0.1526949530430058, 0.32433496167552933, 0.8510654827549644, 0.8041468389910631, 0.5765193440794951, 1.2116830318714702, 0.6406481547678389]], [[1.0975138512811944, 0.5556275987895649, 0.4551648073742559, 0.7344871626734825, 0.9296281747884055, 0.7053903998862566, 0.41073650051442323, 0.8560902183548651, 0.7533587196895981, 1.398586767395238, 0.23674380324386468, 0.8063787048047055, 0.6020884891627212, 0.5485779161526696, 0.42483932221381304, 0.3535290110614524, 0.5487295587980305, 0.8842765687501113, 0.93269953416777, 0.3693925183574796, 0.21231448834605796, 0.25260427098872856, 0.048043297024751457, 0.5479248476738228, 1.3950865171846578, 0.24770449882640355, 0.6804836577509834, 0.6361864084935903, 0.16849959154602953, 0.3668026585103989, 0.8515428923009599, 0.787980401696251, 0.5758806730288797, 1.2668439586838005, 0.683727415843589]], [[1.0955466774966374, 0.52300644476369, 0.442440676105052, 0.7193895304482687, 0.85345680121117, 0.7441444492192236, 0.39881529831793705, 0.8623817061230001, 0.7712478278380673, 1.347213214105612, 0.2307576228788359, 0.7725110304624807, 0.6535205206704877, 0.5356817773473828, 0.3848101736048543, 0.35167224211792314, 0.550055736866498, 0.8943487427648701, 0.8266306270615007, 0.3233726003050373, 0.21219391918052458, 0.1848226530161724, 0.0207406828404881, 0.5296412525542542, 1.3308351506724834, 0.24822203609246052, 0.7270133484278106, 0.6910975706320521, 0.21960749741676427, 0.4395073768143013, 0.8169688301584417, 0.7028811685516176, 0.544653588076734, 1.2795588367785975, 0.6572887579628098]], [[1.0160430712177728, 0.513678118302306, 0.4311688919332871, 0.7322775489047014, 0.7087955002130065, 0.7563806649504919, 0.3947844713708121, 0.8578697525086507, 0.7715635629228885, 1.2205453736812462, 0.21237511495560357, 0.808421845623381, 0.7200705633813296, 0.5209291221459486, 0.33689954951708934, 0.33908707883420786, 0.5655026140412586, 0.9317134024736657, 0.8478839867259043, 0.39485981308689927, 0.24746676160051426, 0.193780005645233, 0.028074954313299767, 0.5592734561930781, 1.350123423560114, 0.23789895603073596, 0.7475529576594313, 0.8350543671012799, 0.2882459798757658, 0.4756745483141408, 0.7317378239032282, 0.632455018072829, 0.5012890844540813, 1.2414047486964421, 0.5748097469300498]], [[0.9373548924753095, 0.5339604578497094, 0.4367244569700351, 0.7196403984480979, 0.5535270885492029, 0.6884647103399794, 0.39839975182209014, 0.876936578079625, 0.7709582270464157, 1.1321819578077008, 0.20861023195393458, 0.7810155938423946, 0.6947631307532898, 0.541771133234159, 0.25467214893025203, 0.33556239945393634, 0.5631484052603121, 0.8927940360345878, 0.8825966617538776, 0.3681028911487772, 0.24152642193439702, 0.14794245176706888, 0.011944899712381708, 0.6267500524609375, 1.3922093190846057, 0.22102106507963026, 0.7909213367721353, 0.9013810185842875, 0.3102103752927289, 0.511962974960264, 0.6867505870661739, 0.5776645386180571, 0.42154073561657956, 1.2109993280461793, 0.5429563627758766]], [[0.9073230304939239, 0.45847511794529006, 0.4275238153701091, 0.8039685896765438, 0.4895247212310775, 0.7781650839295429, 0.3980045121691592, 0.9193611332081701, 0.7989937054201824, 0.9989700977490223, 0.16527566965101026, 0.862745021151883, 0.6188052689768276, 0.5401854114402416, 0.25284864482990155, 0.34488485826163573, 0.5482631176066095, 0.8780037650877431, 0.9205666596582329, 0.38925508431535666, 0.23487498305960278, 0.16975165713737492, 0.011035950642318697, 0.4918921612673116, 1.3585123068594596, 0.2119649196674025, 0.779341379643805, 0.9084399985177627, 0.28767868359308074, 0.5538962489000694, 0.6372811688544191, 0.4623448078991637, 0.4410739033691592, 1.144569528596623, 0.5529328456618008]], [[0.8793040975487896, 0.45485382967124, 0.4870217414475343, 0.8565976300644657, 0.396428790738323, 0.7659177790086447, 0.3879846825871859, 1.0005415459640028, 0.8263191338338487, 0.9026501421449952, 0.13993931958490766, 0.8437144921699462, 0.6089061945964387, 0.5456633038126885, 0.2524110623795397, 0.3152830386110071, 0.5247089018484106, 0.8520570928494494, 0.8032185123918378, 0.4967094201779699, 0.24848501539649478, 0.15103761262249057, 0.011140741946475598, 0.48476926459449526, 1.2295530653384936, 0.22917761905295464, 0.7167742639932244, 0.9953583079758522, 0.2717986447740154, 0.5418275228977182, 0.6135181307930093, 0.4005325560404839, 0.486390059474297, 1.0476924408741608, 0.5526068016239065]], [[0.7991726745290756, 0.4560614019623002, 0.43171642202737265, 0.9505365086238247, 0.38745197969871614, 0.75135494467785, 0.3821823500591204, 1.0094620467348006, 0.8685478096221754, 0.8309068150409372, 0.14953842613182494, 0.8149306913809194, 0.6098495421632886, 0.6155142243080759, 0.2297124509082758, 0.3313712762031717, 0.5334748509309141, 0.8416231504272147, 0.824241595221997, 0.45136985365619064, 0.2338057458146551, 0.14792569584730164, 0.011734065448351599, 0.4968556472975467, 1.1621406515604356, 0.2533830152860881, 0.6558659260908162, 1.0145175394238075, 0.2602449644450816, 0.5215370086771057, 0.5954567250169247, 0.3776526869338219, 0.5594913772840803, 1.005518654767057, 0.5317166765242903]], [[0.7220644350979883, 0.5379509788240943, 0.47006485245408663, 0.9701276080229769, 0.46108198272084205, 0.7651059532782942, 0.3719248115464415, 0.970322103453032, 0.8418495022711903, 0.7074687810865443, 0.08366078217672683, 0.8112253318233469, 0.6240733020115348, 0.6741409068477258, 0.23599520748890299, 0.32417261624997395, 0.5547246791170726, 0.8834202815887664, 0.8066360101523156, 0.4016270004263065, 0.20292906662438187, 0.1417092461830964, 0.008928237650162317, 0.5076706859033209, 1.0291999156819673, 0.26147321132923873, 0.6777707199286747, 0.892621325231798, 0.2693919593618463, 0.5518431511543018, 0.5496016059547617, 0.39992357065222617, 0.5566710590499307, 0.921818534513964, 0.5390589564652436]], [[0.7191590007734888, 0.551088035859231, 0.4781275993462816, 0.9672473321786494, 0.43735565462306597, 0.7632800669571491, 0.37334660542485704, 0.9054921058483991, 0.9181133590230016, 0.5979693658192018, 0.0786902847558061, 0.8241482844676913, 0.6171363772384635, 0.6178591429360021, 0.2546494789237769, 0.34049799651812485, 0.5761402879985649, 0.8864844071105222, 0.7965802763926277, 0.43763219259665476, 0.21937039685145057, 0.13291085572091405, 0.009368540068607802, 0.5337978131702859, 0.920558089245463, 0.2670575698743073, 0.6814480828164091, 0.959638035562745, 0.27726628352536586, 0.5920574016123148, 0.5999706508259904, 0.46272278376866405, 0.5767785483980892, 0.8894870847438736, 0.6838775612757193]], [[0.6254244476277777, 0.5701515426958026, 0.49938067821433374, 0.9849235844484951, 0.5124242173110461, 0.8798943379218807, 0.35802998036367567, 0.9484386930975918, 0.981355694399429, 0.6208523121678893, 0.08873856498864108, 0.7675274535214891, 0.6321639479190839, 0.5836862155001834, 0.25862130792234084, 0.33376129054118553, 0.5698381806535231, 0.9488133207845828, 0.7710317510126103, 0.3939016278947281, 0.2176096641682979, 0.1658529747308484, 0.008264791233553219, 0.5705148965705531, 0.85056270825559, 0.26822222729711864, 0.631277081963169, 1.019373898874361, 0.2930832591090424, 0.6573813523471822, 0.6586417626250218, 0.5140914434252474, 0.5653278878138149, 0.8509640462469217, 0.8558889115683443]], [[0.632483609759985, 0.6179763841289551, 0.5229522524190454, 1.1091934590302053, 0.30709631261589054, 0.8020938298760263, 0.35872325604782296, 0.9648652153742117, 1.0105857702740866, 0.6527812730080348, 0.09402344759822646, 0.747825380618875, 0.6467056981405723, 0.595437096106904, 0.32951676524542484, 0.32874484203328597, 0.6186400589002027, 0.9116627940000019, 0.8021313059070097, 0.43249382721727686, 0.21296958857324375, 0.17607660914182022, 0.010194148665012321, 0.5689038944191038, 0.7819790571767558, 0.26928973635585807, 0.6525660019227364, 1.016055394127001, 0.2901901671343822, 0.706156873500267, 0.7236416510678446, 0.5461228054653289, 0.5906001246857282, 0.8642135611367476, 0.9988064370503525]], [[0.620744182886021, 0.6211472052926557, 0.5770630931064611, 1.2169467443320685, 0.3059839128776251, 0.7414158123450751, 0.45959501986274093, 0.9661245250587072, 0.9621566913683703, 0.6571996834606337, 0.07546235828886193, 0.7658325420138765, 0.6231934127182965, 0.505875856574123, 0.2971386823035724, 0.36218034668441057, 0.6264715670482213, 0.8912902404170584, 0.7235833758327859, 0.44399118172682916, 0.21975377316795414, 0.28392617739848214, 0.013027190293850901, 0.5643468414913471, 0.7344327352983901, 0.29284740358852707, 0.6605738056510695, 0.9658013181120264, 0.30305136873660066, 0.727826980688767, 0.7279298432510433, 0.5794358946876931, 0.6978440545225255, 0.8448987216893181, 1.0327469639054891]], [[0.6124898233137601, 0.5980739408262976, 0.5790107852734868, 1.3426228438096126, 0.36698058414599305, 0.7659845846447668, 0.44953649101899995, 0.9441788151545667, 0.9110167043252773, 0.6737464074429165, 0.10776452858965863, 0.8374811321150184, 0.6642371409776118, 0.4911399290626612, 0.30765440267164174, 0.3932196600597416, 0.5431193769106704, 0.851357747998273, 0.7358987262434515, 0.4425391923609866, 0.2376328440838998, 0.27370642369318454, 0.0178459040190853, 0.5650247681482085, 0.6967094894042126, 0.2868944530111478, 0.6325934420932657, 0.9807097180483875, 0.3262612964501149, 0.7771635954401108, 0.6784007114146401, 0.6247082509841999, 0.8033610101104951, 0.8390967672477505, 1.061030118308865]], [[0.7668873741255808, 0.6857257113067048, 0.648294959530838, 1.3595239789629547, 0.4159477171609363, 0.776725316338952, 0.399042111459404, 0.9181644477951351, 0.8859630799298978, 0.7270673124111634, 0.14403651577313675, 0.8564825460991161, 0.6182968618893656, 0.48871541333759855, 0.2809128487558043, 0.35926779527259356, 0.5301246410402378, 0.7610029922265389, 0.7337552475137361, 0.47851680296129373, 0.22590377271993778, 0.27524420145408957, 0.017699102299979206, 0.6487660106752509, 0.6901662989230513, 0.2605448003594579, 0.5928635569606533, 1.012740621854216, 0.3742110407957491, 0.816859439047164, 0.6376325374080962, 0.6573910677994819, 0.9542323190640329, 0.8448975091780442, 1.073169194738159]], [[0.7864793794428242, 0.6954499316379934, 0.6767154945151885, 1.4106808641837356, 0.4803914661671115, 0.7993971722676918, 0.4023691511550517, 0.8424137878624238, 0.860064661605502, 0.7816133699877783, 0.1761877873032846, 0.8407612406541061, 0.5962362328488973, 0.49902899912135945, 0.30089130082603927, 0.35309348247422917, 0.5918872866381837, 0.7370584050500116, 0.7734829710124091, 0.4478953192778522, 0.2048606423075916, 0.2792397003593779, 0.0344742239525426, 0.6147540176358764, 0.6709540186717103, 0.2560962671558174, 0.5085141119332739, 0.9163997926156472, 0.3905990534808207, 0.8623622961706146, 0.5825266484874134, 0.6387474375316862, 1.0524237360058748, 0.8346582164461225, 1.0836036655812316]], [[0.8704452907246949, 0.6946933734217197, 0.669499329179014, 1.5432566083528083, 0.5439514931265175, 0.7641380034428024, 0.48576728602691205, 0.7439164178436244, 0.9184336933448201, 0.8482718686645818, 0.1795013115446037, 0.8753983855059011, 0.6034037552350019, 0.5009309586311924, 0.24589893212405017, 0.3557619262062133, 0.5999560471385469, 0.7408937803033244, 0.7767235146885442, 0.452422170688716, 0.19014501654058885, 0.27011144152852495, 0.08242367336324116, 0.525296893301616, 0.6810721570706231, 0.2416532740761716, 0.39548325470309637, 0.7751455167460974, 0.4060533550302411, 0.9279603056439881, 0.6394218345239402, 0.6162912427071932, 1.0988628253985235, 0.7887576887604797, 1.1357025116621036]], [[0.927018154031872, 0.6401055011028284, 0.6036214401788385, 1.6896069042129322, 0.6025685064660713, 0.744985478696863, 0.5126477652286727, 0.6309625136474802, 0.923963209046934, 0.8565516407723386, 0.18133408175828794, 0.6979988275996121, 0.6043104748326973, 0.44261835871345767, 0.2584238040101042, 0.36561835492023087, 0.5961405765797616, 0.783701206985488, 0.7667734700920158, 0.4379356464872761, 0.25821830968673953, 0.2656043739576245, 0.013149049095324375, 0.5207075411435564, 0.6321702270374134, 0.23760912999702377, 0.30564197371712043, 0.6791290965581143, 0.4143756434693704, 0.9756017439400884, 0.7062539123795858, 0.5711267977601373, 1.1362329589031894, 0.7648209543564157, 1.1749595005346747]], [[0.9435929977746866, 0.6361710621074805, 0.5842773898389944, 1.6430337916026674, 0.6842157916293954, 0.7851809437323853, 0.5518843884442874, 0.526995279468044, 0.8240913384189925, 0.8935694669450611, 0.16684772017014482, 0.6542362599074987, 0.7174160842769126, 0.4482408047445148, 0.33551910603202423, 0.3810991759666559, 0.5618907084427398, 0.8370392923325085, 0.7831533322131745, 0.47075146011602453, 0.37170494379417907, 0.23077496361828745, 0.012643853666332215, 0.4694453787054557, 0.7208971044268442, 0.2697582279920337, 0.23521049058259436, 0.6672945094654854, 0.4222562596618098, 0.924785087691056, 0.7225969061671402, 0.5579752914253828, 1.1836945080767372, 0.8573673206169278, 1.230761282239856]], [[0.9233955787575072, 0.7254778607278664, 0.691019425909829, 1.4050901907491713, 0.8061011616115303, 0.7999477856051909, 0.5385038265633878, 0.42789694976428533, 0.7182822991949906, 0.9090235743205309, 0.1616652688321828, 0.6032641389826439, 0.7657165396120346, 0.45064132839594984, 0.27402495062597015, 0.36133791642698215, 0.5465941019223196, 0.87532713891071, 0.7950830928718653, 0.5307378584879376, 0.46260289576280117, 0.22206184376594282, 0.01147090701101244, 0.4033006803546082, 0.8562866741243428, 0.3389079717662119, 0.26155613538096933, 0.670318334085567, 0.48237088947098805, 0.8931282425906193, 0.7412659681811957, 0.6041188260473492, 1.2677894770817049, 0.9230434276599376, 1.2334659859859909]], [[0.9045442299107948, 0.72029042846741, 0.680766679684619, 1.18040397388923, 0.9263409372153455, 0.781958705641052, 0.5361147304123219, 0.4805699358688008, 0.6677462372479281, 0.959758171836465, 0.16353965069091217, 0.5847931433985571, 1.0101794992147841, 0.48054021779624867, 0.36241616816252814, 0.33710665752143265, 0.6792416206995231, 0.86271899111187, 0.833379378060099, 0.39218032023716887, 0.38389002465052385, 0.18730422775483135, 0.010216401931970317, 0.3919980736979843, 0.9222840477133836, 0.37733749270529776, 0.3015281695945684, 0.5478156110721484, 0.5858900173329922, 0.7932556226483257, 0.6593482838289729, 0.6076924219094768, 1.255389514518308, 0.964936687343745, 1.2890996275063438]], [[0.8961482623601822, 0.6625563709644022, 0.6370641455251989, 0.9935099411245745, 1.0276500057237004, 0.7223291204703748, 0.569494182281388, 0.5025331527589844, 0.679423760378578, 0.9669279776493415, 0.1785735181583196, 0.7208302771466377, 1.0606441960859132, 0.5472106121905882, 0.47666134449084613, 0.3534953539970925, 0.764697472694758, 0.8276881235135234, 0.9023144551316336, 0.27560166872309527, 0.31018229631375627, 0.19265037263531104, 0.008316340348781606, 0.37902327241101735, 1.0160116101044212, 0.4257888647882848, 0.396520140418529, 0.5817577281570119, 0.7838761404291104, 0.6065474459767858, 0.5827511236871517, 0.6357526400429176, 1.2360917475488384, 0.9593285475496466, 1.263261209763739]], [[0.8654731599198885, 0.6771264387244361, 0.616381571828015, 0.9478746052872739, 1.1294996174624634, 0.7072666288215078, 0.5721122000750718, 0.5089171119097973, 0.7646257723576017, 0.9652916242102583, 0.19054172999748625, 0.6397921077464579, 1.0641392054793484, 0.6316164348668507, 0.5783572223082947, 0.38573936268485476, 0.8035657111391271, 0.7844106233078085, 0.9887515417485127, 0.41785131735813297, 0.3290506294098953, 0.19803913798616177, 0.008435335023423418, 0.3587062883884138, 1.086093767819066, 0.5643344721069166, 0.45492395035089683, 0.6342422402873784, 0.9159675370432651, 0.6420267590972922, 0.5779658597691992, 0.6956690516553519, 1.2194744661012187, 0.9730442686189826, 1.265856345072585]], [[0.8530430309730617, 0.6363116592260781, 0.54186115496623, 0.9393075955482043, 1.2407331107667265, 0.8989417449470365, 0.6801993989913275, 0.5192670850493203, 0.8665845740172602, 0.9132632343403266, 0.19959848239778594, 0.644067354630842, 1.065942100602129, 0.625719574532923, 0.6533307592462628, 0.536725192156985, 0.7508841440332519, 0.8698855330896742, 1.0253649561205491, 0.4269653454634479, 0.3079343375940365, 0.20857841119960463, 0.008775022525610052, 0.31598629067207773, 1.0676195448997732, 0.6416566382213295, 0.45141418486668444, 0.6080070290765701, 1.0061219738125555, 0.7424413172701927, 0.6471902342715479, 0.8190847755921671, 1.172209225964874, 1.0055339341529108, 1.25226886026998]], [[0.9407009894468675, 0.6761871695300379, 0.49919418366301027, 0.8645858162733706, 1.3235622963775633, 1.1014847606364178, 0.8185941399067049, 0.5288702936361818, 0.8103768031154633, 0.8787882284350342, 0.20113148926723778, 0.850944769266156, 1.075736066410572, 0.6694182171826484, 0.7480530185600774, 0.5601146401425605, 0.893085696636455, 1.0215012695925856, 0.9274547706906386, 0.44235275221076464, 0.3767383930521342, 0.19673543086803202, 0.0090056304479786, 0.290960536247345, 1.0299289820898325, 0.6091572210712124, 0.41057402723378833, 0.5699554249751633, 0.9445725074411248, 0.7823488322718399, 0.7297099746635414, 0.8125087814557272, 1.0733605347893005, 1.1674729135836417, 1.2432002133318631]], [[0.9863568556911164, 0.6949208936258882, 0.5078342581213833, 0.8338423308571641, 1.4148910213234238, 1.1027913940200056, 0.8138877059920084, 0.516466745142699, 0.9017044956322728, 0.7731664800699368, 0.20301476331372276, 0.9180087252488492, 1.0668503953736532, 0.7291706177067732, 0.5732584741449442, 0.5285403533271406, 0.9377329621844733, 0.9726050672651086, 0.9676297007709674, 0.394869430010375, 0.3862945643702432, 0.19014201913870793, 0.009762596925872988, 0.3135282102424014, 0.9933500666541726, 0.5512324232598289, 0.3876616739340851, 0.5225458764531584, 0.7989262292131325, 0.6684450686893381, 0.7402560627694479, 0.8298205829125018, 1.0009151215026637, 1.3151485347452736, 1.2394079611403441]], [[1.0060394446356846, 0.6815681737131964, 0.4968529664691256, 0.8366948006681428, 1.4593175222297368, 1.113080548755789, 0.8231536010001471, 0.5103213336804125, 0.9119790395382495, 0.6944545893599525, 0.2342998997196712, 0.7657617110806718, 1.0440879388532933, 0.6177934639835833, 0.4798197599302011, 0.5392890543084854, 0.8724259550722464, 0.9967969602901479, 0.9109476883090615, 0.40097487957889455, 0.40602185412312747, 0.18992876754220595, 0.00902080881280527, 0.3231083895725146, 0.8976849117600009, 0.487214672395477, 0.3821424838560321, 0.5111683354953724, 0.668143746391456, 0.5193987145601919, 0.7594309197383188, 0.8337616039027558, 0.9095831896709177, 1.3239144781902448, 1.2347289705830764]], [[1.0330722789141769, 0.694958709636362, 0.49226586563532226, 0.9139555945195753, 1.4874163294352432, 1.046861667497762, 0.7613585683835397, 0.5331591227492045, 0.9382301978437126, 0.6938208814195788, 0.23896654751563773, 0.6778299417192117, 0.9813670311572441, 0.5706687072570756, 0.5309846636520512, 0.5642732990719078, 0.7254508557071019, 0.9947568867876809, 0.9023251886138178, 0.4232095196500951, 0.4364807315634276, 0.18939624371410801, 0.012368880415937361, 0.3370708876578311, 0.77667306535243, 0.4043117012537533, 0.40398893496962585, 0.48048399034934475, 0.5687705373001101, 0.3367959715562359, 0.7535805398051355, 0.7961560117522534, 0.7960753494897115, 1.3605070520707745, 1.2531271887287114]], [[1.0621826732326165, 0.6886141344255248, 0.48863103100919303, 0.774237064012079, 1.4712871049623126, 1.108618272968865, 0.7204878933113479, 0.590105515950618, 0.9661352772773117, 0.579325746246784, 0.24288016732180046, 0.6343468515850041, 0.9488281619668371, 0.5163549654418338, 0.533674364409993, 0.5659199212019388, 0.6991223629259041, 0.9898716618636, 0.8519353907578215, 0.39781974799867914, 0.4463040050146033, 0.18917673132182722, 0.028208945821489625, 0.3172082292305297, 0.6812021178784431, 0.349568152912534, 0.41952410979699994, 0.49435649214470834, 0.5006253650838381, 0.2109603981843281, 0.7589309072884665, 0.728256413170941, 0.7061825563767476, 1.3831034738622825, 1.2770000375450405]], [[1.1514543283511056, 0.6757550045810875, 0.4576459847155237, 0.9575199579294803, 1.4779255427210831, 1.1943397612794815, 0.7302189446431515, 0.5850594611261675, 1.0748817448846681, 0.4664009141764427, 0.22286519340369365, 0.5984404741439842, 0.9304281686743532, 0.5165712542378417, 0.51916158347879, 0.5775062179138037, 0.6708195940448313, 1.0685064235665123, 0.8679279381972672, 0.40295641987976993, 0.5626639565113418, 0.20727589048684159, 0.06932541577526127, 0.3082338547563672, 0.5690954801172765, 0.3070369956293464, 0.44833199951745306, 0.5214131512475774, 0.4860084193141985, 0.20118286363649884, 0.7714144882027338, 0.6655770881661146, 0.6120164599593301, 1.4053128195050832, 1.3056405097064]], [[0.7572202482679471, 0.6083814647859993, 0.4161382143406317, 0.8185496105479275, 1.2641858498580514, 1.0231601139436328, 0.6515373881160669, 0.4949012080297307, 0.7802004075074277, 0.34235431905663916, 0.09615578902181389, 0.5261054858830179, 0.6422487938394384, 0.5357432847554647, 0.6047885810188762, 0.5857880967288208, 0.6522233605591152, 0.9743402700838121, 0.8251421422847827, 0.3304138745715106, 0.41031842550168673, 0.1948170828772147, 3.4870886584447467, 0.19974931014858194, 0.4312466174585301, 0.18433759439634023, 0.39153723029120097, 0.3811779595795335, 0.27780611702831803, 0.22071288673872158, 0.5599417345145996, 0.45835242670198506, 0.33313113427026714, 0.8766741004255576, 0.8236757638160034]], [[0.679734621938364, 0.7227612993035819, 0.535506183524393, 0.9083049250877511, 1.2434719348410417, 1.0334374616061341, 0.680253572002265, 0.4821397040343132, 0.7502237455193743, 0.31710784096549943, 0.12311053136409195, 0.5922699576944717, 0.6369490239041364, 0.6320238235811961, 0.510612819912577, 0.6173488857020929, 0.565052276518129, 0.9180749930993267, 0.862054459033492, 0.4156507820737554, 0.4009505047405049, 0.1999726188629194, 3.4870886584447467, 0.16875405931621862, 0.4112805100537632, 0.18703936998172443, 0.45576378786904104, 0.3173518646931964, 0.2147198412231725, 0.28259709075715245, 0.5922440436344462, 0.5542002598364277, 0.3072473610509779, 0.859131546870523, 0.7880884609150167]], [[0.6459717907668272, 0.7228640194162346, 0.5264720487011378, 0.8023279023898675, 1.1873350899387909, 0.9432397325185622, 0.5841866451817387, 0.4695835093181382, 0.6756013880759465, 0.3856849488846411, 0.1665608218432802, 0.5582524460475571, 0.6681730458813004, 0.6629002117999583, 0.4682363031038462, 0.6207264387889382, 0.5495304238635766, 0.8302618012715995, 0.8342824074639916, 0.23188710544507918, 0.28938615551866537, 0.2394606816417268, 3.4870886584447467, 0.1717532118358237, 0.38491990268646337, 0.2647893012352921, 0.48861246962811317, 0.27468999070437133, 0.1955059935924715, 0.3310603457684722, 0.6296537335243251, 0.702518789382879, 0.3334884175973216, 0.8429216097160241, 0.7809029779655212]], [[0.6195107795882143, 0.7275172959141654, 0.5406342907942581, 0.8630341248042162, 1.121956268316348, 0.9353623172338308, 0.5771514569508743, 0.5290539018933877, 0.5915525954255401, 0.43992495989327307, 0.16097482124853563, 0.570613560769486, 0.7846903891131756, 0.6623813443528932, 0.6130567331203376, 0.587787618125638, 0.6238753486589732, 0.9745164829905508, 0.7938135169177045, 0.20624083994145576, 0.33755796084768397, 0.26538688961367113, 3.4870886584447467, 0.22764438476380164, 0.37101613552960727, 0.3259570314963902, 0.5005265205907732, 0.23374396085984125, 0.18201728867655648, 0.3302548270963931, 0.6547667408560253, 0.805563742068455, 0.3931171346579067, 0.764706120776914, 0.7549266300086075]], [[0.6139701227931478, 0.7284480528225414, 0.5470932262029763, 0.8005903878499487, 1.0326138295105787, 0.9541448262741795, 0.5642374321891532, 0.6095505466894755, 0.47824470223117826, 0.47821239473000854, 0.21316465862776157, 0.5777742243159462, 0.7873391632864937, 0.6236011153836535, 0.648275731468922, 0.5848612235045957, 0.6055226246759945, 0.9776750842063658, 0.7604292174727475, 0.2243647815517495, 0.3594228683339509, 0.25974144556101053, 3.4870886584447467, 0.2784982568346539, 0.3594218567647115, 0.3440574660154608, 0.4728233785817998, 0.2077045264050092, 0.17609165483774722, 0.34917594850520867, 0.6524283737574381, 0.8587541916215089, 0.45121571129514115, 0.7072857807358264, 0.7090225069497074]], [[0.5672046809945132, 0.6583826776988311, 0.48353839123312176, 0.680081423366879, 0.9574941086666725, 0.9464341129045833, 0.6405673077658751, 0.5382044510923074, 0.41429637344624987, 0.43485881512895735, 0.19480886105141584, 0.5936519975912194, 0.7684416957776179, 0.6049282559666451, 0.5765711125577704, 0.5580469870787234, 0.604183793863648, 0.9094378931830328, 0.8167885854182644, 0.23560312528164917, 0.37039098647958424, 0.24974712230850848, 3.4870886584447467, 0.3049888571632732, 0.4094660737494292, 0.34774548768744185, 0.40844510451036553, 0.23839961044568642, 0.16403421145044617, 0.34904498423735275, 0.6014037474186913, 0.8247098202320421, 0.5207166114521956, 0.5944560162687555, 0.6355600996676694]], [[0.46658149071078836, 0.6536754172065075, 0.4700564325129975, 0.7151300636711221, 0.8695988111275545, 0.947207479932364, 0.6116150491042973, 0.507555010179063, 0.36579199279296876, 0.4096846085983039, 0.22632136067120584, 0.5884438378756899, 0.5548516552670377, 0.5949957255709811, 0.5431703075696981, 0.5836669562494654, 0.5385193520107473, 0.9002765232171845, 0.7177448914794876, 0.2042906570464546, 0.3743400464132163, 0.23825854122216317, 3.4870886584447467, 0.31968369869183055, 0.5448306896944508, 0.33840462674599187, 0.39863419041155224, 0.3218000670774433, 0.16259961811776522, 0.3489471255256231, 0.5173056488955825, 0.7344137833394202, 0.5340122368618383, 0.4480576105179692, 0.5201119879996161]], [[0.4280109782792319, 0.6174803476137318, 0.46615238279105375, 0.7530093900156137, 0.7900793090598082, 0.8925436554769877, 0.5496955605002432, 0.54092169994915, 0.3542018143415326, 0.41958769044937605, 0.2205489529337826, 0.5826860776188442, 0.42913989294688093, 0.5678003848112083, 0.5819380899054168, 0.5949960286767587, 0.5331648402410796, 0.9141678311061434, 0.7417392893802901, 0.17592820533102815, 0.36708166087004, 0.25176579320863673, 3.4870886584447467, 0.2950906170129901, 0.6606399255232064, 0.33239574977411496, 0.3602935724829182, 0.422847311743589, 0.15551618163668351, 0.30663285191733874, 0.42149738581270013, 0.6245686741989475, 0.5308604347399498, 0.2889846256752202, 0.45471635234375574]], [[0.3817131422968494, 0.6487465588065238, 0.5194354785445584, 0.7666184515283607, 0.7705114793158362, 0.823601314841816, 0.5187687038870975, 0.5244910126615437, 0.3624403878974994, 0.44857952237137666, 0.15375720223875025, 0.6007569757293367, 0.23392764461553106, 0.5627060828491601, 0.4899798417781728, 0.5391016671339417, 0.5471036907203776, 0.9353616260209233, 0.7158384072146595, 0.1613204434228105, 0.37026588623109846, 0.2739378221633457, 3.4870886584447467, 0.31942382135627057, 0.7662726057288612, 0.31379722627416246, 0.3535139237731612, 0.48584276945241833, 0.17598312405112881, 0.3197556299936857, 0.31750200632304476, 0.47843782661840634, 0.4762737506724961, 0.16905474829357434, 0.5232211534899749]], [[0.330906909931011, 0.600837547754451, 0.473946827684089, 0.7872120828154454, 0.686623528721612, 0.7948786809302961, 0.549561901567688, 0.4832060748418643, 0.3394504378083376, 0.4317866899018861, 0.12428946306396355, 0.5832901725331796, 0.5201192458654504, 0.5739097197139972, 0.45377295307040527, 0.6079523770440804, 0.5497330821907248, 0.8972728184909016, 0.7350496494032046, 0.14756948650464652, 0.35479272165887354, 0.25119443658890817, 3.4870886584447467, 0.3053208555140141, 0.8574423069248097, 0.3903429805604866, 0.33490257568908327, 0.6131649335003295, 0.24846446795140598, 0.3357413015458054, 0.26782304697339454, 0.4237753913651173, 0.262715904307795, 0.10831524139325086, 0.5626390510687522]], [[0.3145938330426047, 0.5315487917782019, 0.4345763905486677, 0.6930656360632363, 0.617961290912757, 0.7214896753474574, 0.3879629733554068, 0.5589136034577675, 0.32198872556847996, 0.44827153694889244, 0.1229637571535993, 0.5764337561497448, 0.8042205485520071, 0.610957152470261, 0.4699267281684408, 0.5876781094897928, 0.5622043253044295, 0.8875269712000908, 0.7962244355472428, 0.14239944016490436, 0.3805687497733373, 0.24269212530775006, 3.4870886584447467, 0.30205835846038875, 0.9403730028666943, 0.43362004002265475, 0.32799937623745623, 0.6851375177543806, 0.2459368228796541, 0.34693587321283226, 0.35639195672934637, 0.43326493661316656, 0.20055908009362083, 0.15337772651718853, 0.5412362470510416]], [[0.29772534281971835, 0.4580146824139635, 0.4164628872877042, 0.6641694433962482, 0.5901294958859779, 0.7321162816982678, 0.3920178509526479, 0.6291667179315501, 0.28121864673381614, 0.4733097908227252, 0.1115429321483328, 0.5809715736667358, 0.8050689243502525, 0.6329665722053942, 0.383053789272698, 0.42439391379488434, 0.5849256018627954, 0.7912031571000793, 0.8278547203716662, 0.13390730326620792, 0.3869201267085425, 0.23587780143441817, 3.4870886584447467, 0.33196507690375493, 0.9775866652793713, 0.38188123491366877, 0.2603566662916924, 0.6865339836956028, 0.2396082211184721, 0.364096811739403, 0.37474673903803213, 0.40844801773021266, 0.16916513750632972, 0.19859665116769243, 0.5148193259303357]], [[0.30009010859820245, 0.4569315289440652, 0.4430357802060184, 0.6102994426903068, 0.5720639963929841, 0.745977070485778, 0.3900248793644197, 0.6007869328750084, 0.28726733581494757, 0.48616153475179447, 0.10822740864125388, 0.5890667455734921, 0.8151311601564434, 0.6457203980235378, 0.36426957108373575, 0.3785978050102009, 0.5468601676196981, 0.7628629413526895, 0.8774696961761617, 0.10749588964942425, 0.3577031768735819, 0.21759064444687176, 3.4870886584447467, 0.4667885835606833, 1.0024693972764642, 0.319969756239736, 0.2751241837464462, 0.753862359839359, 0.3625047502240685, 0.3677869089295452, 0.38590747108483836, 0.4130725147174865, 0.29368553609286596, 0.20357419100000362, 0.5242244615238247]], [[0.29389510978658556, 0.4994260171579934, 0.45187568125207866, 0.5525983892870638, 0.5329834429674722, 0.7938245594940895, 0.39033452408914393, 0.5744305825601324, 0.2923561303595075, 0.4668388304654497, 0.10279673868326826, 0.6101539517644858, 0.8170099785673834, 0.66652873715133, 0.35983642767303, 0.33659521746155574, 0.5598541935196025, 0.7295799836507599, 0.9024452141665953, 0.10738465339676889, 0.33000449138713517, 0.21681584391350947, 3.4870886584447467, 0.6010932032054067, 1.05291949747279, 0.29633332464861856, 0.3142762209665207, 0.7446181873920665, 0.4014823208026882, 0.3451405338191033, 0.3774329653517118, 0.4234885510898595, 0.4122022213273868, 0.19037950399944, 0.5306205189939894]], [[0.43815054182929614, 0.4199543419707976, 0.4935992100799694, 0.6002275105949625, 0.7075459269193596, 0.8059306122420227, 0.380446845568151, 0.908271903747801, 0.5371814943827848, 0.5645696690779678, 0.13928923721125883, 0.593425612155678, 1.2647998610459905, 0.700506510481073, 0.391267058034168, 0.3619957309440769, 0.5964678317884288, 0.7383600639522019, 0.9991615373496637, 0.09567877598237771, 0.3580230233742886, 0.29323522390522705, 0.16129574111620168, 0.8778910207486041, 1.2642018930726944, 0.46290116732178865, 0.35070553062514503, 0.848277803783016, 0.6892714912223868, 0.5429915022606496, 0.5547229093856395, 0.5390267029389706, 0.8620786228875702, 0.2082875217090189, 0.9675253675836326]], [[0.3893839835900936, 0.3384920117783377, 0.5378642399797051, 0.48318085226576146, 0.6076413759228683, 0.888780381947273, 0.3658270823555883, 0.8406586050435655, 0.5433760478628478, 0.5048474085548735, 0.13528708789164567, 0.592822645551452, 1.2960983702511575, 0.6019647525339794, 0.3733010574375672, 0.3489246392312433, 0.6472549696044028, 0.8214928208794201, 0.9461964518281071, 0.09908251508794141, 0.38761820467449903, 0.3144528454300425, 0.06791006476684394, 0.9273157138205764, 1.2404110172919383, 0.4327239333868875, 0.3606547373085811, 0.8465729187957509, 0.7206326597106115, 0.5639475065301356, 0.5633781428680885, 0.49726107078159065, 1.0071795290843057, 0.2008850380553572, 1.0133370071281393]], [[0.4020208668463105, 0.3803898388213023, 0.6375491240565372, 0.40314160046209224, 0.5635106565719323, 1.1009504391877658, 0.3614671599125431, 0.7724582673444735, 0.5952489594250222, 0.4551253207374292, 0.12828967561304128, 0.5642909018483145, 1.3181392420495959, 0.6211343582394198, 0.37638817635714505, 0.3601508770007301, 0.6425141150408593, 0.8563396069359258, 0.9187093739169567, 0.10861971197048848, 0.39886788618391195, 0.29254562840407006, 0.17406401150822326, 0.9899159656948018, 1.1748780379129566, 0.449837519774689, 0.3542433957092921, 0.8112248494044599, 0.6932581694284781, 0.5851039526187729, 0.5889884332180463, 0.5537977396402847, 1.1008596931663233, 0.20824313112024945, 1.0742180429099255]], [[0.4093094568915051, 0.351376538286844, 0.5450467313051293, 0.30494230165512726, 0.5084532000584957, 0.9275444061602621, 0.3928491216017519, 0.7346553564687983, 0.5489771159689892, 0.40641379756557683, 0.12354974797516052, 0.5726404047818299, 1.3744614462504074, 0.6195782526161571, 0.3830425876519928, 0.35815999275507615, 0.6767988809007807, 0.8441496610893479, 0.9341897540715859, 0.10284424367423191, 0.395140647918923, 0.34085477843282413, 0.14699849649677266, 1.126234216174696, 1.1249810194193601, 0.5277604706372441, 0.2845065670740129, 0.746141165117658, 0.6502666594140898, 0.6460468546410775, 0.6012313998228965, 0.5079829077391649, 1.1783951710661569, 0.18842954180101598, 1.071555056389457]], [[0.5017330914389355, 0.33084915592525843, 0.5644917437903261, 0.25576372971322975, 0.46992480561044764, 0.8920028282984012, 0.585586488440742, 0.6926915824995465, 0.5488111921018642, 0.40822161638564364, 0.12341999490428485, 0.5872183824713462, 1.438245348413337, 0.7224284095179521, 0.37137640379816905, 0.34427797898096957, 0.6983550223378902, 0.7408027095354799, 0.951801762939186, 0.097253779386216, 0.28005922838843783, 0.35058450396450114, 0.12491393731787147, 1.2019375025797563, 1.0428824098780236, 0.5438547173400233, 0.3706311542962317, 0.6830811306938722, 0.6624717059402407, 0.5886662606255872, 0.5647423152647082, 0.470959244420985, 1.1618325194447885, 0.2753387908678625, 1.1024406249036494]], [[0.9584557675220546, 0.3405724298691625, 0.5139401351768713, 0.2767419340355489, 0.4874414065237209, 0.9835092097074625, 0.5204081247900039, 0.6304132670770758, 0.5540659802494097, 0.44639974605982097, 0.15092555193832613, 0.5746483489765426, 1.4140613418207972, 0.6924096325624667, 0.36453902754785145, 0.3734729075692338, 0.7289268358999307, 0.6944693722131882, 1.0705830909225302, 0.09713615464922512, 0.2206170215631481, 0.41602672462248236, 0.11893393109710156, 1.289236284767814, 0.9837804352538688, 0.6124676776029174, 0.41747575985422103, 0.8048965380406546, 0.7308714788845526, 0.4563930756601415, 0.457565861770418, 0.5074791658334014, 1.121975519490445, 0.2569493732644881, 1.118068709094744]], [[0.9682361801646999, 0.3298031929936104, 0.5177238095627921, 0.3545528245940183, 0.5181234140879418, 0.9985414114359941, 0.4612835817270262, 0.5708656630317708, 0.6651987305419937, 0.4821175492729487, 0.17682528235728856, 0.5662592150262291, 1.3930921648052563, 0.7121402945255441, 0.35862887339279054, 0.3740545620937664, 0.763769311359002, 0.7026774757869549, 1.0390853840472576, 0.09910180503676747, 0.21637236900160745, 0.44343614646364715, 0.11656668098823997, 1.3279900179715678, 0.9561346925938966, 0.6078242806565675, 0.3807084189767407, 0.8956462073628632, 0.7295049628227127, 0.34872886250832014, 0.3600764698028275, 0.649107369318624, 1.1269731158321559, 0.3303517493527841, 1.1298232462151883]], [[0.9496129569423366, 0.3403779374156539, 0.4374054183523304, 0.382488146954072, 0.5254892287448444, 0.9819976775441734, 0.4693323556223644, 0.5378512381090683, 0.612081809029748, 0.5200819434036471, 0.17429800894968545, 0.5681979918654945, 1.3611758819945887, 0.6846311253471057, 0.3514176744591029, 0.37182976140356305, 0.7345080097102905, 0.7093593865002888, 0.9249439678314132, 0.0932057730117645, 0.21823722881208032, 0.45270247997795277, 0.120218307827322, 1.3427484567723202, 0.8418453028480775, 0.5932909705876569, 0.3580075669657874, 0.8673555363123107, 0.7664254613578083, 0.3052817434119379, 0.2975374826258781, 0.7276573648422414, 1.1346829456962517, 0.4929473879018787, 1.0936848386977616]], [[0.6534985238364284, 0.34957043293577705, 0.4215165045532526, 0.4102490274140636, 0.6478277720953264, 0.9608173996797695, 0.44910238562757454, 0.4973632784479689, 0.5134748174780861, 0.5434805646777319, 0.16889295349044828, 0.5898040941042838, 1.3642435888653819, 0.6808458966219526, 0.34825639454111634, 0.40904521314036213, 0.6848112679593888, 0.7059135432299801, 0.6882978272063964, 0.07386130011307204, 0.2419969267901552, 0.45809139059760007, 0.054071954449686384, 1.4136339344652646, 0.7071209530797549, 0.5579838984250948, 0.29463551249537323, 0.8498866436963013, 0.747226772521101, 0.2908589026485361, 0.30465175923997234, 0.7040513269565916, 1.1589435504293524, 0.5961903351674107, 1.0551346849955094]], [[0.6317241370891407, 0.35366952106412775, 0.39080323823215957, 0.37476145555348983, 0.5502693157431284, 1.0723891782513448, 0.5265185873642386, 0.4497267624367656, 0.5293198040700234, 0.5788395350783471, 0.1583490872440713, 0.5859777067562681, 1.3125728047630316, 0.6583219269351006, 0.34338458275677064, 0.4286388380902192, 0.7178904343759646, 0.6689532591911156, 0.7217197895805596, 0.08766458203668485, 0.235634228409715, 0.48751991549429163, 0.030044042136924586, 1.469258194848468, 0.6101568264183215, 0.5292330891169608, 0.2925839113473346, 0.7037865194011814, 0.7591045110725513, 0.27036108395549424, 0.30495977806616137, 0.6366610997900712, 1.1715446928634605, 0.47562181813626414, 1.017907741868881]], [[0.5390777376638414, 0.3614561504297489, 0.3802364420374821, 0.3794977894429497, 0.5476759665345987, 1.0259236803214193, 0.5197046649030506, 0.3886647061308643, 0.5913786721234018, 0.5868907172229689, 0.14892584523636193, 0.5653244694811167, 1.334999492008869, 0.6798830162706602, 0.35873502177625993, 0.4257252768423586, 0.7176010283322269, 0.6715211423371477, 0.8335210072793298, 0.09410620129559281, 0.218274846312226, 0.47105357749675947, 0.02833135955422079, 1.318304390895513, 0.5281065158570559, 0.4871059339037044, 0.3094365791369477, 0.7234616490581127, 0.7189398892408791, 0.26853247182453077, 0.30099241280181405, 0.4967199580571705, 1.2066120613511915, 0.43954763838393274, 1.029297546958474]], [[0.5529212169082308, 0.4054440346071138, 0.38986581323738595, 0.41463154464352703, 0.6061598395178364, 1.0025458846034403, 0.5066723487710569, 0.3613659033001588, 0.6070123082812791, 0.5532521312486745, 0.1322561533703071, 0.5683935645418839, 1.3305521174612975, 0.6531457435872866, 0.3446719791326731, 0.33358199835634017, 0.694960430038888, 0.6257998658094317, 0.8326124592816989, 0.09384321091756923, 0.24078125740292305, 0.5142723288779676, 0.09111277350762556, 1.2184360402897207, 0.4490733835958379, 0.4305463057409756, 0.29402262427095077, 0.6907611926064295, 0.7127264829869219, 0.307183990938809, 0.31215570669016973, 0.46479176537408295, 1.2135855434658878, 0.5082568606480348, 1.0676268113299747]], [[0.5995336323413653, 0.42139340636562933, 0.3865869154003357, 0.41599756129493576, 0.5436654729233619, 0.9618648420195279, 0.47800974533386675, 0.3686437100750909, 0.506186327804284, 0.48239420425967633, 0.11940351462210408, 0.5882657141593243, 1.3125161514627206, 0.7287923395307674, 0.33845292330623555, 0.32922738550616004, 0.6750363735901868, 0.6222556416704548, 0.8595669582140859, 0.06438068908696032, 0.2545729331609634, 0.5780143707651756, 0.07399692281172224, 1.0850586728917988, 0.4431388827226654, 0.42112402214097683, 0.27499982413272195, 0.5963359050344469, 0.7031006446716561, 0.359240450576613, 0.3682626198560072, 0.4762173683800655, 1.2043347608354185, 0.5441071639919806, 0.974549828615058]], [[0.646392378748045, 0.5153997601146899, 0.36600703683699104, 0.4242895435056625, 0.513972779249433, 0.9394169744000724, 0.48676081248016023, 0.4248481001521073, 0.49758535252108826, 0.5255392465065553, 0.12954300864036264, 0.5958361329877297, 1.2639461315373548, 0.5902075039750434, 0.3281207810025048, 0.31850357251301564, 0.6461548671267192, 0.6064063423140964, 0.8194989458028493, 0.06950264605702829, 0.23756651432860706, 0.5750793418425872, 0.05379257026372332, 1.039708362791214, 0.4191171377595395, 0.4061483653628611, 0.2735092393665653, 0.5656296603273593, 0.7024362103993039, 0.4336819183936614, 0.4939910299715881, 0.5170622442122838, 1.1870741126915694, 0.6275610043235129, 0.8302098200617778]], [[0.661378939447641, 0.6067866479112713, 0.3422488175020884, 0.4231001298994121, 0.5280625129688199, 0.8235420656024808, 0.44613385777803577, 0.5435431511544482, 0.6198440975046887, 0.5968285259950632, 0.1523574909057744, 0.5943304634360065, 1.1561379609845959, 0.6078966826643681, 0.3199139754935294, 0.3416016103723633, 0.6287667182550741, 0.6260287840296591, 0.8074524109100472, 0.10224384803467829, 0.2262362460411113, 0.5675854100069884, 0.0674151852277562, 1.0195603602385623, 0.4207843617698856, 0.3974482498504964, 0.26552103066837857, 0.5402488306004966, 0.7233116739320178, 0.5428249032509685, 0.6589526134978698, 0.6826163196680003, 1.152940585010289, 0.7089430620677857, 0.7772190981698899]], [[0.7958176666987121, 0.8061942031309333, 0.435864179368887, 0.4273960467215556, 0.5634700274381621, 0.760028916539232, 0.43211736324745154, 0.6538346073836155, 0.6168893595290429, 0.6357925230522563, 0.20720171356352785, 0.6077390100296572, 1.056518008704713, 0.6007269188038856, 0.32127623140364114, 0.3538599896682137, 0.6687360196487488, 0.6224457481634503, 0.8269452890164823, 0.1400769175116504, 0.19209926207378336, 0.5789040908201465, 0.04024139037330668, 0.9866031572193916, 0.44338646622324923, 0.35967302788527655, 0.28635691127482066, 0.5852403204285983, 0.7637869681202176, 0.6492685226728403, 0.7391250346304244, 0.822790273078505, 1.1237667672013552, 0.8282925376434327, 0.8470809100900885]], [[0.8816855671318111, 0.7881220654953445, 0.405411802667487, 0.4458605607845541, 0.6167794711608845, 0.7754633109640781, 0.4501393353080424, 0.5743553276331257, 0.6975298995230726, 0.6255751392393248, 0.18981064407471365, 0.6020373108882058, 1.068527550470247, 0.5594657254697013, 0.3030845345544899, 0.36645466101004365, 0.664947288715529, 0.6522856242907418, 0.7904147255787128, 0.22029915198114342, 0.20939950303351812, 0.6247975268007375, 0.04309568596764292, 0.98441903813579, 0.45887476821877343, 0.3228783668311904, 0.27912783343674613, 0.5429719226608543, 0.7833953259998916, 0.5699310968991986, 0.6515623331809915, 0.6659820296789268, 1.0962953306528047, 0.9187250500781861, 0.92379290218134]], [[0.6007474495869682, 0.5788163000350734, 0.30796903383604646, 0.5226643724263831, 0.6406925076969494, 0.7380310267366802, 0.4450429063170925, 0.540653550039025, 0.686790794857201, 0.6066725980511611, 0.18634677779918474, 0.6676070582049854, 1.1818798931524188, 0.5701482243126845, 0.30163925539494507, 0.3561928680174419, 0.7456209089737328, 0.6672053428624086, 0.8645474925950826, 0.2869604618467698, 0.29589163002271385, 0.5209907253739033, 0.043827327258290125, 1.003721226779247, 0.4871061242681697, 0.3161711317380883, 0.25630029791327863, 0.5812310218973786, 0.8238058437422526, 0.4338457037663848, 0.4744213810385177, 0.5931483826621436, 1.0749866648042925, 1.0151959462841007, 0.9717991495289479]], [[0.6453435703632656, 0.6453958820852658, 0.2710566593906944, 0.5223971777771862, 0.6419908627507313, 0.8746253888629765, 0.5519228553138524, 0.4851828165553397, 0.7180156368998191, 0.6225440577761998, 0.13340733482156686, 0.6298642512561503, 1.0353901206382496, 0.5836010251748012, 0.29193205375368003, 0.42686147296033916, 0.771790787513215, 0.7081035694288524, 0.8462086978488161, 0.23776479876030804, 0.2812968281167306, 0.4371232579905411, 0.04109540329639097, 1.0248256548231478, 0.49167131814510356, 0.3155477576815477, 0.24164934535407134, 0.5311228481060168, 0.8123997366058581, 0.411659774083642, 0.31443236810405933, 0.5899673118258778, 1.084534177662766, 1.190527246376388, 1.0072322569813053]], [[0.6004577986041986, 0.6157287860151106, 0.28321615722704724, 0.5397002138074366, 0.6423914311816804, 0.8740574120896778, 0.5386771988640425, 0.4595604553120497, 0.6906174997359287, 0.6897046599465071, 0.12894245907055615, 0.6203691974768684, 0.9442743387293386, 0.5971256512521987, 0.28991745705030325, 0.4409330904962849, 0.7713253146787908, 0.6979104691047436, 0.8493065599130394, 0.2370106022731997, 0.286917664808451, 0.4073488011130557, 0.04004114951732202, 1.0727731564704324, 0.5217467087963401, 0.32136741320792744, 0.25073256944920297, 0.5138858547397279, 0.8090957328955797, 0.39690195341595425, 0.21631068746528392, 0.657561834987453, 1.0750268085347132, 1.3633232269979243, 0.9521211870825814]], [[0.6533996062995202, 0.74331794102035, 0.28672231347632154, 0.5423698571702532, 0.6444874746690501, 0.9404064143463581, 0.5600366386349139, 0.5072129392882342, 0.7196625249592044, 0.7021150349710442, 0.10706800728865767, 0.6232377336342848, 0.8337047282582467, 0.620363982356753, 0.289099252037563, 0.4482347909476968, 0.7387524621681248, 0.7258282326061016, 0.8974394613999293, 0.20840146779614732, 0.2561840111365997, 0.41893472752070304, 0.04075706037868271, 1.1111380173445018, 0.48867894087850083, 0.33576519221675083, 0.2281658215950322, 0.4211403498757439, 0.8147754552359451, 0.4619162385575486, 0.17231491550201855, 0.7300415094018333, 1.0593123906096644, 1.3811804074226388, 0.841832286078436]], [[0.6907656836093189, 0.6068701784971112, 0.276723004492499, 0.5789419119464421, 0.6186721273641531, 0.9071240081907973, 0.5767597817060236, 0.765139153766331, 0.7166935674735331, 0.6858416707959958, 0.12836426384639715, 0.5871083474491918, 0.7092975920476041, 0.6972226019176847, 0.3022687107456357, 0.42790812236529935, 0.7053257571918967, 0.739068780770676, 0.8720530503752972, 0.19575127980114237, 0.34349889761897817, 0.4582551935878847, 0.04000320676737827, 1.1099308807568478, 0.5443838521525256, 0.37265711363776677, 0.23381156438935044, 0.3210378611270421, 0.8121186097985095, 0.5253796008120954, 0.24006902041946332, 0.795983084732796, 1.0142359094347906, 1.5474142614685782, 0.7668669535127057]], [[0.8056949111039964, 0.5682801905041983, 0.28996089310838874, 0.6068813604148162, 0.6046605580754636, 0.9345786937971774, 0.5857296337245306, 0.8498667588455561, 0.7392729416885753, 0.6732290778572245, 0.10057609343290325, 0.6212337283635433, 0.651146057629599, 0.8003271496700279, 0.29966829748493023, 0.42537258863144534, 0.6898989532553755, 0.7566517514619979, 0.8537386087693846, 0.1978579129246485, 0.2768164069266708, 0.4513892876643518, 0.033157941659840484, 1.0762052918489347, 0.5729613345785229, 0.42877587366899017, 0.25727049305018496, 0.26529502376417896, 0.8059103982103729, 0.6107933637308297, 0.31794082023269127, 0.7882695288037571, 0.9769499787615201, 1.6220937998037304, 0.7498560040722521]], [[0.8719979325235274, 0.5488162993901873, 0.31677160801220844, 0.8090948271866518, 0.5987570658335941, 0.9629226382539726, 0.6437778946418891, 0.9914126831694904, 0.7467935127191926, 0.6991289404504756, 0.08446090334122022, 0.644363936528769, 0.6743371674241989, 0.7437465338957989, 0.32377615936877624, 0.42568282684739145, 0.6603005327222013, 0.7682221079777551, 0.881847358314107, 0.20828588307979967, 0.24342560064683966, 0.3842714441956758, 0.03345909109836431, 1.0567166448143654, 0.5250380016933454, 0.4597191331774046, 0.2431817365936349, 0.23271455709777722, 0.7903062603288168, 0.5863238187976418, 0.39505957652660184, 0.7510278214725228, 0.9394806696826841, 1.7494239008388615, 0.7536865020890005]], [[0.8996842569553727, 0.5275225766224834, 0.28312766281339763, 1.0695632556029477, 0.620096250387425, 1.0006876558591389, 0.6737717353106756, 0.8872691831223074, 0.8422792307284952, 0.6844719593555143, 0.03194361217791621, 0.6443350954617351, 0.7097122814493754, 0.7570999671759171, 0.3149038763144276, 0.4261033469536709, 0.7006120755289834, 0.7481538653259747, 0.8574366189272385, 0.26728141358637114, 0.23306859350825432, 0.3292806139567526, 0.0337632758159354, 1.0093285957776041, 0.5135142676047943, 0.42633779955864487, 0.22006126555873, 0.27506774966296954, 0.7459174690424636, 0.5665220732906907, 0.3587156390243674, 0.7543133740173578, 0.8432234316171554, 1.7912525622482125, 0.7485084202945393]], [[0.9057265705088211, 0.5626564684794143, 0.2802205254009098, 1.238203558053308, 0.6737620996836833, 1.0171003192951482, 0.6687557303669978, 0.8599520353920812, 0.8853260672405366, 0.6342548501238902, 0.0336025351367193, 0.6527182353037131, 0.6501694487123129, 0.6733931587192288, 0.2939699405886288, 0.4230376751724032, 0.7089615381360495, 0.7581674924808413, 0.8268033047326102, 0.24954616289782178, 0.19480327652394042, 0.31356832664276946, 0.037435444020179856, 0.9918747718184384, 0.5004880589959383, 0.3936690746767722, 0.2268276903079514, 0.269337515425091, 0.6813002368850727, 0.5180886612017533, 0.24745734966164473, 0.743117876536391, 0.7630151401060976, 1.7672490571127915, 0.7546963084993483]], [[0.8623759101067416, 0.5216859541683447, 0.3190806498282527, 1.4889322041163122, 0.7003821128484116, 0.9732631348651613, 0.6436310425382896, 1.0617856505678742, 0.8446593773963484, 0.8694837398240746, 0.0729533900973692, 0.6567069519963354, 0.6637281761692557, 0.664548675620617, 0.299016129734597, 0.4472286658786703, 0.7056081563486636, 0.7969141307334736, 0.8322921913964929, 0.27550375568907925, 0.2077247355849598, 0.3127883081593209, 0.036843598292293084, 0.9555573875686163, 0.48027510567288867, 0.3745840136440909, 0.24305099917461973, 0.27853609455879674, 0.6049012832279561, 0.514862909874684, 0.18921130037999115, 0.7467456884862405, 0.7557007967538627, 1.709941300964768, 0.733277692456207]], [[0.8917130377045477, 0.5601274051919989, 0.3062330922796306, 1.5767483351166047, 0.7021481843416207, 0.9903887230483273, 0.6337862928302985, 1.0318704339690976, 0.8443243251335179, 1.0699879891365522, 0.05131987440653037, 0.6495220390263836, 0.707007935059919, 0.5876847473346657, 0.28998423315753696, 0.471530199556359, 0.7854194878193022, 0.7769039464137747, 0.862543727717197, 0.28474152393115304, 0.20350711499214824, 0.30360156502796976, 0.03693134472694029, 0.8783113624185521, 0.4667788278681049, 0.36456953837547973, 0.2634468546907895, 0.30730320378089937, 0.5164920836067974, 0.49607629816197224, 0.18507004845019448, 0.681708151795988, 0.7439291167024089, 1.6985413203492699, 0.7158640117560058]], [[0.8556805573497768, 0.6186339207845155, 0.3143848565991784, 1.680667711219458, 0.6712238198197407, 0.9593701822503291, 0.6180712757589659, 1.050407314162885, 1.034963691783815, 1.21897553719154, 0.039851556943730354, 0.6352509923602326, 0.7908887014171929, 0.5603370787774993, 0.28402392963862244, 0.47562333013778224, 0.8062409662798908, 0.7775419310109886, 0.8548671944427587, 0.28616937066343007, 0.19420261523722254, 0.29987971227575344, 0.039514537863697116, 0.8075994754636603, 0.4915433635845122, 0.37868772872391976, 0.2254633587964331, 0.3564125519644958, 0.4604832754288587, 0.47295858152702325, 0.21626199274312707, 0.6715194819735951, 0.7972105119476245, 1.8723202661080214, 0.7083443072164978]], [[0.9512272228432476, 0.6594379943930951, 0.33618268241589055, 1.828130706959458, 0.677249724759305, 0.960444242396426, 0.6412394234693736, 0.9511539915070844, 1.0153037032918575, 1.3782566268042578, 0.013249064186832474, 0.6694699211723946, 0.7905330632852035, 0.5814848726514256, 0.30486739526949336, 0.4920401453461495, 0.8830313622936616, 0.8346928891113679, 0.7782920272577843, 0.30707949180262656, 0.18665584910554273, 0.3208793588168912, 0.043449322169778204, 0.7524807855248621, 0.4871358693505158, 0.4155486970871725, 0.23081685680502856, 0.4085677657735357, 0.4386400721465543, 0.46548591839403053, 0.2845598376304544, 0.7228837530365841, 0.8957963567698581, 1.7080961483324093, 0.7570532683930749]], [[0.9038772899606831, 0.6820033176605171, 0.3118320519082712, 1.7820750479772585, 0.8143814487674184, 0.8679402425106775, 0.6038254387081698, 0.6955427706021791, 1.031620716338086, 1.4526787008427333, 0.061842217968528235, 0.6417677912726111, 0.7184122533120394, 0.7128322663728075, 0.29956169339532895, 0.46914361269941407, 0.8796353488151978, 0.8046671760569539, 0.7010824319565702, 0.25243545334677486, 0.26908994613632464, 0.3253720133186096, 0.04650093370042825, 0.7143600303350361, 0.4817874983880136, 0.45964301497568955, 0.25645756154109745, 0.4105406606314876, 0.42861664047008574, 0.44311528058622185, 0.42295800340606793, 0.7653044677199897, 0.9061790600471102, 1.5915546078764393, 0.6843871241003804]], [[0.889651231050269, 0.665122991323962, 0.29284045813244036, 1.7054939684190207, 0.7816174929156408, 0.8429519285274152, 0.5996765823201217, 0.5914921572914553, 1.0796266073260588, 1.3273167381930058, 0.1447483775951589, 0.7216905066546399, 0.4946756378916002, 0.6971743570937148, 0.30927861716684013, 0.46009864189329486, 0.8907749465822015, 0.7911671021232585, 0.8259368194634402, 0.2887153848191478, 0.31493961347083543, 0.3948914936253995, 0.0530860120161645, 0.6850930284997859, 0.4433365388465137, 0.39807987456913785, 0.35416867810478064, 0.427730770587549, 0.449955598950443, 0.4269108775552151, 0.5012054802229107, 0.7885960103116707, 0.9315446951785153, 1.560935724796859, 0.6662375018035285]], [[0.8947129669625696, 0.7051674460755448, 0.28493876776135146, 1.617045507824482, 0.7520711165611105, 0.7756439916851198, 0.5585682680319672, 0.6373082199565735, 1.1583695431880412, 1.1416835982955473, 0.2616342595334112, 0.7256157730712977, 0.4043317854453055, 0.6795140508206784, 0.32084468783296566, 0.497036566663036, 0.8907085055496315, 0.8111690742963948, 0.8565128663389943, 0.2902041916348088, 0.3302466573993089, 0.3819122717048751, 0.11157460172108473, 0.7050109851430395, 0.4316455186881307, 0.32921447175100865, 0.4368124637939077, 0.425812580847654, 0.4531615680400646, 0.5038813837547014, 0.5272187878185784, 0.8212157085300898, 0.936478764856, 1.62108100002406, 0.6682224682214027]], [[0.8909418207235424, 0.7208487177409454, 0.2861023388954503, 1.6110381711196984, 0.7245423184714602, 0.7832737584489419, 0.5560170906279348, 0.7520463661494527, 1.2392723458034902, 0.9807274022274947, 0.2578352824620457, 0.6380786938194446, 0.47748254377342514, 0.7227236008501601, 0.3319489467543786, 0.5007407111139329, 0.8842199755669768, 0.7874330437826103, 0.844530512568031, 0.2940545561411254, 0.3598608980482581, 0.3841895097279907, 0.036776235558464695, 0.7705364515954447, 0.42545325228587777, 0.32756129563345954, 0.412937702124058, 0.4568295159288366, 0.45025073283979916, 0.5249617589761824, 0.5413815844328239, 0.8420289018657606, 0.9274924008546823, 1.676612847009953, 0.6547905140553449]], [[0.9544814111844084, 0.6548907533112038, 0.30887467190157647, 1.6335758579158421, 0.7090939390623873, 0.6687415926153171, 0.46244539515250455, 0.731575079930047, 1.2511196952124994, 0.9719839331468904, 0.31256435887550527, 0.6045473967553637, 0.5703131895821097, 0.6411622631099506, 0.3134238397425265, 0.4429395312019079, 0.788748471804719, 0.7950254648409972, 0.974085954158428, 0.25509553701001797, 0.3638850149748033, 0.3436763422025442, 0.020814222403145705, 0.7669869206627773, 0.3791023147014505, 0.4481323844949176, 0.44740341401995654, 0.47152371962743433, 0.578419819427507, 0.5773174585302395, 0.5434854503363283, 0.8346205270664704, 0.9104145243943378, 1.7419832092404268, 0.5971505002776315]], [[0.9201240617013261, 0.7257594171411541, 0.28543134147071564, 1.5546357506353732, 0.6938322950239898, 0.649173436767768, 0.4501158200607373, 0.7338906621609795, 0.9936281577766712, 0.9536612913912108, 0.3070994731104763, 0.6365649499035488, 0.6377993424115773, 0.6369946036069063, 0.30236137486759573, 0.49023173288676014, 0.8203502550791368, 0.7106138379487159, 1.0382202093649315, 0.2009150372098017, 0.31316719732859, 0.33341492255496674, 0.019076559517908077, 0.7177782993976827, 0.43193879106914956, 0.48666166480701356, 0.4332726713067565, 0.5506600789157882, 0.7079683161538075, 0.5971218220896056, 0.5075270672053162, 0.7835292073163883, 0.9198657899897869, 1.5774874019061607, 0.5078946801605732]], [[0.9226830490103752, 0.7786479499072639, 0.282282409569461, 1.4890497509654081, 0.693315550253417, 0.6255227508428622, 0.4512717991686612, 0.7431761679144493, 0.8029496235666717, 0.9898779061899601, 0.26077505757521735, 0.51471048882517, 0.6235697704352358, 0.617666037718939, 0.2902704838406398, 0.46958113740233387, 0.7906125442621641, 0.7607080765307481, 1.0529336521482136, 0.23937492222023155, 0.3445563043273814, 0.3937056291343037, 0.023522025678667534, 0.6390655614672907, 0.38753101227123604, 0.43472555929669665, 0.4268096375918775, 0.7120434347356777, 0.6820340956902405, 0.5280754248020976, 0.4665814861648343, 0.7150549636800714, 0.9107018100632012, 1.3474787240919963, 0.4654886706483423]], [[0.871321377484301, 0.776371919442556, 0.31908898370559086, 1.4525395226484623, 0.727039891199382, 0.6083723356487709, 0.4235022084376999, 0.7270891741378144, 0.7564010883834726, 1.0064060549819045, 0.22868357375610693, 0.5087099621818361, 0.6320726114378872, 0.5886579427928091, 0.30684189725728506, 0.4645482997497298, 0.7827067815798273, 0.7464361993390214, 1.1251004571851428, 0.23007279244979817, 0.3468011353469779, 0.44521603124033415, 0.024003326357840686, 0.5142881629159954, 0.4435873964651474, 0.2793866286916048, 0.4347656592102629, 0.7106531931199116, 0.6452255220611853, 0.4883996024144755, 0.4637981656299367, 0.5902832628445901, 0.6449149229984261, 1.3620354231482743, 0.48500654990822134]], [[0.8619673574886921, 0.7289183371097125, 0.36059756827264866, 1.4604413906857785, 0.7489759689839575, 0.6136907706984411, 0.44075016377693665, 0.7392608453628854, 0.8062931835598401, 0.9297961753477761, 0.21296616218509856, 0.4806200312346577, 0.674826018252632, 0.5785711887927331, 0.3065904357563497, 0.4567418520138081, 0.7308126993217462, 0.766608323093112, 1.1971901266308091, 0.27177284304705795, 0.37412734738420245, 0.497627738339392, 0.02416742792484902, 0.3517374807075986, 0.49148054971386157, 0.20754480094794725, 0.4255878170388145, 0.7659673173256473, 0.6922132797950811, 0.4746560817746045, 0.48239379491389467, 0.5783841722498522, 0.4708900474509657, 1.314579198915951, 0.5090025118270299]], [[0.8648122261737377, 0.7170384331646296, 0.3163343519096743, 1.4595093620161428, 0.7270936354897279, 0.613101349448949, 0.4191893333989237, 0.7387449542521103, 0.8600255769700897, 0.9042200644186424, 0.21408032532821297, 0.4878947636953992, 0.6272771018197634, 0.5764451718624708, 0.29410165984861847, 0.4422248790172618, 0.704583253274589, 0.7398081833264203, 1.1400408590487534, 0.20118629533936977, 0.4233426872814421, 0.5616360319895866, 0.0233766169686784, 0.3990577043246184, 0.5382725902621497, 0.19636533544945567, 0.42539916456715915, 0.8967990326952306, 0.6675410650244754, 0.4239141733648624, 0.5448507681548327, 0.601439398501448, 0.3524214974047296, 1.3548932613432623, 0.5606901344734065]], [[0.9345819271545064, 0.724384970938448, 0.306716303868437, 1.3627399519720802, 0.7531647768577864, 0.6102835199929302, 0.4138344200531928, 0.7924809366683736, 0.8721301795047737, 0.9907867924456814, 0.21938375921448405, 0.6414420642208496, 0.6459075912612173, 0.6565788922371429, 0.31778247014854416, 0.564889584182142, 0.714554200758301, 0.723266931394579, 1.02021085632832, 0.14549560238805836, 0.31925849251888827, 0.44161991553343505, 0.02405342553862423, 0.4482124756358171, 0.606567103989307, 0.20605275304722404, 0.4654744911294413, 0.8280902936073491, 0.6510720552122937, 0.3555178553994789, 0.5668207096955191, 0.6108277180552868, 0.2842575382323338, 1.286553823488514, 0.6298804980458423]], [[0.9659757279776093, 0.702747236941871, 0.3411878093884132, 1.3140710066857764, 0.8858866984249463, 0.6250353423529467, 0.4114774692488459, 0.7331695757074334, 0.8820579222252017, 0.9921031443708551, 0.21784689341959151, 0.7523701050674452, 0.6862186958936023, 0.6033630608942935, 0.32598086705131135, 0.5790501411331394, 0.5967055470860674, 0.6418773818208126, 1.0772777525763404, 0.11398998367965124, 0.27228606855761006, 0.5823841712077638, 0.028784330520543248, 0.4311445000601673, 0.6133139340698331, 0.1759931792325145, 0.4959650221916516, 0.8037595204576591, 0.6759527192149849, 0.329493601299179, 0.5883683968236759, 0.6214710134523938, 0.3048378361630699, 1.3485373284453628, 0.6593183308440639]], [[0.9260723789580778, 0.7335066349560933, 0.34850270845749587, 1.2124750317388615, 0.9534751905613956, 0.6005248975131554, 0.40023891117099836, 0.7470708144550523, 0.840096364955064, 0.9961068826710976, 0.2129790770560158, 0.6495978279538339, 0.8088600882733492, 0.6434416373841731, 0.3295212283915061, 0.481725013618425, 0.5769267484578939, 0.6236579422725359, 0.9712322663503061, 0.14886890617045392, 0.2344361387694564, 0.5721643389323221, 0.028408073119182434, 0.43874276581067906, 0.6409534861472894, 0.15117907586776602, 0.5173995345075675, 0.798870867790715, 0.7569383824782379, 0.3586237392100988, 0.7368534917215219, 0.649820614516667, 0.3862945549244902, 1.3590761549507002, 0.6947727079094517]], [[0.9399856966841774, 0.7019497870818546, 0.31895622277996877, 1.3156176508256123, 0.8654491908002002, 0.6815034492959521, 0.4456973316467158, 0.6725732164845254, 0.7911512038292925, 0.9449477786932767, 0.20956638441222675, 0.5990437440035595, 0.9057878386005638, 0.6418713569685834, 0.4035152854805486, 0.5227184615351868, 0.5976202319524557, 0.6360326555307285, 0.9299874422393256, 0.19327928011131942, 0.23134340233806502, 0.5841869859277519, 0.028161741367797245, 0.4750070980718268, 0.7213607902206749, 0.14020065240578572, 0.4687547544383435, 0.8654877061845891, 0.869597317273582, 0.42310569692198274, 0.9750692405746824, 0.6632261930526383, 0.4612478295098843, 1.369254401845702, 0.6731598563729291]], [[0.97226464365375, 0.6792090362489045, 0.3300346190598108, 1.3574413196901463, 0.7348637186824204, 0.7519519848766041, 0.4626511702059347, 0.6601248084776614, 0.709872923466977, 0.8759594904444349, 0.20997580082540088, 0.6380254041923293, 1.0145000777242936, 0.644707796401089, 0.3621307205912213, 0.544393005630181, 0.6040811276019368, 0.6490194648072611, 0.8789824372982858, 0.16570883220586582, 0.24044431915986703, 0.5280885252526997, 0.027136916621652206, 0.5557306362307901, 0.644048945957417, 0.1403263247449795, 0.4600364513214095, 0.9437526896730624, 1.0082087268213096, 0.4599021810564813, 1.1912208912715267, 0.6638693944313072, 0.5590778585661939, 1.2630743539132039, 0.6542771460815338]], [[1.0667319384329492, 0.7310765850644161, 0.3413634880152502, 1.3869367672832105, 0.7230117216945227, 0.8102594225266028, 0.4811921485349691, 0.7130033745748037, 0.6512753831109301, 0.8709608106640199, 0.21241539019305644, 0.6178928407190478, 1.0715963859814903, 0.6645449465606793, 0.36107673866548085, 0.5733421108781523, 0.5582674669577132, 0.6587950397525721, 0.9002108865024068, 0.16354212405081728, 0.23305796806902507, 0.4016620384089616, 0.03101380424157348, 0.6109898624871413, 0.6243647656049081, 0.12018716098081866, 0.44354921473849535, 0.9864784319037307, 0.9716084588492553, 0.4853280968732882, 1.3538748124444695, 0.6729531703115593, 0.5953064229356115, 0.9961556079984315, 0.6667139861409369]], [[1.1787487655504747, 0.7420016383370788, 0.3327951677846641, 1.3414450080398708, 0.7481872489631243, 0.7900437922534993, 0.4509333342354086, 0.7449486344242304, 0.5982352921035383, 0.8490356065980754, 0.21557533016741065, 0.6278587543943364, 1.0910416133635144, 0.7448828895079949, 0.35894434909001516, 0.5937643142931615, 0.5454841599399627, 0.6704593693352634, 0.9142680973511582, 0.1648864936561242, 0.2251293038593716, 0.3308428695119042, 0.03093391755403603, 0.5845341469341682, 0.653321122376008, 0.11283722726579234, 0.4254089294253933, 1.002313346458951, 0.8943974089033486, 0.5202770383068629, 1.3330971023438134, 0.7280247669580402, 0.6094088118726411, 0.9435377644743979, 0.6385884452806961]], [[1.506425703791719, 0.7345583482381426, 0.33106733238316677, 1.25435820977024, 0.7780657571615981, 0.7765977125691058, 0.43963411081613446, 0.7992218727030389, 0.5858556097760457, 0.864236663302349, 0.2203099809033045, 0.7052724275288329, 0.9644410683361161, 0.9016221259152053, 0.3766316363909704, 0.6016300373508177, 0.6510207359286628, 0.5963529017056359, 0.9251731053183601, 0.15880408317728656, 0.22980936098260754, 0.29679105704426323, 0.03054897181685554, 0.5460732208074768, 0.879906279530881, 0.11334843914295997, 0.5367939356764129, 1.018173608198429, 0.920514284898946, 0.5733107101111947, 1.3029212079785355, 0.7390352947765773, 0.5873235942036923, 0.7585302768462832, 0.6691268147279024]], [[1.405371154362999, 0.6875050367098009, 0.4415931473740903, 1.1162731500261232, 0.7718128712865954, 0.6506785109274296, 0.40871996609345873, 0.7350333695792163, 0.5472527200216927, 0.8613216580475319, 0.21777322351606843, 0.653111335764025, 0.924623806733552, 0.9284115577322962, 0.3603961931705021, 0.6159464676466111, 0.6544167388647967, 0.6084725311750154, 0.8803636523506095, 0.15747400451656218, 0.2240503997285429, 0.3030565760343636, 0.02958943199429094, 0.4757075619956376, 0.9335997870770452, 0.09700515744567878, 0.5883367535798032, 1.022145319550695, 0.9518112204872333, 0.6073910984512989, 1.274273995737895, 0.7678162817562375, 0.586178835995524, 0.589708892466993, 0.7117201219339091]], [[1.1664933909937738, 0.6819268531616023, 0.3129578577709752, 0.9623889780907904, 0.7153435974862812, 0.6661053140931928, 0.37326049948042883, 0.6689109349951052, 0.5797492227337534, 0.8534964236610914, 0.21400165038535418, 0.6711199280827223, 1.0102450960147653, 0.8163877381215473, 0.350473378559333, 0.5680829802571487, 0.6070369703165912, 0.6116419347364812, 0.8948596467008192, 0.15685482797033745, 0.23509859260783786, 0.28989644964961997, 0.05013173941817216, 0.45502548375304364, 0.8329000245443077, 0.08676251939272235, 0.685681284930707, 1.0475571505341068, 0.9918331363919767, 0.6517923834248265, 1.2646137177641648, 0.7849402960712567, 0.5932694163575813, 0.501195329723588, 0.7542502318762445]], [[1.1122338636615712, 0.6541844852617891, 0.31514033819867837, 0.7549999058441105, 0.6618707622076645, 0.6215867332447026, 0.36322068901924065, 0.6740303565519468, 0.5197587041991995, 0.8360013217939204, 0.22090416985587924, 0.6860937053448968, 1.1044344250115132, 0.8266013298098296, 0.3267744004618346, 0.5614810646002399, 0.581047316881633, 0.5611160782089322, 0.9152143852488046, 0.16098132361810863, 0.207271549562513, 0.303577774855055, 0.06445724265654011, 0.4688754763113456, 0.766108111858143, 0.09336198851568776, 0.7555018914596023, 1.0910837787964742, 1.025109481368494, 0.6255850094268992, 1.3203657085604057, 0.8048678787006467, 0.590938480734327, 0.4271976686004761, 0.7195343053037262]], [[1.1330566194324914, 0.6175106482573322, 0.30336566887151994, 0.648090289227814, 0.6774137011466338, 0.6057025222432034, 0.3275527001761458, 0.6982497924700187, 0.4367720348221539, 0.8306398183982855, 0.24094697841961601, 0.6703747980362403, 1.1676893906766779, 0.7629372615409526, 0.3372252700571326, 0.5793836066750065, 0.6014587269537873, 0.6227790829043904, 0.8978235093045293, 0.20002460764180868, 0.19980420024375511, 0.3409595695483968, 0.048062652896399216, 0.4824907360195845, 0.7663128133562469, 0.1349963002782492, 0.7639453626146528, 1.1501633205079629, 1.0802994512798165, 0.6489573972445696, 1.343102619498943, 0.8120209125959714, 0.5401588746928049, 0.48801765731468205, 0.6457555002140083]], [[1.1174418504147499, 0.5908571433253613, 0.34740795642150546, 0.5323116525335325, 0.7345795541732154, 0.5791531142748683, 0.2948562836014893, 0.7107865837248766, 0.43379029816195747, 0.8219805675222774, 0.26898939942056155, 0.6713768161986382, 1.2625565309592741, 0.7651937002793976, 0.3574316177977095, 0.5962166508997798, 0.6149358086842736, 0.6712400679318309, 0.9517078002606807, 0.25357049464988396, 0.17925330013060717, 0.3527086262870816, 0.10422164407138484, 0.5296742969936279, 0.892732898940277, 0.1818349067601775, 0.6466657563484268, 1.2039902756481244, 1.130216720141428, 0.8509543387180708, 1.2689014988467373, 0.7773843221128568, 0.5005811643194887, 0.5893265886406632, 0.5768105459358684]], [[1.1081720372128805, 0.6265128339378058, 0.345110139580069, 0.4928461738018661, 0.8007625221953224, 0.635806393153607, 0.34564610069747553, 0.6871728029248378, 0.4075865930482323, 0.8311489338036191, 0.24195245553082748, 0.6465737482189515, 1.3241932617064736, 0.7311614200321348, 0.3924413998169679, 0.5766534098253028, 0.643445009281426, 0.6493537637590296, 0.8916503695293583, 0.3145371086141593, 0.16380094049855165, 0.3595719046491892, 0.08264938398714283, 0.5215333919954852, 1.0278169740943954, 0.2511200224880368, 0.3752149932582841, 1.2109333419776591, 1.150592918682916, 1.0517186857164809, 1.1373968148190834, 0.6823174653163653, 0.4814386425375695, 0.6320135797393331, 0.5289365544599494]], [[1.1751482426626199, 0.6680300171114041, 0.37643056597623026, 0.4489086815357064, 0.8116375761943446, 0.7175841296832286, 0.4436113940352716, 0.7746638495971825, 0.46269883639519127, 0.820126880562667, 0.22224840321491574, 0.6258596951511177, 1.272148933341798, 0.690981748678497, 0.32748424726710934, 0.5704642938248592, 0.649219650016808, 0.6317630059962629, 0.7597211550709668, 0.2227269028210384, 0.17457953408738214, 0.361722999036634, 0.07921700936021497, 0.5109193036737754, 1.0534047771542738, 0.32620425263970987, 0.34361942072811935, 1.1205035557104088, 1.2404211888160188, 1.2615223340338495, 1.0251282099104717, 0.6010756489675887, 0.5054883433144962, 0.6220490823448632, 0.551238744371891]], [[1.1902932837148974, 0.7008749406763437, 0.44443782902562345, 0.44626866186213354, 0.8095306425878732, 0.6654530300402635, 0.3663333330671741, 0.9266477992045654, 0.5097022263093456, 0.7817417429696796, 0.20605672550217308, 0.6251834706973489, 1.284507690364435, 0.6864934966538869, 0.36092813784259586, 0.5965293516673453, 0.6391299194253824, 0.6049803586806372, 0.7031474971206086, 0.18949672054653005, 0.17703961129000229, 0.33206810657767527, 0.06237118329480905, 0.6371273437666084, 1.0813098185800312, 0.2345148668986147, 0.26731831174291043, 1.0030458559011566, 1.2821618890996285, 1.2441258938353226, 0.8808532333861241, 0.5751141800711345, 0.5186068214357993, 0.5815928889794082, 0.699735179576305]], [[1.0638048677017995, 0.7369551725355379, 0.41960058582313386, 0.6001672863802336, 0.8456236377382281, 0.7454425658425466, 0.42564897807593405, 1.035478258730294, 0.5267566509412327, 0.7288455979224829, 0.18266762349912052, 0.6677320737628546, 1.3141113044703112, 0.7070145955871017, 0.31669389969038564, 0.5936501753875152, 0.5884951527343155, 0.5373722202898972, 0.6269758120778195, 0.2369136175060996, 0.2056542052469526, 0.31438920254372743, 0.057491492309650685, 0.7141022583131638, 1.1139465556692556, 0.21295201448180423, 0.22050472106240102, 0.8360914951519179, 1.0084889583053833, 1.1861385211952227, 0.8066558896595329, 0.6262164173933055, 0.5617738546364047, 0.6143047736479164, 0.809661278469228]], [[1.058246699417647, 0.7575059707503919, 0.40581511564098716, 0.6801971405900299, 0.7957595390435785, 0.7655643837427908, 0.4238190477964452, 1.1201346046063967, 0.5917943845511843, 0.6908838459043469, 0.1742934568962177, 0.6930156969092036, 1.390672494361768, 0.8261220777648295, 0.31906231660869755, 0.6061086490527577, 0.6392992152695233, 0.5612230574941912, 0.5712857907441635, 0.22209467601857927, 0.23276649433672325, 0.3014350312727192, 0.049859169085379906, 0.6678823321271778, 1.0474191177630034, 0.1741704710146054, 0.29740242655645033, 0.7219171302628903, 0.8231209231630207, 1.1064543444886723, 0.7666627476362625, 0.6589147791467675, 0.630505011550267, 0.6432489849731855, 0.8996911129966201]], [[1.0518607803549416, 0.7772542131930795, 0.34256688031657245, 0.7201632147361188, 0.7800438071964019, 0.7404069857429793, 0.3216858363008549, 1.2121069631153722, 0.6370788019774383, 0.672401028233231, 0.24710899063095734, 0.6278405206252783, 1.3932525479398112, 0.7660732045902334, 0.35274538807960243, 0.547090680521972, 0.5443683202933272, 0.5635781961633402, 0.5394442456746282, 0.2128502882911237, 0.24843532843849275, 0.3085099217979578, 0.0685539956522987, 0.6586830567201603, 0.9813473553776295, 0.1844973089387169, 0.2705803295271954, 0.5836387013207438, 0.7672181495726786, 0.9228233311295797, 0.6939582553709781, 0.6523306422051828, 0.6966205076827829, 0.6095744567328618, 0.8688671708295665]], [[1.0510630663212235, 0.772693229801469, 0.34081585852929386, 0.8169192039518687, 0.7799465217839593, 0.7698082402298642, 0.33109474890864166, 1.1958901564948257, 0.5879603304571295, 0.6829760317706409, 0.42100408477168594, 0.5951914707564933, 1.3358993530660639, 0.8045255467704695, 0.31861860248375845, 0.4941729248658826, 0.5615661442252348, 0.5339067377918474, 0.556606601797721, 0.15747607742137626, 0.3167656776887658, 0.3369872925265882, 0.07847486411565821, 0.5820709159683191, 0.9345965152765293, 0.1893119381473386, 0.2439760920521749, 0.476110885436615, 0.7316021237120786, 0.8662398094488661, 0.6255634539097827, 0.643551867808716, 0.7606732651130284, 0.5565123611984781, 0.8026188849448297]], [[1.027724518134392, 0.7605327661154558, 0.3366218040513853, 0.8020062353501431, 0.7575452821487005, 0.7547492222969816, 0.3586706444143804, 1.188246232668641, 0.5583905369224909, 0.6468308086253329, 0.44066806012735876, 0.5845638718992805, 1.235563116233049, 0.8498855265148169, 0.3676311734972729, 0.4880023089327148, 0.5447445314608235, 0.5250056485656185, 0.576880823320994, 0.12683332306683553, 0.2748996824257319, 0.3542178013177033, 0.07484105267035868, 0.4645080208364864, 0.9269059388817598, 0.22495344844944126, 0.24517416621681315, 0.4149134039498939, 0.7200460966019723, 0.7557051937190447, 0.608870640806467, 0.624983649887972, 0.7864372237616248, 0.49451962302209584, 0.7796742420813487]], [[1.014969669132896, 0.7660028605353775, 0.35162759947563194, 0.7862763595655549, 0.742803845812309, 0.7627667413674413, 0.3684151087408927, 1.1642442752926365, 0.6088139851155301, 0.644307682935747, 0.417745046427033, 0.5889034935326002, 1.1067979588904422, 0.9252367940023857, 0.3653778945953883, 0.4893631465054231, 0.6064878950280062, 0.5317347241956805, 0.6016105067312486, 0.1515669367688162, 0.2710882869305284, 0.38889490003101673, 0.07761482208168086, 0.3571236469206911, 0.9748178182727454, 0.269706407697716, 0.24850664166146963, 0.4748057853466606, 0.7097993564129551, 0.6400764228416479, 0.5808188263392087, 0.600894955706722, 0.7623422119345411, 0.45482090587262586, 0.7665575579930485]], [[0.9861342028360761, 0.7785323068931012, 0.2973315640124372, 0.7018944372507192, 0.7331037314294897, 0.7479479480722747, 0.3244840632905912, 1.091853216709474, 0.7311823549038627, 0.6086966758681892, 0.27493467784527714, 0.5841235959819343, 1.0582361505463262, 0.9651680680152455, 0.3868297537310614, 0.5055464686717591, 0.6862946021769827, 0.5531052048827323, 0.5933510588710857, 0.1534088467880563, 0.21497660510755426, 0.4691297237749975, 0.12650917094897396, 0.3487006578817461, 0.9605817260066924, 0.31315228318142346, 0.2699793641116081, 0.6129146943911572, 0.7471064298664727, 0.5951481615791532, 0.6298262152600598, 0.5770230225998809, 0.7471308839209918, 0.4342217422184362, 0.7716941984618988]], [[0.9365553173176511, 0.8082816927678285, 0.3090510786396658, 0.7444819132978622, 0.747285608362178, 0.7376467267002867, 0.29631814015361513, 0.99235562584919, 0.8437991657964738, 0.5585596854842303, 0.15895338520597907, 0.683153112970117, 1.1317963861470484, 0.9556521578133799, 0.3688066182918275, 0.519038845193634, 0.7620235031269311, 0.5184776009320982, 0.6228717844724492, 0.22131878549636008, 0.2514310621136651, 0.6243738064730258, 0.12376399108027586, 0.36512019349897984, 0.8835599438594547, 0.37123242857505484, 0.31073265972148734, 0.726534452679102, 0.7692265030823601, 0.5912081100625028, 0.7059992039692361, 0.616824041152997, 0.7691999214936776, 0.403845842184258, 0.7875487178979446]], [[0.9499475815707165, 0.7914286178323653, 0.29298004909458536, 1.0193079812317527, 0.8079000862926577, 0.7835701637758967, 0.34219957344485197, 0.9255487046803128, 0.9405401431946437, 0.5198232714344987, 0.12157349457097522, 0.6986512033082021, 1.0573609224701104, 0.9073046833133412, 0.37744451262610695, 0.5406818423585567, 0.8514553355606118, 0.5010012287226834, 0.8156862577687864, 0.20336003806556716, 0.1854386786632058, 0.5809132559611112, 0.11856911738911477, 0.3762083662850314, 0.8274865011966059, 0.392106349176375, 0.5042731580316574, 0.7782494965359127, 0.7906245901012768, 0.5992209248994701, 0.7875651440931126, 0.6738225770236403, 0.9052513411239101, 0.4068769016683646, 0.7572259275694265]], [[0.9535827235243434, 0.7837796991246091, 0.3106814402191237, 1.1413827775045706, 0.9113451187629589, 0.7628414966314785, 0.30866526988195314, 0.8788840845748351, 0.9404431457353609, 0.5833507989933251, 0.10303694205900717, 0.5735340024661424, 0.9238195659526314, 0.9195298659789093, 0.3792568707324717, 0.4650820171448786, 0.8063121242818103, 0.5009022466174256, 0.8849211564749224, 0.13751059131500598, 0.18944517385472637, 0.5162609761526982, 0.11489284740766792, 0.37905143468190633, 0.7357208696963737, 0.31416679696159433, 0.5553377449534929, 0.7773679185346668, 0.8261550274037953, 0.6387381237851787, 0.8396572981221446, 0.6970293685230642, 1.0309387564286228, 0.4477762462050134, 0.698697390922286]], [[0.9636343138455876, 0.7743460924955377, 0.2882460206273243, 1.2424611122221, 1.0247465402374507, 0.7770894052551964, 0.33337120207657595, 0.8683378467218378, 0.8294414986891114, 0.6152524327407978, 0.10628833526993431, 0.6878545324558801, 0.9203472651781851, 0.911409412171715, 0.38765312210127234, 0.46880615126282454, 0.8183852452123314, 0.5185548760644632, 0.8644405707335516, 0.11186113895625865, 0.15656279544412144, 0.46177114376115874, 0.10806969901780378, 0.40278302139387184, 0.7028690278308021, 0.2759470859042912, 0.654314829802779, 0.7181954095518334, 0.8502235257709284, 0.6834557982624246, 0.787424139515404, 0.727704157337663, 1.1084280213787037, 0.4935881328306736, 0.6358494461208659]], [[0.9572163136418801, 0.7520201384065117, 0.2624255233209812, 1.0923540249394221, 1.0272013996558098, 0.8091283708752898, 0.3594344238010175, 0.8952984332898378, 0.8266593938540757, 0.6484827964763196, 0.11229968386280537, 0.7565109597923575, 0.8838381929593244, 0.9252159468144946, 0.3933291591643386, 0.4991848931116999, 0.7541610457613891, 0.5276630787162826, 0.9031800531703051, 0.13481119640713213, 0.13408684594591885, 0.4000419768332279, 0.0991204678783118, 0.4029982002037224, 0.6115094052982826, 0.27756416305861964, 0.6480853501035138, 0.6324882408284285, 0.8616442457317474, 0.7212518639628154, 0.7322031420766745, 0.6954512966709034, 1.0961414568571142, 0.4939841608501301, 0.6238342567987867]], [[0.8666159000190489, 0.7573834156872066, 0.2570559094835071, 1.1188785359415563, 0.9909442151192216, 0.8416048959773107, 0.3999627174947545, 0.9362961473405544, 0.8553133566297949, 0.6613832410280701, 0.118073759699579, 0.7280658437381543, 0.8186612523601204, 0.8901761696438067, 0.39553817695406807, 0.4724325064407827, 0.6988143731245959, 0.516520407395963, 0.8275140197155926, 0.16161642758255124, 0.14920070587995, 0.3724247052979732, 0.09550844075194477, 0.38940882203956595, 0.5519794020923328, 0.2622048725050413, 0.7499300891442102, 0.6023896123635765, 0.8773070882574285, 0.719949833339069, 0.6287698494539947, 0.6287694049357104, 1.1033644167308054, 0.49444408756518066, 0.6232601185307465]], [[0.8624929156039495, 0.7686265559813554, 0.24991977352574968, 1.0650609500885777, 0.9921148498990813, 0.8773426286403181, 0.3894072466095685, 0.9426827141100222, 0.8320955487910526, 0.6487120883299484, 0.12436068216522148, 0.6973322484451843, 0.7246212808484616, 0.8631994379219905, 0.41753857510235864, 0.465935071325143, 0.6821876867455509, 0.5239815556039182, 0.8362961341752408, 0.1522222053887986, 0.16410039501151139, 0.3317594370275969, 0.09435154211838259, 0.3854129792396325, 0.5118974374901956, 0.2589320264461824, 0.7656172737609195, 0.5283832536142556, 0.903734303814195, 0.6625423225298055, 0.5889836367942104, 0.7173969770059936, 1.0570050694266246, 0.49797998549682204, 0.641415008093821]], [[0.8646093096159103, 0.7537830056344088, 0.24563472134359338, 1.067047081040078, 0.999363094066101, 0.9369996773570807, 0.37664640423174917, 0.9488654013698731, 0.7728536293809309, 0.6879623294030839, 0.12413095366850474, 0.5612539870095892, 0.6430618246910188, 0.8757033510383483, 0.3833989247614814, 0.5576831326248223, 0.785981113656802, 0.5584790295889844, 0.8524421644574678, 0.17093364787499815, 0.19976019216890653, 0.35072696607337744, 0.0942216243031579, 0.36879439290864835, 0.5426593222942051, 0.25985686850307455, 0.7099812585520097, 0.5179744671069204, 0.9341976154896999, 0.6614891277352253, 0.5842898606017014, 0.7853856829695649, 0.9918706311492028, 0.4790286802655741, 0.6802255590878409]], [[0.8618410362969166, 0.7364520138023133, 0.2530750599525735, 1.0635950826017346, 1.0234067909922484, 0.9266502972898295, 0.38151697051535166, 0.8337651475769194, 0.7358041846927419, 0.7238741733685197, 0.12168309568919819, 0.48525082616897264, 0.5971460840163636, 0.8556248202086496, 0.4167306444388887, 0.6018522952467655, 0.7956102202822125, 0.499050571699883, 0.8868085850569913, 0.2061846770618489, 0.23794667654374707, 0.17095246945971107, 0.10017019415797396, 0.3450537834370079, 0.5337309608729596, 0.24169272139621686, 0.6778199484214235, 0.5588442597944905, 0.9534246395584226, 0.6623688689756067, 0.64176389501197, 0.8155585584616993, 0.9347620606367102, 0.45697972902209816, 0.7138638690902108]], [[0.8679681802918451, 0.7397565541133696, 0.2486350880384276, 1.0722803240055194, 1.023577978521575, 1.1388841081398633, 0.3724469293770786, 0.7658832557693132, 0.75202845759849, 0.6707898058280021, 0.12004261024796042, 0.4872433178702267, 0.47858184552867583, 0.8599365795066969, 0.4104598455910343, 0.47233000153380345, 0.8006103067420022, 0.572232056443725, 0.9259829244530837, 0.20498441664002254, 0.2813431284847342, 0.03219303766808352, 0.10238591511968849, 0.41988648136792445, 0.5352316682771541, 0.2202904636910768, 0.6823393297346108, 0.5349819045366058, 0.9535338221666804, 0.6747024168731743, 0.6428345673486833, 0.7916190683305495, 0.8557190622986021, 0.42899794591913354, 0.662026235794051]], [[0.9056696686497115, 0.7472939186203353, 0.23721044554730691, 0.9957573296506517, 1.0316295845344208, 1.2037378651202175, 0.373710947137196, 0.6897637846316889, 0.9149888354534047, 0.7314119633065769, 0.12081250238085225, 0.5500347789828502, 0.40744639982197506, 0.8751123829537446, 0.4233744358701128, 0.5039038390481302, 0.8158295934805729, 0.6239087924216785, 0.9194323994122772, 0.22195459270632198, 0.26838076726966204, 0.32159731275050624, 0.10407688532526613, 0.4624543159835588, 0.5568767157313104, 0.21042306199864405, 0.6775696703912757, 0.5408933673546202, 0.9559942038873297, 0.7054277178293191, 0.6784398456389714, 0.7731688118169138, 0.7903965747878174, 0.40342870700961525, 0.5857894857697066]], [[0.9489415778185245, 0.7584210437891925, 0.23856792556883222, 0.8067374124652298, 0.9579468316498192, 1.2143543434992852, 0.37401778730741236, 0.38533412737024736, 0.9742487324300229, 0.6996148558716475, 0.11858110045110162, 0.5295290749231135, 0.5407019686447506, 0.9171451328772475, 0.4374093649366002, 0.6127419363677286, 0.8011079563698209, 0.5661344095123686, 0.945298858174579, 0.24916867904834047, 0.27852311088630743, 0.5857060282691793, 0.13753461587415214, 0.434445863563041, 0.5312866476593117, 0.18321326828385318, 0.515241870605547, 0.5356476463089933, 0.9525305330196632, 0.7218582269706697, 0.701431412312943, 0.7552019960607459, 0.7472055778123057, 0.40089852986718244, 0.4966724083626096]], [[0.8633164202097823, 0.7275474020072076, 0.2424643754664767, 0.7649726359767554, 0.8745333335993055, 1.367788782049388, 0.3796206374056231, 0.3560193591369157, 1.0918844130824295, 0.7212556232589927, 0.11787654961329297, 0.5541242362332983, 0.5862010157115546, 0.8877957803414331, 0.41225847536466037, 0.5112776404181312, 0.8234064793226161, 0.6034961030235615, 1.225052582428258, 0.25070560170120476, 0.2935116930205265, 0.7873311956522014, 0.13301568875969097, 0.4539118162257774, 0.46776018002558306, 0.18778685095106135, 0.36966042657795606, 0.5110126816281715, 0.9504565380560474, 0.7200671342397921, 0.6732587293834342, 0.6937387565484501, 0.6561316662498944, 0.32423737531155894, 0.4051900770650395]], [[0.8674823764519324, 0.7241325420499256, 0.23986585402569016, 0.7358035630830309, 0.8082919786171456, 1.3479850965793427, 0.37170242368067674, 0.38094666669780175, 1.125048104480828, 0.6789210156412988, 0.10711730730581098, 0.5929390455313243, 0.8549482458993569, 0.848321952237036, 0.38969727150618116, 0.39552173853908434, 0.7888050447074552, 0.6352854395843688, 1.3369309213332348, 0.23669318208908108, 0.31043783879802356, 0.8551003457562141, 0.1251007202291782, 0.4610210417703462, 0.45682277790671677, 0.2649087407197962, 0.3161348940071753, 0.4626388691941621, 0.9195011642025671, 0.7148224203015074, 0.6718152531969228, 0.631562695705094, 0.5906134043409095, 0.21037775933990893, 0.34361815626643943]], [[0.8651065201283148, 0.7043375733454352, 0.2471089473146979, 0.7506470922394518, 0.7884187938124237, 1.2937861498897911, 0.37168092102620603, 0.36004680782224485, 1.3353505654507694, 0.6736425141504268, 0.10244918691583643, 0.5979238203221349, 1.13107158412241, 0.8204557060195701, 0.3837974998732577, 0.37225289672869916, 0.7627315028925751, 0.6326440469135857, 1.3701918640371757, 0.24192856605252316, 0.32744512007117993, 0.9054105346488192, 0.1339408060522168, 0.4544733480979881, 0.45238814182734743, 0.35736632943757185, 0.3175488937305262, 0.4572210757230336, 0.8714011874770737, 0.6593026843226524, 0.6462961407372829, 0.6255473960572836, 0.5665203809218073, 0.2094739317081568, 0.3853147243113578]], [[0.9013934240090564, 0.7418083191438869, 0.2373946580963341, 0.8074691755312076, 0.7077912946985221, 1.352702205364737, 0.390607674678169, 0.3676751564818539, 1.325081402097413, 0.634019971274513, 0.09015400307188667, 0.5789807654921981, 1.1913009340880731, 0.8805015132479516, 0.3695779631501659, 0.3625848198463565, 0.8440392091425517, 0.7220895011286153, 1.4653698667283073, 0.22699195115341875, 0.3272370873305023, 0.9548757102315099, 0.21452703780596488, 0.4567067515985445, 0.40316564047689907, 0.3665302773092483, 0.3318441490232444, 0.40283900235011016, 0.8409493142456158, 0.6486710091999657, 0.6723575978908307, 0.6299122187673285, 0.5464238533298864, 0.2883815835966383, 0.47191940408392763]], [[1.1032673791529652, 0.80666299815607, 0.2539655656185029, 0.9160621970676772, 0.6643523454348442, 1.1324761720467098, 0.3919505694050352, 0.4537851617091581, 1.1525571306201052, 0.6014693351903022, 0.08779493873003608, 0.6437338095261552, 1.2484241890739138, 0.8152885238781318, 0.3693894713097413, 0.3588961890855284, 0.8575110646153223, 0.7066043146328653, 1.4894463022252662, 0.2199479365953686, 0.32760849585747537, 0.9188701530220121, 0.17130238241364104, 0.4542845077962492, 0.4071785365359471, 0.3921807079242122, 0.40922469438170944, 0.3833839613804196, 0.8186184798980515, 0.6623905845265035, 0.5719548020565511, 0.5596953464133602, 0.45113473773805296, 0.34416610626345434, 0.5806087247717926]], [[1.1640661672268018, 0.7210748879795897, 0.27316671269406256, 1.296709053805958, 0.6845172242616122, 1.1655545163425296, 0.3859360828929735, 0.6522426157697792, 0.9444293600919097, 0.6825308098480467, 0.09580489175027851, 0.6011907912434782, 1.3066953671682324, 0.829590839422778, 0.3501412644124912, 0.354500839994742, 0.822722852355102, 0.6892470815734939, 1.4712545861243012, 0.19827438809034767, 0.31468000474994695, 0.8985384137285182, 0.04776010224828715, 0.49237553105524207, 0.46292216660936425, 0.39614899936663084, 0.4525572419525738, 0.322379330089834, 0.8183199336490478, 0.6650050029367002, 0.5490799735626164, 0.5352838128322805, 0.42706073493145935, 0.40650983774449045, 0.6128479405477785]], [[1.1471317940472, 0.7188057127935241, 0.2593391841400182, 1.5841461810840713, 0.7313788103843732, 1.1357277473931089, 0.3855694028790204, 0.7093865658386571, 0.803224865963159, 0.6320959184619727, 0.12072545172174745, 0.6289977518364193, 1.3348457274441734, 0.816939414661983, 0.3469088286750952, 0.35565609889339106, 0.7647114409723285, 0.6402670204317593, 1.50012738862337, 0.22586233206556672, 0.3947432462769729, 1.0011453715841807, 0.09222250207961552, 0.5556546813890817, 0.5084749628312977, 0.396253987713303, 0.4649171796190984, 0.32068776824712136, 0.8020808420144661, 0.6732848072296438, 0.7189734082194668, 0.5709900070931208, 0.4930669003386818, 0.4273265945476208, 0.7021288786674682]], [[1.151355748445002, 0.7723434101882903, 0.25706866236180853, 1.674367916652407, 0.8197338658255233, 0.9626934903723546, 0.39656219220400535, 0.7240440032383578, 0.7400595027641633, 0.7544100916301865, 0.10388483480093173, 0.670181489031147, 1.3452533025662115, 0.8093795579992337, 0.34931754207725263, 0.34473625569822486, 0.762115283384931, 0.6185665115864416, 1.5778396105487218, 0.23395836254280036, 0.355637583150787, 0.8055440300875949, 0.026344785628411777, 0.5837551434040739, 0.4312677954439851, 0.39771952907596, 0.4208668461084988, 0.2792622719745517, 0.7764100006013239, 0.684497827692535, 0.7484874484045042, 0.5049613180577606, 0.48315074256730456, 0.38451428573381413, 0.41214426238056123]], [[1.1486120356797511, 0.7930341909398158, 0.2967220383941572, 1.8990873747149317, 0.8704008755664814, 0.9958698110115833, 0.4338958838467107, 0.7609368931330558, 0.6836666092994907, 0.9257696497830772, 0.21272709101656612, 0.6746287330991512, 1.3072271169112402, 0.8720184058935305, 0.347185219178739, 0.34955082207862204, 0.8251964714063019, 0.6129672866537166, 1.453153414964607, 0.2682341163940726, 0.29332587187499837, 0.7438863018751276, 0.030023708762373702, 0.5491390766822336, 0.40988560300651444, 0.4882996488919781, 0.42246654151098567, 0.26852633600609777, 0.7320284419991563, 0.6631615937409234, 0.7384258777149975, 0.4335528454590395, 0.49854703190045524, 0.3093688310960825, 0.3438658712775079]], [[1.004783750619049, 0.8060170524605065, 0.2773675659774089, 2.0781123461610145, 0.8893241272298897, 0.9604515034274301, 0.35622776901896747, 0.6796984945566902, 0.6223499175234384, 1.0126559695666466, 0.08755446731882605, 0.6485353390709467, 1.3075675101257607, 0.9028053413466117, 0.34993219599676084, 0.34758845678816286, 1.0716297512812465, 0.6433350364564165, 1.4739431081024685, 0.2959858559857024, 0.28289820759336126, 0.820349583635564, 0.03771493378702045, 0.516403183202565, 0.3840474766342912, 0.6688062905478587, 0.41881569040352984, 0.2604735084594689, 0.6940456701924074, 0.6362394575306778, 0.8183363458708419, 0.37410974716488976, 0.5319377081412532, 0.251740835709309, 0.34250400733923275]], [[0.9107769307323992, 0.7958539023206394, 0.2760504319987331, 2.141184297783974, 0.9131287816604193, 0.9397718880217103, 0.28635686026337204, 0.6038483354980865, 0.5887464022639852, 1.0819606510862436, 0.08466557141489026, 0.6473404758437321, 1.3299587187953692, 0.8946413335976959, 0.3798637144018105, 0.3721610596323276, 1.215642861970776, 0.6955800080324323, 1.438497803811374, 0.32382456294310613, 0.2834442890018561, 0.9289319801851668, 0.032208101883697146, 0.48871634182055945, 0.3394536863573111, 0.7595259210004824, 0.4006064896661051, 0.2687596497656072, 0.7274134411314418, 0.6686358130952326, 0.8299506124598612, 0.3858435894628136, 0.5730093665979248, 0.2648594872811524, 0.4416077386839958]], [[0.8555621639764037, 0.833766055664436, 0.3046518552286949, 2.0161748941827535, 0.9348707298591574, 0.9179635522615393, 0.3624859299520893, 0.4000682571618852, 0.5781220750138678, 0.9047434949972069, 0.06899781487969725, 0.6402306457862336, 1.3306540011038228, 0.8423662139382376, 0.36890814998309324, 0.3654785390138554, 1.0988970305501513, 0.6769558511677009, 1.457222225574335, 0.2974083096288388, 0.27585434252495655, 0.8338412197268668, 0.03921612306684671, 0.5868433283699606, 0.35412679358119303, 0.8347175324589583, 0.3939578457511274, 0.27882503175415646, 0.9058187345117497, 0.7585261459654318, 0.8562723713090827, 0.491950640903294, 0.6189435651276165, 0.3109600228716712, 0.5123373089021983]], [[0.8276063720713331, 0.9188810303856035, 0.2853859663912462, 1.8713401977374362, 0.9346663743495054, 0.8598616776927367, 0.29008646869817706, 0.37059609098208496, 0.6701417392627134, 0.5950546383898602, 0.0442442780610551, 0.6190649220529644, 1.3292285135499196, 0.7909492511816374, 0.38186454888314914, 0.3762617029587042, 0.8510816798582503, 0.7289141620518365, 1.458787673945335, 0.2912543333522364, 0.24065620336900048, 0.7590325023596043, 0.042530312648385075, 0.7316821343553248, 0.3872962783035309, 0.8397180061020988, 0.4668901010217439, 0.29205457265031454, 0.9200133046318708, 0.8435414170736244, 1.0480528843255823, 0.5234996028408592, 0.7212139401792088, 0.349769420344615, 0.5961505395741704]], [[0.8378677829668728, 0.973561746211725, 0.30417999724900785, 1.848544240938614, 0.9902634163500222, 0.921852101564242, 0.37025310982822257, 0.42377091375375986, 0.8038255453358127, 0.6280127363396217, 0.048986789447240324, 0.6325390667234903, 1.2842406328394778, 0.8242928869571253, 0.4158308812304589, 0.418436466849574, 0.787202009073366, 0.7168955010778612, 1.280807797204735, 0.3045551423379649, 0.21847433364933977, 0.7119658347052615, 0.07826394825955482, 0.793781322530084, 0.405535071852998, 0.881966159948794, 0.5138204406024023, 0.30863789762991345, 0.9697325613220158, 0.9181743828129918, 1.0775745067569291, 0.5390215266038588, 0.7735339062608242, 0.3756526996798702, 0.6569113290996528]], [[0.5932601402731805, 0.8863685155780074, 0.3111087275384814, 1.5956048489526145, 0.8497261796986999, 0.9058205930123847, 0.3821806115456672, 0.470711466368963, 0.6451733961651702, 0.5005614995849286, 0.040131884938521274, 0.6285698215218112, 0.7755629496554477, 0.7498894546692787, 0.3819170073358736, 0.4143395173150023, 0.6403187635556238, 0.6296963363923664, 0.9876712046730162, 0.21272994081306976, 0.17759303206425153, 0.41812363755161985, 3.4870886584447467, 0.5173269977600832, 0.39935447856516537, 0.5855070271140396, 0.3668024015344822, 0.3278855351301593, 0.7565980718282264, 0.8044092539610008, 0.6085791272258825, 0.30336947664637876, 0.47142886525212147, 0.2031165178209157, 0.5686946916579474]], [[0.598396066136582, 0.8873030457597384, 0.3067093953816289, 1.5849838288490854, 0.9136409708424653, 0.9204056925975115, 0.40325044865241305, 0.5986861859394699, 0.6476826042831068, 0.5677475141927751, 0.023703233594367382, 0.6008033146193864, 0.7376062318579829, 0.7410591521027187, 0.3802739459016227, 0.43440992352189634, 0.6501250109723976, 0.6520249471854658, 0.9861411843033974, 0.232956524878825, 0.17856985356593047, 0.3948044721130127, 3.4870886584447467, 0.5016868711233167, 0.5634407647652818, 0.6292871329405842, 0.4032582706690026, 0.375263557965339, 0.8217879699119104, 0.8633878158483562, 0.6210982950861185, 0.23392697213214947, 0.4682249096754223, 0.21887441518893777, 0.620022487084093]], [[0.5325244826717473, 0.9213957134383177, 0.31192060212731193, 1.565713077515478, 0.9855178769435575, 0.9657827934875314, 0.3903667526261562, 0.8735031660723571, 0.634864984986693, 0.637422502755437, 0.026046451586913662, 0.6203722181075819, 0.6849302600315506, 0.6690633954480194, 0.38098578537818245, 0.39669544095809894, 0.5086540970594908, 0.6346916900508318, 1.0485797579384961, 0.2333803131336864, 0.17850609574767423, 0.36334663936274875, 3.4870886584447467, 0.4252215224850422, 0.8139791743488413, 0.7371613721906296, 0.46565842567224175, 0.7263896746369576, 0.9874892016268146, 0.8910509995393925, 0.6421029826741234, 0.22200850011882528, 0.4649790604624545, 0.29572742094426047, 0.6658577552605708]], [[0.5569242631348968, 0.8816285008411736, 0.30117388732301453, 1.5920952046062489, 1.0014652890600668, 1.0043994937592529, 0.3918099562511273, 0.9593346554966871, 0.661470106069244, 0.7007786259100097, 0.015531932009899774, 0.6097292395511844, 0.6965137208584997, 0.677983006837719, 0.39898266803354293, 0.4129665130079032, 0.5527248526616272, 0.6777952386364052, 1.0241652132308627, 0.22208545994865442, 0.18639623275676287, 0.3511195344281033, 3.4870886584447467, 0.37722418894730786, 1.1045252578209506, 0.7333246300030729, 0.4367543399323577, 0.999137519477655, 1.0488473275163859, 0.9010847429839174, 0.6463377173519886, 0.2577703826552551, 0.4675578308566414, 0.36881568233006834, 0.6785062296324573]], [[0.5606187574308457, 0.888300430169424, 0.30597473971512723, 1.5848983883532242, 1.0053031537984358, 1.1337842770623627, 0.4060251559865023, 0.99777266538906, 0.7202935950109631, 0.762081923466055, 0.039275280021331724, 0.6683469756168514, 0.7374071014624748, 0.6821622337120443, 0.40972724945421707, 0.4533171354098037, 0.5547814923428354, 0.8140009673675379, 1.026534438465608, 0.23782739845950812, 0.20568641174576113, 0.394477932621315, 3.4870886584447467, 0.3362040349076928, 1.2267863345450118, 0.6221947587902193, 0.4020493881639704, 0.9520875185863367, 0.9675736840771043, 0.9053439912518524, 0.6300334545496437, 0.32419169687684773, 0.4589797742777679, 0.45925427218833226, 0.6707481583323317]], [[0.5828682493762849, 0.8864979131961078, 0.29097555125445745, 1.533088317122044, 1.0374774182463038, 1.1519042674600612, 0.4068573039447614, 0.9159526990438952, 0.8252961541626129, 0.8324004050246774, 0.017671153629582055, 0.6758147965983915, 0.8164816792641314, 0.7079546698363419, 0.41477620713358004, 0.46294953648619874, 0.6057314188743963, 0.765343063716247, 1.0139738612731308, 0.3529064851288391, 0.24501157824041928, 0.43800142517674423, 3.4870886584447467, 0.28362295682629346, 1.249363720812012, 0.5122099209957306, 0.3907553208165929, 0.8703466602274808, 0.9061955467174095, 0.901641238543619, 0.6544261246401444, 0.37153742368631804, 0.45261548743414304, 0.4905067042505895, 0.6287119965881698]], [[0.598303286321417, 0.8672725754531789, 0.28619171328947896, 1.4897469296580175, 1.0428895008338173, 1.1695981355543155, 0.4094959055685774, 0.8640365737728569, 0.9436186437327314, 0.8845431353549893, 0.014768540825331569, 0.7072639335793708, 0.9839622254980533, 0.706041174089518, 0.46500547488854066, 0.4900807892519814, 0.5955672676830641, 0.7888693258814541, 1.0025426410054705, 0.5369583279152952, 0.3559242125771734, 0.5349505207104293, 3.4870886584447467, 0.22406659501772724, 1.1903568182766324, 0.5140294581649032, 0.41978151833317157, 0.7997714359853119, 0.83701912130941, 0.8996120512889118, 0.6215497834976398, 0.3637301501648081, 0.44357632132667835, 0.5242541609376206, 0.6104731249797724]], [[0.5955520702205662, 0.8778398001091549, 0.2727998629203308, 1.4382385033499794, 1.0671046787394232, 1.1360385518680611, 0.4091871377210646, 0.7824319745031493, 1.0334876566294127, 0.91512554555179, 0.013936431148572908, 0.6807259339072729, 1.0355807462917381, 0.7329787496356723, 0.39668708840766215, 0.4719234606363505, 0.5619929882007133, 0.7867102812544018, 0.9802945316812219, 0.6483444525865298, 0.47943791986554124, 0.6257396372659049, 3.4870886584447467, 0.2558554597110255, 1.1767754944450828, 0.5385326864257125, 0.45882579194530637, 0.7554719029264568, 0.7713819967490072, 0.8929760863064318, 0.4859744808241492, 0.32149440424861886, 0.41006934267892703, 0.5758332169323446, 0.5735778982900248]], [[0.6103070764081419, 0.8985239186806474, 0.2884713697687643, 1.379201520503338, 1.0704925932751914, 1.0857136863478898, 0.39827664967959114, 0.6687669787412638, 1.155003875955183, 0.8733988153328961, 0.010394843494808969, 0.6113641210803464, 1.0465061354740144, 0.70641199621807, 0.40634453533184484, 0.46056856646596966, 0.5109212702395115, 0.6789180926069092, 0.8785007778878974, 0.6175259652195895, 0.46122466687101615, 0.6295924439568016, 3.4870886584447467, 0.2783187420938555, 1.1059315089312178, 0.5307595882240295, 0.4406906746787428, 0.6931701958767235, 0.7253542505844017, 0.8569413270680318, 0.23843799656184833, 0.24377561232610573, 0.34480975473870534, 0.5893114000996698, 0.5414994034786608]], [[0.6298058898139665, 0.9011128750387053, 0.28722990582097435, 1.3088471089587732, 1.0508083098915768, 1.0623303193499496, 0.41202856114059405, 0.6313020665332449, 1.279242478345778, 0.8740737628449924, 0.005781048474719398, 0.563574420059689, 1.0674347993545834, 0.7103142019082207, 0.4201134822929509, 0.46357579291178563, 0.4754596998619196, 0.6182744585478471, 0.8328557780541698, 0.5828021003288373, 0.4513223869720218, 0.6190359989964926, 3.4870886584447467, 0.31688329400077764, 1.0982911495485532, 0.5246108187881762, 0.434373487344119, 0.6670328662373977, 0.7263905800398369, 0.827619686323958, 0.1697858385299498, 0.20539472980996598, 0.36343705415564964, 0.6194021173824573, 0.529094522910911]], [[0.5607364319932711, 0.8774942771133641, 0.28907130614334015, 1.2333699244393885, 1.0328991303644297, 1.0137273668779563, 0.4191902579231801, 0.5845795957593807, 1.3640346996699906, 0.8570573741091041, 0.00400914172406001, 0.5815514833404525, 1.0624063012946738, 0.7193402345429024, 0.39564301346137626, 0.46267713321455417, 0.472280117385248, 0.5833313690363037, 0.8049172977769612, 0.5247364384711173, 0.4430801132401891, 0.6550783094162249, 3.4870886584447467, 0.34322610389296454, 1.0748468685717159, 0.45793090449808393, 0.41623648276692904, 0.6499080756689594, 0.7253044276947147, 0.7796873949472509, 0.17817270520448453, 0.19609097408063328, 0.36360995626870446, 0.6134243569388198, 0.5198223191504612]], [[0.5491328630147992, 0.8772805479470663, 0.29881442287497817, 1.1704482954127935, 0.99839750163916, 0.9653503044934331, 0.4224036272154863, 0.6157413402469141, 1.4197827166094146, 0.8436318385411339, 0.005244286496593908, 0.6133251124740611, 1.0750464445003003, 0.7011622197618897, 0.38324126595225944, 0.3963912084592316, 0.5032517302502988, 0.5838821146060618, 0.7935540187486394, 0.48337417412439854, 0.47225297922952686, 0.6374524538051157, 3.4870886584447467, 0.3925641227256872, 1.0287405414962816, 0.37997463385976704, 0.4002629419411055, 0.6454415611784595, 0.7109452969516715, 0.674658110610558, 0.2670263494602499, 0.2273042804028885, 0.41736370541741835, 0.5763906242474486, 0.5291748633965612]], [[0.5308166213710843, 0.8662355415214922, 0.29434148573851543, 1.0975266512577997, 0.9502278485772399, 0.9391988123323893, 0.4905963605069073, 0.6133691426308165, 1.473724015450693, 0.8197428690463797, 0.0054892631523850875, 0.6217515197105021, 0.9635994383876885, 0.7105644355860723, 0.37400396656174667, 0.41589164484581714, 0.5294832284068033, 0.5770234967656045, 0.7046853969811566, 0.40717783089582305, 0.4954335896736406, 0.6170431819481217, 3.487088658444745, 0.4684103756215544, 1.0271963221431837, 0.3045916533333122, 0.4196704024526361, 0.6818468747377695, 0.6682268638270742, 0.5331146208283688, 0.30911275944227234, 0.300924890715215, 0.5112922947099776, 0.5744343033793227, 0.5459786524212186]], [[0.5430928827572612, 0.8160467954187345, 0.28115281333004943, 1.0286559241346531, 0.9196652162265481, 0.8424394611152505, 0.4268316992027905, 0.589350012431385, 1.5342477657346965, 0.7888927276454499, 0.004544095845848322, 0.6607118865544266, 0.94152943598684, 0.6813605002103167, 0.39347766247754107, 0.4268078472036041, 0.5230434539977437, 0.5528699318939008, 0.6819281529354353, 0.37203030408064386, 0.48140171489992645, 0.6206945575563652, 3.4870886584447467, 0.5000229480274179, 1.0295573182621873, 0.2574964626445131, 0.4290618703804018, 0.6514014269449544, 0.6346087692876146, 0.38694893989251267, 0.3615263919242556, 0.4252939020565517, 0.5234063619768448, 0.5651028257392278, 0.5547121270320456]], [[0.555986172225575, 0.7981795565753115, 0.28606742487446374, 1.0001469122328133, 0.8435072008997417, 0.7826020313930745, 0.4419573354616837, 0.5731651848117996, 1.5949185660651057, 0.7412448616666162, 0.017639119334213006, 0.6140090139873644, 0.8866977816025432, 0.6115261636211191, 0.30656952588095676, 0.4243148447380638, 0.4238765798406675, 0.48312390225036417, 0.6411478628757186, 0.3225024270542482, 0.4515901949265308, 0.568295493229031, 3.4870886584447467, 0.5250273172111496, 1.0724394247432227, 0.22334415780397504, 0.511354681523376, 0.79972314592168, 0.6012017467789565, 0.4049601149881944, 0.4626578257442311, 0.6439517449839591, 0.493709777745049, 0.5448430801412543, 0.5674818343607593]], [[0.5579505814942389, 0.7716968299216825, 0.2997899403373338, 0.9538750072412614, 0.7526354222634283, 0.7148363678616465, 0.44947862617031786, 0.5865775794432204, 1.6346355466170572, 0.705029421187123, 0.009426388384040458, 0.5709646101398391, 0.6786252762812739, 0.5781753661540182, 0.3218456067292296, 0.4216325262297942, 0.435853285784874, 0.5114453407250565, 0.6762296072320753, 0.2964095238354532, 0.3880613862392278, 0.5563696796574666, 3.4870886584447467, 0.6807605837809869, 1.0667890075874504, 0.2372648988573945, 0.5682497019269129, 0.92017124104818, 0.5319855228821605, 0.43369940547527863, 0.5631947819197494, 0.7223485811685761, 0.4325052778157994, 0.6043476638753746, 0.5839981624677102]], [[0.5959464367749795, 0.7672789449101635, 0.2909706073692148, 0.8637202434433005, 0.6938461346339613, 0.7003363465306023, 0.4851340479314019, 0.6205172783479687, 1.639548346072585, 0.6656353167075657, 0.005771117961354562, 0.5324956630635822, 0.5096207219887245, 0.529384414304, 0.2906679153517392, 0.4274621627811187, 0.4146681217636863, 0.4305216219571033, 0.5782159602810744, 0.2803201411888166, 0.3380045338733507, 0.5489639707702516, 3.4870886584447467, 0.7743525075080908, 1.0600772512729086, 0.26535263218983085, 0.5939528033905205, 0.9103555631832361, 0.5046120813581473, 0.45848743321903973, 0.5984807313282643, 0.7601415526992406, 0.40436996849803675, 0.6974720494180329, 0.6226965699672737]], [[0.6531477594291039, 0.7459986616764565, 0.298700959618074, 0.8278602721148964, 0.6512067872913205, 0.6329469265822398, 0.4673646741914421, 0.6872273442256195, 1.5981836330272794, 0.6197110616877887, 0.0015170401694573252, 0.5452098625988497, 0.39585392072897374, 0.534506670658813, 0.3046560850942507, 0.42606224670372117, 0.4332059103129131, 0.4591545112027242, 0.6227665511163754, 0.27333144557586425, 0.31191865459457935, 0.5162199424821879, 3.4870886584447467, 0.6580365031669547, 1.103956984373369, 0.2707469831619965, 0.6535249463572529, 0.8221623763054626, 0.519385603625278, 0.45404247437645134, 0.5572846435139178, 0.7171060307445525, 0.42520642178328594, 0.6620636075908062, 0.6680878218468487]], [[0.6724019571216893, 0.735744242757144, 0.29071862081820715, 0.7982438393240917, 0.6154515003573335, 0.6006858774874537, 0.44241416518134624, 0.7642396960237472, 1.5334475081675918, 0.5749762081089953, 0.0014674066155896134, 0.5669980771203846, 0.33467782873003604, 0.5422758031119224, 0.3598816637913331, 0.4002601617154343, 0.45652793924221197, 0.4795422002445181, 0.6484531231958, 0.3052141504091957, 0.291814271678598, 0.47863728045615794, 3.4870886584447467, 0.6746659228033576, 1.1244804358311666, 0.2773181849239061, 0.6804840176300273, 0.7845101528480327, 0.5311760075709062, 0.4596215651690063, 0.5211786920028669, 0.703260132259367, 0.4346449009632808, 0.5649518416318808, 0.7198647921921948]], [[0.681797878359643, 0.6706525239412467, 0.244616184179048, 0.599060478627244, 0.5280958560795237, 0.5090406948710893, 0.42619122228769885, 0.8423181713542249, 1.4496763195606706, 0.5095182105675451, 0.0006657345705040744, 0.5694108189024231, 0.36919491197387444, 0.5348384255983414, 0.2905095719650187, 0.42181772727565914, 0.5439334986188524, 0.5008794112048933, 0.6255639771730181, 0.34280308489523714, 0.2884418353789714, 0.49101136824353464, 3.4870886584447467, 0.6758727849478334, 1.186670311885417, 0.282250933546343, 0.6658542979343489, 0.7846135981924875, 0.5240936214506418, 0.4913537151453105, 0.5394652272782772, 0.7395153730592945, 0.45802197693000046, 0.5362581961831554, 0.7271869213435248]], [[0.6570046265272507, 0.6938145036417636, 0.265713052622201, 0.6300643465280868, 0.5027691315708929, 0.5636923689776967, 0.40931737335232987, 0.8394046057113335, 1.3416389739301984, 0.48223706905075153, 0.0006925861058894203, 0.5156447323376361, 0.38357359546257164, 0.5125979515631167, 0.26586714764305236, 0.45074155057652465, 0.5530216412204951, 0.49108409325709546, 0.602694648418064, 0.3985854718838992, 0.320704429284062, 0.5044132435863365, 3.4870886584447467, 0.6850276856804142, 1.2578937625851248, 0.24526957375441757, 0.6638992813254414, 0.7649656701250873, 0.5096447302965608, 0.5271286781884896, 0.5550533417281421, 0.7558632214475276, 0.45223080553660255, 0.5609791855176938, 0.6928769394322549]], [[0.6363209830384338, 0.6916324335320145, 0.27269641608836287, 0.6916542984474441, 0.47225721086145966, 0.588457947966292, 0.4084478650430845, 0.8250687069893994, 1.2045446689007633, 0.44793583853745444, 0.0006863612673056192, 0.5181793102375758, 0.40047522100052163, 0.5173904274377441, 0.2814846238839578, 0.42002268106173823, 0.5794614636689643, 0.5032324719190071, 0.5832988415348577, 0.4257699625948037, 0.328117055937847, 0.4925950157014181, 3.4870886584447467, 0.7070630943409402, 1.3419336801858086, 0.21365156398817067, 0.6763358894948917, 0.7184580345464879, 0.5142657346212397, 0.5234718041554063, 0.549427273628242, 0.7441400981547763, 0.4316129924275238, 0.5912476136373256, 0.5472288365912059]], [[0.6573130129809152, 0.6024910043030867, 0.21064872801914494, 0.6347407883292557, 0.44145241202136576, 0.5919510306628619, 0.40026885247322813, 0.8238987499210301, 1.076826162411991, 0.41603365502590195, 0.001395631245915456, 0.5410395794379057, 0.39890114703164314, 0.556485720845636, 0.2885202549831527, 0.4315174926400618, 0.5336289049859937, 0.5058848871899291, 0.5697944752651588, 0.42650159268223714, 0.339996597120271, 0.4801021440321978, 3.4870886584447467, 0.6914071002219393, 1.3852015458922091, 0.18603182213811628, 0.6714374058686006, 0.638234586373797, 0.5276954660010151, 0.5183797377848929, 0.5466602221861819, 0.7836618916071132, 0.39099839389058955, 0.5991943174846833, 0.45948836084901457]], [[0.6648343548550306, 0.6138224847403054, 0.22715469608327535, 0.5522987651665352, 0.4119444026416838, 0.566694501551495, 0.3847095628632915, 0.7821987448171503, 0.994013278162311, 0.41375901698182127, 0.0027121639678429816, 0.5510030366896979, 0.4291218882307127, 0.5815661491265208, 0.18982664554033693, 0.45350874382545425, 0.4996570579684431, 0.4817568796412729, 0.47304021188886936, 0.41068267527505364, 0.36542827972285885, 0.47849315271883835, 3.4870886584447467, 0.6872778224034399, 1.3695515360478019, 0.1624318680740445, 0.6549614636586254, 0.6089430945412053, 0.5479082589566103, 0.5024584372466301, 0.5505865048573031, 0.8207611738368212, 0.34960543251271037, 0.5851400028217641, 0.4663158222607249]], [[0.6976081498644846, 0.6173061758983712, 0.25976227304902866, 0.5164929656253248, 0.4085521802957913, 0.5626451699763838, 0.35783441834832524, 0.7614902688976775, 0.9707620886239804, 0.41479122309163696, 0.016991957647392905, 0.5579386415341124, 0.39416413035776277, 0.5738455018214235, 0.2241827298781902, 0.4551612385690436, 0.44741043698682187, 0.46613686505679097, 0.5261296074703289, 0.42186497607022344, 0.4117324932671339, 0.49870247162572456, 3.4870886584447467, 0.6649341188631166, 1.335252723566888, 0.1369865139941264, 0.668017409856268, 0.5839323031773278, 0.5660608519191836, 0.46888697528668294, 0.5463502756404736, 0.8344517410645838, 0.31693841085972807, 0.6011082464466556, 0.4270667591738902]], [[0.728236601630067, 0.5428535975201983, 0.19829388903651785, 0.5326222638177458, 0.4598287232046794, 0.630935049669713, 0.3542985958522279, 0.7393289319094771, 0.918277319092863, 0.4701854127900461, 0.01112696807342127, 0.5498258394234162, 0.28770658240736147, 0.5678179502411899, 0.23935974591031117, 0.46881709545861017, 0.3397120337316046, 0.44926693390740224, 0.5214340896402612, 0.4137407150773138, 0.44352648848524356, 0.5268387385763291, 3.4870886584447467, 0.657126204051574, 1.3114209164759998, 0.11072161479100283, 0.6627973198737523, 0.5791402507300649, 0.5661298328805889, 0.45030930660032664, 0.5447349559261364, 0.8533033061529283, 0.2950291310129405, 0.6865785074637645, 0.4375200018710803]], [[0.711773835091495, 0.5480701510002144, 0.20866813671640705, 0.5109232229058026, 0.5207789008052433, 0.6206414443680248, 0.34817353811754376, 0.7257612492141775, 0.8293599404678175, 0.5195724985258364, 0.0009688581790248806, 0.5379166569247306, 0.28737179881940034, 0.551460600849345, 0.24648972083286907, 0.45352509098750954, 0.3618885170624009, 0.4520189436257055, 0.5432363863524985, 0.3902780363851571, 0.5058128907671312, 0.5856838584505805, 3.4870886584447467, 0.6602602009852863, 1.260630225376857, 0.07357690353438975, 0.6385203868062139, 0.6046339557826406, 0.5486180919266467, 0.4316977584974116, 0.5454813341855953, 0.8698143093157487, 0.28468265431923095, 0.6710811675235717, 0.4355806091533855]], [[0.6958986876669415, 0.6297334507012502, 0.2574003519578259, 0.5248645529470808, 0.51026197861842, 0.6725661361991184, 0.33131597657232686, 0.6913157223086934, 0.7503255479509403, 0.5629946870091785, 0.010066263249356255, 0.5249065535733368, 0.32984459110106445, 0.5430536125394569, 0.24513851124472252, 0.4393969450872157, 0.4060399806141913, 0.4657891379552303, 0.5984198871643411, 0.36392544656757325, 0.4949346737474266, 0.5956501370963398, 3.4870886584447467, 0.6522697485265296, 1.0987704927726054, 0.0399265883795889, 0.6084999896703588, 0.6071767463735898, 0.5416670322254261, 0.438984595033211, 0.5438688801059325, 0.8565841941522414, 0.2975195998019086, 0.620809536175658, 0.418992529708077]], [[0.7103573508229102, 0.6392036422806104, 0.2642604516548673, 0.5799011327293251, 0.5396254924120005, 0.7093928216005249, 0.34093864237104826, 0.6792665904198707, 0.750382617583846, 0.6034792175511513, 0.03487535697070988, 0.5884676151471961, 0.42020780386570417, 0.5797343850995534, 0.24424745003346343, 0.4148233057139278, 0.4337718232061102, 0.4363905978900492, 0.6172268066634533, 0.34170673496896553, 0.45102170468789193, 0.6098608387072112, 3.4870886584447467, 0.6442143312861512, 0.8056329355916765, 0.021485966292125284, 0.5694835014654132, 0.5608108563809244, 0.544562445719539, 0.4491736947238494, 0.5475233948560023, 0.8234056450555765, 0.31641449452933285, 0.5975946164055923, 0.42730743644589525]], [[0.7198516501269865, 0.6452361947738852, 0.26578779352154397, 0.6191128430454518, 0.5867782074104779, 0.7209955767616427, 0.3475428249491879, 0.6948348259420112, 0.7120007224717971, 0.6345527535580495, 0.021507499240844108, 0.6010969412640019, 0.38223603747957335, 0.5874248452712246, 0.24897506411468512, 0.3985774440512616, 0.3969786184096946, 0.45589441100423855, 0.6539970959470189, 0.31387762464799634, 0.43246589739197716, 0.6516576001491177, 3.4870886584447467, 0.630293755378196, 0.528499159058401, 0.012266813359424152, 0.5290435496545477, 0.498171294109432, 0.5382921906177298, 0.4502450720415766, 0.5525006927710803, 0.8237719744246229, 0.3270470736404603, 0.6412299423716323, 0.4775272456714793]], [[0.772473214494113, 0.6587540533063379, 0.2726802407997943, 0.6467581842773662, 0.5895911122799948, 0.6715645120013174, 0.34955409849362634, 0.7594748681126227, 0.6881668702817779, 0.6406510604033128, 0.012716581603886648, 0.6412941530123915, 0.4057645914379236, 0.5975378364532897, 0.25085325125704144, 0.4112307566935326, 0.39163157796683545, 0.4622263590308771, 0.6593476719890329, 0.28689832918914265, 0.40653160419017986, 0.6481408774871666, 3.4870886584447467, 0.604259343417556, 0.5370860002313615, 0.02527906958267198, 0.5139792249629316, 0.5558240107453656, 0.4715781107510896, 0.4645340197988799, 0.5577545774124579, 0.8449147504878334, 0.3740886187493142, 0.6186749601742604, 0.5115203320531525]], [[0.7699573541570255, 0.6383809018789826, 0.2725036936911301, 0.6615397577460671, 0.5576552089991434, 0.6445378332419835, 0.3587574636500702, 0.828435875850263, 0.6833206902144096, 0.5687668602696835, 0.008793108784448155, 0.6003748037180777, 0.40453337696284436, 0.594673209666086, 0.24819972234282747, 0.38140786239839225, 0.4153115502611828, 0.4597652603327734, 0.7202877916960793, 0.3022482790259473, 0.39884476257859486, 0.6472340653881722, 3.4870886584447467, 0.5619439597046798, 0.42089004809471003, 0.08223282650651648, 0.4925366109892272, 0.5330331922183478, 0.39551909792471907, 0.46347433240596475, 0.5534667574394856, 0.8506205609967545, 0.4246189642693824, 0.6588608981664562, 0.5381370507075787]], [[0.7547349142251492, 0.720069230900287, 0.2714566639431751, 0.7252134632391576, 0.5064903995248697, 0.6928340302950036, 0.37388951803427684, 0.8822418435853007, 0.5983751408286914, 0.5826083172251546, 0.0051904180623224, 0.5982425620484757, 0.4537595925215526, 0.5943652557157318, 0.24228614146615166, 0.38412436596550653, 0.4365364056104565, 0.4457764842179139, 0.7186177844534216, 0.306350917167197, 0.4060937700561761, 0.6093666698188523, 3.4870886584447467, 0.5249023692242654, 0.4173070528898037, 0.115746389465457, 0.455118022806521, 0.5392349130980931, 0.3447514393348255, 0.4929725906365006, 0.5513319674223192, 0.7905065178705095, 0.46022801515624046, 0.7728777983849755, 0.5375715539265938]], [[0.7157244279956401, 0.7507210351418051, 0.2712878175744024, 0.8105646669512181, 0.46987270228013917, 0.6971630916808342, 0.3685180063790034, 0.9845898124758152, 0.5339986957537274, 0.5459367716313029, 0.004484958947125927, 0.58357341080177, 0.5362976492272536, 0.6192411211782591, 0.2432938607483465, 0.3911012420824329, 0.4267401664179171, 0.43996270143327554, 0.7383951845940417, 0.3411484672884437, 0.3443598029335562, 0.5467208246610274, 3.4870886584447467, 0.5320794860726452, 0.3859125458201974, 0.14366065706344153, 0.519158524298757, 0.5461862515128022, 0.25309925084468216, 0.5099167698835352, 0.5605775997765505, 0.7562562092685197, 0.4350250737986313, 0.7667440642681778, 0.4985232630182798]], [[0.6607999907562809, 0.8140544080187301, 0.27421583116222764, 0.8708475143763423, 0.49837738289088296, 0.6449816458244366, 0.3746930879362349, 1.2032330452193667, 0.5980911766582373, 0.5017790030361267, 0.029672172544902065, 0.5757135404496245, 0.3939745024962736, 0.6178309669628628, 0.25813883444961006, 0.4410184693839545, 0.4538644697450631, 0.4097771364998738, 0.746166750555073, 0.28557633525686943, 0.3258979379965019, 0.4569964003742909, 3.4870886584447467, 0.6197545583202931, 0.3660880652351847, 0.0762321301298113, 0.5095094360820486, 0.6040522226196761, 0.25994108625321877, 0.5519043569048612, 0.5662250914929143, 0.708926652027843, 0.500267087169946, 0.7699875777332335, 0.4668451534175607]], [[0.6227692263000822, 0.8116199155224619, 0.26375447335112573, 1.0153260566758018, 0.6009211281388085, 0.9633239733309384, 0.429605781288402, 1.325063659715178, 0.6512043021218384, 0.42632139540814695, 0.025768977389632162, 0.6154448824358766, 0.4778287798171035, 0.6426423945685376, 0.2624681982519709, 0.42944183601453945, 0.42065896434342165, 0.39243199224234904, 0.7257737334671839, 0.2669065036908347, 0.2850656639328407, 0.4043688373507585, 3.4870886584447467, 0.677378885593304, 0.3627014900160429, 0.06598671823443564, 0.49441412945396684, 0.6319035649696866, 0.29136007052945406, 0.5919512694248306, 0.5549106216126451, 0.6997508251379656, 0.5875164055391964, 0.7060801219012385, 0.45949985703894625]], [[0.5249785255742541, 0.8268385563490175, 0.2528920621415727, 1.119268608988414, 0.7042763223268964, 0.9025661793596317, 0.3316473351590404, 1.540719394164974, 0.7199390773201277, 0.27052299858543183, 0.029480682695836195, 0.6082071473648841, 0.515673901978136, 0.6376558469274192, 0.2703356871939011, 0.42621099701741527, 0.5016358062383289, 0.39977985341128597, 0.723085098585872, 0.2780362192395089, 0.32133589769762205, 0.30658403578176446, 3.4870886584447467, 0.6889654482473602, 0.3345153470655332, 0.0740663705053807, 0.48211726012639783, 0.6284671510942387, 0.3349387625856671, 0.6457156473055623, 0.5423955407043666, 0.6223529680340707, 0.60274110305957, 0.5781417272483971, 0.4447934559320671]], [[0.5404930953950923, 0.8603458391933047, 0.27038929511730586, 1.1573295122773875, 0.7224006473539801, 0.9961459486366887, 0.3602572768550103, 1.6072147086938713, 0.8439743228673434, 0.4795972761469931, 0.026950508158394497, 0.6030100995455452, 0.6084445856144776, 0.635935773167654, 0.30014534250172675, 0.4439648661410892, 0.4993728571839784, 0.4065873105812392, 0.7149875213554582, 0.2903654798145994, 0.33509456673602667, 0.17912918885394077, 3.4870886584447467, 0.6609107227064243, 0.33994801141580155, 0.08478300793911561, 0.45514414518411545, 0.5648743398517031, 0.36045229671835705, 0.702087023218917, 0.5768436221628036, 0.6068588502616697, 0.6191419835495171, 0.590615101219492, 0.4589040214926799]], [[0.573683770541338, 0.8246794625377845, 0.23445487553507965, 1.1975868791605355, 0.7475198834492418, 1.0266175600995444, 0.32176001324380027, 1.560858913148846, 0.9191135738617694, 0.5422844118265917, 0.021552837187230858, 0.6105274169375308, 0.6027005262418301, 0.6346939393366695, 0.29881889648147797, 0.44026195487178793, 0.5005972866013189, 0.41010843687702647, 0.7040031125531006, 0.30499953521378714, 0.3647527545453826, 0.19618900821418223, 3.4870886584447467, 0.6345672838716678, 0.338418249158951, 0.0828811987852909, 0.48157599827217423, 0.5643348401060448, 0.3736309154443852, 0.7317830555561667, 0.6109356985411466, 0.6007543758867265, 0.6247715116260593, 0.5402048216924791, 0.4540679674506423]], [[0.6463246444943653, 0.7948107487673178, 0.2157601119913229, 1.214692888449157, 0.7544293765949102, 1.069969365829143, 0.3228096578344258, 1.5271910222477663, 0.9847771884228141, 0.6418597577371881, 0.06483175003350841, 0.6153564301861855, 0.6409187571001977, 0.6330823661875975, 0.2876037760922378, 0.4466924696342547, 0.5014882406657604, 0.41643329947463237, 0.7006672377006085, 0.3188738163369769, 0.3577171666207043, 0.3284733487833842, 3.4870886584447467, 0.6355470533405887, 0.34943962704434256, 0.07840377727457376, 0.4885501898194711, 0.5638306385289698, 0.37791711340185463, 0.758656964807619, 0.6193890224591497, 0.6486757934785257, 0.6443853017612063, 0.5115245325464137, 0.44952476616101156]], [[0.7440794748423165, 0.7516382621851412, 0.1928985885079994, 1.248215438126644, 0.7710904020283524, 1.116119676376053, 0.33242179875138766, 1.4498065927611077, 1.0371588865229464, 0.5168053920438483, 0.06540043537318312, 0.5792208122597085, 0.7331418735895985, 0.6061313623077866, 0.30266109078381287, 0.44335546391733216, 0.4974011548940116, 0.42113472298371935, 0.7147100506484316, 0.34367014956962616, 0.4199349205003765, 0.4497083548666446, 3.4870886584447467, 0.6428252494266256, 0.3466861829392207, 0.06600019409528071, 0.451173975207705, 0.5437581580611226, 0.37619054784481654, 0.780803928018352, 0.6217608754011357, 0.650244849511448, 0.6482434714122766, 0.5217432454481711, 0.430039438940946]], [[0.7487165742139643, 0.7549520110276502, 0.1644643355551067, 1.2607405759826542, 0.7674534477662152, 1.181415941293385, 0.34720368268567847, 1.2609174379938721, 1.0901288860282996, 0.5123789919346984, 0.08927793859749444, 0.5974205903782082, 0.8059699085620526, 0.607152859377438, 0.2544104241887095, 0.42838965871821577, 0.4836192252493746, 0.4353254590736965, 0.6827165756306741, 0.38114067215345976, 0.4852305351179602, 0.5430260827071289, 3.4870886584447467, 0.6502578432614243, 0.34974752752434224, 0.060773449732831136, 0.38033770369774467, 0.5471482934552581, 0.3665198266812073, 0.8025834275810253, 0.6243433261793635, 0.6224103519171306, 0.6604100160950666, 0.48225983080196577, 0.4085034802783839]], [[0.5663351104827501, 0.8859168541571093, 0.16192599922656853, 1.210442083696475, 0.7767130888916506, 1.182779989437427, 0.34530005529586455, 1.1373207617524168, 1.149269342381813, 0.5076243960488643, 0.08243804162268847, 0.6308804098683152, 0.9588612504098801, 0.6089768901855817, 0.2638459193091408, 0.41322918519250984, 0.5407000462966143, 0.46989811089570777, 0.6417011727357136, 0.4323827796566932, 0.6554151169472026, 0.6984462113601329, 3.4870886584447467, 0.6509582178750224, 0.34694432409589393, 0.05823179440677012, 0.32559543553288284, 0.5414316104016502, 0.33700826892473806, 0.7839310026653854, 0.6415148217773652, 0.6042392118945087, 0.6700919488579329, 0.38401575986686687, 0.389402103573561]], [[0.552007869896685, 0.8567638387535458, 0.1815726174011028, 1.2240825981359011, 0.8563730781417112, 1.1945565980958806, 0.34473117939266756, 1.0185525568605969, 1.2161577000976245, 0.538546315171209, 0.08146326016110732, 0.6586602614008314, 1.0143553795413751, 0.6058135986136107, 0.2620432176944164, 0.41759044416812574, 0.5175961411115706, 0.47110191607773194, 0.6328014998270473, 0.4500503716339402, 0.7475198084414105, 0.7484252044639776, 3.4870886584447467, 0.6317511844454347, 0.3491459591140554, 0.06071869925538531, 0.31028562087087164, 0.5987260472471251, 0.30224670608834386, 0.7843337625587476, 0.6676004804748846, 0.6268512017497756, 0.6728168214203193, 0.33423770149161225, 0.37987166805395045]], [[0.5813977056837514, 0.8678359236817028, 0.21505111961291062, 1.1839214609711597, 0.9629422299477529, 1.1653801047791887, 0.3311320161343633, 0.9457940498155915, 1.242048755013106, 0.5796608367600843, 0.07479683359724143, 0.8568344421213951, 0.7932211153050515, 0.5858548128874125, 0.3411147011419321, 0.43005681286260433, 0.6272932278449319, 0.508344970317974, 0.7197453657336674, 0.4451196126071252, 0.6632688310248602, 0.6747924668970176, 3.4870886584447467, 0.6058832575588344, 0.3964478753692458, 0.06566149966884, 0.2531036315006715, 0.6578078048160279, 0.2890119557096842, 0.7503789528744134, 0.6855649902223535, 0.6337519771975346, 0.6585443982977262, 0.2311192419689937, 0.39094493691263343]], [[0.6093471196164576, 0.8360184764035388, 0.21277765518644082, 1.0874728041187192, 0.9956280301157723, 1.1466156461362949, 0.3433783735951971, 0.8920166777357724, 1.2515132903720292, 0.5952014229982814, 0.06767283311071096, 0.8290613307406208, 0.8380933053069215, 0.5783093850909651, 0.36723256604322707, 0.4067563270244804, 0.5869649253222062, 0.46352200421814654, 0.6472838239755018, 0.4504240520465377, 0.6382490466256416, 0.5661358491067682, 3.4870886584447467, 0.595003693248729, 0.47068109629868427, 0.07577525195511922, 0.2615164240666036, 0.7348524949633637, 0.29348777183299646, 0.7090657361588297, 0.6925237261641424, 0.6158381685088057, 0.6347345859253553, 0.16908848152855496, 0.3902166973372903]], [[0.5442950075936995, 0.5392018794910503, 0.2860323424715052, 1.0022579507358007, 1.0769884578710458, 1.0716458431072209, 0.41297631068239865, 0.7724954463948541, 1.1964861829239823, 0.6153632543560423, 0.07579571029608784, 0.7964002098667805, 0.7916706367984159, 0.5971551220919389, 0.43619660391628773, 0.43085423067053985, 0.5378687470381982, 0.5050867733249644, 0.6134999584307556, 0.45064360915232116, 0.6746394177321864, 0.4979233104074659, 3.4870886584447467, 0.6051895144041571, 0.537460076747162, 0.0881138868200173, 0.27269684022708496, 0.8994023864927502, 0.3237609445435823, 0.6460589808077604, 0.6918274349240969, 0.6122213186922763, 0.597753287470532, 0.26339650896960387, 0.5628616048474737]], [[0.5581855938531555, 0.6064406865184784, 0.3087406212073006, 0.9116962166969644, 1.0849115760483188, 1.0650275375646219, 0.4036043674270315, 0.8256096377981047, 1.1454450966628735, 0.6234884142468952, 0.07566838982541106, 0.7935483360710254, 0.8892462597994122, 0.6669348531659166, 0.5043677012920986, 0.4352697192070345, 0.4817732951767756, 0.5121440812525111, 0.5522566934886717, 0.4211068816409591, 0.64287941901508, 0.5209731694179647, 3.4870886584447467, 0.5257109350844611, 0.5633262510812045, 0.08934651298880436, 0.2543036548465995, 0.9377316590377509, 0.30453942437398973, 0.5872109314057943, 0.6809271381370472, 0.6057425270122392, 0.5354469403331404, 0.36143485087742944, 0.7178857464472186]], [[0.5983292550047223, 0.6730595738653965, 0.30772315510526327, 0.8260683375630444, 1.1374727159790392, 1.0491060635227907, 0.3935152335761362, 0.9242119561193727, 1.0942510809620822, 0.6362578933622868, 0.07356986640127106, 0.7875154536830592, 0.963879222620216, 0.697722476452961, 0.5236767499447941, 0.4239017290741668, 0.48914787236436175, 0.5362855669542128, 0.5964411703474342, 0.38015866465356196, 0.5207667523798563, 0.49876943760054626, 3.4870886584447467, 0.47035689682684634, 0.4488137889801631, 0.10002146915538138, 0.2615327266195705, 0.8396412845057292, 0.27273055316296735, 0.5201139417132083, 0.6463562567713411, 0.622582416092474, 0.4745074312517912, 0.3709776367810295, 0.7345340456654578]], [[0.6335169642925578, 0.7160597053041835, 0.2914705904612201, 0.756230209110634, 1.1785175365699303, 0.9727480314681074, 0.3695158200203336, 0.870326541246717, 1.0232953131235978, 0.6173846668504633, 0.07165978489876224, 0.7975307171305384, 1.0268077839186112, 0.6657449665152769, 0.5202597933890537, 0.4037025225461399, 0.4609791640491061, 0.5242089529630923, 0.67033023915046, 0.3819089591139614, 0.5281725934885839, 0.46795969288728767, 3.4870886584447467, 0.567173617150311, 0.3827004921867987, 0.09902662546407667, 0.26221530055363845, 0.7343018266557205, 0.25804216523431855, 0.47204982951926255, 0.6098470628830681, 0.6089726588253868, 0.44874898379792955, 0.38453184359517156, 0.6446084232903195]], [[0.6554280588382562, 0.7536732173171571, 0.2865386629601009, 0.7318162164294829, 1.2247447977194643, 0.9261094398161794, 0.3574764281293923, 0.8384681949724656, 0.9314414369089037, 0.5710999336452922, 0.07248994430465941, 0.7855476084344988, 1.040199731303562, 0.6547219991904657, 0.5211610818471532, 0.416646385111871, 0.4789302506930291, 0.541806734137381, 0.6976614330144606, 0.35012643015267564, 0.49203040762638617, 0.455325128035984, 3.4870886584447467, 0.641212064585839, 0.32868997232358615, 0.091264256148815, 0.25253837803752033, 0.6782957775300507, 0.2624808250294419, 0.5471267094839263, 0.5566254587823926, 0.5408524261945042, 0.46125950739628274, 0.42962478094767387, 0.6556596026548421]], [[0.6674597188176696, 0.752313386168515, 0.28281638296970124, 0.7254850816036542, 1.2238542238621002, 0.9057968995482357, 0.36891187899998956, 0.7722211309380902, 0.7969080983955215, 0.49512768975690646, 0.0667892721049835, 0.6655858485566071, 0.9214742794358344, 0.6387081582347542, 0.5188920338095723, 0.41049512639075536, 0.5993779669870533, 0.5875016040617502, 0.7602668672197352, 0.317272903380728, 0.409072348449714, 0.4573326946250863, 3.4870886584447467, 0.7149270344435104, 0.29246063097398417, 0.0884588432212771, 0.2641009880624241, 0.6820688609100509, 0.28412879651136613, 0.46524594310966566, 0.45255199781206584, 0.4347280222342716, 0.47783226155304165, 0.4324277922008053, 0.6136337683957459]], [[0.6928741613082127, 0.7554370019470101, 0.28183589775326306, 0.7122260820808042, 1.201873001180112, 0.9116358291347282, 0.3855796301043544, 0.7361204416129812, 0.6321239630731259, 0.39118846256201933, 0.0675498320214548, 0.6599674955386966, 0.9122639367874874, 0.6511286668025168, 0.4687466094135761, 0.4196041654951388, 0.606241286958443, 0.638898907535167, 0.8515198298050963, 0.32144329335138544, 0.38771863230701153, 0.4394739078229806, 3.4870886584447467, 0.7884535005817811, 0.27186860656835304, 0.09451292121659108, 0.2481920839671496, 0.7469813313479166, 0.3405205497642014, 0.4391775975763654, 0.4257479254997658, 0.3650127813469824, 0.5192590638283171, 0.46973127340533605, 0.5892445303099564]], [[0.6832173376786638, 0.8152610509206024, 0.32038673194229533, 0.6232318440564917, 1.16780632563514, 0.8365619248287597, 0.3929957121508487, 0.7027611594656882, 0.44706099023142154, 0.42724726134932456, 0.07763534545665783, 0.6668852426590336, 0.9032076923918736, 0.6234742552321331, 0.5346896085793083, 0.43079701943419846, 0.6006481871816779, 0.6765269639521367, 1.0223141108981206, 0.35738111615356716, 0.3803425058202474, 0.43982663452456566, 3.4870886584447467, 0.8172538065666388, 0.3124169798208089, 0.07976976227526672, 0.2623068181821808, 0.7841075252695523, 0.40896015983837924, 0.43647243210834685, 0.31889975119515024, 0.34883831107163404, 0.5603909936140793, 0.32740935554657163, 0.5618019952280441]], [[0.7005700682765135, 0.7967603765910845, 0.2998504601328317, 0.5908987243718944, 1.1233033340949374, 0.8204399014296045, 0.3854663787630502, 0.6145072856638079, 0.40113089544702485, 0.4388218888836266, 0.16260116643888234, 0.6231094762014779, 0.7979834026472874, 0.6327554831014375, 0.5103637994918164, 0.4713041227689907, 0.5581400137987843, 0.6854098077348866, 1.0369223180194884, 0.3684483260661091, 0.3454671071575016, 0.46871305081152537, 3.4870886584447467, 0.8365522232205166, 0.31890680312457004, 0.06799990696235338, 0.29706196169470744, 0.7799989521623713, 0.45362012638787735, 0.41472882727949, 0.3048772596078184, 0.3621882064479049, 0.6100206322663357, 0.1486779483060273, 0.5515452995714121]], [[0.6958955779071272, 0.7769678845146171, 0.29233172200646873, 0.6190551544551719, 1.0143426877388821, 0.8508284012487054, 0.37222697826816326, 0.5619614103610022, 0.4209566204149162, 0.4438771149360147, 0.14987390443498677, 0.632866912409849, 0.7888316766177322, 0.6238276148813728, 0.463478125780965, 0.43905661238996596, 0.5500036096141212, 0.7122920046426515, 1.084387247167439, 0.33848247367522444, 0.32341843956856586, 0.4861348711513053, 3.4870886584447467, 0.8281313834563343, 0.31200384840617773, 0.07799273425573368, 0.3254763122673135, 0.7425563495946177, 0.4507952405653846, 0.35280522209787024, 0.3547378519230348, 0.3836996590662412, 0.6420478788619246, 0.09014435609429278, 0.5503814266425648]], [[0.6942677476392751, 0.7936407522906833, 0.29170784716533604, 0.671511296858923, 1.0019449840457024, 0.8139133252501125, 0.37832438188696926, 0.5061798230927229, 0.4648029863677422, 0.43215440230915525, 0.12029111916311067, 0.7123664624307802, 0.8371916010031395, 0.6297175819610858, 0.479090210346241, 0.43949639200999036, 0.5562279016242639, 0.7184313651223359, 1.1051858316986745, 0.3681009919941254, 0.35503687634461123, 0.4806685825510936, 3.4870886584447467, 0.8009018650078791, 0.3005031291759177, 0.11494213672676759, 0.3541604982469869, 0.7177763219844624, 0.4232171907817516, 0.3083094073692173, 0.41735705232209686, 0.43975401348790755, 0.6223534410516234, 0.07601278677605314, 0.5690606302413468]], [[0.5687648807304815, 0.8284882054317694, 0.3506628467443951, 0.7068787530209268, 1.0021696571206995, 0.8431140136978656, 0.38398713828709735, 0.4748109021405795, 0.47204327798147666, 0.4441102849808392, 0.09607819181242198, 0.7894224165018765, 0.8222330709449919, 0.6089321236606908, 0.4379133027751392, 0.449880941529887, 0.5474879676618822, 0.6950555006001728, 1.0884817821505357, 0.37653361658616136, 0.36388932588673195, 0.45035140416701697, 3.4870886584447467, 0.7418928736044524, 0.31115892121065536, 0.1657318630944428, 0.3846670068880044, 0.6956560198988251, 0.42934098794448494, 0.3102040523245768, 0.4599355064281897, 0.4446928030082976, 0.6010431977079865, 0.14637755483463252, 0.5823462258928047]], [[0.5779603999100033, 0.7707824644492854, 0.3333782190864193, 0.7602189135113514, 0.9702902582909646, 0.7838312649824458, 0.38979902525793914, 0.43634720024431195, 0.47654722841334923, 0.4548573586359733, 0.10348344787408717, 0.7825436734992177, 0.718067555570955, 0.5836873005635332, 0.44209202338999865, 0.4510951164122654, 0.5489584690728845, 0.648095235031238, 1.0170624845343985, 0.4028904865311248, 0.3883391465660246, 0.4667437022621177, 3.4870886584447467, 0.6839508712993032, 0.29796259880981724, 0.21784197829032192, 0.42393984886772673, 0.6677683016697245, 0.4475930656202034, 0.3956263599802833, 0.49371432182420677, 0.4572542656598377, 0.5566085060627108, 0.2341503167385601, 0.614145478215671]], [[0.5890768945488023, 0.7547419252693172, 0.3566459436409345, 0.8395274741376604, 1.181464888806668, 0.7481457358313945, 0.39974989405226896, 0.30737953964437914, 0.4762596982114143, 0.4197481255768786, 0.1179142050201713, 0.7351628729192733, 0.6586510598581613, 0.6103788086207127, 0.4215591046300017, 0.4311478074954428, 0.578026451987209, 0.6457165101426863, 0.9452163625688574, 0.37615099788657014, 0.3845147103958978, 0.4813031232183134, 3.4870886584447467, 0.6563943113351087, 0.3423158117613503, 0.25705968933556594, 0.4665027099106581, 0.6573385474062917, 0.4352338452541724, 0.4545326818275572, 0.5346376223369793, 0.4510027521295792, 0.537634855294977, 0.32502168235534784, 0.6056937231078415]], [[0.6353100391257303, 0.7668929968849197, 0.3813958202586839, 0.7993377836369853, 1.1948411047588388, 0.6885558792645562, 0.4051053530348817, 0.3088691276665645, 0.41834830737259954, 0.3742458225496003, 0.11491302907913956, 0.7020110082420785, 0.5897669247553555, 0.6193951147830473, 0.4448823283810636, 0.43376701491391834, 0.6435205724521413, 0.6466093406712132, 0.9357968550858511, 0.3914395931486118, 0.4428092596010293, 0.5512623196938076, 3.4870886584447467, 0.6517056262218003, 0.41312554215096503, 0.26886625142495335, 0.5109324984977227, 0.6648362738250069, 0.44118872310442214, 0.400809937653187, 0.5364611497094418, 0.4008158779086274, 0.5126081356369572, 0.40381813477894574, 0.6249983753720925]], [[0.7114171499562234, 0.765014639321121, 0.377458830508257, 0.854979343469193, 1.2224280898195423, 0.6125545191971306, 0.4045930891434106, 0.314283387711101, 0.3737885028645548, 0.32619359937248527, 0.11182133677192764, 0.6676273536539402, 0.4303985252739422, 0.6078874750615878, 0.4847804862250044, 0.4244979219016991, 0.6613544725822487, 0.6270899918260397, 0.829871732004664, 0.44626721528738833, 0.4870200212047376, 0.6211583252435268, 3.4870886584447467, 0.618500610048494, 0.4116563156784677, 0.24625142636697042, 0.46549491050363434, 0.5963187435832911, 0.484401291829636, 0.2549013781722831, 0.549960812594731, 0.34206800198066506, 0.5054369510926389, 0.4535204164271171, 0.6505251251341695]], [[0.7592111620013404, 0.7450909489409763, 0.37316983537649606, 0.8236169530241091, 1.1845868983945733, 0.5499141934266978, 0.4200854650160044, 0.3138030902905784, 0.3832840084627802, 0.3460103858290347, 0.13887411439189018, 0.6655030867165808, 0.4144049970216341, 0.6180136566242554, 0.49177387202490397, 0.4253805942094143, 0.6701690455519098, 0.6366177310789329, 0.8097821855489935, 0.42267939774642715, 0.43055015066443453, 0.5772883210638718, 3.4870886584447467, 0.565446384762515, 0.38241597002066763, 0.24021066489011728, 0.44024579990476564, 0.47241430374300164, 0.5219582134416211, 0.21893248816201855, 0.5669470516761703, 0.30654249586478105, 0.5267098230869804, 0.49625298981043026, 0.6420602353667615]], [[0.6856319351284409, 0.7014247283646988, 0.3786642093055222, 0.8427250736966063, 1.1270212723813833, 0.6417142819800506, 0.4322816570687109, 0.3079934896279105, 0.3841944847201475, 0.35624076500043145, 0.1457820956647173, 0.6436118272860097, 0.6200797387289292, 0.6041258636296539, 0.4881111745327919, 0.4721049480845617, 0.6484113785360601, 0.6312790610141938, 0.9248226805181968, 0.46904322941285115, 0.3917722836078183, 0.5246822973601901, 3.4870886584447467, 0.5183767714682037, 0.3557807042127063, 0.25617326229713955, 0.45540512557502055, 0.4232311889420984, 0.5198622323749189, 0.21890693475468997, 0.539096503358151, 0.2841901832557822, 0.5470988991131075, 0.5391973815064401, 0.6090758290926198]], [[0.6142922803283335, 0.6847691614378908, 0.3837285426875802, 0.8355729350847616, 1.0746504216843327, 0.6045366921676387, 0.3727440922230263, 0.29466670312870247, 0.38542598268236106, 0.37507296455061967, 0.2127011452605182, 0.6410165282975964, 0.4590152931229091, 0.5876352999483545, 0.4731145580270122, 0.43399740707461376, 0.637661081178833, 0.6180219066057602, 0.9635101106906292, 0.42902044856763133, 0.3818670125338536, 0.5662753715805957, 3.4870886584447467, 0.4493382007164586, 0.32801811742872766, 0.34874916319285887, 0.4327660531822013, 0.30599198802483696, 0.4827816517379274, 0.22124839029369883, 0.47536978991116907, 0.24076074487011745, 0.5340089635192005, 0.5461521137548603, 0.5761630109145357]], [[0.5717824759207798, 0.6568146712133893, 0.3993833675965922, 0.8091483208127528, 1.0213114619501236, 0.6596759971550119, 0.4523283142605066, 0.3497839970320912, 0.38221928426603496, 0.3847971323446522, 0.19538266948709376, 0.6322428065007675, 0.3281112843672089, 0.5745466761557543, 0.4546128339630698, 0.4559559482300063, 0.6490419645670216, 0.6478572297382048, 0.9778482122604879, 0.3933326521764665, 0.3681768982932228, 0.6072378667381546, 3.4870886584447467, 0.4327149882479631, 0.2902869932833635, 0.41201272145740664, 0.44547679140021557, 0.28278730528408796, 0.43589281929007645, 0.27266714175543216, 0.4290869515686265, 0.1877219074783423, 0.5229142612934707, 0.5704857525889562, 0.5982557435350523]], [[0.53051475226756, 0.6514936162228223, 0.3526142564313335, 0.7704474058267275, 1.0435866931233562, 0.6704946475832192, 0.4083533268322841, 0.3473312041764382, 0.392411865862541, 0.4944922462834954, 0.056901200861897094, 0.609777416172376, 0.336709301499759, 0.5949191389261934, 0.4300367051855949, 0.42122940381553986, 0.614513414752026, 0.6412011021959334, 0.9648238442716734, 0.4075719895875546, 0.3417297490306051, 0.4733323370076349, 3.4870886584447467, 0.47309790489575204, 0.27732075577829246, 0.47370291796057445, 0.3962059860973156, 0.2871718950130868, 0.4179267818088983, 0.32405471881436176, 0.43194617323338896, 0.21705752955115035, 0.5263790600617583, 0.6042796489190734, 0.6341623908380063]], [[0.5210428402068965, 0.60663390304096, 0.35617661783827004, 0.7829822911011179, 1.006123341130052, 0.6755098452665678, 0.43427206991620576, 0.39157524091844625, 0.3384246891863673, 0.5668484599988655, 0.0545282430583993, 0.6113408335984016, 0.28152671120003303, 0.5907876448812028, 0.3551388538377629, 0.49411508597743825, 0.5911926994251016, 0.6317912466935469, 0.9638217370500073, 0.4757760014209974, 0.36563194325051374, 0.44417455000610756, 3.4870886584447467, 0.5126578808442279, 0.2765333331979267, 0.48847330570584124, 0.3499865009582418, 0.2849617459084061, 0.418862733604648, 0.39983032446984684, 0.42740204725466224, 0.24299190319644454, 0.5489023776711424, 0.6244523142308381, 0.648970617016444]], [[0.4930808855941085, 0.5833386295701593, 0.37635539860337275, 0.78591789682666, 0.7563162498698717, 0.6990983741038193, 0.4559435014728943, 0.423526186271666, 0.33404334340584646, 0.6593876210639963, 0.05035571950532464, 0.6006486175886472, 0.2963116955339702, 0.5685027485030134, 0.3676970123797573, 0.49129652105790167, 0.5856467265945683, 0.6100991361902818, 0.970667378308466, 0.4006488419615356, 0.2776067655506357, 0.3894963030884208, 3.4870886584447467, 0.5310318750833268, 0.3096708045986849, 0.5306237593848357, 0.32280081035265007, 0.2839501061335225, 0.4245499627579984, 0.4239351917284519, 0.41256669790911193, 0.24437395717553864, 0.559526721269472, 0.6419178731050603, 0.6362748760189397]], [[0.4616568982397995, 0.5791730011748526, 0.3969813076642291, 0.7831169454023658, 0.7824945057148098, 0.6967626988458886, 0.4745337645539568, 0.5033362183512784, 0.34251223822632954, 0.7092408011729857, 0.06111872404933491, 0.6234306941494325, 0.29102545847027655, 0.5575469638922173, 0.38937567488786334, 0.5592602955132052, 0.5799086330493181, 0.625384845509012, 0.9928061824793037, 0.39817616373944553, 0.24059709266370866, 0.45165762170943, 3.4870886584447467, 0.5454417408829152, 0.3275981391532813, 0.5600785355904164, 0.3399271207361997, 0.28653654493020786, 0.43405528046749875, 0.4579929364518478, 0.4028564044846059, 0.2404679879142287, 0.5554218785222671, 0.650685380243189, 0.6361120659608395]], [[0.42544613953796484, 0.5596361400972933, 0.39597522326578416, 0.7058973158947189, 0.8299537486805464, 0.7588051089515659, 0.4725811530938617, 0.5740740783317749, 0.33613554567190823, 0.7012466654163589, 0.04907572033336872, 0.6426284040848169, 0.2717043019131556, 0.5749895077402344, 0.42473165707437166, 0.4645818098501696, 0.5729600915731207, 0.6330739569783999, 1.0512368608298102, 0.4917552657985895, 0.2440893550559331, 0.4891231889069607, 3.4870886584447467, 0.540570595930534, 0.3219676651576804, 0.5556288710195567, 0.31323588017390264, 0.28996309168202056, 0.44301919541314816, 0.49710419460327504, 0.3970330172391865, 0.23026109595703256, 0.5404744604697097, 0.6482859669719939, 0.6223018861857484]], [[0.37780249711974956, 0.5487542233420044, 0.37801680333110566, 0.7078421925877446, 0.730809304176568, 0.7196242785024762, 0.4724012555630209, 0.5228355267984954, 0.3427016806607576, 0.6990006328327949, 0.05409212564044538, 0.6349885686171639, 0.2553635845358836, 0.5751248491001155, 0.3456304324020581, 0.5063472136419541, 0.6083858757691365, 0.6411621878772448, 1.0503362758307642, 0.4957823152667622, 0.2486871168364716, 0.4864321923656353, 3.4870886584447467, 0.3341175858740018, 0.3356324458574883, 0.5336381953838016, 0.24240071255113194, 0.292364514505365, 0.4438391459405361, 0.5175514656220965, 0.4074491201685133, 0.2241045740187879, 0.5206651648188083, 0.6352919429635925, 0.6192798317558706]], [[0.46589226018678936, 0.5621022037879084, 0.36849039427907865, 0.7824750838170885, 0.7885644999966264, 0.72924297166134, 0.4615320663804905, 0.5152605021926432, 0.3655467069855381, 0.6825326088435613, 0.05198309865387047, 0.597807333551041, 0.24336917420043916, 0.5850403923486586, 0.3722546503033164, 0.5170925105811024, 0.6334310656194675, 0.7409427860311775, 1.0915112219978427, 0.5490690020476545, 0.3035266074925195, 0.4860060003322354, 3.4870886584447467, 0.26029011396907764, 0.3400893429993279, 0.5383691742408587, 0.2117571912472403, 0.30767149118170417, 0.4496822234466985, 0.5010028272776723, 0.4125896397250678, 0.23740540567673346, 0.5090662012891652, 0.6232925662019845, 0.6360654581213716]], [[0.43753732576324106, 0.5266059406941177, 0.3718161897310852, 0.8186488798798773, 0.8275210018118775, 0.7272925475199217, 0.4522884589599645, 0.5129317161033312, 0.3869945947056717, 0.621093472854985, 0.06335660405995808, 0.574255861542584, 0.1925505892589144, 0.6100064308930024, 0.36677913867150425, 0.5097908172998363, 0.6266521197471109, 0.8332451765199619, 1.078099938553795, 0.5407003090620088, 0.337695031944792, 0.6193827545169885, 3.4870886584447467, 0.0983234225660093, 0.3094722550802562, 0.5154819564962905, 0.10298923907003524, 0.3174993749475692, 0.43871040574493336, 0.5072069533989063, 0.3887624154467698, 0.24109475440717612, 0.481245471360717, 0.5899045811245749, 0.6262792971225389]], [[0.42785775730388775, 0.5118209855536735, 0.3906425497759275, 0.9639567006007648, 0.8628579416991818, 0.7194022191581126, 0.4474053270665521, 0.5538527705687503, 0.37469386485108347, 0.5778184977921159, 0.0534869426673235, 0.6136335062978955, 0.39681067862984476, 0.6351640928276234, 0.369033371022346, 0.46150534598978954, 0.6726901296367671, 0.808189379796353, 1.0873430097295593, 0.45998399722823624, 0.35378201179744034, 0.6172112649394207, 3.4870886584447467, 0.0906544468586733, 0.2842736088907531, 0.49537137720914803, 0.08593738849333848, 0.37051568518204114, 0.4557505274706371, 0.5281088534891871, 0.3933386646171847, 0.23557590057937994, 0.4882984440351954, 0.5515976340876176, 0.6113584748382337]], [[0.4260194683242483, 0.5184048646522907, 0.3672028796079928, 1.0299229525478417, 0.9051879838481622, 0.6754502500084312, 0.4591646002229828, 0.5353393519644332, 0.3718756732865175, 0.6047860151873132, 0.03657296201195421, 0.6331949079245607, 0.4036466621810889, 0.596179453713461, 0.3588506464086185, 0.48466760923050844, 0.6605329182109105, 0.8309590334902522, 1.1059379188864709, 0.4647344879691626, 0.3533882731663497, 0.6612943665453518, 3.4870886584447467, 0.09225635063312551, 0.278946891841658, 0.47486015690276534, 0.10008860503458555, 0.40655808833419294, 0.46022996764643326, 0.5531868402624657, 0.39085624773777927, 0.24101917211837837, 0.48470149267345813, 0.5241399769387705, 0.5926749014578495]], [[0.4547706262131543, 0.44545537451975326, 0.34208428388395473, 1.1805905729111077, 1.0257516462802967, 0.6864443050806213, 0.35973214878044196, 0.5167470159773396, 0.3706951039916472, 0.6802758450351785, 0.030813722635702555, 0.645666409608405, 0.4073014565251592, 0.5710493576368189, 0.3582908162684767, 0.29886307114276045, 0.47052735551922675, 0.6611041495063027, 1.0889541812263834, 0.4220893228403322, 0.3071831929753953, 0.538585937157092, 3.4870886584447467, 0.13919559666900588, 0.28058234724485465, 0.42298442242100276, 0.13258386763293178, 0.41441979331821704, 0.45848083483220375, 0.6029441255906217, 0.40477898903393905, 0.27085646739227454, 0.4784861803102862, 0.5658763703023828, 0.5822375884191109]], [[0.52945261615879, 0.4597413772494452, 0.3409983877783704, 1.1992740095512913, 1.0194296639331473, 0.7017810982504051, 0.4032757265248814, 0.4178469197534525, 0.4164945307690707, 0.6109946078117723, 0.03198088207503706, 0.6472605229631613, 0.365215926645377, 0.5349487369153116, 0.3349401052283141, 0.34261632572102696, 0.41696519623803163, 0.7240078762490123, 1.1361634723634249, 0.5667633022301127, 0.36918594055361986, 0.5906818543206224, 3.4870886584447467, 0.16882074049667423, 0.22974507944312053, 0.3433070897534005, 0.16252781481106668, 0.4193463066502722, 0.42934492835098925, 0.6355714144303904, 0.4536276165221682, 0.35146680881740644, 0.4584137659748665, 0.5893463539305264, 0.5957772221700749]], [[0.5979378451231565, 0.44522829779035616, 0.3321828908624095, 1.0944688266503007, 0.8367460191207489, 0.779910964071497, 0.38379791218655723, 0.3628788097258797, 0.46934954021164077, 0.5255417616144389, 0.029581150946130747, 0.6568201750251507, 0.2862914342254134, 0.49701041417511715, 0.2867401341527798, 0.2636705862633859, 0.5708458872430111, 0.7140408512005305, 1.201713827856559, 0.44363897893514925, 0.4379047503445748, 0.624165570724763, 3.4870886584447467, 0.16185903426514758, 0.22072116800091268, 0.31365208570421466, 0.15011131058014612, 0.5210168439428772, 0.4011852011073987, 0.6767894440410303, 0.4839478494463442, 0.4505993018863232, 0.3913422235989702, 0.622304204239253, 0.6249147424887522]], [[0.4774478173829575, 0.4622192298722935, 0.3196577716340845, 0.9297800359360885, 0.7242084741661793, 0.7390600578424409, 0.38654217729275714, 0.2749898645817451, 0.5829232693151419, 0.5412662887287574, 0.02733568387973126, 0.648464585975834, 0.44955840055857466, 0.48648227526822757, 0.29257471334992036, 0.24070571281831787, 0.5752093634683855, 0.6613986657524457, 1.1372129987085977, 0.4224271462322272, 0.4515815100204387, 0.6669678571252042, 3.4870886584447467, 0.11661512413770231, 0.233312394921938, 0.3051134345407883, 0.15270036111974336, 0.5702350326578554, 0.38852044207073355, 0.7463007062613258, 0.4630884936268954, 0.46679329768860167, 0.3564055295393638, 0.615764845818114, 0.6624060652748389]], [[0.5781191909710377, 0.45633215439673175, 0.30798911394060907, 0.8062286852270182, 0.6511381528462538, 0.7719282764631737, 0.3673389562401671, 0.2727609273820887, 0.6756125662314032, 0.5325409049567078, 0.01595024450652887, 0.648531255918287, 0.35119232817645585, 0.4937691756531988, 0.3157735937673395, 0.23831100283002205, 0.6854373438921613, 0.6712333733669515, 1.0774666914404387, 0.44359941833770233, 0.43059801420346694, 0.6856486449812975, 3.4870886584447467, 0.10104226323728516, 0.5299452614768347, 0.3062147334389843, 0.148633379572229, 0.7925857555445938, 0.3863434073298494, 0.7852415773598107, 0.4906567409245252, 0.4761201315265782, 0.3260662402339283, 0.6271089122202272, 0.7017823630602211]], [[0.8059700837879371, 0.4423248457865865, 0.3052104358476924, 0.9022389137091966, 0.7924311876082233, 0.7128219051418153, 0.36270361541315804, 0.4024408755997425, 0.6623598688048913, 0.61836846774482, 0.02881774060132839, 0.7132919654867865, 0.3353314053609096, 0.5048249581385119, 0.36281971527077383, 0.21202141248637557, 0.779627088011693, 0.6348118838770384, 1.054831575264165, 0.5621448062839054, 0.5349646020833084, 0.9264404252068099, 0.2085996459192768, 0.19076604658268215, 0.8101980366231029, 0.39198063945449896, 0.2701068135182161, 0.9489653600550555, 0.47522789156880685, 1.301540538828791, 0.7513856947039838, 0.5532896630808636, 0.4804511932991817, 0.9994144865146322, 1.2569826735807057]], [[0.7893301262533474, 0.4396789165305124, 0.2973366811995711, 1.1160751817264218, 0.7675473407972918, 0.674543764056245, 0.3618936982161506, 0.671197441741977, 0.6093974432795535, 0.6162534817885573, 0.056440436614570114, 0.719346985374853, 0.26984020551605903, 0.5828015126470116, 0.299326869688114, 0.20508301409390056, 0.7807694572345087, 0.7050083861605191, 0.8753276521750957, 0.5982958097661065, 0.5092795414494522, 0.9903459950516448, 0.12217854890733483, 0.18584250809538055, 0.9640255811209254, 0.28741470541257275, 0.27430614450306684, 0.8437620704612787, 0.4489692656516052, 1.386291802898802, 0.8682227721102698, 0.4465891471973856, 0.4964322312319861, 0.8671974660446782, 1.4257674144515997]], [[0.7988828966028201, 0.4236895531177861, 0.3346119268599649, 1.2038000098249115, 0.785059611106109, 0.6561987035264001, 0.39924824926865876, 0.8824755271876682, 0.666145093421462, 0.5879931556497435, 0.05376689635734915, 0.6962871508706737, 0.25377303111486926, 0.6945938048779923, 0.3785366504027686, 0.1964233711099428, 0.7935358042469476, 0.7008465230677066, 0.863791324113582, 0.5273481635835768, 0.6618917930015601, 1.0206946301622168, 0.12436784669937975, 0.19272355549264863, 0.850467208155967, 0.2891209936639403, 0.27875349136998817, 0.9518769641007068, 0.40211966995028553, 1.3842683023742415, 1.0162666543352197, 0.4442681870731703, 0.6139606852962651, 0.8105782162089634, 1.4323949175944328]], [[0.8261017248665568, 0.44172806243583185, 0.31887975463146795, 1.2276000981580821, 0.7591500662918391, 0.6629104898305738, 0.38344278970380985, 0.9453896674855418, 0.6998617850100332, 0.5695377369238934, 0.07073026553529178, 0.6613104450787537, 0.2341816683118595, 0.6578393023154712, 0.3587626943790412, 0.16199027090770302, 0.5394591490173066, 0.6481665692189968, 0.8191366012137442, 0.5442492268333259, 0.7568257271309151, 1.0774390323475433, 0.0777732024780318, 0.16739531284458653, 0.8019702513839296, 0.31027661517749383, 0.3116302131039791, 0.861885164736935, 0.35090559017785133, 1.357770649781829, 0.9724620481841808, 0.4632155011907494, 0.6579746816359626, 0.7509323258287445, 1.4065164455931496]], [[0.8892599084224876, 0.4509132231336942, 0.320335431886775, 1.12451071859153, 0.7486701361311584, 0.6841718871186353, 0.41765478763506136, 1.0939560778689266, 0.7818817786403214, 0.5638459412536849, 0.05551690835194831, 0.6698750745481357, 0.18363351699831804, 0.6347533894348687, 0.4520404737619432, 0.1641561203385402, 0.5947519292213019, 0.7027927503585576, 1.0158737933914592, 0.5500973926903681, 0.7306187537887083, 0.9485465711763348, 0.06476218663448849, 0.1637881749492528, 0.7311807770824041, 0.3101017104458225, 0.36427387528844646, 0.9015043445002981, 0.30667447238845535, 1.28496501655635, 0.8939365115124163, 0.4250403635015723, 0.7046237636980426, 0.7254743113475113, 1.334888733492869]], [[0.8814071774663108, 0.4451284737167833, 0.31944073819738633, 1.1295142862792962, 0.738804037283182, 0.6819720344462568, 0.4253784346277141, 1.155690369441022, 0.787817583904213, 0.5379456884521512, 0.06253382993780301, 0.6667024673818229, 0.17711115930407978, 0.6649052598830039, 0.3429851539313015, 0.17976882485929505, 0.7800612158961394, 0.5813340098874817, 1.136434534208141, 0.6046944221375632, 0.5363678457829177, 0.8927886908325944, 0.05631055528449872, 0.16150965967218095, 0.6665762655568892, 0.3493930178264191, 0.38716072062132934, 0.8569409925515175, 0.2936264939971372, 1.1212921342254476, 0.8356174097828322, 0.5140320199890667, 0.695853544407419, 0.6731679515958888, 1.1504340842577583]], [[1.0529237118585208, 0.44085369520596895, 0.3289789365222022, 1.0988355953301157, 0.8019847182224865, 0.6850726517122689, 0.3956204227987512, 1.239106021461914, 0.8209486587185968, 0.5167859089814202, 0.06949638343690347, 0.7120270803983277, 0.19741282334894233, 0.6157022634973092, 0.5037248934653966, 0.17470013199062828, 0.8876070651138371, 0.5402411252854296, 1.286136612619591, 0.5426189155806586, 0.48163187773111027, 0.961442229301515, 0.0524949973255326, 0.16048215294399149, 0.6171368404755477, 0.3808689761495381, 0.4074626298871691, 0.8794056874682737, 0.2835396178768884, 0.9334524238978177, 0.8012152691419658, 0.6303292377849755, 0.7200446451385888, 0.7465387397068282, 0.9603432984628983]], [[1.1083533450163021, 0.4023031969157307, 0.32972705649945633, 1.1158849989398623, 0.817517013807368, 0.6441079442198216, 0.3545224227049596, 1.330261525714549, 0.7667012769189763, 0.4467929004638086, 0.08928566761044043, 0.7323003503636927, 0.22598459046095087, 0.6660072833685896, 0.4159604246415277, 0.22092351935165505, 0.8336380698354835, 0.5231244648919136, 1.2075283837352697, 0.5310618611114466, 0.3239713865373091, 0.949526014985616, 0.05130246327633087, 0.1547924927106924, 0.5037160218087685, 0.38364209485131107, 0.40195148680688375, 0.7202884631398079, 0.353141206927076, 0.7287980757078985, 0.7511847496966821, 0.8133073295740054, 0.7938389471101698, 0.7264212813204398, 0.9124137458897172]], [[1.1199637162095564, 0.3964620725144998, 0.31948548158615603, 1.0611563802360855, 0.8519742399007317, 0.6696225843913579, 0.38214108299427196, 1.3237527586005498, 0.7515590348816428, 0.3536765314324898, 0.08490882754724338, 0.7268934277060634, 0.24398450550292766, 0.6668300228194532, 0.397662295906721, 0.21973509435985766, 0.86788361001938, 0.5490672327202165, 1.2354506000077428, 0.5701331194188163, 0.3523844130041646, 1.00435598216546, 0.05241336929046976, 0.14815151659709502, 0.46261793502188847, 0.39435153812408774, 0.37767229734268914, 0.6490128444704937, 0.4102347929543174, 0.5925497520291404, 0.9306883318483288, 0.8951260093375781, 0.8357572952374298, 0.6999490422481041, 0.9745654416255998]], [[1.15235793215501, 0.37857500956093526, 0.30912224195716814, 1.1063945485887947, 0.8787840291937737, 0.6411904633515737, 0.397845129873973, 1.3303928668295306, 0.7344545182845228, 0.24323923429674732, 0.09434946334678175, 0.7076791650939652, 0.3003116674609358, 0.6589026444009922, 0.44131236285765785, 0.2263128848910957, 0.8605329289910909, 0.5586323718575733, 1.2892756832083452, 0.6189454743977857, 0.3271728026837714, 0.8868008800696677, 0.05536282046707575, 0.14494023717532833, 0.46245076787424017, 0.41653253047007005, 0.37757007469103343, 0.6121844354135282, 0.49495960314423054, 0.6915009408211372, 0.9303703905845496, 0.9306297144977735, 0.8547202757230115, 0.6562613824991372, 1.1110184995160353]], [[1.2198954798967643, 0.3696002660652534, 0.3163365723834301, 1.2862055972890194, 0.9454801594884404, 0.6316349961807861, 0.4327417995440658, 1.3601593729557957, 0.6296568971785124, 0.25516316164515196, 0.03571570977496524, 0.6698301214988003, 0.36692433266561647, 0.7080905014561543, 0.574613794332981, 0.22178483708025432, 0.850282512791188, 0.591676062481877, 1.3563578986793439, 0.662894571711901, 0.3430173571827556, 0.6849172383671852, 0.055970873504161155, 0.14100239929641153, 0.462481116104989, 0.44750571399840294, 0.36702660372226487, 0.5318899126037631, 0.5728929958698677, 0.8822150395508737, 0.9758010763927212, 0.9591776166795853, 0.914552917010814, 0.6068788366064087, 1.1729455367682982]], [[1.2999263861562094, 0.3512949026676968, 0.32593906550700275, 1.350055849365122, 1.0516900685740942, 0.6241066912105172, 0.3865930858152442, 1.382683103524212, 0.6843244160525089, 0.30049172786639056, 0.029893975646933107, 0.6570216654938882, 0.4674315735710005, 0.7196091894807009, 0.5661650816554133, 0.22073990604972182, 0.8561506604645304, 0.5520057540685205, 1.2591036991335436, 0.6654176210245384, 0.4069615616997829, 0.7707752321799434, 0.05804069373250665, 0.12826501027085588, 0.46153320081691884, 0.4449149168631842, 0.3395899460047218, 0.5251056550080755, 0.5755123837452328, 0.9633158673903324, 1.0295191693271368, 0.9702313429917215, 0.9151429632194228, 0.5728031840113694, 1.12658640566901]], [[1.2401202907727806, 0.31921935049363215, 0.3388378394541891, 1.2908085724336809, 0.9336808913279827, 0.6487648255335643, 0.3856081898950795, 1.379777452923218, 0.772155008300379, 0.36082832737243753, 0.039776771897838265, 0.8027557997626253, 0.5667791590796677, 0.6774867999484766, 0.5277940842697183, 0.23020401791989537, 0.8503530904413725, 0.5574660446236462, 1.3329402466774156, 0.6435344496042129, 0.41289645994278484, 0.6568157592495665, 0.058571049521009395, 0.12539098814752211, 0.4596029603306431, 0.38004900691776455, 0.3234018233276339, 0.5368149774588997, 0.6548084860819146, 0.9729827249914724, 1.0401817744900859, 0.9940122022144048, 0.9437732249071799, 0.534008459272584, 1.1826787500084226]], [[1.158614890483309, 0.34786382526439913, 0.368308123343809, 1.314230154423846, 0.7771007836479529, 0.6502494248826869, 0.40064000831049407, 1.3358870848796007, 0.847026963610208, 0.3905116012184231, 0.05977447916535186, 0.809182809483832, 0.6134326090439419, 0.6724423291935268, 0.490301631491432, 0.21605753351336837, 0.859916167892495, 0.6288269464790773, 1.4502080397397115, 0.7467947612614471, 0.48682939403865866, 0.6656994719524458, 0.05974950152328993, 0.1027610987158809, 0.46025767236897824, 0.35576155304986734, 0.31481314300599117, 0.5582487425224194, 0.7363866921005229, 1.0162238149257306, 1.0705222134559969, 0.9943632952495005, 0.9457786371919212, 0.49858286657635603, 1.291334423968419]], [[1.079792689625191, 0.37383469859279694, 0.3508146520559561, 1.230476969345222, 0.6533470114713839, 0.6533595974712406, 0.40635135233567093, 1.2269100494296445, 0.898803696546373, 0.42082098838317367, 0.09371044893284863, 0.69846216392494, 0.6581090346671299, 0.7399407544405996, 0.4667301209119018, 0.21461422317954676, 0.8656458010616438, 0.6340370527968319, 1.3465753906461013, 0.8041393285523181, 0.5062021694592971, 0.7018609942960689, 0.055294470319797874, 0.10825861894036193, 0.44887690189778506, 0.3173771387981109, 0.32089143208450643, 0.6155075757407298, 0.7826077920917285, 1.0205159635792107, 1.1050190404795996, 0.9703453990607024, 0.9668134553172842, 0.5156804680279241, 1.3888801214339153]], [[1.0939244071008158, 0.4360637400161571, 0.3125112064551761, 1.2747835242473688, 0.6159378109307807, 0.657813660603473, 0.4190224588724941, 1.1406507774108183, 0.9458979605236042, 0.4651397491308217, 0.1269472884743061, 0.7227734923088909, 0.6315210310012105, 0.7820072690949602, 0.5130931766120987, 0.20679299337876317, 0.8700406348809859, 0.6752180372917654, 1.2121718288506946, 0.8480792239339434, 0.6186326127943533, 0.7550046397993446, 0.055155354821749915, 0.0997576512249746, 0.4674691024010169, 0.22863707624038937, 0.3316212654034555, 0.7369204239367962, 0.8807281897622999, 1.0549279678843493, 1.1767821915066463, 0.9826401812363745, 0.9692088941631959, 0.5700657004126923, 1.460715590246693]], [[1.029251519642573, 0.47245678716651984, 0.32619185512724636, 1.4138071670508228, 0.7402383555165233, 0.6531525658627447, 0.41083255522176587, 1.0567623656001708, 0.8440222770841382, 0.4618764965484695, 0.12990313396673303, 0.6939936629081427, 0.5824000712249553, 0.7632958216070306, 0.41534281795517614, 0.22694566063099236, 0.8608371586202435, 0.7092278012432617, 0.9585631139691051, 0.8901312934114424, 0.6514136723536104, 0.8077299988888533, 0.05096664524145798, 0.08934208482627065, 0.5328421602878723, 0.18869630217091993, 0.34716400803865194, 0.896163704713357, 0.9052487762819134, 1.0572135437644286, 1.1275846980630777, 0.9961929751241763, 0.9752701322421393, 0.6244061573714133, 1.2724886366965187]], [[1.0015591191031727, 0.5086075250428648, 0.3396246763858204, 1.5834355689788886, 0.763777712994063, 0.6482080003206979, 0.4075850210214138, 0.9797332174402393, 0.8469634330374467, 0.4167536987923155, 0.11350091087419029, 0.6548080294541239, 0.5504323392548219, 0.7676446447935037, 0.3557842316819918, 0.3270712206539892, 0.7921210412443982, 0.727782351237074, 0.8955158196994493, 0.861670818365597, 0.7027062069049256, 0.8337209599532156, 0.047566068464840225, 0.08796825065408559, 0.5505624695594337, 0.17909272646925414, 0.37209823966353667, 0.9271334995701728, 0.8068882483172364, 1.0511179408721032, 0.9738907722435463, 0.9968378362937803, 0.9963156612731227, 0.6287146786004476, 1.1956935162319577]], [[0.9042632947879921, 0.5437957318225237, 0.3863914993450466, 1.6373637230500893, 0.8097654012012516, 0.6522683167869205, 0.3900070186778493, 0.9421682776081092, 0.8226030893292282, 0.3979382056868148, 0.08820846303747144, 0.6590640697521158, 0.5240789206578734, 0.7587927572426063, 0.3478698906376313, 0.28668311106023125, 0.7604634022767467, 0.7715032289062047, 0.8624424978370211, 0.8796603990860299, 0.7449798515191529, 0.8423630807718323, 0.04968991925814555, 0.14116791009736385, 0.6124552042483351, 0.20338242729935008, 0.38820168684105033, 0.9764491260485659, 0.6927231738327904, 1.055761148061372, 0.8413034309579579, 1.056582683436285, 1.0102402380453412, 0.6811236409177508, 1.1440141593108726]], [[0.8612684736854137, 0.53864072984299, 0.3968780606062564, 1.6376727408323277, 0.8397875557092849, 0.6573893570992455, 0.3799266020573756, 0.933914291151084, 0.5689068794070931, 0.3877018218534743, 0.08519908495109449, 0.678724046351874, 0.4802456034430349, 0.7661794589431534, 0.3722915080276984, 0.32881024021846555, 0.7716141431069925, 0.7724469027747091, 0.912582430021122, 0.8632770262641524, 0.7280213025543857, 0.8799123553681942, 0.051106576685280826, 0.18855804670076748, 0.5797179641762513, 0.22968712041671374, 0.4001697565170658, 1.0307250875863057, 0.5597247739751511, 1.0687561964283434, 0.7547493340463765, 1.0086408840160301, 1.0115748617919527, 0.6679562734399966, 1.1772779027543547]], [[0.8534511581014457, 0.5502647639240738, 0.40067548443286044, 1.4908329903778497, 0.843668121065533, 0.6698408971474656, 0.3824768416976804, 1.0593636425342208, 0.6686201076210088, 0.4833700152835141, 0.09363611302711479, 0.6668190530053466, 0.4945823402850319, 0.7945861214244205, 0.4095471628882228, 0.349894491631103, 0.8444750096511928, 0.7573698794451159, 0.9596140515337233, 0.9513345410023121, 0.741573863711842, 0.9535495254986843, 0.052836520638474306, 0.2594002049916277, 0.5917749462349172, 0.24994578228787506, 0.4421215478467979, 0.8900009411111295, 0.4337338592301931, 1.0379464794885085, 0.6421414164969432, 0.9432277483068229, 1.0294362031260045, 0.616276642707857, 1.2206753464128433]], [[0.7815060150196043, 0.5994762840167991, 0.3946615352339703, 1.4425248471769554, 0.8365335685744587, 0.6580162221219236, 0.37192389087591365, 1.465515535770317, 0.8160724017875459, 0.6500778833068377, 0.11792125906576693, 0.5837319534051406, 0.4996750755755438, 0.570815722400813, 0.3803790756982699, 0.3221250149965697, 0.8996581066472991, 0.8153876890428985, 0.9729614572390013, 0.8392755806894474, 0.6697168544250982, 1.0677195804559552, 0.05490832184862444, 0.4029090772630667, 0.6095571404918947, 0.2716555884573263, 0.7919322946573584, 0.8810544812280727, 0.4035185262391811, 1.062473955587079, 0.5518911711311357, 0.8242259835031858, 0.9921703610785453, 0.4384417749092834, 1.2054261150191667]], [[0.7177805018579013, 0.6029618977466241, 0.3695800428138329, 1.3294647582114485, 0.9508850523434205, 0.670324613387681, 0.3720013164785688, 1.4014279786742319, 1.0026455521856277, 0.717432686337395, 0.11430730489695475, 0.5294293193603417, 0.46780448633294125, 0.5643538598871434, 0.416863728437833, 0.3429860445426806, 0.7792451915622051, 0.7236181627525208, 0.8275124512454808, 0.3357215446483349, 0.38102021491062044, 0.7889156218215807, 0.05574000492075784, 0.30230833034591664, 0.5686981717196347, 0.2521900292368069, 0.7878599060533861, 0.8557617597732164, 0.3801159116487715, 1.054223743403106, 0.5429981580676351, 0.6586958287085297, 0.9361013853016964, 0.3781020076412194, 1.0631675071991817]], [[0.8229876208486839, 0.6417760474740363, 0.37955300944241044, 1.2559883174467594, 0.9464969860104001, 0.6720033930885041, 0.3812517479316528, 1.2769553407482896, 1.0936138453989461, 0.6629866550662491, 0.12825232618108595, 0.5288279705701255, 0.4756814837795979, 0.6098283927684021, 0.3508256467319256, 0.33636502018237674, 0.7353578620607751, 0.5970835083482788, 0.88209023469793, 0.19069337409882614, 0.2680443079925008, 0.7455647744148648, 0.059281182186903814, 0.32785289845041377, 0.5482600044973096, 0.2616228376630467, 0.7637507823143386, 0.7079637380656405, 0.37260595704532407, 0.9604977236966334, 0.5167164263398221, 0.5146609311210842, 0.8732759456506567, 0.384353695763873, 0.9928291957207188]], [[0.8656209348651872, 0.6917303238666678, 0.3880424193033993, 1.139743887905756, 0.935879014625993, 0.6939272839733064, 0.3886050229623065, 1.1851890438739308, 1.0791263306774863, 0.6205173255289175, 0.12734034244313414, 0.5045408019391224, 0.5487712574810721, 0.5698295402834626, 0.4011784019769369, 0.3568648442547738, 0.7309801868144027, 0.5763228252559747, 0.8941792603461246, 0.17126365450723174, 0.2603491923073873, 0.7360023549100675, 0.03602387103664667, 0.336745802927777, 0.5364923470562926, 0.2711327462697064, 0.8381659438615253, 0.7473547895135689, 0.3774929268560143, 1.004548310876546, 0.5270461397644859, 0.41651457304982603, 0.9063571990119164, 0.39608007906425147, 0.9813775578561024]], [[0.8599449082499486, 0.5890633203035165, 0.35985086284626744, 1.037592883465577, 0.9461647029916348, 0.7006609394505904, 0.3929681923048829, 1.2092192364152805, 1.0662760215116158, 0.5961115509843018, 0.1348497884647663, 0.49453417755115087, 0.7159017680071482, 0.555567024861461, 0.3490272592185983, 0.3849451712803981, 0.7593505865832014, 0.6606191929274515, 0.8613170307889761, 0.15060793419654564, 0.29303228255991154, 0.7403345668513112, 0.03470878812374918, 0.3893190933926988, 0.49097080694506007, 0.2727551619217241, 0.977259823358152, 0.8244129814201243, 0.3484848690451432, 0.9977707099971209, 0.5756390799965401, 0.3264734781914328, 1.0517447784647704, 0.4058183930825477, 0.8848573136609781]], [[0.858574623959599, 0.5867562581128972, 0.36787147671499726, 0.9740931090553989, 0.8715687501632089, 0.7202804082645, 0.3749925047553251, 1.2909359866559855, 1.1183026840750356, 0.5141200339192318, 0.1354851712157485, 0.4723354866873942, 0.7734657818670487, 0.5276861300322757, 0.3483345977601514, 0.39295880257407456, 0.8022466332392019, 0.6820676124378332, 0.8720141335441445, 0.09402656821817106, 0.2549173899038093, 0.5162005496043739, 0.03606311221242173, 0.43017289016059784, 0.592178972512695, 0.2723546225003952, 1.1240065660619796, 0.9603338280699468, 0.3385946151202148, 0.9799745497440231, 0.6832226719659271, 0.2858186433977227, 1.218554389933684, 0.3984399637974745, 0.8315175064538914]], [[1.0362711393398314, 0.6071374892123242, 0.39478644658330614, 0.912960525408789, 0.7219743568028428, 0.6927030514895473, 0.37935236800722766, 1.2106616373253611, 1.065533115316291, 0.49115178955525574, 0.12019074007123952, 0.46568117320605906, 0.871169076957852, 0.5504912465292968, 0.41427319249873185, 0.46152627320437495, 0.7058128262750909, 0.5672084248188567, 0.7315071416656322, 0.07360729006259203, 0.23211709239889086, 0.42223934430448823, 0.031978699917628264, 0.39479168158657774, 0.6845382249548292, 0.2815360655332919, 1.0117106638522324, 0.8166652345924335, 0.3596072620669443, 0.9602342920724224, 0.7291705288108314, 0.31420014222657766, 0.9893417162425169, 0.38638820087489495, 0.7500126188013778]], [[1.0936556654902685, 0.6238359633424946, 0.38308131784517985, 0.8662533754795401, 0.803322789919271, 0.6968203163162558, 0.3825104135637438, 1.1226696580118793, 1.0037989181821498, 0.5087686578207428, 0.11736790663304641, 0.48978454812158023, 1.1275639153750094, 0.6064391968982821, 0.41080340547088723, 0.4761128396450649, 0.7245813188073932, 0.565443908625695, 0.6999339730305406, 0.0803365511133467, 0.2199305601078308, 0.41668287214391136, 0.033779027090273486, 0.4037451429829648, 0.8818181620614942, 0.3475350821963531, 0.9088674838037947, 0.7703506665902877, 0.4027310975668299, 0.9606889707507289, 0.6753804225062241, 0.37702931715704135, 0.842061625481056, 0.3899348272637877, 0.6394436901560414]], [[1.2573768412601418, 0.6470166317012342, 0.42470871129857435, 0.8292986043046363, 0.8256964406387128, 0.7062035204762929, 0.3785206626196809, 1.0410038597192692, 1.0186303771087224, 0.5399381590247823, 0.1052484669982043, 0.5059069992002061, 1.0942017806823419, 0.5679186079713903, 0.41179851147899665, 0.47339714167433244, 0.6828109736145996, 0.5817012502275029, 0.6763151825352971, 0.06900360514105147, 0.21949102733316495, 0.42996726905782623, 0.03799046346704785, 0.40999295501386956, 0.7382337026156603, 0.3960252034607053, 0.8605637227043983, 0.6460984753731356, 0.4164228331494536, 0.8818876821089591, 0.6451540598617521, 0.4674574321422861, 0.8003717405940053, 0.38255198361392434, 0.6282374282426099]], [[1.3212456351119917, 0.6474568732936865, 0.41834793960288286, 1.0291588019176734, 0.9003207058933188, 0.6968448908159286, 0.3696751661987368, 1.1454387615004311, 1.094342816629778, 0.5990145046962304, 0.11659347201565218, 0.5013151598839574, 1.092862102997392, 0.5612531076691589, 0.41813855651978704, 0.4829476010338084, 0.649302938280606, 0.6091284553543515, 0.7079263179310893, 0.05411062658335175, 0.2152885263713777, 0.47448731918328113, 0.039139552515671694, 0.44069437722410587, 0.6159642591504473, 0.41466895214594235, 0.9524901082520179, 0.6455080672162892, 0.42870663956726807, 0.8343807938822234, 0.6237307681564989, 0.49247194042756437, 0.7799882353107122, 0.3860218206550181, 0.5995496083776627]], [[1.2991181907361615, 0.637058117357716, 0.41628941426506927, 1.0992996928695458, 1.0234320212907146, 0.7071438490749703, 0.3736268343240875, 1.2134825717082247, 1.1963287157347153, 0.6726148590022394, 0.14662963437632343, 0.5179789760243909, 1.0020242188964013, 0.5229789688741596, 0.4052029776304265, 0.4808367676202595, 0.4890664146084317, 0.6552677241657479, 0.7510344103716998, 0.07824726119140074, 0.22082748857362622, 0.47511536548493805, 0.0584874485343349, 0.44852001765806915, 0.5020529747569895, 0.408865408088977, 0.8241310552312306, 0.6258880818536781, 0.42639060851081867, 0.8053295085423885, 0.5740833570129382, 0.45397563569919963, 0.7929170306807993, 0.40487593126806837, 0.5560285600954598]], [[1.3242647794497755, 0.5768307786969131, 0.405859892588569, 1.0911121900901253, 1.0492518838018807, 0.7022167340233698, 0.35643054621181347, 1.1693833593902878, 1.3293993210247244, 0.5979865895544739, 0.18383760293902873, 0.5162538740784062, 0.9125395248757429, 0.40026062565378173, 0.4075861045386538, 0.5499012114428773, 0.5446806720891744, 0.7356071898547838, 0.8628457558166875, 0.12391581921039879, 0.19234924869308467, 0.4675791972158989, 0.0526305767006108, 0.36632665508419576, 0.383738825120401, 0.4062596318389153, 0.5463095467707462, 0.5446879233960269, 0.3974678228354916, 0.7873944836736784, 0.5841567810781841, 0.42311759028911133, 0.7918102458234013, 0.40220574550346383, 0.6366703984344835]], [[1.3580794616914384, 0.549778850120393, 0.4166106420491118, 1.105988175513717, 1.0860073588601853, 0.6948040349487404, 0.37442706947921667, 1.1098940669148698, 1.4316063603840528, 0.49572992145631617, 0.28486039772644567, 0.5128413181411619, 0.8297303648956503, 0.40916694982828006, 0.42682538146690013, 0.5387753839433536, 0.5627865519497812, 0.7191918578828526, 0.9139277391014071, 0.11185899129052508, 0.17209088192840766, 0.45602327266786274, 0.05515745056148971, 0.35149829558189893, 0.35815341266654943, 0.3971592001481633, 0.4715005556462996, 0.45050451926810536, 0.36887243818848725, 0.7964922629862048, 0.7359769818109461, 0.3891733235090994, 0.7894333011028141, 0.4621530365169709, 0.7789429048782579]], [[1.3891066877488076, 0.546232462889608, 0.433905957242458, 1.140209905994579, 1.159708066677581, 0.6734401503373877, 0.36105135757148543, 1.138492215622274, 1.5219984922708534, 0.443063309394648, 0.16719606702306053, 0.5125432799927787, 0.7627309284223518, 0.42005882075066076, 0.4265584892739079, 0.5115883280055898, 0.5593138194406551, 0.7777360853285672, 0.8913242798840592, 0.09095031744390375, 0.17664750781170724, 0.4397259981846755, 0.05342605135546072, 0.3630801735374289, 0.3339440881430725, 0.40445520133882085, 0.483887556437509, 0.4332877960299843, 0.33715786036456075, 0.7827895478658609, 0.9934710638827833, 0.3712931939698302, 0.7459745737405294, 0.3753301809064338, 1.0086817602775104]], [[1.19525172209867, 0.5204380224155272, 0.4424665752238134, 1.1858593321744364, 1.2622893525472412, 0.6871779090553947, 0.35045538536180343, 1.2129303260064919, 1.5853253686513926, 0.40140403079571485, 0.14944001953861047, 0.5112828182829223, 0.7651065330303015, 0.41236276133393573, 0.4068752648407588, 0.47709446670983036, 0.5370861564395351, 0.7782857951438207, 0.917497579948686, 0.08693780929593126, 0.17676104872878667, 0.43219478835313074, 0.05053686671431015, 0.3618508725874599, 0.251995064081433, 0.4072709721397921, 0.5730701954895893, 0.48859578972638723, 0.34715990481301856, 0.726710509096971, 1.0466390453977767, 0.3766144100304154, 0.6399793270632672, 0.4415852586081409, 1.0931927381836892]], [[1.1888654424098404, 0.5435050149369332, 0.42614182489028135, 1.1953444483190792, 1.4211313057027344, 0.6779885181366592, 0.3451039466824866, 1.3058230061636946, 1.6312272616819161, 0.37698927895182116, 0.11846002506407496, 0.5067996869727894, 0.8673774545673322, 0.40528125512674973, 0.4329053051745945, 0.5664399243688351, 0.5053083872847294, 0.7538852118623818, 0.9589024949620355, 0.09217675369514317, 0.18229004184520597, 0.3601689688639216, 0.05072356164674503, 0.327929488886057, 0.3969146591075206, 0.36348536500134965, 0.6827230321790166, 0.5489746257458006, 0.3522540150242918, 0.7183566240293946, 1.052086185811282, 0.4260191040677991, 0.41211214638329313, 0.41789461270222794, 1.191919726684661]], [[1.0825124888440676, 0.4740527453784519, 0.46129451714962755, 1.2341801855445527, 1.379960348679747, 0.730147574162755, 0.3495304807648469, 1.3085207840300297, 1.5525654996472358, 0.3879793922411159, 0.10639724186348795, 0.5225096889819238, 0.9405468250532306, 0.39152530014612746, 0.4325662972437673, 0.4779010337351145, 0.49743594835914595, 0.7728174704786925, 0.9106217018906757, 0.08963550223635905, 0.18300939621045792, 0.3378328703547878, 0.04951857240594818, 0.36602741395056515, 0.34849574376254044, 0.34697547736061074, 0.6831457995088189, 0.6438699854343065, 0.33346308590408624, 0.7035570971369591, 1.0049776877731007, 0.4707643088295726, 0.32118289074450806, 0.5590949099429341, 1.1261434703248971]], [[1.0286489325852557, 0.4247806310084733, 0.5104648778309632, 1.2295632321178813, 1.3408506453087194, 0.71543691221737, 0.3524675405820973, 1.2610398625755148, 1.4978138907296559, 0.41071726863343183, 0.12193468394009635, 0.5273749401900224, 0.9453260212831192, 0.4086187325772085, 0.43883914566822374, 0.46443947397818464, 0.5029463479145587, 0.7857723129534223, 0.9199327407644446, 0.07567468362050211, 0.2065415552669377, 0.3306289779892922, 0.05766053456591845, 0.38287957924373234, 0.27316558653502393, 0.34389894493289863, 0.6473313482625951, 0.6454594517345117, 0.3147456306803561, 0.6673565155881971, 0.9823283365292509, 0.4700833434895502, 0.37640399635552196, 0.6025120069008505, 1.1630751509643944]], [[1.0484297346629219, 0.39016290443589996, 0.5258953594937128, 1.20741332643195, 1.3476262577980922, 0.7305440215315844, 0.3599298759454193, 1.1342869332859764, 1.440227736968948, 0.4716268261436845, 0.18060384251674605, 0.521891253397635, 0.9834614960176701, 0.44969995772115784, 0.43304653686353894, 0.5045719528477324, 0.46625431401173395, 0.8071941843063407, 0.9528162741952411, 0.08642736182196958, 0.22082988654231417, 0.35811910759448096, 0.0480171201227908, 0.3212959760411874, 0.23161068738728963, 0.3473187162814112, 0.6055406674589453, 0.6750673731994655, 0.29507668873386195, 0.569051768693523, 0.9714063094536856, 0.37207741554629187, 0.4546791321447703, 0.6619861722682236, 1.2656750559062204]], [[0.9391078148653966, 0.35529689450623403, 0.5197198597783302, 1.1954466546942304, 1.398431060573897, 0.7425344294784905, 0.3693242437372166, 1.00353095256164, 1.4553653074802528, 0.6451153524697171, 0.18172022243569907, 0.5438445313741578, 0.9628597034894864, 0.4556256954865566, 0.43795573106956964, 0.4862044307049658, 0.4802994592990073, 0.782465739992217, 0.8591600599422642, 0.08340367078980107, 0.21828459171426323, 0.395839475690219, 0.04660225757497342, 0.24559665699890385, 0.1743729060114681, 0.3179771796956061, 0.5743779187536662, 0.6788496191086999, 0.32346019619920074, 0.5410628453547003, 1.0289690557378457, 0.3038267917121641, 0.6246739859730333, 0.6645350769186245, 1.2677366903333427]], [[0.8200625373013053, 0.3420861279461566, 0.5317629972045761, 1.153519587196167, 1.5641307049637563, 0.726048145147522, 0.3617691343664793, 0.8915951508668982, 1.407346941967761, 0.7526626665241433, 0.2027678798697432, 0.5460629057832229, 0.9415976430919488, 0.5059024121081739, 0.4249410700854177, 0.45935996908174737, 0.5376979409056502, 0.8192911980317654, 0.8582070258398522, 0.0859608977123266, 0.2156885854780521, 0.4129076505221127, 0.04670651044251053, 0.14606186627039525, 0.1767162969687704, 0.26385739051547985, 0.5040577133034398, 0.6686288370086387, 0.36875811824544075, 0.5715544288444682, 0.9921331349254137, 0.2674696743941774, 0.6931925151774461, 0.635175237130996, 0.8820133730821929]], [[0.7698032762803815, 0.34040369601086595, 0.5520578796144817, 1.1874410211362296, 1.645553137930066, 0.7933535820787824, 0.401715627459417, 0.845114471996096, 1.334919681109207, 0.7718069618134307, 0.17194830774538783, 0.5426722841848642, 0.965493505279384, 0.5264253998522589, 0.6108406110266904, 0.6972868698641905, 0.5545515368323848, 0.8292086954726005, 0.6783719032553406, 0.09466269112360029, 0.22015568926328372, 0.4162729396465254, 0.044486089678158924, 0.0875429202693987, 0.20417129488973268, 0.2299705564423185, 0.43754517693094275, 0.6813942197821479, 0.41207111691880055, 0.30694843893076285, 0.7010655908730457, 0.32212629136016446, 0.6904361797683861, 0.7075484882518777, 0.48238389551076233]], [[0.7664427348863145, 0.30654334827843155, 0.5636240194028597, 1.1346558490770347, 1.6680092554926391, 0.7186091766513489, 0.3604791539699205, 0.8267569461003321, 1.2846176682642223, 0.8623013520693577, 0.12077425197348451, 0.5478824691999904, 0.9626696589205287, 0.5144960116889798, 0.6226473022601451, 0.6991728771286958, 0.557441737498634, 0.7780345749781847, 0.6287434208093192, 0.10784562064554884, 0.22480388870665924, 0.4207319989909023, 0.044916004419472015, 0.07685715155596592, 0.23027469186387955, 0.21568427104146948, 0.41125044136478817, 0.7054738595438699, 0.4656169910222244, 0.18873258143876806, 0.4847806914623063, 0.3884669764522564, 0.7225569645289656, 0.7794898675488976, 0.37304711658751494]], [[0.8372040299421111, 0.4089047733235521, 0.48959891600031813, 1.1496600650946733, 1.6392861037237991, 0.7177179385648509, 0.35168611485931955, 0.8566931820552242, 1.1656910127950726, 0.8449075075852339, 0.1334464438358418, 0.5538536349091538, 0.9643588279267192, 0.46413276072775983, 0.5899740099344823, 0.6727905501219149, 0.5725592284430487, 0.7441993078373585, 0.647921978401813, 0.09109081246285156, 0.23322530077290277, 0.39695258814679024, 0.03988968702258007, 0.08154868306839622, 0.2813632744647623, 0.205750348437923, 0.42464654538949576, 0.7784664501530436, 0.503424668193761, 0.22509989177388423, 0.26770535174497667, 0.4487140905373941, 0.7094872049315504, 0.8306963693059985, 0.33720378363090653]], [[0.8726496857441881, 0.33085103162573626, 0.5336310770363455, 1.2060006107007315, 1.6327593103124909, 0.657288361838479, 0.3025025176695164, 0.8453109846511244, 1.1786822214848978, 0.7954926513918188, 0.13749458643610352, 0.543674794903573, 0.9317122357968783, 0.4576216170490156, 0.6356563417851966, 0.7012406620193979, 0.5901718941854361, 0.6991052856062614, 0.6335583566207585, 0.10385926743369733, 0.25157741904877, 0.4359019830482826, 0.039822946560628035, 0.08386795915544831, 0.3236993405057515, 0.23243471049858783, 0.4468018850067977, 0.8294799868609894, 0.528775316562063, 0.3303764356985348, 0.1251306177833359, 0.5265893524434228, 0.6782767772240157, 0.8461817790708723, 0.45252137939038095]], [[0.8663162304667262, 0.3374569850374609, 0.5496674672639368, 1.2132428216589044, 1.6030633652539774, 0.7627181973828341, 0.3359494301864907, 0.8331735296231526, 0.9364993993817325, 0.757860659518816, 0.18683102855352984, 0.539655674032748, 0.900879175474754, 0.5576429259399293, 0.5991503646609389, 0.6770692833035125, 0.5911859825903318, 0.7544710679009399, 0.6222558751862097, 0.13289547665348833, 0.24399446837810335, 0.4146401731608821, 0.04045691413063239, 0.09770301446824481, 0.36938401917050834, 0.24208644301291438, 0.4159604147226328, 0.8697832684906701, 0.5565569623679254, 0.38447176511845566, 0.10702360631344665, 0.6671579626371125, 0.667406493020472, 0.8590815885218053, 0.5566888277166456]], [[0.8854128911350736, 0.3380759169908208, 0.5592053934399055, 0.8271737312432526, 1.4045643656564746, 0.6747995774121794, 0.3372482610790405, 0.8642516747882059, 0.5485204659082901, 0.5480512311956024, 0.20307411528415065, 0.5514892748994346, 0.9260510818897878, 0.5126442932128332, 0.5278170670619597, 0.475468823964859, 0.5492922071006501, 0.7209493525625883, 0.6224136845003512, 0.17357419013962777, 0.2530036107617769, 0.41053984203389293, 0.043776893127886476, 0.12369767823401867, 0.436207514862018, 0.27488709251537996, 0.37933946700512433, 0.92173811985957, 0.5612830176919037, 0.38256653341939706, 0.12597810040838958, 0.7651893675644901, 0.47776802048666267, 0.8790441306481249, 0.5831548031208311]], [[0.8164355365504703, 0.33669011170402174, 0.5616897383795609, 0.7457760153874557, 1.422582306422592, 0.6995657722463828, 0.33843688910014713, 0.7986372072215169, 0.7967138959529382, 0.4661175158001314, 0.2190559219435439, 0.5863172536745, 0.9693821720832838, 0.4664109806029934, 0.5215450054347051, 0.667504952009874, 0.5428911334358051, 0.6538460370296206, 0.5785900389925003, 0.1840698011079579, 0.2622348257683038, 0.41946790746452123, 0.04396753927977047, 0.146748567396094, 0.574728427986789, 0.3046080236138477, 0.3227844765290562, 0.9933212700704003, 0.5623160178179661, 0.3661256637707655, 0.1507990642863829, 0.8220633348967704, 0.35536133858921454, 0.9197263163668525, 0.5873661288327328]], [[0.7764648901177429, 0.32114432544038496, 0.5648118756080831, 0.737794695406544, 1.457055479047186, 0.6943396478232372, 0.3403446382322849, 0.6839783929124119, 0.702204892367884, 0.3962205215596012, 0.2337691875366122, 0.5770464713227266, 0.9651051740493619, 0.48484069650065276, 0.5081836575762506, 0.46888067066485956, 0.4915153619078415, 0.719537609966923, 0.564233410116466, 0.20415944406366787, 0.2576563468032803, 0.42053691123082526, 0.04260076491164666, 0.17231195241229041, 0.6550292012351073, 0.3057248523076402, 0.2855117485377854, 1.0184578948569938, 0.5812078995774319, 0.33899512020317724, 0.1632812128173562, 0.850710374195017, 0.2614064691257604, 0.952114659012695, 0.6412103981165155]], [[0.7574675228877941, 0.34772014664190765, 0.5158892824050862, 0.7513400382143894, 1.4680642417907976, 0.6772567564814629, 0.35718195896727395, 0.6794324046266763, 0.6853834143951854, 0.37215619424008767, 0.23476685414498055, 0.5990162925906206, 0.9946040556247531, 0.4814492460285986, 0.481095598767536, 0.4448510457470156, 0.49281936102523605, 0.7444426839967624, 0.571566925615399, 0.20204061246813737, 0.25114124110937613, 0.41707227751176823, 0.046356394880958086, 0.19029070782148605, 0.6906065237379468, 0.2878660702560572, 0.24174505013606987, 1.0179724897940483, 0.5824778575857606, 0.32086457415499137, 0.16595887021476474, 0.8034406275681286, 0.24129996632640716, 0.9337430940425118, 0.669625283927534]], [[0.7648289811041572, 0.3585620169569915, 0.5223959105549194, 0.784397314979222, 1.473267853221099, 0.677531476458435, 0.3643229567852838, 0.6502856547340528, 0.65929688408165, 0.35603826005926764, 0.24585654699473325, 0.5799772614896154, 0.9901189046899512, 0.485331416464468, 0.4892663152187851, 0.4526156605890094, 0.5155739834765232, 0.6909272682933931, 0.5584080424863281, 0.24828518528027888, 0.27800692905790925, 0.4089276208659952, 0.048152013592156144, 0.21338479841355568, 0.6603046074352971, 0.2588803882085475, 0.20340557565684225, 0.9252753080386151, 0.5781464610408144, 0.341203357162513, 0.15696452153946672, 0.778003261815203, 0.2647635078440973, 0.9218152517847726, 0.6528157553965503]], [[0.6804654625458806, 0.371398494319252, 0.47765105075640424, 0.8043454591842686, 1.4911576483168105, 0.6666957576700534, 0.38540453014369835, 0.6775295127407657, 0.6334981737789538, 0.36376630667726395, 0.20622886004322794, 0.4851147555234633, 1.0068762748742777, 0.41650827582691113, 0.48954806300624004, 0.4651651552783618, 0.505258751202285, 0.6378880338835716, 0.5709075260259001, 0.20891577488542035, 0.2871658687551529, 0.382740935984097, 0.03724654730787173, 0.22745439395360173, 0.5408345973424323, 0.20277462002851787, 0.1955960428855048, 0.7810535208399259, 0.5852935820713384, 0.3932174562572441, 0.16815564414630257, 0.7819398529532011, 0.3331673071874041, 1.0457840479846698, 0.6408618684318763]], [[0.6170350727795377, 0.3743809797119282, 0.4690908033891975, 0.7964413856057242, 1.44703841200469, 0.6696384711539123, 0.38501215690754, 0.6662310021987845, 0.6174288595038762, 0.3593117813186999, 0.20320022665493986, 0.4749513226003617, 1.0072175526949028, 0.42312327432475105, 0.4244683513315733, 0.4664565865111699, 0.5077267245620011, 0.6381222930393372, 0.6361663744551672, 0.16389021065661924, 0.29310977968802504, 0.34579820682332374, 0.018083877517079665, 0.2768996547794441, 0.4513101957510889, 0.1483957033802495, 0.18010722869671775, 0.7098854827725362, 0.5872921680725107, 0.37127406662369183, 0.23396017771194555, 0.7375099933472962, 0.4060952908519886, 1.128377003768926, 0.6215685223399124]], [[0.5821429791935303, 0.4195408330516741, 0.4028153103985367, 0.8257807725762107, 1.4238239412871074, 0.6824242174281199, 0.38218282731095243, 0.832906310494868, 0.6041052463600521, 0.3509740168122798, 0.21312068155330305, 0.4782475354679979, 1.0587041184018247, 0.3569807360643993, 0.4574878588822695, 0.502764029616744, 0.4229514068894701, 0.6039909200140003, 0.6372190850661617, 0.2545314613056632, 0.2881444549136843, 0.4772948192984635, 0.022890550944373853, 0.3084931708597617, 0.3412804957042276, 0.09557607224149597, 0.18578150910004332, 0.5569373386999508, 0.5765496794752746, 0.38230730672830787, 0.28235875369487884, 0.6324482462930191, 0.47147543247173596, 1.1510924126183841, 0.5632575246658471]], [[0.5624527269119146, 0.3847974353590521, 0.4442869184193772, 0.7925514884630867, 1.380703616941982, 0.7005276383219756, 0.3995063344163199, 0.8509382843395155, 0.6158962242375161, 0.35088435246217237, 0.2877217033837769, 0.46318244409151943, 1.074056901955776, 0.3504610381230981, 0.4818085046578621, 0.4607248435161624, 0.41935348219136803, 0.6010426101895389, 0.7132057187534503, 0.3262643641401434, 0.31616367395311124, 0.580728097949921, 0.025503568635939604, 0.30638355168720877, 0.29455985673666935, 0.09526296243558408, 0.1701289637022415, 0.45833916806626407, 0.5463516314944272, 0.4536026616532762, 0.31584440913798784, 0.5484231789596664, 0.5605346848449355, 1.1832328030590722, 0.4934057315509397]], [[0.5779984903932704, 0.34003230810450874, 0.28517395861320666, 0.8281008406741395, 1.355171540907322, 0.6933491721333866, 0.41238371122711076, 0.9678326012388755, 0.6078376087165837, 0.3505564735242713, 0.3243502197365299, 0.4814930054324502, 1.1352913734234007, 0.37163826455514964, 0.44930338656364804, 0.4321728653876762, 0.41175888157425705, 0.6387753472013575, 0.7721392711117758, 0.3098542617252103, 0.3134525783033518, 0.5001458306513284, 0.030139003971799715, 0.289965610380996, 0.296370800673452, 0.1224451914878007, 0.1644608721072748, 0.4049713975264555, 0.4631457163203763, 0.4880726303537783, 0.3376541740655693, 0.5108608315477849, 0.6267807689366227, 1.1491189671574888, 0.48532848888162106]], [[0.6735061739581021, 0.43580506015147213, 0.28097089466066083, 0.8225684430421456, 1.380051911072655, 0.6903008994638073, 0.3926588969314757, 1.0083334216277235, 0.5377705805710105, 0.3602898592852809, 0.30978971935582467, 0.46609438867681935, 1.1939677861045381, 0.3827120855746511, 0.4421949692431839, 0.44957367274007143, 0.4026875825690252, 0.6709184358415738, 0.8409839190673332, 0.27987213024652574, 0.29144011996250874, 0.38115353307626654, 0.013786430843634583, 0.3291186604405758, 0.35722889878093156, 0.1820237732035771, 0.15364147241709927, 0.4124765562696763, 0.35689028186813376, 0.5187095875993457, 0.3436940524875213, 0.4905910928488199, 0.7134888656247199, 1.0769866868784814, 0.48354752399127565]], [[0.6839608323898314, 0.449328592310462, 0.24208477715092563, 0.7896494751868541, 1.3440033204345698, 0.7203429725432815, 0.37787228412165025, 0.9882162716896582, 0.5551566374299837, 0.33507722039066046, 0.3080877763488663, 0.561783044569309, 1.0834195926305907, 0.39018275735206476, 0.46904250740276227, 0.4607615255858485, 0.3930670141964197, 0.6426304073510339, 0.7609471290527108, 0.3265553402813, 0.3173083542953037, 0.29736805419328854, 0.011174768981915346, 0.3757474971942185, 0.5106370642066527, 0.28531303000556996, 0.09958824942445321, 0.5365409998023156, 0.30655572660769304, 0.5091126334970228, 0.32372430646765527, 0.45997985830624405, 0.7640221798030348, 0.8761684496213445, 0.5109835113357439]], [[0.7641180673552905, 0.38557930897239123, 0.23430385968869616, 0.8084764775003634, 1.2722799133028198, 0.768972998619623, 0.3880734926202078, 0.9688736602705423, 0.6361302840798855, 0.2796793349464643, 0.2976862819370357, 0.5603543406397198, 1.1449170533818847, 0.3973943381737009, 0.4847177485244323, 0.4850720428310279, 0.3821635293473292, 0.6452933786033264, 0.7610682308137776, 0.29755980060795034, 0.2988658373833787, 0.28946082911692006, 0.013201003794881452, 0.5342542818462606, 0.6248843458503024, 0.38865583437806267, 0.06291627684344259, 0.6691810695867064, 0.34417555942719197, 0.507493605670353, 0.2553103446428173, 0.42617784780144546, 0.7045465986375501, 0.5935593719028356, 0.5276506434586218]], [[0.7612216623499688, 0.40080842137057565, 0.21695436205743093, 0.8610293108502729, 1.2325665165633444, 0.7719132767963343, 0.3820171422734788, 1.010009709280286, 0.5310854066868038, 0.20856327537984948, 0.3034586980954539, 0.557778013130722, 1.061447979210046, 0.40068801483703076, 0.4680797238958393, 0.39751491243664583, 0.4096102118725916, 0.6807137084980491, 0.8174385504569472, 0.30139146896113533, 0.28720947539959985, 0.2620396538853297, 0.016759054265924264, 0.6375797998079, 0.8812864451951588, 0.5047200475712579, 0.1390324224456177, 0.8233815286242159, 0.35693581785845113, 0.5919753269622452, 0.2951836211157923, 0.39412870490205854, 0.6727547138480489, 0.6603401323630802, 0.47835177363952597]], [[0.7610401578089093, 0.37132540444123235, 0.22465774577779063, 0.849401165150348, 1.1951704856003704, 0.7647063619012268, 0.3806026703592014, 1.110410608035986, 0.5566645113456635, 0.1598793158091641, 0.3078989267562371, 0.5737164306015506, 1.1489518420238314, 0.4206246658188674, 0.4962693940118169, 0.37019628022254253, 0.41064512655254637, 0.6954371243015687, 0.818741290469893, 0.29891398942261177, 0.27827896804899865, 0.29557221319969074, 0.017514126082618427, 0.5859760658766295, 1.0974056676491786, 0.5607149558579445, 0.29267205256337964, 0.8776572419532378, 0.3569853192749298, 0.67252439887762, 0.35495970860227144, 0.3729171881523417, 0.5985814079298856, 0.7811479000854544, 0.45730731757197723]], [[0.7688258895744167, 0.3941003516184025, 0.2417117940219928, 0.8192404480230182, 1.14483922668492, 0.7262940123739938, 0.38046026868096033, 1.1685993137297184, 0.5170364245774812, 0.18068256003724176, 0.26859452206425505, 0.5854455574502319, 1.1063706327855667, 0.4232522557501018, 0.5051152406002086, 0.3627479674746763, 0.4053571556856105, 0.7032190483681073, 0.8631258748171156, 0.2839220448758608, 0.2685416472248005, 0.2544646182249888, 0.021967877521537928, 0.5409808390097488, 1.2665780470963555, 0.5037316226110439, 0.2592700537496849, 0.9253481077015638, 0.3353711075171879, 0.7076609742575, 0.4044125703944895, 0.3697006012297985, 0.5142389861506349, 0.9074957611335612, 0.4634452726773985]], [[0.8238196557093538, 0.4080127748856144, 0.2475392793324915, 0.9149625400401898, 1.0569685055666795, 0.7500268918772756, 0.38560575188539514, 1.2386585295347363, 0.41978925724499194, 0.32017920708315606, 0.15935587942617435, 0.5769343583386541, 1.0558140219326368, 0.41043078201674166, 0.5365097340650005, 0.34969402921565174, 0.3847352174429359, 0.6850528326744576, 0.835656564431509, 0.29261823484553967, 0.24639720185775132, 0.23964866716999123, 0.038179220805417816, 0.45935889664620666, 1.442246293697667, 0.4817816468206338, 0.2209439053587732, 0.9342328667327677, 0.3373106003567384, 0.7083346149357721, 0.5226886188869726, 0.43124551276800704, 0.6049327753624086, 1.0074223018731754, 0.4551937356301042]], [[1.0376856515984976, 0.3867305263707611, 0.21931131552441113, 1.195669012233076, 1.0990343862693925, 0.7391532954763758, 0.4043157445017484, 1.2665160335395909, 0.43676690899412773, 0.44184356724058327, 0.13979647355317049, 0.5806900783973381, 0.997758738138415, 0.42762734770320787, 0.552285013546294, 0.3503363402418357, 0.39926937506444415, 0.6897922898491974, 0.8278721114576646, 0.27558792551816624, 0.22248294639001948, 0.24257642639174357, 0.04173413047172937, 0.40142589863653666, 1.5609617980019457, 0.44583427456513913, 0.20640716550294216, 0.9191129983758556, 0.31733089554689287, 0.6926364991930586, 0.6289622579212997, 0.42483713365203307, 0.7209197358415693, 1.025254330553765, 0.46415334810096187]], [[0.9259456455322805, 0.3822304476769781, 0.2198778150698874, 1.415663889500349, 0.8123296900595465, 0.6887035909043817, 0.3879559101575831, 1.2768513213751984, 0.4758183865593434, 0.297471622308106, 0.13507367217696215, 0.5648990762639386, 0.9296399374181363, 0.4169680347510083, 0.5325135372703784, 0.3851635189378192, 0.37093132268930307, 0.7404992023678181, 0.9468465077921001, 0.24157300657841735, 0.20483284858489953, 0.19186801537980186, 0.09093550723010152, 0.351284968552726, 1.6808507822547276, 0.455680259683812, 0.1880117145748519, 0.8968884921201828, 0.2541068715222496, 0.5677896184462472, 0.681933312384359, 0.3797351654375241, 0.800891589010758, 1.0968163953658232, 0.4445392840638348]], [[0.9676847130904116, 0.36516993458969516, 0.21760085526235415, 1.5415811299695317, 0.8057506970192041, 0.6755210158512734, 0.39394607177746016, 1.196775393673601, 0.4573385188533321, 0.24015938918459964, 0.15639405834221806, 0.6058487655829019, 0.9261206986129297, 0.41046252009740225, 0.5211914200120686, 0.375711092966858, 0.39417844289614223, 0.7844735441235475, 0.911849641099356, 0.28541035222607125, 0.2027502751923876, 0.1947663814583039, 0.10432745745447272, 0.3068467990775524, 1.7617170620717977, 0.43173857926837256, 0.1395186016834088, 0.8879094097799604, 0.2230979348947805, 0.5090959184965029, 0.6937983077475997, 0.3212055038419326, 0.833595975426584, 1.084248622918318, 0.4412012810147551]], [[0.9683866271966282, 0.3147058840775343, 0.1947333529972386, 1.6740758005788858, 0.7759906975521419, 0.6333840136082081, 0.4279921003507478, 1.186075752427937, 0.4356976614154852, 0.20595255628678039, 0.19413232893543964, 0.6650091506457687, 0.8783717818915374, 0.3913034129679947, 0.34736640283912057, 0.38063583884497054, 0.4168761279009502, 0.8527268856747539, 0.9390106268798737, 0.3608052876520831, 0.2650119902724099, 0.16446759921421772, 0.17929911000719104, 0.24047453150760079, 1.6795777916172487, 0.38633920872495364, 0.13644265879566259, 0.7351664635399384, 0.25162156634033506, 0.46729168828826767, 0.7730300028946857, 0.3313964024934427, 0.914995797435959, 1.1360224198169748, 0.38595874810419206]], [[0.9913630151498456, 0.24979742741570998, 0.1596570455980776, 1.783135254459069, 0.6865114743160516, 0.635667247431247, 0.4244763674480915, 1.1960879669828697, 0.4341123578545794, 0.3002055955724988, 0.19110661901372938, 0.6552812575389637, 0.8534039542846839, 0.391183258318008, 0.35170376438527495, 0.3840949653378626, 0.40347101137964, 0.8300315221651031, 0.9308818679960971, 0.39057871818290285, 0.3601072519493784, 0.22035699748467413, 0.176002412324053, 0.23585335686701497, 1.5334575964982968, 0.39132688566784846, 0.1292901950000372, 0.6173843540407495, 0.30336198005076365, 0.4782312774670404, 0.8419170465613532, 0.3898093628740416, 1.0535720386136807, 1.1784890322900763, 0.36769151064346683]], [[0.9784124901348532, 0.24180703638738416, 0.16042998086483795, 1.9533079871558674, 0.6252453275294938, 0.6465004133079979, 0.40708388102779486, 1.1870560581082357, 0.43969528522243323, 0.5288005538274888, 0.18824563215266266, 0.6819019270747226, 0.8152578023091182, 0.4409223872249405, 0.36371801775844464, 0.40420085962869656, 0.4218732716751962, 0.8608827257836097, 0.9180776107891351, 0.35494689934629226, 0.2956824344359068, 0.32778131861253446, 0.17401054312305422, 0.24706753014103947, 1.285920475953342, 0.42927777528659494, 0.13956691647743721, 0.5141799255894256, 0.37368017575638357, 0.4921527339508517, 0.8983914428394024, 0.4675403618770517, 1.1285580177762835, 1.1797189402097936, 0.4126409751760691]], [[0.9183315905698414, 0.26121397180085654, 0.16205625842555027, 2.0781855233670323, 0.5333486533175951, 0.6339888606791524, 0.4082551145902205, 1.19533142023945, 0.46329232722473024, 0.6346803247149375, 0.18364495815510934, 0.721185297572299, 0.755807640573716, 0.4266562750937377, 0.4096099206576511, 0.4261692273095349, 0.43488188764648256, 0.8976099316894276, 0.853343755816901, 0.2967928847988358, 0.33504529362652474, 0.30213342465071136, 0.17473582854731512, 0.27168729520210994, 1.1895013097716265, 0.44116955043074013, 0.14703042652211928, 0.5010704448009455, 0.446480749842517, 0.515736500439386, 0.9398891838202676, 0.5621268184476124, 1.0614697273196885, 1.2115880560728434, 0.4927178932107164]], [[0.9030633619418315, 0.2519540714290422, 0.16450547785414255, 2.205948548215972, 0.46065172840317875, 0.6340939420965986, 0.4065292822346008, 1.1664860148061171, 0.48567866551263356, 0.6811400794095919, 0.17867229667109524, 0.7620438266598433, 0.7240213574404696, 0.40200972291728965, 0.4259441664067542, 0.3498728560283342, 0.425079355017442, 0.8169297113415188, 0.8635666651019045, 0.26356724416477617, 0.3350336568024921, 0.31575285246740503, 0.16650524802261507, 0.2917086858254168, 1.1293850013974578, 0.46779713675523915, 0.1612792278694853, 0.4931702896853431, 0.4976849486621735, 0.585721481231663, 0.9751201545915598, 0.6545761919508121, 1.0456386611315345, 1.2201080528579185, 0.5393043513920986]], [[0.8485133767719901, 0.2639189759833236, 0.17578869467287617, 2.2475971192668966, 0.3951822323163031, 0.6253012259874917, 0.4098364501346632, 1.1008925476906133, 0.5166327326454243, 0.7163571354663328, 0.17701130602186152, 0.7640531810415159, 0.7140388949128892, 0.4050654591049938, 0.669216068893369, 0.30826051668845267, 0.41363012710957126, 0.865013364208679, 0.8821521494050594, 0.3181840977586142, 0.3471189855293892, 0.34229374290011005, 0.1627233826907392, 0.3194719435654617, 1.0966736673194615, 0.4608348749772837, 0.22388763226538055, 0.4463399339932448, 0.5383319184715075, 0.4546284822654201, 0.9708347798419961, 0.7484455647790742, 0.9765802181298411, 1.1557891710442127, 0.6026649732912903]], [[0.8448640936072245, 0.24401979463414053, 0.16282081050834132, 2.2687600847630915, 0.44317069333531395, 0.6343906257189829, 0.4124674630662414, 0.98448920440492, 0.5391385307435906, 0.7875355381451954, 0.1464657009504795, 0.8347615145518863, 0.6750136338903242, 0.4069330646977847, 0.5815767157668028, 0.3150747211275855, 0.4218133594058861, 0.8790283113106641, 0.7649083460335775, 0.3461752587903488, 0.33828627998016686, 0.3389518198934823, 0.12701260927646868, 0.3557150208557524, 1.0865928709966806, 0.2650998898857101, 0.2758646636047184, 0.447041971087805, 0.5739209685269397, 0.3092050230179608, 0.9501535983231926, 0.7909646560093091, 0.9355544965602062, 0.9555149160292866, 0.648669463448901]], [[0.8982719284254229, 0.24660079211204555, 0.16034180616616078, 2.3336311592946823, 0.32501374300337815, 0.6390072126031767, 0.3876290089284137, 0.921920622463121, 0.5400497254601461, 0.9890989907658362, 0.46027237225459966, 0.8099644563683921, 0.6594145205353991, 0.4077389954666496, 0.7226842577360126, 0.3110361319048733, 0.44411964334769616, 0.8303728980779481, 0.6753305378766019, 0.355840863781648, 0.3816341482563934, 0.35054327470053515, 0.40917056291584614, 0.3793755867844546, 1.0592729869523234, 0.15125496856662696, 0.3459633833312763, 0.4667382658181635, 0.5993752277226579, 0.41480193624677775, 0.9105867726894457, 0.8226113173668177, 0.9340499760294607, 0.83537909971762, 0.6561355064679608]], [[0.8359839732117582, 0.3026966820176771, 0.21243947238062488, 1.848059421505749, 3.7882351812831665, 0.6087562432547413, 0.3852275702097628, 0.8783725031374923, 0.6010202900088235, 0.8946862680124702, 0.24664493150665395, 0.7884922867894057, 0.49690271015223064, 0.42482559877047243, 0.6605253809544023, 0.30586067550200735, 0.4223533622736323, 0.7762157795524194, 0.6701668493737633, 0.30464946900688417, 0.3555464045999226, 0.2385369619851644, 0.10375994188993622, 0.35233339646158074, 0.849162717039704, 0.1128694181874301, 0.40132443354765257, 0.5058312982446476, 0.5806902295891533, 0.5018074221584403, 0.6920670478966604, 0.5800127445149449, 0.6188831808955475, 0.5824215994344101, 0.5468369540325492]], [[0.5361247683937495, 0.2294308664956537, 0.17338849994609876, 1.8904670964599444, 3.7882351812831665, 0.5964606420029458, 0.4042490451531204, 0.8846461316460548, 0.36779899616336775, 0.8747810683378325, 0.3418320178695603, 0.7623750914213732, 0.49141806021995893, 0.421928665363939, 0.6455799331840968, 0.3120848238624332, 0.421811802783652, 0.7870103126885034, 0.6630520364102509, 0.2882178188963198, 0.36179707693901697, 0.25341252462470465, 0.1689456415312745, 0.41287381837034, 0.8409229928644679, 0.15280982484724565, 0.4396416794705542, 0.5263167457774635, 0.6317506050023393, 0.5942679870597423, 0.655438536338018, 0.556181372530454, 0.633843983629545, 0.5651396473916863, 0.5317128374920355]], [[0.45945919288480963, 0.22284638023523695, 0.15886711124782044, 1.916339947722072, 3.7882351812831665, 0.5826886435541919, 0.42008838533533144, 0.8835985843418641, 0.39175462828725116, 0.7928788217628695, 0.48636741725724275, 0.7565006598408743, 0.5236744125246352, 0.4486315430034985, 0.6660007405724895, 0.3204073679952088, 0.44604464226065754, 0.7730304599502952, 0.6595723216802257, 0.24338769207895605, 0.30896993670000106, 0.28344566311773645, 0.2664544285244198, 0.46827179949611974, 0.8022514827044294, 0.1881772181416918, 0.4674082609418299, 0.5333206193302744, 0.6503853230899467, 0.5532868322661281, 0.6160261643204606, 0.5281998653547917, 0.6200925034631803, 0.5237600722949185, 0.4985668275861355]], [[0.39924716003763805, 0.2817094341919505, 0.23038806858255176, 1.9645175978472744, 3.7882351812831665, 0.5754728512590666, 0.4401722923808178, 0.8755535659525845, 0.35538374107300735, 0.6566629536395836, 0.4676229209249575, 0.8827568872798933, 0.5778997234762642, 0.494540728522983, 0.5502951610825078, 0.39837056263398263, 0.44181195868386136, 0.7986716998436375, 0.7285941111796211, 0.2237962731483809, 0.2819316629690829, 0.24258391258499049, 0.2691620258747912, 0.49944179312955145, 0.7230404822679982, 0.26298396057031326, 0.491203574342839, 0.5239303950850147, 0.7224137085430016, 0.569657539427247, 0.5827319284705595, 0.4983993640532032, 0.6352948245483395, 0.49170094329685365, 0.4630566648602873]], [[0.43510311117788925, 0.27426216139749116, 0.19898558738593747, 2.218569287792232, 0.4313849097606528, 0.5823191231222762, 0.4786068396801431, 1.1208614633555805, 0.5717526117203959, 0.9274339513934704, 0.723206803479892, 0.9791222083810946, 0.7331760670189088, 0.4713562218662945, 0.6320918421643011, 0.3477977769646311, 0.4808897086784666, 0.8215846620065673, 0.6647532878680027, 0.20890915234044383, 0.2731826999119782, 0.3510910769553357, 0.4335703760275554, 0.5989993907418707, 1.1225568498821379, 0.3415956270368408, 0.5073711742371405, 0.4647344597185471, 0.8195863635223487, 0.9225072789242046, 0.7320430422691834, 0.7358355172792416, 0.9385463505422054, 0.8152000554979518, 0.5389961168386994]], [[0.34839565058360966, 0.27282384452724784, 0.2025791175043033, 2.2051223334163, 0.5105552304769146, 0.5758887583175346, 0.4231415293031948, 1.1647078278464422, 0.6424788009171876, 0.8003828560501932, 0.716690728084305, 1.0425166541492406, 0.74207263954827, 0.4560668788708841, 0.6796359699505194, 0.33532011839813003, 0.4660779753733568, 0.8437346498085924, 0.7115795152684536, 0.19125050119828546, 0.31062428189893737, 0.3397699114213026, 0.39000442939609653, 0.6359715188778814, 1.128712424918326, 0.38375930520222906, 0.5103803174607857, 0.42963860294802547, 0.7709420609506893, 0.8324887150552078, 0.7320029122640912, 0.7300564572593677, 0.9591421322831544, 0.8635867077109223, 0.5327121885371705]], [[0.2627067299614, 0.2807771572126482, 0.20847612155669631, 2.1932684284068413, 0.6804418079427721, 0.590478880576306, 0.4335340369669429, 1.1940974168336678, 0.6772949709833795, 0.6891817350743402, 0.7412031498972158, 1.07436085024447, 0.8205734282913923, 0.43258881527497367, 0.5944183542025998, 0.33991812083622797, 0.4485468410504653, 0.880365246813716, 0.7497575743763404, 0.20255100037491336, 0.36246823591947797, 0.3303437976488209, 0.3913812722893659, 0.7017834618645861, 1.1353013981375484, 0.4057083144149155, 0.5333336847835987, 0.4540634679551486, 0.6768597848966733, 0.7901933598939926, 0.7572057331289346, 0.7183797921774577, 1.0191236207244128, 0.9264600124823721, 0.5498072552833554]], [[0.2663724650843742, 0.27822209692878586, 0.2055481364064698, 2.1737408702289707, 0.7959767151043395, 0.6151716321484808, 0.4428229642278011, 1.2012235338264068, 0.6971285691525806, 0.6622555369668429, 0.739404502526217, 0.9515092788329762, 0.804114260062402, 0.4191863543028881, 0.5374817155537641, 0.34648972932801414, 0.4153681835165739, 0.8942296479288829, 0.7572147850500737, 0.2692855532771202, 0.3835209363947873, 0.375968638993715, 0.3871815331658218, 0.7702402381234601, 1.1327175436766233, 0.41648381898270764, 0.5799457616913416, 0.5011428321410825, 0.573276248159189, 0.7696600548331288, 0.7384875807841709, 0.7022371814632073, 1.0392694791848889, 0.9843743281653723, 0.5577210796412564]], [[0.2490447250970386, 0.26453269499317766, 0.18537164634051892, 2.148293691367919, 0.8773265610810175, 0.5950823320479632, 0.44479466594065775, 1.22267375544657, 0.7107706574761976, 0.6858655680526189, 0.6317218422577464, 0.9910796232922456, 0.895236260891704, 0.3812334055106998, 0.7451986541973796, 0.5251581935843158, 0.42856576232038224, 0.9329222065563118, 0.7449278579891103, 0.248998762309261, 0.3304313259594943, 0.3341174376024134, 0.3587156711799156, 0.8123423381423994, 0.985311352785448, 0.45888304036318295, 0.6151738848515578, 0.5484861506587204, 0.5842729916990499, 0.7557861299268028, 0.6829963932553029, 0.6681078838285038, 0.9749963451016641, 0.993772043313432, 0.5186787413080169]], [[0.2430348570445909, 0.34034539738522873, 0.28691136469560763, 2.0720330002209004, 0.913701538079676, 0.585024646811672, 0.46849167385851725, 1.2507941696757119, 0.7405920949528604, 0.7046112811913057, 0.5094700570187146, 0.9626533886296865, 0.9390137044402075, 0.5144301401052489, 0.8415268307918602, 0.5181056076865079, 0.599457196283962, 0.8834919800738295, 0.773768168261421, 0.2289884789398834, 0.31361328303541447, 0.26364325063459204, 0.25654588635972386, 0.8490768867251175, 0.8519213027886401, 0.4152126206153241, 0.6483979550167396, 0.49368511094652795, 0.5966610163286206, 0.7806788684558621, 0.6459209314611212, 0.6598649944201466, 0.8506873043787979, 0.9394633626531481, 0.44375084565842393]], [[0.23199655564257377, 0.35741747597744444, 0.2550200896583277, 2.100294317662565, 0.9567162405370511, 0.6029852135958439, 0.4155979550668788, 1.262985113961995, 0.7709710442005382, 0.7252181164688454, 0.31840292752982435, 1.0012784562064287, 1.007581723405286, 0.665927533794959, 0.756689019236535, 0.512705394363991, 0.6018254311212174, 0.8271094490696225, 0.6438529331529035, 0.22002141822667257, 0.3144893083880092, 0.24186448223021967, 0.17111292750166604, 0.884877292756075, 0.7059275775831148, 0.4388347120518945, 0.6914782442917524, 0.44015919225946387, 0.6300381310742202, 0.7708296737246445, 0.676609109161943, 0.672666763263414, 0.7182400864150138, 0.9238790426098882, 0.3223118029733226]], [[0.22588921353329516, 0.37854202105551193, 0.3439922814383013, 2.0299485089592477, 0.9692737988724394, 0.6146460787493, 0.3866207687174784, 1.2676507493869202, 0.7996604691701655, 0.7593943314969558, 0.3106794131063009, 1.118045826434774, 1.0009422194047795, 0.5548874888920645, 0.6539068341718208, 0.43343610711469893, 0.531889312171546, 0.710439324551046, 0.5581393880095745, 0.23325778776849593, 0.2939943490018007, 0.2819735625573293, 0.20551470904246139, 0.8760668875309441, 0.5758518875820522, 0.4477943475392251, 0.6616715898376604, 0.345326860052877, 0.6911324892241911, 0.7541392813478163, 0.7055628589842603, 0.6894441813443297, 0.5528375728511935, 0.8782324221339899, 0.22776106004973642]], [[0.21436128389975048, 0.361889766362948, 0.32244739478579965, 1.9774534928399023, 0.7988575663071611, 0.6262712713657584, 0.42635431798862006, 1.2473878131446288, 0.7815673266836142, 0.8374453297391916, 0.315278205018867, 0.9634421058276016, 1.010659145467706, 0.5495368274268606, 0.6061236616579488, 0.445617764939189, 0.5109119577878074, 0.5953905379523674, 0.532894518805852, 0.24304918173424153, 0.29392532479997024, 0.28831990583864897, 0.27242065894944134, 0.8260601180670902, 0.5005103901845415, 0.4808502767860511, 0.6455886604684179, 0.2246513924962044, 0.6755631479005229, 0.7368180461699501, 0.7384665796531815, 0.6816756618348645, 0.4170592337264819, 0.9602868239975961, 0.17387847938122106]], [[0.21633860998857737, 0.37400281395693635, 0.3866484520034373, 1.8715093686423365, 0.793463727273969, 0.5987779103146114, 0.4631120010904268, 1.2498004988261617, 0.796831811741917, 0.8438735868500007, 0.30281832335083236, 0.9477682706265553, 1.0824021670198947, 0.5588828272100075, 0.5629670628453931, 0.44529837545557727, 0.5638725473841507, 0.6200154044679114, 0.5600415761311492, 0.22091723127733523, 0.27862001459550906, 0.28305142071890965, 0.3060994183741, 0.8280983514495079, 0.45274526346885846, 0.5129637838545509, 0.5787376152343758, 0.20086202100303113, 0.6527013322976258, 0.7216189061792159, 0.852391186098983, 0.6620999363785731, 0.36356607157635945, 0.982794454226942, 0.13386344934839073]], [[0.24628211594593605, 0.38572026202625664, 0.3931720086866794, 1.754178077931934, 0.7930009515816416, 0.5378603285704078, 0.4174526454055618, 1.2389488231908978, 0.8023781840649034, 0.9223377593311919, 0.2866898660814115, 0.9042067264829785, 1.1348796919390818, 0.5335455316628481, 0.6140190071967098, 0.48887074127893393, 0.5548330230215678, 0.6476839807688779, 0.6278741909897416, 0.21567392328800697, 0.26682342692016414, 0.27039894450609836, 0.2216397420020228, 0.8036116880779131, 0.45957010522407027, 0.45466849994540925, 0.6483247972610111, 0.1873663175248706, 0.7245064916036127, 0.7116926507019996, 0.9157884488331302, 0.6862265013990397, 0.5617930041219978, 1.0275677973264343, 0.13712846310763166]], [[0.2600614765161919, 0.38327925649002054, 0.40989687076211706, 1.7656116385057297, 0.9147514104800905, 0.565712510279521, 0.42262950757773443, 1.1812809882390192, 0.7879386416839453, 1.027668425393995, 0.2724301281477414, 0.8968929333693296, 1.1734914440314088, 0.5363695530701681, 0.6508027483860446, 0.5064198828356257, 0.5583601020973264, 0.6352881887761908, 0.5995982092003336, 0.20419558961104856, 0.269932827720114, 0.2750380147173959, 0.20970070526411982, 0.9442543831215195, 0.4919716942717788, 0.4109235564499866, 0.7789661008098437, 0.16085863831849756, 0.7358071009439102, 0.6493830756259891, 0.9862673164888529, 0.7540199342429494, 0.9122230453540241, 1.113276515308856, 0.12778287261068544]], [[0.2568654932930756, 0.3805921963507579, 0.3993399107192481, 1.7330509429854357, 1.0030311300497432, 0.5843709983669463, 0.44375926109946906, 1.1144088988396013, 0.7827386996818125, 1.154730074976098, 0.2625940770882315, 0.8888740049752815, 1.1498642695297279, 0.5344881438015067, 0.6272585889073459, 0.502931705290184, 0.5112720803449228, 0.6409371913059632, 0.6050733907086826, 0.20692105426870755, 0.29035165546211605, 0.2864462324692217, 0.16649139920962955, 1.0538271454464345, 0.5294871454577009, 0.3653809243082706, 0.9054494316288189, 0.16667096015660948, 0.7219245746017451, 0.5945850949050913, 1.05552677862198, 0.8317803847784586, 1.278835745426735, 1.1971916267177476, 0.13098313659890293]], [[0.28880770472841044, 0.4078272814316253, 0.3932589146961255, 1.6439777197794259, 1.043882810142787, 0.6009220972303502, 0.46133494354228605, 1.0671966111709057, 0.779366247552462, 0.9235723164401063, 0.2430501152421879, 0.9130174740723915, 1.1853425394295498, 0.5487395232402361, 0.5551944889580789, 0.48490508047458847, 0.4895947740107977, 0.6470404444641746, 0.632370395043736, 0.23140085039601777, 0.4043712975268331, 0.2804993264178922, 0.18197981625547976, 1.0716089806402014, 0.50070602413867, 0.3678820175477804, 0.947521978616052, 0.15920960515634755, 0.6508552000293412, 0.6157972784524506, 1.0690028973119419, 0.913218703673379, 1.3312718928731408, 1.2702839070573777, 0.18316625185273722]], [[0.26463757513653885, 0.4412354313799438, 0.3756425604206421, 1.5829306168484147, 1.0591563686538361, 0.618138250460351, 0.4354166955215151, 1.0652980716554215, 0.7754415108001291, 0.7354958439574264, 0.2549257439478289, 0.9436746770358977, 1.2640443497370775, 0.5548016551474735, 0.5518676506846159, 0.4896754842206063, 0.5665322011394835, 0.6527592115023538, 0.6203093792501202, 0.2182830765913938, 0.38111750393038907, 0.30019576339699455, 0.20017037009603117, 1.0509800561603075, 0.3945827905678778, 0.39613418762856156, 0.8902291189411562, 0.17212420668419365, 0.5837558755746812, 0.5964435180784124, 1.0358426829953755, 0.9214427224352397, 1.184430536635996, 1.3387940719831364, 0.23695863953631757]], [[0.2861460556855573, 0.4331197542724235, 0.38577232700597763, 1.6140387692715916, 1.030380600734558, 0.7348688191807808, 0.503425823943697, 1.0543900210797017, 0.7504244711885842, 0.5102184187075731, 0.23346406608150114, 0.9328685569812183, 1.3372948406207994, 0.5348785801780259, 0.5826190086497873, 0.48742212963067555, 0.583249803265624, 0.6547885798346282, 0.6088389459619337, 0.19890447297047453, 0.3176145867739733, 0.3402071579511916, 0.21800112282061473, 0.9495741582700769, 0.33800945572942187, 0.5196252079024082, 0.7703686368189222, 0.19500430477564706, 0.5526403073447896, 0.5167539922405726, 1.0238473809236017, 0.8685022558225769, 1.0362741729496077, 1.3457132406446948, 0.26106040028025207]], [[0.335427498748331, 0.4290358503592748, 0.4199338597664758, 1.428048257950794, 0.9760974545834818, 0.6237738751214696, 0.4627307138890364, 1.0136506301356325, 0.7462548345646739, 0.4857878883345746, 0.2242804965199925, 0.8985090232048332, 1.331507012398991, 0.5292054075028045, 0.5418048673735101, 0.4713361759942297, 0.5865568224354857, 0.6880874548260654, 0.6645335629494576, 0.1843871758786052, 0.27907448869373763, 0.32938179353802544, 0.2266552730001058, 1.0390127910369333, 0.34189946852300845, 0.617006807044888, 0.6350944184504218, 0.3366690263818421, 0.7117837040835596, 0.4497041062253943, 0.9890092815134044, 0.7201361289869208, 0.8399990960585353, 1.342135988463358, 0.27013918955572275]], [[0.3619466554502838, 0.45533260909819967, 0.43914473175290414, 1.260100475907034, 0.9583777935611894, 0.6426642428826999, 0.44115733720463407, 0.9248265205585355, 0.6695059736449103, 0.711447090506516, 0.21712054127521024, 0.859684198827591, 1.3773024870681652, 0.5593573228684725, 0.533215135709266, 0.47028619736783106, 0.48315681002715316, 0.6099702470425556, 0.6639171909632677, 0.22759316555010622, 0.2667805054543815, 0.3164150415876023, 0.14475661725385858, 1.2383235008456557, 0.36761819014466973, 0.5564659035653141, 0.6555117205895479, 0.5860432620818662, 0.701608675718173, 0.4971684349308411, 0.9352107552519713, 0.6692072786395021, 0.8383977549221802, 1.3567518263651859, 0.2887530463388014]], [[0.48552701828195216, 0.4541893675949136, 0.4326568423318039, 1.1974430721557514, 0.9770441346192692, 0.7354770867340926, 0.5294934668176583, 0.8058152048006662, 0.6926330064153696, 0.9035311494803115, 0.2063367984476932, 0.8448198350620078, 1.3952584357753917, 0.5543860007407015, 0.5142795595600946, 0.473867495277952, 0.4651226934021635, 0.6382979023762358, 0.6828342416955129, 0.21284708820840634, 0.2123039178978613, 0.30064243628970855, 0.1286084978506375, 1.3063302266211823, 0.4173405741712487, 0.4615728809260475, 0.6012222720342324, 0.7777729972751212, 0.6795562082779198, 0.5116029958087641, 0.8458493672246465, 0.6963650343700464, 0.8092208349177493, 1.3070101203162774, 0.3004670433980977]], [[0.475904847631353, 0.45152966731649796, 0.4217959507074055, 1.1469043858865513, 1.028636110388248, 0.762022548607338, 0.5393895015435866, 0.9422889871860444, 0.6413263036217272, 1.015341229466773, 0.2013928975166141, 0.8770240152394018, 1.3881850381070298, 0.5640557244656272, 0.5296145462343012, 0.5086252347364599, 0.4658783931458538, 0.6803992490157864, 0.6764470361277528, 0.18753758278748744, 0.1385010488468353, 0.29547515839669053, 0.10324764868344162, 1.4518621892279162, 0.5304081492471427, 0.44772407479507786, 0.5877681594669855, 0.8819322652866455, 0.7071204907087809, 0.5419089002227175, 0.7131057751095642, 0.7394593469170161, 0.8196688534445994, 1.2850047919831906, 0.30818420710419003]], [[0.4981206806717565, 0.4438966746235115, 0.4038778905703895, 1.207251314335654, 1.0422051010462368, 0.7422916971097564, 0.5190019964897215, 0.8864237207729458, 0.5933605075149544, 0.9283706544181267, 0.20635561407111547, 0.8217317642718728, 1.284479036197887, 0.5742553991875943, 0.5626945863364097, 0.5457556109214421, 0.470148618121923, 0.7110136525307253, 0.6877971304036821, 0.20545156527274586, 0.14390770435760844, 0.24589684793581584, 0.09421516260832041, 1.742995770735157, 0.5811151614320614, 0.4121100635819259, 0.6531789830263781, 0.8539964533753173, 0.7245637692964202, 0.5207660214515905, 0.612429077106537, 0.7706841787362837, 0.796622479239304, 1.3157260835760982, 0.3145268855320017]], [[0.38915162773766704, 0.44169065770047433, 0.39354734327092056, 1.185914118615735, 1.0600156468744648, 0.7560821252296945, 0.4732214326762443, 0.7399902626009999, 0.5430986769740517, 0.8424552347861617, 0.19491935551145625, 0.8190097035863017, 1.2457010421590515, 0.5531985820412568, 0.4639555652998095, 0.607015084362435, 0.6299895770141659, 0.8995931258221533, 0.7932192328655461, 0.24406975473320622, 0.16672950992134133, 0.2033791800748731, 0.07419149397210817, 1.8749986379638435, 0.583447994292954, 0.4002750919352351, 0.7637880427868283, 0.8360976915025184, 0.7191322678594934, 0.5000147342363583, 0.5415696544315227, 0.7929459041516818, 0.6382267640139593, 1.2939023716621578, 0.3430138435684903]], [[0.38000279895495886, 0.44806782985444077, 0.37766622726743093, 1.2089743589473774, 1.0452970300165996, 0.7427463228066054, 0.4355947188296255, 0.6364017613733823, 0.4896445526429845, 0.7657022237011042, 0.1725358979483065, 0.6839975217514584, 1.2354915769752122, 0.49843448101211674, 0.34139327659873125, 0.5397299185454988, 0.4735654476905389, 0.7989727804850675, 0.7987443175506153, 0.22027508534795423, 0.15952395416531162, 0.20138184455803448, 0.0647689633569828, 1.9266995335972799, 0.5967036639572715, 0.3693656327412493, 0.8052669270077597, 0.8912614773904283, 0.7237709104330892, 0.4454591689855102, 0.42239675728021775, 0.8019598923769242, 0.639618885125841, 1.3115223522409907, 0.3660585349482075]], [[0.3739181491041754, 0.4369110534819577, 0.36760543196138235, 1.205971456297639, 0.981792707466419, 0.744642783447284, 0.41721018964035417, 0.5821069751823273, 0.4457410926651143, 0.4964742149104381, 0.20274758979745852, 0.6615851082894617, 1.2999028911285864, 0.48371025205995016, 0.3463519144113768, 0.5611727849605705, 0.46054237707391804, 0.7662138382359611, 0.850282561276422, 0.21820904209487307, 0.19714910382763354, 0.19296238079869518, 0.04718881312023619, 1.9477115210097316, 0.5984094748207913, 0.32968314621169426, 0.8286927674648237, 0.9161185321216626, 0.72869225817852, 0.6218433907429584, 0.4087964378718115, 0.813398264513294, 0.6053623501230502, 1.206271946043464, 0.36709834668011043]], [[0.41484355009111873, 0.43504944806058615, 0.37045856103372315, 1.219009840742838, 0.9395865170633553, 0.7842136589920614, 0.41831029119732666, 0.5704777695099794, 0.4068263342679826, 0.3220902698020595, 0.23660091771621716, 0.6662902751977898, 1.3162792056384864, 0.48014509507341496, 0.3430939418865957, 0.5491860584884944, 0.4042284425132159, 0.6889012745995189, 0.8710670847138582, 0.2075368624688084, 0.20493825454431192, 0.24894008709656681, 0.04457737574666377, 1.892383142637513, 0.6489258488925674, 0.3360910678884065, 0.8199316816712431, 0.970955250621935, 0.6982505945431092, 0.7087480800191823, 0.4430124371548431, 0.823158063554057, 0.674137976562394, 1.141361389618997, 0.40386724941065816]], [[0.49252710706733815, 0.4695193421318741, 0.3455240186610097, 1.2476950700898453, 0.9132763048431048, 0.8192909051862789, 0.4456530529655885, 0.6053517208651801, 0.4973339825958423, 0.31377646651092994, 0.21312331314370314, 0.6732832615955617, 1.298940908276887, 0.6122884567202196, 0.31871176340463453, 0.5702041658627238, 0.4529524778385389, 0.7029766815743852, 0.8196878930192131, 0.20542025389459284, 0.22814921074813765, 0.24696379570916915, 0.045562263503879416, 1.7787513093874168, 0.7401900964854393, 0.4076686291695797, 0.8319668648630398, 1.047466836089198, 0.6448669024932012, 0.6527503973885267, 0.4816461979143627, 0.7601738285884596, 0.7198451136564015, 1.1771060728642637, 0.4044384981274294]], [[0.9515196441736214, 0.5339377462889554, 0.3334260863428842, 1.355590126435127, 1.007000939927972, 0.7765229052330024, 0.39259007132034923, 0.7477600687985663, 0.640705438797431, 0.3206082334282459, 0.2108427845164057, 0.6701575002503286, 1.248633731990943, 0.5984759303595082, 0.3249839496798634, 0.5323994791916297, 0.4501318639367865, 0.7005183914848709, 0.835956218588622, 0.21038601814525007, 0.224491730340371, 0.2172259066638656, 0.06919926564732197, 1.4371885600593484, 0.8526412642035281, 0.37001299928682024, 0.8421159213478316, 1.1754488370069132, 0.5595673679975828, 0.6557784556674033, 0.5736652117082692, 0.7381032028940726, 0.7386573737191335, 1.1704583262742523, 0.37067504683267616]], [[0.9507796165845714, 0.5998356395574712, 0.38518356674216425, 1.4012421936463149, 0.9946231702739146, 0.7978159876019584, 0.379175886169232, 0.6836280455039664, 0.849918075231284, 0.34004216127013287, 0.18022203586427316, 0.6300077591967114, 1.3271861171591612, 0.5903618903540836, 0.34598811331054447, 0.5531572195444495, 0.4872682223601434, 0.7480854392362416, 0.8451695255317231, 0.19772572475584552, 0.19799941621093278, 0.19041242647763176, 0.07460620277546405, 1.268017141950039, 0.8252552257621351, 0.4787518443809468, 0.7532625606219834, 1.2018544510370834, 0.6136575522218164, 0.5487803408134044, 0.6380242375781067, 0.6862111933671372, 0.7195290121659823, 1.1630674059096602, 0.32862380783181294]], [[0.9431442978413189, 0.7467711137351758, 0.3940863222843244, 1.3527029056363473, 0.8481325618248323, 0.8387355654196267, 0.3858963409833879, 0.7099309817554975, 1.0211512531746596, 0.4117029892988166, 0.13425265188260355, 0.6526938397605916, 1.2677460542019316, 0.5805138671780992, 0.3746427124741408, 0.5049740563742146, 0.4953652147101131, 0.7859162167797867, 0.8272578387418299, 0.23576529210536723, 0.18981376459035343, 0.20744202035598575, 0.11669821906833056, 1.1193694992083982, 0.8318287688021448, 0.4684817654366431, 0.4414891004100753, 0.9861095109544762, 0.5644654567389297, 0.5416538778991167, 0.7425317827311, 0.7770510419713741, 0.7846352066125287, 1.150766892051503, 0.3087739268502101]], [[0.9107153940361553, 0.623263559121755, 0.3876489966134272, 1.3259228041324207, 0.8158099598174958, 0.9222254181638485, 0.40085574998870444, 0.6668514820720068, 1.1462456736994113, 0.46000307768696314, 0.1412358293688708, 0.6532920150137015, 1.183537501226241, 0.5611802815369877, 0.34499190749624314, 0.48573706644072545, 0.4790842328776949, 0.7926230830864104, 0.778843563107968, 0.2446580506386415, 0.18565124767572722, 0.26745777960599265, 0.11690823242686262, 1.1739277799061165, 0.8506032277755662, 0.47608287365719415, 0.3541234002566138, 0.8538480764980552, 0.4346893636712662, 0.5614630789313229, 0.8983691678000978, 0.9829445585330269, 0.8710756970208311, 1.1730874502341169, 0.348451550508057]], [[0.8588391509559308, 0.675179202573224, 0.39056976229808527, 1.361453246738516, 0.8639272638392823, 0.8595096282417822, 0.38291910163655285, 0.642500256496345, 1.235147738235066, 0.46592327168763004, 0.15379823117842711, 0.5900695088558308, 1.187492677528024, 0.590941532425671, 0.3513051482709945, 0.4349475739853824, 0.4712651861133039, 0.8020519309255193, 0.7855514061768436, 0.20379485683158202, 0.19812883183810853, 0.22648967943510553, 0.11007966107597006, 1.1449599777937767, 0.8866216313657771, 0.4615090945225864, 0.49962837722984144, 0.8702856763816632, 0.3395535478852112, 0.6942519334078185, 0.9187174516779888, 1.1943533623273728, 0.8634645538311884, 1.1995909221540741, 0.3793668906407739]], [[0.7942990548492975, 0.703133518637442, 0.4130885983548114, 1.3078497279835817, 0.8815898109633282, 0.8749857047994715, 0.4167856949022944, 0.5957110244822438, 1.350071634746326, 0.43795648283475863, 0.1492568899189373, 0.6246095804591941, 1.1784072500089886, 0.6491660955339239, 0.3022708628273381, 0.45160539293304214, 0.5262679707523266, 0.771462979204398, 0.8293000703537448, 0.2091997684192994, 0.22369370073210273, 0.22449021951432502, 0.09326638713957092, 1.1521734730544213, 0.9469710059107448, 0.4491937276164969, 0.6220177571974653, 0.9891181045636201, 0.3569453244602642, 0.777309232371027, 0.8919241387517394, 1.1334096305351353, 0.8420647862805632, 1.1125245753022748, 0.42605016389090067]], [[0.7006398027809325, 0.7353563247701198, 0.4385954244159862, 1.2818977439848274, 0.7787240539160691, 0.8673526686220336, 0.43549114831979774, 0.5951583144093385, 1.4306277632302435, 0.44657258106581627, 0.1971017505590127, 0.713652324810019, 0.5963954403010796, 0.7422401786553382, 0.20141958852523084, 0.48372557013564543, 0.5881633857515506, 0.740358046790494, 0.8161643032964223, 0.27926838057166936, 0.23088859327442607, 0.23497588581387224, 0.09181747057140682, 1.290260327641842, 0.9727875376756459, 0.43879563650751396, 0.6886303879766003, 1.0869283796300628, 0.3505543449331465, 0.7952078468511974, 0.7523023335267804, 1.0569597123554217, 0.8365783469270391, 1.0314238174788828, 0.4727876432729017]], [[0.6245408731205734, 0.7861277043355882, 0.4420527218279581, 1.2692382234849924, 0.7712842555591741, 0.9391197755027758, 0.36198052740750636, 0.6088491853715606, 1.434663042687502, 0.43663000215232545, 0.12725873216268305, 0.8210151352030014, 0.48222378115990205, 0.7570754758335173, 0.2997429513008934, 0.4808418944951235, 0.5480200117433263, 0.7422638645141926, 0.8411044627221658, 0.29552931017284917, 0.21005431281528464, 0.2699577152213387, 0.06962461322640129, 1.3972279491987634, 0.8335627553212657, 0.4688937271469264, 0.8698880887291345, 1.040871820014297, 0.38448581302012275, 0.7957288775732693, 0.6195975893276124, 0.8481134345637666, 0.8433579162454089, 1.0584868030604289, 0.5047314762332809]], [[0.5377956019507576, 0.7698808714365374, 0.4392492537695735, 1.1843851083552233, 0.7838605355784516, 0.9037186483946937, 0.3766029694050968, 0.6510421288551285, 1.4532311246988712, 0.4442962919836542, 0.10963232929397902, 0.7516022136507409, 0.5527181276862967, 0.7655943370009342, 0.3596827871178134, 0.48644419714005604, 0.42550090535952556, 0.7321672485186125, 0.798212218626412, 0.29588283785936653, 0.2082734974065257, 0.26409040725706934, 0.09032590307127465, 1.540982446585732, 0.6441289096417149, 0.4986908341238903, 0.8525467842436302, 1.0404067262871717, 0.42101610533121486, 0.7699851824554836, 0.5349031442602822, 0.594102087503166, 0.8900460848269641, 1.0469636678406138, 0.5157565407658643]], [[0.4802774830778862, 0.7964921745115301, 0.4315484336667143, 1.1082246149977144, 0.7491201977983827, 0.8759662841066364, 0.3740175033963965, 0.6806590762201157, 1.4625176506395428, 0.4288032373469224, 0.10892968149810464, 0.8605179417069819, 0.6533280276351396, 0.8133966045717402, 0.36524612351293756, 0.4755934557516053, 0.44651737533036934, 0.7226747604237231, 0.9640844356227443, 0.31801411960146786, 0.21335753510472327, 0.2609716086427247, 0.07054053707512042, 1.6016438906409929, 0.5429242880070706, 0.5558482179743702, 0.7962947061868975, 0.8476137845556377, 0.43159591086139837, 0.7356899153209309, 0.5945169714253735, 0.46706269036691195, 0.9194411217646137, 0.9682187216920333, 0.5431965778866984]], [[0.42422882159579833, 0.8065843221205649, 0.45281635130991893, 1.1234474454250931, 0.7393219922519616, 1.0740263847118885, 0.4048983278163601, 0.6687772092594833, 1.4347238812111334, 0.3955636711528209, 0.09504346415856892, 0.8417596465212596, 0.7822802960543573, 0.7785579676276828, 0.38341439901430147, 0.44915225771884393, 0.49395236267143616, 0.6728349013954966, 0.9686182961263897, 0.287350620337808, 0.23333996196854864, 0.2549044406879984, 0.08439834932963672, 1.7305061920930545, 0.5152385028119146, 0.6721599274780273, 0.7811614499665178, 0.8009606080845277, 0.4792526111326799, 0.7873275068879089, 0.6650627852675748, 0.4115531708375421, 0.9354558777016588, 0.7716333760318475, 0.557324023285499]], [[0.36921958897560964, 0.6976288596189777, 0.36274441975720995, 1.0680186212643727, 1.0671962854852417, 1.256300038210015, 0.3706426554676529, 1.0093887333254088, 1.1617041182398715, 0.3979502173401491, 0.08357232435152341, 0.7293489053423559, 0.871858764649592, 0.9702735810233708, 0.3810197083194671, 0.4758880871178328, 0.5295120760330039, 0.7521711903848238, 0.9468043855246373, 0.313649808785403, 0.23847972526427944, 0.22291510126781758, 0.0776381129958004, 1.799180980345989, 0.5299104672206583, 0.7985072465032961, 0.8313443084074728, 0.8053134586154334, 0.6840128920447583, 0.8511631725814774, 0.7184247044813846, 0.35668313734615814, 0.9436253104410548, 0.5416566054247995, 0.5642419920897703]], [[0.32829012355458465, 0.6359585365911751, 0.3657747559187582, 1.1649315264796007, 0.9820687913817203, 1.3987626136186546, 0.34244894054216696, 0.6701691797912565, 1.0368344353176133, 0.2400596558006376, 0.060426712495209006, 0.7214550840974154, 0.9240735089892889, 1.0565850676226831, 0.40478133463409166, 0.5290316730750527, 0.568679765177325, 0.8114600252804742, 1.0019797300358109, 0.34305875667801033, 0.21933851620809258, 0.2552888556356123, 0.03189189030156606, 1.7852081307790393, 0.265695605601745, 0.6619594546836503, 0.8767194747920954, 0.8092935984551941, 0.6942288095403328, 0.8861486293402832, 0.7462089641091978, 0.3776855531362945, 0.9352032784913564, 0.4737240765577189, 0.540529419561754]], [[0.268336113442317, 0.57502476654805, 0.36911429730211276, 0.9672006367491128, 0.9992064425093302, 1.4190609574162139, 0.3241761174213894, 0.5867373694173597, 0.953442236320455, 0.26590784000627343, 0.06902606720197214, 0.7178940656132329, 0.9872471949244901, 0.9751648941572517, 0.473139668750508, 0.48341082042980255, 0.6027840432349361, 0.7647734077260184, 0.7976179614214827, 0.40380348761323615, 0.2162816814570709, 0.25464748769681755, 0.029446162127462518, 1.858235638901272, 0.11723164070760796, 0.6288110002841008, 1.0533663822912087, 0.7211390157043591, 0.5601573835427553, 0.9306185098758193, 0.7707978461203293, 0.48908477197240147, 0.9418310916872604, 0.3316903899788777, 0.5037656182389714]], [[0.29357163903948724, 0.5534886932287266, 0.353137796140431, 0.9048025823599972, 0.9826050502220413, 1.669344326964527, 0.351500890625998, 0.8357477461910716, 0.8398006073345035, 0.33107253232290673, 0.06625052933347578, 0.7052072809060391, 1.0429612011475258, 1.0407223148137597, 0.42737247234151166, 0.5241728627700928, 0.6194427947231493, 0.7748649306710471, 0.7787879800798725, 0.47844744770672765, 0.2079869554255182, 0.21910703373694346, 0.04111205177369899, 1.788340129595554, 0.22732647012184387, 0.5472235771749383, 1.2059631729601565, 0.7062215834040131, 0.49638562661726676, 0.9537208447876088, 0.7876261791664544, 0.6331555272508753, 0.970019276668601, 0.20106312914241292, 0.4090569033082442]], [[0.44747453146390864, 0.5427835992467847, 0.3683163468628551, 0.8045415132404137, 0.9321686852271378, 2.0032698535560254, 0.5237812102346677, 0.825799146219648, 0.715399207377748, 0.3953968248697006, 0.07241209819810002, 0.733231866785184, 1.13895538062719, 1.0220939197842256, 0.42674091688689475, 0.5507479226757347, 0.5672908863937269, 0.7858634119548126, 0.7708157279239303, 0.42260477958404996, 0.20525179090900073, 0.2099159526992161, 0.038512444725663175, 1.6255234302375596, 0.2848345072580275, 0.5104093033758105, 1.119564920321694, 0.6909827862758283, 0.4824393711241386, 0.9597821650371456, 0.8729891769619924, 0.7494329844348101, 1.0050962731703228, 0.19104613549760674, 0.43045272461784134]], [[0.3301380532397893, 0.5057805696466289, 0.37855933327673263, 0.5587587637977185, 0.8436831638854849, 2.157187919366363, 0.4721530128186778, 0.7351377898566126, 0.6240487779649119, 0.3679903591185083, 0.09903079609135519, 0.6856814137131693, 1.2067924834568329, 1.0424831376267527, 0.39538924514802043, 0.5020932169635486, 0.6598835915964191, 0.7062643188449418, 0.8084343402499934, 0.2619187313198203, 0.2397665130428661, 0.24534518087361, 0.040372183399254, 1.53788804028797, 0.33520964901675177, 0.50839074502456, 1.1082201046001476, 0.6207568725045937, 0.5391979470484256, 0.9776605057732867, 0.9521649914410358, 0.7645457593892798, 0.9495196798915786, 0.23945224757271627, 0.41251211174017444]], [[0.31474295047058465, 0.5121050254617915, 0.3781544719115547, 0.6552277912656297, 0.7430013635580534, 2.25145456458905, 0.4526539141736596, 0.7326061228820206, 0.6383118182682389, 0.3625922852800213, 0.14118489840036635, 0.7139035328551285, 1.266627555920174, 1.1102648984672667, 0.4062131572394052, 0.5043831534157562, 0.6246974905640392, 0.745592841228631, 0.8084526208682491, 0.22684399513459402, 0.2240716024009427, 0.25565338065341203, 0.05057200492313206, 1.5152385900290655, 0.36691232028868415, 0.44491075244007644, 1.0497025419356811, 0.569364002964547, 0.6317889663424807, 0.9733099785521505, 1.0151588809564243, 0.9190020290521442, 0.8298816080501143, 0.4285193722363445, 0.4179450519999446]], [[0.36764525568211337, 0.5299665938368396, 0.39728291506587665, 0.8326014020532462, 0.7554224516256345, 2.2894797827119526, 0.41157205819369136, 0.6743594050848404, 0.6743280438810848, 0.31358992952275544, 0.14829186996835247, 0.7005032297155618, 1.2027077144591471, 1.0279034045107618, 0.41132471514810676, 0.5290341460677835, 0.5716772689377306, 0.6765072682231686, 0.8001087369028729, 0.19987971825086837, 0.20777097756135604, 0.28152055532975934, 0.06529738121402172, 1.4755825571824588, 0.40608949302672326, 0.41899557815583743, 0.9859269030274831, 0.5376579795061327, 0.6456635781829447, 0.999293595055275, 1.076582586755829, 1.0414986410015319, 0.7213504374642086, 0.5034355144903538, 0.463542460514313]], [[0.34700295480392807, 0.5193703305219541, 0.42124855667933037, 0.883543802270755, 0.7520594145078584, 2.4140599765427675, 0.44105588834517206, 0.7199475282153842, 0.6693825663788381, 0.31990734836153334, 0.16115875532607468, 0.6679494087308795, 1.1799917392422876, 0.9193980549470925, 0.4036398702827816, 0.5341154758996222, 0.6016354636794639, 0.7857913022308871, 0.8283592285584287, 0.18822714171729948, 0.19880114913917257, 0.2368277953326268, 0.04403450049426878, 1.3308441832326703, 0.37844041914966986, 0.3806589036739791, 0.9066174682846533, 0.46671146194968854, 0.6685768638933643, 1.061323631088544, 1.1566062115162006, 1.2081838631422885, 0.6044107220285009, 0.5999752762230439, 0.5260679348537719]], [[0.44424055028462084, 0.46632799026851823, 0.3324112880664344, 0.9190852809527236, 0.6493289852363524, 2.496682460397894, 0.48873668441284535, 0.7148893307396817, 0.6512729691177652, 0.3776945298708128, 0.1950272399512343, 0.6319168636380095, 1.1801265061757815, 0.878591776270197, 0.39286373084932524, 0.532933283244258, 0.6096406424024144, 0.7818653510551218, 0.7782863256145853, 0.18150764974020922, 0.18212714302441357, 0.23682364368243597, 0.05229192425202295, 1.1551366653310717, 0.4463690690812392, 0.28896843934454614, 0.8868017840476317, 0.4764553156306345, 0.690048807358065, 1.1167304116872778, 1.2633510450396064, 1.2817879274486341, 0.5062969766127926, 0.6187211832698386, 0.5472871585269257]], [[0.4260065792193115, 0.4694387786711592, 0.3364343326059259, 0.9773368253080272, 0.6189014771152077, 2.4197823060676305, 0.43334144772987887, 0.761108984282085, 0.6624239836578678, 0.3921449603302777, 0.24644278710299503, 0.5869086454734551, 1.1867280193273542, 0.8716568532945865, 0.33014633214456157, 0.5536006868489618, 0.5907952219488921, 0.7861589721114575, 0.7873007079446072, 0.19824769137427856, 0.17802961528133415, 0.20892680109425918, 0.08096010714482214, 1.1277654471611462, 0.5227486727446695, 0.09985047121167304, 0.6771208109936331, 0.519164416115658, 0.7533613744120036, 1.1377681262262112, 1.3463901730836136, 1.2747547680833333, 0.4191536642018617, 0.6249734455864087, 0.6995826530711258]], [[0.47210455791139, 0.4636197679008355, 0.3312508114318543, 1.0825270573094818, 0.5673350089633262, 2.4352463853765838, 0.44501626008358186, 0.9021775271317174, 0.6545681894987984, 0.4140012319983443, 0.2944545924618034, 0.5902819835204574, 1.145226634725066, 0.8607936217748806, 0.34656856277953907, 0.5189190094817172, 0.5932621008492367, 0.7961015274788181, 0.7835691846471591, 0.2184912179120967, 0.17848178815425583, 0.18641426367937863, 0.08608008448220156, 1.0517181925574568, 0.6231778138552637, 0.08374505223613074, 0.5720784590603261, 0.5443585706168612, 0.7061325635248147, 1.1484061049298673, 1.4018619191031614, 1.322136422493306, 0.324328416240347, 0.6145601695143834, 0.814911409163253]], [[0.3797882498577657, 0.4811088768232642, 0.33620320608295223, 1.184344494123584, 0.42381647542292067, 2.3250432715817917, 0.39815062233906173, 0.8618311193476416, 0.5629211657475149, 0.42660997839820103, 0.4017898035056722, 0.5907122510441212, 1.1248517018397117, 0.8408501874754366, 0.35072645365603095, 0.510175058531421, 0.5684117096230958, 0.7950131529663579, 0.7412126118194988, 0.2042812094881979, 0.18893152394748738, 0.1821783306815731, 0.09449399092110478, 0.8525516498654855, 0.6890706615144933, 0.13724834118249982, 0.5209996916282906, 0.5214535675816552, 0.6993186952467835, 1.0575885781287795, 1.4338216950139828, 1.3824620570605517, 0.24254535992453957, 0.6470261776220207, 0.8393377892889522]], [[0.38797442186734177, 0.4788652213629173, 0.3349537681228607, 1.1687375983473536, 0.34298244688008384, 2.2940644601951083, 0.4094702846292574, 0.9297718997977554, 0.46501339306189704, 0.4264833722590589, 0.3935508523106586, 0.610113537697057, 1.115867052438627, 0.8250089744722428, 0.3474309514070826, 0.48973511867224073, 0.5076927567536713, 0.7699787875853112, 0.7643664884839357, 0.19429264978018962, 0.21185296646942586, 0.19517218938563916, 0.09601514275108386, 0.7756797860670667, 0.7757872967107933, 0.18837509001100275, 0.45420125379399645, 0.47833313773088715, 0.7190673499142161, 0.9890579316976167, 1.4484706798604932, 1.4155559440422638, 0.2253869352117861, 0.6535569899304652, 0.8020246306039664]], [[0.3668478028928712, 0.4575304353625512, 0.33020403187536174, 1.2527717274181331, 0.3236347245357831, 2.2704669690050614, 0.38262993222998143, 0.9658007733864515, 0.42737616065023115, 0.3833619942940394, 0.30886316735441066, 0.6189262344342431, 1.1322560184035364, 0.8626094552704539, 0.3521958061286056, 0.4956081446840238, 0.44749312575018785, 0.7686214970556928, 0.8298876633387993, 0.21508647987309587, 0.1940829869757425, 0.21269592771065549, 0.10870857354550725, 0.47259484107365246, 0.8878493582096112, 0.27585390258889464, 0.41766105175025303, 0.43398811463475684, 0.6194832529992449, 0.9712930963513143, 1.4642925979897057, 1.4421343257365158, 0.2557753292962205, 0.5905037639244876, 0.7791365284243945]], [[0.36055273127172904, 0.4265497307136341, 0.33461026617678147, 1.2120612264192574, 0.5134815951124472, 2.221839390283304, 0.38121432544645767, 0.9671184518209142, 0.4410424047247954, 0.35630576815523163, 0.28678566646559966, 0.5888420486452217, 1.1546056863143057, 0.9174079032308988, 0.36951395723738845, 0.5009017619701605, 0.43155358899346663, 0.7656465757758725, 0.7860240654032872, 0.20533252609624802, 0.22314849891839403, 0.3062193571523719, 0.09688830810018217, 0.31276076938431313, 0.9482056733628346, 0.3604059767522194, 0.5295564705785967, 0.39675879370977307, 0.4887803806762375, 1.0098621260057798, 1.4786553690473758, 1.4577934595587039, 0.3612989126353879, 0.5726680943038445, 0.7971329254099541]], [[0.3422919442790239, 0.4117589838741563, 0.3394514952677906, 1.2404323190586128, 0.71144763774371, 2.2225781856375098, 0.4050878507650981, 0.9594727386265536, 0.5167231942553239, 0.3228543340673852, 0.29165285915360634, 0.5859520531877078, 1.1836868900537751, 0.8962400235154242, 0.5508280451018328, 0.6274786602837278, 0.4327986610186234, 0.7001353761354026, 0.7444328800596514, 0.18632113804279127, 0.25420089156309816, 0.3025597264437352, 0.10002731888857855, 0.24042590800665026, 0.978892943295715, 0.43004939036917855, 0.6065131626690715, 0.35463055138675614, 0.48338420355272665, 1.0354400993854016, 1.4834989420165314, 1.4618638010809093, 0.34665986357166767, 0.6128095581291357, 0.796124817235752]], [[0.34403234091629753, 0.4004295203122752, 0.3433213975574979, 1.133167359417689, 0.8142856874315878, 2.141072016277312, 0.368539565159095, 0.9486269725162391, 0.5786730761726241, 0.3004713926640489, 0.31258036371642345, 0.5759932689065879, 1.1991602868110809, 0.8737807914943507, 0.5924456926056931, 0.5806446484097134, 0.4255805692024166, 0.6741643910038685, 0.7078422326284534, 0.17484705471334572, 0.2661709059988576, 0.25029594129516153, 0.09619717404689312, 0.22918511182533277, 0.9934962225749905, 0.4845252345487174, 0.6105904260076648, 0.33391907709698004, 0.44190762733727074, 1.0508805366119178, 1.4895654029107546, 1.49712977188803, 0.36044458573580607, 0.640821181321031, 0.7961000325711316]], [[0.3511346715416571, 0.40006864941028963, 0.3391553898645387, 1.0433468425200534, 0.8799083683144305, 2.142181376833042, 0.39635177440510677, 0.9400846444773372, 0.5810707062746947, 0.3068504202665808, 0.3295582059086314, 0.6333863543550573, 1.152648778566898, 0.7428396527621864, 0.6272543397822556, 0.5504943431944099, 0.38963173645084614, 0.697893965455774, 0.6785988483662091, 0.1693077300362742, 0.20912613095780525, 0.22910173973728948, 0.09437811793862813, 0.26401262985055307, 1.0150564018819352, 0.5192170022676244, 0.5644318969140242, 0.31897684874505566, 0.42253831292283334, 0.9889481500850293, 1.486205897793268, 1.5076318763257148, 0.2968200530722892, 0.7003477521315916, 0.7862633234661851]], [[0.34237613463127226, 0.408294404324762, 0.34087685929504724, 1.0347322016937992, 0.9038896534914211, 2.1055837536603623, 0.40101389452479896, 0.8990774999913376, 0.5971693291343566, 0.3049918731529582, 0.3296106424547903, 0.6435156215542, 1.0845156921724186, 0.7991964044583435, 0.7184611536098904, 0.5123227382811885, 0.40285472376835074, 0.7088283369369781, 0.6450591627242278, 0.21039056679155427, 0.25318035910773107, 0.20847152995708262, 0.058225205732784044, 0.28614759776378484, 1.0317074293426451, 0.5366783139345738, 0.5359269989206222, 0.3071927942215129, 0.4311325644249671, 0.9610534809441567, 1.4833684103030764, 1.4981735916135726, 0.2615763938707533, 0.729447822739763, 0.7847230035324212]], [[0.36397540333475054, 0.41518803169960716, 0.34342721047702973, 0.9828606207110133, 0.8980800162010427, 1.9522271741586885, 0.3565803119053904, 0.9423074685505425, 0.6132753226363569, 0.3089956859882279, 0.3532591232817825, 0.6731745237617107, 1.0115068296870107, 0.7552087968363234, 0.7128495315968577, 0.4894395082584385, 0.3991138419357957, 0.7178095934188289, 0.6111487477398989, 0.21082819924151847, 0.25644751539512, 0.19954321531086458, 0.04677437459006274, 0.27518366867640054, 1.0667982824476487, 0.53168225871587, 0.5741182615381004, 0.3019803769572179, 0.4168496408273178, 0.9163912227689981, 1.4757211750248143, 1.4979681167056649, 0.27953628583837525, 0.7736591836725697, 0.817895187746225]], [[0.3357928698024687, 0.4202136655714114, 0.34424589144373524, 0.9414276165201376, 0.9104909285759799, 2.0012624782002892, 0.38196055113936705, 0.9583260539196399, 0.6232507585107421, 0.31596207492365747, 0.3390623253663707, 0.6365400269555571, 0.9178238067637308, 0.6962633098956922, 0.7287247078351802, 0.4718093426581279, 0.3936286604631586, 0.6868193839040047, 0.5884801182381643, 0.20445907646981276, 0.21127093966849664, 0.23406281841561494, 0.06139998672888819, 0.27408962497753153, 1.0001045478680823, 0.4971935414008698, 0.5839539175497889, 0.2720242609795755, 0.44457024474999673, 0.9002353629852573, 1.492226173800589, 1.5104447775843002, 0.27759969524898925, 0.8032965198585964, 0.8359427963745197]], [[0.34438401084474213, 0.4370458584458137, 0.3491121405340049, 0.9784908196276825, 0.9662848639140073, 1.956461607516311, 0.414263110625194, 0.9642873156220667, 0.627631558462888, 0.3335999821486316, 0.31472730630987544, 0.6345378360277438, 0.8628768741626283, 0.6920846400093472, 0.6901504006313031, 0.45381923292631243, 0.4201925275710122, 0.6765701077114705, 0.5915941508683998, 0.18411217992219836, 0.23391598395972435, 0.2821942442257104, 0.0832717174097763, 0.2932526898740424, 0.9908324026134838, 0.47107902570990867, 0.5928279162115278, 0.294531192266139, 0.5248492519759667, 0.8875147857591783, 1.549158806240119, 1.5602403627583834, 0.28119039648246846, 0.7835313980459091, 0.8769432363926145]], [[0.38160026183072737, 0.4214045103491143, 0.34738452090108357, 0.9857752505525796, 1.012524829206667, 1.8982346698087627, 0.4437326914492227, 0.9469063912009601, 0.597982884332447, 0.31084386298345257, 0.28347302198072166, 0.6710836624908498, 0.7983473760617315, 0.6465658847271739, 0.635768719033768, 0.548403892573328, 0.4310869863961373, 0.6305298111408653, 0.5609613358299343, 0.16452981773768682, 0.2837614028612774, 0.2598307678886349, 0.0736531518532173, 0.3332022557552003, 1.0227480019570732, 0.4464388362703092, 0.6049226655280988, 0.3864950273175127, 0.5767918939643136, 0.8576151020345395, 1.5103625597866817, 1.581213568641927, 0.30645227992276314, 0.7498863609893015, 0.9000373396070296]], [[0.38726114859907235, 0.4168527793530633, 0.3544254883063232, 0.933849163140882, 1.0239848095251467, 1.8733465010752752, 0.4814720703188953, 0.9817561099497583, 0.5430964881049815, 0.2682626707389943, 0.25337427469119034, 0.6759599687586609, 0.8339949054766413, 0.7322466079410488, 0.6157368979144835, 0.5673667034671146, 0.4210381071007966, 0.6354062267997415, 0.6243588683798154, 0.1833568786622545, 0.27702924481626284, 0.2520468418725056, 0.08156288236478235, 0.38106225724729836, 1.0781553830861959, 0.4327678974719815, 0.605993368679812, 0.44216246861007336, 0.5992167609251411, 0.8828178495964167, 1.3704839219428668, 1.556529047907878, 0.35266034459783957, 0.8454024092430539, 0.9607884520923049]], [[0.39528407465124415, 0.47497096396492317, 0.4122173195156853, 0.9200073305467015, 1.036752369152367, 1.687138433292181, 0.40797523250723533, 0.9683885427351493, 0.504169910308682, 0.2504613564140803, 0.23784746357173037, 0.6694623862148383, 0.6221219738474796, 0.7856000645983803, 0.693650728010472, 0.5938854403347669, 0.41711152587302625, 0.6612119240553372, 0.720785185252728, 0.2357979663265284, 0.28312443905593754, 0.2483074564614788, 0.03323740287811833, 0.4510034803367117, 1.1500941632325485, 0.36238485813039967, 0.6247565563471382, 0.5397285662356686, 0.5459269278916685, 0.8821410608322322, 1.270359758275809, 1.4860423618751635, 0.3949292156574335, 0.846515915631227, 1.0365906970767165]], [[0.3886022513941788, 0.41250555680942214, 0.36259396278625206, 0.951576128215506, 1.0410536204110126, 1.5508709424001017, 0.36351383476976157, 0.9133630911099521, 0.48957164160744165, 0.2528981384408039, 0.24113935783204454, 0.6676960151191016, 0.5543196304186067, 0.7716085144166095, 0.7899174857202274, 0.5602593748574237, 0.42426807667009814, 0.6446281237398885, 0.6780101415083678, 0.21588968775071865, 0.23088338176480616, 0.20738440291565988, 0.023775249525861544, 0.5047824810661692, 1.2476010855588515, 0.2879510142895526, 0.6174497532229672, 0.6118920617763481, 0.45645723621027745, 0.8872374715854454, 1.1804573388993331, 1.4368796495803426, 0.4395026533659576, 0.8882819943623396, 1.0561453876076927]], [[0.37956127887044155, 0.4037433418267264, 0.35676405795389826, 0.9348905903160266, 1.0088907043952746, 1.3865656912399653, 0.34172153748368783, 0.8666168056462442, 0.5143118240097779, 0.26170147321866055, 0.27019299103911193, 0.6916365324163345, 0.5614323979906539, 0.7607038710388524, 0.7946932364395416, 0.5904534736583676, 0.4172716182628039, 0.6412299045833205, 0.6524285968066357, 0.22378704155976728, 0.22255387294236895, 0.18074957733381147, 0.01819986868242179, 0.5596137333602954, 1.2438086337982721, 0.26139378103738686, 0.6297764238420984, 0.6664672413406448, 0.4033521562096838, 0.9083658218653214, 1.064474284553782, 1.3944262377464112, 0.43861481901161337, 0.8966988841150214, 1.0285229900600203]], [[0.353798727561743, 0.4042157635206538, 0.347678901735983, 0.6938821963033639, 0.8041174694388087, 1.0630484993899734, 0.3356426532395214, 0.8628087945138185, 0.5159293020236204, 0.2623263066908935, 0.2515546307865451, 0.6383665810832129, 0.49288096520792657, 0.7355717603634597, 0.4713622076054434, 0.43671805747343495, 0.4017481971121848, 0.5956271248981448, 0.71796192175861, 0.2606784539482576, 0.22611952811942693, 0.15998663804457608, 0.01573145513301193, 0.6458715430821229, 1.1783373240388704, 0.2506534028448568, 0.5889451245974155, 0.6933029457446771, 0.35758914786303836, 0.8726514349597476, 1.0175827956710486, 1.2687745728575606, 0.47804902305182273, 0.8126536413938781, 0.9614406719424775]], [[0.34815181817070084, 0.43554975834424403, 0.3755636156093183, 0.731684935748974, 0.8486972659106521, 0.9889117755715681, 0.34284296059702, 0.8585050564688191, 0.8374799738334491, 0.9882670021081055, 0.24387496056131797, 0.9895346825093654, 0.37415201686030436, 0.7825718860587476, 0.49040653375105103, 0.36007310855043984, 0.3118913933756264, 0.5812053901318859, 0.6484955040924167, 0.2501118494357013, 0.1642437696409506, 0.13837889255460645, 0.00647894015065872, 0.7330651036730705, 1.0779182603974933, 0.24016673931656177, 0.5824904015803799, 0.7081377084839731, 0.2418924532540056, 0.7999241743420047, 1.1243902757110376, 1.1195665133702482, 0.6775225871314287, 0.8298755280272512, 0.8791846191855228]], [[0.34110432372533483, 0.40675875809073336, 0.3523199915008792, 0.8093120643257004, 0.8117171669267725, 0.8431172681943058, 0.34931707947788604, 0.8180329563245898, 0.903578570451685, 1.150149714302932, 0.24563069213598282, 1.0135137994960157, 0.4209341334447658, 0.7967299735558733, 0.4723226268765098, 0.39271083515888805, 0.3240294358494198, 0.613903129593568, 0.695751074158747, 0.21387227330510628, 0.15331425334069002, 0.11692387578778118, 0.01550701882315753, 0.8233981556085548, 0.9052613343631342, 0.2511976133874053, 0.618363351136086, 0.7017511087256068, 0.23005922357221698, 0.7832843741982083, 1.0566454548190611, 0.936379889835889, 0.7793128695495346, 0.8370000982950483, 0.7827547234593892]], [[0.3273275996141437, 0.4078713368054415, 0.35889906395994026, 0.8892474117827791, 0.79598526216518, 0.798941956298578, 0.3625289597215188, 0.7909030418597204, 0.6820551161797415, 1.1520672135228198, 0.2238763359117958, 1.040309279983902, 0.48669650117271956, 0.7168321890942666, 0.5056076478374665, 0.3213448066122857, 0.34051530703544625, 0.6857553666429388, 0.7953612862157695, 0.25451435649048865, 0.14887057873264375, 0.15670532810437018, 0.013969375116087979, 0.9174751243966222, 0.7360428369316754, 0.37176668952335795, 0.6786536001812584, 0.6940289810833836, 0.23317302526329153, 0.7749826516427032, 1.0181697203471238, 0.7320054017395825, 0.8314595867020071, 0.8666591461954585, 0.737545107987138]], [[0.31282916025663576, 0.38909698269588067, 0.35742762280591706, 1.030325095159927, 0.8034802390671982, 0.8789028673313376, 0.41077369876124586, 0.847163299539655, 0.6576487482011669, 1.122614999511201, 0.23926363138038526, 1.0618184952915954, 0.5338577697174421, 0.6606697284833783, 0.43886393439661936, 0.3863566109818001, 0.5106664080631576, 0.7363382638743193, 0.8523575376562961, 0.3016724867603612, 0.14164802919215708, 0.1540497691912207, 0.013486913041017602, 0.9731921424318812, 0.682557976503864, 0.44125345560198714, 0.7051111175204691, 0.6898472455383866, 0.2324401348236424, 0.747715478297579, 0.8998091124881783, 0.6373870882424192, 0.8405030830989224, 0.8264967471533994, 0.7281161209957825]], [[0.224773853229426, 0.409693674141576, 0.3343978654150357, 1.1280579223089784, 0.8074554731148487, 0.8837520117522302, 0.4022442686639086, 0.8832725734215914, 0.7127906847182611, 1.0460354498170203, 0.2627590929855796, 0.9489022936814102, 0.5679103006067707, 0.6354094963065389, 0.4634361648480701, 0.3426362198917158, 0.5446247171637547, 0.740014609180971, 0.9014846557430176, 0.3264189672906396, 0.14516658338669253, 0.18957056938173308, 0.018919299934196256, 1.0339849711679325, 0.648250624407865, 0.4320411933624436, 0.7184765599913717, 0.7578421853604655, 0.22544198205109178, 0.740769531863513, 0.9158112241292446, 0.5632925680388734, 0.7577981297463612, 0.7222789653122119, 0.7434950383215455]], [[0.23365354111715902, 0.40460865598653317, 0.3470421990114033, 1.1693633476830958, 0.8490766507251185, 0.9307233903188442, 0.38661947237873995, 0.899718032221923, 0.7193005230522601, 0.955597062154083, 0.2855140807489559, 0.9874911337075438, 0.6088638185032713, 0.6260591446283144, 0.4538752325732328, 0.36595902153999915, 0.6725572424143126, 0.7758280744444817, 0.967915443809799, 0.3419894347817533, 0.15165861126517324, 0.21539249701082072, 0.016312122624247628, 1.0842773762158968, 0.5694261514951892, 0.3751319723294671, 0.675321174146679, 0.8259814801604355, 0.21233016009165104, 0.7994255357535739, 0.9462101747768578, 0.583020979943368, 0.7138500257373814, 0.6836037401213717, 0.747525219734576]], [[0.21812215524665768, 0.4022638893906258, 0.3469937160259634, 1.1537902381142306, 0.8671980410884629, 0.8653169506069729, 0.3880191666742829, 0.9468670234502635, 0.7389633066836193, 1.0547251721327333, 0.3120436250310761, 1.027150499425546, 0.6213148436805181, 0.6031835449082471, 0.4668670212263409, 0.34352196161405346, 0.716912581018773, 0.8006667993659815, 1.1072373012738477, 0.3621660600943195, 0.16309514478430812, 0.2607006860919444, 0.01565749783768655, 1.1011869447609544, 0.5218142294814732, 0.3661079544790648, 0.6269753426093421, 0.7707421209737862, 0.33304290183914076, 0.8067974221095409, 0.9880897677658945, 0.6147836440569154, 0.6809797153264167, 0.5879914315478824, 0.77333473075961]], [[0.20840623079324913, 0.4009175756286778, 0.3420047331308906, 1.1455907862179193, 0.8679084830343875, 0.8704862093738834, 0.38588513502229993, 0.921320782467615, 0.7751223142425719, 1.095713180663884, 0.32661551744004125, 1.1058363953665298, 0.641398645782675, 0.6259713870690586, 0.45257877963713833, 0.32788174211261495, 0.7350571914672085, 0.8086008691168229, 1.1372762353256354, 0.40376465009338774, 0.1652149276246808, 0.27764208068579666, 0.0335992034088127, 1.114686343677064, 0.4979665869375808, 0.36000115686931333, 0.5790278667651348, 0.7668227605504988, 0.41041434924751086, 0.8549689871698748, 1.0212763701480179, 0.6575496639202367, 0.7501617218247767, 0.5565817859118325, 0.833123565882435]], [[0.20114662404138337, 0.41058203514545705, 0.3450828988376323, 1.0904929034028583, 0.8686418869484851, 0.8696483849586474, 0.3846824738108616, 0.9291202945366803, 0.8085454191486746, 1.0821996334083297, 0.329979795629849, 1.0094047784653764, 0.6366481681381126, 0.6458210086669419, 0.46740842536471955, 0.32313750337500136, 0.6334124900502085, 0.7588125102620724, 1.0444379108736475, 0.41531103089893684, 0.18609878667609847, 0.2716768027674461, 0.018524097391831938, 1.0801464494116637, 0.5543469085735923, 0.30534361703343904, 0.5010336731265366, 0.8150909739739989, 0.435150157070075, 0.9558660133384961, 1.0603271649024286, 0.7134257888730282, 0.7096848081702684, 0.4968029056173488, 0.8928979032522328]], [[0.18087176745670971, 0.40605200622787463, 0.3386627823746673, 1.068639017583098, 0.8646224430410139, 0.8222425657764554, 0.3896893574143681, 0.8358933810830714, 1.0026323032685918, 1.1927839880252529, 0.34783443333317876, 1.005064630005508, 0.6310419502083746, 0.6043544125071953, 0.5061982246560496, 0.32428965456005643, 0.6188045293286257, 0.7222792983761149, 0.9899850893966102, 0.38573676515943833, 0.20726093238400592, 0.2823843661700118, 0.028747326237430492, 1.0168608113312256, 0.6740719524248346, 0.3323528685805147, 0.4004317214077616, 0.849016718754978, 0.46820512439882234, 0.9438689591547642, 1.0581276077175863, 0.7917632689624061, 0.6146511610227431, 0.4696138252243299, 0.9828463480295359]], [[0.17548872930384926, 0.4088255870167522, 0.336593896296653, 1.0808567104188531, 0.8977388209405865, 0.790912302403266, 0.3960447358583165, 0.8399543466296816, 1.0931196517735522, 1.1711563194541428, 0.4666145640404165, 0.9139280759023264, 0.6151483339097813, 0.620384757588685, 0.41534533405347956, 0.33475741410956705, 0.4680511629123515, 0.6696297922310163, 0.9613816470231767, 0.3936498088446612, 0.21705030051108243, 0.29066841173201685, 0.01329449597820059, 1.0633291372120364, 0.7093951249116567, 0.3847046309243254, 0.3895691061553962, 0.8214478059590009, 0.49545538426596364, 0.7827456797569907, 1.0311051330500782, 0.8407857401410694, 0.5610080669183621, 0.48488456526235013, 1.1169391709683545]], [[0.17331012330198106, 0.42943560278372017, 0.3448701823442207, 1.161713252165514, 0.8705483088359847, 0.7954493166308145, 0.39157673602176046, 0.9765105603905417, 1.1352646893398595, 1.1480655640438004, 0.4769520443745998, 0.8866339379249133, 0.5913142792557198, 0.6366312407643835, 0.4091875540367364, 0.32649671152055904, 0.4410955682629625, 0.6378560228867292, 0.8999284434089301, 0.38665310269906555, 0.23365171601193788, 0.3164646519551401, 0.015279212682114554, 1.1434768514586438, 0.6327323944723657, 0.42657641058779616, 0.5378118715298283, 0.7373032716084197, 0.4789142698197207, 0.6292620406502465, 1.0211591855995903, 0.9147443280070644, 0.49618961541565976, 0.6061694340966628, 1.2180041613658605]], [[0.1607420698133015, 0.41400563245047606, 0.3246788829692412, 1.2840623530045836, 0.8764622312032055, 0.7765315286043206, 0.38956577876032183, 0.9798067258130919, 1.1232841780293135, 1.0608609897644325, 0.42249767147238443, 0.8711506080482591, 0.5482911530102846, 0.6526776644879702, 0.41384427474193414, 0.30124340553619783, 0.44257174372808206, 0.5483231337741594, 0.8014290285485705, 0.4129639092703333, 0.26034714000773324, 0.3331540242366499, 0.01236751991802586, 1.1441787988701122, 0.6013346145671645, 0.478252104265669, 0.6645790202574208, 0.7210875211423611, 0.4955750350818923, 0.5189608864400153, 1.0198506464942085, 0.960676064270561, 0.4594231605598813, 0.6439891062194851, 1.2161602636738245]], [[0.154523348461072, 0.42080466427862906, 0.3196214863986523, 1.6228687979286767, 0.8190262125997998, 0.896410864798592, 0.38283509261059984, 0.9422479489200566, 1.0943429162409313, 1.0461774173852392, 0.46899673772225536, 0.8558365136097564, 0.5478689554932255, 0.6910325164361908, 0.49293557683120687, 0.29649343538294604, 0.4050898492172269, 0.5373015099304512, 0.7805345748926155, 0.4381797680671203, 0.23682343056255423, 0.3780335381265798, 0.016276325252311413, 1.1589007034882495, 0.5807322162691468, 0.4950476155764515, 1.1575393914303223, 0.7605221006142664, 0.8803386063982709, 0.4573254839353035, 1.0185773918290677, 0.9338987647848058, 0.4629019914661311, 0.6697765869874419, 1.1828756627052701]], [[0.1560793069870505, 0.4192544284248708, 0.32066863483419716, 1.9276686373145546, 0.769654342881243, 0.9076765769613505, 0.361915570711546, 1.0040909355233392, 1.1069180343564329, 0.9435200529687131, 0.4306220393614455, 0.820182401276471, 0.5316143359924009, 0.7264207835312809, 0.5061882869151988, 0.28022045638228316, 0.3960113661898875, 0.5313596507294638, 0.780967675466284, 0.36813220488073983, 0.2290777817641771, 0.3991259982692558, 0.018102902865654434, 1.2763579846736968, 0.5311484320468425, 0.49575677845642185, 1.5979473309118053, 0.6588049683724896, 1.2680752026976927, 0.37849749323772786, 1.0142635787048988, 0.7979064065962165, 0.46466978149647176, 0.6607642255227772, 1.1605577398615687]], [[0.1516355760979405, 0.43783436885945004, 0.3182435687388824, 2.18119197106751, 0.8702547547351253, 0.9849786275461427, 0.354361383912034, 1.2420146978655404, 1.1451495698820628, 0.9009146060615418, 0.6127996582973789, 0.8108025837767373, 0.5062892492601049, 0.768716961011992, 0.5527335226474033, 0.2748337701807715, 0.39337238036776345, 0.5090962120000633, 0.8280726151086271, 0.3012204555835538, 0.2198309118418761, 0.4138928687911651, 0.017482313181676618, 1.3179525268812267, 0.47122894768098655, 0.48228947316787524, 1.6427610718254306, 0.6318343305235049, 1.2523619662019612, 0.29694000307897417, 0.9956562487059702, 0.7084891736949813, 0.4408984493028345, 0.6143746789889069, 1.153707701328778]], [[0.1638263013904436, 0.45860607500055595, 0.33560192058779337, 2.2943865297564883, 0.9008994474729781, 0.9926484629739338, 0.36297578292890276, 1.3252005374224165, 1.0363958530190507, 0.8482633683383576, 0.66121287580874, 0.8344014454665409, 0.5061817692124011, 0.6942447019182783, 0.5887142217601145, 0.3060231801665777, 0.43330496785989014, 0.5595058457030131, 0.8727628299160339, 0.28489043647923606, 0.20065908002345081, 0.4213648465718629, 0.04220420868968558, 1.4093276611482906, 0.382969225413574, 0.5130460159299627, 1.7285134446025698, 0.5899104250768222, 1.294603758180719, 0.21886235015098324, 0.9231131236849163, 0.5675403074217925, 0.416450659104342, 0.45336669054287937, 1.1298177366744453]], [[0.19059666751744583, 0.4190185806097975, 0.36130297103629955, 2.3553318619647046, 0.9448271156574564, 1.0666768020268338, 0.36566458960642273, 1.0514110517532242, 1.2196174955422827, 0.6451538256680002, 0.6448419478566136, 0.9475467895670026, 0.5578117431116703, 0.6904872708308608, 0.534424557949551, 0.3335640301734202, 0.48296809708258226, 0.5356416718333139, 0.9078913186972963, 0.2659117073560824, 0.19006863575949465, 0.41995848467947544, 0.04882280217489167, 1.4994455269039426, 0.3720665367143421, 0.565048149975844, 1.967107404485283, 0.6506374713510843, 1.344956582972518, 0.20765957948866542, 0.8173276598416694, 0.4497040369055302, 0.40521924665289444, 0.4651125999660515, 1.0364059029120467]], [[0.17976505155654657, 0.42455497062396186, 0.3702299173108155, 2.378474965736604, 0.9937324336071911, 1.042367656474331, 0.3667549548600338, 1.0813254625708413, 1.2780383301593845, 0.5173306942015432, 0.710009361401982, 1.0768159435806277, 0.5943593688902147, 0.6752023224782944, 0.48818855831907443, 0.3988369919657194, 0.6478124932024364, 0.5383518410161341, 0.9614557488125048, 0.25285093853386625, 0.170050236571284, 0.44291589199619485, 0.029483573309883465, 0.9821461336414992, 0.5094001530848347, 0.5862175962896791, 1.6578984528332916, 0.6147334227436264, 0.5794868659572208, 0.3428694376106835, 0.6680440663791452, 0.3570543080370262, 0.37648630739040856, 0.47557049425381276, 1.0014179784228736]], [[0.1805510633330615, 0.42714366206720183, 0.37404550435165407, 2.3666951005639296, 0.9832499384831629, 0.9877472845785896, 0.31716121206484893, 1.1196597978223761, 1.247875304687202, 0.4399497026337943, 0.7330051577323711, 0.9626403235563455, 0.6316674878712583, 0.6275193387723892, 0.4814408505621309, 0.39366998335560077, 0.5264388884044833, 0.49158145595553154, 0.9557420794983474, 0.21498992896498703, 0.17263533630978364, 0.3920977948768327, 0.022400915346727234, 0.45106583613603735, 0.5074682277744348, 0.49182131156776104, 1.2996991480600615, 0.6633536111436128, 0.2918698248765018, 0.6404865424953314, 0.5586602397007637, 0.3367114573119527, 0.3723413489086347, 0.4933175927536713, 0.9057164280746586]], [[0.18121183291088913, 0.41962834688765926, 0.3725545784262596, 2.355667619111749, 0.950541732868803, 0.988899677031735, 0.31686731678609814, 1.124363008085754, 1.1418673277840303, 0.45419420917529507, 0.7299169061951367, 0.9352184308614876, 0.6565396688811564, 0.6768870155410691, 0.48853443927813384, 0.3924014790452741, 0.5279282594749335, 0.523191212719482, 0.9845763947016015, 0.2193363403919421, 0.14248449628026488, 0.22238639008050629, 0.02406013971877375, 0.3298247781739094, 0.7026944355424963, 0.38234490464022086, 0.670541934303666, 0.7175568812312254, 0.24265506872843218, 0.9370945546278795, 0.5414150218817746, 0.36705372728121954, 0.3623923659274821, 0.5517070766078267, 0.8810124026936772]], [[0.18161013417070762, 0.39165655404311994, 0.3607656053751345, 2.379062956040146, 0.9910399796006057, 1.01691777555203, 0.3145488003818415, 1.1380749887974284, 1.0687712760483732, 0.5822808979271161, 0.7426371188708292, 0.8943680727455907, 0.6134889217051895, 0.5781295704903909, 0.4934623659680497, 0.34905778553220645, 0.529128024639574, 0.5167632091381658, 0.9273087311146497, 0.3112211098179035, 0.14749313368513448, 0.17815797113496923, 0.020297832556689, 0.3833788669492449, 0.7549549329549949, 0.433037459439347, 0.4599505210364149, 0.7398275516450963, 0.31978235117576714, 0.9855185623048547, 0.53056732368179, 0.3832972597648131, 0.34162045408624564, 0.6318902492220106, 0.7894255783496475]], [[0.1812501489332443, 0.38822030629163434, 0.34992287532729527, 2.111758412167748, 0.9013532243465896, 0.882734269121235, 0.25369925639285545, 1.2246032223996743, 0.9279562687786073, 0.6830776248004936, 0.6439104770302087, 1.0045837688364112, 0.7512492230803038, 0.6099080252143213, 0.5548634671209838, 0.35011264849876156, 0.4593151142341827, 0.5037501079357672, 0.9642949939562779, 0.20164969888948886, 0.19424469530156432, 0.132923913504235, 0.018189730964111788, 0.41661740072192, 0.9030833032372025, 0.4712150359980951, 0.6601752597791027, 0.7596120780300234, 0.5414345394648507, 0.8713574660144487, 0.5805323223239549, 0.378567092795432, 0.311954022852375, 0.7604897653366468, 0.6815070688487581]], [[0.18151989918269637, 0.3988675687170612, 0.3502209913893714, 1.9689109636030888, 0.7437462344555992, 0.9378319542790889, 0.3083759653653205, 1.254052174792826, 0.8539924692826266, 0.7056920536472864, 0.6181776231285329, 1.0112791821284117, 0.8002655440283821, 0.6461015744023362, 0.6330898946657377, 0.34300347731226943, 0.408272538719996, 0.514886383456194, 0.9950205445101168, 0.15257534616236568, 0.20222326937052199, 0.13742055281451965, 0.01389894718269101, 0.4538775019793153, 1.0452454477941358, 0.4980597177411894, 0.7807231663057618, 0.7934561791274568, 0.6400786860826181, 0.8080139542667264, 0.6265387692108798, 0.4020232449950435, 0.31572857044842195, 0.8685128566901112, 0.6915692639567759]], [[0.18144057018189994, 0.39424088880481134, 0.3558010174803955, 1.8740656178946034, 0.6999685901182294, 0.9274205096436408, 0.2830857632810674, 1.1708266083029129, 0.7617364502928148, 0.681918169368118, 0.5367016999986984, 0.9650848511486175, 0.8048485243601531, 0.6851297229752287, 0.6256749893964779, 0.3390629392514184, 0.4289408577716092, 0.5179166508231883, 1.0026720865350305, 0.1106105436918079, 0.18385510958706636, 0.13392445831304048, 0.040395542726211854, 0.48739008864673483, 1.1621878023991623, 0.5050758870350098, 0.8454373413932323, 0.7757408476565952, 0.6173456346463859, 0.5577316116335906, 0.5691880501358219, 0.418431522752557, 0.33958913311588906, 0.82732429506053, 0.4860240805409305]], [[0.18148455744403189, 0.4040400780221445, 0.36447381646336086, 1.8442534706886655, 0.6615110289319646, 0.8966576064794349, 0.26890743844135934, 1.0642391988159092, 0.681075806045923, 0.6117655498576845, 0.45152795575023735, 0.9246848134737572, 0.9244215870543218, 0.6715290092943459, 0.5706005140297918, 0.35380688388599624, 0.42370549134014523, 0.5196494668176728, 0.9914644232935328, 0.12116686828370882, 0.1741325576007618, 0.13785481744784586, 0.03634549489194128, 0.5298844972945146, 1.2816722072134692, 0.49963457252851673, 0.8557949253803528, 0.7704984328865419, 0.6391183918214409, 0.4474721485455597, 0.38768297338769975, 0.38548491267209706, 0.36714868661289846, 0.8326895115941593, 0.48190225775437345]], [[0.19151935305806891, 0.36981934914372105, 0.40509273936426116, 1.7527103654339904, 0.5978773095590709, 0.8307075490565246, 0.21046833382458263, 1.0595647661253835, 0.6371262512803736, 0.6091082530917106, 0.3031894383764636, 0.9571139703088267, 0.9194326355706415, 0.6736672030030615, 0.6605659331122689, 0.36668224637069835, 0.46891334145930763, 0.524484573692227, 0.9957907370728437, 0.1101356261725185, 0.18972586444153688, 0.15386418315196543, 0.03261775918235591, 0.5873072305653887, 1.390438658970043, 0.48422905933582383, 0.8055722977492369, 0.7417950533443711, 0.6171745922925859, 0.48857535823143683, 0.272490671133045, 0.3230043763130107, 0.4163181981567686, 0.8352871385232468, 0.5179201979397577]], [[0.1929576698880164, 0.3726382815119368, 0.3948638782011352, 1.675797897845091, 0.6200385929466593, 0.7605120659764157, 0.1782885384456812, 1.0783069206655846, 0.575236918506795, 0.6462602243957671, 0.3250175369339983, 0.9524805128899669, 0.9227276676359009, 0.6059390079834799, 0.5774417324950045, 0.3795868739072848, 0.4937006610363984, 0.5421593256506397, 1.0683895797966623, 0.10481809822216621, 0.15558822173457942, 0.16113458060777372, 0.020366158970432712, 0.6383245039641438, 1.4288692139148844, 0.4988923074153857, 0.7777248681347555, 0.766561930218133, 0.6009706594000374, 0.49136860389279513, 0.1946472169218253, 0.3393625297297439, 0.5704722933133578, 0.8031334240475105, 0.5613314888725752]], [[0.18877491457411186, 0.3960818474401375, 0.36057344182654716, 1.5349162699313321, 0.41473917936858434, 0.6862805154695656, 0.16291874419173846, 1.085497419783303, 0.5969818793374393, 0.6488536568431607, 0.22032963230439218, 0.9482842967974499, 0.9346436293122922, 0.5875729451758083, 0.6219376843210518, 0.3870308933950553, 0.5435522461959624, 0.5331133088052684, 1.068373228737528, 0.1149572554672676, 0.13014128357121332, 0.20366560225539804, 0.029640271246766138, 0.659760881432088, 1.3940784555096855, 0.5151555020334617, 0.8265940364342799, 0.7634052625104089, 0.5220612176466689, 0.5090832840160692, 0.2189451971881327, 0.38414399619267126, 0.6617279381068308, 0.691353535896951, 0.6293761912102055]], [[0.18874210178623863, 0.37979638423732165, 0.3708452981551692, 1.5081161322997862, 0.4059387098108367, 0.6315553124549317, 0.18660088461093918, 1.0713481855991978, 0.6579259208432333, 0.6399052122235076, 0.24680302860322112, 0.9094053862090501, 0.9568866349852556, 0.5978729220158965, 0.643056262132502, 0.5116990993611452, 0.5001070119220385, 0.5661546351130644, 1.0611775619664434, 0.14431102602341228, 0.12713778649136057, 0.27338711400616933, 0.04670140162430741, 0.5969122989494833, 1.352193200184249, 0.46958988401323953, 0.8866644663779075, 0.6492907500266158, 0.5590874865388697, 0.5112047023499579, 0.2518701257563958, 0.46357708044181434, 0.6741208393095883, 0.5356167608939179, 0.7329727834589296]], [[0.20835165297938937, 0.37954082737246686, 0.40631305586252825, 1.3518250896407626, 0.4220403563894556, 0.6296451673920568, 0.22781814594457359, 1.0882994561831534, 0.7006397836127384, 0.714059706767765, 0.15299811661393592, 0.5377404627370386, 1.2717608145414967, 0.6263037292712239, 0.4973396183928841, 0.5603410097787944, 0.48864096937762924, 0.6902251171254307, 1.162767292362528, 0.1635542994897375, 0.1487986713393738, 0.2828758871465375, 0.10378572390855896, 0.5420548706672167, 1.2697602237719703, 0.4563220983752829, 0.9235058106757051, 0.587817183810235, 0.5874179670455129, 0.506932164044856, 0.2953535818728448, 0.49860829046348953, 0.6972671330426268, 0.36870065988574185, 0.7515627252302216]], [[0.22726496497144377, 0.38615358572150416, 0.39444457327864496, 1.1639005627052814, 0.44420997797660794, 0.6079681048623098, 0.19415291531779394, 1.106123619789382, 0.6719090597393392, 0.7140297605228508, 0.1463035878313236, 0.5317650189928871, 1.1393726536288606, 0.6290247078546097, 0.5486566537439151, 0.594611338372762, 0.47913282275370694, 0.675239990416034, 1.1370465012535709, 0.15448764789095726, 0.16964755027465453, 0.21482373441037161, 0.11860500222011623, 0.5369501273600775, 1.1307097872739111, 0.4091731019092083, 0.8814739397463003, 0.5780204197906593, 0.5886641842565279, 0.524925150508005, 0.33223962278243946, 0.5455754215585724, 0.7001103576819481, 0.31299649518616546, 0.752456050909223]], [[0.23431650067322307, 0.3466849143291031, 0.4214866165217268, 0.9540476082482277, 0.46329345971914015, 0.672452386908516, 0.2579437632247299, 1.0914852612772696, 0.7303709139176192, 0.6935077840617909, 0.11438036919649491, 0.5302040028031948, 1.0409000001232354, 0.664302129469186, 0.5033719717559536, 0.5960242049390387, 0.48417675322786596, 0.6783561677533831, 1.0282619316633017, 0.13729906216712934, 0.16308978461076354, 0.18367088002367726, 0.1263377291282143, 0.6514909967086546, 1.1383712297777362, 0.3811377898554603, 0.8943352465357314, 0.5651246804611991, 0.5961890265583567, 0.5380197595307096, 0.3421199294418702, 0.5462260218557107, 0.6813377946185294, 0.31047700469561734, 0.7411531336653203]], [[0.25481803013898985, 0.3404373871986311, 0.4253777552904954, 0.7548746927429824, 0.49320272020730827, 0.61487204465796, 0.30182458761191466, 1.0071281993466417, 0.7083467635094901, 0.6255884539266913, 0.10464715529007373, 0.5421939307296105, 0.9497850778919248, 0.6586555669360581, 0.5186435946162538, 0.5457182617716734, 0.494827785132387, 0.7509187235383441, 1.0275296923103159, 0.12838567266938564, 0.18311353573689876, 0.1423567891630667, 0.11109931910928914, 0.6981833601766221, 1.292128081487025, 0.3576297034280216, 0.9595614890584687, 0.5578249084662046, 0.595635157475537, 0.5430159823584271, 0.37440601333999013, 0.4686265341020622, 0.6829015075527656, 0.41444932077918784, 0.640859499350885]], [[0.19260144806055454, 0.3365780902413902, 0.4417114452166011, 0.7404374190978418, 0.5026782848078757, 0.6364802884071398, 0.37780898496756493, 0.9434461089222199, 0.7114014825970101, 0.5130668620549511, 0.0831506576149203, 0.5978566742400978, 0.7385022854343033, 0.604760558534239, 0.3471358729037715, 0.365792371426665, 0.48676542169468495, 0.8027374579856681, 1.019837998579663, 0.1284802160877517, 0.19894714301180544, 0.13222748772606652, 0.09810531349156026, 0.7342113671124768, 1.3740336235742154, 0.32544349959520374, 0.9538728140790004, 0.5687827633197053, 0.5419458872268554, 0.5429768922020433, 0.37097855023313675, 0.46567525972321583, 0.5983914989712806, 0.462683355245393, 0.565580592624196]], [[0.1996234965426736, 0.34352144364566417, 0.41658600021107495, 0.7483234069367617, 0.4994994742587181, 0.6384200941472509, 0.4068272627784051, 0.7904302078417016, 0.6957263814609898, 0.4871930889919426, 0.07429962619393832, 0.5518356332670779, 0.645389740917753, 0.5606363595306026, 0.46946254819760386, 0.3343673025945606, 0.493450601641201, 0.7760136404681355, 0.9199051902416504, 0.15539497695227003, 0.1887260324550367, 0.10798864063238126, 0.1080789766341844, 0.7107601413173377, 1.425368883157994, 0.3076399589613374, 0.9061696173025181, 0.571323771296672, 0.5201989438749965, 0.5414917397746283, 0.34430559123128957, 0.45915788734975027, 0.5617702736763516, 0.48539645070346954, 0.4739512409182838]], [[0.20470960357330945, 0.3437092624806998, 0.4279863291253913, 0.7783068445528607, 0.5085279344253375, 0.6068272185703985, 0.47462799252178156, 0.7952673054275615, 0.6348766423135669, 0.45577771832105585, 0.090947531504214, 0.5612428509559202, 0.639487234925873, 0.5798013286822742, 0.4624716608211434, 0.3468960462505468, 0.48399025022573416, 0.8013132602194705, 0.8329451497094197, 0.16112891367121035, 0.2059549882883828, 0.11059661013226613, 0.09939351218958858, 0.6605878340360479, 1.4715457475743585, 0.29784512304231175, 0.8226924068158098, 0.5513017359950714, 0.4999979816547134, 0.5241311668984796, 0.3549651732190847, 0.48078990017582746, 0.5536912960582016, 0.5183373298803092, 0.3925998384578414]], [[0.21935918687300568, 0.3634662428003377, 0.416304179860903, 0.7688813233333494, 0.6533811591763117, 0.6155295051105404, 0.48622776796084033, 0.8469951898296798, 0.4451041136197774, 0.38220506889262407, 0.12047507992389413, 0.548006727495067, 0.6198396225124017, 0.5973675127252254, 0.45057976585752463, 0.3611963131507414, 0.5065896395262374, 0.826787401676196, 0.7593440357040322, 0.17145253412079875, 0.21169188884589363, 0.10804424587391787, 0.10005965479026233, 0.6043938890360274, 1.505867980016121, 0.23413146847471922, 0.7433367897129101, 0.4913768571540389, 0.4856102604595609, 0.4578106790350961, 0.3826714129646295, 0.5630658579633467, 0.5872396458407529, 0.6737908768319407, 0.3084089019402174]], [[0.2373639162704074, 0.3768528847763677, 0.4150811559369079, 0.9254935573883778, 0.5084328735485066, 0.6763441550942407, 0.5131457620369811, 0.8563993008021455, 0.39289311804807037, 0.4912934200031728, 0.14434481423064785, 0.61078652089986, 0.5271926255049135, 0.6138400161631434, 0.4216921226824096, 0.35987390963715105, 0.5648104137578213, 0.8225491290585936, 0.713602307632957, 0.21270740130449722, 0.2017054293036627, 0.11079174477181647, 0.10372844008829807, 0.5994926219183704, 1.4850654436160597, 0.22934387265634693, 0.7708601474690249, 0.48715930175838473, 0.4487986327757286, 0.4535666683690698, 0.40268023344885784, 0.5884468262338709, 0.5777533698372433, 0.738528634689789, 0.3422309472510596]], [[0.24059564151789356, 0.37381735929897325, 0.3934664876179988, 0.9775667213472997, 0.44777193138137567, 0.6734104287157736, 0.5382212898796517, 0.8247574916813065, 0.5254506930531705, 0.6394411594382661, 0.17393087345127123, 0.6053930109235348, 0.5143564417929022, 0.6430158460183539, 0.4539652732438994, 0.3469410528033584, 0.5859097965196992, 0.805596097889361, 0.7127014584731532, 0.22595751624056482, 0.21698950437887898, 0.11251624370076539, 0.1498594364661379, 0.6405051840129782, 1.3603121635162907, 0.31931225185358564, 0.7909966948643695, 0.5550232008717972, 0.44367656818742135, 0.447969400700238, 0.4673166029529423, 0.6537217421778598, 0.5845120450669298, 0.7382752273967053, 0.3788379703319511]], [[0.24780757784169857, 0.4131510797316005, 0.4076366352509026, 1.0176742561903978, 0.4168413540011333, 0.6786957169504171, 0.5052469393900036, 0.8003974616101048, 0.6771427617507405, 0.7208250176921078, 0.1620133215200925, 0.6435169371601284, 0.5546801854543495, 0.5853533891244891, 0.4404999878412303, 0.35355945015555956, 0.5527992845923067, 0.7353362059756541, 0.7100941865001846, 0.206707965325193, 0.22380220515820964, 0.13603073225675466, 0.15393322304242796, 0.6506466012441077, 1.2469525825023258, 0.5440170455196085, 0.8042713836510833, 0.6685203835259101, 0.6914514943671383, 0.4171905625329979, 0.5332040950709351, 0.6334804494119979, 0.585433708891153, 0.7437640904829064, 0.44939924148742705]], [[0.25261857699217305, 0.48422605737051483, 0.4213814484423537, 0.9503440617434585, 0.41653976894676614, 0.7268435365265085, 0.5923978264229307, 0.7639523058287073, 0.7477925418512233, 0.8497539075168845, 0.14810175891119043, 0.6206263061550213, 0.586068450555397, 0.6041067954071746, 0.3241975122150977, 0.4058658427701355, 0.5055052714314255, 0.7350489454098099, 0.6048614355570996, 0.1849733241013988, 0.22252267994378494, 0.15165261271022334, 0.1020986543002201, 0.6458870141465352, 1.1656282538961773, 0.6430546518714666, 0.8702747019055275, 0.7178545201089832, 0.8797466553190414, 0.3939006557409428, 0.5329192052120671, 0.6306946404533891, 0.5650375723972592, 0.7055153655658017, 0.4670558465379657]], [[0.26702645659491714, 0.5207093796547315, 0.43048113367995, 0.9590974535404356, 0.42040183012534066, 0.7701892906213212, 0.5717629715578059, 0.7314220158135009, 0.785279869855068, 0.9666434298863233, 0.16432450429371265, 0.6751970738571507, 0.6152616594642761, 0.708851738661596, 0.34874031772405073, 0.42449534158882224, 0.5522271203978284, 0.7092614807598562, 0.5947123216259622, 0.16097548198041225, 0.23598732951222914, 0.22079495965743723, 0.10934005207436931, 0.6502772740174165, 1.1415725952736022, 0.7092096417222213, 0.9584872218355822, 0.7552276671181524, 1.1079963435693585, 0.3913953200646187, 0.5255091125718947, 0.6041000494945986, 0.5579285197583355, 0.7061627770489554, 0.5481577027120741]], [[0.28282776742490534, 0.5487475753973152, 0.4530119700101227, 1.0130010027995104, 0.4584409905952399, 0.7914299626004555, 0.5381880982621797, 0.752199485352665, 0.8197496758007893, 0.9951240760871285, 0.16018709429419048, 0.6541453777501578, 0.6280282580616485, 0.6692991389179486, 0.3544198106675201, 0.43334620876553764, 0.5651673825215392, 0.6827843353461424, 0.6086163698118656, 0.14575696389271692, 0.20973297497985113, 0.24846236697159574, 0.1003756683161342, 0.6497537875060166, 1.0860969929199822, 0.7754720965669071, 0.9841963708378938, 0.784583387147122, 1.3334149784966192, 0.3927148785068106, 0.5454169982263712, 0.5331990212373905, 0.6096261771148853, 0.6842544629297815, 0.6110170024077566]], [[0.22171816089409369, 0.6548676499811786, 0.4012459601132674, 0.892597112657582, 0.3159921671205792, 0.8317610045586861, 0.5122829985053038, 0.6925581938151597, 0.868004086416574, 1.0347561031346393, 0.09394491129649703, 0.6653081415287552, 0.6286633501530527, 0.6316570198126347, 0.36964139598021756, 0.4122684897162103, 0.5511574685086835, 0.7754188197203288, 0.6557865492617747, 0.13810071292810114, 0.19619935165638727, 0.23071099955245053, 0.10537090115935745, 0.6605305811432148, 0.9400283665963939, 0.877712241191531, 0.8979323475233668, 0.79639068037345, 1.5320169669798511, 0.3606246145047285, 0.5500758927056579, 0.45811453682115977, 0.6158362210046013, 0.6723832124979794, 0.6829037790842062]], [[0.21447624870737592, 0.7069006086733007, 0.43772002204390803, 0.8347818327731784, 0.32017802583523136, 0.850063675703378, 0.5021429792479415, 0.7825100891596184, 0.9908061269381708, 1.2158222098774747, 0.09328807748182018, 0.6663045273479457, 0.6552827942704502, 0.5874533384723153, 0.38802620838893004, 0.40738561227312453, 0.5192721941038112, 0.7421186966015862, 0.6556792918772925, 0.13428920340626938, 0.20123826253286053, 0.18138571681297005, 0.10399802309741547, 0.6665680868961138, 0.818360190312858, 0.9631617340961809, 0.803483251579003, 0.8125165515901083, 1.6347139674325355, 0.3844424272010492, 0.5525729004376998, 0.4861634575034044, 0.5572760709253237, 0.65482365694538, 0.6997917405204588]], [[0.23407540839793317, 0.6929283235665631, 0.43501350841744846, 0.8118873616651923, 0.34703556082275355, 0.9169782811186317, 0.466400671399264, 1.3033795095426377, 1.4239907702761925, 1.5214646672110033, 0.08201627646442762, 0.652930781917133, 0.6815251372177452, 0.5793929192676602, 0.41402845222540435, 0.35965186160533946, 0.5152387531723006, 0.7879106510273411, 0.6456631222910976, 0.15863860133075636, 0.2136249857221796, 0.14280027233879825, 0.11655513070513968, 0.665910628744605, 0.8005259563030656, 1.0188910567308682, 0.786374079086529, 0.8210039823320866, 1.709649457775383, 0.3700796705525504, 0.5626523462800558, 0.5026362689281537, 0.5591539156171625, 0.6755401404546906, 0.9172502578447335]], [[0.3205151000965544, 0.8429286724646082, 0.3369730424840837, 0.8318550494075874, 0.39693126459216077, 0.9656207921802704, 0.5085641475401735, 1.087697559160237, 1.489530514857509, 1.5983410708386185, 0.08981754081667001, 0.6246453233760465, 0.6756928639007813, 0.5584230677943209, 0.41079427589153, 0.34955401450189105, 0.5257006966822295, 0.7833156581089954, 0.649102575225107, 0.1628735440711102, 0.21927024527876593, 0.13711002972789332, 0.12018309655784892, 0.687459246741396, 0.8180887273379913, 1.059204195158871, 0.7644667483847594, 0.8619804942131294, 1.6864170712724482, 0.3515865338404405, 0.561641839160228, 0.5285851752588352, 0.6126942406307185, 0.6327440206272041, 0.9872937500429733]], [[0.30903106284199977, 0.933436354449879, 0.4039182807421554, 0.8607054089035657, 0.3984603939286563, 0.9477244353877569, 0.5102513845493034, 1.0083987138382777, 1.5133146221055445, 1.4989868426683497, 0.0985515727027391, 0.602174493054558, 0.708400064150124, 0.574543889076454, 0.35003608196053365, 0.3296092589004207, 0.5359072841426273, 0.7150406927779414, 0.6605978443875147, 0.16640384788211113, 0.22026958744809705, 0.15920147087918668, 0.10776089478240926, 0.7476874916948983, 0.8859031042743681, 1.0726029279870075, 0.7896041834208551, 0.7449967420447278, 1.6710053911275329, 0.3469093538204353, 0.534914787366073, 0.5583032186782148, 0.6061785227458202, 0.5505010135889705, 1.0025300335813179]], [[0.334914157837196, 0.8048153149405745, 0.35570798598657877, 0.9626812922877366, 0.39898157337031187, 0.8968896079757901, 0.4496662405381582, 1.009938524713751, 1.4199218862560459, 1.1667570394546725, 0.08043602295991287, 0.6550693314458975, 0.8505543832254059, 0.6428815558391511, 0.3531775554010888, 0.3221009010940483, 0.5640501570353549, 0.7485825705553777, 0.6550642527576629, 0.167653407250011, 0.22540029442798126, 0.20471708436565308, 0.07887258179986906, 0.7408047795188495, 0.9457412246458463, 1.1175024547344659, 1.062941272270092, 0.6760625685234204, 1.6790597213533085, 0.33191305458182624, 0.453483609837633, 0.5705122265054767, 0.5636389949379808, 0.4606417472024012, 1.0657950134699503]], [[0.3018754396268637, 0.7329382948954581, 0.33817876370133465, 0.9445225117635432, 0.4061457967088252, 0.7858124680275824, 0.42204013880706087, 0.7655052238730684, 1.141456375652492, 0.7980011080323907, 0.044139994909469134, 0.5617554713982867, 0.9259577902448818, 0.6132835130375045, 0.38956761710543203, 0.3343248105443937, 0.5923846388001887, 0.7446795494580366, 0.641273791715705, 0.19336759420543537, 0.21413652640240136, 0.2034092372308664, 0.06582548493960824, 0.7332518739123721, 0.9973390155928819, 1.1242494738540434, 1.1117165057025962, 0.6037021025570634, 1.7197262871219778, 0.3738744635641145, 0.4222714370690608, 0.5754713450090936, 0.5558106630065387, 0.40971717267839874, 1.0947393924098159]], [[0.2670832135620964, 0.7841442561570704, 0.3855431234251375, 0.9356453230445754, 0.3972218119642815, 0.7840219237655608, 0.42823676586421616, 0.5208350695504808, 0.985591535869342, 0.701694589005675, 0.034840993788579196, 0.5234978121141017, 0.8598348932830409, 0.5834909101868625, 0.4099139543411479, 0.3252986488827909, 0.6396824009793705, 0.8349561897638256, 0.6208320240279717, 0.28471279803586175, 0.2433347755256531, 0.193822358905536, 0.0918978982243321, 0.7661791582204789, 0.9912617697294354, 1.1229271290845315, 1.1830141594305745, 0.6207805851111698, 1.8045698571637978, 0.47594055363627774, 0.4006761790433251, 0.5860328435877219, 0.5474796025878687, 0.3976513583611567, 1.1847446178429828]], [[0.237997697319503, 0.8127329336052187, 0.38320541459750807, 0.9017437004275333, 0.3817442620778541, 0.7878200036986427, 0.4315264643059824, 0.5197260640962695, 0.9416515616596162, 0.6566811894688848, 0.027582516092839935, 0.48319198838502453, 0.6827950836234581, 0.5318098665858999, 0.3335541734806872, 0.31432525483266976, 0.604666520822084, 0.7904339948470078, 0.6432273625761215, 0.2714149200797952, 0.2365025287937138, 0.18877306583653142, 0.06951410978382286, 0.7565699665850972, 0.9348788334508736, 1.110694937561544, 1.0563200580123335, 0.6171905839815378, 1.8321911927042445, 0.6624873790810044, 0.4369302823606246, 0.6158628514852266, 0.5546330835178639, 0.3627047975868926, 1.2719617378929686]], [[0.24857896500398258, 0.7523294538092864, 0.38376866308992563, 0.9284930406541514, 0.3762308342032783, 0.8237593134066277, 0.4476044405584342, 0.44195291947157433, 0.9090590907746452, 0.7048842000930292, 0.03517228717446575, 0.4446570019772835, 0.6178728613075677, 0.5265627240091686, 0.2732769130119601, 0.30500498134041754, 0.5883013607365456, 0.7742852500937285, 0.6661788852921767, 0.2580616033298506, 0.2036871231552413, 0.19185770618805992, 0.0732432421563317, 0.7610818994675113, 0.882508459177524, 1.117110363594732, 0.9715629486346311, 0.7070022239086841, 1.832638094424719, 1.0576641883287983, 0.5329832085271223, 0.6272219378826072, 0.5435106893046506, 0.3053370389332962, 1.4296068230821812]], [[0.24188644063760723, 0.7779979765879622, 0.4079496688691866, 0.9600321273253769, 0.35572747123022463, 0.8035694141193404, 0.44340562612312384, 0.5279819254833704, 0.9448112095967378, 0.6901830571097284, 0.029575532478224316, 0.4122491464140819, 0.6150030637573197, 0.5091004685004992, 0.27587901632546913, 0.3180255205785021, 0.5689681095238606, 0.7996702455533571, 0.6653770386557681, 0.19009793177815354, 0.20162469967304977, 0.16596897580982173, 0.07888287986171491, 0.7541475182314892, 0.8490844953947342, 1.1231873211739756, 0.9387204311822277, 0.7238294014135007, 1.831336681147729, 1.2181101471732032, 0.7103728002613304, 0.6518646442438687, 0.5517344861727141, 0.3476583112321701, 1.5509059774796963]], [[0.24444737647435966, 0.7233943848821169, 0.41657020937977357, 0.8713757184801673, 0.3527124296009746, 0.742050948354613, 0.40194621984963663, 0.5456394169642677, 0.8984605116363356, 0.6701617476650169, 0.013655291330929218, 0.4063200237183441, 0.6145278497138972, 0.5180880427156211, 0.29073696643659613, 0.29516390032796025, 0.5780531061114942, 0.7555716172718395, 0.6659436176882909, 0.20093029356447184, 0.1440713569379644, 0.14343363407222182, 0.08198277669102771, 0.7266555143929457, 0.7892911851145088, 1.1249274994203553, 0.8798107975260687, 0.6629548890439612, 1.8633243827420722, 1.1520690225220487, 0.8130877436347184, 0.6832761800204366, 0.4163157199746602, 0.3492026047877955, 1.5444059336394944]], [[0.26214917068360954, 0.7605282258657609, 0.424662182204105, 0.8969004990615901, 0.33885399873043276, 0.7099583893256034, 0.39706436627908936, 0.6245298712226799, 0.9082431454873026, 0.656083676603164, 0.04562172446302835, 0.5134891888028293, 0.63994559416428, 0.6133211869252053, 0.30206932094514966, 0.2924521479008017, 0.5656959350914501, 0.758523305922703, 0.6642432858556505, 0.19669346891370787, 0.13741485025846273, 0.1492643809406827, 0.09405772985524447, 0.7660974809959555, 0.7672216025427632, 1.1223832132525116, 0.8160738211047427, 0.5990152994619293, 1.8938124050480023, 1.0285720612826088, 0.7359547310264121, 0.700757764240094, 0.3961666738590466, 0.21645370146252557, 1.4978408598864514]], [[0.2652692421799517, 0.7689417606897899, 0.4281988618212113, 0.9444744504981293, 0.324632884759947, 0.7157672744135033, 0.37714694849256364, 0.7067673232967191, 0.9606042526643449, 0.7319224560303951, 0.06579624369608565, 0.433655751171781, 0.6564201353293182, 0.5903577218029666, 0.2711043176767523, 0.27756865137747155, 0.593469746922848, 0.757089603896468, 0.6757540076290773, 0.19752715995950593, 0.12348432086408243, 0.1702276843332463, 0.07536345250967436, 0.9061289774306044, 0.7044456432035262, 1.116125999705741, 0.7790910527621597, 0.5590091181908154, 1.9253210031575372, 0.9345062959906271, 0.6567158023402888, 0.7116800958587753, 0.4427324365152083, 0.11254220626076335, 1.3409889626979445]], [[0.2375778401141917, 0.7513843004240265, 0.4044457248930533, 0.9773176060969653, 0.29795564089089577, 0.6882752366582623, 0.38153941902073657, 0.8210617227863812, 1.1528204857879139, 0.9069721471073046, 0.09366629187582237, 0.46804883663977603, 0.6370703694335935, 0.5526268216591891, 0.25453076580652373, 0.2639237462716204, 0.624814253499331, 0.7582374164617092, 0.679074559771442, 0.20170899703250922, 0.10546756331360826, 0.1420359067176927, 0.07662382025161199, 1.0220653397139983, 0.7311536579583803, 1.1126603988303851, 0.6809124188470311, 0.6083146893383788, 1.9659641192529294, 0.9196032981450224, 0.6827672634823108, 0.7265159435812334, 0.5371635758705399, 0.13153243141329848, 1.262200040099717]], [[0.23824835902190178, 0.6805353652363536, 0.40233234928721057, 1.0609101925380278, 0.2927724818445564, 0.6646631517207013, 0.36517665302274066, 0.9089770142408392, 1.282986992170551, 1.0779911430681401, 0.07550872409842842, 0.47485405357522004, 0.6681481079039315, 0.5601721177192407, 0.24827367022649166, 0.2729020686928082, 0.6384352631861677, 0.735527598074011, 0.6874078530369537, 0.18251300444054727, 0.1056426222143731, 0.1198240231099074, 0.06848590006613817, 1.1338819564583642, 0.7757680370014324, 1.117537597412025, 0.6733136379898839, 0.665945169221222, 2.000736572386192, 0.8804797247337335, 0.8060722889505458, 0.7456875138621705, 0.619513398470609, 0.2755973436816564, 1.2264321395276703]], [[0.25385417231253965, 0.6234290364700319, 0.3579509323691769, 1.0692591069166593, 0.30230784658588117, 0.6841508875772813, 0.3636694682078259, 0.9746041415737721, 1.265647365323173, 1.008134996092352, 0.049904052587975756, 0.4861622525616051, 0.8244171165092726, 0.5704610137192205, 0.29116384189215255, 0.27833747523489816, 0.6440416842485983, 0.739563076245272, 0.7246517452981049, 0.2087232700259971, 0.11871205125155361, 0.1363503267809063, 0.06524954824058515, 1.1458607883599492, 0.8221630690913497, 1.106477244928757, 0.7034952919664741, 0.7016050839183032, 1.9925854052404806, 0.8136782246650812, 0.7926712885789189, 0.7988303252266479, 0.6846070052403695, 0.3215443178067998, 1.1860957239587169]], [[0.27860463884364806, 0.5183375856197627, 0.2989726404066106, 1.117032925683, 0.3041993533739633, 0.6976393077390243, 0.42350254905374274, 1.087096361246067, 1.231188096952326, 0.8471534816546411, 0.03954269011626355, 0.47875414787268256, 0.9013766938253394, 0.5334475176230931, 0.3217484899599758, 0.2870050424607596, 0.6131801457175632, 0.742836597392659, 0.7322554479894408, 0.22517580706382406, 0.13643576507849886, 0.16487567190666055, 0.07034226799891026, 1.2105169901618877, 0.8416689642157802, 1.0871002043822213, 0.7473447199099253, 0.6666151318429696, 1.9347520413715016, 0.820407132018662, 0.6886382732219349, 0.8420879475078054, 0.7597195420465171, 0.28004545205079223, 0.9344522213325968]], [[0.2731043146391301, 0.5081803531120054, 0.3209544619790014, 1.14936094888587, 0.3044024869972016, 0.7021245785284276, 0.4613629244473795, 1.1529213077809823, 1.133538952039687, 0.6857027821475228, 0.05323491075365484, 0.4792232503060213, 0.9677755040630838, 0.516413290321917, 0.33208450164593584, 0.3298526012364599, 0.6393613989117233, 0.7359998661623484, 0.8140395839944352, 0.1955938442491632, 0.14708001667066853, 0.16118335147483304, 0.06483008762549511, 1.2225481533782667, 0.9077016637900888, 1.0655100979720475, 0.7497108423581968, 0.6067877613433171, 1.8084860885838292, 0.8261532355495237, 0.6802234238540465, 0.856645198987264, 0.8335191281634711, 0.2717667051778806, 0.7748094312672635]], [[0.39673420691765726, 0.494914920488906, 0.42089227018035097, 1.2063590297785245, 0.30523932267759313, 0.7042013565891242, 0.4940338760540445, 1.191870239187922, 1.0742673714534183, 0.6159837356328269, 0.06152996706187548, 0.530294066791127, 0.9722993736893021, 0.5324444086564182, 0.32520349674736376, 0.3620211385979624, 0.5962077071880458, 0.7432697834950289, 0.8069809558733603, 0.27314698522214165, 0.13741350650402065, 0.30847959284552773, 0.0587363612011873, 1.2065480003644748, 0.9785744319249584, 1.051542598332841, 0.6732449415628005, 0.5983975916840436, 1.6823926678055727, 0.8218018812255521, 0.6891314627395795, 0.8356996122993953, 0.9094949506880987, 0.36574296205384965, 0.7707533331174241]], [[0.36459163381727144, 0.5146434192880849, 0.4234517348994574, 1.236944710053158, 0.30611166779496046, 0.6559849046353213, 0.5051699843425002, 1.2205643382242921, 1.0217719722784646, 0.6230279095018073, 0.07259783463017505, 0.5004457446831488, 0.9425313976289789, 0.566026629596283, 0.33724649657484324, 0.3193869633666771, 0.6754028715578604, 0.7407796003911922, 0.766888299112912, 0.3678505894267501, 0.13714283118193513, 0.2719749113705681, 0.07290096383164649, 1.189003246628655, 1.0104932573748509, 1.0256749614717977, 0.6046622222311773, 0.6637063762841047, 1.6162570823641862, 0.7982820315123613, 0.7205858773540528, 0.784818100753154, 1.0024962640294368, 0.38338314496480697, 0.7416166925117166]], [[0.34829308223797106, 0.5209362168203534, 0.3645546961123082, 1.2702193940760549, 0.30709692541994393, 0.6636090771829353, 0.5160078598643483, 1.2155063102925532, 0.991212010081322, 0.6942883152356072, 0.0633051726487389, 0.5343211581140059, 0.8805901325275269, 0.5875419842805925, 0.2868510594558485, 0.34078933866422806, 0.667306097872981, 0.8004518109804358, 0.8349114094902741, 0.41331365054764946, 0.2396945972076189, 0.27207135960709994, 0.04020704461090545, 1.1618534205946225, 1.0124309256075046, 1.0008080108918802, 0.5129602229349128, 0.730278355089386, 1.5783413616677906, 0.751461989758883, 0.7161212953489468, 0.7228898270072979, 1.0649508294101149, 0.3775007905658231, 0.7122184007316003]], [[0.31790439685449495, 0.47594790035148893, 0.30462920507895913, 1.3877982192446243, 0.3046934430269598, 0.6463301898071581, 0.5480266652543065, 1.2029393594205253, 1.459157781574594, 0.7190297987205577, 0.05412489007401275, 0.533338506122994, 0.8784848596183046, 0.6630542282794822, 0.27949497464448325, 0.3522930106654868, 0.7267370291780455, 0.7397156993656178, 0.7989546227056673, 0.43337881348925966, 0.258894361555886, 0.19882955917719378, 0.05319669694433998, 1.1833438468970743, 1.011774989583114, 0.9707701454031373, 0.49984723576975976, 0.7262791636574609, 1.592712058752661, 0.72289236399097, 0.6955802592351253, 0.629164138047251, 1.0583090357928653, 0.36117617638088295, 0.7078454580678605]], [[0.3652873735333107, 0.42602037666044684, 0.29198704588974983, 1.3439873809300398, 0.3043531275600877, 0.6384235200834654, 0.5791049296461617, 1.1799630297113797, 1.4291909509419174, 0.7598540829981986, 0.051852680146714625, 0.5318611086316558, 0.8134242392729163, 0.6711092947689676, 0.3132089719173809, 0.3526016935333023, 0.7122890042944988, 0.7897379092653869, 0.7616793849865877, 0.45022166538595043, 0.2760924809521152, 0.14092436581866472, 0.04943410160310227, 1.0783878985362298, 1.030394940566716, 0.9396986833765943, 0.5367898059130041, 0.7257664770575802, 1.731201131710328, 0.7337176672238742, 0.6647046162432529, 0.5178819855625061, 1.0871901125225911, 0.3136011188874286, 0.7097658498813445]], [[0.4451936550477217, 0.4137625030048409, 0.2906272060057177, 1.2207380166436255, 0.30173612951446027, 0.564173825338531, 0.5781392150186195, 1.1631892209639412, 1.4357482830839121, 0.7939026492143031, 0.0415309777364478, 0.48806171456073677, 0.7992522782823086, 0.6840912028814508, 0.4069426552849356, 0.35364115755071457, 0.6589939227114556, 0.866131594258857, 0.7503174782920483, 0.4428091028747148, 0.2855634873935431, 0.14954449952142393, 0.05952221053776241, 1.034198817039155, 1.032928236678158, 1.0039876338804616, 0.6014658759884584, 0.6798154328692235, 1.8923200720549636, 0.7599259443776083, 0.6734727102180671, 0.45463776349973695, 1.0792299400348868, 0.2565894315251943, 0.7348619130250285]], [[0.4873227174381414, 0.3653384375912877, 0.24965945450398203, 1.2603959677735213, 0.28972756767071883, 0.6376054228690804, 0.6761913126677255, 1.1325110017657525, 1.4128676808599416, 0.7591168393546611, 0.04348683015991664, 0.5049592246634445, 0.805668940308524, 0.6804809746473957, 0.3584482333821252, 0.35332598653344915, 0.6897519410771816, 0.8840944538389724, 0.808059993826508, 0.5240930125821237, 0.34876053894160847, 0.1806523102413691, 0.06794918822474194, 0.9999450532846607, 1.0586285424197892, 1.117627052326885, 0.5631944442801868, 0.657881307627265, 1.989612074287923, 0.7964833870685273, 0.6880996582908292, 0.4122647486188731, 1.0471966598141746, 0.20256788224798716, 0.794382896511928]], [[0.5894213259605945, 0.3455845220956517, 0.24449174739933194, 1.1419053898855647, 0.26804867535036436, 0.5759664924754376, 0.6893001022379872, 1.1048520274904157, 1.4266226879108013, 0.7390784421839159, 0.04414250869714578, 0.5561293183008783, 0.7480769835955724, 0.7083155407854638, 0.4567805964526404, 0.38847388817635453, 0.6641276862121562, 0.8186479839754104, 1.2926709507656478, 0.60217631013451, 0.37642544814128776, 0.450660741977642, 0.049681507289643975, 1.0023914217923735, 1.0536031277031972, 1.2540493338865266, 0.550592594280642, 0.703912487775678, 2.030625606803352, 0.8150971615441698, 0.6738632845275724, 0.4228594596857116, 0.9488030614921328, 0.19267765740814746, 0.7763967266839029]], [[0.6398326047809637, 0.32878920020269625, 0.25216374302956657, 1.1498385121392356, 0.2413700139694439, 0.6395821185245368, 0.7200523819359306, 1.0682195846396603, 1.472179745425247, 0.7908340646429831, 0.04947938127463259, 0.5607381626151011, 0.7107980509314689, 0.7094357949747243, 0.5724009869124718, 0.37266512517438344, 0.6906868393799557, 0.7791622924472383, 1.310520715074985, 0.5988601379772852, 0.44765017664102547, 0.6397665884058119, 0.06425888598241832, 0.9612711373040849, 1.0418164894745958, 1.3435430988203714, 0.5395358207964582, 0.7404813146611962, 1.999706504966096, 0.8300448297574998, 0.6418843128356944, 0.4411317339003794, 0.9308579463181121, 0.17383472741925124, 0.7657812050911725]], [[0.6577429119563851, 0.377878212368734, 0.23055122292585234, 1.1109525759491563, 0.22886778993533163, 0.6717748841111169, 0.7707631375882877, 1.0058832825160913, 1.4641929428993397, 0.8207033886929678, 0.07114516987112764, 0.5241628692228153, 0.7651154004297915, 0.7449086540392688, 0.5398547031591341, 0.37257155078111515, 0.6276161110787595, 0.7745336722716054, 1.2431165281045933, 0.4614929739472371, 0.4737400943683395, 0.815398244318046, 0.037046094063822185, 1.0319064304162973, 1.0254069674872568, 1.3839703062898132, 0.5242610455261634, 0.7806145147041653, 1.9191279975375093, 0.8606574714123533, 0.6270605261935538, 0.4602120487070006, 0.8999438653272722, 0.1987232127824734, 0.8232598329823192]], [[0.73522043515466, 0.35680234282496476, 0.2176376558568507, 1.1012342875553585, 0.2218519872230872, 0.697062144317548, 0.7605804124090256, 0.9566078239081743, 1.4802893154902517, 0.8580722337205724, 0.09889908136827716, 0.5619702885941817, 0.7636847784894474, 0.7656575689615077, 0.5504347213951459, 0.3918602539185899, 0.677519512305794, 0.9386021490126488, 1.2986692339143426, 0.20019742626154863, 0.40141742998642954, 0.9039166517175339, 0.03551556899088585, 1.147689040346509, 1.0410272915402436, 1.39810469713706, 0.47044006477197153, 0.7005306458876439, 1.8128341850719292, 0.9297766208344983, 0.5875250441101101, 0.44578796471279947, 0.8575672380522634, 0.3050338538925974, 0.8641306438674932]], [[0.862670559916195, 0.49195061851804, 0.29336283961941007, 1.056440447522404, 0.21556676583397938, 0.660196920029537, 0.8094263170626166, 0.9436421325536046, 1.5138933144990614, 0.895324832766951, 0.11957127163253733, 0.5526819319849399, 0.8543462186584934, 0.7546666054287162, 0.5857298540168255, 0.4071827119549406, 0.7013420768020858, 0.8345344530509913, 1.3783237631688823, 0.15746611399522903, 0.3435975682242248, 1.0031688921855724, 0.03476912721050289, 1.012751896556962, 1.0601505706504735, 1.319985459310077, 0.472648046594772, 0.700068363247395, 1.6790338072664182, 0.8571375320726193, 0.607192258023571, 0.4253768420730909, 0.8101375061884513, 0.3739497455952741, 0.8584785321865023]], [[0.8935869637156343, 0.5243411422066877, 0.29731213601560047, 1.072509554796879, 0.3296355518877351, 0.6995052509998123, 0.7199240464913353, 0.884512997714901, 1.4350749315999258, 0.9017378958690992, 0.1333795031483729, 0.6362659513978703, 0.7798436441142356, 0.7805098822241658, 0.5795989095420457, 0.42970921444081556, 0.7331570710327941, 0.8608148936272304, 1.3731641195543394, 0.14945025118772176, 0.31209347455317493, 1.0969704946784753, 0.04212089268670169, 0.8541797595371861, 1.0523068906319974, 1.2913309274050575, 0.48698758335670733, 0.6711551565561064, 1.5435105263992135, 0.8483840516184361, 0.691534576914897, 0.40993019381781237, 0.8107735580456098, 0.5630520411470721, 0.8552624619556635]], [[1.0068945280932313, 0.5587820775107605, 0.2839042677520003, 1.0200210260758813, 0.3526649842679069, 0.6865190500108329, 0.6894185785352691, 0.8608966883198087, 1.4366549824529469, 0.8669904919532883, 0.154396034905428, 0.6067490832397694, 0.7409849849803445, 0.7685261913052506, 0.5768867444802308, 0.454362218050832, 0.7731579508829127, 0.920611930886876, 1.399074690174835, 0.19497069014899276, 0.322379362836445, 1.0422275451140517, 0.04472802812864468, 0.8562957584469129, 1.018851286789186, 1.2578403520650274, 0.4536335577518261, 0.6788673243923942, 1.452970390282044, 0.8701001919557609, 0.6950104261194364, 0.40195192406762015, 0.802821954786126, 0.5968776029603791, 0.8538398003680351]], [[1.2797614940303825, 0.5724197220965717, 0.27813601619241685, 0.8766732167847681, 0.39817229396249665, 0.7234859630271593, 0.6746283557340956, 0.8023249470123253, 1.4492929395950718, 0.8469275732554377, 0.16112159596318365, 0.6037661259726513, 0.673377691508956, 0.7792331013168048, 0.5568620770203754, 0.4264358366154817, 0.8489438795435882, 0.9148148329192161, 1.3031899426847282, 0.203965730434219, 0.2763944710572571, 0.6797450479035829, 0.04146539069796343, 0.8300627239086459, 0.9812976352020083, 1.223823386346952, 0.5468309736880042, 0.7463870686874257, 1.401736790530694, 0.8601409023097472, 0.6707418436063293, 0.4032467793645679, 0.7767654704367145, 0.4981961702103996, 0.878529921906162]], [[1.2906616561261415, 0.577609324423396, 0.2680188737631394, 0.9168762354824191, 0.4681945323582188, 0.7868592288646978, 0.69084322610753, 0.7491658559998906, 1.480880365921011, 0.8597154883567906, 0.18492327219738672, 0.6025231646748168, 0.644092755433904, 0.7787436873903648, 0.5891755493258303, 0.4300455773902041, 0.8394252825065399, 0.7695604205110027, 1.1695140085117435, 0.20533119532457744, 0.28244037796365556, 0.4109679533769047, 0.05264862045637772, 0.8071458694660003, 0.9098769022584692, 1.1664091924153226, 0.6154677530952675, 0.7263673943576026, 1.3426621875702918, 0.8809173079874325, 0.6942490686297348, 0.3817242167675968, 0.7646821771057264, 0.42738342434649174, 0.9235139518212049]], [[1.4966710754737602, 0.5618615754473794, 0.25898860412169156, 0.8605524642586129, 0.5777928133149379, 0.7606679722080462, 0.6761412956638373, 0.5985210946058479, 1.4504819752668163, 0.8342949047329209, 0.18962007208794912, 0.585904676888557, 0.6168814938296846, 0.7940645165271487, 0.5800627862585783, 0.428375202396, 0.8630149523320713, 0.7701589282603597, 1.186105343661857, 0.2508626059711076, 0.35903175920157143, 0.3780178282885637, 0.028674530340453003, 0.8247291413968381, 0.8343663641730725, 1.06871838401203, 0.5859746277820197, 0.6774956855427383, 1.2305765646763813, 0.9192136344282036, 0.6949836587310617, 0.33707957190129545, 0.6720722601077993, 0.3910374266272815, 0.8924384788450357]], [[1.512814438809326, 0.564995815991157, 0.25736132207948065, 0.740197984187388, 0.6715769591339834, 0.8054455128661024, 0.6712315461648446, 0.39582349460178967, 1.3592426383312999, 0.8410163073140194, 0.19281268492328058, 0.5710613260886455, 0.6421919185446411, 0.7717353908902245, 0.6018609066362988, 0.4395685483217724, 0.8931819339831522, 0.7555230477679342, 1.2644548487253138, 0.33828206025424523, 0.34822674455172653, 0.3632232261261017, 0.03985444794694266, 0.8038071307193775, 0.7061734123508773, 0.9446136797390965, 0.587860964963516, 0.6659040478771572, 1.012765705303088, 0.9250756228016082, 0.7391306114051505, 0.24929908907823278, 0.5200384069043174, 0.35915362714536786, 0.8257545049376516]], [[1.40185861086372, 0.6055094312289839, 0.2468003699846828, 0.7231270350201627, 0.771760732517043, 0.8008921344424964, 0.659152717503685, 0.4195945972278635, 1.2152474530537398, 0.6122058620220292, 0.2008140703031533, 0.4959582886527938, 0.5952966309647734, 0.8150470556749077, 0.5788683458035478, 0.33363277837918126, 0.8338114618563032, 0.5856858786626172, 1.355481863835432, 0.4602068843594558, 0.38679733006603423, 0.3713977302270983, 0.04605373728722105, 0.8242646251738336, 0.5513279128266778, 0.7947353174047378, 0.5814448815073664, 0.5753318428009189, 0.7901297610448625, 0.8684756174201748, 0.7282352029614609, 0.42803172083237295, 0.41289026390000727, 0.3783457042943832, 0.7593086592401703]], [[1.5254264627089236, 0.6787884545107725, 0.23779512507147177, 0.8044169884816038, 0.7532324380788218, 0.7805402660833909, 0.6393929914228311, 0.4431577064631616, 1.0703269569556355, 0.45542942485905313, 0.15879323466114673, 0.5277113904711623, 0.5635630429908134, 0.891552243341218, 0.533475669229871, 0.35716390103450135, 0.8605051199193093, 0.7239416411781872, 1.1552670320767446, 0.48550946332277056, 0.3917934906731103, 0.43694505474356227, 0.04529400622267489, 0.788168203596038, 0.4130568404944907, 0.5379118287819062, 0.5481586273316026, 0.47656044662627567, 0.6210579881976066, 0.9700981092568322, 0.7607122137579958, 0.6721705766580015, 0.35024673579408694, 0.4605842340822317, 0.7036487682968633]], [[1.518184368422419, 0.7084038520064732, 0.2268025139778888, 0.8733246850010863, 0.7906574699186162, 0.7567792187622618, 0.6202392021883472, 0.5536626295204977, 1.020773287529153, 0.3510058700383769, 0.16275998696261984, 0.5251121579589701, 0.5558562378212206, 0.8815741887452404, 0.5131171879283771, 0.38164614121791185, 0.8905376787458656, 0.8757771000223503, 1.1241069211659727, 0.5017171407898654, 0.4880808268705812, 0.4788108158955847, 0.04024015611539306, 0.8130517461379551, 0.44218691935843, 0.41972379715109165, 0.6875978309218214, 0.30523467003737104, 0.5183710074534695, 1.1314820180334069, 0.7527390093985915, 0.6775787414466576, 0.33147439928417244, 0.4926517985937824, 0.7644666823009777]], [[1.4965777304794141, 0.8829865871565739, 0.2362010659441311, 0.9918615111177108, 0.813753540657062, 0.7330683054699624, 0.5695264872253409, 0.7226397353975466, 0.9471381194286947, 0.2871994759301899, 0.22465004913965386, 0.5359446465669273, 0.5600847648457778, 0.8856694871308156, 0.49649980788312087, 0.4003238835225954, 0.8828120263192223, 0.8729341715248129, 1.1813463364203776, 0.5130930097492145, 0.4976946516430848, 0.5553660135717337, 0.04273150222454357, 0.8724417357062927, 0.39860870043938085, 0.4378704108026654, 0.5985749790151772, 0.2262820529261392, 0.506089102202364, 1.3172360018346554, 0.7527232485173985, 0.7033810534938854, 0.3155040566771987, 0.6212561850095708, 0.8127612726753359]], [[1.5286428166603112, 0.9405661945737634, 0.2416251990041059, 0.9754477535062014, 1.0938270581042597, 0.7078974085591174, 0.574679081794397, 0.8547532066846909, 0.9283977800225062, 0.27476506215231755, 0.23025729004879297, 0.5382268809429809, 0.658749914604996, 0.8581461999028839, 0.5724661369986273, 0.4326488185659014, 0.8682282078176708, 0.9104033761714871, 1.236596946392769, 0.5358409915811356, 0.49259734177073744, 0.5686958527319999, 0.04032649712743784, 0.8870976796801279, 0.37399394102404004, 0.4752921849663343, 0.5041131869417954, 0.22713844660972668, 0.5127419141924517, 1.344973338021317, 0.8509552101513655, 0.6780973996374065, 0.36877577162084574, 0.7694407600150155, 0.9258905597302043]], [[1.4062125426134116, 0.9925514985608971, 0.24113197222037142, 0.920150110822417, 1.1438792935707294, 0.69564855175161, 0.4802474944310868, 0.9252405470552237, 0.8838181609990525, 0.266833127204369, 0.3190744884105601, 0.47737966870829684, 0.844561869402418, 0.6482750779260746, 0.5194486750106635, 0.43444538232464236, 0.8614015512377662, 0.86471542587493, 1.1867667898004859, 0.5279465381073547, 0.47047775741405934, 0.6082811601775419, 0.02620017453414614, 0.8488629469892479, 0.3197675761953437, 0.48818029218510517, 0.45588815164001245, 0.22238491247689754, 0.6052944663074308, 1.330811547910736, 0.8196541689890253, 0.5900205629034909, 0.42506518495383716, 0.9833372565732919, 1.0473456480876155]], [[1.3119432722637556, 1.0329289506488384, 0.24009239273332084, 0.9158034400562296, 1.2084851581484992, 0.6875340584535649, 0.42339542758039683, 0.9699734717652408, 0.8611022256752572, 0.2658138438630788, 0.3249391619681755, 0.9745682638468169, 1.097460814258429, 0.6587241807754287, 0.6011094709846179, 0.4315825372742289, 0.4511163660256209, 0.6633943209483042, 1.243386284077773, 0.2767034041281571, 0.37293419675223743, 0.6046956442134982, 0.026479124311655146, 0.6925110984316529, 0.2884380418229878, 0.5058199546065323, 0.41987727719696155, 0.35848321865670374, 0.8046334776302957, 1.3337620034466986, 0.8040762623673978, 0.5044877612589488, 0.4252550337205845, 0.9850200520238014, 1.0670151575181699]], [[1.2417477503229077, 1.1661916174704312, 0.2881713009300596, 0.8482854026483131, 1.275216287286248, 0.6868046456270676, 0.4364791547106328, 0.9751744256275261, 0.8270711260992946, 0.2635196892177036, 0.395781261589068, 0.7906562546432908, 1.1300028499192438, 0.5773955050523754, 0.6307257438311663, 0.43051901709808027, 0.45593430284576486, 0.7532328450753555, 1.3097157206692374, 0.24060131509069943, 0.3328147623960602, 0.6513561157168022, 0.020237963210660948, 0.6023114392236691, 0.28848620837819483, 0.4842979042090465, 0.410638480775285, 0.4354123764623266, 0.9772794732116521, 1.2746708893889185, 0.7846455256240471, 0.4298267439592688, 0.42962874273492835, 0.8887857605822767, 0.9742013304528321]], [[1.1599348351059695, 1.153348533456841, 0.255208930876331, 0.8473337036394284, 1.331374230360566, 0.671694039095982, 0.42310065974575006, 1.0038853989673053, 0.8079224995694879, 0.28466812621951204, 0.3761034171940228, 0.8097215201566653, 0.8980529737983762, 0.5979798490076103, 0.5665304856965843, 0.4272344576661975, 0.45027207326377505, 0.7378059694440495, 1.2928114187645185, 0.2521961436914364, 0.3432962464718531, 0.6541211328347062, 0.020400812055251877, 0.6464900324921415, 0.22588802144530115, 0.4608646867168718, 0.4308238442350615, 0.4263244791535015, 1.0330915564868037, 1.1704214776599167, 0.7295755612606093, 0.42666069893298836, 0.4494309229295479, 0.83860357302201, 0.8516005513523053]], [[1.231998169888898, 1.2054724786875728, 0.25476285532248794, 0.859554006051577, 1.3789397846234934, 0.6629428595702922, 0.4326853558891219, 1.050110717463479, 0.8315664375282725, 0.342714634030357, 0.3171561722273356, 0.6969148700801371, 0.7748583388682271, 0.5985040768529426, 0.43292943756695684, 0.4033615959573783, 0.46574846173935897, 0.6142400047192451, 1.1541499588069497, 0.26494903362327804, 0.34877261512123436, 0.5956037657801243, 0.024916345506904386, 0.6733294857001086, 0.16934787173325438, 0.3886280594794077, 0.476189804322538, 0.4001170667943378, 1.0870987333222886, 1.0552319978159441, 0.6630284447677935, 0.45463372475329256, 0.4297307794969426, 0.8483353579392121, 0.6710529484547859]], [[1.2228703824527238, 1.2238548121786477, 0.2478702333366617, 0.83876599697235, 1.458431797565719, 0.6913659468271072, 0.4085877703284993, 1.0681976532625415, 0.8557335703660385, 0.3821023340419547, 0.2863142365534157, 0.6736023841147197, 0.6140102647077171, 0.5944048133952018, 0.4412348902065335, 0.40524779753936935, 0.4832599553134336, 0.641189740467533, 1.1041271044745724, 0.2249524722583735, 0.32115113072437795, 0.636503303992044, 0.03051188599195888, 0.6635696733597267, 0.1556679783270906, 0.26371441832496223, 0.5403741117071409, 0.24033435782965148, 1.1907842763030951, 0.8831724144584681, 0.6929323097013997, 0.529842205685683, 0.39960979831125176, 0.8690077821636351, 0.5904097534259568]], [[1.153392061920072, 1.2802182290027169, 0.2587985643338221, 0.77418924618245, 1.5020964691986878, 0.7120837916108833, 0.40953113348198195, 1.029970197602675, 0.8691063825983062, 0.38951850877900274, 0.2739469133131518, 0.7669647069214348, 0.5582563162602961, 0.5919286973461492, 0.5000271589214564, 0.4170114875124816, 0.5627908275471415, 0.6094804798856849, 1.188010888912704, 0.19557791468513383, 0.29298537013062365, 0.6658807057298519, 0.04382600088819416, 0.6379274466567785, 0.2154152339138239, 0.35489567348771217, 0.5308814850972232, 0.2007087338420603, 1.1262390008245173, 0.6627212109526299, 0.7860432935969046, 0.5406626843949566, 0.39184286117127926, 0.8700097118087433, 0.5409500414921862]], [[1.1485027267221826, 1.2154402512662805, 0.2567128343897367, 0.687054410881013, 1.5894149087082403, 0.7018156466052198, 0.4103938230851963, 0.99227419657627, 0.8778035603290724, 0.39327814892188717, 0.26750652946793174, 0.7679377494637639, 0.5700788438526205, 0.6560934191242126, 0.5044698008223004, 0.44049901271998626, 0.5528098956060683, 0.5627880140545198, 1.1430698881834345, 0.1895168576892782, 0.28968586606861835, 0.5983298012096487, 0.049570693995590207, 0.7445915153978201, 0.20505773616923137, 0.3409715398962539, 0.4461038868194816, 0.20277681566840194, 1.0031419736204663, 0.4901268359131269, 0.8569104287769677, 0.5535291958777514, 0.39134251691231814, 0.8669636010494215, 0.4860589628116787]], [[1.1214296125200869, 1.2775412846733778, 0.2563588830273953, 0.6521360513305522, 1.6162040660512726, 0.7146596684868101, 0.40007333606560846, 0.9216795682020607, 0.95963382940296, 0.4297779545050268, 0.28207713139347157, 0.7109631646684509, 0.5381053218048204, 0.7100198258630117, 0.4866525958552357, 0.3941161746238539, 0.4996183650290317, 0.5464928767489786, 1.1193598157146805, 0.18833872513096472, 0.27587890623957395, 0.6017999791097465, 0.04864423173034184, 0.7927355914724006, 0.2852418810586847, 0.38184566377692636, 0.4230655300389539, 0.19519753907486548, 0.8954057668410004, 0.42693347732567444, 0.9153844034769709, 0.6062306554860704, 0.3849292075238574, 0.8243804312294412, 0.4242214920460303]], [[1.084606779854843, 1.3302523535558655, 0.2573987989747793, 0.6029764002182745, 1.5956800333630656, 0.7218680809992186, 0.4257628750834689, 0.9063792698753386, 0.9309186463572543, 0.4044223173673896, 0.2930314328615422, 0.7185651791248074, 0.5228530910889116, 0.6920414927392313, 0.4789925185860677, 0.43533099063740577, 0.46067089047536014, 0.5637670740068869, 1.0587320532085926, 0.18475509478080762, 0.2795332388834355, 0.4849801223380743, 0.07272018837915106, 0.8441186806631218, 0.3015399214003352, 0.40756106702651257, 0.46856677585026574, 0.2312456179948675, 1.0124315795273404, 0.37729996635932933, 0.9631918782603412, 0.6647756932094291, 0.39489667531447287, 0.7853362866853495, 0.40253801273114287]], [[1.038130877367947, 1.361611582057461, 0.24777332872008467, 0.6236345406632757, 1.6397807534729534, 0.7368406327525957, 0.394679614564722, 0.9620492117409423, 0.9447027081838509, 0.3648232922078021, 0.23643513728609802, 0.6950354496921527, 0.5215512880503169, 0.6340129072879715, 0.5333560214666475, 0.43460792709600526, 0.4686065114893446, 0.548839530738922, 1.032868362161314, 0.19937128483200525, 0.2903978064833591, 0.4260060505105622, 0.03379784977621363, 1.0058357115635272, 0.2638310464654862, 0.36320091909244956, 0.5571650273404791, 0.40143510327046483, 1.135958875159914, 0.3724868626213411, 0.9954811206339373, 0.6249082673214728, 0.39207516387468144, 0.7534658817961013, 0.3953944565541]], [[0.9945204129138283, 1.3459124340290376, 0.24928083512623878, 0.7281330856562673, 1.6753700027750411, 0.7423863985721468, 0.3988636118128974, 1.0418397093946274, 0.9390976891605828, 0.35226499768227804, 0.1864044673834296, 0.7267693962548227, 0.4562817021216339, 0.6318988690839853, 0.5291761832024102, 0.43289204814429555, 0.45218359644032047, 0.5145146371079468, 0.8692734082986221, 0.16259957580220594, 0.2988260494779519, 0.3283561332143226, 0.0617787727522973, 0.9915986191210007, 0.21204081905808644, 0.34395599928785225, 0.5612878556503865, 0.3456181184998351, 1.0906156339654782, 0.3980738910590345, 0.9836764073496927, 0.6282622700872642, 0.3915124237049553, 0.7373804883994091, 0.44423200799863133]], [[0.959943401290679, 1.3638731616948905, 0.2512204622236477, 0.6994351608288334, 1.3484536947787336, 0.715866399598728, 0.4202879815146466, 1.091463902698196, 0.9745873502803053, 0.35911471834190783, 0.17965228853391707, 0.7088038786100019, 0.4203614065592505, 0.6588109866181324, 0.4938476340071228, 0.4453892173237185, 0.44735327900471367, 0.48652148822765057, 0.8910039339607874, 0.13538961961985763, 0.2905469722126238, 0.32321558203268624, 0.04994178682452887, 0.9908135699474199, 0.17864842932953595, 0.3451074245901271, 0.5465715417434426, 0.3659738061141523, 0.9928818621661406, 0.5053510022557752, 0.9880133538789785, 0.6694112293001542, 0.3875948862856804, 0.7066754398377061, 0.45696552214893604]], [[0.9993435556746454, 1.3259404722397201, 0.25127672673592005, 0.7298440941407379, 1.3271075094264106, 0.7017052739040581, 0.39161261120736746, 1.0605684845966716, 1.001199331876306, 0.38357148744630865, 0.12901817256248305, 0.6848456474537613, 0.43482247530301166, 0.7077530009551694, 0.48397402873454987, 0.42984117692550416, 0.45771343332272274, 0.5107263107249227, 0.9076844182949042, 0.12290928521916833, 0.2729203685390707, 0.3225922979043242, 0.013379483379531504, 1.0522541056921166, 0.17521533688694124, 0.4075615044425292, 0.5873639431079041, 0.3412031748280703, 0.9191075699703173, 0.6528950178775299, 0.9805649791039257, 0.7164943246048792, 0.42597792042766525, 0.6860509899517587, 0.5096561615845858]], [[0.9964655992460458, 1.3199352312445172, 0.23852692938919784, 0.720754193880171, 1.3331172340759982, 0.7086554234382554, 0.3935416127898482, 1.0487780996756941, 1.1103788740845628, 0.40750773974650767, 0.07974316535335366, 0.6765589223244437, 0.38215734701732573, 0.6847129896066295, 0.3954085201143649, 0.4516002960294441, 0.45016755499310673, 0.5314345637465807, 0.7816029458383591, 0.13154304679374643, 0.2376593917575616, 0.29221440081614813, 0.01641992206461745, 1.0345537795219848, 0.19995611416034068, 0.5466394676980324, 0.5156327099360786, 0.337325486041667, 0.8503558686915709, 0.6640056220025512, 0.9418826856816102, 0.6958575015438184, 0.4136523242243838, 0.7467190659258796, 0.6182843539626715]], [[0.9864615137922581, 1.3351044864377988, 0.2421848888098695, 0.7287531779238595, 1.3569502449009163, 0.7223960287557779, 0.397384896217331, 1.036341061466541, 1.0517495066662506, 0.38873685465412744, 0.07714935408981632, 0.7195273099460606, 0.4550422177962635, 0.7108834253624063, 0.336557964578524, 0.45749316597990763, 0.40319268133756125, 0.5748064216573022, 0.793277966192872, 0.13573041138816863, 0.19560791993520488, 0.27045784211736806, 0.014811604550007938, 1.0217189703873086, 0.21890987533461054, 0.4280895715674141, 0.5061490369400711, 0.35408821316185285, 0.7687168032982235, 0.6283121377209345, 0.8753541877102633, 0.7586188560796185, 0.3678780056331009, 0.7642604642787597, 0.5454455355825958]], [[0.9909948078730205, 1.300807080492869, 0.252714079410188, 0.763830433398889, 1.3648554301757365, 0.6980126442790968, 0.39343096854720794, 1.0309756081399843, 1.0272252734187688, 0.43935750783465993, 0.12120036482925114, 0.748345811470164, 0.5372343391581917, 0.6935441469749126, 0.33696854776861973, 0.4178002421026288, 0.46879973754523196, 0.6325653471296062, 0.8320605444292268, 0.15198316301879206, 0.19269336868190767, 0.2874900241960326, 0.015522484207049594, 1.0031624993411263, 0.2573801485402167, 0.3454039915385978, 0.5046276237893247, 0.37528737065867684, 0.7881528288357704, 0.6150900009646043, 0.7934285446612523, 0.7903754792985009, 0.3515789561774062, 0.7813344830055005, 0.4301775527816941]], [[0.9852082523807253, 1.2866720817577924, 0.25304321645642225, 0.7847478670334945, 1.3547471325905076, 0.712056705750251, 0.40234011960117316, 0.9843347846331956, 0.9748957320513734, 0.4466969505265054, 0.11736220769541508, 0.7740581250879308, 0.6362298787746624, 0.6787316195384787, 0.354458223281584, 0.41072554852543885, 0.4780986025099172, 0.6600486056522975, 0.8112651946263395, 0.14000233100234724, 0.18769298616148872, 0.33297864859010945, 0.0204449393300176, 1.0052776451159182, 0.2905184456557286, 0.32796797445148085, 0.5755716256834164, 0.39283662946043285, 0.7613602156532684, 0.5806506034702956, 0.7381241634706587, 0.767549484594807, 0.430476528372129, 0.7822783844933248, 0.4553889516386851]], [[0.975162520756802, 1.3550182056029398, 0.24892448061863243, 0.8053774892294407, 1.3269845072736217, 0.8058895313342704, 0.3797584468349944, 0.8315653531709077, 0.8388658512028758, 0.40722567013479877, 0.13845613241813465, 0.8588218555170298, 0.6946775155812683, 0.6846962792603983, 0.3404805292148986, 0.4336499909573829, 0.5254274938505117, 0.6992701361538888, 0.8653444655018805, 0.13099651894518882, 0.18108801927990315, 0.37639850392783175, 0.027333094938553903, 1.0526106712092713, 0.30291012843014414, 0.2992098291666154, 0.863020101715231, 0.4352314505182243, 0.5809472690329618, 0.49054241217206485, 0.6492920604804548, 0.7669650272716706, 0.4722917961649117, 0.7895331448767713, 0.45295514821685845]], [[0.9861241349553309, 1.3900570623671828, 0.24604997651228327, 0.7627839394078794, 1.3345074287417809, 0.8010224885396463, 0.37685693122681374, 0.7855800087025986, 0.828382341165999, 0.3249752816371454, 0.15316541121273133, 0.8535227246620964, 0.7358071974594573, 0.6478076690949899, 0.3332433776541694, 0.43038973888855603, 0.5232021999510033, 0.7252779472878809, 0.8845425143061674, 0.13436470616449683, 0.1724705485117023, 0.44117170375817005, 0.037534623587969926, 1.1262304009586988, 0.2933070945269093, 0.27630688907928375, 0.9402371324938457, 0.4533547198927543, 0.4792273715428494, 0.4601413660371054, 0.6582647048295285, 0.7532238492154731, 0.4969940054313015, 0.8125984671296134, 0.471351704125053]], [[1.0567436295871657, 1.3793697034485453, 0.2461878776831245, 0.7734657038568286, 1.3168922012261985, 0.783499682089654, 0.3837613566164231, 0.7424598428459313, 0.9153552153074173, 0.28699082317830044, 0.14046164803667716, 0.8491771096039934, 0.7110769226101047, 0.6531638306593258, 0.309149062896159, 0.41528582693673394, 0.5046318455274856, 0.7278132064653041, 0.8908174546486991, 0.13160984444786347, 0.22631023149795534, 0.5142498313151715, 0.04985214392198997, 1.2081178850989933, 0.2772303985542053, 0.2623204789745388, 0.9373812526766341, 0.4888636105094063, 0.4660319188926828, 0.35709439653825314, 0.5708839337472125, 0.7549739458808524, 0.5118639614790677, 0.803445637085642, 0.5809739920333628]], [[1.2399408961863632, 1.3646948134904684, 0.24452837616331577, 0.8728335989506555, 1.2469768142031983, 0.7777258319376879, 0.3940002010150179, 0.7383809777372815, 0.9707261788112715, 0.27446302313259047, 0.1271075122555916, 0.8454107135206166, 0.6719715984641255, 0.646246078468085, 0.3223196560991683, 0.41676427708060443, 0.5368055508462686, 0.7591244352355915, 0.9260797596966666, 0.1465558200406128, 0.2129106770307618, 0.5314350450673049, 0.039779947084945436, 1.1993641206302081, 0.2953810109652934, 0.24916807506475408, 0.9135328861292149, 0.45089518826434016, 0.42193298398984724, 0.24137744282930287, 0.45282300598603237, 0.7486949637870461, 0.5237294886929393, 0.8149456932633405, 0.6182287927342978]], [[1.2107435594765588, 1.3530256245679517, 0.24315425442335725, 0.8963842078930698, 1.2702106178781292, 0.7796384087033195, 0.4030394501374338, 0.7051594157627238, 1.0008502990734809, 0.2433603244295427, 0.11072352358538123, 0.8383187711612368, 0.7151381419741094, 0.5767672532469805, 0.3400240166384876, 0.40909269627720146, 0.6480053594139813, 0.8183868610148448, 1.0049504283109787, 0.15321277046285334, 0.21289303749921343, 0.5362166669362457, 0.03850493391851953, 1.2793853768996168, 0.3422757075255951, 0.23200463323501053, 0.9505991141979504, 0.41354365584892006, 0.3591868429637192, 0.21720127605046918, 0.4498380988987648, 0.7483147261274172, 0.6220658850438971, 0.6679114202041725, 0.6824028740500445]], [[1.1551794135499684, 1.2745903194695307, 0.24860269650246286, 0.8871942092240026, 1.3221242095888055, 0.8374027231249263, 0.40450133369705704, 0.6220656482496423, 1.0327661922550324, 0.2263510195199729, 0.11510193622371476, 0.8420139468598482, 0.790098255798821, 0.5924782088556179, 0.34505484459088254, 0.45084846506645315, 0.6229120297825856, 0.8165749144824537, 0.959087760861415, 0.16192040034665559, 0.20729130064771092, 0.5238277320716584, 0.07067970832306744, 1.2505678378445824, 0.38956420190547153, 0.2417160652837067, 0.9598765121603028, 0.4195975819131014, 0.36715735702206753, 0.1344972515977887, 0.6478694456805314, 0.786424799830856, 0.7613043499843839, 0.46530172298133504, 0.7553180958357693]], [[1.1735774037488778, 1.2620169529212932, 0.2500202154329266, 0.8242931956730268, 1.3210258917009257, 0.8446166409447986, 0.4298541081257511, 0.47388058552718165, 1.0344135099542922, 0.23851858556945288, 0.1221281622875468, 0.8557691620609871, 0.8553773108947793, 0.5872237303918252, 0.30520659864845356, 0.4371484850724406, 0.6111117279740776, 0.7918988715934975, 0.8656542384222373, 0.1673093795661233, 0.20437449525781623, 0.47975529287230934, 0.06861262374357441, 1.2866580168887618, 0.38658480387610095, 0.26398399441087217, 0.921915284940767, 0.3663957439808223, 0.3141815724793117, 0.15334483270113702, 0.7422675048088179, 0.8780055818457595, 0.8890261539881696, 0.2950754981655123, 0.8639952697049558]], [[1.1868968470399384, 1.2126140674617643, 0.2386234205097809, 0.7922690347806208, 1.3442300751050462, 0.9490900699520308, 0.45865059291049226, 0.38034597084479843, 1.0976164848500658, 0.33496991820091504, 0.12867263378044494, 0.9415299982865639, 0.8947604342067776, 0.631676010280507, 0.30126712000894185, 0.4802878596327884, 0.6115831283318836, 0.7985865309862676, 0.7774795885894302, 0.17438354997776417, 0.1792156356261353, 0.5234126149834479, 0.09227803271277027, 1.24680655854071, 0.36412402072086214, 0.24581222244494835, 0.9313613536879282, 0.3295811847989194, 0.24691579667479438, 0.2534485372833389, 0.7698530682244653, 0.8766000989840381, 0.8898625228272492, 0.2759806428065986, 0.9581664564679322]], [[1.1896213571784202, 1.2117954707318628, 0.24128378088951397, 0.833075551031961, 1.2790089908923694, 0.8777439391977475, 0.4073521234861307, 0.40423052783366564, 1.0899785744920156, 0.37101800605797153, 0.12285399172357553, 0.94238279700502, 0.862015666442657, 0.6339481255914414, 0.2897938120099376, 0.4564525514754656, 0.6148016880379576, 0.7618033514046545, 0.7402391234495975, 0.1840492647617617, 0.1696233632437761, 0.5122664345104417, 0.08131911151757601, 1.1455833258744303, 0.36164753284243445, 0.23719729250586816, 0.9360716417805008, 0.3208486875480838, 0.2558971881642631, 0.33547472667631306, 0.7993968296021334, 0.7709849960762275, 0.9056353588492407, 0.2814478096841561, 0.9480351051617538]], [[1.235422978488789, 1.1740067118882882, 0.25710198060877376, 0.8553147447257203, 1.1421097200871757, 0.831928916244632, 0.3918397498120616, 0.4411173500172613, 1.0155354610122533, 0.3504671641791267, 0.1197823200273915, 0.9378805142756748, 0.8234950380750877, 0.6242697758792581, 0.2904950528718808, 0.44695129508678555, 0.6257394934354394, 0.751933564421821, 0.6990552140448826, 0.1938835956584585, 0.16670638333722998, 0.5058494919743819, 0.06789604305538206, 0.9965441952678246, 0.35764018015903, 0.28120527036618637, 0.8829594965163929, 0.31507549056449047, 0.29169522868401054, 0.39944068461487037, 0.9240566967782742, 0.7056499150661276, 0.9186775431564169, 0.31038705324694926, 0.9246505387655608]], [[1.2962423005319954, 1.2286462523898543, 0.23912848574076273, 0.8446463448634582, 1.144062241388502, 0.9146146166961003, 0.4123509663660351, 0.6317040017696778, 0.9664396321134456, 0.3601972553540168, 0.1178059150092985, 0.8853031346046879, 0.7931639321794131, 0.682037623815765, 0.28641682092014054, 0.5084323253840182, 0.6319225334029759, 0.8091356064047435, 0.6453650916356579, 0.22574457378556434, 0.17460546916256525, 0.5131119315931163, 0.06834931310516451, 0.9698105484428157, 0.3776738470710025, 0.35140831085943625, 0.8677648652310912, 0.30555009323121085, 0.2988381103143974, 0.4635535870713841, 1.091919987767033, 0.6302176482849284, 0.9435528831865956, 0.300291966712162, 0.9119105103124622]], [[1.3019938728351221, 1.3199498338523314, 0.268010886882628, 0.8240455175782995, 1.1473206521967518, 0.8509281506761167, 0.42166858274591673, 0.5109686203003034, 0.9378298425144288, 0.3060763907639857, 0.12040233293715974, 0.9291607773431536, 0.7889201385058247, 0.6563730314499446, 0.27958502738731855, 0.4882222184180269, 0.5962914482868704, 0.7839924644758527, 0.618601195538065, 0.22626450335595388, 0.1754211238511325, 0.47082285432182913, 0.0962772652545981, 0.9785231735359216, 0.3992059862203793, 0.3778346886216496, 0.8336305559880266, 0.30903074837017686, 0.2796182700645344, 0.5003201132146518, 1.161269478174776, 0.5401641133060254, 0.921610722533359, 0.3539834408867945, 0.9318834956633597]], [[1.2974220874067957, 1.32090589878569, 0.27680564317058376, 0.8124342353329089, 1.1782212954607294, 0.8463696635834639, 0.4470225002578036, 0.5171692963201001, 0.8925492292781261, 0.28891001186864745, 0.1336010719328155, 0.846344146399437, 0.8314463667870069, 0.6299735265271387, 0.2939756981247712, 0.45640593238363475, 0.6317880219591503, 0.778590688821212, 0.6740009995467362, 0.2512740815583662, 0.17405142681119004, 0.42041572348177425, 0.058425461129637965, 0.7982920854787643, 0.40936741697529844, 0.4070617318283612, 0.8322885837716812, 0.31155709924984953, 0.2736440388227153, 0.5720685217123417, 1.2028695474432662, 0.5475175076001911, 0.9103759619672369, 0.32216475364233826, 0.921891549315576]], [[1.3923301350943464, 1.30089711077497, 0.2737522580995204, 0.7871968702497811, 1.2063062018320356, 0.8658661868960218, 0.4468803451528326, 0.5041998968057803, 0.8629838480356105, 0.23610577305322455, 0.10470835903597016, 0.6302106163918539, 0.9876261226658818, 0.7208927187731512, 0.3360570273665017, 0.48240117582276004, 0.834177703578203, 0.8146356048623925, 0.6532931809354487, 0.2925193091306172, 0.18479295612229546, 0.38776474434867825, 0.06480904784118076, 0.6934857449069511, 0.41307615258057007, 0.42199046812558344, 0.7451631365323139, 0.30762615669567916, 0.28760284358370997, 0.6330037580912905, 1.2673475845080278, 0.5848952711771981, 0.9200701717598765, 0.26905716793918333, 0.8959527339654973]], [[1.4043535035644366, 1.3508206253761668, 0.29393120607460166, 0.7848989058451907, 1.188861492103455, 0.861856779368469, 0.4291993828477594, 0.5008515469064553, 0.6390510300613028, 0.2046456312986894, 0.07169636869011622, 0.6149213925186267, 0.9401478305519628, 0.5637860925358448, 0.3472659117807877, 0.4587956138232501, 0.8151713558785325, 0.8302328410127413, 0.6684195476485315, 0.34054367703502747, 0.23272925385254642, 0.40943411826940257, 0.06777188872782428, 0.648495035401184, 0.4146048053521406, 0.43308349761056275, 0.6893173261568937, 0.2914590460253869, 0.28536031495052355, 0.6570140190617769, 1.2420163183030484, 0.6321330874939043, 0.898249792774267, 0.2606804090207134, 0.8816881667073266]], [[1.387164981858626, 1.2939348140021392, 0.2821333107660149, 0.8056947144906876, 1.1672289278849257, 0.854991788838754, 0.42639363363075017, 0.4939583227474115, 0.46163277677670117, 0.17581213473285034, 0.082555015264179, 0.6025782903554525, 0.9562119377655676, 0.5871441906522072, 0.29922557470003214, 0.4749131406435656, 0.7777323684552884, 0.858868294831186, 0.6867064442340528, 0.34457863771658825, 0.2574714824054526, 0.4159445025025137, 0.07044807450365276, 0.5501343526183324, 0.38697969149661166, 0.4846059769343554, 0.6325728067328886, 0.2804396128550531, 0.32825362112313333, 0.6978948736879415, 1.2678160085189463, 0.6500142393255397, 0.944402978821745, 0.30835707643346955, 0.8660789954974417]], [[1.4778644643926993, 1.2736057210257872, 0.2808716970476699, 0.7829957981877369, 0.9935751633415709, 0.8531557378062218, 0.41352206472086267, 0.5161048346086133, 0.36351238359784066, 0.17129967610805819, 0.11528480221644394, 0.5506498270516432, 0.9089051019532182, 0.6048915514204201, 0.2613668130131134, 0.5171133504172497, 0.6426930906847746, 0.7291219803249047, 0.7047136567103467, 0.3738019358467131, 0.1928266911684793, 0.4336258517651806, 0.07204735759735131, 0.39364133345428065, 0.31430135973952644, 0.4446936868541771, 0.5589768211468821, 0.29365096752304126, 0.3196817898552688, 0.7287083779097365, 1.1741242069782905, 0.7092786035535762, 1.0117507596710031, 0.33482715675127106, 0.9874254392029485]], [[1.5618731762834217, 1.2786451658482547, 0.2938868541744545, 0.759469325001424, 0.954675409406057, 0.8615626897380939, 0.4349599801952504, 0.530570943674539, 0.3254077305634649, 0.1708613456894174, 0.1177110426676681, 0.5423069795159561, 0.8488669313692229, 0.6382433286542297, 0.26787474479822726, 0.5080917334743202, 0.6453762528968222, 0.7240383078616817, 0.7111197550968257, 0.4036431599804903, 0.22052846714719368, 0.44020665169235984, 0.06255584279106682, 0.2895604262610571, 0.22349874437857048, 0.4169880696946113, 0.5359927192108193, 0.2931883437326045, 0.3259856159881028, 0.7371558599006601, 1.124258449404992, 0.7021725937131469, 1.0641544425231608, 0.33533759566323257, 1.1772891383561064]], [[1.5395511561087283, 1.2102142550897521, 0.2919811216446567, 0.6974501336064187, 0.9070144956743771, 0.8760383071486999, 0.4297007858175013, 0.5567865325794126, 0.23579416856233826, 0.18142987827353635, 0.13350168345311814, 0.5425973234871858, 0.7877490199278485, 0.6340904011280368, 0.2604711238442513, 0.5155581195299154, 0.6268595222496421, 0.7089966480341221, 0.6428450916057216, 0.47369257012684196, 0.23859266611973956, 0.4858885022843865, 0.05382042381647949, 0.26900570833326654, 0.17851340749664718, 0.4288289776324351, 0.5058542527251498, 0.2923979698177122, 0.3407785795838805, 0.7438012624843462, 1.072629333382157, 0.6574419414593502, 1.029305480822273, 0.40072505233706, 1.096311967696645]], [[1.485434487505898, 1.2057798482674205, 0.29128868611684045, 0.7225064082772743, 0.8811738428693495, 0.8423668453950656, 0.44813807265033107, 0.540536462697056, 0.2413467960881502, 0.18911512803898356, 0.14411294041329628, 0.5755913516691407, 0.7380057628804687, 0.5940975209791077, 0.2420942803389073, 0.5093999851330862, 0.6563108954911392, 0.7406211080870644, 0.6766066022302923, 0.4534988814874666, 0.2238561865421254, 0.3596190420950782, 0.042879032117944305, 0.3145664112864428, 0.1258099421384693, 0.38925517545616106, 0.5024343121518057, 0.2969162440059837, 0.3587977968096501, 0.7398758816640343, 0.9553145747120757, 0.668716465391487, 1.020106069266403, 0.4150443009098792, 1.025174221233275]], [[1.4871200095684292, 1.1893478092094496, 0.2631682637484598, 0.74129614485914, 0.8389988135999331, 0.8423920679958485, 0.5140382255533763, 0.43390624052981847, 0.2772937957969963, 0.22635565242839026, 0.15023437073399235, 0.6116938234414497, 0.6633409599835294, 0.6015732745136085, 0.25912981345446706, 0.46620266983332403, 0.6440951565355435, 0.6736139761361808, 0.68223315196137, 0.421083947494981, 0.21260386139936144, 0.3650518807445556, 0.09649501921727027, 0.3375060158515242, 0.06458501564253322, 0.2939438484820853, 0.4901492576478702, 0.32768730292415826, 0.4019195683991387, 0.763224234371385, 0.8427113462443706, 0.6704410167204019, 0.9962757135471311, 0.41790794537436726, 1.002575343498259]], [[1.5077893118017172, 1.1655085768173898, 0.24655410900862318, 0.7046982144518311, 0.7864889304061491, 0.8273762953812378, 0.47006207572530523, 0.36393802364016375, 0.31080900927390653, 0.3108145287581594, 0.14028364521632022, 0.5663628272702813, 0.6749632682334687, 0.6113883903409274, 0.2533427381963818, 0.44005778590812744, 0.6960455068709555, 0.6291596808559371, 0.6509452583832371, 0.529692600458957, 0.31640577954820415, 0.33118849854868976, 0.04463775163290053, 0.3172605353642157, 0.15799268272336708, 0.3325016867377838, 0.4130086702180521, 0.3432169323017148, 0.6247678602749428, 0.7922331426275078, 0.7713385129048589, 0.6638742620093584, 0.9624414173461388, 0.42594211808725374, 1.0430355835759262]], [[1.5329340039571393, 1.0777846470096106, 0.23749232204748533, 0.6916021967116424, 0.7805430883803937, 0.8228125975697573, 0.4454474563598344, 0.40603933247044244, 0.3344629300703796, 0.4407064538102665, 0.0933906142611964, 0.5680187933610173, 0.6788736743241593, 0.6390227516398483, 0.33025309850373746, 0.4401637896120999, 0.69478539846025, 0.6605550250707435, 0.6998250784744371, 0.5849601634350059, 0.33472700521385956, 0.3517136805752224, 0.05130449766870457, 0.2735647552925522, 0.2971134176843627, 0.3766396268629922, 0.4024280390978055, 0.3401802108699889, 0.7511159676825898, 0.8089527872544354, 0.7375877862482632, 0.6492369969987402, 0.9255507583275697, 0.4469584205922889, 1.0532302201563404]], [[1.6335138996857215, 1.0273698786956342, 0.22754510774387873, 0.6662871297013807, 0.9469414361095917, 0.8397973386522832, 0.42199028636571734, 0.4420380597533945, 0.36232105483046584, 0.5414933573111492, 0.068468968199351, 0.5892211560998903, 0.6022333474272736, 0.638880547312239, 0.3286119975526353, 0.4197023163629431, 0.7154334720290924, 0.6225805091309389, 0.6691794293436568, 0.4409135557479645, 0.3533158097398441, 0.42132918670549246, 0.06245638517656278, 0.23511740910548984, 0.3458243734197054, 0.4005916293781789, 0.3354647922505744, 0.3379558572971274, 0.8382169584057975, 0.8680648946803213, 0.6850711641286356, 0.6234694082938333, 0.8888279209841952, 0.47564318554382995, 1.0077082419932823]], [[1.735191901402938, 1.0681937895411642, 0.24104779195232093, 0.6345218654782382, 0.9464100518425931, 0.8339486660163304, 0.46888584022703994, 0.43987918955099736, 0.39260998185697893, 0.6112893323745214, 0.07909508265518825, 0.6318434866996177, 0.39225176437967174, 0.57885734110191, 0.36268914814042436, 0.3990404738931954, 0.7486917110251081, 0.6803242169701367, 0.6674616136162683, 0.21788371152944547, 0.2977287174127521, 0.5038934545022029, 0.07009586608022564, 0.19662613941569762, 0.42460824312126366, 0.3827971072817843, 0.32132416783865597, 0.3538670046001002, 0.9057283836901463, 0.8845742370509941, 0.6880735216470001, 0.7266253414993653, 0.8359875774724774, 0.46177866822211866, 0.9497138423420357]], [[1.7301050477263797, 0.9921477326836525, 0.23495402134539528, 0.6743660222523876, 0.9462568936970466, 0.8568962337279147, 0.405081939742187, 0.46203377927888345, 0.3955416564390963, 0.6373399840896214, 0.1369086933060067, 0.56187412828826, 0.3185897251303788, 0.5213301610966588, 0.42178716598595783, 0.36570551715613686, 0.707478682494281, 0.7635293630001274, 0.7122064731063353, 0.13487282795815922, 0.27658715914366866, 0.49732319218216886, 0.03804050810062807, 0.18192101525390364, 0.4936671070448524, 0.3888197935898092, 0.32260260177549216, 0.3548360099098713, 1.0000573196335973, 0.89062249399115, 0.6880561037237324, 0.6504444670279108, 0.7402408704440129, 0.4521233055242305, 0.8552214358494672]], [[1.722037629567914, 0.9912227255124505, 0.23017149162906975, 0.681841824476915, 0.855967551963743, 0.9141030901449065, 0.4265226150006656, 0.40524524512571447, 0.49076076269746566, 0.5583171893639187, 0.18885723906238944, 0.5371674454422366, 0.26828272042263507, 0.5249136566014303, 0.4327627652454607, 0.381450741592128, 0.7033060347047704, 0.7672802638288868, 0.7233761766425025, 0.16343928271260122, 0.3489138946577372, 0.4434625271513134, 0.04992921162831467, 0.18614992576751632, 0.5319484539689645, 0.4072600845871353, 0.29761064773269125, 0.3915318716059313, 1.0087703025848054, 0.8741683113734251, 0.6199780604761531, 0.5506628407855012, 0.6211735488666392, 0.4367092748254175, 0.7154924389485292]], [[1.7495575896205633, 0.9897846546580669, 0.24001263399556538, 0.690266246161787, 0.8921800154692385, 0.90481563354258, 0.41857104492674047, 0.40443105098392496, 0.6107945360718032, 0.4971785469571449, 0.20450390851665828, 0.5511287970671627, 0.2753597350454562, 0.48422970410790234, 0.48989078749989085, 0.3858332346694143, 0.7005200746291758, 0.8287745442270866, 0.7504477773128617, 0.18705793009794414, 0.35914849659574216, 0.44912077358491875, 0.05129464672270101, 0.20571569323470568, 0.5878836956223804, 0.3927170245235979, 0.2834500096023689, 0.3978936775625402, 0.9133399656847978, 0.8734996497502434, 0.5792610527545976, 0.5435351948087203, 0.5663074113647371, 0.39035493318274167, 0.6457123304935853]], [[1.7909980800311942, 1.0148775339473637, 0.24608094924689694, 0.7228391135062331, 0.8713986518342043, 0.8842514083769701, 0.39687419955940073, 0.42675497815879204, 0.6793961811282028, 0.49149412359771283, 0.22087446998866594, 0.585604366985321, 0.2603430449679456, 0.49287252908309215, 0.546178643356815, 0.3786877536039476, 0.7039428395840965, 0.8344276459073829, 0.7808883705024791, 0.19221197706808035, 0.32141000669580855, 0.4288303804040234, 0.06087645248974672, 0.2187140065006604, 0.5726739147605591, 0.3385941257533088, 0.2619658202364578, 0.4250925412585835, 0.8634649705789782, 0.8990768482729463, 0.60553642418843, 0.5017940829457783, 0.5387503409414801, 0.3593874786404262, 0.6130978017859716]], [[1.7669183292859767, 0.9141269625620458, 0.20480025915436884, 0.6533025568441421, 0.8223985888380503, 0.9257567369501865, 0.383686582937165, 0.40905899891694153, 0.655041671693568, 0.4516860746086435, 0.22413234289785866, 0.6893559346475843, 0.3550265425131073, 0.5042680268179645, 0.5826753277621255, 0.37396922935436977, 0.6531226985480969, 0.8251841957087909, 0.8521352531756599, 0.1963153784027897, 0.3229296611664209, 0.43797622825604066, 0.05972800326610243, 0.21563775224720255, 0.5386696217105793, 0.2026563450358939, 0.2584046224133978, 0.41583471455155097, 0.7011666276775548, 0.8878031123910926, 0.6039178575064409, 0.5455146496267753, 0.4770207437627566, 0.3232022489940848, 0.5581387772174666]], [[1.8710090490150924, 0.8513663059582134, 0.1617318172070199, 0.6830813134591067, 0.7927429047038248, 0.8947112157672903, 0.40680005078456705, 0.4852063654997616, 0.6575224012708318, 0.5032228372349768, 0.18264385964235863, 0.6845718414692075, 0.4158409307490042, 0.4871832741733989, 0.5896130474325811, 0.32740701625015756, 0.6768603479067927, 0.8586667040275043, 0.8895276610276633, 0.18375974349864593, 0.27904066563672936, 0.3800528293805494, 0.09110016678364301, 0.22752094908911505, 0.5469580095868737, 0.13376155551213176, 0.25861353055319947, 0.45417685895258825, 0.6940892500441436, 0.8456883114709084, 0.5316753074926284, 0.42384737919252924, 0.5016297312946862, 0.2871130007801786, 0.4955551705630563]], [[1.8063978561850953, 0.8970665908743967, 0.21534069415939572, 0.623567647788465, 0.843569890207539, 0.8313953450202473, 0.40827695904404154, 0.4347540918065731, 0.6214173106650949, 0.5734817546932018, 0.20150874338726887, 0.6755681481338123, 0.38599784714317964, 0.4647178975844509, 0.6197164970167721, 0.32601559141248543, 0.6405901485712213, 0.9675456267447569, 0.9127002468479319, 0.17384457343673757, 0.2630800749103525, 0.3809601151157229, 0.09201307186146195, 0.23229998097167412, 0.5720262309500537, 0.15396633331036652, 0.2852357633970211, 0.4776820496554689, 0.7043686350374303, 0.8081320236911952, 0.38200518649467996, 0.27927187709933937, 0.522323699590247, 0.2794195862972301, 0.5604425610289845]], [[1.8055703633903126, 0.9279392811804585, 0.2286911824329464, 0.6154043934212676, 0.8745693641198958, 0.7616036191127291, 0.3744549365844823, 0.4038273021842479, 0.573536448156039, 0.6179882814846899, 0.19127574200977293, 0.7742701257184951, 0.2969458111875764, 0.5089636121122946, 0.7063915994234093, 0.3319880248253662, 0.6364590634796075, 0.8531019321537656, 0.8068348339481578, 0.16497407341199916, 0.3080895154478758, 0.34148998678478154, 0.06721774552941864, 0.21303412176817496, 0.6140714234276754, 0.15087233531378508, 0.4283790367657278, 0.6578937400702476, 0.6470790248122338, 0.7295787499587851, 0.25822147720288635, 0.15028947328289022, 0.5441096246183901, 0.27039535310090596, 0.5609471077421451]], [[1.8394353001107098, 0.9335258458008158, 0.19152238320473736, 0.6217481806498762, 0.9146324097081997, 0.7905451849906056, 0.3965985852397422, 0.3167113456623122, 0.5244987908895704, 0.6312247842200287, 0.17749267337844365, 0.7496725176478635, 0.1964958549909292, 0.4922570518272063, 0.7037980951326135, 0.33779936984444314, 0.6226595708909627, 0.7960752830049929, 0.7753918305470809, 0.15910383155425487, 0.31836245417197706, 0.3371927511153634, 0.1314416999211747, 0.19137664390911474, 0.618594460973395, 0.22565156392407001, 0.4701345892259107, 0.78164075287074, 0.7125766337235048, 0.7246178361637516, 0.1749165211463603, 0.10316341488377792, 0.5255557425661126, 0.31620319686372006, 0.4940501905362128]], [[1.8597131968704117, 0.915442591447279, 0.15272454360450538, 0.6263149154050967, 0.9925892542323911, 0.7540279465837895, 0.38162853333948893, 0.28538394818161616, 0.5520605349283245, 0.6087381955067306, 0.15065359384283586, 0.7345034867915115, 0.19095434061392239, 0.4896428629452153, 0.7093330814354586, 0.3426101966275266, 0.6423119498768803, 0.7850785066855777, 0.7849878755106358, 0.1599392940054735, 0.3405385608929442, 0.32301386977686375, 0.1216426756409747, 0.19673371490039854, 0.5872461852301712, 0.21089859669836822, 0.4782889699346182, 0.8421575349701216, 0.7416589036210255, 0.6764850515716208, 0.16580928293044214, 0.08304940346760159, 0.5767495948218294, 0.3720705812915012, 0.4856691985152935]], [[1.827565781348776, 0.939141320448466, 0.15147895607041068, 0.5813959357648644, 1.058117125655304, 0.727401686764879, 0.37451132943588683, 0.2640341494320515, 0.6253391457113249, 0.6167787605737511, 0.14520721886633167, 0.7532570897750017, 0.22742826237691727, 0.4993026768203293, 0.7374479871751076, 0.3430219031132594, 0.7087885379403666, 0.7940874236003848, 0.7342117327183421, 0.15142829723875384, 0.3314193208663544, 0.3235672873701222, 0.12934537741883406, 0.16496203677220933, 0.606847281162102, 0.17714868875067438, 0.5153048605315658, 0.8955577101588799, 0.714836714416884, 0.521494279944128, 0.16057314547680243, 0.11415450283108713, 0.6424354204507472, 0.3879825350903628, 0.6036017333122075]], [[1.754935395757414, 0.955746182285249, 0.1648971140219314, 0.579541296315183, 1.1391981346163067, 0.6889689396721885, 0.375356851172843, 0.30243012046878415, 0.676265520898709, 0.5989971436453765, 0.12608772456093514, 0.7431920742868809, 0.2435745946814401, 0.5003787418428577, 0.7065856105668431, 0.3391096552036946, 0.6989101730603519, 0.7898003061047767, 0.7398843203924731, 0.1615745535570484, 0.33240005201679546, 0.3288585499075888, 0.11108795281019253, 0.1646868001384142, 0.6529721409958913, 0.16444400431080836, 0.5175178152505207, 0.9324277230296683, 0.6851586885530587, 0.4197813674881471, 0.18274667629890412, 0.18128594137335524, 0.671555598840034, 0.39219417995022343, 0.6436046979043432]], [[1.7008668447223685, 1.0606197610674242, 0.2114669671166886, 0.5787354814993311, 1.2186931269283918, 0.6858222931203102, 0.3793877661544649, 0.32518559592749074, 0.7108192708123754, 0.6050059432141888, 0.0973901178178325, 0.6781740592544788, 0.25078630810625524, 0.4593213913058602, 0.7073614562893276, 0.34008419949506224, 0.6873476877545579, 0.7560245483700471, 0.8272071689593242, 0.15501316337476334, 0.38300281042149686, 0.3551048086383083, 0.08447453629319904, 0.16576209026514793, 0.9124819932649463, 0.15056388363164555, 0.5137672842367529, 0.998224268255808, 0.6694313301777705, 0.3453771073614584, 0.19181548224912406, 0.2560992919704399, 0.6888809212981108, 0.41598739475231616, 0.6570390987497796]], [[1.7213687019218928, 1.0835401885406932, 0.2364490832108852, 0.6969897900466094, 1.272670467684697, 0.7028195842564864, 0.3787565647781541, 0.31056689418156347, 0.8133608822191937, 0.5960445219278877, 0.08707847497703251, 0.7097507238867523, 0.31268775436941254, 0.45128228867031683, 0.8618718404305261, 0.38702310532162115, 0.7136305704510258, 0.7507510814716115, 0.813152571526006, 0.14770651085707898, 0.3693213112610136, 0.32830806850274696, 0.07937432230068522, 0.1679143177108829, 0.6932556156954821, 0.15600345238070312, 0.5234320901775438, 1.0337296698230969, 0.6765254438018764, 0.3903011261508216, 0.21755249341371644, 0.20354390401494005, 0.6318225606185789, 0.41062325718510867, 0.5442832111157428]], [[1.7140482279468598, 0.98866940967896, 0.24463106395446388, 0.9406943592509511, 1.3055498170474722, 0.7073106389346393, 0.37960067421282506, 0.3354127068555162, 0.8428033657390346, 0.5873864572772309, 0.09390383672438082, 0.7073832461031878, 0.4463963782182867, 0.43218744090593486, 0.8718101785060643, 0.43664076238723554, 0.7193646161113962, 0.7691727146146446, 0.780910130177576, 0.1520961622948477, 0.3591960421730737, 0.30631698351620745, 0.08316871673573754, 0.19748431612158102, 0.6592822272377844, 0.176632703733994, 0.55362613795354, 1.093947743701906, 0.7294660066764442, 0.4067049965889735, 0.3505726904321629, 0.26753108652799373, 0.627681510761158, 0.45228598071982473, 0.5836262337259123]], [[1.55284444171865, 0.9971499884937596, 0.21655605855632443, 1.1240296489885009, 1.3612211351943846, 0.9222040818120003, 0.44109689087950665, 0.30748938018663, 0.8286822888702292, 0.5948010473464251, 0.08244420093587985, 0.5968574081687338, 0.496827526955282, 0.4133992734066292, 0.8397162355210142, 0.325980509890559, 0.8022042989453702, 0.7598851496655946, 0.7409402590478947, 0.16778622134467572, 0.3569874781090434, 0.307784409904588, 0.09432780753843822, 0.23039848119173256, 0.6467475368248481, 0.1547893777611476, 0.5603635272444145, 1.2222785531587061, 0.7140881220694442, 0.4525579902329378, 0.4712801672917171, 0.37801788374366824, 0.5873486388635483, 0.4528025988145742, 0.6400212333091098]], [[1.4457220441660708, 1.11852193684806, 0.2012087531059188, 1.0458839218682523, 1.5049692234528336, 0.8246892937153294, 0.4444106749557833, 0.37824972767160137, 0.7868516357526733, 0.6290050347879634, 0.07305638598789585, 0.520080335665364, 0.48841584161877927, 0.3869280144937889, 1.015190576622388, 0.31685471325024694, 0.861289206367368, 0.7337773296502498, 0.7400318498713172, 0.17188702208776135, 0.26586154104957266, 0.3080646845543973, 0.09853804466533617, 0.23265940351526404, 0.5904612860595576, 0.15536667365233972, 0.5672874175881559, 1.297596575704491, 0.7303888925021901, 0.4860450210091387, 0.5269757220769649, 0.4693563999203826, 0.6105993755503751, 0.3846428360155657, 0.6600678592549425]], [[1.4739786579429865, 1.1887740426706253, 0.2429966868437145, 1.0915967010770031, 1.5860234826154131, 0.8052486425721352, 0.45523521554274804, 0.4386328288398277, 0.8227028164184524, 0.6302682275763175, 0.06593030557417655, 0.5984857617149029, 0.4886129907171056, 0.4524942043479896, 1.0972108390619686, 0.3265381901783876, 0.9552984764953828, 0.724408588388329, 0.7221821527181018, 0.1709021915190285, 0.28447625166333007, 0.30699473211363615, 0.09970069891940783, 0.2381761972074673, 0.5883234606130602, 0.12037954407611702, 0.5696859233493545, 1.3395068194605513, 0.6699381837314171, 0.52144330667864, 0.5489796383992338, 0.4474642599877731, 0.5888901767735555, 0.34542677692735807, 0.693532285471619]], [[1.3816306339669886, 1.2165773576908867, 0.26990482906245766, 1.1847982666609898, 1.6330924617065647, 0.7918889685270916, 0.4641713955250757, 0.45529854440390183, 0.810042575703056, 0.553576229520495, 0.06029854170445819, 0.6511205048488689, 0.5305624010583875, 0.456918420071469, 0.8214593195861535, 0.3566729115855325, 0.7678446612093347, 0.7544646846370615, 0.7221856543015509, 0.15373679120261632, 0.2505970943208034, 0.30344853243985476, 0.0998726869089182, 0.2453462960444377, 0.5225119592514101, 0.1141778801574536, 0.5177166991393671, 1.3083434514715933, 0.6996997091635733, 0.5975526729624225, 0.5689560087585761, 0.4039866566346338, 0.5544328226415655, 0.36684330652997954, 0.6215874727329402]], [[1.1803055334478847, 1.1774264694131074, 0.2704325747352601, 1.316733581584173, 1.6412507819793967, 0.8385244110540406, 0.502526069780359, 0.504288140494584, 0.7125689553541262, 0.4407375686856249, 0.048770335677425065, 0.6251246540397798, 0.5756814415309794, 0.45544086070868744, 0.8635143866549543, 0.34869464038778986, 0.8209919762048371, 0.7536917185745343, 0.711575960226535, 0.13758109585514613, 0.27583046436400477, 0.2932131585058946, 0.10111061870542482, 0.23557321755460692, 0.582568906350801, 0.14200786926731243, 0.4956614292382875, 1.269644433621104, 0.7073166041095805, 0.6213776166524994, 0.5993954594887008, 0.39115621396966155, 0.5017111973445402, 0.45723697497787774, 0.46527408310262075]], [[1.1256924790337133, 1.1872193398385917, 0.2695458342318961, 1.423860986667956, 1.5699769624857711, 0.9726572425153259, 0.5230072760161881, 0.5685624826725149, 0.6295838348176677, 0.3927822667687218, 0.05692388408634695, 0.6416633858591316, 0.741498575947964, 0.44129729062045464, 0.6125942501664823, 0.3442467535252879, 0.7677550565102034, 0.7529179711683519, 0.7062874674347784, 0.10478396019775701, 0.25580896388052754, 0.1728232590906518, 0.11189739501940921, 0.21342721811980814, 0.6449140698951841, 0.17789709703892154, 0.4708672904264496, 1.2368593468450402, 0.7377870495191908, 0.5979590644456838, 0.6170879850338168, 0.45233072158795035, 0.42089193491706783, 0.5287657174612999, 0.4695050071189415]], [[1.0967639937967224, 1.21806230121735, 0.27362965377705023, 1.5431678616443048, 1.549198008847967, 0.9572335662407465, 0.5077318948054518, 0.6773983200431508, 0.6607161195074853, 0.4742605368725227, 0.04495634763557691, 0.5958900462137783, 0.7023578330007226, 0.44222931905689, 0.6220313292213591, 0.3226291674319035, 0.6868577076008782, 0.7468311246359023, 0.6836359449296703, 0.09629639903294707, 0.255156957251379, 0.1338389055553554, 0.10649347830841238, 0.19218485941729246, 0.6794270236213087, 0.1968909588605154, 0.4104232887112514, 1.122811843190462, 0.7935127956125465, 0.6143985551744412, 0.628231373446929, 0.54271791007405, 0.3621785758258955, 0.6042146586477007, 0.47055051913169377]], [[1.0666950419590457, 1.2924273503032149, 0.2712703262791854, 1.787469779397193, 1.550138954577128, 0.9299981660444755, 0.49450290776542233, 0.7465252427502118, 0.7400917746520256, 0.5182775709960833, 0.042110681803378576, 0.617635360418116, 0.6306381351031004, 0.4404157576223845, 0.6160808636554194, 0.32342258921245937, 0.700766118115028, 0.733035980678553, 0.6733663810616143, 0.09718748144977254, 0.25640773129225547, 0.11412359332573072, 0.09862292590848923, 0.16474674763803165, 0.7111899167979607, 0.196694402299467, 0.33858257648557777, 0.9895274461307668, 0.8646901297827841, 0.6386036267568347, 0.6515750980058984, 0.5895464167651767, 0.32030106833507627, 0.7215430442603084, 0.47094114609267235]], [[1.2107983140803427, 1.3503905888536656, 0.32666930566020524, 1.7887864443394108, 1.5618979380361264, 0.9047873257139452, 0.484178684302739, 0.882675338035261, 0.7457737646881205, 0.4276614223800579, 0.04229857280778493, 0.6746419115763518, 0.7197573739759072, 0.5342356070861343, 0.715090753673451, 0.3268610568257718, 0.8105383679206336, 0.6883543983110788, 0.6577405649987645, 0.12459503644765035, 0.21650507425404836, 0.17156075261043274, 0.09213558783921554, 0.1393794229774482, 0.7764858326153228, 0.1747410222321824, 0.28344108294562553, 0.9206587012426835, 0.9232241109914561, 0.6473431926895614, 0.6701399002692847, 0.5539117759097394, 0.3101813723514238, 0.891676727856886, 0.4728566944729937]], [[1.2778215724857414, 1.3938545046353714, 0.3207304535201704, 1.7884506609565163, 1.5376870860018745, 0.9413378556597055, 0.49739545789479545, 0.87462105606338, 0.9050065247126402, 0.4063957758863828, 0.04390578907257894, 0.7235571223667103, 0.8062152323256468, 0.5715864988632794, 0.6687157399963994, 0.3440873665615336, 0.6561962990755035, 0.6223924509357266, 0.682436307576088, 0.10407213344076238, 0.17478727741165012, 0.1691727698010174, 0.10449673420509537, 0.13124407920480965, 0.8664808206327025, 0.15034531668955248, 0.2946853021765865, 0.9516102383566084, 0.9797521849616331, 0.6681614313885028, 0.7097020427513755, 0.5466187332396881, 0.3883449379797677, 0.8678395695299024, 0.4871220179664576]], [[1.3396900064017851, 1.3547393380314576, 0.2898957609845508, 1.7347202128125574, 1.4765410178061957, 0.9698322091793883, 0.5055221156363165, 0.97802616996245, 0.9722415214505999, 0.4624205961005747, 0.04243046998465669, 0.6712698950201312, 1.0906291722780388, 0.6157232575338634, 0.5734429176122395, 0.37231294043041047, 0.6220867065979394, 0.6883709065310988, 0.8034701021967271, 0.14076023364603235, 0.17225445957795915, 0.21909397900326194, 0.09952547348093668, 0.26323070169491464, 0.894739395518724, 0.1234021989931088, 0.49319932768787045, 0.939176958369641, 1.0052767326874033, 0.6754476231958079, 0.7261326079690407, 0.5411079840100803, 0.4250837345906987, 0.875042995296738, 0.4956803324021324]], [[1.406262987998955, 1.2211495127208911, 0.33083209179194817, 1.6133669650506663, 1.3918985533122283, 0.9534370022034666, 0.5143185045641034, 0.858677233632009, 1.054915314114341, 0.7210756779234322, 0.032751223804404886, 0.6826490789275229, 1.1776963417463202, 0.6317887402831269, 0.6321219424891613, 0.38138840298026205, 0.625666869524665, 0.7108689799213735, 0.8964058240172458, 0.13384964488615944, 0.16491059906250471, 0.2117563651254778, 0.10731548750324366, 0.528942954516688, 0.9052418923947424, 0.1009286065864855, 0.7441358361368353, 0.8203029265957742, 1.0058624109103231, 0.7132371186502833, 0.7453290839480219, 0.5967621805121299, 0.36838904882879375, 0.8895156665121866, 0.45170446199564185]], [[1.4125213757213984, 1.1875491293877896, 0.3402719775424723, 1.6100747484870912, 1.275243078840744, 0.8735510564863609, 0.5126970020136528, 0.8113351595470091, 1.0240945844369103, 0.7683522414777078, 0.04027200192260946, 0.6847519120361634, 1.1107117707667156, 0.6099835356529041, 0.45429274795111174, 0.3800492313727193, 0.6107770785951154, 0.7467933897382808, 1.0477546626914918, 0.14669980416866885, 0.18969718163846516, 0.2244918635791089, 0.11760524185373197, 0.576864969618172, 1.03976289331405, 0.17550205775985697, 0.6908396418184517, 0.6387293619465504, 1.0045281758714657, 0.6869776938972271, 0.7505245077557694, 0.6467284412628944, 0.31847067424026765, 0.8922134526801497, 0.47285166344709817]], [[1.4897206631769406, 1.1623299265970664, 0.33368279038329396, 1.5145158935377085, 1.201020843729506, 0.8435138168994649, 0.5217203476331767, 0.7875908676229215, 0.9490299870814303, 0.6664931599780454, 0.05409396218250243, 0.6764396535717518, 1.1134734495998226, 0.5948033470578422, 0.4499336965952808, 0.3855335179829326, 0.5584372377206943, 0.7833174781078358, 1.1568999649956146, 0.1825184958823064, 0.19612220541414757, 0.23059950662008438, 0.11502724999016817, 0.6671829318173705, 1.1481405006667673, 0.3086301512973999, 0.6708402683905532, 0.5673412729314892, 1.007034655557569, 0.6661912558721779, 0.7635980821602096, 0.5678688926355104, 0.26882361391838716, 0.8735797583320809, 0.46166189133033475]], [[1.5226409297579924, 1.1214278710609678, 0.33667919081124337, 1.3814227815798488, 1.1714175886345868, 0.8792747837275923, 0.570494103868474, 0.675398815124019, 0.8626830503052698, 0.4957003473203102, 0.16445944150479244, 0.6670351042220042, 1.177888446748037, 0.5828849646254592, 0.44503468869063284, 0.3761825409708537, 0.5352036554645978, 0.7726517988485294, 1.1943007297193904, 0.2274376305221412, 0.20043408087366485, 0.26879561973114885, 0.05165455638448481, 0.7170318507292088, 1.184506689139544, 0.3321423880752622, 0.7204275082383086, 0.5420680812056402, 1.0058848464777976, 0.637490589351897, 0.8044336762574367, 0.5605541049753364, 0.20079164674008743, 0.8633947240561766, 0.4417046591557672]], [[1.7077017984885743, 1.3734221446546775, 0.3019349014784678, 1.429381064725273, 1.6334079128278853, 0.8675326903435197, 0.6001437953843596, 0.6879734926958478, 1.0109763829455647, 0.4271058648841006, 0.11156060118438185, 0.6700691172494793, 1.1734119763776565, 0.5638140864210457, 0.42917348611180584, 0.3643411615558094, 0.5468045852283467, 0.782537068534786, 1.0951701768312916, 0.2590955598774099, 0.19253003147290848, 0.3093288170851758, 0.06371738901327034, 0.7937406059476613, 1.2242324125002275, 0.36526635163899135, 0.7378463696895214, 0.6436211180336744, 0.9066705855684081, 0.684596877819027, 0.8447973035444793, 0.5276193572233652, 0.22399331599299374, 0.8523740650078688, 0.4496834933501467]], [[1.6383727101928307, 1.0697963917759499, 0.3018861648020986, 1.1859068517160134, 1.6630398504330142, 0.8764141008534284, 0.5253207721434108, 0.9102670322943505, 0.8083290345862604, 0.44238307781833797, 0.1422827140489723, 0.6932550928262946, 1.1671926140893973, 0.5683243861663159, 0.44292020446672614, 0.3703257397916206, 0.52892675918635, 0.7641111385321071, 1.0562429198583883, 0.23296521665132852, 0.17485287002311514, 0.3500863106949163, 0.06187580945698766, 0.8541807751280015, 1.3409718063093221, 0.36277796036408977, 0.7593234116249374, 0.6880456148708567, 0.8843929933415691, 0.7286521326281725, 0.8512747495023358, 0.46250037696315577, 0.2215727524623747, 0.8815373407044287, 0.5728922785959254]], [[1.6315586868503842, 0.9000126272389862, 0.3420098271663864, 1.1981848144243872, 1.5120410210820707, 0.8509971486052299, 0.6140059559233235, 0.9535054715224199, 0.601388554488218, 0.47963010107076387, 0.17620275169009159, 0.6550611655063818, 1.121999535642772, 0.5704302921775171, 0.43969581555141746, 0.3756958572230007, 0.5221964720313272, 0.7683802514971385, 1.0466145247582386, 0.22249742891596444, 0.16917247497440724, 0.3148728363269719, 0.08094721703535335, 0.8303136923371026, 1.4526502812781894, 0.3122157164117027, 0.7453440442582057, 0.8654167040437062, 0.9051309268078289, 0.764088750189158, 0.880223731325586, 0.41106062545757727, 0.21146611850652555, 0.8938835437278376, 0.5757288741283342]], [[1.5928047129868155, 0.8493084755146593, 0.33587278501576023, 1.2437503780550105, 1.3968415916111017, 0.7941420974413831, 0.5328567136350432, 1.003063307271948, 0.49817940784544723, 0.5282639071686781, 0.23647231899090204, 0.6770934205890575, 1.0402061328654144, 0.5420397368881388, 0.40701295490766354, 0.37358474753054993, 0.5245199043440734, 0.7585800516951664, 1.0771125845301572, 0.21503316934991196, 0.18309452076859845, 0.27893555837168627, 0.11758099271335509, 0.8016824671976186, 1.454956381232035, 0.2756836982745883, 0.7911189707423972, 1.0206850906817642, 0.9989721932901332, 0.7961235458105969, 0.9105627227275741, 0.4815598834824262, 0.24484456115020595, 0.9018053532808201, 0.6412955094671285]], [[1.6093715509695388, 0.8561210127344382, 0.307927726840287, 1.2830996754645365, 1.1092051928306526, 0.8119722764127986, 0.5493100809197333, 1.0828107297315412, 0.47317905507416336, 0.5642317583888781, 0.26899406725599173, 0.6421722823170934, 1.047219425722804, 0.5549161430300606, 0.39759184454990115, 0.41055627814346374, 0.5259756123405392, 0.7586542081570633, 1.0212264618709128, 0.22378438883961138, 0.17707268338618914, 0.18547484863746552, 0.05759770480811416, 0.7567441944854612, 1.530226099673305, 0.25233893698684473, 0.7834062504846618, 1.160440449435006, 1.1001195823867196, 0.7805767497718705, 0.9040944806339368, 0.5527560112331011, 0.3303545997291874, 0.9131740292322308, 0.6510855344847701]], [[1.6758170209484775, 0.8581463369063747, 0.273375542673431, 1.2651211119062642, 0.9645525584738077, 0.7922464258597315, 0.49992884268474813, 1.0763587518044186, 0.5779783978838403, 0.5982094514825619, 0.34886611075507806, 0.6530387560392417, 1.0798875269318466, 0.5548957526348628, 0.383234623040104, 0.4224017887774125, 0.4883609517699733, 0.7317744592215716, 1.0185396011821368, 0.22437849327868176, 0.1902694493285224, 0.1547288507063253, 0.08187804607936348, 0.8129905314367822, 1.6267009480420196, 0.39889132512356984, 0.8171702968059084, 1.2274571439881472, 1.290545719660588, 0.7345214768858598, 0.8781518383249238, 0.5540558721649055, 0.37336782420351755, 0.9020130481988367, 0.7090672686227]], [[1.6438218806470601, 0.8681997590078743, 0.24922106596472937, 1.3473225831537445, 0.9291028396581724, 0.7295386741854349, 0.4443814139004232, 1.0461287653099258, 0.751956287189153, 0.6474288362972086, 0.17661521657910423, 0.6217232221672573, 1.1059945944176062, 0.5722761609027633, 0.40323838073997087, 0.4670101782033321, 0.4703086362682934, 0.7507135511875379, 0.9726016821681263, 0.2613246099352725, 0.1769858709398717, 0.12371692076294191, 0.08445122328398776, 0.8576603351826019, 1.5853907422665774, 0.5336338471138034, 0.8302286529915159, 1.245451835292851, 1.483542490339881, 0.6088971775691936, 0.8575736860744655, 0.5673335801174847, 0.5126424986222724, 0.9010056783254625, 0.7929776138365147]], [[1.6087703643031979, 0.8134551588778626, 0.23719290140322374, 1.4144341926774073, 0.8813313647416223, 0.7142711874930727, 0.42114284098934807, 1.0059540502197484, 0.8194364518564501, 0.6610919300268568, 0.14874782812546541, 0.6118630086115223, 1.07617549182994, 0.5657238002173852, 0.397411941691246, 0.5061468421465147, 0.5015627962451638, 0.7292551800936071, 0.9263903869345012, 0.2821252882576861, 0.26564127987155745, 0.1348621250994989, 0.060513188253974816, 0.8556966621140745, 1.4984477844800708, 0.6314451660725366, 0.8730652694214361, 1.2306594832424809, 1.5599080972888115, 0.5651837280757137, 0.8547667270342113, 0.6235977259247236, 0.6737855235044528, 0.7723271947356902, 0.8514380254949194]], [[1.5565281341062058, 0.8284165247338182, 0.25003374005034434, 1.3869273109303513, 0.8026255312966519, 0.7212838181337415, 0.4024982178966034, 0.9722134587934995, 0.8825591561879161, 0.6384562549699632, 0.1704031850873064, 0.5935460042906515, 1.0894742700794902, 0.5733175315278727, 0.36859160237440425, 0.5114829044118661, 0.5339893536791547, 0.7707622330356969, 0.9180216814898358, 0.27235250234972136, 0.33934566817240464, 0.22737005172535457, 0.04072370637538397, 0.8006926632080203, 1.3659279264535293, 0.7494539908584676, 0.8022340441277656, 1.2108430932592782, 1.569775340278799, 0.47613715071557355, 0.9544417297496646, 0.7281102221387687, 0.7120816449688725, 0.7193930725401227, 0.998595462850753]], [[1.5980427035553493, 0.8164342240890846, 0.2709144304807523, 1.2481926058086903, 0.7330378909358035, 0.7434509687519474, 0.39992670281709797, 0.9798297225463897, 0.9256667007412642, 0.6184092366654302, 0.16425869494547762, 0.5721994121905343, 1.060497866039622, 0.5743833883839644, 0.35404854798137386, 0.5260913112529089, 0.5423199097330208, 0.7746227163426918, 0.8917960948666197, 0.26758852305066577, 0.3086200074226123, 0.3216074250990625, 0.0295848678032411, 0.7636381965401953, 1.3107465381540846, 0.7745216274768478, 0.8063381196618065, 1.2025982380435385, 1.6653030914139608, 0.5597547236841205, 0.921111344357319, 0.8194675650980552, 0.7281011739335831, 0.8905549224303762, 1.1005618132724035]], [[1.5495454121735213, 0.8392201854978303, 0.27563775684207614, 1.0686229303852977, 0.7765814652977828, 0.7724792453211875, 0.41254595253327625, 0.9692591344417691, 0.9906053642464229, 0.6650023576495382, 0.1369441414524032, 0.565952735482715, 1.0321569177042844, 0.5615482554628262, 0.3302505012631853, 0.5413325702248726, 0.5035222408591786, 0.8208483992411322, 0.8966541359239533, 0.20928579025119795, 0.2536755531134143, 0.3963148568258827, 0.02904243772339907, 0.7662541169469791, 1.404728948702938, 0.8079713222641849, 0.6866460608288385, 1.044016297561024, 1.8923374566964735, 0.7373559888291972, 0.8709472827054612, 0.8525156160004556, 0.7423928093233918, 1.0110268456878184, 1.074039269356413]], [[1.4930482585632467, 0.7822770451596667, 0.28628890077274866, 0.9152962863305152, 0.820494716420452, 0.8023928380495036, 0.4011177982960633, 1.0259069469616457, 0.9913830583199996, 0.7927725778075994, 0.11985554422633579, 0.5382819894904731, 1.0056143497151033, 0.5866299364754208, 0.3141941299680305, 0.544418008104198, 0.5167153033121125, 0.8052209334394397, 0.9123630328613239, 0.18547280143945774, 0.2588707135528089, 0.5124818383666393, 0.03199775051950722, 0.7718188564510036, 1.5545618360758788, 0.8763026000811218, 0.6206651497313975, 0.8817923282173191, 2.1634624301285514, 0.8422760651652759, 0.8319823716012038, 0.8289160095462539, 0.7824497831003767, 1.0237198431886414, 1.0823734428968252]], [[1.523753509757205, 0.7943214549995543, 0.2990678117268343, 0.7856953885390772, 0.8378640938673979, 0.819442650967249, 0.40503509923004055, 1.0469403578022378, 1.0721759358438891, 0.8605047746290614, 0.1181382302672721, 0.5491929533307707, 1.007868959023192, 0.6005333780426418, 0.298227331016523, 0.5715383571555218, 0.4864361029286932, 0.8388386958529233, 0.9412992176988509, 0.1827316605845434, 0.23042115480282024, 0.6346409088196049, 0.052261534707151426, 0.8307643379857372, 1.6850578221448522, 0.9414069894971757, 0.5377581863177995, 0.8263467313919546, 2.328900825055477, 0.9158124294251586, 0.8285931500547145, 0.817703783409019, 0.8659531293660359, 1.0246290083762897, 1.1650325395207586]], [[1.546050560761715, 0.8714335289682998, 0.31871850876659885, 0.8372202372186569, 0.836808776448728, 0.7732408806226398, 0.37560025359465443, 1.117750809650433, 1.1087936462555734, 0.9087256678792915, 0.11363027401750922, 0.5700779198161782, 1.032956936853585, 0.6158236747982629, 0.3003616066473054, 0.5902679712762924, 0.4800000846042147, 0.8155829556153176, 0.9919950448371337, 0.1610254819670987, 0.14350011017978642, 0.5107352791397284, 0.059107679995198836, 0.8752122905614778, 1.716999695753215, 0.8089551378552645, 0.41731037830430917, 0.8170321375674093, 2.302332218827816, 0.9910466923861097, 0.8059913575917692, 0.790135277472963, 0.9303387666985217, 0.901899088774786, 1.1158824968788061]], [[1.5621740952903602, 0.8717763080336085, 0.3316677675196568, 0.8890076650430145, 0.9007798135772505, 0.7538838405824189, 0.3716152900895109, 1.1483453743707637, 1.116485219643899, 0.9648933375596853, 0.08634593956336953, 0.5892625191944872, 1.0945506692636138, 0.6282601212789904, 0.3324097340287917, 0.6159847544942849, 0.4980475430573057, 0.8413256379426621, 1.0848833949194745, 0.1710836851371842, 0.14183538238304685, 0.49681599164914314, 0.03803376517283736, 0.915761597676351, 1.7799713202367045, 0.7119790121156496, 0.29533545636273456, 0.8334076040439853, 2.2287875895227645, 1.0104490143781313, 0.8193805157203982, 0.7724450752965497, 0.9583700472510258, 0.7599297270859473, 1.0444916561512239]], [[1.5316951707563076, 0.8079719118656713, 0.2958929045613337, 0.9303217667724973, 0.8913052279213158, 0.7527938196859996, 0.3897834635132426, 1.1209205187959848, 1.192423838476798, 1.092900850969643, 0.08815473590966241, 0.6232237076892059, 0.958513405678816, 0.6181694932810838, 0.3362243623453448, 0.5456258702250274, 0.5227618840559183, 0.8051898775051602, 1.183810770708138, 0.1883740091568016, 0.14246492076567763, 0.410825323067589, 0.05500783056316563, 0.9484133613889605, 1.715924608031181, 0.6458304309250378, 0.31831372604278435, 0.8626434854345197, 2.107904825538231, 1.0452374656214714, 0.8687948054825098, 0.6761083272668292, 0.9497671465155555, 0.6636291798849416, 0.9594465383067367]], [[1.4227799606163005, 0.9256439005328305, 0.32757793812995434, 0.9549067691243951, 0.9183346706416218, 0.8375187923520688, 0.38980170586349067, 1.0917826859800774, 1.15957843469175, 1.2594933693927515, 0.08679706807060372, 0.6420158550208703, 0.9292263774311249, 0.6079754611677062, 0.37536961167696437, 0.4936438157144606, 0.5346984144824384, 0.8467822925748518, 1.3228455342867356, 0.2023744524962363, 0.1443156681975976, 0.3645285867136817, 0.060605895342638876, 0.9763526505169964, 1.6916000641224807, 0.5580824898738783, 0.5309519622576152, 0.8705778538199878, 2.070019866097952, 1.103868644296211, 0.8845505128286204, 0.5681972668679178, 0.9881709321458609, 0.5629292303048198, 0.9038314443738451]], [[1.3349404879951083, 0.9710675974874687, 0.3506341347431308, 0.9708232075343696, 0.9290787387011181, 0.8446237930899367, 0.388339063859148, 1.0020362637119558, 1.0988113477248542, 1.2767241789972932, 0.0664919838624883, 0.6423028970066718, 0.9398066805370522, 0.6242600986757934, 0.3644741468313071, 0.47977138807094055, 0.543731721688993, 0.8814305836093416, 1.3339444151150672, 0.2269270220557119, 0.14602172711216785, 0.3214571139481821, 0.06271413299240716, 0.901054372190522, 1.6870207839489575, 0.5352296817522126, 0.5730963100525382, 0.9058799076190844, 2.077202986392222, 1.1374671362441129, 0.9006542523254931, 0.47221604129312933, 1.005224324789187, 0.49292436402262485, 0.8106175148618678]], [[1.167926496397501, 0.9926527599049445, 0.3382977629585356, 1.0090028617082203, 0.832016239037456, 0.8332886613096393, 0.3771398233412268, 0.9600869955643314, 1.001979462182848, 1.2792168715108403, 0.06383153525108079, 0.6018437951760284, 0.9331653687323912, 0.6577897399769446, 0.3400018586512138, 0.5283631962916822, 0.5674338678141677, 0.8582828574494983, 1.2644055563159342, 0.26546251524809367, 0.16081333666264777, 0.2984073791832677, 0.061699213129708726, 0.8490405707246076, 1.6446028617575261, 0.5668980829828345, 0.5820203563445698, 0.8949152442974202, 2.049363602355391, 1.1400469949512642, 0.9072653120610723, 0.43345773513082403, 0.9642878086100239, 0.4797111434275844, 0.8006252592302928]], [[1.002106493405626, 0.941989907317874, 0.33033319847001913, 1.0252856297929438, 0.7556393214545196, 0.8324408112368853, 0.3915934972212055, 0.9333345447086312, 0.9145948473210006, 1.3133317932212163, 0.06378518964988271, 0.5971015323019662, 0.8690694331458281, 0.6576571722940019, 0.33768968979717906, 0.5235702394376474, 0.5295883492159403, 0.7655290517762324, 1.2468122956109797, 0.24261492366374, 0.16422946924823026, 0.28572804727840206, 0.07241560897264505, 0.7909146500179158, 1.6439222833949643, 0.599981887538846, 0.5752396822393255, 0.9339295220589501, 1.997769828629519, 1.1183139240293036, 0.9343708400147739, 0.41078961058740066, 0.958396912556704, 0.4637926506477001, 0.8819888192045324]], [[0.8593948488128866, 0.8457701907308869, 0.3110891176051381, 1.0552259433270166, 0.6229387793428018, 0.8153960883855141, 0.4031176009100456, 0.8569004173386441, 0.7360029542661164, 1.3831916271960973, 0.12011807123834309, 0.6661049920768112, 0.9276219234164671, 0.7055178704267431, 0.41981733897029894, 0.5450991665748479, 0.5352143477257, 0.6936319435225495, 1.1303656803532416, 0.31718405344412, 0.18114719387467634, 0.2611488646917452, 0.11047795894839312, 0.7241911397771749, 1.6428237817786906, 0.5885725983933009, 0.5276628788487608, 0.9185724903544225, 1.8776180724196634, 1.108257034735671, 0.9522809736558364, 0.33210741363528595, 0.9143667658372052, 0.4158790623363599, 0.9505496927255774]], [[0.8886270660222458, 0.978830356918974, 0.33131934189709294, 1.0444507324206205, 0.4149037065741301, 0.7589011974840222, 0.38672477714593095, 0.8862327711796153, 0.4470604318429249, 1.3515277337114817, 0.14720042891104124, 0.7150304719652879, 0.8006019155244497, 0.6807432755346718, 0.44599478165199813, 0.5256698108290291, 0.5582978254787494, 0.7369598614051072, 1.0204878574422178, 0.3445246083451898, 0.20099302780160977, 0.2854542658037933, 0.06665482731651312, 0.6868244954218887, 1.6354713567858692, 0.6520023933367308, 0.4128090213392254, 0.8389429267397541, 1.874002158827711, 1.1074256668362934, 0.9601610391033645, 0.2878584885737457, 0.9550355203169243, 0.4573080924690918, 0.9383870382563643]], [[1.188885870052478, 1.0448489235511758, 0.2938866700648027, 0.9916252442129143, 0.40264883703730087, 0.7219228241563982, 0.37889994349669465, 0.9711346500568562, 0.4424952923780257, 1.4335447229957525, 0.1748738220739237, 0.7702555556493734, 0.7498879383925466, 0.6797317618386869, 0.40523587395710303, 0.5079612891419727, 0.5522806010953876, 0.7463137668003061, 1.1179446111665814, 0.3314764172144958, 0.22488995401417577, 0.2757477358703385, 0.05460544525145108, 0.6598782189086833, 1.6722408526702177, 0.6575959373581809, 0.42332857537238033, 0.7849466908764662, 1.9248318295737867, 1.166021894312121, 0.9507793812745651, 0.27050467505435366, 0.9237449283436137, 0.5371536632742472, 0.8663878283731021]], [[1.2711669626148225, 1.1648190622134962, 0.31080260387471786, 1.0420136777644817, 0.43335841850692725, 0.770701951556302, 0.37764425107148214, 1.0223234102629686, 0.5436368639975384, 1.4570241944677136, 0.21064503774715754, 0.7584500684602982, 0.6823920771335972, 0.6219366714894785, 0.3870301652124437, 0.4885271281668687, 0.5475179033358175, 0.7030474946748384, 1.2446620477497619, 0.2680422238188288, 0.2782760748971089, 0.28847310670700266, 0.08020328920624613, 0.620729575768962, 1.7053279184902472, 0.6777530461492429, 0.4562522311956904, 0.8531331394215794, 1.9055587608549829, 1.1863794140821504, 0.9472064761971195, 0.34697857025178874, 0.901121438952561, 0.6119642138110334, 0.8157348770503299]], [[1.2280620116764076, 1.191328637191495, 0.29828666007440535, 1.0184490732539293, 0.4975734213656967, 0.8643271705211195, 0.38506630220217447, 1.002199072409141, 0.6935079280647658, 1.4296911904910377, 0.20509737713683424, 0.7542187565082346, 0.5788617747672231, 0.5748951385620426, 0.38602717242716084, 0.431145471725361, 0.5266481326630534, 0.7376326409480816, 1.1499729062185675, 0.2386259710189332, 0.2762642639627956, 0.2762071934615466, 0.09563854149998899, 0.5213431673397585, 1.6862260176020896, 0.6963722383628483, 0.46326395784356095, 0.8597505705893234, 1.8564273520090495, 1.1858430295126567, 0.9704096443584093, 0.4040843971058341, 0.7948589928907577, 0.6125631673021266, 0.799759478965548]], [[1.1759845161305078, 1.2021598184941944, 0.3022882761008396, 0.9701077436361549, 0.5462869515281292, 1.013494209135386, 0.4060958041518366, 0.9243037988116007, 0.7588970517553022, 1.4221750212016773, 0.20692397012156, 0.7075022962273464, 0.5363417621043896, 0.544145936886572, 0.4008946014923585, 0.40942322935813386, 0.5353370955357336, 0.7678700851478549, 1.1615461351487848, 0.24109743322520016, 0.24705893962636563, 0.3245765076470159, 0.13639857792481996, 0.46983332773452424, 1.606829944650552, 0.8498509691520507, 0.48061345259422283, 0.8205093705646184, 1.8545502713825868, 1.2191643673818746, 0.9871678711767539, 0.433649651573072, 0.6785398057175203, 0.6571769033977409, 0.8042406004949496]], [[1.1920426678396279, 1.1897347502661184, 0.2801391464663891, 0.8769124762199629, 0.5695299101382109, 1.0160538151278793, 0.41080404712272284, 0.912897556336177, 0.7895160336957393, 1.4205160119349824, 0.21502786184762046, 0.8306743546876282, 0.5224177873922445, 0.5009620658252528, 0.4512421154372522, 0.3597520474818116, 0.5812783460449805, 0.7602932720334379, 1.1858260560605849, 0.34938588488296746, 0.20969507008385543, 0.42528083236391395, 0.1585750109976012, 0.4548244162607679, 1.4876987672177018, 0.9219847906184222, 0.5159311447735112, 0.7982936091679645, 1.8454881722705931, 1.201191920908131, 0.9844613417956454, 0.46645947440519664, 0.6049665119527976, 0.7031356177491661, 0.7629996018604148]], [[1.123637194704143, 1.2218129177926673, 0.28290262682690476, 0.8068551976217107, 0.6045552151292626, 0.9925224034839665, 0.40781584058019443, 0.973772565625968, 0.8710701603830142, 1.4593311697959876, 0.22105405007658058, 0.7832047665077178, 0.5519140006219471, 0.6530821787466241, 0.4568214539956154, 0.36907126559363995, 0.6241109138744866, 0.7509676313366748, 1.1009580884009693, 0.4425202970930654, 0.20012733859304624, 0.44031350375316697, 0.2118262126741513, 0.5250095556955102, 1.3651824861519075, 0.9353912876463686, 0.5735496949734986, 0.7311114281761919, 1.8290962421549577, 1.1648092000778372, 0.9286859513885882, 0.5372492484489036, 0.5779178699356715, 0.740310905425676, 0.7384936955087062]], [[1.0892425091155946, 1.2295155603391679, 0.2843992712378281, 0.726820235803889, 0.6082174446112849, 0.9950674129514097, 0.4295554763397094, 1.05454889294275, 0.9373959228501638, 1.3989949296280664, 0.21202025309601435, 0.8698889284431908, 0.6241466118271488, 0.6320496306518212, 0.44462326927634765, 0.36385933829982287, 0.6151426114205583, 0.7126639999729385, 1.0848617733100385, 0.4077663205204128, 0.19214150711872235, 0.3898520669587721, 0.19772246766596363, 0.5626344555546983, 1.3212832582617624, 0.9612671031419413, 0.6704015107825163, 0.7750406078254299, 1.7668358041558716, 1.0709484712847663, 0.8616629307085923, 0.5831879105070004, 0.48958308761583225, 0.7635932131602041, 0.7284285539976475]], [[1.1023888533227955, 1.187424601922294, 0.2888482111655543, 0.7165639513760971, 0.6216061881591204, 0.9687422558474434, 0.39610754859430786, 1.1870770264128017, 1.084190782017336, 1.423692547548272, 0.2249809514887885, 0.8675054861244249, 0.7256811458384544, 0.5569415200196309, 0.4600202814949352, 0.3768092018571799, 0.6764877387269715, 0.712266178356047, 1.0889329283432336, 0.3487340791628668, 0.18327520720290263, 0.2827791927855033, 0.15982616232903554, 0.5834943762398338, 1.269724761608433, 1.0593388616932204, 0.7701875765423403, 0.7691304102830072, 1.6572209449897808, 1.006072805548413, 0.8317059686504189, 0.6927576104170217, 0.4040302156383062, 0.7681981724403883, 0.7347611332874899]], [[1.1988669356266741, 1.3503170684882952, 0.31315550898934646, 0.7602112041755577, 0.6049043770340948, 0.9279688406857103, 0.3931818440845021, 1.3202667125468575, 1.1782834884665268, 1.4075120621024673, 0.20225384396778112, 0.7786298398815292, 0.79249486025378, 0.5029421001492755, 0.42526425032137793, 0.37577843289840396, 0.6437968755196863, 0.7077064456890558, 1.1097845661762666, 0.39683966151875333, 0.1593617967120994, 0.21565527515686117, 0.17201924286683745, 0.6095377089509095, 1.230560240725076, 1.0911729905236374, 0.8036057986068359, 0.7950405376486283, 1.4837538498305942, 0.9931214718893664, 0.7931695066251786, 0.8089070707588697, 0.39652549779933416, 0.7602986362033795, 0.7621570801830952]], [[1.1794801576996767, 1.3104778838249553, 0.3309978413559092, 0.7451419838156932, 0.6312682553404967, 0.9269064623750206, 0.4027373224269902, 1.4583237017554747, 1.1660031583372896, 1.3568882442560728, 0.15999146943461784, 0.7159085146540148, 0.7851221618187565, 0.519175445130891, 0.428291669473451, 0.383042459920271, 0.6849473145389163, 0.7151792898483944, 1.1026313834112003, 0.4209288433944144, 0.17091651322934764, 0.259683743127504, 0.21112428884873405, 0.6569547883883011, 1.1734114187050944, 1.0978273531402785, 0.7715357377420043, 0.82176810869741, 1.3370352962339498, 0.9430666899849729, 0.8009215547105336, 0.8767455526230352, 0.6224804895137461, 0.7625941038220162, 0.6741738255617455]], [[1.093707278193692, 1.3322254161513944, 0.28080979945434564, 0.6462096518338902, 0.6813972544085325, 1.0085766916728043, 0.4423291785841937, 1.5491252318556445, 1.0621936257472144, 1.280091033052082, 0.14352552306105892, 0.7307337766069261, 0.7967916203817218, 0.5346848765842217, 0.4158117195862159, 0.3883382498820003, 0.706508885686057, 0.7556023884892934, 1.1528165658880307, 0.4585875688059703, 0.18126977990882986, 0.29305274945222737, 0.2123943688238132, 0.747795371323293, 1.0503582624833907, 0.8610632570305561, 0.7865947072578927, 0.7238395683973384, 1.2329624860277233, 0.8556925010524167, 0.8535882186368243, 1.0351837574047233, 0.6355527404918953, 0.8799984761651378, 0.5347323724112678]], [[1.0681228382401424, 1.3957957400524768, 0.31520893727534754, 0.6462402019419669, 0.9497895516156811, 0.9523053915846577, 0.42984566938138336, 1.6423805996570067, 0.9295654254821514, 1.149415222312605, 0.15418767592260885, 0.7177382857924355, 0.7441810724597153, 0.5230044658505018, 0.4496488616760588, 0.4174227293529833, 0.67574812457257, 0.7778651592753958, 1.0607205862843574, 0.3733566594765624, 0.16800657822575146, 0.31370907702016526, 0.18527525532791045, 0.7644805817930129, 0.6484917143028166, 0.7874973573022217, 0.7677132221933051, 0.6013607408046231, 0.8508132877756115, 0.7299973587208162, 0.922451285611833, 1.110496048992422, 0.5095585466901928, 0.994505082964043, 0.509438566654636]], [[1.1437429403513901, 1.3778305536315403, 0.31766474670044015, 0.6510285960806331, 0.8863643346931587, 0.9233345881932742, 0.38456437833097484, 1.5786442262724338, 0.9745945230674868, 0.9816753258931403, 0.15947821722875452, 0.7255188080671424, 0.7576352182845785, 0.5253171865617888, 0.4691055928139104, 0.36764243173251787, 0.6565392131655644, 0.7367398169794743, 1.0643049344047983, 0.3185138703195648, 0.17592430206460757, 0.29303504761611887, 0.1379536135723976, 0.7612206916425183, 0.4951063074449401, 0.6252440710005457, 0.7917289671114152, 0.4871360480874267, 0.5200501321131601, 0.6315226383502681, 0.9234448117557859, 1.113574574173435, 0.3747949337452165, 1.0512298199081145, 0.49463625998788996]], [[1.1381284978029331, 1.2771059687241235, 0.3754952014505855, 0.6520289782440317, 0.8903883709056295, 0.8642101961005291, 0.37693270756312397, 1.5679757704280677, 1.0846098679523215, 1.0196134872938891, 0.23244134954899492, 0.6381522548650735, 0.7022288076906079, 0.5230984281519917, 0.47657995726145513, 0.39839718821170633, 0.7131022481291481, 0.8049866001389111, 1.0985307782329514, 0.31004501936751805, 0.16492250851300777, 0.2721596661381908, 0.11091199378843537, 0.7067020307748881, 0.5818230433484026, 0.6402072926962146, 0.7778453509951234, 0.41492570254632694, 0.6065425711551476, 0.4948049205643905, 0.9160438069549572, 1.0540102113309313, 0.49655757458705324, 1.1468919263953996, 0.491542736686312]], [[1.120492506483586, 1.194292286805641, 0.36757882976967166, 0.7106808253257962, 0.9518105189606478, 0.9661168685734796, 0.37942671117809434, 1.57505580237318, 1.1410056363438221, 0.972784168242105, 0.29723464874988204, 0.7216916376880119, 0.6678095480917914, 0.5443377147762123, 0.531644694913268, 0.3683636250253094, 0.7788652545095265, 0.7694097167749714, 1.170132065924455, 0.3027546895196521, 0.1544828451962008, 0.2723819070121991, 0.11153261060863363, 0.6891341141465717, 0.6868339269622498, 0.586922910018223, 0.7661979247063448, 0.42640437770861594, 0.6836436613542961, 0.37667799571989236, 0.8797461117472116, 1.109501505220534, 0.5897201552689292, 1.1865581836982462, 0.49935701415981026]], [[1.1612840612432107, 1.3586874781054785, 0.33092058643897054, 0.9071985919160929, 0.9454777219193303, 0.9325931602583166, 0.3870485229043216, 1.6016651243987927, 1.1265669084842544, 1.0863130493398685, 0.32617740290643815, 0.5473744963434914, 0.5381141712211113, 0.6045879972685456, 0.5299344521972083, 0.33991516247593573, 0.6744912822571645, 0.7242716233480992, 1.06715066270385, 0.3589376444122674, 0.12839604321306253, 0.29383437225512665, 0.10205833871849673, 0.6401882141422297, 0.793500744299869, 0.5707359095335174, 0.697455771348774, 0.36755840250725463, 0.6954357168513547, 0.28529291041073135, 0.9520122336086294, 1.1243765863339814, 0.5289257761521582, 1.1843377593021196, 0.46117437195527256]], [[1.183001956771486, 1.2999052195542178, 0.32805984264072746, 0.8873535890695603, 0.8954821575151156, 0.8452995158762129, 0.3870917550591599, 1.635413936152944, 1.017307221409109, 1.2969159004946254, 0.32676171947773003, 0.5825474558488142, 0.5786105828856586, 0.603097957267958, 0.49056666931931253, 0.3301030030807224, 0.6661687926343617, 0.7431529881568798, 1.2006170922214907, 0.31798531921044854, 0.12794314816545477, 0.31025144939769145, 0.08458105057418, 0.6199465456413945, 0.647881920294361, 0.496898336657131, 0.672373763520209, 0.363166515759971, 0.7319019263186637, 0.2801248947492893, 0.9923444806329997, 1.1742725225416868, 0.5870637600386601, 1.013331904680839, 0.37249882197335754]], [[1.180634935350819, 1.2203782127117258, 0.33954798942634073, 0.871321929544072, 0.8640283802722161, 0.8172769097662398, 0.3766865764016804, 1.616760924441581, 0.9928028495651531, 1.344335970443114, 0.29597628250196406, 0.6614311000222703, 0.7134774295028017, 0.5968048844207093, 0.5133453482638397, 0.3331924939982556, 0.8399580604106668, 0.749297091469912, 1.245878747993805, 0.3692111764346587, 0.14137571939233484, 0.3238057297690726, 0.10504701183953372, 0.6186579090150711, 0.6433771455942022, 0.3196451611699188, 0.6861695072386573, 0.3780692781694922, 0.6514481457803771, 0.34246782741226073, 0.9772818420403866, 1.222406787545952, 0.7785978917397612, 0.9742555305056033, 0.32991291165343734]], [[1.146493043129506, 1.0769385498333348, 0.3294810833758994, 0.8659833350738937, 0.8438484459875191, 0.8507164812698221, 0.38022133031132815, 1.5601502173946622, 1.0013389983372174, 1.2646858452642946, 0.2570203955739959, 0.6106262019758439, 0.6992549736215358, 0.5980363334286722, 0.5066148778466368, 0.3048440294804486, 0.85834339418661, 0.7246392128531782, 1.3390207714639688, 0.42789025483544774, 0.16322135814713867, 0.3695840436149941, 0.05738552280605128, 0.643197207161327, 0.6367903394027326, 0.3134089100026912, 0.6623137172677611, 0.36022851409518764, 0.5428252782656355, 0.3540941936961238, 1.0173904207796463, 1.1714688103985325, 0.9503415207076824, 1.0177105590709903, 0.3199878515879774]], [[1.0685271751661543, 1.0389091117358917, 0.3000880721012628, 0.8373683648267372, 0.800068681906316, 0.7150569649713671, 0.36814443282812503, 1.4760626319258958, 1.003008771234858, 1.3878266504736736, 0.258705882857732, 0.6312560363037141, 0.7837677889596449, 0.5958106749252947, 0.34656463502293355, 0.3078264117904682, 0.7492593033267472, 0.739709900209934, 1.3918271041406327, 0.45270938623488044, 0.19255316616516588, 0.4761751811187017, 0.043476008001311836, 0.6693092770317339, 0.6458671110961209, 0.30217406304028893, 0.631060828362457, 0.3851892077407449, 0.5729444607926228, 0.4527856796206591, 1.0356904392657094, 1.1669858262442323, 1.065401367208129, 1.0128968686633648, 0.2945969805599108]], [[1.0103851681637335, 1.0199552719160547, 0.28962506372504454, 0.7947915160015102, 0.7410232985311729, 0.6994225362539241, 0.36215511443811144, 1.5260656691503591, 1.028887167991346, 1.289178161841921, 0.2925252190006272, 0.6387854932065296, 0.8000005522532315, 0.591166990708019, 0.29996845001175493, 0.29596986682739324, 0.7608365058776331, 0.7367872954126615, 1.3256312962795647, 0.3998248033500361, 0.27999207891461225, 0.5237898803394562, 0.026409381072670303, 0.680828123136742, 0.6045276831330136, 0.2641527652383139, 0.6281655711446479, 0.3850061355733583, 0.5779786845993534, 0.5556677279787325, 1.0482685363010118, 1.1910616103796432, 1.120637852105304, 0.9995881347637328, 0.2614118448881316]], [[1.0811246699298598, 1.135606481283921, 0.29206006818497915, 0.8156059763326893, 0.6867079721292452, 0.6737825434426448, 0.36258833084736447, 1.6391180097111686, 1.009844320863784, 1.2989975580873951, 0.2723605609176384, 0.6550802340957513, 0.8947554056895053, 0.5870822789967827, 0.30991023056696687, 0.3028248015763285, 0.7125384025775005, 0.7317113336629417, 1.241624667460596, 0.26968267789704137, 0.3285478876591252, 0.4611349787028978, 0.014042638290969255, 0.7187587301707825, 0.5380707551914705, 0.24879331098871033, 0.6326199550627385, 0.446104261380896, 0.6085326928268705, 0.7363523243580501, 1.0337611411617236, 1.1340385188233213, 1.253988616877019, 0.9168258400768466, 0.21284566031343205]], [[1.1031571904309425, 1.0616382373057602, 0.3131631172194445, 0.9265267330155933, 0.7227078447139454, 0.6817006030944296, 0.34176490223096795, 1.6280182011586666, 0.9842222296405121, 1.287257676522887, 0.2674794386594966, 0.6778909285635581, 0.9202427393299548, 0.5783744948199827, 0.29228243162274037, 0.29151882319354666, 0.7016113868412311, 0.7177385179140857, 1.2368409708258143, 0.26318098714140337, 0.3829437724466358, 0.47453204844595365, 0.01772449937608376, 0.7452333629534705, 0.5752073700534601, 0.318090414587243, 0.6634707458720821, 0.4743633725346486, 0.6112527421621394, 0.7984927407291966, 1.0284728992032703, 1.1228144439926626, 1.3240603336587635, 0.759401997588116, 0.25758269704467895]], [[1.2681336095598372, 0.9623273614477984, 0.401231028551998, 0.943468115295856, 0.7293337134357392, 0.6869993646341228, 0.36114490998357374, 1.6896445467715406, 1.0199979857561905, 1.2589695680458153, 0.26317114081267856, 0.5646318042093875, 0.9088388336729163, 0.5909517759978624, 0.2988492813566055, 0.31168195799578824, 0.7211479292243643, 0.7823420937962482, 1.0491505463997692, 0.21450802909575692, 0.23150231488789502, 0.5304148698154236, 0.02292797044697369, 0.7654052034358916, 0.49990868896942764, 0.35353956312156165, 0.6600058055320789, 0.4731947134536796, 0.6411375217044829, 0.9161908446585634, 1.0319214519418174, 1.1065423575369855, 1.3565391998423357, 0.6702612886068637, 0.3572358076388119]], [[1.3392657291547496, 0.9969518134875457, 0.4358559456008634, 0.8902313738705931, 0.7594288414808278, 0.7083635530434966, 0.3598327030224896, 1.6532205288035993, 1.0429534417970328, 1.3306128491543565, 0.25952753475354273, 0.5604728698871388, 0.922850176878678, 0.6768290373295675, 0.3038908733069533, 0.34769028329643054, 0.6907779993724839, 0.763762126239788, 0.9554452087220493, 0.22637794200674824, 0.24308018436957451, 0.5645766977958725, 0.029289500199752964, 0.7943836946276046, 0.4349556977413872, 0.3899712034506293, 0.646849912737903, 0.4680386081014245, 0.6566470390076664, 0.87613732022231, 1.0310412244436291, 0.982687146100691, 1.3951854586489074, 0.5996650623838284, 0.35245749663402737]], [[1.3243965793052659, 1.085706697880919, 0.38253599662803944, 0.8902075469716964, 0.8024683683315635, 0.7170792423597374, 0.36542380099521465, 1.5729450613741363, 1.074813269259498, 1.4745075684786242, 0.2803514964948459, 0.5642618497741313, 0.971308278675141, 0.6445869998572418, 0.3057134852892234, 0.3446668199782075, 0.6381203543831439, 0.7481061966693047, 0.9015197959956197, 0.24227468313029704, 0.24346629654785484, 0.5440710809047823, 0.011391929219434285, 0.8232037181380542, 0.3774379185255321, 0.41463839778406203, 0.6525066483558498, 0.4437426986806595, 0.6828661682538995, 0.8718322517055852, 1.0293921707441918, 0.8703020252576563, 1.4071511918051123, 0.526865238063195, 0.33144185505769186]], [[1.4080954899428297, 1.2321731693223932, 0.32037461678161155, 0.8796800633142696, 0.913494170578405, 0.7272272968938387, 0.3656611834333544, 1.543522746324766, 1.0713024289020887, 1.5628091523111838, 0.33415263203461565, 0.6052921841776611, 1.0136608674597387, 0.6303632166311228, 0.2591690910905582, 0.33850943147319007, 0.6132674196779329, 0.7664033753617859, 0.897077064324228, 0.2585674849745342, 0.20978203859519667, 0.4813688587262356, 0.009170662696097783, 0.8356679767564212, 0.4363631032210045, 0.43170663026601064, 0.6336442544187632, 0.48542012224187425, 0.6759932697324303, 0.8500499893404351, 1.0231677054126609, 0.7772746778190248, 1.4041809894263055, 0.5031303381201134, 0.3070172249206391]], [[1.2926415705054604, 0.9476907666817271, 0.29785234822866186, 0.8345776050464031, 0.60885387577379, 0.7415091346133741, 0.3680430391011266, 1.5401717008215383, 1.0536874554555489, 1.6248499413505493, 0.3027352363198211, 0.6214287576134317, 1.0254081125098828, 0.6086892537021322, 0.2647763325426533, 0.3240473577643822, 0.6167940911564012, 0.7895230300124552, 0.9658986078632017, 0.24842010074986254, 0.2158834268125544, 0.47431565778334034, 0.036870205392535736, 0.8430708075317239, 0.5416354541338841, 0.4190027821192695, 0.62143342910448, 0.5892344345910786, 0.6583469949653561, 0.8383740716866034, 0.9841125216648309, 0.7406684800211842, 1.3752387483141217, 0.4437506531903791, 0.34596819105297705]], [[1.1962246010581101, 0.779639520588218, 0.275499988211879, 0.8300814301917177, 0.3342725729283703, 0.7773725762789209, 0.3723639336946875, 1.538898606106165, 0.9786351757544673, 1.588787512432892, 0.2687356153915424, 0.4708034006154874, 0.9864042567364901, 0.47195413790782487, 0.31098949374642454, 0.39618533996721694, 0.5526822766535029, 0.7622622207950999, 0.9465382709311878, 0.21965688452747062, 0.2769569640573939, 0.523543486756342, 0.0363871459150307, 0.8231581090936522, 0.5610683856446841, 0.4071813105001385, 0.5982122468310864, 0.5487548512447429, 0.6597545825500539, 0.8287272073248096, 0.9676550021746542, 0.8007185803298325, 1.3536200529911717, 0.4208066635468847, 0.45941815498569333]], [[1.178975664490061, 0.7744622159637783, 0.34453186814222464, 0.860683457439155, 0.3367690748033503, 0.7652399533063151, 0.3691549027498936, 1.5958552151351957, 1.0560856838622055, 1.5922520407605467, 0.24947897228116528, 0.48507178397431205, 1.0147243671859463, 0.4544294346254293, 0.3502400620201297, 0.3964426058249608, 0.5151463071979354, 0.7647268951799663, 0.9084031495135361, 0.21452361921408764, 0.26223894255517866, 0.5346624544988459, 0.026300440544861733, 0.8520901686082221, 0.59648968046407, 0.38284862501269323, 0.6386331463384363, 0.5193451170672123, 0.6640405492548652, 0.7582886399566287, 0.9525716639572492, 0.8055182764285956, 1.3161849006250366, 0.4093707871805826, 0.5993353492370499]], [[1.1287286443378517, 0.8663670948172398, 0.3622524706985453, 0.8301168842708562, 0.5075852964352141, 0.7675588857302644, 0.3685917960323653, 1.6519987625558077, 1.1747925491148308, 1.558675358185266, 0.23887995222716896, 0.48720016358514634, 0.8985751337392728, 0.44357595648135617, 0.30915192688453097, 0.4045010099977086, 0.5349412019205951, 0.730619315368034, 0.8693931942750956, 0.23977914295183367, 0.23113713313369216, 0.4770078118828709, 0.05301669704886448, 0.9774905082328083, 0.5711266433390548, 0.3785521693952108, 0.683162886071814, 0.4146289929620083, 0.6728764156846185, 0.794348392607036, 0.9101187220424538, 0.8172827398258434, 1.2433998008057594, 0.4358041778920466, 0.719061557285543]], [[1.131193907816288, 0.8551114856832184, 0.39230985825693454, 0.8052740681185102, 0.36072951165627276, 0.7496823747379682, 0.37034321495580963, 1.5693982429430413, 1.1277350917256665, 1.4395288235129904, 0.21364758438242137, 0.493058016750784, 0.7026728542494185, 0.45074069227366437, 0.32059447994237755, 0.44256520133711574, 0.5277625467941861, 0.7118109605526629, 0.8087731490496366, 0.2625310468827462, 0.22644128682539877, 0.46256739875258535, 0.04364005934919163, 1.081283350593945, 0.542510229209123, 0.3924044267035287, 0.7401955288185313, 0.34380856505780555, 0.715790263063769, 0.764946112646932, 0.858354062755303, 0.6758162289342825, 1.1807568425455832, 0.3261582930026957, 0.7774102943281909]], [[1.1089194133106883, 0.9032053667337804, 0.35040544736095014, 0.7834666976759452, 0.49216105467184623, 0.6772258428977818, 0.37151700771653556, 1.5963919892934046, 1.1249981806030935, 1.4367171161714647, 0.18664494986504726, 0.47315048981209373, 0.6112454572422681, 0.455977379794481, 0.37299405677773, 0.4656430486668931, 0.5399145402577672, 0.7377294976994893, 0.7920151160963036, 0.252953207328545, 0.23271814869324758, 0.39088883335758295, 0.04440266206990956, 1.0464701297758912, 0.4951891640695154, 0.3911578313201326, 0.7006438877698788, 0.36187771538247726, 0.738697058354722, 0.7931767580742068, 0.7195797635355687, 0.6047258892633969, 1.156046631380677, 0.32219432125313235, 0.7385995544872574]], [[0.9876920234157922, 0.9764630101159072, 0.333408279523316, 0.8130414271331947, 0.39562208672305826, 0.6645709630705185, 0.3707179048302762, 1.6243900516423229, 1.1642618503578341, 1.3618274899556981, 0.1711475554360296, 0.4835003399813477, 0.5911740669246791, 0.4450240933760928, 0.34777716913945494, 0.44029920082117135, 0.6152953925793285, 0.7454963379471351, 0.7508645840038036, 0.2536264185201051, 0.22631657336938626, 0.3896026800827867, 0.015990348491129312, 0.9904455899061839, 0.48363464628459896, 0.4034787564382908, 0.4438543846279246, 0.3760068327658628, 0.7565988471173698, 1.0088826297598616, 0.48018948302255604, 0.5439847656924621, 1.100376274606576, 0.39360946746113507, 0.7596812972659335]], [[0.9065472484496688, 1.0071286708966032, 0.33671074639638565, 0.8222094912486231, 0.5036146731953449, 0.6787237472941035, 0.3702849121257981, 1.5782934539025613, 1.1497710461267399, 1.4029427484258372, 0.14018256092900253, 0.4890819083327923, 0.5967275204065312, 0.4402283397125767, 0.3197337518927999, 0.46425910460083164, 0.5438364958118952, 0.7777756473712167, 0.7778490148354269, 0.2512642491334864, 0.22978306558986272, 0.3698930307376538, 0.02025852541134082, 0.7729246453787911, 0.4710340495628237, 0.4089045752609367, 0.22312903422543962, 0.40992392123355503, 0.7499262860007742, 1.0805208766995982, 0.4694914219686298, 0.5440539200143755, 1.0278721471653254, 0.41604061908883583, 0.7054311538039514]], [[0.9014364318730009, 1.0348418169113134, 0.35617968243623943, 0.8399886961134836, 0.5675109421682543, 0.6797253145625479, 0.3682051388451773, 1.5210402305944548, 1.1647848467298585, 1.428388833019275, 0.13786309988825068, 0.48947028546769705, 0.6131227683227288, 0.36497844451726597, 0.2532685062901575, 0.464472315303403, 0.5885845373298011, 0.7411532653222428, 0.7950270409874238, 0.23242160338853834, 0.23818433181872758, 0.3383728159640025, 0.02798070593814957, 0.6107005175493853, 0.49666189621199563, 0.4031540787505458, 0.20603912052017898, 0.4902213053497748, 0.7211739849969684, 1.058000814164271, 0.4692374066819334, 0.5511441264368617, 0.9135488905371341, 0.4811146300954425, 0.7273736859601537]], [[0.9152348120496544, 1.0852289385591238, 0.3456178860380643, 0.9027248089151283, 0.6108283102147302, 0.6705226903262462, 0.3685759711390425, 1.4252176685183886, 1.127198923698899, 1.301770982619785, 0.07186278740005766, 0.4630824019166858, 0.623001337582725, 0.3683722585088457, 0.26021100646007617, 0.44093693951478774, 0.5548308886329072, 0.7326706755538429, 0.7877133806833084, 0.21496254307445853, 0.31743533202456653, 0.28238149650297595, 0.0497042528737966, 0.4692034428291627, 0.4761851165909188, 0.40685313795285377, 0.07657324299559246, 0.5377700007550051, 0.7039606137259757, 0.9681054390104405, 0.4300068378480031, 0.5767976028423377, 0.7473545886940381, 0.4944203417278621, 0.7482596048282439]], [[0.9457339377027782, 1.190813794616443, 0.32796130361632647, 0.845827513404968, 0.7024426605922697, 0.6750427347527039, 0.36706152207797477, 1.2422058825285687, 1.1541287398162785, 1.0532356446166435, 0.10825177867624242, 0.4599149640154967, 0.6592122839620347, 0.35560940300730853, 0.23076670016724926, 0.4541769791829263, 0.5963738021839192, 0.7464638434108674, 0.7907434318406066, 0.2050367794450739, 0.31839260025682425, 0.2826625999733939, 0.0729365525262303, 0.39256119895508834, 0.5169172517555561, 0.36273785758180155, 0.0649049873939433, 0.613208224557971, 0.6765332843034149, 0.9127301936691514, 0.4170045710346497, 0.5720229338694061, 0.63418592882417, 0.5421484003654051, 0.7728850110230487]], [[0.9055877108387604, 1.1968651707848923, 0.37272020340031253, 0.773604573738025, 0.8198458307633562, 0.6883539433425394, 0.36750305869669386, 1.147677294214383, 1.091338729125566, 0.9411514983139254, 0.15421955564872838, 0.48513550711537257, 0.6827641822676089, 0.4487727160814735, 0.2751264270044096, 0.4218251805365386, 0.6813332564629913, 0.7621686762004278, 0.7898875764879069, 0.20343774593397065, 0.2607862763746864, 0.32621066714441255, 0.06356139298074595, 0.34694214011052066, 0.5588570488803689, 0.31608631347148053, 0.07233194182119378, 0.6587875266746606, 0.6642061112849104, 0.8669782734523486, 0.4210840523313151, 0.5519857453285173, 0.6165648911859788, 0.679472635748251, 0.8195297971522864]], [[0.9278994079026677, 1.2468681650921367, 0.3463359265655171, 0.7282088316606286, 0.9071581948155758, 0.707932167926175, 0.37427656250781577, 1.0912800764912305, 1.0857946150419056, 0.9620618093424185, 0.1560729246398834, 0.4679562066873689, 0.6706910424594645, 0.44075522644180953, 0.27014679955055815, 0.4057522861684386, 0.651718993918926, 0.7773324957363599, 0.7640156607161941, 0.19189151471175542, 0.24061109470649125, 0.3378423835176195, 0.05422708164798766, 0.3223747034361139, 0.5462370729358104, 0.3014048250176293, 0.06420226104934537, 0.6786990383397653, 0.6787286100530485, 0.8359100734731923, 0.4260538675842237, 0.5464245840785168, 0.5667603392520613, 0.7171731784603668, 0.8786769286366822]], [[1.1070042567114173, 1.3654953291698628, 0.3989353707742468, 0.7645380283024262, 0.987238079940792, 0.7329692528559573, 0.3708799874951242, 1.0309685961076314, 1.1142656593732903, 1.0688017163096546, 0.1019681061582078, 0.4604505377159825, 0.6896641752727672, 0.42546287895172774, 0.2710599180299843, 0.37180166133972464, 0.6504935900624067, 0.770266030319135, 0.6250336059393626, 0.1844875007709837, 0.24703585551405036, 0.3458437218204552, 0.04202751169027649, 0.31336702489492696, 0.5285852514036906, 0.3083899655890058, 0.051937000893211197, 0.6998643131439186, 0.6600977308870104, 0.8281314845443328, 0.4677814862772176, 0.5491401288483434, 0.5127642374602897, 0.7751319439603959, 0.9004666220848687]], [[1.0872767493694484, 1.3602069241020232, 0.3773049382117267, 0.8054085627403453, 1.047793265625359, 0.737255100457187, 0.36958990329867025, 1.0517206558789665, 1.1259281257541713, 1.0553544609485275, 0.07246918322029593, 0.4676006047834431, 0.6627471453260998, 0.43034996450336477, 0.28317278432861964, 0.3774383054306509, 0.6728615854499702, 0.7673470910260067, 0.6500834309751267, 0.17614864596304028, 0.23569392532140934, 0.3957648586525707, 0.0444858710911221, 0.30782869824242187, 0.5249029415362485, 0.3274925020064727, 0.043714659504023756, 0.7204146676301051, 0.6307279622877945, 0.8268888277619629, 0.4753472022671438, 0.5651349041765477, 0.4702924253009807, 0.6828012389995788, 0.9013961292422742]], [[1.0498631002566787, 1.3612788946802727, 0.34227190411017194, 0.8330591213384546, 1.1173391012575007, 0.7701569282852873, 0.3854064312174116, 1.0085935908958459, 1.1726154036555707, 0.985674153477482, 0.06184004832302682, 0.4755545845675754, 0.7027901675987834, 0.45140471014667943, 0.2885029959656809, 0.35225191639724646, 0.6632009781449042, 0.7679245540941784, 0.6964051608537801, 0.17533109419765983, 0.18796382213719381, 0.4074507841730712, 0.05850882522127106, 0.3049799965569918, 0.5117767493376094, 0.3542080683674773, 0.05055084471393226, 0.732947706865643, 0.6562072193650847, 0.8353386947568924, 0.5028136981894831, 0.5950523550272773, 0.4458098465839384, 0.5885285451160364, 0.8846081415341217]], [[0.9760993536125956, 1.3219891323470887, 0.3467458453192348, 0.8351516222512729, 1.2008148970150434, 0.7414605780321897, 0.3929496996758189, 0.9663994556057802, 1.225627908314003, 0.9260476576873466, 0.04988664412297625, 0.4830052002532048, 0.6955071035263471, 0.4334738062538252, 0.30693553888885494, 0.3303938470648612, 0.6521345330001536, 0.8619554526152875, 0.757100157574184, 0.1727519526293147, 0.1809682269166183, 0.4079763307802994, 0.09353715883844857, 0.33070186776775445, 0.504936946070649, 0.3743106726316721, 0.07353735859186007, 0.7500554647611974, 0.6523515569954705, 0.8463241305058543, 0.538509711535212, 0.6196473646723026, 0.49681993182798634, 0.4598760201724499, 0.8132150361320833]], [[0.9402126695361139, 1.343997232381647, 0.3149845073908102, 0.8516339457086366, 1.2501956464982862, 0.7237130007845851, 0.3784491870267118, 0.911289670806561, 1.111251234538488, 0.8277787848112708, 0.04438425246898134, 0.49868156955512466, 0.7315772870293487, 0.45340328878585345, 0.3081735493282576, 0.33404688807326555, 0.6601925150510215, 0.8786395455746654, 0.7552409967808353, 0.17860652358333753, 0.1919452818103085, 0.3891835155791863, 0.10380175143183143, 0.3612718504669091, 0.5659352471396755, 0.42581050835908396, 0.0897206160883662, 0.7230026484662864, 0.663742568512397, 0.862934059890527, 0.559793047144154, 0.6202448640873724, 0.5611842435317607, 0.3819281610208701, 0.7574371416032178]], [[1.0036380937572706, 1.3881685671922925, 0.3180816192867453, 0.8635483810281326, 1.2753762662516177, 0.7251268916827014, 0.3640130026253985, 0.8133811301054854, 1.0338007263071614, 0.7588757250949221, 0.05117305166360149, 0.5340011119801666, 0.737256705840035, 0.44992440309563553, 0.3027931939146383, 0.3286025870680671, 0.6711310227636453, 0.9261396183765982, 0.7493542573134537, 0.18062539884367101, 0.1967405711697673, 0.32608008867750915, 0.09812638225077337, 0.29029994238344853, 0.5610061608282143, 0.5615443544526532, 0.10756045325478594, 0.7443025639667709, 0.6920608838205156, 0.8672363453244318, 0.6285107423151438, 0.6031639501092367, 0.6587091731311705, 0.4237541393068192, 0.7209695008628059]], [[0.9641421938452321, 1.3554144635713772, 0.31379597382671576, 0.8879882318772865, 1.2855184458298576, 0.7126994421348218, 0.34645149350185866, 0.7510883840660795, 1.004420487550901, 0.6834095398412835, 0.06550155458469759, 0.553596547125038, 0.7195297514788008, 0.44361036076919036, 0.2956924600170097, 0.32589926536687003, 0.6691209462880436, 0.9154149727320955, 0.743288823378243, 0.17026532663458963, 0.20856491344324063, 0.29132796115609794, 0.08489493182573028, 0.26073352694813473, 0.5478434244448906, 0.6931885806998153, 0.08819250185572333, 0.7628430133010708, 0.7057154794261516, 0.8633689673151766, 0.6818220348681573, 0.5869189274159877, 0.776184947866356, 0.47607890543403797, 0.6896628581970634]], [[0.9365157142112195, 1.3684348049637707, 0.31450182173729824, 0.8642036546352974, 1.302316055222261, 0.7224669168060612, 0.36103151680362006, 0.695712761813379, 1.0128147670361787, 0.679322943134885, 0.084468008919144, 0.5910588827084184, 0.7147457077374464, 0.44714306774738566, 0.2951479010436665, 0.32961228535171844, 0.6330097317161437, 0.9280225657648293, 0.7550653058178208, 0.16548088025820684, 0.20418142392335273, 0.28844147522438457, 0.08714719136508622, 0.2734826242312376, 0.5823888027599038, 0.6793535792023526, 0.09495324373623398, 0.7549772353527029, 0.6803596408801773, 0.8489453137920426, 0.7247625610402411, 0.5725767693444153, 0.7629481661039825, 0.4475759111694799, 0.6429339772700007]], [[0.9734948940072243, 1.377355194291963, 0.3123229604812161, 0.8777104385307353, 1.2849534961601394, 0.6943532568718698, 0.36783999303715237, 0.631821951365967, 1.0194038011432123, 0.6524705848955215, 0.05186782772598242, 0.5597283397238454, 0.6971564672744663, 0.41493403180350463, 0.3096676644205444, 0.33611789766001543, 0.5269293523816724, 0.9606569129660594, 0.7732166512759292, 0.1696661916060014, 0.18673144467780328, 0.25259108591554236, 0.12531382525592244, 0.23111471211053453, 0.5835414092384352, 0.7057881449001352, 0.12631641460799464, 0.7396054338714535, 0.6972068080520655, 0.826489352842768, 0.7512467944027287, 0.5622819948164416, 0.7393919793301018, 0.4327525740174284, 0.6216596722245789]], [[0.9731493378006151, 1.3324240205701234, 0.29830862913827827, 0.8598058352091368, 1.2832614099980648, 0.6809603586023287, 0.34357870280469965, 0.5789555249576855, 1.0491263130432764, 0.5181167354735646, 0.03851771306154323, 0.5270505523403484, 0.6603425418878768, 0.4403696345155321, 0.3404149403282979, 0.28909949124912554, 0.5888421159412804, 0.9877999565236124, 0.7943437285232804, 0.1941770758046604, 0.17720296262967156, 0.28349846187158456, 0.11528261773887406, 0.20401540864867007, 0.558677895745442, 0.7527606167193739, 0.22614263267616008, 0.733586132073821, 0.738369159595619, 0.8147533686187609, 0.756715167610204, 0.5518947575040372, 0.7240036573077063, 0.44653584621048403, 0.5702387365463543]], [[1.000728391157895, 1.3339844835988828, 0.3123721893467394, 0.8711902423486225, 1.287306144835736, 0.6721597683491277, 0.3195895534190133, 0.5320550380851254, 1.08484796930275, 0.5026224575578007, 0.03581291266015343, 0.5170848261412287, 0.702342373853396, 0.45628953177087594, 0.39999073405829494, 0.30061746079278795, 0.5774202186065811, 1.0151598721558506, 0.7994125716856411, 0.29389010900229645, 0.23570391161262053, 0.34097563003371983, 0.12480369531346483, 0.21082789970834181, 0.5119848229407649, 0.8380637235753431, 0.27710830456220625, 0.7274371277626436, 0.8336242647981427, 0.7774025798644714, 0.7496727513576398, 0.5271026243630783, 0.7352553090340419, 0.527340853724119, 0.7432996578632001]], [[0.9720041314055643, 1.16519895064421, 0.34939614793151674, 0.9153655650392142, 1.249015317200626, 0.6955043561874192, 0.3569284283117872, 0.46174650140539947, 1.0794491708055811, 0.4488681404846453, 0.046315924428253906, 0.4994826246704194, 0.7406843390579765, 0.42581229757678707, 0.3354733640623463, 0.30741569089663356, 0.5755116324960308, 0.9881457097995049, 0.7485509745260804, 0.23362478957195243, 0.22896376375017474, 0.3436416181425075, 0.10344482620535836, 0.19392906458682602, 0.37876698657862423, 0.8185942941069579, 0.3261072763652286, 0.7548936478078736, 0.8342126472461262, 0.7799190645803044, 0.7469283652199505, 0.5032681107105311, 0.684061672379096, 0.5896567494703566, 0.7019091199875227]], [[0.9221163581500673, 1.0720438639749768, 0.37941279802890937, 0.9749033900648945, 1.178829918733928, 0.7109054749302355, 0.35467037287482445, 0.3763502182294074, 1.0473101451915467, 0.41706633823105843, 0.04228249464288821, 0.45800931473284856, 0.7842854203097102, 0.40913566588927697, 0.3097487552473862, 0.30433117487395855, 0.5657677314893371, 0.9290834431089074, 0.741751177144931, 0.14436028806929463, 0.24881518848136844, 0.3347919646314439, 0.09719362407798608, 0.2182292983413594, 0.27529815865834095, 0.8102504891015844, 0.3116763954700052, 0.7984681963046605, 0.8373806319342674, 0.7804618145050137, 0.7634865595036044, 0.4913496075233781, 0.6467786172962695, 0.6606533948089827, 0.6777910194333654]], [[0.9062337586830109, 1.0507945846704914, 0.35150868423847925, 1.0089781918582734, 1.1049366170593533, 0.6779480142265868, 0.36883928564469176, 0.3565059447258151, 1.0123116244718955, 0.3746604465475273, 0.05452572842172563, 0.48903137455787016, 0.8115624202176035, 0.4427079365656651, 0.31505465309265834, 0.3176389670799013, 0.6045822337881028, 0.9068065543125223, 0.7407767726570149, 0.1503740697263894, 0.24124964062248383, 0.2805512990822888, 0.11898048320860836, 0.2599029205778882, 0.24476832299271453, 0.8005796636495193, 0.3031711654899799, 0.8398534949293546, 0.7980471365209142, 0.7836375316464357, 0.773241868173831, 0.48385396445332907, 0.5883382015532985, 0.7571168163601072, 0.5318860993424473]], [[0.8935320194254587, 1.006187182578799, 0.3268491428930823, 1.0859234748425497, 1.0463303472210141, 0.6646902236429062, 0.36435796510259144, 0.31718244396175144, 0.9848199813143318, 0.3387618765781526, 0.0719882253868312, 0.5050239276405804, 0.8572868759565797, 0.4425144624438424, 0.31091310594504673, 0.34043545292300215, 0.608662648195923, 0.9528053197488695, 0.7319555660453585, 0.12772948627999398, 0.22590080622216396, 0.31554378151184975, 0.11219889716811358, 0.36253816636826053, 0.28098462303723226, 0.8018960353392472, 0.31541121211324985, 0.8678459843091464, 0.7005039096714919, 0.7846084609164662, 0.7752854010619054, 0.5023960818944255, 0.5973828023860321, 0.8016685513016726, 0.6331450088357071]], [[0.874515564733622, 1.0028207892327492, 0.30942666430128296, 1.1954072409047427, 0.9737051654922415, 0.6737318097165169, 0.39169514950930245, 0.3341869806430048, 0.9486948786220298, 0.36488843226559065, 0.06995695618691616, 0.5045966718125141, 0.8981960130726749, 0.4278223484475087, 0.3017246284978867, 0.3823604164094415, 0.6062833643062981, 0.9683498433456795, 0.7715847832326099, 0.15271643395149082, 0.21794863262176223, 0.3231148124220368, 0.11683795291059021, 0.43891542828416186, 0.3715084111117829, 0.8278825649733879, 0.39326465491536444, 0.8675400086267693, 0.6018154342052108, 0.7791565298642498, 0.799918239558087, 0.5344576728389427, 0.6116086845773636, 0.806277153985582, 0.7810629219540071]], [[0.81009697547834, 1.0478215161395117, 0.3314461685551465, 1.2354678944888486, 0.867593024665579, 0.7346092353370604, 0.41066296060093016, 0.4104197488206195, 0.8999544669360856, 0.5361646527956798, 0.0475234290153722, 0.4704084274024229, 0.9631677759242918, 0.39183483580104683, 0.2860138580888507, 0.3757812034460474, 0.5975327302387263, 0.9175614369188857, 0.7438405283489216, 0.16374145778978902, 0.21584419086316203, 0.3212290981286589, 0.15591534931337156, 0.474027849379385, 0.3864229581785804, 0.7980108551671462, 0.4952040120877481, 0.8150999365260787, 0.5366597578788144, 0.683702741871493, 0.7143081748042456, 0.5754133529024089, 0.567822706362298, 0.8297196349832983, 0.9405836302155105]], [[0.7650127662464551, 0.9550508029401873, 0.3382014821186867, 1.279895486199912, 0.7219211288949885, 0.763437224104077, 0.41444723615653706, 0.5162630614331547, 0.869576354780335, 0.7978202277202334, 0.08899273725066437, 0.4675094639656588, 0.98453621774662, 0.41114043437860903, 0.3256575541175547, 0.36935495651152916, 0.6269116801237775, 0.8774470729516937, 0.6737537078884985, 0.17431733768307991, 0.20419438630185666, 0.35001760365072904, 0.1182919145477763, 0.3848814465889618, 0.3656524859286031, 0.7811256455929925, 0.6112575656497669, 0.6592239385340447, 0.612551996833979, 0.6731023914385186, 0.6135928059687408, 0.6141205320436667, 0.6100391181603297, 0.8355564602828971, 0.9491222776592044]], [[0.7298903317754939, 0.9356030585204373, 0.34361594989260313, 1.3617685551944378, 0.596460631351202, 0.6604934820972939, 0.4286311641073105, 0.5659780542848667, 0.8948251407507685, 0.875739861689961, 0.07462893432229062, 0.49940925645631185, 0.9642607200001028, 0.5194288760735368, 0.3167187930001761, 0.3609476163469637, 0.7164868160321685, 0.9078063046500527, 0.6491507598260098, 0.16522499998618764, 0.20760754776977638, 0.4526968726485475, 0.08775121162125682, 0.25913987348905876, 0.276448129157156, 0.7549140682098814, 0.5800841651839024, 0.6218240339934857, 0.48611566889275504, 0.6263240877205797, 0.46326471977753003, 0.5809447445799989, 0.6459165804035059, 0.8778223278696935, 0.8321428494897631]], [[0.6706111298863606, 0.8929616021754705, 0.35993182251566425, 1.4079630659295266, 0.5163394879683703, 0.6693224177071637, 0.42481907116241935, 0.5268091259904351, 0.9409815923348128, 1.0947604473141364, 0.06722923107275958, 0.4084428269658272, 0.9984075958427696, 0.5123693707804247, 0.3081686229303813, 0.374615408958372, 0.7245641597579333, 0.8255289240361574, 0.6702252987867253, 0.18832750624381933, 0.19860881498996683, 0.417949887909177, 0.061463679224247676, 0.1908782916228533, 0.17232471963391427, 0.7460550085389608, 0.5160317309907527, 0.5542954100021675, 0.3924600152158737, 0.6389769084208333, 0.5065219206311377, 0.6640535523741526, 0.6260033807620824, 0.9048257149707248, 0.7325494325078092]], [[0.6161767488366279, 0.9143435297327154, 0.342501285938251, 1.4708466589835358, 0.5661396735132064, 0.671115363875723, 0.44223226994276393, 0.5513527524282502, 1.1128130892720471, 1.2854832676534627, 0.06533753092977322, 0.4163531414428397, 0.9787963862908342, 0.45733325780077305, 0.3110808383934583, 0.38472378866939355, 0.684933383416066, 0.7831668610720548, 0.6760720114949667, 0.19590618453734399, 0.21263004819249998, 0.41974630780780336, 0.07328099419475011, 0.16227388326513897, 0.1554991093789062, 0.7084494715951182, 0.5026685579377975, 0.5914580296575531, 0.36114116203289576, 0.6842846159295763, 0.6702253143389334, 0.8248380066386057, 0.5902693587659372, 0.9324845714231996, 0.7353673787616295]], [[0.5461530606015913, 0.9656727299569462, 0.3525596391448922, 1.4538887952865756, 0.4893409245248735, 0.6756619074021969, 0.44171729081754574, 0.5734369292503266, 1.2409874130096135, 1.0994746437653222, 0.07782542020756374, 0.4244709341412749, 0.9303375540167615, 0.47857791084345747, 0.31360284793877286, 0.3730422863422214, 0.7348812260704074, 0.7683453135301195, 0.707229728646156, 0.2076739648863396, 0.2157205856078066, 0.41965250559749456, 0.12995520710013617, 0.15771831718105006, 0.21145904899827578, 0.6608777438820808, 0.47012047325306794, 0.6182648104677013, 0.2923424378036479, 0.6695021307020288, 0.7722752390956373, 0.8324210241546307, 0.5980185594937165, 0.901020937075104, 0.724374989040977]], [[0.6654557070456312, 0.9597245285497952, 0.3240210814470306, 1.4704394539935504, 0.6995785784763662, 0.6714548578300814, 0.440494681846058, 0.5533594571377656, 1.317692883666966, 1.057187844287225, 0.08661179784949449, 0.39212996751853363, 0.9172097565457562, 0.46092474211115986, 0.32037870418620823, 0.3520672731875987, 0.713449445697008, 0.7955255253136659, 0.7221943360538701, 0.2228087212749412, 0.22208798779190364, 0.3699926007188178, 0.10795615056372632, 0.14905444218181357, 0.25402557901767414, 0.6478040255814608, 0.41215805442665177, 0.6278601010865449, 0.3220120480151516, 0.6228726027358009, 0.7505086425762284, 0.8110218645835304, 0.45832617579373003, 0.8458025726818115, 0.7726072751420059]], [[0.6469330802297091, 0.8243962566811233, 0.3387111361748546, 1.5055033800040931, 0.3231410864988349, 0.6926904702037766, 0.4374537450664456, 0.5679043089046594, 1.2052374929993215, 0.9153245951518056, 0.09196627947804223, 0.40355867142991175, 0.8318679880984472, 0.48794992713682706, 0.3347263838599736, 0.3654746144171389, 0.5881055865514091, 0.8190912799203645, 0.7230672252416198, 0.2289227187012149, 0.23798653242996032, 0.4072375870044319, 0.040384473382085576, 0.1377074814746579, 0.2980455498751702, 0.6350525883351621, 0.38056473631619586, 0.6498127669227504, 0.34448812671395235, 0.45791082209875356, 0.6903739659882195, 0.7929209027759744, 0.4427439841650874, 0.7821736925984908, 0.8279612788376818]], [[0.645685189920107, 0.78780722458308, 0.36576098166885046, 1.467108951229575, 0.33297325462032634, 0.68447035901705, 0.4301559839829266, 0.5990114515945044, 1.247191915171657, 0.8464362463784869, 0.09003908564178113, 0.4167230650455087, 0.7113509238804288, 0.48589621118594867, 0.3487194627342841, 0.3639207056289063, 0.5787308695083505, 0.7845662544590152, 0.6549317951801883, 0.25267927609158747, 0.23515681434735175, 0.3991597994449225, 0.04325153806819332, 0.13422903799538055, 0.3160477803714637, 0.6318314519952788, 0.36712763942868926, 0.7194730821596993, 0.37249708943387067, 0.4949845722762741, 0.7647722489335025, 0.7647181210120213, 0.48364148167285664, 0.8316115331800036, 1.023494102131502]], [[0.6368787611869379, 0.7917789716477882, 0.4111859538781609, 1.499049068677561, 0.4143772598999532, 0.6572208959818179, 0.4238537573181975, 0.6077836298122958, 1.3758887569851452, 0.9326292971905881, 0.11699009468468863, 0.39640541564655085, 0.6650062077743832, 0.4598338619910335, 0.36487685454080054, 0.3402779048743324, 0.5700583217329427, 0.8950870538783364, 0.674981070599682, 0.231667320830992, 0.2638036746913257, 0.4130619858541797, 0.04619407514141842, 0.11832297037611861, 0.2602008052741597, 0.6605014635036237, 0.343181091651673, 0.7007305683222244, 0.3907294345317015, 0.5734590743165552, 0.9141166634413431, 0.701178221533277, 0.48337780790559715, 0.9336020762308219, 1.1812802917126106]], [[0.632034228408024, 0.7407414901875936, 0.42080345191603785, 1.3788662621978638, 0.44817270436688766, 0.6638786482361025, 0.4236589227029976, 0.6238414196656878, 1.0766728159734718, 0.9287161969076096, 0.09548462024079911, 0.4239187868239631, 0.53029988082848, 0.4249971345827939, 0.38258588212127914, 0.32999645413310974, 0.48507120759081584, 0.8980344244474627, 0.6844272018143083, 0.2961350179399832, 0.28743003189414523, 0.44231753304613924, 0.026421815409291005, 0.11461216440981026, 0.19373732365293492, 0.6960305517216542, 0.34358626539243553, 0.6814269485313181, 0.4095755893299601, 0.6451519002833112, 0.945633430063207, 0.6626041777154252, 0.46767173966479336, 0.8651199964466902, 1.036595186171798]], [[0.5478978995791484, 0.6602497851391576, 0.41855134668436816, 1.2221229136917855, 0.6279725822876571, 0.6643082721311657, 0.43951262977687205, 0.6415563246906602, 1.0328786612467242, 0.8961775557618409, 0.12198888242499303, 0.45041211395259884, 0.329363637820662, 0.45410719649291487, 0.3486787941280692, 0.31874845012095754, 0.4164822568568529, 0.8486719579471869, 0.7494144795193554, 0.29151889946889437, 0.3204851362250483, 0.4670583111053097, 0.02933436188795467, 0.0947200714384327, 0.18352749093018858, 0.7071374950988862, 0.33861557004359977, 0.6316596868111932, 0.45121045448173197, 0.689376465010764, 0.9549309863426685, 0.6416159010762398, 0.4601364459039108, 0.8226133573189587, 0.9279188613445785]], [[0.5081872588453578, 0.6239791571558805, 0.49756625936370213, 1.03866790249435, 0.9183758995764711, 0.6380147532484959, 0.4815458507952086, 0.6685926606010238, 0.8921280780546572, 0.8627245572928006, 0.07128128831812358, 0.4676749285592636, 0.23247525780078865, 0.4533598023998355, 0.38908977593462507, 0.3081314466977495, 0.46173261497989804, 0.8104880853707546, 0.6154354568509879, 0.2721864687067146, 0.3261382801716949, 0.4783619522950988, 0.017063909863255993, 0.09673549877630247, 0.18390687218864324, 0.7258326669354118, 0.37414974233203585, 0.5857121024015547, 0.5074421252602884, 0.7051241799665617, 0.957909501013161, 0.606356044404186, 0.4954564547557798, 0.580947636777041, 0.7691927054159992]], [[0.46198866946228956, 0.5660138044258627, 0.5186035498475108, 0.9050295942958424, 1.0255153954869403, 0.673988885217616, 0.4677233132090008, 0.6675058301372073, 0.8669177941847415, 0.8269024169162371, 0.09195482387286119, 0.5287020291076925, 0.17686419574846973, 0.41747557668637647, 0.4202788199186515, 0.3414846075480442, 0.4540529696908022, 0.8488612729448652, 0.588226272001105, 0.2134302095426085, 0.3362875076034388, 0.49148088862559974, 0.009867284584679692, 0.10354388681048912, 0.21809718446556808, 0.6952897092016903, 0.3692787311320417, 0.5540965669025079, 0.5329557864173431, 0.6905754582099761, 0.9244225497746448, 0.5637832301158032, 0.5177207093958709, 0.42183435870622765, 0.5748875587542158]], [[0.39660666675296263, 0.5832122478956532, 0.5007637218830909, 0.873021784961359, 1.1410108461730608, 0.6782805644532695, 0.48830952929501426, 0.660434391678029, 0.8162313968472537, 0.8371204156098511, 0.07673620193091642, 0.47856327053759173, 0.18244022796196221, 0.4584346643923309, 0.3749638948087381, 0.35771846184509176, 0.5227297873201777, 0.922775972479319, 0.6649101893034302, 0.21196342445900485, 0.33423626583159566, 0.5016350435929119, 0.04446469754907548, 0.1248106292841582, 0.2593159874222423, 0.654710912800792, 0.33728392631085674, 0.49578358324736194, 0.5537840043658392, 0.6837265805781381, 0.905320234759069, 0.5696992469665407, 0.5392842738968219, 0.3943272016623346, 0.3687965003843521]], [[0.3750044192389521, 0.48496327561009445, 0.5725197570384303, 0.9718108816752123, 1.244445294526565, 0.669237278905106, 0.434494322382211, 0.6633288964082826, 0.9020402743296643, 0.8611038004554628, 0.07936169090463562, 0.4768565697472842, 0.1852082448919625, 0.45847208290302766, 0.4161929323656319, 0.3723303959931971, 0.5557937338603828, 0.9326714025900295, 0.6500368376855445, 0.21931685631607917, 0.3692368832216163, 0.5454304493968537, 0.017913352922863612, 0.1384881029370083, 0.3508853745515974, 0.6772068614585389, 0.2733133789595879, 0.43442316716998, 0.675030998151249, 0.7389599708784051, 0.8541581368405832, 0.6486442759707012, 0.5941825750255149, 0.4156596508933008, 0.24339081429654816]], [[0.46393126688800373, 0.5227901763397357, 0.6669104575026383, 1.1570341540024718, 1.3003473221755073, 0.7217490152559007, 0.499060816382073, 0.6633224017734217, 0.9368321978227203, 0.9103716794458634, 0.08889298421708614, 0.4793196155562815, 0.23662146313553828, 0.4731427880984833, 0.433412674233201, 0.3589077762793894, 0.5263004348799053, 0.8735530845624304, 0.7910152529979411, 0.17663584312694502, 0.3148827368231597, 0.704031583025416, 0.058637806392502434, 0.16110786438385394, 0.48546909608068156, 0.7541558741174599, 0.23712836936468024, 0.5583322043085855, 1.0618139586835487, 0.7485508433069199, 0.8144135222300002, 0.6930466688230071, 0.7270470976696416, 0.5010002712899687, 0.22986659203138793]], [[0.37188085377374386, 0.49630334036326706, 0.6375392276430756, 1.1089177837880404, 1.285966547339064, 0.7340076455094735, 0.4974339148442758, 0.5958497948028973, 0.9747591074544433, 0.9677363247070885, 0.08154839373916145, 0.4944139119909444, 0.28925420049382083, 0.5031168787783846, 0.492569722347822, 0.3331878909282604, 0.5144910352449946, 0.7134072392203623, 0.8163502593686676, 0.22535653386155768, 0.32358362828461956, 0.9053066522992879, 0.038876634610827916, 0.1958069098371363, 0.6637719597817344, 0.7589095371268662, 0.28165014561114216, 0.8195778293818505, 1.4870137152294065, 0.7300991194743947, 0.73120832168474, 0.581908810326786, 0.6513189777978445, 0.45944246334622246, 0.2957326284374967]], [[0.34051451924792125, 0.4944594184931606, 0.6591382218852014, 1.0406685568467158, 1.2758497119575838, 0.696734454957145, 0.5128935733181517, 0.5855324989792726, 0.9296664762402751, 0.8785671543214078, 0.08458229836671889, 0.5043129830567986, 0.30924301322140113, 0.468557003182817, 0.48780263857421974, 0.35876464027348803, 0.502944287986933, 0.7121437796443831, 0.7585387783035897, 0.2543815388386673, 0.3514684382316854, 0.9119550943548524, 0.07888497885174502, 0.27524871349916463, 0.8572667254211297, 0.7487514536848956, 0.2820317119877753, 0.9498901496041516, 1.5393852982678708, 0.686557272226763, 0.5009825308621205, 0.49830825225230824, 0.5800917718890598, 0.46267282866076265, 0.368243945670804]], [[0.40419592329583437, 0.48883826007593867, 0.6751536071577455, 0.8727523967246499, 1.2557631831997418, 0.6794768917095282, 0.5272247867859734, 0.5883473989959436, 0.9136472036647144, 0.7216817868909495, 0.0946704307035095, 0.49376470496834723, 0.32216493446297023, 0.4592560490278829, 0.49290033573961806, 0.3468264488536691, 0.5409118791483991, 0.745644866296663, 0.710296405624943, 0.25022948056270977, 0.3683689624929121, 0.8313922882848469, 0.05067976680990735, 0.33168342537031953, 0.9426197398025586, 0.7340757087540984, 0.2713432128714092, 0.9488862785110744, 1.515611570626584, 0.6738557747467326, 0.31504923772504934, 0.4376243057484791, 0.5215354919853501, 0.4675387444687864, 0.42761740584047925]], [[0.4177650965227432, 0.50863142836081, 0.6411257080059485, 0.6536134548146841, 1.2391814818467124, 0.6893957189802015, 0.5890304210565933, 0.5446322056908255, 0.8997442270013526, 0.6732245471846364, 0.0867958438673317, 0.5168838282138999, 0.30397072024230704, 0.47068908947264787, 0.4854946353049037, 0.33419700847551875, 0.5345396110089019, 0.7227583784694925, 0.7279156454674134, 0.24566451631847597, 0.354384742527826, 0.6488641527236171, 0.053772338775256036, 0.3612724529280531, 1.0872673236710595, 0.6829572147507135, 0.26030414221788095, 0.8905689858829655, 1.3815231882427996, 0.6534781366411624, 0.2487063424858823, 0.4119699085428218, 0.46745714245254133, 0.518970733780015, 0.429219614100917]], [[0.4517305641464373, 0.6661625430351429, 0.6205579906310619, 0.5585455441366984, 0.9894686384708106, 0.6877187534878821, 0.600012501167958, 0.3582453861809071, 0.44146056292497976, 0.3992534082468622, 0.08301729824072782, 0.5256951281987354, 0.3009289858714837, 0.4535986522089433, 0.47538629281247474, 0.36145989793620803, 0.5316250836679973, 0.7072832896424144, 0.7457298378558297, 0.2497671966831233, 0.33256286385141637, 0.4989076511270973, 0.05120334467286866, 0.4427584450584477, 1.2978916140786563, 0.6231984012122457, 0.2737439152332539, 1.070808914700267, 1.3214726779860484, 0.6122175292343079, 0.17569900037877295, 0.4383717465151483, 0.46628929725028334, 0.7033252159708707, 0.44357715945990006]], [[0.4102085689618572, 0.7086453166193303, 0.6253079166930703, 0.5035142929799036, 0.9261244155805252, 0.7322368542517915, 0.4727183187508318, 0.2978566111291596, 0.18233922685921947, 0.24733405545749887, 0.10559552585780149, 0.5503902802294387, 0.3321579663832492, 0.4632094517693933, 0.5029336585827273, 0.32347251397019827, 0.5355324508669297, 0.7023469483885567, 0.8099571040358965, 0.2670726231108199, 0.39917598629076373, 0.3393859062980865, 0.07307800960653413, 0.5805705197713167, 1.34885894964292, 0.610551594382265, 0.40338413899838865, 1.19031711158443, 1.3465062985450653, 0.6207079475933079, 0.18002798869059863, 0.5394315055172064, 0.4947003697478738, 0.8559993901971517, 0.48421906228102396]], [[0.3848851081801705, 0.6904052121640477, 0.5755929982282086, 0.49483024277114457, 0.9978418068079307, 0.6893338561852772, 0.4876889983089754, 0.266611576572455, 0.17778468606566739, 0.18695218038066888, 0.11533399085074125, 0.5370471764412612, 0.3229594907526172, 0.45995332747221773, 0.5066078752760794, 0.3357985267853492, 0.5512178413760243, 0.7820209394713442, 0.7904422573449958, 0.2899210250241946, 0.39403407616500236, 0.29375850553808514, 0.07315004059573604, 0.8430931008705056, 1.4523423126618606, 0.6104240343747591, 0.5890277144749447, 1.1069006671515078, 1.5132626558245097, 0.6535627682364932, 0.24300135617227597, 0.804530090677311, 0.49749840453596056, 0.8771257479798058, 0.5169077885426295]], [[0.37250138504078484, 0.723797670613082, 0.552877741616702, 0.48010097659872386, 1.0450633189568406, 0.6649260145885417, 0.5018158401727444, 0.24469852596397437, 0.25177664358948837, 0.12640834055732003, 0.09441815137205248, 0.511222971198565, 0.32152329157022386, 0.4529162234402601, 0.4834269674562454, 0.4527165260279732, 0.5083210526572404, 0.7661506761038506, 0.7975102921076683, 0.3011756265946627, 0.36103286926796785, 0.300004209908017, 0.07748199268709886, 1.1066583268443038, 1.6529983586339456, 0.6954969386713639, 0.7366171990671944, 1.0463593283917896, 1.6666374627511376, 0.6752916462260483, 0.3256136013775557, 1.0682888673901392, 0.5092088607837071, 0.768235071762055, 0.6017092074680973]], [[0.3321416890877963, 0.7301063108296, 0.4141240755471055, 0.4986939834195895, 1.0530993365061485, 0.6938975418252875, 0.4899452214145488, 0.22388534652973657, 0.41856615556990706, 0.18645831988662032, 0.1282673493071728, 0.5095362859249603, 0.32393829850899614, 0.44102285726606705, 0.48582378734733817, 0.4168315677252916, 0.44224369830820603, 0.7356119084490751, 0.7298672134014387, 0.3110163318848612, 0.36062212098837754, 0.29451651034081416, 0.05527800845090917, 1.2477738887999796, 0.2573255212126311, 0.3172222974930904, 0.6657458151341268, 0.6881818043641114, 1.0510400539584994, 0.6601070853593693, 0.4069698913616373, 0.9712997631296825, 0.5528692951205492, 0.5876140225043782, 0.7698632125505211]], [[0.33548870731864755, 0.7618676062649123, 0.3906691460550325, 0.5533866702464378, 1.092003339742302, 0.6763550183672773, 0.5133745943503274, 0.2473408603845818, 0.5187887357064612, 0.20031720523728558, 0.14134605562866734, 0.5102034753286746, 0.3245811366382537, 0.45909520269657395, 0.40001758128015646, 0.4574850745726008, 0.46853165686297543, 0.7431239497022213, 0.7498245951932822, 0.3280451852432713, 0.36379087827186496, 0.32375713012184787, 0.07811236381098514, 1.1735330918126874, 0.28709894172887124, 0.3057433000495833, 0.6018066527676714, 0.6943146553071922, 1.0648792153484832, 0.6711255629999997, 0.6411988032664552, 0.983490482965673, 0.5790320994608353, 0.7843115303992287, 0.8891516127232639]], [[0.33741266732541664, 0.7210399697283771, 0.3821749940562541, 0.5673911974825165, 1.047735535787339, 0.690523002103226, 0.5111018959955242, 0.4294780253372455, 0.2919403007469208, 0.14327191375507004, 0.1278868012743562, 0.521667895909993, 0.21572573135386877, 0.47585740175959507, 0.4610423828991666, 0.3813008034884119, 0.4131645975082323, 0.7597781336953829, 0.6734289913943307, 0.36379463455949856, 0.322177460954019, 0.3432656729704249, 0.09690219691904953, 1.1727001048616135, 0.3015260868664923, 0.2706766431704588, 0.5809698219624639, 0.5505209826824433, 1.0834934385307124, 0.6494147465239192, 0.6249450162429057, 0.9810934065420005, 0.596043098030534, 1.0226087041257719, 0.9555027835983414]], [[0.33985420141141864, 0.6942977783496719, 0.40712560784307794, 0.5562497863730251, 0.9900235506380193, 0.6733533961947289, 0.5075417165087032, 0.5551536538631084, 0.3656592532698933, 0.11491927574109981, 0.14746160401182992, 0.5149331812239427, 0.25037434452897317, 0.46697138980727504, 0.4074121900155651, 0.3706212534489062, 0.38611310362631796, 0.7642742854970479, 0.6816859412490222, 0.3703382374077405, 0.36123394856699015, 0.35641333567547145, 0.11170701382626183, 1.1454503680910004, 0.3613596010949207, 0.23079607195522012, 0.524554959817949, 0.5637359097400277, 1.0451369173570573, 0.6536439890315925, 0.6663031552414393, 0.9179323566778264, 0.567941179451823, 1.1457614825982816, 1.005782327387066]], [[0.3159737646943537, 0.699073791504953, 0.37498835901161187, 0.5826010480296214, 0.9923193807988266, 0.6720850474003883, 0.49094283796231675, 0.7529550568473451, 0.4857187807423804, 0.13378160248397705, 0.18005863424914417, 0.4976986277807803, 0.2826483616291611, 0.47787908778646715, 0.40647636799986847, 0.37932233286783235, 0.4106656252870785, 0.7820079150908799, 0.6686724870460856, 0.38643808599169205, 0.4425284959403922, 0.3672677805124035, 0.1467584932037539, 1.0249059178986963, 0.3971586171941481, 0.2576944437179931, 0.48636545217184896, 0.6075296489814512, 0.9991892114720942, 0.6858328473921369, 0.8144101242011308, 0.8248319991709806, 0.5279562328879221, 1.1992299479493367, 1.0428998473005717]], [[0.30042102417582695, 0.673552651248332, 0.39104060653309225, 0.6230931726439802, 0.9158641620628002, 0.6999964490587275, 0.43795669659534675, 0.9167726037532167, 0.517898177072819, 0.14662929217620824, 0.10579342867342209, 0.47126011810942936, 0.33178309903590353, 0.44872683975237937, 0.34834555314838106, 0.3961335120426004, 0.4202485108270931, 0.8063325660409042, 0.6337542119369778, 0.4196395171996219, 0.44798727929006715, 0.36701491553012394, 0.1806820198770776, 0.9232305158902037, 0.4696387840042674, 0.305105083564837, 0.4834717334449131, 0.6549719890321426, 0.9593542925728238, 0.6176903079451341, 0.8189502223195078, 0.6420737747255649, 0.4670540053607567, 1.2058567952273325, 0.9980109423133876]], [[0.32987716796513006, 0.7520335823518944, 0.3851144896669636, 0.6611192997274493, 0.7299670777716307, 0.7044698227693429, 0.4368585996663191, 0.9523772741635488, 0.379825974039024, 0.1295901653490934, 0.04487659097280145, 0.48967194427350674, 0.3943685557427233, 0.460146236491916, 0.3517171153260422, 0.4004829200860984, 0.40632475077813207, 0.8098173790823889, 0.639884883262968, 0.34686696531051886, 0.4224932254008947, 0.36539723967668025, 0.09971180380843447, 0.8381735314204617, 0.5355925495896413, 0.3917942857302459, 0.5197399928947373, 0.6678817253355296, 0.9491540580151308, 0.5571029265192482, 0.8932140690530661, 0.5032907696311233, 0.3837593434401086, 1.1732268221293054, 0.8913676573956377]], [[0.4086787094415487, 0.7208796618380511, 0.3407158602379189, 0.7547253141187912, 0.6422069200342794, 0.6933588647824328, 0.46070304510433935, 0.9059517848735817, 0.3480150971021792, 0.12185265512305254, 0.05402288643157451, 0.48346624632386725, 0.4724309129629964, 0.462590750808254, 0.37327017097178444, 0.3811754924885704, 0.4213830108226241, 0.8520206759192266, 0.6351033618007214, 0.3482785638298096, 0.4179760225746219, 0.3857739086431844, 0.12717966954202828, 0.8295321976679287, 0.6084350279055981, 0.4811132839514024, 0.4520520735198936, 0.6718488351197446, 0.9791590148008533, 0.6141215784478581, 0.9595302069806079, 0.4999137625345531, 0.36839455062938165, 1.1599435946134715, 0.8189921410906655]], [[0.5066459156466995, 0.7283514462640006, 0.3262149240611242, 0.9903221491874615, 0.6917277634371188, 0.7058442485683099, 0.45283306806400075, 0.858812224132901, 0.3581582823747327, 0.19520634784479124, 0.058026090550270915, 0.4798021009684009, 0.5531838990956759, 0.455133888595377, 0.38536627121035355, 0.329483425555966, 0.36995944466118713, 0.7070520799014763, 0.6257951876739434, 0.36254341865720063, 0.43457582959286584, 0.3925407479366114, 0.16667698995631347, 0.9542705077046802, 0.6875318582319796, 0.5537226508789127, 0.391345143686454, 0.6313204801405647, 1.0656253905710578, 0.6561494117541966, 0.9775812506908781, 0.5207416078911983, 0.3599638416869258, 1.121003696417442, 0.7115787070341781]], [[0.5022788306831598, 0.7832655529333776, 0.33373997783143006, 1.173653823564519, 0.7241154791960382, 0.6351159645472926, 0.43424113393785113, 0.9208562667820296, 0.334362050239967, 0.2953013719325213, 0.04986690251056661, 0.4902275962822507, 0.6202920432948984, 0.4630994555244043, 0.35905673328287985, 0.3302628845287503, 0.41314002449647425, 0.7207909145962526, 0.6388737844034218, 0.3723417090299127, 0.4783972517757913, 0.3801351050788443, 0.15971487375292096, 1.020267273139452, 0.7390167076177898, 0.5949101295192059, 0.47774792645103703, 0.6146318302192367, 1.129840254798985, 0.7300948165896989, 0.997346117578263, 0.5488360214692622, 0.3909065926932701, 1.0814214311932684, 0.627833836265959]], [[0.4937910550389926, 0.7801148973551255, 0.36452284073536956, 1.4283798341838656, 0.6851580747169733, 0.6552936333893058, 0.46033950175507005, 0.8336451116636137, 0.32975985205646896, 0.3461149084659566, 0.04325855920289544, 0.4792635002561988, 0.6809179565160794, 0.4642674401112505, 0.46689307674244657, 0.3842313740893136, 0.40902606696033234, 0.6623754949039111, 0.6587556822584373, 0.3398900880168766, 0.4191394238264327, 0.36251280797201585, 0.14983013711907422, 1.0139382460232622, 0.7999716511359198, 0.6052698067961196, 0.5147122943917761, 0.5932968941311153, 1.1763834420484565, 0.7096608913330796, 0.9550237651129828, 0.6157979867125829, 0.2696453230179041, 1.0528799643360092, 0.6860781991913272]], [[0.4711225916330936, 0.83184634899481, 0.4214991819617072, 1.687914236238503, 0.6666277488229679, 0.6921357280738695, 0.49572130799397296, 0.7083858461713441, 0.32442204733338054, 0.3678296268957227, 0.08455918178601851, 0.47494018967392465, 0.7962700120322148, 0.461296545631359, 0.49539775433388195, 0.44959810716332127, 0.41912442691781177, 0.6584856175126064, 0.6803775496232713, 0.3814562131046979, 0.4376900829587777, 0.3824453581328501, 0.1778504413541545, 1.0314484831160313, 0.8503073240413237, 0.6025647811830322, 0.5566690558589598, 0.5801726383297261, 1.1800161225981052, 0.7767596145325422, 0.9054078137342088, 0.6891580164821438, 0.3747503427964755, 1.00818569954856, 0.6319552689435253]], [[0.5017369860942384, 0.9473122231851018, 0.44972025884641714, 1.6591862747277306, 0.6559402001337862, 0.6283935623074168, 0.46372282389873926, 0.6586259974501176, 0.3651651802895688, 0.44027623101594693, 0.0854957919918692, 0.4562276100419681, 0.798258963134013, 0.4696560252069286, 0.5043050410081227, 0.5228935358803464, 0.4064387623426527, 0.6707094488648083, 0.7194178817712025, 0.40216913466819626, 0.45785067188181056, 0.3757269110240002, 0.16625571180226872, 1.0245972053755747, 0.8580351507167077, 0.6443892624109404, 0.568569073759639, 0.5672885172100331, 1.2034333556843553, 0.7837390217510078, 0.8337081186829118, 0.7671841867890373, 0.6445445772081123, 0.9863119276806647, 0.5246880040653874]], [[0.7195661573212766, 1.0152758305251295, 0.4645458123982604, 1.8336782627972903, 0.7056901200460336, 0.6732922385367475, 0.5168811126769559, 0.6294381992652666, 0.3357354324184765, 0.4449526358458571, 0.08691511483904983, 0.4413149894543633, 0.8590709885702104, 0.45747459119776807, 0.5620257118522269, 0.6411403044648046, 0.49063316266834367, 0.7218241627258476, 0.7712116902097493, 0.4064567612651595, 0.5037178488696799, 0.3716849060854889, 0.17506596419323717, 1.0100896977373164, 0.8937924326073552, 0.70638477758566, 0.559346897410816, 0.5607163947789353, 1.2162338351456992, 0.7517140008978528, 0.7707911583464904, 0.8221231652019674, 0.687024582952236, 0.9505113036791776, 0.4377752949276787]], [[0.8435764909676817, 1.0418125431480814, 0.4823979048422668, 1.9659209655104217, 0.8552548141951811, 0.6571649203283538, 0.4924688228825207, 0.6236840209021093, 0.32252753414933233, 0.4584954487338003, 0.08999059541690352, 0.4447299983559474, 0.8927337177738688, 0.4448425703444714, 0.6778328247102468, 0.6760788506292251, 0.5237157192076457, 0.698271868914644, 0.7597919167999443, 0.4014496555960906, 0.5387600866770657, 0.3767064289042066, 0.1383596156109305, 0.8937017178787354, 0.9405072837905882, 0.7778791409745052, 0.5648374448682522, 0.5582679066531236, 1.2027540613016439, 0.7488690177012964, 0.7274614762819369, 0.865489713253125, 0.6756839823293085, 0.9115778403169961, 0.43779572564514624]], [[0.8371018451205188, 1.0743193354904297, 0.49114733713174574, 1.9940866003232243, 0.9878253685273327, 0.662986932102615, 0.5082418390011376, 0.6526443184144772, 0.3724661636378153, 0.5284385776612497, 0.12133690018019075, 0.4883976683207546, 0.9554111660589641, 0.49467975222122834, 0.713627467843019, 0.7030926508916805, 0.5163035332180623, 0.6916860733430374, 0.8187668215159842, 0.35177483405868615, 0.5436193460959056, 0.3253574325293198, 0.13115860794631842, 0.7439872503423026, 0.9159454524159927, 0.8065691857550004, 0.5643697611271341, 0.5440408428991301, 1.1970332831557196, 0.7082480553047694, 0.7141387190011423, 0.9088252973722173, 0.6907776261083666, 0.8843057068837507, 0.447068756670939]], [[0.781928812595202, 1.159922657728193, 0.4994432129083731, 1.9524293241334894, 1.092659735466174, 0.6218486284670569, 0.47327713000617927, 0.6850401438864535, 0.4292215970360006, 0.620279870426969, 0.19208222865346322, 0.5112946322810734, 1.081512231223714, 0.5157810223342391, 0.9069720062613345, 0.6957063189181557, 0.5551872031633615, 0.6758051698269555, 0.8485225736760046, 0.421620411202422, 0.543188527166254, 0.2545782390295566, 0.12912711471245295, 0.6113319870328913, 0.8270912745913036, 0.8076351247325673, 0.573466834159353, 0.519044263226656, 1.1972592047428625, 0.6025603188320999, 0.7345081676143089, 0.9217762839144563, 0.6722063547479022, 0.8727680521049885, 0.48365104967600614]], [[0.8667739512954172, 1.1583485843687518, 0.4697612314759873, 1.8350315290941592, 0.8822025579073309, 0.6495337830977389, 0.4491724618579903, 0.6975642444190423, 0.5239668442510464, 0.6834500990850485, 0.09337875528415987, 0.49683140186638797, 1.1599847819844002, 0.525985641128611, 0.9294874308232691, 0.613661622229346, 0.5523579527036112, 0.6655394416881227, 0.8530596392904104, 0.40413360071702553, 0.44055886242954556, 0.2881612943082572, 0.1406703299956421, 0.41944863396423676, 0.7316553702936428, 0.7127084547064313, 0.5053569866008996, 0.510490865316036, 1.1064016981155598, 0.560044737482601, 0.7569196764323622, 0.9883781530245042, 0.6277813747462888, 0.8635618301267736, 0.637135314077077]], [[0.941330052714759, 1.1475342983042913, 0.46695722816712965, 1.7960167093354504, 0.8676855548815217, 0.662373035996823, 0.4645056881305918, 0.7118403451974404, 0.6532473582732672, 0.7350660706291026, 0.09367973224068976, 0.5748233632426432, 1.1679639336305727, 0.604304615856261, 1.0929209597615022, 0.4375594421516439, 0.6014102791869879, 0.6770107015953237, 0.8477080886258432, 0.4273092652703476, 0.31628348279664403, 0.23572317515726082, 0.13541795875059004, 0.3817506620007769, 0.543983154212005, 0.6501696274595256, 0.464391426595377, 0.5082739028437446, 1.1210835809041755, 0.5735276232956763, 0.7806071023079285, 1.0114865779062359, 0.5197773924908375, 0.9831709720487825, 1.0251175541482782]], [[1.009062676321997, 1.1185728121879053, 0.4619166658252248, 1.7772938844113781, 0.863521599004504, 0.644834434775923, 0.4551930726877942, 0.7520650197117392, 0.7945535545899542, 0.7554836918071753, 0.09411621007040229, 0.5548908025121162, 1.1692775959025845, 0.6337092332352499, 0.8652276193602909, 0.4445253713472277, 0.5390638860403308, 0.6594729584336927, 0.8280899816623686, 0.394609310719384, 0.2168160069063096, 0.2049155461697139, 0.11778892375045524, 0.2990678719842671, 0.45636738194373183, 0.572308014164645, 0.5310268826554252, 0.5151356617604796, 1.0966126691425127, 0.5627207565023479, 0.9282996033044758, 1.0297930410432181, 0.4679129762504538, 1.030232162781993, 1.3529449180056412]], [[1.1707917413717193, 1.0955844933107781, 0.39987423327373883, 1.7189587130180517, 0.877206276570692, 0.6436323001860049, 0.44664425638358585, 0.8684771764744649, 0.9206135469121943, 0.7697136731975178, 0.09547202384416084, 0.4974871916485966, 1.2465824602474942, 0.6374683067730846, 0.937917107612307, 0.4437216451280245, 0.5536172547706617, 0.6674234275650242, 0.8497501449399545, 0.4310554480232923, 0.28722967449608106, 0.30097942164077696, 0.10289725031422622, 0.3342270957902864, 0.3787659676727443, 0.5799007657539325, 0.5112980030725929, 0.5071154825920207, 1.0219447745576713, 0.5112614232430196, 0.8199325297114841, 1.002184481909361, 0.4334368581825554, 1.032624082215616, 1.349484163021109]], [[1.4050269219238438, 1.035890004180572, 0.34046945723478605, 1.6943535955842297, 0.8918106157368834, 0.6506560734352687, 0.399890001947226, 0.8772305536385817, 1.0528537740172736, 0.7794780724505155, 0.10765628433186897, 0.5533908809377825, 1.2174989435330146, 0.6339054006731111, 0.9764446128921191, 0.4696675116660951, 0.5652719772433673, 0.6849356499889986, 0.8235861416143189, 0.4845170275331706, 0.2461472104005658, 0.4437246287430678, 0.11850654484764846, 0.34649357204460535, 0.3570678908341849, 0.5981303981397788, 0.4455330553855229, 0.47656508344723814, 0.9816218378488393, 0.5002275199401517, 0.655792258687055, 0.9847555887071419, 0.42991166716988816, 0.9086370776685627, 1.0484159423019714]], [[1.5144892946370154, 1.0237182113248309, 0.3066434469551592, 1.7177266125553274, 0.8931063594941772, 0.656124772243089, 0.397231842606446, 0.950526250806734, 1.228132900053785, 0.8021431642928242, 0.13901500389643753, 0.5281179323025527, 1.2205946694419276, 0.6157070637202249, 0.8798167215130098, 0.4311711001727203, 0.5006634982290169, 0.693967589821411, 0.8320527648936817, 0.4545422590209103, 0.26779923738236894, 0.47156000742459137, 0.08832500747539698, 0.359580786387458, 0.3134411349229954, 0.6138057946013653, 0.4712932215007853, 0.43220922045913057, 0.9547868332845518, 0.5677193500583613, 0.4726087314126647, 1.0124463484021506, 0.46934981884068544, 0.8080399362563384, 0.904252710901958]], [[1.5278832527160766, 1.1653763843859526, 0.33255325462098684, 1.6691437756496923, 0.8796675477956957, 0.6669480974207574, 0.396990967028245, 0.9409993284185342, 1.327958294936621, 0.7731312683388807, 0.19983276387047455, 0.5238009529562417, 1.195177030564107, 0.6235492078317094, 0.8762037353675503, 0.41117252780154856, 0.5142331029815272, 0.6928743333513794, 0.8377931213451613, 0.3973297008371184, 0.28310576244773034, 0.484005310736254, 0.10026278681268315, 0.3499622908882204, 0.2966019262521274, 0.6213316539054656, 0.4414529780471639, 0.4068478889918459, 0.9018944010111558, 0.6625101647679046, 0.313771342097478, 0.9989515858937075, 0.45877976154320615, 0.6565129138524746, 0.8662570969216761]], [[1.5390887317134494, 1.0559323664171036, 0.30040716433020587, 1.6037643936374615, 0.891183517196777, 0.6425198411157119, 0.3909824448348989, 1.0223149618372867, 1.4538406984035233, 0.7473994246179343, 0.30913641783593493, 0.4944335216900746, 1.2533331261596836, 0.5736140029911267, 0.7197779329652512, 0.41032799943038123, 0.4633836052029972, 0.6730822572504511, 0.8348670839058716, 0.38065875011705874, 0.1388986013624367, 0.4379652018627847, 0.09478978847399136, 0.31690248048379077, 0.23942835116049224, 0.6446913458130854, 0.4365736387019686, 0.3689413098439479, 0.8598684086536819, 0.7076183881169436, 0.35166102011144973, 0.9769806423079743, 0.47598785798455573, 0.606006963225072, 0.8560302911686368]], [[1.54746467100256, 1.0124544131697946, 0.30007392496307916, 1.5256210024775216, 0.9199391049562154, 0.6328035741591171, 0.3803653149427404, 1.1226124696823245, 1.4878245477859637, 0.7047484028873475, 0.38742995855198187, 0.4689763532073765, 1.2046903428924685, 0.6297213844567991, 0.6498902629554871, 0.4259137011680053, 0.48074243018475826, 0.6882193730380932, 0.8294299179506064, 0.37836231624438754, 0.1324803212446048, 0.46531616149416766, 0.07016307682514483, 0.3102797894099152, 0.19769924969216282, 0.6550752803221683, 0.4345125852154893, 0.3209527804746222, 0.8391285003372493, 0.7287012187206992, 0.5648058887445799, 0.9612339430347692, 0.5066970845719635, 0.5316289729925743, 0.8805377290755243]], [[1.56778925417433, 0.9919496827636705, 0.2756876251812237, 1.4640169582412632, 0.9397283158011567, 0.6836438310710051, 0.40367208953827566, 1.136995229752962, 1.516408844357204, 0.7328348738862562, 0.3487700819445968, 0.4947849914960968, 1.2024139926736825, 0.6969345041006358, 0.6535713581562841, 0.4634825999247121, 0.45339694931711705, 0.6730986338875476, 0.8366041734270766, 0.4128662594313115, 0.13606313138893195, 0.2933842576671512, 0.06582569924028589, 0.3079022418327223, 0.19791768863866688, 0.6710892276409397, 0.4306323104285461, 0.2689997257340811, 0.786636837261836, 0.7116234752323924, 0.4522167332146425, 0.9124738798276875, 0.5431056314461433, 0.4449660828210196, 0.818262208594008]], [[1.5941440136624343, 0.9538384029643835, 0.27880303166441933, 1.4495545988219638, 0.9467223249924812, 0.6917460547860076, 0.39490438342431583, 1.1309520055239604, 1.5583817816958636, 0.649624626985026, 0.31749126271280514, 0.5521415456898883, 1.1465650469563036, 0.7507260591738283, 0.6427371087293682, 0.4212268219437799, 0.4436171027105241, 0.6418417876503413, 0.8567064508616217, 0.41916939604463377, 0.12515448398662743, 0.25305901831889, 0.11945134096814153, 0.2778478923121338, 0.1981731417191912, 0.6884042871972651, 0.4727599616114872, 0.23572478528860072, 0.7168242319944594, 0.7056539077816841, 0.4037060269989421, 0.8381818167933079, 0.5960036809654492, 0.2962365736668041, 0.7575990097972413]], [[1.6276885066573308, 0.9429329050145873, 0.2910318654284135, 1.3801396780785713, 0.9394094737450616, 0.7164666738833039, 0.3967125882451365, 1.1374207356957449, 1.539813017648492, 0.6479358861964837, 0.29013623582646314, 0.5217121057179691, 1.0440920024579923, 0.7761360998486808, 0.610128637036666, 0.29837869997685684, 0.4687475323971392, 0.5910107411880411, 0.7326012628829093, 0.4250432133339751, 0.13944425081105571, 0.2615571568865068, 0.0893703604350059, 0.25857378117791013, 0.2723149461432822, 0.6270030468021006, 0.48357401336411343, 0.26561783629298974, 0.5820926266309436, 0.7123110469674427, 0.42606164528927076, 0.8114840603709361, 0.6646045072693185, 0.21786061580541166, 0.7115835755827674]], [[1.650110751173972, 0.8781991279228671, 0.29057488307767687, 1.3645822744226397, 0.9259333208796053, 0.6982798903034924, 0.41352785539345427, 1.0652406274654125, 1.5161800422629765, 0.6287099279145625, 0.24976979978813663, 0.5437181340436211, 1.0476336787494844, 0.7536984897400968, 0.6017088573054926, 0.30183964966541615, 0.5095101410204503, 0.5519528218553723, 0.6803900836528036, 0.4635982521246724, 0.14383440229672603, 0.347636679197977, 0.08305737476354408, 0.2636687753392323, 0.5051917733565018, 0.5353864615431966, 0.5191062997926493, 0.3255903464392316, 0.49077131391028217, 0.7143989201238197, 0.5532873800938758, 0.8306467289216102, 0.7176022445954657, 0.3550900630293562, 0.715667258820446]], [[1.64638455006517, 0.8366982741713589, 0.25545374092345485, 1.3057874430404615, 0.9340873274090876, 0.7239490536053459, 0.4312689984966658, 1.0579966738942792, 1.5321412945588413, 0.6078658667294067, 0.2205154827529754, 0.49939332411684534, 0.8295767688996508, 0.7135709336521544, 0.5764268105989878, 0.33164500848240086, 0.4192657413460328, 0.5309426636186099, 0.7668457957809452, 0.5388206412185647, 0.14665207017860607, 0.3257394954360826, 0.10980472940258998, 0.2873501001595156, 0.4801122651160463, 0.4868085803586932, 0.5484283056473732, 0.4674148669382856, 0.39738044182569776, 0.6791092488160575, 0.5422114958908006, 0.7732422718899117, 0.7029930544499636, 0.4402087273441019, 0.70533811708155]], [[1.6528391373390143, 0.9050926621809587, 0.26163117065866637, 1.2658015166791279, 0.9300832724552665, 0.7087167268650387, 0.4193047415682414, 1.187908575820596, 1.4984606580212347, 0.5765494851786143, 0.22478954733883919, 0.4735699502662276, 0.7218718295498943, 0.6376790858057244, 0.5581187075060171, 0.3337957540918135, 0.411941645371221, 0.5296159659234587, 0.7687101463911873, 0.5522445096857295, 0.15468270846304483, 0.27903782358955354, 0.140963900640748, 0.349274914898848, 0.4271551253969402, 0.43126385396481737, 0.6152456246776133, 0.7497151384006018, 0.2453301146125595, 0.67394271682923, 0.5783468458283543, 0.7028647115277968, 0.6908148855041429, 0.5363039076955751, 0.7691800470659331]], [[1.561428934898889, 1.0096528683146153, 0.3135461084058532, 1.1847754704381483, 0.9614206995905563, 0.6950852021615295, 0.3936057785958686, 1.2315334191146945, 1.6266899620665591, 0.5558821300828463, 0.22818703803010526, 0.4797860086706769, 0.7696072165247733, 0.618941855044252, 0.6755697775440828, 0.3451267082563113, 0.46460314066911446, 0.549783019660605, 0.9046208791099808, 0.6536751097183658, 0.18047056083538462, 0.2649569997150193, 0.1645103602200904, 0.4368317888840527, 0.38937994616496, 0.37253556971406765, 0.6732540239004804, 0.732176272430441, 0.15404352503501967, 0.6553146591148107, 0.5920918291719162, 0.6337497598124504, 0.6838132297156079, 0.5559480315060736, 0.8316020236201644]], [[1.5747710281809866, 1.050412500579022, 0.26394276011990925, 1.0855074499987505, 1.0040912886891777, 0.6988295729675139, 0.38356665510893756, 1.2186904481606955, 1.6548970608150337, 0.5396329164821647, 0.21752440099067183, 0.5104648046035474, 0.7680593462755159, 0.6048329134095348, 0.650149329223221, 0.3130561305764321, 0.45318059725851445, 0.5392090627860859, 0.9109158247369809, 0.761051488593594, 0.17741714940863534, 0.2444086639717421, 0.13885806648889923, 0.5410851821383269, 0.33366021777333077, 0.32568225738847, 0.66997639821834, 0.6094925974725602, 0.1315957118622596, 0.6366387165062115, 0.629362772516937, 0.5848422255771861, 0.7052834363005, 0.581350044555188, 0.8624795566739702]], [[1.653090730095035, 1.0715544604602931, 0.2606954522971212, 1.0918648859671238, 0.9900015802911932, 0.6832796638968548, 0.3876222009550817, 1.24104576057589, 1.6564730773531524, 0.5421655947090617, 0.18575406688946516, 0.498338482205123, 0.7511751523378307, 0.6034033959056274, 0.7821958316389317, 0.35117800571082436, 0.4370110381695298, 0.5882697669248388, 0.885487185595523, 0.8035590329428772, 0.16662026202959235, 0.23421765752927254, 0.1427517602191997, 0.7145812950213987, 0.3218710099472384, 0.2777496729797903, 0.7353996157699578, 0.5668408911498797, 0.11741621685906628, 0.6319174243938681, 0.6602655076745371, 0.5048591367839793, 0.7314922986763033, 0.64153094552751, 0.8417155808948671]], [[1.692141294358647, 1.0079347724450332, 0.2716933257664526, 1.0030523807692124, 1.0003692819536658, 0.6942792402247095, 0.3999979354066359, 1.1798171633771535, 1.6888201853611569, 0.5068007123470805, 0.183490279854948, 0.5091900853710366, 0.695366921325004, 0.7290180653133547, 0.718646964761819, 0.3518805345136732, 0.4302374035317368, 0.5973636007252016, 0.8518965867574788, 0.921121589110339, 0.14978388001235649, 0.2517379267465863, 0.15152533022555348, 0.8032753636102967, 0.397364511868571, 0.3507660574264392, 0.6226066649569297, 0.536985209450801, 0.14669532168630656, 0.6395634835849009, 0.6775780348174657, 0.4077479713380252, 0.829914102372463, 0.7915754673059741, 0.8514584620929164]], [[0.8691823700645906, 0.9392712677588827, 0.2682593049549382, 0.9985736565010268, 1.0065441972624898, 0.7189554048572333, 0.3919100747559248, 0.984774586430577, 1.847860542607729, 0.5753221294311589, 0.18896195945071967, 0.5070287911934069, 0.840057248065248, 0.7464275322899008, 0.7217420686740006, 0.35187905678150877, 0.45844893875022086, 0.5402574645781534, 0.8816708600429927, 0.9334212652789716, 0.14693654027563957, 0.2679705222903711, 0.23021978042088304, 0.9422271956431574, 0.4527595428496072, 0.5469820278131284, 0.4985027361432981, 0.45185446139112756, 0.20346468053418149, 0.6519276244176284, 0.6334839850464591, 0.3341631980512049, 0.9315096443243887, 0.8376859128919605, 0.8820765607515679]], [[0.7954094036256887, 0.9684413564474607, 0.31337381841107903, 1.066468493658391, 1.017042829956047, 0.7183123140986453, 0.4333722755516362, 0.8555944763863114, 1.7620328697583603, 0.5033829261000358, 0.19993089826183774, 0.448923694214947, 0.9477123664068255, 0.7787661680676707, 0.7178639050280935, 0.4525161976812411, 0.5386221574329836, 0.5403904146929986, 0.8543073251137644, 0.9943515130251496, 0.12952968551580446, 0.22719145015822698, 0.22021700721568693, 0.9632651727390812, 0.5243403753530388, 0.9661784828365751, 0.4798880685591931, 0.4381770349392222, 0.39560145613610953, 0.6493165333120346, 0.5174393891186938, 0.36763711214064176, 1.0363366009323114, 0.7979065097505476, 0.9604352607078538]], [[0.7219327657807264, 0.9586790149765341, 0.3133099067330113, 1.09435488210269, 1.016503278624335, 0.7546733313708631, 0.46565515814634295, 0.8238136206201458, 1.7151842457278663, 0.47688195554406687, 0.20965455834346294, 0.47028720919184736, 0.9430999877386068, 0.7920230062576813, 0.6433667141461121, 0.44964185250272526, 0.562403141306918, 0.5355728466994223, 0.8470099858023289, 1.0415763047785376, 0.2355939980163126, 0.24520701914396656, 0.210937744853793, 0.9016315378193551, 0.6123273169014078, 1.0422825457149334, 0.5520203914139146, 0.4493344019540235, 0.5993346569419933, 0.664483994775737, 0.5812056534307237, 0.4776142565880671, 0.8706337503498385, 0.7530716062428786, 0.9872779660594768]], [[0.668399620428062, 0.9940487748925289, 0.3108114103887274, 1.0891761534345368, 1.065292089312179, 0.7413157921539208, 0.4616028676586829, 0.8123861703406671, 1.5595317790739576, 0.4680235797676157, 0.2315840333951624, 0.48283359087182737, 0.8770116842528792, 0.7062897760504625, 0.626661359993973, 0.4524458503488673, 0.5444212313801751, 0.563428758164681, 0.8476549167670472, 1.0375155222897137, 0.3236119707857532, 0.18693573054270898, 0.2173059414117817, 0.9466664389008768, 0.6656874927433457, 1.0756974992215194, 0.5828074887504952, 0.4682518077231293, 0.7140728076390408, 0.682117170417027, 0.7427108680207596, 0.5326976819531531, 0.7371047053187634, 0.7854888755168751, 1.0194905736695943]], [[0.6609397536472221, 1.0104769719516766, 0.3092404685397818, 1.1524070092931828, 1.073384829552813, 0.700594028371784, 0.46079791322427727, 0.7846758792433608, 1.4811613655562414, 0.4556006212020949, 0.21308568369521988, 0.5031650428337013, 0.9114556212487464, 0.7111031963922407, 0.5013932005728245, 0.4316720247039204, 0.5932362087162597, 0.6211333499353604, 0.8497112980840228, 1.0361733166609346, 0.32289683391795426, 0.19479253139288646, 0.2196375171441396, 1.0475948900993166, 0.6756056889508468, 1.1530429462481375, 0.43972556882540725, 0.46642167038671545, 0.6713893431955955, 0.6721748221661892, 0.8808360373712645, 0.5236465761079253, 0.7885212483878357, 0.849588317997963, 1.0343929991218883]], [[0.6737969177897358, 1.055361261267972, 0.2913369254298936, 1.2527534599900512, 1.0745721543296574, 0.6522882604342854, 0.4392145369933205, 0.741239862046807, 1.462669583537027, 0.4309814846737905, 0.17408109160255175, 0.46407769061988924, 0.9310105437032145, 0.7057917899733137, 0.4856077997032124, 0.4122768098855789, 0.5914809689109517, 0.5695283944839873, 0.9331884062632709, 1.0243059221217505, 0.3108769194781723, 0.20493866519280607, 0.20443765581453222, 1.0850899787537125, 0.6835110997482211, 1.0480308152072533, 0.3456323907972776, 0.42372792906738566, 0.578390850661177, 0.6623247152085817, 0.8764607859973925, 0.5419079261145319, 0.8563936479553005, 0.904268585105305, 1.013073744451104]], [[0.6666847691123277, 1.0878523630485546, 0.2819400899771099, 1.319130495315971, 1.0588321225971231, 0.6467977089979746, 0.4329483574541023, 0.8050581957256641, 1.3218078499148196, 0.40835652774547765, 0.16125940715684073, 0.48537813634948834, 0.9391714707369238, 0.6839907165016526, 0.48611261725487226, 0.39533175263577824, 0.6997319686529772, 0.5586754838589905, 0.8954943199704412, 1.0566212302124354, 0.3624126221641121, 0.21101351140828398, 0.20080037400965967, 1.120128686768428, 0.6738975794453776, 0.8954300767528154, 0.26709631691845, 0.5116948636666585, 0.48467708406424487, 0.6718635600883346, 0.8514475448090959, 0.6259027786581571, 0.8757512114702302, 0.9874265549001049, 1.009485712734406]], [[0.6349317211466717, 1.1675979738622306, 0.30056438897757326, 1.3492360146891864, 1.0561039504620728, 0.5777750050391657, 0.400482159019276, 0.7943660448542151, 1.1948681715913703, 0.45308181789318286, 0.13776224037881224, 0.49599601155471795, 0.9953120315395498, 0.7176358234123539, 0.4591687036659154, 0.36775770166132304, 0.6454377805118323, 0.49033424471484277, 0.8101082014927111, 1.0939367971600085, 0.4196164892665934, 0.21195428526496327, 0.18482586384008728, 1.1389039677555488, 0.6700689581896044, 0.800497742234928, 0.30575400660834867, 0.5302596130010184, 0.5176295306773506, 0.7035847643404746, 0.9480002742005078, 0.6958724270855889, 0.8779776090068483, 1.0466489643604981, 0.9710451308581987]], [[0.6498557327722577, 1.15723059718856, 0.283138348107255, 1.2842661434020988, 1.084644701815177, 0.6070198016420614, 0.4172442728166914, 0.7954241217293823, 1.4146141172577587, 0.5013247253329359, 0.15494895337207965, 0.5001967941607713, 1.1408061446503703, 0.7262416605902207, 0.4448652038539611, 0.34058920187646213, 0.6950611344874774, 0.5152410311802152, 0.8645170505742916, 1.116287289530014, 0.5023692310385981, 0.18221342843301533, 0.18780823452115242, 1.135427554395398, 0.6681138492664083, 0.6925771231111832, 0.38036838034544374, 0.5270018608328032, 0.5877600769480468, 0.7328794276916442, 1.0899394154965254, 0.8648725705191983, 0.8702565982736804, 1.0833173177947133, 0.959188096806343]], [[0.6721111507400407, 1.1414390101875205, 0.28204406877443905, 1.2843766916319899, 1.099336314716572, 0.6148388182373418, 0.4332564035435563, 0.8134055825872292, 1.3533362535275995, 0.255643021841082, 0.13430449996049432, 0.47732030877016085, 1.1353479851717738, 0.6871651009889048, 0.4342680865196257, 0.3431402977661947, 0.7165662708174124, 0.474647842997001, 0.8737983560959715, 1.0771523773407021, 0.5385975738216862, 0.26389994497914576, 0.1844949914426698, 1.1395738121446357, 0.6397047492769785, 0.47789624175206064, 0.6046853675254122, 0.6259071416267099, 0.57364832335223, 0.7982010815369922, 1.0300472570298185, 1.058751292792163, 0.8597183722820788, 1.1726372594478258, 0.9197641883589989]], [[0.6628731194202501, 1.1563035269616468, 0.29136293044303646, 1.2399982014356088, 1.2077316656402002, 0.5828014046318954, 0.3963272062516735, 0.871883008202563, 1.1112777264821603, 0.42339385257690454, 0.1364888048911574, 0.4798360170116103, 1.2432266588102658, 0.6834693737005846, 0.48325004489500956, 0.329609018927278, 0.6869720402523993, 0.48255554681506185, 0.9166768343051174, 0.9348292391065686, 0.5249936979700575, 0.29217664671653687, 0.12060998022548268, 1.0671166857979013, 0.5921090145878469, 0.3402502549492957, 0.629808671774921, 0.5967446447727194, 0.5843558017225053, 0.8588661461655605, 1.0340418831400102, 0.9834350324819459, 0.8967677074862938, 1.120066582760939, 0.9505153070619143]], [[0.6507516856853162, 1.1604383456996745, 0.27726877004924905, 1.137718972507752, 1.2378441690157158, 0.7244151147717809, 0.3514176167958863, 0.9363501303768043, 0.8180381382647789, 0.5288069040394721, 0.08540758961032696, 0.521508742398477, 1.1508870593867866, 0.7351539654042614, 0.4343579846317428, 0.43761185620426735, 0.7264631350841745, 0.5550499919435804, 0.9919562478117403, 0.7069039000521962, 0.5145567252510379, 0.31476902382836064, 0.13164131830385156, 0.9171151938486671, 0.5870780699611842, 0.3149222391822116, 0.6556839470335156, 0.596987274472266, 0.5826947285907773, 0.8544715514169132, 0.9824501401940832, 1.0050414932848408, 0.8375530033447329, 1.0954522718059967, 0.9512006895322767]], [[0.6496676828554905, 1.1367681248524284, 0.2798452341350736, 1.1412663320069816, 1.2144992999232622, 0.6136303337044123, 0.3502082203325205, 0.852778928255756, 0.6852741749653334, 0.706920394408671, 0.11934058091465152, 0.5421772651038831, 1.0760516182212905, 0.7298857316061861, 0.35727293160711715, 0.5705808630662584, 0.719974442369751, 0.5042989420310456, 1.1576336801873963, 0.6920541300864274, 0.4977132702484801, 0.3508954798593368, 0.09436990126701152, 0.8190706043627134, 0.609359916719289, 0.3077966944859631, 0.7095644373085923, 0.5250831432718783, 0.5826998628029155, 0.9709919899498327, 0.975700740109742, 0.9303514436860345, 0.8072137538921026, 1.058859404612061, 1.045190066190857]], [[0.6367368376093118, 1.1537155327713864, 0.3103148151231708, 1.1380673338083862, 1.189644594789638, 0.6151095215747764, 0.38209214319568346, 0.7284475891422239, 0.6478020310971386, 0.6899178432294083, 0.1634600191501404, 0.5340117653262859, 0.8026022500498244, 0.6771189606182715, 0.3913346317428607, 0.7632767085829291, 0.7479167265385203, 0.5107186626191791, 1.149881020414778, 0.738402329981296, 0.4151688437058194, 0.35649829309729686, 0.12099464825200003, 0.7280682127264375, 0.6292838309803581, 0.3878866950146346, 0.679623190176113, 0.5037602386111499, 0.6172089384351835, 0.8081866265381429, 0.8046883502960183, 0.9311775168794572, 0.7770987359410034, 1.0208033910767635, 1.024281416521627]], [[0.6369590583906883, 1.1135622061799308, 0.2920675956566608, 1.1261762762137408, 1.1884273726662606, 0.6252736191499215, 0.39180431973747476, 0.6714383400147793, 0.6382039533306638, 0.5079423097400155, 0.20017731963409008, 0.5346413074459371, 0.7057235335155885, 0.6515959521358861, 0.3887633892316578, 0.7220062437285243, 0.86080091599244, 0.5161677130579485, 1.1187088403448155, 0.703887200870773, 0.46060354314514534, 0.3589994121819149, 0.10739322403881196, 0.5690417721904545, 0.6296045731482232, 0.4641384050577947, 0.44110934077418607, 0.5832023039866168, 0.6285343829730391, 0.6794489729349995, 0.7830436259300548, 1.0050111952909884, 0.8295024469733076, 0.9582079855021901, 0.7857810397615521]], [[0.639549607627617, 1.0962395748628404, 0.2887891994549309, 1.2751999574036357, 1.1517399996743707, 0.6680329963932707, 0.41983694462358134, 0.5751533666111099, 0.8964030379274244, 0.3907187649885409, 0.2187639550352184, 0.5170625559700939, 0.5741140205576862, 0.6500771645627224, 0.3853519346028418, 0.6951371765408896, 0.823968316066753, 0.4888991431094384, 1.1367576387477984, 0.6310437881642971, 0.4315244335999975, 0.33089625695312347, 0.09309682062731345, 0.5527116240606755, 0.605568874262528, 0.5112116569825953, 0.33283062509146455, 0.601080776537317, 0.6170184366104552, 0.6569629844927478, 0.8425871431109065, 1.065699664155285, 0.8520227232912833, 0.9229788280293774, 0.6067942369813615]], [[0.6418300301415799, 1.0551647389450762, 0.27990850596500366, 1.2474679057102407, 1.1985082668622642, 0.6790197653443657, 0.40306099968960013, 0.421150801493726, 1.2294588153304213, 0.44695245457914073, 0.23872354167986765, 0.507498372696453, 0.5403364558818217, 0.648476789506708, 0.4113270091134107, 0.7140038064022999, 0.7821073319204079, 0.4910635449144849, 1.1143437611859128, 0.5506425809637246, 0.39108227958235314, 0.3117870166278226, 0.06962387981395933, 0.5367895732421173, 0.5206269765523672, 0.4943752400366749, 0.3408739076665516, 0.5930324286020664, 0.6268538574012371, 0.652714816688767, 0.9060607282018492, 1.1175156546601683, 0.8651338595525628, 0.9597678519813981, 0.4866236971727934]], [[0.6235146143990846, 1.045690654631816, 0.2756667046408648, 1.12249494822161, 1.3092503173229009, 0.6769690100139154, 0.4066260773345678, 0.5172408499225268, 1.2623005130579958, 0.3973378292687583, 0.2648635993492715, 0.5264223168769684, 0.3794193753494404, 0.655738508536416, 0.43159741185137546, 0.6529718321417498, 0.7827961484198444, 0.6622885356716257, 1.1173427423061697, 0.48508088009833744, 0.377520879342632, 0.3644292176676221, 0.08182290786123614, 0.45056383961402624, 0.4006161468193933, 0.4282762500802373, 0.33018892356284235, 0.5159467960271414, 0.6474028279158208, 0.634788168511208, 0.967802978059457, 1.1805485684810728, 0.8783151541285468, 0.984903885561614, 0.4019549105404099]], [[0.6075494174217464, 1.0300942474456107, 0.28311532271442524, 1.1240796439738432, 1.3139655582760557, 0.6742250729871674, 0.4609851602420375, 0.636794226011269, 1.230934061158179, 0.4015868351708447, 0.20442569647195707, 0.5249889594682708, 0.5725786240622069, 0.7263573376758554, 0.4460298477585034, 0.6558532186292889, 0.7622007917967415, 0.657607895261964, 1.15792872501227, 0.4301161170026328, 0.33102673737023175, 0.3969992459033095, 0.1317356504108989, 0.36857172792846205, 0.3319291653132482, 0.2376639892503708, 0.3311524558398004, 0.4487008593254035, 0.5048313614437501, 0.5680738312928901, 0.9924088239078451, 1.254251121197043, 0.8933234567519359, 0.9742991198378124, 0.41317378404972394]], [[0.602214351166311, 0.9844780341395649, 0.2845963532356017, 1.1367350751325116, 1.2701608211192883, 0.6626718578901948, 0.41987554362304885, 0.6088558810565644, 1.0393886451967114, 0.4104747219947653, 0.20312019972812678, 0.5055881358355361, 0.7540426984158521, 0.6751240035779813, 0.4917551629799511, 0.6247987901083455, 0.6512057293781378, 0.4926835067612946, 1.1471708837751065, 0.4428873587900949, 0.34563817234576716, 0.4062274425536687, 0.1397213891640602, 0.3457385157055549, 0.28353688294542123, 0.19312545631867523, 0.3535176050469212, 0.4044910036022422, 0.4556465236206724, 0.577252694001209, 1.0196480813008328, 1.322445277938502, 0.9082793558290386, 0.9425518917540621, 0.4527535994045495]], [[0.6067560266791698, 0.9274273570060567, 0.29870843098874483, 1.044892486964355, 1.2841095181958684, 0.6392589984335403, 0.4141923345686911, 0.5152505088650218, 0.8715070920636944, 0.4329559688646316, 0.21778169088205873, 0.488897286256177, 0.8759744757481847, 0.6715985069441379, 0.5438125100033815, 0.6476663689423072, 0.5638425999351584, 0.4899884132446598, 1.1451534341184144, 0.4242958016725878, 0.3511539809358034, 0.4145874767381956, 0.17906176392404638, 0.4301799844202755, 0.2523420207720085, 0.204802249195881, 0.45644323484942784, 0.3626606057457609, 0.5690969891832309, 0.6061436213302417, 1.0246864023788076, 1.3693257430668757, 0.9296942473451, 0.9284885991937886, 0.5165197284535443]], [[0.5909894522474678, 0.9321020451342313, 0.2841459493382454, 0.992170151548537, 1.2865998446100626, 0.6320416351948085, 0.39925681593095674, 0.42974045015652756, 0.7449405418131853, 0.45611045160312996, 0.2504649133727732, 0.9275754511394342, 0.5243142761006298, 0.4311495930473125, 0.4467295159655113, 0.4012611355844319, 0.23692467966029024, 0.2940043003638686, 0.7248392279925182, 0.25916606896336736, 0.3205827293647301, 0.3017695435065642, 0.17065870836662034, 0.5076563604591386, 0.2623535218788273, 0.1714374080951092, 0.6234590382275969, 0.43235299196067944, 0.46921249879134797, 0.5894343221650494, 1.049273690877746, 1.3993684079818118, 0.9131013762922962, 0.9251993881663111, 0.7222983732029564]], [[0.5703601205104016, 0.9169774468446222, 0.2803018484959254, 0.9210187117850022, 1.2899405609871013, 0.6094620917407796, 0.40389391832802496, 0.4181646050413878, 0.7437464881866406, 0.3967843591088419, 0.2672310500817795, 1.0658383094426833, 0.5359823951004555, 0.3914156944949875, 0.45239488616027246, 0.4893644500111869, 0.24727005962821713, 0.31045688026273904, 0.7364522209229911, 0.1677769856139594, 0.27085049750461543, 0.28513401338830846, 0.18802474773976355, 0.6661481339081108, 0.2974745392647882, 0.19765197741169174, 0.6874078745890472, 0.5020864233578225, 0.4751936650029176, 0.5829590287209769, 1.0411599434140257, 1.3965196825918418, 0.900412754846701, 0.9046662197008849, 0.8387356891568079]], [[0.6155232596545157, 0.8309997220111177, 0.29933315027379714, 0.9478731829420207, 1.261779113796396, 0.6128174124510738, 0.4177324881096073, 0.41960220647628005, 0.780001605727501, 0.31514351689706716, 0.27666412395891615, 1.111498427769528, 0.5660553148186023, 0.4000144758742125, 0.45140133287907214, 0.4732575324817826, 0.2643819308204518, 0.30393615215001046, 0.7580510363942702, 0.17910217486505708, 0.25842935097609127, 0.3259619007901915, 0.16884081720656263, 0.746629634940742, 0.3911794054989584, 0.21966223344607413, 0.7103076908731437, 0.4779242119920589, 0.4947628898937851, 0.5768531604034928, 0.9991489709712719, 1.3633147355894293, 0.9172216111122597, 0.8703421103564708, 0.8091615424884457]], [[0.6004712716127862, 0.8711707633359291, 0.30949555730088824, 1.0066592783097335, 1.213991031481822, 0.642497655842545, 0.4292752447548085, 0.424759520731214, 0.7453276182877443, 0.35926612206805447, 0.2960967129534039, 1.1026507230197427, 0.5827667343151803, 0.40471597029164375, 0.41231797593229347, 0.4613957999600954, 0.270291923513537, 0.3044953513360411, 0.7227394107974627, 0.19420234832033714, 0.24409651990579628, 0.3392821509508645, 0.16637173864760826, 0.8390962130808948, 0.3281910238043565, 0.20499184240123552, 0.6850085530236825, 0.44556807449492153, 0.5076686290967434, 0.5297368824056523, 0.8824223401717748, 1.3489122649518088, 0.8931541466202702, 0.8361784814075188, 0.9294428944142141]], [[0.6063373520245642, 0.8822213818512431, 0.34234950700952155, 1.0694314249363437, 1.1669467573804453, 0.6514799984038617, 0.4303875078872955, 0.48718342190087804, 0.7454875034136443, 0.4044912254014401, 0.3057016393049017, 1.1133425236861607, 0.5957374680831777, 0.4157437014823248, 0.39845866497814747, 0.45450754237012536, 0.2534427576932277, 0.3135582515469129, 0.7368232923511144, 0.21379656329118418, 0.23428669297130422, 0.33683640296957945, 0.15993967584028992, 0.8462831229961344, 0.3793418081680933, 0.20128337625801723, 0.6441826416879235, 0.4472816955958406, 0.5387874550535394, 0.4801709742660183, 0.7651613484004439, 1.310766951647571, 0.8724373330325192, 0.8081089839925554, 0.9383922736486998]], [[0.607434517314811, 0.914722920869929, 0.3669594641854855, 1.134958613291778, 1.1174846763637276, 0.6417466051511258, 0.44134479903970003, 0.5091931757988388, 0.7710774530967867, 0.4015352689518559, 0.28384274682872646, 1.1213571530868944, 0.5864962477280335, 0.40935242992119586, 0.32523532509187825, 0.45235722718098437, 0.24543646707965106, 0.3136855994271482, 0.646468262381561, 0.20671666593025473, 0.20555624851714285, 0.33142617756211823, 0.1588913561405092, 0.8647924644730115, 0.3676671237577386, 0.19545436563932184, 0.5487536148663623, 0.3689346481301451, 0.5764903075583606, 0.4136205946348889, 0.5654225954224064, 1.2566914315277415, 0.877885008621073, 0.793250545997001, 0.9131921007291255]], [[0.6358573219375616, 0.9038542564184577, 0.3364848266563612, 1.1314424398928145, 1.081681154274989, 0.6638012914024094, 0.4523917987837064, 0.5542785952286603, 0.7807170084175643, 0.39248836848471746, 0.2971035930511644, 1.1268678380106105, 0.6065136115505436, 0.41813400746904844, 0.27877244682558255, 0.4752038593571589, 0.2446965823199196, 0.3179279666982681, 0.6456842450662981, 0.1495965095244337, 0.17587633513111245, 0.27358456780692336, 0.19308608365405233, 0.8900052862441115, 0.47959518297697734, 0.19792197397840255, 0.5130201778048435, 0.3664904489660227, 0.6128880480085914, 0.3757630900326363, 0.4154599114009949, 1.3054914573851675, 0.8587136265807066, 0.7589957774573419, 0.902015047145064]], [[0.6471066212404061, 0.9150290362418312, 0.29246494867238754, 1.1461533251931442, 1.021646231428048, 0.6566399870202919, 0.4554495587686824, 0.5252995360889398, 0.7813561332460971, 0.39457384514308286, 0.27905918967765825, 1.0964271749750971, 0.558383032683449, 0.41356547097202806, 0.28478898427315325, 0.4526027109587779, 0.2622184435000526, 0.322459424191587, 0.6633038165824909, 0.08255945306744356, 0.14508483163415528, 0.15552659459653204, 0.1793032165756442, 0.9048565109798115, 0.5310544929673086, 0.1785250681682633, 0.4821475656706584, 0.47708271098775856, 0.6544150002722221, 0.3675740154744308, 0.31280162550516427, 1.3156025882762439, 0.8629849642091136, 0.7657071145613965, 0.9033409480046872]], [[0.710495790535803, 0.8813680798868793, 0.31606627884415445, 1.124166276343248, 0.9931290069623155, 0.661713128211393, 0.4380871711422476, 0.5683760556981962, 0.7251595260130196, 0.45752032710126217, 0.27168828917216575, 1.0666304464619296, 0.540965256391349, 0.41458662078350245, 0.27234702527667703, 0.4582697342196691, 0.2028320087980895, 0.3247713122580855, 0.6379154664066605, 0.05988382008103183, 0.14420233919639708, 0.11669241180805487, 0.17609490135540676, 0.8676566733719031, 0.6631301980879012, 0.16861272742528832, 0.5206396851127824, 0.5774230108303784, 0.7164053005621398, 0.34135095493957146, 0.35378938824386497, 1.2535586549635735, 0.8263572952282006, 0.7619138737654254, 0.9321416057936968]], [[2.036947299572411, 1.0204382422395923, 0.3092065653855182, 1.100291237701039, 1.0422948918640769, 0.6280094102400029, 0.42291360397794797, 0.5554458201899367, 0.6756047377681941, 0.482402188548941, 0.24309984504444898, 1.0758588309037076, 0.5394473047569995, 0.4092658232944506, 0.2595501815457808, 0.47474840431800736, 0.19679133719347286, 0.3006178124078164, 0.5856622139831028, 0.04532641698279563, 0.16650002849421114, 0.10089511155270164, 0.17152378515653133, 0.8048091027201212, 0.8709944360771938, 0.13418566038158158, 0.49829703437424044, 0.7032384493600021, 0.8544944197909965, 0.3462573386608522, 0.612169747526957, 1.0809309519545511, 0.76564023138323, 0.8607402514139547, 0.9831328758907567]], [[2.05902610531033, 1.0221188687993459, 0.2999274077916186, 1.1022063442183396, 1.0346237575512491, 0.6172761359985892, 0.4024915127154658, 0.49889363926178476, 0.7156355320372736, 0.45503336024493507, 0.2403351022657817, 1.0825705564297259, 0.536975758980387, 0.41251454783291086, 0.2632611380952612, 0.5107574144777393, 0.1836161734224055, 0.2958738001991258, 0.5620377264496114, 0.053848667608427825, 0.18565928918745855, 0.11064538478434877, 0.16166602221331014, 0.7620138457578902, 0.7962652131589549, 0.12320150120335563, 0.5962378006135307, 0.9332342575857345, 1.0123991572688742, 0.3932080345782627, 0.7249243720867695, 0.9207688019713987, 0.6270240838008455, 0.843505404236869, 0.9695230290874856]], [[2.068678678187087, 1.049053407788138, 0.2797244684710242, 1.039301106883487, 1.0978936234476635, 0.5858703928336056, 0.3910444745402287, 0.46559466154198104, 0.7773141974686172, 0.40455179210677183, 0.24394481877922056, 1.088802296534663, 0.5341042899501528, 0.42168375520829793, 0.27863079899828713, 0.44698887557180866, 0.18077788333649475, 0.2909418146631695, 0.5490227945806603, 0.04968515821933888, 0.20227099721736988, 0.12413968863987258, 0.1678398959542962, 0.7455473521994975, 0.8189677439828922, 0.12695493124283852, 0.5330678516948395, 0.991025653105264, 1.1035477286972684, 0.5012941108644553, 0.743767700032935, 0.7551507155746493, 0.46596349536281756, 0.7601151775741926, 0.9718299488632222]], [[2.0664978929279463, 0.9661747318687979, 0.30614981495550064, 1.1007182948361183, 1.0177070533692891, 0.5913618413244326, 0.38825141778472383, 0.4379942646109798, 0.6737130861982251, 0.36690664589841127, 0.2609528425199285, 1.346085750438042, 0.5153180240006479, 0.6861283096720375, 0.2567297687588239, 0.4884117765921465, 0.23720896655359447, 0.3035576681871379, 0.5399666326826629, 0.05148469476964118, 0.16410381824763295, 0.10911826246261569, 0.16739153036179824, 0.7247591906259175, 0.8636832790165849, 0.1265843239834227, 0.4803286492552864, 0.968286139157677, 1.191577319789535, 0.5481169693528706, 0.7152571027812231, 0.7642265213679975, 0.3591448455546654, 0.6306158484969505, 0.9508206155517371]], [[2.054463845689107, 0.9655653161364274, 0.3106507389189941, 1.2060569083156867, 0.9210579693789485, 0.6311887324675258, 0.39142438865355, 0.44305305686938395, 0.5932871581241499, 0.36167929077901495, 0.26090678670406264, 1.3822889519508947, 0.5123863157757591, 0.6818583937456786, 0.2447873311773393, 0.47448647720942116, 0.2520474154115063, 0.2994841302810808, 0.5365890636354365, 0.0530060980300477, 0.14955801257591717, 0.0929358568559542, 0.16279362493326796, 0.7095526132321912, 0.9224039286891721, 0.12362771384386008, 0.5099397181910553, 0.9475696131972994, 1.2551197422566516, 0.5667467940857935, 0.684953473302008, 0.7712893142965839, 0.31389535519637046, 0.6384515059442903, 0.9139815853166159]], [[2.0207919894376833, 0.9823292709449919, 0.30748024686466, 1.2551920656034548, 0.8172768973624605, 0.6494639547843654, 0.3763899538359894, 0.4409557264989737, 0.6116197724932788, 0.29559143509558017, 0.2561685247581339, 1.401936406208725, 0.5407931330778354, 0.7153315921457563, 0.2389275913917449, 0.5001838517842134, 0.29751499009508964, 0.30403566304555196, 0.5207042877364545, 0.060740932588372536, 0.15275714678616337, 0.08586546060741618, 0.14910615937463367, 0.7362568429538804, 0.9133453267704522, 0.1357489123418802, 0.5435750986152674, 0.907553260207887, 1.2525917257202215, 0.5352094555239063, 0.6566991991342035, 0.7880970236948882, 0.296925519650879, 0.6303461472899358, 0.900750331954485]], [[2.029543578054184, 0.9622521807794053, 0.31485031385274287, 1.2387613474193704, 0.8044264581138468, 0.6505312936301878, 0.3642404744394222, 0.4399864386017274, 0.6795327246527165, 0.3237582370850959, 0.2473189200668835, 1.4335284294198338, 0.6282936119784701, 0.6991106417410882, 0.2607129544480198, 0.47739487590754903, 0.24952188336052972, 0.2995065539480293, 0.505531103389099, 0.08622911619276763, 0.16576814328781617, 0.07647737928120507, 0.1253965590216582, 0.736784817844325, 0.8184024572762644, 0.19689346843148514, 0.5930111329193515, 0.9261050231322642, 1.220775028301783, 0.4465195204306829, 0.676706439732444, 0.7730907142375918, 0.3023091728010575, 0.7195606333115405, 0.8828271265969249]], [[2.0410373097804246, 0.9888436738050955, 0.30956057909324697, 1.1933153818916926, 0.8766773958266665, 0.6629340530715828, 0.37395868820295314, 0.41619669750844607, 0.7549699010841084, 0.4059819126296201, 0.1516373584060286, 1.3210856326920593, 0.5663367522048811, 0.6522096879476575, 0.2403877792092104, 0.37645201975861875, 0.1872050631096691, 0.30671941628177724, 0.5046176003040227, 0.08297254914667473, 0.17805278862364157, 0.07536690973020468, 0.11089801559913752, 0.7355814669491678, 0.7223455569500061, 0.274637688647733, 0.6708432020182948, 0.8851083089550044, 1.126196395586887, 0.40421527889309744, 0.69384375117253, 0.7408002006958182, 0.3466086281509838, 0.7932238612320139, 0.8160114645408569]], [[2.06494169561659, 0.967714233792939, 0.31465509435749994, 1.183545497379992, 0.9363040303246639, 0.6606129202499154, 0.376955162944806, 0.4205680125370405, 0.779666432578128, 0.46249599978098677, 0.09386101327717593, 1.2959611182838513, 0.5278579720726547, 0.6906259352887031, 0.22551407357795072, 0.39237761517330505, 0.18563367630435212, 0.3110055238718099, 0.4826421637628582, 0.07135779955373828, 0.20143015189944136, 0.1015026793836164, 0.11791735966194022, 0.722297078948408, 0.677968809164156, 0.2866340574141778, 0.6727254978550185, 0.7150393482829335, 1.07355392102146, 0.4638282126071939, 0.7612827593570504, 0.7714937087517371, 0.3948060242684007, 0.8102646383945753, 0.7469258969322222]], [[2.092086275601689, 0.999327367058849, 0.29185429859458845, 1.2433180300075324, 1.0055385815818862, 0.6673845119050786, 0.3719563515524579, 0.41315332089843804, 0.8183148604043811, 0.4148854321396082, 0.05141613876206047, 1.242479866112895, 0.5065328857191582, 0.6892358583382938, 0.23298923974284858, 0.38233597318872903, 0.18702413033136955, 0.33251111264808597, 0.49761222579361175, 0.08549480182982765, 0.19724514975203922, 0.10762110985000606, 0.09834303007147935, 0.7071945209733914, 0.5730184435284854, 0.25505999930278334, 0.6627552468788475, 0.4453975756890689, 0.9692526346939349, 0.5355709785431646, 0.8662447161914656, 0.7787844190887345, 0.4346691184637556, 0.7833662476497429, 0.7311817112520324]], [[2.067750915091335, 1.0149375110588714, 0.2990598941464281, 1.1631428511153314, 1.050447644507978, 0.6595197914010971, 0.3893374249849687, 0.431317262549657, 0.8253562838368016, 0.4892586248564583, 0.22044457614816265, 1.17741885282024, 0.38291557877468685, 0.6289873873307195, 0.23466330581898465, 0.37042804760722786, 0.19933093861880052, 0.336413605294695, 0.4948275992739216, 0.08149855614769769, 0.19327216890421622, 0.12445640104297054, 0.20541684996699355, 0.6800044117830581, 0.554052727018574, 0.21773173053569134, 0.5467752344851434, 0.3832043422025684, 0.9347152562816137, 0.6015778842885855, 0.9019610485546414, 0.8132740207052634, 0.5016871277975725, 0.736896425361476, 0.6883361413050642]], [[2.035268672953564, 0.9942077449299254, 0.29962380176969616, 1.0522220938067988, 1.0586852110118157, 0.6264739775288408, 0.3749802740445506, 0.43176116844164375, 0.764579537180949, 0.553985123125708, 0.21960878170388964, 1.2657996203915505, 0.29874256554793743, 0.6034828041127891, 0.23304066012229496, 0.3644767348623749, 0.2018359802865828, 0.3159065508698137, 0.49441083957228005, 0.06413052310446118, 0.20115259136428548, 0.1322614773877506, 0.1893349621325382, 0.6461819797508697, 0.48143498130076284, 0.18904653238886043, 0.48143503523221387, 0.3481033387522489, 0.8825103915520218, 0.53693274630303, 0.9415545986424927, 0.8231504947172101, 0.6176747648392042, 0.7333843618194478, 0.6597453753448221]], [[2.022902496854014, 0.9166873746069261, 0.3138805643846665, 1.0600609031170005, 1.0750679040585158, 0.6664389308699643, 0.3868440605879573, 0.4475121187244547, 0.6251261842958912, 0.5195202990597512, 0.23570125988923746, 1.2585607422890988, 0.21432998679804638, 0.6651800817747442, 0.23887103014086475, 0.3639342694697527, 0.19582518194363197, 0.3271869707999254, 0.477216056744203, 0.05539149430192236, 0.19991830722962634, 0.15842738798670192, 0.1780053650224723, 0.6244799867348741, 0.48918906343836327, 0.18154283225894255, 0.4540117730117207, 0.39690496488602245, 0.8092554184112888, 0.46502424550491883, 0.9634632930052016, 0.7450204928991201, 0.5288823688449056, 0.7368062497758856, 0.5634106731815497]], [[1.993449292728136, 1.0003571754330747, 0.32052984326370326, 1.037589756632837, 1.0409221389809038, 0.6617821770454027, 0.39094804948460865, 0.48417852940455375, 0.610382507327354, 0.4759472579393353, 0.25932036848721945, 1.315731726405346, 0.2081892375763198, 0.7220697093748295, 0.21400947270012582, 0.24118821753175806, 0.19685891233169622, 0.3080397321486734, 0.4475714991726728, 0.054381589396434116, 0.20538979600759413, 0.18511710895317596, 0.17550334173782178, 0.6005612159066893, 0.46801308555336973, 0.1654129698818235, 0.4602009104606495, 0.3925807879659333, 0.7289515417592387, 0.4316903455742691, 1.0130075620035035, 0.7180579747036566, 0.5239686882101717, 0.7543326611132355, 0.5647630659557222]], [[1.9510794249555432, 0.906170701691313, 0.29768287176536173, 0.9948443740455373, 0.9917836398906228, 0.663480987037812, 0.41279592229584383, 0.575595044072458, 0.49801824733102323, 0.44359325023556273, 0.2553020325256341, 1.304267828008566, 0.19029860243554408, 0.6658189548372443, 0.22415570350741357, 0.24280381897207826, 0.19134906415459432, 0.3295983844199001, 0.4244500484345897, 0.054343480313426895, 0.20877384532277143, 0.19485009833058387, 0.1624686526422623, 0.6538889072328776, 0.43449490258259954, 0.176617775641365, 0.46738883552078536, 0.3295767204737558, 0.6541167572592312, 0.4345802531171522, 0.9963141070884403, 0.7144616008205315, 0.5120439211509267, 0.7512056611085856, 0.5701143996751303]], [[1.8747474803486877, 0.9016304244414068, 0.3565712688769957, 0.9937431947377593, 0.9535267950444564, 0.6578267124440922, 0.41692589529128793, 0.6698145080830968, 0.5280301277518531, 0.42521368129436404, 0.22293744591450815, 1.2688605369526544, 0.11932659753732033, 0.6775971164421005, 0.21908579944710868, 0.2516971511738346, 0.21007575040936458, 0.33786142785104323, 0.4429237678007971, 0.09032605474354705, 0.21216517421011458, 0.21940847442204792, 0.18555593559542033, 0.6981031090968135, 0.390279549199382, 0.3172941092607591, 0.4424464916437224, 0.28209240968619737, 0.6130754466927206, 0.4893917507148096, 0.9390218211055037, 0.8109325433016428, 0.5244415258204724, 0.7538308806958058, 0.612477181210665]], [[1.7683726860931803, 1.017360488521476, 0.34561638983466364, 0.8615392829111981, 0.9724052904732755, 0.6373066844583006, 0.459369045181422, 0.71548719081922, 0.5947302379111364, 0.5694439637733404, 0.19539064288755015, 1.2835853322429311, 0.15223484979800034, 0.6466781989638866, 0.219166074421815, 0.2503814311697714, 0.2377656331440288, 0.3424975763960319, 0.43876510132108787, 0.10937189774566097, 0.22084338026921757, 0.25768558279641474, 0.20974538386971464, 0.7754275826269238, 0.32077355008125685, 0.4505273753982693, 0.4186911733812301, 0.24725939974410382, 0.7440208223724368, 0.48949658672426033, 0.8354959463241408, 0.8954437662472416, 0.5453691845170865, 0.7712611070529125, 0.4688390886674353]], [[1.7992497391044304, 1.0237355600644094, 0.31423157679488883, 0.7119043150922926, 0.99073671574108, 0.6064306948710856, 0.4312260715999814, 0.8205586594726155, 0.6752262700411279, 0.6775736483648227, 0.23032307732065016, 1.2358459989637596, 0.15466578971555708, 0.694612677095914, 0.2049196579419414, 0.24444362728022784, 0.21263157588338577, 0.32206165333061265, 0.4601529815527007, 0.11567737254780727, 0.22326452562792756, 0.2902182665261903, 0.2324918546370418, 0.8218610940798083, 0.2605486788866094, 0.5038528298320833, 0.37459756907017555, 0.2414113240707605, 0.8977962439987436, 0.48222377437998293, 0.7600034653380994, 0.9702028641548074, 0.5707132576553029, 0.7940246947167037, 0.3108907284044358]], [[1.7359584894008149, 1.035782983708703, 0.3031376875457909, 0.6852013444967517, 1.0062572109339802, 0.6188901132390867, 0.413509354173663, 0.9264017663082589, 0.6504842371213924, 0.5969305913977979, 0.35386079918882674, 1.2200108722973542, 0.14110968126685244, 0.6181352643840201, 0.2086262159958985, 0.25049008203145995, 0.21145702205299408, 0.32052253266071323, 0.4584041799865436, 0.11891268013681347, 0.23419933496215983, 0.2945058310064472, 0.16161534272815, 0.8167396539411985, 0.27786245232477613, 0.46857061859425975, 0.3676895546051357, 0.24714390394246438, 0.8674462011330804, 0.43812438568235335, 0.7879870867519532, 0.9086409931971584, 0.4422537787019167, 0.8197103488159299, 0.23916801045004635]], [[1.6660739775552296, 1.0557197374525322, 0.29204799854846925, 0.6405098425975474, 1.0199035305684605, 0.6265878923242092, 0.42392701620907836, 1.027320142014671, 0.6487025244700286, 0.5017615457228013, 0.32005657638201196, 1.2575965148905606, 0.10460462090236351, 0.5954712454061578, 0.21112608775944633, 0.25028369571036496, 0.21272178784517187, 0.3283990927183088, 0.48447762518028115, 0.1598511950869605, 0.22102342262739705, 0.4284676612316907, 0.16117378483272154, 0.7991222749844382, 0.2749746692401465, 0.36463684752169934, 0.3585497614244431, 0.2745288280572684, 0.8163865625201603, 0.4544889283646413, 0.8168649251598243, 0.6564267284866333, 0.3513620077547813, 0.8680208333747856, 0.36652200844675553]], [[1.1565538540945453, 0.9658146681643169, 0.26912460847815806, 0.541109508347637, 1.0434560567722104, 0.5748141727184897, 0.4373199135655984, 1.138261283858494, 0.6842205332646449, 0.45501934032892977, 0.30090462050328004, 1.3076612331247508, 0.21255892478925362, 0.5973314590425868, 0.21596474135325436, 0.252944781094841, 0.2157547474289695, 0.34292673390285416, 0.5002622247187576, 0.3631617411866881, 0.3197496778425764, 0.6123690655332676, 0.11257243181328434, 0.7580086668132271, 0.3284366067390111, 0.3269447457917929, 0.36213721584294034, 0.2686599580987592, 0.9614292153671757, 0.45253189616089573, 0.8518230077672906, 0.49159657083127895, 0.2825504343834689, 0.8300901906811177, 0.49918277626923013]], [[1.082377368486247, 1.1528479330213017, 0.2758409768373366, 0.6523119182250081, 1.0775283872946835, 0.6167090699619624, 0.5044363944808322, 1.2244237304462269, 0.6489541945196448, 0.41799762585273914, 0.2703381865369576, 0.7251777551771607, 0.15277946071366566, 0.8716084868263395, 0.2795870122333851, 0.49231001482516135, 0.5091866050933161, 0.6277277794672568, 0.6933206648051704, 0.45352819915438025, 0.45074439531104743, 0.5710696278254936, 0.12324573488293292, 0.6809865206084036, 0.5122172916002535, 0.2463110812476671, 0.37853136449157554, 0.3306435956697115, 0.9503091597812324, 0.4191074470425389, 0.8359341881052245, 0.42486788167833206, 0.26566748774160776, 0.8405847260926356, 0.3067148097275687]], [[1.012833124603479, 1.1435509790029739, 0.2506598655120191, 0.7761812441405227, 0.9738595772631565, 0.6560075448461811, 0.527040766009539, 1.2956643575319855, 0.603655716321663, 0.4619138273510087, 0.2775298403799332, 0.7030349806508029, 0.19558360331445485, 0.7935089320350293, 0.31184149593082733, 0.5126383279844311, 0.5740941810239037, 0.6058962903478042, 0.6477912786068851, 0.5100096666261191, 0.3960127835795564, 0.6417970751138086, 0.044051294313512804, 0.6748757692268499, 0.5326017093674856, 0.2560528943695322, 0.35862507910167557, 0.29242898296602504, 0.9298154004263102, 0.4239093336394176, 0.7812454462135112, 0.39615834817778756, 0.2833738823489049, 0.6104570145189838, 0.2572145674760489]], [[1.0592141512730346, 1.1427884780935325, 0.2247194800184644, 0.826957275546421, 0.9764222038349848, 0.625615910058029, 0.49450047556114923, 1.393638990038673, 0.6113045883708177, 0.4046132267255472, 0.2700096915156672, 0.7225480539587795, 0.21813405273591774, 0.7947037463577215, 0.3333972646325287, 0.513771803768809, 0.5579619451085934, 0.5747882764898251, 0.6204057771736314, 0.5463094745518984, 0.39989295150743454, 0.6989170800770181, 0.0614867440144502, 0.8007266503340906, 0.5189523819936824, 0.24985959332174718, 0.3206470057992884, 0.2482938386219734, 0.8638501497366093, 0.38658542140292157, 0.6635323981465249, 0.39061816046490905, 0.2829471054800371, 0.4791482920378107, 0.23063489966106065]], [[1.0917977970484873, 1.1472115852721958, 0.22120303027644211, 0.8606476309344939, 0.9504081356687649, 0.5766465983508735, 0.3908586013918183, 1.4192696131393971, 0.5838262292431202, 0.385276776278146, 0.20791509798457758, 0.6542681320098318, 0.2643859766654688, 0.7960515288151528, 0.2726039161069669, 0.4023788361400549, 0.542501540388027, 0.5680254721615969, 0.5855060199227551, 0.47380006419914567, 0.37170067467477136, 0.738958153202651, 0.03877653305563357, 0.7885586002732596, 0.458018147395878, 0.25096830092280664, 0.3323906085430762, 0.2751404868663837, 0.8345978414831693, 0.41613212339152417, 0.6026381719354675, 0.3909357725465741, 0.2870082226330671, 0.5496618138899027, 0.23777174729537628]], [[1.0563505724597377, 1.1042362380372726, 0.2584126621455372, 0.8670082662037181, 0.9812409485510287, 0.5727104576668808, 0.3477990568806377, 1.4397331439676055, 0.5499955144980668, 0.37342302485760304, 0.22789299974334504, 0.9474900799041488, 0.29669852461925406, 0.6278559793131525, 0.24737494369030438, 0.40680294378325554, 0.4653216117568917, 0.4242546946558238, 0.49031457674413165, 0.3595109433050471, 0.38455837389641856, 0.6378061628480829, 0.03830612521974978, 0.789746237439767, 0.4458614340652648, 0.23931746781065832, 0.38718281783141495, 0.22723408890451877, 0.7345559440634608, 0.5141812476560759, 0.6038693645143841, 0.44128858465599263, 0.363374152405734, 0.5941248588720076, 0.26019829821326107]], [[1.0651962387107974, 1.0614848056805122, 0.2528799013073492, 0.8873362122381971, 0.972101060718795, 0.5591042245008375, 0.30110350290380866, 1.4655627929903576, 0.5087588130044254, 0.3719054515813078, 0.26223772385218636, 1.0354883832349784, 0.2793187164686238, 0.613255463554097, 0.25380984283286306, 0.40829318397652914, 0.4485352368081945, 0.407976368851091, 0.4969783935562323, 0.3953531489966218, 0.37764280058777394, 0.688720811595202, 0.04463464024382975, 0.7667914416242743, 0.5400746075187345, 0.21613062354482576, 0.4044224495720554, 0.23840876621866475, 0.7080527815815987, 0.5395588801384399, 0.6085328186807453, 0.5010531227179182, 0.4643841980663478, 0.6873084659920857, 0.29763323946125064]], [[1.1072642825111219, 1.0378466635137498, 0.27112151619626085, 1.0666705124024867, 0.9463597550905929, 0.5370847696972134, 0.27011652915228135, 1.4865243158975343, 0.5334991562885503, 0.42301738729753807, 0.31497504655241604, 1.0073896729123453, 0.36153965792783826, 0.6011424542106321, 0.22385562565530542, 0.47849508743516966, 0.4137310956306275, 0.40324157574924707, 0.5039101825106614, 0.3551998558967401, 0.40620175646122875, 0.6680461500618283, 0.06714282775405442, 0.6723069607781615, 0.5514972140169425, 0.18292677069869834, 0.4137728900742661, 0.21732434792980232, 0.6496786492050385, 0.6467758664536789, 0.607272609696416, 0.5447755090672461, 0.6447868411949234, 1.0084620494801557, 0.4381405669134679]], [[1.2056694924410563, 1.1161809656243413, 0.24622560272035404, 1.0569922298417815, 0.7238022340057922, 0.5364194255384759, 0.26215751157044387, 1.5066343414828143, 0.5929665619631471, 0.4728862583741341, 0.34310182570766246, 1.0700546225442633, 0.387688390679935, 0.5994559384578246, 0.21768142467312063, 0.4381665231261714, 0.40533784511690374, 0.3555002517073499, 0.5931313198563759, 0.22375285687295648, 0.38786002094726785, 0.5906728334135813, 0.06387236013017165, 0.6183318870850747, 0.6258497048461349, 0.15401618266902106, 0.4215893115419036, 0.20451590467294958, 0.5254247326603441, 0.6739076397953148, 0.6821777172449353, 0.6231921474982602, 0.8015520436399202, 1.1627739239094443, 0.685723605299814]], [[1.2005695259404348, 1.0310380718461951, 0.22758420357615844, 0.8608037337334995, 0.9277830162969588, 0.5268383226813013, 0.2604248493410367, 1.549854874312503, 0.6069055159518466, 0.5085966089298755, 0.3553357459541814, 1.022536174625176, 0.4415273709673784, 0.597071221970236, 0.2677368001012736, 0.4351088310104323, 0.3958733795888397, 0.3617718599414574, 0.5458489253656256, 0.2412366401237016, 0.3294590549843093, 0.5582146263063407, 0.049631523064297314, 0.5623584530901049, 0.702451133696015, 0.1294053054268781, 0.46932805239305575, 0.2586958088552665, 0.45389555684503785, 0.6429826625044476, 0.7301233473156366, 0.7173145681455804, 0.958702680540477, 1.2237033412431493, 0.9076507771285889]], [[1.193396052637787, 1.0585881469580805, 0.23076137806305025, 0.7679978336336848, 0.9134785897661422, 0.5576463249783491, 0.28533052018061045, 1.4842056259281915, 0.6692421187612091, 0.5149341529615842, 0.38430831572422497, 1.0041961569459479, 0.4810369461566507, 0.6048887045886562, 0.2847653972649738, 0.449348363166277, 0.37663387738303, 0.38442800533132526, 0.5801093268153754, 0.2937148953186138, 0.33450358923393264, 0.4996185088833576, 0.05209753546175741, 0.7559542360698218, 0.7066851863050292, 0.11870490148069174, 0.4959161506511711, 0.26729811429472916, 0.4254416321470672, 0.6017148616620372, 0.7581290664073692, 0.8083095099836807, 1.0739844548386746, 1.2206119932441033, 0.957479528140796]], [[1.1478093372167972, 1.027552073285833, 0.22951337998992521, 0.7964286629965802, 0.8778772677416966, 0.5036691360598937, 0.2441542595483939, 1.4678945545362858, 0.6947743685489496, 0.513169142530354, 0.36372031275451233, 1.038581803281513, 0.5350181571599096, 0.602392997447103, 0.28012637787760414, 0.43262891309704654, 0.3760526784390934, 0.3910142118170224, 0.6002638195831127, 0.3060851341820398, 0.3444921792091649, 0.43368693031625344, 0.04049725034491594, 1.0643879440364548, 0.67144863971648, 0.10641623667676295, 0.48379549626751533, 0.218535818564425, 0.3952896867371274, 0.3267645589823364, 0.8717860106203517, 0.9068969636098514, 1.091532183851687, 1.2047456174633706, 0.8447175339301375]], [[1.1312751012008393, 1.0243933273085934, 0.23800789019697072, 0.7728527644338499, 0.8383152259081488, 0.4637604756749262, 0.19157802427191128, 1.3394958473813001, 0.6878147384778568, 0.4925630289348085, 0.24103168974898248, 1.0657042631709626, 0.5890967278798652, 0.584357889498144, 0.24401926780907385, 0.45005476545632633, 0.44596356129182047, 0.43640843320783773, 0.6690894593040662, 0.27455443640600286, 0.32263721055778294, 0.35855657202449664, 0.029498918507374454, 1.2566825163697186, 0.6110170763290299, 0.1009719846578428, 0.46727296882359426, 0.17895704757548192, 0.40389414520856765, 0.314715133240588, 1.0212251363197062, 1.0055397309289107, 1.042048394819946, 1.1936511373459624, 0.8414821809400055]], [[1.1069296208185184, 1.0934609072595494, 0.23463799034349958, 0.7648851047166851, 0.714257649216741, 0.46195470809623895, 0.19054298468302966, 1.2274668345586637, 0.6829499926800432, 0.4715942264108368, 0.23018884184350924, 1.1225241626699893, 0.5991831970976316, 0.5383356092288706, 0.2073668076807466, 0.4364651786402751, 0.4503855585682863, 0.44691602048859724, 0.7117445131740476, 0.22693719513194865, 0.2664730341507922, 0.3201617362706428, 0.025048195778613672, 1.230005176114998, 0.5516998826618338, 0.08843325066865909, 0.5738408227815002, 0.14270953202762926, 0.47306160817192056, 0.22491194739797918, 1.1381590739973138, 1.1037586569918, 0.8540779512929837, 1.2327477206945268, 0.8558528346805043]], [[1.117576841986477, 1.0649740854008736, 0.2960560563146311, 0.7925371065160371, 0.7289703779412187, 0.4608333265048573, 0.19200422188837896, 1.1812016171598445, 0.7198436605109566, 0.46551864172955015, 0.1868795658211381, 1.0897711347991015, 0.6388955407055212, 0.5308190612954388, 0.22731440985020818, 0.4347251639188884, 0.48518998061974705, 0.4770709923954662, 0.6576845617421764, 0.27627755183954905, 0.2852974661582259, 0.24514916527421016, 0.024795392642873712, 1.1708075222746337, 0.5205689456805799, 0.07567118194712408, 0.647738579410565, 0.15122085543861935, 0.5036470430160442, 0.24076719103721012, 1.1802758003024822, 1.1847112896582361, 0.7203347695322911, 1.2637954189056588, 0.847105673558807]], [[1.0975279773471822, 1.0401404052340135, 0.32539204944829053, 0.8659598592579787, 0.834717646329131, 0.5734942148235553, 0.32093091984250055, 1.1335448502380303, 0.6712254312241481, 0.4414970926023457, 0.17376921871313372, 1.0528559786012164, 0.6728406391829365, 0.5649056295300021, 0.23405825914780937, 0.4407736863195826, 0.5694234125575615, 0.5470045673538217, 0.6701009084600144, 0.33889266763624737, 0.29687425975033926, 0.19073688333857006, 0.02502360983187956, 1.1027957174445278, 0.5609357349095624, 0.0706195573708919, 0.6589244960774058, 0.1719945726974559, 0.5457856265339354, 0.2820137905739736, 1.2285534160738236, 1.2595179902299372, 0.701646272804895, 1.3621286804364245, 0.9642657291554415]], [[1.1007698896716196, 1.0215510139284474, 0.3469053889069327, 0.8979310867150262, 0.825495928528083, 0.5836272244725718, 0.3236470922127216, 1.1256064330248041, 0.5977990456549109, 0.42685471041097234, 0.14411208545604431, 1.1073469140484982, 0.6962661288666752, 0.5690106550060399, 0.23961650175608618, 0.4358896407245454, 0.5689751237184353, 0.5385674925767345, 0.6653496865643856, 0.2942678538591746, 0.2721751072442234, 0.22848702271608362, 0.02461215344836171, 1.0659230730348057, 0.6766439034504207, 0.06496058232594032, 0.72525367173655, 0.21946799650785956, 0.5989911041380525, 0.3486303046552526, 1.281103550262469, 1.2592984366198183, 0.6494453549811159, 1.3983436749224398, 1.16202004589114]], [[1.061785044228816, 1.0491355669666875, 0.3250569818700898, 0.9403529036911277, 0.885052397483488, 0.5813942349138386, 0.3228297840221875, 1.0788550076958312, 0.5947168622163528, 0.45977302345965454, 0.1442563943151413, 1.2229759059416807, 0.733707246728242, 0.5877985786100476, 0.23280102618243542, 0.4300846543684751, 0.6606146448753337, 0.5298186022024821, 0.6823563531816215, 0.29880950236057563, 0.2833177274562404, 0.22714670942079465, 0.010996432058035814, 1.0729150474723619, 0.7110852515680026, 0.0796463938604163, 0.760504260407875, 0.2574620845591928, 0.5849190125067244, 0.43073634738331246, 1.374762918182971, 1.1351862094643774, 0.6871215167280061, 1.3705112604974294, 1.282290418637644]], [[1.0205980455441053, 1.1767689933666914, 0.263717843272774, 0.9548191376760877, 0.9150201259778175, 0.6148068089148775, 0.3735635675063898, 1.0474123303821266, 0.6610175199698122, 0.4795370312331454, 0.17716259329425027, 1.4038657771025944, 0.7225851972320682, 0.5479863305797289, 0.25889242433150417, 0.42483431806013905, 0.9115294425923679, 0.5737404590510198, 0.7263779710774976, 0.2520387621920393, 0.23232794787870567, 0.21370658044907176, 0.008170971366658522, 1.07059587899251, 0.8679053614939523, 0.10764614812959905, 0.8193392577209554, 0.4834314572934283, 0.569617218237607, 0.5064489248522196, 1.4725692932698806, 1.0747922832416423, 0.7326273926171085, 1.3737542806706895, 1.2575454767824237]], [[0.9721306359542887, 1.202390878675589, 0.2689085671761467, 1.0069483766094565, 0.950761149411617, 0.5575961614548577, 0.29331882874449855, 1.0221778550964378, 0.7198039935134818, 0.43392526378985286, 0.14050115559829693, 1.4026667778944888, 0.6453935176934733, 0.5515309819978216, 0.2547046372586896, 0.42644198593441784, 0.9150493667458998, 0.5374267693700113, 0.7235667489341522, 0.29657093881074176, 0.33825004154561567, 0.2543888746317305, 0.012604540583811661, 1.039423321574011, 1.1514724812603747, 0.15203838563406857, 0.7340843592851587, 0.6220781519717468, 0.5891368882793653, 0.5601700544444108, 1.586019221723816, 1.151212492359969, 0.6830146671853188, 1.337896058417807, 1.043179109136794]], [[0.9301746611205817, 1.2286581156901573, 0.2694150479322372, 1.017991531069831, 0.9973812619948246, 0.6110915602716642, 0.353314135473286, 0.9880149620082876, 0.7700002067823647, 0.4630825221174596, 0.12910636163041628, 1.267626885074588, 0.6255002966737044, 0.5609646188033376, 0.23768603915671652, 0.4283748526999014, 0.8042044222276767, 0.5708578145067662, 0.693560297826957, 0.2698790329481736, 0.29881089706276376, 0.28835373163744654, 0.006034691366983254, 1.0463866578264729, 1.4223758394937762, 0.3393142573083956, 0.6688747639512844, 0.7501029911726707, 0.6877634388704388, 0.5964719212887485, 1.5524202454301674, 1.1081187311328162, 0.6944064114792691, 1.1990921128224197, 0.7344452929664189]], [[0.8963684593626235, 1.2453441339446634, 0.2747453116632124, 0.9963587423346588, 0.9961249521268618, 0.5879663413600739, 0.3267687060266189, 0.9940846014101983, 0.7845457223399462, 0.4587704783797921, 0.10382687807634862, 1.2576460027683085, 0.6150916023522663, 0.5780577932620571, 0.2470284434442128, 0.42549825730637825, 0.7943388220142891, 0.5655053681569588, 0.670301300876409, 0.200878433206079, 0.23576243983347436, 0.2933650816023833, 0.011729942845374253, 1.0596707821396802, 1.761730980740305, 0.646244195296009, 0.6429444423116742, 0.8187101932960648, 0.8890948368669734, 0.6505569398654857, 1.3879477272824206, 0.9799574285934936, 0.7191787913596789, 1.0459891834196688, 0.4452465018153642]], [[0.9880409294291795, 1.3157938003248582, 0.27001600065798476, 1.0048680926208253, 1.0002431054858247, 0.6196376794765778, 0.33950915782855007, 1.2578887896148485, 0.9884056558526567, 0.4221065045360528, 0.16304089634835972, 1.3697149450146275, 0.5752876022828508, 0.5518877340667208, 0.24467507099044183, 0.4329043813927669, 0.8716501240876913, 0.5753378940138635, 0.6180488552705321, 0.12781071349695072, 0.21332035074948325, 0.3102946970611698, 0.02001367787710421, 1.0712273258149045, 2.12030183605262, 0.8914234606779539, 0.6463873058545271, 0.8954966807616763, 1.0636154147745107, 0.7637670018590501, 1.2385431527036048, 0.8513907613273456, 0.8074299938614096, 0.9721388187624179, 0.4072572472231783]], [[1.1015291759968675, 1.299552070787596, 0.2723870483794815, 1.005939009112328, 1.0004759115532216, 0.6171487807105267, 0.3369092818043788, 1.4403829370500296, 1.0730176927964106, 0.4992677783135347, 0.17066001634592567, 1.209235337781548, 0.5617295825690127, 0.5707104016444537, 0.31573591733582973, 0.42845335367796733, 0.8399998604840069, 0.5770868460205039, 0.6343321221167548, 0.10824186977302364, 0.1723401065932936, 0.30887478457861367, 0.02353488205892828, 1.0667863615611655, 2.2167329143707697, 1.0749344212224308, 0.6341374822380357, 0.8416025949272619, 1.2592143037696837, 0.8371794526499217, 1.1656643787393857, 0.7374475950285757, 0.9155309321068738, 0.9412559311617461, 0.5245529522024273]], [[1.0735522870913872, 1.270889627503117, 0.2626335470486379, 1.0223882463898233, 1.013523374294472, 0.5665838849405325, 0.2571395779814147, 1.4988378816418533, 1.0204801750452421, 0.5874846732922342, 0.13884936890278704, 1.2349919804741645, 0.5667330640516163, 0.5887260837029971, 0.3656897549847884, 0.4373058739530695, 0.8902074885230045, 0.5855029488217265, 0.6271739435100518, 0.1226368966187048, 0.16263587037159027, 0.26506840424332034, 0.017660737447543727, 1.0449583969254421, 2.186462390714955, 1.080428277389771, 0.5133280029813044, 0.834309941814864, 1.3293304505335084, 0.9084059017176094, 1.1112944031090652, 0.7191103296236921, 0.9550255467184607, 0.9323045292350136, 0.6026482453607793]], [[1.0203967923776402, 1.239782162759548, 0.26142622693007705, 1.033855260578571, 0.8662489259927371, 0.5800213700723923, 0.25207704178582885, 1.4244894284067375, 1.017097586083751, 0.593449448805014, 0.1431335797164187, 1.1834256933195288, 0.5813975510321414, 0.6355969043041425, 0.364718893170659, 0.45243863869258905, 0.8775908797677165, 0.5858009358982438, 0.6765591692673149, 0.13799117208768819, 0.15567217716019843, 0.2874367885695266, 0.016748767282730812, 1.0099260669208647, 2.048898422183999, 1.0721527448712074, 0.4726012942513961, 0.8104155920226347, 1.3651072220477687, 0.921091749355374, 1.0704559850135311, 0.8083189762559291, 0.9861793782262869, 0.9429995088113678, 0.6458921003436755]], [[0.8830998895667089, 1.2432651577268274, 0.27531447102842593, 1.0309745247864923, 0.8583689790895221, 0.5555621850754295, 0.23205247848351415, 1.278277600712178, 0.9373152178737253, 0.5015317509782591, 0.15625424690820297, 1.057780215997837, 0.6322777742236345, 0.6800293419362597, 0.3687384403733188, 0.45446888420807763, 0.8864055510268534, 0.5614984211900182, 0.6804869177203446, 0.14957654467279485, 0.15537788717990375, 0.3268822543129697, 0.019146103185717044, 1.0163448982007832, 1.9163770279868637, 1.1638426619318079, 0.443778695870088, 0.8425634487781082, 1.4041395016146907, 0.893923494603184, 1.0410001794361792, 0.850799557597939, 1.054840136514731, 0.917829900547855, 0.6846300352038845]], [[0.8340865523719416, 1.2311855807866894, 0.2825323106608543, 1.0751698256102924, 0.8426522885318726, 0.5708845847911964, 0.24612440716138326, 1.1379408529942643, 0.9199427111169217, 0.542214426085477, 0.13995863566921207, 1.1311190408420149, 0.6120353874822446, 0.6595595908589771, 0.35643249511423336, 0.40256896257517716, 0.8744923897422797, 0.5683989720502783, 0.7153203294319801, 0.15574794582483115, 0.15855144172574054, 0.3441976934166972, 0.021742646021722156, 0.9986263017532204, 1.696622856514161, 1.1958755312133889, 0.40388858047954035, 0.8030117540685244, 1.481400108142869, 0.8592496414626818, 1.0058182470380572, 0.860217122536647, 1.0704043647904664, 0.9499057083701413, 0.7273924099140319]], [[0.8501087245469909, 1.2689109357017354, 0.29584692875558943, 1.1042404690137049, 0.8375873071620767, 0.5860904326647645, 0.26634397671411414, 1.0870447677656008, 0.9652308623524598, 0.5434625778905593, 0.13742221784688008, 1.120401501208014, 0.5780285885439413, 0.6500145336246279, 0.32681628697969567, 0.4311607425908398, 0.9295809073627694, 0.5554149760712903, 0.7082323109079753, 0.1618654388002394, 0.16866093477626087, 0.31763551767635123, 0.014798111296177351, 0.9441219768790818, 1.6181480544799758, 1.260213658993945, 0.3295017892608873, 0.782476612177377, 1.50992342603752, 0.8259940381236632, 0.9427984654912568, 0.8327916429449489, 1.107932876530874, 0.948138658620056, 0.8163130751907278]], [[0.8904674728460118, 1.3760475840933577, 0.31058648195332084, 1.024393214062152, 0.8233582358678533, 0.5893712098353129, 0.2623725739851548, 1.0746303474981305, 0.8945063777590317, 0.5390389682083727, 0.14901413161151653, 1.1349648877243452, 0.5681436814736462, 0.6295704590785535, 0.3419767904857808, 0.44725025390941653, 0.9076849319939955, 0.5370774254755171, 0.7038425846824359, 0.17738099407947494, 0.17666928527315096, 0.3049780321458217, 0.01829932606361817, 0.8900968339174922, 1.7115290885994063, 1.1392016542441208, 0.2979301082856072, 0.7731194722740417, 1.5210218258505181, 0.7785335320970743, 0.8732999365769016, 0.814723316209296, 1.0113677660523541, 0.9197865202713853, 0.8404995696662212]], [[1.0779346243441488, 1.23732053131564, 0.28838718083913656, 0.9971317089461191, 0.8021635157541037, 0.6378603439143974, 0.29504599242882745, 1.1192772583630808, 0.7775095347473108, 0.5522236410525307, 0.13448943400654645, 1.1273691351615607, 0.6516341476114148, 0.6357125470387206, 0.3489860369630498, 0.44142240073308914, 0.9728221660770694, 0.5572445808546354, 0.71493581730275, 0.19715903975537086, 0.16831185451895736, 0.32050287634651653, 0.030655809803489326, 0.8241677689029014, 1.7943919876249161, 0.9955061837533599, 0.27207799579581743, 0.8003582767662906, 1.4897530712500373, 0.7349869798103034, 0.7930902392253458, 0.8336704923364158, 0.8829187905493325, 0.8770248825747228, 0.9116279739554513]], [[1.2005725497330642, 1.1918643273766172, 0.2837846687630358, 0.8925562449644867, 0.8479507901986826, 0.709977817974868, 0.3239484620718981, 1.121062151174797, 0.7036149547562164, 0.5818880782351773, 0.10013178576219564, 0.9501913844735361, 0.6315061522095455, 0.6423264665069583, 0.33570541127494413, 0.4204209941378829, 0.854941532006465, 0.492708974102784, 0.6761956432768751, 0.20852265286767066, 0.15747048652276885, 0.3254057829600461, 0.07581874940402766, 0.7632213369083938, 1.9515260591291308, 0.8909962731970803, 0.30776765606272627, 0.8723826186280097, 1.4747701194254161, 0.7146505891018524, 0.7282391913223345, 0.8355328364793134, 0.7865469231318998, 0.8367225377349741, 0.9570034647516704]], [[1.2305872638859903, 1.1390079588927309, 0.27999980787247275, 0.9475516969189806, 0.8544072235659976, 0.762945392030451, 0.3565437461364084, 1.2321468013379546, 0.7304892999691732, 0.6416463090622602, 0.09694664412937545, 0.943866463172337, 0.5949113041466636, 0.633545815357931, 0.36842375702733887, 0.42241419245440925, 0.7460854043538773, 0.497432469040846, 0.7346431274955746, 0.20397645776332543, 0.16226802332265494, 0.32159982767231454, 0.06286688346595151, 0.7866327418751267, 1.984993251615994, 0.7400500533365197, 0.38090261947595394, 0.8423425075211184, 1.2784297206575692, 0.6905760016056894, 0.7636430236968199, 0.8636293590482713, 0.7653718559385001, 0.8312742939673683, 0.911298737095402]], [[1.2082708228274255, 1.1131219598240232, 0.26857036964562914, 0.9697516900357498, 0.8595904408268309, 0.8178698139340548, 0.33826723303376394, 1.283869637068509, 0.7390347320683888, 0.6688106956544289, 0.0941430009601775, 0.9382390607585539, 0.4845259921916617, 0.6212332665919353, 0.373407703287056, 0.4252150457080619, 0.7448547582260239, 0.4932943091583676, 0.7409262599417323, 0.2382845064948878, 0.1784536374699413, 0.3511730074953109, 0.03652534757197817, 0.7759519948805753, 1.888075989603387, 0.6226825852820664, 0.40432452635779204, 0.8346135822718941, 1.139221211623875, 0.7354167295026054, 0.7423514016169672, 0.8626810273682343, 0.8408359960342391, 0.8941295569815679, 0.9086380357831793]], [[1.1836552917848682, 1.0897440198055528, 0.268571555413537, 0.932651381057808, 0.8316462859901143, 0.7717496209780106, 0.35179684030418146, 1.2879578198110604, 0.7381197172854046, 0.6512542439039468, 0.10259669938986866, 0.947976838147905, 0.43915917952378003, 0.6301124837072547, 0.356568243514085, 0.4143500805577284, 0.7393504441533696, 0.49558694475263887, 0.8083242472623922, 0.2147899975584346, 0.17203966928909242, 0.34305604247876387, 0.04477053015341882, 0.8554431556505524, 1.8811566411004546, 0.5062443309701588, 0.4443708393279106, 0.8219170107086348, 1.0946278894992036, 0.7563996859122626, 0.7010102132587652, 0.8787884999275473, 0.9103359950996636, 1.0076010477366568, 0.9100872091689303]], [[1.2151632874828797, 1.1242033830536402, 0.2864627924286966, 0.9243517630867999, 0.8309532633952723, 0.8151133675604296, 0.35950592152230404, 1.2107935988415695, 0.8038814063530567, 0.6384942723126873, 0.11805786150481115, 0.6276343934765758, 0.48450623486717026, 0.690318999528083, 0.3711760221694186, 0.41549638133374345, 0.6918397010160053, 0.5555631320465388, 0.8687467517100071, 0.17810487985889528, 0.16796154188132337, 0.39407634591062835, 0.06410697646888232, 0.8500095619819092, 1.8561116931218837, 0.44572280566612016, 0.49720496937481395, 0.7973365906163673, 1.0753725514074435, 0.7567505348705459, 0.6338653091211257, 0.9473499059413417, 0.986708320149172, 1.0987281220812828, 0.9252005087988107]], [[1.292825145945355, 1.170387522320412, 0.2880389411837467, 0.8059907070030049, 0.9173224327144447, 0.8690937838067184, 0.38734222517168954, 1.1498782074690497, 0.8663644721915936, 0.5062493108471077, 0.1336097483563531, 0.5458630361833314, 0.34605848524284694, 0.6626015079571008, 0.38972334540282816, 0.42287906262259944, 0.6765778493939409, 0.5529422341353583, 0.8048676317681895, 0.15288353986705833, 0.18066344368803028, 0.47607983783346763, 0.08649442407810609, 0.7306678891729125, 1.8186085573861224, 0.3985651067530193, 0.5623137999925192, 0.8471145457097567, 1.0728468362945769, 0.7250543065189814, 0.6719335097320704, 0.9782348156092009, 1.062650571029919, 1.1230730639854685, 0.9554864157757725]], [[1.4028598791556681, 1.0351861652477763, 0.26571004744085286, 0.6989317097269272, 0.958266309687422, 0.941175225204341, 0.39793237042801, 1.1311450385432331, 0.89788618494143, 0.42578946489777614, 0.09636099525847282, 0.5772644024203881, 0.3195146885818604, 0.6324522367800753, 0.3873584096226733, 0.451952504104002, 0.6710907192919215, 0.560848827764403, 0.7933202157372059, 0.15111702132082205, 0.18569621388484292, 0.5334376536452434, 0.04248314723531403, 0.7828124659614792, 1.6114885988164809, 0.2630660941657542, 0.6659929367547981, 0.8215517404215965, 1.0239019726631569, 0.6706318498041852, 0.6150108955363026, 0.9671993378907651, 1.0916031379709097, 1.079440814719617, 0.9597128562857214]], [[1.3816209958673298, 1.0121229114529404, 0.2677104172218025, 0.7413583764259829, 0.8626410341767922, 0.9844149818873627, 0.4069175628915324, 1.0490269356604922, 0.9304666859945241, 0.39405350225625313, 0.08356310417454636, 0.5695428982761762, 0.34058314386417265, 0.6342717880018525, 0.3923558014216264, 0.429477589406424, 0.6159821981994414, 0.572344522826693, 0.7895717012016138, 0.16763255009624692, 0.1748490427260837, 0.6077990561076234, 0.044925720195118535, 0.8170186652737664, 1.3670646851435972, 0.2801085050625558, 0.6751684993615176, 0.7101387865202118, 0.9596308951115481, 0.554605570933141, 0.6191109776743187, 0.9293131321228165, 1.0242551985422796, 1.16064509469629, 0.9878192321497142]], [[1.3483732248361564, 0.9641518287385902, 0.27478466348462666, 0.6964067459001824, 0.8229428885375674, 1.0230575846527437, 0.4007422267261197, 0.9891424820927573, 0.8612228140560931, 0.39741667980894274, 0.05561196587090291, 0.5169517566644722, 0.3890040633579456, 0.551331515923046, 0.36270138190950707, 0.4114371103779919, 0.7084369585313398, 0.5851162709003617, 0.8028113882538569, 0.13987308297304973, 0.17282481291629637, 0.6064972046721736, 0.05102648665914407, 0.914686855952056, 1.279173715499719, 0.3023256866142989, 0.6168845633175475, 0.6723158866872674, 0.9217509293656665, 0.5380811092215452, 0.6320951619331531, 0.8687939532262232, 0.9106126234055307, 1.259664967830363, 0.9515131940923809]], [[1.3189825066640828, 1.0035544203120401, 0.2661983201267855, 0.7705324862100716, 0.8183705119850136, 1.0911546171005266, 0.42194027034937315, 1.00868291810138, 0.843052197454301, 0.4407181199129244, 0.03570840979347356, 0.5140442122506467, 0.5198708175603051, 0.528501813466225, 0.32825974846216843, 0.42880884639168293, 0.7768874612454002, 0.6061698494883834, 0.795212214967896, 0.12287134084421625, 0.16337112057308223, 0.5331502164637439, 0.04713560335187185, 1.0806583789600082, 1.223651629334663, 0.30956835318588066, 0.6218651972309872, 0.6702551703917372, 0.9250194669310685, 0.623616954631141, 0.6572768434274243, 0.8883107778691769, 0.8781948175989089, 1.3803547796792748, 0.9308929346549977]], [[0.9873636377125882, 0.8580630901129513, 0.2657558236123854, 0.6572972323346287, 0.6105127434643867, 1.108446278118873, 0.4057756832482345, 0.9559214785912182, 0.8939993854176942, 0.522875055936026, 0.045263000156369554, 0.523435498611611, 0.64058984013343, 0.5160114977665406, 0.33458259423383385, 0.4234211172291158, 0.8539386720251474, 0.666248330770653, 0.7932570245080046, 0.12374883889273236, 0.15216682869283582, 0.45662270280281336, 0.04301063613585435, 1.1142022450830882, 1.278567902310209, 0.2867517602030199, 0.6429111209847109, 0.6224735008252783, 0.9060804973247364, 0.730802175186449, 0.6737460840178452, 0.8763909784021526, 0.852101385500654, 1.426802637870881, 1.0191368683088957]], [[0.9453880931854588, 0.8802829619135724, 0.266117682470342, 0.6187550922744752, 0.42577585417128466, 1.2036527034915618, 0.40100184931247435, 0.8249903451636463, 0.9097392460251568, 0.5872032901273008, 0.053405920678338104, 0.5331873908528191, 0.6438745660127712, 0.5102276116878302, 0.4293829596723767, 0.42747490789383147, 0.8663816270485307, 0.6414683983371032, 0.805454472344683, 0.11942545253157313, 0.1490822056539356, 0.45610223546577366, 0.058887756900936096, 1.1136120650519477, 1.4317705680942006, 0.278048317998461, 0.6619297790880517, 0.5840314135885566, 0.892598242151375, 0.7679495477473576, 0.6850358911634438, 0.873116797524737, 0.8116654969455951, 1.4693719600646038, 1.0803821620997713]], [[0.9693497266346538, 0.9034228054350999, 0.26687792947553474, 0.6319963586919887, 0.38720931855810514, 1.22311433056389, 0.4059961637651913, 0.7951224185321768, 1.0065764949549685, 0.6200940203708672, 0.056432035413933324, 0.5494945227772088, 0.6450959004042343, 0.5081104925016359, 0.4348438936082096, 0.40369358372022, 0.7969998884405215, 0.6499020009543445, 0.7579917230484465, 0.13762340305577658, 0.1609219270639727, 0.5070245588617117, 0.05193568443384211, 1.0747295777222212, 1.7667472808322735, 0.2727637398456304, 0.676302366359764, 0.6489668394428508, 0.9435874959029595, 0.8140872472426128, 0.6804476681776364, 0.7878138786919135, 0.7608121598525636, 1.4961862728824913, 1.0697406914571894]], [[1.0135662668214127, 0.9256223487289369, 0.2680112672727349, 0.654742054075033, 0.3808214766875806, 1.2386446944095995, 0.4068666174043153, 0.7604109194080246, 1.1290923867835905, 0.6435385523656851, 0.08411133203647948, 0.5280159329265764, 0.6644706327150895, 0.5472336436230717, 0.4173079256098967, 0.4018112249343624, 0.7297288188126886, 0.7835022145636835, 0.7727932172299875, 0.129084301589106, 0.17850050641409346, 0.5031379387697974, 0.05642308751937884, 1.0487823987627454, 1.823142965841241, 0.25931276119025937, 0.6981070199447152, 0.6303400573088056, 0.9926061291699009, 0.7270781879602675, 0.6629218653493194, 0.7772980210452574, 0.680843779162648, 1.5457169631686063, 1.020731744828634]], [[0.977429280478131, 0.8899947832112967, 0.2743808225433782, 0.6875725930621972, 0.3494020135793168, 1.1205187907100969, 0.38196194268819705, 0.7364324383681797, 1.1894437609325932, 0.6556450826282304, 0.10099741295866434, 0.5297281862919335, 0.6679889272009485, 0.48780076021815433, 0.45485482898878704, 0.4403349286175019, 0.7134228725698871, 0.7529557083294989, 0.7450948684670917, 0.1506594580792886, 0.1817315786646783, 0.46283304613133763, 0.07042961868435735, 1.0329502609238708, 1.838866622353407, 0.2198901053022415, 0.666226225540623, 0.6306827833669446, 0.9622907467859085, 0.679453596811042, 0.6487352984351975, 0.776055264844374, 0.6618342647938737, 1.5888821736744334, 0.8045544450228072]], [[0.9718338306748187, 0.7585077679225816, 0.27201934349329965, 0.7048764701346188, 0.3436218462845674, 0.9948900233944019, 0.3748607299175635, 0.7266348318187862, 1.290378583024795, 0.6628985138006976, 0.06816175993963317, 0.5610190000029285, 0.6394837597564955, 0.479617697834976, 0.39732360100796965, 0.4619349848689749, 0.6959306205928144, 0.7235821221622191, 0.6705703316223441, 0.20328793037915463, 0.18262162450896596, 0.4544748294304025, 0.06988797565331137, 1.0168861348180385, 1.8286854439138178, 0.2644832447359896, 0.6791562017711195, 0.640034940607174, 0.9315414358253847, 0.6553064024160998, 0.6290499101682026, 0.7719041900679464, 0.6559479791183073, 1.5118281937074944, 0.5824558692127841]], [[0.49414567703982426, 0.7602033522167735, 0.27026645555928636, 0.6267794004182097, 0.3783590573328709, 1.0930907186710686, 0.3753545047556873, 0.7492767930319055, 1.2279906681765835, 0.6799736973632894, 0.0663024345517941, 0.5746617285035909, 0.6869685921326001, 0.5274441004314729, 0.41216651845986024, 0.4669519378319923, 0.6726520297768335, 0.7379430522441542, 0.5948426490926588, 0.2061222246869452, 0.17409775010552103, 0.4458625029421547, 0.04041091389732115, 0.9580936157313166, 2.02025431287271, 0.2940787259471634, 0.7220525068886103, 0.6878662094461823, 0.8967711846014361, 0.6668975538109068, 0.5812400102727346, 0.7574047094831275, 0.637552150281256, 1.5215824772591118, 0.5704829836782281]], [[0.47364003574155944, 0.816704250011392, 0.2773353237904056, 0.6394011104953174, 0.38066434789520925, 0.9968039265422461, 0.37083829620449116, 0.7119086789932989, 1.2860983937841635, 0.7030302839373221, 0.057499664523409016, 0.6014330303719657, 0.7033363041506542, 0.5064112073154583, 0.3343063339296789, 0.4738513560174996, 0.7971669392081975, 0.7526443513854447, 0.7627366094307163, 0.18196177652358356, 0.20240979077378551, 0.47954007823341066, 0.04332404908250351, 0.801692255884383, 2.1710581093094143, 0.3237089220124182, 0.9259942861667674, 0.6834513033519698, 0.9164263857525123, 0.6292823907794679, 0.5642888040822684, 0.7490531509090441, 0.5969670675078452, 1.5643493386777763, 0.5761218905560561]], [[0.4729639158666683, 0.7459320067343391, 0.29362656426463474, 0.6176101220523331, 0.36505689677267505, 0.920142852570484, 0.37343800117960907, 0.7476765241036074, 1.3209453380641552, 0.7180233974932773, 0.05464757040420322, 0.600074599582501, 0.7189403095922916, 0.5418785496673268, 0.37748772863927915, 0.4836475472643518, 0.748241822600552, 0.7213019855611048, 0.8092380450609991, 0.16028427852200208, 0.24582533861012476, 0.4343442446886614, 0.05765057371152748, 0.550399392255417, 2.0893275830907942, 0.35285887296830815, 0.8204693587488687, 0.7551475872819293, 1.008082616423776, 0.5881197482739967, 0.5247371296918149, 0.8051438025295492, 0.5386389686506368, 1.5390945149683968, 0.3936177909343589]], [[0.46225664748971385, 0.7905832709705415, 0.3139229163038362, 0.6339002702812707, 0.381879943049468, 0.866744823282091, 0.36034250393158457, 0.7277799986997122, 1.3670486043969245, 0.722282790161908, 0.06724811933138464, 0.6351443583459715, 0.7459430421618233, 0.520371576509877, 0.4354382442517082, 0.4710921916239875, 0.7367440512377763, 0.7298264866563507, 0.8566279353393297, 0.15828445152825593, 0.1974386750500662, 0.4008752914847318, 0.050627710545152145, 0.5742748838038233, 1.968626931234206, 0.36434931073647625, 0.7495657203924831, 0.7606649963734062, 0.995605088635257, 0.4969395874784292, 0.4544631180044539, 0.8248255609683879, 0.5108599994599772, 1.508597295687063, 0.37679769673865793]], [[0.4586824494878133, 0.7540173740313033, 0.28256613567435324, 0.6887885826526612, 0.3803656059155973, 0.8829156860190298, 0.3566607429266159, 0.7682187195291181, 1.3568208923172966, 0.7226261500822568, 0.06779930912303389, 0.6010109846986345, 0.7859501708274302, 0.5082150523067689, 0.4594189419879516, 0.5151806066624436, 0.6717083100828294, 0.7250587892078989, 0.8380481977802808, 0.16995035189337604, 0.1967122477057403, 0.3841197937031181, 0.055434105068653206, 0.6381454218788143, 1.9458179701391298, 0.3239710201542681, 0.7771215203684921, 0.7097174487125115, 0.9743011124182606, 0.4277732806846569, 0.43938171891877875, 0.7943009163999162, 0.5100520544152686, 1.4673797573981502, 0.5666137594477215]], [[0.4348492277121786, 0.8092013588846482, 0.2793700067638265, 0.8364266332764752, 0.3188077727519987, 0.8851303615863204, 0.35407232254553733, 0.7848078382120911, 1.190779834007506, 0.7168817051919456, 0.09423588756317619, 0.6069639443140075, 0.7947176777974481, 0.59961284036741, 0.46946918439201296, 0.5261948302860502, 0.5690472519275994, 0.7093329248228389, 0.7280122848997231, 0.2038509714146317, 0.19422038864568109, 0.4169204452883934, 0.06134927750288716, 0.6492416955602336, 2.03249845513628, 0.26747180512079033, 0.8147333382094718, 0.726456603133599, 0.9621306217161807, 0.3847928166531955, 0.4673140375842386, 0.624990781215714, 0.5304336116136883, 1.3649695229963323, 0.7240939865244744]], [[0.42152073170465226, 0.845972994684881, 0.3081026949324133, 0.9459802033269279, 0.3428558321686659, 0.9225084697757217, 0.3652274198961003, 0.9651376039133429, 1.0215227588068838, 0.7108593683382277, 0.09281166707692158, 0.6520288686362792, 0.7853189823682512, 0.598820068188359, 0.3908363226187457, 0.5381088662910102, 0.6157854995070027, 0.724801576236222, 0.7906289643947725, 0.20978595494829166, 0.199848058753764, 0.5767924801424995, 0.10935882733321114, 0.6701255735249372, 2.0953615633641283, 0.2841770840428287, 0.7911185001050565, 0.8314965985605705, 0.9419661545214566, 0.3600323354239784, 0.4148944661876117, 0.42776694895915607, 0.5532470583917375, 1.3343961004470886, 0.9271665172704613]], [[0.390231712990915, 0.8758986465170571, 0.26861618706750573, 1.062548475863504, 0.2804701611202649, 0.997702077171079, 0.3545488012725018, 1.0582748297168307, 0.8109092418405437, 0.7173643483325727, 0.09637585037174314, 0.7188207134909725, 0.7963357788532841, 0.6131992445085432, 0.3980759986360517, 0.5279771858947901, 0.5665185785646976, 0.731693998496504, 0.6685263324618499, 0.2115399594198586, 0.18985361936458225, 0.44519870887129287, 0.04975363461465356, 0.6585053801250074, 2.104352164413677, 0.27914502262173807, 0.8264381725984637, 0.9053179284922256, 0.9452630562821608, 0.42514020171644873, 0.37955262717149374, 0.337636687274848, 0.5708361073348672, 1.203091040029673, 1.021974296815197]], [[0.3947616001012903, 0.9027213984778479, 0.26128192768489544, 1.0596140164690722, 0.3171497916201068, 0.8960366055935514, 0.3559008776381945, 1.0836114840990456, 0.7153470838403251, 0.7353169602284659, 0.0962648295427568, 0.7510802591692731, 0.8563265467526734, 0.6550182140565097, 0.3974998878993373, 0.524714947550939, 0.635815268373639, 0.6584301468353314, 0.6871803907596037, 0.23746800636587867, 0.21397290266355082, 0.4768583188129061, 0.0699189076343697, 0.6288595120713654, 2.0896458920974457, 0.2294801602900547, 0.8408888859057186, 0.9018677472081897, 0.9510845703190478, 0.47894192171421446, 0.3639698443980662, 0.45874947073663513, 0.5941170906065671, 1.0706293053875378, 1.0856194969751065]], [[0.35127755269878136, 0.9459423542646037, 0.2641418398960851, 1.124169825918695, 0.32238282107125305, 0.8881769189630013, 0.34841112627223775, 1.069760141271363, 0.7355156987464887, 0.7538731104354975, 0.10255159322842526, 0.9142513045982755, 0.8395650559867445, 0.6137718132799383, 0.40989689957277375, 0.5362385334718947, 0.7680555120166951, 0.6487525829014785, 0.6154626948145221, 0.1365834832479247, 0.220091966592865, 0.24780322353627776, 0.03951211853876431, 0.6306666122601068, 2.0886739744157756, 0.21883733364603186, 0.7817860239570189, 0.9168872438974931, 0.9678892040994828, 0.5833431894466968, 0.374566565582628, 0.34145612192960795, 0.5930599439244518, 0.9599802473859387, 1.077158370004205]], [[0.3265966944076725, 0.9816538408021644, 0.2710425131052805, 1.1908213125280247, 0.3215455955533082, 0.8302483972548429, 0.3411789029535832, 1.0977655051360107, 0.7553468078874765, 0.8194144410269358, 0.09495692475340535, 0.7610467283851794, 0.8248580344290977, 0.6242836353044412, 0.33035935383875437, 0.44967031459928586, 0.7858302858592132, 0.6773502008703947, 0.5564533998839561, 0.09777670526663723, 0.1629240132271381, 0.16723279880909786, 0.04530915250450602, 0.715419824943893, 1.9898775875583854, 0.21810053514033512, 0.8238213351471309, 1.010285875815527, 0.9557277647531405, 0.709614523356268, 0.4690085053477949, 0.38431896860095655, 0.5952685435938083, 0.908744945632703, 1.0428481441767952]], [[0.32039816749917294, 0.9345085103046661, 0.27975104521642635, 1.2257814792807866, 0.3174228611618224, 0.7855220500235753, 0.35328147220048645, 1.168439904004371, 0.7894305280701772, 0.9458093310737727, 0.06706621935263143, 0.7546628369426363, 0.7810553513578522, 0.6009063679625822, 0.3497080012255584, 0.45001152797594957, 0.5768671478844764, 0.6941415977073236, 0.5460855086769272, 0.1165678583195005, 0.14870639430612287, 0.15817005590435246, 0.042137832342759404, 0.7834151789612704, 1.8955672100721344, 0.27457593938602454, 0.796482991458152, 0.9600384040989999, 0.9281210407673881, 0.7224957725209408, 0.5314064748902639, 0.37564634203644603, 0.6103932961000811, 0.7815756168536739, 0.9669764822699123]], [[0.3064080552670793, 0.9536027643742143, 0.31504686623481937, 1.3001382182488246, 0.292933678036415, 0.7917253080718348, 0.3510965471302911, 1.2183405744828344, 0.7992319399677242, 0.9200180883185451, 0.07718177475217831, 0.7094091116348804, 0.7227912705419599, 0.5872656549230035, 0.3368013727723105, 0.4454283756027979, 0.5184292232898107, 0.6828407546192556, 0.5841923774334432, 0.13568748176703171, 0.15729717951040986, 0.16703760573361726, 0.04910958620773865, 0.9171515988442567, 1.8604921467202407, 0.31484927470369906, 0.848865126047252, 0.8803893951090098, 0.8744237912084021, 0.7714556544965788, 0.5506021525754544, 0.34089594220299035, 0.6016495477467609, 0.7157456110645756, 1.040054613951493]], [[0.29325812317920424, 0.9528605801154416, 0.2998476084204226, 1.3201827674004774, 0.3535158994533245, 0.809159684461624, 0.35506035878276054, 1.2431379819249344, 0.8008810069576682, 0.8629765581095392, 0.09285310211118733, 0.7384836359311822, 0.6376768294018458, 0.6417161125371953, 0.29611377423577423, 0.4595306528408586, 0.5440809429904755, 0.6760353982395093, 0.6174766002997896, 0.13446072712871787, 0.15140660001725964, 0.15355686034834948, 0.059733326579608725, 0.9066450356707554, 1.8167965439134612, 0.3379021139936097, 0.8622987468642977, 0.8268031058764056, 0.7639205363903759, 0.7790421266294114, 0.5799183665169264, 0.30047323105659013, 0.6016280796881418, 0.6724342908209051, 1.1255451331563535]], [[0.29615612550554604, 0.9520372497301569, 0.3035528700607373, 1.2775917517229622, 0.5142544276917932, 0.8248343901366963, 0.38770346566920955, 1.2466954878175551, 0.8396731931885895, 0.8576333213830782, 0.10781171164436508, 0.7213234104949879, 0.6042727493282155, 0.6718842503252556, 0.30768324087002286, 0.4146849894587965, 0.49738457955303395, 0.6868837207046301, 0.6612004449804341, 0.0968148650116872, 0.15401494969861146, 0.1441211073801331, 0.08482923029425775, 0.9071140915937489, 1.7420183982095865, 0.35205673842864155, 0.8506889042227412, 0.8051587243031529, 0.6284754680681318, 0.8488555275160656, 0.6474750248572527, 0.29636225543191863, 0.5793584492232015, 0.6047035986542258, 1.1568435633999519]], [[0.2775109300742622, 0.9401742890967322, 0.30878372328906917, 1.2082576083060008, 0.5463846868157627, 0.7987164406489962, 0.4077990904090123, 1.286599877557764, 0.9264271225733459, 0.9066499389287441, 0.10071573957903013, 0.7270779304573638, 0.5709271085043514, 0.6740367286308365, 0.29400886526105696, 0.4261607190775393, 0.5357606093373138, 0.7047941105363137, 0.6681992416254094, 0.10832148610181863, 0.15395224699789473, 0.16464601950778046, 0.07352116033425683, 0.9507789685805252, 1.6585927441053971, 0.36019014689492, 0.8108252899688105, 0.8068132850755333, 0.5526128904012151, 0.838099198696818, 0.671895631716031, 0.3827332174944188, 0.5184501096649213, 0.6211410818436562, 1.1410775091106355]], [[0.27372702526564674, 0.8952970773146784, 0.313476401267168, 1.1920634860339354, 0.5839675991952038, 0.7863211151225737, 0.38724832750158966, 1.258146770126942, 0.985164996910844, 0.9605655996169871, 0.15189805014307167, 0.735593111991931, 0.47944766667903094, 0.720304564230219, 0.2955059212040178, 0.41003247715046875, 0.5976237112304374, 0.7136390422113698, 0.6307930885003042, 0.09386241464748973, 0.16188512073387226, 0.15526498538636774, 0.057659551887727895, 1.0837275257066488, 1.562902612153777, 0.3738870006519913, 0.7581782514341193, 0.8624051701267201, 0.48498799931468867, 0.913161698146641, 0.6293761635601236, 0.408258528413174, 0.5137486679500903, 0.7171524856214941, 1.1454673640616277]], [[0.2563325921741925, 0.8950819770315398, 0.31814366392444937, 1.1647591430455473, 0.7047857891339869, 0.7488089508727334, 0.38376901365967103, 1.2192437421607432, 1.0572997412311178, 1.0581997916390287, 0.14713210187099018, 0.7116528359624832, 0.4189998449410137, 0.693751740322865, 0.3017686429426373, 0.4269667482965376, 0.5191557821655841, 0.7337466452726208, 0.6366669482561502, 0.08966758280489329, 0.16256259466427528, 0.14561239587823635, 0.056284472270527665, 1.1308047154489023, 1.4877559943913357, 0.3309703007081554, 0.767446061713107, 0.8488629831060639, 0.4769407795432325, 0.9319571817043302, 0.599422285064565, 0.4777484758423099, 0.506683756436616, 0.7303602066538348, 1.154608658917204]], [[0.24991854977429387, 0.8961449137078605, 0.32727284141484486, 1.1732382995181787, 0.8086720808492442, 0.7475301299055559, 0.4114308515792649, 1.2554139577771721, 1.149062100150258, 1.269763377372915, 0.14601609172895802, 0.6167034701818384, 0.3838729654674981, 0.6613241376803001, 0.2900868899721992, 0.45007403303549154, 0.48792425537167805, 0.6947865683624006, 0.6342212103741405, 0.09829610402717781, 0.1739704244097549, 0.1446207623768577, 0.05564331735972786, 1.10386121362402, 1.4232298502052778, 0.2783421210167148, 0.7387520544849907, 0.8547554501564374, 0.49311213972204154, 0.9459018278103681, 0.5274534892015385, 0.5094719445147927, 0.4414955813508785, 0.7180184452489358, 1.1764646924848674]], [[0.2799407661188015, 0.8608294555295694, 0.3148233600558309, 1.0692103453974338, 0.8527306290271836, 0.769564750229528, 0.3833904171418043, 1.2640513507822615, 1.207317402594641, 1.246607329315237, 0.15609593652113196, 0.6510563551877901, 0.2858011289393187, 0.683395187499495, 0.2884836325759793, 0.4606362137550699, 0.5121308640755554, 0.6715790572083334, 0.6304495438140078, 0.12922501632642508, 0.19905840257025545, 0.15211223183777917, 0.07152330466942491, 1.1051888610572198, 1.3659107810109439, 0.30042694163550265, 0.661890878049659, 0.8371234406797545, 0.498053123963089, 0.8676376952991501, 0.4314707278251399, 0.5244632749681687, 0.35838953511971006, 0.7357911502510299, 1.171549547579797]], [[0.26527264295860215, 0.8291836183346327, 0.30413115721712036, 1.1471306742383802, 0.8646529623553454, 0.768966862327972, 0.39076081437178994, 1.15534394928298, 1.2540622555661018, 1.1027036718160232, 0.16889602442710736, 0.6322485771008773, 0.24929274466069457, 0.71885400806712, 0.2985477571945713, 0.4825400276790258, 0.5466732258186884, 0.7240591844913034, 0.6052589307570414, 0.1395122429298994, 0.19017095757515687, 0.1677490152818662, 0.01835381012717518, 1.0524829931001152, 1.2144338067199691, 0.3310899418160383, 0.5736945321787762, 0.85960992848321, 0.5003648473325106, 0.8333380715995118, 0.37227917239433383, 0.40695843347284893, 0.3674132766985262, 0.7495177426267217, 1.1971116677937528]], [[0.2538889125644811, 0.7961595091518193, 0.32035826372782994, 1.0519716466121574, 0.8579306209240427, 0.7990561716896489, 0.3958130672699084, 1.057496475067435, 1.3092159142236621, 0.9900043683281181, 0.18008336694328356, 0.8351811377514925, 0.5525120769914822, 0.7492775118044797, 0.28689112331134087, 0.43728966598515867, 0.5119163866948184, 0.7401103030707774, 0.6014536711286185, 0.1463395663398731, 0.16755343421949412, 0.17570392387256673, 0.017433552767483804, 1.0296517911079286, 1.075549822820784, 0.3832821292157358, 0.5526659200519777, 0.9059275088629806, 0.5536786849367998, 0.8173901434238602, 0.2912048840450341, 0.29226785895974616, 0.5539782532675837, 0.7723144009121763, 1.1755973859939337]], [[0.28541560402903177, 0.7802143738605705, 0.3495971881853925, 0.9281480668265613, 0.8859464276220255, 0.8540398222005992, 0.3963705517106932, 0.9225234372441591, 1.3519151958537734, 0.9680315764135143, 0.17805338686905564, 0.9126825845403335, 0.28289119820781233, 0.7081734464844892, 0.29240743103029054, 0.4504632807881442, 0.510517868808926, 0.6784712398874087, 0.5976396961746546, 0.12982018827023883, 0.1811597267360531, 0.11073153687158557, 0.015861999995535357, 0.9157647501141009, 0.9099137904155293, 0.4642707724374753, 0.4948295104218958, 0.9674782515615772, 0.5865753828801761, 0.7889584905455294, 0.2390824877440783, 0.2859215818905344, 0.6332751830641874, 0.8056070842586838, 1.1568275191050787]], [[0.31859305111526254, 0.7865314033272872, 0.3435846349041567, 0.8625461314942482, 0.9115042191959998, 0.9647495548619084, 0.38916357000810536, 0.7414374544628317, 1.4091084763766903, 1.0708377470509283, 0.25052167120733515, 0.9253528992475535, 0.26412937403102454, 0.6728455859664565, 0.285719664107165, 0.45869131961627246, 0.5592158177647742, 0.6674999503826188, 0.6188306498195035, 0.1591876748157844, 0.185934984784064, 0.07735435164955046, 0.014355246472220706, 0.7253147101764863, 0.8121687172693487, 0.49361763731598685, 0.3854799457382023, 1.097558504681314, 0.5992225025329587, 0.71768322381693, 0.2969853296822491, 0.3804936622850284, 0.7303374398449896, 0.7707952694026855, 1.0334652457794384]], [[0.33094520626931134, 0.7506752133633916, 0.32632257711795, 0.7817485216666001, 0.9391503443309623, 1.1583933081848605, 0.3882053751513944, 0.5604613842511095, 1.463301061839387, 1.1949626422731552, 0.25563196361127793, 0.9564262189555464, 0.2690912990119678, 0.6142536937660124, 0.26822571946282364, 0.42896001422016655, 0.5922330783075409, 0.6317289188462561, 0.6538550424799309, 0.1949969662493709, 0.1851240227389401, 0.09730187768879346, 0.012915691900009161, 0.5603651128548334, 0.6799695696711041, 0.5658227818054815, 0.263689750034777, 1.1633806485906426, 0.632715132897568, 0.5771683334426434, 0.4845015425960991, 0.543294212152854, 0.800534474544011, 0.7622998861155587, 0.8726306161350087]], [[0.27517938160019795, 0.7176511136792615, 0.3430938168378491, 0.6990510087153756, 0.8646449904127209, 1.1661693389907082, 0.4124168550757138, 0.4886281925232302, 1.4535630865862932, 1.202210913557455, 0.22441862346841215, 0.9460281654151703, 0.2666310240783303, 0.6103012843573898, 0.2728769656786866, 0.4178694276646288, 0.5892770108191354, 0.6486202683775164, 0.6905822219590966, 0.23605908259000807, 0.1932638214792569, 0.07856405551573263, 0.010468928758567501, 0.4630180911499002, 0.598975912135121, 0.5919456480825134, 0.2486731015883557, 1.160674712730208, 0.6891757048869638, 0.44962384280165624, 0.6274005731084575, 0.5642390504423175, 0.9218759281336655, 0.7650537237962045, 0.7781957314833587]], [[0.3311726418263814, 0.7330218211127058, 0.34648804634856506, 0.6311794270881131, 0.7816566754561758, 1.1572871167453809, 0.45855368911202515, 0.5244419001412406, 1.40914333213698, 1.21544892481028, 0.22931981943769608, 1.0319411034204995, 0.2678941233229841, 0.56416496944331, 0.31985332608518136, 0.39367026092525415, 0.6061007265510296, 0.6482950120368881, 0.6772601735901236, 0.2511407537063, 0.2028627693887281, 0.07240029952631608, 0.053699355852797115, 0.392675741537892, 0.5455431676462504, 0.6913060949483476, 0.33213289007143354, 1.2565066657239579, 0.7496358259996254, 0.5096190226873442, 0.6253436669018779, 0.5319673712137013, 1.0315849493178542, 0.8327184889671015, 0.8831178078489299]], [[0.37708597803893934, 0.7955066263851144, 0.3543652515104139, 0.5978084379463015, 0.786637852224362, 1.1065426931899052, 0.47684407284071223, 0.6047700258271124, 1.333353770393598, 1.1381909413359215, 0.35848595134031846, 0.9169579292720709, 0.2693595829987694, 0.5428218845897066, 0.33903490680828746, 0.40532304903059746, 0.6760575387027685, 0.6632402458601996, 0.6558207843434442, 0.2524367232146272, 0.2142901450226593, 0.07549787739894019, 0.05362432845568743, 0.353314771988705, 0.5723830216654493, 0.7705580452785661, 0.42267322886894343, 1.0636372407126444, 0.8376861796578314, 0.3876346232180303, 0.6676056896849074, 0.5503731342631293, 1.0655602672912985, 0.8629708486511334, 0.9682104186588305]], [[0.3686370859985779, 0.8065692248485574, 0.41279556100030046, 0.6045636245305521, 0.7618544001346224, 1.0987866678684983, 0.4933127102027256, 0.6885493705692616, 1.2689349305538586, 1.1094560558403077, 0.30921138692039607, 0.8921797621401699, 0.2791001065029377, 0.5637680469314377, 0.33914274100013725, 0.4205013819263004, 0.7144212021288949, 0.6514783525392283, 0.6663512313310773, 0.2591195760943168, 0.2512691869770615, 0.0909387741796657, 0.09264549653870179, 0.3118194060957476, 0.6073081155278228, 0.8168615189912708, 0.5390195938918072, 0.908923629861426, 0.9231842280711028, 0.29637315381378937, 0.6997564886560027, 0.550137210780049, 1.0561573870735323, 0.8453258751330073, 1.0579666878497629]], [[0.24980607548885153, 0.8177569801256634, 0.4122351168037296, 0.6253406005842213, 0.7555282722590906, 1.2162476998646214, 0.5172708382770541, 0.6767392377542671, 1.1798066374486855, 1.1644901193402997, 0.3713229661456088, 0.8989225371531407, 0.28454368944230657, 0.5927296898435, 0.3325492505607544, 0.4005221789306018, 0.7221053054364502, 0.6595961185991568, 0.7032955183187293, 0.27264993898506434, 0.2654637924731862, 0.09330382503340981, 0.11439804378361564, 0.32337610968916697, 0.6352953426334706, 0.8168559906253061, 0.7114572214739591, 0.7969310225544788, 0.856045286400308, 0.24335003793260887, 0.7618929624127289, 0.6054083178624575, 1.0469012588224276, 0.7613604435815602, 1.0362083954818566]], [[0.24561134072616275, 0.7758041702811583, 0.4325094661561518, 0.6742266937744563, 0.7305090882585056, 1.251049446867773, 0.4998188205910293, 0.7185120187655762, 1.0994864522024108, 1.1011356092256759, 0.27209207965709126, 0.8878047494763942, 0.29571887247663453, 0.5695496091976942, 0.3141314561597022, 0.39991979819835033, 0.6161008057444406, 0.789438525996545, 0.8299202215430733, 0.17075967200243314, 0.2527989066616414, 0.07467515800684638, 0.10224536654899134, 0.3658712238268427, 0.6081911409674992, 0.7210113880118313, 0.8650056531148133, 0.6228875850001389, 0.8131108632260362, 0.22898422153111694, 0.7117152605432787, 0.6233206923558043, 1.0197966558027323, 0.7219988134675535, 0.9669685016404307]], [[0.29382457098234893, 0.8119463355916061, 0.4800281510666964, 0.7055299197848003, 0.6969267575809674, 1.229345208752855, 0.4653794817261244, 0.8724401811503882, 1.0369833811752072, 1.019195163594026, 0.19481827723599165, 0.8501280905966466, 0.29054777812528576, 0.5828357280459764, 0.2957410729722474, 0.3931716992914355, 0.6118153246790434, 0.7582102457910798, 0.8444919533202786, 0.20391413970902655, 0.2937326688912813, 0.07732444729644258, 0.08599622482847508, 0.4232127324445692, 0.5718935665155366, 0.7224055481787947, 1.0439514455937675, 0.6423027122629079, 0.7575555670425393, 0.1939483404482928, 0.6890408477100308, 0.6066477787291329, 1.0136517700990635, 0.6716975684470681, 0.8583251269866179]], [[0.2765011029818118, 0.9085542877140121, 0.46638534643009477, 0.7002085766700823, 0.7646377052696004, 1.1952838386504965, 0.4789539107902762, 0.8902357270401594, 1.0087408367863606, 1.0275373956152443, 0.16726661722886876, 0.8251875656407237, 0.28342943878954163, 0.5882261695604754, 0.31268449741331705, 0.3827845946530336, 0.6256643121757159, 0.6769991687529598, 0.8862331731670515, 0.246041913259644, 0.35391793000099575, 0.09000349276896547, 0.08200818673308388, 0.5250174563777184, 0.508665476873492, 0.8084425230136576, 1.1833271075670382, 0.5969792890809229, 0.7470752366642084, 0.20852957426751342, 0.6836268472964255, 0.5629003402646702, 1.0299254218668386, 0.6105347005418685, 0.7484655327842775]], [[0.27448430793841416, 0.9743382863989689, 0.5010376888207982, 0.7224578323858797, 0.7172363530343175, 1.2020675431235002, 0.4546421194096539, 0.9265996089471156, 0.8938354067885277, 0.9278479776338441, 0.13116767425803283, 0.8124769898549495, 0.2050268477463958, 0.5475633069374823, 0.3567631874898786, 0.3650389840628364, 0.6200067805706417, 0.7108466277615962, 0.9780541789791661, 0.25633021103676357, 0.2716266203409532, 0.11166132340465157, 0.09047987485919341, 0.5862741546822242, 0.4392470083953632, 0.8631041747231643, 1.2800846217119932, 0.5259698738764822, 0.6734862755384192, 0.3170233395659086, 0.7210156006286425, 0.5546449231252772, 1.0681138680599935, 0.6376195953936729, 0.6349818631761588]], [[0.22319838971266268, 0.9373328285495793, 0.4798577477570206, 0.7394257648128186, 0.5390067838221497, 1.128093988936439, 0.4418067275575706, 0.9324159832200959, 0.6125776982039679, 0.7317250055042139, 0.10849627396136065, 0.8014961483190985, 0.2133302943556319, 0.5451019935028747, 0.34998025181156023, 0.3450502045270784, 0.5778691828433309, 0.7276572784281667, 0.934507561475818, 0.2502505489732995, 0.24985533314709754, 0.11482766675565441, 0.09695896713965012, 0.668401425631501, 0.43823688096009794, 0.9532286503940226, 1.3929679815411975, 0.550673837265858, 0.5802992307928422, 0.5574034848827929, 0.7900719354765753, 0.5612495866007882, 1.08335869247803, 0.6067979321856358, 0.5082330531191882]], [[0.184395801005638, 0.9185251452374998, 0.4535462510630037, 0.7658156276371116, 0.34672234734925245, 1.0654296549152114, 0.43054466419918913, 0.9065483223860875, 0.41847558389434436, 0.4531884295783454, 0.10371923557600232, 0.7872720745219579, 0.21068838649272376, 0.5135390295538838, 0.39289308642239396, 0.3453692477518332, 0.599665968841132, 0.7214843185374373, 0.8856201455255135, 0.23281182363426625, 0.248610587102987, 0.10325644316666122, 0.10214857780719089, 0.7652093501836436, 0.4218640362241894, 0.9774413360941305, 1.4424425218743264, 0.6488281301889588, 0.7852351717872024, 0.9326709730887461, 0.8902256821517378, 0.5653310452991882, 1.0895446355722558, 0.6619370957705957, 0.4867834820453556]], [[0.19053390572908413, 0.8683069692783382, 0.48758352455921994, 0.7716812251053596, 0.27735261536442724, 0.8427142073289544, 0.37248903878364303, 0.8801090095267873, 0.3557958088139364, 0.48065025451373333, 0.09436431247817176, 0.7707147274606525, 0.21172974829260185, 0.5043941218409852, 0.3429936541758889, 0.38404487536267395, 0.5860291334730721, 0.7047276548295365, 0.7327305957919301, 0.2289365889712345, 0.2569236041465288, 0.07920778151087597, 0.10285986366658173, 0.8062993604558738, 0.4010931476132638, 1.0090470299637377, 1.4108461890248079, 0.8278081631074069, 0.8598148687902866, 1.243431283820504, 0.9493761263539894, 0.6083600278989703, 1.0927557481984818, 0.9654826367942246, 0.5426386068703676]], [[0.2711123979763916, 0.8146582728377504, 0.46489417220540386, 0.7689126424685062, 0.41623431169930203, 0.7187843825755962, 0.3484303411682613, 0.9522225886901309, 0.5258599549510043, 0.5952032683771078, 0.10205521474436416, 0.7504345122972149, 0.2555377509440981, 0.4989897293087895, 0.335475040529049, 0.3641661956485555, 0.6032885095530738, 0.7254545118798139, 0.752996811647145, 0.2278693641401388, 0.25349256351543853, 0.06415206107951495, 0.10924554333416811, 0.7965581336910866, 0.39735597354433055, 1.0460028587071848, 1.2937586314721017, 0.9606560024791524, 0.9191526737981855, 1.4309093684427077, 0.9990118801981295, 0.6504266315069507, 1.12078418718625, 0.9674895297636594, 0.5293673590403104]], [[0.3459213973733916, 0.8257969297132183, 0.45835088497723975, 0.7681190936349745, 0.6686052658157748, 0.7159665027287944, 0.29881738567738003, 1.0018253056668307, 0.6006990012120788, 0.6626719598985041, 0.10352727900542204, 0.7704966857746198, 0.29698907284917786, 0.4955161659641789, 0.3383209544746336, 0.32220282182701454, 0.6092499978742134, 0.7435787046060257, 0.7382637704625099, 0.2146609564259356, 0.2577492211738849, 0.07995661516069996, 0.11602778114902412, 0.7584944791951607, 0.38043677249339347, 1.0318424115104696, 1.1156598262024229, 1.0500858283394299, 0.8938405810206035, 1.4728944324326716, 0.996450425249629, 0.699873973864988, 1.136856556354605, 1.1084751077413304, 0.5509958117179847]], [[0.3847315409068071, 0.9176867077024506, 0.45015248159473287, 0.795504730456907, 0.5467412596897547, 0.6967589706394156, 0.316716767346534, 1.1188549180477996, 0.613851276390905, 0.6489039076060368, 0.10422938991367753, 0.7363159551934715, 1.0437690381314448, 0.5774151983336279, 0.2959355419532056, 0.35006760064144504, 0.6191731391285659, 0.740570899069206, 0.7160943782705691, 0.1603297512688381, 0.24601010750015873, 0.11058518602729381, 0.1021760560719998, 0.7103398914682729, 0.34519598780559735, 1.025375392889631, 0.9520011450130857, 1.165073571113449, 0.907458205314708, 1.4541497772814842, 0.9527305125114109, 0.7491735073571983, 1.1073384494807772, 1.1481589226671114, 0.5016790558109018]], [[0.415101154893561, 0.9784603878527451, 0.465286297179743, 0.8110212311098557, 0.5508380045457346, 0.7465292650856179, 0.3803471432349735, 0.8290930011712647, 0.6110378421067828, 0.6482372555040166, 0.0966149517834263, 0.7734799458047817, 1.1209187452097522, 0.5721072512613008, 0.38125873679184463, 0.3391768126740917, 0.4992172394580553, 0.7471231097678066, 0.6975817410335028, 0.1459740803608037, 0.2574120921295817, 0.10126840407628032, 0.14465863232756787, 0.6956451222269595, 0.2745998741824728, 1.0712459561252718, 0.7855575794136953, 1.1185512589377153, 0.8470567906093072, 1.5083219209066345, 0.8991493276125768, 0.7631295918670944, 1.0897485322833589, 1.1758958666890127, 0.42013068717880153]], [[0.44834768272295356, 0.97207039833241, 0.3906885341925048, 0.7935269163200149, 0.7066373986191848, 0.798508690188767, 0.37382495257242154, 0.839706191442591, 0.5616393203011667, 0.6734470090854593, 0.09436929699135763, 0.7022548063678935, 1.2160759056489456, 0.5682221481696861, 0.35559788313279816, 0.3436823476363211, 0.5740117793072153, 0.7655929314763166, 0.7451673893779314, 0.14929307362830432, 0.27919436591598995, 0.0831064392074182, 0.12444907680498322, 0.7636337877919233, 0.3249034157894297, 1.0653466525989586, 0.6555819290764106, 1.1147942437509786, 0.8539146896132535, 1.5009464745798051, 0.7755551095977794, 0.6369409003765765, 0.9943646757703506, 1.2004524517891333, 0.3647226590336549]], [[0.5018046088337034, 0.9789638865773246, 0.399155036606731, 0.8933297693327504, 1.1832066404961217, 0.6956015055525977, 0.3101002673398679, 0.8242290448676646, 0.5314188286931725, 0.6328982385258632, 0.09212554284563065, 0.670068133210024, 1.3021977209120221, 0.5666763971676244, 0.36351696587043597, 0.35594055052677587, 0.6382372610856328, 0.7574353000318026, 0.7901646008698283, 0.15558541952209265, 0.343382041419795, 0.11241030659882578, 0.1013126454180802, 0.7541863132245061, 0.481209319080445, 1.0110924759288265, 0.5581640796276451, 1.177107352271073, 0.8234977856497092, 1.5058808293484454, 0.679849078175641, 0.6029853236759004, 0.9698966935562177, 1.2958646652327643, 0.4261868140934826]], [[0.5418936053061468, 1.0042216673282363, 0.37862570124668715, 0.9813536151667381, 1.2097690651691564, 0.7719613125321227, 0.32995565997065657, 0.8296697855992186, 0.4739040416617428, 0.5186731750444828, 0.07577575809608185, 0.7080913130671218, 1.2795968105504303, 0.5815297471141381, 0.35974362835594853, 0.37899121834653027, 0.6743939337550005, 0.73656813828799, 0.7896669920454261, 0.15579373864103352, 0.2936935128160808, 0.13145293837634114, 0.08619684076210121, 0.7795329429499247, 0.562003782443177, 0.8759750445452097, 0.4795040320298355, 1.2366663218807465, 0.7662431985183378, 1.561995329168226, 0.6047370658053292, 0.535296489880805, 0.948209105627551, 1.3149754309081358, 0.4847294341480456]], [[0.5795163755334067, 1.0772939752902835, 0.485395437716197, 1.0084146950926784, 1.228521292946064, 0.7192519279117872, 0.2862675603640038, 0.8621971798325847, 0.5281924762901408, 0.46402577916679977, 0.05474156679193256, 0.7392541036703176, 1.2670166271111758, 0.6012498738967542, 0.42648055925354295, 0.43986070038603714, 0.72778281271938, 0.7256328338304739, 0.828728057473481, 0.1727157798139961, 0.2867789288931417, 0.13540764719022252, 0.09169422361086627, 0.7850060013363152, 0.5194753610769498, 0.7828564756658619, 0.43089176078237407, 1.2529546884708083, 0.701520484348213, 1.5271519069529274, 0.5146362697376464, 0.46067661714003344, 0.9029558684263503, 1.313184760442281, 0.5400692349647119]], [[0.6383267985618439, 1.094742866611604, 0.6227771895780536, 0.9638716844044781, 1.2453930287426813, 0.6635187053186633, 0.4121737773964028, 0.8386987739650467, 0.7032609535736742, 0.3925503184413809, 0.05678648354849739, 0.7580220601196703, 1.3014063415240065, 0.585024525563598, 0.397686746161923, 0.40922864390895575, 0.7109567998387589, 0.7242583844564969, 0.7826931185655885, 0.23644993253032087, 0.2692564159866357, 0.1949468922591132, 0.10557601673305157, 0.7746244703343877, 0.5834363373438354, 0.6537188039905715, 0.4189234291898243, 1.2132404854110526, 0.6199553574363752, 1.5513507359164103, 0.40489179510711193, 0.4361177621822107, 0.8137487518512507, 1.3467778323113375, 0.6041957708578567]], [[0.7403118271688917, 1.0934268472683697, 0.7135066329767523, 0.8834302023583867, 1.23486864008638, 0.6360377561218773, 0.39923698366287325, 0.8625982344467655, 0.8194476079099531, 0.4074975014102474, 0.0547351042826562, 0.7199460059794341, 1.272007789974949, 0.6095549041618282, 0.36696409373255473, 0.43670428426062213, 0.676110117814583, 0.698149583483955, 0.7315274735700289, 0.1918803128840556, 0.23238096185369603, 0.2279297301325918, 0.10840292583165946, 0.7530265804675729, 0.610574372832577, 0.5835208549994048, 0.43167649144019504, 1.1950256009487963, 0.6529573187134399, 1.5379631338848114, 0.2725086069448608, 0.3808612858031701, 0.6300934183215702, 1.4151916086132372, 0.6136132521093378]], [[0.7665253198145903, 1.2310246743889335, 0.8173597172519649, 0.8538101347685878, 1.0041022834124567, 0.5849102014553134, 0.49769580336795116, 0.6954465263295038, 0.8119383959429856, 0.6394778986325512, 0.05082534710355274, 0.7015344029089212, 1.2935674160321788, 0.6369973431147307, 0.3774955755596851, 0.4445184063966161, 0.7022670154137157, 0.6628131697344276, 0.7272782932718331, 0.22337239693493405, 0.21980469517617907, 0.3306279329111754, 0.12005886467273222, 0.7207708652312397, 0.6196846081041508, 0.3548345609252054, 0.5067999373793761, 1.050254540141923, 0.48389524981405946, 1.6139589011466353, 0.21976815811682104, 0.3143903495462297, 0.5243135901686975, 1.4611891431155848, 0.5553142098407546]], [[0.8033358347769082, 1.217481965259108, 0.938543160525887, 0.8420536163932961, 0.924134329395258, 0.6144977159150866, 0.5889324959222785, 0.847180221937208, 0.8203674379604192, 0.7297085135747254, 0.07623552185183868, 0.7337061575544337, 1.3130465441160042, 0.6393905860110343, 0.3369023142933384, 0.4539616851533216, 0.7146389352059086, 0.729293321072683, 0.73014050804048, 0.24229769464776352, 0.2178297486071743, 0.42447426137122657, 0.10687833622071669, 0.7230040842813962, 0.6472452652901682, 0.30920831783009106, 0.49994355177671224, 1.0528876586527665, 0.4640174529361612, 1.650961500577679, 0.23505019270151176, 0.26773092061433346, 0.562075453095514, 1.5490982358823775, 0.5212055746240587]], [[0.8083789820682394, 1.1742808161915925, 0.8909335073221867, 0.9927081427729522, 0.8968925302494735, 0.5992642913096227, 0.5554879844023801, 1.03399329737917, 0.7926555156699557, 0.8259997287375539, 0.10651445055174195, 0.7157186262139692, 1.3848592615987632, 0.667111806196331, 0.3409334319141403, 0.4663014481814574, 0.7053658840477017, 0.7276030372472769, 0.7198080323490482, 0.2442681527462789, 0.20380020075885774, 0.4997288192310776, 0.0925584919911841, 0.5045426968355468, 0.6837764342132553, 0.26906002520681693, 0.42505713694745545, 1.0194879135399662, 0.42993971337231485, 1.4256795915553993, 0.1671017858675166, 0.22477502942076996, 0.6344599152070871, 1.5249847691543872, 0.48478311035568444]], [[0.8086736584014868, 1.1947167527063387, 0.8599035752981483, 1.0554529198641651, 0.9712917322116698, 0.5807007638003424, 0.5372204259102638, 1.2750857069810717, 0.7256559176645984, 1.0022538385449327, 0.13515733252223938, 0.7400465698763905, 1.4733295236805735, 0.6546327081083969, 0.34724236526030083, 0.43266811143493633, 0.6972230573805733, 0.7381135499222387, 0.762248534340713, 0.24124562157934168, 0.16833748587806463, 0.5141005199695186, 0.0969637836663925, 0.37023084172947374, 0.6278332782330276, 0.32764745384966076, 0.3158276900894934, 0.9427516711694216, 0.4191693597690357, 1.1582316228390737, 0.15220411285809765, 0.2015777427937755, 0.6462988047094594, 1.5246622671713455, 0.5011359518028251]], [[0.725567426184925, 1.1241123456559003, 0.8139715398491597, 1.0537190096415825, 1.0006973207156382, 0.6281992120262998, 0.6000598003436439, 1.4464642957568445, 0.7404123098778027, 1.0774839758975303, 0.1285873227728719, 0.6912003525549046, 1.501990516898462, 0.7146070364895437, 0.3456034642612674, 0.460293667404742, 0.6845669480254759, 0.6894771489554745, 0.8553734135364038, 0.20616729658924227, 0.18406066279950828, 0.43845227671384446, 0.12833541289660522, 0.39892921472709064, 0.6500828498040392, 0.40315899170870206, 0.267218992777732, 0.8255876532298984, 0.4043665504130364, 1.1157368306278408, 0.24389751254836622, 0.23945222922299633, 0.602302521222445, 1.5961982966024304, 0.5354531260120134]], [[0.7203288691705625, 1.0470883227539576, 0.7321958522773412, 0.9976848818203189, 1.0055238540245204, 0.6266698140218181, 0.5720639104319951, 1.5947303358931024, 0.6784210708461181, 1.0233271846515417, 0.13012001390987754, 0.6389467452958136, 1.5012507160611621, 0.7011625088815729, 0.3384583258753696, 0.46404078192182624, 0.6897757351433529, 0.6801864860292797, 0.8872686163815163, 0.15947782673482186, 0.21392836656683284, 0.5339099763742621, 0.10175026820292016, 0.3959002478460032, 0.717654509083125, 0.36477877140424947, 0.23060432704056788, 0.7400700732366854, 0.37944561889630407, 1.0584464426602318, 0.4002958068703224, 0.3069365621357054, 0.549065573493364, 1.5360078302335571, 0.5688115070854377]], [[0.6743829651859139, 0.9092762217256098, 0.6826385037029361, 1.0853835849410534, 1.0229443170897454, 0.6476565895728966, 0.5279636798954492, 1.646487969603413, 0.624080192222326, 0.9506407058985259, 0.1181575847212272, 0.6217048271396691, 1.4881160772089337, 0.6916230881193779, 0.42554549819266796, 0.4823790146557704, 0.6810213287046193, 0.7153585040673479, 0.8489923667531167, 0.15575286752016698, 0.21551476134148734, 0.6053154220216984, 0.04911132936317502, 0.42733430690488117, 0.7576853698667223, 0.2802152554718687, 0.2373684933511795, 0.659532843525415, 0.3527084530365024, 0.936357921904589, 0.5216849202438087, 0.4721663944109545, 0.46960455660072187, 1.3805942433256821, 0.6008872880459292]], [[0.6818472978514883, 0.8715784715155264, 0.6425186724331244, 1.1395125559951373, 1.0337539702440863, 0.6695908945575731, 0.5385543262183456, 1.6810180514260562, 0.6468478737051907, 0.9008620692799254, 0.11590274117220659, 0.59696384043779, 1.502951283295134, 0.6903850847593742, 0.39194889286010137, 0.4520066434331937, 0.6988834329843077, 0.7323084307493637, 0.9101069736853812, 0.1726229359949303, 0.25439533429757216, 0.6659324999892993, 0.04500302889702838, 0.511045240417479, 0.7466069390320172, 0.27825834890042733, 0.29153810345772346, 0.6305736326589599, 0.35256621073282723, 0.8672846439739135, 0.6145334413616022, 0.5799067910379374, 0.43993953279974185, 1.3034682412575425, 0.5986784144159453]], [[0.7034344617854709, 0.8614648141670177, 0.6092539977409388, 1.1158117123018774, 0.9667811931753996, 0.6815118569139897, 0.532716167234682, 1.6957192480384646, 0.6228181733416264, 0.7418453343033109, 0.1129113764684817, 0.5839448773379531, 1.5356594950007123, 0.6911884722856403, 0.4188551878609257, 0.4808579871080749, 0.7523628825305759, 0.7649761203974401, 0.9280626362151023, 0.22931448087283268, 0.28507468331679314, 0.8683360924720404, 0.043730585572086436, 0.558052202649437, 0.7111097358475968, 0.34589119180002736, 0.3568888257115582, 0.6004781090439559, 0.40161111347984696, 0.8779918719649897, 0.6657089213736025, 0.6592499740420369, 0.44796475467089597, 1.2130584547174479, 0.616616919324762]], [[0.724673181535832, 0.8449702188989336, 0.5871359676804018, 1.151269390264824, 0.8429924792851149, 0.6965053515935731, 0.5310604775405672, 1.6541970168694804, 0.6443907005787368, 0.7594124718996613, 0.11231033467193631, 0.5613512076980299, 1.561328724122781, 0.7091569306721831, 0.4163280028898686, 0.5199732405500745, 0.7826628280200374, 0.8030408145561293, 1.1968962732393424, 0.23902532683854727, 0.29648535050524893, 1.032947633540946, 0.12808247302563303, 0.6201008127634704, 0.7637584097427366, 0.6022835960725513, 0.4374939700822156, 0.5705506778829141, 0.4604078368114734, 0.885813016211729, 0.815244391714229, 0.7810904722699088, 0.46003458451573437, 1.087549432734165, 0.6762392925171128]], [[0.7607558131738329, 0.8369468139741227, 0.5694596396156854, 1.2083408751272662, 0.8283466788979255, 0.6944008722701936, 0.5060487782763218, 1.5987906885158252, 0.6411121160759865, 0.7610207740875855, 0.10934325183539696, 0.5544973518930505, 1.5907227257875831, 0.6889740676298669, 0.39303001574078744, 0.5411129277764424, 0.7606370632013287, 0.7996159030597715, 1.287541524544046, 0.27206923558316387, 0.37151399036479343, 0.9790151440981013, 0.18520961717325102, 0.6071891106034115, 0.911092915031685, 0.8474587517594, 0.44719384301003623, 0.5358147807029745, 0.5126808772527662, 0.917222711868285, 0.8944874474084072, 0.8366846185630148, 0.46436611330532396, 1.0289844759156768, 0.7915165379188032]], [[0.7725128050666913, 0.8416473951908212, 0.5579040095736261, 1.2624258926911285, 0.8454372991643195, 0.6750485613782988, 0.5129812554502341, 1.5684096385127526, 0.6541511972079261, 0.5768173061027557, 0.12613858059095184, 0.5516698522078731, 1.6359825343606031, 0.7142744721509193, 0.4064012744066396, 0.42026781013027736, 0.6992054107619246, 0.8541150426337989, 1.4675937479183585, 0.3139731456151997, 0.43293116858181396, 0.8563743036670766, 0.14773491978538755, 0.5938492149449344, 1.0473526797672066, 1.0890248930039257, 0.4154001929057467, 0.3294360453013474, 0.5009899933072085, 0.8939784051264467, 0.9349866736055807, 0.8604371171163738, 0.456895391554505, 0.8719369763494529, 0.8461892169397358]], [[0.8527879957681704, 0.8101547212998121, 0.5526146523689205, 1.3495275934230677, 0.8888868183129621, 0.7110090965828834, 0.5168954736619417, 1.5907877138410822, 0.6786793427569018, 0.4958360847889271, 0.14356764217559193, 0.5507711783326406, 1.674717863058014, 0.7854275965719631, 0.42060801313639834, 0.43778245662261417, 0.7330565175006487, 0.8021533954267197, 1.486295983135462, 0.3312992708955213, 0.45702650979507353, 0.7412901532279826, 0.15889290880318202, 0.6162676038605313, 1.0285027166564045, 1.1376358694790145, 0.3011882374219345, 0.22511721768189868, 0.44773933828576784, 0.8277816612739188, 0.9290247441777615, 0.8628896369910324, 0.442685772644682, 0.8108706552797942, 0.7764061626518342]], [[0.7676765204358346, 0.7168199090302665, 0.40366701191037285, 1.4639020283484114, 0.9163376448766702, 0.6923879099859016, 0.44971711364852374, 1.5887776387631634, 0.6440231641705133, 0.5417221415262389, 0.13739237435265295, 0.5425102931416997, 1.6940002046485167, 0.7523589177031591, 0.47114008025051307, 0.39559811911488596, 0.71573081659889, 0.7588505346023653, 1.433943507911548, 0.32695230799532154, 0.42794360537107606, 0.6304154128144612, 0.1323283446859727, 0.6351887617047476, 0.9451007235112516, 0.9639055085950431, 0.16666112368829616, 0.1625786871240298, 0.36848879725489053, 0.640871759350606, 0.9030643684396729, 0.854325666844097, 0.422698129959177, 0.7350950517921742, 0.651531979671726]], [[0.7304322858623269, 0.7309089235939692, 0.40707670129063456, 1.398529296738585, 0.9696382584892175, 0.6822499902881398, 0.44294469736335973, 1.5419578405628165, 0.5131751324966363, 0.6262996174146114, 0.10341769067806401, 0.5477613344765593, 1.64860368111668, 0.719259663783172, 0.45676303568602933, 0.4169615676605644, 0.7168693266588231, 0.7713618066916521, 1.3929493156857458, 0.32260265925642384, 0.3852248812204236, 0.6576113480739846, 0.11453220345789773, 0.6486291014381104, 0.8293482031955979, 0.8501032760751687, 0.085447465965101, 0.1608681764028897, 0.3863551857009945, 0.5855399745120922, 0.8909118303617518, 0.8602827119434074, 0.4064004107783567, 0.6613545436318027, 0.567350140913177]], [[0.7131601431353553, 0.7218768656172292, 0.3921058856098638, 1.5004621854191027, 1.0438964424347994, 0.7100091565759035, 0.42475527188950996, 1.5238794536217994, 0.617694895422205, 0.7181608897879508, 0.13660715024468817, 0.5847300080229869, 1.6017848163750004, 0.7413038671945755, 0.39392995436693057, 0.40660686845804356, 0.7676831907266956, 0.7614676009704582, 1.325240437815981, 0.36213084279454355, 0.414772612702125, 0.7655161053226067, 0.10907662601556832, 0.6795122181615891, 0.6665838943367115, 0.905371841171111, 0.10645939688753026, 0.18573440244254763, 0.4306299035504815, 0.5833755358267723, 0.8977249840414212, 0.8865898418103083, 0.3895499722950627, 0.5740182971373597, 0.44233540402431876]], [[0.7477230804980916, 0.6946102692970203, 0.3592081039064492, 1.5031785136224216, 1.0873362426142565, 0.7565385349145421, 0.43285618660094444, 1.5240860934626548, 0.6966404700004121, 0.7556618055744085, 0.09369246623506657, 0.5819409258386347, 1.528086017029834, 0.8314922385375085, 0.3667477979351181, 0.40204484225520626, 0.8898343150401611, 0.7476995706721863, 1.3412999509252255, 0.41107074474269933, 0.44725762895247895, 0.8222289915041495, 0.11200348744350151, 0.6955903205669642, 0.5338053823176373, 0.9836200450788523, 0.18584345824209958, 0.2434859357363201, 0.4408064118455618, 0.5997844068867954, 0.9218482766990304, 0.9021055258241876, 0.36931698965543075, 0.443067486592113, 0.3089598712949128]], [[0.7417757370301034, 0.7223220250344657, 0.3267306627058097, 1.425234416902544, 1.1421111703955447, 0.7490717010952983, 0.46480374045997414, 1.4608187345470063, 0.6846296227050377, 0.7657281586896623, 0.10334729352008523, 0.5894570231245269, 1.3818841837976865, 0.8872921225249171, 0.36691364717413644, 0.40030099065166924, 0.7872103150537668, 0.747052133477255, 1.3464950190275697, 0.4502921427908858, 0.45682568102146687, 0.8579268993169257, 0.10697973021982807, 0.7141778209301493, 0.49510052550411043, 1.0482367470618876, 0.3330798756351201, 0.31400481665993907, 0.4567925204200271, 0.6017542588975889, 0.9440102027147832, 0.9212687056801614, 0.35573601317184966, 0.4045968114585441, 0.13336318357329296]], [[0.7344514171335452, 0.7296507850292486, 0.3046438565453864, 1.3255529915954272, 1.0968197676864495, 0.7477254891016076, 0.49382229701623326, 1.3761330205395477, 0.7066955589284876, 0.7286301793347315, 0.10182843750742454, 0.583897862787719, 1.267714254797743, 0.9145606153116598, 0.34041881703805665, 0.4023511440504305, 0.7827935588871089, 0.7503716622852757, 1.4032532481571025, 0.43210151422744514, 0.4192890045806984, 0.7685425664944725, 0.10846377215395624, 0.7274063197362056, 0.4535160733529151, 1.0534084784172673, 0.4021798609642029, 0.30568969326506207, 0.45486452503933317, 0.6261614800068677, 0.9896799344363642, 0.9517392710726889, 0.353332334773793, 0.3488500494786321, 0.15290081966579902]], [[0.7085804810989953, 0.790559051786925, 0.31175447101596054, 1.4146614014743544, 1.090603727648166, 0.7130341151012558, 0.4911768676949792, 1.3029973174174065, 0.7010395586205731, 0.7127226860878879, 0.06665152852967826, 0.5761593181129852, 1.1919856370729138, 0.965724027997944, 0.34150968425182826, 0.37175734869321775, 0.6793022028967008, 0.6906280361452592, 1.253989705939824, 0.4105734491035403, 0.3976903859786617, 0.7153582943349065, 0.09618242074112578, 0.7098907082578292, 0.4172723436569286, 1.0241856212257558, 0.40976549838474163, 0.32798149405239974, 0.4191843604565687, 0.6444553453896977, 0.9959766635446056, 0.9649648132856267, 0.3469558073314852, 0.34070968324742734, 0.28707181767717893]], [[0.7079898420028891, 0.7243400768409436, 0.3130110700142148, 1.3804133165824433, 1.074439663309928, 0.7083132306700566, 0.43977952112161, 1.2673804612246522, 0.7160601733773955, 0.6902906732331219, 0.05764772594813439, 0.5649213464780805, 1.1844146851386732, 1.021386693370326, 0.38766533940663733, 0.3583575870243155, 0.5674838945796765, 0.7176839335564655, 1.1512864186230956, 0.41344274523834756, 0.4048398907403557, 0.685763277896133, 0.09586863062318018, 0.6930263861770123, 0.42948228770826813, 1.0024031520763734, 0.41303207476543236, 0.3723804921023266, 0.3965143797304226, 0.6496949468251947, 0.9986871771752698, 0.9624023839926227, 0.34843483177649254, 0.4293940043464687, 0.4410969168785165]], [[0.69540231866467, 0.6578366941836029, 0.31387327501721896, 1.3559756126484854, 1.114973970466513, 0.7471491721571771, 0.4354940727080685, 1.300590203969621, 0.7173748869040306, 0.6780848412044036, 0.058569190569471186, 0.5673186346551367, 1.2396049255677721, 1.0243757461354033, 0.46763285957831907, 0.3822894507526525, 0.5321327779053548, 0.6662438007613695, 0.918786472025872, 0.22104286095396797, 0.24809783584910733, 0.4152244156912519, 0.10935810855110281, 0.6793355936259098, 0.41407332681899545, 0.9716869307639077, 0.3977030420006832, 0.36789683486441627, 0.37579108491567925, 0.5500897990895823, 1.0069610064822643, 0.9222067802408696, 0.44335752643368637, 0.5009225491749092, 0.5949569482544665]], [[0.6992136146541, 0.6369423946970807, 0.2969819474049852, 1.2431097278266305, 1.137847042184491, 0.7310533442069862, 0.4369680368196842, 1.157223112918214, 0.6821039375660942, 0.6380344806938107, 0.05699064333331476, 0.5653305814886483, 1.2224696110552458, 1.0451379171063622, 0.42657335734698304, 0.36512734248289963, 0.44610929957086254, 0.6719421647690463, 0.9466955969081354, 0.18825514281170388, 0.244037939496491, 0.23437859070287403, 0.1427486738085308, 0.6714089934132711, 0.3787911456864541, 0.8780345821722737, 0.41140775241686833, 0.3834466480815639, 0.39224939614261, 0.4877944156312723, 0.999622566401752, 0.8516650063971742, 0.4856838870815397, 0.5672007820488779, 0.6944829933479612]], [[0.7214988789775418, 0.5975528787424607, 0.34140525680876666, 1.0612318433545862, 1.1349387291495063, 0.6991555551759363, 0.43731481072745887, 0.9987797711584626, 0.5263651071510365, 0.5728501363750007, 0.05412158853318012, 0.5840404774978576, 1.268391933266458, 1.104295513591367, 0.4660631329463683, 0.3794993328749624, 0.5760317150484772, 0.636653604178515, 0.9090651265615198, 0.18689201846111284, 0.23721266626508808, 0.20573303522489633, 0.13823275404910745, 0.6468527237218437, 0.39086362452425416, 0.7885204941321605, 0.3890653474529643, 0.39856233791396317, 0.41015760786032235, 0.44099086528473647, 0.9442416182665965, 0.8199572672962275, 0.4560654234067868, 0.6467102935904112, 0.7440309239404149]], [[0.7248984429631623, 0.5341656367856557, 0.34231501291162036, 0.9410909843857919, 1.262484406057621, 0.673353234484408, 0.3140820669791654, 0.8343186357420987, 0.4394991792193577, 0.45593253082749796, 0.07203434225303777, 0.5867370152154952, 1.3649497263230383, 1.081318813483081, 0.3830234016312225, 0.32360157895372066, 0.4581192425663557, 0.6258776093080772, 0.7892933249793999, 0.21658026529268776, 0.25759442481895745, 0.2108069472639611, 0.14324849520249777, 0.605132977374298, 0.4024409133438762, 0.7800003079037285, 0.33342155227942943, 0.39227797585490354, 0.42701992644247994, 0.40829875252460873, 0.8560549185512475, 0.8143833900078008, 0.44555184608863097, 0.5770376302362268, 0.7648449099549463]], [[0.7398107862459712, 0.4623218212293495, 0.41191299283752303, 0.7878086179465366, 1.364493997772775, 0.7360557870969149, 0.42232661795255444, 0.7131117666585831, 0.3882493552449978, 0.3927439886671192, 0.0638606258161066, 0.5692287046973298, 1.4988051422020552, 1.111493709621425, 0.38365012105988733, 0.30879659633246015, 0.48202086700404084, 0.6316914087069769, 0.7402441394918429, 0.2341462340995954, 0.26468662595164655, 0.21573674495541817, 0.14757867461083574, 0.5364285411947612, 0.41282988116645586, 0.7923102326531127, 0.3062461771637298, 0.42127390927609853, 0.44708868909637245, 0.3722369064218281, 0.7845984500452947, 0.7594607650521482, 0.48371451567979523, 0.6321004003638909, 0.7607494400876517]], [[0.7873807602344268, 0.4471893286097225, 0.5128403323261401, 0.6899678757813407, 1.4016355720734208, 0.7422301339775725, 0.45469259924722194, 0.701399352671159, 0.4600703986950133, 0.37407480109034397, 0.07053007789392834, 0.5560299521251517, 1.593601706304697, 0.9271300928462869, 0.4181769105906756, 0.32084529519611765, 0.4560791573988262, 0.6389655714711938, 0.7448949331907696, 0.1934873905049696, 0.27321953775797314, 0.2166809608438465, 0.15174248706062984, 0.4875935611228014, 0.42050132367585724, 0.7652274528867526, 0.28237238177851576, 0.44705301481251225, 0.43348736982702296, 0.3703723079175426, 0.6727825407701157, 0.7516157904916159, 0.5179860991657157, 0.676040199527595, 0.7418066030931852]], [[0.8161235160099539, 0.5822413003110362, 0.44853359414351734, 0.7293245644701134, 1.479198604865983, 0.6524432616564576, 0.45671939828131747, 0.7041656371874259, 0.39681702692070775, 0.30900816430715994, 0.07643873624033896, 0.662728137719075, 1.5849970583123572, 0.838412963144082, 0.407266194192861, 0.3292608416906813, 0.5318682464073434, 0.6501978800005015, 0.7483663934066078, 0.1824055194860272, 0.2831603687942931, 0.22361445678223585, 0.16774775268811762, 0.38701081711065183, 0.4473252159402656, 0.7276303603843131, 0.2366562264150563, 0.4711755905005596, 0.42360407274308814, 0.3590298963095607, 0.5128791179320388, 0.7752811785025597, 0.5964653810967245, 0.6942343237253199, 0.71261379347531]], [[0.8185782172980101, 0.6262610386737673, 0.48901399684259284, 0.671606293875695, 1.5156481724645121, 0.5299109825991581, 0.41524060404707586, 0.6435915463810782, 0.3840857085934198, 0.2752411063028437, 0.06843718611648437, 0.6715422273450952, 1.5678568406374598, 0.7881410372774915, 0.3961685552328073, 0.3265147089474851, 0.5349662860104328, 0.6424713904715231, 0.7429876885489781, 0.18418569466614057, 0.2846240983210647, 0.20397576916532312, 0.189056153431741, 0.30193105808243775, 0.4501275327749421, 0.6827091788028568, 0.21972774646269716, 0.4680009269958531, 0.4074788927354659, 0.3689918179775965, 0.41334766103919385, 0.8477510213873469, 0.6639734102015802, 0.7285686852626774, 0.7179965401218399]], [[0.8654668648231385, 0.7208354996044173, 0.6389483012246091, 0.7669460293801815, 1.5215239763267017, 0.5575314372407945, 0.4209164693442279, 0.6296825898680358, 0.4058901632097384, 0.2287577002428875, 0.06871259571832335, 0.7010832100901527, 1.6218302587744668, 0.7762716521467413, 0.34794447640116866, 0.4042186445823081, 0.5804374841950386, 0.6434085350172464, 0.7387104985525205, 0.19901219128043893, 0.28482202674365814, 0.2036480444488208, 0.20566395907571602, 0.26366008884009023, 0.42149272173992214, 0.6995123686802908, 0.20733986511943542, 0.4883960420792559, 0.43072749615716244, 0.3881203990402119, 0.3954272792802156, 0.8729726058102651, 0.7176714684413079, 0.7803364114568774, 0.7417497343627175]], [[0.896895718196555, 0.757914197471631, 0.6384399226791471, 0.7924935969003346, 1.5302452411162069, 0.544120605666034, 0.3975610497319779, 0.6185589551958719, 0.43562209783384837, 0.23331209375751694, 0.07784939133886623, 0.6554588243415322, 1.6152540504095083, 0.744359043495609, 0.347075232244474, 0.38077563636151135, 0.5473603697511435, 0.5877562049262282, 0.7507247767442045, 0.19568000763111992, 0.28431694453114087, 0.19036510928995606, 0.20098002073125087, 0.23520705289970406, 0.34341461059159556, 0.7190669369859446, 0.18961809007248065, 0.5679760816896525, 0.54544199996917, 0.3622555983873058, 0.37954236594507657, 0.8195649011249727, 0.6629140764516926, 0.7921129731542595, 0.7027209167132549]], [[0.9203260869888996, 0.8144619246028881, 0.6828755569792012, 0.959210066532751, 1.5628018814443763, 0.531925244962269, 0.40253964391428076, 0.6645814292256518, 0.489688989027541, 0.19670773109654852, 0.1006829629006123, 0.6256947320651383, 1.6127067815677105, 0.5812622403179927, 0.33993017219249033, 0.36973300251240965, 0.4962876586296814, 0.5631417418636825, 0.7519894370876097, 0.17432152826407404, 0.28485451683302065, 0.1918697392493999, 0.20246811427958483, 0.28337133638598055, 0.2919635539000446, 0.7144143799326013, 0.19152118311596672, 0.7340931309016713, 0.8613728853030438, 0.3889987523167603, 0.4615814040373152, 0.7853566261936664, 0.5942446460709199, 0.7925321638751508, 0.693808820403967]], [[1.041940374200777, 0.8372131846276653, 0.7813798486366592, 1.3508688690297974, 1.6067667886632062, 0.5281256154965785, 0.43012743741197845, 0.6707051103667925, 0.7449807027200612, 0.16640451831056996, 0.1129899986869342, 0.6341173197394682, 1.564052567616587, 0.636592413580318, 0.3390160379454189, 0.4026743232123423, 0.4751596178071792, 0.5611657602333142, 0.7782389975180344, 0.1490664542896975, 0.28167788676912187, 0.19407992908971328, 0.17607329141720976, 0.3820753999537426, 0.28712081285875407, 0.6600158073182203, 0.1695577395744671, 0.8657854394599827, 1.0955260309253534, 0.4212706338187766, 0.4858989067198487, 0.7556820776674817, 0.5787296253413015, 0.7870060674100896, 0.7191364683382502]], [[1.1525566667718712, 0.8102458493225483, 0.6990971192053411, 1.5148839319424774, 1.6401052139197294, 0.5383190722275519, 0.41891177408242847, 0.6597951795779382, 0.7895903478370316, 0.14979694279447514, 0.197295228084903, 0.6359528641358073, 1.4874233443219136, 0.6600860730359601, 0.35332424224586784, 0.4271708566532126, 0.4281981217302898, 0.5412369304694784, 0.8143718694401196, 0.14301375422589113, 0.2640868496278598, 0.21835493485773835, 0.1763848547521343, 0.5111229101528812, 0.34330280176487404, 0.6841685494698897, 0.2657996585320921, 0.8661727962265411, 1.0205788079133202, 0.43439350228802265, 0.66513200217944, 0.8143837193915355, 0.7259016083457954, 0.8226763984543988, 0.6900409205130424]], [[1.093640172390843, 0.7788141054586314, 0.5946359007896103, 1.5608867477677615, 1.6614374455695748, 0.549598502369835, 0.4196095416729708, 0.6264303398610717, 0.6555750012272695, 0.13840386717162212, 0.1460612992150187, 0.6499657021349281, 1.4330793890911353, 0.6337884925123396, 0.35640120604138203, 0.3891697171993735, 0.39982294892718717, 0.5146452321732938, 0.822046254487904, 0.15171466146083687, 0.24353288077187285, 0.23275913218125907, 0.1994449548689671, 0.6048744703953595, 0.386205320795528, 0.5521444911802861, 0.4212493600147379, 0.839221110952715, 0.7914682960254391, 0.5002068795486411, 0.8863885039022986, 1.0337589076381888, 0.9202529160597877, 0.8606114960622506, 0.6623190091401563]], [[1.116821827491675, 0.7949957839630581, 0.593388308284768, 1.692964942125353, 1.6615726400750201, 0.5363783091244221, 0.4415829517309683, 0.5827666552999745, 0.6116215471121018, 0.12708636696933998, 0.1521503434458551, 0.6679721805561232, 1.4588682207461887, 0.6350851278569016, 0.2836503386740137, 0.36917378796119105, 0.38166189616068574, 0.4966931353753584, 0.8474559334514155, 0.15204517200984813, 0.24234840034279703, 0.23205088920342687, 0.230371917824988, 0.6329066675469758, 0.4078626650045821, 0.5396101329996915, 0.39842694342011464, 0.7608559704656637, 0.5932099197251629, 0.5925102245871197, 0.9886840707366716, 1.1197938000488472, 0.9906635204491878, 0.8746764739462822, 0.650592520537465]], [[1.0914183640451252, 0.7775115273607934, 0.5941761786153337, 1.632829268778253, 1.6760340518351637, 0.5070328591952045, 0.4488140528530582, 0.5839683950359079, 0.5721930102897612, 0.1222019041360054, 0.11073958043547566, 0.6866055866081471, 1.472625772039751, 0.6561932226837205, 0.3283175937866035, 0.38365030403809874, 0.3817844055980851, 0.47656426269971497, 0.863122986915719, 0.1271453287054254, 0.2362518506532374, 0.22800772673491387, 0.1924799815904231, 0.6263094850245239, 0.44623924405106447, 0.6123189080346116, 0.3861125023898755, 0.7316610929301703, 0.5282419324798896, 0.6538769826163147, 0.8526886201809545, 1.1082209632825315, 0.9713664784709912, 0.8688198684073952, 0.6461555403809908]], [[1.08428334634484, 0.7708180747566413, 0.6023200210265669, 1.5716748917511176, 1.6714286831737835, 0.5186628055488001, 0.4477271008566457, 0.6584006741175608, 0.5077661093196357, 0.11062036585884791, 0.09392642230914339, 0.5895581755449142, 1.440556370174349, 0.6300668577040952, 0.24878430701332888, 0.32150076916339054, 0.38828127412762115, 0.454641697379089, 0.9100602786772309, 0.11663468499744306, 0.22531638366293996, 0.19086215207457763, 0.2033492730684436, 0.5711060546005284, 0.5999674912254386, 0.8108068279351808, 0.37431069145445445, 0.6840470675557867, 0.5456994498486946, 0.8538035141073493, 0.7155501963998816, 1.0721922958451229, 0.9201909358686504, 0.9034118668284955, 0.6744929794308597]], [[1.037079642841493, 0.7690272918295091, 0.6174856357222749, 1.6329253211798322, 1.5968910211615952, 0.5173663458027676, 0.4511131525533904, 0.5823465625034504, 0.4882976381810391, 0.0847487127214287, 0.07266484493260746, 0.5753996455506394, 1.4025117280220476, 0.6506094225041408, 0.2472214072413667, 0.32758576727968114, 0.3979830717934534, 0.49452848967841584, 0.885675118724716, 0.07662561222271812, 0.2316807974981323, 0.1650514427197608, 0.205523223506773, 0.5166023546795289, 0.6101131072940856, 0.788316418825425, 0.4581667570986148, 0.6326957749466519, 0.632204998981647, 0.8767103633038643, 0.7289497924717216, 0.9910015210046529, 0.8752641157535715, 0.858227543675474, 0.6232003187743533]], [[1.034064951723893, 0.7482412824184994, 0.6155860864531242, 1.6401889958578815, 1.5283144717311976, 0.5431011048594011, 0.462427130994663, 0.601324398160696, 0.43555091254635747, 0.07952493244656125, 0.07354658318096356, 0.6227932518282951, 1.3510723186210287, 0.6311526474459697, 0.2514486670164533, 0.36970177823370315, 0.36080984578038017, 0.5212456300446546, 0.8633075549668354, 0.0574656494844266, 0.2514522052133653, 0.1697287937127172, 0.15980153099324204, 0.4797634042229014, 0.5712098719611571, 0.6476536759616454, 0.5626608506990044, 0.723809467886307, 0.5409785932570973, 0.9117261205825271, 0.8156713340384222, 0.9507262261396382, 0.8470914563650537, 0.7575713322284898, 0.5343240470250563]], [[0.9901648009363329, 0.6346961301653459, 0.5953240709972896, 1.6721389132507616, 1.4276243619965838, 0.5642761694714331, 0.4505190105153548, 0.6041138612823246, 0.4820558765430639, 0.18451888820181195, 0.06967610391307574, 0.5624732320277668, 1.3275802177933695, 0.557616211597804, 0.3335130078352616, 0.4846025179575911, 0.38344479315088265, 0.5669344852760576, 0.8670692219683049, 0.034323051490610046, 0.27045485755967086, 0.20371959361961267, 0.11641387007171612, 0.5262040120531878, 0.6297072201299557, 0.5130803711229195, 0.5230678874510605, 0.6963869002095882, 0.473416447853994, 0.923352628383027, 0.8628959609869862, 0.9269598624819181, 0.8428815054995711, 0.6795978881926977, 0.484059532343971]], [[0.9752371799916442, 0.6007020577883179, 0.5689425216410793, 1.606263872352914, 1.3225107516955688, 0.5689195633847431, 0.43397470457869414, 0.5994122060609475, 0.5672342000693062, 0.26926996478237586, 0.0659122640299436, 0.5617162415275183, 1.2865992673476294, 0.5834461260575372, 0.3231256988477885, 0.3781781078091764, 0.43104595290271774, 0.5117586823078291, 0.6952636825983218, 0.03345840411907916, 0.2528133687849, 0.2323177142746783, 0.140613660098035, 0.5508367522319473, 0.552711761712235, 0.47510177055156927, 0.4096670677048899, 0.6239746098494243, 0.3621281296658414, 0.8738111060380521, 0.8690609187405627, 0.8869598725578477, 0.7803225960758371, 0.6590056877256274, 0.4423799260118839]], [[0.971739824526605, 0.549247082513822, 0.545108607142375, 1.5541621444600917, 1.2499939569281722, 0.5823935895806038, 0.4757433839833919, 0.5999930412590688, 0.6125371860675193, 0.3163702868712932, 0.06513270126057757, 0.5317433777828448, 1.2128915009032673, 0.6357729986912336, 0.31877731958035516, 0.44114862883771855, 0.4514756972884447, 0.4733785461516305, 0.6514876452498611, 0.03483229778768546, 0.2471046630062046, 0.2103776759939096, 0.09848736673684332, 0.6458534077218286, 0.5493239305484167, 0.48128289128446017, 0.37221922729625695, 0.5682118044362678, 0.3144325716661297, 0.8483157505593659, 0.872391818680742, 0.8650396208099613, 0.735247276062308, 0.6588555616267802, 0.4132149226795777]], [[0.9406436230003759, 0.5279719166601103, 0.47159252605796004, 1.5106395944000406, 1.2173792171304605, 0.5877984452461285, 0.40164158669206285, 0.49314484674123477, 0.6369172382431483, 0.30906424153270595, 0.07965956431007583, 0.5427702853507923, 1.126428332661948, 0.6368430428420156, 0.30390055537152993, 0.4414597468991329, 0.4343715161258451, 0.46787251780130323, 0.6205950557040218, 0.026108890586311898, 0.23196095917345344, 0.19761605325144138, 0.10517959230717874, 0.8042182964010761, 0.6054442273750572, 0.5651868367869619, 0.3859751638496021, 0.4798946463473792, 0.33420536190310607, 0.8178912484040624, 0.8943620997149724, 0.8603560870500154, 0.6654578603169082, 0.6258752665160912, 0.3930119714908782]], [[0.8603341350791492, 0.5397263184820813, 0.514715241070552, 1.491350943897564, 1.1767932040879028, 0.6278972663591891, 0.36171833664597036, 0.5103975222573209, 0.6494908117013921, 0.2949764765520517, 0.11395146718502067, 0.5423133664076021, 1.2368514375908706, 0.6579846423016128, 0.25069254499907206, 0.412818393384085, 0.44728745437926587, 0.47505935237489294, 0.5832164223725056, 0.029270995340349215, 0.22655773358531572, 0.18092685240780793, 0.11833374143580017, 0.9728102822371066, 0.45738683463827734, 0.608162757019888, 0.3682190984648338, 0.23963918690207142, 0.38226820424286595, 0.8833859287338097, 0.9307080238410497, 0.8411550819693402, 0.5304814537436777, 0.5966202623672101, 0.34164418391710893]], [[0.8009734924394442, 0.5092088170577613, 0.46182977757145527, 1.4283387576669269, 1.1539815493005687, 0.6486830592014305, 0.38417100257148545, 0.5227950659758193, 0.7425499243957365, 0.29253499609604916, 0.10897635210984655, 0.5329751321552159, 1.3514349596020558, 0.6238151565050575, 0.2621194814067468, 0.41913948346502117, 0.428248844958786, 0.4661772472147048, 0.6146886804294547, 0.04237900908543629, 0.2575662310729559, 0.17621757795561194, 0.11846802013455046, 1.043290677252305, 0.4052274783629794, 0.5392661071812019, 0.365537017324318, 0.27760761845849974, 0.47166668797171746, 0.8221625277944702, 0.9717441288021083, 0.7963501300385628, 0.429672937358373, 0.5496672220221792, 0.3209063807656404]], [[0.7747456135152103, 0.46863057533164176, 0.4152545186321054, 1.3828886651722108, 1.1004589336777792, 0.7210587763759082, 0.3628209937836224, 0.5398683139754106, 0.8753434402686724, 0.32408249122923377, 0.13855979688771788, 0.5190264341356818, 1.3628950252882226, 0.647011998033205, 0.2176676863998943, 0.4283865386949295, 0.4000835566806794, 0.5029937143827479, 0.6191452576434004, 0.035698406560531054, 0.25599450675330315, 0.17347106743374588, 0.06555372824699995, 1.0017759127378127, 0.38204149291871864, 0.3958178446123174, 0.35158707197542416, 0.4569514923882084, 0.5295606200842229, 0.7709122894724737, 1.0132659981985122, 0.7524250816894672, 0.38490904194945325, 0.47634615218028253, 0.41899765397665634]], [[0.7207119915086396, 0.4300763626921764, 0.41624237117860363, 1.5623415040022566, 1.1072603343018241, 0.35137942379196807, 0.08971909621893365, 0.5134050157146196, 0.9843869821063057, 0.37041645370759974, 0.1274523606457266, 0.5126142631459019, 1.3197502390957507, 0.730960623075581, 0.22575656340549313, 0.5466207238502926, 0.43954798799340855, 0.4958889908660341, 0.6036428279825523, 0.030921883356372415, 0.28821122981011865, 0.1763200595824634, 0.0730562486876364, 0.9847074731530008, 0.342752341179116, 0.21298753651951327, 0.24181287282378, 0.5702837395026452, 0.5020955056434309, 0.7463200389459297, 1.0511235204360037, 0.7089673819089638, 0.36258718961456143, 0.5543989591365635, 0.5886045723382974]], [[0.4366402862843809, 0.4128558732258908, 0.5074555344669287, 0.36998874089026557, 0.3999191424861622, 0.4323264014928433, 0.4166556336902915, 0.37753058662776073, 0.38066751011617417, 0.24083541778245743, 0.8238585625999513, 0.5380465825697971, 0.34668196327136724, 0.548392642847571, 0.22088650097579232, 0.45676116877394746, 0.30120978070067045, 0.4472035422983923, 0.5617662345281436, 0.05077948356492251, 0.20334266062494918, 0.0871116654644078, 0.024490465224494683, 0.21410065294948485, 0.43503863611127624, 0.3648941821340038, 0.5354403261018396, 0.6127366952392386, 0.7779219908179418, 0.4788681351815879, 0.48251243686501566, 0.8280446408107671, 0.6781529613264872, 0.5054529728894042, 0.4732976175236684]], [[0.45150016662983916, 0.4385920075960469, 0.5285036510651273, 0.4000738761975456, 0.4162440886317001, 0.47028887871487746, 0.44272089062914116, 0.412727713455863, 0.39775673125195965, 0.306818497858062, 0.814344425732914, 0.5478459173630514, 0.36293958917966973, 0.4898261303343275, 0.2427155562078659, 0.47122984757264536, 0.350868101595287, 0.5247623013743159, 0.6426625729730063, 0.0597349258601656, 0.20177595226685432, 0.09642420401228131, 0.021065979442174643, 0.3557445145181156, 0.5480468944438219, 0.4721701664657821, 0.5867187666103557, 0.6613395627224843, 0.783223352662267, 0.46338711356906953, 0.4919679404197468, 0.7929494936732757, 0.6766981401484997, 0.5711356643556977, 0.5079023982697932]], [[0.45753177264181655, 0.45045520871725064, 0.53780532681917, 0.43505114681563717, 0.425474629952249, 0.48824375782623186, 0.45519934609870105, 0.42706195985145834, 0.4154385052695504, 0.3431167618055263, 0.808533693750394, 0.5361101558709189, 0.4398191544712632, 0.439099020161007, 0.2741329131568493, 0.4916329366444442, 0.45700758962576693, 0.680156846163592, 0.7313557911322708, 0.07694549057467348, 0.19609542685564085, 0.10773241731909874, 0.015168880247418155, 0.4267125636149309, 0.5962491710892331, 0.525943261408125, 0.6137125343374463, 0.6902103837257839, 0.7787665332437808, 0.44357072085970695, 0.5090243246845386, 0.7706417724934447, 0.6772022994681842, 0.5788936933449234, 0.5053690374588764]], [[0.45979515165556517, 0.4547326654665754, 0.540880429454219, 0.4559801091985579, 0.4298573159405313, 0.49536539601150825, 0.4567634415595554, 0.4132883139649642, 0.4240082097585847, 0.3534369589759344, 0.8084819134610091, 0.5675106190100304, 0.37979093722568963, 0.45576386943366554, 0.22419162063780862, 0.46683953068938533, 0.45399978866912305, 0.7208656649678165, 0.7101390392226617, 0.07965623656395102, 0.19642132135047016, 0.1103102882175509, 0.013743195423061724, 0.42395520300132883, 0.51508468068069, 0.4949110017993053, 0.6196142646440768, 0.6797157572310563, 0.7708432592095245, 0.4217482990171287, 0.5271916474527721, 0.7759450181815064, 0.6796158288164778, 0.5711155044341867, 0.4530169975956949]], [[0.46146421133285676, 0.4577178994389727, 0.545135744837992, 0.46026320789616715, 0.43342074292992105, 0.49981383336649865, 0.4582466529572696, 0.39985994073982245, 0.4316705444660307, 0.36181292366208867, 0.8085493706637437, 0.5567597105750179, 0.33915200009408575, 0.44467488029800456, 0.3029186005154968, 0.4391886144651993, 0.5276783060631907, 0.7197078917507064, 0.7516979605530444, 0.10272610996614479, 0.20520828324614793, 0.12089795514672634, 0.01524191477717628, 0.42427363212240915, 0.4574722548152747, 0.4774250254152952, 0.6283072539573101, 0.6718902449114103, 0.7697650726374048, 0.40995211441345286, 0.5343365531644305, 0.7654594826068333, 0.6808654424839817, 0.5771164508266466, 0.43138966922472277]], [[0.4625927646931165, 0.46024368634365964, 0.5471074005342782, 0.4601258934203611, 0.43727164311507627, 0.5039753739725655, 0.4597902408047132, 0.39578950040608163, 0.43851633165926, 0.369895498772676, 0.808568484353575, 0.57367746268494, 0.3559613896808037, 0.473735289742945, 0.2906750860820747, 0.514135706492863, 0.4189186313048609, 0.6562480038903743, 0.7881424487071258, 0.16331472468601127, 0.22950699043237408, 0.15853392031168506, 0.02409025528603339, 0.4278822083518611, 0.4270927757078621, 0.4703294369881656, 0.6375736597508288, 0.6686509008527519, 0.7726100925740216, 0.39183196171539736, 0.5406685877341445, 0.7550628302707998, 0.6815153329478405, 0.58307374645214, 0.42586417201607335]], [[0.46217157712038104, 0.46266435024196584, 0.548470511770734, 0.4601614284081632, 0.44171220169809855, 0.5086440814594342, 0.4607443356805069, 0.39477105883200003, 0.4459966182534758, 0.37687224140612574, 0.808241120664036, 0.5417641688015014, 0.3286844262729297, 0.4691146555755463, 0.23093014421933478, 0.671971752195775, 0.39826870342095344, 0.6502664400984073, 0.7083371766802327, 0.1973619463351294, 0.22950642856253944, 0.1516898730508725, 0.020584200400195037, 0.429502600323681, 0.4144120969010169, 0.4663326680642751, 0.6432664697301531, 0.6694065185204485, 0.7784330632215755, 0.36876598086849904, 0.545574484264989, 0.7397628514864485, 0.6823610136489275, 0.5872382168867959, 0.42789180644474256]], [[0.46105943304542385, 0.46526263700966064, 0.5501194535503182, 0.46420799208933616, 0.4468978403130312, 0.5137712848395257, 0.4622779246969403, 0.40389376515072256, 0.4532800642882796, 0.37943644248827973, 0.806486302382912, 0.5141420794217778, 0.29870492921147296, 0.44546923013864176, 0.22184519056275387, 0.5580823795729241, 0.4452330792134341, 0.6812759941915455, 0.658773897362602, 0.18610050268262776, 0.20959173837945358, 0.1611155861976205, 0.040849941114479656, 0.4338650985220389, 0.4120013460507313, 0.461320525007718, 0.6463590365927199, 0.6694117535212524, 0.7828754843337056, 0.3547526835067911, 0.5473239803214356, 0.7191528387650168, 0.681821129354379, 0.5849738885356072, 0.4307932477298253]], [[0.46001240214259487, 0.4679352023534176, 0.5506609679591448, 0.4750234599945193, 0.45174847126115647, 0.519922378636944, 0.464070369056718, 0.424554202153802, 0.45429457182669897, 0.38154464299175134, 0.8047747785732577, 0.5242873514695815, 0.30692899295452036, 0.45628143045359115, 0.24858709229390596, 0.4982195987814272, 0.4753178879796601, 0.6849143561075579, 0.6626722554109921, 0.16762883476383753, 0.18874248226591803, 0.15490880467498702, 0.0342176440790063, 0.43308128517261885, 0.4192323312626699, 0.4602833973826509, 0.6481678915156708, 0.6699675532265035, 0.7859451288048512, 0.36351431202355516, 0.5588232268070417, 0.6974601241622524, 0.6738172447083031, 0.5758201165223806, 0.43748240327443155]], [[0.45873183830486897, 0.46994188301426665, 0.5508265123968743, 0.48119922897189005, 0.45496298567262583, 0.5236901015715316, 0.4655870402237631, 0.4424517059373669, 0.4503119010150903, 0.3795034112675542, 0.8019193220902545, 0.46075054503478546, 0.33492462549376256, 0.44723951356985625, 0.23809021852759682, 0.6232500664112447, 0.43803440823497675, 0.6368541420042587, 0.6650755645581676, 0.15634582966483893, 0.19552663110173382, 0.14581480060419547, 0.024665056774770586, 0.43848578625292667, 0.43059521166940107, 0.45916443904552834, 0.6507661276516516, 0.671940842006558, 0.7897849718378427, 0.4124391015978136, 0.5636754163478185, 0.6848960559395632, 0.6641112061707367, 0.561261399261205, 0.4295812156439883]], [[0.4576843581095002, 0.4719490591712281, 0.5513687429818945, 0.48505538795901015, 0.45758429070551526, 0.5275022621092618, 0.4675967576748489, 0.4587137453914758, 0.44408738454287344, 0.37680363792000054, 0.7998090001786151, 0.4298290632474968, 0.3555408707235352, 0.45890656690767706, 0.29418394764969447, 0.6001647003639639, 0.4528321790022112, 0.584335691615187, 0.6994051185995238, 0.1579560421554908, 0.21829360337547485, 0.14238478362640655, 0.016865866961061304, 0.4446759829904841, 0.4446549991095468, 0.4589493383341427, 0.6548366489410757, 0.6752130808098826, 0.7955726360695614, 0.47300859306181625, 0.5715998341012027, 0.6804047272214097, 0.6553408874851341, 0.5399803140250327, 0.4264753805944119]], [[0.4560302696863099, 0.47264577704611854, 0.5508698640931837, 0.4868236673987213, 0.4585235679735736, 0.5290644158874528, 0.46863455422348, 0.4680998434431177, 0.4356225172352176, 0.3762563267655998, 0.798501250401746, 0.5406353045066246, 0.3468269603253611, 0.49541720144892876, 0.3028974152497226, 0.44780660689467455, 0.47342221838926823, 0.5949002717387817, 0.6930225861596224, 0.16461494471838997, 0.20932262940411114, 0.13147202245118694, 0.01120791656162445, 0.4546308822357652, 0.4564513647947094, 0.4582021314270457, 0.6592131734313431, 0.681164901056956, 0.8019756378585293, 0.5205350736505772, 0.5834778861126855, 0.6794819287458767, 0.6506536650626656, 0.516831024213095, 0.4364682010738088]], [[0.4553657320086141, 0.4732277700153813, 0.5514222150263558, 0.4884337622262522, 0.4579515785005649, 0.5290455094349538, 0.4689415241800459, 0.47239247087432756, 0.4262861611864771, 0.3761184793056496, 0.7981573830025019, 0.5210619464909878, 0.30810796392527995, 0.44887018183695726, 0.29296326047815946, 0.3897895458861789, 0.4795066972488574, 0.5494107883097987, 0.7272952060930065, 0.2117886538149053, 0.2482463222808311, 0.15412253987332714, 0.013752572707244374, 0.45736060202804224, 0.4698921795204648, 0.4592781477589803, 0.6647710564044245, 0.6859472181296471, 0.806553657923654, 0.5473027100361252, 0.6052721930180609, 0.6871180677114683, 0.6490654736553493, 0.5008458674856209, 0.43737655595048763]], [[0.45472799074663134, 0.4732269970517958, 0.5512992795545166, 0.4894973503591103, 0.45736516483106116, 0.5289127277615178, 0.46880002647252245, 0.4728628353596409, 0.4123400358516956, 0.374482685747154, 0.7977307760855448, 0.5595731111508842, 0.26597507555866473, 0.4093249918855807, 0.28180275030622526, 0.3684389132805883, 0.46386912545465453, 0.5168704978000138, 0.7527439214048459, 0.24527539612762528, 0.29303666695506125, 0.19364869103661642, 0.016543795646134625, 0.46835156974440184, 0.48056738833289236, 0.46080509123189983, 0.6719279318670504, 0.6920292638255748, 0.8093564578258853, 0.5559315042239441, 0.6323507789786154, 0.7025178763727689, 0.648453888616294, 0.49278823626885865, 0.4272321995200069]], [[0.45341465819601356, 0.4709543198014595, 0.5511405299405288, 0.49011773338918185, 0.4550793869848835, 0.526496340737882, 0.46681212867697974, 0.46319379234756286, 0.39938529738780143, 0.37162661834174626, 0.7980626829916813, 0.6575140236149721, 0.31385109834374425, 0.43097210193799673, 0.278156357936363, 0.37012662986516, 0.4902098921292489, 0.5121120243467797, 0.7612126856052049, 0.258505382707102, 0.3479946609437476, 0.25930755317076115, 0.017415838409365312, 0.47865883146437066, 0.49324945974168216, 0.4580041624111498, 0.6816811227395514, 0.7000821936632675, 0.811987296410074, 0.5581596913633198, 0.6460996331216282, 0.7259197704652278, 0.6498465836556488, 0.4891073492835589, 0.40968323185841904]], [[0.4526530949155654, 0.46852322314512757, 0.5496699099383315, 0.49615755593291844, 0.45171068941140263, 0.5223104044415701, 0.4642829292858047, 0.45474510349315106, 0.3896203601923202, 0.36768612475575996, 0.7995831449693906, 0.6549589074810203, 0.3270201963433015, 0.44222584354402106, 0.2702843174643555, 0.36942426709157217, 0.4659336638114404, 0.5098579864928945, 0.7371195640207122, 0.23715549728238666, 0.3531841252218697, 0.2834150317179995, 0.02413086598870451, 0.49075616704472597, 0.5012528174639825, 0.4553152533054502, 0.6896729852891823, 0.707237760674793, 0.8136622562665089, 0.5558748753452494, 0.6464769025711198, 0.7500175159407361, 0.6539890627624466, 0.47992789558963767, 0.40294861806484006]], [[0.4522312117470484, 0.4665242286015925, 0.5488197857026762, 0.4994291254290225, 0.44788319898619133, 0.5182828353611793, 0.4621047672090555, 0.4474551372801552, 0.3788066190448077, 0.3635362329149495, 0.8013445367355994, 0.636569275000926, 0.33365119416623645, 0.4348817076914248, 0.2741838463948399, 0.3724219783285266, 0.49302980723655165, 0.5148837123809096, 0.6924588441628742, 0.21104583316818182, 0.3443321906013849, 0.22390517930752749, 0.02017699809015406, 0.5116957480453012, 0.497835489909483, 0.45625716405329353, 0.6939611521812368, 0.7111083154953907, 0.8103438384310258, 0.5561344206710523, 0.630199531512073, 0.7733634602224703, 0.6579602262953732, 0.46431994374578717, 0.38657463196661035]], [[0.450793326300129, 0.46237570078815615, 0.5462972228992283, 0.501598545725193, 0.441947750588661, 0.511457750149942, 0.45844237033216106, 0.4429507555124197, 0.371592955478647, 0.35791818548435095, 0.8038895053448071, 0.548598214756067, 0.2616060396077061, 0.41911418293490665, 0.22994683176238045, 0.3807301469128443, 0.5069936272383703, 0.5278530857355808, 0.6653684329342748, 0.19833975441769514, 0.31966422491065466, 0.15803358829001302, 0.03601685523241663, 0.5369699909114263, 0.4974508567029107, 0.45573007573567453, 0.6963892934406377, 0.7168468170802045, 0.8047838808821512, 0.5514072126877232, 0.5937007606623886, 0.7733274876197231, 0.6642634079678822, 0.45053383353392684, 0.37868801295806653]], [[0.44989519939140654, 0.4587682254443104, 0.544623387223943, 0.4969249027271912, 0.4365648332932743, 0.5067098040991302, 0.4553256295567352, 0.43873848771326995, 0.372615695461885, 0.3516622992717411, 0.8060974792367874, 0.5624105700039019, 0.1743581026461617, 0.43248834384788837, 0.243273042599843, 0.37884844572606735, 0.5447739538981269, 0.5168670428527609, 0.6490061704204918, 0.17661943935710933, 0.29643323457053994, 0.13091943307193166, 0.024364726377321073, 0.5638637255271447, 0.5067499998342717, 0.4561339728408002, 0.70316451595405, 0.7230678202743088, 0.79693006814434, 0.5471690433842832, 0.5713541646433417, 0.7756540145215389, 0.6718296603947289, 0.44329586154849643, 0.3696353320696736]], [[0.44859439995358635, 0.45535283484230715, 0.5423564022690347, 0.49337561488583453, 0.4320633187411881, 0.5020668954516271, 0.45288627451142127, 0.4351698267674894, 0.37753702315566773, 0.3482348020290845, 0.8079517965604075, 0.5608073482785835, 0.11338576736038747, 0.44591054506976335, 0.23524926035790467, 0.38300326628239134, 0.6084215784413048, 0.53688303861013, 0.6137070839920029, 0.18057792247248067, 0.2974276993936949, 0.17736205642807473, 0.025349751410587866, 0.5824686519106015, 0.5150444171172758, 0.4617312014576721, 0.7077385321425413, 0.7300344972530886, 0.7896745293559178, 0.5365839408271412, 0.5579788579785586, 0.7766637714932099, 0.6774705376705834, 0.4375387885296873, 0.3574185006308256]], [[0.4479261035608664, 0.45319898203308173, 0.540284668690717, 0.49525046601015743, 0.4291782650571658, 0.4987751020924207, 0.45147933735999873, 0.4335678443293668, 0.3842679862903028, 0.35032313349180993, 0.8093162269976442, 0.5600291326567948, 0.14752583857514553, 0.43672794886573507, 0.19167808252541835, 0.3881469351629131, 0.6381129306777641, 0.540616781342971, 0.6121760971975831, 0.20459433073469957, 0.3210105349475061, 0.23182483945110238, 0.031250207175180164, 0.5988314784848106, 0.5293225436538899, 0.4728515061771219, 0.7130038803407457, 0.7377109186298487, 0.7815941677010214, 0.5261292169850104, 0.5646625439395423, 0.7773545086337017, 0.6837934952624656, 0.43309188150523664, 0.3589263532782241]], [[0.44863270502234703, 0.4519252784250605, 0.5390846075681337, 0.4965292718010157, 0.4264880588580247, 0.49607526221147324, 0.45154616486304106, 0.4327619496556469, 0.39380976044937793, 0.35696512178175155, 0.8107283244094861, 0.5619173565195783, 0.09906236982462531, 0.4704015076503013, 0.18918664651613332, 0.40053807245916734, 0.5661960527826095, 0.5315284343619318, 0.5772606235094973, 0.18182418377856135, 0.3111075139245833, 0.25252750519395795, 0.044654765695838164, 0.6047922938873224, 0.554434429857556, 0.4919001030759074, 0.7184763865919884, 0.7461834369927332, 0.7739637389041385, 0.5177830793244561, 0.565304843427094, 0.7774779486218549, 0.6928983612650725, 0.43187926651290065, 0.34923929347484706]], [[0.44982232899077995, 0.4503947938117357, 0.5375994913602788, 0.49612321461524195, 0.42289416146352615, 0.49269152363830593, 0.451247506806382, 0.4325467557250674, 0.401102000855057, 0.3669014794346306, 0.812160863812154, 0.5994272401894222, 0.09356097844423765, 0.4762902764205575, 0.1924501093103989, 0.40829373879507014, 0.5718759040011185, 0.5163687560743982, 0.5284084221378808, 0.16080276209350008, 0.2964823878939607, 0.2710836473334863, 0.0530265861620077, 0.6013501975834402, 0.585849414697062, 0.5273538291999916, 0.7200247821246664, 0.7540337744944827, 0.773647183309604, 0.5182127770826862, 0.5630584480439232, 0.7739429820827579, 0.7037030633735353, 0.43467631476629653, 0.3436871725528563]], [[0.45197941306525785, 0.4512396347642934, 0.5378238224667078, 0.5010849010840192, 0.4222567504816354, 0.49254547387765446, 0.4531759615460444, 0.43265519930327, 0.4095553418561324, 0.3781115150755261, 0.8124261569833349, 0.5854323035001121, 0.13043686009721936, 0.442398161667482, 0.23881289874914208, 0.388058169397157, 0.5719415181131864, 0.5662889853138338, 0.6975071488938351, 0.15667477933026042, 0.28814424435080965, 0.2680925347499447, 0.028085092464421618, 0.6011769920194864, 0.6280017262996956, 0.5792910521044945, 0.7165475329343947, 0.7629908303940172, 0.7713226403476904, 0.5124544937548197, 0.5632555740938902, 0.7686715638845543, 0.6997784806222368, 0.4398243854535103, 0.337588034635095]], [[0.45324907680012666, 0.4514995289189756, 0.537022855663043, 0.5031888689815757, 0.42086489334309657, 0.49217422257793536, 0.4563775971718125, 0.4312682880241845, 0.41955911439062005, 0.3985371431747496, 0.8123423745024265, 0.5406908910022976, 0.15037706939538145, 0.40978119787809625, 0.23464660552827096, 0.3827548625334244, 0.5804217037526268, 0.5836237324446794, 0.7068254433497314, 0.13711010049333772, 0.2841488026655313, 0.28316638233325214, 0.035707651760611185, 0.599770906170783, 0.6742729513371757, 0.6395651866920449, 0.7132053407467736, 0.7790061395330131, 0.7800292717880954, 0.4981062189801942, 0.5709363821218221, 0.7606634435628346, 0.6952549566601852, 0.4458943203297982, 0.34947454561063485]], [[0.4523636949552199, 0.44894846987898296, 0.5358455480969792, 0.5111566643531967, 0.4173075088226399, 0.49156681311912226, 0.4561924358672439, 0.44231722827679454, 0.4255621082322991, 0.4117925467879351, 0.8121439222186114, 0.536212094343014, 0.17354802470199576, 0.41597939505849657, 0.22832284801167413, 0.4007057442050304, 0.6329867677946865, 0.5963913313242076, 0.7300168396548364, 0.15299748445862793, 0.27269011592120096, 0.3511279743929868, 0.0456279036466925, 0.5927284249243836, 0.703124225680664, 0.6895419785711947, 0.7098698243976027, 0.8027844324438245, 0.7988198076171193, 0.49291406382045655, 0.5758969024813362, 0.7523627407238211, 0.6924231593900989, 0.44865840870300483, 0.38233562738470495]], [[0.45088672759285076, 0.44780214050481665, 0.5346974025073905, 0.5156161151806911, 0.4157869976318184, 0.4908901185989078, 0.4550946094959459, 0.4537175097491307, 0.4270666745307309, 0.41655057361169107, 0.8113706967502934, 0.5370935321674772, 0.19570817704675517, 0.4106829519592588, 0.229807102839998, 0.4143650826958039, 0.6243399989684102, 0.5959508463624351, 0.7729445809276609, 0.1472980741670244, 0.25906292261448083, 0.35113169693219254, 0.03769236448401764, 0.5961139007483958, 0.7081709136039918, 0.7187509431655977, 0.6994958715221395, 0.8165860800971316, 0.7982341157745774, 0.4989812869352814, 0.5798902990334093, 0.7431858718297264, 0.6879339595018612, 0.45059548328557814, 0.4074497047738119]], [[0.4492313937125826, 0.44850656081587487, 0.5345180255570481, 0.5223812327452249, 0.4172601803259121, 0.49360033206021636, 0.45681061305252685, 0.4570557680305308, 0.4288125840312753, 0.41763666682572, 0.8087349304817374, 0.4735431162811157, 0.1846967942108071, 0.3963275031519648, 0.26533378213196995, 0.417074323774866, 0.6201080955541466, 0.5930667190270908, 0.7761291454546266, 0.15284464016197052, 0.22452162568615164, 0.343044584556147, 0.030124802324434964, 0.612594074789168, 0.7039576248841595, 0.7063001852514359, 0.6973870605911817, 0.8068677726228664, 0.7907479654364691, 0.5036097469793037, 0.5735415898855175, 0.7303773682979975, 0.6821045888761359, 0.45229281035982144, 0.4096586966097755]], [[0.908165094290777, 0.7973996643752481, 0.9852122201603449, 0.8453977854034491, 0.7783626301221723, 0.7871657663511642, 0.8480100092066218, 0.8336590513330171, 0.7821680299993087, 0.8148136963686776, 1.5605664214179382, 0.4465495012013904, 0.24640572586099874, 0.3729591764006628, 0.2569459694375836, 0.4770127863929834, 0.6431288409092611, 0.5009908434124554, 0.8778036323336487, 0.17009516818349724, 0.28032872310775664, 0.35262123528751127, 0.040763070286784324, 1.132258879147459, 1.0893494703302107, 1.198130229184527, 1.2697110588554503, 1.3635100806780402, 1.2687024366285053, 0.9121383922358823, 0.9804167884260362, 1.358912358433591, 1.3205841594123087, 0.9083731917484628, 0.7951140894803163]], [[0.9091403427638002, 0.8057533476016052, 0.9883501371193855, 0.8722612206953211, 0.7897022994829411, 0.8118231986165667, 0.8601255878901455, 0.8119537571144442, 0.7918672345641797, 0.8284880968030366, 1.5524520584302275, 0.4546863219145102, 0.17704874242146848, 0.3966713264604638, 0.24857653624022655, 0.39818549093952504, 0.6083552206502116, 0.5331498189909589, 0.8307436417258085, 0.11403672864663791, 0.29284279599788654, 0.3821319913111394, 0.0446616661157741, 1.2145729441949418, 1.1097665620355757, 1.1327952963862633, 1.283469879890065, 1.3611528560666146, 1.255762403833968, 0.9183585596176047, 0.9638329462675385, 1.3599372735436082, 1.2971826926753063, 0.9364025119479255, 0.9059136392195134]], [[0.443838332304374, 0.44095035950229383, 0.5285510422267177, 0.5395151743979221, 0.41195174840343274, 0.4892860404467838, 0.4505198874834328, 0.43980732215208956, 0.4254101887340768, 0.4157588461135141, 0.8079226817141395, 0.5804208184856686, 0.16670989874841863, 0.4578969639146101, 0.24214063804958025, 0.38475542650859595, 0.6212887954657821, 0.600105666679428, 0.7980099468928774, 0.12448931190316725, 0.2585030539112737, 0.3292172234034819, 0.03293743255604936, 0.6592764843915091, 0.6661533730629772, 0.6162860876015303, 0.7227114556967935, 0.825622085199225, 0.7584089969944366, 0.4830555412253851, 0.5114265648800886, 0.7359021192692197, 0.6620390419286794, 0.44107558308778044, 0.4595400697134029]], [[0.43260540483293897, 0.41408485119733757, 0.5117810987377005, 0.5258867602562252, 0.38510205867871905, 0.4538394274395571, 0.4244355033011768, 0.43625194914170107, 0.40199553124125176, 0.4037170140564897, 0.8222158735276206, 0.5927259444436577, 0.21034057071544038, 0.49145426950106286, 0.2496057844842401, 0.3888040206910858, 0.6004454017482886, 0.6026507360088204, 0.7839410311691174, 0.12520382106459532, 0.24979379917890832, 0.32065645099468165, 0.03217716124214646, 0.6308150242126209, 0.6635611285052061, 0.5534491631934952, 0.7392724839676248, 0.8501144136027952, 0.7292535252740124, 0.4739078016989956, 0.4677694478475668, 0.7482936752538394, 0.6586268666987989, 0.39094705718674094, 0.4420244531209595]], [[1.0988750999759938, 0.7749915168162997, 0.37409463644980545, 1.2699321019901322, 2.2452572440478007, 0.630365302758923, 0.40886711915842755, 0.5158117255475718, 0.7465922178181318, 1.0752461542115934, 0.09915877083474663, 0.5675429888423629, 1.1957576221651083, 0.36194285405184157, 0.2554877774867684, 0.3362296999079042, 0.5196341880434597, 0.5345657348800923, 0.9307668722863804, 0.12526775585708502, 0.3276981449244564, 0.45649693025964144, 0.0709506536416691, 0.12951737107146508, 1.1867284135994813, 0.7006295153856117, 0.08736729060403928, 0.8388241730336105, 0.9745040305782418, 1.3212158874975524, 0.8680208517446657, 0.7580253955656697, 0.799734446045449, 0.6500638120490725, 0.6485292930259107]], [[1.1388008919474168, 0.8036463501030847, 0.35364113677997217, 1.384307128095068, 2.1484210775733343, 0.625312408775135, 0.44796252856002583, 0.46630549240289854, 0.7693075375288788, 1.0141386419916958, 0.11510313347379873, 0.6069815702688934, 1.1985202873300027, 0.4374288478750297, 0.29604355378998737, 0.3441906497001582, 0.48131623083815983, 0.5826845989373218, 0.9011081641521004, 0.12979928708270952, 0.34887785233597596, 0.4206147125136603, 0.06902645591529365, 0.13057155950174743, 1.1727472125453056, 0.644352051907473, 0.2194824946006835, 1.0377829360945776, 1.0024239672797868, 1.1763794223899158, 0.8366382617956147, 0.7213929575260816, 0.6742829561379028, 0.6502844532139875, 0.6565346964455868]], [[1.121023269660666, 0.8801972195391001, 0.34012541887686154, 1.3790646614242033, 2.0890056520132116, 0.6706061113710116, 0.4052422144518276, 0.48707550950198175, 0.7234397443493269, 1.0538380069500604, 0.13256063113113173, 0.623665764536131, 1.2189098464908534, 0.44675475775626133, 0.3403748918169035, 0.35902955021912036, 0.5383540172865391, 0.5764727867425727, 0.9554726643598321, 0.15352741181362767, 0.35958945127701725, 0.4235640361303502, 0.06993867625048948, 0.22624155782188146, 1.167071310059466, 0.6337937437508201, 0.3645986294523642, 1.0416504404104066, 1.0313883779170823, 1.1242083579682038, 0.8096863341768703, 0.6703764259136151, 0.6376381090870908, 0.6322518449140174, 0.6366872275765014]], [[1.1213795935696487, 0.8942117583745066, 0.32445171240465775, 1.3024704464090024, 2.091564896108979, 0.7174140808398101, 0.42545658893231897, 0.5645033677636795, 0.6577602562598317, 1.1782282662122576, 0.178532708990541, 0.8042677095466162, 1.2091898943660175, 0.5044602506017148, 0.3467175698753758, 0.33660635804322425, 0.4638445076818856, 0.5343584473109749, 0.8543469323500903, 0.18293075132978287, 0.3864043056542125, 0.3792376302886293, 0.07177659985156254, 0.25081487717171497, 1.2031130645410093, 0.6262396800076323, 0.49538990982276343, 1.1995858361409688, 1.038076188250769, 1.1800051298631482, 0.7759762365919225, 0.585742887326784, 0.6224812467185077, 0.6353140758832905, 0.671963247206819]], [[1.0781501685890937, 0.874033322987851, 0.31368080367795814, 1.1056125806271615, 2.0412703299012103, 0.7320158215689978, 0.3956896512473001, 0.5430905245549651, 0.6508612259458291, 1.2556315020967517, 0.1722335911473859, 0.8894841630491931, 1.1328587724337074, 0.5443624764495539, 0.3747305426363497, 0.2692787473599594, 0.48418927281275753, 0.6157919308143156, 0.979804060025762, 0.0988702722085523, 0.3855020964028005, 0.3210272320000641, 0.07671603296921617, 0.3084077392819943, 1.15070363926195, 0.5793408152334496, 0.6240071493679114, 1.1879225582368445, 1.0032708054893642, 1.11884863377671, 0.8597614587896163, 0.5212423713408862, 0.618821879053367, 0.5978080713436218, 0.5982630106977596]], [[1.0592594256172374, 0.8623773535406152, 0.3074149848776717, 0.9016813384716538, 2.054772691767774, 0.79475377701071, 0.40686836014658945, 0.5471807353164648, 0.6957547488538445, 1.2972010369297093, 0.09424131517926085, 0.8748334206057016, 1.0445250478684998, 0.566479793493855, 0.3318215528551392, 0.2565781441049151, 0.4340461096213247, 0.6112183492419577, 0.9785469968005037, 0.11389543241972183, 0.39731380148967615, 0.31709347849974745, 0.08577758003337727, 0.3794798201676538, 1.1616601109680338, 0.49610724789407135, 0.6875264130822616, 1.1993977003506786, 0.9547973924342174, 0.9167565125901608, 0.9098295539284473, 0.47892451055004703, 0.5622950227688133, 0.583728696906372, 0.5316288700552854]], [[1.108822906466185, 0.8695262916035423, 0.2977575206140728, 0.7832212341704087, 2.066171450528894, 0.7818468153594194, 0.42804060159345725, 0.6345596529959561, 0.7342496399394313, 1.2816707927708304, 0.0899476395003741, 0.9093294249273154, 0.97229850988656, 0.5950670952111132, 0.3827400934978334, 0.2930085012977776, 0.4198540629024767, 0.6126169189934902, 1.0947010683327598, 0.12344146871725487, 0.39790205132992673, 0.31821378673824774, 0.09384710437603698, 0.4025077828018296, 1.0758287911444664, 0.4656853392271533, 0.7097625391999359, 1.1017180443798642, 0.8943497525812214, 0.7609041618594934, 0.7824087669482218, 0.45068154052914766, 0.557833317339469, 0.5897497977499822, 0.5562863513954679]], [[1.0960571136621826, 0.8243629913692823, 0.31202609265827574, 0.6901011232456791, 2.0195678178884764, 0.8011263940692714, 0.3997786543244196, 0.669678634724048, 0.7533363899149075, 1.2000832759083786, 0.07512914646354535, 0.9186524117379993, 0.9820356461417825, 0.5410823191976799, 0.3747840142296339, 0.26585250933756543, 0.4179977545735716, 0.611911156646371, 1.1006878105846016, 0.11104258881235424, 0.3923534519163885, 0.3540144454875754, 0.09874308852748934, 0.43367435043915126, 1.0149603233715583, 0.42497752160432034, 0.7662215459847936, 1.1523992152575078, 0.8382973000524805, 0.6617024604953972, 0.7048180290051438, 0.41150624573164507, 0.5519625354399313, 0.6997614344959582, 0.6096321737818644]], [[1.1560832950405306, 0.7915806045583216, 0.3335172510984791, 0.6709104551189164, 1.916762496256646, 0.7476628043265381, 0.42039144042860344, 0.6103412858368669, 0.8533935429451834, 1.063205177781283, 0.07841643139616328, 0.8686587630841027, 0.9622690357183875, 0.5414398000802481, 0.3843132296659299, 0.26807633317479157, 0.4154971630385159, 0.5829112524157778, 1.0777514150128849, 0.09678214350638792, 0.39212458788692983, 0.3670057531882238, 0.0878382664050318, 0.4152152714386335, 0.9758830508238349, 0.33428644138452845, 0.7678878738853093, 1.1067691971336426, 0.7715617878042957, 0.6432263744680475, 0.7102464456002306, 0.3889746828953111, 0.5614834682731749, 0.6491156155249457, 0.575769957783038]], [[1.145324928055531, 0.7544432575575757, 0.3059647218879009, 0.720685050859879, 1.8427441148918389, 0.8079521895498036, 0.4148300753170917, 0.6223215710430753, 0.8671017381282953, 1.0411708284171237, 0.07247927210906421, 0.8032232063840379, 1.068544556780432, 0.5511118979280871, 0.38149853140253626, 0.27004557154569236, 0.4089067164310015, 0.5966120734974076, 1.0847631637647335, 0.06930021706540962, 0.405428626028833, 0.40454982091139635, 0.08808860109331132, 0.39712544811057626, 0.9087636051778556, 0.4391115804801305, 0.7874732287352373, 1.1521088965793682, 0.7411740121690737, 0.54291385540071, 0.7976322174792054, 0.5851811954520659, 0.6356539339482432, 0.6417070830358023, 0.5365298750742903]], [[1.243415677825538, 0.7449324908127896, 0.30473530378517494, 0.7405668568930572, 1.9126014478314275, 0.7474675922557457, 0.42327374321936245, 0.6116439553929552, 0.7402565028044201, 1.0273210722365853, 0.06408213295604363, 0.786459104193795, 1.0739297633158587, 0.6460069587549194, 0.38639113430006433, 0.3003261518340064, 0.4151814421081704, 0.5815789525784353, 0.9462279035565868, 0.07098277160137034, 0.37347179894269744, 0.3974876382472702, 0.0898640270350948, 0.38705749406681, 0.8935432954029086, 0.3966736666853411, 0.817477418592115, 1.1038505285048719, 0.5895352046127091, 0.47277987689094914, 0.8318070964930352, 0.6264223089961809, 0.7178848144799568, 0.5444207390046316, 0.5142897995489055]], [[1.3422626975811942, 0.6880065546119133, 0.3203974554543526, 0.7736585270837979, 1.9363338680951125, 0.7566871441887999, 0.42753302190474035, 0.644960118831347, 0.502595679062255, 1.0319238908335437, 0.07316811747766676, 0.8487677230521073, 1.0850197770759815, 0.6753101993616172, 0.4467422915981319, 0.4542580807880901, 0.4772211188255492, 0.5985138716668871, 0.8059052496391259, 0.0718033373041125, 0.33925475249402315, 0.36946715767460747, 0.10138264346372727, 0.3654061198769857, 0.93505257586397, 0.384594961066554, 0.8498544178087412, 1.1336854692812626, 0.5245836054258315, 0.3972923583214215, 0.8779715594855408, 0.6551411152105693, 0.7503951491315561, 0.3892344342837133, 0.4273603153091613]], [[1.3561499495476594, 0.5894032947299064, 0.34066230677258713, 0.7884949651507474, 1.8429606602899526, 0.7485214678886469, 0.40364649283544146, 0.6698432221780607, 0.46753468371999946, 0.9616854468557257, 0.09459216227712108, 0.8582367354388525, 1.095241010201737, 0.6101130000768114, 0.44915593205615656, 0.42774506124938233, 0.4876375965836304, 0.5871285263988277, 0.7606580872696292, 0.09251322269950243, 0.33066847544200717, 0.36664330362030223, 0.10723226225298013, 0.3282173763971411, 0.9926701677001688, 0.37942989169583574, 0.8292574191824398, 0.9994552629840758, 0.45718095182721685, 0.5373037489482934, 0.868955793628366, 0.6761851958242404, 0.8092864916325134, 0.3712929041786954, 0.4635393189574948]], [[1.4013741795133163, 0.5988483626023344, 0.3532751641872125, 0.837081331341335, 1.8271582814686438, 0.6797915041344459, 0.40039956763659024, 0.6467174429981073, 0.4705871117368926, 0.9194328236586642, 0.07478806027983133, 0.8465031447836722, 1.1271741010491119, 0.6707374808422069, 0.45366716041166355, 0.4649209950485487, 0.5656942869113835, 0.6147759420702021, 0.740484685954881, 0.09022141759424636, 0.3280141220764715, 0.34806043805031195, 0.10032013387882538, 0.3360946664591522, 1.0218635721322413, 0.36133944152629227, 0.8078516811138605, 0.9251704325055288, 0.5534991183873358, 0.6130861312811668, 0.8264324221130659, 0.6932175737115989, 0.8086169911256189, 0.408695776581137, 0.48214922477709565]], [[1.4184850473352024, 0.5877007729721416, 0.3875860149563012, 0.8335510721800694, 1.773121788485438, 0.6446770329523751, 0.39199353418334176, 0.6130531905174639, 0.47229174738199764, 0.8463534576944891, 0.0479374682707317, 0.8314838760876495, 1.1592306216814312, 0.6241402765493645, 0.45430014076977554, 0.4366384939885968, 0.5566836882642763, 0.5906323997510541, 0.7314168353915341, 0.09330148656976432, 0.3521901055422546, 0.3262340193698164, 0.0964742484261411, 0.4143397897904765, 1.006475168029841, 0.17856447642855586, 0.8584409385991295, 0.8035565759645742, 0.502255639562757, 0.7480867261842704, 0.7478779769959087, 0.7038850063461155, 0.8107144638130457, 0.36672953974048533, 0.43344577345702673]], [[1.4359834674261356, 0.5484479708171227, 0.4109124613997196, 0.9766243786717725, 1.6829945517567895, 0.6673572938924862, 0.4029155607385636, 0.5939374538872841, 0.5152880592654823, 0.7813970921561704, 0.03994346300964825, 0.9004341090460808, 1.1699367790086415, 0.6061503727711064, 0.43915043628065337, 0.4112860770295767, 0.525171491257045, 0.5888473920241063, 0.8132093961464976, 0.1399750060354254, 0.3929887013194654, 0.35768086655726455, 0.11077209915105328, 0.5629520868921063, 0.9636901138661982, 0.16232320129952024, 0.9069722099201487, 0.732006549908025, 0.33663025697735216, 0.7744300826716141, 0.685489709790821, 0.7184356746329599, 0.8039479776805072, 0.22575682514315004, 0.3993306592332496]], [[1.4434995113630005, 0.4774559405797745, 0.3813660873639443, 0.9559089122731671, 1.612135659585986, 0.6757624010515839, 0.4066921744676492, 0.576041742233024, 0.4784847901991617, 0.6457897422315273, 0.051337707963564475, 0.9152259321909388, 1.1831338196219434, 0.5970721073426625, 0.4001560760778232, 0.41259952446047043, 0.5171413393912269, 0.604140865263209, 0.7268828861084129, 0.13720943218809706, 0.4045133717456037, 0.4464883079540359, 0.12236082782238107, 0.7601750569078591, 0.9289242254589685, 0.27965569977290883, 0.9586337272984551, 0.6731748771584304, 0.33577514319055035, 0.7213580190919382, 0.7061389244481575, 0.7149257878489027, 0.7130777699985129, 0.20317947432416414, 0.37190425420807316]], [[1.465632468206871, 0.4074515623626963, 0.38097984479038316, 0.9149868279797593, 1.6017526290419783, 0.6572786197300443, 0.40276270742078063, 0.4809566850226357, 0.4620410842390593, 0.6060337406005365, 0.03735130106052817, 0.8603133049746314, 1.2066547167546915, 0.5595434834411452, 0.4239552537157378, 0.4721919009978697, 0.4892361420700163, 0.619358003106953, 0.6985136997552797, 0.1545665232485652, 0.3752283949844595, 0.5734189010186153, 0.13102001859916435, 0.9111038840649752, 0.8376426903840872, 0.35594415793391065, 0.9632595285572279, 0.6264327694019349, 0.3713427190930486, 0.8053837464631435, 0.7206663061873615, 0.7565144581955404, 0.6427731935387067, 0.21742180488666743, 0.3452510570894393]], [[1.4984596578269507, 0.43723038173360845, 0.37699664206648903, 0.833063629838424, 1.586306716230625, 0.617263104466587, 0.41223110275927755, 0.49346896760537556, 0.4412207142813202, 0.6070438498401884, 0.023788869256448755, 0.8444058311985663, 1.2339404406433945, 0.5569300316055521, 0.47426983303340947, 0.4236879458598691, 0.47016548585576967, 0.59306882818586, 0.7323538120377234, 0.1987893539247429, 0.3432053906355275, 0.6301213881166693, 0.12465516194369157, 0.9446780863420994, 0.8288160929212219, 0.38523346181030627, 0.905663272672939, 0.5529007505045125, 0.3699711780693503, 0.8581342987848328, 0.7707006298054143, 0.8729619609963838, 0.60482925179724, 0.2520551513703746, 0.35621204843356424]], [[1.5018523303564333, 0.5151176550873625, 0.36408928443110344, 0.8418097538122938, 1.5532792725230173, 0.6360933077538409, 0.3953275160490117, 0.5153246475679408, 0.4380251721115266, 0.5203360916629419, 0.01816035551700335, 0.8539837947053237, 1.2516607683044814, 0.5708725311925219, 0.49643526640943136, 0.40904171361938313, 0.5182391651452274, 0.5838423777480315, 0.7230135173300182, 0.22231608603170816, 0.30781688155353254, 0.7271123755366965, 0.12325966746592006, 0.9579794259307061, 0.746791720462017, 0.4129554488747282, 0.9111242402063284, 0.5437419811360559, 0.41988988439318264, 0.8570553772733625, 0.7539066525049671, 0.9478581212613628, 0.6350823304819195, 0.2851769740161305, 0.48781367643930057]], [[1.5363778813773459, 0.37820314291050394, 0.34411382351134534, 0.8524539249231353, 1.7338123271173727, 0.5879462628403277, 0.4123193063296377, 0.5218276601958893, 0.6331903116561794, 0.3884973386376528, 0.013986286511108786, 0.8769810246493814, 1.1230835811608497, 0.5697236974649281, 0.4626514383776357, 0.3986898145353919, 0.5736179677092423, 0.5499775270509922, 0.6945101182044924, 0.27358329650574154, 0.3157744923917148, 0.7144546274686698, 0.1408230430169115, 0.9011915417105003, 0.6639472955990641, 0.41358284573095117, 0.9116963739666227, 0.5209702363966701, 0.45923582097078, 0.8718168236428647, 0.7819078988328854, 0.9082924357678028, 0.6178549757302281, 0.2845185482897159, 0.5534397541627027]], [[1.5958108342029638, 0.36399845346527254, 0.32710457219116507, 0.9393577404862961, 1.806679854761674, 0.6180543338497323, 0.42350596513640054, 0.5482785431058266, 0.6216258908516925, 0.3276277214667276, 0.01532254415687567, 0.7759954808379048, 1.0611650627928104, 0.6552219191291296, 0.433285976724737, 0.4361433547397361, 0.5939369526530911, 0.5672324459024232, 0.6081425026185984, 0.28304390053618056, 0.2863234621175611, 0.7282110005176621, 0.07075884428342669, 0.8333950081180976, 0.5315157839742897, 0.36702722448861874, 0.8682456629623407, 0.5279795227849287, 0.4798945594942933, 0.9932000893458409, 0.7531644809506877, 0.8760494955558716, 0.6479356818552084, 0.2652609984626515, 0.6872264236085599]], [[1.5528473546073331, 0.3642382820025196, 0.3467217442617656, 1.025239134722077, 1.770614532176109, 0.5926571822465336, 0.4317335453185832, 0.5096716899600227, 0.6829985917803688, 0.29305853332234594, 0.011198633429541327, 0.5796072456878616, 1.0032418333328916, 0.5604326209561419, 0.4046816468804677, 0.38479828645477865, 0.596283419715593, 0.6335932422185265, 0.5938535122162063, 0.24791781722161585, 0.2249976342914037, 0.5830709013717629, 0.0204774208188264, 0.7717403044597075, 0.5017282838448696, 0.346153577167889, 0.8325738294938378, 0.5386847096481692, 0.49319856326598677, 0.9974444993492129, 0.6895206633560084, 0.8862262163635115, 0.6480640092606711, 0.23296995609223903, 0.9220422608761746]], [[1.539070174416923, 0.3775234898457124, 0.3131174698902194, 1.1163741628375683, 1.73082367426807, 0.6222854343177553, 0.43270347668347925, 0.4377852825948288, 0.7673085142123832, 0.24911318880838526, 0.01990789213086941, 0.48785810811160873, 0.9213497093063283, 0.47788277818396635, 0.37612775529222753, 0.38450910445701053, 0.5912346798007495, 0.6195825121846628, 0.5657134700193209, 0.22135927589230758, 0.2294319559475278, 0.35977345684652995, 0.04651406077386644, 0.8212482098008111, 0.523544816312158, 0.3833705738601274, 0.8309117544889024, 0.572224337187972, 0.48911734417134534, 1.0097613072542115, 0.6169217066204653, 0.9045958043759941, 0.5606612118086169, 0.3115080755087731, 0.9759116492487049]], [[1.3799503282861116, 0.37103447375468024, 0.3319246652285305, 1.1514690504262766, 1.64354705701106, 0.6125265703240608, 0.40532513582633695, 0.4102286889573476, 0.7544090486334304, 0.23776057843657677, 0.02247539376948459, 0.4867350283347205, 0.8145741845946581, 0.48575347422390847, 0.3730550886476033, 0.3815792337911174, 0.6417887323476893, 0.5853113499831732, 0.558692538540248, 0.2224486599264111, 0.22681851622554533, 0.29495514093995445, 0.04494586201640263, 0.8570706279579525, 0.5565762744682758, 0.4138601020244544, 0.8926948345427104, 0.6338131834566892, 0.5228161198398463, 0.9662904478818187, 0.6502455693954367, 0.9863055585715619, 0.5091292156735483, 0.37829642709249317, 1.0780477590243818]], [[1.4452737292238969, 0.35528707671620513, 0.3685809301658778, 1.2243884793771413, 1.3030762393086914, 0.6099017564385276, 0.40897369284209467, 0.4790366903610039, 0.4352601115882079, 0.22511758093506695, 0.021529400362209263, 0.4871228193343664, 0.6863835938444, 0.5026203507641247, 0.42713728635220166, 0.3827665570253271, 0.6512180001688196, 0.5828706471835812, 0.5695278065898945, 0.20998857660885456, 0.237030419372252, 0.2752021139543659, 0.04282734569468595, 0.8489435181251886, 0.4429768967729282, 0.4574092710273516, 0.8348206458326597, 0.5691988546677577, 0.5349421833516907, 1.0230604906280132, 0.69399412481737, 1.0332736994894363, 0.4768509310681049, 0.40874830655093614, 1.0940196112876936]], [[1.5231262869756887, 0.3642363958468414, 0.4282983783815203, 1.323593744138979, 1.2683831518461064, 0.6159696015252141, 0.40009618209092257, 0.5100995114913078, 0.39685294888242156, 0.22526032075408908, 0.04077852327418055, 0.48669667032062314, 0.5545811428708938, 0.5070431709846323, 0.5501276821362446, 0.41797416517984043, 0.6950388710256707, 0.5852536323062347, 0.615881715592062, 0.16746619332875395, 0.24514542991107185, 0.3022282519105665, 0.04226719853393686, 0.8610755941508947, 0.3990624928843621, 0.5232333673166041, 0.6805172615863654, 0.5600395311094596, 0.5532248888443687, 1.0681857702583724, 0.7050959358679993, 1.0513320281392284, 0.4268808700679988, 0.4536663262137384, 0.984434596900045]], [[1.528591037662896, 0.3777415421040607, 0.34880492573348076, 1.5753775983321125, 1.3692678233799476, 0.6049595898050224, 0.39461566253245645, 0.4701532396060709, 0.38335421740709474, 0.24215760997101765, 0.009383553105350802, 0.4737842465782436, 0.4060604820177423, 0.47084561197340474, 0.5074981303208601, 0.449767389074452, 0.6802688147709959, 0.5699718563774473, 0.5696879906007375, 0.13701874903070765, 0.2394537404826051, 0.2848887351799566, 0.04532024218561945, 0.8882378497076928, 0.37508037003299005, 0.5397774463096252, 0.7010140153387268, 0.542342765865035, 0.5606778897589058, 1.1196903124280597, 0.7345194274225846, 1.042070463093443, 0.42937940623942394, 0.4812797526083421, 0.8067079430911718]], [[1.4761592015777736, 0.37627269671008645, 0.3521961394628802, 1.7314131677979927, 1.4690798041331747, 0.6074700007894748, 0.3968377417929126, 0.5775693073442634, 0.4004476529221478, 0.23577190339074827, 0.0024927044704159563, 0.46540615149924136, 0.4175550159997612, 0.41737958699248295, 0.45472965673364957, 0.4655043027937771, 0.6808349841687817, 0.5880497687616256, 0.5899859610098185, 0.12342060573265427, 0.23274202153007625, 0.28022784955847446, 0.04620201915463601, 0.830769744940067, 0.34979458046822676, 0.5322136346737868, 0.7388515507893214, 0.526298145865851, 0.5786882647954988, 1.057305308757103, 0.7917525935163692, 1.0236754150409755, 0.3862790812368177, 0.5021229148653121, 0.8353171856845393]], [[1.4576252003967274, 0.37735588293807903, 0.35483359283472304, 1.8047643856563294, 1.4409142219573017, 0.6025013334875149, 0.3891447611988857, 0.6722334166012933, 0.5134524589842041, 0.3375552180375381, 0.0022756554687821677, 0.45231321364512533, 0.5058611878482004, 0.43041844123333955, 0.41916667501252863, 0.44033879702650397, 0.6579934905181095, 0.6425224487934493, 0.6122457215170074, 0.11992431831391663, 0.23466408610208517, 0.28586729332962235, 0.0456010233143124, 0.7797452798335985, 0.327407613516579, 0.5359453844827948, 0.7173191557996625, 0.4357998795054676, 0.6562334055956122, 0.9188997147882105, 0.8618739375157616, 0.9884513849097183, 0.46765294952950215, 0.5413479630725481, 0.9169097282958827]], [[1.4410777102147105, 0.37939701488256716, 0.3681420314481866, 1.9275501035303135, 1.4504359001136315, 0.6107593762921486, 0.3963798707654123, 0.7603445250138929, 0.6226083117525543, 0.3809946161228015, 0.0030555524630525837, 0.4563025680672759, 0.7629299320636637, 0.46603195308717116, 0.4129347853533581, 0.41417662801534433, 0.6873799088171728, 0.6517619710943865, 0.6103334549531323, 0.1188238687136176, 0.23822598735405717, 0.31471181981023655, 0.04614749980135678, 0.763136434057903, 0.3250546972756171, 0.5587001759156395, 0.5967153278737014, 0.3878737951159007, 0.7350899693005284, 0.8427599200663571, 0.9037558347894584, 0.9372858476142287, 0.4924218526876928, 0.5679038219148947, 0.9954952787597905]], [[1.423912089487907, 0.376446889598702, 0.3066607335017877, 2.0019431764390943, 1.415549169633202, 0.6034014887837948, 0.3985288782531865, 0.8197259061320856, 0.7126547878833651, 0.40722476643640226, 0.004035523596935474, 0.4519270936599497, 1.0113260416814573, 0.4092855583028798, 0.3805220940411963, 0.4036070984956841, 0.6726830950069422, 0.6611505077563378, 0.6413095476791233, 0.1282109608131301, 0.2471721489688139, 0.3159673115036995, 0.050213126425268186, 0.7411623471702172, 0.36729010949570906, 0.5196105168476293, 0.5379527919089515, 0.38952481679077544, 0.7654583617432831, 0.7736235203927256, 0.8633635286479394, 0.8866506284248309, 0.5484462048449905, 0.5819381078387545, 1.058189200795033]], [[1.3558807437813662, 0.3746798438526081, 0.34179542801790064, 2.004405649493881, 1.4141230980043613, 0.569600543573829, 0.41294244682306364, 0.8979490577567776, 0.8225185766754131, 0.41895095810560157, 0.008070593565322822, 0.48223316821641254, 1.2266102727452075, 0.41767251136985717, 0.36371480467096035, 0.3806309783084029, 0.8297856200755868, 0.6844430179467069, 0.6323589199867022, 0.1379824336622169, 0.249195935369672, 0.3101571945119083, 0.0516802950073308, 0.7027770535674238, 0.43160099790224893, 0.43236845387462763, 0.5566584896915481, 0.4750436874076188, 0.7309175834956321, 0.6717582823034116, 0.832211341948629, 0.8444720035681399, 0.6112785417096603, 0.5908754727065002, 1.0864702193111158]], [[0.7843218944667757, 0.3797794095537247, 0.31973520718326454, 1.7826703541079862, 0.9438096597096088, 0.5706137503367843, 0.3843896809778408, 0.8148766196480434, 0.7860676950865682, 0.41347823403357575, 0.012537217035766112, 0.4662664418919862, 0.7817855560722005, 0.38685567820821964, 0.3451532732502798, 0.35967317211448613, 0.7117996773870461, 0.7114823710049543, 0.659401177027005, 0.12599994937759884, 0.20461852014363963, 0.2101621616651403, 3.4870886584447467, 0.48152941771395236, 0.358176021318538, 0.2844109137004171, 0.3327417222210882, 0.4379474600870037, 0.5282238509570075, 0.40417265398649754, 0.5541785049636109, 0.5276073052540886, 0.573139153206426, 0.49958372613492025, 0.8000499486173139]], [[0.7606687651983821, 0.3825512341061721, 0.3096479319778629, 1.8357773080762092, 0.9119400394955841, 0.5678119965217135, 0.3949712851456942, 0.8653023326595373, 0.8498980729766908, 0.43877630321651884, 0.026212302986173403, 0.46050854518136347, 0.7619919743245662, 0.38751951712457994, 0.34441967661442163, 0.36281064434896537, 0.711074962756003, 0.6633299644925452, 0.6494455926955618, 0.1363673169772165, 0.201597268102728, 0.21542715984743055, 3.4870886584447467, 0.43106193780942914, 0.39103083358076657, 0.22896925376963786, 0.2408087190016038, 0.4815324546831453, 0.45764462634353104, 0.38720882630729136, 0.5897236080759269, 0.5466017442705475, 0.6603579814208576, 0.5220527218670786, 0.8241489963910423]], [[0.7476209030590916, 0.38075727895761086, 0.31398531393636264, 1.8578161930858454, 0.8722368054307787, 0.5698320393491206, 0.38974281659081467, 0.898626493233521, 0.9002866708921188, 0.498202415915956, 0.02448938755263555, 0.47094661476722877, 0.766584400395971, 0.39381269189799584, 0.3378500719668467, 0.36102204949372446, 0.620156991960722, 0.6693749716340507, 0.6540477544771162, 0.1297500025535196, 0.1984053261100061, 0.22666929839212874, 3.4870886584447467, 0.41960636111313127, 0.5136695485020335, 0.19447387655682702, 0.12344190183761183, 0.43968207512146573, 0.4213160629123765, 0.39334790437665856, 0.6377598372177345, 0.5944521113860943, 0.702086642052855, 0.5229420010458892, 0.8574691952285926]], [[0.7890779345203383, 0.39830634050394703, 0.3527535989965508, 1.9143351815783545, 0.9132800240926939, 0.5922916903284932, 0.37826266307282824, 0.8911269924348661, 0.8408432225410523, 0.4379405755351679, 0.03691431457072434, 0.4558692304216557, 0.8026107958850828, 0.393497454142672, 0.3159866020236285, 0.3829974314526823, 0.5755168642683614, 0.6752397910628942, 0.6283658036486067, 0.13265953792976182, 0.20067237986230746, 0.2275059965265256, 3.4870886584447467, 0.4384374573752047, 0.6459917278434952, 0.1632585308026756, 0.15266473371022976, 0.4414690868122264, 0.3962397832654414, 0.35934877917121905, 0.6873639370892032, 0.6410613847503517, 0.6923200657724586, 0.49415155915443565, 0.8688606729948072]], [[0.7996104571556404, 0.4045392204200593, 0.35822239028859004, 1.9677212108636442, 0.8911403115019387, 0.5942823146076374, 0.37739375032463457, 0.8773773796504316, 0.795263456836458, 0.37434664133244755, 0.0453692072150121, 0.4562701269802192, 0.8197886902324676, 0.3911005774739892, 0.3340043566814106, 0.3756806305353726, 0.6061223376067091, 0.6951946602447217, 0.6439601086022897, 0.13612664101566474, 0.20634379839212064, 0.23359210756211463, 3.4870886584447467, 0.4245817168725705, 0.718140692390137, 0.13456348876929675, 0.20565863079001984, 0.5255813879312408, 0.3776322915710111, 0.3401525194219286, 0.7290612005897112, 0.676182660806204, 0.6569124118210299, 0.46948339654724525, 0.862457719047162]], [[0.7824949704759963, 0.4050414391937045, 0.3512776138046502, 2.0233976322885985, 0.8553494153407117, 0.5870143032085687, 0.3807083167513766, 0.8677600396512837, 0.7623362604663658, 0.2896576274826552, 0.05420811664918869, 0.46591535214268637, 0.8123129196342089, 0.4008802674984987, 0.3595872422610269, 0.37087620030115487, 0.6167215836740035, 0.7145932156768218, 0.6491655019421009, 0.13270321965328602, 0.20785616010694308, 0.23734922340016745, 3.4870886584447467, 0.3559112908226236, 0.6997422816292018, 0.12687852213742284, 0.25787410197000843, 0.6461419100035752, 0.3707694361462054, 0.3253114808402207, 0.7714647282954553, 0.6929179358255609, 0.5857389984283585, 0.44724520531086714, 0.8460873913218451]], [[0.7695468133342706, 0.40934793473360154, 0.34719921077802685, 2.0764663778950023, 0.8862213440702131, 0.5931019913425577, 0.38436199935262294, 0.8601398439045552, 0.7467974209226216, 0.3322786927759618, 0.0636121432328627, 0.4590873321136732, 0.8168656370961862, 0.39936681431697657, 0.3611328923032368, 0.3823072103271429, 0.6408111083363885, 0.7177007747422789, 0.6638516606242013, 0.12397652517640595, 0.21362227366906822, 0.22157542878306247, 3.4870886584447467, 0.3020129542258194, 0.641016967384664, 0.1262904615362852, 0.2571682282526647, 0.7876773675847195, 0.36539614980457347, 0.31315293590811866, 0.7847719759186318, 0.7348027645466251, 0.49942891727132094, 0.46662204421409315, 0.828683839162264]], [[0.7401316060054681, 0.4041755718761507, 0.3514170793245752, 2.0937931332746, 0.8389963793494086, 0.5680320004482926, 0.3790596838974389, 0.8540391664070783, 0.6988291706607628, 0.45492033978791363, 0.07551058694494972, 0.4426553436552716, 0.8220005830465748, 0.376935708569627, 0.35842259997683673, 0.3688642024907882, 0.6350459883951753, 0.7193702591882267, 0.6789940818603548, 0.10474769085287222, 0.214681974997398, 0.19746399365215717, 3.4870886584447467, 0.2819856286751217, 0.5425811613820906, 0.12335717702868398, 0.24024482924957039, 0.7846018666438198, 0.3635903710789605, 0.47864413767585584, 0.718197958980638, 0.5114563033589057, 0.451428132300707, 0.32974360081649556, 0.8996165698913544]], [[0.7599192549376873, 0.3989268662942441, 0.3573257058835626, 2.147959704522318, 0.8051025165175579, 0.5693844358749129, 0.3813977808785819, 0.8468769265651271, 0.6407000073136875, 0.4851960615985019, 0.08241234391200965, 0.4504646594941326, 0.8156417175355453, 0.3814872581044989, 0.3386415846694643, 0.38166899062862225, 0.6218751186259922, 0.6485046036250847, 0.6807983568419123, 0.09288080791403064, 0.21583623026372215, 0.18365144797822675, 3.4870886584447467, 0.28280403785314856, 0.4713849481083052, 0.13787374789828566, 0.24398738852317883, 0.7810794561020056, 0.34328835219697024, 0.6001591959812813, 0.6550476193282628, 0.43614528991356777, 0.4169868937701787, 0.30451697409233625, 0.9644460046595886]], [[0.8080128367546511, 0.4114579732402441, 0.38374700382783705, 2.0149070495944095, 0.7487141254103564, 0.566138980315413, 0.3995678233340809, 0.8778803640632318, 0.6179465970121782, 0.5340669461289406, 0.09272358072903528, 0.4503588224058641, 0.8185396775937765, 0.3847506917339686, 0.27899417381559627, 0.3824134939870721, 0.6435949988078187, 0.6375622596676276, 0.7238524053769274, 0.0777333675891414, 0.21018316456183628, 0.17649953018198072, 3.4870886584447467, 0.2884252799639433, 0.34452040493098873, 0.15400414009560182, 0.2450957574293864, 0.7646689124441618, 0.32516732310935836, 0.6662335784038012, 0.5911896845926816, 0.37879287558424757, 0.399570449696002, 0.29046575811766706, 1.0242124486087083]], [[0.8594285210859757, 0.4080901305299873, 0.3824327095323924, 2.0577911857618663, 0.6927482339251961, 0.5872726305466147, 0.3971608470921613, 0.8817458085842478, 0.564369488374649, 0.5040898524676685, 0.0931672908170269, 0.4475573498621829, 0.8090886481820695, 0.3640995386871419, 0.32167292365503375, 0.3284125418110952, 0.6522794655733065, 0.7395783483373745, 0.7626372515119892, 0.07679871404747124, 0.19412921843143865, 0.18806813362006694, 3.4870886584447467, 0.28066341834448344, 0.2559705568800981, 0.19239443322768618, 0.20977818959021868, 0.6816586577695699, 0.3104407279771744, 0.7157307917431296, 0.5280829006265929, 0.35379217827647547, 0.3619170671129941, 0.329353103956741, 1.0648822868579682]], [[0.9118360887845566, 0.3968365156051893, 0.377914132155415, 2.1041300652485337, 0.6036117796465155, 0.597998555421164, 0.39490278404198575, 0.8930698191331512, 0.5523231531331132, 0.507017024684236, 0.11481849445728201, 0.472551505282983, 0.7929326354905435, 0.37536375535473787, 0.3279810323913991, 0.34615065368587916, 0.6537033607306533, 0.7683090971694058, 0.7407670108767668, 0.08942043921823628, 0.18824035667038097, 0.18874331905576514, 3.4870886584447467, 0.28343790088698073, 0.24355585742797453, 0.2258984463060919, 0.15923674887038597, 0.589414901514898, 0.32815624472916377, 0.7214966845374842, 0.4746213120490975, 0.38208018946691424, 0.3151649132339578, 0.37629432583960293, 1.0949266545049892]], [[0.9718920501517293, 0.4058656678397821, 0.3971529393531746, 2.137617779204086, 0.6093234985942326, 0.6119403212987198, 0.39060538845366366, 0.910102578445107, 0.553050162264201, 0.5670580738338127, 0.13014576091586139, 0.4855480424935974, 0.7876155997274853, 0.38374287834770265, 0.35157065851003544, 0.3350685700722969, 0.5934348444587009, 0.7815756650551078, 0.7127698913790671, 0.08741468174346154, 0.1806339394970049, 0.1717935734403974, 3.4870886584447467, 0.1877557509077658, 0.22295218772410003, 0.21736319750244407, 0.14764232739323158, 0.5563394662148295, 0.3583003736283511, 0.7043762465604725, 0.44784934059372095, 0.4062539123507941, 0.264984550304888, 0.40123949038315954, 1.110959764903908]], [[0.9574507241894952, 0.43034441585265437, 0.42909916207573884, 2.1080037749414537, 0.6326821085664154, 0.6059616576266866, 0.37493762133355923, 0.9410094820483855, 0.5503008086384523, 0.6143731759755124, 0.1429193991588404, 0.48409953190413557, 0.7682060011442166, 0.3849523838074058, 0.3692293103058924, 0.3367383109022538, 0.5897993624674838, 0.7826408433228955, 0.7092477314904164, 0.10137051613199646, 0.16812422773539276, 0.16190964475182335, 3.4870886584447467, 0.1535217774825643, 0.221325594718421, 0.239868280584062, 0.1457230278176846, 0.5543489756790478, 0.35207578129989026, 0.679072578550675, 0.4354434166481017, 0.4313774261905392, 0.2500852940069822, 0.4064455725587345, 1.113528640536079]], [[0.9772528437238797, 0.42938890669365526, 0.4343596628173859, 2.0956389518363285, 0.6598212968775162, 0.6028836852384298, 0.38091858740891016, 1.0142777557575098, 0.5937760841451161, 0.6923521899373262, 0.14492137207286804, 0.4859019113677215, 0.7527449907082382, 0.4085162266021544, 0.36182745636741803, 0.362846291421121, 0.5698271249856119, 0.7755961812461649, 0.7208839240118214, 0.09917080198579911, 0.17474246828967469, 0.15317285834063354, 3.4870886584447467, 0.1476228435201451, 0.2817109925800474, 0.24499875093022824, 0.14561299323845633, 0.5774850370982305, 0.3345436728490364, 0.6617638756303548, 0.463683190842309, 0.44191012156033005, 0.2388396217443103, 0.4123663392713235, 1.1070894099080326]], [[0.9775645179759296, 0.46344252427021115, 0.441050046303052, 1.9826581811137305, 0.6532430353315622, 0.6054192220629764, 0.40118202998727387, 1.0441763049996249, 0.6733508506705713, 0.7909506445032746, 0.1430990917871325, 0.48158057630457, 0.743537025134184, 0.4148942985273128, 0.344635174727218, 0.35919642666739005, 0.5709696549240523, 0.7666201736806743, 0.7351462820466573, 0.09751361290186897, 0.18003853224586952, 0.13342099292319998, 3.4870886584447467, 0.1489460108823803, 0.3230741562036482, 0.24398729018178067, 0.12616096216062933, 0.6264152937115517, 0.3195947026112868, 0.6566093771373276, 0.5128929854304565, 0.4600413927019858, 0.23049247075681314, 0.3896287036765295, 1.077743538963833]], [[0.9857897224145247, 0.448982367965719, 0.4343983068241839, 1.9020584857774767, 0.6219584596199743, 0.5886204062561374, 0.37010930003273285, 1.0013409042974408, 0.7196305750279021, 0.8383837678830246, 0.13356726024166146, 0.4706179623765808, 0.7346663787123263, 0.39500004306132397, 0.34139809742208466, 0.3589403423233536, 0.5757850952951098, 0.7737968513781555, 0.7098379314706644, 0.10508686039455292, 0.18365206774709386, 0.12935170172524263, 3.4870886584447467, 0.18475404461848496, 0.3215657066301645, 0.22521979053027272, 0.14407307739434985, 0.701591134215919, 0.3053749936425215, 0.6701045327755966, 0.5332985927322693, 0.4540174259921842, 0.2247175298686615, 0.3233786914523988, 1.003745220224941]], [[0.9112385499078164, 0.49210186984168836, 0.4684297664244838, 1.6872687367633317, 0.5103834341102708, 0.6227957302871718, 0.3683119041084752, 1.1187911066093756, 0.7534588846676638, 0.9491825069832243, 0.1177335137981024, 0.4605395336072591, 0.7098709562278966, 0.41113495074235185, 0.3437339404760672, 0.40466598115736907, 0.5821833750094771, 0.765821957436241, 0.7143022895806103, 0.0679668895175925, 0.17549018577802292, 0.13418943996432453, 3.4870886584447467, 0.2272836632520487, 0.32050703999466773, 0.23652190760878797, 0.175981295178212, 0.7834119025653197, 0.30927618212723296, 0.6768274633001408, 0.4520157219755837, 0.41736864794894013, 0.18483806361496743, 0.19961528058434416, 0.9094284173186018]], [[0.9727467372502749, 0.5211588321849296, 0.4877924814586074, 1.6116398291276606, 0.4400740238003022, 0.6283021179040449, 0.3829866303322767, 1.1119907315686728, 0.7372434460611257, 1.022694489488423, 0.12391879205990994, 0.46623424829849175, 0.6979711393998138, 0.4003320572185824, 0.3540496280806418, 0.37537428196997535, 0.5711859853218693, 0.7681599089956301, 0.7166896069253365, 0.04757938541400744, 0.17309036807810746, 0.17815566577500525, 3.4870886584447467, 0.2657664604725312, 0.36329756672945107, 0.2800331999237854, 0.21916697660079168, 0.8755253067155101, 0.34990335773474457, 0.6968102814159347, 0.418515509989268, 0.45296680300973713, 0.1916466931433788, 0.14291175094030373, 0.8374024665113868]], [[0.7977896091446466, 0.5527867904014658, 0.5037059239875896, 1.4983677224849303, 0.43299643605701715, 0.6884656196717615, 0.43948855640978934, 1.1969534184940303, 0.6588011645251204, 1.0052856041650773, 0.13159921416992396, 0.5662680490064249, 0.6816796449953628, 0.44351726360914967, 0.339508979849953, 0.3667165338003049, 0.5636921291535137, 0.7636364083661564, 0.7267787088650027, 0.04395517058054943, 0.1855855167317525, 0.19116239881952152, 3.4870886584447467, 0.29218796334359576, 0.33305686088423586, 0.42935481878907056, 0.2668661209605133, 0.9999282152826623, 0.574436790527754, 0.7083743563488104, 0.3999747018873837, 0.5207768065113232, 0.21490249594016692, 0.1647063258483435, 0.7804908846040587]], [[0.8213980389306343, 0.5558652983815812, 0.5173024443133796, 1.4839257187018184, 0.3420341186008474, 0.6427769261939948, 0.3931387239442428, 1.078116377901326, 0.6134045644659756, 1.0459077369931298, 0.14029216828397892, 0.5786518399245074, 0.7263211859767842, 0.49278361832440387, 0.3446003264677483, 0.337849831662585, 0.5425533491256618, 0.7786188960995251, 0.7654414330761079, 0.03640537104171496, 0.17441809863957028, 0.1868654687523434, 3.4870886584447467, 0.3036832803449433, 0.3635019024075306, 0.5032111299910274, 0.28645581225125555, 1.062567419863439, 0.9191303784923822, 0.7267063126134123, 0.40527181599415274, 0.578579481792945, 0.21707457914408945, 0.20181746665845057, 0.7268567107520602]], [[0.7873430139417645, 0.5816734469240379, 0.5326614164763073, 1.4009818611112577, 0.4534916889530894, 0.7901562967734082, 0.3855084207189532, 1.048854186548603, 0.5776564279713814, 0.8854478240487879, 0.14552584990562398, 0.5990274360876824, 0.7730975537400085, 0.4885721794960058, 0.36308883281868154, 0.33312196608447175, 0.5031351714976873, 0.7375496882756751, 0.7602306763280482, 0.05158576634326062, 0.18032328849249366, 0.1897710479029553, 3.487088658444745, 0.3165841368545449, 0.3664520694993654, 0.44794969752663094, 0.2896075250907069, 0.9505555617273191, 0.9820682516131368, 0.7591214686453263, 0.4055436691094011, 0.594893461655543, 0.24177716454638007, 0.2740802442249621, 0.6804346397874734]], [[0.6817617039333079, 0.5766400714754687, 0.49629312816776017, 1.3576613590077353, 0.2755954654227389, 0.6690653736354251, 0.38564034258921237, 0.8696127087427235, 0.48495284365804847, 0.9114811568948818, 0.16459465558471367, 0.6391323702532172, 0.7826381564189211, 0.4770016114352406, 0.3772874383501683, 0.3160575829888095, 0.5225201535927877, 0.7537467296713498, 0.7221186723668165, 0.07575575540618987, 0.19817423726494388, 0.2118717311013846, 3.4870886584447467, 0.29658624419141344, 0.34419876316170883, 0.37753202273219116, 0.28425178027686837, 0.8512750895942531, 0.6728527816278717, 0.7220810064188229, 0.4115170584519161, 0.5669874751445081, 0.2600262972340791, 0.3738717510171454, 0.6620508622846586]], [[0.6291788816294434, 0.5537065491766475, 0.4621891599269154, 1.381549021117932, 0.2587464874703609, 0.6750260074725243, 0.3500448953228226, 0.8074525598565363, 0.4499919442597596, 1.0159354452900298, 0.12623484359118842, 0.7354805479669011, 0.7703268549615614, 0.5063903178322896, 0.3520185556871461, 0.31268561495867914, 0.504481017993831, 0.7961507692139431, 0.6841602455850768, 0.14040242239537393, 0.18506884413681757, 0.1971461307659311, 3.4870886584447467, 0.2970843489368609, 0.3650125755834144, 0.3391577130886237, 0.2603270142792248, 0.7706606381115095, 0.4850887972729978, 0.669130639107852, 0.43470409348357864, 0.5808647963096314, 0.26448801130641064, 0.44761461657155366, 0.6549970106959468]], [[0.5890033096110452, 0.5589339640123625, 0.4728407664816213, 1.3988760113200391, 0.2467302676354229, 0.6809980762502692, 0.3345111640587499, 0.7131239306804655, 0.4284286440422083, 1.0658813536173506, 0.11791219242372136, 0.7813629346708553, 0.7517270630104137, 0.4969376347927251, 0.3475364077595031, 0.3168880855886331, 0.5493569115011325, 0.77603727179332, 0.7098766685358726, 0.15994285260526336, 0.17397378550466144, 0.20409704022359135, 3.4870886584447467, 0.29144845104869044, 0.3541260357934439, 0.3402166663880042, 0.20671950452502436, 0.6968221206841533, 0.43724611121208473, 0.6425455107976025, 0.46022479009150496, 0.5885529102031827, 0.2759343208869228, 0.5667307856767939, 0.6437534626168673]], [[0.35825363374563984, 0.5996705312325524, 0.4890640060885002, 1.322566272891566, 0.24469552559105837, 0.6868880509271071, 0.3487398844163889, 0.6387474357410271, 0.4229868124991024, 1.0437928925997297, 0.11219564192842377, 0.7545415750685145, 0.6909387425482942, 0.5038617166717108, 0.350894781726084, 0.37749285752593964, 0.5348586846747192, 0.7723660041717495, 0.7399197938479614, 0.15485610497830787, 0.17246740793824256, 0.19528137436236326, 3.4870886584447467, 0.2884771492512984, 0.3264927975779276, 0.390323567871219, 0.1562822844017126, 0.5711094270829533, 0.3827708340234629, 0.6005078177977781, 0.40703316546488777, 0.49493235988584383, 0.292720565344596, 0.6299363495802889, 0.6203506421772493]], [[0.34281259477723225, 0.5879152497123902, 0.47147834389816734, 1.1923358190236313, 0.23077588960771958, 0.6623073796534091, 0.37547918274815284, 0.5404485270678352, 0.4725674182684796, 1.0909830304750459, 0.09820831842392673, 0.8085005615806071, 0.6723790262355889, 0.49576793160669697, 0.3464013675419908, 0.3951691934643437, 0.5199013240634129, 0.7513421173826347, 0.7112702757924341, 0.15250238945417605, 0.17933179596291365, 0.17159268464113941, 3.4870886584447467, 0.30344253501404733, 0.2982749985615879, 0.36743294428859474, 0.1620115740626149, 0.47104912899916185, 0.35764932205431094, 0.5918367145593869, 0.41806231126090054, 0.4335886725418252, 0.3226547746615416, 0.6925888849152642, 0.6049454172367026]], [[0.3213525508336049, 0.5899458544324183, 0.45430837939952884, 1.0797394549379336, 0.21198339895985938, 0.7004682671548867, 0.40168203484068, 0.4876657285907091, 0.5402914416890399, 1.1126827965493524, 0.09323609769085053, 0.8003657908334152, 0.6601949762645225, 0.5028216847792669, 0.3453081580559242, 0.4077600004939226, 0.5488807794763925, 0.7848768428804307, 0.7360362099401307, 0.17537920117205769, 0.18247721990043092, 0.25869434857204576, 3.4870886584447467, 0.3767169058588082, 0.3172346274766563, 0.35496240007142166, 0.18553994230600823, 0.4810238674821691, 0.3695808779793378, 0.6150319628381469, 0.47055576606936167, 0.4841043730472912, 0.327627962167344, 0.7850600276836799, 0.5887659121819481]], [[0.29007213454131714, 0.5840636808130023, 0.4609785274883872, 0.992809953723125, 0.19058879190626135, 0.728482471903402, 0.395056561838712, 0.44603673293071655, 0.574233974778325, 1.101207046500685, 0.10525915440630942, 0.811231883578343, 0.6771303081920741, 0.48549501039022824, 0.3470085789024435, 0.40091403822629124, 0.5374385899224321, 0.7556681650105896, 0.7087703926812852, 0.16924740698027896, 0.19096360382918792, 0.2851241079133971, 3.4870886584447467, 0.44586044609312403, 0.32376760737501453, 0.3638305486057889, 0.2227170641494059, 0.49890592684427354, 0.4465338898436897, 0.6455199968576839, 0.4772635239158872, 0.5587290055999852, 0.34118891073021573, 0.8363401499239123, 0.5776232332410516]], [[0.2886569727865159, 0.5667706895545646, 0.4361638185840849, 0.833223663381085, 0.19102780910635098, 0.721500943602086, 0.39304672774478394, 0.39677642594792284, 0.6052810507377695, 1.1502538713690929, 0.12123341962326858, 0.6756066471667408, 0.7673471965429846, 0.45066016407148685, 0.36042730744183005, 0.42112021719769854, 0.6153247745958449, 0.7847202136067564, 0.7002158885206976, 0.17597194202416644, 0.19631780692794026, 0.2821167291933166, 3.4870886584447467, 0.4262537548419343, 0.33725732559111776, 0.3650617028526699, 0.27617259919539355, 0.5466392485443523, 0.5276916531415252, 0.6609244276867066, 0.4899764307425214, 0.618443541142099, 0.34360523649124647, 0.8850976620363606, 0.564880930251963]], [[0.2635773772554964, 0.5681949830840921, 0.43937296002910237, 0.7713790548282724, 0.21869554699619037, 0.7330402208513576, 0.389726142362665, 0.4059563380837184, 0.6857014274941047, 1.1374122719043258, 0.1491456975161103, 0.7357495488148936, 0.8100159618727516, 0.6043587897212184, 0.40557034687834215, 0.41295562558853294, 0.7355077254113089, 0.752340577767411, 0.6608814550288376, 0.09423652291906487, 0.18180113217971403, 0.28459030788400563, 3.4870886584447467, 0.37088186507403414, 0.36354649715343046, 0.33743010406115637, 0.3300953139697748, 0.5216782503283082, 0.5365897453515025, 0.6513995328414014, 0.5438603681317125, 0.6204923154710461, 0.3253799369410457, 0.9439600019248593, 0.5234452599435462]], [[0.2338084159264503, 0.5551757125972104, 0.4518257779766134, 0.721030885001759, 0.25623571673199513, 0.7696773036249643, 0.3855850134128134, 0.4226857570613922, 0.7901003071786171, 0.9712099268109123, 0.16184827215830658, 0.7575531152554325, 0.7502887152134838, 0.6242380704574799, 0.39834640579446334, 0.4258718897060516, 0.7441770925593064, 0.7277656217775529, 0.6820687475939795, 0.08750309216607632, 0.18265328781006016, 0.31026954225157655, 3.4870886584447467, 0.3075632231222425, 0.3815874169810598, 0.29757568499640713, 0.321397255314567, 0.5599755783400396, 0.5531089034769042, 0.6204052916263414, 0.5840379088223662, 0.6360699356586521, 0.3060196297650666, 1.0515515412391143, 0.445859015508054]], [[0.2320306349137915, 0.5491701510846003, 0.45383757762308446, 0.6671552749791382, 0.26131796423707837, 0.7848653479523996, 0.37816402781489916, 0.46789155487228795, 0.8118199979084232, 0.9210876716777163, 0.17929085268799952, 0.7649591610910391, 0.6801003768337559, 0.6762241980165965, 0.3433775246285773, 0.45496215287954245, 0.7776126364271767, 0.7301408724968029, 0.7014768735562801, 0.0995490726123438, 0.18294658803897532, 0.29932578281976074, 3.4870886584447467, 0.24533497332180618, 0.393982294433555, 0.26936178979213254, 0.27447611455835697, 0.5689813684538038, 0.5454164128419746, 0.5606922006051935, 0.549348929146945, 0.6629358940260915, 0.284734276779692, 1.0350616698479032, 0.34363129930196534]], [[0.2769394355999132, 0.57442393033084, 0.45593769302941656, 0.6135467358955007, 0.36837298273572344, 0.826830581564354, 0.367907351309395, 0.5058706876818035, 1.275601685923161, 1.1602087551830897, 0.263492715119645, 0.9164879801055268, 1.08028190771615, 0.7091221485774275, 0.37052706319179407, 0.4947950122895247, 0.8199274581883278, 0.7538143557045052, 0.7340372446737563, 0.0680013960583693, 0.19544776744731923, 0.4198348004689036, 0.051271318816271165, 0.27587541213357836, 0.46089217446394287, 0.41181224131557836, 0.24723345758684256, 0.7545043140110848, 0.6257244137057657, 0.7152593244979453, 0.8262190642607594, 0.7620661590884421, 0.45782869316132224, 1.486829009204277, 0.3653383957192497]], [[0.2665407731503124, 0.5314545803907923, 0.3944701857355231, 0.5115638493306746, 0.35116933162317837, 0.8057979909344136, 0.3883609702762969, 0.41967913786787786, 1.273141617389384, 0.9764764951975227, 0.28002710069806874, 0.9137904716946152, 1.1646271270369941, 0.6954543692928425, 0.36575230890361654, 0.4961291085863838, 0.8251814373930931, 0.7319232123673087, 0.7302980585450323, 0.06635726049906862, 0.1945903386744157, 0.31336599880519933, 0.0322632798866746, 0.14646819318615484, 0.6093124512516002, 0.3554445107425863, 0.20930746607548295, 0.7242277566544846, 0.6744121395943171, 0.6249266125800488, 0.7089368787512531, 0.7136409714152302, 0.39316145780937223, 1.5563067711006395, 0.5731814679634056]], [[0.23845865101867936, 0.5961391736367266, 0.4503807547090061, 0.49975478754423214, 0.34014056623441946, 0.8775619862383061, 0.3757768281931021, 0.42517135251151117, 1.23956757028481, 0.9551428994634192, 0.2600901303261717, 0.9383386113100893, 1.2389947636599978, 0.8132519337471065, 0.3744063188765948, 0.4941036759648113, 0.8643883346589742, 0.708258406678617, 0.763717295632973, 0.07507643135457381, 0.18676503975565262, 0.257971762043509, 0.02802092974562491, 0.11249788951041984, 0.8224447991414306, 0.36865702475981343, 0.28184165068333505, 0.7324203223499566, 0.7600220458785326, 0.6464006560850956, 0.7643981168147904, 0.7271895634255063, 0.29526934716580666, 1.673200003133629, 0.8685555835159937]], [[0.21274513187400806, 0.5867781841743639, 0.46392249778807926, 0.46081084494408586, 0.29470295098767657, 0.8944193690732027, 0.3963462875052706, 0.4119843695231813, 1.2016810462229237, 0.8390363975005632, 0.2695080517171468, 0.9586344934794349, 1.394918936831553, 0.8146509341105564, 0.34925823819216373, 0.47948249031839607, 0.8643014975912837, 0.7512237856579183, 0.7884749817300741, 0.05700145593925765, 0.17779661160686908, 0.2258838213311668, 0.031065719399000505, 0.1462030150641495, 0.8772114978843943, 0.41415471118279557, 0.3358062300587898, 0.784824297769586, 0.8140881132301226, 0.7943860348420781, 0.9029790864724092, 0.7359541845398854, 0.3251461109646302, 1.6288844495840906, 0.9443220200708671]], [[0.1974211906417424, 0.6082337440757921, 0.5111480761574075, 0.433277972813346, 0.2711248779275196, 0.9760972845092865, 0.36947214170088066, 0.3146553002748393, 1.120015804637929, 0.8936021842410918, 0.2635956334419857, 0.9945182502745247, 1.3798724180785131, 0.7659689913763359, 0.35147826078403854, 0.47287258531366916, 0.859193202609242, 0.7405509161063462, 0.7669809762304243, 0.06058111095830776, 0.1647332702045075, 0.21077639956917055, 0.03161585657033497, 0.18548804192854546, 0.8104425307393901, 0.4195588662715699, 0.41624886501478986, 0.7913532652820243, 0.8744124435756611, 0.8343905619084053, 0.9396810477387199, 0.6244869994958884, 0.21953038993757817, 1.6387952101778778, 0.9326949025260034]], [[0.22847457293343626, 0.579484321837898, 0.4959555772313944, 0.3997144172172982, 0.2587435977061834, 0.9658250300217148, 0.381709531784927, 0.24644011369899782, 1.091016645478843, 0.8879766035634898, 0.25657096119832323, 0.9696243001236404, 1.3778526631563963, 0.7909867866601634, 0.3391939569314796, 0.47866792708603634, 0.8489291585096824, 0.7221463458767458, 0.8172791784441741, 0.08588087169881103, 0.15995648001805068, 0.1944730383871931, 0.034045793769589196, 0.2205830074132226, 0.8846564191351713, 0.4076371083798034, 0.4963541717940198, 0.898627263918784, 0.8993135427068186, 0.9014482662408703, 0.9489572166550849, 0.5705387529186268, 0.23822969722795678, 1.6955884265259842, 0.961972165385824]], [[0.2031434205788098, 0.594224636254855, 0.5017428756520613, 0.281021272263279, 0.2489236926102587, 1.0330632995672704, 0.38093167999131017, 0.1439314686412161, 1.1710780104806644, 0.8464719055871734, 0.2817517720371372, 0.9510041705683653, 1.3631494491915461, 0.7997764551532804, 0.3409847507355199, 0.47067793349652554, 0.8209076396405605, 0.7006066536101252, 0.9032632262859099, 0.13045304987251727, 0.20192170402485607, 0.23062187281953156, 0.03548673057019224, 0.27387873722909006, 1.1002571651284856, 0.3978110115064593, 0.5776147723449205, 1.0850600863991826, 0.8492819539911453, 0.8852597622148495, 0.9534943567149778, 0.569304852767953, 0.3059967415916974, 1.6593571093076354, 0.9338690273887197]], [[0.2021254194217944, 0.5722121114173303, 0.49577644477674015, 0.28476552735784316, 0.2520192370744229, 0.9806937532697635, 0.3883967308063126, 0.11184565597206514, 1.165479992156596, 0.7241369304350066, 0.28361758557392486, 0.9398070048863014, 1.323371748279286, 0.7973760491823898, 0.34030258983365413, 0.5183009126176555, 0.7478310055318095, 0.7130336259692103, 0.9266815981554957, 0.12580823607715524, 0.18294267721470125, 0.24204908203364361, 0.03533073487026057, 0.24708804280578509, 1.069884552728044, 0.4343943117908134, 0.581145790038106, 1.0891701612084186, 0.8405162937638148, 0.8911665561691728, 0.9688756749408751, 0.5737712669630755, 0.37263336986697604, 1.637986816799019, 0.9949451835217912]], [[0.1980049678944823, 0.5566499864064375, 0.47062476014072263, 0.30295782639132823, 0.24836241414866003, 0.9997013656214413, 0.4013049907836275, 0.09761690611623147, 1.1470034937600968, 0.7793554909996027, 0.29022234206122427, 0.9376661316345384, 1.2666616268776352, 0.9509038480798764, 0.3415126870261168, 0.5214034235541863, 0.7958262608260945, 0.8273520432016679, 0.9312066265093558, 0.12921711051163234, 0.15053607734141622, 0.2508397780321794, 0.034350620804966966, 0.21566977016286237, 0.9784931550178648, 0.48897450707163065, 0.5612865152136884, 1.1125607314384491, 0.8298286385990161, 0.9233986255791822, 0.9958332967203496, 0.5667064955678622, 0.44774931697749143, 1.5708901756862121, 1.0851648729361565]], [[0.23967429736050577, 0.5709114111573066, 0.48884773508202095, 0.34060600376640876, 0.2182950967634007, 1.0162381766465385, 0.3946270738419255, 0.0974258861202201, 1.117555323901795, 0.782435799515784, 0.2742868107914048, 0.9000749508157299, 1.3404093772248464, 0.9391678731949706, 0.3361238697444271, 0.515659436485654, 0.8387840968446787, 0.866078607928787, 0.9678507722914438, 0.14939318819477232, 0.14101285688239204, 0.28416636709658616, 0.03382936291753517, 0.2069592773555028, 0.8540539483570677, 0.48681658565437014, 0.54463048030716, 1.0905619597500196, 0.7742841980591703, 0.9381599283581565, 0.9292137757971983, 0.5451411912662232, 0.48173396628373333, 1.5405007257364967, 1.0701339115440185]], [[0.3898325254758548, 0.6015773296227441, 0.5383063085456541, 0.4767747346718528, 0.16655630142525707, 1.0647922988837166, 0.3878885233770197, 0.11654284248936952, 0.944637942331231, 0.772653412243772, 0.30216240391399696, 1.0154886888641632, 1.2772619001495111, 0.8118024828504995, 0.3362208832814171, 0.5171138682674185, 0.7326831960905302, 0.8014977564257619, 0.9130038769119608, 0.17905344453320998, 0.16130267940108148, 0.27515585564716877, 0.030500952523708567, 0.2031070647362401, 0.8004948411837811, 0.5197331721916809, 0.5195032315624432, 1.098137353941953, 0.7228539221540564, 0.9566483576376551, 0.876268352771211, 0.5421746688129367, 0.5720424023256128, 1.3314287840321146, 1.0467769368944193]], [[0.46922137581541495, 0.5757633895199199, 0.574399421181638, 0.5654823599069175, 0.12675280950953452, 1.0571888722223366, 0.384578441183918, 0.18345548939620182, 0.8331338654842804, 0.7225602092712267, 0.3395905572918089, 0.9889075645688861, 1.347931756391245, 0.7695540779102998, 0.34231722095449646, 0.4699594325821732, 0.7628343149368284, 0.7150374587857663, 0.9489334284467911, 0.19141215948562285, 0.17900094782894538, 0.29351370009482286, 0.02927607382611125, 0.19071731698541622, 0.7951207595443397, 0.5603796186704535, 0.517313185060549, 1.0671339791912848, 0.7383546842924095, 0.9581331312738011, 0.9021822682374694, 0.5387159058393473, 0.6794798093149885, 1.0517728817718326, 1.0319797793370051]], [[0.449545792153677, 0.579303039550766, 0.49551077242852154, 0.6697078228742761, 0.17437278999434447, 0.9953395112141994, 0.3816096723841, 0.29707264669204153, 0.9114237894813937, 0.6640593599444238, 0.3403762370214139, 0.9417092389618549, 1.2986696913093212, 0.7069684526816431, 0.37823646389061993, 0.4467927157101347, 0.7270787996284551, 0.7188506917921504, 0.9351638959021858, 0.19415778625705357, 0.20921871694408173, 0.4078823667976036, 0.017995295020459958, 0.14997334282678113, 0.8155745737471751, 0.5213809979832097, 0.5263717169927671, 1.070565434858981, 0.7505950712849079, 1.0405534043343723, 0.9260562064958568, 0.6231576372699487, 0.7558553567252708, 0.8908315084309115, 1.033132898693156]], [[0.4448357152480368, 0.571340237044299, 0.49016663817915307, 0.8286541427061477, 0.35954807240750475, 1.0028836732825834, 0.3587938167179598, 0.48435069274476883, 1.1250642579940175, 0.6257501108898191, 0.31130834802868246, 0.9887361991428343, 1.1806480467127618, 0.7230078934848909, 0.34177719772610515, 0.4725336860572723, 0.7530384892640324, 0.7142002875684024, 0.9422421937271401, 0.3880957323080507, 0.28617866850178336, 0.5792541184797606, 0.01551239357632261, 0.12446966495496484, 0.8323544117687884, 0.3085948592329947, 0.5335028647387793, 0.9119155648751099, 0.7668482480590901, 1.1486307441896013, 0.9690656634741608, 0.7410389972528659, 0.7913653224153379, 0.8721011335611337, 1.0213827314988788]], [[0.4235897435722715, 0.5859272440141099, 0.4856889539526885, 1.0149721164526897, 0.3168601555373034, 0.9125286763799622, 0.3726772794904167, 0.6392994034661619, 0.9972209289562708, 0.6046405237501353, 0.24732901934566243, 0.8400766975650871, 1.233054689288011, 0.8537307464812334, 0.3367551594284156, 0.46659293332864293, 1.1624203502147763, 0.73085916429604, 0.9106677893204957, 0.4131085969100688, 0.24346585098837023, 0.6035305851281378, 0.015221233918045161, 0.11682026274355836, 0.7784746352746955, 0.13463044568103424, 0.5489765033598402, 0.7468921965132698, 0.728407771112179, 1.1506419349168342, 0.9549765675863299, 0.7393778730867737, 0.8125835026006967, 0.8647546205261507, 1.0323517369360422]], [[0.3861667057993827, 0.5678615359446005, 0.45813387168650527, 1.112592748850384, 0.3884152529638067, 0.8322781234460938, 0.3853132501768977, 0.805794411184291, 1.0508244063838772, 0.5821095508768541, 0.22621007497763845, 0.7891813511506144, 1.0991630730672781, 0.8622355851021253, 0.3371344860113582, 0.4553824709673907, 1.1471712780457566, 0.7434355790298661, 0.9806135791152797, 0.3654927880084693, 0.20075320987990147, 0.49168228086840654, 0.015636789505500882, 0.10357729419455969, 0.6237999344791552, 0.11492938041138168, 0.5693991455277775, 0.49101529973504643, 0.6244191639415136, 1.1399073140862896, 0.8255444393512171, 0.6608455701268443, 0.8000021181852587, 0.8836216191481326, 1.1016832674433585]], [[0.3834370187579212, 0.5649797025543186, 0.44933013032083424, 1.2079551923441614, 0.3981408030019149, 0.7882386558625938, 0.3857464045795667, 0.9191003071547377, 0.9521747221965582, 0.5713329892127716, 0.18460724130708855, 0.6783112906960241, 1.235253851119432, 0.8654816042184242, 0.3461333568896554, 0.4278525843278427, 1.0634487289981887, 0.7810995078411767, 0.985249902019153, 0.27494679361540486, 0.18593764155352177, 0.38511130888560335, 0.016079447514004773, 0.08429613135049452, 0.5722584822688739, 0.19058495914388762, 0.5704418139364624, 0.4094905228513324, 0.43741014275152995, 0.9975449820892006, 0.858820124300839, 0.5421909815126645, 0.7807385216140206, 0.8712566852890604, 1.1284982493735527]], [[0.4019216709180878, 0.5570347452468811, 0.4229482804817886, 1.246530878237638, 0.5430943275563592, 0.7601548913709621, 0.3852752090489886, 1.0779280864835052, 0.8987701305669961, 0.5976635702309671, 0.1624098888261724, 0.7372041474764899, 0.7789222874464117, 0.77996793407091, 0.3600367049815145, 0.4103863674746807, 0.8926047402010915, 0.6451719112306873, 0.9153218464279874, 0.341563258105454, 0.16528370771675763, 0.331753661700428, 0.016113051635742913, 0.0983716917624225, 0.6894941218613415, 0.2604545709694821, 0.5706111598034757, 0.3582436862356055, 0.41215435612907597, 0.8178042949520401, 0.935124598301915, 0.47567001165178213, 0.7196288182243056, 0.8286519392992469, 1.0505308967767735]], [[0.3858075903858046, 0.6360597130287623, 0.424948792921684, 1.2938168905642686, 0.7408340137082653, 0.8168607357996693, 0.36275607655998504, 1.1294267912457352, 0.958800767399894, 0.718266136360714, 0.15060950240581963, 0.7235002910952419, 0.912680360759539, 0.806025461840613, 0.34622410701875206, 0.4675165535121616, 0.893116462286889, 0.5901336479979644, 0.8634697122274655, 0.4773447469458032, 0.16151478443850026, 0.3142846565411851, 0.015749604371819895, 0.09079221639956642, 0.8596421202040515, 0.29859199808702175, 0.5740608491216281, 0.49624681919192437, 0.41345655756569377, 0.6984428168619653, 1.029407743107901, 0.49282279707488297, 0.6670661396244966, 0.8058375081169763, 0.9442507276359524]], [[0.3912998005747453, 0.6056370506096128, 0.3818602718171929, 1.3623368437254684, 0.8801492066624615, 0.7871888172651306, 0.36956063553187113, 1.1735456598858367, 0.9843152023838441, 0.7286792437394881, 0.12327965639425936, 0.6379819899835542, 0.9762084186270126, 0.8193582901133643, 0.34924680001705277, 0.4776713774849428, 0.7281315769822734, 0.6158054771512255, 0.9655617380820805, 0.49020091215938877, 0.19144839740621183, 0.313239904330956, 0.014765007164060196, 0.057387464392013005, 0.8979714735693252, 0.2911893111874322, 0.5512437578262235, 0.5754871194182061, 0.4177648614243487, 0.6156874416147128, 1.1056007391678762, 0.5745249058776174, 0.5242191798907527, 0.7779285599043213, 0.8874849735972195]], [[0.4230666490214006, 0.6064176848643429, 0.39164000106964997, 1.4556894412564079, 1.002956871335006, 0.7637665332896976, 0.3958256462777894, 1.1854299963484796, 1.0168863775502595, 0.6495773227030724, 0.13033827238937695, 0.5860772112574764, 1.056403113029179, 0.7852405549783095, 0.2836019709369544, 0.5019258664051455, 0.7092315024123431, 0.6047865813247251, 0.9252648311807949, 0.4775993992004538, 0.18336409973866025, 0.3190649207380937, 0.01393734876959224, 0.048388434616083054, 0.8384474044395953, 0.30091978523677726, 0.5372323860242514, 0.5481141347783447, 0.37295665347539764, 0.5368962773749308, 1.0769351263859952, 0.5724861117862706, 0.4336186971514198, 0.7497541100713943, 0.8470733612505276]], [[0.4458024362353803, 0.5994157100162145, 0.3969512361146541, 1.5335062415997638, 1.0996987725131082, 0.7766687044193472, 0.4505660411094141, 1.2185940946176463, 1.151700349600016, 0.6404580119572133, 0.19329533957340989, 0.5751727824240066, 1.1349124634603807, 0.8041864439897703, 0.2802467562824019, 0.5246597015136942, 0.7625629639392365, 0.6257774756872159, 0.8835159956019458, 0.4598550712419787, 0.166295931940414, 0.3317282582983672, 0.017009735207250173, 0.05817143607421682, 0.7521916570861469, 0.3302557234974131, 0.5292518667196398, 0.43604040081961576, 0.4051139834656475, 0.5364769514175799, 0.9709026773574182, 0.6921004282946659, 0.4220955496968068, 0.7178216938634157, 0.8453269377898147]], [[0.6749848286621067, 0.5694407524053972, 0.4081255969113278, 1.5358100159322434, 1.0522525357532082, 0.8036505801905379, 0.4208122763925627, 1.2874009802837025, 1.1651735049058032, 0.5391421862257894, 0.24161138945645344, 0.5886625756599045, 1.1849795857320093, 0.7091731092535504, 0.23057980320251603, 0.4209268321750361, 0.731402591583035, 0.6369572875092256, 0.8136020915987242, 0.4707967427051498, 0.1665071802930796, 0.33962189696760586, 0.018159557813655195, 0.08082804793692971, 0.751256438471138, 0.3134429223241501, 0.5014891822116534, 0.4370143687161234, 0.379690088456606, 0.48405811222812767, 0.9666603479740589, 0.8039376477821457, 0.4272464003362514, 0.8400869081256619, 0.8178256577711718]], [[0.6781469661388, 0.4827348399945336, 0.4356150966104534, 1.4635536439718815, 1.1095026462986821, 0.7104484765897681, 0.4062879139947005, 1.2526178746188152, 1.165830475749754, 0.5373551274731582, 0.282098463493734, 0.6017269531539243, 1.18692390528699, 0.7121287287681901, 0.23088716449084684, 0.3464166023662244, 0.699234758699387, 0.6585689369642456, 0.9017852789431702, 0.4902742066299264, 0.1634372970205064, 0.3819571378886519, 0.017645916512566195, 0.0969921157043038, 0.7689929836305114, 0.3130520856810608, 0.4610394530774793, 0.5033115659244729, 0.4223593474278632, 0.5134466672619047, 0.9572076422848856, 0.7722523190372572, 0.43058197394587816, 0.8449354022004545, 0.8076978602000958]], [[0.6985897845310902, 0.5364775154591193, 0.4270398628968683, 1.4317745505167705, 1.1625593807814643, 0.6722260322915671, 0.39973089715043597, 1.1989728055543036, 1.2606655389874613, 0.6057291474400097, 0.279794333507102, 0.6008836798858781, 1.196406327436222, 0.7266202824762745, 0.2303352742031567, 0.34561873791796405, 0.6696856259310943, 0.6657522092814103, 0.911907751911512, 0.5048924118633382, 0.16814925635161418, 0.39617079846319664, 0.017213129129063844, 0.11325572297111527, 0.7725683853348149, 0.3234807394287759, 0.4054427400951901, 0.583342383891224, 0.5059927722061112, 0.5130128871890086, 0.9263154209899728, 0.7241657972018827, 0.41674688255927633, 0.8093985709209325, 0.7805933747473839]], [[0.6970766096476362, 0.5675609124421559, 0.42846928446838206, 1.422715507046242, 1.1234658843522285, 0.6733530612985025, 0.39269756004822165, 1.1101653301310985, 1.1647202203278524, 0.6798739834315662, 0.3001001085810274, 0.6071561456981529, 1.11826175999169, 0.7346815622239643, 0.2459393051428184, 0.3641573014005608, 0.7024604238881594, 0.6866684218992403, 0.8830135636645331, 0.4940499539762785, 0.17170306201669272, 0.36554754987237614, 0.017831852629676598, 0.1633359478960354, 0.7628728301210569, 0.3518621148579568, 0.3660431714093469, 0.5874300285198257, 0.617676732999827, 0.4421444611891434, 0.9068131440396034, 0.6241521275220702, 0.3921586046104854, 0.7865717097092472, 0.6973723923117167]], [[0.7087138397809314, 0.52333838483455, 0.4059421468015247, 1.3866289865042998, 1.188481797293887, 0.680204986475331, 0.3834579600386712, 1.0375361354599384, 1.0963349695557372, 0.7699384507841376, 0.32732808774616884, 0.5982745351397379, 1.249970013097243, 0.7521043276669921, 0.25130841938342385, 0.4029356678008009, 0.6184483657789785, 0.6507537791441955, 1.0564798625380338, 0.4584843438261221, 0.1860051706144196, 0.3822336577449155, 0.0199893319429468, 0.2508032368050147, 0.7478020337018302, 0.4117441575200087, 0.32438500322785613, 0.5696697145249932, 0.7473796088443728, 0.4298072896413576, 0.8949486688665522, 0.6751871215484155, 0.37651449491109085, 0.6970263343992555, 0.6537652549618449]], [[0.7195515510849955, 0.5500339385626191, 0.40977707883425785, 1.343449694409867, 1.2913481609327544, 0.6813926047064289, 0.38328089035422075, 1.0396487250749757, 1.1207991597668803, 0.9144007959786304, 0.35052377509028987, 0.6045858697272136, 1.185857822480999, 0.7361374855630191, 0.23614761033343878, 0.4415809541863688, 0.6560634542787747, 0.6540428787513386, 0.9758867109973932, 0.47508967146025155, 0.19302917380653456, 0.4255693993555152, 0.02320145186704882, 0.3544118606449334, 0.764814962369611, 0.5346625038663784, 0.2820437230590879, 0.5510238968875598, 0.8488160801131859, 0.4166673954661789, 0.7974157614681023, 0.8401534763676246, 0.3829547547144594, 0.6466853959045897, 0.611942390277322]], [[0.6571808721217895, 0.5201201125712709, 0.42342479193909566, 1.3088672792600664, 1.2253950010975274, 0.6813588766083627, 0.38273429102750084, 0.9881912373069914, 1.3190224863860065, 0.9580702751809173, 0.3901682065679517, 0.5996152293992869, 1.1711363141624833, 0.7161739300423204, 0.24619394967142594, 0.4381541963946469, 0.685713599064257, 0.6916213233792036, 1.04180087186805, 0.4445253710730021, 0.19383913207068432, 0.4280136654611276, 0.02842402247576191, 0.4574721879986091, 0.7028036840201345, 0.6460221283364247, 0.2748677389024934, 0.5282816314326467, 0.9799395079244486, 0.45279220959295885, 0.4791533991879739, 0.9903157868194263, 0.42123818769915855, 0.7319751388572793, 0.5527155428903656]], [[0.6036434428695407, 0.5490144407223332, 0.4168578192147393, 1.230333095194104, 1.184971966881094, 0.6693189141986267, 0.40554251431603794, 0.9647789633991661, 1.4022276592690048, 1.0788412156963014, 0.3722105279517878, 0.6019353379944614, 1.1630667867189068, 0.6173571218515963, 0.25858385823701774, 0.41138399724632857, 0.6747148483125862, 0.7205985059001428, 1.0127879452493969, 0.3706610706101195, 0.19402225978560736, 0.4523038097287914, 0.06870701594555592, 0.549143824604841, 0.6247708635294138, 0.7186146567281279, 0.27695149287390125, 0.5496484154560315, 1.0523587391814457, 0.5193351154216239, 0.2077332575734452, 1.1009056334996594, 0.44135286313369493, 0.726302962534692, 0.5212601210789751]], [[0.41640871459254614, 0.5330698067775176, 0.40046662501334984, 1.0863618128625507, 1.025259999547399, 0.6397038195226289, 0.3993877315159599, 0.8105589230371865, 0.874620442754168, 0.7447759374363192, 0.29155191013859033, 0.6086116104525954, 0.7417806776520248, 0.5902388278238866, 0.263728198829412, 0.41444949369648554, 0.7079508279402894, 0.7190294001784765, 0.9024289937574099, 0.3829005699090672, 0.2042334743498734, 0.33594978225369676, 3.4870886584447467, 0.33603376448190847, 0.492087509474773, 0.47535534624841547, 0.2567528994560272, 0.5460983918020939, 0.7750295802932445, 0.4960581391489437, 0.15334401944864479, 0.6454903647799768, 0.2936860178083459, 0.4647445573099196, 0.3930513583653848]], [[0.39285956250411136, 0.5378494820827145, 0.4039575582043541, 1.094074303033617, 1.0401147516687377, 0.6028327196154661, 0.416706086504402, 0.7975037158811638, 0.7738830001751125, 0.7204791020252255, 0.2811524835922151, 0.6202543599472977, 0.6007772210672558, 0.5839678090558993, 0.283695535268393, 0.3754300140486504, 0.550041756080777, 0.6968287247171274, 0.9049387934057782, 0.38458034806817476, 0.21963656348399335, 0.2570152416366207, 3.4870886584447467, 0.3158190559730226, 0.41499338709566846, 0.40020686495717306, 0.2877264480010985, 0.54229625673113, 0.7926082962258845, 0.5489157867846312, 0.17136161964991772, 0.6253147877147116, 0.34147990708412995, 0.3786070797815172, 0.35097922505616497]], [[0.4099426440255691, 0.5851045666196538, 0.4380392329133215, 0.9891589691885336, 1.0328564794664943, 0.5937325909690176, 0.42770414246719235, 0.7867791080491248, 0.6275448286090819, 0.6075060731052588, 0.31467500938210946, 0.6155700741919093, 0.5611419007649121, 0.5512036422581605, 0.2579272622499032, 0.3998078111605081, 0.6081245009336282, 0.7038979043330706, 0.8555649545108241, 0.19622357271379565, 0.2431547283885669, 0.2391596699512127, 3.4870886584447467, 0.3181371292210121, 0.3213782189475063, 0.3972700074031271, 0.3109217097905346, 0.506773212318482, 0.8129811650210563, 0.5965571965523023, 0.242837804682883, 0.6138610824756092, 0.38364138879532406, 0.3630700070125562, 0.35474026724869695]], [[0.4167152584615962, 0.5701418829475744, 0.4654417215597396, 0.8265438921056614, 1.0332015994105859, 0.6202609082404699, 0.39732712943405185, 0.7150310373300306, 0.5438136541618546, 0.6331358083044383, 0.31891361528285317, 0.6094278766042799, 0.5771795212744244, 0.5498985228389234, 0.22989533846951174, 0.4047973677142148, 0.6096082389496689, 0.6937134018282157, 0.7907070065898772, 0.17199789046110142, 0.24013946305441297, 0.22748160514943225, 3.4870886584447467, 0.33033943479819533, 0.27623180603278086, 0.4367062670773921, 0.33280620070029043, 0.4992895705785754, 0.8475621945051733, 0.6360214975932732, 0.2586339689054722, 0.6063151078341715, 0.39844683019404237, 0.2845181944992254, 0.3545859621134556]], [[0.4101409701255312, 0.5901694060475681, 0.4954956297286076, 0.7158138557607359, 0.9888341848375113, 0.5859094142265722, 0.4614889049499984, 0.5887201649270856, 0.6278741091808858, 0.7938030789680899, 0.2932836669006205, 0.6104216884923861, 0.5242134312796333, 0.5519009471538757, 0.2393398844059468, 0.39636897109383695, 0.5984556428241325, 0.7320195537682745, 0.7691730476475631, 0.2091492436655783, 0.22323965990087308, 0.22069050819853528, 3.4870886584447467, 0.39353356789513994, 0.20419076272076192, 0.5133337823264754, 0.3353508072868534, 0.4592952584130374, 0.8824464207820495, 0.5422591786382434, 0.28407620357173813, 0.6148069322704198, 0.1933974145913417, 0.24640403194415095, 0.3300196182083378]], [[0.4035501328178399, 0.5704738536616882, 0.49093241420455963, 0.6197073966760545, 0.9519814207161219, 0.5815853275194751, 0.45437610317576116, 0.5826259577319481, 0.5908241909785354, 0.8127476333478424, 0.2680622199861158, 0.6023127766592712, 0.5250933246143378, 0.5564651631540309, 0.26244528902440256, 0.3774960673705203, 0.6391031002838985, 0.7495910396695086, 0.8107294572167778, 0.31373562751354306, 0.25183463056308925, 0.2190382308813058, 3.4870886584447467, 0.4072922923880451, 0.18828845841212616, 0.5755682888797977, 0.3394192161678984, 0.46237241432029985, 0.8804415192904134, 0.5219034202973477, 0.31843431981196446, 0.6161436272423051, 0.2874796643753399, 0.2520219518486013, 0.32599977161753396]], [[0.4636898426718811, 0.5567848978969897, 0.4935148518626892, 0.5947875112030787, 0.9270230440346945, 0.600994426525331, 0.44443389329999927, 0.5866019002150817, 0.5648723283507678, 0.8054308368371481, 0.2576483536712023, 0.596211647427316, 0.598345803709611, 0.5517366434540895, 0.2303592372383219, 0.40512350910957207, 0.6089648749674049, 0.7119588290758768, 0.7854659605927756, 0.32706910537551726, 0.2944577250157159, 0.22953838487733352, 3.4870886584447467, 0.43019559968131293, 0.18550072635754877, 0.607903361786048, 0.3452953758364479, 0.47759758409385583, 0.8748011095255426, 0.47823447714950945, 0.3529266380551803, 0.624770420590776, 0.2308326990848577, 0.28816143527038784, 0.29508445212562084]], [[0.4619346498067123, 0.5377160506595358, 0.5020166137018943, 0.6163631050082466, 0.8404822427177044, 0.5968879653140338, 0.4636501663341577, 0.592309296655629, 0.5497169037718715, 0.8562568288430725, 0.22295059069191303, 0.6009175757251471, 0.6356799957152762, 0.5547072489552625, 0.21501157260681972, 0.3940774490762743, 0.6181778076078763, 0.7163176322996786, 0.7827912709451754, 0.2727070416410145, 0.3017283462583099, 0.24970342715152327, 3.4870886584447467, 0.46381734985562656, 0.15646131028328103, 0.6149435004586132, 0.3244775788330429, 0.47952158197551187, 0.9188821286959876, 0.44730268061071327, 0.4045624892691506, 0.63711418314396, 0.27773957603250354, 0.30660724224570335, 0.22489543975581316]], [[0.5554086351595993, 0.5282446477406407, 0.5146135468222608, 0.6474458132468199, 0.8076524495052999, 0.6342192099559791, 0.43116232881532834, 0.6317054681705961, 0.46367206240299463, 0.8034838621067963, 0.17768899854457645, 0.6015749310397659, 0.709287917394761, 0.5946268370607497, 0.23990753161034872, 0.3977839191377157, 0.5983768405271164, 0.706281092162465, 0.8610724013021565, 0.2074894807432736, 0.29628318899683437, 0.28404719888594754, 3.4870886584447467, 0.5198908344646267, 0.1721441606380611, 0.56768386478322, 0.3003779744488865, 0.4702849052556346, 0.8281896089359249, 0.4329020591083437, 0.4441511739033488, 0.5990023810823016, 0.32472995868991766, 0.3494665619551923, 0.15066808512724778]], [[0.5571029149248157, 0.5133082886018752, 0.5204843217565414, 0.7711588457378702, 0.7757207169585263, 0.6233294889105738, 0.40438405488580637, 0.6003981608045891, 0.49329980275904733, 0.7701395227777235, 0.16151642242964914, 0.5922269565467247, 0.8188241415403308, 0.6367605306508848, 0.23390865890366594, 0.41395045707800193, 0.5611928384651675, 0.6652146549562459, 0.9140927871328454, 0.15622769399462255, 0.2611937624577248, 0.29977336826698425, 3.4870886584447467, 0.510080737963077, 0.1914358690832783, 0.4964731642636381, 0.24308882984852798, 0.44971372441807655, 0.604074524943525, 0.40164286281344597, 0.45333399368892324, 0.5408008639550252, 0.36714520707461207, 0.3246621167917635, 0.12000243529343035]], [[0.5135603200325538, 0.49549359084786543, 0.5044807403497553, 0.8228566861961171, 0.7555972055505752, 0.6164659881867169, 0.39581871569052446, 0.609536484379547, 0.5823828254282872, 0.6625194855063434, 0.12853763595858125, 0.5756587926737897, 0.8987743896871168, 0.6893204415174872, 0.22429726883541615, 0.6145757580552657, 0.6357753433444115, 0.6349214739116562, 0.8627524970490443, 0.1753045543163093, 0.22664278107666186, 0.2468759099257019, 3.4870886584447467, 0.4184397065130229, 0.25230171440294563, 0.4609424813319904, 0.19493249191565165, 0.42318741254304865, 0.532903053536557, 0.2755441944471235, 0.42936578094068467, 0.5687143034590552, 0.3907316737740998, 0.3276736442579699, 0.13473945751979044]], [[0.5112903219745114, 0.46419173528088503, 0.49118116351743146, 0.868972508033379, 0.7235003794392063, 0.6113925676017766, 0.3957455838089439, 0.5824070730269725, 0.6843710780743423, 0.6161933717152458, 0.09650813826069658, 0.5711439824776784, 0.9052015922839964, 0.7417885160471492, 0.26335028615422684, 0.6169700095916978, 0.6012557230543565, 0.6226604983318287, 0.9405587813491006, 0.16216255741874663, 0.254885493936477, 0.2207817191906589, 3.4870886584447467, 0.36384369783951515, 0.31796134072751436, 0.42801319377856545, 0.2007692390205923, 0.46081676119460346, 0.5041182189325352, 0.16154801690297388, 0.4121869070403712, 0.5660654019823679, 0.4467582082249462, 0.3410471679232061, 0.15382996454100883]], [[0.48997706831392046, 0.4261902917190329, 0.44068843158394044, 0.9252934150506333, 0.6506656813610934, 0.5925714779284852, 0.4556721194757763, 0.6130044573155883, 0.7667739296493628, 0.547937947435698, 0.10409121611601163, 0.57876762230835, 0.9552046532457521, 0.7590277705938416, 0.32403398420458307, 0.5676477522287584, 0.5989711188601163, 0.6145928443616773, 0.9589146981356934, 0.1707008580953971, 0.23916432218606754, 0.22611141044657046, 3.4870886584447467, 0.34607111217085906, 0.3506817737065188, 0.40690405787051737, 0.2043005863823823, 0.5653542566224953, 0.45980546446629444, 0.08938474091837785, 0.42041036919116515, 0.5460853136852821, 0.45677080338179227, 0.36902366180444096, 0.1995517781337743]], [[0.4869996988011994, 0.417749228548751, 0.44300787611863457, 0.984981990822235, 0.636220966925385, 0.5961982505670543, 0.46953199352336467, 0.582356554095431, 0.8516002766437188, 0.4857064650989526, 0.08301406301625422, 0.5670080448177197, 0.9315214787040115, 0.7636056511605467, 0.3023916684729956, 0.49728228085489457, 0.5821867064654231, 0.6089847721945416, 0.9262644344809627, 0.10675002477329315, 0.22186524465620483, 0.2593994004747524, 3.4870886584447467, 0.30665375271597206, 0.40858260383032075, 0.40753773401898186, 0.2615199028596112, 0.6615664926933507, 0.4358179993755972, 0.12812552243378172, 0.49392982577339145, 0.564549585529355, 0.4722409093533592, 0.43601345595680235, 0.25161758213007923]], [[0.5296727619684374, 0.46402322953236286, 0.4400983987417826, 1.0522660919459312, 0.7025933849114396, 0.6103655179770366, 0.4187197764195505, 0.5713572265707761, 0.8481945475748058, 0.5242792966398316, 0.0576186956702328, 0.5604342520784183, 0.916766229196204, 0.6729848335947243, 0.3018269644361554, 0.4502673252642191, 0.48232246507426985, 0.5668342996464069, 0.93737636458177, 0.11349574328281492, 0.24390627923829183, 0.23804849788258256, 3.4870886584447467, 0.3039780660474789, 0.45166316415899427, 0.4463901216837079, 0.2552256013608274, 0.7014030522007971, 0.3883127709724429, 0.20367724555820818, 0.510521938080658, 0.5608317541634866, 0.4759843905898611, 0.5082261418071123, 0.3229859111627324]], [[0.576483534461347, 0.39529303167718866, 0.4535515413793319, 1.1207683110385713, 0.860261424437622, 0.6177814517230952, 0.4044712476947277, 0.5589222977444844, 0.8256218908628786, 0.5583528643689867, 0.03228390805777061, 0.5764067592777774, 0.8894437243023263, 0.6800825754299193, 0.32082162266002623, 0.4665145033196548, 0.4495061734802205, 0.5631617436968782, 0.8176118018249943, 0.1195201892703418, 0.2734161377825313, 0.2257683704426706, 3.4870886584447467, 0.27236564205271335, 0.49216865365213813, 0.46986669522116903, 0.25321760097322055, 0.7299578535350577, 0.33562151595451234, 0.20528529326602524, 0.5520168354376034, 0.5320480842266451, 0.5041659061870503, 0.5325623915561511, 0.4205155797041853]], [[0.5945475206395227, 0.40580899053914204, 0.4390505546583884, 1.2666774338605522, 0.9095695442564176, 0.633267889108176, 0.4334527479174407, 0.5501402974363164, 0.762489439400924, 0.5783014211669922, 0.052748867779534726, 0.5996481190920369, 0.8907827962295477, 0.6133365606638634, 0.30170846827277487, 0.47103901078908883, 0.4690927358818302, 0.5818660372190418, 0.8347588707854091, 0.1364464558018615, 0.2790591990410815, 0.24389343685761028, 3.4870886584447467, 0.28769570588267646, 0.5485389852172937, 0.4699201861777463, 0.2561751903753796, 0.7108414341025036, 0.32619284700949125, 0.37662295949926156, 0.55748194950955, 0.4658789851939871, 0.521746212231555, 0.5454133268528967, 0.4217124870040352]], [[0.6402164826086117, 0.39676694415270186, 0.4489054118890067, 1.3168342410853255, 0.8987024779294654, 0.6258045993896986, 0.43385592032425235, 0.586863355601603, 0.6942027500906894, 0.6914472725701185, 0.04797733701333427, 0.5123540012719581, 0.8852634713767864, 0.6089428574616851, 0.3230415461866163, 0.5282145182666211, 0.5238129525007285, 0.5846804270025147, 0.8694937801243661, 0.1529478824080505, 0.27763362645454304, 0.2246189920624008, 3.4870886584447467, 0.2782157444069978, 0.5577297358323978, 0.4454092261319805, 0.26657466678126124, 0.6814584093758993, 0.31105422108845315, 0.45251499460176553, 0.5707171912622624, 0.45175656517855306, 0.5280127479786471, 0.5939277445983928, 0.42658100029980894]], [[0.6656158831537589, 0.39296412240030754, 0.4437908871254055, 1.3617785467613912, 0.8575529324653932, 0.6237387872402134, 0.5092048528906847, 0.5545771991148778, 0.64313959906591, 0.7136527178323833, 0.012099262906631169, 0.524213227085095, 0.8810207235994127, 0.602675337477075, 0.4023026737113212, 0.5147729434148135, 0.558651514307268, 0.619033304330693, 0.9172540518383563, 0.14376053429927077, 0.3026102146035903, 0.22190615336526734, 3.4870886584447467, 0.2800748240755554, 0.5853944982023302, 0.4368213617470375, 0.2733585603386197, 0.6929296098076771, 0.33891368064201666, 0.5698327736748902, 0.5943628673554068, 0.4345937959538346, 0.5222656677513562, 0.6088512425215806, 0.40085585140318714]], [[0.6128068054549758, 0.4021764972283946, 0.4260242647213534, 1.3823585586091671, 0.5984545522716828, 0.6305327839088685, 0.4635195231404103, 0.47775532732848036, 0.6875063762268293, 0.7052423499027057, 0.005086153939402187, 0.5926320948683332, 0.8759502989239987, 0.6502741619816331, 0.41332055161964387, 0.4724479774642236, 0.5021757549143595, 0.6249540192679199, 0.8958696202901073, 0.14138303123780013, 0.295771394305273, 0.20368187670645885, 3.4870886584447467, 0.21985453408547517, 0.6185626727634319, 0.3919962231796619, 0.2852421315578605, 0.6886280507375729, 0.3466701851446339, 0.6184447577758578, 0.42428568521107984, 0.3819016161406848, 0.5198502661898252, 0.6351648368390326, 0.3556934882544782]], [[0.593419582660412, 0.4129125891831227, 0.42526597741539174, 1.3827014086170855, 0.5133567504337885, 0.6128604133580594, 0.4626234034288895, 0.46897384253169, 0.7503080963666029, 0.7496811946671613, 0.007617330056489699, 0.660315218947419, 0.8731179350163195, 0.6786851394128629, 0.47029836986041745, 0.5234067233241136, 0.4819814436340265, 0.6274753275727699, 0.8497699794676712, 0.14926523437549852, 0.3245509411353654, 0.2012399028304808, 3.4870886584447467, 0.18662096120854416, 0.6290206677593073, 0.3321201140804947, 0.2757444273681121, 0.6628187029062018, 0.3829887457810909, 0.6350349748443509, 0.3726206285961685, 0.4117785294494899, 0.5118770855161037, 0.62265749491224, 0.3463653337996445]], [[0.5692821363826729, 0.40375859517374213, 0.3955031980678103, 1.4888361178443068, 0.41544394469791956, 0.5974515009362893, 0.4806509250940931, 0.4506527192553068, 0.7460973686054488, 0.7382847693178437, 0.009271632947378072, 0.6542828730839999, 0.8534708484981852, 0.740815491883658, 0.4373531084929858, 0.49599762199086395, 0.49655524436830123, 0.5707383136790106, 0.7776715013793052, 0.14991056029003996, 0.3110082400413677, 0.21272344314648922, 3.4870886584447467, 0.14363223631128363, 0.5874690084418241, 0.2132439893107112, 0.2375238830806005, 0.5643349575126796, 0.43972292644590183, 0.7026239630925956, 0.3181439644248218, 0.46423361911417393, 0.5156851301814366, 0.611824450459506, 0.35986905016500614]], [[0.5992742509754989, 0.32997590905371443, 0.30860639712137256, 1.4286758956967216, 0.390210634369613, 0.5628010002219332, 0.4944884801095246, 0.518279980985855, 0.7107214087664105, 0.6978456060942024, 0.013705591207061149, 0.5954505341006002, 0.8247720598145779, 0.6216041147710988, 0.4360673892571604, 0.5135800108454827, 0.5142433738028468, 0.5639017442845766, 0.7735345315553018, 0.2043400628939065, 0.3146853048887196, 0.233747751907048, 3.4870886584447467, 0.12854426619795278, 0.5785630893982103, 0.14629328270274825, 0.2262040751222845, 0.4972095691140186, 0.48888089538027696, 0.7604295295612673, 0.2932901960828356, 0.5215388647061376, 0.5174314637659928, 0.6112400734781356, 0.3722057489309282]], [[0.5922748232611293, 0.30286889413891455, 0.27503601367291797, 1.388656896037074, 0.3978814975754891, 0.5901421422878229, 0.46324489126486557, 0.5544310815328987, 0.6730798672824188, 0.6507780420063973, 0.024861412807690813, 0.5719200039930493, 0.8525480012871143, 0.6734096993805538, 0.3532991808840621, 0.4924945445650236, 0.4500126243356931, 0.5454414589498795, 0.7750005042970931, 0.23949456324053717, 0.36005324133040056, 0.2227977770719895, 3.4870886584447467, 0.1373500767472975, 0.7127882572846997, 0.13029771955188613, 0.23865183741597096, 0.5561047468679506, 0.4843204974578432, 0.7568017147087449, 0.2670907213012531, 0.5964915447183372, 0.502775477328303, 0.6097207937417247, 0.38807011847313405]], [[0.6007467251527678, 0.3282184504250179, 0.3207311101402446, 1.4801475948131295, 0.4021824857836054, 0.6063272250185069, 0.41690338247235575, 0.553376507030289, 0.6343629229811529, 0.6381962991570727, 0.1615778283000935, 0.46822964865175576, 0.8875983772717904, 0.6029569339292872, 0.31098483224883744, 0.522794505337136, 0.4302932699892734, 0.532232653651834, 0.8228383744842656, 0.2358311527771832, 0.3561489577603493, 0.23908011478844543, 3.4870886584447467, 0.18296039940840936, 0.745244129920674, 0.12609601751466604, 0.23951275699387, 0.5414891066446544, 0.4713298517128873, 0.7601822899549272, 0.2462955043786228, 0.7078115030586636, 0.4708044564447425, 0.5826132439178964, 0.41910070149623013]], [[0.5712949527849005, 0.29121714905824736, 0.2651587854558881, 1.47741827122946, 0.4098307802158768, 0.6115544769580852, 0.3997208372890698, 0.5811905161838965, 0.5833566840588615, 0.6636057272174172, 0.16363973009725136, 0.4882560721097241, 0.8748421282712939, 0.6104268875872326, 0.3358847434305977, 0.5642326585225462, 0.42447229541776343, 0.5350637385001283, 0.8509985567490597, 0.25405383703832407, 0.2704212840112821, 0.2638060012524569, 3.4870886584447467, 0.20522448521533898, 0.7821227148600355, 0.13068073942938913, 0.2661383650040262, 0.5173568425140991, 0.4941957829179585, 0.7548211971757721, 0.2060291385703106, 0.7399346450519142, 0.47366341023718994, 0.5809903898827763, 0.4270694316999658]], [[0.5406144823929167, 0.25509548715662633, 0.23113780725457322, 1.5136616426195513, 0.4397265359663708, 0.6179542819850188, 0.39510669256297515, 0.5821340437057698, 0.5489375761006625, 0.6939075089438814, 0.2261154425402762, 0.4754477437052035, 0.8465043286540945, 0.5648840970010203, 0.3482438269007607, 0.5534306704590859, 0.42706798850295136, 0.5418649847738868, 0.9068833665361389, 0.270491004882912, 0.24076419747814654, 0.31004253340148213, 3.4870886584447467, 0.2065534252217721, 0.7753015936788459, 0.15478438523124777, 0.3077047251437483, 0.43729613711950105, 0.5372713995644218, 0.7328854290311749, 0.21417518907607275, 0.7723099280507773, 0.4683751485399233, 0.599216438737377, 0.41747963076199046]], [[0.5397281272138564, 0.24815234366067906, 0.25278322721739155, 1.456154716871442, 0.4359706489076667, 0.6486159613756176, 0.38633294912710636, 0.6002084542988831, 0.5246187904938568, 0.711919396915824, 0.26148265616162597, 0.4718879185094136, 0.825088579428408, 0.5521945553077834, 0.3317972764363251, 0.5408649710752833, 0.48765198075316263, 0.5463277989638335, 0.9380567881203994, 0.28935732863122376, 0.20726359236883296, 0.355898908738634, 3.4870886584447467, 0.19619581428739907, 0.7799196354414166, 0.16610833925739255, 0.32068543771067504, 0.4483529242648642, 0.5500149388559886, 0.7179867752348006, 0.27870458081752747, 0.7362519576484537, 0.4717920385371617, 0.6122119158386434, 0.4505174883120964]], [[0.5284824104488491, 0.2662173498442105, 0.26474433367100947, 1.3831835253389033, 0.4799610451210432, 0.6526651520438095, 0.38184489204827365, 0.6374592238383422, 0.5791908945844985, 0.7634471472624289, 0.29193480779827513, 0.4643260554281494, 0.8109392904201818, 0.5673108942469809, 0.33737287685441886, 0.560179374448996, 0.455191140598607, 0.5323513106070507, 0.9248921536812416, 0.2982057486739322, 0.19367599879617842, 0.3847105972183146, 3.4870886584447467, 0.19033406665957064, 0.7254538379005776, 0.1931481700224727, 0.3244353749101293, 0.4417114946266818, 0.5316625169319152, 0.7389379722329336, 0.4299622607529311, 0.7410689226218938, 0.47635896619047224, 0.6145462648183885, 0.4999931292756971]], [[0.5603554888830544, 0.281121228563214, 0.21698107870714012, 1.3564188828178325, 0.5468008931778646, 0.6440131394416896, 0.3800184573244265, 0.6828081912636752, 0.6475835675354769, 0.7658050116941763, 0.301076173189907, 0.41730319792683157, 0.7406670847385597, 0.567943003331834, 0.34666749120831175, 0.6038243195648285, 0.4471750715335605, 0.5372565759968644, 0.9443097513024767, 0.33883056763582947, 0.20631783787659208, 0.36773930893132056, 3.4870886584447467, 0.19812029426740352, 0.6487369925217872, 0.1966359944027, 0.3443882966239563, 0.40976975967978163, 0.5005135374080987, 0.7597230571882866, 0.5698316148661764, 0.755034768071897, 0.48282137787793167, 0.6004381672825696, 0.5326972984748691]], [[0.5619982194477284, 0.36972075191093057, 0.346482380693538, 1.3273799928303731, 0.5424583625918042, 0.6397380409148286, 0.38081211611043053, 0.6941991214228733, 0.698786297156851, 0.750931375604686, 0.3172016451549627, 0.4664110543844754, 0.6597335305781542, 0.5898865569332961, 0.3497107549051617, 0.579979272133714, 0.5357221588929179, 0.5650193286130019, 0.8950164292474997, 0.36447561695281666, 0.18402461050254948, 0.3959810479653013, 3.4870886584447467, 0.190211121867152, 0.6678373827263973, 0.21230973154649127, 0.3449330127884056, 0.38092171381027773, 0.4722772786645746, 0.77009947727889, 0.5751675986047822, 0.7487583823686867, 0.49012504948528385, 0.5927037471315991, 0.5564371660666958]], [[0.5757441267522765, 0.3920480068242491, 0.36271015152884756, 1.2765077699680116, 0.5250197789506471, 0.6368884639803396, 0.3756851799591151, 0.716399694141344, 0.7891002607325236, 0.7929867544121936, 0.24881054121438267, 0.46056752792214034, 0.6713092995021042, 0.5815335539342803, 0.3492985458125064, 0.5777786226618916, 0.532914661947268, 0.5524781942104838, 0.9535561240079407, 0.382336742167682, 0.17977961545968993, 0.3994708257608779, 3.4870886584447467, 0.1693820590903669, 0.6885828354891097, 0.2202233366061795, 0.3429217083263696, 0.38873532622352125, 0.4520995337930818, 0.7682670059646878, 0.5355835925198751, 0.7577169761724736, 0.4892815448472228, 0.575071413523803, 0.5736847139786643]], [[0.957841868248425, 0.4975868260020999, 0.3894578237766211, 1.3823905094765934, 0.7739912439582088, 0.636630116185501, 0.40833580318787366, 1.2189593254861946, 1.3016348596255773, 1.3306415649043206, 0.2688129114957465, 0.42239322231297016, 1.1909195940420148, 0.5690278072481452, 0.3482619085768808, 0.6602482461632336, 0.503311549791114, 0.5679772151558862, 1.1213980057211475, 0.44466520284980293, 0.1910416379750295, 0.5363338896367129, 0.1657705162609946, 0.25809708487299615, 0.8118867777925121, 0.31465249539203666, 0.5530099844034542, 0.4000500384201312, 0.6461647458660223, 1.0342257866026823, 0.9048476704003974, 1.320040943138249, 0.8505032195225635, 0.7185089298735228, 0.7922565261670844]], [[1.0103638341202459, 0.5432233258469824, 0.37190013641010167, 1.285881150623744, 0.7276159806998745, 0.6419295034489759, 0.3916499112125854, 1.2446262730547324, 1.7015177690272258, 1.381906301185876, 0.26459964001048275, 0.4059336022409922, 1.3289975098585065, 0.5906166146508015, 0.37137322721210414, 0.683086110495781, 0.5425604987194945, 0.5699494420631613, 1.1806484287292762, 0.4324025605929763, 0.22079062143046266, 0.5061177480040298, 0.07619989423111845, 0.23296114318333488, 0.8998697193710573, 0.377481286661548, 0.5412366451609734, 0.4758249915760797, 0.8255387257296168, 1.0407497035300093, 0.94061126084159, 1.321772966239156, 0.8371010874018947, 0.7342776219028389, 0.7692632165931732]], [[1.0560275490688276, 0.5270975047217956, 0.3827146369222983, 1.2830969772181868, 0.6728572408241256, 0.6667644509011061, 0.39113123010008727, 1.2386927349089454, 1.7495498574135584, 1.199478779605012, 0.237005595526521, 0.4386848098791467, 1.324472964105137, 0.5901224283965276, 0.39178435786072047, 0.6711234134209125, 0.5883138747430865, 0.5707883808523109, 1.0761218015717309, 0.4316840617168267, 0.22878064921761876, 0.5242913539827397, 0.059636910069719835, 0.20310063465075123, 0.9151844610583811, 0.36518962345962, 0.538847929165677, 0.49543300767553755, 0.8001796152373974, 1.0053534488664797, 0.9578489096469909, 1.2993053079751875, 0.8379802270276259, 0.6856805913751909, 0.7116552021532787]], [[1.115407942774206, 0.5131165950510133, 0.35288278149706886, 1.2646892100557294, 0.6824384944271144, 0.6355972276194406, 0.3877187946826929, 1.1750674060741244, 1.7932159459810284, 1.0004016358286267, 0.20115732498026817, 0.47905955554781043, 1.2793240237623917, 0.6184227392150838, 0.3702133738387147, 0.6369455235625145, 0.5939705100057722, 0.564389481126541, 0.9946681929429051, 0.4534392393963263, 0.2235439878839316, 0.5424905739915925, 0.06231990891442081, 0.2020684253040061, 0.905019687411663, 0.322466309100052, 0.5312263160086432, 0.5444083055121973, 0.7655245959816562, 0.9681211482820813, 0.9781358211135802, 1.2060989965131519, 0.8649599556092162, 0.6485499674357924, 0.6771840796772769]], [[1.148072728741912, 0.4893525357423175, 0.32529901057124716, 1.2332819766630299, 0.7012625946397183, 0.623512522246213, 0.38392618385431976, 1.0736984069271107, 1.7943086658805774, 0.8487066117402051, 0.21322573440098427, 0.48737411963234645, 1.210809031278195, 0.7336928371231415, 0.40450199950413873, 0.5771565791096686, 0.5886773788195585, 0.5337349651057702, 0.9942100468169308, 0.46735790704803126, 0.22239534502554195, 0.6147073544337385, 0.052764430177194566, 0.22688057496309227, 0.8559546728228895, 0.3698444943691217, 0.5482416394600307, 0.6020386227058591, 0.7358443111623405, 0.9241283664993786, 1.0572245359864443, 1.0619279844691554, 0.9442160565230677, 0.6709560487906674, 0.6973365776127881]], [[1.1870300839615373, 0.4761498107861911, 0.33709947515660793, 1.2450496018147512, 0.7132262430597216, 0.6906345574706159, 0.4158038786840772, 1.048680087950398, 1.7832744996630465, 0.8030822977433317, 0.2181704821735048, 0.4758749167259994, 1.1933156278964085, 0.6902011191063677, 0.42530553256333853, 0.5246101216923564, 0.651243555366296, 0.5629670479193296, 1.0615361270433583, 0.459647525857511, 0.20648125325612043, 0.6895089954306524, 0.04507317012338196, 0.2262051654652128, 0.8224773553630477, 0.4109625990317259, 0.5550024716832376, 0.6512253095127043, 0.8195308356209368, 0.8867574562465954, 1.1383544762703697, 0.9208861701584765, 0.9833937771765997, 0.7410752351765475, 0.69052180742666]], [[1.2337406854499977, 0.5284891690557623, 0.3947028315327695, 1.3382508621480762, 0.7262800647463953, 0.7054274004220771, 0.453277099641699, 1.0697899471163874, 1.7784326074578036, 0.80071112689711, 0.2282415718957155, 0.4628898008395446, 1.2105421523700817, 0.6620508670411602, 0.43759301386454, 0.5309390975016758, 0.6827024158278605, 0.5519470144840167, 1.0668603916111343, 0.40847886207945483, 0.21753114419441594, 0.6867114644167231, 0.04440139053419467, 0.24185491568089557, 0.7691370972193281, 0.40146847400975494, 0.5906676743976405, 0.6923156221574329, 0.8488119118721922, 0.848668832650162, 1.1920561594491188, 0.7988936092475363, 0.9670802744731495, 0.7950575537626416, 0.751975815055151]], [[1.2653225955422776, 0.5073331995909758, 0.3217846028816084, 1.4212106593476366, 0.7438157448626905, 0.6590700999030727, 0.40018931043577904, 1.125019689501513, 1.7082541548110273, 0.756693315831767, 0.1761035803562065, 0.46606800781878177, 1.21822134616431, 0.6354557481633398, 0.42140191612628347, 0.5173800104055566, 0.673296576278644, 0.5628040689162319, 1.068279837100007, 0.3735571960633681, 0.2693625295630315, 0.6886212549803921, 0.0434150646159191, 0.2864994534374474, 0.7057948246115514, 0.4111561763623654, 0.6072547742692781, 0.762156433019295, 0.8385645096374906, 0.835706581558687, 1.1946440500271747, 0.7052619494615235, 0.9194244222242447, 0.7704437596273324, 0.8665169942835]], [[1.2069099187671029, 0.5918353936235614, 0.3985189343661133, 1.5305492209677243, 0.8071212335273994, 0.7322186822594685, 0.45375208479538776, 1.1158791242820332, 1.5551211958554592, 0.7466606612345809, 0.18683683924182862, 0.4567944239550358, 1.2434288162486062, 0.5556581566888157, 0.4338067918858861, 0.4588717375194693, 0.6912332020201937, 0.599622623628267, 1.0511743799805977, 0.34835954214050957, 0.2757295353188203, 0.7869214605042418, 0.04617194817821511, 0.4373223343251751, 0.6856314037481912, 0.4582910219457929, 0.6168867111260479, 0.7592569317593219, 0.8539616082479002, 0.7825882894643341, 1.1025288757728833, 0.6428347521210385, 0.8945664147898846, 0.7900072980010153, 0.9491333664697441]], [[1.1774524687707226, 0.537833125748021, 0.3330858424530011, 1.519314570337484, 0.8371046109241659, 0.6373886426662916, 0.36532364823452973, 1.170065094683184, 1.4429440679032757, 0.8389360676711812, 0.19705363969042478, 0.4705503627968157, 1.309071095365716, 0.5206813713117415, 0.4311732442818875, 0.43576986164418074, 0.7170293209628044, 0.595869605667262, 1.024075617333202, 0.31029483125943313, 0.2878816390284729, 0.8600609262955345, 0.043508754609885354, 0.6083614915397482, 0.6955052060988156, 0.5077820818538348, 0.6607013133109391, 0.7813945693473402, 0.8827997532456374, 0.7940439977533673, 1.0135101849131125, 0.5897380020204939, 0.8769009927691405, 0.6851852424684975, 0.9364521439250917]], [[1.2353154259413146, 0.45503616327424723, 0.3702671415011673, 1.4831854222187746, 0.7913794450517633, 0.6429096061433484, 0.3737880373544098, 1.1827581634883317, 1.2688362436733676, 0.9473079255780003, 0.2023112040068927, 0.7451491845480375, 1.2938350983914653, 0.6299367286678088, 0.3391178364258051, 0.3677502245231637, 0.7615574761209614, 0.5561021849636929, 1.0009948798876334, 0.3174759564745064, 0.3099691721274942, 0.9748033008041432, 0.03692251941209858, 0.6261401584712173, 0.688952054438152, 0.5729434624157346, 0.6899940290857112, 0.7590995522716579, 0.8513368203742359, 0.8056048152134252, 0.9503130839700983, 0.5573642828430448, 0.9043050927959528, 0.6009759568335067, 0.8762799672333019]], [[1.1847650027925019, 0.5587995022079648, 0.4381233887661932, 1.645725640675641, 0.7394702417241161, 0.6651250548249217, 0.3866642256543413, 1.1696286718843214, 1.1619741419064864, 0.9318821853121086, 0.19490929091920442, 0.7420488644262534, 1.3189889997894653, 0.630800281586678, 0.29425321178342306, 0.3738238207626068, 0.7456167305057768, 0.5551387012312428, 0.9744582826231745, 0.3410551300675813, 0.346790799771042, 1.0048923921408905, 0.0409579194837274, 0.5106495013195416, 0.6843621710394028, 0.6667317466755891, 0.6844699554167295, 0.741926262838198, 0.8783593177568912, 0.8508442297870201, 0.884158709629203, 0.574914469903791, 0.9421825220612889, 0.6995977300767704, 0.8056276715484711]], [[1.0942070734417564, 0.6612690341267852, 0.4577035665349195, 1.7230968478966284, 0.7002930513458516, 0.6844754175564685, 0.4110788615421613, 1.1488003952402166, 1.1229553510578576, 0.950389322248168, 0.19003561264082164, 0.8561753553714044, 1.389066429399175, 0.6653688444186151, 0.30514496018083076, 0.3622325576484967, 0.8030096061326496, 0.5672092203581098, 0.9378515621209664, 0.3791944394900406, 0.3844737693252419, 0.9903988064590485, 0.04222255416304207, 0.3741542096359571, 0.7335815756099168, 0.6956459834718334, 0.6585965210617932, 0.6884280042396533, 0.9256808588304475, 0.9535296707855424, 0.8504253031222941, 0.6339778187021475, 0.9967208906323919, 0.8752489086112818, 0.7108326087882508]], [[1.1539805348182623, 0.5880143296098339, 0.36179226204395787, 1.7496027039123432, 0.6380871500106998, 0.6836406044312358, 0.43666190592499265, 1.1123717529473416, 1.1054581857073007, 1.039332263943464, 0.18798801762887896, 0.800347735439115, 1.4381598180562236, 0.670814610921739, 0.29679952777816804, 0.3727096496613625, 0.811856155090143, 0.5825456169878647, 0.978021002173046, 0.4168427716276646, 0.40581395709497864, 0.975981842231483, 0.04374706896063435, 0.2904204774425042, 0.7574322929227865, 0.6894377994697153, 0.6559488241181171, 0.6512029436739853, 1.0768679745458662, 0.9772986980907686, 0.8086587076654781, 0.6877005305782047, 1.0473139175338388, 1.0515267670644084, 0.6346860829124269]], [[1.2005563294796653, 0.6056493605231188, 0.3371703833557989, 1.9014336642295513, 0.5286395567606319, 0.6843924200652134, 0.42417452386594745, 0.965915145484692, 1.1317620495339855, 1.141170020096377, 0.16166804167167706, 0.816235476203893, 1.4347038889441608, 0.688937365728457, 0.31335561569472664, 0.3983909942050473, 0.7615013530721063, 0.5840487757929844, 1.0351114227847589, 0.35997805813904427, 0.4880358483779499, 0.9141640021636437, 0.0459728397351842, 0.23243101857013518, 0.7496840754707081, 0.6633535199463343, 0.6211128679948611, 0.6868420596100718, 1.2713182146832533, 1.0574645689177975, 0.746679797003681, 0.7185959346153628, 1.1326096402911485, 1.0962308404086718, 0.5310139310726327]], [[1.2327241714736537, 0.7635202907699375, 0.3692505244391556, 1.9097285684484997, 0.428326856711961, 0.6945924291686194, 0.42803968321450114, 0.8003832256202125, 1.247637725242317, 1.0471878586125685, 0.1256190269819339, 0.7761949835385241, 1.4420296160501036, 0.6924186436542851, 0.2707023925589497, 0.3915707858264018, 0.6334104340066908, 0.5441545559268918, 1.1023393910772357, 0.3344860534359984, 0.38728673203124786, 0.6205219667709239, 0.039902164091264306, 0.30527187432617764, 0.787560370450174, 0.3966173439960111, 0.5561966349812992, 0.6977421054507315, 1.1455793703950932, 0.9771518168007991, 0.6917335326614799, 0.6879969187670143, 1.1626556740347342, 1.1266341939121594, 0.4353034416566225]], [[1.2159851532516064, 0.7491186244549751, 0.3866954600303751, 1.8731893520400442, 0.3398072127693511, 0.7007525211583706, 0.3957848482946996, 0.7438071122097683, 1.2563235029395767, 1.0214243429137322, 0.1524487038906945, 0.7423363595690079, 1.4345939096275715, 0.6879777038466683, 0.2954707007665061, 0.38788718142502987, 0.5586162366048428, 0.5121855961503035, 1.127805587617318, 0.37629371596580413, 0.3086874738846258, 0.49050067307679657, 0.035987615326559745, 0.5074314167482292, 0.838815117104063, 0.3178547089656012, 0.4614606723686082, 0.6852563836589971, 1.0128444350933847, 0.9572923765383183, 0.6649303577931935, 0.6295085210711452, 1.1659130576352015, 1.140278138754303, 0.4266072416349632]], [[1.2323891973364112, 0.7458258148762844, 0.3811749458900928, 1.8639784502477263, 0.3535090910562769, 0.6553918673091655, 0.3802309998262208, 0.7081535530883485, 1.212921502673609, 1.0222275057212302, 0.15366155948879792, 0.882988042426131, 1.4123605976301081, 0.60921169631286, 0.30947526511987145, 0.37044824567606427, 0.5403979805221804, 0.4493909265647462, 1.066677736875218, 0.35017313957215157, 0.3196916931478375, 0.46247577467614487, 0.029922475616634407, 0.6709280100836544, 0.8146723319188582, 0.28387031362164916, 0.4264242783087944, 0.7325301797352382, 0.9643257661484375, 0.9180597779297169, 0.6387290458746663, 0.5351932912765054, 1.1084387766822061, 1.1431691013613245, 0.402014390494556]], [[1.2828874182453636, 0.7142892069964352, 0.3880076785438084, 1.8123223167190408, 0.33855970374098276, 0.6552109701424562, 0.37023489770549844, 0.6681197641057277, 1.230657761756581, 1.0435977101162792, 0.17088699806992708, 0.8505529638682237, 1.3582771315995883, 0.6201808752605209, 0.27754500102717544, 0.35762047429487515, 0.6577488748613327, 0.439962062201983, 1.0525290315451141, 0.3444690621400284, 0.30222875942812455, 0.4411001420154247, 0.030371352107998927, 0.799997270042121, 0.7724250226297493, 0.32066556860834616, 0.39520601651900256, 0.9201331765972399, 0.843653404352559, 0.8680969150238405, 0.7879237614420269, 0.40771795852102466, 0.9516051764736881, 1.1559796689569677, 0.5307866312788141]], [[1.2897525615404983, 0.6882856792693636, 0.36122109426734994, 1.8754642142777778, 0.34002626851502105, 0.6437663831911635, 0.37668440691233673, 0.6371301307686117, 1.1316983567957895, 1.1067299651621725, 0.20197035229851817, 0.8592816645615865, 1.3142479908357618, 0.6193441749643502, 0.281181830184907, 0.3401432881127848, 0.6634437728102687, 0.4613789114623812, 1.061635605343163, 0.37711039312095174, 0.29530082800914903, 0.42839028515924854, 0.046113228576661336, 0.9203138984292525, 0.7326208211403827, 0.278488677252856, 0.31906337883609004, 0.9254351098159922, 0.7501776080376459, 0.7971489618169807, 0.8341191447075, 0.2903675821262137, 0.7788876020917226, 1.146359225261003, 0.68273632233078]], [[1.171140222479728, 0.6162431586549648, 0.4587750458902936, 1.9828630370273448, 0.3515473084535008, 0.6752243909636062, 0.37903053600816117, 0.664476450832, 0.8569001400021221, 1.1424450681631184, 0.23763054171520753, 0.863032361347765, 1.2858433146507369, 0.6682013913827114, 0.28647668840286294, 0.37593483731617516, 0.677867092460849, 0.4733618908711559, 1.066560655421012, 0.3716953648780699, 0.2761673140025772, 0.41496960621360346, 0.04665875854723109, 0.9834548090764854, 0.716273451480381, 0.2366028624636335, 0.22816270828457158, 0.9125309938615674, 0.7029931615499585, 0.7878774048056045, 0.8124737150188057, 0.2179684466380101, 0.6525194995330513, 1.13491005224884, 0.7557384253887234]], [[1.1053005719649023, 0.59483732300163, 0.43138279059163986, 1.941837597448961, 0.3515175432905764, 0.6827454021799955, 0.37717016881653503, 0.7629008153749806, 0.8362268736663767, 1.112578787481655, 0.24519933605548822, 0.8618609210407753, 1.3123053550125314, 0.6695019403150628, 0.2760276383494224, 0.34637023537290973, 0.7412333171549375, 0.49006305208586953, 1.0947469824254004, 0.33533325614203185, 0.14510457311668182, 0.39832369220602554, 0.050992853025031246, 1.0204121417072758, 0.7665198943191929, 0.196687504359482, 0.20408741144077552, 0.8573440119442952, 0.687270282223209, 0.830294829269186, 0.7649939187021042, 0.2104077316275681, 0.5796240515011846, 1.124402028206999, 0.9092883180394549]], [[1.029409559395641, 0.6464515903881625, 0.5514694462701877, 1.9489728498936971, 0.35550147792048925, 0.6757096081709106, 0.3780148222448968, 0.7285057597734224, 0.8574796545456085, 1.0316989905943816, 0.22371857469498035, 0.8776060166936938, 1.383760018170602, 0.6732242329741073, 0.2821383381524141, 0.2944214264118976, 0.6265873457183622, 0.5089940263462727, 1.0117961536372981, 0.3091161399308265, 0.17777507058463055, 0.3170324593638529, 0.05144212462155837, 0.9562915721311047, 0.8931115693667591, 0.2286249960662902, 0.31231220340521154, 0.9591620777836162, 0.7157534346158525, 0.8240695143409813, 0.6712996600607116, 0.27678919231234544, 0.5615543159677017, 1.1398350854154164, 0.8439407588533304]], [[0.9736308016281379, 0.5556182225549469, 0.4350820446241138, 1.7794678322348199, 0.358766567149985, 0.6788923347606965, 0.3865960202640078, 0.6274045038246856, 0.8147604798438368, 0.9482821203767969, 0.2645077130584912, 0.9437180010822185, 1.4364579572755405, 0.6587922448134338, 0.28099925380960583, 0.26375532178278593, 0.624992999211702, 0.4921481947303143, 1.065009314759716, 0.266111807898167, 0.22081249865928232, 0.2862000257859074, 0.05144997305541201, 0.8698400202714005, 1.0176804312274037, 0.26838968562234294, 0.36725293064731135, 0.9976011839866648, 0.7763786848118419, 0.8504376996396932, 0.6079400014294507, 0.34378173895611625, 0.6931867595328589, 1.0871521944428473, 0.8316083925886039]], [[0.9440087706415723, 0.4915790475791164, 0.3516839824386172, 1.6648020619563935, 0.34216516867390645, 0.6845296455175182, 0.38208800145050104, 0.5508869489526862, 0.8365507963391194, 0.8212437214090244, 0.22832841238827778, 0.9694503251828064, 1.484496577465301, 0.6666853804935393, 0.3133881183368316, 0.24960716342732675, 0.5974762245477601, 0.475489868647736, 0.9320703815439303, 0.23090473480686766, 0.29994186153579494, 0.2717052075825224, 0.04903758330784549, 0.8659185358425534, 1.14410562570313, 0.2998009868431034, 0.40139093452988495, 0.9063691702820891, 0.7736369570714857, 0.7842104052234946, 0.6393040020317305, 0.45126989792247835, 0.7489089805292547, 1.0039747484863044, 0.8378898585755139]], [[0.7398501933692537, 0.5294658301665909, 0.4508589791763094, 1.4026934050201116, 0.3023166569517885, 0.6645736157699194, 0.38238170495890644, 0.5318997178903473, 0.8484255318669184, 0.8876708035974565, 0.16543352799089656, 1.080177726789193, 1.490214005088181, 0.6080909038229418, 0.30889949716329096, 0.22778789905931834, 0.4538707068024439, 0.47649031732521985, 0.8546022559150468, 0.22055609307931656, 0.3438372182120941, 0.3236887496316574, 0.05073273600654121, 0.8909433176638832, 1.2047664354130994, 0.4475489412090146, 0.44479676255450945, 0.8137574865560635, 0.7881562160991145, 0.7335169364258378, 0.6481891525379093, 0.5495663338631622, 0.7894753192644556, 0.9495415162047666, 0.9346644248815796]], [[0.7633823791688158, 0.5370864234123981, 0.5783913764923739, 1.3400829841995785, 0.369539881111149, 0.6612013745211077, 0.3816744863434244, 0.599107541450419, 0.8931401726646941, 1.1135806489854945, 0.12012837630153056, 1.1427807781834662, 1.4941539572024873, 0.6045901892734789, 0.31066174835423727, 0.2344617758888855, 0.53744645832314, 0.44653158295043804, 0.9467617785702438, 0.19912133205475024, 0.37813531271404993, 0.4999513899262054, 0.13370552405739838, 0.9027443217677342, 1.1918421797165628, 0.4038779608323222, 0.482377107964136, 0.7199712359780327, 0.7182094919075017, 0.6511512436795401, 0.6446718615342655, 0.608114611146336, 0.8021284250031283, 0.9323823029709222, 1.0556489256846726]], [[0.7234695213790412, 0.4925402529912373, 0.5236551615271606, 1.3048909827113988, 0.3429887517957013, 0.6348076841177384, 0.3793829820563945, 0.6285279994914182, 0.8295915136505018, 1.1226810300328829, 0.016944925849462153, 1.0242081452539264, 1.5378647922838546, 0.5922478564200123, 0.3267377464091922, 0.25598925509470427, 0.4754419824553943, 0.43486419840409196, 0.8105780570030375, 0.23324056540006124, 0.42693426208064356, 0.7002268125505997, 0.08826795479229155, 0.9185170855625946, 1.1403132972480847, 0.29652894989544326, 0.4567118570156816, 0.4619806751937545, 0.6642863121091173, 0.5265297210290767, 0.6750228732549886, 0.7132049636735217, 0.8282111970752719, 0.9430915474546173, 1.046828424963448]], [[0.7489080593334106, 0.4778680275279191, 0.5381450302089918, 1.4499168388388377, 0.3560763651544383, 0.624192931158077, 0.398255550587472, 0.546643529882537, 0.8445774317126941, 1.0984240855657887, 0.03930891284640282, 1.0518910950698497, 1.5245635329944134, 0.6005554915529692, 0.34781002647099307, 0.22918308045054087, 0.4230413010893447, 0.4331709833809285, 0.8598622658943819, 0.2379597719030358, 0.4487197204968791, 0.817276817727553, 0.07772700376202574, 0.9225684401659195, 1.0227897277010332, 0.34944207096642277, 0.42947451368442474, 0.36042739718229283, 0.5719552828722607, 0.48450645412144755, 0.6284144921979544, 0.7057391409318076, 0.8239810597786422, 0.908697556473164, 1.0610381510884843]], [[0.7399636016129936, 0.4659920701774227, 0.5122933187439312, 1.4896662541669188, 0.37781668518193934, 0.6224822445127045, 0.39739686191895573, 0.5706419894839461, 0.9431766947002138, 1.0308298803442937, 0.043166008829667334, 1.061390081218175, 1.5455645011884376, 0.655586008491277, 0.3598753180082522, 0.24759112140154543, 0.41886420342260233, 0.4559022303497125, 0.8031024227862793, 0.3026920905014813, 0.4233743254561324, 0.9939941992179988, 0.06806958128218976, 0.9938426966642527, 0.8335846813833395, 0.4444888677633844, 0.4431895421492977, 0.42930722497433854, 0.6356730234312803, 0.6046799526311375, 0.6056017428578473, 0.7191109139296205, 0.838711142625002, 0.8668752202994103, 1.1215151726993065]], [[0.8375745917293114, 0.47292124655037643, 0.5332308726667874, 1.5357718534903333, 0.38079724431661865, 0.6489664374668684, 0.40764834436433284, 0.5699117750721501, 1.0070029089461006, 0.9181686808601304, 0.04322715506847089, 1.0186190840497926, 1.54522010910707, 0.6675443979722774, 0.3629926870350935, 0.2546375758793879, 0.35983133388508026, 0.47040190013505145, 0.7000787362869014, 0.3495332533724799, 0.4201483442016989, 1.103151624959011, 0.07243509033511855, 1.0868979106155245, 0.6911528968084301, 0.5272231567316217, 0.5005089562129417, 0.45384637229578917, 0.7191047580200101, 0.6691459056094363, 0.6042230408823643, 0.7454610870145555, 0.8868640090114697, 0.7760554557812187, 1.1655936716716364]], [[0.8898085817212007, 0.4788179043003946, 0.5489227659621144, 1.5164657089561613, 0.3713196596595506, 0.6569135700281907, 0.4124835490856331, 0.5900849694489503, 1.1152313379836365, 0.7281395330929935, 0.047676783752327395, 1.0571975801565316, 1.512449636041285, 0.7145180015581354, 0.3309618944182401, 0.2481799417503628, 0.37860166326745215, 0.45950289787978976, 0.7640936734941997, 0.32612977399255044, 0.4276467182629861, 1.1822471585177885, 0.06334836004937797, 1.0058574087629584, 0.5902448518781833, 0.586142171556999, 0.47845323075791135, 0.5036137903046427, 0.8192142143259837, 0.6313990609732305, 0.580205411476086, 0.8272308911824213, 0.8901374475678022, 0.6471471825276625, 0.878963385297399]], [[0.9112934389258414, 0.4562074444426497, 0.5559160387363378, 1.6374747013904025, 0.3531722146937964, 0.6484788921679261, 0.3979883726313828, 0.6615420655964049, 1.1763250696711745, 0.7133213272300201, 0.024721092668873283, 0.9295071578826891, 1.5343669905570108, 0.6966738760023498, 0.3083593373776082, 0.25012468784154784, 0.40452573058961305, 0.4924304563323181, 0.8181983246530427, 0.2846349921663075, 0.4456262174512603, 1.0952635842600884, 0.06334646542525464, 0.8981099352991895, 0.4861569909997224, 0.4729728004529673, 0.4507578809956003, 0.6009744214282995, 0.8333944967045597, 0.6174323484944835, 0.591082096247502, 0.829852869042712, 0.9021862705217362, 0.5956732520171736, 0.7675308382143614]], [[0.9268244701289389, 0.4306563425724693, 0.5389174074200581, 1.696191686070471, 0.33396904224029395, 0.6141787975189548, 0.3932328177222779, 0.7466883609464567, 1.1731471020727773, 0.591669956184533, 0.01040287946546987, 0.9214263039836041, 1.5616792513513476, 0.7238941617787552, 0.3146024790192393, 0.318629619525495, 0.5150328332351234, 0.5456684087264454, 0.8123254400511395, 0.27952017548900576, 0.4589096067079105, 1.0797872221457279, 0.06287874325273464, 0.8771751508012231, 0.4445316128284962, 0.4755069703256852, 0.5057584319270924, 0.5848097117185023, 0.8539646794789969, 0.6012791743659862, 0.6858253178302632, 0.8556603297615706, 0.9032107659702902, 0.5553858396938137, 0.7063977797022389]], [[0.9533420542065552, 0.42408016107612934, 0.5002045189199414, 1.6130146348635939, 0.39462710123661127, 0.6273215990293233, 0.39827806477096855, 0.8524839075429319, 1.1773924777247236, 0.4970622774868414, 0.007265944778266028, 1.0160298794044516, 1.5878311625843415, 0.5901622989686082, 0.34875754524011837, 0.3171783483956965, 0.4854908360975999, 0.6135797184454705, 0.809250725596077, 0.28367601875623893, 0.46883296029461846, 1.0124014536900297, 0.0624731006031494, 0.7507554885489389, 0.4817088689903536, 0.4950922599649998, 0.47494322237926045, 0.5715561651091902, 0.8274069383308801, 0.6205675613471878, 0.7211923725672731, 0.8738242414667433, 0.858647517073732, 0.4787813788973598, 0.622743829131431]], [[1.0092655529719226, 0.4133900331038486, 0.4825321093725995, 1.6207251322687553, 0.3484729215305674, 0.6227016006289212, 0.412789797302834, 0.8844786631276218, 1.164617426678991, 0.42651062325873595, 0.012061849765836763, 1.0508062859235507, 1.5720490219210372, 0.5519749752066451, 0.3508515456864928, 0.2929274492362546, 0.3949333325007842, 0.5775897536350478, 0.79290638827117, 0.28392248060580594, 0.4032824029600288, 0.9490170980821457, 0.06299122162958926, 0.6622141191810147, 0.4270051007487792, 0.5262255176868473, 0.4159129806730089, 0.5624107032955744, 0.7709805782307293, 0.6288526446099652, 0.7323130137093313, 0.8823297376568875, 0.7991683622462018, 0.49271014314703837, 0.5420930695027943]], [[1.0985562419330461, 0.42739418792549383, 0.4887372921336386, 1.5220506340470565, 0.3100221803495026, 0.6419644238257118, 0.4086639484133151, 0.8179936027716489, 1.0703351517322206, 0.442452294983983, 0.05741437931824199, 1.0421004506301252, 1.5570996988167956, 0.5563611193951492, 0.310428480500474, 0.2947775854171279, 0.41773750994659375, 0.6192662465946986, 0.8441273250892606, 0.3096907571130997, 0.35700267910694944, 0.8528860955624227, 0.0592374669530599, 0.6162668335685555, 0.3419367942842528, 0.4088655124842444, 0.4401016786920561, 0.5762858361359574, 0.7896157795220557, 0.6048183591643802, 0.6994961660856869, 0.8779773973643452, 0.7782207099574767, 0.47979197514095767, 0.47691984520336306]], [[0.9040479811450401, 0.5164667521854789, 0.48883450948164253, 1.4128108484462376, 0.34230262770606573, 0.6513128301102926, 0.4530384616479826, 0.7338383767657539, 1.0700415913393888, 0.40519807208505965, 0.07390397495656034, 1.062818246435866, 1.5577438897947882, 0.5308141729912101, 0.3076087212052909, 0.2864411128304921, 0.44713344469776706, 0.5962470831534123, 0.7583598298949008, 0.32710759153480407, 0.3664646095760518, 0.8298326248392169, 0.05697577243118458, 0.5604138814186196, 0.32466705281993, 0.15568613447277602, 0.5007017663617824, 0.6025694800079852, 0.6517940867007763, 0.5340081322567551, 0.5766845909667606, 0.8734157579082236, 0.7760731219879953, 0.41286857940171917, 0.3870976445352463]], [[0.8579242517579766, 0.5047079236028782, 0.48349467868937723, 1.4189727874081481, 0.3755557383400485, 0.6443042510904371, 0.4628446409739655, 0.6023895673126991, 1.0264303426442087, 0.46197808632861337, 0.07745145851776751, 1.0690534147507336, 1.588431735275785, 0.5972115904172669, 0.3129491299941931, 0.30237654947036774, 0.4194883331134597, 0.6179318407077556, 0.7712868005090312, 0.36832996147918773, 0.3676651237302751, 0.7903388200951054, 0.058158998470004206, 0.45354348869683214, 0.37537403764469207, 0.0954527718148594, 0.5053252038784206, 0.5835960170536526, 0.4343482582302314, 0.4928448370409854, 0.42169168642594845, 0.885743029442962, 0.7896371196904872, 0.3841905092810353, 0.31192784523200745]], [[0.9087985608563742, 0.48428790311536485, 0.4480233899490932, 1.1512676462660003, 0.38078129964141894, 0.723999441676199, 0.5182937763500326, 0.5484024010843436, 0.964947907436304, 0.45283472520341406, 0.09920977334319342, 1.0929292928832197, 1.6280337561579885, 0.6174641356011741, 0.32981010899229113, 0.3326188182839037, 0.45949755798469094, 0.6242647311476164, 0.8141411623498687, 0.5192867005938241, 0.4081712714240401, 0.7589655631797683, 0.05947392913063303, 0.4219827172732423, 0.4837613589235352, 0.07115235912231566, 0.5025727371946226, 0.5113552969004389, 0.23802484372030686, 0.4543474980913788, 0.2519396790217958, 0.9406092551367666, 0.7844085966682188, 0.3813593611972581, 0.3228918315083482]], [[0.9522797369119593, 0.5329658838323703, 0.415860494743143, 0.9785549166622268, 0.3781497176678872, 0.7463930693406412, 0.5169608972022045, 0.4728969214858613, 0.9790669570579658, 0.4459222803860416, 0.09190383667109893, 0.9283922149232722, 1.5133660569817493, 0.616697169306601, 0.35941214143219835, 0.2927757497393483, 0.41664052700740584, 0.6064713215557512, 0.7924770087150549, 0.5316968891587874, 0.39387392667651555, 0.7186119966829804, 0.05534245350137265, 0.31056724468625363, 0.5899022043082605, 0.060279597015470365, 0.525671304911753, 0.5026377865627427, 0.1398770751497571, 0.37764632977050283, 0.24496149682988505, 1.0100718259922914, 0.745968614426979, 0.37400011696835184, 0.3780689092757875]], [[0.9722182109639395, 0.6241984586551884, 0.44414357702585894, 0.7109351064249063, 0.3490113495803589, 0.751457329584059, 0.4775609300258994, 0.5181352084221675, 1.0477202002546342, 0.47364109386487613, 0.07310237716022298, 0.9324446496114298, 1.478177235426002, 0.6370976067040862, 0.35658762841382985, 0.2688956106466487, 0.4454393168263815, 0.6037043092632435, 0.8884387609306992, 0.40845279559875813, 0.4206958677276845, 0.6737929104736478, 0.06073029168693083, 0.3289752529052099, 0.6184040515670997, 0.05936456904902593, 0.5528529013643568, 0.4629783970654461, 0.10084249449423816, 0.33941430745341483, 0.3317489052385792, 1.0394887808187834, 0.668300032087571, 0.4127852074395466, 0.4096482126607389]], [[0.970999617635088, 0.6547613269175725, 0.47267660539572764, 0.6537266736315694, 0.3818905722833881, 0.767591269382669, 0.4769128260944454, 0.5007687532792914, 1.1944342649963988, 0.5527152913952125, 0.1264703367793853, 0.9391647915993877, 1.4327375388047368, 0.620062721053608, 0.3519082435509636, 0.2790108156358403, 0.4978964384118808, 0.6114310823080051, 0.9271892683143457, 0.48027777995928106, 0.38049142260056934, 0.6120500671957585, 0.05808779466097217, 0.36534045817185756, 0.6382739448548014, 0.05629470494246358, 0.612638696986306, 0.41146862194691414, 0.20277298034359573, 0.46042294998125544, 0.4857959273492576, 1.0549863339062016, 0.6006823795990232, 0.3608560124583297, 0.46318359599103875]], [[0.966435741923412, 0.6865175965748254, 0.4089113776685007, 0.6208096674265717, 0.39227624676449246, 0.7301421776361053, 0.4486195496703461, 0.6359966481523996, 1.1390982975522284, 0.6315270020752366, 0.11157785406119944, 0.9364768748323474, 1.4830226814490943, 0.661870789455935, 0.3112744949938794, 0.2879944884220629, 0.39891399365578056, 0.5841425091852532, 0.8785918324593576, 0.7722385866827972, 0.5471954309281948, 0.6588851544412153, 0.0798941165405527, 0.40611889872856144, 0.6350216935066758, 0.06984613493573982, 0.7345335549929634, 0.2687658241030043, 0.2851432325174743, 0.6000075290458395, 0.592801696935081, 1.0427074936659093, 0.48790673969049847, 0.33867540820316555, 0.5798529685088307]], [[0.949729098332101, 0.7068132766139623, 0.36902789409951076, 0.5756652443122297, 0.3772618752265476, 0.705340362661659, 0.4763995061648984, 0.7864652898870421, 1.0402455075213606, 0.6947816082858818, 0.07527203354576592, 0.9589101040269425, 1.4800034271865317, 0.6234759851504191, 0.31564422942956216, 0.28332849371704405, 0.37083997095675114, 0.6129743202650586, 0.8019893196539698, 0.8208561100052072, 0.5572327659174947, 0.6744003150659272, 0.08093194649240457, 0.47903558039157657, 0.487023739348259, 0.12818595372983327, 0.8226070983226096, 0.29086103031671506, 0.43845412868671174, 0.6966333342988207, 0.6197065385300785, 1.0431262015953155, 0.47256001099361855, 0.4358141913791449, 0.7013930855394863]], [[0.9451714549224709, 0.6946181551282069, 0.3441319382135243, 0.5698623668847516, 0.3905529606861465, 0.7098205464134657, 0.4832092248108718, 0.834049195985381, 0.9184242169606449, 0.6003121445002862, 0.07062940694507357, 0.9474272134559356, 1.4013439225893825, 0.6118092677178644, 0.3578265993834636, 0.2571083179544754, 0.38927616558270317, 0.5930528719306234, 0.7748410837963193, 0.7179123323862372, 0.5378932885728852, 0.5714839360907369, 0.08007609419556397, 0.5544300068123945, 0.41329743232436833, 0.17114894447515994, 0.8652552912692231, 0.294510386233791, 0.5183485625122948, 0.3222128305899569, 0.5402991861970605, 1.0562375620096112, 0.624198459747431, 0.5978864439276921, 0.7485759578456849]], [[0.9456127411710474, 0.8004825415318098, 0.3318372387783023, 0.5571845174695109, 0.3769926238430123, 0.7296128297952206, 0.45824196059017425, 0.866795146081547, 0.789327413051524, 0.6648266658617543, 0.07986312902459154, 0.990057436708754, 1.4952899484368878, 0.6003294016802314, 0.3597095591249226, 0.278287206475912, 0.37610152217076764, 0.5918700155989111, 0.8267095215509029, 0.6855324458985486, 0.5033202587607267, 0.5124735793464036, 0.08073019212759872, 0.6371139562410435, 0.3450971121219212, 0.2400861651757199, 0.8766207165545175, 0.28844260962677987, 0.5714871142385736, 0.31399366228563136, 0.4539952605158001, 1.0155156175811866, 0.8222726357727481, 0.6935049949972836, 0.6885672625727084]], [[0.9532900733866748, 0.8474224641958387, 0.32685847606553387, 0.5784817172477911, 0.4946567580254772, 0.7164252117846934, 0.38432361698896494, 0.9098114569592481, 0.7117378045525751, 0.7238103675952829, 0.12621180067557963, 0.6122216573406085, 1.683387759965043, 0.7666460601555416, 0.32236120437765764, 0.28425549495235547, 0.3550996691173116, 0.5563226232103232, 0.8979858317306598, 0.7215448791700827, 0.4804571536308717, 0.4489829406596933, 0.08245956631804288, 0.6969265998334997, 0.3379381336395451, 0.3121032079647865, 0.8576281895449884, 0.25284724619283905, 0.6826538821009129, 0.3167710179904277, 0.4244137131908547, 0.9233154921944963, 0.9610752237707054, 0.7733759559267921, 0.6101906469833893]], [[0.9913671109621996, 0.9794566028236651, 0.32423485827618237, 0.5624345597903141, 0.5993897824822867, 0.7354975593045973, 0.36246997682789844, 0.8952374218090191, 0.652533433438407, 0.8145692586811613, 0.12777947246615382, 0.6043674168810801, 1.7561434982071527, 0.718243992587678, 0.33123860496938073, 0.2969879394521096, 0.34038989322015856, 0.5368959885716001, 0.8700156261022345, 0.793238065912607, 0.5020209566777544, 0.4575377011303277, 0.08313573296457868, 0.7414453319183442, 0.3417760645677758, 0.3575266082016796, 0.8676877833460248, 0.23074790442356502, 0.7697523096086591, 0.37704356578382603, 0.4457279417035743, 0.8321116864153676, 0.9960837848058992, 0.8352478372085048, 0.4835154537070605]], [[1.0447626735598023, 1.1479458419936395, 0.33788874730805646, 0.58945859521862, 0.690446968392568, 0.7440496168699413, 0.3663637762485071, 0.8915728077929277, 0.5708074689446193, 0.7415857796392304, 0.0955782703351277, 0.5814687224787638, 1.6583772580922995, 0.7204040549338896, 0.36353358733100183, 0.2984399677849149, 0.3161198403814348, 0.5483921032016675, 0.8216921396739016, 0.8356211408398659, 0.49500042359017177, 0.46426575948892035, 0.0688128412855123, 0.7586105897314959, 0.34510588487238597, 0.39112109586189114, 0.8582449687565912, 0.234618126931753, 0.8380551403130443, 0.4123567899042172, 0.5166093022331377, 0.8377074400546464, 1.0353444373277156, 0.8949467731475309, 0.3932396462992532]], [[1.0454414877304448, 1.1240268491952616, 0.31833189412463786, 0.6088370448711321, 0.7550507603113472, 0.7497081665304646, 0.38854842379635557, 0.9571709280123798, 0.5114933794144156, 0.6594353193070175, 0.1372848221224117, 0.6054471279768696, 1.625809538094546, 0.6955072351062472, 0.31146220257507085, 0.30111646924376984, 0.30145124051471117, 0.5372997750133051, 0.8156560805284918, 0.9090708808059935, 0.5175566892136225, 0.46527733495943124, 0.06056615245523852, 0.7575862995217936, 0.3159318158464125, 0.43282181649686224, 0.7790258521372067, 0.22058431181366545, 0.8632433927253514, 0.42644607270523305, 0.6049412970808796, 0.9375068758165728, 1.06757017744614, 0.8668948134289112, 0.3453508022301004]], [[1.0463649585501558, 1.1762244292515516, 0.3043392629951666, 0.7020438242877058, 0.7750176995249024, 0.7493676439991435, 0.3996587048852796, 1.0292508103657945, 0.5338862090368904, 0.5762308242910461, 0.1508565143724906, 0.6107566479747462, 1.551947629878101, 0.6817397372670987, 0.2985951295153378, 0.30731403659169043, 0.29538291621321666, 0.5933659031657854, 0.7976229107293775, 1.0069242841799686, 0.5701589349383711, 0.5106175030262131, 0.05980976289913654, 0.6841039074410142, 0.27207635020430804, 0.4489100106264301, 0.6960858647818624, 0.2239430260781027, 0.8301426615890668, 0.4372793682831645, 0.6766964811868856, 1.08300986221199, 0.978126553787009, 0.8254362697513014, 0.2989174195631096]], [[1.0526720025202945, 1.398655067058642, 0.31894403666374166, 1.066774576501691, 0.8318240385260169, 0.6600788573336518, 0.30556078748525684, 1.1333656094417262, 0.6721711529291219, 0.4665232634943683, 0.1732851113305851, 0.5995090293069716, 1.434602907305059, 0.729342165328885, 0.2965096373840447, 0.31270207953063145, 0.2935474935470083, 0.5938973127133205, 0.7438763868807022, 0.9856660468783955, 0.5687506471291064, 0.5240951741070458, 0.06774992522558053, 0.574916977794349, 0.23339255489458866, 0.4660080901746658, 0.5758735651816184, 0.20695111196908247, 0.812200102026464, 0.46963519698970835, 0.722801839900642, 1.2087000016058116, 0.9074455362193978, 0.8990830506375197, 0.25628508211797046]], [[0.972268246659382, 1.3322228622518306, 0.3199945498492596, 1.0888252474516857, 0.8934931076518411, 0.6503223451193669, 0.3142771952208837, 1.1503721254841115, 0.7122497288243317, 0.38676927068162714, 0.17603254356889558, 0.5923979843782223, 1.4275115835992147, 0.6844668320599304, 0.29779823066564737, 0.3058353148479564, 0.30208812328174484, 0.5707035646942579, 0.7438065952987936, 1.036321150942694, 0.5709988368930505, 0.5936727550284844, 0.07626551885459265, 0.5558313418865142, 0.19619141868496903, 0.47824166368038323, 0.4845424980907219, 0.21626073064120757, 0.7978064468014413, 0.5058773212019065, 0.7442150854550857, 1.277208116307166, 0.8114475904071492, 1.0090949617728093, 0.2340981287561829]], [[0.9701503561265792, 1.18267607895588, 0.2939018217425658, 1.1456883730976057, 0.9815195952094408, 0.6686618233992532, 0.32745723856048986, 1.1860738445242465, 0.7509089698741803, 0.30907549796609857, 0.2061084749836167, 0.5430018257776976, 1.4357923696002224, 0.7123365799930577, 0.29556130337291653, 0.29809331095951985, 0.31358199252061575, 0.5606209971747584, 0.814937893682937, 0.9182686623451719, 0.47593944994670073, 0.5246165983120237, 0.08125284286314866, 0.6020116330515102, 0.27285496927471453, 0.48639095025330015, 0.42611164231695176, 0.2896734321083812, 0.709235796296069, 0.545826858498214, 0.7568876483904052, 1.248229166763243, 0.7610784581486392, 1.0772963616903246, 0.2625163654259681]], [[0.9403919012620496, 1.0652954420639384, 0.29005120180898686, 1.1381904493470076, 1.0365172311520146, 0.7572886377315999, 0.35524251381377964, 1.2224545417747916, 0.7532840992942677, 0.2456806950284932, 0.22600988315375514, 0.5693482714581984, 1.4281119300278182, 0.6886374468477978, 0.28405587113321346, 0.2852435997940376, 0.3023702281609426, 0.5413459316936063, 0.8093576141714058, 0.835116802146822, 0.448393367363055, 0.46508023596718995, 0.11215194255783892, 0.5747812318116938, 0.3011334056216246, 0.4891511404391051, 0.3956126205998021, 0.31008380897173393, 0.6047243638391002, 0.594889973682575, 0.7950829422932759, 1.0552761137593634, 0.7347611584993041, 0.8466064080176398, 0.32685743094778336]], [[0.7591584262695784, 0.9200274533149845, 0.267827652596961, 0.9587346444220838, 0.6628077334154152, 0.7772390719380033, 0.38879640128740633, 1.06886156985807, 0.5154648932390815, 0.16511462296986168, 0.2141440045110985, 0.5510771232749918, 0.7828152547993406, 0.6757973168063345, 0.2787556043710374, 0.3336767026767469, 0.3313856095324694, 0.5586376396541033, 0.73722436961718, 0.7036301082374465, 0.41510655471051755, 0.43142416404673534, 3.4870886584447467, 0.40212618447561566, 0.21997164294931434, 0.3276498480381688, 0.35722696650929087, 0.2726359591250641, 0.3692117271351019, 0.5322771105229449, 0.699079093138212, 0.6358210986799258, 0.4752219269610792, 0.44234453716005817, 0.2974596225637033]], [[0.7510498945648851, 0.9326570353249807, 0.27708664878009276, 0.8760671402126259, 0.7241152637425885, 0.7985587754279345, 0.40542784358472567, 1.0508667378837673, 0.5078311966424434, 0.21753121997445818, 0.24931469822889074, 0.5713702584622993, 0.7581545879010324, 0.6352468138912473, 0.28015997426105477, 0.3677057993169596, 0.3465676384998394, 0.6741955096213135, 0.7415822994270059, 0.5286072116875442, 0.26369726774287355, 0.5223976474988922, 3.487088658444745, 0.37109096575389666, 0.17955949666965787, 0.34832464452744366, 0.40380901030462196, 0.25486869794836126, 0.3477796464401858, 0.6167515265771852, 0.7192454002614086, 0.6171377665958915, 0.45880913690923786, 0.5155032378516735, 0.39327449440139617]], [[0.7533590044774072, 1.0064531372723287, 0.2764861969658617, 0.8643946028727051, 0.7259164334201484, 0.7402114933881301, 0.3591231173249697, 1.0116976231464485, 0.5478663286787824, 0.32335241229488054, 0.25896536520976454, 0.6026485537826873, 0.7594654518883657, 0.6215577558521708, 0.2857232710610792, 0.3240924151453965, 0.37523027007526455, 0.6797430139368448, 0.7424021050306198, 0.3857392255988801, 0.1818933126673658, 0.5188220670247641, 3.4870886584447467, 0.36006838756404497, 0.16679572098574907, 0.36140453170712783, 0.4727195194652274, 0.259729663266338, 0.3218840810959752, 0.722010201727219, 0.7379958593026721, 0.5678828289739539, 0.4672857028072792, 0.5333740827495853, 0.5236384865714153]], [[0.7509705971171123, 0.8498373895465592, 0.26952655930741937, 0.8425120339383809, 0.6979413461290762, 0.7783971907299475, 0.3877550410681473, 1.0114098781835412, 0.5529286787726101, 0.28922744886488666, 0.2810449424125493, 0.6151004681755281, 0.7900805288995566, 0.6195838809095725, 0.2866046564675556, 0.3027027534183726, 0.3843184517105391, 0.7341862503317667, 0.7448367769036865, 0.36380582672945205, 0.16478517966346906, 0.4487436798857276, 3.4870886584447467, 0.3490004155473113, 0.19661207484360865, 0.3540945545510097, 0.527977131869687, 0.2678007653578184, 0.30256873779006416, 0.8073794841578671, 0.7470204980830741, 0.5182451313144432, 0.43661348853336535, 0.5498897070632224, 0.6481338702490793]], [[0.7423203755035315, 0.8237785144587764, 0.2715171115924954, 0.8252112369839661, 0.6860935159306047, 0.7920459228550503, 0.39808390561837176, 1.0181469294995567, 0.622470520566491, 0.29740951257991327, 0.30464574399729083, 0.6393078520922986, 0.8146743193923541, 0.6223853053434142, 0.2804620107937421, 0.3047880579120223, 0.3801305694704078, 0.7014173305294329, 0.7754802502776549, 0.348761921931517, 0.16281658854008307, 0.4333891747601176, 3.4870886584447467, 0.38343879250528146, 0.2312742431677068, 0.3477552710329651, 0.6248906390159851, 0.2795630114220473, 0.2877931311590053, 0.8700711163461132, 0.7499940873214855, 0.4660735229474715, 0.4275756367028859, 0.5622645995087309, 0.7330855760735323]], [[0.7284969317264937, 0.7960247172615277, 0.2664347262592314, 0.787347412901331, 0.703170400768343, 0.758027728745485, 0.37707079870605165, 1.0319063467332925, 0.6039595557899583, 0.2841555695450493, 0.32023976230218837, 0.6424910867577847, 0.8031100543861648, 0.5852648329607947, 0.26952626612890895, 0.3026893461359277, 0.4611969763379571, 0.7262878966096259, 0.7610580167888188, 0.3307814627208272, 0.21313310520773726, 0.4102000436191172, 3.4870886584447467, 0.45831600925280247, 0.25045068240162666, 0.3337044188072265, 0.7407103439075473, 0.28922327640458845, 0.27961570829114296, 0.9259361482345302, 0.7354468917658057, 0.4259526563678505, 0.41649501267480404, 0.580980317844564, 0.7919899316913844]], [[0.7132604855782207, 0.7214036336301937, 0.2664353748146274, 0.8084535725395511, 0.705186784254532, 0.7061687322559363, 0.36980462910097905, 1.051700937137078, 0.6476314163255028, 0.29911987445201416, 0.3034944140637988, 0.6193627411760689, 0.7948820009199642, 0.5955703903310534, 0.23829463241678092, 0.3023839162482814, 0.4496853113171882, 0.7843778689193608, 0.745780296328074, 0.2881733516389813, 0.20877155708026107, 0.31806886365668263, 3.4870886584447467, 0.4687233216578392, 0.2897608445554536, 0.30380665299967813, 0.8219039601994614, 0.28720928917580135, 0.27448781127230887, 0.9708157117087712, 0.7200004367872227, 0.39857563624643505, 0.4349350352060496, 0.5978833833052825, 0.8188144707730651]], [[0.6694348115382647, 0.5985822374141281, 0.2974970281563699, 0.82861984824976, 0.7258480540773006, 0.6815017263178271, 0.369624772169989, 1.0964396094065427, 0.6789163871658452, 0.32868400412984133, 0.2671543914460743, 0.6038682093328386, 0.7491326216752381, 0.6313430189466587, 0.2104184558049012, 0.3333219847038963, 0.4488052971557646, 0.7749830201191841, 0.7191299367784145, 0.2747528643539808, 0.22725213625514434, 0.21935798196479392, 3.4870886584447467, 0.4911245367761684, 0.35707726615106405, 0.2568084706155469, 0.8937676225861926, 0.2852297007379195, 0.26274209790915726, 1.0137642743735715, 0.70341449049102, 0.3753044273553646, 0.4621033948854132, 0.6370581722785335, 0.838700144135187]], [[0.6457714257105339, 0.5377848104909504, 0.3116868509432069, 0.8469216908358164, 0.7206121059428188, 0.6831524535177592, 0.37105030374713455, 1.0784935866730971, 0.6729215340361294, 0.32595984399341826, 0.25613727201887404, 0.6475387152693417, 0.7135186390460059, 0.6247160960505889, 0.23009226426959425, 0.30768000768584136, 0.4654744328310626, 0.7913093902788947, 0.7291932351982999, 0.2767560277035277, 0.2662542810538354, 0.19078485700405154, 3.4870886584447467, 0.45903526948027074, 0.3919925045451597, 0.2123069321953312, 0.9491941884043148, 0.2810284252442635, 0.2511952067841558, 1.0443650474050532, 0.6908288852780957, 0.36064688614721363, 0.4970205730582321, 0.6560708054464289, 0.8570615401289771]], [[0.6167998024620627, 0.4528695935867365, 0.3255533496086003, 0.8520713463906482, 0.7209555001286603, 0.661028894133728, 0.3725778315661737, 1.0639950949522468, 0.6666209240385825, 0.41631485316779954, 0.18252945423988323, 0.7475431349763528, 0.6659820346420973, 0.5574723734727358, 0.18985092116612695, 0.2824599931731734, 0.415789704618199, 0.7510407505298464, 0.7085243089975992, 0.24777251804973383, 0.2369859924700205, 0.1671919531329572, 3.4870886584447467, 0.3989430712399932, 0.35793465159028925, 0.1807955606259687, 0.9817488772687996, 0.26779000847313594, 0.23938874305151905, 1.0710405127727105, 0.6876653523530147, 0.3442975227302483, 0.5179264886621564, 0.6671976653417626, 0.8670759150591325]], [[0.6005063272341387, 0.41215584979046227, 0.3389585737582052, 0.8170384326851785, 0.7697862403729954, 0.6701039594764714, 0.37172506898846946, 0.9653688585027458, 0.7071653079964689, 0.4002101247108629, 0.16738123304065203, 0.7406754325749484, 0.6345496895719827, 0.5555321454524174, 0.1518402709062524, 0.284334405392052, 0.40893753818692724, 0.698047675019147, 0.6266709802833347, 0.17583635498162586, 0.21029205307320278, 0.1257870397860017, 3.4870886584447467, 0.38250785080736943, 0.24383752500844288, 0.14453931441778534, 1.02416071916067, 0.2498143911924791, 0.23612655141386243, 1.1023072997728804, 0.6917064574666105, 0.328306944476819, 0.5375569092760178, 0.6748532570592087, 0.8898398764336728]], [[0.5875263500583804, 0.4064534534798252, 0.34011646400154305, 0.8270207735610364, 0.8079186541897936, 0.6841678805141967, 0.37389232965031843, 0.8951347933736916, 0.7521791125921411, 0.42565018057747633, 0.1607460065180437, 0.7461821835691389, 0.6511078644778943, 0.5968738111036007, 0.20458105221621484, 0.2483062901148444, 0.41958312860201374, 0.7001729374210259, 0.618105271837665, 0.15886620274250668, 0.21082451127384277, 0.12190206790895214, 3.4870886584447467, 0.3488993225883424, 0.15476481217878882, 0.12600333731863522, 1.0285717555918574, 0.23230581226749114, 0.21864045232086377, 1.1172504697474879, 0.6896019533624315, 0.32004762248809043, 0.5458077117135353, 0.672060382511189, 0.9079202079673262]], [[0.571774535233785, 0.39622938005693553, 0.34177639158133205, 0.75373573335428, 0.8551635297163487, 0.6883614017756685, 0.37292459037797066, 0.7714935717460457, 0.7651696126427409, 0.4699063736189375, 0.15415025751997333, 0.7565729529031675, 0.6757382675147873, 0.5573623212176553, 0.27408381837868756, 0.26307782113074546, 0.45127457200259746, 0.7423863590043291, 0.6809677643485441, 0.1729775212263369, 0.21327653702485905, 0.1361421615876292, 3.4870886584447467, 0.39363069452669164, 0.09900864175650383, 0.10859391896959408, 1.0445698195313706, 0.201641345753318, 0.18826670653391425, 1.120423506762588, 0.6874132304000404, 0.3092644148951717, 0.5628047454953238, 0.6725705194640947, 0.9149247950021819]], [[0.5500544239911701, 0.39198490645850115, 0.34657306529120396, 0.697372102934449, 0.8772343637281733, 0.6917059328185473, 0.37543583100015415, 0.7514963375019219, 0.7590189800137419, 0.4974588963316322, 0.13438195753902754, 0.7894626417093857, 0.691295081031072, 0.5477262288905653, 0.2091375475085239, 0.27842794650474095, 0.462254197970694, 0.7508762319830768, 0.6768734573004247, 0.19317475929237643, 0.22800873772599714, 0.1705499932109258, 3.4870886584447467, 0.43083471757935915, 0.07926853310023325, 0.0865305501927934, 1.074038227958972, 0.18546512035346208, 0.16676401455836726, 1.1029121963853261, 0.7056087948734845, 0.2944959606100529, 0.5788852743772006, 0.6877900851461162, 0.9087921433958619]], [[0.544216657478687, 0.38844809765866584, 0.3438819748590246, 0.7255588273402522, 0.8836492102566976, 0.694839043109255, 0.37402641203564646, 0.7888446928201492, 0.7798188187343503, 0.48617803093903256, 0.10755512168448851, 0.7397841508339169, 0.6737162651195033, 0.5627243420621909, 0.2499770751336563, 0.2925744852516494, 0.4809909093205603, 0.7396822430727351, 0.7006734671280233, 0.1697660364966937, 0.21431779784516028, 0.20465465537731142, 3.4870886584447467, 0.4578648206756924, 0.07233159839969926, 0.07014619108245967, 1.0861792214895623, 0.1786991008797415, 0.1787687289500294, 1.0866346993003044, 0.7406552883219666, 0.30466195323531897, 0.614325309740106, 0.6836314906654992, 0.898271858286148]], [[0.5311378646173108, 0.3891920591697986, 0.34376816271568755, 0.7126033776776024, 0.9153525649055292, 0.6952137164997128, 0.37410213339529996, 0.8660015332715115, 0.8181564925284859, 0.49033154530728373, 0.0926889161344427, 0.7405352819735751, 0.6814709987956389, 0.5619306768674623, 0.26020812178045255, 0.2847695977587956, 0.4894317951493462, 0.7401594234833477, 0.6883237733875289, 0.14380101969899395, 0.21414474053834615, 0.2296977343669383, 3.4870886584447467, 0.47770002431070885, 0.07272676205764686, 0.06126030063239361, 1.0969540378172475, 0.1865945707745283, 0.2198069133943073, 1.080670165382568, 0.7070457430832379, 0.4083824418287378, 0.6414373163039361, 0.6911404715372946, 0.8826016959472416]], [[0.5295166808685463, 0.38800043664785083, 0.34366647994384575, 0.6901080963490958, 0.9556213181073783, 0.7200843353062141, 0.37403536505632956, 0.8498954221665869, 0.8437173016166294, 0.4885225272809943, 0.0947282186440295, 0.7220234758112697, 0.6926130706383229, 0.5669190302091418, 0.30066470413361257, 0.27618885495711315, 0.49216877519801494, 0.7151270626673114, 0.7416642343429121, 0.11807159274055945, 0.2153737961724601, 0.22716060026412166, 3.4870886584447467, 0.4975988987206785, 0.0698271159658465, 0.061888657419787205, 1.1120484541604816, 0.22168544392487113, 0.2601726991806874, 1.0801209730280017, 0.6260230244407552, 0.44284413735476297, 0.6670279303107617, 0.7175097348522725, 0.8718203354045245]], [[0.5094668891888741, 0.3837085041316115, 0.34437082166314886, 0.6814692214653804, 0.9647662928177381, 0.6849737359024706, 0.38177350179037983, 0.8663790194308447, 0.8585005018824885, 0.4966979974391704, 0.09046744302825249, 0.7826055244715846, 0.7070901803420984, 0.5552928884987165, 0.3074806480747885, 0.2892058868894475, 0.51774105057157, 0.7944523314816587, 0.7758609257833401, 0.11603610466632797, 0.2234520257637887, 0.22661964307247062, 3.4870886584447467, 0.5236494117954941, 0.07638722915474006, 0.07702515805984647, 1.1190034707589611, 0.21503813326544274, 0.21988557797913072, 1.0793431872873964, 0.49359788894774387, 0.40908068085062554, 0.7208629888972915, 0.7431361456657526, 0.8488723318321398]], [[0.49690972905777464, 0.37502380387454837, 0.3387580855041957, 0.6825035073536991, 0.9954074963888704, 0.6923024196933352, 0.37630230609086956, 0.9052455697399693, 0.7917424553481146, 0.5166008472716316, 0.09031743786301395, 0.767200296802349, 0.7346014968744866, 0.6150792484482346, 0.3230966081445461, 0.3221621974053309, 0.5308879270141657, 0.7432909172740652, 0.75517273707741, 0.1430707568852605, 0.24142840754467232, 0.23909177693889028, 3.4870886584447467, 0.5863181660872323, 0.08039447143323955, 0.17071424416185352, 1.1388259223679056, 0.23297605767345997, 0.19645257260650145, 1.0434810256603675, 0.4055391774149324, 0.48607256406406796, 0.7550535690513391, 0.7668367573341113, 0.779799649845193]], [[0.5464822765189749, 0.37476289328314244, 0.34204005497776213, 0.6570686284750369, 1.2411540001774066, 0.6914589452809874, 0.37790934782079305, 0.9710980457856501, 0.8635980539615196, 0.5390013316745892, 0.0786010817908756, 0.7753200817531657, 0.7237023076667558, 0.6289392756499957, 0.30453983622019987, 0.3617578766239601, 0.593592948424585, 0.7037634899370231, 0.7577094356449587, 0.14649477600550104, 0.23708743203773802, 0.2352672837102218, 3.4870886584447467, 0.589156808555791, 0.08472172801105685, 0.15700102950054243, 1.0929513390442753, 0.2254056532584922, 0.21743055689502527, 1.0063808658848092, 0.340014942876094, 0.5432613503341748, 0.7475148887723786, 0.7679544220556267, 0.6952003535674622]], [[0.4885389974148825, 0.376864423669134, 0.3431791095824972, 0.657328581882183, 1.3149253381814807, 0.6922321887028517, 0.3792353112640682, 0.9669678223217154, 0.8921163903894171, 0.5443934981708912, 0.06988749821952986, 0.7590349708479375, 0.7237416066296192, 0.6355137170508529, 0.29815462976028595, 0.43602368792311685, 0.5940617230063241, 0.6870560258524014, 0.7815680543618333, 0.14482048532378122, 0.22842327512685326, 0.25398067397926183, 3.4870886584447467, 0.5709434314182942, 0.08644691016760046, 0.14488141257170356, 1.034319868541821, 0.2096072750754978, 0.19536765522423383, 0.9165726206012877, 0.2513703829047909, 0.4792069375480621, 0.7237235533062465, 0.7590394374625241, 0.625031321151604]], [[0.3826804843069965, 0.3860191511542806, 0.3385394835808487, 0.6506569249623462, 1.3052202270909232, 0.688585152036983, 0.37506770517753646, 0.9538762808727701, 0.8875935269529162, 0.5449602251453394, 0.06285316483344894, 0.7692262299580795, 0.7495320105417196, 0.5848485819333282, 0.2893824414506717, 0.41542154471246207, 0.5619309949600788, 0.7297666124395426, 0.8231396496419098, 0.15039705552857158, 0.2230516227545742, 0.28365671372270995, 3.4870886584447467, 0.5508889833972643, 0.1083757149661696, 0.14787535300646043, 0.9452242306444704, 0.17981865154782428, 0.17198661578723767, 0.8379637447485091, 0.23434245627903527, 0.4386406722882555, 0.6983905233364076, 0.7298309654020495, 0.5872703593305791]], [[0.39231174571052974, 0.3855769522373275, 0.34538693453876124, 0.6688354137505197, 1.300627427665097, 0.7050769697893369, 0.37532482354819235, 0.8864506327274916, 0.9300043479072135, 0.5690198315983329, 0.07921961204399007, 0.8324190821084487, 0.7430960831942677, 0.5640201954810351, 0.27914560548073974, 0.3788823099311466, 0.4948041151405073, 0.8147823056536583, 0.8761868787286449, 0.16054587246839597, 0.21012865353133642, 0.30519792570803034, 3.4870886584447467, 0.5042929968517171, 0.134614333372089, 0.13565619373909205, 0.867529082230596, 0.1806424879767756, 0.21048662219977765, 0.7766703063115943, 0.25906826521517556, 0.4171227324432779, 0.667953836539587, 0.7253155025073086, 0.5643614211714495]], [[0.4072208584406549, 0.45305140257021687, 0.329229719279866, 0.6662914317736968, 1.2660949239485686, 0.699816188238308, 0.3753543559826804, 0.8472385695067114, 0.9836707859005902, 0.614111621452283, 0.08129084644174944, 0.7961291052601311, 0.6690374615022681, 0.5590000258289726, 0.2857422354816328, 0.4040155719226307, 0.5002975440525229, 0.8199991558392505, 0.8564809353016267, 0.16612249069954588, 0.2007122777082597, 0.3117898244564533, 3.4870886584447467, 0.478587804380964, 0.1715523234711725, 0.14110003015704115, 0.8043792157098026, 0.18885042335427066, 0.24138422998615128, 0.7158978507370876, 0.3027256750554337, 0.42083210885275957, 0.6423328804857771, 0.7127728461679254, 0.5570262266307382]], [[0.433015115712436, 0.5087295619480523, 0.3129562566224802, 0.6752538393588161, 1.218545988041205, 0.7297911760843052, 0.3846590168475273, 0.7484995489622255, 1.0269475700365918, 0.6498252538171572, 0.11626419594249142, 0.8009249632436243, 0.6438062769233872, 0.5494348579232748, 0.2772925001986324, 0.40511637976161996, 0.4985693799839091, 0.8187561864889954, 0.9106452087252985, 0.17145603525088876, 0.19296971632721993, 0.34767335642425523, 3.4870886584447467, 0.39657734220393986, 0.22439752480436684, 0.1853318998004594, 0.7232894471358071, 0.2014973456866857, 0.2583941340823538, 0.6464197906096141, 0.303134769927903, 0.4586007319545181, 0.6119702726394212, 0.7072898192440351, 0.56457933227921]], [[0.44198461030919833, 0.5002523322037904, 0.3124922311265279, 0.6472956145952872, 1.1309432180605385, 0.721147162467586, 0.4160553864887568, 0.7092782508458662, 1.0001703568521951, 0.5984411068954842, 0.1156038286894605, 0.7137436921357593, 0.643237969606021, 0.5467551353756284, 0.2764666842615262, 0.3712938728262737, 0.4689702610979294, 0.744415832504658, 0.8893215446627003, 0.14813030970908547, 0.19966174962490552, 0.34491000112684234, 3.4870886584447467, 0.32561102459542224, 0.28424366827675196, 0.23708111573185114, 0.6707282196504946, 0.2395355067098594, 0.31806380667998824, 0.5708339765331947, 0.30677366806934014, 0.46768874388670933, 0.5523572363634927, 0.7247486664391671, 0.5566459685210171]], [[0.5311818320846622, 0.5489500282029411, 0.29448869511901676, 0.6565332846898508, 1.0630148069135104, 0.7568814271364264, 0.4113751343397764, 0.5905635651981839, 0.9588831369598801, 0.4732826055754644, 0.10991529176458587, 0.6897613238912366, 0.6474480498585694, 0.5397012937905716, 0.26413627941916723, 0.343640966129554, 0.4681712543764672, 0.7533986020328812, 0.957711276822229, 0.15187905549763703, 0.20248403914489208, 0.2962982789091771, 3.4870886584447467, 0.2507678210381007, 0.3107147785477472, 0.25578113301667094, 0.6412913484503636, 0.22823233490395814, 0.4354815374288644, 0.5084413942654358, 0.29985587368788325, 0.47689664934654136, 0.4861304320804905, 0.7318698372887142, 0.558761436480799]], [[0.5204218193021862, 0.5275608394448679, 0.30390089777279017, 0.6761818959695945, 1.0076469294581145, 0.7558208919591863, 0.4039582938625588, 0.5360086640395766, 0.9544059137592282, 0.39250086935367445, 0.09839153673210789, 0.6829444740227772, 0.653103856226946, 0.5427666587620918, 0.2771869546266998, 0.3589724416347625, 0.4629780709961845, 0.79620544644543, 0.8762265407222696, 0.17070166544070287, 0.21937372728609883, 0.30371560201565284, 3.4870886584447467, 0.20159347602845198, 0.32944430618018317, 0.23490153519586698, 0.6542499424764219, 0.21570640076286568, 0.5247279788671111, 0.5098219055881381, 0.3127616124051318, 0.49814983831220133, 0.4631060653192074, 0.7637136865670093, 0.5605674422264614]], [[0.5149566218108974, 0.48426993808938007, 0.3123182774897213, 0.6823402542548074, 0.9155614765025804, 0.7449042900390994, 0.3906852922639537, 0.501962226811085, 0.8963919105573221, 0.4450106183693063, 0.11782195809815625, 0.6589256225951647, 0.6569670880321818, 0.5593748263190101, 0.27839354332500404, 0.3496419959458422, 0.4429297867655161, 0.7976369848984097, 0.8089217223829743, 0.21043636171789049, 0.24852588155673877, 0.32575001078196403, 3.4870886584447467, 0.22996060892135523, 0.3402236014548199, 0.19520496247140487, 0.6922743236527367, 0.23420780377010902, 0.49277257851150297, 0.5091033636122777, 0.35218779711524495, 0.5092416586613249, 0.41967122722254224, 0.7663358756170691, 0.5689093104624731]], [[0.5368338955095904, 0.5167014961353942, 0.2991134804163804, 0.7237359857342673, 0.8408667185499742, 0.6975152676777419, 0.3799674798877225, 0.5022024791856761, 0.8550297651584007, 0.4814958372977655, 0.11432036131070379, 0.623479057834472, 0.6432688370341949, 0.5726559556050074, 0.28084897696841266, 0.3628263727316846, 0.43192879346818114, 0.8008565139543631, 0.7602049224990495, 0.2952421388622361, 0.3013672102223288, 0.3331833271327204, 3.4870886584447467, 0.2828678272257179, 0.33637856057086, 0.162992525183331, 0.7581725931157037, 0.2650007389279518, 0.48185809160857546, 0.4502580324459404, 0.3794643526701992, 0.5022570865983911, 0.41761667448405015, 0.7564646216672407, 0.5652301806940687]], [[0.5336163463744631, 0.5035639301409807, 0.3064600397221545, 0.744046027104805, 0.9521353927097579, 0.6908122702653526, 0.38078241970811666, 0.49202031964994936, 0.8092655928359963, 0.4191972460747242, 0.1043717646244228, 0.6288412862680685, 0.6171655732055176, 0.6115310266773598, 0.29583737441071956, 0.3467282153227616, 0.44562398923674607, 0.8207635243232074, 0.7118186439574921, 0.30631239235076335, 0.3630214121919672, 0.27548936677733565, 3.4870886584447467, 0.31599391283902784, 0.31767609129050006, 0.16973497173381383, 0.8264115635018, 0.25743261423506797, 0.5012682175210912, 0.4469786745670331, 0.40611116849787143, 0.5094309326945676, 0.43454451146447387, 0.7643356957539654, 0.5519214315448441]], [[0.5501041037384558, 0.5059862269665207, 0.30434408806479757, 0.6936786384196427, 0.9199416035759426, 0.6694831800828882, 0.38839457995936216, 0.5639555025814728, 0.8347134118916046, 0.35220969882202735, 0.07419604322507017, 0.6497408056768555, 0.6189098647067689, 0.6083759512713255, 0.29940119351260586, 0.340094348719739, 0.42191488182596115, 0.7564967382298412, 0.7074369332801149, 0.30211499761921534, 0.3600690672833353, 0.28678254970345285, 3.4870886584447467, 0.34885138015529255, 0.3401420203338858, 0.18118617654347877, 0.8887807941565651, 0.23485604779706742, 0.48762102572394594, 0.4147036642280566, 0.4362803384639552, 0.5534472240682473, 0.4526463063553482, 0.7709568883312172, 0.531583995180686]], [[0.5563122885096852, 0.5186689624859069, 0.30093988498488483, 0.6708816339450807, 0.879859781105068, 0.7276448219577742, 0.462444006745138, 0.5919234212021146, 0.846625875259085, 0.3780292558261819, 0.07590610649243953, 0.6839206834931522, 0.594071950902783, 0.6125477824916166, 0.3033533367832609, 0.3543166181358244, 0.4039514829897828, 0.7417997462510716, 0.721352093106507, 0.30181624748086566, 0.3395057843763028, 0.23298996498520994, 3.4870886584447467, 0.37918284546269154, 0.34915831465331115, 0.17762861521987344, 0.9734536364648153, 0.23443458772782105, 0.405218163594802, 0.39269036823785997, 0.4536501468337928, 0.6065953973729603, 0.43581472624120826, 0.7357376185213873, 0.5151592185675187]], [[0.5359445001967442, 0.4992963205290174, 0.3091049209410115, 0.6877052168750792, 0.846348543987259, 0.7042220238270627, 0.4463436497661005, 0.5873323284595737, 0.8549761498270075, 0.4004709071168891, 0.07989308874214411, 0.6427501142700711, 0.6334380125773642, 0.6186765162695997, 0.29911401057035025, 0.34355471940623705, 0.38380379223150485, 0.7144263726663043, 0.7432729915154755, 0.30584755717318135, 0.3513921156293662, 0.19975117716840057, 3.4870886584447467, 0.389678025739712, 0.3533015823297718, 0.22263277053197666, 1.047803098991896, 0.23836719053650804, 0.3878749421720069, 0.34316917786935275, 0.4700872951639874, 0.670380980093246, 0.43055750597762094, 0.6894266976388161, 0.5010153010743164]], [[0.5608510852912421, 0.5202952930202158, 0.30724264796926487, 0.7340427963009384, 0.7269483886579797, 0.6942609355968921, 0.45970937385608973, 0.5893301176859818, 0.8520910776174584, 0.43097174885115686, 0.07726232065267684, 0.6887025825415762, 0.7071402417956938, 0.6427737937788163, 0.3010462125716188, 0.3499811653778972, 0.3859828437133134, 0.7161653196960205, 0.7430024290296244, 0.28748823917001753, 0.36177785994175204, 0.21735378460712884, 3.4870886584447467, 0.3622236980727249, 0.36715790523239517, 0.24307496314896898, 1.09413914649113, 0.24867681921796803, 0.44222295740303613, 0.3484328389686584, 0.48861699718865226, 0.7031179983929217, 0.42795951006956323, 0.6565340556151615, 0.49809818527212424]], [[0.5639010283536348, 0.580131550737397, 0.30106847987636487, 0.8111508044487343, 0.7098571586172925, 0.6890447121252852, 0.38883665694018227, 0.4638043428827237, 0.8091057840489346, 0.46329333292910213, 0.08420584099079495, 0.6958735919394414, 0.7218493224822055, 0.6062693748342928, 0.32804541066311266, 0.35012112549180263, 0.38362906599469865, 0.7169406259410065, 0.7569124971941399, 0.2807727354511411, 0.34989293342279154, 0.34867831124799126, 3.4870886584447467, 0.28004974378032665, 0.39541295727919423, 0.2906980569227758, 1.038600004372268, 0.30293981855684793, 0.4557800485566522, 0.29655723141189066, 0.5235571586675867, 0.7250863684572955, 0.4342669035491403, 0.5863202178326495, 0.4849700191034776]], [[0.5723713431264621, 0.638899847857161, 0.3210199739260463, 1.157280738206973, 0.6220783109236878, 0.7048906655754592, 0.38294148079838325, 0.5921556334589952, 0.7982952086188702, 0.494136615797284, 0.10701864913245321, 0.6698006837436105, 0.9958574438523247, 0.5792082664753496, 0.3280677142756139, 0.3523170097287339, 0.380959099680263, 0.7368547736848138, 0.7103797000882996, 0.2937549068022848, 0.33590107557826143, 0.2524663183965686, 3.4870886584447467, 0.2338946955491143, 0.4839996101768011, 0.30848831124173065, 0.8771640865418315, 0.33950919749515723, 0.4416917318328307, 0.23838285641604468, 0.5267724994231393, 0.7094526660252609, 0.4232351104290308, 0.522750748741778, 0.48192016199371035]], [[0.5938247864868592, 0.7314190698996141, 0.2823799295818811, 1.5249621602708467, 0.5526968685094057, 0.7325151995582806, 0.3649882568591979, 0.6264640167079774, 0.8023597136764153, 0.4597931755583273, 0.1231116978401377, 0.6677583185443994, 1.1448453897988242, 0.5685771172125115, 0.33528219475605775, 0.3440671917881333, 0.3803749475984969, 0.7650499793380567, 0.7151013575596827, 0.3482472628851275, 0.33433415424533675, 0.23698342462634658, 3.4870886584447467, 0.2553824079020918, 0.529781877753051, 0.30772641020742764, 0.7724959177736435, 0.3571408511199079, 0.43582687234028594, 0.2615076376194107, 0.4922701937738373, 0.6892977245413665, 0.4126087910399175, 0.4490005669352315, 0.47986061526266766]], [[0.5852798329049123, 0.7892501543803399, 0.28091216646887557, 1.646103702777141, 0.519554003503744, 0.7052458371550672, 0.39093409942055274, 0.7363093083485526, 0.7640212429301763, 0.4301420820699349, 0.12763859031412364, 0.620045981402262, 1.1779779835515165, 0.5119924446287073, 0.3508920074761282, 0.3407515970115106, 0.3743421457412117, 0.76747301829551, 0.7414039752773279, 0.5027215378851583, 0.34573124786090004, 0.2394000320173898, 3.4870886584447467, 0.22359125695676307, 0.6121829242521892, 0.34077677062465533, 0.6553718770026249, 0.400806461703452, 0.4305707572591915, 0.24825374644465806, 0.4811338607895153, 0.6730899342929079, 0.41636776315590773, 0.38518009801130304, 0.47797749856609206]], [[0.5513738986123249, 0.9253120620208299, 0.3277441834006839, 1.8424016033430446, 0.4915373313174046, 0.7254737895255012, 0.4072782177840305, 0.8846505467292145, 0.7036787345088602, 0.41888161981666305, 0.13566012620694795, 0.5140603579676364, 1.1499255335467098, 0.4467985963337713, 0.32647306989206815, 0.36200611319860654, 0.44304133450373007, 0.794369265318132, 0.749554338621701, 0.5674383274189809, 0.35257065414348965, 0.23935976062656616, 3.4870886584447467, 0.19620953579880274, 0.8723653654037771, 0.5059553052338268, 0.5608813921626584, 0.5541040651341568, 0.4257599063033436, 0.2955402855050864, 0.4924947723017188, 0.7017701491173483, 0.4145867144070261, 0.33952829759547154, 0.47680765990837704]], [[0.5313497364924662, 0.9284140904307185, 0.32148180319877834, 1.9231895440451048, 0.5986042126358992, 0.7297256903264846, 0.4666749247014254, 1.0589368163978603, 0.6720907288166735, 0.4190981384849347, 0.14096232134867653, 0.5205878957371852, 1.129673036293525, 0.44764262950491424, 0.3338724903110601, 0.3559714664454913, 0.45928401124189827, 0.8012902066021438, 0.8074875327060342, 0.5430630704574293, 0.3356159166921133, 0.240268627180347, 3.4870886584447467, 0.17814495996892082, 1.1499813400367294, 0.6283302525633228, 0.49293306937503933, 0.8237725259862152, 0.518014361166389, 0.2972799592635317, 0.46727818982940195, 0.6963965772016403, 0.3955910954351053, 0.3394024404110532, 0.4799998736730009]], [[0.5342961372016648, 0.9200852691867742, 0.3329210308057001, 2.0046649184633947, 0.5930802894302427, 0.7060738632288897, 0.42412608417746067, 0.907599536414055, 0.6427789102889065, 0.41092379478599517, 0.14826575998836522, 0.5034247559155006, 1.041618491461195, 0.43811365036702926, 0.2831428635540556, 0.37516640532487633, 0.465426738518421, 0.831615043675484, 0.829443217787736, 0.5154875098021299, 0.34647280886764886, 0.23273238973295496, 3.4870886584447467, 0.1608455644589673, 1.1423789274850003, 0.5603953593114748, 0.4349695619270397, 0.8043230650356322, 0.5986429073768434, 0.2568942611895513, 0.462979967522937, 0.6870192575552653, 0.350232050603422, 0.25097725287975275, 0.4717903502083973]], [[0.487953330592249, 0.9248522683954561, 0.28854520965103014, 2.064868670871202, 0.6124329434467672, 0.6930728575384746, 0.42603421968620603, 0.8344080018810016, 0.5708270934188722, 0.41054585818720457, 0.16323065767176975, 0.49901934363698525, 0.9192488856296698, 0.40704469959824735, 0.273327606887491, 0.3704119699759525, 0.5006074681451999, 0.9123026721276424, 0.7986979323770769, 0.4760407955144768, 0.3184389442245833, 0.22291521003303816, 3.4870886584447467, 0.21236130907495712, 1.085152610173253, 0.49974090595046883, 0.39357407466053773, 0.7090115856918603, 0.6022939799409129, 0.2507125234729859, 0.5093719201243323, 0.6884225574448593, 0.32281228167221665, 0.2742678617608141, 0.46796888628358424]], [[0.4844612069013333, 0.9022857558516564, 0.29216903027275587, 2.0676942696632037, 0.6371118506669466, 0.6980738700553899, 0.43048618966752294, 0.8634741180366399, 0.5098111834773634, 0.4143049898286353, 0.1709138304016178, 0.5056454238880539, 0.9644384397050662, 0.45127831476747304, 0.34849036274186407, 0.3370268148798409, 0.5197411901968181, 0.8187944148899294, 0.837182494817228, 0.5260104240498991, 0.30431166176383984, 0.20929223945948727, 3.4870886584447467, 0.22993522651509143, 0.9951632219387762, 0.4552301118729246, 0.35425823261884243, 0.6824424593699603, 0.6189088396354263, 0.2677428253140556, 0.6128570603304981, 0.7308682475050775, 0.3078279225240113, 0.3321591028364672, 0.4594921572171743]], [[0.47052448594545826, 0.9069018206560459, 0.28854979201113545, 2.05657337411597, 0.5102193283383297, 0.6893750242182264, 0.41595979373596165, 0.9970043240732542, 0.46279025945600055, 0.4627701576104286, 0.175334671664289, 0.5885236995326383, 0.9347240699651742, 0.4882029188741117, 0.32568682535935184, 0.36244956315666343, 0.5647810205785578, 0.8150410955662044, 0.8677174004748858, 0.5340687013486386, 0.30338479310889255, 0.1868711278034497, 3.4870886584447467, 0.2547318252218751, 0.899980778756868, 0.3963100935668943, 0.34474164569835086, 0.6107084226382556, 0.6414801154590392, 0.26709874876061335, 0.7382909982523604, 0.781759666486443, 0.2851777165178803, 0.41025862727173834, 0.47546988662874934]], [[0.5052330559912859, 0.901949616006815, 0.31348913819146657, 2.0641493929919053, 0.5321286962198379, 0.6726672052026172, 0.40789652508665863, 1.035377990594718, 0.4889965444024381, 0.5353815395495622, 0.17931443512585352, 0.5663537122609833, 0.8743473856731534, 0.49629148704695847, 0.3541265675891727, 0.38246088190482197, 0.6017109150262832, 0.8332534980223503, 0.8949487164264873, 0.5147105965711591, 0.28722072976189594, 0.17569766771488843, 3.4870886584447467, 0.28432687963723186, 0.9621647759682742, 0.3573045160958574, 0.3739416888194363, 0.5752956957802173, 0.6946720373285369, 0.35841533693657535, 0.8446235002750265, 0.8028513969748827, 0.30514130368175885, 0.5178265003760074, 0.46852425184619473]], [[0.4745061251115516, 0.9788886461289545, 0.3288752388650343, 1.986804978735528, 0.5637661069934282, 0.66255660220397, 0.41409574688275974, 1.0754280846321072, 0.38056915371016686, 0.5709293671093851, 0.1859150369071022, 0.5514556880217999, 0.8145906014806703, 0.5111061814081966, 0.3373065333227871, 0.3838433260759152, 0.67895874747647, 0.8178833365927176, 0.9806836792429038, 0.48136903304874984, 0.30219128583944366, 0.17122125217859724, 3.4870886584447467, 0.35011655965894256, 0.9548453683824426, 0.33277955547247245, 0.3898200036547208, 0.5457849766931202, 0.7727398026884837, 0.4535517082728803, 0.8603277612905136, 0.7611047554701202, 0.2918232509174573, 0.5485182537067493, 0.42760031011979727]], [[0.4782570886126588, 0.8896452150068939, 0.29014142535933696, 1.9629097891891867, 0.6160383355728585, 0.6829211187970129, 0.4095060893809147, 1.1754145218430818, 0.3868655933517812, 0.5199336685816272, 0.20122109040609387, 0.5361614552894722, 0.7548790377101804, 0.4835996580012839, 0.3924568297254817, 0.36256562815481974, 0.7349099063167079, 0.8546774499129497, 0.9961254638152265, 0.47224909860721237, 0.3304257515363957, 0.18069212925767503, 3.4870886584447467, 0.4022406460401555, 0.9418333266143925, 0.3146403156640437, 0.38651558330041846, 0.561467997310442, 0.7980599109787756, 0.5298291249153346, 0.8265315916041304, 0.6615336526653592, 0.2760733067392538, 0.5795929129945456, 0.3939785171919652]], [[0.520225881482077, 0.8145081310890857, 0.3139793619305768, 1.8713465928077893, 0.6065377078212308, 0.6721796283925019, 0.38844041084410424, 1.2164150041096369, 0.37070682064926624, 0.5535826596926944, 0.21384451578773667, 0.5306164009088903, 0.7455454101306236, 0.4676704334037594, 0.3876750700006528, 0.3230611976217377, 0.7332641755481168, 0.8489054273028, 1.0389164324106717, 0.5576298414395955, 0.3505860120630616, 0.1859722904873199, 3.4870886584447467, 0.43498662922605436, 1.0599499237315726, 0.2965519918759674, 0.3930560398089632, 0.6297534344981357, 0.9271421575064741, 0.5624292517055811, 0.7786019861944566, 0.671669828617788, 0.22391143812639952, 0.6040801991323506, 0.37264868036566046]], [[0.583772342678007, 0.7634264668858373, 0.3021557314586824, 1.8086359898193658, 0.6435590581915381, 0.6740198706313815, 0.4106243599994954, 1.1996319331136172, 0.47515947869007097, 0.5672053326056742, 0.21157604760720256, 0.5263592495284478, 0.7144431925886497, 0.45977087276510864, 0.36879958760411546, 0.31786596838364967, 0.7001800626119967, 0.8523382356567202, 1.0270632364964332, 0.59028441318315, 0.3607564375639063, 0.19841684378146968, 3.4870886584447467, 0.44319541916644833, 1.1198909791378338, 0.2740469916755025, 0.4063086045698876, 0.6240425743091669, 0.9080822448318895, 0.5943261347314448, 0.7881716106896418, 0.684281373499397, 0.2045663575569077, 0.6576638867889126, 0.3901280951129459]], [[0.5773803885083473, 0.7414714822806416, 0.3115736159205511, 1.730743211230367, 0.6375410512262129, 0.6757357086833107, 0.41545562585047613, 1.1677626168643045, 0.6028417749865005, 0.6105521255207575, 0.23575850819018718, 0.5515311968539136, 0.6600283178829327, 0.4931090556114281, 0.35467006805456214, 0.28124943385622236, 0.6013131268350433, 0.8219266212797622, 0.9721140720033496, 0.6257263409528676, 0.3802757987737086, 0.19578438610571652, 3.4870886584447467, 0.4213500474640137, 1.1471040741688965, 0.251247516219551, 0.38038078357317373, 0.5935840571643992, 0.8726122560248692, 0.6036131078973489, 0.8590728061876154, 0.7004540571561502, 0.18369895958986776, 0.7313354266151313, 0.4367889355579214]], [[0.5920883693874183, 0.7248120695872005, 0.3088178788978833, 1.6896464729149032, 0.6379118430694428, 0.6598299699885034, 0.41599961118464357, 1.1136234070097197, 0.7039830436161323, 0.6380684491777695, 0.2879979627476029, 0.574522726342458, 0.6851185474231325, 0.49599153193815415, 0.3525555713079682, 0.2881385989769369, 0.5925241971199223, 0.8452345443682824, 0.9530978086761384, 0.6393836538720988, 0.37377860142369873, 0.20095557384790336, 3.4870886584447467, 0.35162008106657006, 1.113376915034763, 0.21878753531167053, 0.3482824962333909, 0.5777323233615212, 0.8247636017968693, 0.5988340647605943, 0.8605840995999423, 0.7316118567082408, 0.14446709678786765, 0.7643694444995863, 0.4282826968553244]], [[0.5731733731795561, 0.755096543633492, 0.33127100676124305, 1.5506951901199708, 0.6180220275594207, 0.650019359668262, 0.441736005137008, 0.8901058734644368, 0.8094630862097443, 0.6224244762815488, 0.23407682181323447, 0.5181871357799321, 0.691776957168712, 0.5400550263104458, 0.3375050975487347, 0.2789317821091956, 0.5895338530772842, 0.7676039643580608, 0.9921543268571977, 0.6340632123409924, 0.43753523145216633, 0.2361759216110949, 3.4870886584447467, 0.3042756755911226, 0.9490759464225802, 0.14568175466943104, 0.34545958747238614, 0.5163356986383836, 0.6372933235455183, 0.615658678632214, 0.8567979984755842, 0.8135001632674115, 0.12992507427095712, 0.7767719285422097, 0.44893599776173315]], [[0.5892613269297852, 0.7483560010914763, 0.3141789690767501, 1.5066402765343443, 0.6262585683527815, 0.6655426865120974, 0.4330267911966171, 0.8910820472085459, 0.8834861038392655, 0.5934351234263063, 0.22448222676333612, 0.5346110231326058, 0.7100118518105728, 0.5100413861020674, 0.34531437198521253, 0.25874660938945143, 0.5813307512410277, 0.7632447535381287, 1.0623341804744646, 0.632410471399892, 0.4754514435850725, 0.2830663547434478, 3.4870886584447467, 0.30067617880504394, 0.7428288807023147, 0.10959913052414884, 0.37790643492536274, 0.4503306417619523, 0.44654527704179997, 0.57760066934886, 0.843358273415096, 0.8072867509100695, 0.08239850419987033, 0.7912884619714124, 0.4177929023129025]], [[0.5949677165110263, 0.7401444799306389, 0.3149452832201732, 1.4577724451275924, 0.6331724305336514, 0.6881399847961903, 0.4352780059419464, 0.9018145022445406, 0.9487400682424306, 0.5575383948481311, 0.1939316657403265, 0.5766503626497386, 0.7241673417464876, 0.5366334926528719, 0.33328018011994315, 0.2487555285393343, 0.5639298209309, 0.7127875769393702, 1.0186625370629236, 0.638597841925537, 0.48249147223260636, 0.30245782159329193, 3.4870886584447467, 0.28963638288748456, 0.6831733959958604, 0.09815887038192525, 0.41159603427352365, 0.38770160193749803, 0.38098559429199363, 0.5561103352687312, 0.839689718831989, 0.8058902464931279, 0.07678288608393892, 0.7760434423283782, 0.37222118125456516]], [[0.5886473017284357, 0.74128479252029, 0.3105018123382226, 1.4016812231714664, 0.6459347555032502, 0.6882568945848544, 0.43512918690963887, 0.9018957240582163, 0.9902804614279461, 0.48016018211407585, 0.16481681587353983, 0.5617663310410146, 0.7256730039598493, 0.5702336128454095, 0.3321786511214563, 0.2503221748794034, 0.5719690305799546, 0.7121024375647576, 0.9650677939027353, 0.5903892948752052, 0.4102690074176359, 0.33105396310899127, 3.4870886584447467, 0.24455240062078049, 0.6434340788130675, 0.11015787301819996, 0.40569824610556127, 0.3968096415781377, 0.38623841175954177, 0.5428679270499406, 0.821928993461893, 0.793862840075513, 0.08680122527291607, 0.7489588320935527, 0.3505062096108419]], [[0.5990945658024038, 0.8078423829508381, 0.2949482019141151, 1.3830341060324578, 0.6829834907501893, 0.6702934824185536, 0.43390856871941447, 0.928878590428299, 1.0333099587160783, 0.4017719619841136, 0.1327618703703971, 0.5467812291087266, 0.7060281308226497, 0.4959392709842192, 0.3362498640849752, 0.262553502713626, 0.5615741371926124, 0.8036455947777008, 0.9965912528518273, 0.542614094220062, 0.39772785329214577, 0.33783026572308433, 3.4870886584447467, 0.25431134631935043, 0.6134157894657095, 0.15299180168324747, 0.39537703226821674, 0.40666377947600174, 0.4083798729110199, 0.5366255391544037, 0.8007494866715094, 0.7775776029858121, 0.16241980721039595, 0.7825574852320644, 0.38069835306359295]], [[0.5704534630762657, 0.713428568930457, 0.32429718234033666, 1.343387491953812, 0.6754200074901148, 0.681641874271745, 0.4338005368946406, 0.9196084119350958, 1.0695990697532407, 0.36913163983872355, 0.11982084221294886, 0.5933143682504007, 0.7165697462985647, 0.5024352333563398, 0.33523076648864236, 0.2785506272290012, 0.538061377520978, 0.8036232297680965, 1.0568954265635857, 0.48123469871121344, 0.37491766508268487, 0.29409187698185135, 3.4870886584447467, 0.27298263450579496, 0.6240978928393776, 0.19450837996730222, 0.3910964434285531, 0.4214828439045751, 0.44600528858940713, 0.5627561916800314, 0.7781207985241531, 0.7315667707822757, 0.24967031340645882, 0.8008058726757245, 0.4177215589797123]], [[0.4847958223822778, 0.571744698809026, 0.3106646618446548, 1.2919964091536813, 0.6767668468969121, 0.7079902815495454, 0.4332772074935404, 0.8375192273156662, 1.0647797199002202, 0.35619290160525374, 0.0652608481878544, 0.6001200286823523, 0.7125159379892588, 0.46976396798540176, 0.4456635882842538, 0.30201504543741553, 0.588851915581982, 0.788974218390393, 1.134283845217194, 0.45247035387000584, 0.32234814240038956, 0.265952822858469, 3.4870886584447467, 0.2698704976852139, 0.6136311217414907, 0.21395993789906148, 0.36518856843796466, 0.43459124765000345, 0.4715866398223494, 0.5582767548023992, 0.7823097484132324, 0.6763317681705237, 0.24173830074525443, 0.842905731003196, 0.4860766442632579]], [[0.44009892643730264, 0.569439717979154, 0.3153688615622547, 1.2039251279318168, 0.7257652061703194, 0.7125581449512037, 0.4279378758595382, 0.8137949244467886, 1.0437320134854449, 0.4055959253734748, 0.02993840349418355, 0.624518886643797, 0.7039491579272786, 0.46784689225749354, 0.4280973721234006, 0.3057865005684773, 0.570069406765761, 0.7694146928363761, 1.0930225397154338, 0.39831201116070813, 0.29126486546511104, 0.26493191918215553, 3.4870886584447467, 0.2592238394461813, 0.623621885581285, 0.25038790564951363, 0.3391762359983905, 0.4643895608907189, 0.5023860516539176, 0.5713085217231784, 0.7667160611344639, 0.6987715531327319, 0.22114098842722124, 0.8701919803374691, 0.5636241161898479]], [[0.4248872136039653, 0.4995251695008067, 0.32343381735644183, 1.2334568418688168, 0.7617916121557339, 0.7365597794053197, 0.4320053283709917, 0.7831414843555351, 1.041231070806223, 0.3328179465553609, 0.025891123338507804, 0.6160685326385288, 0.6372295510404589, 0.45676695729071864, 0.39217157132136493, 0.3025620905218575, 0.6076891437525848, 0.8412248252036381, 1.0502853427286276, 0.3681896193332502, 0.2631299737459115, 0.22152781913529562, 3.4870886584447467, 0.24924497180871533, 0.6056090881043815, 0.24634612628955804, 0.3004127950915436, 0.4860403067342423, 0.573844752133693, 0.595075289940764, 0.6750650650251913, 0.723203056105977, 0.19877970160761727, 0.8997219242394641, 0.6178051013365364]], [[0.44482280356647963, 0.4547509906884044, 0.33881795234022155, 1.2463540780036722, 0.7638795067500638, 0.7321112810400315, 0.419022105970148, 0.7756732732751173, 1.0644200795341943, 0.2727365402013875, 0.03340542951229709, 0.6024676710363455, 0.6767430290504266, 0.4622349895238328, 0.38640624098324255, 0.2953158717077278, 0.5267318499332343, 0.8530054113728089, 0.9942361634832246, 0.31165298075572556, 0.2548070325092137, 0.2593515675888367, 3.4870886584447467, 0.2636641888013812, 0.5898164356926157, 0.17713988104241327, 0.28825465187054466, 0.4986639993003117, 0.595632517446907, 0.6192330661563505, 0.6248321491495643, 0.704802659714332, 0.1976109416554958, 0.9103569023843662, 0.6481521963279209]], [[0.4284733175113129, 0.4337782852536653, 0.34173812036211726, 1.2379675836973005, 0.7898398381949604, 0.7347739029672944, 0.4187301584571468, 0.7893812807564988, 1.0519925988927314, 0.23725404916212073, 0.025827695403425506, 0.5945423873974999, 0.6350025489114547, 0.46026269881461374, 0.3325832863650424, 0.29686287379619547, 0.5339646433210765, 0.8572473441591317, 0.9426108782249276, 0.3219550018427396, 0.24983700872597187, 0.20885879305261673, 3.4870886584447467, 0.25977029632148363, 0.6026594141112874, 0.13336705837042254, 0.2533150095899984, 0.5140008426491367, 0.5954273701975445, 0.6398193024186427, 0.5596960291212965, 0.6432030273936549, 0.21372437548898274, 0.9307613575706507, 0.6324178493048456]], [[0.38662742619422363, 0.39732127798556727, 0.3577430748012831, 1.2052943834072263, 0.7984238270518692, 0.7488746530632261, 0.40147423495574686, 0.7955765423136709, 1.0870362872352288, 0.223243983386227, 0.03680230719648384, 0.6255571976650133, 0.6387715988470559, 0.47703968086111154, 0.32927507632379494, 0.2954283681248583, 0.5496949976950936, 0.8592689027549958, 0.8865574790363494, 0.34614218719561046, 0.259536512049154, 0.18941889328844566, 3.4870886584447467, 0.24976415425071155, 0.6442907095446285, 0.09624770457196327, 0.2288927536334432, 0.5684071101321286, 0.6154714575947164, 0.6450267610009346, 0.5024309621723955, 0.56770173518499, 0.223763861862725, 0.9374838446011319, 0.5906035522140562]], [[0.35165654228581783, 0.3837904745498106, 0.35523704865476313, 1.1541612005535244, 0.8292334551086094, 0.7682224624668987, 0.4084953373899437, 0.7880637097444407, 1.0593958667803398, 0.2143415375656523, 0.030369417570662645, 0.6300330336689781, 0.6741383838538301, 0.46285558727294546, 0.3484081480750513, 0.2859758372043548, 0.576891522579356, 0.8731921472971318, 0.8648503694511218, 0.3844756304136906, 0.26309700664101376, 0.19666834998494706, 3.4870886584447467, 0.26143055794247116, 0.6546626166250616, 0.07288473341679429, 0.23405936788302092, 0.588599317636131, 0.60288929871401, 0.6430605932854423, 0.450321397326823, 0.5706668318410809, 0.2390265692854326, 0.9416459713515154, 0.5478761461194576]], [[0.3130192764449105, 0.3655638375799761, 0.351573752651948, 1.196952225991995, 0.8407085549063351, 0.8180009524123231, 0.43447226541187267, 0.8519879111736015, 1.0712332992527365, 0.21137219040207775, 0.06272630110896292, 0.655497138507549, 0.6643966927603385, 0.47293023437099285, 0.29817561490178573, 0.2982099452920015, 0.6188903288055077, 0.8726647140500972, 0.8180325986726178, 0.41355281614324474, 0.2820186711646195, 0.2556835801191775, 3.4870886584447467, 0.2438148635604734, 0.691020819980902, 0.06634590889120838, 0.2592626511778612, 0.6798135288354097, 0.5996953244252512, 0.6253304675482221, 0.37732984103471157, 0.4729552994814974, 0.22261099598486422, 0.9558373840898988, 0.5770246586535304]], [[0.28052263333313426, 0.3557049662561201, 0.3674435917082898, 1.1601564832644526, 0.797631184616022, 0.7238739826220838, 0.37801054727865646, 0.8947737572179547, 1.0393296122111466, 0.23000623764458636, 0.09924095554418459, 0.7228842585313331, 0.7691760191156367, 0.49281719843432537, 0.39558476448883273, 0.2908937096339206, 0.5964489121203951, 0.7485458494802942, 0.7130639863304119, 0.4082256639323527, 0.27826726515625533, 0.23737037408506614, 3.4870886584447467, 0.2733239814647118, 0.629470062317886, 0.07266413757627214, 0.2718296690274661, 0.6865870873302757, 0.5847107152433304, 0.529775327566995, 0.3154902389600661, 0.3950339220679294, 0.19508172433554677, 0.9624934945574468, 0.5964297823719107]], [[0.28066511331324184, 0.37497475930040836, 0.37237119918028383, 1.0700893058968792, 0.8760144327051348, 0.678883068104714, 0.39604693564435167, 0.8890423926688211, 0.9773678887867873, 0.23849312837871592, 0.1506592846884047, 0.7399628192137649, 0.82645937487596, 0.5481624237120014, 0.45382713270387004, 0.29456634580118346, 0.6226130357058147, 0.6938817014248437, 0.8082076771412254, 0.41292233715111554, 0.28239433085769594, 0.21309964229201397, 3.4870886584447467, 0.3368872235506944, 0.5352590402736744, 0.08181678128584664, 0.311331901394714, 0.6741758894670522, 0.5624669934677515, 0.46325508364698553, 0.2915015157817914, 0.30202840268573394, 0.2071252550801282, 0.9569512156933092, 0.6309963860482364]], [[0.31847998699588986, 0.34805662046832303, 0.3542540666717756, 1.0358402456545057, 0.9169709238288031, 0.6932510637843812, 0.3752326961846545, 0.9068320301342602, 0.94731909622976, 0.24280464738795451, 0.16633009260261583, 0.7221376973946618, 0.8298861735728246, 0.5559709071366797, 0.4704187018243688, 0.30392130221375924, 0.5530223986352123, 0.7064235425477388, 0.9125180651786331, 0.45132139318887976, 0.27583971961194015, 0.20176600054600108, 3.4870886584447467, 0.32936616401055896, 0.47576493960634547, 0.06945518944557247, 0.35808972559722885, 0.6258059634334814, 0.5368564033353409, 0.4823711360069749, 0.31024363582122116, 0.20601828050560803, 0.21544854058426655, 0.9268090841072862, 0.6690525730693474]], [[0.38944932454038894, 0.36551183739404186, 0.3610893254155416, 0.9733370228216875, 0.9635057659473109, 0.6683551874788827, 0.3776932853014562, 0.8713075416706981, 0.8947877610203332, 0.27419899783781954, 0.11608981905320598, 0.7565432461096564, 0.8454546752049062, 0.5477250185457008, 0.47733041937988907, 0.3018326813448375, 0.5075896724750041, 0.6766100012064213, 0.8512868520718961, 0.5057593491635997, 0.29864091840602514, 0.18828719008535952, 3.4870886584447467, 0.31666300654284685, 0.4290467195778973, 0.07884668449457483, 0.37468088708994324, 0.5696155172804269, 0.5037777274456723, 0.44785962776197186, 0.33201469874708106, 0.1439564712356377, 0.23046594762873007, 0.8567813719315857, 0.7122849938179505]], [[0.44435946511383506, 0.3979070518229828, 0.34447007650936295, 0.8916834734316039, 0.9864781615024336, 0.6902650261559673, 0.37238955242618355, 0.8671333118215617, 0.8487110597102738, 0.2917100226825439, 0.11126447604786077, 0.7841288830329762, 0.8417501520134675, 0.5459937861426339, 0.41959715928177044, 0.3050175182423398, 0.5762159696555241, 0.6596425203505721, 0.8363931532377573, 0.48327037467477435, 0.3240622763439733, 0.18188147237742114, 3.4870886584447467, 0.2996898685945707, 0.36533160387349783, 0.0817438533911595, 0.378542151405262, 0.5538018508167025, 0.4702916922418652, 0.37591057020873736, 0.3124616677332632, 0.09986654584849147, 0.26025785951124414, 0.7657843530817546, 0.7456351175309365]], [[0.4390416074472454, 0.39925265007756366, 0.33479712095708114, 0.8238905093702991, 1.0260688089077394, 0.6800803971448294, 0.37350150480032795, 0.8317017323615709, 0.7628426769116808, 0.2850278083555738, 0.11202634062512708, 0.760061682445077, 0.8191775537808073, 0.531158055215168, 0.43191189114047374, 0.30423105891285224, 0.5512379815983115, 0.6494236187375257, 1.0291988035686352, 0.46241917950863354, 0.3263823950062142, 0.2160710426992321, 3.4870886584447467, 0.28758607691013527, 0.3148452157493259, 0.09412453544010925, 0.388168261062268, 0.5621814045538641, 0.4241470250731331, 0.3469811135695434, 0.26187474229617336, 0.09358123777628122, 0.2616779700113288, 0.6676861221813017, 0.7471613956253076]], [[0.7589948873489777, 0.3993295513389909, 0.31276110069669505, 0.8102950090305301, 1.2884976282631149, 0.7040912617627241, 0.3822835232903568, 0.892466528659039, 0.7256492550332381, 0.30873160507054687, 0.19271922658620944, 0.7952876401821147, 1.2876761916548178, 0.5792835610031752, 0.47259251150117704, 0.2901593716345966, 0.5606987607008246, 0.6294567028217211, 1.244522674188992, 0.5594616450192191, 0.34245049041746367, 0.2750303051185538, 0.09570002647806608, 0.3449978247226857, 0.3266412684844734, 0.16579776358670398, 0.5062606679656717, 0.7638748141081435, 0.46043792929112004, 0.4531243678545362, 0.3284182867620208, 0.17473218150532308, 0.37534616565042656, 0.7222234797779186, 1.266127929243201]], [[0.7957876246691462, 0.37720880895967734, 0.3404221364482819, 0.8085347868850563, 1.3326454890981245, 0.7018703093211214, 0.3802316169242462, 0.8568101558570349, 0.6371621047843639, 0.32512296834300236, 0.1355470799181738, 1.3655692782614544, 1.420322173753006, 0.5555477236133771, 0.425684561507584, 0.3003983543813639, 0.5449323749766429, 0.6372065964363587, 1.1022773450696715, 0.6281828858690105, 0.3109497652790825, 0.2630749015113594, 0.07597895487529682, 0.2509353275679446, 0.4295649205898547, 0.16693525759021258, 0.39999529039929077, 0.8091105727200205, 0.42660325762086054, 0.3722428362954053, 0.2898569329515476, 0.20960516961012546, 0.3443090411653377, 0.6125687136889869, 1.2697574994047462]], [[0.8450478129833954, 0.39364011236820645, 0.3486368433457568, 0.8181090052865816, 1.3698582617922828, 0.8118868248718888, 0.37486011728854, 0.8240168839365134, 0.5345233843779635, 0.26762311204194955, 0.11224619459167601, 1.116367013106254, 1.4033897297037128, 0.534040261343001, 0.3970756292791592, 0.28368914301933784, 0.5121719352481046, 0.6470657000964797, 1.280836989105207, 0.42255508366017497, 0.266961110878488, 0.2949635667168309, 0.037965004887494797, 0.20257335364029094, 0.44860524354699594, 0.19902676513330408, 0.3162004509181138, 0.6990170888380668, 0.4062543441443423, 0.45003891599798834, 0.2766016339715405, 0.2118666486099473, 0.33853170473352723, 0.4877524899771376, 1.2466454377401914]], [[0.8767597935744214, 0.4168906688027582, 0.3482815825364942, 0.7964378670132022, 1.3972531466511138, 0.8240979994874281, 0.3836636407341419, 0.804748627197062, 0.4156768004962358, 0.2487576870413869, 0.15078096700025148, 1.1201211540131428, 1.3883663870639626, 0.546854601797518, 0.38094038558634125, 0.2776078384676499, 0.5076330756509357, 0.6714033306869438, 1.330263051427154, 0.45384346685055915, 0.24353161919207963, 0.2720295244670737, 0.02760408500582353, 0.20446680536260212, 0.49830300979183506, 0.2632452518505803, 0.3026305345472975, 0.6889838771090089, 0.37601396638644163, 0.5767397413813538, 0.2755604971216006, 0.24654519428837118, 0.347652352351651, 0.38516917820795094, 1.202985664080079]], [[0.9038850265635878, 0.4812475849304225, 0.33982592666525624, 0.7600568432104011, 1.4169458488415563, 0.7223258472854056, 0.37432213741816095, 0.8499272811415403, 0.4389556144918108, 0.3815301637552479, 0.11264302610395058, 1.0872761599220242, 1.269100448604401, 0.5667054740298945, 0.4213799274369646, 0.3012781192639057, 0.4567568951684757, 0.6109636922688727, 1.360191872706662, 0.5190398947183398, 0.23590835966919071, 0.23190505524283356, 0.023883226831284736, 0.3630102092526886, 0.5450068820139768, 0.3145690754389139, 0.3191202522664357, 0.6499083963246877, 0.3533589563913002, 0.64776022021728, 0.2786948048624349, 0.3130549597938526, 0.39590993895616405, 0.3410493108750703, 1.1355093424295768]], [[0.9199224296694843, 0.5462359319841983, 0.3400868224159179, 0.6916848953877888, 1.412206576510572, 0.6685512581086506, 0.3616801917449222, 0.9221704661988485, 0.4760945493059993, 0.50054461632423, 0.085127807321798, 1.0324598000161764, 1.2052916069793964, 0.6084204671880413, 0.38840814895685905, 0.2713145929773778, 0.47095740083778787, 0.6287089329306583, 1.1935958642632905, 0.5357118836279736, 0.29632203330840656, 0.16026570518881444, 0.022846787604934218, 0.612126030550618, 0.6479516411282178, 0.36463943931646287, 0.3601523749101025, 0.7346268284082498, 0.3545712577767757, 0.7621445408543541, 0.32272898626068436, 0.3979512789731236, 0.56005832374439, 0.3199512956983249, 1.0481776470328574]], [[0.9160891282549177, 0.5800305139296438, 0.3379779633035172, 0.6854973537451043, 1.385841441014435, 0.6765727191208131, 0.36362484943852924, 0.8887422802098064, 0.5148759139471029, 0.5408318068498232, 0.09782080936960358, 0.9951359648365496, 1.174941977169698, 0.5895023729110698, 0.38765798858218803, 0.25921384930032804, 0.43976091546905194, 0.6082694547908523, 1.1298269918961845, 0.4883537472033808, 0.3376438016809187, 0.1414726597213207, 0.021244622203314877, 0.7703431352591521, 0.7339162110158487, 0.39512697473916963, 0.3811565588225215, 0.720726198160926, 0.35917141274210784, 0.716515229707456, 0.42659765095117563, 0.4218011228526125, 0.5854376455935, 0.35067200349762706, 0.9394735268403451]], [[0.8791341431042481, 0.5871854633149387, 0.3304972987970137, 0.7318405976212026, 1.3744715336789497, 0.6725805974920969, 0.36474791687502994, 0.9503797291818215, 0.5479363788352086, 0.5376436255225687, 0.12309752284974466, 1.0114734950445081, 1.1745312088728475, 0.5981951564144921, 0.3814906336069067, 0.275942299470921, 0.40935675706360936, 0.5804994976252593, 1.0160699463145513, 0.35423329776685697, 0.3031830753175267, 0.1536825412822424, 0.02053611330119036, 0.7717566723698461, 0.7866325066542936, 0.4250382428189111, 0.5069179842599119, 0.78604033991769, 0.340033236968584, 0.7276143924242788, 0.5119824948613845, 0.3936858422578254, 0.6722731078108792, 0.3650878286121218, 0.8294661246720125]], [[0.8904935774432452, 0.6114333696456669, 0.3225149196678108, 0.7923636235934277, 1.3896939236169012, 0.6301602155236281, 0.3636758522723044, 1.0557667789376914, 0.5833945261879241, 0.47603183413781014, 0.1394435224219049, 0.9616862876187078, 1.2207987640584834, 0.6057257255756799, 0.3774834243018104, 0.3091208006620527, 0.41212792536586085, 0.5756929184256615, 0.9622731754812495, 0.3313034685589554, 0.28449514889137484, 0.1931316609350753, 0.020331489250472512, 0.6991294386028784, 0.823348938390843, 0.4625659560161084, 0.6128895375326673, 0.6959232227751466, 0.2892587500615853, 0.7675321326021479, 0.4852085622648443, 0.38925149351130955, 0.6337792636895343, 0.37608119124200745, 0.6974697526242948]], [[0.9089886708801461, 0.6326251215994361, 0.3143601101628192, 0.8016024992948926, 1.481956358504136, 0.6828912204513958, 0.38517317777286264, 1.2402475786844391, 0.6256794770419634, 0.5716415093343888, 0.09688529108126837, 0.9358908515830364, 1.3823580033917053, 0.5940598320739623, 0.3876745755277992, 0.33504840709401185, 0.38868687585659456, 0.5580061963167555, 0.9622319436513245, 0.3228036363669635, 0.37159453664771186, 0.281685118365561, 0.019252790014810485, 0.5131144097359458, 0.8617381969121118, 0.5072371985534291, 0.6787849152755964, 0.6566144730559365, 0.35360576644488595, 0.9016004282685073, 0.29406301225852277, 0.36203099499107727, 0.5793731528150441, 0.3425104728058863, 0.75052512273983]], [[0.8946896349344067, 0.6754120443643268, 0.31031668430989057, 0.8189634642774506, 1.5087837892642595, 0.6058983625793756, 0.37411914978979294, 1.3440569973124008, 0.6650063488225477, 0.47979326256294474, 0.16612597225285122, 0.9784940726404372, 1.4647993670818413, 0.5043017704900923, 0.33726107682315054, 0.26597099282484415, 0.38034929434307574, 0.5367598754439346, 0.912092584774701, 0.3152569429536169, 0.368967575077849, 0.32445860659814285, 0.015383130811346868, 0.4203923676338917, 0.8745061569532845, 0.48466944933117834, 0.6884900090808317, 0.6372644838686461, 0.43809745764425, 0.9911271931633148, 0.25370102779160986, 0.34263551548611054, 0.5344157165280803, 0.32087424345077153, 0.6515680064726795]], [[1.0326396428367057, 0.644552250106981, 0.3336458264443397, 1.089936785185555, 1.2976717834145206, 0.5992842221685382, 0.3739401206632578, 0.7792072001361587, 0.6628013688503039, 0.46152589898715324, 0.104107743321871, 1.0269353898438554, 1.4511120623179372, 0.4986280645360306, 0.3165987149361617, 0.2644188113268693, 0.3082698144412681, 0.5612792355979357, 0.916075202007974, 0.3044388855334106, 0.3492048114471811, 0.3345365392656702, 0.009936673719764854, 0.4145704928698433, 0.8740631664060423, 0.4122028428056183, 0.7487720381325432, 0.5825689377663196, 0.44903505821274636, 0.9969934557528902, 0.3104074486479564, 0.4484389735133398, 0.43657054816987595, 0.37358405760944113, 0.6105294633672755]], [[1.0569966688870074, 0.6872265094077791, 0.3200341445386284, 1.3342521270763035, 1.3044482108952855, 0.6152503372896941, 0.4050834007326402, 0.7977156340605458, 0.6546084216457967, 0.4411184994316442, 0.08553756278804381, 0.9800473762149523, 1.3765481378335052, 0.5068809909117233, 0.35051394783812106, 0.2819602309343511, 0.29232569591453783, 0.5651738826952268, 0.9235206313849296, 0.28065072159281357, 0.3314225814788432, 0.33851991085118094, 0.007157192217877304, 0.44183902167334344, 0.8531379856274234, 0.4141975182660662, 0.7524009421416018, 0.6073801446430797, 0.48864405138430533, 0.9689962059141646, 0.42238566434531916, 0.4516687905318512, 0.3950730719009966, 0.4919980541971619, 0.5690435438452218]], [[1.1308775471233865, 0.6667453506957028, 0.39493001393726235, 1.484205987399547, 1.3025553439684499, 0.5583670606994178, 0.3735200111568688, 0.8738466798927431, 0.9194187103583878, 0.5620063837132325, 0.06318794780454122, 0.9648808944273051, 1.3085293811324976, 0.49709379739214615, 0.34801383523856133, 0.29948840334665067, 0.32225801102543133, 0.5372484410431074, 0.8734648385050613, 0.2585440005954365, 0.3516490816780074, 0.34056525360388773, 0.007406416294952612, 0.565140886096515, 0.835482137518172, 0.3843908166019746, 0.8105481360216767, 0.5945710670157173, 0.488529741542003, 0.9846147151253399, 0.5445839218218219, 0.44461765554083116, 0.4316845287378397, 0.6592149324630505, 0.5068417347950258]], [[0.7603028398029255, 0.6287475196002166, 0.3716747338577973, 1.647244871774151, 1.2436019695532967, 0.588029987859559, 0.37288160554135413, 0.8478231488357238, 1.1411162032411544, 0.5627153094008293, 0.06595039779386962, 0.977151858714303, 1.2725980864153377, 0.5073784790759541, 0.3062574985610359, 0.2764663022573142, 0.3545043083993815, 0.5579009942005855, 0.8801061637616747, 0.22117020710836063, 0.36103282435280803, 0.323832858295133, 0.0074710226216516745, 0.71543054814174, 0.8177540895588538, 0.32849203203802824, 1.0790056949425697, 0.6176507545775205, 0.45277595032651335, 1.0271371095368962, 0.643684831706831, 0.4190732077727363, 0.4648404534116135, 0.7093659247853089, 0.49286551084991925]], [[0.8524707461782883, 0.658602748084929, 0.3015039084327444, 1.7554948974698108, 1.2138598157637461, 0.5887257664007106, 0.3646421048622535, 0.7398468725556403, 1.5506368280399832, 0.48429235627687445, 0.07554497570651933, 1.011027228239901, 1.3060223600601235, 0.4864000197917001, 0.3305567182391074, 0.28331492143762876, 0.34481458037866364, 0.5641999783110956, 0.8876487293532673, 0.22721820604406348, 0.32243169135806404, 0.24809701135500917, 0.007506532931320506, 0.6646165986191799, 0.7954413171578998, 0.3509304840091438, 1.2126303738636395, 0.6614903185429672, 0.45956349606712643, 1.0374122076376349, 0.7397677634924773, 0.3907998588485463, 0.4879142348671075, 0.805056560589072, 0.4998508765970434]], [[0.8119113743302723, 0.5998311490866227, 0.3045581610060314, 1.7876757572771054, 1.17237118814704, 0.5948860329868018, 0.362788213734027, 0.7096860879490712, 1.694241555826421, 0.4816990181207012, 0.06308872707712283, 1.0595097216878113, 1.3271872046640367, 0.49080906871819563, 0.2507344312584691, 0.27041765799231093, 0.33272225991676635, 0.5654008122350966, 0.853969214872088, 0.21065470012625004, 0.3299031328139215, 0.2633450565065007, 0.006958651325666623, 0.5294969903990892, 0.7662475210651416, 0.35609770040055844, 1.2062052978096225, 0.7589945478322152, 0.3923354447789015, 1.0572943166066313, 0.8929451004688951, 0.3604053673936971, 0.5053400482277286, 0.8702924689465176, 0.5758161578364711]], [[0.7673428590215943, 0.6020351007279823, 0.31134465491130076, 1.8627618531626977, 1.1030228645870686, 0.5882875807067621, 0.3700983589887813, 0.6841015256666177, 1.7502041985222312, 0.4772900166798147, 0.06142043579056575, 1.0692620540373143, 1.329736155395476, 0.5044167185581474, 0.22157749642572727, 0.265322228113591, 0.3627884266468099, 0.5474371370823848, 0.8506892264932421, 0.19285241480321982, 0.31245841889515286, 0.2806583432635799, 0.0066747174080848196, 0.5136194408846935, 0.6946016091515378, 0.4105246562653555, 1.1926040895104288, 0.8135469103525772, 0.3909070335923225, 1.0640286728962771, 0.9548184277206215, 0.3755885287302652, 0.5581924778373026, 0.9958381616840588, 0.6986642669813472]], [[0.7569415400906636, 0.5318910250152287, 0.36303822450528517, 1.9539943719727504, 1.0491012684761525, 0.5930737903390605, 0.3718480959650858, 0.6802193457677481, 1.7726497103723637, 0.5014639880577605, 0.07531516394908842, 1.0793842546405759, 1.3230706302180921, 0.5026667958089373, 0.23428802363313256, 0.2719277454424113, 0.35939186927263134, 0.5096389269699881, 0.8090497810011823, 0.22854779001239867, 0.28388303163607365, 0.3146860189068166, 0.006332011787384116, 0.6946600897763745, 0.6404843140629944, 0.5040064361387413, 1.1948738183295085, 0.8987359534556866, 0.4044390046453246, 0.9639439800669398, 1.0155159803011142, 0.36857639850353713, 0.5623849411775608, 1.0366465266642004, 0.8081622521065783]], [[0.7373671977187151, 0.5838697701147743, 0.3615365626727403, 1.967799083196486, 0.9193991291369484, 0.5548236548346998, 0.3619816281540264, 0.7052660625788729, 1.7982530800390437, 0.5109220957548772, 0.09741043468628541, 1.1304824619086689, 1.3025690016428042, 0.5774605137159565, 0.30514996518503845, 0.30719351667087375, 0.31709000693082556, 0.5586868280398831, 0.866861118374697, 0.23089233561275574, 0.2673387091625663, 0.24803093379904934, 0.0066551360470561444, 0.8790019029774587, 0.7083651292904851, 0.4818329459344862, 1.107031451447324, 0.9952649547310255, 0.521335841273886, 0.9903250853020745, 1.0228471895414029, 0.33531834748363193, 0.5834131303795971, 1.1163530680155265, 0.9424239473863826]], [[0.6990646373499003, 0.5265491184312017, 0.37690016743636023, 1.9979004152296465, 0.7312154129729312, 0.5642470196926851, 0.3605057727807082, 0.7559428027583542, 1.6275946619902917, 0.48776997409320577, 0.11491989038868376, 1.0941190048717035, 1.273081150847754, 0.5916037185748289, 0.3002760952659235, 0.31740020393475843, 0.3186528167553972, 0.5575395537303698, 0.8805013727189039, 0.2277618769967167, 0.28915313662672404, 0.23668372949274358, 0.008061760156569278, 0.9464991879834713, 0.7583522605152954, 0.47596906078738926, 1.0667137166110094, 1.0435583798906385, 0.6126132785851962, 0.9829553663028202, 1.030629891336445, 0.2347450019435056, 0.547341689326952, 1.1937435057268004, 0.966216098668302]], [[0.6223100440297594, 0.5770220086289153, 0.36733210006316325, 1.891771172693183, 0.6261952025706018, 0.5626669769089476, 0.3578441479300618, 0.7837338058181431, 1.3837350715377434, 0.4845026041386423, 0.1185079770988609, 1.0955095460066395, 1.2151819807452973, 0.591998539642705, 0.35745520164157457, 0.3228712366842657, 0.30014784770299646, 0.5107597177215906, 0.758793102498585, 0.22438006628681695, 0.2940662338195089, 0.24423761399033594, 0.009819909490449412, 0.9370059518628144, 0.7526540394744969, 0.4378563550691193, 1.1089806383808742, 1.079103886286168, 0.677897057167081, 0.9647557534505058, 1.029784679174388, 0.26019726695173384, 0.48411105803295135, 1.2236842703182096, 0.9988023091340541]], [[0.5550845209342713, 0.6436055741760468, 0.355788528702324, 1.9053570217856317, 0.3833470932921131, 0.5704717571308988, 0.37766241987690374, 0.8143130236251933, 1.3028398799194598, 0.42463653569687476, 0.16035044352954092, 1.0245324954484833, 1.1479720778477378, 0.5843996103358415, 0.31072998817850583, 0.3504988498224176, 0.29322997897836534, 0.5183510841664922, 0.7282700420563899, 0.22596212408309338, 0.30980002734590456, 0.27426176630689447, 0.010552144355212725, 0.9166903942621933, 0.8042292945112659, 0.4214364482068767, 1.1701581029597894, 1.230713628005903, 0.7138690208821795, 0.9334959310676483, 1.0424950552323873, 0.27043644161046715, 0.4118663189652572, 1.259024755491899, 1.043305210513522]], [[0.46733043844575134, 0.6028332508343901, 0.3820429774000249, 1.921542405517613, 0.3199221620443622, 0.5970078009565675, 0.3845326090103801, 0.8309415462445305, 1.05931384190164, 0.3463905654927667, 0.14516527726519504, 1.0060002455072927, 1.1397561965189047, 0.586593487369092, 0.33240897436643757, 0.3320717319893829, 0.26860566136047337, 0.5122870107201876, 0.7012415612145387, 0.21926141217835848, 0.30488408960116925, 0.33045877156250825, 0.011263021217770607, 0.9185700890983337, 0.7107329993123872, 0.42751069873469505, 1.2084968285126998, 1.2232113194833052, 0.6720071207350548, 0.9388717435489299, 1.040073537002746, 0.32515738460617605, 0.31197536491854555, 1.2652625095029446, 1.1356401164521324]]]}]}


In [15]:
arr = np.array(arr2['predictions'][0]['values'])

In [24]:
arr[:,0,:].shape

(1439, 35)

In [37]:
arr[:,0,0:4].shape

(1439, 4)

In [33]:
df_1 = pd.DataFrame(columns=['A','B','C','D'])
df_1[['A','B','C','D']] = arr[:,0,0:3]

In [38]:
df_1['A']

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
1434    0.737367
1435    0.699065
1436    0.622310
1437    0.555085
1438    0.467330
Name: A, Length: 1439, dtype: float64

In [54]:
ir = range(0, 432533, 400)
for r in ir:
    print(r)

0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
6000
6400
6800
7200
7600
8000
8400
8800
9200
9600
10000
10400
10800
11200
11600
12000
12400
12800
13200
13600
14000
14400
14800
15200
15600
16000
16400
16800
17200
17600
18000
18400
18800
19200
19600
20000
20400
20800
21200
21600
22000
22400
22800
23200
23600
24000
24400
24800
25200
25600
26000
26400
26800
27200
27600
28000
28400
28800
29200
29600
30000
30400
30800
31200
31600
32000
32400
32800
33200
33600
34000
34400
34800
35200
35600
36000
36400
36800
37200
37600
38000
38400
38800
39200
39600
40000
40400
40800
41200
41600
42000
42400
42800
43200
43600
44000
44400
44800
45200
45600
46000
46400
46800
47200
47600
48000
48400
48800
49200
49600
50000
50400
50800
51200
51600
52000
52400
52800
53200
53600
54000
54400
54800
55200
55600
56000
56400
56800
57200
57600
58000
58400
58800
59200
59600
60000
60400
60800
61200
61600
62000
62400
62800
63200
63600
64000
64400
64800
65200
65600
66000
66400
66800
67200
67600
68000
6840

In [41]:


f = open('/home/markus/MODEL_results', 'rb')
res_ = f.read()
f.close()
res = pickle.loads(res_)
res

array([[2.19756099, 0.82051618, 1.00868397, ..., 0.35914272, 0.81013005,
        0.63541357],
       [2.17871288, 0.83878964, 1.01131494, ..., 0.31389662, 0.78070435,
        0.5811306 ],
       [2.16026406, 0.83924264, 0.96929057, ..., 0.2969239 , 0.76952044,
        0.56777975],
       ...,
       [0.59311255, 0.51546357, 0.62910743, ..., 0.49750863, 0.94655086,
        1.26268869],
       [0.50332343, 0.55425651, 0.34586288, ..., 0.42595373, 0.98036609,
        1.27421145],
       [0.41633517, 0.39291964, 0.24563999, ..., 0.32113994, 1.06938151,
        1.26858504]])

In [42]:
res.shape

(471, 35)

In [102]:
#f = open('./xxxxxxxx', "r")
#arr = f.read()
#f.close()
#arr.replace('NaN','np.nan')
arr = exec('np.array(' + arr + ')')

In [104]:
arr

In [88]:
arr

array([7.66632853e+170, 1.94920557e-153, 5.61858287e+175, ...,
       4.17633291e-062, 3.53541455e-057, 5.56440456e+141])

In [2]:
db_schema=None
class DatabaseDummy:
    tenant_id = '###_IBM_###'
    db_type = 'db2'
    model_store = FileModelStore()
    def _init(self):
        return

db = DatabaseDummy()

2022-07-11T13:30:16.035 INFO iotfunctions.dbtables.__init__ Init FileModelStore with path: 


In [3]:
df = pd.read_csv('../mmfunctions/data/AzureAnomalysample.csv')
df['entity'] = 'myentity'
df = df.set_index(['entity','timestamp'])

In [5]:
from mmfunctions.Customanomaly import AnomalyThreshold
jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}

vsmi = AnomalyThreshold('value', .9, 'Min','Max','cycle','outlier')
et = vsmi._build_entity_type(columns = [Column('test',Float())], **jobsettings)
vsmi._entity_type = et
vsmi.auto_train = True
df = vsmi.execute(df=df)

2022-07-11T13:30:17.791 INFO iotfunctions.metadata.__init__ The entity type is not connected to a metric input table.
2022-07-11T13:30:17.807 INFO iotfunctions.dbtables.retrieve_model Model model.TEST_ENTITY_FOR_ANOMALYTHRESHOLD.SupervisedLearningTransformer.Min.Max.cycle.outlier.myentity of size 102 bytes has been retrieved from filesystem
Here 5


In [6]:
df

value  Min  Max  cycle  outlier
entity   timestamp                                            
myentity 2017-10-09 20:32:00  3.0000 -9.2    6      0     True
         2017-10-09 20:33:00  3.0000 -9.2    6      0     True
         2017-10-09 20:34:00  3.0000 -9.2    6      0     True
         2017-10-09 20:35:00  3.0000 -9.2    6      0     True
         2017-10-09 20:36:00  3.0000 -9.2    6      0     True
...                              ...  ...  ...    ...      ...
         2017-10-12 22:17:00  2.1875 -9.2    6      0     True
         2017-10-12 22:18:00  2.1250 -9.2    6      0     True
         2017-10-12 22:19:00  2.0000 -9.2    6      0     True
         2017-10-12 22:20:00  2.0000 -9.2    6      0     True
         2017-10-12 22:21:00  2.0625 -9.2    6      0     True

[4000 rows x 5 columns]

In [7]:
jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}

invi = InvokeWMLModel(df, 'wml_score', 'out')
et = invi._build_entity_type(columns = [Column('test',Float())], **jobsettings)
invi._entity_type = et
df = invi.execute(df=df)

2022-07-12T13:28:27.456 INFO iotfunctions.metadata.__init__ The entity type is not connected to a metric input table.
2022-07-12T13:28:27.457 INFO mmfunctions.bif.execute InvokeWML exec
2022-07-12T13:28:27.458 INFO mmfunctions.bif.execute db is <__main__.DatabaseDummy object at 0x7fc649b27040>
2022-07-12T13:28:27.467 INFO mmfunctions.bif.login Constants {'__dict__': {'local_columns': [Column('test', Float(), table=None)], 'logical_name': 'test_entity_for_InvokeWMLModel', 'name': 'TEST_ENTITY_FOR_INVOKEWMLMODEL', '_metric_table_name': 'TEST_ENTITY_FOR_INVOKEWMLMODEL', 'description': '', 'activity_tables': {}, 'scd': {}, 'db': <__main__.DatabaseDummy object at 0x7fc649b27040>, 'tenant_id': '###_IBM_###', '_system_columns': ['deviceid', '_timestamp', 'logicalinterface_id', 'devicetype', 'format', 'updated_utc', 'evt_timestamp'], '_stage_type_map': [('preload', 'is_preload'), ('get_data', 'is_data_source'), ('transform', 'is_transformer'), ('aggregate', 'is_data_aggregator'), ('simple_aggr

RuntimeError: No WML credentials specified